In [ ]:
############################################
#                 KUNBUS                   #
############################################

from scapy.all import rdpcap
import datetime
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import time
import copy
from scapy.compat import raw
from scapy.error import Scapy_Exception
from scapy.config import conf
from scapy.packet import Packet, bind_layers
from scapy.layers.l2 import Ether
from scapy.layers.inet import UDP
from scapy.fields import (
    XShortEnumField, BitEnumField, XBitField,
    BitField, StrField, PacketListField,
    StrFixedLenField, ShortField,
    FlagsField, ByteField, XIntField, X3BytesField
)
import scapy.modules.six as six

fileName = "manufacturer.pcap"
userID = 'Marcio Rafael Buzoli'

PNIO_FRAME_IDS = {
    0x0020: "PTCP",#-RTSyncPDU-followup
    0x0080: "PTCP",#-RTSyncPDU
    0xFC01: "Alarm High",
    0xFE01: "Alarm Low",
    0xFEFC: "DCP",#-Hello-Req
    0xFEFD: "DCP",#-Get-Set
    0xFEFE: "DCP",#-Identify-ReqPDU
    0xFEFF: "DCP",#-Identify-ResPDU
    0xFF00: "PTCP",#-AnnouncePDU
    0xFF20: "PTCP",#-FollowUpPDU
    0xFF40: "PTCP",#-DelayReqPDU
    0xFF41: "PTCP",#-DelayResPDU-followup
    0xFF42: "PTCP",#-DelayFuResPDU
    0xFF43: "PTCP",#-DelayResPDU
}


def i2s_frameid(x):
    """ Get representation name of a pnio frame ID
    :param x: a key of the PNIO_FRAME_IDS dictionary
    :returns: str
    """
    try:
        return PNIO_FRAME_IDS[x]
    except KeyError:
        pass
    if 0x0100 <= x < 0x1000:
        return "RT_CLASS_3"# (%4x)" % x
    if 0x8000 <= x < 0xC000:
        return "RT_CLASS_2"# (%4x)" % x
    if 0xC000 <= x < 0xFC00:
        return "RT_CLASS_1"# (%4x)" % x
    if 0xFF80 <= x < 0xFF90:
        return "FragmentationFrameID"# (%4x)" % x
    return x


def s2i_frameid(x):
    """ Get pnio frame ID from a representation name
    Performs a reverse look-up in PNIO_FRAME_IDS dictionary
    :param x: a value of PNIO_FRAME_IDS dict
    :returns: integer
    """
    try:
        return {
            "RT_CLASS_3": 0x0100,
            "RT_CLASS_2": 0x8000,
            "RT_CLASS_1": 0xC000,
            "FragmentationFrameID": 0xFF80,
        }[x]
    except KeyError:
        pass
    try:
        return next(key for key, value in six.iteritems(PNIO_FRAME_IDS)
                    if value == x)
    except StopIteration:
        pass
    return x


#################
#  PROFINET IO  #
#################

class ProfinetIO(Packet):
    """ Basic PROFINET IO dispatcher """
    fields_desc = [
        XShortEnumField("frameID", 0, (i2s_frameid, s2i_frameid))
    ]

    def guess_payload_class(self, payload):
        # For frameID in the RT_CLASS_* range, use the RTC packet as payload
        if self.frameID in [0xfefe, 0xfeff, 0xfefd]:
            from scapy.contrib.pnio_dcp import ProfinetDCP
            return ProfinetDCP
        elif (
                (0x0100 <= self.frameID < 0x1000) or
                (0x8000 <= self.frameID < 0xFC00)


        ):
            return PNIORealTimeCyclicPDU
        return super(ProfinetIO, self).guess_payload_class(payload)


bind_layers(Ether, ProfinetIO, type=0x8892)
bind_layers(UDP, ProfinetIO, dport=0x8892)


#####################################
#  PROFINET Real-Time Data Packets  #
#####################################

conf.contribs["PNIO_RTC"] = {}


class PNIORealTime_IOxS(Packet):
    """ IOCS and IOPS packets for PROFINET Real-Time payload """
    name = "PNIO RTC IOxS"
    fields_desc = [
        # IOxS.DataState -- IEC-61158 - 6 - 10 / FDIS ED 3, Table 181
        BitEnumField("dataState", 1, 1, ["bad", "good"]),
        # IOxS.Instance -- IEC-61158 - 6 - 10 / FDIS ED 3, Table 180
        BitEnumField("instance", 0, 2,
                     ["subslot", "slot", "device", "controller"]),
        # IOxS.reserved -- IEC-61158 - 6 - 10 / FDIS ED 3, line 2649
        XBitField("reserved", 0, 4),
        # IOxS.Extension -- IEC-61158-6-10/FDIS ED 3, Table 179
        BitField("extension", 0, 1),
    ]

    @classmethod
    def is_extension_set(cls, _pkt, _lst, p, _remain):
        ret = cls if isinstance(p, type(None)) or p.extension != 0 else None
        return ret

    @classmethod
    def get_len(cls):
        return sum(type(fld).i2len(None, 0) for fld in cls.fields_desc)

    def guess_payload_class(self, p):
        return conf.padding_layer


class PNIORealTimeCyclicDefaultRawData(Packet):
    name = "PROFINET IO Real Time Cyclic Default Raw Data"
    fields_desc = [
        # 4 is the sum of the size of the CycleCounter + DataStatus
        #     + TransferStatus trailing from PNIORealTimeCyclicPDU
        StrField("data", '', remain=4)
    ]

    def guess_payload_class(self, payload):
        return conf.padding_layer


class PNIORealTimeCyclicPDU(Packet):
    """ PROFINET cyclic real-time """
    __slots__ = ["_len", "_layout"]
    name = "PROFINET Real-Time"

    fields_desc = [
        # C_SDU ^ CSF_SDU -- IEC-61158-6-10/FDIS ED 3, Table 163
        PacketListField(
            "data", [],
            next_cls_cb=lambda pkt, lst, p, remain: pkt.next_cls_cb(
                lst, p, remain)
        ),
        # RTCPadding -- IEC - 61158 - 6 - 10 / FDIS ED 3, Table 163
        StrFixedLenField("padding", '',
                         length_from=lambda p: p.get_padding_length()),
        # APDU_Status -- IEC-61158-6-10/FDIS ED 3, Table 164
        ShortField("cycleCounter", 0),
        FlagsField("dataStatus", 0x35, 8, [
            "primary",
            "redundancy",
            "validData",
            "reserved_1",
            "run",
            "no_problem",
            "reserved_2",
            "ignore",
        ]),
        ByteField("transferStatus", 0)
    ]

    def pre_dissect(self, s):
        # Constraint from IEC-61158-6-10/FDIS ED 3, line 690
        self._len = min(1440, len(s))
        return s

    def get_padding_length(self):
        if hasattr(self, "_len"):
            pad_len = (
                self._len -
                sum(len(raw(pkt)) for pkt in self.getfieldval("data")) -
                2 -  # Cycle Counter size (ShortField)
                1 -  # DataStatus size (FlagsField over 8 bits)
                1  # TransferStatus (ByteField)
            )
        else:
            pad_len = len(self.getfieldval("padding"))

        # Constraints from IEC-61158-6-10/FDIS ED 3, Table 163
        assert(0 <= pad_len <= 40)
        q = self
        while not isinstance(q, UDP) and hasattr(q, "underlayer"):
            q = q.underlayer
        if isinstance(q, UDP):
            assert(0 <= pad_len <= 12)
        return pad_len

    def next_cls_cb(self, _lst, _p, _remain):
        if hasattr(self, "_layout") and isinstance(self._layout, list):
            try:
                return self._layout.pop(0)
            except IndexError:
                self._layout = None
                return None

        ether_layer = None
        q = self
        while not isinstance(q, Ether) and hasattr(q, "underlayer"):
            q = q.underlayer
        if isinstance(q, Ether):
            ether_layer = q

        pnio_layer = None
        q = self
        while not isinstance(q, ProfinetIO) and hasattr(q, "underlayer"):
            q = q.underlayer
        if isinstance(q, ProfinetIO):
            pnio_layer = q

        self._layout = [PNIORealTimeCyclicDefaultRawData]
        if not (ether_layer is None and pnio_layer is None):
            # Get from config the layout for these hosts and frameid
            layout = type(self).get_layout_from_config(
                ether_layer.src,
                ether_layer.dst,
                pnio_layer.frameID)
            if not isinstance(layout, type(None)):
                self._layout = layout

        return self._layout.pop(0)

    @staticmethod
    def get_layout_from_config(ether_src, ether_dst, frame_id):
        try:
            return copy.deepcopy(
                conf.contribs["PNIO_RTC"][(ether_src, ether_dst, frame_id)]
            )
        except KeyError:
            return None

    @staticmethod
    def build_fixed_len_raw_type(length):
        return type(
            "FixedLenRawPacketLen{}".format(length),
            (conf.raw_layer,),
            {
                "name": "FixedLenRawPacketLen{}".format(length),
                "fields_desc": [StrFixedLenField("data", '', length=length)],
                "get_data_length": lambda _: length,
                "guess_payload_class": lambda self, p: conf.padding_layer,
            }
        )


# From IEC 61784-3-3 Ed. 3 PROFIsafe v.2.6, Figure 20
profisafe_control_flags = [
    "iPar_EN", "OA_Req", "R_cons_nr", "Use_TO2",
    "activate_FV", "Toggle_h", "ChF_Ack", "Loopcheck"
]
# From IEC 61784-3-3 Ed. 3 PROFIsafe v.2.6, Figure 19
profisafe_status_flags = [
    "iPar_OK", "Device_Fault/ChF_Ack_Req", "CE_CRC",
    "WD_timeout", "FV_activated", "Toggle_d", "cons_nr_R", "reserved"
]


class PROFIsafeCRCSeed(Packet):
    __slots__ = ["_len"] + Packet.__slots__

    def guess_payload_class(self, p):
        return conf.padding_layer

    def get_data_length(self):
        """ Must be overridden in a subclass to return the correct value """
        raise Scapy_Exception(
            "This method must be overridden in a specific subclass"
        )

    def get_mandatory_fields_len(self):
        # 5 is the len of the control/status byte + the CRC length
        return 5

    @staticmethod
    def get_max_data_length():
        # Constraints from IEC-61784-3-3 ED 3, Figure 18
        return 13


class PROFIsafeControlCRCSeed(PROFIsafeCRCSeed):
    name = "PROFISafe Control Message with F_CRC_Seed=1"
    fields_desc = [
        StrFixedLenField("data", '',
                         length_from=lambda p: p.get_data_length()),
        FlagsField("control", 0, 8, profisafe_control_flags),
        XIntField("crc", 0)
    ]


class PROFIsafeStatusCRCSeed(PROFIsafeCRCSeed):
    name = "PROFISafe Status Message with F_CRC_Seed=1"
    fields_desc = [
        StrFixedLenField("data", '',
                         length_from=lambda p: p.get_data_length()),
        FlagsField("status", 0, 8, profisafe_status_flags),
        XIntField("crc", 0)
    ]


class PROFIsafe(Packet):
    __slots__ = ["_len"] + Packet.__slots__

    def guess_payload_class(self, p):
        return conf.padding_layer

    def get_data_length(self):
        """ Must be overridden in a subclass to return the correct value """
        raise Scapy_Exception(
            "This method must be overridden in a specific subclass"
        )

    def get_mandatory_fields_len(self):
        # 4 is the len of the control/status byte + the CRC length
        return 4

    @staticmethod
    def get_max_data_length():
        # Constraints from IEC-61784-3-3 ED 3, Figure 18
        return 12

    @staticmethod
    def build_PROFIsafe_class(cls, data_length):
        assert(cls.get_max_data_length() >= data_length)
        return type(
            "{}Len{}".format(cls.__name__, data_length),
            (cls,),
            {
                "get_data_length": lambda _: data_length,
            }
        )


class PROFIsafeControl(PROFIsafe):
    name = "PROFISafe Control Message with F_CRC_Seed=0"
    fields_desc = [
        StrFixedLenField("data", '',
                         length_from=lambda p: p.get_data_length()),
        FlagsField("control", 0, 8, profisafe_control_flags),
        X3BytesField("crc", 0)
    ]


class PROFIsafeStatus(PROFIsafe):
    name = "PROFISafe Status Message with F_CRC_Seed=0"
    fields_desc = [
        StrFixedLenField("data", '',
                         length_from=lambda p: p.get_data_length()),
        FlagsField("status", 0, 8, profisafe_status_flags),
        X3BytesField("crc", 0)
    ]


#def removeDuplicates(l):
#    return list(set(l))


inicio = time.time()

packets = rdpcap(fileName)

############################################################################
############################################################################
############################################################################

cycleCounter_list = []
cod_equipments = []
connections2 = list()
frame_id_list = [] 
alarm_low_list = []
alarm_high_list = []
alarm_info_list = []
alarm_low = 0
alarm_high = 0
alarm_type_list = []
nRT_Throughput = []
chronology_list = []
rtThroughputSecond = [0]
nRtThroughputSecond = [0]

chronology_throughput = ['%.3f'%float(0)]
support_rtThroughputSecond = []
support_NrtThroughputSecond = []
counter_throughput = 1.000000000

liveList = []
srcList = []
dstList = []
RT_liveListThroughput = []
nRT_liveListThroughput = []
packetsRT = 0
packetsNRT = 0

for i in range(len(packets)):
###########################################
    source = packets[i].src
    destination = packets[i].dst
    srcList.append(source)
    dstList.append(destination)
###FRAME ID FILTER########    START
    flagFrame_id = str(packets[i].show2(1)[0:])
    flagFrame_id2 = flagFrame_id.split('frameID   = ',1)
    if len(flagFrame_id2) == 2:
        flagFrame_id3 = flagFrame_id2[1].split(" ", 1)
        flagFrame_id4 = flagFrame_id3[0]
        if flagFrame_id4[0:8] != 'RT_CLASS':
            flagFrame_id4= 'nRT'
    else:
        flagFrame_id4= 'nRT'
 ###FRAME ID FILTER########    END   
    
    if source not in liveList:
        liveList.append(source)
    if destination not in liveList:
        liveList.append(destination)
    if flagFrame_id4[0:3] == 'RT_':
        RT_liveListThroughput.append(packets[i].wirelen)
        packetsRT = packetsRT +1
        nRT_liveListThroughput.append(0)
    if flagFrame_id4[0:3] != 'RT_':
        packetsNRT = packetsNRT +1
        RT_liveListThroughput.append(0)
        nRT_liveListThroughput.append(packets[i].wirelen)   

###########################################
    support_nRT_Throughput = packets[i].wirelen
    nRT_Throughput.append(support_nRT_Throughput)
    flag_alarm_info = 'None'
    alarm_type = 0
    flag = str(packets[i].show2(1)[0:])
    flag1 = flag.split('cycleCounter= ')
    frame_split = flag.split('frameID   = ')
    if len(frame_split) == 2:
        frame_split1 = frame_split[1]
        frame_split2 = frame_split1.split('\n')
        frame_split = frame_split2[0]

        if frame_split == 'Alarm Low':
            flag1 = "-"
            alarm_type = frame_split
            alarm_low = alarm_low + 1
            flag_alarm_info1 = str(packets[i].load[len(packets[i].load)-1:len(packets[i].load)])
            if flag_alarm_info1[3:6] == 'x01':
                flag_alarm_info='Short circuit'
            if flag_alarm_info1[3:6] == 'x02':
                flag_alarm_info='Undervoltage'
            if flag_alarm_info1[3:6] == 'x03':
                flag_alarm_info='Overvoltage'
            if flag_alarm_info1[3:6] == 'x04':
                flag_alarm_info='Overloading'    
            if flag_alarm_info1[3:6] == 'x05':
                flag_alarm_info='Overtemperature'
            if flag_alarm_info1[3:6] == 'x06':
                flag_alarm_info='Line break: Open circuit'
            if flag_alarm_info1[3:6] == 'x07':
                flag_alarm_info='Upper limit value exceeded: Upper limit value overshot '
            if flag_alarm_info1[3:6] == 'x08':
                flag_alarm_info='Lower limit value exceeded: Lower limit value undershot'
            if flag_alarm_info1[3:6] == 'x09':
                flag_alarm_info='Error'
            if flag_alarm_info1[3:6] == 'x0a':
                flag_alarm_info='Simulation active'
            if flag_alarm_info1[3:6] == 'x0f':
                flag_alarm_info='Parameter missing: The channel requires an additional parameter. No or too few parameters written '
            if flag_alarm_info1[3:6] == 'x10':
                flag_alarm_info='Parameterization fault: Parameter error. Incorrect or too many parameters written'    
            if flag_alarm_info1[3:6] == 'x11':
                flag_alarm_info='Power supply fault'
            if flag_alarm_info1[3:6] == 'x12':
                flag_alarm_info='Fuse blown/open: Fuse defective'
            if flag_alarm_info1[3:6] == 'x13':
                flag_alarm_info='Communication fault: Communication fault. Incorrect sequence or sequence number'
            if flag_alarm_info1[3:6] == 'x14':
                flag_alarm_info='Ground fault'
            if flag_alarm_info1[3:6] == 'x15':
                flag_alarm_info='Reference point lost'
            if flag_alarm_info1[3:6] == 'x16':
                flag_alarm_info='Process event lost / Sampling error'
            if flag_alarm_info1[3:6] == 'x17':
                flag_alarm_info='Threshold warning'
            if flag_alarm_info1[3:6] == 'x18':
                flag_alarm_info='Output disabled'    
            if flag_alarm_info1[3:6] == 'x19':
                flag_alarm_info='Functional safety event'
            if flag_alarm_info1[3:6] == 'x1a':
                flag_alarm_info='External fault'
            if flag_alarm_info1[3:6] == 'x1f':
                flag_alarm_info='Temporary fault'
            if flag_alarm_info1[3:6] == 'x40':
                flag_alarm_info='Functional safety 0: Safety target addresses do not match'
            if flag_alarm_info1[3:6] == 'x41':
                flag_alarm_info='Functional safety 1: Safety target address invalid'
            if flag_alarm_info1[3:6] == 'x42':
                flag_alarm_info='Functional safety 2: Safety source address invalid or does not match'
            if flag_alarm_info1[3:6] == 'x43':
                flag_alarm_info='Functional safety 3: Safety watchdog time is set to 0 ms'
            if flag_alarm_info1[3:6] == 'x44':
                flag_alarm_info='Functional safety 4: Parameter "F_SIL" exceeds SIL of the device application'    
            if flag_alarm_info1[3:6] == 'x45':
                flag_alarm_info='Functional safety 5: Parameter "F_CRC_Length" does not match the generated value'
            if flag_alarm_info1[3:6] == 'x46':
                flag_alarm_info='Functional safety 6: Version of the F-Parameter invalid'
            if flag_alarm_info1[3:6] == 'x47':
                flag_alarm_info='Functional safety 7: Data in received FParameter block is inconsistent (CRC1 error)'
            if flag_alarm_info1[3:6] == 'x48':
                flag_alarm_info='Functional safety 8: Device-specific or undefined diagnostic information, see data sheet'  
            if flag_alarm_info1[3:6] == 'x49':
                flag_alarm_info='Functional safety 9: Store iParameter: Watchdog time exceeded' 
            if flag_alarm_info1[3:6] == 'x4a':
                flag_alarm_info='Functional safety 10: Restore iParameter: Watchdog time exceeded'
            if flag_alarm_info1[3:6] == 'x4b':
                flag_alarm_info='Functional safety 11: Inconsistent iParameters (iParCRC error)'
            if flag_alarm_info1[3:6] == 'x4c':
                flag_alarm_info='Functional safety 12: F_Block_ID not supported'
            if flag_alarm_info1[3:6] == 'x4d':
                flag_alarm_info='Functional safety 13: Transfer error: Inconsistent data (CRC2 error)'
            if flag_alarm_info1[3:6] == 'x4e':
                flag_alarm_info='Functional safety 14: Transfer error: Timeout'
            if flag_alarm_info1[3:6] == 'x4f':
                flag_alarm_info='Functional safety 15: Acknowledge needed to enable the channel(s)'  
            if flag_alarm_info1[3:6] == 'x00':
                flag_alarm_info='Unknown error'
        if frame_split == 'Alarm High':
            flag1 = "-"
            alarm_type = frame_split
            alarm_high = alarm_high + 1
            flag_alarm_info1 = str(packets[i].load[len(packets[i].load)-1:len(packets[i].load)])
            if flag_alarm_info1[3:6] == 'x01':
                flag_alarm_info='Short circuit'
            if flag_alarm_info1[3:6] == 'x02':
                flag_alarm_info='Undervoltage'
            if flag_alarm_info1[3:6] == 'x03':
                flag_alarm_info='Overvoltage'
            if flag_alarm_info1[3:6] == 'x04':
                flag_alarm_info='Overloading'    
            if flag_alarm_info1[3:6] == 'x05':
                flag_alarm_info='Overtemperature'
            if flag_alarm_info1[3:6] == 'x06':
                flag_alarm_info='Line break: Open circuit'
            if flag_alarm_info1[3:6] == 'x07':
                flag_alarm_info='Upper limit value exceeded: Upper limit value overshot '
            if flag_alarm_info1[3:6] == 'x08':
                flag_alarm_info='Lower limit value exceeded: Lower limit value undershot'
            if flag_alarm_info1[3:6] == 'x09':
                flag_alarm_info='Error'
            if flag_alarm_info1[3:6] == 'x0a':
                flag_alarm_info='Simulation active'
            if flag_alarm_info1[3:6] == 'x0f':
                flag_alarm_info='Parameter missing: The channel requires an additional parameter. No or too few parameters written '
            if flag_alarm_info1[3:6] == 'x10':
                flag_alarm_info='Parameterization fault: Parameter error. Incorrect or too many parameters written'    
            if flag_alarm_info1[3:6] == 'x11':
                flag_alarm_info='Power supply fault'
            if flag_alarm_info1[3:6] == 'x12':
                flag_alarm_info='Fuse blown/open: Fuse defective'
            if flag_alarm_info1[3:6] == 'x13':
                flag_alarm_info='Communication fault: Communication fault. Incorrect sequence or sequence number'
            if flag_alarm_info1[3:6] == 'x14':
                flag_alarm_info='Ground fault'
            if flag_alarm_info1[3:6] == 'x15':
                flag_alarm_info='Reference point lost'
            if flag_alarm_info1[3:6] == 'x16':
                flag_alarm_info='Process event lost / Sampling error'
            if flag_alarm_info1[3:6] == 'x17':
                flag_alarm_info='Threshold warning'
            if flag_alarm_info1[3:6] == 'x18':
                flag_alarm_info='Output disabled'    
            if flag_alarm_info1[3:6] == 'x19':
                flag_alarm_info='Functional safety event'
            if flag_alarm_info1[3:6] == 'x1a':
                flag_alarm_info='External fault'
            if flag_alarm_info1[3:6] == 'x1f':
                flag_alarm_info='Temporary fault'
            if flag_alarm_info1[3:6] == 'x40':
                flag_alarm_info='Functional safety 0: Safety target addresses do not match'
            if flag_alarm_info1[3:6] == 'x41':
                flag_alarm_info='Functional safety 1: Safety target address invalid'
            if flag_alarm_info1[3:6] == 'x42':
                flag_alarm_info='Functional safety 2: Safety source address invalid or does not match'
            if flag_alarm_info1[3:6] == 'x43':
                flag_alarm_info='Functional safety 3: Safety watchdog time is set to 0 ms'
            if flag_alarm_info1[3:6] == 'x44':
                flag_alarm_info='Functional safety 4: Parameter "F_SIL" exceeds SIL of the device application'    
            if flag_alarm_info1[3:6] == 'x45':
                flag_alarm_info='Functional safety 5: Parameter "F_CRC_Length" does not match the generated value'
            if flag_alarm_info1[3:6] == 'x46':
                flag_alarm_info='Functional safety 6: Version of the F-Parameter invalid'
            if flag_alarm_info1[3:6] == 'x47':
                flag_alarm_info='Functional safety 7: Data in received FParameter block is inconsistent (CRC1 error)'
            if flag_alarm_info1[3:6] == 'x48':
                flag_alarm_info='Functional safety 8: Device-specific or undefined diagnostic information, see data sheet'  
            if flag_alarm_info1[3:6] == 'x49':
                flag_alarm_info='Functional safety 9: Store iParameter: Watchdog time exceeded' 
            if flag_alarm_info1[3:6] == 'x4a':
                flag_alarm_info='Functional safety 10: Restore iParameter: Watchdog time exceeded'
            if flag_alarm_info1[3:6] == 'x4b':
                flag_alarm_info='Functional safety 11: Inconsistent iParameters (iParCRC error)'
            if flag_alarm_info1[3:6] == 'x4c':
                flag_alarm_info='Functional safety 12: F_Block_ID not supported'
            if flag_alarm_info1[3:6] == 'x4d':
                flag_alarm_info='Functional safety 13: Transfer error: Inconsistent data (CRC2 error)'
            if flag_alarm_info1[3:6] == 'x4e':
                flag_alarm_info='Functional safety 14: Transfer error: Timeout'
            if flag_alarm_info1[3:6] == 'x4f':
                flag_alarm_info='Functional safety 15: Acknowledge needed to enable the channel(s)'  
            if flag_alarm_info1[3:6] == 'x00':
                flag_alarm_info='Unknown error' 
                
    if len(flag1) == 2:
        flag2 = flag1[1]
        flag3 = flag2.split('\n')
        flag1 = flag3[0]
        
        
#inicio - modificacoes 22-09

    support_chronology = float(packets[i].time) - float(packets[0].time)
    support_chronology1 ='%.6f'%float(support_chronology)+'000'
    chronology_list.append(support_chronology1)
    
    if packets[i].time < packets[0].time:
        support_chronology = float(packets[i-1].time) - float(packets[0].time)
        support_chronology1 ='%.6f'%float(support_chronology)+'000'
        chronology_list.append(support_chronology1)
    
    #counter_throughput = 1
    
    
    if flagFrame_id4[0:3] == 'RT_':
        flag_x = packets[i].wirelen
        support_rtThroughputSecond.append(flag_x)
    else:
        flag_x = packets[i].wirelen
        support_NrtThroughputSecond.append(flag_x)
    if packets[i] ==  packets[len(packets)-1]:
        flag_rt = '%.3f'%float((sum(support_rtThroughputSecond)*8)/1000000)
        flag_nrt = '%.3f'%float((sum(support_NrtThroughputSecond)*8)/1000000)
        flag_crt = '%.3f'%float(support_chronology1)
        chronology_throughput.append(flag_crt)
        rtThroughputSecond.append(flag_rt)
        nRtThroughputSecond.append(flag_nrt)
             
    if float(support_chronology) > float(counter_throughput):
        flag_rt = '%.3f'%float((sum(support_rtThroughputSecond)*8)/1000000)
        flag_nrt = '%.3f'%float((sum(support_NrtThroughputSecond)*8)/100000)
        flag_crt = '%.3f'%float(support_chronology1)
        chronology_throughput.append(flag_crt)
        rtThroughputSecond.append(flag_rt)
        nRtThroughputSecond.append(flag_nrt)
        support_rtThroughputSecond = []
        support_NrtThroughputSecond = []
        counter_throughput = counter_throughput + 1.000000000      
        
#fim - modificacoes 22-09

    frame_id_list.append(frame_split)
    cycleCounter_list.append(flag1)
    flag_connection2 = (packets[i].src,packets[i].dst)
    connections2.append(flag_connection2)
    alarm_info_list.append(flag_alarm_info)
    alarm_type_list.append(alarm_type)
    

def remove_equal(list_src_dst):
    l = []
    for i in list_src_dst:
        if i not in l:
            l.append(i)
    l.sort()
    return l

list_src_dst = connections2
list_src_dst = remove_equal(list_src_dst)
devices = []

#Devices on-line
for i in range(len(list_src_dst)):
    flag = list_src_dst[i][0]
    devices.append(flag)

def remove_equal(devices):
    l = []
    for i in devices:
        if i not in l:
            l.append(i)
    l.sort()
    return l  
devices = remove_equal(devices)

all_db = []
pn_ptcp = 0
non_profinet = 0
non_profinetname = []

############################## Kunbus TAP


kunbus_timestamp = []
for i in range(len(packets)):
    b7 = str(hex(packets[i].original[len(packets[i].original)-8]))
    b7_1 = b7.split('0x')
    if len(b7_1[1]) == 1:
        b7='0'+b7_1[1]
    else:
        b7 = b7_1[1]
   
    b6 = str(hex(packets[i].original[len(packets[i].original)-7]))
    b6_1 = b6.split('0x')
    if len(b6_1[1]) == 1:
        b6='0'+b6_1[1]
    else:
        b6 = b6_1[1]
    
    b5 = str(hex(packets[i].original[len(packets[i].original)-6]))
    b5_1 = b5.split('0x')
    if len(b5_1[1]) == 1:
        b5='0'+b5_1[1]
    else:
        b5 = b5_1[1]    

    b4 = str(hex(packets[i].original[len(packets[i].original)-5]))
    b4_1 = b4.split('0x')
    if len(b4_1[1]) == 1:
        b4='0'+b4_1[1]
    else:
        b4 = b4_1[1]
     
    b3 = str(hex(packets[i].original[len(packets[i].original)-4]))
    b3_1 = b3.split('0x')
    if len(b3_1[1]) == 1:
        b3='0'+b3_1[1]
    else:
        b3 = b3_1[1]    

    b2 = str(hex(packets[i].original[len(packets[i].original)-3]))
    b2_1 = b2.split('0x')
    if len(b2_1[1]) == 1:
        b2='0'+b2_1[1]
    else:
        b2 = b2_1[1]    

    b1 = str(hex(packets[i].original[len(packets[i].original)-2]))
    b1_1 = b1.split('0x')
    if len(b1_1[1]) == 1:
        b1='0'+b1_1[1]
    else:
        b1 = b1_1[1]    

    b0 = str(hex(packets[i].original[len(packets[i].original)-1]))
    b0_1 = b0.split('0x')
    if len(b0_1[1]) == 1:
        b0='0'+b0_1[1]
    else:
        b0 = b0_1[1]
    
    kunbus_timestamp_support = [i ,'0x'+b0+b1+b2+b3+b4+b5+b6+b7]
    kunbus_timestamp.append(kunbus_timestamp_support)
    
kunbusTime = []
errorCount = 0
for i in range(len(kunbus_timestamp)):

    support1= (int(kunbus_timestamp[i][1],16) - int(kunbus_timestamp[i-1-errorCount][1],16))/1000000000
    #print(support1)
    if kunbus_timestamp[i][0]==0:
        support1 = "%.9f"%float(0)
        support2 = "%.9f"%float(0)
    if kunbus_timestamp[i][0] > 0:
        if support1 > 1000 or support1 <0:
            #print('errorkunbustimestamp = ',i,support1)
            support2= (int(kunbus_timestamp[i][1],16) - int(kunbus_timestamp[0][1],16))/1000000000
            if (float(support2)) > 1000 or (float(support2)) < 0:
                
                support1 = chronology_list[i]
                errorCount = errorCount +1
            support1 = "%.9f"%float(support2)
            errorCount = 0   
        else:    
            support1 = "%.9f"%float(float("%.9f"%float(support1)) + float(kunbusTime[-1-errorCount]))
            errorCount = 0
        #print(i,support1)
    if kunbus_timestamp[i][0]==kunbus_timestamp[len(kunbus_timestamp)-1][0]:
        kunbusTime.append(str(support1))
        break 

    kunbusTime.append(str(support1))    
    #print(i,kunbus_timestamp[i][1],int(kunbus_timestamp[i][1],16),"%.9f"%float(support1))  
    
for i in range(len(kunbusTime)):
    if kunbusTime[i] == kunbusTime[-1]:
        #print(kunbusTime[i])
        break
    if float(kunbusTime[i]) < 0 or float(kunbusTime[i]) > float(chronology_list[-1])+1000:
        kunbusTime[i] = chronology_list[i]
        #print(kunbusTime[i])

chronology_list = kunbusTime

############## ALL_DB

for i in range(len(list_src_dst)):
    for x in range (len(connections2)):
        if connections2[x] == list_src_dst[i]:
            if packets[x].type == 34962: #8892 - Identificação PN_IO Profinet
                if packets[x].dst != '01:80:c2:00:00:0e':   #mac PN_TCP                
                    support = [i+1,x, connections2[x], float(packets[x].time),packets[x].wirelen, 
                               hex(packets[x].type),alarm_type_list[x],alarm_info_list[x], frame_id_list[x], 
                               cycleCounter_list[x], chronology_list[x]
                              ]
                    
                    all_db.append(support)
                    x = x+1
                else:
                    pn_ptcp = pn_ptcp + 1
                    support = [i+1, connections2[x]]
                    non_profinetname.append(support)
            else:
                non_profinet = non_profinet + 1
                support = [i+1,connections2[x]]
                non_profinetname.append(support)
devices = []
for i in range(len(list_src_dst)):
    flag = list_src_dst[i][0]
    devices.append(flag)
def remove_equal(devices):
    l = []
    for i in devices:
        if i not in l:
            l.append(i)
    l.sort()
    return l  
devices = remove_equal(devices)
##########################
manufacture = str("01:80:C2IEEERegi,FF:FF:FFBroadcast,01:80:C2:00:00:00/xxSpanning_Tree_Protocol,01:80:C2:00:00:01/xxEthernet_flow_control,01:80:C2:00:00:02/xxIEEE_802.3ah,01:80:C2:00:00:03/xxPort_authentication,01:80:C2:00:00:08/xxSpanning_Tree_Protocol,01:80:C2:00:00:0D/xxMultiple_VLAN_Registration_Protocol,01:80:C2:00:00:0E/xxLLDP_Multicast,01:80:C2:00:00:21/xxMultiple_VLAN_Registration_Protocol,01:80:C2:00:00:30/xxEthernet_CFM_Protocol,01:1B:19:00:00:00/xxLLDP_Multicast,01:00:5EIPv4_Multicast,33:33:00IPv6_multicast,01:0C:CDIEC_61850-8-1,01:00:0C:CC:CC:CC/xxCisco_Discovery_Protocol,01:00:0C:CC:CC:CD/xxCisco_Shared_Spanning_Tree_Protocol_Address,00:00:01Xerox,00:00:02Xerox,00:00:03Xerox,00:00:04Xerox,00:00:05Xerox,00:00:06Xerox,00:00:07Xerox,00:00:08Xerox,00:00:09Powerpip,00:00:0AOmronTat,00:00:0BMatrix,00:00:0CCisco,00:00:0DFibronic,00:00:0EFujitsu,00:00:0FNext,00:00:10Sytek,00:00:11Normerel,00:00:12Informat,00:00:13Camex,00:00:14Netronix,00:00:15Datapoin,00:00:16DuPontPi,00:00:17Oracle,00:00:18WebsterC,00:00:19AppliedD,00:00:1AAdvanced,00:00:1BNovellNo,00:00:1CBellTech,00:00:1DCabletro,00:00:1ETelsistI,00:00:1FTelco,00:00:20Dataindu,00:00:21SuremanC,00:00:22VisualTe,00:00:23AbbAutom,00:00:24ConnectA,00:00:25Ramtek,00:00:26Sha-Ken,00:00:27JapanRad,00:00:28Prodigy,00:00:29ImcNetwo,00:00:2ATrw-Sedd,00:00:2BCrispAut,00:00:2CAutotote,00:00:2DChromati,00:00:2ESocieteE,00:00:2FTimeplex,00:00:30VgLabora,00:00:31QpsxComm,00:00:32Marconi,00:00:33EganMach,00:00:34NetworkR,00:00:35Spectrag,00:00:36Atari,00:00:37OxfordMe,00:00:38CssLabs,00:00:39Toshiba,00:00:3AChyron,00:00:3BiControl,00:00:3CAuspex,00:00:3DUnisys,00:00:3ESimpact,00:00:3FSyntrex,00:00:40Applicon,00:00:41Ice,00:00:42MetierMa,00:00:43MicroTec,00:00:44Castelle,00:00:45FordAero,00:00:46Olivetti,00:00:47NicoletI,00:00:48SeikoEps,00:00:49ApricotC,00:00:4AAdcCoden,00:00:4BIclData,00:00:4CNEC,00:00:4DDci,00:00:4EAmpex,00:00:4FLogicraf,00:00:50Radisys,00:00:51HobElect,00:00:52Intrusio,00:00:53Compucor,00:00:54Schneide,00:00:55Commissa,00:00:56DrBStruc,00:00:57Scitex,00:00:58RacoreCo,00:00:59Hellige,00:00:5ASkSchnei,00:00:5BEltecEle,00:00:5CTelemati,00:00:5DCsTeleco,00:00:5EICANNIAN,00:00:5FSumitomo,00:00:60KontronE,00:00:61GatewayC,00:00:62BullHnIn,00:00:63BarcoCon,00:00:64Yokogawa,00:00:65NetworkG,00:00:66Talaris,00:00:67Soft*Rit,00:00:68Rosemoun,00:00:69ConcordC,00:00:6AComputer,00:00:6BSiliconG,00:00:6CPrivate,00:00:6DCrayComm,00:00:6EArtisoft,00:00:6FMadgeNet,00:00:70Hcl,00:00:71Adra,00:00:72Miniware,00:00:73Siecor,00:00:74Ricoh,00:00:75BellNort,00:00:76AbekasVi,00:00:77Interpha,00:00:78Labtam,00:00:79Networth,00:00:7ADanaComp,00:00:7BResearch,00:00:7CAmpere,00:00:7DOracle,00:00:7EClustrix,00:00:7FLinotype,00:00:80CrayComm,00:00:81BayNetwo,00:00:82LectraSy,00:00:83TadpoleT,00:00:84Supernet,00:00:85Canon,00:00:86Megahert,00:00:87Hitachi,00:00:88BrocadeC,00:00:89CaymanSy,00:00:8ADatahous,00:00:8BInfotron,00:00:8CAlloyCom,00:00:8DCryptek,00:00:8ESolbourn,00:00:8FRaytheon,00:00:90Microcom,00:00:91Anritsu,00:00:92CogentDa,00:00:93Proteon,00:00:94AsanteMa,00:00:95SonyTekt,00:00:96MarconiE,00:00:97DellEMC,00:00:98Crosscom,00:00:99Mtx,00:00:9ARcComput,00:00:9BInformat,00:00:9CRolmMil-,00:00:9DLocusCom,00:00:9EMarli,00:00:9FAmerista,00:00:A0SANYOEle,00:00:A1Marquett,00:00:A2BayNetwo,00:00:A3NetworkA,00:00:A4AcornCom,00:00:A5Tattile,00:00:A6NetworkG,00:00:A7NetworkC,00:00:A8StratusT,00:00:A9Network,00:00:AAXeroxXer,00:00:ABLogicMod,00:00:ACConwareN,00:00:ADBrukerIn,00:00:AEDassault,00:00:AFCanberra,00:00:B0Rnd-RadN,00:00:B1AlphaMic,00:00:B2Televide,00:00:B3Cimlinc,00:00:B4EdimaxTe,00:00:B5Databili,00:00:B6Micro-Ma,00:00:B7DoveFast,00:00:B8Seikosha,00:00:B9Mcdonnel,00:00:BASiig,00:00:BBTri-Data,00:00:BCRockwell,00:00:BDMitsubis,00:00:BENtiGroup,00:00:BFSymmetri,00:00:C0WesternD,00:00:C1Madge,00:00:C2Informat,00:00:C3Harris,00:00:C4WatersDi,00:00:C5ARRISGro,00:00:C6HpIntell,00:00:C7Arix,00:00:C8AltosCom,00:00:C9EmulexTe,00:00:CAARRISGro,00:00:CBCompu-Sh,00:00:CCDensan,00:00:CDAlliedTe,00:00:CEMegadata,00:00:CFHayesMic,00:00:D0Develcon,00:00:D1Adaptec,00:00:D2Sbe,00:00:D3WangLabo,00:00:D4PureData,00:00:D5Microgno,00:00:D6PunchLin,00:00:D7Dartmout,00:00:D8OldNovel,00:00:D9NipponTe,00:00:DAAtex,00:00:DBBritishT,00:00:DCHayesMic,00:00:DDTcl,00:00:DECetia,00:00:DFBellHowe,00:00:E0Quadram,00:00:E1Grid,00:00:E2AcerTech,00:00:E3Integrat,00:00:E4In2Group,00:00:E5Sigmex,00:00:E6AptorPro,00:00:E7StarGate,00:00:E8AcctonTe,00:00:E9Isicad,00:00:EAUpnod,00:00:EBMatsushi,00:00:ECMicropro,00:00:EDApril,00:00:EENetworkD,00:00:EFKti,00:00:F0SamsungE,00:00:F1MagnaCom,00:00:F2SpiderCo,00:00:F3GandalfD,00:00:F4AlliedTe,00:00:F5DiamondS,00:00:F6AppliedM,00:00:F7YouthKee,00:00:F8DigitalE,00:00:F9Quotron,00:00:FAMicrosag,00:00:FBRechnerZ,00:00:FCMeiko,00:00:FDHighLeve,00:00:FEAnnapoli,00:00:FFCamtecEl,00:01:00EquipTra,00:01:01Private,00:01:023Com,00:01:033Com,00:01:04DVICO,00:01:05Beckhoff,00:01:06TewsDate,00:01:07Leiser,00:01:08AVLABTec,00:01:09NaganoJa,00:01:0ACisTechn,00:01:0BSpaceCyb,00:01:0CSystemTa,00:01:0DTeledyne,00:01:0EBri-Link,00:01:0FBrocadeC,00:01:10GothamNe,00:01:11iDigm,00:01:12SharkMul,00:01:13Olympus,00:01:14KandaTsu,00:01:15Extratec,00:01:16Netspect,00:01:17Canal+,00:01:18EZDigita,00:01:19RTUnetAu,00:01:1AHoffmann,00:01:1BUnizoneT,00:01:1CUniversa,00:01:1DCentilli,00:01:1EPrecidia,00:01:1FRCNetwor,00:01:20Oscilloq,00:01:21WatchGua,00:01:22TrendCom,00:01:23Schneide,00:01:24Acer,00:01:25YaesuMus,00:01:26PACLabs,00:01:27OPENNetw,00:01:28EnjoyWeb,00:01:29DFI,00:01:2ATelemati,00:01:2BTELENET,00:01:2CAravoxTe,00:01:2DKomodoTe,00:01:2EPCPartne,00:01:2FTwinhead,00:01:30ExtremeN,00:01:31BoschSec,00:01:32Dranetz-,00:01:33KYOWAEle,00:01:34Selectro,00:01:35KDC,00:01:36CyberTAN,00:01:37ITFarm,00:01:38XAViTech,00:01:39PointMul,00:01:3AShelcadC,00:01:3BBna,00:01:3CTiw,00:01:3DRiscStat,00:01:3EAscomTat,00:01:3FNeighbor,00:01:40Sendtek,00:01:41CablePri,00:01:42Cisco,00:01:43Ieee802,00:01:44DellEMC,00:01:45Winsyste,00:01:46TescoCon,00:01:47ZhoneTec,00:01:48X-traWeb,00:01:49Tdt,00:01:4ASony,00:01:4BEnnovate,00:01:4CBerkeley,00:01:4DShinKinE,00:01:4EWINEnter,00:01:4FAdtran,00:01:50GilatCom,00:01:51Ensemble,00:01:52Chromate,00:01:53ArchtekT,00:01:54G3M,00:01:55PromiseT,00:01:56Firewire,00:01:57Syswave,00:01:58ElectroI,00:01:59S1,00:01:5ADigitalV,00:01:5BITALTELR,00:01:5CCadant,00:01:5DOracle,00:01:5EBestTech,00:01:5FDIGITALD,00:01:60ELMEX,00:01:61MetaMach,00:01:62CygnetTe,00:01:63NdcNatio,00:01:64Cisco,00:01:65AirSwitc,00:01:66TcGroup,00:01:67HiokiEE,00:01:68Vitana,00:01:69Celestix,00:01:6AAlitec,00:01:6BLightChi,00:01:6CFoxconn,00:01:6DCarrierC,00:01:6EConklin,00:01:6FInkel,00:01:70ESEEmbed,00:01:71AlliedDa,00:01:72TechnoLa,00:01:73Amcc,00:01:74CyberOpt,00:01:75RadiantC,00:01:76OrientSi,00:01:77Edsl,00:01:78MARGI,00:01:79Wireless,00:01:7AChengduM,00:01:7BHeidelbe,00:01:7CAG-E,00:01:7DThermoQu,00:01:7EADTEKSys,00:01:7FExperien,00:01:80AOpen,00:01:81NortelNe,00:01:82DicaTech,00:01:83AniteTel,00:01:84SiebMeye,00:01:85HitachiA,00:01:86UweDisch,00:01:87I2SE,00:01:88LXCOTech,00:01:89Refracti,00:01:8ARoiCompu,00:01:8BNetLinks,00:01:8CMegaVisi,00:01:8DAudeSiTe,00:01:8ELogitec,00:01:8FKenetec,00:01:90Smk-M,00:01:91SYREDDat,00:01:92TexasDig,00:01:93HanbyulT,00:01:94CapitalE,00:01:95SenaTech,00:01:96Cisco,00:01:97Cisco,00:01:98DarimVis,00:01:99HeiSeiEl,00:01:9ALEUNIG,00:01:9BKyotoMic,00:01:9CJDSUniph,00:01:9DE-Contro,00:01:9EESSTechn,00:01:9FReadyNet,00:01:A0Infinili,00:01:A1Mag-Tek,00:01:A2Logical,00:01:A3GenesysL,00:01:A4Microlin,00:01:A5Nextcomm,00:01:A6Scientif,00:01:A7UnexTech,00:01:A8Welltech,00:01:A9Bmw,00:01:AAAirspanC,00:01:ABMainStre,00:01:ACSitaraNe,00:01:ADCoachMas,00:01:AETrexEnte,00:01:AFArtesynE,00:01:B0FulltekT,00:01:B1GeneralB,00:01:B2DigitalP,00:01:B3Precisio,00:01:B4Wayport,00:01:B5TurinNet,00:01:B6SAEJINT&,00:01:B7Centos,00:01:B8Netsensi,00:01:B9SKFUK,00:01:BAIC-Net,00:01:BBFrequent,00:01:BCBrains,00:01:BDPeterson,00:01:BEGigalink,00:01:BFTeleforc,00:01:C0CompuLab,00:01:C1VitesseS,00:01:C2ARKResea,00:01:C3Acromag,00:01:C4NeoWave,00:01:C5SimplerN,00:01:C6QuarryTe,00:01:C7Cisco,00:01:C8ThomasCo,00:01:C9Cisco,00:01:CAGeocastN,00:01:CBEvr,00:01:CCJapanTot,00:01:CDARtem,00:01:CECustomMi,00:01:CFAlphaDat,00:01:D0VitalPoi,00:01:D1CoNetCom,00:01:D2inXtron,00:01:D3PAXCOMM,00:01:D4LeisureT,00:01:D5HaedongI,00:01:D6manrolan,00:01:D7F5Networ,00:01:D8Teltroni,00:01:D9Sigma,00:01:DAWINCOMM,00:01:DBFreecomT,00:01:DCActivete,00:01:DDAvailNet,00:01:DETrango,00:01:DFISDNComm,00:01:E0Fast,00:01:E1KinpoEle,00:01:E2AndoElec,00:01:E3Siemens,00:01:E4Sitera,00:01:E5Supernet,00:01:E6HewlettP,00:01:E7HewlettP,00:01:E8Force10N,00:01:E9LittonMa,00:01:EACirilium,00:01:EBC-COM,00:01:ECEricsson,00:01:EDSETA,00:01:EEComtrolE,00:01:EFCamtelTe,00:01:F0Tridium,00:01:F1Innovati,00:01:F2MarkUnic,00:01:F3QPS,00:01:F4Enterasy,00:01:F5Erim,00:01:F6Associat,00:01:F7ImageDis,00:01:F8TexioTec,00:01:F9TeraGlob,00:01:FAHoroscas,00:01:FBDoTopTec,00:01:FCKeyence,00:01:FDDigitalV,00:01:FEDigitalE,00:01:FFDataDire,00:02:00NetSys,00:02:01IFMElect,00:02:02AminoCom,00:02:03Woonsang,00:02:04BodmannI,00:02:05HitachiD,00:02:06TelitalR,00:02:07VisionGl,00:02:08UnifyNet,00:02:09Shenzhen,00:02:0AGefran,00:02:0BNativeNe,00:02:0CMetro-Op,00:02:0DMicronpc,00:02:0EECITelec,00:02:0FAatr,00:02:10Fenecom,00:02:11NatureWo,00:02:12SierraCo,00:02:13SDEL,00:02:14Dtvro,00:02:15CotasCom,00:02:16EsiExten,00:02:17Cisco,00:02:18Advanced,00:02:19ParalonT,00:02:1AZumaNetw,00:02:1BKollmorg,00:02:1CNetworkE,00:02:1DDataGene,00:02:1ESimtel,00:02:1FAculab,00:02:20CanonFin,00:02:21DSPAppli,00:02:22Chromisy,00:02:23ClickTV,00:02:24C-Cor,00:02:25OneStop,00:02:26XESystem,00:02:27ESDElect,00:02:28Necsom,00:02:29Adtec,00:02:2AAsoundEl,00:02:2BSAXA,00:02:2CABBBomem,00:02:2DAgere,00:02:2ETEACR&D,00:02:2FP-Cube,00:02:30Intersof,00:02:31Ingersol,00:02:32Avision,00:02:33MantraCo,00:02:34Imperial,00:02:35ParagonN,00:02:36INIT,00:02:37CosmoRes,00:02:38SeromeTe,00:02:39Visicom,00:02:3AZSKStick,00:02:3BEricsson,00:02:3CCreative,00:02:3DCisco,00:02:3ESeltaTel,00:02:3FCompalEl,00:02:40Seedek,00:02:41Amercom,00:02:42Videofra,00:02:43Raysis,00:02:44SURECOMT,00:02:45Lampus,00:02:46All-WinT,00:02:47GreatDra,00:02:48Pilz,00:02:49AvivInfo,00:02:4ACisco,00:02:4BCisco,00:02:4CSiByte,00:02:4DMannesma,00:02:4EDatacard,00:02:4FIPMDatac,00:02:50GeyserNe,00:02:51SomaNetw,00:02:52Carrier,00:02:53Televide,00:02:54WorldGat,00:02:55IBM,00:02:56AlphaPro,00:02:57Microcom,00:02:58FlyingPa,00:02:59TsannKue,00:02:5ACatenaNe,00:02:5BCambridg,00:02:5CSCIKunsh,00:02:5DCalixNet,00:02:5EHighTech,00:02:5FNortelNe,00:02:60Accordio,00:02:61Tilgin,00:02:62SoyoGrou,00:02:63UPSManuf,00:02:64AudioRam,00:02:65Virditec,00:02:66Thermalo,00:02:67NodeRunn,00:02:68HarrisGo,00:02:69Nadatel,00:02:6ACocessTe,00:02:6BBCMCompu,00:02:6CPhilipsC,00:02:6DAdeptTel,00:02:6ENeGeNAcc,00:02:6FSenaoInt,00:02:70Crewave,00:02:71ZhoneTec,00:02:72CC&CTech,00:02:73Coriolis,00:02:74TommyTec,00:02:75SMARTTec,00:02:76PrimaxEl,00:02:77CashSyst,00:02:78SamsungE,00:02:79ControlA,00:02:7AIOITechn,00:02:7BAmplifyN,00:02:7CTrilithi,00:02:7DCisco,00:02:7ECisco,00:02:7Fask-tech,00:02:80MuNet,00:02:81Madge,00:02:82ViaClix,00:02:83Spectrum,00:02:84UKGridSo,00:02:85Riversto,00:02:86OccamNet,00:02:87Adapcom,00:02:88GlobalVi,00:02:89DNETechn,00:02:8AAmbitMic,00:02:8BVDSL,00:02:8CMicrel-S,00:02:8DMovitaTe,00:02:8ERapid5Ne,00:02:8FGlobetek,00:02:90Woorigis,00:02:91OpenNetw,00:02:92LogicInn,00:02:93SolidDat,00:02:94TokyoSok,00:02:95IPAccess,00:02:96Lectron,00:02:97C-CORnet,00:02:98Broadfra,00:02:99Apex,00:02:9AStorageA,00:02:9BKreatelC,00:02:9C3Com,00:02:9DMerix,00:02:9EInformat,00:02:9FL-3Commu,00:02:A0Flatstac,00:02:A1WorldWid,00:02:A2Hilscher,00:02:A3ABBSwitz,00:02:A4AddPacTe,00:02:A5HewlettP,00:02:A6Effinet,00:02:A7VivaceNe,00:02:A8AirLinkT,00:02:A9RACOMsro,00:02:AAPLcom,00:02:ABCTCUnion,00:02:AC3PARdata,00:02:ADHOYA,00:02:AEScannexE,00:02:AFTeleCruz,00:02:B0HokubuCo,00:02:B1Anritsu,00:02:B2Cablevis,00:02:B3Intel,00:02:B4Daphne,00:02:B5Avnet,00:02:B6Acrosser,00:02:B7Watanabe,00:02:B8WhiKonsu,00:02:B9Cisco,00:02:BACisco,00:02:BBContinuo,00:02:BCLVL7,00:02:BDBionet,00:02:BETotsuEng,00:02:BFdotRocke,00:02:C0BencentT,00:02:C1Innovati,00:02:C2NetVisio,00:02:C3Arelnet,00:02:C4OPTMachi,00:02:C5EvertzMi,00:02:C6DataTrac,00:02:C7Alpsalpi,00:02:C8Technoco,00:02:C9Mellanox,00:02:CAEndPoint,00:02:CBTriState,00:02:CCMCCI,00:02:CDTeleDrea,00:02:CEFoxJet,00:02:CFZyGateCo,00:02:D0Comdial,00:02:D1Vivotek,00:02:D2Workstat,00:02:D3NetBotz,00:02:D4PDAPerip,00:02:D5Acr,00:02:D6NICE,00:02:D7EMPEG,00:02:D8BRECISCo,00:02:D9Reliable,00:02:DAExiOComm,00:02:DBNetsec,00:02:DCFujitsuG,00:02:DDBromaxCo,00:02:DEAstrodes,00:02:DFNetCom,00:02:E0ETAS,00:02:E1Integrat,00:02:E2NDCInfar,00:02:E3LITE-ONC,00:02:E4JCHYUN,00:02:E5Timeware,00:02:E6GouldIns,00:02:E7CAB,00:02:E8ED&A,00:02:E9CSSystem,00:02:EAFocusEnh,00:02:EBPicoComm,00:02:ECMaschoff,00:02:EDDXOTelec,00:02:EENokiaDan,00:02:EFCCCNetwo,00:02:F0AMEOptim,00:02:F1Pinetron,00:02:F2eDevice,00:02:F3MediaSer,00:02:F4PCTEL,00:02:F5VIVESyne,00:02:F6EquipeCo,00:02:F7Arm,00:02:F8SEAKREng,00:02:F9MIMOSBer,00:02:FADXAntenn,00:02:FBBaumulle,00:02:FCCisco,00:02:FDCisco,00:02:FEViditec,00:02:FFHandanBr,00:03:00Barracud,00:03:01Exfo,00:03:02CharlesI,00:03:03JAMAElec,00:03:04PacificB,00:03:05MSCVertr,00:03:06FusionIn,00:03:07SecureWo,00:03:08AMCommun,00:03:09TexcelTe,00:03:0AArgusTec,00:03:0BHunterTe,00:03:0CTelesoft,00:03:0DUniwillC,00:03:0ECoreComm,00:03:0FDigitalC,00:03:10E-Global,00:03:11MicroTec,00:03:12TRsystem,00:03:13AccessMe,00:03:14Teleware,00:03:15Cidco,00:03:16NobellCo,00:03:17Merlin,00:03:18Cyras,00:03:19Infineon,00:03:1ABeijingB,00:03:1BCellvisi,00:03:1CSvenskaH,00:03:1DTaiwanCo,00:03:1EOptranet,00:03:1FCondev,00:03:20Xpeed,00:03:21RecoRese,00:03:22IDIS,00:03:23CornetTe,00:03:24SANYOTec,00:03:25ArimaCom,00:03:26IwasakiI,00:03:27ActL,00:03:28MaceGrou,00:03:29F3,00:03:2AUniDataC,00:03:2BGAIDaten,00:03:2CABBSwitz,00:03:2DIBASETec,00:03:2EScopeInf,00:03:2FGlobalSu,00:03:30Imagenic,00:03:31Cisco,00:03:32Cisco,00:03:33Digitel,00:03:34OmegaEng,00:03:35MiraeTec,00:03:36ZetesTec,00:03:37Vaone,00:03:38OakTechn,00:03:39Eurologi,00:03:3ASiliconW,00:03:3BTAMITech,00:03:3CDaiden,00:03:3DILSHinLa,00:03:3ETateyama,00:03:3FBigBandN,00:03:40FlowareW,00:03:41AxonDigi,00:03:42NortelNe,00:03:43MartinPr,00:03:44Tietech,00:03:45RoutrekN,00:03:46HitachiK,00:03:47Intel,00:03:48NorscanI,00:03:49Vidicode,00:03:4ARIAS,00:03:4BNortelNe,00:03:4CShanghai,00:03:4DChiaroNe,00:03:4EPosData,00:03:4FSur-Gard,00:03:50Bticino,00:03:51Diebold,00:03:52Colubris,00:03:53Mitac,00:03:54FiberLog,00:03:55TeraBeam,00:03:56WincorNi,00:03:57Intervoi,00:03:58HanyangD,00:03:59DigitalS,00:03:5APhotron,00:03:5BBridgeWa,00:03:5CSaintSon,00:03:5DBosungHi,00:03:5EMetropol,00:03:5FPrüftech,00:03:60PACInter,00:03:61Widcomm,00:03:62VodtelCo,00:03:63Miraesys,00:03:64ScenixSe,00:03:65KiraInfo,00:03:66ASMPacif,00:03:67JasmineN,00:03:68Embedone,00:03:69NipponAn,00:03:6AMainnet,00:03:6BCisco,00:03:6CCisco,00:03:6DRuntop,00:03:6ENicon,00:03:6FTelsey,00:03:70NXTV,00:03:71AcomzNet,00:03:72Ulan,00:03:73Aselsan,00:03:74ControlM,00:03:75NetMedia,00:03:76Graphtec,00:03:77GigabitW,00:03:78HUMAX,00:03:79Proscend,00:03:7ATaiyoYud,00:03:7BIDECIZUM,00:03:7CCoaxMedi,00:03:7DStellcom,00:03:7EPORTechC,00:03:7FAtherosC,00:03:80SSHCommu,00:03:81Ingenico,00:03:82A-One,00:03:83MeteraNe,00:03:84Aeta,00:03:85ActelisN,00:03:86HoNet,00:03:87BlazeNet,00:03:88Fastfame,00:03:89Plantron,00:03:8AAmericaO,00:03:8BPLUS-ONE,00:03:8CTotalImp,00:03:8DPCSReven,00:03:8EAtoga,00:03:8FWeinsche,00:03:90DigitalV,00:03:91Advanced,00:03:92HyundaiT,00:03:93Apple,00:03:94ConnectO,00:03:95Californ,00:03:96EZCast,00:03:97FireBric,00:03:98Wisi,00:03:99DongjuIn,00:03:9ASiConnec,00:03:9BNetChipT,00:03:9COptiMigh,00:03:9DQisda,00:03:9ETeraSyst,00:03:9FCisco,00:03:A0Cisco,00:03:A1HIPERInf,00:03:A2Catapult,00:03:A3MAVIX,00:03:A4Imation,00:03:A5Medea,00:03:A6TraxitTe,00:03:A7UnixtarT,00:03:A8IDOTComp,00:03:A9AXCENTMe,00:03:AAWatlow,00:03:ABBridgeIn,00:03:ACFroniusS,00:03:ADEmersonE,00:03:AEAlliedAd,00:03:AFParageaC,00:03:B0XsenseTe,00:03:B1Hospira,00:03:B2Radware,00:03:B3IALink,00:03:B4Macrotek,00:03:B5EntraTec,00:03:B6QSI,00:03:B7ZACCESS,00:03:B8NetKitSo,00:03:B9HualongT,00:03:BAOracle,00:03:BBSignalCo,00:03:BCCOT,00:03:BDOmniClus,00:03:BENetility,00:03:BFCenterpo,00:03:C0RFTNC,00:03:C1PacketDy,00:03:C2Solphone,00:03:C3Micronik,00:03:C4TomraASA,00:03:C5Mobotix,00:03:C6ICUE,00:03:C7hopfElek,00:03:C8CMLEmerg,00:03:C9TECOM,00:03:CAMTS,00:03:CBSystemGe,00:03:CCMomentum,00:03:CDCloverte,00:03:CEETENTech,00:03:CFMuxcom,00:03:D0KOANKEIS,00:03:D1Takaya,00:03:D2Crossbea,00:03:D3Internet,00:03:D4Alloptic,00:03:D5Advanced,00:03:D6RADVisio,00:03:D7NextNetW,00:03:D8iMPathNe,00:03:D9Secheron,00:03:DATakamisa,00:03:DBApogeeEl,00:03:DCLexarMed,00:03:DDComarkIn,00:03:DEOTCWirel,00:03:DFDesana,00:03:E0ARRISGro,00:03:E1WinmateC,00:03:E2Comspace,00:03:E3Cisco,00:03:E4Cisco,00:03:E5Hermsted,00:03:E6Entone,00:03:E7Logostek,00:03:E8Wavesigh,00:03:E9AkaraCan,00:03:EAMegaSyst,00:03:EBAtrica,00:03:ECICGResea,00:03:EDShinkawa,00:03:EEMKNet,00:03:EFOneline,00:03:F0RedfernB,00:03:F1CicadaSe,00:03:F2SenecaNe,00:03:F3DazzleMu,00:03:F4NetBurne,00:03:F5Chip2Chi,00:03:F6AllegroN,00:03:F7Plast-Co,00:03:F8SanCastl,00:03:F9Pleiades,00:03:FATiMetraN,00:03:FBENEGATE,00:03:FCIntertex,00:03:FDCisco,00:03:FECisco,00:03:FFMicrosof,00:04:00LexmarkP,00:04:01OsakiEle,00:04:02NexsanTe,00:04:03Nexsi,00:04:04MakinoMi,00:04:05ACNTechn,00:04:06FaMetabo,00:04:07TopconPo,00:04:08SankoEle,00:04:09CratosNe,00:04:0ASage,00:04:0B3ComEuro,00:04:0CKannoWor,00:04:0DAvaya,00:04:0EAVM,00:04:0FAsusNetw,00:04:10Spinnake,00:04:11InkraNet,00:04:12WaveSmit,00:04:13snomtech,00:04:14UmezawaM,00:04:15Rasteme,00:04:16ParksCom,00:04:17Elau,00:04:18Teltroni,00:04:19Fibercyc,00:04:1AInesTest,00:04:1BBridgewo,00:04:1CipDialog,00:04:1DCoregaAm,00:04:1EShikokuI,00:04:1FSonyInte,00:04:20SlimDevi,00:04:21OcularNe,00:04:22StudioTe,00:04:23Intel,00:04:24TMC,00:04:25Atmel,00:04:26Autosys,00:04:27Cisco,00:04:28Cisco,00:04:29Pixord,00:04:2AWireless,00:04:2BITAccess,00:04:2CMinet,00:04:2DSarian,00:04:2ENetousTe,00:04:2FInternat,00:04:30Netgem,00:04:31GlobalSt,00:04:32VoyetraT,00:04:33Cyberboa,00:04:34Accelent,00:04:35InfiNet,00:04:36ELANsatT,00:04:37PowinInf,00:04:38NortelNe,00:04:39RoscoEnt,00:04:3AIntellig,00:04:3BLavaComp,00:04:3CSONOS,00:04:3DIndel,00:04:3ETelencom,00:04:3FESTeemWi,00:04:40cyberPIX,00:04:41HalfDome,00:04:42Nact,00:04:43AgilentT,00:04:44WesternM,00:04:45LMSSkala,00:04:46CYZENTEC,00:04:47Acrowave,00:04:48Polaroid,00:04:49Mapletre,00:04:4AiPolicyN,00:04:4BNvidia,00:04:4CJenoptik,00:04:4DCisco,00:04:4ECisco,00:04:4FSchubert,00:04:50DMDCompu,00:04:51Medrad,00:04:52RocketLo,00:04:53YottaYot,00:04:54Quadriga,00:04:55ANTARAne,00:04:56CambiumN,00:04:57Universa,00:04:58FusionX,00:04:59Veristar,00:04:5ALinksysG,00:04:5BTechsanE,00:04:5CMobiwave,00:04:5DBEKAElek,00:04:5EPolyTrax,00:04:5FAvalueTe,00:04:60KnilinkT,00:04:61EPOXComp,00:04:62DAKOSDat,00:04:63BoschSec,00:04:64Pulse-Li,00:04:65istisdn-,00:04:66ARMITEL,00:04:67WuhanRes,00:04:68Vivity,00:04:69Innocom,00:04:6ANaviniNe,00:04:6BPalmWire,00:04:6CCyberTec,00:04:6DCisco,00:04:6ECisco,00:04:6FDigitelI,00:04:70ipUnplug,00:04:71IPrad,00:04:72Telelynx,00:04:73Photonex,00:04:74Legrand,00:04:753Com,00:04:763Com,00:04:77Scalant,00:04:78GStarTec,00:04:79Radius,00:04:7AAxxessit,00:04:7BSchlumbe,00:04:7CSkidata,00:04:7DMotorola,00:04:7ESiqura,00:04:7FChrMayr,00:04:80BrocadeC,00:04:81Econolit,00:04:82Medialog,00:04:83DeltronT,00:04:84Amann,00:04:85PicoLigh,00:04:86ITTCUniv,00:04:87CogencyS,00:04:88Eurother,00:04:89YAFONetw,00:04:8ATemiaVer,00:04:8BPoscon,00:04:8CNaynaNet,00:04:8DTeoTechn,00:04:8EOhmTechL,00:04:8FTD,00:04:90OpticalA,00:04:91Technovi,00:04:92HiveInte,00:04:93Tsinghua,00:04:94Breezeco,00:04:95TejasNet,00:04:96ExtremeN,00:04:97MacroSys,00:04:98MahiNetw,00:04:99Chino,00:04:9ACisco,00:04:9BCisco,00:04:9CSurgient,00:04:9DIpanemaT,00:04:9EWirelink,00:04:9FFreescal,00:04:A0VerityIn,00:04:A1PathwayC,00:04:A2LSIJapan,00:04:A3Microchi,00:04:A4NetEnabl,00:04:A5BarcoPro,00:04:A6SAFTehni,00:04:A7FabiaTec,00:04:A8Broadmax,00:04:A9SandStre,00:04:AAJetstrea,00:04:ABMavenir,00:04:ACIbmPcmci,00:04:ADMalibuNe,00:04:AESullair,00:04:AFDigitalF,00:04:B0ELESIGN,00:04:B1SignalTe,00:04:B2Essegi,00:04:B3Videotek,00:04:B4Ciac,00:04:B5Equitrac,00:04:B6StratexN,00:04:B7AMBit,00:04:B8Kumahira,00:04:B9SISoubou,00:04:BAKDDMedia,00:04:BBBardac,00:04:BCGiantec,00:04:BDARRISGro,00:04:BEOptXCon,00:04:BFVersaLog,00:04:C0Cisco,00:04:C1Cisco,00:04:C2Magnipix,00:04:C3CASTORIn,00:04:C4Audioton,00:04:C5ASETechn,00:04:C6YamahaMo,00:04:C7NetMount,00:04:C8LIBAMasc,00:04:C9MicroEle,00:04:CAFreeMs,00:04:CBTdsoftCo,00:04:CCPeekTraf,00:04:CDExtenway,00:04:CEPatriaAi,00:04:CFSeagateT,00:04:D0Softlink,00:04:D1DrewTech,00:04:D2AdconTel,00:04:D3Toyokeik,00:04:D4ProviewE,00:04:D5HitachiI,00:04:D6TakagiIn,00:04:D7OmitecIn,00:04:D8IPWirele,00:04:D9TitanEle,00:04:DARelaxTec,00:04:DBTellusGr,00:04:DCNortelNe,00:04:DDCisco,00:04:DECisco,00:04:DFTeracomT,00:04:E0ProcketN,00:04:E1Infinior,00:04:E2SMCNetwo,00:04:E3AcctonTe,00:04:E4Daeryung,00:04:E5Glonet,00:04:E6BanyanNe,00:04:E7Lightpoi,00:04:E8IER,00:04:E9Infinisw,00:04:EAHewlettP,00:04:EBPaxonetC,00:04:ECMemobox,00:04:EDBillionE,00:04:EELincolnE,00:04:EFPolestar,00:04:F0Internat,00:04:F1WhereNet,00:04:F2Polycom,00:04:F3FSFORTH-,00:04:F4Infinite,00:04:F5SnowShor,00:04:F6Amphus,00:04:F7OmegaBan,00:04:F8QUALICAB,00:04:F9XteraCom,00:04:FANBSTechn,00:04:FBCommtech,00:04:FCStratusT,00:04:FDJapanCon,00:04:FEPelagoNe,00:04:FFAcronet,00:05:00Cisco,00:05:01Cisco,00:05:02ApplePci,00:05:03Iconag,00:05:04NarayInf,00:05:05Integrat,00:05:06ReddoNet,00:05:07FineAppl,00:05:08Inetcam,00:05:09AVOCNish,00:05:0AICS,00:05:0BSICOM,00:05:0CNetworkP,00:05:0DMidstrea,00:05:0E3ware,00:05:0FTanakaSS,00:05:10Infinite,00:05:11Compleme,00:05:12ZebraTec,00:05:13VTLinxMu,00:05:14KDT,00:05:15Nuark,00:05:16SMARTMod,00:05:17Shellcom,00:05:18Jupiters,00:05:19SiemensB,00:05:1A3ComEuro,00:05:1BMagicCon,00:05:1CXnetTech,00:05:1DAirocon,00:05:1EBrocadeC,00:05:1FTaijinMe,00:05:20Smartron,00:05:21ControlM,00:05:22LEA*D,00:05:23AVLList,00:05:24BTLSyste,00:05:25PuretekI,00:05:26IPAS,00:05:27SJTek,00:05:28NewFocus,00:05:29Shanghai,00:05:2AIkegamiT,00:05:2BHORIBA,00:05:2CSupremeM,00:05:2DZoltrixI,00:05:2ECintaNet,00:05:2FLevitonN,00:05:30Andiamo,00:05:31Cisco,00:05:32Cisco,00:05:33BrocadeC,00:05:34Northsta,00:05:35ChipPC,00:05:36DanamCom,00:05:37NetsTech,00:05:38Merilus,00:05:39ABrandNe,00:05:3AWillowgl,00:05:3BHarbourN,00:05:3CXircom,00:05:3DAgere,00:05:3EKIDSyste,00:05:3FVisionTe,00:05:40FAST,00:05:41Advanced,00:05:42Otari,00:05:43IQWirele,00:05:44ValleyTe,00:05:45Internet,00:05:46KDDINetw,00:05:47StarentN,00:05:48Disco,00:05:49SaliraOp,00:05:4AArioData,00:05:4BEatonAut,00:05:4CRFInnova,00:05:4DBransTec,00:05:4EPhilips,00:05:4FGarminIn,00:05:50VcommsCo,00:05:51FSElektr,00:05:52XycotecC,00:05:53DVC,00:05:54Rangesta,00:05:55JapanCas,00:05:56360,00:05:57AgileTV,00:05:58Synchron,00:05:59Intracom,00:05:5APowerDsi,00:05:5BCharlesI,00:05:5CKowa,00:05:5DD-Link,00:05:5ECisco,00:05:5FCisco,00:05:60LeaderCo,00:05:61nacImage,00:05:62DigitalV,00:05:63J-Works,00:05:64Tsinghua,00:05:65TailynCo,00:05:66Secuicom,00:05:67Etymonic,00:05:68Piltofis,00:05:69VMware,00:05:6AHeuftSys,00:05:6BCPTechno,00:05:6CHungChan,00:05:6DPacific,00:05:6ENational,00:05:6FInnomedi,00:05:70Baydel,00:05:71SeiwaEle,00:05:72Deonet,00:05:73Cisco,00:05:74Cisco,00:05:75CDS-Elec,00:05:76NSMTechn,00:05:77SMInform,00:05:78Private,00:05:79Universa,00:05:7AOverture,00:05:7BChungNam,00:05:7CRCOSecur,00:05:7DSunCommu,00:05:7EEckelman,00:05:7FAcqisTec,00:05:80FibroLAN,00:05:81Snell,00:05:82ClearCub,00:05:83ImageCom,00:05:84Absolute,00:05:85JuniperN,00:05:86LucentTe,00:05:87Locus,00:05:88Sensoria,00:05:89National,00:05:8ANetcom,00:05:8BIPmental,00:05:8COpentech,00:05:8DLynxPhot,00:05:8EFlextron,00:05:8FCLCsoft,00:05:90Swissvoi,00:05:91ActiveSi,00:05:92Pultek,00:05:93GrammarE,00:05:94HMSIndus,00:05:95Alesis,00:05:96Genotech,00:05:97EagleTra,00:05:98CRONOS,00:05:99DRSTesta,00:05:9ACisco,00:05:9BCisco,00:05:9CKleinkne,00:05:9DDanielCo,00:05:9EZinwell,00:05:9FYottaNet,00:05:A0MOBILINE,00:05:A1Zenocom,00:05:A2CELOXNet,00:05:A3QEI,00:05:A4LucidVoi,00:05:A5Kott,00:05:A6ExtronEl,00:05:A7HYPERCHI,00:05:A8WyleElec,00:05:A9Princeto,00:05:AAMooreInd,00:05:ABCyberFon,00:05:ACNorthern,00:05:ADTopspinC,00:05:AEMediapor,00:05:AFInnoScan,00:05:B0KoreaCom,00:05:B1ASBTechn,00:05:B2Medison,00:05:B3Asahi-En,00:05:B4Aceex,00:05:B5Broadcom,00:05:B6INSYSMic,00:05:B7ArborTec,00:05:B8Electron,00:05:B9Airvana,00:05:BAAreaNetw,00:05:BBMyspace,00:05:BCResource,00:05:BDRoax,00:05:BEKongsber,00:05:BFJustEzyT,00:05:C0DigitalN,00:05:C1A-KyungM,00:05:C2Soronti,00:05:C3PacificI,00:05:C4Telect,00:05:C5FlagaHF,00:05:C6TrizComm,00:05:C7IF-Com,00:05:C8Verytech,00:05:C9LGInnote,00:05:CAHitronTe,00:05:CBROISTech,00:05:CCSumtelCo,00:05:CDD&MHoldi,00:05:CEProlinkM,00:05:CFThunderR,00:05:D0Solinet,00:05:D1Metavect,00:05:D2DAPTechn,00:05:D3eProduct,00:05:D4FutureSm,00:05:D5Speedcom,00:05:D6L-3Linka,00:05:D7VistaIma,00:05:D8Arescom,00:05:D9TechnoVa,00:05:DAApexAuto,00:05:DBPSINente,00:05:DCCisco,00:05:DDCisco,00:05:DEGiFoneKo,00:05:DFElectron,00:05:E0Empirix,00:05:E1TrellisP,00:05:E2CreativN,00:05:E3LightSan,00:05:E4RedLionC,00:05:E5Renishaw,00:05:E6Egenera,00:05:E7Netrakea,00:05:E8TurboWav,00:05:E9UnicessN,00:05:EARednix,00:05:EBBlueRidg,00:05:ECMosaic,00:05:EDTechniku,00:05:EEVanderbi,00:05:EFADOIRDig,00:05:F0Satec,00:05:F1Vrcom,00:05:F2PowerR,00:05:F3Webyn,00:05:F4SystemBa,00:05:F5Geospace,00:05:F6YoungCha,00:05:F7AnalogDe,00:05:F8RealTime,00:05:F9TOA,00:05:FAIPOptica,00:05:FBShareGat,00:05:FCSchenckP,00:05:FDPacketLi,00:05:FETraficon,00:05:FFSNSSolut,00:06:00ToshibaT,00:06:01Otanikei,00:06:02Cirkitec,00:06:03BakerHug,00:06:04@TrackCo,00:06:05InncomIn,00:06:06RapidWAN,00:06:07OmniDire,00:06:08At-SkySA,00:06:09Crosspor,00:06:0ABlue2spa,00:06:0BArtesynE,00:06:0CMelcoInd,00:06:0DWave7Opt,00:06:0EIGYS,00:06:0FNaradNet,00:06:10AbeonaNe,00:06:11ZeusWire,00:06:12Accusys,00:06:13Kawasaki,00:06:14PrismHol,00:06:15KimotoEl,00:06:16TelNet,00:06:17Redswitc,00:06:18DigiPowe,00:06:19Connecti,00:06:1AZetari,00:06:1BNotebook,00:06:1CHoshinoM,00:06:1DMIPTelec,00:06:1EMaxan,00:06:1FVisionCo,00:06:20SerialSy,00:06:21Hinox,00:06:22ChungFuC,00:06:23MGEUPSFr,00:06:24GentnerC,00:06:25LinksysG,00:06:26MWE,00:06:27UniwideT,00:06:28Cisco,00:06:29IbmRisc6,00:06:2ACisco,00:06:2BIntraser,00:06:2CBivioNet,00:06:2DTouchSta,00:06:2EAristosL,00:06:2FPivotech,00:06:30AdtranzS,00:06:31Calix,00:06:32MescoEng,00:06:33CrossMat,00:06:34GTEAirfo,00:06:35PacketAi,00:06:36JedaiBro,00:06:37Toptrend,00:06:38SungjinC,00:06:39Newtec,00:06:3ADuraMicr,00:06:3BArcturus,00:06:3CIntrinsy,00:06:3DMicrowav,00:06:3EOpthos,00:06:3FEverexCo,00:06:40WhiteRoc,00:06:41Itcn,00:06:42Genetel,00:06:43SONOComp,00:06:44NextGenB,00:06:45MeiseiEl,00:06:46ShenZhen,00:06:47Etrali,00:06:48Seedswar,00:06:493MDeutsc,00:06:4AHoneywel,00:06:4BAlexon,00:06:4CInvictaN,00:06:4DSencore,00:06:4EBroadNet,00:06:4FPRO-NETS,00:06:50TiburonN,00:06:51AspenNet,00:06:52Cisco,00:06:53Cisco,00:06:54Winpresa,00:06:55Yipee,00:06:56Tactel,00:06:57MarketCe,00:06:58HelmutFi,00:06:59EALApeld,00:06:5AStrix,00:06:5BDell,00:06:5CMalachit,00:06:5DHeidelbe,00:06:5EPhoturis,00:06:5FECITelec,00:06:60NADEX,00:06:61NIAHomeT,00:06:62MBMTechn,00:06:63HumanTec,00:06:64Fostex,00:06:65SunnyGik,00:06:66RovingNe,00:06:67TrippLit,00:06:68ViconInd,00:06:69Datasoun,00:06:6AInfiniCo,00:06:6BSysmex,00:06:6CRobinson,00:06:6DCompupri,00:06:6EDeltaEle,00:06:6FKoreaDat,00:06:70Upponett,00:06:71Softing,00:06:72Netezza,00:06:73TKHSecur,00:06:74Spectrum,00:06:75Banderac,00:06:76NovraTec,00:06:77Sick,00:06:78D&MHoldi,00:06:79Konami,00:06:7AJMP,00:06:7BToplinkC,00:06:7CCisco,00:06:7DTakasago,00:06:7EWinCom,00:06:7FDigeo,00:06:80CardAcce,00:06:81GoepelEl,00:06:82Convedia,00:06:83BravaraC,00:06:84Biacore,00:06:85NetNearU,00:06:86ZARDCOM,00:06:87Omnitron,00:06:88TelwaysC,00:06:89yLezTech,00:06:8ANeuronNe,00:06:8BAirRunne,00:06:8C3Com,00:06:8DSEPATON,00:06:8EHID,00:06:8FTelemoni,00:06:90EuracomC,00:06:91PTInovac,00:06:92Intruver,00:06:93FlexusCo,00:06:94Mobillia,00:06:95EnsureTe,00:06:96AdventNe,00:06:97RDCenter,00:06:98egnite,00:06:99VidaDesi,00:06:9AeTel,00:06:9BAVTAudio,00:06:9CTransmod,00:06:9DPetards,00:06:9EUNIQA,00:06:9FKuokoaNe,00:06:A0MxImagin,00:06:A1CelsianT,00:06:A2Microtun,00:06:A3Bitran,00:06:A4INNOWELL,00:06:A5PINON,00:06:A6Artistic,00:06:A7Primario,00:06:A8KCTechno,00:06:A9Universa,00:06:AAVTMiltop,00:06:ABW-Link,00:06:ACIntersof,00:06:ADKBElectr,00:06:AEHimachal,00:06:AFXaltedNe,00:06:B0ComtechE,00:06:B1Sonicwal,00:06:B2Linxtek,00:06:B3Diagraph,00:06:B4VorneInd,00:06:B5SourcePh,00:06:B6Nir-OrIs,00:06:B7TELEM,00:06:B8Bandspee,00:06:B9A5TEK,00:06:BAWestwave,00:06:BBATITechn,00:06:BCMacrolin,00:06:BDBNTECHNO,00:06:BEBaumerOp,00:06:BFAccellaT,00:06:C0UnitedIn,00:06:C1Cisco,00:06:C2Smartmat,00:06:C3Schindle,00:06:C4Piolink,00:06:C5INNOVITe,00:06:C6lesswire,00:06:C7RFNETTec,00:06:C8Sumitomo,00:06:C9Technica,00:06:CAAmerican,00:06:CBJotronEl,00:06:CCJMIElect,00:06:CDLeafImag,00:06:CEDateno,00:06:CFThalesAv,00:06:D0ElgarEle,00:06:D1TahoeNet,00:06:D2TundraSe,00:06:D3AlphaTel,00:06:D4Interact,00:06:D5Diamond,00:06:D6Cisco,00:06:D7Cisco,00:06:D8MapleOpt,00:06:D9IPM-Net,00:06:DAITRANCom,00:06:DBICHIPS,00:06:DCSyabasTe,00:06:DDATTLabor,00:06:DEFlashTec,00:06:DFAIDONIC,00:06:E0MAT,00:06:E1TechnoTr,00:06:E2CeemaxTe,00:06:E3Quantita,00:06:E4CitelTec,00:06:E5FujianNe,00:06:E6DongYang,00:06:E7BitBlitz,00:06:E8OpticalN,00:06:E9Intime,00:06:EAELZET80M,00:06:EBGlobalDa,00:06:ECHarris,00:06:EDInaraNet,00:06:EEShenyang,00:06:EFMaxxan,00:06:F0Digeo,00:06:F1Optillio,00:06:F2PlatysCo,00:06:F3AcceLigh,00:06:F4PrimeEle,00:06:F5Alpsalpi,00:06:F6Cisco,00:06:F7Alpsalpi,00:06:F8Boeing,00:06:F9MitsuiZo,00:06:FAIPSQUARE,00:06:FBHitachiP,00:06:FCFnet,00:06:FDComjetIn,00:06:FEAmbrado,00:06:FFSheba,00:07:00Zettamed,00:07:01Racal-Da,00:07:02VarexIma,00:07:03CSEETran,00:07:04Alpsalpi,00:07:05EndressH,00:07:06Sanritz,00:07:07Interali,00:07:08Bitrage,00:07:09Westerst,00:07:0AUnicomAu,00:07:0BNovabase,00:07:0CSVA-Intr,00:07:0DCisco251,00:07:0ECisco,00:07:0FFujant,00:07:10Adax,00:07:11Acterna,00:07:12JALInfor,00:07:13IPOne,00:07:14Brightco,00:07:15GeneralR,00:07:16JSMarine,00:07:17WielandE,00:07:18iCanTek,00:07:19Mobiis,00:07:1AFinedigi,00:07:1BCDVIAmer,00:07:1CAt&T,00:07:1DSatelsaS,00:07:1ETri-MEng,00:07:1FEuropean,00:07:20Trutzsch,00:07:21FormacEl,00:07:22Nielsen,00:07:23ELCONSys,00:07:24Telemax,00:07:25Bematech,00:07:26Shenzhen,00:07:27ZiHK,00:07:28NeoTelec,00:07:29KistlerI,00:07:2AInnovanc,00:07:2BJungMyun,00:07:2CFabricom,00:07:2DCNSystem,00:07:2ENorthNod,00:07:2FIntransa,00:07:30Hutchiso,00:07:31Ophir-Sp,00:07:32AAEONTec,00:07:33DANCONTR,00:07:34ONStor,00:07:35FlarionT,00:07:36DataVide,00:07:37Soriya,00:07:38YoungTec,00:07:39ScottyGr,00:07:3AInventel,00:07:3BTenovis,00:07:3CTelecomD,00:07:3DNanjingP,00:07:3EChinaGre,00:07:3FWoojyunS,00:07:40Buffalo,00:07:41SierraAu,00:07:42Ormazaba,00:07:43ChelsioC,00:07:44Unico,00:07:45RadlanCo,00:07:46TURCK,00:07:47Mecalc,00:07:48ImagingS,00:07:49CENiX,00:07:4ACarlVale,00:07:4BDaihen,00:07:4CBeicom,00:07:4DZebraTec,00:07:4EIPFRONT,00:07:4FCisco,00:07:50Cisco,00:07:51m-u-t,00:07:52RhythmWa,00:07:53BeijingQ,00:07:54XyterraC,00:07:55Lafon,00:07:56JuyoungT,00:07:57TopcallI,00:07:58DragonWa,00:07:59BorisMan,00:07:5AAirProdu,00:07:5BGibsonGu,00:07:5CEastmanK,00:07:5DCellerit,00:07:5EAmetekPo,00:07:5FVCSVideo,00:07:60TOMISInf,00:07:6129530,00:07:62GroupSen,00:07:63Sunniwel,00:07:64YoungWoo,00:07:65JadeQuan,00:07:66ChouChin,00:07:67YuxingEl,00:07:68Danfoss,00:07:69Italiana,00:07:6ANEXTEYE,00:07:6BStralfor,00:07:6CDaehanet,00:07:6DFlexligh,00:07:6ESinetica,00:07:6FSynoptic,00:07:70Ubiquoss,00:07:71Embedded,00:07:72Alcatel-,00:07:73AscomPow,00:07:74GuangZho,00:07:75ValenceS,00:07:76FederalA,00:07:77Motah,00:07:78GERSTEL,00:07:79SungilTe,00:07:7AInfoware,00:07:7BMillimet,00:07:7CWestermo,00:07:7DCisco,00:07:7EElrest,00:07:7FJCommuni,00:07:80Bluegiga,00:07:81Itron,00:07:82Oracle,00:07:83SynComNe,00:07:84Cisco,00:07:85Cisco,00:07:86Wireless,00:07:87IdeaSyst,00:07:88Clipcomm,00:07:89Allradio,00:07:8AMentorDa,00:07:8BWegenerC,00:07:8CElektron,00:07:8DNetEngin,00:07:8EGarzFric,00:07:8FEmkayInn,00:07:90Tri-MTec,00:07:91Internat,00:07:92SütronEl,00:07:93ShinSate,00:07:94SimpleDe,00:07:95Elitegro,00:07:96LSI,00:07:97Netpower,00:07:98Selea,00:07:99TippingP,00:07:9AVerint,00:07:9BAuroraNe,00:07:9CGoldenEl,00:07:9DMusashi,00:07:9EIlinx,00:07:9FActionDi,00:07:A0e-Watch,00:07:A1VIASYSHe,00:07:A2Opteon,00:07:A3OsitisSo,00:07:A4GNNetcom,00:07:A5YDK,00:07:A6LevitonM,00:07:A7A-Z,00:07:A8HaierGro,00:07:A9Novasoni,00:07:AAQuantumD,00:07:ABSamsungE,00:07:ACEolring,00:07:ADPentacon,00:07:AEBritestr,00:07:AFRedLionC,00:07:B0OfficeDe,00:07:B1EquatorT,00:07:B2Transacc,00:07:B3Cisco,00:07:B4Cisco,00:07:B5AnyOneWi,00:07:B6TelecomT,00:07:B7SamuraiI,00:07:B8Corvalen,00:07:B9Ginganet,00:07:BAUTStarco,00:07:BBCandera,00:07:BCIdentix,00:07:BDRadionet,00:07:BEDataLogi,00:07:BFArmillai,00:07:C0NetZerve,00:07:C1Overture,00:07:C2NetsysTe,00:07:C3Thomson,00:07:C4JEAN,00:07:C5Gcom,00:07:C6VDSVossk,00:07:C7Synectic,00:07:C8Brain21,00:07:C9TechnolS,00:07:CACreatixP,00:07:CBFreeboxS,00:07:CCKabaBenz,00:07:CDKumohEle,00:07:CECabletim,00:07:CFAnoto,00:07:D0AutomatE,00:07:D1Spectrum,00:07:D2LogopakS,00:07:D3SPGPrint,00:07:D4Zhejiang,00:07:D53eTechno,00:07:D6Commil,00:07:D7CaporisN,00:07:D8HitronTe,00:07:D9Spliceco,00:07:DANeuroTel,00:07:DBKiranaNe,00:07:DCAtek,00:07:DDCradleTe,00:07:DEeCopilt,00:07:DFVbrick,00:07:E0Palm,00:07:E1WISCommu,00:07:E2Bitworks,00:07:E3NavcomTe,00:07:E4SoftRadi,00:07:E5Coup,00:07:E6edgeflow,00:07:E7FreeWave,00:07:E8EdgeWave,00:07:E9Intel,00:07:EAMassana,00:07:EBCisco,00:07:ECCisco,00:07:EDAltera,00:07:EEtelcoInf,00:07:EFLockheed,00:07:F0LogiSync,00:07:F1TeraBurs,00:07:F2IOA,00:07:F3Thinkeng,00:07:F4Eletex,00:07:F5Bridgeco,00:07:F6QqestSof,00:07:F7Galtroni,00:07:F8ITDevice,00:07:F9Sensapho,00:07:FAITT,00:07:FBGigaStre,00:07:FCAdept,00:07:FDLANergy,00:07:FERigaku,00:07:FFGluonNet,00:08:00Multitec,00:08:01HighSpee,00:08:02HewlettP,00:08:03CosTron,00:08:04ICA,00:08:05Techno-H,00:08:06Raonet,00:08:07AccessDe,00:08:08PPTVisio,00:08:09Systemon,00:08:0AEspera-W,00:08:0BBirkaBPA,00:08:0CVDAGroup,00:08:0DToshiba,00:08:0EARRISGro,00:08:0FProximio,00:08:10KeyTechn,00:08:11VOIX,00:08:12GM-2,00:08:13Diskbank,00:08:14TILTechn,00:08:15CATS,00:08:16BluelonA,00:08:17EmergeCo,00:08:18Pixelwor,00:08:19Banksys,00:08:1ASanradIn,00:08:1BWindigo,00:08:1C@poscom,00:08:1DIpsil,00:08:1ERepeatit,00:08:1FPouYuenT,00:08:20Cisco,00:08:21Cisco,00:08:22InProCom,00:08:23Texa,00:08:24NuanceDo,00:08:25AcmePack,00:08:26Colorado,00:08:27ADBBroad,00:08:28KoeiEngi,00:08:29TokyoEle,00:08:2APowerwal,00:08:2BWooksung,00:08:2CHomag,00:08:2DIndusTeq,00:08:2EMultiton,00:08:2FCisco,00:08:30Cisco,00:08:31Cisco,00:08:32Cisco,00:08:4EDivergeN,00:08:4FQualstar,00:08:50ArizonaI,00:08:51Canadian,00:08:52Davolink,00:08:53Schleich,00:08:54Netronix,00:08:55NASA-God,00:08:56Gamatron,00:08:57PolarisN,00:08:58Novatech,00:08:59ShenZhen,00:08:5AIntiGate,00:08:5BHanbitEl,00:08:5CShanghai,00:08:5DMitel,00:08:5EPco,00:08:5FPicanol,00:08:60LodgeNet,00:08:61SoftEner,00:08:62NECElumi,00:08:63Entrisph,00:08:64Fasy,00:08:65Jascom,00:08:66DSXAcces,00:08:67UptimeDe,00:08:68PurOptix,00:08:69Command-,00:08:6ASecurito,00:08:6BMipsys,00:08:6CPlasmonL,00:08:6DMissouri,00:08:6EHyglo,00:08:6FResource,00:08:70Rasvia,00:08:71NORTHDAT,00:08:72Sorenson,00:08:73DapTechn,00:08:74Dell,00:08:75AcorpEle,00:08:76SDSystem,00:08:77Liebert-,00:08:78Benchmar,00:08:79CEM,00:08:7AWipotec,00:08:7BRTXTelec,00:08:7CCisco,00:08:7DCisco,00:08:7EBonElect,00:08:7FSPAUNele,00:08:80BroadTel,00:08:81DigitalH,00:08:82Sigma,00:08:83HewlettP,00:08:84IndexBra,00:08:85EMSDrTho,00:08:86HansungT,00:08:87Maschine,00:08:88OULLIMIn,00:08:89DishTech,00:08:8AMinds@Wo,00:08:8BTropicNe,00:08:8CQuantaNe,00:08:8DSigma-Li,00:08:8ENihonCom,00:08:8FAdvanced,00:08:90Avilinks,00:08:91Lyan,00:08:92EMSoluti,00:08:93LeInform,00:08:94InnoVISI,00:08:95DIRCTech,00:08:96Printron,00:08:97QuakeTec,00:08:98GigabitO,00:08:99Netbind,00:08:9AAlcatelM,00:08:9BICPElect,00:08:9CElecsInd,00:08:9DUHD-Elek,00:08:9EBeijingE,00:08:9FEFMNetwo,00:08:A0StotzFei,00:08:A1CNetTech,00:08:A2ADIEngin,00:08:A3Cisco,00:08:A4Cisco,00:08:A5Peninsul,00:08:A6Multiwar,00:08:A7iLogic,00:08:A8Systec,00:08:A9SangSang,00:08:AAKaram,00:08:ABEnerLinx,00:08:ACBST,00:08:ADToyo-Lin,00:08:AEPacketFr,00:08:AFNovatec,00:08:B0HUBER+SU,00:08:B1ProQuent,00:08:B2Shenzhen,00:08:B3Fastwel,00:08:B4Syspol,00:08:B5TaiGuenE,00:08:B6RouteFre,00:08:B7HIT,00:08:B8EFJohnso,00:08:B9Kaonmedi,00:08:BAErskine,00:08:BBNetExcel,00:08:BCIlevo,00:08:BDTepg-Us,00:08:BEXENPAKMS,00:08:BFAptusEle,00:08:C0Asa,00:08:C1AvistarC,00:08:C2Cisco,00:08:C3Contex,00:08:C4Hikari,00:08:C5Liontech,00:08:C6PhilipsC,00:08:C7HewlettP,00:08:C8Sonetico,00:08:C9TechniSa,00:08:CATwinHanT,00:08:CBZetaBroa,00:08:CCRemotec,00:08:CDWith-Net,00:08:CEIPMobile,00:08:CFNipponKo,00:08:D0MusashiE,00:08:D1Karel,00:08:D2ZOOMNetw,00:08:D3Hercules,00:08:D4IneoQues,00:08:D5Vanguard,00:08:D6HASSNET,00:08:D7How,00:08:D8DowkeyMi,00:08:D9Mitadens,00:08:DASofaWare,00:08:DBCorrigen,00:08:DCWiznet,00:08:DDTelenaCo,00:08:DE3UP,00:08:DFAlistel,00:08:E0ATOTechn,00:08:E1Barix,00:08:E2Cisco,00:08:E3Cisco,00:08:E4Envenerg,00:08:E5IDK,00:08:E6Littlefe,00:08:E7SHIContr,00:08:E8ExcelMas,00:08:E9NextGig,00:08:EAMotionCo,00:08:EBROMWin,00:08:ECOpticalZ,00:08:EDST&TInst,00:08:EELogicPro,00:08:EFDibal,00:08:F0NextGene,00:08:F1Voltaire,00:08:F2C&STechn,00:08:F3Wany,00:08:F4Bluetake,00:08:F5YESTECHN,00:08:F6Sumitomo,00:08:F7HitachiS,00:08:F8UtcCcs,00:08:F9ArtesynE,00:08:FAKEBAutom,00:08:FBSonoSite,00:08:FCGigaphot,00:08:FDBlueKore,00:08:FEUNIKC&C,00:08:FFTrilogyC,00:09:00Tmt,00:09:01Shenzhen,00:09:02RedlineC,00:09:03Panasas,00:09:04MONDIALe,00:09:05iTECTech,00:09:06EsteemNe,00:09:07Chrysali,00:09:08VTechTec,00:09:09TelenorC,00:09:0ASnedFarT,00:09:0BMTLInstr,00:09:0CMayekawa,00:09:0DLeaderEl,00:09:0EHelixTec,00:09:0FFortinet,00:09:10SimpleAc,00:09:11Cisco,00:09:12Cisco,00:09:13SystemK,00:09:14Computro,00:09:15CAS,00:09:16ListmanH,00:09:17WEMTechn,00:09:18SamsungT,00:09:19MDSGatew,00:09:1AMacatOpt,00:09:1BDigitalG,00:09:1CCacheVis,00:09:1DProteamC,00:09:1EFirstech,00:09:1FA&D,00:09:20EpoXCOMP,00:09:21Planmeca,00:09:22TSTBiome,00:09:23HeamanSy,00:09:24Telebau,00:09:25VSNSyste,00:09:26YodaComm,00:09:27Toyokeik,00:09:28Telecore,00:09:29SanyoInd,00:09:2AMYTECS,00:09:2BiQstorNe,00:09:2CHitpoint,00:09:2DHTC,00:09:2EB&TechSy,00:09:2FAkomTech,00:09:30AeroConc,00:09:31FutureIn,00:09:32Omnilux,00:09:33Ophit,00:09:34Dream-Mu,00:09:35Sandvine,00:09:36Ipetroni,00:09:37Inventec,00:09:38AllotCom,00:09:39ShibaSok,00:09:3AMolexCMS,00:09:3BHyundaiN,00:09:3CJacquesT,00:09:3DNewisys,00:09:3EC&ITechn,00:09:3FDouble-W,00:09:40AGFEO,00:09:41AlliedTe,00:09:42Wireless,00:09:43Cisco,00:09:44Cisco,00:09:45Palmmicr,00:09:46ClusterL,00:09:47Aztek,00:09:48VistaCon,00:09:49GlyphTec,00:09:4AHomenetC,00:09:4BFillFact,00:09:4CCommunic,00:09:4DBraintre,00:09:4EBartechI,00:09:4Felmegt,00:09:50Independ,00:09:51ApogeeIm,00:09:52Auerswal,00:09:53LinkageS,00:09:54AMiTspol,00:09:55YoungGen,00:09:56NetworkG,00:09:57Supercal,00:09:58Intelnet,00:09:59Sitecsof,00:09:5ARacewood,00:09:5BNetgear,00:09:5CPhilipsM,00:09:5DDialogue,00:09:5EMasstech,00:09:5FTelebyte,00:09:60YOZAN,00:09:61Switchge,00:09:62SonitorT,00:09:63Dominion,00:09:64Hi-Techn,00:09:65HyunJuCo,00:09:66TrimbleE,00:09:67Tachyon,00:09:68Technove,00:09:69MeretOpt,00:09:6ACloverle,00:09:6BIBM,00:09:6CImediaSe,00:09:6DPowernet,00:09:6EGiantEle,00:09:6FBeijingZ,00:09:70Vibratio,00:09:71TimeMana,00:09:72Secureba,00:09:73LentenTe,00:09:74InnopiaT,00:09:75fSONACom,00:09:76Datasoft,00:09:77BrunnerE,00:09:78AIJISyst,00:09:79Advanced,00:09:7ALouisDes,00:09:7BCisco,00:09:7CCisco,00:09:7DSecWellN,00:09:7EImiTechn,00:09:7FVsecure2,00:09:80PowerZen,00:09:81NewportN,00:09:82LoeweOpt,00:09:83GlobalTo,00:09:84MyCasaNe,00:09:85AutoTele,00:09:86Metalink,00:09:87NishiNip,00:09:88NudianEl,00:09:89VividLog,00:09:8AEqualLog,00:09:8BEntropic,00:09:8COptionWi,00:09:8DVelocity,00:09:8Eipcas,00:09:8FCetacean,00:09:90ACKSYSCo,00:09:91Intellig,00:09:92InterEpo,00:09:93Visteon,00:09:94CronyxEn,00:09:95CastleTe,00:09:96Rdi,00:09:97NortelNe,00:09:98Capinfo,00:09:99CpGeorge,00:09:9AElmo,00:09:9BWesternT,00:09:9CNavalRes,00:09:9DHaliplex,00:09:9ETestech,00:09:9FVidex,00:09:A0Microtec,00:09:A1Telewise,00:09:A2Interfac,00:09:A3LeadflyT,00:09:A4HARTEC,00:09:A5HansungE,00:09:A6IgnisOpt,00:09:A7BangOluf,00:09:A8Eastmode,00:09:A9IkanosCo,00:09:AADataComm,00:09:ABNetcontr,00:09:ACLanvoice,00:09:ADHyundaiS,00:09:AEOkanoEle,00:09:AFe-generi,00:09:B0Onkyo,00:09:B1Kanemats,00:09:B2L&F,00:09:B3MCM,00:09:B4KisanTel,00:09:B53JTech,00:09:B6Cisco,00:09:B7Cisco,00:09:B8Entise,00:09:B9ActionIm,00:09:BAMAKUInfo,00:09:BBMathStar,00:09:BCUtility,00:09:BDEpygiTec,00:09:BEMamiya-O,00:09:BFNintendo,00:09:C06Wind,00:09:C1Proces-D,00:09:C2Onity,00:09:C3Netas,00:09:C4Medicore,00:09:C5KINGENET,00:09:C6Visionic,00:09:C7Movistec,00:09:C8Sinagawa,00:09:C9BlueWINC,00:09:CAiMaxNetw,00:09:CBHBrain,00:09:CCMoog,00:09:CDHudsonSo,00:09:CESpaceBri,00:09:CFiAd,00:09:D0SolacomT,00:09:D1SeranoaN,00:09:D2MaiLogic,00:09:D3WesternD,00:09:D4Transtec,00:09:D5SignalCo,00:09:D6KNCOne,00:09:D7DCSecuri,00:09:D8FältComm,00:09:D9Neoscale,00:09:DAControlM,00:09:DBeSpace,00:09:DCGalaxisT,00:09:DDMavinTec,00:09:DESamjinIn,00:09:DFVestelEl,00:09:E0Xemics,00:09:E1GemtekTe,00:09:E2SinbonEl,00:09:E3AngelIgl,00:09:E4KTechInf,00:09:E5Hottinge,00:09:E6CyberSwi,00:09:E7ADCTecho,00:09:E8Cisco,00:09:E9Cisco,00:09:EAYEM,00:09:EBHuMANDAT,00:09:ECDaktroni,00:09:EDCipherOp,00:09:EEMeikyoEl,00:09:EFVoceraCo,00:09:F0ShimizuT,00:09:F1YamakiEl,00:09:F2CohuElec,00:09:F3WELLComm,00:09:F4AlconLab,00:09:F5EmersonN,00:09:F6Shenzhen,00:09:F7SEDadivi,00:09:F8UnimoTec,00:09:F9ArtJapan,00:09:FBPhilipsP,00:09:FCIPFLEX,00:09:FDUbinetic,00:09:FEDaisyTec,00:09:FFXnet2000,00:0A:00Mediatek,00:0A:01SOHOware,00:0A:02Annso,00:0A:03EndesaSe,00:0A:043Com,00:0A:05Widax,00:0A:06Teledex,00:0A:07WebWayOn,00:0A:08AlpsAlpi,00:0A:09TaraComI,00:0A:0ASUNIX,00:0A:0BSealevel,00:0A:0CScientif,00:0A:0DAmphenol,00:0A:0EInvivoRe,00:0A:0FIlryungT,00:0A:10FASTmedi,00:0A:11ExPetTec,00:0A:12AzylexTe,00:0A:13Honeywel,00:0A:14TECO,00:0A:15SiliconD,00:0A:16LassenRe,00:0A:17NestarCo,00:0A:18Vichel,00:0A:19ValerePo,00:0A:1AImerge,00:0A:1BStreamLa,00:0A:1CBridgeIn,00:0A:1DOpticalC,00:0A:1ERed-MPro,00:0A:1FARTWARET,00:0A:20SVANetwo,00:0A:21IntegraT,00:0A:22Amperion,00:0A:23ParamaNe,00:0A:24OctaveCo,00:0A:25CeragonN,00:0A:26CEIA,00:0A:27Apple,00:0A:28Motorola,00:0A:29PanDacom,00:0A:2AQSI,00:0A:2BEtherstu,00:0A:2CActiveTc,00:0A:2DCabotCom,00:0A:2EMapleNet,00:0A:2FArtnix,00:0A:30Visteon,00:0A:31HCVConsu,00:0A:32Xsido,00:0A:33Emulex,00:0A:34Identica,00:0A:35Xilinx,00:0A:36SynelecT,00:0A:37ProceraN,00:0A:38ApaniNet,00:0A:39LoPAInfo,00:0A:3AJ-THREEI,00:0A:3BGCTSemic,00:0A:3CEnerpoin,00:0A:3DEloSiste,00:0A:3EEADSTele,00:0A:3FDataEast,00:0A:40CrownAud,00:0A:41Cisco,00:0A:42Cisco,00:0A:43Chunghwa,00:0A:44AveryDen,00:0A:45Audio-Te,00:0A:46AroWeldi,00:0A:47AlliedVi,00:0A:48Albatron,00:0A:49F5Networ,00:0A:4ATarga,00:0A:4BDataPowe,00:0A:4CMolecula,00:0A:4DNoritz,00:0A:4EUNITEKEl,00:0A:4FBrainBox,00:0A:50Remotek,00:0A:51GyroSign,00:0A:52AsiaRF,00:0A:53Intronic,00:0A:54LagunaHi,00:0A:55MARKEM,00:0A:56HITACHIM,00:0A:57HewlettP,00:0A:58FreyerSi,00:0A:59HWserver,00:0A:5AGreenNET,00:0A:5BPower-On,00:0A:5CCarel,00:0A:5DFingerTe,00:0A:5E3Com,00:0A:5Falmedio,00:0A:60Autostar,00:0A:61Cellinx,00:0A:62CrinisNe,00:0A:63DHD,00:0A:64EracomTe,00:0A:65GentechM,00:0A:66Mitsubis,00:0A:67OngCorp,00:0A:68Solarfla,00:0A:69SUNNYbel,00:0A:6ASVMMicro,00:0A:6BTadiranT,00:0A:6CWalchem,00:0A:6DEKSElekt,00:0A:6EHarmonic,00:0A:6FZyFLEXTe,00:0A:70MPLSForu,00:0A:71AvrioTec,00:0A:72Stec,00:0A:73Scientif,00:0A:74Manticom,00:0A:75Caterpil,00:0A:76BeidaJad,00:0A:77Bluewire,00:0A:78Olitec,00:0A:79corega,00:0A:7AKyoritsu,00:0A:7BCorneliu,00:0A:7CTecton,00:0A:7DValo,00:0A:7EAdvantag,00:0A:7FTeradonI,00:0A:80Telkonet,00:0A:81TEIMAAud,00:0A:82TatsutaS,00:0A:83SaltoSL,00:0A:84RainsunE,00:0A:85PLATC2,00:0A:86Lenze,00:0A:87Integrat,00:0A:88InCypher,00:0A:89Creval,00:0A:8ACisco,00:0A:8BCisco,00:0A:8CGuardwar,00:0A:8DEurother,00:0A:8EInvacom,00:0A:8FAskaInte,00:0A:90BaysideI,00:0A:91HemoCue,00:0A:92Presonus,00:0A:93W2Networ,00:0A:94ShangHai,00:0A:95Apple,00:0A:96MewtelTe,00:0A:97SONICblu,00:0A:98M+FGwinn,00:0A:99CalampWi,00:0A:9AAiptekIn,00:0A:9BTBGroup,00:0A:9CServerTe,00:0A:9DKingYoun,00:0A:9EBroadWeb,00:0A:9FPannaway,00:0A:A0CedarPoi,00:0A:A1VVS,00:0A:A2Systek,00:0A:A3Shimafuj,00:0A:A4Shanghai,00:0A:A5MaxlinkI,00:0A:A6Hochiki,00:0A:A7FEIElect,00:0A:A8ePipe,00:0A:A9BrooksAu,00:0A:AAAltiGenC,00:0A:ABToyotaTe,00:0A:ACTerraTec,00:0A:ADStargame,00:0A:AERosemoun,00:0A:AFPipal,00:0A:B0LOYTECel,00:0A:B1GENETEC,00:0A:B2FresnelW,00:0A:B3FaGIRA,00:0A:B4ETICTele,00:0A:B5DigitalE,00:0A:B6Compunet,00:0A:B7Cisco,00:0A:B8Cisco,00:0A:B9AsteraTe,00:0A:BAArconTec,00:0A:BBTaiwanSe,00:0A:BCSeabridg,00:0A:BDRupprech,00:0A:BEOPNETTec,00:0A:BFHirotaSs,00:0A:C0FuyohVid,00:0A:C1Futurete,00:0A:C2WuhanFib,00:0A:C3eMTechni,00:0A:C4DaewooTe,00:0A:C5ColorKin,00:0A:C6Overture,00:0A:C7Unicatio,00:0A:C8ZPSYSPla,00:0A:C9Zambeel,00:0A:CAYOKOYAMA,00:0A:CBXPAKMSAG,00:0A:CCWinnowNe,00:0A:CDSunrichT,00:0A:CERadiante,00:0A:CFPROVIDEO,00:0A:D0NiigataD,00:0A:D1Mws,00:0A:D2JEPICO,00:0A:D3INITECH,00:0A:D4CoreBell,00:0A:D5Brainchi,00:0A:D6BeamReac,00:0A:D7OriginEL,00:0A:D8IPCservT,00:0A:D9Sony,00:0A:DAVindicat,00:0A:DBTrillian,00:0A:DCRuggedCo,00:0A:DDAllworx,00:0A:DEHappyCom,00:0A:DFGennum,00:0A:E0FujitsuS,00:0A:E1EGTechno,00:0A:E2Binatone,00:0A:E3YangMeiT,00:0A:E4Wistron,00:0A:E5ScottCar,00:0A:E6Elitegro,00:0A:E7Eliop,00:0A:E8CathayRo,00:0A:E9AirVastT,00:0A:EAAdamElek,00:0A:EBTp-LinkT,00:0A:ECKoatsuGa,00:0A:EDHARTINGE,00:0A:EEGCDHard-,00:0A:EFOtrumAsa,00:0A:F0Shin-OhE,00:0A:F1ClarityD,00:0A:F2NeoAxiom,00:0A:F3Cisco,00:0A:F4Cisco,00:0A:F5AirgoNet,00:0A:F6EmersonC,00:0A:F7Broadcom,00:0A:F8American,00:0A:F9HiConnec,00:0A:FATraverse,00:0A:FBAmbri,00:0A:FCCoreTecC,00:0A:FDKentecEl,00:0A:FENovaPal,00:0A:FFKilchher,00:0B:00FujianSt,00:0B:01DaiichiE,00:0B:02Dallmeie,00:0B:03Taekwang,00:0B:04Volktek,00:0B:05PacificB,00:0B:06ARRISGro,00:0B:07VoxpathN,00:0B:08PillarDa,00:0B:09Ifoundry,00:0B:0AdBmOptic,00:0B:0BCorrent,00:0B:0CAgile,00:0B:0DAir2U,00:0B:0ETrapezeN,00:0B:0FBoschRex,00:0B:1011waveTe,00:0B:11HimejiAb,00:0B:12NURITele,00:0B:13Zetron,00:0B:14ViewSoni,00:0B:15Platypus,00:0B:16Communic,00:0B:17MKSInstr,00:0B:18Private,00:0B:19VernierN,00:0B:1AIndustri,00:0B:1BSystroni,00:0B:1CSIBCO,00:0B:1DLayerZer,00:0B:1EKAPPAopt,00:0B:1FICONComp,00:0B:20Hirata,00:0B:21G-StarCo,00:0B:22Environm,00:0B:23SiemensH,00:0B:24AirLogic,00:0B:25Aeluros,00:0B:26Wetek,00:0B:27Scion,00:0B:28Quatech,00:0B:29LSLGIndu,00:0B:2AHOWTEL,00:0B:2BHostnet,00:0B:2CEikiIndu,00:0B:2DDanfoss,00:0B:2ECal-Comp,00:0B:2Fbplan,00:0B:30BeijingG,00:0B:31YantaiZh,00:0B:32Vormetri,00:0B:33VivatoTe,00:0B:34ShangHai,00:0B:35QuadBitS,00:0B:36Producti,00:0B:37Manufact,00:0B:38Knürr,00:0B:39KeisokuG,00:0B:3APesa,00:0B:3Bdevolo,00:0B:3CCygnalIn,00:0B:3DCONTALOK,00:0B:3EBittWare,00:0B:3FAntholog,00:0B:40Cambridg,00:0B:41IngBüroD,00:0B:42commax,00:0B:43Microsca,00:0B:44ConcordI,00:0B:45Cisco,00:0B:46Cisco,00:0B:47Advanced,00:0B:48sofrel,00:0B:49RF-LinkS,00:0B:4AVisimetr,00:0B:4BVisiowav,00:0B:4CClarionM,00:0B:4DEmuzed,00:0B:4ECommunic,00:0B:4FVerifone,00:0B:50Oxygnet,00:0B:51MicetekI,00:0B:52JoymaxEl,00:0B:53INITIUM,00:0B:54BiTMICRO,00:0B:55ADInstru,00:0B:56Cybernet,00:0B:57SiliconL,00:0B:58Astronau,00:0B:59ScriptPr,00:0B:5AHyperEdg,00:0B:5BRinconRe,00:0B:5CNewtech,00:0B:5DFujitsu,00:0B:5EAudioEng,00:0B:5FCisco,00:0B:60Cisco,00:0B:61Friedric,00:0B:62ib-mohne,00:0B:63Kaleides,00:0B:64KiebackP,00:0B:65SyAC,00:0B:66Teralink,00:0B:67TopviewT,00:0B:68Addvalue,00:0B:69FrankeFi,00:0B:6AAsiarock,00:0B:6BWistronN,00:0B:6CSychip,00:0B:6DSolectro,00:0B:6ENeffInst,00:0B:6FMediaStr,00:0B:70LoadTech,00:0B:71Litchfie,00:0B:72Lawo,00:0B:73KodeosCo,00:0B:74Kingwave,00:0B:75Iosoft,00:0B:76ET&TTech,00:0B:77Cogent,00:0B:78Taifatec,00:0B:79X-COM,00:0B:7AL-3Linka,00:0B:7BTest-Um,00:0B:7CTelexCom,00:0B:7DSolomonE,00:0B:7ESAGINOMI,00:0B:7FAlignEng,00:0B:80LyciumNe,00:0B:81Kaparel,00:0B:82Grandstr,00:0B:83Datawatt,00:0B:84Bodet,00:0B:85Cisco,00:0B:86ArubaaHe,00:0B:87American,00:0B:88Vidisco,00:0B:89TopGloba,00:0B:8AMITEQ,00:0B:8BKerajet,00:0B:8CFlextron,00:0B:8DAvvioNet,00:0B:8EAscent,00:0B:8FAkitaEle,00:0B:90ADVAOpti,00:0B:91AglaiaGe,00:0B:92AscomDan,00:0B:93RitterEl,00:0B:94DigitalM,00:0B:95eBetGami,00:0B:96Innotrac,00:0B:97Matsushi,00:0B:98NiceTech,00:0B:99SensAble,00:0B:9AShanghai,00:0B:9BSiriusSy,00:0B:9CTriBeamT,00:0B:9DTwinMOST,00:0B:9EYasingTe,00:0B:9FNeueELSA,00:0B:A0T&LInfor,00:0B:A1Fujikura,00:0B:A2Sumitomo,00:0B:A3Siemens,00:0B:A4ShironSa,00:0B:A5QuasarCi,00:0B:A6Miyakawa,00:0B:A7MarantiN,00:0B:A8HanbackE,00:0B:A9CloudShi,00:0B:AAAiphone,00:0B:ABAdvantec,00:0B:AC3Com,00:0B:ADPC-PoS,00:0B:AEVitalsSy,00:0B:AFWOOJUCOM,00:0B:B0SysnetTe,00:0B:B1SuperSta,00:0B:B2Smallbig,00:0B:B3RiTtechn,00:0B:B4RDCSemic,00:0B:B5nStorTec,00:0B:B6Metallig,00:0B:B7Micro,00:0B:B8KihokuEl,00:0B:B9Imsys,00:0B:BAHarmonic,00:0B:BBEtin,00:0B:BCEnGarde,00:0B:BDConnexio,00:0B:BECisco,00:0B:BFCisco,00:0B:C0ChinaIWN,00:0B:C1BayMicro,00:0B:C2CorinexC,00:0B:C3Multiple,00:0B:C4BIOTRONI,00:0B:C5SMCNetwo,00:0B:C6ISAC,00:0B:C7ICET,00:0B:C8AirFlowN,00:0B:C9Electrol,00:0B:CADatavanT,00:0B:CBFagorAut,00:0B:CCJusan,00:0B:CDHewlettP,00:0B:CEFree2mov,00:0B:CFAgfaNdt,00:0B:D0XiMetaTe,00:0B:D1Aeronix,00:0B:D2RemoproT,00:0B:D3cd3o,00:0B:D4BeijingW,00:0B:D5Nvergenc,00:0B:D6PaxtonAc,00:0B:D7DORMATim,00:0B:D8Industri,00:0B:D9GeneralH,00:0B:DAEyeCross,00:0B:DBDell,00:0B:DCAkcp,00:0B:DDTOHOKURI,00:0B:DETELDIX,00:0B:DFShenzhen,00:0B:E0SercoNet,00:0B:E1NokiaNET,00:0B:E2Lumenera,00:0B:E3KeyStrea,00:0B:E4Hosiden,00:0B:E5HIMSInte,00:0B:E6DatelEle,00:0B:E7ComfluxT,00:0B:E8Aoip,00:0B:E9Actel,00:0B:EAZultysTe,00:0B:EBSystegra,00:0B:ECNipponEl,00:0B:EDELM,00:0B:EEjet,00:0B:EFCode,00:0B:F0MoTEXPro,00:0B:F1LAPLaser,00:0B:F2Chih-Kan,00:0B:F3Bae,00:0B:F4Private,00:0B:F5Shanghai,00:0B:F6Nitgen,00:0B:F7Nidek,00:0B:F8Infinera,00:0B:F9Gemstone,00:0B:FAExemys,00:0B:FBD-NETInt,00:0B:FCCisco,00:0B:FDCisco,00:0B:FECASTELBr,00:0B:FFBerkeley,00:0C:00BEBIndus,00:0C:01Abatron,00:0C:02ABB,00:0C:03HDMILice,00:0C:04Tecnova,00:0C:05RPAReser,00:0C:06NixvuePt,00:0C:07Iftest,00:0C:08HUMEXTec,00:0C:09HitachiI,00:0C:0AGuangdon,00:0C:0BBroadbus,00:0C:0CApproTec,00:0C:0DCommunic,00:0C:0EXtremeSp,00:0C:0FTechno-O,00:0C:10PNI,00:0C:11NipponDe,00:0C:12Micro-Op,00:0C:13MediaQ,00:0C:14Diagnost,00:0C:15CyberPow,00:0C:16Concorde,00:0C:17AJAVideo,00:0C:18ZenisuKe,00:0C:19TelioCom,00:0C:1AQuestTec,00:0C:1BORACOM,00:0C:1CMicroWeb,00:0C:1DMettlerF,00:0C:1EGlobalCa,00:0C:1FGlimmerg,00:0C:20FiWIn,00:0C:21FacultyS,00:0C:22DoubleDE,00:0C:23BeijingL,00:0C:24Anator,00:0C:25AlliedTe,00:0C:26WeintekL,00:0C:27Sammy,00:0C:28Rifatron,00:0C:29VMware,00:0C:2AOCTTELCo,00:0C:2BELIASTec,00:0C:2CEnwiser,00:0C:2DFullWave,00:0C:2EOpenetin,00:0C:2FSeorimTe,00:0C:30Cisco,00:0C:31Cisco,00:0C:32AvionicD,00:0C:33Compucas,00:0C:34Vixen,00:0C:35KaVoDent,00:0C:36SharpTak,00:0C:37Geomatio,00:0C:38TelcoBri,00:0C:39Sentinel,00:0C:3AOxance,00:0C:3BOrionEle,00:0C:3CMediaCho,00:0C:3DGlsystec,00:0C:3ECrestAud,00:0C:3FCogentDe,00:0C:40AltechCo,00:0C:41Cisco-Li,00:0C:42Routerbo,00:0C:43RalinkTe,00:0C:44Automate,00:0C:45Animatio,00:0C:46AlliedTe,00:0C:47SKTelete,00:0C:48QoStek,00:0C:49Dangaard,00:0C:4ACygnusMi,00:0C:4BCheopsEl,00:0C:4CArcorAG,00:0C:4DCurtiss-,00:0C:4EWinbestT,00:0C:4FUDTechJa,00:0C:50SeagateT,00:0C:51Scientif,00:0C:52Roll,00:0C:53Private,00:0C:54Pedestal,00:0C:55Microlin,00:0C:56MegatelC,00:0C:57MACKIEEn,00:0C:58M&S,00:0C:59IndymeEl,00:0C:5AIBSmmEmb,00:0C:5BHanwangT,00:0C:5CGTN,00:0C:5DChicTech,00:0C:5ECalypsoM,00:0C:5FAvtec,00:0C:60ACM,00:0C:61ACTechDB,00:0C:62ABBCewe-,00:0C:63ZenithEl,00:0C:64X2MSAGro,00:0C:65SuninTel,00:0C:66ProntoNe,00:0C:67OyoElect,00:0C:68SigmaTel,00:0C:69National,00:0C:6AMbari,00:0C:6BKurzIndu,00:0C:6CEve,00:0C:6DEdwards,00:0C:6EASUSTekC,00:0C:6FAmteksys,00:0C:70ACC,00:0C:71Wybron,00:0C:72Tempearl,00:0C:73TelsonEl,00:0C:74Rivertec,00:0C:75Oriental,00:0C:76Micro-St,00:0C:77LifeRaci,00:0C:78In-TechE,00:0C:79ExtelCom,00:0C:7ADaTARIUS,00:0C:7BALPHAPRO,00:0C:7CInternet,00:0C:7DTeikokuE,00:0C:7ETellium,00:0C:7Fsynertro,00:0C:80Opelcomm,00:0C:81Schneide,00:0C:82NetworkT,00:0C:83LogicalS,00:0C:84Eazix,00:0C:85Cisco,00:0C:86Cisco,00:0C:87Amd,00:0C:88ApacheMi,00:0C:89ACElectr,00:0C:8ABose,00:0C:8BConnectT,00:0C:8CKodicom,00:0C:8DMATRIXVI,00:0C:8EMentorEn,00:0C:8FNergal,00:0C:90Octasic,00:0C:91Riverhea,00:0C:92WolfVisi,00:0C:93Xeline,00:0C:94UnitedEl,00:0C:95PrimeNet,00:0C:96OQO,00:0C:97ADBTTVTe,00:0C:98LETEKCom,00:0C:99HITELLIN,00:0C:9AHitechEl,00:0C:9BEESoluti,00:0C:9CChonghoi,00:0C:9DUbeeAirW,00:0C:9EMemoryLi,00:0C:9FNKE,00:0C:A0StorCase,00:0C:A1SIGMACOM,00:0C:A2Harmonic,00:0C:A3RanchoTe,00:0C:A4Promptte,00:0C:A5NamanNZ,00:0C:A6Mintera,00:0C:A7MetroSuz,00:0C:A8GarudaNe,00:0C:A9Ebtron,00:0C:AACubicTra,00:0C:ABCommendI,00:0C:ACCitizenW,00:0C:ADBTUInter,00:0C:AEAilocom,00:0C:AFTriTerm,00:0C:B0StarSemi,00:0C:B1SallandE,00:0C:B2UNION,00:0C:B3ROUND,00:0C:B4AutoCell,00:0C:B5PremierT,00:0C:B6NanjingS,00:0C:B7NanjingH,00:0C:B8Medion,00:0C:B9Lea,00:0C:BAJamex,00:0C:BBIskraeme,00:0C:BCIscutum,00:0C:BDInterfac,00:0C:BEInnomina,00:0C:BFHolySton,00:0C:C0Genera,00:0C:C1Eaton,00:0C:C2ControlN,00:0C:C3BeWAN,00:0C:C4Tiptel,00:0C:C5Nextlink,00:0C:C6Ka-Roele,00:0C:C7Intellig,00:0C:C8Xytronix,00:0C:C9IlwooDat,00:0C:CAHGSTaWes,00:0C:CBDesignCo,00:0C:CCAeroscou,00:0C:CDIec-Tc57,00:0C:CECisco,00:0C:CFCisco,00:0C:D0Symetrix,00:0C:D1SFOMTech,00:0C:D2Schaffne,00:0C:D3PrettlEl,00:0C:D4Positron,00:0C:D5Passave,00:0C:D6PartnerT,00:0C:D7Nallatec,00:0C:D8MKJuchhe,00:0C:D9Itcare,00:0C:DAFreeHand,00:0C:DBBrocadeC,00:0C:DCBECSTech,00:0C:DDAOStechn,00:0C:DEABBSTOTZ,00:0C:DFJAIManuf,00:0C:E0TrekDiag,00:0C:E1OpenGrou,00:0C:E2Rolls-Ro,00:0C:E3OptionIn,00:0C:E4NeuroCom,00:0C:E5ARRISGro,00:0C:E6Fortinet,00:0C:E7MediaTek,00:0C:E8GuangZho,00:0C:E9Bloomber,00:0C:EAaphonaKo,00:0C:EBCNMPNetw,00:0C:ECOroliaUS,00:0C:EDRealDigi,00:0C:EEjp-embed,00:0C:EFOpenNetw,00:0C:F0MN,00:0C:F1Intel,00:0C:F2GAMESAEó,00:0C:F3CallImag,00:0C:F4Akatsuki,00:0C:F5InfoExpr,00:0C:F6SitecomE,00:0C:F7NortelNe,00:0C:F8NortelNe,00:0C:F9XylemWat,00:0C:FADigital,00:0C:FBKoreaNet,00:0C:FCS2ioTech,00:0C:FDHyundaiI,00:0C:FEGrandEle,00:0C:FFMRO-TEKR,00:0D:00SeawayNe,00:0D:01P&EMicro,00:0D:02NECPlatf,00:0D:03Matrics,00:0D:04FoxboroE,00:0D:05cybernet,00:0D:06Compulog,00:0D:07CalrecAu,00:0D:08AboveCab,00:0D:09YuehuaZh,00:0D:0ABarcoPro,00:0D:0BBuffalo,00:0D:0CMDISecur,00:0D:0DITSuppor,00:0D:0EInqnet,00:0D:0FFinlux,00:0D:10Embedtro,00:0D:11DENTSPLY,00:0D:12AXELL,00:0D:13WilhelmR,00:0D:14VtechInn,00:0D:15Voipacsr,00:0D:16UHS,00:0D:17TurboNet,00:0D:18Mega-Tre,00:0D:19ROBEShow,00:0D:1AMustekSy,00:0D:1BKyotoEle,00:0D:1CAmesysDe,00:0D:1DHigh-Tek,00:0D:1EControlT,00:0D:1FAVDigita,00:0D:20Asahikas,00:0D:21WISCORE,00:0D:22Unitroni,00:0D:23SmartSol,00:0D:24SentecE&,00:0D:25Sanden,00:0D:26Primagra,00:0D:27MICROPLE,00:0D:28Cisco,00:0D:29Cisco,00:0D:2AScanmati,00:0D:2BRacalIns,00:0D:2CNet2Edge,00:0D:2DNCTDeuts,00:0D:2EMatsushi,00:0D:2FAINCommT,00:0D:30IceFyreS,00:0D:31Compelle,00:0D:32Dispense,00:0D:33Prediwav,00:0D:34ShellInt,00:0D:35PACInter,00:0D:36WuHanRou,00:0D:37Wiplug,00:0D:38Nissin,00:0D:39NetworkE,00:0D:3AMicrosof,00:0D:3BMicroele,00:0D:3CiTechDyn,00:0D:3DHammerhe,00:0D:3EAPLUXCom,00:0D:3FVTIInstr,00:0D:40VerintLo,00:0D:41SiemensI,00:0D:42NewbestD,00:0D:43DRSTacti,00:0D:44AudioBU-,00:0D:45TottoriS,00:0D:46ParkerSS,00:0D:47Collex,00:0D:48AEWINTec,00:0D:49TritonDe,00:0D:4ASteagETA,00:0D:4BRoku,00:0D:4COutlineE,00:0D:4DNinelane,00:0D:4ENDR,00:0D:4FKenwood,00:0D:50GalazarN,00:0D:51DIVR,00:0D:52Comartsy,00:0D:53Beijing5,00:0D:543Com,00:0D:55SANYCOMT,00:0D:56Dell,00:0D:57FujitsuI,00:0D:58Private,00:0D:59Amity,00:0D:5ATiesse,00:0D:5BSmartEmp,00:0D:5CRobertBo,00:0D:5DRaritanC,00:0D:5ENECPerso,00:0D:5FMinds,00:0D:60IBM,00:0D:61Giga-Byt,00:0D:62Funkwerk,00:0D:63DENTInst,00:0D:64COMAGHan,00:0D:65Cisco,00:0D:66Cisco,00:0D:67Ericsson,00:0D:68Vinci,00:0D:69TMT&D,00:0D:6ARedwoodT,00:0D:6BMita-Tek,00:0D:6CM-Audio,00:0D:6DK-TechDe,00:0D:6EK-Patent,00:0D:6FEmber,00:0D:70Datamax,00:0D:71boca,00:0D:722Wire,00:0D:73Technica,00:0D:74SandNetw,00:0D:75KobianPt,00:0D:76HokutoDe,00:0D:77FalconSt,00:0D:78Engineer,00:0D:79DynamicS,00:0D:7ADiGATTOA,00:0D:7BConsensy,00:0D:7CCodian,00:0D:7DAfco,00:0D:7EAxiowave,00:0D:7FMIDASCOM,00:0D:80OnlineDe,00:0D:81Pepperl+,00:0D:82Phsnet,00:0D:83Sanmina-,00:0D:84Makus,00:0D:85Tapwave,00:0D:86Huber+Su,00:0D:87Elitegro,00:0D:88D-Link,00:0D:89BilsTech,00:0D:8AWinnersE,00:0D:8BT&D,00:0D:8CShanghai,00:0D:8DProsoftT,00:0D:8EKodenEle,00:0D:8FKingTsus,00:0D:90FactumEl,00:0D:91EclipseH,00:0D:92ARIMACom,00:0D:93Apple,00:0D:94AFARComm,00:0D:95Opti-cel,00:0D:96VteraTec,00:0D:97ABBTropo,00:0D:98SWACSchm,00:0D:99OrbitalS,00:0D:9AInfotec,00:0D:9BHeraeusE,00:0D:9CElan,00:0D:9DHewlettP,00:0D:9ETOKUDENO,00:0D:9FRFMicroD,00:0D:A0Nedap,00:0D:A1MIRAEITS,00:0D:A2InfrantT,00:0D:A3Emerging,00:0D:A4DoschAma,00:0D:A5Fabric7,00:0D:A6Universa,00:0D:A7Private,00:0D:A8Teletron,00:0D:A9TEAMSL,00:0D:AATehnolog,00:0D:ABParkerHa,00:0D:ACJapanCBM,00:0D:ADDataprob,00:0D:AESamsungH,00:0D:AFPlexusUK,00:0D:B0Olym-tec,00:0D:B1JapanNet,00:0D:B2Ammasso,00:0D:B3SDOCommu,00:0D:B4Stormshi,00:0D:B5Globalsa,00:0D:B6Broadcom,00:0D:B7SankoEle,00:0D:B8Schiller,00:0D:B9PCEngine,00:0D:BAOcéDocum,00:0D:BBNipponDe,00:0D:BCCisco,00:0D:BDCisco,00:0D:BEBelFuseE,00:0D:BFTekToneS,00:0D:C0SpagatAS,00:0D:C1SafeWeb,00:0D:C2Private,00:0D:C3FirstCom,00:0D:C4Emcore,00:0D:C5EchoStar,00:0D:C6DigiRose,00:0D:C7CosmicEn,00:0D:C8AirMagne,00:0D:C9THALESEl,00:0D:CATaitElec,00:0D:CBPetcomko,00:0D:CCNEOSMART,00:0D:CDGroupeTx,00:0D:CEDynavacT,00:0D:CFCidra,00:0D:D0TetraTec,00:0D:D1Stryker,00:0D:D2SimradOp,00:0D:D3SAMWOOTe,00:0D:D4Symantec,00:0D:D5ORiteTec,00:0D:D6Iti,00:0D:D7Bright,00:0D:D8Bbn,00:0D:D9AntonPaa,00:0D:DAAlliedTe,00:0D:DBAirwaveT,00:0D:DCVac,00:0D:DDProfiloT,00:0D:DEJoyteck,00:0D:DFJapanIma,00:0D:E0ICPDAS,00:0D:E1ControlP,00:0D:E2CMZSiste,00:0D:E3ATSweden,00:0D:E4DIGINICS,00:0D:E5SamsungT,00:0D:E6YoungboE,00:0D:E7Snap-onO,00:0D:E8NasacoEl,00:0D:E9Napatech,00:0D:EAKingtelT,00:0D:EBCompXs,00:0D:ECCisco,00:0D:EDCisco,00:0D:EEAndrewRF,00:0D:EFSocCoopB,00:0D:F0QcomTech,00:0D:F1Ionix,00:0D:F2Private,00:0D:F3AsmaxSol,00:0D:F4Watertek,00:0D:F5Teletron,00:0D:F6Technolo,00:0D:F7SpaceDyn,00:0D:F8ORGAKart,00:0D:F9NDS,00:0D:FAMicroCon,00:0D:FBKomax,00:0D:FCITFOR,00:0D:FDHugesHi-,00:0D:FEHauppaug,00:0D:FFChenming,00:0E:00Atrie,00:0E:01ASIPTech,00:0E:02Advantec,00:0E:03Emulex,00:0E:04CMAMicro,00:0E:05Wireless,00:0E:06TeamSimo,00:0E:07Sony,00:0E:08Cisco-Li,00:0E:09Shenzhen,00:0E:0ASakumaDe,00:0E:0BNetacTec,00:0E:0CIntel,00:0E:0DHeschSch,00:0E:0EESAelett,00:0E:0FErmme,00:0E:10C-guys,00:0E:11BDTBürou,00:0E:12Adaptive,00:0E:13Accu-Sor,00:0E:14Visionar,00:0E:15Tadlys,00:0E:16SouthWin,00:0E:17Private,00:0E:18MyATechn,00:0E:19LogicaCM,00:0E:1AJPSCommu,00:0E:1BIAV,00:0E:1CHach,00:0E:1DARIONTec,00:0E:1EQLogic,00:0E:1FTCLNetwo,00:0E:20ACCESSAm,00:0E:21MTUFried,00:0E:22Private,00:0E:23Incipien,00:0E:24HuwellTe,00:0E:25HannaeTe,00:0E:26GincomTe,00:0E:27CrereNet,00:0E:28DynamicR,00:0E:29ShesterC,00:0E:2APrivate,00:0E:2BSafariTe,00:0E:2CNetcodec,00:0E:2DHyundaiD,00:0E:2EEdimaxTe,00:0E:2FRocheDia,00:0E:30AERASNet,00:0E:31OlympusS,00:0E:32KontronM,00:0E:33ShukoEle,00:0E:34NexGenCi,00:0E:35Intel,00:0E:36HEINESYS,00:0E:37HarmsWen,00:0E:38Cisco,00:0E:39Cisco,00:0E:3ACirrusLo,00:0E:3BHawkingT,00:0E:3CTransact,00:0E:3DTelevic,00:0E:3ESunOptro,00:0E:3FSoronti,00:0E:40NortelNe,00:0E:41NihonMec,00:0E:42MoticInc,00:0E:43G-TekEle,00:0E:44Digital5,00:0E:45BeijingN,00:0E:46NiigataS,00:0E:47NCISyste,00:0E:48LipmanTr,00:0E:49ForswayS,00:0E:4AChangchu,00:0E:4Batriumca,00:0E:4CBermai,00:0E:4DNumesa,00:0E:4EWaveplus,00:0E:4FTrajet,00:0E:50ThomsonT,00:0E:51tecnaele,00:0E:52Optium,00:0E:53AvTech,00:0E:54AlphaCel,00:0E:55Auvitran,00:0E:564G,00:0E:57IworldNe,00:0E:58Sonos,00:0E:59Sagemcom,00:0E:5ATELEFIEL,00:0E:5BParkerVi,00:0E:5CARRISGro,00:0E:5DTriplePl,00:0E:5ERaisecom,00:0E:5Factiv-ne,00:0E:60360SUNDi,00:0E:61Microtro,00:0E:62NortelNe,00:0E:63LemkeDia,00:0E:64Elphel,00:0E:65TransCor,00:0E:66HitachiI,00:0E:67EltisMic,00:0E:68E-TOPNet,00:0E:69ChinaEle,00:0E:6A3Com,00:0E:6BJanitzae,00:0E:6CDeviceDr,00:0E:6DMurataMa,00:0E:6EMATMircr,00:0E:6FIRISBerh,00:0E:70in2Netwo,00:0E:71GemstarT,00:0E:72CTSelect,00:0E:73Tpack,00:0E:74SolarTel,00:0E:75NewYorkA,00:0E:76GemsocIn,00:0E:77Decru,00:0E:78Amtelco,00:0E:79AmpleCom,00:0E:7AGemWonCo,00:0E:7BToshiba,00:0E:7CTeleves,00:0E:7DElectron,00:0E:7EionSign,00:0E:7FHewlettP,00:0E:80ThomsonT,00:0E:81Devicesc,00:0E:82Infinity,00:0E:83Cisco,00:0E:84Cisco,00:0E:85Catalyst,00:0E:86AlcatelN,00:0E:87adpGause,00:0E:88Videotro,00:0E:89Clematic,00:0E:8AAvaraTec,00:0E:8BAstarteT,00:0E:8CSiemens,00:0E:8DinProgre,00:0E:8ESparkLAN,00:0E:8FSercomm,00:0E:90Ponico,00:0E:91NavicoAu,00:0E:92OpenTele,00:0E:93Milénio3,00:0E:94MaasInte,00:0E:95FujiyaDe,00:0E:96CubicDef,00:0E:97Ultracke,00:0E:98HMEClear,00:0E:99Spectrum,00:0E:9ABoeTechn,00:0E:9BAmbitMic,00:0E:9CBenchmar,00:0E:9DTiscaliU,00:0E:9ETopfield,00:0E:9FTEMICSDS,00:0E:A0NetKlass,00:0E:A1FormosaT,00:0E:A2McAfee,00:0E:A3CNCR-ITH,00:0E:A4Quantum,00:0E:A5BLIP,00:0E:A6ASUSTekC,00:0E:A7EndaceTe,00:0E:A8UnitedTe,00:0E:A9Shanghai,00:0E:AAScalent,00:0E:ABCray,00:0E:ACMintronE,00:0E:ADMetanoia,00:0E:AEGawellTe,00:0E:AFCastel,00:0E:B0Solution,00:0E:B1Newcotec,00:0E:B2Micro-Re,00:0E:B3HewlettP,00:0E:B4Guangzho,00:0E:B5EcastleE,00:0E:B6Riverbed,00:0E:B7Knovativ,00:0E:B8Iiga,00:0E:B9HASHIMOT,00:0E:BAHanmiSem,00:0E:BBEverbeeN,00:0E:BCParagonF,00:0E:BDBurdicka,00:0E:BEB&BElect,00:0E:BFRemsdaq,00:0E:C0NortelNe,00:0E:C1MYNAHTec,00:0E:C2Lowrance,00:0E:C3LogicCon,00:0E:C4IskraTra,00:0E:C5DigitalM,00:0E:C6AsixElec,00:0E:C7Motorola,00:0E:C8Zoran,00:0E:C9YOKOTech,00:0E:CAWTSS,00:0E:CBVineSysT,00:0E:CCTableau,00:0E:CDSkov,00:0E:CESITTI,00:0E:CFPROFIBUS,00:0E:D0Privaris,00:0E:D1OsakaMic,00:0E:D2Filtroni,00:0E:D3Epicente,00:0E:D4CresittI,00:0E:D5COPAN,00:0E:D6Cisco,00:0E:D7Cisco,00:0E:D8Positron,00:0E:D9Aksys,00:0E:DAC-TechUn,00:0E:DBXiNCOM,00:0E:DCTellion,00:0E:DDShure,00:0E:DEREMEC,00:0E:DFPLXTechn,00:0E:E0Mcharge,00:0E:E1ExtremeS,00:0E:E2CustomEn,00:0E:E3ChiyuTec,00:0E:E4BoeTechn,00:0E:E5bitWalle,00:0E:E6Adimos,00:0E:E7AacElect,00:0E:E8ZioncomE,00:0E:E9WayTechD,00:0E:EAShadongL,00:0E:EBSandmart,00:0E:ECOrban,00:0E:EDNokiaDan,00:0E:EEMucoIndu,00:0E:EFPrivate,00:0E:F0Festo,00:0E:F1Ezquest,00:0E:F2Infinico,00:0E:F3Smartlab,00:0E:F4KasdaNet,00:0E:F5iPACTech,00:0E:F6E-TENInf,00:0E:F7VulcanPo,00:0E:F8SbcAsi,00:0E:F9REAElekt,00:0E:FAOptowayT,00:0E:FBMaceyEnt,00:0E:FCJTAGTech,00:0E:FDFujinon,00:0E:FEEndRunTe,00:0E:FFMegasolu,00:0F:00Legra,00:0F:01Digitalk,00:0F:02Digicube,00:0F:03Com&C,00:0F:04cim-usa,00:0F:053BSystem,00:0F:06NortelNe,00:0F:07Mangrove,00:0F:08Indagon,00:0F:09Private,00:0F:0AClearEdg,00:0F:0BKentimaT,00:0F:0CSynchron,00:0F:0DHuntElec,00:0F:0EWaveSpli,00:0F:0FRealIDTe,00:0F:10RDM,00:0F:11Prodrive,00:0F:12Panasoni,00:0F:13Nisca,00:0F:14Mindray,00:0F:15Icotera,00:0F:16JayHowTe,00:0F:17InstaEle,00:0F:18Industri,00:0F:19BostonSc,00:0F:1AGamingSu,00:0F:1BEgo,00:0F:1CDigitAll,00:0F:1DCosmoTec,00:0F:1EChengduK,00:0F:1FDell,00:0F:20HewlettP,00:0F:21Scientif,00:0F:22Helius,00:0F:23Cisco,00:0F:24Cisco,00:0F:25AimValle,00:0F:26WorldAcc,00:0F:27TEALElec,00:0F:28Itronix,00:0F:29Augmenti,00:0F:2ACablewar,00:0F:2BGreenbel,00:0F:2CUplogix,00:0F:2DChung-Hs,00:0F:2EMegapowe,00:0F:2FW-LinxTe,00:0F:30RazaMicr,00:0F:31AlliedVi,00:0F:32LootomTe,00:0F:33DUALi,00:0F:34Cisco,00:0F:35Cisco,00:0F:36Accurate,00:0F:37Xambala,00:0F:38Netstar,00:0F:39IrisSens,00:0F:3AHisharp,00:0F:3BFujiSyst,00:0F:3CEndeleo,00:0F:3DD-Link,00:0F:3ECardioNe,00:0F:3FBigBearN,00:0F:40OpticalI,00:0F:41Zipher,00:0F:42Xalyo,00:0F:43Wasabi,00:0F:44Tivella,00:0F:45Stretch,00:0F:46Sinar,00:0F:47Robox,00:0F:48Polypix,00:0F:49Northove,00:0F:4AKyushu-k,00:0F:4BOracle,00:0F:4CElextech,00:0F:4DTalkSwit,00:0F:4ECellink,00:0F:4FPCSSyste,00:0F:50StreamSc,00:0F:51Azul,00:0F:52YORKRefr,00:0F:53Solarfla,00:0F:54Entrelog,00:0F:55Datawire,00:0F:56Continuu,00:0F:57CABLELOG,00:0F:58AdderTec,00:0F:59Phonak,00:0F:5APeribitN,00:0F:5BDeltaInf,00:0F:5CDayOneDi,00:0F:5DGenexis,00:0F:5EVeo,00:0F:5FNicetyTe,00:0F:60Lifetron,00:0F:61HewlettP,00:0F:62AlcatelB,00:0F:63ObzervTe,00:0F:64D&RElect,00:0F:65icube,00:0F:66Cisco-Li,00:0F:67WestInst,00:0F:68VavicNet,00:0F:69SEWEurod,00:0F:6ANortelNe,00:0F:6BGateWare,00:0F:6CADDI-DAT,00:0F:6DMidasEng,00:0F:6EBBox,00:0F:6FFTACommu,00:0F:70WintecIn,00:0F:71SanmeiEl,00:0F:72Sandburs,00:0F:73RSAutoma,00:0F:74QamcomTe,00:0F:75FirstSil,00:0F:76DigitalK,00:0F:77Dentum,00:0F:78Datacap,00:0F:79Bluetoot,00:0F:7ABeiJingN,00:0F:7BArceSist,00:0F:7CACTi,00:0F:7DXirrus,00:0F:7EAblerexE,00:0F:7FUBSTORAG,00:0F:80TrinityS,00:0F:81PALPacif,00:0F:82MortaraI,00:0F:83Brainium,00:0F:84AstuteNe,00:0F:85ADDO-Jap,00:0F:86BlackBer,00:0F:87MaxcessI,00:0F:88AMETEK,00:0F:89Winnerte,00:0F:8AWideView,00:0F:8BOrionMul,00:0F:8CGigawave,00:0F:8DFASTTV-S,00:0F:8EDongyang,00:0F:8FCisco,00:0F:90Cisco,00:0F:91Aerotele,00:0F:92Microhar,00:0F:93Landis+G,00:0F:94Genexis,00:0F:95ELECOMLa,00:0F:96Telco,00:0F:97Avanex,00:0F:98Avamax,00:0F:99APACopto,00:0F:9ASynchron,00:0F:9BRossVide,00:0F:9CPanduit,00:0F:9DDisplayL,00:0F:9EMurrelek,00:0F:9FARRISGro,00:0F:A0CanonKor,00:0F:A1Gigabit,00:0F:A22xWirele,00:0F:A3AlphaNet,00:0F:A4Sprecher,00:0F:A5BWATechn,00:0F:A6S2Securi,00:0F:A7RaptorNe,00:0F:A8Photomet,00:0F:A9PCFabrik,00:0F:AANexusTec,00:0F:ABKyushuEl,00:0F:ACIeee8021,00:0F:ADFMNcommu,00:0F:AEE2OCommu,00:0F:AFDialog,00:0F:B0CompalEl,00:0F:B1Cognio,00:0F:B2Broadban,00:0F:B3Actionte,00:0F:B4Timespac,00:0F:B5Netgear,00:0F:B6Europlex,00:0F:B7Cavium,00:0F:B8CallURL,00:0F:B9Adaptive,00:0F:BATevebox,00:0F:BBNokiaSie,00:0F:BCOnkeyTec,00:0F:BDMRVCommu,00:0F:BEe-wyou,00:0F:BFDGTSp,00:0F:C0DELCOMp,00:0F:C1WAVE,00:0F:C2Uniwell,00:0F:C3PalmPalm,00:0F:C4NST,00:0F:C5KeyMed,00:0F:C6EurocomI,00:0F:C7DionicaR,00:0F:C8ChantryN,00:0F:C9Allnet,00:0F:CAA-JinTec,00:0F:CB3Com,00:0F:CCARRISGro,00:0F:CDNortelNe,00:0F:CEKikusuiE,00:0F:CFDataWind,00:0F:D0Astri,00:0F:D1AppliedW,00:0F:D2EWATechn,00:0F:D3Digium,00:0F:D4Soundcra,00:0F:D5Schwecha,00:0F:D6Sarotech,00:0F:D7HarmanMu,00:0F:D8Force,00:0F:D9FlexDSLT,00:0F:DAYazaki,00:0F:DBWestellT,00:0F:DCUedaJapa,00:0F:DDSordin,00:0F:DESony,00:0F:DFSOLOMONT,00:0F:E0NComputi,00:0F:E1IdDigita,00:0F:E2Hangzhou,00:0F:E3DammCell,00:0F:E4Pantech,00:0F:E5MercuryS,00:0F:E6MBTech,00:0F:E7LutronEl,00:0F:E8Lobos,00:0F:E9GwTechno,00:0F:EAGiga-Byt,00:0F:EBCylonCon,00:0F:ECARKUS,00:0F:EDAnamElec,00:0F:EEXTec,00:0F:EFThalese-,00:0F:F0Sunray,00:0F:F1nex-GPte,00:0F:F2LoudTech,00:0F:F3JungMyou,00:0F:F4Gunterma,00:0F:F5GN&S,00:0F:F6DarfonLi,00:0F:F7Cisco,00:0F:F8Cisco,00:0F:F9Valcrete,00:0F:FAOptinel,00:0F:FBNipponDe,00:0F:FCMeritLi-,00:0F:FDGlorytek,00:0F:FEG-ProCom,00:0F:FFControl4,00:10:00CableTel,00:10:01Citel,00:10:02Actia,00:10:03Imatron,00:10:04Brantley,00:10:05UecComme,00:10:06ThalesCo,00:10:07CiscoSys,00:10:08Vienna,00:10:09Horanet,00:10:0AWilliams,00:10:0BCisco,00:10:0CIto,00:10:0DCiscoSys,00:10:0EMicroLin,00:10:0FIndustri,00:10:10Initio,00:10:11CiscoSys,00:10:12Processo,00:10:13KontronA,00:10:14Cisco,00:10:15OOmon,00:10:16TSqware,00:10:17BoschAcc,00:10:18Broadcom,00:10:19SIRONADE,00:10:1APictureT,00:10:1BCornetTe,00:10:1COhmTechn,00:10:1DWinbondE,00:10:1EMatsushi,00:10:1FCiscoSys,00:10:20HandHeld,00:10:21EncantoN,00:10:22SatComMe,00:10:23NetworkE,00:10:24NagoyaEl,00:10:25Grayhill,00:10:26Accelera,00:10:27L-3Commu,00:10:28Computer,00:10:29CiscoSys,00:10:2AZfMicros,00:10:2BUmaxData,00:10:2CLasatNet,00:10:2DHitachiS,00:10:2ENetworkT,00:10:2FCiscoSys,00:10:30EION,00:10:31Objectiv,00:10:32AltaTech,00:10:33Accessla,00:10:34GNPCompu,00:10:35Elitegro,00:10:36Inter-Te,00:10:37CYQveTec,00:10:38MicroRes,00:10:39Vectron,00:10:3ADiamondN,00:10:3BHippiNet,00:10:3CIcEnsemb,00:10:3DPhasecom,00:10:3ENetschoo,00:10:3FTollgrad,00:10:40Intermec,00:10:41BristolB,00:10:42Alacrite,00:10:43A2,00:10:44InnoLabs,00:10:45NortelNe,00:10:46AlcornMc,00:10:47EchoElet,00:10:48HtrcAuto,00:10:49ShoreTel,00:10:4AParvus,00:10:4B3com3c90,00:10:4CTeledyne,00:10:4DSurtecIn,00:10:4ECeologic,00:10:4FOracle,00:10:50Rion,00:10:51Cmicro,00:10:52Mettler-,00:10:53Computer,00:10:54Cisco,00:10:55FujitsuM,00:10:56Sodick,00:10:57Rebelcom,00:10:58ArrowPoi,00:10:59DiabloRe,00:10:5A3comFast,00:10:5BNetInsig,00:10:5CQuantumD,00:10:5DDraegerM,00:10:5ESpirentS,00:10:5FZodiacDa,00:10:60Billingt,00:10:61Hostlink,00:10:62NxServer,00:10:63Starguid,00:10:64Dnpg,00:10:65Radyne,00:10:66Advanced,00:10:67Ericsson,00:10:68ComosTel,00:10:69HeliossC,00:10:6ADigitalM,00:10:6BSonusNet,00:10:6CEDNT,00:10:6DAxxceler,00:10:6ETadiranC,00:10:6FTrentonT,00:10:70CaradonT,00:10:71Advanet,00:10:72GvnTechn,00:10:73Technobo,00:10:74AtenInte,00:10:75SegateTe,00:10:76EUREM,00:10:77SafDrive,00:10:78NueraCom,00:10:79Cisco550,00:10:7AAmbicomW,00:10:7BCisco,00:10:7CP-Com,00:10:7DAuroraCo,00:10:7EBACHMANN,00:10:7FCrestron,00:10:80Metawave,00:10:81Dps,00:10:82JnaTelec,00:10:83Hp-UxE90,00:10:84K-BotCom,00:10:85PolarisC,00:10:86ATTOTech,00:10:87Xstreami,00:10:88American,00:10:89WebSonic,00:10:8ATeraLogi,00:10:8BLaserani,00:10:8CFujitsuS,00:10:8DJohnsonC,00:10:8EHUGHSYMO,00:10:8FRaptor,00:10:90Cimetric,00:10:91NoWiresN,00:10:92Netcore,00:10:93CmsCompu,00:10:94Performa,00:10:95Thomson,00:10:96Tracewel,00:10:97WinNetMe,00:10:98StarnetT,00:10:99InnoMedi,00:10:9ANetline,00:10:9BEmulex,00:10:9CM-System,00:10:9DClarinet,00:10:9EAware,00:10:9FPavo,00:10:A0InnovexT,00:10:A1KendinSe,00:10:A2Tns,00:10:A3Omnitron,00:10:A4XircomRe,00:10:A5OxfordIn,00:10:A6Cisco,00:10:A7UnexTech,00:10:A8Reliance,00:10:A9AdhocTec,00:10:AAMedia4,00:10:ABKoitoEle,00:10:ACImciTech,00:10:ADSoftroni,00:10:AEShinkoEl,00:10:AFTac,00:10:B0Meridian,00:10:B1For-A,00:10:B2Coactive,00:10:B3NokiaMul,00:10:B4Atmosphe,00:10:B5AcctonTe,00:10:B6EntrataC,00:10:B7CoyoteTe,00:10:B8Ishigaki,00:10:B9Maxtor,00:10:BAMartinho,00:10:BBDataInfo,00:10:BCAastraTe,00:10:BDTelecomm,00:10:BEMarchNet,00:10:BFInterAir,00:10:C0ARMA,00:10:C1OiElectr,00:10:C2Willnet,00:10:C3Csi-Cont,00:10:C4MediaGlo,00:10:C5Protocol,00:10:C6Universa,00:10:C7DataTran,00:10:C8Communic,00:10:C9Mitsubis,00:10:CATelco,00:10:CBFacit,00:10:CCCLPCOMPU,00:10:CDInterfac,00:10:CEVolamp,00:10:CFFiberlan,00:10:D0Witcom,00:10:D1TopLayer,00:10:D2NittoTsu,00:10:D3GripsEle,00:10:D4StorageC,00:10:D5ImasdeCa,00:10:D6Exelis,00:10:D7ArgosyEn,00:10:D8Calista,00:10:D9IbmJapan,00:10:DAKollmorg,00:10:DBJuniperN,00:10:DCMicro-St,00:10:DDEnableSe,00:10:DEInternat,00:10:DFRiseComp,00:10:E0Oracle,00:10:E1SITech,00:10:E2ArrayCom,00:10:E3HewlettP,00:10:E4Nsi,00:10:E5Solectro,00:10:E6AppliedI,00:10:E7Breezeco,00:10:E8Telocity,00:10:E9Raidtec,00:10:EAAdeptTec,00:10:EBSelsius,00:10:ECRpcg,00:10:EDSundance,00:10:EECtiProdu,00:10:EFDbtel,00:10:F0RITTAL-W,00:10:F1I-O,00:10:F2Antec,00:10:F3NexcomIn,00:10:F4Vertical,00:10:F5Amherst,00:10:F6Cisco,00:10:F7IRIICHIT,00:10:F8TexioTec,00:10:F9Unique,00:10:FAApple,00:10:FBZidaTech,00:10:FCBroadban,00:10:FDCocom,00:10:FEDigitalE,00:10:FFCisco,00:11:00Schneide,00:11:01CETTechn,00:11:02AuroraMu,00:11:03kawamura,00:11:04Telexy,00:11:05SunplusT,00:11:06SiemensB,00:11:07RGBNetwo,00:11:08OrbitalD,00:11:09Micro-St,00:11:0AHewlettP,00:11:0BFranklin,00:11:0CAtmarkTe,00:11:0DSANBlaze,00:11:0ETsurusak,00:11:0Fnetplat,00:11:10MaxannaT,00:11:11Intel,00:11:12Honeywel,00:11:13Fraunhof,00:11:14EverFocu,00:11:15EPINTech,00:11:16CoteauVe,00:11:17Cesnet,00:11:18BLXICDes,00:11:19Solteras,00:11:1AARRISGro,00:11:1BTargaDiv,00:11:1CPleoraTe,00:11:1DHectrix,00:11:1EETHERNET,00:11:1FDoremiLa,00:11:20Cisco,00:11:21Cisco,00:11:22CIMSYS,00:11:23Appointe,00:11:24Apple,00:11:25IBM,00:11:26Venstar,00:11:27TASI,00:11:28Streamit,00:11:29Paradise,00:11:2ANiko,00:11:2BNetModul,00:11:2CIZT,00:11:2DiPulse,00:11:2ECeicom,00:11:2FASUSTekC,00:11:30AlliedTe,00:11:31Unatech,00:11:32Synology,00:11:33SiemensA,00:11:34MediaCel,00:11:35Grandeye,00:11:36Goodrich,00:11:37AichiEle,00:11:38Taishin,00:11:39STOEBERA,00:11:3AShinbora,00:11:3BMicronet,00:11:3CMicronas,00:11:3DKnSoltec,00:11:3EJL,00:11:3FAlcatelD,00:11:40Nanometr,00:11:41GoodMan,00:11:42e-SMARTC,00:11:43Dell,00:11:44Assuranc,00:11:45ValuePoi,00:11:46Telecard,00:11:47Secom-In,00:11:48ProlonCo,00:11:49Proliphi,00:11:4AKAYABAIN,00:11:4BFrancoty,00:11:4Ccaffeina,00:11:4DAtsumiEl,00:11:4E690885On,00:11:4FUSDigita,00:11:50Belkin,00:11:51Mykotron,00:11:52Eidsvoll,00:11:53TridentT,00:11:54WebproTe,00:11:55Sevis,00:11:56PharosNZ,00:11:57OkiElect,00:11:58NortelNe,00:11:59MatisseN,00:11:5AIvoclarV,00:11:5BElitegro,00:11:5CCisco,00:11:5DCisco,00:11:5EProMinen,00:11:5FITXSecur,00:11:60ARTDIO,00:11:61NetStrea,00:11:62StarMicr,00:11:63SystemDe,00:11:64ACARDTec,00:11:65ZNYXNetw,00:11:66TaelimEl,00:11:67Integrat,00:11:68HomeLogi,00:11:69EMSSatco,00:11:6ADomo,00:11:6BDigitalD,00:11:6CNanwangM,00:11:6DAmerican,00:11:6EPeplinkI,00:11:6FNetforyo,00:11:70Gsc,00:11:71DEXTERCo,00:11:72Cotron,00:11:73SMARTSto,00:11:74MojoNetw,00:11:75Intel,00:11:76Intellam,00:11:77CoaxialN,00:11:78ChironTe,00:11:79Singular,00:11:7ASingimIn,00:11:7BBüchiLab,00:11:7Ce-zynet,00:11:7DZMDAmeri,00:11:7EMidmark,00:11:7FNeotuneI,00:11:80ARRISGro,00:11:81InterEne,00:11:82IMINorgr,00:11:83Datalogi,00:11:84HumoLabo,00:11:85HewlettP,00:11:86Prime,00:11:87Category,00:11:88Enterasy,00:11:89Aerotech,00:11:8AViewtran,00:11:8BAlcatel-,00:11:8CMissouri,00:11:8DHanchang,00:11:8EHalytech,00:11:8FEutechIn,00:11:90DigitalD,00:11:91CTS-Clim,00:11:92Cisco,00:11:93Cisco,00:11:94ChiMeiCo,00:11:95D-Link,00:11:96Actualit,00:11:97Monitori,00:11:98PrismMed,00:11:992wcom,00:11:9AAlkeria,00:11:9BTelesyne,00:11:9CEP&TEner,00:11:9DDiginfoT,00:11:9ESolectro,00:11:9FNokiaDan,00:11:A0VtechEng,00:11:A1VisionNe,00:11:A2Manufact,00:11:A3LanReady,00:11:A4JStreamT,00:11:A5FortunaE,00:11:A6SypixxNe,00:11:A7InfilcoD,00:11:A8QuestTec,00:11:A9MOIMSTON,00:11:AAUniclass,00:11:ABTrustabl,00:11:ACSimtecEl,00:11:ADShanghai,00:11:AEARRISGro,00:11:AFMedialin,00:11:B0Fortelin,00:11:B1BlueExpe,00:11:B22001Tech,00:11:B3Yoshimiy,00:11:B4Westermo,00:11:B5Shenzhen,00:11:B6OpenInte,00:11:B7Octalix,00:11:B8Liebherr,00:11:B9InnerRan,00:11:BAElexol,00:11:BBCisco,00:11:BCCisco,00:11:BDBombardi,00:11:BEAGPTelec,00:11:BFAESYS,00:11:C0AdayTech,00:11:C14PMobile,00:11:C2UnitedFi,00:11:C3Transcei,00:11:C4Terminal,00:11:C5TENTechn,00:11:C6SeagateT,00:11:C7Raymarin,00:11:C8Powercom,00:11:C9MTT,00:11:CALongRang,00:11:CBJacobson,00:11:CCGuangzho,00:11:CDAxsunTec,00:11:CEUbisense,00:11:CFThraneTh,00:11:D0Tandberg,00:11:D1SoftImag,00:11:D2Percepti,00:11:D3NextGenT,00:11:D4NetEnric,00:11:D5Hangzhou,00:11:D6HandEra,00:11:D7eWerks,00:11:D8ASUSTekC,00:11:D9TiVo,00:11:DAVivaasTe,00:11:DBLand-Cel,00:11:DCGlunzJen,00:11:DDFROMUSTE,00:11:DEEurilogi,00:11:DFCurrentE,00:11:E0U-MEDIAC,00:11:E1Arcelik,00:11:E2HuaJungC,00:11:E3Thomson,00:11:E4DanelecE,00:11:E5KCodes,00:11:E6Scientif,00:11:E7WORLDSAT,00:11:E8TixiCom,00:11:E9Starnex,00:11:EAIWICS,00:11:EBInnovati,00:11:ECAvix,00:11:ED802Globa,00:11:EEEstari,00:11:EFConitecD,00:11:F0Wideful,00:11:F1QinetiQ,00:11:F2Institut,00:11:F3NeoMedia,00:11:F4woori-ne,00:11:F5AskeyCom,00:11:F6AsiaPaci,00:11:F7Shenzhen,00:11:F8AIRAYA,00:11:F9NortelNe,00:11:FARane,00:11:FBHeidelbe,00:11:FCHARTINGE,00:11:FDKorg,00:11:FEKeiyoSys,00:11:FFDigitroT,00:12:00Cisco,00:12:01Cisco,00:12:02DecraneA,00:12:03ActivNet,00:12:04u10Netwo,00:12:05Terrasat,00:12:06iQuestNZ,00:12:07HeadStro,00:12:08GantnerI,00:12:09Fastrax,00:12:0AEmersonC,00:12:0BChinasys,00:12:0CCE-Infos,00:12:0DAdvanced,00:12:0EAboCom,00:12:0FIeee8023,00:12:10WideRay,00:12:11Protechn,00:12:12PLUS,00:12:13Metrohm,00:12:14KoenigBa,00:12:15iStorNet,00:12:16ICPInter,00:12:17Cisco-Li,00:12:18ARUZE,00:12:19GeneralD,00:12:1ATechnoSo,00:12:1BSoundDev,00:12:1CParrot,00:12:1DNetfabri,00:12:1EJuniperN,00:12:1FHardingI,00:12:20Cadco,00:12:21BBraunMe,00:12:22SkardinU,00:12:23Pixim,00:12:24NexQL,00:12:25ARRISGro,00:12:26JapanDir,00:12:27Franklin,00:12:28Data,00:12:29BroadEas,00:12:2AVTechTel,00:12:2BVirbiage,00:12:2CSoenenCo,00:12:2DSiNett,00:12:2ESignalTe,00:12:2FSaneiEle,00:12:30PicasoIn,00:12:31MotionCo,00:12:32LeWizCom,00:12:33JRCTOKKI,00:12:34CamilleB,00:12:35Andrew,00:12:36ConSentr,00:12:37TexasIns,00:12:38SetaBoxT,00:12:39SNet,00:12:3APosystec,00:12:3BKeRoApS,00:12:3CSecondRu,00:12:3DGES,00:12:3EERUNEtec,00:12:3FDell,00:12:40AmoiElec,00:12:41a2imarke,00:12:42Millenni,00:12:43Cisco,00:12:44Cisco,00:12:45Zellwege,00:12:46TOMTechn,00:12:47SamsungE,00:12:48DellEMC,00:12:49DeltaEle,00:12:4ADedicate,00:12:4BTexasIns,00:12:4CBBWM,00:12:4DInducon,00:12:4EXacAutom,00:12:4FnVent,00:12:50TokyoAir,00:12:51Silink,00:12:52Citronix,00:12:53AudioDev,00:12:54SpectraT,00:12:55NetEffec,00:12:56LgInform,00:12:57LeapComm,00:12:58TechVoIP,00:12:59ThermoEl,00:12:5AMicrosof,00:12:5BKaimeiEl,00:12:5CGreenHil,00:12:5DCyberNet,00:12:5ECaen,00:12:5FAWIND,00:12:60StantonM,00:12:61Adaptix,00:12:62NokiaDan,00:12:63DataVoic,00:12:64daumelec,00:12:65Enerdyne,00:12:66Swisscom,00:12:67Panasoni,00:12:68IPSdoo,00:12:69ValueEle,00:12:6AOPTOELEC,00:12:6BAscalade,00:12:6CVisonicT,00:12:6DUniversi,00:12:6ESeidelEl,00:12:6FRaysonTe,00:12:70NGESDenr,00:12:71Measurem,00:12:72ReduxCom,00:12:73Stoke,00:12:74NITlab,00:12:75Sentilla,00:12:76CGPowerI,00:12:77KorenixT,00:12:78Internat,00:12:79HewlettP,00:12:7ASanyuInd,00:12:7BVIANetwo,00:12:7CSwegon,00:12:7DMobileAr,00:12:7EDigitalL,00:12:7FCisco,00:12:80Cisco,00:12:81MarchNet,00:12:82Qovia,00:12:83NortelNe,00:12:84Lab33,00:12:85Gizmondo,00:12:86Endevco,00:12:87DigitalE,00:12:882Wire,00:12:89AdvanceS,00:12:8AARRISGro,00:12:8BSensoryN,00:12:8CWoodward,00:12:8DSTBDaten,00:12:8EQ-FreeAS,00:12:8FMontilio,00:12:90KYOWAEle,00:12:91KWSCompu,00:12:92GriffinT,00:12:93ABBPower,00:12:94Sumitomo,00:12:95Aiware,00:12:96Addlogix,00:12:97O2Micro,00:12:98MicoElec,00:12:99KtechTel,00:12:9AIRTElect,00:12:9BE2SElect,00:12:9CYulinet,00:12:9DFirstInt,00:12:9ESurfComm,00:12:9FRAE,00:12:A0NeoMerid,00:12:A1BluePack,00:12:A2Vita,00:12:A3TrustInt,00:12:A4ThingMag,00:12:A5DolphinI,00:12:A6DolbyAus,00:12:A7ISRTECHN,00:12:A8intec,00:12:A93Com,00:12:AAIEE,00:12:ABWiLife,00:12:ACOntimete,00:12:ADVivavis,00:12:AEHLSHARD-,00:12:AFELPROTec,00:12:B0Efore,00:12:B1DaiNippo,00:12:B2Avolites,00:12:B3AdvanceW,00:12:B4WorkMicr,00:12:B5Vialta,00:12:B6SantaBar,00:12:B7PTWFreib,00:12:B8G2Micros,00:12:B9FusionDi,00:12:BAFSI,00:12:BBTelecomm,00:12:BCEcholab,00:12:BDAvantecM,00:12:BEAstek,00:12:BFArcadyan,00:12:C0HotLava,00:12:C1CheckPoi,00:12:C2ApexElec,00:12:C3Wit,00:12:C4Viseon,00:12:C5V-ShowTe,00:12:C6TGCAmeri,00:12:C7SECURAYT,00:12:C8Perfectt,00:12:C9ARRISGro,00:12:CAMechatro,00:12:CBCSS,00:12:CCBitatek,00:12:CDASEM,00:12:CEAdvanced,00:12:CFAcctonTe,00:12:D0Gossen-M,00:12:D1TexasIns,00:12:D2TexasIns,00:12:D3Zetta,00:12:D4Princeto,00:12:D5MotionRe,00:12:D6JiangsuY,00:12:D7InventoN,00:12:D8Internat,00:12:D9Cisco,00:12:DACisco,00:12:DBZIEHLind,00:12:DCSunCorpI,00:12:DDShengquI,00:12:DERadioCom,00:12:DFNovomati,00:12:E0Codan,00:12:E1AlliantN,00:12:E2ALAXALAN,00:12:E3AgatSoft,00:12:E4ZIEHLind,00:12:E5TimeAmer,00:12:E6SpectecC,00:12:E7Projecte,00:12:E8Fraunhof,00:12:E9Abbey,00:12:EATrane,00:12:EBPDHSolut,00:12:ECMovacolo,00:12:EDAVGAdvan,00:12:EESony,00:12:EFOneAcces,00:12:F0IntelCor,00:12:F1Ifotec,00:12:F2BrocadeC,00:12:F3connectB,00:12:F4BelcoInt,00:12:F5ImardaNe,00:12:F6Mdk,00:12:F7XiamenXi,00:12:F8WNIResou,00:12:F9UryuSeis,00:12:FAThx,00:12:FBSamsungE,00:12:FCPLANETSy,00:12:FDOptimusI,00:12:FELenovoMo,00:12:FFLelyIndu,00:13:00It-Facto,00:13:01IronGate,00:13:02IntelCor,00:13:03GateConn,00:13:04Flaircom,00:13:05Epicom,00:13:06AlwaysOn,00:13:07Paravirt,00:13:08NuveraFu,00:13:09OceanBro,00:13:0ANortelNe,00:13:0BMextal,00:13:0CHFSystem,00:13:0DGalileoA,00:13:0EFocusrit,00:13:0FEGEMENBi,00:13:10Cisco-Li,00:13:11ARRISGro,00:13:12AmediaNe,00:13:13GuangZho,00:13:14Asiamajo,00:13:15SonyInte,00:13:16L-S-BBro,00:13:17GNNetcom,00:13:18DGSTATIO,00:13:19Cisco,00:13:1ACisco,00:13:1BBeCellIn,00:13:1CLiteTouc,00:13:1DScanvaeg,00:13:1Epeikerac,00:13:1FNxtPhase,00:13:20IntelCor,00:13:21HewlettP,00:13:22DAQElect,00:13:23Cap,00:13:24Schneide,00:13:25Cortina,00:13:26ECM,00:13:27DataAcqu,00:13:28WestechK,00:13:29VSST,00:13:2ASitronic,00:13:2BPhoenixD,00:13:2CMAZBrand,00:13:2DiWiseCom,00:13:2EITianCop,00:13:2FInteract,00:13:30EuroProt,00:13:31CellPoin,00:13:32BeijingT,00:13:33BaudTec,00:13:34Arkados,00:13:35VSIndust,00:13:36Tianjin7,00:13:37OrientPo,00:13:38Freseniu,00:13:39CCVDeuts,00:13:3AVadaTech,00:13:3BSpeedDra,00:13:3CQuintron,00:13:3DMicroMem,00:13:3EMetaSwit,00:13:3FEppendor,00:13:40ADEL,00:13:41Shandong,00:13:42VisionRe,00:13:43Matsushi,00:13:44FargoEle,00:13:45Eaton,00:13:46D-Link,00:13:47RedLionC,00:13:48ArtilaEl,00:13:49ZyxelCom,00:13:4AEngim,00:13:4BToGolden,00:13:4CYDTTechn,00:13:4DInepro,00:13:4EValox,00:13:4FRapidusW,00:13:50SilverSp,00:13:51NilesAud,00:13:52Naztec,00:13:53HYDACFil,00:13:54ZcomaxTe,00:13:55TOMENCyb,00:13:56FLIRRadi,00:13:57SoyalTec,00:13:58Realm,00:13:59ProTelev,00:13:5AProjectT,00:13:5BPanelLin,00:13:5COnSite,00:13:5DNTTPCCom,00:13:5EEabRwiK,00:13:5FCisco,00:13:60Cisco,00:13:61Biospace,00:13:62ShinHeun,00:13:63Verascap,00:13:64Paradigm,00:13:65NortelNe,00:13:66Neturity,00:13:67Narayon,00:13:68SaabDanm,00:13:69HondaEle,00:13:6AHachLang,00:13:6BE-Tec,00:13:6CTomTom,00:13:6DTentacul,00:13:6ETechmetr,00:13:6FPacketMo,00:13:70NokiaDan,00:13:71ARRISGro,00:13:72Dell,00:13:73BLwaveEl,00:13:74AtherosC,00:13:75American,00:13:76TaborEle,00:13:77SamsungE,00:13:78QsanTech,00:13:79PonderIn,00:13:7ANetvoxTe,00:13:7BMovon,00:13:7CKaicom,00:13:7DDynalab,00:13:7ECorEdgeN,00:13:7FCisco,00:13:80Cisco,00:13:81CHIPS,00:13:82CetaceaN,00:13:83Applicat,00:13:84Advanced,00:13:85Add-OnTe,00:13:86ABBTotal,00:13:8727MTechn,00:13:88WiMediaA,00:13:89RedesdeT,00:13:8AQingdaoG,00:13:8BPhantomT,00:13:8CKumyoung,00:13:8DKinghold,00:13:8EFOABElek,00:13:8FAsiarock,00:13:90TermtekC,00:13:91Ouen,00:13:92RuckusWi,00:13:93Panta,00:13:94Infohand,00:13:95congatec,00:13:96AcbelPol,00:13:97Oracle,00:13:98TrafficS,00:13:99STAC,00:13:9AK-ubique,00:13:9BioIMAGE,00:13:9CExaveraT,00:13:9DMaxLinea,00:13:9ECiaraTec,00:13:9FElectron,00:13:A0ALGOSYST,00:13:A1CrowElec,00:13:A2MaxStrea,00:13:A3SiemensH,00:13:A4KeyEyeCo,00:13:A5GeneralS,00:13:A6Extricom,00:13:A7Battelle,00:13:A8TanisysT,00:13:A9Sony,00:13:AAALSTEC,00:13:ABTelemoti,00:13:ACSunmyung,00:13:ADSendo,00:13:AERadiance,00:13:AFNUMATech,00:13:B0Jablotro,00:13:B1Intellig,00:13:B2Carallon,00:13:B3EcomComm,00:13:B4AppearTV,00:13:B5Wavesat,00:13:B6SlingMed,00:13:B7Scantech,00:13:B8RyCoElec,00:13:B9Bm,00:13:BAReadyLin,00:13:BBSmartvue,00:13:BCArtimi,00:13:BDHymatom,00:13:BEVirtualC,00:13:BFMediaSys,00:13:C0TrixTecn,00:13:C1AsokaUSA,00:13:C2WACOM,00:13:C3Cisco,00:13:C4Cisco,00:13:C5Lightron,00:13:C6OpenGear,00:13:C7IONOS,00:13:C8ADBBroad,00:13:C9BeyondAc,00:13:CAAtx,00:13:CBZenitelN,00:13:CCTallMapl,00:13:CDMTI,00:13:CEIntelCor,00:13:CF4AccessC,00:13:D0t+Medica,00:13:D1KIRKtele,00:13:D2PageIber,00:13:D3Micro-St,00:13:D4ASUSTekC,00:13:D5RuggedCo,00:13:D6TiiNetwo,00:13:D7SPIDCOMT,00:13:D8Princeto,00:13:D9MatrixPr,00:13:DADiskware,00:13:DBSHOEIEle,00:13:DCIbtek,00:13:DDAbbottDi,00:13:DEAdapt4,00:13:DFRyvor,00:13:E0MurataMa,00:13:E1Iprobe,00:13:E2GeoVisio,00:13:E3CoViTech,00:13:E4Yangjae,00:13:E5Tenosys,00:13:E6Technolu,00:13:E7Halcro,00:13:E8IntelCor,00:13:E9VeriWave,00:13:EAKamstrup,00:13:EBSysmaste,00:13:ECNetsnapp,00:13:EDPsia,00:13:EEJBXDesig,00:13:EFKingjonD,00:13:F0Wavefron,00:13:F1AMODTech,00:13:F2Klas,00:13:F3Giga-byt,00:13:F4Psitek,00:13:F5Akimbi,00:13:F6Cintech,00:13:F7SMCNetwo,00:13:F8DexSecur,00:13:F9Cavera,00:13:FALifeSize,00:13:FBRkcInstr,00:13:FCSiCortex,00:13:FDNokiaDan,00:13:FEGrandtec,00:13:FFDage-MTI,00:14:00MinervaK,00:14:01Rivertre,00:14:02kk-elect,00:14:03Renasis,00:14:04ARRISGro,00:14:05OpenIB,00:14:06GoNetwor,00:14:07SperianP,00:14:08Eka,00:14:09MAGNETIM,00:14:0AWEPIO,00:14:0BFirstInt,00:14:0CGkbCctv,00:14:0DNortelNe,00:14:0ENortelNe,00:14:0FFederalS,00:14:10SuzhouKe,00:14:11Deutschm,00:14:12S-TECele,00:14:13TrebingH,00:14:14Jumpnode,00:14:15IntecAut,00:14:16ScoscheI,00:14:17RSEInfor,00:14:18C4Line,00:14:19Sidsa,00:14:1ADeicy,00:14:1BCisco,00:14:1CCisco,00:14:1DKEBAIndu,00:14:1EPASemi,00:14:1FSunKwang,00:14:20G-Linksn,00:14:21TotalWir,00:14:22Dell,00:14:23J-SNEURO,00:14:24MerryEle,00:14:25Galactic,00:14:26NLTechno,00:14:27JazzMuta,00:14:28Vocollec,00:14:29VCenterT,00:14:2AElitegro,00:14:2BEdataCom,00:14:2CKonceptI,00:14:2DToradex,00:14:2E77Elektr,00:14:2FSavvius,00:14:30ViPowER,00:14:31PDLElect,00:14:32Tarallax,00:14:33EmpowerT,00:14:34Keri,00:14:35CityCom,00:14:36QwertyEl,00:14:37GSTelete,00:14:38HewlettP,00:14:39BlonderT,00:14:3ARaytalkI,00:14:3BSensovat,00:14:3CRheinmet,00:14:3DAevoe,00:14:3EAirLinkC,00:14:3FHotwayTe,00:14:40ATOMIC,00:14:41Innovati,00:14:42Atto,00:14:43Consultr,00:14:44Grundfos,00:14:45Telefon-,00:14:46SuperVis,00:14:47BOAZ,00:14:48Inventec,00:14:49SichuanC,00:14:4ATaiwanTh,00:14:4BHifn,00:14:4CGeneralM,00:14:4DIntellig,00:14:4ESrisa,00:14:4FOracle,00:14:50Heim,00:14:51Apple,00:14:52Calculex,00:14:53Advantec,00:14:54Symwave,00:14:55CoderEle,00:14:56EdgeProd,00:14:57T-VipsAs,00:14:58HSAutoma,00:14:59Moram,00:14:5AWestermo,00:14:5BSeekerNe,00:14:5CIntronic,00:14:5DWJCommun,00:14:5EIBM,00:14:5FAditec,00:14:60KyoceraW,00:14:61Corona,00:14:62Digiwell,00:14:63Idcs,00:14:64Cryptoso,00:14:65NovoNord,00:14:66Kleinhen,00:14:67ArrowSpa,00:14:68CelPlanI,00:14:69Cisco,00:14:6ACisco,00:14:6BAnagran,00:14:6CNetgear,00:14:6DRFTechno,00:14:6EHStoll,00:14:6FKohler,00:14:70ProkomSo,00:14:71EasternA,00:14:72ChinaBro,00:14:73Bookham,00:14:74K40Elect,00:14:75WilineNe,00:14:76MultiCom,00:14:77Trillian,00:14:78Tp-LinkT,00:14:79NECMagnu,00:14:7AEubus,00:14:7BIteris,00:14:7C3Com,00:14:7DAeonDigi,00:14:7EInnerWir,00:14:7FThomsonT,00:14:80Hitachi-,00:14:81Multilin,00:14:82AuroraNe,00:14:83eXS,00:14:84CermateT,00:14:85Giga-Byt,00:14:86EchoDigi,00:14:87American,00:14:88Akorri,00:14:89B1540210,00:14:8AElinEbgT,00:14:8BGloboEle,00:14:8CGeneralD,00:14:8DCubicDef,00:14:8ETelePowe,00:14:8FProtroni,00:14:90ASP,00:14:91DanielsE,00:14:92LiteonMo,00:14:93Systimax,00:14:94Esu,00:14:952Wire,00:14:96Phonic,00:14:97ZHIYUANE,00:14:98VikingDe,00:14:99Helicomm,00:14:9AARRISGro,00:14:9BNokotaCo,00:14:9CHF,00:14:9DSoundID,00:14:9EUbONE,00:14:9FSysteman,00:14:A0Accsense,00:14:A1Synchron,00:14:A2CoreMicr,00:14:A3Vitelec,00:14:A4HonHaiPr,00:14:A5GemtekTe,00:14:A6Teraneti,00:14:A7NokiaDan,00:14:A8Cisco,00:14:A9Cisco,00:14:AAAshlyAud,00:14:ABSenhaiEl,00:14:ACBountifu,00:14:ADGassnerW,00:14:AEWizlogic,00:14:AFDatasymP,00:14:B0NaeilCom,00:14:B1AxellWir,00:14:B2mCubelog,00:14:B3CoreStar,00:14:B4GeneralD,00:14:B5Physiome,00:14:B6EnswerTe,00:14:B7ARInfote,00:14:B8Hill-Rom,00:14:B9MstarSem,00:14:BACarvers,00:14:BBOpenInte,00:14:BCSynectic,00:14:BDincNETWO,00:14:BEWinkcomm,00:14:BFCisco-Li,00:14:C0Symstrea,00:14:C1USRoboti,00:14:C2HewlettP,00:14:C3SeagateT,00:14:C4Vitelcom,00:14:C5AliveTec,00:14:C6Quixant,00:14:C7NortelNe,00:14:C8Contempo,00:14:C9BrocadeC,00:14:CAKeyRadio,00:14:CBLifeSync,00:14:CCZetec,00:14:CDDigitalZ,00:14:CENf,00:14:CFINVISIOC,00:14:D0BTI,00:14:D1TRENDnet,00:14:D2KyudenTe,00:14:D3Sepsa,00:14:D4KTechnol,00:14:D5DatangTe,00:14:D6Jeongmin,00:14:D7Datastor,00:14:D8bio-logi,00:14:D9IPFabric,00:14:DAHuntleig,00:14:DBElmaTren,00:14:DCCommunic,00:14:DDCovergen,00:14:DESageInst,00:14:DFHI-PTech,00:14:E0LETS,00:14:E1DataDisp,00:14:E2datacom,00:14:E3mm-lab,00:14:E4infinias,00:14:E5Alticast,00:14:E6AIMInfra,00:14:E7Stolinx,00:14:E8ARRISGro,00:14:E9NortechI,00:14:EASDigmSaf,00:14:EBAwarePoi,00:14:ECAcroTele,00:14:EDAirak,00:14:EEWesternD,00:14:EFTZeroTec,00:14:F0Business,00:14:F1Cisco,00:14:F2Cisco,00:14:F3ViXS,00:14:F4DekTecDi,00:14:F5OSISecur,00:14:F6JuniperN,00:14:F7CREVIS,00:14:F8Scientif,00:14:F9VantageC,00:14:FAAsGa,00:14:FBTechnica,00:14:FCExtandon,00:14:FDThecusTe,00:14:FEArtechEl,00:14:FFPreciseA,00:15:00IntelCor,00:15:01LexBox,00:15:02BETAtech,00:15:03PROFIcom,00:15:04GamePlus,00:15:05Actionte,00:15:06NeoPhoto,00:15:07Renaissa,00:15:08GlobalTa,00:15:09PlusTech,00:15:0ASonoa,00:15:0BSageInfo,00:15:0CAVM,00:15:0DHoanaMed,00:15:0EOpenbrai,00:15:0Fmingjong,00:15:10Techsphe,00:15:11DataCent,00:15:12ZurichUn,00:15:13EFSsas,00:15:14HuZhouNA,00:15:15Leipold,00:15:16Uriel,00:15:17IntelCor,00:15:18Shenzhen,00:15:19StoreAge,00:15:1AHunterEn,00:15:1BIsilon,00:15:1CLeneco,00:15:1DM2I,00:15:1EETHERNET,00:15:1FMultivis,00:15:20Radiocra,00:15:21Horoquar,00:15:22DeaSecur,00:15:23MeteorCo,00:15:24Numatics,00:15:25Chamberl,00:15:26RemoteTe,00:15:27BalboaIn,00:15:28BeaconMe,00:15:29N3,00:15:2ANokia,00:15:2BCisco,00:15:2CCisco,00:15:2DTenXNetw,00:15:2EPacketHo,00:15:2FARRISGro,00:15:30DellEMC,00:15:31Kocom,00:15:32Consumer,00:15:33Nadam,00:15:34ABeltrón,00:15:35OTE,00:15:36Powertec,00:15:37VentusNe,00:15:38RFID,00:15:39Technodr,00:15:3AShenzhen,00:15:3BEMHmeter,00:15:3CKprotech,00:15:3DElimProd,00:15:3EQ-MaticS,00:15:3FAlcatelA,00:15:40NortelNe,00:15:41StrataLi,00:15:42Microhar,00:15:43Aberdeen,00:15:44coMt,00:15:45SEECODE,00:15:46ITGWorld,00:15:47AiZenSol,00:15:48CubeTech,00:15:49DixtalBi,00:15:4AWanshihE,00:15:4BWondePro,00:15:4CSaunders,00:15:4DNetronom,00:15:4EIec,00:15:4FoneRFTec,00:15:50NitsTech,00:15:51RadioPul,00:15:52Wi-Gear,00:15:53Cytyc,00:15:54AtalumWi,00:15:55DFM,00:15:56Sagemcom,00:15:57Olivetti,00:15:58Foxconn,00:15:59Securapl,00:15:5ADainippo,00:15:5BSampo,00:15:5CDresserW,00:15:5DMicrosof,00:15:5EMorganSt,00:15:5FGreenPea,00:15:60HewlettP,00:15:61JJPlus,00:15:62Cisco,00:15:63Cisco,00:15:64BEHRINGE,00:15:65XiamenYe,00:15:66A-FirstT,00:15:67RADWIN,00:15:68Dilithiu,00:15:69PECOII,00:15:6ADG2LTech,00:15:6BPerfisan,00:15:6CSaneSyst,00:15:6DUbiquiti,00:15:6EAWCommun,00:15:6FXiranetC,00:15:70ZebraTec,00:15:71Nolan,00:15:72Red-Lemo,00:15:73NewSoftT,00:15:74HorizonS,00:15:75NevisNet,00:15:76LABiTec-,00:15:77AlliedTe,00:15:78AudioVid,00:15:79Lunatone,00:15:7ATelefin,00:15:7BLeuzeele,00:15:7CDaveNetw,00:15:7DPosdata,00:15:7EWeidmüll,00:15:7FChuanGIn,00:15:80U-Way,00:15:81MAKUS,00:15:82PulseEig,00:15:83IVT,00:15:84SchenckP,00:15:85Aonvisio,00:15:86XiamenOv,00:15:87Takenaka,00:15:88Salutica,00:15:89D-MAXTec,00:15:8ASURECOMT,00:15:8BParkAir,00:15:8CLiabApS,00:15:8DJennic,00:15:8EPlustek,00:15:8FNTTAdvan,00:15:90Hectroni,00:15:91RLW,00:15:92FacomUKM,00:15:93U4EATech,00:15:94Bixolon,00:15:95QuesterT,00:15:96ARRISGro,00:15:97AetaAudi,00:15:98Kolektor,00:15:99SamsungE,00:15:9AARRISGro,00:15:9BNortelNe,00:15:9CB-KYUNGS,00:15:9DTrippLit,00:15:9EMadCatzI,00:15:9FTerascal,00:15:A0NokiaDan,00:15:A1Eca-Sint,00:15:A2ARRISGro,00:15:A3ARRISGro,00:15:A4ARRISGro,00:15:A5DCI,00:15:A6DigitalE,00:15:A7Robatech,00:15:A8ARRISGro,00:15:A9KwangWoo,00:15:AARextechn,00:15:ABProSound,00:15:ACCapelon,00:15:ADAccedian,00:15:AEkyungil,00:15:AFAzureWav,00:15:B0Autotele,00:15:B1Ambient,00:15:B2Advanced,00:15:B3Caretech,00:15:B4PolymapW,00:15:B5CINetwor,00:15:B6ShinMayw,00:15:B7Toshiba,00:15:B8Tahoe,00:15:B9SamsungE,00:15:BAiba,00:15:BBSMASolar,00:15:BCDevelco,00:15:BDGroup4Te,00:15:BEIqua,00:15:BFtechnico,00:15:C0DigitalT,00:15:C1SonyInte,00:15:C23MGerman,00:15:C3RufTelem,00:15:C4Flovel,00:15:C5Dell,00:15:C6Cisco,00:15:C7Cisco,00:15:C8FlexiPan,00:15:C9Gumstix,00:15:CATeraReco,00:15:CBSurfComm,00:15:CCUquest,00:15:CDExartech,00:15:CEARRISGro,00:15:CFARRISGro,00:15:D0ARRISGro,00:15:D1ARRISGro,00:15:D2Xantech,00:15:D3Pantech&,00:15:D4Emitor,00:15:D5Nicevt,00:15:D6OSLiNKSp,00:15:D7Reti,00:15:D8Interlin,00:15:D9PKCElect,00:15:DAIritelAD,00:15:DBCanesta,00:15:DCKT&C,00:15:DDIPContro,00:15:DENokiaDan,00:15:DFClivet,00:15:E0Ericsson,00:15:E1Picochip,00:15:E2DrIngHer,00:15:E3DreamTec,00:15:E4ZimmerEl,00:15:E5Cheertek,00:15:E6MOBILETE,00:15:E7QuantecT,00:15:E8NortelNe,00:15:E9D-Link,00:15:EATellumat,00:15:EBzte,00:15:ECBocaDevi,00:15:EDFulcrumM,00:15:EEOmnexCon,00:15:EFNECTOKIN,00:15:F0Ego,00:15:F1KYLINKCo,00:15:F2ASUSTekC,00:15:F3Peltor,00:15:F4Eventide,00:15:F5Sustaina,00:15:F6ScienceA,00:15:F7Wintecro,00:15:F8Kingtron,00:15:F9Cisco,00:15:FACisco,00:15:FBsetexsch,00:15:FCLittelfu,00:15:FDComplete,00:15:FESchillin,00:15:FFNovatelW,00:16:00CelleBri,00:16:01Buffalo,00:16:02CeyonTec,00:16:03COOLKSKY,00:16:04Sigpro,00:16:05Yorkvill,00:16:06IdealInd,00:16:07CurvesIn,00:16:08SequansC,00:16:09Uniteche,00:16:0ASWEEXEur,00:16:0BTVWorks,00:16:0CLplDevel,00:16:0DBeHere,00:16:0EOpticaTe,00:16:0FBadgerMe,00:16:10CarinaTe,00:16:11Altecon,00:16:12OtsukaEl,00:16:13LibreStr,00:16:14Picoseco,00:16:15Nittan,00:16:16BrowanCo,00:16:17Msi,00:16:18HIVION,00:16:19Lancelan,00:16:1ADametric,00:16:1BMicronet,00:16:1Cecue,00:16:1DInnovati,00:16:1EWoojinne,00:16:1FSUNWAVET,00:16:20Sony,00:16:21Colorado,00:16:22Bbh,00:16:23Interval,00:16:24Teneros,00:16:25Impinj,00:16:26ARRISGro,00:16:27embedded,00:16:28Magicard,00:16:29Nivus,00:16:2AAntikcom,00:16:2BTogamiEl,00:16:2CXanboo,00:16:2DSTNet,00:16:2ESpaceShu,00:16:2FGeutebrü,00:16:30VativTec,00:16:31Xteam,00:16:32SamsungE,00:16:33OxfordDi,00:16:34Mathtech,00:16:35HewlettP,00:16:36QuantaCo,00:16:37CITEL,00:16:38TECOM,00:16:39Ubiquam,00:16:3AYvesTech,00:16:3BCommunic,00:16:3CRebox,00:16:3DTsinghua,00:16:3EXensourc,00:16:3FCReTE,00:16:40Asmobile,00:16:41Universa,00:16:42Pangolin,00:16:43Sunhillo,00:16:44LITE-ONT,00:16:45PowerDis,00:16:46Cisco,00:16:47Cisco,00:16:48SSD,00:16:49SetOne,00:16:4AVibratio,00:16:4BQuorionD,00:16:4CPLANETIN,00:16:4DAlcatel-,00:16:4ENokiaDan,00:16:4FWorldEth,00:16:50KratosEP,00:16:51Exeo,00:16:52HoatechT,00:16:53LEGOSyst,00:16:54Flex-PIn,00:16:55FUHOTECH,00:16:56Nintendo,00:16:57Aegate,00:16:58Fusionte,00:16:59ZMPRadwa,00:16:5AHarmanSp,00:16:5BGripAudi,00:16:5CTrackflo,00:16:5DAirDefen,00:16:5EPrecisio,00:16:5FFairmoun,00:16:60NortelNe,00:16:61Novatium,00:16:62LiyuhTec,00:16:63KBTMobil,00:16:64Prod-El,00:16:65CellonFr,00:16:66Quantier,00:16:67A-TECSub,00:16:68EishinEl,00:16:69MRVCommu,00:16:6ATps,00:16:6BSamsungE,00:16:6CSamsungE,00:16:6DYulongCo,00:16:6EArbitron,00:16:6FIntelCor,00:16:70SKNET,00:16:71SymphoxI,00:16:72Zenwayen,00:16:73Bury,00:16:74EuroCBPh,00:16:75ARRISGro,00:16:76IntelCor,00:16:77Bihl+Wie,00:16:78Shenzhen,00:16:79eOnCommu,00:16:7ASkyworth,00:16:7BHaver&Bo,00:16:7CiRexTech,00:16:7DSky-Line,00:16:7EDiboss,00:16:7FBluebird,00:16:80BallyGam,00:16:81VectorIn,00:16:82ProDex,00:16:83WEBIOInt,00:16:84Donjin,00:16:85Elisa,00:16:86KarlStor,00:16:87ChubbCSC,00:16:88ServerEn,00:16:89PilkorEl,00:16:8Aid-Confi,00:16:8BParalan,00:16:8CDSLPartn,00:16:8DKORWIN,00:16:8EVimicro,00:16:8FGNNetcom,00:16:90J-TekInc,00:16:91Moser-Ba,00:16:92Scientif,00:16:93PowerLin,00:16:94Sennheis,00:16:95AVCTechn,00:16:96QDITechn,00:16:97NEC,00:16:98T&AMobil,00:16:99TonicDVB,00:16:9AQuadrics,00:16:9BAlstomTr,00:16:9CCisco,00:16:9DCisco,00:16:9ETVOne,00:16:9FVimtronE,00:16:A0Auto-Mas,00:16:A13LeafNet,00:16:A2CentraLi,00:16:A3Ingeteam,00:16:A4Ezurio,00:16:A5Tandberg,00:16:A6DovadoFZ,00:16:A7AwetaG&P,00:16:A8Cwt,00:16:A92Ei,00:16:AAKeiCommu,00:16:ABDansenso,00:16:ACTohoTech,00:16:ADBT-Links,00:16:AEInventel,00:16:AFShenzhen,00:16:B0VK,00:16:B1Kbs,00:16:B2DriveCam,00:16:B3Photonic,00:16:B4Private,00:16:B5ARRISGro,00:16:B6Cisco-Li,00:16:B7SeoulCom,00:16:B8Sony,00:16:B9ProCurve,00:16:BAWeathern,00:16:BBLaw-Chai,00:16:BCNokiaDan,00:16:BDATIIndus,00:16:BEINFRANET,00:16:BFPaloDExG,00:16:C0Semtech,00:16:C1Eleksen,00:16:C2Avtec,00:16:C3BA,00:16:C4SiRFTech,00:16:C5Shenzhen,00:16:C6NorthAtl,00:16:C7Cisco,00:16:C8Cisco,00:16:C9NATSeatt,00:16:CANortelNe,00:16:CBApple,00:16:CCXcuteMob,00:16:CDHijiHigh,00:16:CEHonHaiPr,00:16:CFHonHaiPr,00:16:D0ATechele,00:16:D1ZAT,00:16:D2Caspian,00:16:D3Wistron,00:16:D4CompalCo,00:16:D5Synccom,00:16:D6TDATech,00:16:D7Sunways,00:16:D8Senea,00:16:D9NingboBi,00:16:DAFutronic,00:16:DBSamsungE,00:16:DCArchos,00:16:DDGigabeam,00:16:DEFAST,00:16:DFLundinov,00:16:E03Com,00:16:E1SiliconS,00:16:E2American,00:16:E3AskeyCom,00:16:E4Vanguard,00:16:E5FordleyD,00:16:E6Giga-Byt,00:16:E7DynamixP,00:16:E8Lumissil,00:16:E9TibaMedi,00:16:EAIntelCor,00:16:EBIntelCor,00:16:ECElitegro,00:16:EDUtility,00:16:EERoyaldig,00:16:EFKokoFitn,00:16:F0Dell,00:16:F1OmniSens,00:16:F2DmobileS,00:16:F3CASTInfo,00:16:F4Eidicom,00:16:F5DalianGo,00:16:F6VideoPro,00:16:F7L-3Commu,00:16:F8Aviqtech,00:16:F9CETRTAPO,00:16:FAECITelec,00:16:FBShenzhen,00:16:FCTohken,00:16:FDJatyElec,00:16:FEAlpsalpi,00:16:FFWaminOpt,00:17:00ARRISGro,00:17:01KDE,00:17:02OsungMid,00:17:03MOSDANIn,00:17:04ShincoEl,00:17:05MethodeE,00:17:06Techfait,00:17:07InGrid,00:17:08HewlettP,00:17:09ExaltCom,00:17:0AInewDigi,00:17:0BContela,00:17:0CTwigCom,00:17:0DDustNetw,00:17:0ECisco,00:17:0FCisco,00:17:10Casa,00:17:11CytivaSw,00:17:12ISCOInte,00:17:13TigerNet,00:17:14BRContro,00:17:15Qstik,00:17:16QnoTechn,00:17:17LeicaGeo,00:17:18VanscoEl,00:17:19Audiocod,00:17:1AWinegard,00:17:1BInnovati,00:17:1CNTMicroS,00:17:1DDigit,00:17:1ETheoBenn,00:17:1FIMV,00:17:20ImageSen,00:17:21FITRE,00:17:22Hanazede,00:17:23SummitDa,00:17:24StuderPr,00:17:25LiquidCo,00:17:26m2cElect,00:17:27ThermoRa,00:17:28SelexCom,00:17:29Ubicod,00:17:2AProwareT,00:17:2BGlobalTe,00:17:2CTaejinIn,00:17:2DAxcenPho,00:17:2EFXC,00:17:2FNeuLion,00:17:30Automati,00:17:31ASUSTekC,00:17:32Science-,00:17:33Sfr,00:17:34ADCTelec,00:17:35IntelWir,00:17:36iiTron,00:17:37Industri,00:17:38Internat,00:17:39BrightHe,00:17:3ACloudast,00:17:3BCisco,00:17:3CExtremeE,00:17:3DNeology,00:17:3ELeucotro,00:17:3FBelkinIn,00:17:40BluberiG,00:17:41Defidev,00:17:42Fujitsu,00:17:43Deck,00:17:44Araneo,00:17:45INNOTZ,00:17:46Freedom9,00:17:47Trimble,00:17:48Neokoros,00:17:49HyundaeY,00:17:4ASocomec,00:17:4BNokiaDan,00:17:4CMillipor,00:17:4DDynamicN,00:17:4EParama-t,00:17:4FiCatch,00:17:50GSIGroup,00:17:51Online,00:17:52DAGS,00:17:53nForeTec,00:17:54ArkinoHi,00:17:55GESecuri,00:17:56VinciLab,00:17:57RixTechn,00:17:58ThruVisi,00:17:59Cisco,00:17:5ACisco,00:17:5BACSSolut,00:17:5CSharp,00:17:5DDongseos,00:17:5EZed-3,00:17:5FXENOLINK,00:17:60NaitoDen,00:17:61Private,00:17:62SolarTec,00:17:63Essentia,00:17:64ATMedia,00:17:65NortelNe,00:17:66AccenseT,00:17:67Earforce,00:17:68Zinwave,00:17:69Cymphoni,00:17:6AAvagoTec,00:17:6BKiyon,00:17:6CPivot3,00:17:6DCore,00:17:6EDucatiSi,00:17:6FPAXCompu,00:17:70ArtiIndu,00:17:71APDCommu,00:17:72ASTROStr,00:17:73Laketune,00:17:74Elesta,00:17:75TTEGerma,00:17:76MesoScal,00:17:77Obsidian,00:17:78CentralM,00:17:79QuickTel,00:17:7AAssaAblo,00:17:7BAzaleaNe,00:17:7CSmartlin,00:17:7DIDTTechn,00:17:7EMeshcomT,00:17:7FWorldsma,00:17:80AppliedB,00:17:81Greyston,00:17:82LoBenn,00:17:83TexasIns,00:17:84ARRISGro,00:17:85SparrEle,00:17:86wisembed,00:17:87BrotherB,00:17:88PhilipsL,00:17:89Zenitron,00:17:8ADartsTec,00:17:8BTeledyne,00:17:8CIndepend,00:17:8DCheckpoi,00:17:8EGunneboC,00:17:8FNingboYi,00:17:90HYUNDAID,00:17:91LinTech,00:17:92FalcomWi,00:17:93Tigi,00:17:94Cisco,00:17:95Cisco,00:17:96Rittmeye,00:17:97TelsyEle,00:17:98AzonicTe,00:17:99SmarTire,00:17:9AD-Link,00:17:9BChantSin,00:17:9CDEPRAGSC,00:17:9DKelman,00:17:9ESirit,00:17:9FApricorn,00:17:A0RoboTech,00:17:A13soft,00:17:A2Camrivox,00:17:A3MIX,00:17:A4HewlettP,00:17:A5RalinkTe,00:17:A6YosinEle,00:17:A7MobileCo,00:17:A8EDM,00:17:A9Sentivis,00:17:AAelab-exp,00:17:ABNintendo,00:17:ACONeilPro,00:17:ADAceNet,00:17:AEGAI-Tron,00:17:AFEnermet,00:17:B0NokiaDan,00:17:B1ACISTMed,00:17:B2SKTelesy,00:17:B3AftekInf,00:17:B4RemoteSe,00:17:B5Peerless,00:17:B6Aquantia,00:17:B7TonzeTec,00:17:B8Novatron,00:17:B9GambroLu,00:17:BASedo,00:17:BBSyrinxIn,00:17:BCTouchtun,00:17:BDTibetsys,00:17:BETratecTe,00:17:BFCoherent,00:17:C0PureTech,00:17:C1CMPrecis,00:17:C2ADBBroad,00:17:C3KTFTechn,00:17:C4QuantaMi,00:17:C5SonicWAL,00:17:C6CrossMat,00:17:C7MARACons,00:17:C8KYOCERAD,00:17:C9SamsungE,00:17:CAQisda,00:17:CBJuniperN,00:17:CCAlcatel-,00:17:CDCECWirel,00:17:CEScreenSe,00:17:CFiMCA,00:17:D0OpticomC,00:17:D1NortelNe,00:17:D2Thinlinx,00:17:D3Etymotic,00:17:D4MonsoonM,00:17:D5SamsungE,00:17:D6Bluechip,00:17:D7IONGeoph,00:17:D8MagnumSe,00:17:D9AAI,00:17:DASpansLog,00:17:DBCankoTec,00:17:DCDaemyung,00:17:DDClipsalA,00:17:DEAdvantag,00:17:DFCisco,00:17:E0Cisco,00:17:E1DACOSTec,00:17:E2ARRISGro,00:17:E3TexasIns,00:17:E4TexasIns,00:17:E5TexasIns,00:17:E6TexasIns,00:17:E7TexasIns,00:17:E8TexasIns,00:17:E9TexasIns,00:17:EATexasIns,00:17:EBTexasIns,00:17:ECTexasIns,00:17:EDWooJooIT,00:17:EEARRISGro,00:17:EFIBM,00:17:F0SZCOMBro,00:17:F1RenuElec,00:17:F2Apple,00:17:F3Harris,00:17:F4ZeronAll,00:17:F5LigNeopt,00:17:F6PyramidM,00:17:F7CEMSolut,00:17:F8MotechIn,00:17:F9ForcomSp,00:17:FAMicrosof,00:17:FBFa,00:17:FCSuprema,00:17:FDAmuletHo,00:17:FETalosSys,00:17:FFPLAYLINE,00:18:00Unigrand,00:18:01Actionte,00:18:02AlphaNet,00:18:03ArcSoftS,00:18:04E-TekDig,00:18:05BeijingI,00:18:06HokkeiIn,00:18:07Fanstel,00:18:08SightLog,00:18:09Cresyn,00:18:0ACiscoMer,00:18:0BBrillian,00:18:0COptelian,00:18:0DTerabyte,00:18:0EAvega,00:18:0FNokiaDan,00:18:10IPTrade,00:18:11NeurosTe,00:18:12BeijingX,00:18:13Sony,00:18:14Mitutoyo,00:18:15GZTechno,00:18:16Ubixon,00:18:17DEShawRe,00:18:18Cisco,00:18:19Cisco,00:18:1AAVerMedi,00:18:1BTaiJinMe,00:18:1CExterity,00:18:1DAsiaElec,00:18:1EGDXTechn,00:18:1FPalmmicr,00:18:20w5networ,00:18:21Sindoric,00:18:22CecTelec,00:18:23DeltaEle,00:18:24KimaldiE,00:18:25Private,00:18:26CaleAcce,00:18:27NecUnifi,00:18:28e2vtechn,00:18:29Gatsomet,00:18:2ATaiwanVi,00:18:2BSoftier,00:18:2CAscendNe,00:18:2DArtecDes,00:18:2EXStreamH,00:18:2FTexasIns,00:18:30TexasIns,00:18:31TexasIns,00:18:32TexasIns,00:18:33TexasIns,00:18:34TexasIns,00:18:35Thoratec,00:18:36REJ,00:18:37Universa,00:18:38PanAcces,00:18:39Cisco-Li,00:18:3AWestellT,00:18:3BCENITS,00:18:3CEncoreSo,00:18:3DVertexLi,00:18:3EDigilent,00:18:3F2Wire,00:18:403Phoenix,00:18:41HighTech,00:18:42NokiaDan,00:18:43Dawevisi,00:18:44HeadsUpT,00:18:45Pulsar-T,00:18:46Crypto,00:18:47AceNetTe,00:18:48VecimaNe,00:18:49nVentSch,00:18:4ACatcher,00:18:4BLasVegas,00:18:4CBogenCom,00:18:4DNetgear,00:18:4ELianheTe,00:18:4F8WaysTec,00:18:50SecfoneK,00:18:51SWsoft,00:18:52StorLink,00:18:53AteraNet,00:18:54Argard,00:18:55Aeromari,00:18:56EyeFi,00:18:57Unilever,00:18:58TagMaste,00:18:59Strawber,00:18:5AuControl,00:18:5BNetworkC,00:18:5CEDSLABTe,00:18:5DTaiguenT,00:18:5ENexterm,00:18:5FTAC,00:18:60SIMTechn,00:18:61Ooma,00:18:62SeagateT,00:18:63Veritech,00:18:64Eaton,00:18:65SiemensH,00:18:66LeutronV,00:18:67Datalogi,00:18:68CiscoSPV,00:18:69Kingjim,00:18:6AGlobalLi,00:18:6BSambuCom,00:18:6CNeonode,00:18:6DZhenjian,00:18:6E3Com,00:18:6FSethaInd,00:18:70E28Shang,00:18:71HewlettP,00:18:72Expertis,00:18:73Cisco,00:18:74Cisco,00:18:75AnaCiseT,00:18:76WowWee,00:18:77Amplex,00:18:78Mackware,00:18:79dSys,00:18:7AWiremold,00:18:7B4NSYS,00:18:7CIntercro,00:18:7DArmorlin,00:18:7ERGBSpect,00:18:7FZodianet,00:18:80MaximInt,00:18:81BuyangEl,00:18:82HuaweiTe,00:18:83Formosa2,00:18:84FonTechn,00:18:85Motorola,00:18:86El-Tech,00:18:87Metasyst,00:18:88GOTIVE,00:18:89WinNetSo,00:18:8AInfinova,00:18:8BDell,00:18:8CMobileAc,00:18:8DNokiaDan,00:18:8EEkahau,00:18:8FMontgome,00:18:90RadioCOM,00:18:91Zhongsha,00:18:92ads-tec,00:18:93Shenzhen,00:18:94NPCore,00:18:95HansunTe,00:18:96GreatWel,00:18:97JESS-LIN,00:18:98Kingstat,00:18:99ShenZhen,00:18:9AHANAMicr,00:18:9BThomson,00:18:9CWeldex,00:18:9DNavcast,00:18:9EOMNIKEY,00:18:9FLenntek,00:18:A0CiermaAs,00:18:A1TiqitCom,00:18:A2XIPTechn,00:18:A3ZippyTec,00:18:A4ARRISGro,00:18:A5ADigitTe,00:18:A6Persiste,00:18:A7YoggieSe,00:18:A8AnNealTe,00:18:A9Ethernet,00:18:AAProtecFi,00:18:ABBeijingL,00:18:ACShanghai,00:18:ADNidecSan,00:18:AETvt,00:18:AFSamsungE,00:18:B0NortelNe,00:18:B1IBM,00:18:B2AdeunisR,00:18:B3TECWizHo,00:18:B4DawonMed,00:18:B5MagnaCar,00:18:B6S3C,00:18:B7D3Led,00:18:B8NewVoice,00:18:B9Cisco,00:18:BACisco,00:18:BBEliwellC,00:18:BCNVPBolid,00:18:BDShenzhen,00:18:BEANSA,00:18:BFEssenceT,00:18:C0ARRISGro,00:18:C1AlmitecI,00:18:C2Firetide,00:18:C3CS,00:18:C4RabaTech,00:18:C5NokiaDan,00:18:C6OPWFuelM,00:18:C7RealTime,00:18:C8ISONAS,00:18:C9EOpsTech,00:18:CAViprinet,00:18:CBTecobest,00:18:CCAxiohmSa,00:18:CDEraeElec,00:18:CEDreamtec,00:18:CFBaldorEl,00:18:D0AtRoadAT,00:18:D1SiemensH,00:18:D2High-Gai,00:18:D3Teamcast,00:18:D4UnifiedD,00:18:D5Reigncom,00:18:D6Swirlnet,00:18:D7JAVADGNS,00:18:D8ARCHMETE,00:18:D9Santosha,00:18:DAWürthEle,00:18:DBEPLTechn,00:18:DCProstar,00:18:DDSilicond,00:18:DEIntelCor,00:18:DFMorey,00:18:E0Anaveo,00:18:E1VerkerkS,00:18:E2TopdataS,00:18:E3Visualga,00:18:E4Yiguang,00:18:E5Adhoco,00:18:E6Computer,00:18:E7CameoCom,00:18:E8Hacetron,00:18:E9Numata,00:18:EAAlltec,00:18:EBBlueZenE,00:18:ECWeldingT,00:18:EDAccutech,00:18:EEVideolog,00:18:EFEscapeCo,00:18:F0JOYTOTO,00:18:F1Chunichi,00:18:F2BeijingT,00:18:F3ASUSTekC,00:18:F4EOTECHNI,00:18:F5Shenzhen,00:18:F6ThomsonT,00:18:F7Kameleon,00:18:F8Cisco-Li,00:18:F9VVOND,00:18:FAYushinPr,00:18:FBComproTe,00:18:FCAltecEle,00:18:FDOptimalT,00:18:FEHewlettP,00:18:FFPowerQua,00:19:00Intelliv,00:19:01F1Media,00:19:02Cambridg,00:19:03BigfootN,00:19:04WBElectr,00:19:05SCHRACKS,00:19:06Cisco,00:19:07Cisco,00:19:08Duaxes,00:19:09DEVI-Dan,00:19:0AHasware,00:19:0BSouthern,00:19:0CEncoreEl,00:19:0DIEEE1394,00:19:0EAtechTec,00:19:0FAdvansus,00:19:10KnickEle,00:19:11JustInMo,00:19:12Welcat,00:19:13Chuang-Y,00:19:14Winix,00:19:15TECOM,00:19:16PayTec,00:19:17Posiflex,00:19:18Interact,00:19:19ASTEL,00:19:1AIrlink,00:19:1BSputnikE,00:19:1CSensicas,00:19:1DNintendo,00:19:1EBeyondwi,00:19:1FMicrolin,00:19:20KUMEelec,00:19:21Elitegro,00:19:22CMComand,00:19:23PhonexKo,00:19:24LBNLEngi,00:19:25Intelici,00:19:26BitsGen,00:19:27ImCoSys,00:19:28SiemensT,00:19:292M2BMont,00:19:2AAntiopeA,00:19:2BAclaraRF,00:19:2CARRISGro,00:19:2DNokia,00:19:2ESpectral,00:19:2FCisco,00:19:30Cisco,00:19:31Balluff,00:19:32Gude,00:19:33Strix,00:19:34TrendonT,00:19:35DuerrDen,00:19:36Sterlite,00:19:37Commerce,00:19:38UMBCommu,00:19:39Gigamips,00:19:3AOesoluti,00:19:3BLigoWave,00:19:3CHighPoin,00:19:3DGMCGuard,00:19:3EADBBroad,00:19:3FRDItechn,00:19:40Rackable,00:19:41PitneyBo,00:19:42OnSoftwa,00:19:43Belden,00:19:44FossilPa,00:19:45RFCOncep,00:19:46CianetIn,00:19:47CiscoSPV,00:19:48AireSpid,00:19:49TentelCo,00:19:4ATesto,00:19:4BSagemcom,00:19:4CFujianSt,00:19:4DAvagoTec,00:19:4EUltraEle,00:19:4FNokiaDan,00:19:50HarmanMu,00:19:51NETCONSs,00:19:52ACOGITO,00:19:53Chainlea,00:19:54Leaf,00:19:55Cisco,00:19:56Cisco,00:19:57SaafnetC,00:19:58Bluetoot,00:19:59Staccato,00:19:5AJenaerAn,00:19:5BD-Link,00:19:5CInnotech,00:19:5DShenZhen,00:19:5EARRISGro,00:19:5FValemoun,00:19:60DoCoMo,00:19:61Blaupunk,00:19:62Commerci,00:19:63Sony,00:19:64Doorking,00:19:65YuHuaTel,00:19:66Asiarock,00:19:67TELDATSp,00:19:68DigitalV,00:19:69NortelNe,00:19:6AMikroM,00:19:6BDanpex,00:19:6CEtrovisi,00:19:6DRaybitKo,00:19:6EMetacom,00:19:6FSensoPar,00:19:70Z-Com,00:19:71Guangzho,00:19:72PlexusXi,00:19:73Zeugma,00:19:7416063,00:19:75BeijingH,00:19:76XipherTe,00:19:77ExtremeN,00:19:78Datum,00:19:79NokiaDan,00:19:7AMAZeT,00:19:7BPicotest,00:19:7CRiedelCo,00:19:7DHonHaiPr,00:19:7EHonHaiPr,00:19:7FPlantron,00:19:80Gridpoin,00:19:81Vivox,00:19:82SmarDTV,00:19:83CCTR&D,00:19:84ESTIC,00:19:85ITWatchd,00:19:86ChengHon,00:19:87Panasoni,00:19:88Wi2Wi,00:19:89Sonitrol,00:19:8ANorthrop,00:19:8BNoveraOp,00:19:8CiXSea,00:19:8DOceanOpt,00:19:8EOticon,00:19:8FNokiaBel,00:19:90ELMDATA,00:19:91avinfo,00:19:92Adtran,00:19:93Changshu,00:19:94JorjinTe,00:19:95JurongHi,00:19:96TurboChe,00:19:97SoftDevi,00:19:98Sato,00:19:99FujitsuT,00:19:9AEdo-Evi,00:19:9BDiversif,00:19:9CCtring,00:19:9DVizio,00:19:9ENifty,00:19:9FDkt,00:19:A0NihonDat,00:19:A1LgInform,00:19:A2OrdynTec,00:19:A3asteelel,00:19:A4AustarTe,00:19:A5RadarFin,00:19:A6ARRISGro,00:19:A7Itu-T,00:19:A8WiQuestC,00:19:A9Cisco,00:19:AACisco,00:19:ABRaycom,00:19:ACGSP,00:19:ADBobst,00:19:AEHoplingT,00:19:AFRigolTec,00:19:B0HanYangS,00:19:B1Arrow7,00:19:B2XYnetsof,00:19:B3Stanford,00:19:B4Intellio,00:19:B5FamarFue,00:19:B6EuroEmme,00:19:B7NokiaDan,00:19:B8Boundary,00:19:B9Dell,00:19:BAParadoxS,00:19:BBHewlettP,00:19:BCElectroC,00:19:BDNewMedia,00:19:BEAltaiTec,00:19:BFCitiwayt,00:19:C0ARRISGro,00:19:C1Alpsalpi,00:19:C2Equustek,00:19:C3Qualitro,00:19:C4Infocryp,00:19:C5SonyInte,00:19:C6zte,00:19:C7Cambridg,00:19:C8AnyDATA,00:19:C9S&CElect,00:19:CABroadata,00:19:CBZyxelCom,00:19:CCRCGHK,00:19:CDChengdue,00:19:CEProgress,00:19:CFSalicru,00:19:D0Cathexis,00:19:D1IntelCor,00:19:D2IntelCor,00:19:D3TRAKMicr,00:19:D4ICXTechn,00:19:D5IPInnova,00:19:D6LSCablea,00:19:D7Fortunet,00:19:D8Maxfor,00:19:D9Zeutsche,00:19:DAWelltran,00:19:DBMicro-St,00:19:DCENENSYST,00:19:DDFEI-Zyfe,00:19:DEMobitek,00:19:DFThomson,00:19:E0Tp-LinkT,00:19:E1NortelNe,00:19:E2JuniperN,00:19:E3Apple,00:19:E42Wire,00:19:E5LynxStud,00:19:E6ToyoMedi,00:19:E7Cisco,00:19:E8Cisco,00:19:E9S-Inform,00:19:EATeraMage,00:19:EBPyronix,00:19:ECSagamore,00:19:EDAxesstel,00:19:EECARLOGAV,00:19:EFShenzhen,00:19:F0Unionman,00:19:F1StarComm,00:19:F2Teradyne,00:19:F3Cetis,00:19:F4Converge,00:19:F5Imaginat,00:19:F6AcconetP,00:19:F7OnsetCom,00:19:F8Embedded,00:19:F9TDK-Lamb,00:19:FACableVis,00:19:FBBSkyB,00:19:FCPTUfoaks,00:19:FDNintendo,00:19:FEShenzhen,00:19:FFFinnzyme,00:1A:00Matrix,00:1A:01SmithsMe,00:1A:02SecureCa,00:1A:03AngelEle,00:1A:04InterayS,00:1A:05Optibase,00:1A:06OpVista,00:1A:07ArecontV,00:1A:08Simoco,00:1A:09Wayfarer,00:1A:0AAdaptive,00:1A:0BBonaTech,00:1A:0CSwe-Dish,00:1A:0DHandHeld,00:1A:0EChengUei,00:1A:0FSistemas,00:1A:10LucentTr,00:1A:11Google,00:1A:12Essilor,00:1A:13WanlidaG,00:1A:14XinHuaCo,00:1A:15gemaltoe,00:1A:16NokiaDan,00:1A:17TeakTech,00:1A:18Advanced,00:1A:19Computer,00:1A:1AGentexEl,00:1A:1BARRISGro,00:1A:1CGT&TEngi,00:1A:1DPChomeOn,00:1A:1EArubaaHe,00:1A:1FCoastalE,00:1A:20CMOTECH,00:1A:21Brookhui,00:1A:22eQ-3Entw,00:1A:23IceQube,00:1A:24GalaxyTe,00:1A:25DeltaDor,00:1A:26Deltanod,00:1A:27Ubistar,00:1A:28ASWTTaiw,00:1A:29JohnsonO,00:1A:2AArcadyan,00:1A:2BAyecomTe,00:1A:2CSATEC,00:1A:2DNavvoGro,00:1A:2EZiovaCop,00:1A:2FCisco,00:1A:30Cisco,00:1A:31ScanCoin,00:1A:32ActivaMu,00:1A:33ASICommu,00:1A:34KonkaGro,00:1A:35BARTEC,00:1A:36Aipermon,00:1A:37Lear,00:1A:38Sanmina-,00:1A:39MertenGm,00:1A:3ADongahel,00:1A:3BDoahElec,00:1A:3CTechnowa,00:1A:3DAjinVisi,00:1A:3EFasterTe,00:1A:3FIntelbra,00:1A:40A-FourTe,00:1A:41INOCOVA,00:1A:42Techcity,00:1A:43LogicalL,00:1A:44JWTradin,00:1A:45GNNetcom,00:1A:46DigitalM,00:1A:47Agami,00:1A:48Takacom,00:1A:49MicroVis,00:1A:4AQumranet,00:1A:4BHewlettP,00:1A:4CCrossbow,00:1A:4DGiga-Byt,00:1A:4ENTILinMo,00:1A:4FAVM,00:1A:50PheeNetT,00:1A:51AlfredMa,00:1A:52Meshlinx,00:1A:53Zylaya,00:1A:54HipShing,00:1A:55ACA-Digi,00:1A:56ViewTel,00:1A:57MatrixDe,00:1A:58CCVDeuts,00:1A:59Ircona,00:1A:5AKoreaEle,00:1A:5BNetCareS,00:1A:5CEuchnerG,00:1A:5DMobinnov,00:1A:5EThincomT,00:1A:5FKitWorks,00:1A:60WaveElec,00:1A:61PacStar,00:1A:62DataRobo,00:1A:63ElsterSo,00:1A:64IBM,00:1A:65Seluxit,00:1A:66ARRISGro,00:1A:67Infinite,00:1A:68WeltecEn,00:1A:69WuhanYan,00:1A:6ATranzas,00:1A:6BUniversa,00:1A:6CCisco,00:1A:6DCisco,00:1A:6EImproTec,00:1A:6FMITEL,00:1A:70Cisco-Li,00:1A:71Diostech,00:1A:72MosartSe,00:1A:73GemtekTe,00:1A:74ProcareI,00:1A:75Sony,00:1A:76SDTinfor,00:1A:77ARRISGro,00:1A:78ubtos,00:1A:79Telecomu,00:1A:7ALismoreI,00:1A:7BTeleco,00:1A:7CHirschma,00:1A:7Dcyber-bl,00:1A:7ELNSritha,00:1A:7FGCIScien,00:1A:80Sony,00:1A:81Zelax,00:1A:82PROBABui,00:1A:83PegasusT,00:1A:84VOneMult,00:1A:85MichelVa,00:1A:86NewWaveD,00:1A:87CanholdI,00:1A:88Venergy,00:1A:89NokiaDan,00:1A:8ASamsungE,00:1A:8BChunilEl,00:1A:8CSophos,00:1A:8DAVECSBer,00:1A:8E3WayNetw,00:1A:8FNortelNe,00:1A:90TrópicoS,00:1A:91FusionDy,00:1A:92ASUSTekC,00:1A:93ERCOLeuc,00:1A:94Votronic,00:1A:95HisenseM,00:1A:96Ecler,00:1A:97fitivisi,00:1A:98AsotelCo,00:1A:99SmartyHZ,00:1A:9ASkyworth,00:1A:9BADECPart,00:1A:9CRightHan,00:1A:9DSkipperW,00:1A:9EICONDigi,00:1A:9FA-Link,00:1A:A0Dell,00:1A:A1Cisco,00:1A:A2Cisco,00:1A:A3Delorme,00:1A:A4FutureUn,00:1A:A5BRNPhoen,00:1A:A6Telefunk,00:1A:A7TorianWi,00:1A:A8MamiyaDi,00:1A:A9FujianSt,00:1A:AAAnalogic,00:1A:ABeWings,00:1A:ACCorelatu,00:1A:ADARRISGro,00:1A:AESavant,00:1A:AFBlusensT,00:1A:B0SignalNe,00:1A:B1AsiaPaci,00:1A:B2CyberSol,00:1A:B3Visionit,00:1A:B4FFEI,00:1A:B5HomeNetw,00:1A:B6TexasIns,00:1A:B7EthosNet,00:1A:B8Anseri,00:1A:B9Pmc,00:1A:BACatonOve,00:1A:BBFontalTe,00:1A:BCU4EATech,00:1A:BDImpatica,00:1A:BEComputer,00:1A:BFTRUMPFLa,00:1A:C0JoybienT,00:1A:C13Com,00:1A:C2YEC,00:1A:C3Scientif,00:1A:C42Wire,00:1A:C5Keysight,00:1A:C6MicroCon,00:1A:C7Unipoint,00:1A:C8ISLInstr,00:1A:C9Suzuken,00:1A:CATilera,00:1A:CBAutocomP,00:1A:CCCelestia,00:1A:CDTidelEng,00:1A:CEYupiteru,00:1A:CFCTElettr,00:1A:D0AlbisTec,00:1A:D1Fargo,00:1A:D2Eletroni,00:1A:D3Vamp,00:1A:D4iPOXTech,00:1A:D5KmcChain,00:1A:D6JiagnsuA,00:1A:D7Christie,00:1A:D8AlsterAe,00:1A:D9Internat,00:1A:DABiz-2-Me,00:1A:DBARRISGro,00:1A:DCNokiaDan,00:1A:DDPePWave,00:1A:DEARRISGro,00:1A:DFInteract,00:1A:E0Mytholog,00:1A:E1EdgeAcce,00:1A:E2Cisco,00:1A:E3Cisco,00:1A:E4MedicisT,00:1A:E5MvoxTech,00:1A:E6AtlantaA,00:1A:E7AztekNet,00:1A:E8UnifySof,00:1A:E9Nintendo,00:1A:EARadioTer,00:1A:EBAlliedTe,00:1A:ECKeumbeeE,00:1A:EDINCOTEC,00:1A:EEShenztec,00:1A:EFLoopcomm,00:1A:F0Alcatel-,00:1A:F1Embedded,00:1A:F2Dynavisi,00:1A:F3Samyoung,00:1A:F4Handream,00:1A:F5Pentaone,00:1A:F6Woven,00:1A:F7datascha,00:1A:F8CopleyCo,00:1A:F9AeroVIro,00:1A:FAWelchAll,00:1A:FBJoby,00:1A:FCModusLin,00:1A:FDEvolis,00:1A:FESofacrea,00:1A:FFWizyoung,00:1B:00NeopostT,00:1B:01AppliedR,00:1B:02ED,00:1B:03ActionTe,00:1B:04Affinity,00:1B:05Ymc,00:1B:06Ateliers,00:1B:07Mendocin,00:1B:08DanfossD,00:1B:09MatrixTe,00:1B:0AIntellig,00:1B:0BPhidgets,00:1B:0CCisco,00:1B:0DCisco,00:1B:0EInoTecOr,00:1B:0FPetratec,00:1B:10ShenZhen,00:1B:11D-Link,00:1B:12Apprion,00:1B:13IcronTec,00:1B:14CarexLig,00:1B:15Voxtel,00:1B:16Celtro,00:1B:17PaloAlto,00:1B:18TsukenEl,00:1B:19IEEEI&MS,00:1B:1Ae-treesJ,00:1B:1BSiemens,00:1B:1CCoherent,00:1B:1DPhoenixI,00:1B:1EHARTComm,00:1B:1FDELTA-Da,00:1B:20TPineTec,00:1B:21IntelCor,00:1B:22PalitMic,00:1B:23SimpleCo,00:1B:24QuantaCo,00:1B:25NortelNe,00:1B:26RON-Tele,00:1B:27MerlinCS,00:1B:28Polygon,00:1B:29Avantis,00:1B:2ACisco,00:1B:2BCisco,00:1B:2CATRONele,00:1B:2DMed-Eng,00:1B:2ESinkyoEl,00:1B:2FNetgear,00:1B:30Solitech,00:1B:31NeuralIm,00:1B:32QLogic,00:1B:33NokiaDan,00:1B:34FocusSys,00:1B:35ChongQin,00:1B:36TsubataE,00:1B:37Computec,00:1B:38CompalIn,00:1B:39Proxicas,00:1B:3ASIMS,00:1B:3BYi-Qing,00:1B:3CSoftware,00:1B:3DEuroTel,00:1B:3ECurtis,00:1B:3FProCurve,00:1B:40NetworkA,00:1B:41GeneralI,00:1B:42WiseBlue,00:1B:43BeijingD,00:1B:44SanDisk,00:1B:45ABBASDiv,00:1B:46BlueoneT,00:1B:47Futarque,00:1B:48Shenzhen,00:1B:49RobertsR,00:1B:4AW&WCommu,00:1B:4BSANION,00:1B:4CSigntech,00:1B:4DArecaTec,00:1B:4ENavmanNe,00:1B:4FAvaya,00:1B:50NizhnyNo,00:1B:51VectorTe,00:1B:52ARRISGro,00:1B:53Cisco,00:1B:54Cisco,00:1B:55HurcoAut,00:1B:56TehutiNe,00:1B:57Semindia,00:1B:58ACECADEn,00:1B:59Sony,00:1B:5AApolloIm,00:1B:5B2Wire,00:1B:5CAzuretec,00:1B:5DVololink,00:1B:5EBPL,00:1B:5FAlienTec,00:1B:60Navigon,00:1B:61DigitalA,00:1B:62JHTOptoe,00:1B:63Apple,00:1B:64IsaacLan,00:1B:65ChinaGri,00:1B:66Sennheis,00:1B:67Cisco,00:1B:68Modnnet,00:1B:69Equaline,00:1B:6APowerwav,00:1B:6BSwyxSolu,00:1B:6CLookXDig,00:1B:6DMidtroni,00:1B:6EKeysight,00:1B:6FTeletrak,00:1B:70IRIUbite,00:1B:71Telular,00:1B:72Sicep,00:1B:73DTLBroad,00:1B:74MiraLink,00:1B:75Hypermed,00:1B:76Ripcode,00:1B:77IntelCor,00:1B:78HewlettP,00:1B:79Faiveley,00:1B:7ANintendo,00:1B:7BTintomet,00:1B:7CARCambri,00:1B:7DCXRAnder,00:1B:7EBeckmann,00:1B:7FTMNTechn,00:1B:80LORD,00:1B:81DATAQIns,00:1B:82TaiwanSe,00:1B:83Finsoft,00:1B:84ScanEngi,00:1B:85MANEnerg,00:1B:86BoschAcc,00:1B:87Deepsoun,00:1B:88DivinetA,00:1B:89EMZAVisu,00:1B:8A2MElectr,00:1B:8BNECPlatf,00:1B:8CJMicronT,00:1B:8DElectron,00:1B:8EHuluSwed,00:1B:8FCisco,00:1B:90Cisco,00:1B:91Efkon,00:1B:92l-acoust,00:1B:93JCDecaux,00:1B:94TEMpA,00:1B:95Video,00:1B:96GeneralS,00:1B:97ViolinTe,00:1B:98SamsungE,00:1B:99KSSystem,00:1B:9AApolloFi,00:1B:9BHose-McC,00:1B:9CSATELsp,00:1B:9DNovusSec,00:1B:9EAskeyCom,00:1B:9FCalyptec,00:1B:A0Awox,00:1B:A1Åmic,00:1B:A2IDSImagi,00:1B:A3FlexitGr,00:1B:A4EAfikim,00:1B:A5MyungMin,00:1B:A6intotech,00:1B:A7LoricaSo,00:1B:A8UBI&MOBI,00:1B:A9Brotheri,00:1B:AAXenICs,00:1B:ABTelchemy,00:1B:ACCurtissW,00:1B:ADiControl,00:1B:AEMicroCon,00:1B:AFNokiaDan,00:1B:B0BharatEl,00:1B:B1WistronN,00:1B:B2Intellec,00:1B:B3Condalo,00:1B:B4Airvod,00:1B:B5Cherry,00:1B:B6BirdElec,00:1B:B7AltaHeig,00:1B:B8BluewayE,00:1B:B9Elitegro,00:1B:BANortelNe,00:1B:BBRFTech,00:1B:BCSilverPe,00:1B:BDFMCKongs,00:1B:BEICOPDigi,00:1B:BFSagemcom,00:1B:C0JuniperN,00:1B:C1HOLUXTec,00:1B:C2Integrat,00:1B:C3Mobisolu,00:1B:C4Ultratec,00:1B:C5IEEERegi,00:1B:C5:00:00:00/36Convergi,00:1B:C5:00:10:00/36OpenRBco,00:1B:C5:00:20:00/36GORAMO-J,00:1B:C5:00:30:00/36MicroSig,00:1B:C5:00:40:00/36Intellvi,00:1B:C5:00:50:00/36Private,00:1B:C5:00:60:00/36TRIAX-HI,00:1B:C5:00:70:00/36EnergyAw,00:1B:C5:00:80:00/36DalajEle,00:1B:C5:00:90:00/36SolomonS,00:1B:C5:00:A0:00/36MercuryH,00:1B:C5:00:B0:00/36Private,00:1B:C5:00:C0:00/36QuantumT,00:1B:C5:00:D0:00/36Advanced,00:1B:C5:00:E0:00/36VigorEle,00:1B:C5:00:F0:00/36Simavita,00:1B:C5:01:00:00/36Softel,00:1B:C5:01:10:00/36NPPMera,00:1B:C5:01:20:00/36TokyoCos,00:1B:C5:01:30:00/36ZamirRec,00:1B:C5:01:40:00/36Private,00:1B:C5:01:50:00/36Corporat,00:1B:C5:01:60:00/36Energote,00:1B:C5:01:70:00/36cPacketN,00:1B:C5:01:90:00/36DunlopCo,00:1B:C5:01:A0:00/36AbaElect,00:1B:C5:01:B0:00/36Commonwe,00:1B:C5:01:C0:00/36Coolit,00:1B:C5:01:D0:00/36Rose+Her,00:1B:C5:01:E0:00/36Private,00:1B:C5:01:F0:00/36SaturnSo,00:1B:C5:02:00:00/36Momentum,00:1B:C5:02:10:00/36Openpeak,00:1B:C5:02:20:00/36CjscStcS,00:1B:C5:02:30:00/36MAGOdiDe,00:1B:C5:02:40:00/36AnnecyEl,00:1B:C5:02:50:00/36andersen,00:1B:C5:02:60:00/36DIMEPSis,00:1B:C5:02:70:00/36CAMEAspo,00:1B:C5:02:80:00/36Stechwin,00:1B:C5:02:90:00/362FranceM,00:1B:C5:02:A0:00/36Analytic,00:1B:C5:02:B0:00/36SaturnSo,00:1B:C5:02:C0:00/36CareEver,00:1B:C5:02:D0:00/36DDTRONIK,00:1B:C5:02:E0:00/36Bettini,00:1B:C5:02:F0:00/36Fibrain,00:1B:C5:03:00:00/36OctoGate,00:1B:C5:03:10:00/36Adixein,00:1B:C5:03:20:00/36OsborneC,00:1B:C5:03:30:00/36JESuunni,00:1B:C5:03:40:00/36InterCEL,00:1B:C5:03:50:00/36RTLS,00:1B:C5:03:60:00/36Lomar,00:1B:C5:03:70:00/36ITWReyfl,00:1B:C5:03:80:00/36SEEDInte,00:1B:C5:03:90:00/36Euresys,00:1B:C5:03:A0:00/36MindMade,00:1B:C5:03:B0:00/36Promixis,00:1B:C5:03:C0:00/36Xiphos,00:1B:C5:03:D0:00/36rioxo,00:1B:C5:03:E0:00/36Daylight,00:1B:C5:03:F0:00/36ELTRADE,00:1B:C5:04:00:00/36Actidata,00:1B:C5:04:10:00/36DesignAE,00:1B:C5:04:20:00/36ChamSys,00:1B:C5:04:30:00/36Coincide,00:1B:C5:04:40:00/36&quot;RA,00:1B:C5:04:50:00/36MarvelDi,00:1B:C5:04:60:00/36Géant,00:1B:C5:04:70:00/36PTAmanin,00:1B:C5:04:80:00/36XPossibl,00:1B:C5:04:90:00/36EUROCONT,00:1B:C5:04:A0:00/36CertisTe,00:1B:C5:04:B0:00/36SiliconC,00:1B:C5:04:C0:00/36RhinoCon,00:1B:C5:04:D0:00/36eirakuel,00:1B:C5:04:E0:00/36Mitsubis,00:1B:C5:04:F0:00/36Orbital,00:1B:C5:05:00:00/36TeliSwit,00:1B:C5:05:10:00/36QQNaviga,00:1B:C5:05:20:00/36Engineer,00:1B:C5:05:30:00/36Metrycom,00:1B:C5:05:40:00/36Private,00:1B:C5:05:50:00/36Lumiplan,00:1B:C5:05:60:00/36ThinKomS,00:1B:C5:05:70:00/36EREEElec,00:1B:C5:05:80:00/36optiMEAS,00:1B:C5:05:90:00/36Inpixal,00:1B:C5:05:A0:00/36PostecDa,00:1B:C5:05:B0:00/36konzeptp,00:1B:C5:05:C0:00/36Suretrak,00:1B:C5:05:D0:00/36Prominfo,00:1B:C5:05:E0:00/36Ecomed-C,00:1B:C5:05:F0:00/36Klingent,00:1B:C5:06:00:00/36Enstech,00:1B:C5:06:10:00/36Scientif,00:1B:C5:06:20:00/36Sulaon,00:1B:C5:06:30:00/36Check-It,00:1B:C5:06:40:00/36Enkora,00:1B:C5:06:50:00/36PlairMed,00:1B:C5:06:60:00/36Manufact,00:1B:C5:06:70:00/36Embit,00:1B:C5:06:80:00/36HCSKABLO,00:1B:C5:06:90:00/36DatasatD,00:1B:C5:06:A0:00/36IST,00:1B:C5:06:B0:00/36Verified,00:1B:C5:06:C0:00/36LuxconSy,00:1B:C5:06:D0:00/36TESElect,00:1B:C5:06:E0:00/36TwoDimen,00:1B:C5:06:F0:00/36Emzior,00:1B:C5:07:00:00/36SiemensI,00:1B:C5:07:10:00/36Centerfo,00:1B:C5:07:20:00/36OhioSemi,00:1B:C5:07:30:00/36tado,00:1B:C5:07:40:00/36Dynasthe,00:1B:C5:07:50:00/36Kitron,00:1B:C5:07:60:00/36PLAiRMed,00:1B:C5:07:70:00/36Momentum,00:1B:C5:07:80:00/36DonbassS,00:1B:C5:07:90:00/36HPIHighP,00:1B:C5:07:A0:00/36Servicio,00:1B:C5:07:B0:00/36QCOREMed,00:1B:C5:07:C0:00/36head,00:1B:C5:07:D0:00/36Greatcom,00:1B:C5:07:E0:00/36BioMolec,00:1B:C5:07:F0:00/36Hitechla,00:1B:C5:08:00:00/36LUMINO,00:1B:C5:08:10:00/36WonATech,00:1B:C5:08:20:00/36TGSGeoph,00:1B:C5:08:30:00/36Diwel,00:1B:C5:08:40:00/36AppliedI,00:1B:C5:08:50:00/36Oberonmi,00:1B:C5:08:60:00/36CASTGrou,00:1B:C5:08:70:00/36OnnetTec,00:1B:C5:08:80:00/36UABKitro,00:1B:C5:08:90:00/36Signatur,00:1B:C5:08:A0:00/36Topicon,00:1B:C5:08:B0:00/36Nistica,00:1B:C5:08:C0:00/36Triax,00:1B:C5:08:D0:00/36Eurek,00:1B:C5:08:E0:00/36TrendPoi,00:1B:C5:08:F0:00/36Unilever,00:1B:C5:09:00:00/36SevenSol,00:1B:C5:09:10:00/363greenAp,00:1B:C5:09:20:00/36ArnouseD,00:1B:C5:09:30:00/36AmbientD,00:1B:C5:09:40:00/36reelyAct,00:1B:C5:09:50:00/36PREVACsp,00:1B:C5:09:60:00/36Sanstrea,00:1B:C5:09:70:00/36Plexstar,00:1B:C5:09:80:00/36Cubic,00:1B:C5:09:90:00/36UABKitro,00:1B:C5:09:A0:00/36Shenzhen,00:1B:C5:09:B0:00/36YIK,00:1B:C5:09:C0:00/36SICES,00:1B:C5:09:D0:00/36Navitar,00:1B:C5:09:E0:00/36K+KMesst,00:1B:C5:09:F0:00/36ENTESp,00:1B:C5:0A:00:00/36Silvair,00:1B:C5:0A:10:00/36Hangzhou,00:1B:C5:0A:20:00/36HettichB,00:1B:C5:0A:30:00/36PANetwor,00:1B:C5:0A:40:00/36Radmor,00:1B:C5:0A:50:00/36TeslaCon,00:1B:C5:0A:60:00/36BalterSe,00:1B:C5:0A:70:00/36LGLElect,00:1B:C5:0A:80:00/36LinkPrec,00:1B:C5:0A:90:00/36Elektrom,00:1B:C5:0A:A0:00/36Senceive,00:1B:C5:0A:B0:00/36Evondos,00:1B:C5:0A:C0:00/36AVnuAlli,00:1B:C5:0A:D0:00/36TierraJa,00:1B:C5:0A:E0:00/36TechlanR,00:1B:C5:0A:F0:00/36Enerwise,00:1B:C5:0B:00:00/36J-DCom,00:1B:C5:0B:10:00/36RoslenEc,00:1B:C5:0B:20:00/36SKODAELE,00:1B:C5:0B:30:00/36FSMSolut,00:1B:C5:0B:40:00/36Coban,00:1B:C5:0B:50:00/36Exibea,00:1B:C5:0B:60:00/36Veilux,00:1B:C5:0B:70:00/36Autelis,00:1B:C5:0B:80:00/36Private,00:1B:C5:0B:90:00/36DenkiKog,00:1B:C5:0B:A0:00/36NtMicros,00:1B:C5:0B:B0:00/36Triax,00:1B:C5:0B:C0:00/36kuwatec,00:1B:C5:0B:D0:00/36BridgeDi,00:1B:C5:0B:E0:00/36YESpayIn,00:1B:C5:0B:F0:00/36TNCore,00:1B:C5:0C:00:00/36DigitalL,00:1B:C5:0C:10:00/36EREEElec,00:1B:C5:0C:20:00/36TechSolu,00:1B:C5:0C:30:00/36inomatic,00:1B:C5:0C:40:00/36Eldes,00:1B:C5:0C:50:00/36GillInst,00:1B:C5:0C:60:00/36Connode,00:1B:C5:0C:70:00/36Wizzilab,00:1B:C5:0C:80:00/36Dialine,00:1B:C5:0C:90:00/36UABKitro,00:1B:C6StratoRe,00:1B:C7StarVedi,00:1B:C8Miura,00:1B:C9FsnDispl,00:1B:CABeijingR,00:1B:CBPempek,00:1B:CCKingtekC,00:1B:CDDaviscom,00:1B:CEMeasurem,00:1B:CFDataupia,00:1B:D0IdentecS,00:1B:D1Sogestma,00:1B:D2ULTRA-XA,00:1B:D3Panasoni,00:1B:D4Cisco,00:1B:D5Cisco,00:1B:D6KelvinHu,00:1B:D7CiscoSPV,00:1B:D8FLIR,00:1B:D9Edgewate,00:1B:DAUTStarco,00:1B:DBValeoVEC,00:1B:DCVencer,00:1B:DDARRISGro,00:1B:DERenkus-H,00:1B:DFIskraSis,00:1B:E0TELENOTE,00:1B:E1ViaLogy,00:1B:E2AhnLab,00:1B:E3HealthHe,00:1B:E4Townet,00:1B:E5802autom,00:1B:E6Vr,00:1B:E7PostekEl,00:1B:E8Ultratro,00:1B:E9Broadcom,00:1B:EANintendo,00:1B:EBDMPElect,00:1B:ECNetioTec,00:1B:EDBrocadeC,00:1B:EENokiaDan,00:1B:EFBlossoms,00:1B:F0ValuePla,00:1B:F1NanjingS,00:1B:F2KworldCo,00:1B:F3TRANSRAD,00:1B:F4KenwinIn,00:1B:F5TellinkS,00:1B:F6CONWISET,00:1B:F7LundIPPr,00:1B:F8Digitrax,00:1B:F9Intellit,00:1B:FAGiNmbH,00:1B:FBAlpsalpi,00:1B:FCASUSTekC,00:1B:FDDignsys,00:1B:FEZavio,00:1B:FFMillenni,00:1C:00EntryPoi,00:1C:01ABBDrive,00:1C:02PanoLogi,00:1C:03BettyTVT,00:1C:04Airgain,00:1C:05NoninMed,00:1C:06SiemensN,00:1C:07Cwlinux,00:1C:08Echo360,00:1C:09SAEElect,00:1C:0AShenzhen,00:1C:0BSmartAnt,00:1C:0CTANITA,00:1C:0DG-Techno,00:1C:0ECisco,00:1C:0FCisco,00:1C:10Cisco-Li,00:1C:11ARRISGro,00:1C:12ARRISGro,00:1C:13OptsysTe,00:1C:14VMware,00:1C:15iPhotoni,00:1C:16ThyssenK,00:1C:17NortelNe,00:1C:18Sicert,00:1C:19secunetS,00:1C:1AThomasIn,00:1C:1BHypersto,00:1C:1CCenterCo,00:1C:1DChenzhou,00:1C:1Eemtrion,00:1C:1FQuestRet,00:1C:20CLBBenel,00:1C:21Nucsafe,00:1C:22AerisEle,00:1C:23Dell,00:1C:24FormosaW,00:1C:25HonHaiPr,00:1C:26HonHaiPr,00:1C:27SunellEl,00:1C:28Sphairon,00:1C:29CoreDigi,00:1C:2AEnvisaco,00:1C:2BAlertmec,00:1C:2CSynapse,00:1C:2DFlexRadi,00:1C:2EHPNSuppl,00:1C:2FPfister,00:1C:30ModeLigh,00:1C:31MobileXP,00:1C:32Telian,00:1C:33Sutron,00:1C:34HueyChia,00:1C:35NokiaDan,00:1C:36iNEWiT,00:1C:37Callpod,00:1C:38Bio-RadL,00:1C:39SNetsyst,00:1C:3AElementL,00:1C:3BAmRoadTe,00:1C:3CSeonDesi,00:1C:3DWaveStor,00:1C:3EECKey,00:1C:3FInternat,00:1C:40VDG-Secu,00:1C:41scemtecT,00:1C:42Parallel,00:1C:43SamsungE,00:1C:44BoschSec,00:1C:45ChenbroM,00:1C:46Qtum,00:1C:47Hangzhou,00:1C:48WiDeFi,00:1C:49ZoltanTe,00:1C:4AAVM,00:1C:4BGener8,00:1C:4CPetrotes,00:1C:4DAplixIPH,00:1C:4ETASAInte,00:1C:4FMacab,00:1C:50TCLTechn,00:1C:51CelenoCo,00:1C:52Visionee,00:1C:53SynergyL,00:1C:54Hillston,00:1C:55Shenzhen,00:1C:56Pado,00:1C:57Cisco,00:1C:58Cisco,00:1C:59DevonIt,00:1C:5AAdvanced,00:1C:5BChubbEle,00:1C:5CIntegrat,00:1C:5DLeicaMic,00:1C:5EASTONFra,00:1C:5FWinlandE,00:1C:60CSPFront,00:1C:61GalaxyMi,00:1C:62LGElectr,00:1C:63Truen,00:1C:64Landis+G,00:1C:65JoeScan,00:1C:66Ucamp,00:1C:67PumpkinN,00:1C:68AnhuiSun,00:1C:69PacketVi,00:1C:6AWeissEng,00:1C:6BCOVAX,00:1C:6C30805,00:1C:6DKyohrits,00:1C:6ENewburyN,00:1C:6FEmfit,00:1C:70Novacomm,00:1C:71Emergent,00:1C:72MayerCie,00:1C:73AristaNe,00:1C:74SyswanTe,00:1C:75Segnet,00:1C:76Wandswor,00:1C:77Prodys,00:1C:78WyplaySa,00:1C:79Cohesive,00:1C:7APerfecto,00:1C:7BCastlene,00:1C:7CPerq,00:1C:7DExcelpoi,00:1C:7EToshiba,00:1C:7FCheckPoi,00:1C:80NewBusin,00:1C:81NextGenV,00:1C:82GenewTec,00:1C:83NewLevel,00:1C:84STLSolut,00:1C:85Eunicorn,00:1C:86Cranite,00:1C:87Uriver,00:1C:88Transyst,00:1C:89ForceCom,00:1C:8ACirrasca,00:1C:8BMJInnova,00:1C:8CDialTech,00:1C:8DMesaImag,00:1C:8EAlcatel-,00:1C:8FAdvanced,00:1C:90Empacket,00:1C:91Gefen,00:1C:92Tervela,00:1C:93ExaDigm,00:1C:94LI-CORBi,00:1C:95Opticomm,00:1C:96Linkwise,00:1C:97EnzytekT,00:1C:98LuckyTec,00:1C:99ShunraSo,00:1C:9ANokiaDan,00:1C:9BFEIGELEC,00:1C:9CNortelNe,00:1C:9DLiecthi,00:1C:9EDualtech,00:1C:9FRazorstr,00:1C:A0Producti,00:1C:A1AkamaiTe,00:1C:A2ADBBroad,00:1C:A3Terra,00:1C:A4Sony,00:1C:A5Zygo,00:1C:A6Win4NET,00:1C:A7Internat,00:1C:A8AirTiesW,00:1C:A9Audiomat,00:1C:AABellon,00:1C:ABMeyerSou,00:1C:ACQniqTech,00:1C:ADWuhanTel,00:1C:AEWiChorus,00:1C:AFPlatoNet,00:1C:B0Cisco,00:1C:B1Cisco,00:1C:B2Bpt,00:1C:B3Apple,00:1C:B4IridiumS,00:1C:B5NeihuaNe,00:1C:B6DuzonCNT,00:1C:B7USCDigiA,00:1C:B8CBC,00:1C:B9KwangSun,00:1C:BAVerScien,00:1C:BBMusician,00:1C:BCCastGrab,00:1C:BDEzzeMobi,00:1C:BENintendo,00:1C:BFIntelCor,00:1C:C0IntelCor,00:1C:C1ARRISGro,00:1C:C2PartIIRe,00:1C:C3ARRISGro,00:1C:C4HewlettP,00:1C:C53Com,00:1C:C6ProStor,00:1C:C7Rembrand,00:1C:C8INDUSTRO,00:1C:C9KaiseEle,00:1C:CAShanghai,00:1C:CBForthPub,00:1C:CCBlackBer,00:1C:CDAlektron,00:1C:CEByTechde,00:1C:CFLimetek,00:1C:D0Circleon,00:1C:D1WavesAud,00:1C:D2KingCham,00:1C:D3ZPEngine,00:1C:D4NokiaDan,00:1C:D5ZeeVee,00:1C:D6NokiaDan,00:1C:D7HarmanBe,00:1C:D8BlueAntW,00:1C:D9GlobalTo,00:1C:DAExeginTe,00:1C:DBCarpoint,00:1C:DCCustomCo,00:1C:DDCowbellE,00:1C:DEInteract,00:1C:DFBelkinIn,00:1C:E0DasanTps,00:1C:E1IndraSis,00:1C:E2AtteroTe,00:1C:E3Optimedi,00:1C:E4EleSy,00:1C:E5MBSElect,00:1C:E6Innes,00:1C:E7RoconRes,00:1C:E8Cummins,00:1C:E9GalaxyTe,00:1C:EAScientif,00:1C:EBNortelNe,00:1C:ECMobileso,00:1C:EDEnvironn,00:1C:EESHARP,00:1C:EFPrimaxEl,00:1C:F0D-Link,00:1C:F1SUPoXTec,00:1C:F2TenlonTe,00:1C:F3EvsBroad,00:1C:F4MediaTec,00:1C:F5Wiseblue,00:1C:F6Cisco,00:1C:F7AudioSci,00:1C:F8ParadeTe,00:1C:F9Cisco,00:1C:FAAlarmcom,00:1C:FBARRISGro,00:1C:FCSumitomo,00:1C:FDUniversa,00:1C:FEQuartics,00:1C:FFNaperaNe,00:1D:00Brivo,00:1D:01NeptuneD,00:1D:02Cybertec,00:1D:03DesignSo,00:1D:04ZipitWir,00:1D:05CooperLi,00:1D:06HMElectr,00:1D:07Shenzhen,00:1D:08JiangsuY,00:1D:09Dell,00:1D:0ADavisIns,00:1D:0BPowerSta,00:1D:0CMobileCo,00:1D:0DSonyInte,00:1D:0EAgaphaTe,00:1D:0FTp-LinkT,00:1D:10LightHau,00:1D:11Analogue,00:1D:12Rohm,00:1D:13NextGTV,00:1D:14Speradto,00:1D:15Shenzhen,00:1D:16Sfr,00:1D:17DigitalS,00:1D:18PowerInn,00:1D:19Arcadyan,00:1D:1AOvisLink,00:1D:1BSangeanE,00:1D:1CGennet,00:1D:1DInter-M,00:1D:1EKyushuTe,00:1D:1FSiauliuT,00:1D:20Comtrend,00:1D:21AlcadSL,00:1D:22FossAnal,00:1D:23Sensus,00:1D:24AclaraPo,00:1D:25SamsungE,00:1D:26Rockridg,00:1D:27Nac-Inte,00:1D:28Sony,00:1D:29Doro,00:1D:2AShenzhen,00:1D:2BWuhanPon,00:1D:2CWavetren,00:1D:2DPylone,00:1D:2ERuckusWi,00:1D:2FQuantumV,00:1D:30YXWirele,00:1D:31HighproI,00:1D:32LongkayC,00:1D:33Maverick,00:1D:34SYRISTec,00:1D:35Viconics,00:1D:36Electron,00:1D:37Thales-P,00:1D:38SeagateT,00:1D:39Moohadig,00:1D:3Amhacoust,00:1D:3BNokiaDan,00:1D:3CMuscle,00:1D:3DAvidyne,00:1D:3ESakaTech,00:1D:3FMitron,00:1D:40Intel–GE,00:1D:41HardyIns,00:1D:42NortelNe,00:1D:43Shenzhen,00:1D:44Krohne,00:1D:45Cisco,00:1D:46Cisco,00:1D:47Covote,00:1D:48Sensor-T,00:1D:49Innovati,00:1D:4ACarestre,00:1D:4BGridConn,00:1D:4CAlcatel-,00:1D:4DAdaptive,00:1D:4ETCMMobil,00:1D:4FApple,00:1D:50Spinetix,00:1D:51BabcockW,00:1D:52Defzone,00:1D:53S&OElect,00:1D:54SunnicTe,00:1D:55ZANTAZ,00:1D:56KramerEl,00:1D:57CAETECMe,00:1D:58CQ,00:1D:59MitraEne,00:1D:5A2Wire,00:1D:5BTecvanIn,00:1D:5CTomCommu,00:1D:5DControlD,00:1D:5EComingMe,00:1D:5FOverSpee,00:1D:60ASUSTekC,00:1D:61BIJ,00:1D:62InPhaseT,00:1D:63MieleCie,00:1D:64AdamComm,00:1D:65Microwav,00:1D:66HyundaiT,00:1D:67Amec,00:1D:68ThomsonT,00:1D:69Knorr-Br,00:1D:6AAlphaNet,00:1D:6BARRISGro,00:1D:6CClariPhy,00:1D:6DConfidan,00:1D:6ENokiaDan,00:1D:6FChainzon,00:1D:70Cisco,00:1D:71Cisco,00:1D:72Wistron,00:1D:73Buffalo,00:1D:74TianjinC,00:1D:75Radiosca,00:1D:76Eyeheigh,00:1D:77NSGate,00:1D:78InvengoI,00:1D:79Signamax,00:1D:7AWideband,00:1D:7BIceEnerg,00:1D:7CABEElett,00:1D:7DGiga-Byt,00:1D:7ECisco-Li,00:1D:7FTekronIn,00:1D:80BeijingH,00:1D:81Guangzho,00:1D:82GNNetcom,00:1D:83Emitech,00:1D:84Gateway,00:1D:85CallDire,00:1D:86ShinwaIn,00:1D:87VigTechL,00:1D:88Clearwir,00:1D:89VaultSto,00:1D:8ATechTrex,00:1D:8BADBBroad,00:1D:8CLaCrosse,00:1D:8DFlukePro,00:1D:8EAlereon,00:1D:8FPureWave,00:1D:90EMCOFlow,00:1D:91Digitize,00:1D:92Micro-St,00:1D:93Modacom,00:1D:94ClimaxTe,00:1D:95Flash,00:1D:96WatchGua,00:1D:97AlertusT,00:1D:98NokiaDan,00:1D:99CyanOpti,00:1D:9AGodexInt,00:1D:9BHokuyoAu,00:1D:9CRockwell,00:1D:9DArtjoyIn,00:1D:9EAxionTec,00:1D:9FMATTRPTr,00:1D:A0HengYuEl,00:1D:A1Cisco,00:1D:A2Cisco,00:1D:A3SabiOso,00:1D:A4Hangzhou,00:1D:A5WBElectr,00:1D:A6MediaNum,00:1D:A7Seamless,00:1D:A8Takahata,00:1D:A9CastlesT,00:1D:AADrayTek,00:1D:ABSwissQua,00:1D:ACGigamon,00:1D:ADSinotech,00:1D:AEChangTse,00:1D:AFNortelNe,00:1D:B0FuJianHe,00:1D:B1Crescend,00:1D:B2Hokkaido,00:1D:B3HPNSuppl,00:1D:B4KumhoEng,00:1D:B5JuniperN,00:1D:B6BestComm,00:1D:B7TendrilN,00:1D:B8Intoto,00:1D:B9Wellspri,00:1D:BASony,00:1D:BBDynamicS,00:1D:BCNintendo,00:1D:BDVersamed,00:1D:BEARRISGro,00:1D:BFRadiient,00:1D:C0EnphaseE,00:1D:C1Audinate,00:1D:C2Xortec,00:1D:C3RIKORTV,00:1D:C4AIOI,00:1D:C5BeijingJ,00:1D:C6SNR,00:1D:C7L-3Commu,00:1D:C8Navionic,00:1D:C9GainSpan,00:1D:CAPAVElect,00:1D:CBExénsDev,00:1D:CCAyonCybe,00:1D:CDARRISGro,00:1D:CEARRISGro,00:1D:CFARRISGro,00:1D:D0ARRISGro,00:1D:D1ARRISGro,00:1D:D2ARRISGro,00:1D:D3ARRISGro,00:1D:D4ARRISGro,00:1D:D5ARRISGro,00:1D:D6ARRISGro,00:1D:D7Algolith,00:1D:D8Microsof,00:1D:D9HonHaiPr,00:1D:DAMikroele,00:1D:DBC-BEL,00:1D:DCHangZhou,00:1D:DDDatHK,00:1D:DEZhejiang,00:1D:DFSunitecE,00:1D:E0IntelCor,00:1D:E1IntelCor,00:1D:E2Radionor,00:1D:E3Intuicom,00:1D:E4Visionee,00:1D:E5Cisco,00:1D:E6Cisco,00:1D:E7MarineSo,00:1D:E8NikkoDen,00:1D:E9NokiaDan,00:1D:EACommtest,00:1D:EBDINECInt,00:1D:ECMarusys,00:1D:EDGridNet,00:1D:EENextvisi,00:1D:EFTrimm,00:1D:F0Vidient,00:1D:F1Intego,00:1D:F2Netflix,00:1D:F3SBSScien,00:1D:F4Magellan,00:1D:F5Sunshine,00:1D:F6SamsungE,00:1D:F7RSTAHLSc,00:1D:F8WebproVi,00:1D:F9Cybiotro,00:1D:FAFujianLA,00:1D:FBNETCLEUS,00:1D:FCKsic,00:1D:FDNokiaDan,00:1D:FEPalm,00:1D:FFNetworkC,00:1E:00ShantouI,00:1E:01RenesasT,00:1E:02SougouKe,00:1E:03LiComm,00:1E:04HansonRe,00:1E:05XseedTec,00:1E:06Wibrain,00:1E:07WinyTech,00:1E:08CentecNe,00:1E:09ZEFATEK,00:1E:0ASybaTech,00:1E:0BHewlettP,00:1E:0CSherwood,00:1E:0DMicran,00:1E:0EMaxiView,00:1E:0FBriotInt,00:1E:10HuaweiTe,00:1E:11EleluxIn,00:1E:12Ecolab,00:1E:13Cisco,00:1E:14Cisco,00:1E:15BeechHil,00:1E:16Keytroni,00:1E:17Stn,00:1E:18RadioAct,00:1E:19Gtri,00:1E:1ABestSour,00:1E:1BDigitalS,00:1E:1CSWSAustr,00:1E:1DEastCoas,00:1E:1EHoneywel,00:1E:1FNortelNe,00:1E:20Intertai,00:1E:21Qisda,00:1E:22ARVOOIma,00:1E:23Electron,00:1E:24Zhejiang,00:1E:25IntekDig,00:1E:26Digifrie,00:1E:27SBNTECH,00:1E:28Lumexis,00:1E:29Hyperthe,00:1E:2ANetgear,00:1E:2BRadioDes,00:1E:2CCyVerse,00:1E:2DStim,00:1E:2ESIRTI,00:1E:2FDiMoto,00:1E:30Shireen,00:1E:31infomark,00:1E:32Zensys,00:1E:33Inventec,00:1E:34CryptoMe,00:1E:35Nintendo,00:1E:36Ipte,00:1E:37Universa,00:1E:38Bluecard,00:1E:39ComsysCo,00:1E:3ANokiaDan,00:1E:3BNokiaDan,00:1E:3CLyngboxM,00:1E:3DAlpsalpi,00:1E:3EKMW,00:1E:3FTrellisW,00:1E:40Shanghai,00:1E:41Microwav,00:1E:42Teltonik,00:1E:43AisinAw,00:1E:44Santec,00:1E:45Sony,00:1E:46ARRISGro,00:1E:47PTHariff,00:1E:48Wi-Links,00:1E:49Cisco,00:1E:4ACisco,00:1E:4BCityThea,00:1E:4CHonHaiPr,00:1E:4DWelkinSc,00:1E:4EDAKOEDV-,00:1E:4FDell,00:1E:50Battisto,00:1E:51Converte,00:1E:52Apple,00:1E:53FurtherT,00:1E:54TOYOELEC,00:1E:55COWON,00:1E:56BallyWul,00:1E:57ALCOMAsp,00:1E:58D-Link,00:1E:59SiliconT,00:1E:5AARRISGro,00:1E:5BUnitron,00:1E:5CRBGenera,00:1E:5DHolosysd,00:1E:5ECOmputim,00:1E:5FKwikByte,00:1E:60DigitalL,00:1E:61ITEC,00:1E:62Siemon,00:1E:63Vibro-Me,00:1E:64IntelCor,00:1E:65IntelCor,00:1E:66RESOLEle,00:1E:67IntelCor,00:1E:68QuantaCo,00:1E:69Thomson,00:1E:6ABeijingB,00:1E:6BCiscoSPV,00:1E:6COpaque,00:1E:6DITR&DCen,00:1E:6EShenzhen,00:1E:6FMagna-Po,00:1E:70CobhamAn,00:1E:71MIrcomGr,00:1E:72Pcs,00:1E:73zte,00:1E:74Sagemcom,00:1E:75LGElectr,00:1E:76ThermoFi,00:1E:77Air2App,00:1E:78OwitekTe,00:1E:79Cisco,00:1E:7ACisco,00:1E:7BRI,00:1E:7CTaiwick,00:1E:7DSamsungE,00:1E:7ENortelNe,00:1E:7FCBMAmeri,00:1E:80Icotera,00:1E:81CNBTechn,00:1E:82SanDisk,00:1E:83LANMANSt,00:1E:84PikaTech,00:1E:85Lagotek,00:1E:86MEL,00:1E:87Realease,00:1E:88AndorSys,00:1E:89CRFS,00:1E:8AeCopy,00:1E:8BInfraAcc,00:1E:8CASUSTekC,00:1E:8DARRISGro,00:1E:8EHunkeler,00:1E:8FCanon,00:1E:90Elitegro,00:1E:91KIMINEle,00:1E:92Jeulin,00:1E:93CiriTech,00:1E:94Supercom,00:1E:95Sigmalin,00:1E:96Sepura,00:1E:97MediumLi,00:1E:98GreenLin,00:1E:99Vantanol,00:1E:9AHAMILTON,00:1E:9BSan-Eish,00:1E:9CFidustro,00:1E:9DRecallTe,00:1E:9Eddmhopt+,00:1E:9FVisionee,00:1E:A0XLN-t,00:1E:A1Brunata,00:1E:A2Symx,00:1E:A3NokiaDan,00:1E:A4NokiaDan,00:1E:A5ROBOTOUS,00:1E:A6BestITWo,00:1E:A7Actionte,00:1E:A8DatangMo,00:1E:A9Nintendo,00:1E:AAE-SenzaT,00:1E:ABTeleWell,00:1E:ACArmadeus,00:1E:ADWingtech,00:1E:AEContinen,00:1E:AFOphirOpt,00:1E:B0ImesDEle,00:1E:B1Cryptsof,00:1E:B2LGInnote,00:1E:B3PrimexWi,00:1E:B4UnifatTe,00:1E:B5EverSpar,00:1E:B6TAGHeuer,00:1E:B7TBTech,00:1E:B8Aloys,00:1E:B9SingFaiT,00:1E:BAHighDens,00:1E:BBBlueligh,00:1E:BCWintechA,00:1E:BDCisco,00:1E:BECisco,00:1E:BFHaasAuto,00:1E:C0Microchi,00:1E:C13ComEuro,00:1E:C2Apple,00:1E:C3Kozio,00:1E:C4Celio,00:1E:C5MiddleAt,00:1E:C6ObviusHo,00:1E:C72Wire,00:1E:C8RapidMob,00:1E:C9Dell,00:1E:CANortelNe,00:1E:CBRPCEnerg,00:1E:CCCdvi,00:1E:CDKYLANDTe,00:1E:CEBISATech,00:1E:CFPhilipsE,00:1E:D0Ingespac,00:1E:D1Keyproce,00:1E:D2RayShine,00:1E:D3DotTechn,00:1E:D4DobleEng,00:1E:D5Tekon-Au,00:1E:D6AlentecO,00:1E:D7H-Stream,00:1E:D8DigitalU,00:1E:D9Mitsubis,00:1E:DAWesemann,00:1E:DBGikenTra,00:1E:DCSony,00:1E:DDWasko,00:1E:DEByd,00:1E:DFMasterIn,00:1E:E0Urmet,00:1E:E1SamsungE,00:1E:E2SamsungE,00:1E:E3T&WElect,00:1E:E4ACSSolut,00:1E:E5Cisco-Li,00:1E:E6Shenzhen,00:1E:E7Epic,00:1E:E8Mytek,00:1E:E9Stonerid,00:1E:EASensorSw,00:1E:EBTalk-A-P,00:1E:ECCompalIn,00:1E:EDAdventiq,00:1E:EEETL,00:1E:EFCantroni,00:1E:F0GigafinN,00:1E:F1Servimat,00:1E:F2MicroMot,00:1E:F3From2,00:1E:F4L-3Commu,00:1E:F5HitekAut,00:1E:F6Cisco,00:1E:F7Cisco,00:1E:F8Emfinity,00:1E:F9PascomKo,00:1E:FAPROTEI,00:1E:FBTrioMoti,00:1E:FCMassa-K,00:1E:FDMicrobit,00:1E:FELEVELsro,00:1E:FFMueller-,00:1F:00NokiaDan,00:1F:01NokiaDan,00:1F:02Pixelmet,00:1F:03Num,00:1F:04Granch,00:1F:05iTASTech,00:1F:06Integrat,00:1F:07AZTEQMob,00:1F:08Risco,00:1F:09Jastec,00:1F:0ANortelNe,00:1F:0BFederalS,00:1F:0CIntellig,00:1F:0DL3Commun,00:1F:0EJapanKya,00:1F:0FSelectEn,00:1F:10ToledoDo,00:1F:11Openmoko,00:1F:12JuniperN,00:1F:13S,00:1F:14NexG,00:1F:15Bioscryp,00:1F:16Wistron,00:1F:17IDX,00:1F:18HakusanM,00:1F:19Ben-RiEl,00:1F:1AProminve,00:1F:1BRoyalTek,00:1F:1CKOBISHIE,00:1F:1DAtlasMat,00:1F:1EAstecTec,00:1F:1FEdimaxTe,00:1F:20Logitech,00:1F:21InnerMon,00:1F:22SourcePh,00:1F:23Interaco,00:1F:24Digitvie,00:1F:25MBS,00:1F:26Cisco,00:1F:27Cisco,00:1F:28HPNSuppl,00:1F:29HewlettP,00:1F:2AAccm,00:1F:2BOrangeLo,00:1F:2CStarbrid,00:1F:2DElectro-,00:1F:2ETriangle,00:1F:2FBerker,00:1F:30Travelpi,00:1F:31Radiocom,00:1F:32Nintendo,00:1F:33Netgear,00:1F:34LungHwaE,00:1F:35Air802,00:1F:36BellwinI,00:1F:37GenesisI,00:1F:38Positron,00:1F:39Construc,00:1F:3AHonHaiPr,00:1F:3BIntelCor,00:1F:3CIntelCor,00:1F:3DQbit,00:1F:3ERP-Techn,00:1F:3FAVM,00:1F:40Speakerc,00:1F:41RuckusWi,00:1F:42Ethersta,00:1F:43EntesEle,00:1F:44GETransp,00:1F:45Enterasy,00:1F:46NortelNe,00:1F:47MCSLogic,00:1F:48Mojix,00:1F:49Manhatta,00:1F:4AAlbentia,00:1F:4BLineageP,00:1F:4CRosemanE,00:1F:4DSegnetic,00:1F:4EConMedLi,00:1F:4FThinkwar,00:1F:50Swissdis,00:1F:51HDCommun,00:1F:52UVTUnter,00:1F:53GEMACChe,00:1F:54LorexTec,00:1F:55Honeywel,00:1F:56DigitalF,00:1F:57PhonikIn,00:1F:58EMHEnerg,00:1F:59Kronback,00:1F:5ABeckwith,00:1F:5BApple,00:1F:5CNokiaDan,00:1F:5DNokiaDan,00:1F:5EDynaTech,00:1F:5FBlatand,00:1F:60Compass,00:1F:61TalentCo,00:1F:62Stilsoft,00:1F:63Goodwin-,00:1F:64BeijingA,00:1F:65KoreaEle,00:1F:66Planar,00:1F:67Hitachi,00:1F:68Martinss,00:1F:69PingoodT,00:1F:6APacketFl,00:1F:6BLGElectr,00:1F:6CCisco,00:1F:6DCisco,00:1F:6EVtechEng,00:1F:6FFujianSu,00:1F:70BotikTec,00:1F:71xGTechno,00:1F:72QingDaoH,00:1F:73Teraview,00:1F:74EigenDev,00:1F:75GiBahnMe,00:1F:76AirLogic,00:1F:77HeolDesi,00:1F:78BlueFoxP,00:1F:79LodamEle,00:1F:7AWiWide,00:1F:7BTechNexi,00:1F:7CWitelcom,00:1F:7DEmbedded,00:1F:7EARRISGro,00:1F:7FPhabrix,00:1F:80Lucas,00:1F:81AccelSem,00:1F:82Cal-Comp,00:1F:83Teleplan,00:1F:84GigleSem,00:1F:85AprivaIS,00:1F:86digEcor,00:1F:87Skydigit,00:1F:88FMSForce,00:1F:89Signalio,00:1F:8AEllionDi,00:1F:8BCacheIQ,00:1F:8CCCS,00:1F:8DIngenieu,00:1F:8EMetrisUS,00:1F:8FShanghai,00:1F:90Actionte,00:1F:91DBSLodgi,00:1F:92Motorola,00:1F:93Xiotech,00:1F:94LascarEl,00:1F:95Sagemcom,00:1F:96Aprotech,00:1F:97BERTANA,00:1F:98Daiichi-,00:1F:99SERONICS,00:1F:9ANortelNe,00:1F:9BPosbro,00:1F:9CLedco,00:1F:9DCisco,00:1F:9ECisco,00:1F:9FThomsonT,00:1F:A0A10Netwo,00:1F:A1Gtran,00:1F:A2DatronWo,00:1F:A3T&WElect,00:1F:A4Shenzhen,00:1F:A5Blue-Whi,00:1F:A6Stilo,00:1F:A7SonyInte,00:1F:A8SmartEne,00:1F:A9AtlantaD,00:1F:AATaseon,00:1F:ABISHighTe,00:1F:ACGoodmill,00:1F:ADBrownInn,00:1F:AEBlickSou,00:1F:AFNextIO,00:1F:B0TimeIPS,00:1F:B1Cybertec,00:1F:B2Sontheim,00:1F:B32Wire,00:1F:B4SmartSha,00:1F:B5IOInterc,00:1F:B6ChiLinTe,00:1F:B7WiMateTe,00:1F:B8Universa,00:1F:B9Paltroni,00:1F:BABoyoungT,00:1F:BBXenatech,00:1F:BCEVGA,00:1F:BDKyoceraW,00:1F:BEShenzhen,00:1F:BFFulhuaMi,00:1F:C0ControlE,00:1F:C1HanlongT,00:1F:C2JowTongT,00:1F:C3SmartSyn,00:1F:C4ARRISGro,00:1F:C5Nintendo,00:1F:C6ASUSTekC,00:1F:C7CasioHit,00:1F:C8Up-Today,00:1F:C9Cisco,00:1F:CACisco,00:1F:CBNIWSolut,00:1F:CCSamsungE,00:1F:CDSamsungE,00:1F:CEQtech,00:1F:CFMSITechn,00:1F:D0Giga-Byt,00:1F:D1Optex,00:1F:D2Commtech,00:1F:D3RIVANetw,00:1F:D44Ipnet,00:1F:D5MICRORIS,00:1F:D6Shenzhen,00:1F:D7Telerad,00:1F:D8A-TrustC,00:1F:D9RSDCommu,00:1F:DANortelNe,00:1F:DBNetworkS,00:1F:DCMobileSa,00:1F:DDGdi,00:1F:DENokiaDan,00:1F:DFNokiaDan,00:1F:E0EdgeVelo,00:1F:E1HonHaiPr,00:1F:E2HonHaiPr,00:1F:E3LGElectr,00:1F:E4Sony,00:1F:E5In-Circu,00:1F:E6Alphion,00:1F:E7Simet,00:1F:E8KURUSUGA,00:1F:E9Printrex,00:1F:EAAppliedM,00:1F:EBTrioData,00:1F:ECSynapseÉ,00:1F:EDTecan,00:1F:EEubisyste,00:1F:EFShinseiI,00:1F:F0AudioPar,00:1F:F1ParadoxH,00:1F:F2VIATechn,00:1F:F3Apple,00:1F:F4PowerMon,00:1F:F5Kongsber,00:1F:F6PSAudioI,00:1F:F7Nakajima,00:1F:F8SiemensS,00:1F:F9Advanced,00:1F:FACoretree,00:1F:FBGreenPac,00:1F:FCRiccius+,00:1F:FDIndigoMo,00:1F:FEHPNSuppl,00:1F:FFRespiron,00:20:00LexmarkP,00:20:01DspSolut,00:20:02Seritech,00:20:03PixelPow,00:20:04Yamatake,00:20:05SimpleTe,00:20:06GarrettC,00:20:07Sfa,00:20:08CableCom,00:20:09PackardB,00:20:0ASource-C,00:20:0BOctagon,00:20:0CAdastra,00:20:0DCarlZeis,00:20:0ENSSLGlob,00:20:0FEBRAINS,00:20:10JeolSyst,00:20:11Canopus,00:20:12Camtroni,00:20:13Diversif,00:20:14GlobalVi,00:20:15ActisCom,00:20:16ShowaEle,00:20:17Orbotech,00:20:18CisTechn,00:20:19Ohler,00:20:1AMRVCommu,00:20:1BNorthern,00:20:1CExcel,00:20:1DKatanaPr,00:20:1ENetquest,00:20:1FBestPowe,00:20:20Megatron,00:20:21Algorith,00:20:22NMSCommu,00:20:23TCTechno,00:20:24PacificC,00:20:25ControlT,00:20:26Amkly,00:20:27MingFort,00:20:28WestEgg,00:20:29Teleproc,00:20:2ADzine,00:20:2BAtmlAdva,00:20:2CWelltron,00:20:2DTaiyo,00:20:2EDaystarD,00:20:2FZetaComm,00:20:30AnalogDi,00:20:31Tattile,00:20:32AlcatelT,00:20:33SynapseT,00:20:34RotecInd,00:20:35IbmInter,00:20:36BmcSoftw,00:20:37SeagateT,00:20:38VmeMicro,00:20:39Scinets,00:20:3ADigitalB,00:20:3BWisdm,00:20:3CEurotime,00:20:3DHoneywel,00:20:3ELogiCanT,00:20:3FJuki,00:20:40ARRISGro,00:20:41DataNet,00:20:42Datametr,00:20:43Neuron,00:20:44Genitech,00:20:45IONNetwo,00:20:46Ciprico,00:20:47Steinbre,00:20:48MarconiC,00:20:49Comtron,00:20:4APronet,00:20:4BAutocomp,00:20:4CMitronCo,00:20:4DInovis,00:20:4ENetworkS,00:20:4FDeutsche,00:20:50KoreaCom,00:20:51Verilink,00:20:52Ragula,00:20:53Huntsvil,00:20:54Sycamore,00:20:55Altech,00:20:56Neoprodu,00:20:57TITZEDAT,00:20:58AlliedSi,00:20:59MiroComp,00:20:5AComputer,00:20:5BKentrox,00:20:5CInterNet,00:20:5DNanomati,00:20:5ECastleRo,00:20:5FGammadat,00:20:60ALCATELI,00:20:61Dynatech,00:20:62Scorpion,00:20:63WiproInf,00:20:64ProtecMi,00:20:65Supernet,00:20:66GeneralM,00:20:67NodeRunn,00:20:68Isdyne,00:20:69Isdn,00:20:6AOsakaCom,00:20:6BKonicaMi,00:20:6CEvergree,00:20:6DDataRace,00:20:6EXact,00:20:6FFlowpoin,00:20:70Hynet,00:20:71Ibr,00:20:72Worklink,00:20:73Fusion,00:20:74Sungwoon,00:20:75Motorola,00:20:76Reudo,00:20:77Kardios,00:20:78Runtop,00:20:79Mikron,00:20:7AWiSEComm,00:20:7BIntel,00:20:7CAutec,00:20:7DAdvanced,00:20:7EFinecom,00:20:7FKyoeiSan,00:20:80SynergyU,00:20:81TitanEle,00:20:82Oneac,00:20:83Prestico,00:20:84OcePrint,00:20:85Eaton,00:20:86Microtec,00:20:87Memotec,00:20:88GlobalVi,00:20:89T3PlusNe,00:20:8ASonixCom,00:20:8BLapisTec,00:20:8CGalaxyNe,00:20:8DCmdTechn,00:20:8EChevinSo,00:20:8FECITelec,00:20:90Advanced,00:20:91J125Nati,00:20:92ChessEng,00:20:93Landings,00:20:94Cubix,00:20:95RivaElec,00:20:96Invensys,00:20:97AppliedS,00:20:98Hectroni,00:20:99BonElect,00:20:9A3Do,00:20:9BErsatEle,00:20:9CPrimaryA,00:20:9DLippertA,00:20:9EBrownSOp,00:20:9FMercuryC,00:20:A0OaLabora,00:20:A1Dovatron,00:20:A2GalcomNe,00:20:A3Harmonic,00:20:A4Multipoi,00:20:A5ApiEngin,00:20:A6Proxim,00:20:A7Pairgain,00:20:A8SastTech,00:20:A9WhiteHor,00:20:AAEricsson,00:20:ABMicroInd,00:20:ACInterfle,00:20:ADLinq,00:20:AEOrnetDat,00:20:AF3Com,00:20:B0GatewayD,00:20:B1ComtechR,00:20:B2GKDGesel,00:20:B3Tattile,00:20:B4TermaEle,00:20:B5YaskawaE,00:20:B6AgileNet,00:20:B7NamaquaC,00:20:B8PrimeOpt,00:20:B9Metricom,00:20:BACenterFo,00:20:BBZax,00:20:BCLongReac,00:20:BDNiobrara,00:20:BELanAcces,00:20:BFAehrTest,00:20:C0PulseEle,00:20:C1SAXA,00:20:C2TexasMem,00:20:C3CounterS,00:20:C4Inet,00:20:C5EagleNe2,00:20:C6Nectec,00:20:C7AKAIProf,00:20:C8Larscom,00:20:C9Victron,00:20:CADigitalO,00:20:CBPretecEl,00:20:CCDigitalS,00:20:CDHybridNe,00:20:CELogicalD,00:20:CFTestMeas,00:20:D0Versalyn,00:20:D1Microcom,00:20:D2RadDataC,00:20:D3OstOuetS,00:20:D4Cabletro,00:20:D5Vipa,00:20:D6Breezeco,00:20:D7JAPANMIN,00:20:D8NortelNe,00:20:D9Panasoni,00:20:DAAlcatel-,00:20:DBXnetTech,00:20:DCDensitro,00:20:DDCybertec,00:20:DEJapanDig,00:20:DFKyosanEl,00:20:E0Actionte,00:20:E1AlamarEl,00:20:E2Informat,00:20:E3McdKenco,00:20:E4HsingTec,00:20:E5ApexData,00:20:E6Lidkopin,00:20:E7B&WNucle,00:20:E8Datatrek,00:20:E9Dantel,00:20:EAEfficien,00:20:EBCincinna,00:20:ECTechware,00:20:EDGiga-Byt,00:20:EEGtech,00:20:EFUsc,00:20:F0Universa,00:20:F1AltosInd,00:20:F2Oracle,00:20:F3Raynet,00:20:F4Spectrix,00:20:F5Pandatel,00:20:F6NetTekKa,00:20:F7Cyberdat,00:20:F8CarreraC,00:20:F9Paralink,00:20:FAGde,00:20:FBOctelCom,00:20:FCMatrox,00:20:FDItvTechn,00:20:FETopwareG,00:20:FFSymmetri,00:21:00GemtekTe,00:21:01Aplicaci,00:21:02UpdateLo,00:21:03GHIElect,00:21:04GigasetC,00:21:05Alcatel-,00:21:06RIMTesti,00:21:07Seowonin,00:21:08NokiaDan,00:21:09NokiaDan,00:21:0Abydsign,00:21:0BGeminiTr,00:21:0CCymtec,00:21:0DSamsinIn,00:21:0EOrpakLTD,00:21:0FCernium,00:21:10Clearbox,00:21:11Uniphone,00:21:12WiscomSy,00:21:13Padtec,00:21:14HylabTec,00:21:15PHYWESys,00:21:16Transcon,00:21:17Tellord,00:21:18AthenaTe,00:21:19SamsungE,00:21:1ALInTech,00:21:1BCisco,00:21:1CCisco,00:21:1DDataline,00:21:1EARRISGro,00:21:1FShinsung,00:21:20SequelTe,00:21:21VRmagic,00:21:22Chip-pro,00:21:23AerosatA,00:21:24Optos,00:21:25KUKJETON,00:21:26Shenzhen,00:21:27Tp-LinkT,00:21:28Oracle,00:21:29Cisco-Li,00:21:2AAudiovox,00:21:2BMSAAuer,00:21:2CSemIndia,00:21:2DScimolex,00:21:2Edresden-,00:21:2FPhoebeMi,00:21:30KeicoHig,00:21:31Blynke,00:21:32Mastercl,00:21:33Building,00:21:34Brandywi,00:21:35Alcatel-,00:21:36ARRISGro,00:21:37BayContr,00:21:38Cepheid,00:21:39Escherlo,00:21:3AWinchest,00:21:3BBerkshir,00:21:3CAliphCom,00:21:3DCermetek,00:21:3ETomTomIn,00:21:3FA-TeamTe,00:21:40ENTechno,00:21:41Radlive,00:21:42Advanced,00:21:43ARRISGro,00:21:44SSTeleco,00:21:45Semptian,00:21:46Sanmina-,00:21:47Nintendo,00:21:48KacoSola,00:21:49ChinaDah,00:21:4APixelVel,00:21:4BShenzhen,00:21:4CSamsungE,00:21:4DGuangzho,00:21:4EGSYuasaP,00:21:4FAlpsalpi,00:21:50EyeviewE,00:21:51Millinet,00:21:52GeneralS,00:21:53SeaMicro,00:21:54D-TACQSo,00:21:55Cisco,00:21:56Cisco,00:21:57National,00:21:58StyleFly,00:21:59JuniperN,00:21:5AHewlettP,00:21:5BSenseAny,00:21:5CIntelCor,00:21:5DIntelCor,00:21:5EIBM,00:21:5FIHSE,00:21:60HideaSol,00:21:61Yournet,00:21:62NortelNe,00:21:63AskeyCom,00:21:64SpecialD,00:21:65Presstek,00:21:66NovAtel,00:21:67HWAJINT&,00:21:68iVeia,00:21:69Prologix,00:21:6AIntelCor,00:21:6BIntelCor,00:21:6COdva,00:21:6DSoltech,00:21:6EFunction,00:21:6FSymCom,00:21:70Dell,00:21:71WesungTN,00:21:72Seoultek,00:21:73IonTorre,00:21:74AvaLANWi,00:21:75PacificS,00:21:76YMaxTele,00:21:77WLGoreAs,00:21:78Matusche,00:21:79IOGEAR,00:21:7ASejinEle,00:21:7BBastec,00:21:7C2Wire,00:21:7DPyxis,00:21:7ETelitCom,00:21:7FIntracoT,00:21:80ARRISGro,00:21:81Si2Micro,00:21:82SandLink,00:21:83ANDRITZH,00:21:84Powersof,00:21:85Micro-St,00:21:86Universa,00:21:87Imacs,00:21:88EMC,00:21:89AppTech,00:21:8AElectron,00:21:8BWesconTe,00:21:8CTopContr,00:21:8DAPRouter,00:21:8EMekics,00:21:8FAvantgar,00:21:90GoliathS,00:21:91D-Link,00:21:92BaodingG,00:21:93Videofon,00:21:94PingComm,00:21:95GWDMedia,00:21:96Telsey,00:21:97Elitegro,00:21:98ThaiRadi,00:21:99Vacon,00:21:9ACambridg,00:21:9BDell,00:21:9CHoneywld,00:21:9DAdesys,00:21:9ESony,00:21:9FSatel,00:21:A0Cisco,00:21:A1Cisco,00:21:A2EKE-Elec,00:21:A3Micromin,00:21:A4DbiiNetw,00:21:A5ERLPhase,00:21:A6Videotec,00:21:A7HantleSy,00:21:A8Telephon,00:21:A9Mobilink,00:21:AANokiaDan,00:21:ABNokiaDan,00:21:ACInfrared,00:21:ADNordicID,00:21:AEAlcatel-,00:21:AFRadioFre,00:21:B0TycoTele,00:21:B1DigitalS,00:21:B2Fiberbla,00:21:B3RossCont,00:21:B4AproMedi,00:21:B5Galvanic,00:21:B6TriactaP,00:21:B7LexmarkI,00:21:B8Inphi,00:21:B9Universa,00:21:BATexasIns,00:21:BBRikenKei,00:21:BCZalaComp,00:21:BDNintendo,00:21:BECiscoSPV,00:21:BFHitachiH,00:21:C0MobileAp,00:21:C1ABBMediu,00:21:C2GLCommun,00:21:C3CORNELLC,00:21:C4Consiliu,00:21:C53DSP,00:21:C6CSJGloba,00:21:C7Russound,00:21:C8LOHUISNe,00:21:C9WavecomA,00:21:CAARTSyste,00:21:CBSmsTecno,00:21:CCFlextron,00:21:CDLiveTV,00:21:CENTC-Metr,00:21:CFCryptoGr,00:21:D0GlobalDi,00:21:D1SamsungE,00:21:D2SamsungE,00:21:D3BocomSec,00:21:D4VollmerW,00:21:D5X2E,00:21:D6LXIConso,00:21:D7Cisco,00:21:D8Cisco,00:21:D9Sekonic,00:21:DAAutomati,00:21:DBSantachi,00:21:DCTECNOALA,00:21:DDNorthsta,00:21:DEFireproW,00:21:DFMartinCh,00:21:E0CommAgil,00:21:E1NortelNe,00:21:E2visagoCo,00:21:E3SerialTe,00:21:E4I-Win,00:21:E5DisplayS,00:21:E6Starligh,00:21:E7Informat,00:21:E8MurataMa,00:21:E9Apple,00:21:EABystroni,00:21:EBEsp,00:21:ECSolutron,00:21:EDTelegesi,00:21:EEFullSpec,00:21:EFKapsys,00:21:F0EW3Techn,00:21:F1TutusDat,00:21:F2EASY3CAL,00:21:F3Si14,00:21:F4INRange,00:21:F5WesternE,00:21:F6Oracle,00:21:F7HPNSuppl,00:21:F8Enseo,00:21:F9WIRECOMT,00:21:FAA4SPTech,00:21:FBLGElectr,00:21:FCNokiaDan,00:21:FDLacroixT,00:21:FENokiaDan,00:21:FFCyfrowyP,00:22:00IBM,00:22:01AksysNet,00:22:02ExcitoEl,00:22:03Glensoun,00:22:04Koratek,00:22:05WeLinkSo,00:22:06Cyberdyn,00:22:07IntenoBr,00:22:08Certicom,00:22:09OmronHea,00:22:0AOnLive,00:22:0BNational,00:22:0CCisco,00:22:0DCisco,00:22:0EIndigoSe,00:22:0FMoCAMult,00:22:10ARRISGro,00:22:11Rohati,00:22:12CAINetwo,00:22:13Pci,00:22:14RinnaiKo,00:22:15ASUSTekC,00:22:16Shibaura,00:22:17NeatElec,00:22:18AkamaiTe,00:22:19Dell,00:22:1AAudioPre,00:22:1BMorega,00:22:1CPrivate,00:22:1DFreegene,00:22:1EMediaDev,00:22:1FeSangTec,00:22:20MitacTec,00:22:21ItohDenk,00:22:22Schaffne,00:22:23TimeKeep,00:22:24GoodWill,00:22:25ThalesAv,00:22:26Avaak,00:22:27uv-elect,00:22:28BreezeIn,00:22:29Compumed,00:22:2ASoundEar,00:22:2BNucomm,00:22:2CCeton,00:22:2DSMCNetwo,00:22:2Emaintech,00:22:2FOpenGrid,00:22:30FutureLo,00:22:31SMT&C,00:22:32DesignDe,00:22:33ADBBroad,00:22:34Corventi,00:22:35Strukton,00:22:36VectorSp,00:22:37Shinhint,00:22:38Logiplus,00:22:39IndianaL,00:22:3ACiscoSPV,00:22:3BCommunic,00:22:3CRATIOEnt,00:22:3DJumpGen,00:22:3EIRTrans,00:22:3FNetgear,00:22:40Universa,00:22:41Apple,00:22:42Alacron,00:22:43AzureWav,00:22:44ChengduL,00:22:45LeineLin,00:22:46EvocInte,00:22:47DacEngin,00:22:48Microsof,00:22:49HomeMult,00:22:4ARaylase,00:22:4BAirtechT,00:22:4CNintendo,00:22:4DMitacInt,00:22:4ESEEnergy,00:22:4FByzoroNe,00:22:50PointSix,00:22:51Lumasens,00:22:52ZOLLLife,00:22:53Entorian,00:22:54BigelowA,00:22:55Cisco,00:22:56Cisco,00:22:573ComEuro,00:22:58TaiyoYud,00:22:59Guangzho,00:22:5AGardeSec,00:22:5BTeradici,00:22:5CMultimed,00:22:5DDigicabl,00:22:5EUwinTech,00:22:5FLiteonTe,00:22:60AFREEY,00:22:61Frontier,00:22:62BEPMarin,00:22:63KoosTech,00:22:64HewlettP,00:22:65NokiaDan,00:22:66NokiaDan,00:22:67NortelNe,00:22:68HonHaiPr,00:22:69HonHaiPr,00:22:6AHoneywel,00:22:6BCisco-Li,00:22:6CLinkSpri,00:22:6DShenzhen,00:22:6EGowellEl,00:22:6F3onedata,00:22:70ABKNorth,00:22:71JägerCom,00:22:72American,00:22:73Techway,00:22:74FamilyPh,00:22:75BelkinIn,00:22:76TripleEY,00:22:77NECAustr,00:22:78Shenzhen,00:22:79NipponCo,00:22:7ATelecomD,00:22:7BApogeeLa,00:22:7CWooriSMT,00:22:7DYeData,00:22:7EChengdu3,00:22:7FRuckusWi,00:22:80A2BElect,00:22:81Daintree,00:22:828086Cons,00:22:83JuniperN,00:22:84DesayA&V,00:22:85NomusCom,00:22:86Astron,00:22:87TitanWir,00:22:88Sagrad,00:22:89Vanderla,00:22:8ATeratron,00:22:8BKensingt,00:22:8CPhotonEu,00:22:8DGBSLabor,00:22:8ETv-Numer,00:22:8FCnrs,00:22:90Cisco,00:22:91Cisco,00:22:92Cinetal,00:22:93zte,00:22:94Kyocera,00:22:95SGMTechn,00:22:96LinoWave,00:22:97XMOSSemi,00:22:98Sony,00:22:99SeaMicro,00:22:9ALastar,00:22:9BAverLogi,00:22:9CVerismoN,00:22:9DPyung-Hw,00:22:9ESocialAi,00:22:9FSensysTr,00:22:A0AptivSer,00:22:A1HuaweiSy,00:22:A2XtramusT,00:22:A3Californ,00:22:A42Wire,00:22:A5TexasIns,00:22:A6SonyComp,00:22:A7TycoElec,00:22:A8Ouman,00:22:A9LGElectr,00:22:AANintendo,00:22:ABShenzhen,00:22:ACHangzhou,00:22:ADTelesisT,00:22:AEMattel,00:22:AFSafetyVi,00:22:B0D-Link,00:22:B1Elbit,00:22:B24RFCommu,00:22:B3Sei,00:22:B4ARRISGro,00:22:B5Novita,00:22:B6Superflo,00:22:B7GSSGrund,00:22:B8Norcott,00:22:B9Analogix,00:22:BAHUTHElek,00:22:BBbeyerdyn,00:22:BCJDSUFran,00:22:BDCisco,00:22:BECisco,00:22:BFSieAmpGr,00:22:C0Shenzhen,00:22:C1ActiveSt,00:22:C2ProviewE,00:22:C3ZeeportT,00:22:C4epro,00:22:C5INFORSON,00:22:C6Sutus,00:22:C7SEGGERMi,00:22:C8AppliedI,00:22:C9LenordBa,00:22:CAAnvizBio,00:22:CBIONODES,00:22:CCSciLog,00:22:CDAredTech,00:22:CECiscoSPV,00:22:CFPlanexCo,00:22:D0PolarEle,00:22:D1Albrecht,00:22:D2AllEarth,00:22:D3Hub-Tech,00:22:D4ComWorth,00:22:D5EatonEle,00:22:D6Cypak,00:22:D7Nintendo,00:22:D8Shenzhen,00:22:D9FortexIn,00:22:DAAnatek,00:22:DBTranslog,00:22:DCVigilHea,00:22:DDProtecta,00:22:DEOPPODigi,00:22:DFTAMUZMon,00:22:E0Atlantic,00:22:E1ZORTLabs,00:22:E2WABTECTr,00:22:E3Amerigon,00:22:E4ApassTec,00:22:E5Fisher-R,00:22:E6Intellig,00:22:E7WPSParki,00:22:E8Applitio,00:22:E9ProVisio,00:22:EARustelco,00:22:EBDataResp,00:22:ECIdealbtT,00:22:EDTSIPower,00:22:EEAlgoComm,00:22:EFiWDLTech,00:22:F03GreensA,00:22:F1Private,00:22:F2SunPower,00:22:F3SHARP,00:22:F4AMPAKTec,00:22:F5Advanced,00:22:F6Syracuse,00:22:F7Conceptr,00:22:F8PIMAElec,00:22:F9PollinEl,00:22:FAIntelCor,00:22:FBIntelCor,00:22:FCNokiaDan,00:22:FDNokiaDan,00:22:FEAdvanced,00:22:FFNivis,00:23:00CayeeCom,00:23:01WitronTe,00:23:02CobaltDi,00:23:03LITE-ONI,00:23:04Cisco,00:23:05Cisco,00:23:06Alpsalpi,00:23:07FutureIn,00:23:08Arcadyan,00:23:09JanamTec,00:23:0AARBURG,00:23:0BARRISGro,00:23:0CCloverEl,00:23:0DNortelNe,00:23:0EGorba,00:23:0FHirschEl,00:23:10LNCTechn,00:23:11Gloscom,00:23:12Apple,00:23:13QoolTech,00:23:14IntelCor,00:23:15IntelCor,00:23:16KisanEle,00:23:17Lasercra,00:23:18Toshiba,00:23:19Sielox,00:23:1AITF,00:23:1BDanaherM,00:23:1CFourier,00:23:1DDeltacom,00:23:1ECezzerMu,00:23:1FGuangdaE,00:23:20NiciraNe,00:23:21AvitechI,00:23:22KISSTekn,00:23:23ZylinAS,00:23:24G-ProCom,00:23:25IOLAN,00:23:26Fujitsu,00:23:27ShouyoEl,00:23:28AlconTel,00:23:29DDRdrive,00:23:2AeonasIT-,00:23:2BIrd,00:23:2CSenticar,00:23:2DSandForc,00:23:2EKedahEle,00:23:2FAdvanced,00:23:30Dizipia,00:23:31Nintendo,00:23:32Apple,00:23:33Cisco,00:23:34Cisco,00:23:35Linkflex,00:23:36METELsro,00:23:37GlobalSt,00:23:38OJ-Elect,00:23:39SamsungE,00:23:3ASamsungE,00:23:3BC-Matic,00:23:3CAlflex,00:23:3DLairdTec,00:23:3EAlcatel-,00:23:3FPurechoi,00:23:40MiXTelem,00:23:41Vanderbi,00:23:42CoffeeEq,00:23:43Tem,00:23:44Objectiv,00:23:45Sony,00:23:46Vestac,00:23:47ProCurve,00:23:48Sagemcom,00:23:49Helmholt,00:23:4APrivate,00:23:4BInyuanTe,00:23:4CKtc,00:23:4DHonHaiPr,00:23:4EHonHaiPr,00:23:4FLuminous,00:23:50RDCdbaLy,00:23:512Wire,00:23:52DATASENS,00:23:53FETElett,00:23:54ASUSTekC,00:23:55KincoAut,00:23:56PacketFo,00:23:57Pitronot,00:23:58Systel,00:23:59Benchmar,00:23:5ACompalIn,00:23:5BGulfstre,00:23:5CAprius,00:23:5DCisco,00:23:5ECisco,00:23:5FSiliconM,00:23:60LookitTe,00:23:61Unigen,00:23:62Goldline,00:23:63ZhuhaiRa,00:23:64PowerIns,00:23:65InstaEle,00:23:66BeijingS,00:23:67UniContr,00:23:68ZebraTec,00:23:69Cisco-Li,00:23:6ASmartRG,00:23:6BXembedde,00:23:6CApple,00:23:6DResMed,00:23:6EBurster,00:23:6FDAQSyste,00:23:70Snell,00:23:71SOAMSyst,00:23:72MoreStar,00:23:73GridIron,00:23:74ARRISGro,00:23:75ARRISGro,00:23:76HTC,00:23:77IsotekEl,00:23:78GNNetcom,00:23:79UnionBus,00:23:7ARim,00:23:7BWhdi,00:23:7CNeotion,00:23:7DHewlettP,00:23:7EElster,00:23:7FPlantron,00:23:80Nanoteq,00:23:81LengdaTe,00:23:82LihRonge,00:23:83InMage,00:23:84GGHEngin,00:23:85Antipode,00:23:86IMIHydro,00:23:87ThinkFlo,00:23:88VTTelema,00:23:89Hangzhou,00:23:8ACiena,00:23:8BQuantaCo,00:23:8CPrivate,00:23:8DTechnoDe,00:23:8EADBBroad,00:23:8FNidecCop,00:23:90Algolwar,00:23:91Maxian,00:23:92ProteusI,00:23:93Ajinexte,00:23:94Samjeon,00:23:95ARRISGro,00:23:96AndesTec,00:23:97WestellT,00:23:98Vutlansr,00:23:99SamsungE,00:23:9AEasyData,00:23:9BElsterSo,00:23:9CJuniperN,00:23:9DMapowerE,00:23:9EJiangsuL,00:23:9FInstitut,00:23:A0HanaCNS,00:23:A1TrendEle,00:23:A2ARRISGro,00:23:A3ARRISGro,00:23:A4NewConce,00:23:A5SageTV,00:23:A6E-Mon,00:23:A7RedpineS,00:23:A8Marshall,00:23:A9BeijingD,00:23:AAHFR,00:23:ABCisco,00:23:ACCisco,00:23:ADXmark,00:23:AEDell,00:23:AFARRISGro,00:23:B0COMXIONT,00:23:B1Longchee,00:23:B2Intellig,00:23:B3Lyyn,00:23:B4NokiaDan,00:23:B5Ortana,00:23:B6Securite,00:23:B7Q-Light,00:23:B8SichuanJ,00:23:B9AirbusDe,00:23:BAChroma,00:23:BBAccretec,00:23:BCEQ-SYS,00:23:BDDigitalA,00:23:BECiscoSPV,00:23:BFMainpine,00:23:C0Broadway,00:23:C1Securita,00:23:C2SAMSUNGE,00:23:C3LogMeIn,00:23:C4LuxLumen,00:23:C5Radiatio,00:23:C6SMC,00:23:C7AVSystem,00:23:C8Team-R,00:23:C9SichuanT,00:23:CABehindSe,00:23:CBShenzhen,00:23:CCNintendo,00:23:CDTp-LinkT,00:23:CEKitaDens,00:23:CFCummins-,00:23:D0UnilocUS,00:23:D1Trg,00:23:D2InhandEl,00:23:D3AirLinkW,00:23:D4TexasIns,00:23:D5WAREMAel,00:23:D6SamsungE,00:23:D7SamsungE,00:23:D8Ball-It,00:23:D9BannerEn,00:23:DAIndustri,00:23:DBsaxnet,00:23:DCBenein,00:23:DDElgin,00:23:DEAnsync,00:23:DFApple,00:23:E0INOThera,00:23:E1CavenaIm,00:23:E2SEASigna,00:23:E3Microtro,00:23:E4IPnect,00:23:E5IPaXiomN,00:23:E6Innovati,00:23:E7Hinke,00:23:E8Demco,00:23:E9F5Networ,00:23:EACisco,00:23:EBCisco,00:23:ECAlgorith,00:23:EDARRISGro,00:23:EEARRISGro,00:23:EFZuendSys,00:23:F0Shanghai,00:23:F1Sony,00:23:F2TVLogic,00:23:F3Glocom,00:23:F4Masterna,00:23:F5WiloSe,00:23:F6Softwell,00:23:F7Private,00:23:F8ZyxelCom,00:23:F9Double-T,00:23:FARGNets,00:23:FBIPDatate,00:23:FCUltraSte,00:23:FDAFTAtlas,00:23:FEBiodevic,00:23:FFBeijingH,00:24:00NortelNe,00:24:01D-Link,00:24:02Op-Tecti,00:24:03NokiaDan,00:24:04NokiaDan,00:24:05DilogNor,00:24:06Pointmob,00:24:07TelemSas,00:24:08PacificB,00:24:09Toro,00:24:0AUSBevera,00:24:0BVirtualC,00:24:0CDELEC,00:24:0DOnePathN,00:24:0EInventec,00:24:0FIshiiToo,00:24:10NUETEQTe,00:24:11PharmaSm,00:24:12BenignTe,00:24:13Cisco,00:24:14Cisco,00:24:15Magnetic,00:24:16AnyUse,00:24:17ThomsonT,00:24:18Nextwave,00:24:19Private,00:24:1ARedBeetl,00:24:1BiWOWComm,00:24:1CFuGangEl,00:24:1DGiga-Byt,00:24:1ENintendo,00:24:1FDCT-Delt,00:24:20NetUP,00:24:21Micro-St,00:24:22KnappLog,00:24:23AzureWav,00:24:24AceAxis,00:24:25Shenzhen,00:24:26NohmiBos,00:24:27SsiCompu,00:24:28EnergyIC,00:24:29MkMaster,00:24:2AHittiteM,00:24:2BHonHaiPr,00:24:2CHonHaiPr,00:24:2EDatastri,00:24:2FMicron,00:24:30RubyTech,00:24:31Uni-v,00:24:32NeostarT,00:24:33Alpsalpi,00:24:34Lectroso,00:24:35Wide,00:24:36Apple,00:24:37Motorola,00:24:38BrocadeC,00:24:39DigitalB,00:24:3ALudlElec,00:24:3BCSSISPte,00:24:3CAA,00:24:3DEmersonA,00:24:3FStorwize,00:24:40HaloMoni,00:24:41WanzlMet,00:24:42Axona,00:24:43NortelNe,00:24:44Nintendo,00:24:45Adtran,00:24:46MMBResea,00:24:47Kaztek,00:24:48SpiderCl,00:24:49ShenZhen,00:24:4AVoyantIn,00:24:4BPerceptr,00:24:4CSolartro,00:24:4DHokkaido,00:24:4ERadChips,00:24:4FAsantron,00:24:50Cisco,00:24:51Cisco,00:24:52SiliconS,00:24:53Initrado,00:24:54SamsungE,00:24:55MuLogic,00:24:562Wire,00:24:58PABastio,00:24:59ABBAutom,00:24:5ANanjingP,00:24:5BRaidonTe,00:24:5CDesign-C,00:24:5DTerbergb,00:24:5EHivision,00:24:5FVineTele,00:24:60GiavalSc,00:24:61ShinWang,00:24:62Rayzone,00:24:63Phybridg,00:24:64BridgeTe,00:24:65Elentec,00:24:66Unitron,00:24:67AOCInter,00:24:68Sumavisi,00:24:69Fasttel-,00:24:6ASolidYea,00:24:6BCovia,00:24:6CArubaaHe,00:24:6DWeinzier,00:24:6EPhihongU,00:24:6FOndaComm,00:24:70AUROTECH,00:24:71FusionMu,00:24:72ReDriven,00:24:733ComEuro,00:24:74Autronic,00:24:75CompassS,00:24:76TAPtv,00:24:77TibboTec,00:24:78MagTechE,00:24:79OptecDis,00:24:7AFUYICHEN,00:24:7BActionte,00:24:7CNokiaDan,00:24:7DNokiaDan,00:24:7EUniversa,00:24:7FNortelNe,00:24:80Meteocon,00:24:81HewlettP,00:24:82RuckusWi,00:24:83LGElectr,00:24:84BangandO,00:24:85ConteXtr,00:24:86DesignAr,00:24:87Transact,00:24:88CentreFo,00:24:89Vodafone,00:24:8AKagaElec,00:24:8BHybus,00:24:8CASUSTekC,00:24:8DSonyInte,00:24:8EInfoware,00:24:8FDo-Monix,00:24:90SamsungE,00:24:91SamsungE,00:24:92Motorola,00:24:93ARRISGro,00:24:94Shenzhen,00:24:95ARRISGro,00:24:96Ginzinge,00:24:97Cisco,00:24:98Cisco,00:24:99AquilaTe,00:24:9ABeijingZ,00:24:9BActionSt,00:24:9CBimengCo,00:24:9DNESTechn,00:24:9EADC-Elek,00:24:9FRIMTesti,00:24:A0ARRISGro,00:24:A1ARRISGro,00:24:A2HongKong,00:24:A3SonimTec,00:24:A4SikluCom,00:24:A5Buffalo,00:24:A6TELESTAR,00:24:A7Advanced,00:24:A8ProCurve,00:24:A9LeaderTe,00:24:AADycorTec,00:24:ABA7Engine,00:24:ACHangzhou,00:24:ADAdolfThi,00:24:AEIdemia,00:24:AFDishTech,00:24:B0Esab,00:24:B1CoulombT,00:24:B2Netgear,00:24:B3Graf-Syt,00:24:B4ESCATRON,00:24:B5NortelNe,00:24:B6SeagateT,00:24:B7GridPoin,00:24:B8freealli,00:24:B9WuhanHig,00:24:BATexasIns,00:24:BBCENTRAL,00:24:BCHuRob,00:24:BDHainzlIn,00:24:BESony,00:24:BFCiat,00:24:C0NtiComod,00:24:C1ARRISGro,00:24:C2Asumo,00:24:C3Cisco,00:24:C4Cisco,00:24:C5Meridian,00:24:C6HagerEle,00:24:C7Mobilarm,00:24:C8Broadban,00:24:C9Broadban,00:24:CATobiiTec,00:24:CBAutonetM,00:24:CCFascinat,00:24:CDWillowGa,00:24:CEExeltech,00:24:CFInscapeD,00:24:D0Shenzhen,00:24:D1Thomson,00:24:D2AskeyCom,00:24:D3QUALICA,00:24:D4FreeboxS,00:24:D5WinwardI,00:24:D6IntelCor,00:24:D7IntelCor,00:24:D8IlSungPr,00:24:D9BICOM,00:24:DAInnovar,00:24:DBAlcoholM,00:24:DCJuniperN,00:24:DDCentrak,00:24:DEGLOBALTe,00:24:DFDigitalb,00:24:E0DSTech,00:24:E1ConveyCo,00:24:E2Hasegawa,00:24:E3CAOGroup,00:24:E4Withings,00:24:E5SeerTech,00:24:E6InMotion,00:24:E7PlasterN,00:24:E8Dell,00:24:E9SamsungE,00:24:EAirisinfr,00:24:EBClearPat,00:24:ECUnitedIn,00:24:EDYTElec,00:24:EEWynmax,00:24:EFSony,00:24:F0Seanodes,00:24:F1Shenzhen,00:24:F2Uniphone,00:24:F3Nintendo,00:24:F4Kaminari,00:24:F5NDSSurgi,00:24:F6MiyoshiE,00:24:F7Cisco,00:24:F8Technica,00:24:F9Cisco,00:24:FAHilgeruK,00:24:FBPrivate,00:24:FCQuoPin,00:24:FDAccedian,00:24:FEAVM,00:24:FFQLogic,00:25:00Apple,00:25:01Supertel,00:25:02NaturalP,00:25:03IBM,00:25:04ValiantC,00:25:05eksEngel,00:25:06AIAntita,00:25:07ASTAK,00:25:08MaquetCa,00:25:09SHARETRO,00:25:0ASecurity,00:25:0BCentrofa,00:25:0CSenet,00:25:0DGZTTelko,00:25:0Egtgerman,00:25:0FOn-RampW,00:25:10Pico-Tes,00:25:11Elitegro,00:25:12zte,00:25:13CxpDigit,00:25:14PCWorthI,00:25:15Sfr,00:25:16Integrat,00:25:17Venntis,00:25:18PowerPLU,00:25:19Viaas,00:25:1APsiberDa,00:25:1BPhilipsC,00:25:1CEdt,00:25:1DDSAEncor,00:25:1ERotelTec,00:25:1FZynusVis,00:25:20SMARailw,00:25:21LogitekE,00:25:22ASRockIn,00:25:23OCP,00:25:24Lightcom,00:25:25CTERANet,00:25:26GenuineT,00:25:27Bitrode,00:25:28DaidoSig,00:25:29ComelitG,00:25:2AChengduG,00:25:2BStirling,00:25:2CEntourag,00:25:2DKiryungE,00:25:2ECiscoSPV,00:25:2FEnergy,00:25:30Aetas,00:25:31CloudEng,00:25:32DigitalR,00:25:33Wittenst,00:25:35Minimax,00:25:36OkiElect,00:25:37RuncomTe,00:25:38SamsungE,00:25:39IfTA,00:25:3ACEVA,00:25:3BdinDietm,00:25:3C2Wire,00:25:3DDRSConso,00:25:3ESensusMe,00:25:40QuasarTe,00:25:41MaquetCr,00:25:42Pittasof,00:25:43Moneytec,00:25:44LoJack,00:25:45Cisco,00:25:46Cisco,00:25:47NokiaDan,00:25:48NokiaDan,00:25:49JeorichT,00:25:4ARingCube,00:25:4BApple,00:25:4CVideonCe,00:25:4DSingapor,00:25:4EVertexWi,00:25:4FELETTROL,00:25:50Riverbed,00:25:51SE-Elekt,00:25:52VXi,00:25:53ADBBroad,00:25:54Pixel8Ne,00:25:55VisonicT,00:25:56HonHaiPr,00:25:57BlackBer,00:25:58Mpedia,00:25:59SyphanTe,00:25:5ATantalus,00:25:5BCoachCom,00:25:5CNEC,00:25:5DMornings,00:25:5EShanghai,00:25:5FSenTec,00:25:60IbridgeN,00:25:61ProCurve,00:25:62interbro,00:25:63Luxtera,00:25:64Dell,00:25:65Vizimax,00:25:66SamsungE,00:25:67SamsungE,00:25:68HuaweiTe,00:25:69Sagemcom,00:25:6AinIT-Ins,00:25:6BATENIXEE,00:25:6CAzimutPr,00:25:6DBroadban,00:25:6EVanBreda,00:25:6FDantherm,00:25:70EasternC,00:25:71Zhejiang,00:25:72Nemo-QIn,00:25:73STElectr,00:25:74KUNIMIME,00:25:75FiberPle,00:25:76NeliTech,00:25:77D-BOXTec,00:25:78ConcernS,00:25:79JFLabs,00:25:7ACAMCOPro,00:25:7BStjElect,00:25:7CHuachent,00:25:7DPointRed,00:25:7ENewPosTe,00:25:7FCallTech,00:25:80Equipson,00:25:81x-starne,00:25:82MaksatTe,00:25:83Cisco,00:25:84Cisco,00:25:85KOKUYOS&,00:25:86Tp-LinkT,00:25:87Vitality,00:25:88GenieInd,00:25:89HillsInd,00:25:8APoleZero,00:25:8BMellanox,00:25:8CEsusElek,00:25:8DHaier,00:25:8EWeatherC,00:25:8FTridentM,00:25:90SuperMic,00:25:91NEXTEK,00:25:92Guangzho,00:25:93DatNetIn,00:25:94Eurodesi,00:25:95Northwes,00:25:96GIGAVISI,00:25:97KalkiCom,00:25:98ZhongSha,00:25:99Hedoned,00:25:9ACEStroni,00:25:9BBeijingP,00:25:9CCisco-Li,00:25:9DPrivate,00:25:9EHuaweiTe,00:25:9FTechnoDi,00:25:A0Nintendo,00:25:A1Enalasys,00:25:A2AltaDefi,00:25:A3TrimaxWi,00:25:A4EuroDesi,00:25:A5WalnutMe,00:25:A6CentralN,00:25:A7itron,00:25:A8KontronB,00:25:A9Shanghai,00:25:AABeijingS,00:25:ABAioLcdPc,00:25:ACI-Tech,00:25:ADManufact,00:25:AEMicrosof,00:25:AFCOMFILET,00:25:B0Schmartz,00:25:B1Maya-Cre,00:25:B2MBDADeut,00:25:B3HewlettP,00:25:B4Cisco,00:25:B5Cisco,00:25:B6TelecomF,00:25:B7Costarel,00:25:B8AgileCom,00:25:B9CypressS,00:25:BAAlcatel-,00:25:BBINNERINT,00:25:BCApple,00:25:BDItaldata,00:25:BETektrap,00:25:BFWireless,00:25:C0ZillionT,00:25:C1NawooKor,00:25:C2RingBell,00:25:C321168,00:25:C4RuckusWi,00:25:C5StarLink,00:25:C6kasercor,00:25:C7altek,00:25:C8S-Access,00:25:C9Shenzhen,00:25:CALSResear,00:25:CBReinerSC,00:25:CCMobileCo,00:25:CDSkylaneO,00:25:CEInnerSpa,00:25:CFNokiaDan,00:25:D0NokiaDan,00:25:D1EasternA,00:25:D2InpegVis,00:25:D3AzureWav,00:25:D4GeneralD,00:25:D5Robonica,00:25:D6Kroger,00:25:D7Cedo,00:25:D8KoreaMai,00:25:D9DataFab,00:25:DASecuraKe,00:25:DBATIElect,00:25:DCSumitomo,00:25:DDSunnytek,00:25:DEProbits,00:25:DFPrivate,00:25:E0CeedTecS,00:25:E1Shanghai,00:25:E2Everspri,00:25:E3Hanshini,00:25:E4OMNI-WiF,00:25:E5LGElectr,00:25:E6BelgianM,00:25:E7Sony,00:25:E8IdahoTec,00:25:E9i-mateDe,00:25:EAIphion,00:25:EBReutechR,00:25:ECHumanwar,00:25:EDNuVoTech,00:25:EEAvtex,00:25:EFI-TEC,00:25:F0SugaElec,00:25:F1ARRISGro,00:25:F2ARRISGro,00:25:F3Nordwest,00:25:F4KoCoConn,00:25:F5DVSKorea,00:25:F6netTALKc,00:25:F7AnsaldoS,00:25:F9GMKelect,00:25:FAJ&MAnaly,00:25:FBTunstall,00:25:FCEndaEndu,00:25:FDOBRCentr,00:25:FEPilotEle,00:25:FFCreNovaM,00:26:00TEACAust,00:26:01Cutera,00:26:02SMARTTem,00:26:03Shenzhen,00:26:04AudioPro,00:26:05CC,00:26:06RAUMFELD,00:26:07Enabling,00:26:08Apple,00:26:09Phyllis,00:26:0ACisco,00:26:0BCisco,00:26:0CDataram,00:26:0DMercury,00:26:0EAblaze,00:26:0FLinnProd,00:26:10Apacewav,00:26:11Licera,00:26:12SpaceExp,00:26:13EngelAxi,00:26:14Ktnf,00:26:15Teracom,00:26:16Rosemoun,00:26:17OEMWorld,00:26:18ASUSTekC,00:26:19Frc,00:26:1AFemtocom,00:26:1BLaurelBa,00:26:1CNeovia,00:26:1DCopSecur,00:26:1EQingbang,00:26:1FSAEMagne,00:26:20ISGUS,00:26:21InteliCl,00:26:22CompalIn,00:26:23JRDCommu,00:26:24Thomson,00:26:25MediaSpu,00:26:26Geophysi,00:26:27Truesell,00:26:28companyt,00:26:29JuphoonS,00:26:2AProxense,00:26:2BWongsEle,00:26:2CIKTAdvan,00:26:2DWistron,00:26:2EChengduJ,00:26:2FHamamats,00:26:30AcorelS,00:26:31Commtact,00:26:32Instrume,00:26:33MIR-Medi,00:26:34Infineta,00:26:35Bluetech,00:26:36ARRISGro,00:26:37SamsungE,00:26:38XiaMenJo,00:26:39TMElectr,00:26:3ADigitec,00:26:3BOnbnetec,00:26:3CBachmann,00:26:3DMIA,00:26:3ETrapezeN,00:26:3FLIOSTech,00:26:40BaustemB,00:26:41ARRISGro,00:26:42ARRISGro,00:26:43Alpsalpi,00:26:44ThomsonT,00:26:45Circontr,00:26:46Shenyang,00:26:47WfeTechn,00:26:48Emitech,00:26:4AApple,00:26:4CShanghai,00:26:4DArcadyan,00:26:4Er2p,00:26:4FKrüger&G,00:26:502Wire,00:26:51Cisco,00:26:52Cisco,00:26:53DaySeque,00:26:543Com,00:26:55HewlettP,00:26:56Sansonic,00:26:57NppEkra,00:26:58T-Platfo,00:26:59Nintendo,00:26:5AD-Link,00:26:5BHitronTe,00:26:5CHonHaiPr,00:26:5DSamsungE,00:26:5EHonHaiPr,00:26:5FSamsungE,00:26:60Logiways,00:26:61Irumtek,00:26:62Actionte,00:26:63Shenzhen,00:26:64CoreSyst,00:26:65Protecte,00:26:66EFMNetwo,00:26:67Carecom,00:26:68NokiaDan,00:26:69NokiaDan,00:26:6AEssensiu,00:26:6BShineUni,00:26:6CInventec,00:26:6DMobileAc,00:26:6ENissho-d,00:26:6FCoordiwi,00:26:70CinchCon,00:26:71AUTOVISI,00:26:72AAMPAmer,00:26:73Ricoh,00:26:74HunterDo,00:26:75AztechEl,00:26:76COMMidtA,00:26:77Deif,00:26:78LogicIns,00:26:79Euphonic,00:26:7Awuhanhon,00:26:7BGSIHelmh,00:26:7CMetz-Wer,00:26:7DA-MaxTec,00:26:7EParrot,00:26:7FZenterio,00:26:80SIL3,00:26:81Interspi,00:26:82GemtekTe,00:26:83AjohoEnt,00:26:84KisanSys,00:26:85DigitalI,00:26:86Quantenn,00:26:87corega,00:26:88JuniperN,00:26:89GeneralD,00:26:8ATerrierS,00:26:8BGuangzho,00:26:8CStarLeaf,00:26:8DCellTel,00:26:8EAltaSolu,00:26:8FMTA,00:26:90IDoIt,00:26:91Sagemcom,00:26:92Mitsubis,00:26:93QVidiumT,00:26:94Senscien,00:26:95ZTGroupI,00:26:96NOOLIX,00:26:97AlphaTec,00:26:98Cisco,00:26:99Cisco,00:26:9ACarinaSy,00:26:9BSOKRAT,00:26:9CItusJapa,00:26:9DM2Mnet,00:26:9EQuantaCo,00:26:9FPrivate,00:26:A0moblic,00:26:A1Megger,00:26:A2Instrume,00:26:A3FQIngeni,00:26:A4NovusPro,00:26:A5Microrob,00:26:A6Trixell,00:26:A7Connect,00:26:A8DaehapHy,00:26:A9StrongTe,00:26:AAKenmecMe,00:26:ABSeikoEps,00:26:ACShanghai,00:26:ADArada,00:26:AEWireless,00:26:AFDuelco,00:26:B0Apple,00:26:B1NavisAut,00:26:B2Setrix,00:26:B3ThalesCo,00:26:B4FordMoto,00:26:B5ICOMMTel,00:26:B6AskeyCom,00:26:B7Kingston,00:26:B8Actionte,00:26:B9Dell,00:26:BAARRISGro,00:26:BBApple,00:26:BCGeneralJ,00:26:BDJTECCard,00:26:BESchoonde,00:26:BFShenZhen,00:26:C0EnergyHu,00:26:C1Artray,00:26:C2SCDI,00:26:C3Insighte,00:26:C4Cadmosmi,00:26:C5Guangdon,00:26:C6IntelCor,00:26:C7IntelCor,00:26:C8SystemSe,00:26:C9Proventi,00:26:CACisco,00:26:CBCisco,00:26:CCNokiaDan,00:26:CDPurpleCo,00:26:CEKozumiUS,00:26:CFDekaR&D,00:26:D0Semihalf,00:26:D1SSquared,00:26:D2Pcube,00:26:D3ZenoInfo,00:26:D4IRCA,00:26:D5OrySoluc,00:26:D6NingboAn,00:26:D7KMElecto,00:26:D8MagicPoi,00:26:D9ARRISGro,00:26:DAUniversa,00:26:DBIonicsEM,00:26:DCOpticalD,00:26:DDFivalSci,00:26:DEFdiMatel,00:26:DFTaiDocTe,00:26:E0Asiteq,00:26:E1Stanford,00:26:E2LGElectr,00:26:E3Dti,00:26:E4Canal+,00:26:E5AEGPower,00:26:E6Visionhi,00:26:E7Shanghai,00:26:E8MurataMa,00:26:E9SP,00:26:EACheerchi,00:26:EBAdvanced,00:26:ECLegrandH,00:26:EDzte,00:26:EETKM,00:26:EFTechnolo,00:26:F0cTrixsIn,00:26:F1ProCurve,00:26:F2Netgear,00:26:F3SMCNetwo,00:26:F4Nesslab,00:26:F5XRPLUS,00:26:F6Military,00:26:F7Nivetti,00:26:F8GoldenHi,00:26:F9SEM,00:26:FABandRich,00:26:FBAirDioWi,00:26:FCAcSiPTec,00:26:FDInteract,00:26:FEMKDTechn,00:26:FFBlackBer,00:27:00Shenzhen,00:27:01INCOstar,00:27:02SolarEdg,00:27:03TestechE,00:27:04Accelera,00:27:05Sectroni,00:27:06Yoisys,00:27:07LiftComp,00:27:08NordiagA,00:27:09Nintendo,00:27:0AIee,00:27:0BAduraTec,00:27:0CCisco,00:27:0DCisco,00:27:0EIntelCor,00:27:0FEnvision,00:27:10IntelCor,00:27:11LanPro,00:27:12MaxVisio,00:27:13Universa,00:27:14Grainmus,00:27:15ReboundT,00:27:16Adachi-S,00:27:17CEDigita,00:27:18SuzhouNE,00:27:19Tp-LinkT,00:27:1AGeenovoT,00:27:1BAlecSich,00:27:1CMercury,00:27:1DCombaTel,00:27:1EXagylCom,00:27:1FMIPROEle,00:27:20New-SolC,00:27:21Shenzhen,00:27:22Ubiquiti,00:27:90Cisco,00:27:E3Cisco,00:27:F8BrocadeC,00:28:9FSemptian,00:28:F8IntelCor,00:29:26AppliedO,00:29:C2Cisco,00:2A:10Cisco,00:2A:6ACisco,00:2A:AFLARsys-A,00:2B:67LCFCHeFe,00:2C:C8Cisco,00:2D:76TITECH,00:2D:B3AMPAKTec,00:2E:C7HuaweiTe,00:2F:5CCisco,00:2F:D9Fiberhom,00:30:00AllwellT,00:30:01Smp,00:30:02ExpandNe,00:30:03Phasys,00:30:04LeadtekR,00:30:05FujitsuS,00:30:06Superpow,00:30:07Opti,00:30:08AvioDigi,00:30:09TachionN,00:30:0AAztechEl,00:30:0BmPHASETe,00:30:0CCongruen,00:30:0DMMCTechn,00:30:0EKlotzDig,00:30:0FIMT-Info,00:30:10Visionet,00:30:11HMSIndus,00:30:12DigitalE,00:30:13NEC,00:30:14Divio,00:30:15CpClare,00:30:16Ishida,00:30:17BlueArcU,00:30:18JetwayIn,00:30:19Cisco,00:30:1ASmartbri,00:30:1BShuttle,00:30:1CAltvater,00:30:1DSkystrea,00:30:1E3ComEuro,00:30:1FOpticalN,00:30:20TSI,00:30:21HsingTec,00:30:22FongKaiI,00:30:23CogentCo,00:30:24Cisco,00:30:25Checkout,00:30:26HeiTelDi,00:30:27Kerbango,00:30:28FASESald,00:30:29Opicom,00:30:2ASouthern,00:30:2BInalpNet,00:30:2CSylantro,00:30:2DQuantumB,00:30:2EHoftWess,00:30:2FGEAviati,00:30:30Harmonix,00:30:31Lightwav,00:30:32MagicRam,00:30:33OrientTe,00:30:34SetEngin,00:30:35Corning,00:30:36RmpElekt,00:30:37PackardB,00:30:38Xcp,00:30:39Softbook,00:30:3AMaatel,00:30:3BPowerCom,00:30:3COnnto,00:30:3DIva,00:30:3ERadcom,00:30:3FTurboCom,00:30:40Cisco,00:30:41SaejinTM,00:30:42DeTeWe-D,00:30:43IdreamTe,00:30:44CradlePo,00:30:45VillageN,00:30:46Controll,00:30:47NisseiEl,00:30:48SuperMic,00:30:49BryantTe,00:30:4AFraunhof,00:30:4BOrbacom,00:30:4CAppianCo,00:30:4DEsi,00:30:4EBustecPr,00:30:4FPLANETTe,00:30:50VersaTec,00:30:51OrbitAvi,00:30:52ElasticN,00:30:53Basler,00:30:54Castlene,00:30:55RenesasT,00:30:56HMSIndus,00:30:57QTelNet,00:30:58ApiMotio,00:30:59KontronE,00:30:5ATelgen,00:30:5BToko,00:30:5CSMARLabo,00:30:5DDigitra,00:30:5EAbelkoIn,00:30:5FHasselbl,00:30:60Powerfil,00:30:61MobyTEL,00:30:62IPVideoN,00:30:63Santera,00:30:64AdlinkTe,00:30:65Apple,00:30:66Rfm,00:30:67BIOSTARM,00:30:68Cybernet,00:30:69ImpacctT,00:30:6APentaMed,00:30:6BCmos,00:30:6CHitex,00:30:6DLucentTe,00:30:6EHewlettP,00:30:6FSeyeonTe,00:30:701Net,00:30:71Cisco,00:30:72Intellib,00:30:73Internat,00:30:74Equiinet,00:30:75Adtech,00:30:76Akamba,00:30:77OnpremNe,00:30:78Cisco,00:30:79Cqos,00:30:7AAdvanced,00:30:7BCisco,00:30:7CAdid,00:30:7DGreAmeri,00:30:7ERedflexC,00:30:7FIrlan,00:30:80Cisco,00:30:81AltosC&C,00:30:82TaihanEl,00:30:83Ivron,00:30:84AlliedTe,00:30:85Cisco,00:30:86Transist,00:30:87VegaGrie,00:30:88Ericsson,00:30:89Spectrap,00:30:8ANicotraS,00:30:8BBrixNetw,00:30:8CQuantum,00:30:8DPinnacle,00:30:8ECrossMat,00:30:8FMICRILOR,00:30:90CyraTech,00:30:91TaiwanFi,00:30:92KontronE,00:30:93SonnetTe,00:30:94Cisco,00:30:95ProcompI,00:30:96Cisco,00:30:97Regin,00:30:98GlobalCo,00:30:99BoenigUn,00:30:9AAstroTer,00:30:9BSmartwar,00:30:9CTimingAp,00:30:9DNimbleMi,00:30:9EWorkbit,00:30:9FAmberNet,00:30:A0TycoSubm,00:30:A1WEBGATE,00:30:A2Lightner,00:30:A3Cisco,00:30:A4Woodwind,00:30:A5ActivePo,00:30:A6VianetTe,00:30:A7Schweitz,00:30:A8OlECommu,00:30:A9Netivers,00:30:AAAxusMicr,00:30:ABDeltaNet,00:30:ACSystemeL,00:30:ADShanghai,00:30:AETimesNSy,00:30:AFHoneywel,00:30:B0Converge,00:30:B1TrunkNet,00:30:B2L-3Sonom,00:30:B3SanValle,00:30:B4Intersil,00:30:B5TadiranM,00:30:B6Cisco,00:30:B7Teletrol,00:30:B8RiverDel,00:30:B9Ectel,00:30:BAAc&TSyst,00:30:BBCacheFlo,00:30:BCOptronic,00:30:BDBelkinCo,00:30:BECity-Net,00:30:BFMultidat,00:30:C0LaraTech,00:30:C1HewlettP,00:30:C2Comone,00:30:C3Flueckig,00:30:C4CanonIma,00:30:C5CadenceD,00:30:C6ControlS,00:30:C7Macromat,00:30:C8GadLine,00:30:C9LuxNN,00:30:CADiscover,00:30:CBOmniFlow,00:30:CCTenorNet,00:30:CDConexant,00:30:CEZaffire,00:30:CFTwoTechn,00:30:D0Tellabs,00:30:D1Inova,00:30:D2WinTechn,00:30:D3AgilentT,00:30:D4AAE,00:30:D5DResearc,00:30:D6MscVertr,00:30:D7Innovati,00:30:D8Sitek,00:30:D9Datacore,00:30:DAComtrend,00:30:DBMindread,00:30:DCRightech,00:30:DDIndigita,00:30:DEWAGOKont,00:30:DFKbTelTel,00:30:E0OxfordSe,00:30:E1NetworkE,00:30:E2Garnet,00:30:E3SedonaNe,00:30:E4ChiyodaS,00:30:E5AmperDat,00:30:E6DraegerM,00:30:E7CnfMobil,00:30:E8Ensim,00:30:E9GmaCommu,00:30:EATeraForc,00:30:EBTurbonet,00:30:ECBorgardt,00:30:EDExpertMa,00:30:EEDSGTechn,00:30:EFNeonTech,00:30:F0UniformI,00:30:F1AcctonTe,00:30:F2Cisco,00:30:F3AtWorkCo,00:30:F4StardotT,00:30:F5WildLab,00:30:F6Securelo,00:30:F7Ramix,00:30:F8Dynapro,00:30:F9Sollae,00:30:FATelica,00:30:FBAZSTechn,00:30:FCTerawave,00:30:FDIntegrat,00:30:FEDSA,00:30:FFDataFab,00:31:46JuniperN,00:31:92TP-Link,00:32:17Cisco,00:32:3Aso-logic,00:33:6CSynapSen,00:34:A1Rf-Lambd,00:34:DALGElectr,00:34:F1RadicomR,00:34:FEHuaweiTe,00:35:1ACisco,00:35:32Electro-,00:35:60RosenAvi,00:35:FFTexasIns,00:36:76ARRISGro,00:36:BENorthwes,00:36:F8ContiTem,00:36:FESuperVis,00:37:6DMurataMa,00:37:B7Sagemcom,00:38:DFCisco,00:3A:7DCisco,00:3A:98Cisco,00:3A:99Cisco,00:3A:9ACisco,00:3A:9BCisco,00:3A:9CCisco,00:3A:9DNECPlatf,00:3A:AFBlueBit,00:3C:10Cisco,00:3C:84SiliconL,00:3C:C5WONWOOEn,00:3D:41Hattelan,00:3D:E1HuaweiDe,00:3D:E8LGElectr,00:3E:E1Apple,00:40:00PciCompo,00:40:01ZeroOneT,00:40:02Perle,00:40:03EmersonP,00:40:04Icm,00:40:05AniCommu,00:40:06SampoTec,00:40:07TelmatIn,00:40:08APlusInf,00:40:09Tachiban,00:40:0APivotalT,00:40:0BCisco,00:40:0CGeneralM,00:40:0DLannetDa,00:40:0EMemotec,00:40:0FDatacomT,00:40:10SonicMac,00:40:11AndoverC,00:40:12Windata,00:40:13NttDataC,00:40:14Comsoft,00:40:15AscomInf,00:40:16ADC-Glob,00:40:17SilexTec,00:40:18Adobe,00:40:19Aeon,00:40:1AFujiElec,00:40:1BPrinter,00:40:1CAstPenti,00:40:1DInvisibl,00:40:1EIcc,00:40:1FColorgra,00:40:20CommScop,00:40:21RasterGr,00:40:22KleverCo,00:40:23Logic,00:40:24Compac,00:40:25Molecula,00:40:26Buffalo,00:40:27SmcMassa,00:40:28Netcomm,00:40:29Compex,00:40:2ACanoga-P,00:40:2BTrigemCo,00:40:2CIsisDist,00:40:2DHarrisAd,00:40:2EPrecisio,00:40:2FXlntDesi,00:40:30GkComput,00:40:31KokusaiE,00:40:32DigitalC,00:40:33AddtronT,00:40:34Bustek,00:40:35Opcom,00:40:36Minim,00:40:37Sea-Ilan,00:40:38TalentEl,00:40:39OptecDai,00:40:3AImpactTe,00:40:3BSynerjet,00:40:3CForks,00:40:3DTeradata,00:40:3ERasterOp,00:40:3FSsangyon,00:40:40RingAcce,00:40:41Fujikura,00:40:42NAT,00:40:43NokiaDat,00:40:44QnixComp,00:40:45Twinhead,00:40:46UdcResea,00:40:47WindRive,00:40:48SmdInfor,00:40:49RocheDia,00:40:4AWestAust,00:40:4BMapleCom,00:40:4CHypertec,00:40:4DTelecomm,00:40:4EFluent,00:40:4FSpaceNav,00:40:50Ironics,00:40:51Garbeean,00:40:52StarTech,00:40:53AmproCom,00:40:54Thinking,00:40:55Metronix,00:40:56McmJapan,00:40:57Lockheed,00:40:58Kronos,00:40:59YoshidaK,00:40:5AGoldstar,00:40:5BFunasset,00:40:5CFuture,00:40:5DStar-Tek,00:40:5ENorthHil,00:40:5FAfeCompu,00:40:60Comendec,00:40:61Datatech,00:40:62EGarland,00:40:63ViaTechn,00:40:64KlaInstr,00:40:65GteSpace,00:40:66APRESIA,00:40:67Omnibyte,00:40:68Extended,00:40:69Lemcom,00:40:6AKentekIn,00:40:6BSysgen,00:40:6CCoperniq,00:40:6DLanco,00:40:6ECorollar,00:40:6FSyncRese,00:40:70Interwar,00:40:71AtmCompu,00:40:72AppliedI,00:40:73BassAsso,00:40:74CableAnd,00:40:75Tattile,00:40:76SunConve,00:40:77MaxtonTe,00:40:78WearnesA,00:40:79JukoManu,00:40:7ASocieteD,00:40:7BScientif,00:40:7CQume,00:40:7DExtensio,00:40:7EEvergree,00:40:7FFLIR,00:40:80Athenix,00:40:81Mannesma,00:40:82Laborato,00:40:83TdaIndus,00:40:84Honeywel,00:40:85SaabInst,00:40:86MichelsK,00:40:87Ubitrex,00:40:88MobuisNu,00:40:89Meidensh,00:40:8ATpsTelep,00:40:8BRaylan,00:40:8CAxisComm,00:40:8DGoodyear,00:40:8ETattile,00:40:8FWm-DataM,00:40:90AnselCom,00:40:91ProcompI,00:40:92AspCompu,00:40:93PaxdataN,00:40:94Shograph,00:40:95RPTInter,00:40:96Cisco,00:40:97DatexDiv,00:40:98Dressler,00:40:99Newgen,00:40:9ANetworkE,00:40:9BHalCompu,00:40:9CTranswar,00:40:9DDigiboar,00:40:9EConcurre,00:40:9FTelco,00:40:A0Goldstar,00:40:A1ErgoComp,00:40:A2Kingstar,00:40:A3Microuni,00:40:A4RoseElec,00:40:A5Clinicom,00:40:A6CrayRese,00:40:A7ItautecP,00:40:A8ImfInter,00:40:A9Datacom,00:40:AAValmetAu,00:40:ABRolandDg,00:40:ACSuperWor,00:40:ADSmaRegel,00:40:AEDeltaCon,00:40:AFDigitalP,00:40:B0BytexEng,00:40:B1Codonics,00:40:B2Systemfo,00:40:B3ParTech,00:40:B4Nextcom,00:40:B5VideoTec,00:40:B6Computer,00:40:B7StealthC,00:40:B8IdeaAsso,00:40:B9MacqElec,00:40:BAAlliantC,00:40:BBGoldstar,00:40:BCAlgorith,00:40:BDStarligh,00:40:BEBoeingDe,00:40:BFChannelI,00:40:C0VistaCon,00:40:C1Bizerba-,00:40:C2AppliedC,00:40:C3FischerA,00:40:C4KinkeiSy,00:40:C5MicomCom,00:40:C6Fibernet,00:40:C7RubyTech,00:40:C8MilanTec,00:40:C9Ncube,00:40:CAFirstInt,00:40:CBLanwanTe,00:40:CCSilcomMa,00:40:CDTeraMicr,00:40:CENet-Sour,00:40:CFStrawber,00:40:D0MitacInt,00:40:D1FukudaDe,00:40:D2Pagine,00:40:D3Kimpsion,00:40:D4GageTalk,00:40:D5Sartoriu,00:40:D6Locamati,00:40:D7StudioGe,00:40:D8OceanOff,00:40:D9American,00:40:DATelspec,00:40:DBAdvanced,00:40:DCTritecEl,00:40:DDHongTech,00:40:DEElsagDat,00:40:DFDigalog,00:40:E0Atomwide,00:40:E1MarnerIn,00:40:E2MesaRidg,00:40:E3Quin,00:40:E4E-MTechn,00:40:E5Sybus,00:40:E6CAEN,00:40:E7ArnosIns,00:40:E8CharlesR,00:40:E9Accord,00:40:EAPlaintre,00:40:EBMartinMa,00:40:ECMikasaSy,00:40:EDNetworkC,00:40:EEOptimem,00:40:EFHypercom,00:40:F0Micro,00:40:F1ChuoElec,00:40:F2JanichKl,00:40:F3Netcor,00:40:F4CameoCom,00:40:F5OemEngin,00:40:F6KatronCo,00:40:F7Polaroid,00:40:F8Systemha,00:40:F9Combinet,00:40:FAMicroboa,00:40:FBCascadeC,00:40:FCIbrCompu,00:40:FDLxe,00:40:FESymplexC,00:40:FFTelebitC,00:41:B4WuxiZhon,00:41:D2Cisco,00:42:38IntelCor,00:42:52RLXTechn,00:42:5ACisco,00:42:68Cisco,00:42:79SunitecE,00:43:FFKetron,00:45:01MidmarkR,00:45:1DCisco,00:45:E2CyberTAN,00:46:4BHuaweiTe,00:48:54DigitalS,00:4A:77zte,00:4B:F3Shenzhen,00:4D:32AndonHea,00:4E:01Dell,00:4E:35HewlettP,00:4F:1AHuaweiTe,00:4F:49Realtek,00:4F:4BPineTech,00:50:00NexoComm,00:50:01Yamashit,00:50:02Omnisec,00:50:03Xrite,00:50:043com3c90,00:50:06Tac,00:50:07SiemensT,00:50:08TivaMicr,00:50:09PhilipsB,00:50:0AIrisTech,00:50:0BCisco,00:50:0Ce-TekLab,00:50:0DSatoriEl,00:50:0EChromati,00:50:0FCisco,00:50:10NovaNETL,00:50:12Cbl-,00:50:13SeagateC,00:50:14Cisco,00:50:15BrightSt,00:50:16MolexCan,00:50:17Rsr,00:50:18AMIT,00:50:19SpringTi,00:50:1AIQinVisi,00:50:1BAblCanad,00:50:1CJatom,00:50:1EGrassVal,00:50:1FMrg,00:50:20Mediasta,00:50:21EisInter,00:50:22ZonetTec,00:50:23PgDesign,00:50:24Navic,00:50:26Cosystem,00:50:27Genicom,00:50:28AvalComm,00:50:291394Prin,00:50:2ACisco,00:50:2BGenrad,00:50:2CSoyoComp,00:50:2DAccel,00:50:2ECambex,00:50:2FTollBrid,00:50:30FuturePl,00:50:31Aeroflex,00:50:32PicazoCo,00:50:33MayanNet,00:50:36Netcam,00:50:37KogaElec,00:50:38DainTele,00:50:39MarinerN,00:50:3ADatongEl,00:50:3BMediafir,00:50:3CTsinghua,00:50:3ECisco,00:50:3FAnchorGa,00:50:40Panasoni,00:50:41Coretron,00:50:42SciManuf,00:50:43MarvellS,00:50:44Asaca,00:50:45Rioworks,00:50:46MenicxIn,00:50:47Private,00:50:48Infolibr,00:50:49ArborNet,00:50:4AElteco,00:50:4BBarconet,00:50:4CGalilMot,00:50:4DTokyoEle,00:50:4ESierraMo,00:50:4FOlencomE,00:50:50Cisco,00:50:51IwatsuEl,00:50:52TiaraNet,00:50:53Cisco,00:50:54Cisco,00:50:55Doms,00:50:56VMware,00:50:57Broadban,00:50:58SangomaT,00:50:59iBAHN,00:50:5ANetworkA,00:50:5BKawasaki,00:50:5CTundo,00:50:5EDigitekM,00:50:5FBrandInn,00:50:60Tandberg,00:50:62KouwellE,00:50:63ComselSy,00:50:64CaeElect,00:50:65TDK-Lamb,00:50:66AtecoMad,00:50:67Aerocomm,00:50:68Electron,00:50:69PixStrea,00:50:6AEdeva,00:50:6BSpx-Ateg,00:50:6CBeijerEl,00:50:6DVideojet,00:50:6ECorderEn,00:50:6FG-Connec,00:50:70Chaintec,00:50:71Aiwa,00:50:72Corvis,00:50:73Cisco,00:50:74Advanced,00:50:75KestrelS,00:50:76IBM,00:50:77Prolific,00:50:78MegatonH,00:50:79Private,00:50:7AXpeed,00:50:7BMerlotCo,00:50:7CVideocon,00:50:7DIfp,00:50:7ENewerTec,00:50:7FDrayTek,00:50:80Cisco,00:50:81MurataMa,00:50:82Foresson,00:50:83Gilbarco,00:50:84Quantum,00:50:86Telkom,00:50:87Terasaki,00:50:88Amano,00:50:89SafetyMa,00:50:8BHewlettP,00:50:8CRsi,00:50:8DAbitComp,00:50:8EOptimati,00:50:8FAsitaTec,00:50:90Dctri,00:50:91Netacces,00:50:92RigakuOs,00:50:93Boeing,00:50:94ARRISGro,00:50:95PeracomN,00:50:96SalixTec,00:50:97MMC-EMBE,00:50:98Globaloo,00:50:993ComEuro,00:50:9ATagElect,00:50:9BSwitchco,00:50:9CBetaRese,00:50:9DIndustre,00:50:9ELesTechn,00:50:9FHorizonC,00:50:A0DeltaCom,00:50:A1CarloGav,00:50:A2Cisco,00:50:A3TransMed,00:50:A4IoTech,00:50:A5CapitolB,00:50:A6Optronic,00:50:A7Cisco,00:50:A8OpenCon,00:50:A9MoldatWi,00:50:AAKonicaMi,00:50:ABNALTEC,00:50:ACMapleCom,00:50:ADCommUniq,00:50:AEFDK,00:50:AFIntergon,00:50:B0Technolo,00:50:B1Giddings,00:50:B2BRODEL,00:50:B3Voiceboa,00:50:B4Satchwel,00:50:B5FichetSe,00:50:B6GoodWayI,00:50:B7BoserTec,00:50:B8InovaCom,00:50:B9XitronTe,00:50:BAD-Link,00:50:BBCmsTechn,00:50:BCHammerSt,00:50:BDCisco,00:50:BEFastMult,00:50:BFMetallig,00:50:C0Gatan,00:50:C1GemflexN,00:50:C2IEEERegi,00:50:C2:00:00:00/36TLS,00:50:C2:00:10:00/36JMBSDeve,00:50:C2:00:20:00/36Integrat,00:50:C2:00:30:00/36Microsof,00:50:C2:00:40:00/36SCITechn,00:50:C2:00:50:00/36GDCalifo,00:50:C2:00:60:00/36ProjectM,00:50:C2:00:70:00/36CliveGre,00:50:C2:00:80:00/36Portable,00:50:C2:00:90:00/36Datakine,00:50:C2:00:A0:00/36Tharsys,00:50:C2:00:B0:00/36IO,00:50:C2:00:C0:00/36Vbrick,00:50:C2:00:D0:00/36OpusTele,00:50:C2:00:E0:00/36TTTech,00:50:C2:00:F0:00/36XLN-t,00:50:C2:01:00:00/36Moisture,00:50:C2:01:10:00/36Bihl+Wie,00:50:C2:01:20:00/36FlowareS,00:50:C2:01:30:00/36SensysTe,00:50:C2:01:40:00/36Canal+,00:50:C2:01:50:00/36LeroyAut,00:50:C2:01:60:00/36DSPDesig,00:50:C2:01:70:00/36HunterTe,00:50:C2:01:80:00/36CAD-UL,00:50:C2:01:90:00/36EmtacTec,00:50:C2:01:A0:00/36SkylakeT,00:50:C2:01:B0:00/36CrossPro,00:50:C2:01:C0:00/36TadiranS,00:50:C2:01:D0:00/36Princeto,00:50:C2:01:E0:00/36CallTech,00:50:C2:01:F0:00/36KBSIndus,00:50:C2:02:00:00/36IconRese,00:50:C2:02:10:00/36DRSTechn,00:50:C2:02:20:00/36AshlingM,00:50:C2:02:30:00/36Zabacom,00:50:C2:02:40:00/36Ipitek,00:50:C2:02:50:00/36TeracomT,00:50:C2:02:60:00/36Abatis,00:50:C2:02:70:00/36Industri,00:50:C2:02:80:00/36Frensch,00:50:C2:02:90:00/36Grossenb,00:50:C2:02:A0:00/36VersaLog,00:50:C2:02:B0:00/36NovaEngi,00:50:C2:02:C0:00/36Narrowba,00:50:C2:02:D0:00/36Innocor,00:50:C2:02:E0:00/36TurtleMo,00:50:C2:02:F0:00/36Sinetica,00:50:C2:03:00:00/36Lockheed,00:50:C2:03:10:00/36Eloquenc,00:50:C2:03:20:00/36MotionIO,00:50:C2:03:30:00/36DobleEng,00:50:C2:03:40:00/36IngBuero,00:50:C2:03:50:00/36AlliantT,00:50:C2:03:60:00/36Arcturus,00:50:C2:03:70:00/36EISM,00:50:C2:03:80:00/36EtheiraT,00:50:C2:03:90:00/36ApexSign,00:50:C2:03:A0:00/36PLLBElet,00:50:C2:03:B0:00/36VNRElect,00:50:C2:03:C0:00/36BrainBox,00:50:C2:03:D0:00/36ISDNGate,00:50:C2:03:E0:00/36MSUUK,00:50:C2:03:F0:00/36Celotek,00:50:C2:04:00:00/36MiSPO,00:50:C2:04:10:00/36DamlerCh,00:50:C2:04:20:00/36BEARSolu,00:50:C2:04:30:00/36Curtis,00:50:C2:04:40:00/36Private,00:50:C2:04:50:00/36ChaseMan,00:50:C2:04:60:00/36Private,00:50:C2:04:70:00/36BRElectr,00:50:C2:04:80:00/36Cybectec,00:50:C2:04:90:00/36Computer,00:50:C2:04:A0:00/36TelecomA,00:50:C2:04:B0:00/36TecstarD,00:50:C2:04:C0:00/36NewStand,00:50:C2:04:E0:00/36Industri,00:50:C2:04:F0:00/36Luma,00:50:C2:05:00:00/36Dataprob,00:50:C2:05:10:00/36JSRUltra,00:50:C2:05:20:00/36MayoFoun,00:50:C2:05:40:00/36Optionex,00:50:C2:05:50:00/36SanCastl,00:50:C2:05:60:00/36Base2,00:50:C2:05:70:00/36LiteF,00:50:C2:05:80:00/36VisionRe,00:50:C2:05:90:00/36AustcoMa,00:50:C2:05:A0:00/36Sonifex,00:50:C2:05:B0:00/36Radiomet,00:50:C2:05:C0:00/36NortelNe,00:50:C2:05:D0:00/36IgnitusC,00:50:C2:05:E0:00/36DIVA,00:50:C2:05:F0:00/36MaldenEl,00:50:C2:06:00:00/36Private,00:50:C2:06:10:00/36SimpleNe,00:50:C2:06:20:00/36Private,00:50:C2:06:30:00/36Ticketma,00:50:C2:06:40:00/36Private,00:50:C2:06:50:00/36CleverDe,00:50:C2:06:60:00/36Private,00:50:C2:06:70:00/36Riverlin,00:50:C2:06:80:00/36Seabridg,00:50:C2:06:90:00/36ECElettr,00:50:C2:06:A0:00/36Unimark,00:50:C2:06:B0:00/36NCast,00:50:C2:06:C0:00/36WaveComE,00:50:C2:06:D0:00/36Advanced,00:50:C2:06:E0:00/36AvtronMa,00:50:C2:06:F0:00/36DigitalS,00:50:C2:07:00:00/36Katchall,00:50:C2:07:10:00/36NetVisio,00:50:C2:07:20:00/36Neuberge,00:50:C2:07:30:00/36AlstomSi,00:50:C2:07:40:00/36EdgeTech,00:50:C2:07:50:00/36ENTTEC,00:50:C2:07:60:00/36LittonGu,00:50:C2:07:70:00/36SacoSmar,00:50:C2:07:80:00/36Reselec,00:50:C2:07:90:00/36Flextel,00:50:C2:07:A0:00/36RadioTel,00:50:C2:07:B0:00/36TrikonTe,00:50:C2:07:C0:00/36PLLBelet,00:50:C2:07:D0:00/36CaspianN,00:50:C2:07:E0:00/36JL-tekni,00:50:C2:07:F0:00/36Dunti,00:50:C2:08:00:00/36Aim,00:50:C2:08:10:00/36Matusche,00:50:C2:08:20:00/36GFIChron,00:50:C2:08:30:00/36ard,00:50:C2:08:40:00/36DIALOG4S,00:50:C2:08:50:00/36Crosspor,00:50:C2:08:60:00/36Validyne,00:50:C2:08:70:00/36MonitorB,00:50:C2:08:80:00/36TELINC,00:50:C2:08:90:00/36FenwalIt,00:50:C2:08:A0:00/36RisingEd,00:50:C2:08:B0:00/36HYPERCHI,00:50:C2:08:C0:00/36IPUnity,00:50:C2:08:D0:00/36KylinkCo,00:50:C2:08:E0:00/36Bsquare,00:50:C2:08:F0:00/36GeneralI,00:50:C2:09:00:00/36Invensys,00:50:C2:09:10:00/36StorLogi,00:50:C2:09:20:00/36DigitAll,00:50:C2:09:30:00/36Korealin,00:50:C2:09:40:00/36Analytic,00:50:C2:09:50:00/36Seatech,00:50:C2:09:60:00/36UtronixE,00:50:C2:09:70:00/36IMVInver,00:50:C2:09:80:00/36EPELIndu,00:50:C2:09:90:00/36CaseInfo,00:50:C2:09:A0:00/36NBODevel,00:50:C2:09:B0:00/36SeffleIn,00:50:C2:09:C0:00/36RFApplic,00:50:C2:09:D0:00/36Zelpos,00:50:C2:09:E0:00/36Infinite,00:50:C2:09:F0:00/36MetaWave,00:50:C2:0A:00:00/36Cynaps,00:50:C2:0A:10:00/36VisableG,00:50:C2:0A:20:00/36JägerCom,00:50:C2:0A:30:00/36BaSyTec,00:50:C2:0A:40:00/36Bounty,00:50:C2:0A:50:00/36Mobiltex,00:50:C2:0A:60:00/36Arula,00:50:C2:0A:70:00/36WaterCov,00:50:C2:0A:80:00/36KaveriNe,00:50:C2:0A:90:00/36RadiantN,00:50:C2:0A:A0:00/36Log-In,00:50:C2:0A:B0:00/36Fastware,00:50:C2:0A:C0:00/36Honeywel,00:50:C2:0A:D0:00/36BMCMesss,00:50:C2:0A:E0:00/36Zarak,00:50:C2:0A:F0:00/36LatusLig,00:50:C2:0B:00:00/36LMITechn,00:50:C2:0B:10:00/36BeelineN,00:50:C2:0B:20:00/36RFMicroD,00:50:C2:0B:30:00/36SMX,00:50:C2:0B:40:00/36Wavefly,00:50:C2:0B:50:00/36ExtremeC,00:50:C2:0B:60:00/36ApSecure,00:50:C2:0B:70:00/36Rymic,00:50:C2:0B:80:00/36LANContr,00:50:C2:0B:90:00/36HelmutMa,00:50:C2:0B:A0:00/36Pro-Acti,00:50:C2:0B:B0:00/36MAZet,00:50:C2:0B:C0:00/36Infolink,00:50:C2:0B:D0:00/36Tattile,00:50:C2:0B:E0:00/36StellaEl,00:50:C2:0B:F0:00/36Private,00:50:C2:0C:00:00/36Imigix,00:50:C2:0C:10:00/36Casabyte,00:50:C2:0C:20:00/36AlchemyS,00:50:C2:0C:30:00/36Tonbu,00:50:C2:0C:40:00/36InterEpo,00:50:C2:0C:50:00/36SAIABurg,00:50:C2:0C:60:00/36Advanced,00:50:C2:0C:70:00/36TransCom,00:50:C2:0C:80:00/36Trane,00:50:C2:0C:90:00/36DSSNetwo,00:50:C2:0C:A0:00/36JDRichar,00:50:C2:0C:B0:00/36Studiel,00:50:C2:0C:C0:00/36AlphaMed,00:50:C2:0C:D0:00/36Linet,00:50:C2:0C:E0:00/36RFLElect,00:50:C2:0C:F0:00/36Pcsc,00:50:C2:0D:00:00/36Telefran,00:50:C2:0D:10:00/36Renaissa,00:50:C2:0D:20:00/36RealWorl,00:50:C2:0D:30:00/36LakeTech,00:50:C2:0D:40:00/36Palm,00:50:C2:0D:50:00/36Zelax,00:50:C2:0D:60:00/36IncoStar,00:50:C2:0D:70:00/36SummitAv,00:50:C2:0D:80:00/36Charlott,00:50:C2:0D:90:00/36LoeweOpt,00:50:C2:0D:A0:00/36MotionAn,00:50:C2:0D:B0:00/36Cyberex,00:50:C2:0D:C0:00/36Elbit,00:50:C2:0D:D0:00/36Interisa,00:50:C2:0D:E0:00/36Frederic,00:50:C2:0D:F0:00/36Innovati,00:50:C2:0E:00:00/36EMAC,00:50:C2:0E:10:00/36Inspirat,00:50:C2:0E:20:00/36VisualCi,00:50:C2:0E:30:00/36Lanex,00:50:C2:0E:40:00/36CollaboT,00:50:C2:0E:50:00/36Clearwat,00:50:C2:0E:60:00/36RouteFre,00:50:C2:0E:70:00/36CenturyG,00:50:C2:0E:80:00/36AudioDes,00:50:C2:0E:90:00/36Smartmed,00:50:C2:0E:A0:00/36iReady,00:50:C2:0E:B0:00/36iREZTech,00:50:C2:0E:C0:00/36KeithKoe,00:50:C2:0E:D0:00/36ValleyPr,00:50:C2:0E:E0:00/36Industri,00:50:C2:0E:F0:00/36MovazNet,00:50:C2:0F:00:00/36VHBTechn,00:50:C2:0F:10:00/36Steelcas,00:50:C2:0F:20:00/36KMS,00:50:C2:0F:30:00/36YoungCom,00:50:C2:0F:40:00/36Sysnet,00:50:C2:0F:50:00/36SpectraT,00:50:C2:0F:60:00/36CarlBaas,00:50:C2:0F:70:00/36FossNIRS,00:50:C2:0F:80:00/36TecnintH,00:50:C2:0F:90:00/36RavenInd,00:50:C2:0F:A0:00/36GETransp,00:50:C2:0F:B0:00/36PIUSYS,00:50:C2:0F:C0:00/36KimmonMa,00:50:C2:0F:D0:00/36Inducomp,00:50:C2:0F:E0:00/36EnergyIC,00:50:C2:0F:F0:00/36IPAXS,00:50:C2:10:00:00/36Corelatu,00:50:C2:10:10:00/36LAUDElec,00:50:C2:10:20:00/36MillionT,00:50:C2:10:30:00/36GreenHil,00:50:C2:10:40:00/36Adescom,00:50:C2:10:50:00/36Lumentis,00:50:C2:10:60:00/36Matsuoka,00:50:C2:10:70:00/36NewHer,00:50:C2:10:80:00/36Balogh,00:50:C2:10:90:00/36ITKDrKas,00:50:C2:10:A0:00/36Quinx,00:50:C2:10:B0:00/36MarekMic,00:50:C2:10:C0:00/36Photonic,00:50:C2:10:D0:00/36Implemen,00:50:C2:10:E0:00/36Unipower,00:50:C2:10:F0:00/36Percepti,00:50:C2:11:00:00/36QuesCom,00:50:C2:11:10:00/36Endusis,00:50:C2:11:20:00/36Compuwor,00:50:C2:11:30:00/36AceElect,00:50:C2:11:40:00/36QuestInn,00:50:C2:11:50:00/36Vidco,00:50:C2:11:60:00/36DSPDesig,00:50:C2:11:70:00/36Wintegra,00:50:C2:11:80:00/36Microbit,00:50:C2:11:90:00/36GlobalOp,00:50:C2:11:A0:00/36Teamaxes,00:50:C2:11:B0:00/36DigitalV,00:50:C2:11:C0:00/36Stonefly,00:50:C2:11:D0:00/36DestinyN,00:50:C2:11:E0:00/36VolvoCar,00:50:C2:11:F0:00/36CSSIndus,00:50:C2:12:00:00/36XStore,00:50:C2:12:10:00/36COE,00:50:C2:12:20:00/36Diva,00:50:C2:12:30:00/36SeranoaN,00:50:C2:12:40:00/36TokaiSof,00:50:C2:12:50:00/36Tecwings,00:50:C2:12:60:00/36MaxLinea,00:50:C2:12:70:00/36TPATraff,00:50:C2:12:80:00/36Pycon,00:50:C2:12:90:00/36TTPCom,00:50:C2:12:A0:00/36Symbolic,00:50:C2:12:B0:00/36DongAElt,00:50:C2:12:C0:00/36DeltaTau,00:50:C2:12:D0:00/36Megisto,00:50:C2:12:E0:00/36Runcom,00:50:C2:12:F0:00/36Haag-Str,00:50:C2:13:00:00/36USTraffi,00:50:C2:13:10:00/36InBusEng,00:50:C2:13:20:00/36ProconEl,00:50:C2:13:30:00/36ChipWrig,00:50:C2:13:40:00/36DRSPhotr,00:50:C2:13:50:00/36Elad,00:50:C2:13:60:00/36Tensilic,00:50:C2:13:70:00/36UniwellU,00:50:C2:13:80:00/36DelphinT,00:50:C2:13:90:00/36SrResear,00:50:C2:13:A0:00/36TexCompu,00:50:C2:13:B0:00/36Vaisala,00:50:C2:13:C0:00/36NBGIndus,00:50:C2:13:D0:00/36FormulaO,00:50:C2:13:E0:00/36AVerMedi,00:50:C2:13:F0:00/36SentitoN,00:50:C2:14:00:00/36ITS,00:50:C2:14:10:00/36TimeTerm,00:50:C2:14:20:00/36Instrume,00:50:C2:14:30:00/36Aartesys,00:50:C2:14:40:00/36PhytecMe,00:50:C2:14:50:00/36ELCLight,00:50:C2:14:60:00/36APCON,00:50:C2:14:70:00/36UniSUR,00:50:C2:14:80:00/36Alltec,00:50:C2:14:90:00/36Haag-Str,00:50:C2:14:A0:00/36Dycec,00:50:C2:14:B0:00/36HECUBAEl,00:50:C2:14:C0:00/36Optibase,00:50:C2:14:D0:00/36wellink,00:50:C2:14:E0:00/36CorinexG,00:50:C2:14:F0:00/36Telephon,00:50:C2:15:00:00/36Torse,00:50:C2:15:10:00/36ReduxCom,00:50:C2:15:20:00/36AirVastT,00:50:C2:15:30:00/36Advanced,00:50:C2:15:40:00/36Jostra,00:50:C2:15:50:00/36EneaReal,00:50:C2:15:60:00/36CommServ,00:50:C2:15:70:00/36nCore,00:50:C2:15:80:00/36Communic,00:50:C2:15:90:00/36Standard,00:50:C2:15:A0:00/36Plextek,00:50:C2:15:B0:00/36DuneNetw,00:50:C2:15:C0:00/36AoptixTe,00:50:C2:15:D0:00/36Cepheid,00:50:C2:15:E0:00/36Celite,00:50:C2:15:F0:00/36Pulsar,00:50:C2:16:00:00/36TTI-Tele,00:50:C2:16:10:00/36J&BEngin,00:50:C2:16:20:00/36Teseda,00:50:C2:16:30:00/36Computer,00:50:C2:16:40:00/36Acunia,00:50:C2:16:50:00/36Ipcast,00:50:C2:16:60:00/36Infineer,00:50:C2:16:70:00/36Precisio,00:50:C2:16:80:00/36ExtremeS,00:50:C2:16:90:00/36Nordson,00:50:C2:16:A0:00/36TimeDoma,00:50:C2:16:B0:00/36Mastercl,00:50:C2:16:C0:00/36BrijingE,00:50:C2:16:D0:00/36PostecDa,00:50:C2:16:E0:00/36Pmc,00:50:C2:16:F0:00/36DicksonT,00:50:C2:17:00:00/36Taishodo,00:50:C2:17:10:00/36Quantron,00:50:C2:17:20:00/36SAETIS,00:50:C2:17:30:00/36DeMeTec,00:50:C2:17:40:00/36N&PTechn,00:50:C2:17:50:00/36Sei,00:50:C2:17:60:00/36Wavium,00:50:C2:17:70:00/36Unicoi,00:50:C2:17:80:00/36PartnerV,00:50:C2:17:90:00/36Verifibe,00:50:C2:17:A0:00/36WOLFAdva,00:50:C2:17:B0:00/36Broadsto,00:50:C2:17:C0:00/36Jeffress,00:50:C2:17:D0:00/36Cognex,00:50:C2:17:E0:00/36BinaryWa,00:50:C2:17:F0:00/36PDQManuf,00:50:C2:18:00:00/36ZultysTe,00:50:C2:18:10:00/36Task84,00:50:C2:18:20:00/36wolf-inf,00:50:C2:18:30:00/36Mixbaal,00:50:C2:18:40:00/36HMComput,00:50:C2:18:50:00/36OpticalW,00:50:C2:18:60:00/36PantecEn,00:50:C2:18:70:00/36CyanTech,00:50:C2:18:80:00/36dresden-,00:50:C2:18:90:00/36CC,00:50:C2:18:A0:00/36BaslerEl,00:50:C2:18:B0:00/36Teradyne,00:50:C2:18:C0:00/36Technodr,00:50:C2:18:D0:00/36CCII,00:50:C2:18:E0:00/36SparrEle,00:50:C2:18:F0:00/36MatsuiMf,00:50:C2:19:00:00/36Goerlitz,00:50:C2:19:10:00/36PartnerV,00:50:C2:19:20:00/36Advanced,00:50:C2:19:30:00/36LaserBit,00:50:C2:19:40:00/36Cominfo,00:50:C2:19:50:00/36Momentum,00:50:C2:19:60:00/36Netsynt,00:50:C2:19:70:00/36EPMTecno,00:50:C2:19:80:00/36PotsTek,00:50:C2:19:90:00/36Survalen,00:50:C2:19:A0:00/36AzioTech,00:50:C2:19:B0:00/36Wilcoxon,00:50:C2:19:C0:00/36ArtecDes,00:50:C2:19:D0:00/36Electrex,00:50:C2:19:E0:00/36Paltroni,00:50:C2:19:F0:00/36Fleetwoo,00:50:C2:1A:00:00/36SCAData,00:50:C2:1A:10:00/36Portalpl,00:50:C2:1A:20:00/36ABBSwitz,00:50:C2:1A:30:00/36TidelEng,00:50:C2:1A:40:00/36ProtechO,00:50:C2:1A:50:00/36Norco,00:50:C2:1A:60:00/36RFCode,00:50:C2:1A:70:00/36AlphaBet,00:50:C2:1A:80:00/36AnovaBro,00:50:C2:1A:90:00/36AxotecTe,00:50:C2:1A:A0:00/36BitBox,00:50:C2:1A:B0:00/36Streamin,00:50:C2:1A:C0:00/36Beckmann,00:50:C2:1A:D0:00/36Remiasro,00:50:C2:1A:E0:00/36HomeDire,00:50:C2:1A:F0:00/36Pesa,00:50:C2:1B:00:00/36BLANKOMA,00:50:C2:1B:10:00/36AxesTech,00:50:C2:1B:20:00/36SIGOSSys,00:50:C2:1B:30:00/36DspDesig,00:50:C2:1B:40:00/36DSPGroup,00:50:C2:1B:50:00/36ThraneTh,00:50:C2:1B:60:00/36DTS,00:50:C2:1B:70:00/36MosChipU,00:50:C2:1B:80:00/36Electron,00:50:C2:1B:90:00/36EmComTec,00:50:C2:1B:A0:00/36Interzea,00:50:C2:1B:B0:00/36EmailMet,00:50:C2:1B:C0:00/36DINECInt,00:50:C2:1B:D0:00/36AIOI,00:50:C2:1B:E0:00/36SediaEle,00:50:C2:1B:F0:00/36Internat,00:50:C2:1C:00:00/36WillMoni,00:50:C2:1C:10:00/36Infiniti,00:50:C2:1C:20:00/36Weltroni,00:50:C2:1C:30:00/36TTelectr,00:50:C2:1C:40:00/36PalmSolu,00:50:C2:1C:50:00/36Flander,00:50:C2:1C:60:00/36RemcoIta,00:50:C2:1C:70:00/36TwinDeve,00:50:C2:1C:80:00/36Euphonyt,00:50:C2:1C:90:00/36modas,00:50:C2:1C:A0:00/36EVERSp,00:50:C2:1C:B0:00/36quantumB,00:50:C2:1C:C0:00/36WaveIP,00:50:C2:1C:D0:00/36INCAAInf,00:50:C2:1C:E0:00/36DatatekA,00:50:C2:1C:F0:00/36Lifetime,00:50:C2:1D:00:00/36YazakiNo,00:50:C2:1D:10:00/36Benchmar,00:50:C2:1D:20:00/36Shenyang,00:50:C2:1D:30:00/36Synopsys,00:50:C2:1D:40:00/36PhaseIVE,00:50:C2:1D:50:00/36Redpoint,00:50:C2:1D:60:00/36shanghai,00:50:C2:1D:70:00/36PleoraTe,00:50:C2:1D:80:00/36Guardian,00:50:C2:1D:90:00/36Edc,00:50:C2:1D:A0:00/36GFIChron,00:50:C2:1D:B0:00/36AppliedE,00:50:C2:1D:C0:00/36ImardaNe,00:50:C2:1D:D0:00/36peikerac,00:50:C2:1D:E0:00/36ReliOn,00:50:C2:1D:F0:00/36LuleaUni,00:50:C2:1E:00:00/36Cognex,00:50:C2:1E:10:00/36Automaat,00:50:C2:1E:20:00/36DIGITRON,00:50:C2:1E:30:00/36Bluesock,00:50:C2:1E:40:00/36Soronti,00:50:C2:1E:50:00/36DorletSa,00:50:C2:1E:60:00/36UnitedTr,00:50:C2:1E:70:00/36SmithMet,00:50:C2:1E:80:00/36Metrotec,00:50:C2:1E:90:00/36RanchNet,00:50:C2:1E:A0:00/36Dave,00:50:C2:1E:B0:00/36DataResp,00:50:C2:1E:C0:00/36COSMO,00:50:C2:1E:D0:00/36EMKA-ele,00:50:C2:1E:E0:00/36PertoPer,00:50:C2:1E:F0:00/36M2Techno,00:50:C2:1F:00:00/36EXIWirel,00:50:C2:1F:10:00/36SKYCompu,00:50:C2:1F:20:00/36Tattile,00:50:C2:1F:30:00/36Radionor,00:50:C2:1F:40:00/36Covia,00:50:C2:1F:50:00/36AbestCom,00:50:C2:1F:60:00/36BAEContr,00:50:C2:1F:70:00/36ARCCréat,00:50:C2:1F:80:00/36Ultracke,00:50:C2:1F:90:00/36FrSauter,00:50:C2:1F:A0:00/36SPContro,00:50:C2:1F:B0:00/36Willowgl,00:50:C2:1F:C0:00/36EDD,00:50:C2:1F:D0:00/36SouthWin,00:50:C2:1F:E0:00/36Safetran,00:50:C2:1F:F0:00/36ProductD,00:50:C2:20:00:00/36Whittier,00:50:C2:20:10:00/36OlympusN,00:50:C2:20:20:00/36AudioRid,00:50:C2:20:30:00/36Vocality,00:50:C2:20:40:00/36AlgodueE,00:50:C2:20:50:00/36SystIng,00:50:C2:20:60:00/36Windmill,00:50:C2:20:70:00/36Solectro,00:50:C2:20:80:00/36nNovia,00:50:C2:20:90:00/36LK,00:50:C2:20:A0:00/36Ferrarie,00:50:C2:20:B0:00/36Rafael,00:50:C2:20:C0:00/36Communic,00:50:C2:20:D0:00/36Varisys,00:50:C2:20:E0:00/36PYRAMIDC,00:50:C2:20:F0:00/36OMICRONe,00:50:C2:21:00:00/36Innovics,00:50:C2:21:10:00/36Hochschu,00:50:C2:21:20:00/364Links,00:50:C2:21:30:00/36SysAware,00:50:C2:21:40:00/36OshimiSy,00:50:C2:21:50:00/36VoiceCom,00:50:C2:21:60:00/36LevelCon,00:50:C2:21:70:00/36LinnProd,00:50:C2:21:80:00/36NansenSA,00:50:C2:21:90:00/36Aeroflex,00:50:C2:21:A0:00/36Mst,00:50:C2:21:B0:00/36GDMissio,00:50:C2:21:C0:00/36Fracarro,00:50:C2:21:D0:00/36ESGElekt,00:50:C2:21:E0:00/36AppliedT,00:50:C2:21:F0:00/36MonitorB,00:50:C2:22:00:00/36Serveron,00:50:C2:22:10:00/36GetingeI,00:50:C2:22:20:00/36imo-elek,00:50:C2:22:30:00/36visicont,00:50:C2:22:40:00/36PANNOCOM,00:50:C2:22:50:00/36nVentSch,00:50:C2:22:60:00/36RossVide,00:50:C2:22:70:00/36Intellig,00:50:C2:22:80:00/36Intellig,00:50:C2:22:90:00/36eko,00:50:C2:22:A0:00/36Crescend,00:50:C2:22:B0:00/36RieglLas,00:50:C2:22:C0:00/36Intrinsi,00:50:C2:22:D0:00/36asetek,00:50:C2:22:E0:00/36LordInge,00:50:C2:22:F0:00/36HTEC,00:50:C2:23:00:00/36AutoTOOL,00:50:C2:23:10:00/36Legra,00:50:C2:23:20:00/36Simet,00:50:C2:23:30:00/36EdenTree,00:50:C2:23:40:00/36Silverba,00:50:C2:23:50:00/36PolimarE,00:50:C2:23:60:00/36JLCooper,00:50:C2:23:70:00/36Tandata,00:50:C2:23:80:00/36Schwer+K,00:50:C2:23:90:00/36StinsCom,00:50:C2:23:A0:00/36ChantryN,00:50:C2:23:B0:00/36Envara,00:50:C2:23:C0:00/36Wheatsto,00:50:C2:23:D0:00/36Gauging,00:50:C2:23:E0:00/36Kallastr,00:50:C2:23:F0:00/36Hallibur,00:50:C2:24:00:00/36Geoquip,00:50:C2:24:10:00/36Controni,00:50:C2:24:20:00/36MdsSciex,00:50:C2:24:30:00/36RGBSpect,00:50:C2:24:40:00/36intec,00:50:C2:24:50:00/36Hauppaug,00:50:C2:24:60:00/36Hardmeie,00:50:C2:24:70:00/36GradualT,00:50:C2:24:80:00/36DixtalBi,00:50:C2:24:90:00/36Bender,00:50:C2:24:A0:00/36CDSRail,00:50:C2:24:B0:00/36Azimuth,00:50:C2:24:C0:00/36Supertel,00:50:C2:24:D0:00/36MettlerT,00:50:C2:24:E0:00/36ScharffW,00:50:C2:24:F0:00/36Macronet,00:50:C2:25:00:00/36ACDElekt,00:50:C2:25:10:00/36DGTSp,00:50:C2:25:20:00/36ads-tec,00:50:C2:25:30:00/36DSM-Mess,00:50:C2:25:40:00/36ThalesCo,00:50:C2:25:50:00/36STMicroe,00:50:C2:25:60:00/36Informat,00:50:C2:25:70:00/36Digicast,00:50:C2:25:80:00/36Spacesav,00:50:C2:25:90:00/36OmicronC,00:50:C2:25:A0:00/36Zendex,00:50:C2:25:B0:00/36WinfordE,00:50:C2:25:C0:00/36Softhill,00:50:C2:25:D0:00/36Rdtech,00:50:C2:25:E0:00/36MITEHrad,00:50:C2:25:F0:00/36AlbertHa,00:50:C2:26:00:00/36Biotage,00:50:C2:26:10:00/36Tattile,00:50:C2:26:20:00/36Shanghai,00:50:C2:26:30:00/36VanscoEl,00:50:C2:26:40:00/36Confiden,00:50:C2:26:50:00/36BelikSPR,00:50:C2:26:60:00/36ATOMGIKE,00:50:C2:26:70:00/36AllenMar,00:50:C2:26:80:00/36Parabit,00:50:C2:26:90:00/36Technisy,00:50:C2:26:A0:00/36FgSynery,00:50:C2:26:B0:00/36Continen,00:50:C2:26:C0:00/36CrystalV,00:50:C2:26:D0:00/36DspDesig,00:50:C2:26:E0:00/36ZPEngine,00:50:C2:26:F0:00/36DigitalR,00:50:C2:27:00:00/36S4Techno,00:50:C2:27:10:00/36VlsipTec,00:50:C2:27:20:00/36VerexTec,00:50:C2:27:30:00/36Servicio,00:50:C2:27:40:00/36Fundació,00:50:C2:27:50:00/36ExtremeE,00:50:C2:27:60:00/36TielineR,00:50:C2:27:70:00/36TR,00:50:C2:27:80:00/36Replicom,00:50:C2:27:90:00/36PATLITE,00:50:C2:27:A0:00/36Maestro,00:50:C2:27:B0:00/36LinkSecu,00:50:C2:27:C0:00/36Danlaw,00:50:C2:27:D0:00/36AlliedTe,00:50:C2:27:E0:00/36AnaLogic,00:50:C2:27:F0:00/36AirBroad,00:50:C2:28:00:00/36Agecodag,00:50:C2:28:10:00/36Cabtroni,00:50:C2:28:20:00/36Telvent,00:50:C2:28:30:00/36Ansitex,00:50:C2:28:40:00/36Micronet,00:50:C2:28:50:00/36Littwin,00:50:C2:28:60:00/36Ateme,00:50:C2:28:70:00/36TECNEWEl,00:50:C2:28:80:00/36RPM,00:50:C2:28:90:00/36Rototype,00:50:C2:28:A0:00/36RealTime,00:50:C2:28:B0:00/36OrionTec,00:50:C2:28:C0:00/36Futaba,00:50:C2:28:D0:00/36Axode,00:50:C2:28:E0:00/36Tattile,00:50:C2:28:F0:00/36Spellman,00:50:C2:29:00:00/36Ebneuro,00:50:C2:29:10:00/36ChauvinA,00:50:C2:29:20:00/36AMIRIX,00:50:C2:29:30:00/36IPUnity,00:50:C2:29:40:00/36EPSa,00:50:C2:29:50:00/36Logosol,00:50:C2:29:60:00/36OpVista,00:50:C2:29:70:00/36Kinetics,00:50:C2:29:80:00/36HarvadUn,00:50:C2:29:90:00/36CAD-UL,00:50:C2:29:A0:00/36PacketTe,00:50:C2:29:B0:00/36ACDElekt,00:50:C2:29:C0:00/362NTELEKO,00:50:C2:29:D0:00/36GlobeWir,00:50:C2:29:E0:00/36SELEXCom,00:50:C2:29:F0:00/36Baudisch,00:50:C2:2A:00:00/36Sterling,00:50:C2:2A:10:00/36Infineti,00:50:C2:2A:20:00/36EpelsaSL,00:50:C2:2A:30:00/36West-Com,00:50:C2:2A:40:00/36XipherEm,00:50:C2:2A:50:00/36SeptierC,00:50:C2:2A:60:00/36Brannstr,00:50:C2:2A:70:00/36MicroSys,00:50:C2:2A:80:00/36DVTelIsr,00:50:C2:2A:90:00/36DrStaige,00:50:C2:2A:A0:00/36DEUTAWer,00:50:C2:2A:B0:00/36AUMInfot,00:50:C2:2A:C0:00/36BBIEngin,00:50:C2:2A:D0:00/36ABBT&D,00:50:C2:2A:E0:00/36QuestRet,00:50:C2:2A:F0:00/36CSACompu,00:50:C2:2B:00:00/36TeldaEle,00:50:C2:2B:10:00/36Private,00:50:C2:2B:20:00/36SmithsDe,00:50:C2:2B:30:00/36Embedded,00:50:C2:2B:40:00/36Polatis,00:50:C2:2B:50:00/36HobbesCo,00:50:C2:2B:60:00/36Softier,00:50:C2:2B:70:00/36RAFI,00:50:C2:2B:80:00/36AdmiralS,00:50:C2:2B:90:00/36Richmond,00:50:C2:2B:A0:00/36NorcoInd,00:50:C2:2B:B0:00/36TAInstru,00:50:C2:2B:C0:00/36UsterTec,00:50:C2:2B:D0:00/36StorLink,00:50:C2:2B:E0:00/36Lipowsky,00:50:C2:2B:F0:00/36Perax,00:50:C2:2C:00:00/36Magellan,00:50:C2:2C:10:00/36StageTec,00:50:C2:2C:20:00/36Smarteye,00:50:C2:2C:30:00/36DigitalS,00:50:C2:2C:40:00/36Invensys,00:50:C2:2C:50:00/36Elman,00:50:C2:2C:60:00/36InitialE,00:50:C2:2C:70:00/36Siliquen,00:50:C2:2C:80:00/36Selco,00:50:C2:2C:90:00/36RosemanE,00:50:C2:2C:A0:00/36Putercom,00:50:C2:2C:B0:00/36FACTSEng,00:50:C2:2C:C0:00/36Embedded,00:50:C2:2C:D0:00/36DataWind,00:50:C2:2C:E0:00/36RossVide,00:50:C2:2C:F0:00/36Diseñode,00:50:C2:2D:00:00/36WorthDat,00:50:C2:2D:10:00/36Miritek,00:50:C2:2D:20:00/36AirnetCo,00:50:C2:2D:30:00/36GerberSc,00:50:C2:2D:40:00/36Integrat,00:50:C2:2D:50:00/36Pixy,00:50:C2:2D:60:00/36WISTechn,00:50:C2:2D:70:00/36NeoElect,00:50:C2:2D:80:00/36Syn-Tech,00:50:C2:2D:90:00/36Private,00:50:C2:2D:A0:00/36PYRAMIDC,00:50:C2:2D:B0:00/36AutoTOOL,00:50:C2:2D:C0:00/36WienerPl,00:50:C2:2D:D0:00/36WestekTe,00:50:C2:2D:E0:00/36Research,00:50:C2:2D:F0:00/36Micrel-N,00:50:C2:2E:00:00/36BaxterIn,00:50:C2:2E:10:00/36AccessIS,00:50:C2:2E:20:00/36BallardT,00:50:C2:2E:30:00/36MGIndust,00:50:C2:2E:40:00/36iamba,00:50:C2:2E:50:00/36Transtec,00:50:C2:2E:60:00/36Dalsa,00:50:C2:2E:70:00/36SafeView,00:50:C2:2E:80:00/36SMVSyste,00:50:C2:2E:90:00/36SRIInter,00:50:C2:2E:A0:00/36QUBIsoft,00:50:C2:2E:B0:00/36LinggJan,00:50:C2:2E:C0:00/36ChengduB,00:50:C2:2E:D0:00/364RFCommu,00:50:C2:2E:E0:00/36SHFCommu,00:50:C2:2E:F0:00/36Profline,00:50:C2:2F:00:00/36LECO,00:50:C2:2F:10:00/36Geometri,00:50:C2:2F:20:00/36Eurotek,00:50:C2:2F:30:00/36Crossbow,00:50:C2:2F:40:00/36Efficien,00:50:C2:2F:50:00/36ADChips,00:50:C2:2F:60:00/36Clifford,00:50:C2:2F:70:00/36Gillam-F,00:50:C2:2F:80:00/36SavvyCor,00:50:C2:2F:90:00/36Digilent,00:50:C2:2F:A0:00/36TornadoM,00:50:C2:2F:B0:00/36ArthurIn,00:50:C2:2F:C0:00/36Blacklin,00:50:C2:2F:D0:00/36American,00:50:C2:2F:E0:00/36Saab,00:50:C2:2F:F0:00/36PatriaAd,00:50:C2:30:00:00/36SoredexI,00:50:C2:30:10:00/36DelphiDi,00:50:C2:30:20:00/36EuroDesi,00:50:C2:30:30:00/36CI,00:50:C2:30:40:00/36COMERSON,00:50:C2:30:50:00/36Symbium,00:50:C2:30:60:00/36NoranTel,00:50:C2:30:70:00/36Uniondig,00:50:C2:30:80:00/36FiveCo,00:50:C2:30:90:00/36Rackmast,00:50:C2:30:A0:00/36InningsT,00:50:C2:30:B0:00/36VXTechno,00:50:C2:30:C0:00/36Teamlog,00:50:C2:30:D0:00/36Setaram,00:50:C2:30:E0:00/36Obvius,00:50:C2:30:F0:00/36Digicont,00:50:C2:31:00:00/36Cybertro,00:50:C2:31:10:00/36Comodo,00:50:C2:31:20:00/36DeseTech,00:50:C2:31:30:00/36SAIABurg,00:50:C2:31:40:00/36MicroBee,00:50:C2:31:50:00/36ifaksyst,00:50:C2:31:60:00/36Dataline,00:50:C2:31:70:00/36Cosine,00:50:C2:31:80:00/36Milmega,00:50:C2:31:90:00/36Invatron,00:50:C2:31:A0:00/36ZodiakDa,00:50:C2:31:B0:00/36Datacon,00:50:C2:31:C0:00/36Casa,00:50:C2:31:D0:00/36ImardaNe,00:50:C2:31:E0:00/36C3-ilex,00:50:C2:31:F0:00/36GeotechI,00:50:C2:32:00:00/36DTASENSO,00:50:C2:32:10:00/36Uxp,00:50:C2:32:20:00/36BQTSolut,00:50:C2:32:30:00/36RedRockN,00:50:C2:32:40:00/36Odixion,00:50:C2:32:50:00/36FederalA,00:50:C2:32:60:00/36Navionic,00:50:C2:32:70:00/36Dornier,00:50:C2:32:80:00/36ICSElect,00:50:C2:32:90:00/36Imax,00:50:C2:32:A0:00/36PhytecMe,00:50:C2:32:B0:00/36DigitalM,00:50:C2:32:C0:00/36Integrat,00:50:C2:32:D0:00/36ConsensZ,00:50:C2:32:E0:00/36Manusa-G,00:50:C2:32:F0:00/36Pultroni,00:50:C2:33:00:00/36Sicon,00:50:C2:33:10:00/36Broadcas,00:50:C2:33:20:00/36PunjabCo,00:50:C2:33:30:00/36Radix,00:50:C2:33:40:00/36PictureE,00:50:C2:33:50:00/36NimcatNe,00:50:C2:33:60:00/36GoldenRi,00:50:C2:33:70:00/36Eti,00:50:C2:33:80:00/36Ernitec,00:50:C2:33:90:00/36SecureSe,00:50:C2:33:A0:00/36UnitedTe,00:50:C2:33:B0:00/36Multimed,00:50:C2:33:C0:00/36SkipJam,00:50:C2:33:D0:00/36GDMissio,00:50:C2:33:E0:00/36CATechno,00:50:C2:33:F0:00/36EXYSbvba,00:50:C2:34:00:00/36Virtu,00:50:C2:34:10:00/36Novx,00:50:C2:34:20:00/36StMichae,00:50:C2:34:30:00/36ABBXiame,00:50:C2:34:40:00/36ads-tec,00:50:C2:34:50:00/36Act,00:50:C2:34:60:00/36biokeysy,00:50:C2:34:70:00/36RowSeven,00:50:C2:34:80:00/36KoolSpan,00:50:C2:34:90:00/36SSISchae,00:50:C2:34:A0:00/36NIE,00:50:C2:34:B0:00/36Ecutel,00:50:C2:34:C0:00/36ChuoElec,00:50:C2:34:D0:00/36BMKprofe,00:50:C2:34:E0:00/36ABBPower,00:50:C2:34:F0:00/36NorthPol,00:50:C2:35:00:00/36KinesysP,00:50:C2:35:10:00/36Finesyst,00:50:C2:35:20:00/36Luceo,00:50:C2:35:30:00/36Crossing,00:50:C2:35:40:00/36Advanced,00:50:C2:35:50:00/36Ihm,00:50:C2:35:60:00/36BaytechC,00:50:C2:35:70:00/36AthenaSe,00:50:C2:35:80:00/36Alcea,00:50:C2:35:90:00/36KramerEl,00:50:C2:35:A0:00/36Advanced,00:50:C2:35:B0:00/36VlsipTec,00:50:C2:35:C0:00/36Ratotec,00:50:C2:35:D0:00/36NetTest,00:50:C2:35:E0:00/36JobinYvo,00:50:C2:35:F0:00/36FImm,00:50:C2:36:00:00/36DigitalR,00:50:C2:36:10:00/36Contec,00:50:C2:36:20:00/36AZDPraha,00:50:C2:36:30:00/36Septentr,00:50:C2:36:40:00/36Tattile,00:50:C2:36:50:00/36Vpg,00:50:C2:36:60:00/36Vanguard,00:50:C2:36:70:00/36CANMAXTe,00:50:C2:36:80:00/36Aspel,00:50:C2:36:90:00/36AlwaysOn,00:50:C2:36:A0:00/36Optronic,00:50:C2:36:B0:00/36MinervaT,00:50:C2:36:C0:00/36RISCOGro,00:50:C2:36:D0:00/36OplinkCo,00:50:C2:36:E0:00/36MinicomA,00:50:C2:36:F0:00/36XIMEAsro,00:50:C2:37:00:00/36EuropeTe,00:50:C2:37:10:00/36DigitalA,00:50:C2:37:20:00/36ELVElekt,00:50:C2:37:30:00/36Companio,00:50:C2:37:40:00/36OwasysAd,00:50:C2:37:50:00/36TIR,00:50:C2:37:60:00/36Cleode,00:50:C2:37:70:00/36XycomVME,00:50:C2:37:80:00/36Daintree,00:50:C2:37:90:00/36ControlL,00:50:C2:37:A0:00/36IDA,00:50:C2:37:B0:00/36Freescal,00:50:C2:37:C0:00/36MODIA,00:50:C2:37:D0:00/36VeroTrak,00:50:C2:37:E0:00/36Ni,00:50:C2:37:F0:00/36Foresear,00:50:C2:38:00:00/36EKE-Elec,00:50:C2:38:10:00/36Realtime,00:50:C2:38:20:00/36Colorado,00:50:C2:38:30:00/36ICSElect,00:50:C2:38:40:00/36Wireless,00:50:C2:38:50:00/36SUNGJINN,00:50:C2:38:60:00/36Precisio,00:50:C2:38:70:00/36Inoteska,00:50:C2:38:80:00/36IEE,00:50:C2:38:90:00/36Exavio,00:50:C2:38:A0:00/36Embedtro,00:50:C2:38:B0:00/36InterBri,00:50:C2:38:C0:00/36Epsilon,00:50:C2:38:D0:00/36A&GSoluz,00:50:C2:38:E0:00/36NordicAl,00:50:C2:38:F0:00/36TTCTelec,00:50:C2:39:00:00/36TCCommun,00:50:C2:39:10:00/36Esensors,00:50:C2:39:20:00/36PhytecMe,00:50:C2:39:30:00/36SYSTECel,00:50:C2:39:40:00/36Embedit,00:50:C2:39:50:00/36vidisys,00:50:C2:39:60:00/36RapidWav,00:50:C2:39:70:00/36MANGODSP,00:50:C2:39:80:00/36InhandEl,00:50:C2:39:90:00/36Advanced,00:50:C2:39:A0:00/36OpticalA,00:50:C2:39:B0:00/36YUYAMAMF,00:50:C2:39:C0:00/36TiyodaMf,00:50:C2:39:D0:00/36DigitalD,00:50:C2:39:E0:00/36ARGElect,00:50:C2:39:F0:00/36Isensix,00:50:C2:3A:00:00/36StreetFi,00:50:C2:3A:10:00/36Samsoft,00:50:C2:3A:20:00/36VegasAmu,00:50:C2:3A:30:00/36StarLink,00:50:C2:3A:40:00/36Silvertr,00:50:C2:3A:50:00/36LabJack,00:50:C2:3A:60:00/36IntelliD,00:50:C2:3A:70:00/36Elektrot,00:50:C2:3A:80:00/36Engim,00:50:C2:3A:90:00/36Westroni,00:50:C2:3A:A0:00/36Networke,00:50:C2:3A:B0:00/36taskitRe,00:50:C2:3A:C0:00/36InAccess,00:50:C2:3A:D0:00/36SpirentC,00:50:C2:3A:E0:00/36HankukTa,00:50:C2:3A:F0:00/36NorbitOD,00:50:C2:3B:00:00/36Microtar,00:50:C2:3B:10:00/36Specstro,00:50:C2:3B:20:00/36Tennesse,00:50:C2:3B:30:00/36MediaLab,00:50:C2:3B:40:00/36Contrôle,00:50:C2:3B:50:00/36NECTOKIN,00:50:C2:3B:60:00/36ArecontV,00:50:C2:3B:70:00/36Mindspee,00:50:C2:3B:80:00/36KeithKoe,00:50:C2:3B:90:00/36Gilbarco,00:50:C2:3B:A0:00/36PhytecMe,00:50:C2:3B:B0:00/36IMAGOTec,00:50:C2:3B:C0:00/36Tyzx,00:50:C2:3B:D0:00/36BigbangL,00:50:C2:3B:E0:00/36PaulySte,00:50:C2:3B:F0:00/36AudioPro,00:50:C2:3C:00:00/36EDAIndus,00:50:C2:3C:10:00/36MicroTek,00:50:C2:3C:20:00/36Casabyte,00:50:C2:3C:30:00/364gTechno,00:50:C2:3C:40:00/36SyprisEl,00:50:C2:3C:50:00/36SiliconO,00:50:C2:3C:60:00/36NetOptic,00:50:C2:3C:70:00/36SalentTe,00:50:C2:3C:80:00/36WheelsZe,00:50:C2:3C:90:00/36DilaxInt,00:50:C2:3C:A0:00/36ABB,00:50:C2:3C:B0:00/36Analytic,00:50:C2:3C:C0:00/36Linkwell,00:50:C2:3C:D0:00/36Micro-Me,00:50:C2:3C:E0:00/36WardLeon,00:50:C2:3C:F0:00/36Technova,00:50:C2:3D:00:00/36Micro-Ro,00:50:C2:3D:10:00/36Braintro,00:50:C2:3D:20:00/36AdilecEn,00:50:C2:3D:30:00/36American,00:50:C2:3D:40:00/36Wisnuand,00:50:C2:3D:50:00/36FlukeBio,00:50:C2:3D:60:00/36Comlab,00:50:C2:3D:70:00/36TTCTELEK,00:50:C2:3D:80:00/36Key,00:50:C2:3D:90:00/36BavariaD,00:50:C2:3D:A0:00/36M5Data,00:50:C2:3D:B0:00/36Osmetech,00:50:C2:3D:C0:00/363Dpercep,00:50:C2:3D:D0:00/36ELMIC,00:50:C2:3D:E0:00/36ABBPower,00:50:C2:3D:F0:00/36BiODE,00:50:C2:3E:00:00/36Stinghor,00:50:C2:3E:10:00/36NeuLion,00:50:C2:3E:20:00/36SysNova,00:50:C2:3E:30:00/36CSIRO-Di,00:50:C2:3E:40:00/36CUE,00:50:C2:3E:50:00/36Vacon,00:50:C2:3E:60:00/36Crde,00:50:C2:3E:70:00/36Revoluti,00:50:C2:3E:80:00/36Conforma,00:50:C2:3E:90:00/36MedAvant,00:50:C2:3E:A0:00/36AlroInfo,00:50:C2:3E:B0:00/36ISSInter,00:50:C2:3E:C0:00/36Teneros,00:50:C2:3E:D0:00/36BoardRoo,00:50:C2:3E:E0:00/36Commoca,00:50:C2:3E:F0:00/36PATIndus,00:50:C2:3F:00:00/36megatece,00:50:C2:3F:10:00/36SallandE,00:50:C2:3F:20:00/36STL,00:50:C2:3F:30:00/36HitachiA,00:50:C2:3F:40:00/36MCTECHNO,00:50:C2:3F:50:00/36Phaedrus,00:50:C2:3F:60:00/36dAFTdATA,00:50:C2:3F:70:00/36Advantag,00:50:C2:3F:80:00/36Superna,00:50:C2:3F:90:00/36Sintium,00:50:C2:3F:A0:00/36Tumsan,00:50:C2:3F:B0:00/36nVentSch,00:50:C2:3F:C0:00/36Weinberg,00:50:C2:3F:D0:00/36HARTMANN,00:50:C2:3F:E0:00/36HaiVisio,00:50:C2:3F:F0:00/36CastIron,00:50:C2:40:00:00/36SmartMot,00:50:C2:40:10:00/36Promess,00:50:C2:40:20:00/36NumeronS,00:50:C2:40:30:00/36Rohde&Sc,00:50:C2:40:40:00/36NanShanB,00:50:C2:40:50:00/36Guralp,00:50:C2:40:60:00/36CoreStre,00:50:C2:40:70:00/36AIEEtude,00:50:C2:40:80:00/36TERN,00:50:C2:40:90:00/36Ktec,00:50:C2:40:A0:00/36ContecSt,00:50:C2:40:B0:00/36CenterVO,00:50:C2:40:C0:00/36AppliedM,00:50:C2:40:D0:00/36AfonicsF,00:50:C2:40:E0:00/36ads-tec,00:50:C2:40:F0:00/36Bir,00:50:C2:41:00:00/36Grossenb,00:50:C2:41:10:00/36Multimes,00:50:C2:41:20:00/36TSBSolut,00:50:C2:41:30:00/36Goodrich,00:50:C2:41:40:00/36Talleres,00:50:C2:41:50:00/36SensoTec,00:50:C2:41:60:00/36SELCO,00:50:C2:41:70:00/36QT,00:50:C2:41:80:00/36Planea,00:50:C2:41:90:00/36Mecsel,00:50:C2:41:A0:00/36Bluewate,00:50:C2:41:B0:00/36LogiMSof,00:50:C2:41:C0:00/36Infrasaf,00:50:C2:41:D0:00/36Altronic,00:50:C2:41:E0:00/36Videotek,00:50:C2:41:F0:00/36Avionica,00:50:C2:42:00:00/36Boundles,00:50:C2:42:10:00/36Efsys,00:50:C2:42:20:00/36ads-tec,00:50:C2:42:30:00/36Power-On,00:50:C2:42:40:00/36Metrolab,00:50:C2:42:50:00/36Pinnacle,00:50:C2:42:60:00/36STOMSyst,00:50:C2:42:70:00/36ScheidtB,00:50:C2:42:80:00/36Roxar,00:50:C2:42:90:00/36Matthews,00:50:C2:42:A0:00/36DspDesig,00:50:C2:42:B0:00/36VlsipTec,00:50:C2:42:C0:00/36TrapezeS,00:50:C2:42:D0:00/36Argo-Tec,00:50:C2:42:E0:00/36OelmannE,00:50:C2:42:F0:00/36Win4NET,00:50:C2:43:00:00/36ArcomDig,00:50:C2:43:10:00/36Octatron,00:50:C2:43:20:00/36TopwayIn,00:50:C2:43:30:00/36IdetechE,00:50:C2:43:40:00/36Imperati,00:50:C2:43:50:00/36AdatelTe,00:50:C2:43:60:00/36Satellit,00:50:C2:43:70:00/36PowerWAN,00:50:C2:43:80:00/36TelecomP,00:50:C2:43:90:00/36PeletonP,00:50:C2:43:A0:00/36ProDesig,00:50:C2:43:B0:00/36A3Ip,00:50:C2:43:C0:00/36Ducommun,00:50:C2:43:D0:00/36AnnArbor,00:50:C2:43:E0:00/36Copperco,00:50:C2:43:F0:00/36ARVOOIma,00:50:C2:44:00:00/36Advanced,00:50:C2:44:10:00/36SammiInf,00:50:C2:44:20:00/36PicoComp,00:50:C2:44:30:00/36Pickerin,00:50:C2:44:40:00/36Offshore,00:50:C2:44:50:00/36MICRONIC,00:50:C2:44:60:00/36MicroTec,00:50:C2:44:70:00/36GrupoEpe,00:50:C2:44:80:00/36Comtech,00:50:C2:44:90:00/36BLEILEDA,00:50:C2:44:A0:00/36Elettron,00:50:C2:44:B0:00/36Solace,00:50:C2:44:C0:00/36Computim,00:50:C2:44:D0:00/36Electro-,00:50:C2:44:E0:00/36QQTechno,00:50:C2:44:F0:00/36kippdata,00:50:C2:45:00:00/36Enconair,00:50:C2:45:10:00/36HAMEG,00:50:C2:45:20:00/36ScameSis,00:50:C2:45:30:00/36Erhardt+,00:50:C2:45:40:00/36Brivo,00:50:C2:45:50:00/36GogoBA,00:50:C2:45:60:00/36DRDCValc,00:50:C2:45:70:00/36Danbridg,00:50:C2:45:80:00/36HRZdata,00:50:C2:45:90:00/36PhytecMe,00:50:C2:45:A0:00/36Funkwerk,00:50:C2:45:B0:00/36MatraEle,00:50:C2:45:C0:00/36DeisterE,00:50:C2:45:D0:00/36DigitalE,00:50:C2:45:E0:00/36Hallibur,00:50:C2:45:F0:00/36T2CMarke,00:50:C2:46:00:00/36Vitelnet,00:50:C2:46:10:00/36Tattile,00:50:C2:46:20:00/36CT,00:50:C2:46:30:00/36Codem,00:50:C2:46:40:00/36XYTACsys,00:50:C2:46:50:00/36PDTS,00:50:C2:46:60:00/36LONAP,00:50:C2:46:70:00/36UnitedWe,00:50:C2:46:80:00/36NetworkI,00:50:C2:46:90:00/36BiPOMEle,00:50:C2:46:A0:00/36ISE,00:50:C2:46:B0:00/36EASYTECH,00:50:C2:46:C0:00/36CAMCO,00:50:C2:46:D0:00/36PaulSche,00:50:C2:46:E0:00/36AvenirTe,00:50:C2:46:F0:00/36Neurowar,00:50:C2:47:00:00/36Cybectec,00:50:C2:47:10:00/36PixtreeT,00:50:C2:47:20:00/36KOP,00:50:C2:47:30:00/36SensusMe,00:50:C2:47:40:00/36Venue1,00:50:C2:47:50:00/36ISEPOS,00:50:C2:47:60:00/36Ascon,00:50:C2:47:70:00/36SEVTidsy,00:50:C2:47:80:00/36Metafix,00:50:C2:47:90:00/36Unlimite,00:50:C2:47:A0:00/36Efficien,00:50:C2:47:B0:00/36PitneyBo,00:50:C2:47:C0:00/36AUCONET,00:50:C2:47:D0:00/36WIT,00:50:C2:47:E0:00/36EnergieU,00:50:C2:47:F0:00/36BRIT,00:50:C2:48:00:00/36SELKOM,00:50:C2:48:10:00/36Computer,00:50:C2:48:20:00/36PriamusS,00:50:C2:48:30:00/36Ses,00:50:C2:48:40:00/36Kooltech,00:50:C2:48:50:00/36PhytecMe,00:50:C2:48:60:00/36Safegate,00:50:C2:48:70:00/36Eridon,00:50:C2:48:80:00/36DaSistem,00:50:C2:48:90:00/36EREEElec,00:50:C2:48:A0:00/36MobileMa,00:50:C2:48:B0:00/36ads-tec,00:50:C2:48:C0:00/36Uniton,00:50:C2:48:D0:00/36MetronSp,00:50:C2:48:E0:00/36Teledyne,00:50:C2:48:F0:00/36DENGYOSH,00:50:C2:49:00:00/36Cloanto,00:50:C2:49:10:00/36FrSauter,00:50:C2:49:20:00/36TrafsysA,00:50:C2:49:30:00/36Artis,00:50:C2:49:40:00/36Ultimate,00:50:C2:49:50:00/36VAZAElek,00:50:C2:49:60:00/36Acutelog,00:50:C2:49:70:00/36Advanced,00:50:C2:49:80:00/36QuartetT,00:50:C2:49:90:00/36TrelliaN,00:50:C2:49:A0:00/36TelASICC,00:50:C2:49:B0:00/36vgcontro,00:50:C2:49:C0:00/36Envisaco,00:50:C2:49:D0:00/36Critical,00:50:C2:49:E0:00/36Armorlin,00:50:C2:49:F0:00/36GCS,00:50:C2:4A:00:00/36Advanced,00:50:C2:4A:10:00/36nVentSch,00:50:C2:4A:20:00/36SPECS,00:50:C2:4A:30:00/36ProtiumT,00:50:C2:4A:40:00/36IeeeP160,00:50:C2:4A:50:00/36Teledyne,00:50:C2:4A:60:00/36BuyangEl,00:50:C2:4A:70:00/36isegSpez,00:50:C2:4A:80:00/36CYJAYAKo,00:50:C2:4A:90:00/36FaberEle,00:50:C2:4A:A0:00/36HEINENEL,00:50:C2:4A:B0:00/36JVFCommu,00:50:C2:4A:C0:00/36Doramu,00:50:C2:4A:D0:00/36OpenPeak,00:50:C2:4A:E0:00/36ads-tec,00:50:C2:4A:F0:00/36Orbis,00:50:C2:4B:00:00/36EsmartDi,00:50:C2:4B:10:00/36NSFOCUSI,00:50:C2:4B:20:00/36TESLA,00:50:C2:4B:30:00/36ANSA,00:50:C2:4B:40:00/36MatrixAu,00:50:C2:4B:50:00/36ValleyTe,00:50:C2:4B:60:00/36GeneralR,00:50:C2:4B:70:00/36GFIChron,00:50:C2:4B:80:00/36Shenzhen,00:50:C2:4B:90:00/36RoseTech,00:50:C2:4B:A0:00/36Mistleto,00:50:C2:4B:B0:00/36Protonic,00:50:C2:4B:C0:00/36SaiaBurg,00:50:C2:4B:D0:00/36ArgonST,00:50:C2:4B:E0:00/36DigitalD,00:50:C2:4B:F0:00/36Westingh,00:50:C2:4C:00:00/36Bio-logi,00:50:C2:4C:10:00/36MovazNet,00:50:C2:4C:20:00/36Elbit,00:50:C2:4C:30:00/36Quantum3,00:50:C2:4C:40:00/36BlackDia,00:50:C2:4C:50:00/36eXrayBro,00:50:C2:4C:60:00/36Rubin,00:50:C2:4C:70:00/36Transbit,00:50:C2:4C:80:00/36Neets,00:50:C2:4C:90:00/36Scirocco,00:50:C2:4C:A0:00/36YargBiom,00:50:C2:4C:B0:00/36Verint,00:50:C2:4C:C0:00/36ImpediMe,00:50:C2:4C:D0:00/36Securito,00:50:C2:4C:E0:00/36OpenDate,00:50:C2:4C:F0:00/36Ziehl-Ab,00:50:C2:4D:00:00/36RCSEnerg,00:50:C2:4D:10:00/36SLICANsp,00:50:C2:4D:20:00/36TwowayCA,00:50:C2:4D:30:00/36ELPROCsp,00:50:C2:4D:40:00/36Herholdt,00:50:C2:4D:50:00/36SEBADesi,00:50:C2:4D:60:00/36Ingenieu,00:50:C2:4D:70:00/36DeltaTau,00:50:C2:4D:80:00/36Avantry,00:50:C2:4D:90:00/36GESecuri,00:50:C2:4D:A0:00/36MEDIORNE,00:50:C2:4D:B0:00/36AlfingMo,00:50:C2:4D:C0:00/36AceElect,00:50:C2:4D:D0:00/36TruteqWi,00:50:C2:4D:E0:00/36GDMissio,00:50:C2:4D:F0:00/36ThermoEl,00:50:C2:4E:00:00/36Telematr,00:50:C2:4E:10:00/36SSTeleco,00:50:C2:4E:20:00/36AppliedR,00:50:C2:4E:30:00/36Romteck,00:50:C2:4E:40:00/36Embigenc,00:50:C2:4E:50:00/36Sedo,00:50:C2:4E:60:00/36Photonic,00:50:C2:4E:70:00/36Computer,00:50:C2:4E:80:00/36SATELsp,00:50:C2:4E:90:00/36Seachang,00:50:C2:4E:A0:00/36Pmc,00:50:C2:4E:B0:00/36Mandozzi,00:50:C2:4E:C0:00/36ThalesDe,00:50:C2:4E:D0:00/36LabXTech,00:50:C2:4E:E0:00/36BeijingC,00:50:C2:4E:F0:00/36Creative,00:50:C2:4F:00:00/36MedAvant,00:50:C2:4F:10:00/36PacketIs,00:50:C2:4F:20:00/36Tantroni,00:50:C2:4F:30:00/36Autronic,00:50:C2:4F:40:00/36O2Run,00:50:C2:4F:50:00/36MonroeEl,00:50:C2:4F:60:00/36RealD,00:50:C2:4F:70:00/36WaveIP,00:50:C2:4F:80:00/36ProdcoIn,00:50:C2:4F:90:00/36RTDSTech,00:50:C2:4F:A0:00/36Cambridg,00:50:C2:4F:B0:00/36BESTechn,00:50:C2:4F:C0:00/36Hwayoung,00:50:C2:4F:D0:00/36NetworkA,00:50:C2:4F:E0:00/36GEMELETT,00:50:C2:4F:F0:00/36Dakty,00:50:C2:50:00:00/36Orenco,00:50:C2:50:10:00/36IBEXUK,00:50:C2:50:20:00/36Criterio,00:50:C2:50:30:00/36Respiron,00:50:C2:50:40:00/36Aphex,00:50:C2:50:50:00/36Computer,00:50:C2:50:60:00/367+Kft,00:50:C2:50:70:00/36MicroCon,00:50:C2:50:80:00/36Putercom,00:50:C2:50:90:00/36Hillcres,00:50:C2:50:A0:00/36MonitorB,00:50:C2:50:B0:00/36LogicBea,00:50:C2:50:C0:00/36AirwiseT,00:50:C2:50:D0:00/36Clearson,00:50:C2:50:E0:00/36Fibresav,00:50:C2:50:F0:00/36Polystar,00:50:C2:51:00:00/36SummitDe,00:50:C2:51:10:00/36Tecna,00:50:C2:51:20:00/36LinearAc,00:50:C2:51:30:00/36GenieNet,00:50:C2:51:40:00/36TadianEl,00:50:C2:51:50:00/36Monaghan,00:50:C2:51:60:00/36SowaElec,00:50:C2:51:70:00/36SolidSta,00:50:C2:51:80:00/36ChristEl,00:50:C2:51:90:00/36DBMCORP,00:50:C2:51:A0:00/36SpeasTec,00:50:C2:51:B0:00/36BetaLase,00:50:C2:51:C0:00/36TOA,00:50:C2:51:D0:00/36Velux,00:50:C2:51:E0:00/36AlconTec,00:50:C2:51:F0:00/36Traquair,00:50:C2:52:00:00/36McCainTr,00:50:C2:52:10:00/36ArisTech,00:50:C2:52:20:00/36MarkIVID,00:50:C2:52:30:00/36AMRDECPr,00:50:C2:52:40:00/36Motec,00:50:C2:52:50:00/36VASTech,00:50:C2:52:60:00/36ACsro,00:50:C2:52:70:00/36IRTrans,00:50:C2:52:80:00/36Tattile,00:50:C2:52:90:00/36PhytecMe,00:50:C2:52:A0:00/36Omnitron,00:50:C2:52:B0:00/36Sicon,00:50:C2:52:C0:00/36VitecMul,00:50:C2:52:D0:00/36Smartcom,00:50:C2:52:E0:00/36DspDesig,00:50:C2:52:F0:00/36Gesellsc,00:50:C2:53:00:00/36Innovati,00:50:C2:53:10:00/36OrionTec,00:50:C2:53:20:00/36NVE,00:50:C2:53:30:00/36NanShanB,00:50:C2:53:40:00/36HyundaiJ,00:50:C2:53:50:00/36MMSServi,00:50:C2:53:60:00/36C2Diagno,00:50:C2:53:70:00/36DstContr,00:50:C2:53:80:00/36EtherTek,00:50:C2:53:90:00/36Detectio,00:50:C2:53:A0:00/36ImageCon,00:50:C2:53:B0:00/36Teleks,00:50:C2:53:C0:00/36Marposs,00:50:C2:53:D0:00/36Digitalc,00:50:C2:53:E0:00/36Honeywel,00:50:C2:53:F0:00/36Ellips,00:50:C2:54:00:00/36MesureCo,00:50:C2:54:10:00/36WavesSys,00:50:C2:54:20:00/36AVerMedi,00:50:C2:54:30:00/36DigiSesn,00:50:C2:54:40:00/36Zetera,00:50:C2:54:50:00/36SecuInfo,00:50:C2:54:60:00/36Universi,00:50:C2:54:70:00/36BLANKOMA,00:50:C2:54:80:00/36ITWBetap,00:50:C2:54:90:00/36Netsynt,00:50:C2:54:A0:00/36IPTCTech,00:50:C2:54:B0:00/36Innopsys,00:50:C2:54:C0:00/36Sintecno,00:50:C2:54:D0:00/36SilentSy,00:50:C2:54:E0:00/36Converge,00:50:C2:54:F0:00/36Valtroni,00:50:C2:55:00:00/36LJUAutom,00:50:C2:55:10:00/36Innovati,00:50:C2:55:20:00/36Elfiq,00:50:C2:55:30:00/36ATHsyste,00:50:C2:55:40:00/36Weinzier,00:50:C2:55:50:00/36ControlA,00:50:C2:55:60:00/36Freiburg,00:50:C2:55:70:00/36Netcomse,00:50:C2:55:80:00/36BedoElek,00:50:C2:55:90:00/36FailSafe,00:50:C2:55:A0:00/36Valde,00:50:C2:55:B0:00/36MatrixTe,00:50:C2:55:C0:00/36ads-tec,00:50:C2:55:D0:00/36ACDElekt,00:50:C2:55:E0:00/36HanzasEl,00:50:C2:55:F0:00/36MoogBroa,00:50:C2:56:00:00/36ProconEl,00:50:C2:56:10:00/36SeitecEl,00:50:C2:56:20:00/36C21,00:50:C2:56:30:00/36OrtratSL,00:50:C2:56:40:00/36LastMile,00:50:C2:56:50:00/36Workpowe,00:50:C2:56:60:00/36ubinetsy,00:50:C2:56:70:00/36Tess,00:50:C2:56:80:00/36GeoFocus,00:50:C2:56:90:00/36TwinwinT,00:50:C2:56:A0:00/36GrupoEpe,00:50:C2:56:B0:00/36DatatonU,00:50:C2:56:C0:00/36Targeted,00:50:C2:56:D0:00/36Computro,00:50:C2:56:E0:00/36Lab-ElEl,00:50:C2:56:F0:00/36Gma,00:50:C2:57:00:00/36EllexMed,00:50:C2:57:10:00/36OberonSe,00:50:C2:57:20:00/36ChellIns,00:50:C2:57:30:00/36DATAMICR,00:50:C2:57:40:00/36Ingenier,00:50:C2:57:50:00/36Solystic,00:50:C2:57:60:00/36Visi-tec,00:50:C2:57:70:00/36Advanced,00:50:C2:57:80:00/36DelphiDi,00:50:C2:57:90:00/36Gastager,00:50:C2:57:A0:00/36nVentSch,00:50:C2:57:B0:00/36ptswitch,00:50:C2:57:C0:00/36éolane,00:50:C2:57:D0:00/36SierraVi,00:50:C2:57:E0:00/36DigitalW,00:50:C2:57:F0:00/36Orderite,00:50:C2:58:00:00/36BuyangEl,00:50:C2:58:10:00/36Devitech,00:50:C2:58:20:00/36AllSun,00:50:C2:58:30:00/36JungerAu,00:50:C2:58:40:00/36ToyotaMo,00:50:C2:58:50:00/36Wireless,00:50:C2:58:60:00/36Genetix,00:50:C2:58:70:00/36Dynalco,00:50:C2:58:80:00/36FederalE,00:50:C2:58:90:00/36HoribaAb,00:50:C2:58:A0:00/36Dixell,00:50:C2:58:B0:00/36Innovati,00:50:C2:58:C0:00/36LatticeS,00:50:C2:58:D0:00/36Zao,00:50:C2:58:E0:00/36PennyGil,00:50:C2:58:F0:00/36XoIP,00:50:C2:59:00:00/36EMMotors,00:50:C2:59:10:00/36Grosveno,00:50:C2:59:20:00/36PaloDExG,00:50:C2:59:30:00/36Broadlig,00:50:C2:59:40:00/36PixelVel,00:50:C2:59:50:00/36Callpod,00:50:C2:59:60:00/36Spansion,00:50:C2:59:70:00/36Nautel,00:50:C2:59:80:00/36Bundesam,00:50:C2:59:90:00/36FenTechn,00:50:C2:59:A0:00/36MultiTro,00:50:C2:59:B0:00/36Sapec,00:50:C2:59:C0:00/36DelsatGr,00:50:C2:59:D0:00/36DSSNetwo,00:50:C2:59:E0:00/36Legerity,00:50:C2:59:F0:00/36ads-tec,00:50:C2:5A:00:00/36RudolphT,00:50:C2:5A:10:00/36Vestfold,00:50:C2:5A:20:00/36iNET,00:50:C2:5A:30:00/36Lumel,00:50:C2:5A:40:00/36FederalS,00:50:C2:5A:50:00/36Equiposd,00:50:C2:5A:60:00/36PlasticL,00:50:C2:5A:70:00/36PhytecMe,00:50:C2:5A:80:00/36Etap,00:50:C2:5A:90:00/36AYCTelec,00:50:C2:5A:A0:00/36Transenn,00:50:C2:5A:B0:00/36EatonEle,00:50:C2:5A:C0:00/36Kinemetr,00:50:C2:5A:D0:00/36Emcom,00:50:C2:5A:E0:00/36CpsEurop,00:50:C2:5A:F0:00/36DorletSa,00:50:C2:5B:00:00/36INCOTEC,00:50:C2:5B:10:00/36Rosta,00:50:C2:5B:20:00/36Syntroni,00:50:C2:5B:30:00/36HITECOMS,00:50:C2:5B:40:00/36Terrasci,00:50:C2:5B:50:00/36Rafael,00:50:C2:5B:60:00/36KontronB,00:50:C2:5B:70:00/36AVerMedi,00:50:C2:5B:80:00/36Westfali,00:50:C2:5B:90:00/36TaiwanVi,00:50:C2:5B:A0:00/36SAIABurg,00:50:C2:5B:B0:00/36UnicTech,00:50:C2:5B:C0:00/36Guangzho,00:50:C2:5B:D0:00/36NomusCom,00:50:C2:5B:E0:00/36CardAcce,00:50:C2:5B:F0:00/36Techimp,00:50:C2:5C:00:00/36Pyott-Bo,00:50:C2:5C:10:00/36RLDrake,00:50:C2:5C:20:00/36Intuitiv,00:50:C2:5C:30:00/36KSSystem,00:50:C2:5C:40:00/36ProMik,00:50:C2:5C:50:00/36RadiantI,00:50:C2:5C:60:00/36Technisc,00:50:C2:5C:70:00/36InSyncTe,00:50:C2:5C:80:00/36GeorgiaT,00:50:C2:5C:90:00/36Shenzhen,00:50:C2:5C:A0:00/36BuyangEl,00:50:C2:5C:B0:00/36KoboldSi,00:50:C2:5C:C0:00/36Enseo,00:50:C2:5C:D0:00/36RADAElec,00:50:C2:5C:E0:00/36RokeMano,00:50:C2:5C:F0:00/36InnomedM,00:50:C2:5D:00:00/36Automata,00:50:C2:5D:10:00/36MeucciSo,00:50:C2:5D:20:00/36DA-Desig,00:50:C2:5D:30:00/36Wexiodis,00:50:C2:5D:40:00/36BuyangEl,00:50:C2:5D:50:00/36CannonTe,00:50:C2:5D:60:00/36BioAcces,00:50:C2:5D:70:00/36Synrad,00:50:C2:5D:80:00/36Technifo,00:50:C2:5D:90:00/36CrimsonM,00:50:C2:5D:A0:00/36TonnaEle,00:50:C2:5D:B0:00/36SecureSe,00:50:C2:5D:C0:00/36RMMichae,00:50:C2:5D:D0:00/36SomerDat,00:50:C2:5D:E0:00/36MagalSen,00:50:C2:5D:F0:00/36Gnutek,00:50:C2:5E:00:00/36PhytecMe,00:50:C2:5E:10:00/36IttiamP,00:50:C2:5E:20:00/36PYRAMIDC,00:50:C2:5E:30:00/36Computec,00:50:C2:5E:40:00/36BuyangEl,00:50:C2:5E:50:00/36Stresste,00:50:C2:5E:60:00/36Musatel,00:50:C2:5E:70:00/36EadsTest,00:50:C2:5E:80:00/36Info-Chi,00:50:C2:5E:90:00/36MicroTec,00:50:C2:5E:A0:00/36MicroEle,00:50:C2:5E:B0:00/36GarperTe,00:50:C2:5E:C0:00/36ASiSTech,00:50:C2:5E:D0:00/36Aquarott,00:50:C2:5E:E0:00/36Condre,00:50:C2:5E:F0:00/36pikkerto,00:50:C2:5F:00:00/36DIASInfr,00:50:C2:5F:10:00/36Technoma,00:50:C2:5F:20:00/36ESEMGrün,00:50:C2:5F:30:00/36POSNETPo,00:50:C2:5F:40:00/36TeamProj,00:50:C2:5F:50:00/36Genesis,00:50:C2:5F:60:00/36Cambridg,00:50:C2:5F:70:00/36Metrolog,00:50:C2:5F:80:00/36GrupoEpe,00:50:C2:5F:90:00/36ROTHARYS,00:50:C2:5F:A0:00/36LEAdoo,00:50:C2:5F:B0:00/36AllElect,00:50:C2:5F:C0:00/36FilmLigh,00:50:C2:5F:D0:00/36MEGElect,00:50:C2:5F:E0:00/36Novacomm,00:50:C2:5F:F0:00/36GazelleM,00:50:C2:60:00:00/36ProtecFi,00:50:C2:60:10:00/36MedAvant,00:50:C2:60:20:00/36ChauvinA,00:50:C2:60:30:00/36Cerus,00:50:C2:60:40:00/36HCJBGlob,00:50:C2:60:50:00/36Swistec,00:50:C2:60:60:00/36Shenzhen,00:50:C2:60:70:00/36TelecomF,00:50:C2:60:80:00/36SilexInd,00:50:C2:60:90:00/36Toptech,00:50:C2:60:A0:00/36GradualT,00:50:C2:60:B0:00/36Shanghai,00:50:C2:60:C0:00/36Identic,00:50:C2:60:D0:00/36Sicon,00:50:C2:60:E0:00/36Automati,00:50:C2:60:F0:00/36Kommunik,00:50:C2:61:00:00/36FDTManuf,00:50:C2:61:10:00/36Brookhav,00:50:C2:61:20:00/36IHP,00:50:C2:61:30:00/36Tattile,00:50:C2:61:40:00/36Proserv,00:50:C2:61:50:00/36AxisElec,00:50:C2:61:60:00/36Honeywel,00:50:C2:61:70:00/36Narinet,00:50:C2:61:80:00/36Intergra,00:50:C2:61:90:00/36Linkbit,00:50:C2:61:A0:00/36Communic,00:50:C2:61:B0:00/36NCITechn,00:50:C2:61:C0:00/36TestPro,00:50:C2:61:D0:00/36Sutus,00:50:C2:61:E0:00/36LesterEl,00:50:C2:61:F0:00/36ImagineC,00:50:C2:62:00:00/36HarmanBe,00:50:C2:62:10:00/36Version-,00:50:C2:62:20:00/362NTELEKO,00:50:C2:62:30:00/36Safeline,00:50:C2:62:40:00/36ComtestN,00:50:C2:62:50:00/36EBNeuro,00:50:C2:62:60:00/36WinsysIn,00:50:C2:62:70:00/36JungleSy,00:50:C2:62:80:00/36DAREDeve,00:50:C2:62:90:00/36MacDonal,00:50:C2:62:A0:00/36PrismaEn,00:50:C2:62:B0:00/36FirstCon,00:50:C2:62:C0:00/36AirMatri,00:50:C2:62:D0:00/36ProconEl,00:50:C2:62:E0:00/36TDMIngén,00:50:C2:62:F0:00/36Qes,00:50:C2:63:00:00/36AuroraFl,00:50:C2:63:10:00/36Fraunhof,00:50:C2:63:20:00/36RoseTech,00:50:C2:63:30:00/36RiceUniv,00:50:C2:63:40:00/36Sohon,00:50:C2:63:50:00/36Shockfis,00:50:C2:63:60:00/36dSPACE,00:50:C2:63:70:00/36Omnitrol,00:50:C2:63:80:00/36HUNGAROC,00:50:C2:63:90:00/36Qstreams,00:50:C2:63:A0:00/363DSP,00:50:C2:63:B0:00/36Powis,00:50:C2:63:C0:00/36dPictIma,00:50:C2:63:D0:00/36IDERs,00:50:C2:63:E0:00/36T2Commun,00:50:C2:63:F0:00/36SpeechTe,00:50:C2:64:00:00/36Iac,00:50:C2:64:10:00/36NEOInfor,00:50:C2:64:20:00/36StantonT,00:50:C2:64:30:00/36Enatel,00:50:C2:64:40:00/36PhytecMe,00:50:C2:64:50:00/36Software,00:50:C2:64:60:00/36Trutouch,00:50:C2:64:70:00/36R&DTechn,00:50:C2:64:80:00/36Fidelity,00:50:C2:64:90:00/36Pan-STAR,00:50:C2:64:A0:00/36CPqD,00:50:C2:64:B0:00/36MangoDSP,00:50:C2:64:C0:00/36CIS,00:50:C2:64:D0:00/36TeraInfo,00:50:C2:64:E0:00/36Northern,00:50:C2:64:F0:00/36MALighti,00:50:C2:65:00:00/36LiquidBr,00:50:C2:65:10:00/36Staer,00:50:C2:65:20:00/36WidecoSw,00:50:C2:65:30:00/36DobleEng,00:50:C2:65:40:00/36PaloDExG,00:50:C2:65:50:00/36PhysikIn,00:50:C2:65:60:00/36LDAAudio,00:50:C2:65:70:00/36Monytel,00:50:C2:65:80:00/36OpenPKG,00:50:C2:65:90:00/36DorsettT,00:50:C2:65:A0:00/36Hisstema,00:50:C2:65:B0:00/36Silverbr,00:50:C2:65:C0:00/36VTZdoo,00:50:C2:65:D0:00/36RedfoneC,00:50:C2:65:E0:00/36Cantion,00:50:C2:65:F0:00/36Invocon,00:50:C2:66:00:00/36Izisoft,00:50:C2:66:10:00/36PCE,00:50:C2:66:20:00/36AsiaPaci,00:50:C2:66:30:00/36COE,00:50:C2:66:40:00/36WestelWi,00:50:C2:66:50:00/36NetworkS,00:50:C2:66:60:00/36XworksNZ,00:50:C2:66:70:00/36Vocality,00:50:C2:66:80:00/36KeithKoe,00:50:C2:66:90:00/36DspDesig,00:50:C2:66:A0:00/36ABBTrans,00:50:C2:66:B0:00/36flsystem,00:50:C2:66:C0:00/36Desy,00:50:C2:66:D0:00/36DIGITEK,00:50:C2:66:E0:00/36Linear,00:50:C2:66:F0:00/36Nilan,00:50:C2:67:00:00/36NaimAudi,00:50:C2:67:10:00/36SkylineP,00:50:C2:67:20:00/36DDSElett,00:50:C2:67:30:00/36Ferrarie,00:50:C2:67:40:00/36ProtechO,00:50:C2:67:50:00/36KentonRe,00:50:C2:67:60:00/36Eds,00:50:C2:67:70:00/36ProconX,00:50:C2:67:80:00/36Ihm,00:50:C2:67:90:00/36Industri,00:50:C2:67:A0:00/36CC,00:50:C2:67:B0:00/36SpartonE,00:50:C2:67:C0:00/36GogoBA,00:50:C2:67:D0:00/36ESAMesst,00:50:C2:67:E0:00/36SAIABurg,00:50:C2:67:F0:00/36PhytecMe,00:50:C2:68:00:00/36HoneyNet,00:50:C2:68:10:00/36OwasysAd,00:50:C2:68:20:00/36Commet,00:50:C2:68:30:00/36MEGGITTS,00:50:C2:68:40:00/36REASONTe,00:50:C2:68:50:00/36Datamars,00:50:C2:68:60:00/36ANNAXAnz,00:50:C2:68:70:00/36AccessSp,00:50:C2:68:80:00/36ElkProdu,00:50:C2:68:90:00/36RFCode,00:50:C2:68:A0:00/36ZhuhaiJi,00:50:C2:68:B0:00/36Simtek,00:50:C2:68:C0:00/36Isochron,00:50:C2:68:D0:00/36CXRLarus,00:50:C2:68:E0:00/36Selco,00:50:C2:68:F0:00/36Bertroni,00:50:C2:69:00:00/36GHLBerha,00:50:C2:69:10:00/36Interopi,00:50:C2:69:20:00/36MateMedi,00:50:C2:69:30:00/36TechComm,00:50:C2:69:40:00/36Initel,00:50:C2:69:50:00/36Purelink,00:50:C2:69:60:00/36Casabyte,00:50:C2:69:70:00/36MonarchI,00:50:C2:69:80:00/36NavtechR,00:50:C2:69:90:00/36Bulleten,00:50:C2:69:A0:00/36StoreTec,00:50:C2:69:B0:00/36TsienUK,00:50:C2:69:C0:00/36BugLabs,00:50:C2:69:D0:00/36Dvation,00:50:C2:69:E0:00/36Ideus,00:50:C2:69:F0:00/36TotalRF,00:50:C2:6A:00:00/36GFPLab,00:50:C2:6A:10:00/36Pricol,00:50:C2:6A:20:00/36CadiScie,00:50:C2:6A:30:00/36CreaTech,00:50:C2:6A:40:00/36Teletask,00:50:C2:6A:50:00/36EATONFHF,00:50:C2:6A:60:00/36VictoryC,00:50:C2:6A:70:00/36HoerIndu,00:50:C2:6A:80:00/36DelcanTe,00:50:C2:6A:90:00/36ArmidaTe,00:50:C2:6A:A0:00/36Ifox-Ind,00:50:C2:6A:B0:00/36Software,00:50:C2:6A:C0:00/36ThalesUK,00:50:C2:6A:D0:00/36Heim-Bür,00:50:C2:6A:E0:00/36Qualisys,00:50:C2:6A:F0:00/36Nanoradi,00:50:C2:6B:00:00/36SmartKey,00:50:C2:6B:10:00/36BurkTech,00:50:C2:6B:20:00/36Edgeware,00:50:C2:6B:30:00/364RFCommu,00:50:C2:6B:40:00/36Somesca,00:50:C2:6B:50:00/36Triumf,00:50:C2:6B:60:00/36CommoDaS,00:50:C2:6B:70:00/36SystemLS,00:50:C2:6B:80:00/36Epec,00:50:C2:6B:90:00/36unipo,00:50:C2:6B:A0:00/36FertronC,00:50:C2:6B:B0:00/36EleMag,00:50:C2:6B:C0:00/36Paraytec,00:50:C2:6B:D0:00/36Mitron,00:50:C2:6B:E0:00/36ESTEC,00:50:C2:6B:F0:00/36Optoplan,00:50:C2:6C:00:00/36GlosterS,00:50:C2:6C:10:00/36RADIUSSw,00:50:C2:6C:20:00/36HoseoTel,00:50:C2:6C:30:00/36iTRACS,00:50:C2:6C:40:00/36REXXON,00:50:C2:6C:50:00/36Oerlikon,00:50:C2:6C:60:00/36MedAvant,00:50:C2:6C:70:00/36QuickCir,00:50:C2:6C:80:00/36B&SMEDIA,00:50:C2:6C:90:00/36Netami,00:50:C2:6C:A0:00/36DynamicH,00:50:C2:6C:B0:00/36StreamPr,00:50:C2:6C:C0:00/36WidmerTi,00:50:C2:6C:D0:00/36Rgm,00:50:C2:6C:E0:00/36EMITALLS,00:50:C2:6C:F0:00/36Microway,00:50:C2:6D:00:00/36EDSSyste,00:50:C2:6D:10:00/36Schnick-,00:50:C2:6D:20:00/36Lumistar,00:50:C2:6D:30:00/36DigiSens,00:50:C2:6D:40:00/36EtaniEle,00:50:C2:6D:50:00/36BeckerEl,00:50:C2:6D:60:00/36ADLElect,00:50:C2:6D:70:00/36MavenirS,00:50:C2:6D:80:00/36BLHealth,00:50:C2:6D:90:00/36Ajeco,00:50:C2:6D:A0:00/36TechnoFi,00:50:C2:6D:B0:00/36Gebhardt,00:50:C2:6D:C0:00/36L-3Commu,00:50:C2:6D:D0:00/36Zmicro,00:50:C2:6D:E0:00/36LaserToo,00:50:C2:6D:F0:00/36QRScienc,00:50:C2:6E:00:00/36FIRSTTRU,00:50:C2:6E:10:00/36NewOnSys,00:50:C2:6E:20:00/36PhytecMe,00:50:C2:6E:30:00/36MirosAS,00:50:C2:6E:40:00/36MangoDSP,00:50:C2:6E:50:00/36Boeckele,00:50:C2:6E:60:00/36Lanetco,00:50:C2:6E:70:00/36AceAxis,00:50:C2:6E:80:00/36Anymax,00:50:C2:6E:90:00/36Bandoele,00:50:C2:6E:A0:00/36Firstec,00:50:C2:6E:B0:00/36Harrison,00:50:C2:6E:C0:00/36Netistix,00:50:C2:6E:D0:00/36SechanEl,00:50:C2:6E:E0:00/36Interact,00:50:C2:6E:F0:00/36Pneumopa,00:50:C2:6F:00:00/36StanleyS,00:50:C2:6F:10:00/36ITSTelec,00:50:C2:6F:20:00/36LaserEle,00:50:C2:6F:30:00/36E3Switch,00:50:C2:6F:40:00/36Cryogeni,00:50:C2:6F:50:00/36KitronMi,00:50:C2:6F:60:00/36AVSatCom,00:50:C2:6F:70:00/36infoplan,00:50:C2:6F:80:00/36RVTechno,00:50:C2:6F:90:00/36Revox,00:50:C2:6F:A0:00/36Dcns,00:50:C2:6F:B0:00/36WaveIP,00:50:C2:6F:C0:00/36ActeSp,00:50:C2:6F:D0:00/36SAIABurg,00:50:C2:6F:E0:00/36BlueOrig,00:50:C2:6F:F0:00/36StMichae,00:50:C2:70:00:00/36Gem-MedS,00:50:C2:70:10:00/36KeithKoe,00:50:C2:70:20:00/36SPMInstr,00:50:C2:70:30:00/36SAEIT,00:50:C2:70:40:00/36DiniGrou,00:50:C2:70:50:00/36HauchBac,00:50:C2:70:60:00/36DioDigiW,00:50:C2:70:70:00/36DTechLab,00:50:C2:70:80:00/36Smartekd,00:50:C2:70:90:00/36ROVERLab,00:50:C2:70:A0:00/36Efficien,00:50:C2:70:B0:00/36BEARSolu,00:50:C2:70:C0:00/36Exertus,00:50:C2:70:D0:00/36ela-soft,00:50:C2:70:E0:00/36Audico,00:50:C2:70:F0:00/36ZumbachE,00:50:C2:71:00:00/36WhartonE,00:50:C2:71:10:00/36Linkit,00:50:C2:71:20:00/36Pasan,00:50:C2:71:30:00/363DX-Ray,00:50:C2:71:40:00/36TEAmSA,00:50:C2:71:50:00/36RIEXINGE,00:50:C2:71:60:00/36Mitrol,00:50:C2:71:70:00/36MBconnec,00:50:C2:71:80:00/36illunis,00:50:C2:71:90:00/36ennovati,00:50:C2:71:A0:00/36LogusBro,00:50:C2:71:B0:00/36ADVAOpti,00:50:C2:71:C0:00/36Elmec,00:50:C2:71:D0:00/36MG,00:50:C2:71:E0:00/36ASKIIndu,00:50:C2:71:F0:00/36ASCtelec,00:50:C2:72:00:00/36Colorado,00:50:C2:72:10:00/36Spectrum,00:50:C2:72:20:00/36CentricT,00:50:C2:72:30:00/36PowerEle,00:50:C2:72:40:00/36HSC-Rege,00:50:C2:72:50:00/36DspDesig,00:50:C2:72:60:00/36etasyste,00:50:C2:72:70:00/36Pelwecky,00:50:C2:72:80:00/36InterDig,00:50:C2:72:90:00/36SPContro,00:50:C2:72:A0:00/36PhytecMe,00:50:C2:72:B0:00/36Sequeste,00:50:C2:72:C0:00/36RichardG,00:50:C2:72:D0:00/36Physical,00:50:C2:72:E0:00/36SncfEimP,00:50:C2:72:F0:00/36Priority,00:50:C2:73:00:00/36haberkoe,00:50:C2:73:10:00/36SpirentC,00:50:C2:73:20:00/36Schlumbe,00:50:C2:73:30:00/36Cimetric,00:50:C2:73:40:00/36CardioME,00:50:C2:73:50:00/36AntLamp,00:50:C2:73:60:00/36Nika,00:50:C2:73:70:00/36Teradici,00:50:C2:73:80:00/36MiracomT,00:50:C2:73:90:00/36Tattile,00:50:C2:73:A0:00/36Naturela,00:50:C2:73:B0:00/36OnAirNet,00:50:C2:73:C0:00/36Simicon,00:50:C2:73:D0:00/36cryptiri,00:50:C2:73:E0:00/36QuantecN,00:50:C2:73:F0:00/36MEDAV,00:50:C2:74:00:00/36McQuayCh,00:50:C2:74:10:00/36Dain,00:50:C2:74:20:00/36Fantuzzi,00:50:C2:74:30:00/36Elektro-,00:50:C2:74:40:00/36Avonaco,00:50:C2:74:50:00/36Acisa,00:50:C2:74:60:00/36Realtron,00:50:C2:74:70:00/36CDSADamN,00:50:C2:74:80:00/36Letechni,00:50:C2:74:90:00/36Affolter,00:50:C2:74:A0:00/36MonitorE,00:50:C2:74:B0:00/36STAR-Dun,00:50:C2:74:C0:00/36Saia-Bur,00:50:C2:74:D0:00/36BeceemCo,00:50:C2:74:E0:00/36Tronico,00:50:C2:74:F0:00/36GermanTe,00:50:C2:75:00:00/36Brightli,00:50:C2:75:10:00/36e&sEngin,00:50:C2:75:20:00/36Lober,00:50:C2:75:30:00/36Abb,00:50:C2:75:40:00/36Abeo,00:50:C2:75:50:00/36TeletekE,00:50:C2:75:60:00/36Chesapea,00:50:C2:75:70:00/36ESPTechn,00:50:C2:75:80:00/36AixSolve,00:50:C2:75:90:00/36Sequentr,00:50:C2:75:A0:00/36GaislerR,00:50:C2:75:B0:00/36DMTSyste,00:50:C2:75:C0:00/36STÖRK-TR,00:50:C2:75:D0:00/36FluidAna,00:50:C2:75:E0:00/36Sky-Skan,00:50:C2:75:F0:00/36BRexroth,00:50:C2:76:00:00/36ArS,00:50:C2:76:10:00/36ElbitAme,00:50:C2:76:20:00/36Assembly,00:50:C2:76:30:00/36XtendWav,00:50:C2:76:40:00/36Argus-Sp,00:50:C2:76:50:00/36PhytecMe,00:50:C2:76:60:00/36Guterman,00:50:C2:76:70:00/36Eid,00:50:C2:76:80:00/36ControlS,00:50:C2:76:90:00/36BES,00:50:C2:76:A0:00/36Digidriv,00:50:C2:76:B0:00/36Putercom,00:50:C2:76:C0:00/36EFGCZspo,00:50:C2:76:D0:00/36Mobilism,00:50:C2:76:E0:00/36Crinia,00:50:C2:76:F0:00/36Controla,00:50:C2:77:00:00/36CadexEle,00:50:C2:77:10:00/36ZigBeeAl,00:50:C2:77:20:00/36IESElekt,00:50:C2:77:30:00/36PointeCo,00:50:C2:77:40:00/36GeoSIG,00:50:C2:77:50:00/36Laserdyn,00:50:C2:77:60:00/36Integrat,00:50:C2:77:70:00/36EuroDisp,00:50:C2:77:80:00/36SunGardV,00:50:C2:77:90:00/36CoralTel,00:50:C2:77:A0:00/36SmithMet,00:50:C2:77:B0:00/36ItibiaTe,00:50:C2:77:C0:00/36Atec,00:50:C2:77:D0:00/36LincolnI,00:50:C2:77:E0:00/36Cominfo,00:50:C2:77:F0:00/36ACDElekt,00:50:C2:78:00:00/36IQSoluti,00:50:C2:78:10:00/36Starling,00:50:C2:78:20:00/36PhytecMe,00:50:C2:78:30:00/36NORMA,00:50:C2:78:40:00/36LewisCon,00:50:C2:78:50:00/36IconTime,00:50:C2:78:60:00/36KeithKoe,00:50:C2:78:70:00/36AustcoMa,00:50:C2:78:80:00/36HosaTech,00:50:C2:78:90:00/36Rosslare,00:50:C2:78:A0:00/36LevelTel,00:50:C2:78:B0:00/36OMICRONe,00:50:C2:78:C0:00/36Giga-tro,00:50:C2:78:D0:00/36Telairit,00:50:C2:78:E0:00/36Globalco,00:50:C2:78:F0:00/36ELMARele,00:50:C2:79:00:00/36GESecuri,00:50:C2:79:10:00/36MSquared,00:50:C2:79:20:00/36SMARTRO,00:50:C2:79:30:00/36EnertexB,00:50:C2:79:40:00/36Comsonic,00:50:C2:79:50:00/36Ameli,00:50:C2:79:60:00/36DorletSa,00:50:C2:79:70:00/36Tiefenba,00:50:C2:79:80:00/36Indefia,00:50:C2:79:90:00/36Aavd,00:50:C2:79:A0:00/36JMCAmeri,00:50:C2:79:B0:00/36Schniewi,00:50:C2:79:C0:00/36Vital,00:50:C2:79:D0:00/36MiraTrek,00:50:C2:79:E0:00/36BenshawC,00:50:C2:79:F0:00/36Npc,00:50:C2:7A:00:00/36MedAvant,00:50:C2:7A:10:00/36FieldDes,00:50:C2:7A:20:00/36RaySatIs,00:50:C2:7A:30:00/36ABBTrans,00:50:C2:7A:40:00/36CalibreU,00:50:C2:7A:50:00/36QuantumM,00:50:C2:7A:60:00/36AsianaId,00:50:C2:7A:70:00/36Guidance,00:50:C2:7A:80:00/36Integrat,00:50:C2:7A:90:00/36DeltaTau,00:50:C2:7A:A0:00/36GrupoEpe,00:50:C2:7A:B0:00/36GeneralM,00:50:C2:7A:C0:00/36Iusa,00:50:C2:7A:D0:00/36TurunTur,00:50:C2:7A:E0:00/36GlobalTe,00:50:C2:7A:F0:00/36C2Micros,00:50:C2:7B:00:00/36IMPTelek,00:50:C2:7B:10:00/36Ateme,00:50:C2:7B:20:00/36ADIVideo,00:50:C2:7B:30:00/36Elmec,00:50:C2:7B:40:00/36T1Engine,00:50:C2:7B:50:00/36DIT-MCOI,00:50:C2:7B:60:00/36AlstomSc,00:50:C2:7B:70:00/36Tattile,00:50:C2:7B:80:00/36Design20,00:50:C2:7B:90:00/36Technova,00:50:C2:7B:A0:00/36InfodevE,00:50:C2:7B:B0:00/36InRaySol,00:50:C2:7B:C0:00/36Eidos,00:50:C2:7B:D0:00/36PromateE,00:50:C2:7B:E0:00/36Powerlin,00:50:C2:7B:F0:00/36ZoeMedic,00:50:C2:7C:00:00/36European,00:50:C2:7C:10:00/36PrimaryI,00:50:C2:7C:20:00/36DSRInfor,00:50:C2:7C:30:00/36Ast,00:50:C2:7C:40:00/36MoBaCon,00:50:C2:7C:50:00/36VentureR,00:50:C2:7C:60:00/36Lyngdorf,00:50:C2:7C:70:00/36Pyrosequ,00:50:C2:7C:80:00/36FrSauter,00:50:C2:7C:90:00/36Bluebell,00:50:C2:7C:A0:00/36CEDARAud,00:50:C2:7C:B0:00/36ViewPlus,00:50:C2:7C:C0:00/36SwecoJap,00:50:C2:7C:D0:00/36Precisio,00:50:C2:7C:E0:00/36GogoBA,00:50:C2:7C:F0:00/36Emitech,00:50:C2:7D:00:00/36RadarTro,00:50:C2:7D:10:00/36PhytecMe,00:50:C2:7D:20:00/36Bittital,00:50:C2:7D:30:00/36Highrail,00:50:C2:7D:40:00/36WR,00:50:C2:7D:50:00/36DEUTA-WE,00:50:C2:7D:60:00/36Internat,00:50:C2:7D:70:00/36Newtec,00:50:C2:7D:80:00/36InnoScan,00:50:C2:7D:90:00/36Volumati,00:50:C2:7D:A0:00/36HTEC,00:50:C2:7D:B0:00/36MuellerE,00:50:C2:7D:C0:00/36aiXtrusi,00:50:C2:7D:D0:00/36LSElektr,00:50:C2:7D:E0:00/36CascadeT,00:50:C2:7D:F0:00/36Private,00:50:C2:7E:00:00/36C&DTechn,00:50:C2:7E:10:00/36ZeltiqAe,00:50:C2:7E:20:00/36Digitrol,00:50:C2:7E:30:00/36Progente,00:50:C2:7E:40:00/36MetaVisi,00:50:C2:7E:50:00/36NystromE,00:50:C2:7E:60:00/36EmpirixI,00:50:C2:7E:70:00/36V2Green,00:50:C2:7E:80:00/36MistralS,00:50:C2:7E:90:00/36Sicon,00:50:C2:7E:A0:00/36MonitorB,00:50:C2:7E:B0:00/36SesolInd,00:50:C2:7E:C0:00/36Lyngsoe,00:50:C2:7E:D0:00/36GenesisA,00:50:C2:7E:E0:00/36NHResear,00:50:C2:7E:F0:00/36GFIChron,00:50:C2:7F:00:00/36NetworkH,00:50:C2:7F:10:00/36STUHLReg,00:50:C2:7F:20:00/36Logother,00:50:C2:7F:30:00/36Sorec,00:50:C2:7F:40:00/36Wireless,00:50:C2:7F:50:00/36ACECarwa,00:50:C2:7F:60:00/36Saia-Bur,00:50:C2:7F:70:00/36MangoDSP,00:50:C2:7F:80:00/36WiseIndu,00:50:C2:7F:90:00/36KarlDUNG,00:50:C2:7F:A0:00/36Automati,00:50:C2:7F:B0:00/36Qtron,00:50:C2:7F:C0:00/36TISDialo,00:50:C2:7F:D0:00/36Adeneo,00:50:C2:7F:E0:00/36Wireless,00:50:C2:7F:F0:00/36Shenzhen,00:50:C2:80:00:00/36DelphiDi,00:50:C2:80:10:00/36JANUS,00:50:C2:80:20:00/36Private,00:50:C2:80:30:00/36dBBroadc,00:50:C2:80:40:00/36SoftSwit,00:50:C2:80:50:00/36Multimed,00:50:C2:80:60:00/36Cet,00:50:C2:80:70:00/36Technoma,00:50:C2:80:80:00/36ITBCompu,00:50:C2:80:90:00/36VarmaEle,00:50:C2:80:A0:00/36PhytecMe,00:50:C2:80:B0:00/36OpenVide,00:50:C2:80:C0:00/36LuxpertT,00:50:C2:80:D0:00/36Acube,00:50:C2:80:E0:00/36BrunoInt,00:50:C2:80:F0:00/36Selekron,00:50:C2:81:00:00/36Alphion,00:50:C2:81:10:00/36OpenSyst,00:50:C2:81:20:00/36Femto,00:50:C2:81:30:00/36Intellef,00:50:C2:81:40:00/36Telvent,00:50:C2:81:50:00/36microCDe,00:50:C2:81:60:00/36Inteligh,00:50:C2:81:70:00/36OdinTele,00:50:C2:81:80:00/36Wireless,00:50:C2:81:90:00/36Cabinpla,00:50:C2:81:A0:00/36InfoGLOB,00:50:C2:81:B0:00/36BrainTec,00:50:C2:81:C0:00/36Telcom,00:50:C2:81:D0:00/36ItSalux,00:50:C2:81:E0:00/36Channelo,00:50:C2:81:F0:00/362NTELEKO,00:50:C2:82:00:00/36TESCANsr,00:50:C2:82:10:00/36MISCORef,00:50:C2:82:20:00/36WinnerTe,00:50:C2:82:30:00/36RobotVis,00:50:C2:82:40:00/36SMTdoo,00:50:C2:82:50:00/36Funkwerk,00:50:C2:82:60:00/36HEWIHein,00:50:C2:82:70:00/36EneroSol,00:50:C2:82:80:00/36SLICANsp,00:50:C2:82:90:00/36Intellec,00:50:C2:82:A0:00/36VDCDispl,00:50:C2:82:B0:00/36KeithKoe,00:50:C2:82:C0:00/36VitelNet,00:50:C2:82:D0:00/36Elmec,00:50:C2:82:E0:00/36LogiCom,00:50:C2:82:F0:00/36Momentum,00:50:C2:83:00:00/36CompuSho,00:50:C2:83:10:00/36StJudeMe,00:50:C2:83:20:00/36S1nn,00:50:C2:83:30:00/36LaserLin,00:50:C2:83:40:00/36ANTEK,00:50:C2:83:50:00/36Communic,00:50:C2:83:60:00/36DspDesig,00:50:C2:83:70:00/36ID-KARTA,00:50:C2:83:80:00/36TProjeMu,00:50:C2:83:90:00/36IMSRöntg,00:50:C2:83:A0:00/36Syr-TecE,00:50:C2:83:B0:00/36OBay,00:50:C2:83:C0:00/36hemaelec,00:50:C2:83:D0:00/36beroNet,00:50:C2:83:E0:00/36KPEspols,00:50:C2:83:F0:00/36PhytecMe,00:50:C2:84:00:00/36Resident,00:50:C2:84:10:00/36Connecti,00:50:C2:84:20:00/36QuantumC,00:50:C2:84:30:00/36Xtensor,00:50:C2:84:40:00/36ProdigyE,00:50:C2:84:50:00/36VisualSo,00:50:C2:84:60:00/36ESP-Plan,00:50:C2:84:70:00/36LarsMori,00:50:C2:84:80:00/36DASAROBO,00:50:C2:84:90:00/36DesignAn,00:50:C2:84:A0:00/36Keystone,00:50:C2:84:B0:00/36TaskSist,00:50:C2:84:C0:00/36Performa,00:50:C2:84:D0:00/36Bmti,00:50:C2:84:E0:00/36Draco,00:50:C2:84:F0:00/36Gamber-J,00:50:C2:85:00:00/36Rocky,00:50:C2:85:10:00/36SPJEmbed,00:50:C2:85:20:00/36eInfochi,00:50:C2:85:30:00/36EttusRes,00:50:C2:85:40:00/36Ratiopla,00:50:C2:85:50:00/36Rohde&Sc,00:50:C2:85:60:00/36CT,00:50:C2:85:70:00/36GrupoEpe,00:50:C2:85:80:00/36Wireless,00:50:C2:85:90:00/36Nuvation,00:50:C2:85:A0:00/36Art,00:50:C2:85:B0:00/36Boreste,00:50:C2:85:C0:00/36BSE,00:50:C2:85:D0:00/36IngKnaus,00:50:C2:85:E0:00/36Radiomet,00:50:C2:85:F0:00/36GDMissio,00:50:C2:86:00:00/36Eutron,00:50:C2:86:10:00/36Grantron,00:50:C2:86:20:00/36Elsys,00:50:C2:86:30:00/36Advanced,00:50:C2:86:40:00/36ATGAutom,00:50:C2:86:50:00/36PersyCon,00:50:C2:86:60:00/36SaiaBurg,00:50:C2:86:70:00/36Syntroni,00:50:C2:86:80:00/36Aethon,00:50:C2:86:90:00/36Funkwerk,00:50:C2:86:A0:00/36USM,00:50:C2:86:B0:00/36OMBSiste,00:50:C2:86:C0:00/36CondigiT,00:50:C2:86:D0:00/36TielineR,00:50:C2:86:E0:00/36HanyangE,00:50:C2:86:F0:00/36b-plus,00:50:C2:87:00:00/36Logel,00:50:C2:87:10:00/36R-S-IEle,00:50:C2:87:20:00/36Oliotalo,00:50:C2:87:30:00/36XRONET,00:50:C2:87:40:00/36ArcosTec,00:50:C2:87:50:00/36PhytecMe,00:50:C2:87:60:00/36Privatqu,00:50:C2:87:70:00/36MotionAn,00:50:C2:87:80:00/36Acoustic,00:50:C2:87:90:00/36Milesys,00:50:C2:87:A0:00/36Spectrum,00:50:C2:87:B0:00/36UAVNavig,00:50:C2:87:C0:00/36Arcontia,00:50:C2:87:D0:00/36AT&TGove,00:50:C2:87:E0:00/36ScmProdu,00:50:C2:87:F0:00/36Optoelet,00:50:C2:88:00:00/36Creation,00:50:C2:88:10:00/36InnoTran,00:50:C2:88:20:00/36Warecube,00:50:C2:88:30:00/36Neocontr,00:50:C2:88:40:00/36IPThinki,00:50:C2:88:50:00/36NtkImos,00:50:C2:88:60:00/36Wartsila,00:50:C2:88:70:00/36Inventis,00:50:C2:88:80:00/36IAdea,00:50:C2:88:90:00/36ACSMotio,00:50:C2:88:A0:00/36Continen,00:50:C2:88:B0:00/36HollisEl,00:50:C2:88:C0:00/36Z-App,00:50:C2:88:D0:00/36L3Commun,00:50:C2:88:E0:00/36Cardinal,00:50:C2:88:F0:00/36KeynoteS,00:50:C2:89:00:00/36BAEHäggl,00:50:C2:89:10:00/36AdmiralS,00:50:C2:89:20:00/36Trakce,00:50:C2:89:30:00/36EIZOTech,00:50:C2:89:40:00/36Shockfis,00:50:C2:89:50:00/36MarineCo,00:50:C2:89:60:00/36Blankom,00:50:C2:89:70:00/36ODFOptro,00:50:C2:89:80:00/36VeecoPro,00:50:C2:89:90:00/36InicoTec,00:50:C2:89:A0:00/36NeptuneT,00:50:C2:89:B0:00/36SensataT,00:50:C2:89:C0:00/36Mediana,00:50:C2:89:D0:00/36Systemte,00:50:C2:89:E0:00/36Broadcas,00:50:C2:89:F0:00/36Datalink,00:50:C2:8A:00:00/36Speciali,00:50:C2:8A:10:00/36IntuneNe,00:50:C2:8A:20:00/36UAVISION,00:50:C2:8A:30:00/36RTW,00:50:C2:8A:40:00/36BALOGHTA,00:50:C2:8A:50:00/36Mocon,00:50:C2:8A:60:00/36Selco,00:50:C2:8A:70:00/36Pixeye,00:50:C2:8A:80:00/36AltekEle,00:50:C2:8A:90:00/36Intellig,00:50:C2:8A:A0:00/36ATSElekt,00:50:C2:8A:B0:00/36Nanomoti,00:50:C2:8A:C0:00/36Telsa,00:50:C2:8A:D0:00/36ThalesCo,00:50:C2:8A:E0:00/36Desarrol,00:50:C2:8A:F0:00/36Xelerate,00:50:C2:8B:00:00/36BKInnova,00:50:C2:8B:10:00/36RingCube,00:50:C2:8B:20:00/36Servaind,00:50:C2:8B:30:00/36VTQVidet,00:50:C2:8B:40:00/36SandarTe,00:50:C2:8B:50:00/36KeithKoe,00:50:C2:8B:60:00/36Shadrins,00:50:C2:8B:70:00/36CalnexSo,00:50:C2:8B:80:00/36DSSNetwo,00:50:C2:8B:90:00/36ACDElekt,00:50:C2:8B:A0:00/36FrSauter,00:50:C2:8B:B0:00/36smtagint,00:50:C2:8B:C0:00/36Honeywel,00:50:C2:8B:D0:00/36MatrixSw,00:50:C2:8B:E0:00/36Pennsylv,00:50:C2:8B:F0:00/36ARISTOGr,00:50:C2:8C:00:00/36SCE,00:50:C2:8C:10:00/36HeraeusN,00:50:C2:8C:20:00/36AccessCo,00:50:C2:8C:30:00/36BytePara,00:50:C2:8C:40:00/36SoldigIn,00:50:C2:8C:50:00/36VortexEn,00:50:C2:8C:60:00/36GradualT,00:50:C2:8C:70:00/36Tattile,00:50:C2:8C:80:00/36Pumatron,00:50:C2:8C:90:00/36A+SAktua,00:50:C2:8C:A0:00/36Altairse,00:50:C2:8C:B0:00/36Beonic,00:50:C2:8C:C0:00/36LyconSys,00:50:C2:8C:D0:00/36Cambridg,00:50:C2:8C:E0:00/36PhytecMe,00:50:C2:8C:F0:00/36GigaLinx,00:50:C2:8D:00:00/36Saia-Bur,00:50:C2:8D:10:00/36Bachmann,00:50:C2:8D:20:00/36TTi,00:50:C2:8D:30:00/36IFAM,00:50:C2:8D:40:00/36Internet,00:50:C2:8D:50:00/36PeekTraf,00:50:C2:8D:60:00/36UltraVis,00:50:C2:8D:70:00/36PolygonI,00:50:C2:8D:80:00/36ArrayTec,00:50:C2:8D:90:00/36Industri,00:50:C2:8D:A0:00/36Docutemp,00:50:C2:8D:B0:00/36DCOMNetw,00:50:C2:8D:C0:00/36Frame,00:50:C2:8D:D0:00/36Gimcon,00:50:C2:8D:E0:00/36Coherix,00:50:C2:8D:F0:00/36Dipl-Ing,00:50:C2:8E:00:00/36Shenzhen,00:50:C2:8E:10:00/36Deutsche,00:50:C2:8E:20:00/36Wireless,00:50:C2:8E:30:00/36bioMérie,00:50:C2:8E:40:00/36MaCaPSIn,00:50:C2:8E:50:00/36Berthel,00:50:C2:8E:60:00/36SandelAv,00:50:C2:8E:70:00/36MKTSyste,00:50:C2:8E:80:00/36Friedric,00:50:C2:8E:90:00/36Unidata,00:50:C2:8E:A0:00/36Ateme,00:50:C2:8E:B0:00/36C-COMSat,00:50:C2:8E:C0:00/36BalfourB,00:50:C2:8E:D0:00/36AT-Autom,00:50:C2:8E:E0:00/36Pcsc,00:50:C2:8E:F0:00/36Technolo,00:50:C2:8F:00:00/36XentrasC,00:50:C2:8F:10:00/36Detectio,00:50:C2:8F:20:00/36Schneide,00:50:C2:8F:30:00/36CurtisDo,00:50:C2:8F:40:00/36Critical,00:50:C2:8F:50:00/36tec5,00:50:C2:8F:60:00/36K-MAC,00:50:C2:8F:70:00/36TGE,00:50:C2:8F:80:00/36Rmsd,00:50:C2:8F:90:00/36Honeywel,00:50:C2:8F:A0:00/36TELIUMsc,00:50:C2:8F:B0:00/36AlfredKu,00:50:C2:8F:C0:00/36Symetric,00:50:C2:8F:D0:00/36SindomaM,00:50:C2:8F:E0:00/36CrossCou,00:50:C2:8F:F0:00/36Luceat,00:50:C2:90:00:00/36MagorCom,00:50:C2:90:10:00/36Research,00:50:C2:90:20:00/36ChinaRai,00:50:C2:90:30:00/36EcoAxis,00:50:C2:90:40:00/36R2Sonic,00:50:C2:90:50:00/36LinkComm,00:50:C2:90:60:00/36Gidel,00:50:C2:90:70:00/36CristalC,00:50:C2:90:80:00/36CodexDig,00:50:C2:90:90:00/36Elisra,00:50:C2:90:A0:00/36BoardLev,00:50:C2:90:B0:00/36EONESSve,00:50:C2:90:C0:00/36LSS,00:50:C2:90:D0:00/36EVKDIKer,00:50:C2:90:E0:00/36PhytecMe,00:50:C2:90:F0:00/36INTEGRAB,00:50:C2:91:00:00/36Autotank,00:50:C2:91:10:00/36VaporRai,00:50:C2:91:20:00/36ASSETInt,00:50:C2:91:30:00/36Leonardo,00:50:C2:91:40:00/36IO-Conne,00:50:C2:91:50:00/36Verint,00:50:C2:91:60:00/36CHKGridS,00:50:C2:91:70:00/36Cirtem,00:50:C2:91:80:00/36DesignLi,00:50:C2:91:90:00/36AHV,00:50:C2:91:A0:00/36XtoneNet,00:50:C2:91:B0:00/36Embedded,00:50:C2:91:C0:00/36MangoDSP,00:50:C2:91:D0:00/36Rosendah,00:50:C2:91:E0:00/36Automati,00:50:C2:91:F0:00/362NcommDe,00:50:C2:92:00:00/36RogueEng,00:50:C2:92:10:00/36iQueRFID,00:50:C2:92:20:00/36MetrumSw,00:50:C2:92:30:00/36AmicusWi,00:50:C2:92:40:00/36LinkElec,00:50:C2:92:50:00/36PHBEletr,00:50:C2:92:60:00/36DiTESTFa,00:50:C2:92:70:00/36ATISgrou,00:50:C2:92:80:00/36Cinetix,00:50:C2:92:90:00/36FlightDe,00:50:C2:92:A0:00/36Rohde&Sc,00:50:C2:92:B0:00/36DspDesig,00:50:C2:92:C0:00/36Exatrol,00:50:C2:92:D0:00/36APProSof,00:50:C2:92:E0:00/36GoannaTe,00:50:C2:92:F0:00/36PhytecMe,00:50:C2:93:00:00/36NETAElek,00:50:C2:93:10:00/36KoreaTel,00:50:C2:93:20:00/36SMAVIS,00:50:C2:93:30:00/36Saia-Bur,00:50:C2:93:40:00/36Xilar,00:50:C2:93:50:00/36ImageVid,00:50:C2:93:60:00/36Margarit,00:50:C2:93:70:00/36BigBear,00:50:C2:93:80:00/36PostecDa,00:50:C2:93:90:00/36MosaicDy,00:50:C2:93:A0:00/36ALPHATRO,00:50:C2:93:B0:00/36Reliatro,00:50:C2:93:C0:00/36Fracture,00:50:C2:93:D0:00/36Lighting,00:50:C2:93:E0:00/36RCSCommu,00:50:C2:93:F0:00/36TSBSolut,00:50:C2:94:00:00/36Phitek,00:50:C2:94:10:00/36Rolbit,00:50:C2:94:20:00/36KeithKoe,00:50:C2:94:30:00/36QuanZhou,00:50:C2:94:40:00/36Wireonai,00:50:C2:94:50:00/36ExiFlowM,00:50:C2:94:60:00/36MEGWAREC,00:50:C2:94:70:00/36IMEXHIGH,00:50:C2:94:80:00/36Electron,00:50:C2:94:90:00/36taskit,00:50:C2:94:A0:00/36TruMedia,00:50:C2:94:B0:00/36Pilleren,00:50:C2:94:C0:00/36Temix,00:50:C2:94:D0:00/36C&Htechn,00:50:C2:94:E0:00/36ZynixOri,00:50:C2:94:F0:00/36IT-Desig,00:50:C2:95:00:00/36TeleandR,00:50:C2:95:10:00/36ELCAsocc,00:50:C2:95:20:00/36TechFass,00:50:C2:95:30:00/36GrupoEpe,00:50:C2:95:40:00/36PhytecMe,00:50:C2:95:50:00/36Roessman,00:50:C2:95:60:00/36Sicon,00:50:C2:95:70:00/36STRATECC,00:50:C2:95:80:00/36Sensopti,00:50:C2:95:90:00/36DECTRIS,00:50:C2:95:A0:00/36TechnoAP,00:50:C2:95:B0:00/36ASSolar,00:50:C2:95:C0:00/36Resurgen,00:50:C2:95:D0:00/36fullelec,00:50:C2:95:E0:00/36BEEcube,00:50:C2:95:F0:00/36Metronic,00:50:C2:96:00:00/36kuroneko,00:50:C2:96:10:00/36Picsolve,00:50:C2:96:20:00/36Shockfis,00:50:C2:96:30:00/36Lécureux,00:50:C2:96:40:00/36IQAutoma,00:50:C2:96:50:00/36Emitech,00:50:C2:96:60:00/36PCMIndus,00:50:C2:96:70:00/36Watthour,00:50:C2:96:80:00/36BuLogics,00:50:C2:96:90:00/36GehrkeKo,00:50:C2:96:A0:00/36BittiumW,00:50:C2:96:B0:00/36Electron,00:50:C2:96:C0:00/36AquaCool,00:50:C2:96:D0:00/36KeeneEle,00:50:C2:96:E0:00/36PeekTraf,00:50:C2:96:F0:00/36Varec,00:50:C2:97:00:00/36TsujiEle,00:50:C2:97:10:00/36Ipitek,00:50:C2:97:20:00/36SwitchSc,00:50:C2:97:30:00/36Systèmes,00:50:C2:97:40:00/36EMAC,00:50:C2:97:50:00/36PyramidT,00:50:C2:97:60:00/36SandsIns,00:50:C2:97:70:00/36Saia-Bur,00:50:C2:97:80:00/36LOGITALD,00:50:C2:97:90:00/36FarSouth,00:50:C2:97:A0:00/36KSTTechn,00:50:C2:97:B0:00/36Smartqua,00:50:C2:97:C0:00/36CreaconT,00:50:C2:97:D0:00/36SoehnleP,00:50:C2:97:E0:00/36RFIndust,00:50:C2:97:F0:00/36C&I,00:50:C2:98:00:00/36DigitalP,00:50:C2:98:10:00/36Novotron,00:50:C2:98:20:00/36TripleRi,00:50:C2:98:30:00/36BogartEn,00:50:C2:98:40:00/36Atel,00:50:C2:98:50:00/36Earnestc,00:50:C2:98:60:00/36Dsci,00:50:C2:98:70:00/36Joinsoon,00:50:C2:98:80:00/36PantelIn,00:50:C2:98:90:00/36Psigenic,00:50:C2:98:A0:00/36MEV,00:50:C2:98:B0:00/36Ti2000Te,00:50:C2:98:C0:00/36MGM-Devi,00:50:C2:98:D0:00/36Mecos,00:50:C2:98:E0:00/36LinkTech,00:50:C2:98:F0:00/36BelikSPR,00:50:C2:99:00:00/36KeithKoe,00:50:C2:99:10:00/36UGL,00:50:C2:99:20:00/36IDTSound,00:50:C2:99:30:00/36Unetconv,00:50:C2:99:40:00/36XafaxNed,00:50:C2:99:50:00/36InterCon,00:50:C2:99:60:00/36Commerci,00:50:C2:99:70:00/36DeproÉle,00:50:C2:99:80:00/36PhytecMe,00:50:C2:99:90:00/36Cambusti,00:50:C2:99:A0:00/36Miromico,00:50:C2:99:B0:00/36Bettini,00:50:C2:99:C0:00/36CaTs3,00:50:C2:99:D0:00/36Powersen,00:50:C2:99:E0:00/36EngageTe,00:50:C2:99:F0:00/36SietronE,00:50:C2:9A:00:00/36Trs,00:50:C2:9A:10:00/36ComApsro,00:50:C2:9A:20:00/36SAMsyste,00:50:C2:9A:30:00/36Computer,00:50:C2:9A:40:00/36Entwickl,00:50:C2:9A:50:00/36Conolog,00:50:C2:9A:60:00/36Metodo2,00:50:C2:9A:70:00/36ThalesCo,00:50:C2:9A:80:00/36Domis,00:50:C2:9A:90:00/36GDMissio,00:50:C2:9A:A0:00/36TEKOTELE,00:50:C2:9A:B0:00/36Electrod,00:50:C2:9A:C0:00/36QuestekA,00:50:C2:9A:D0:00/36ChronosT,00:50:C2:9A:E0:00/36Esensors,00:50:C2:9A:F0:00/36KRESS-NE,00:50:C2:9B:00:00/36Ebru,00:50:C2:9B:10:00/36BonHora,00:50:C2:9B:20:00/36MesaLabs,00:50:C2:9B:30:00/36KahlerAu,00:50:C2:9B:40:00/36EukreaEl,00:50:C2:9B:50:00/36Telegamm,00:50:C2:9B:60:00/36Actech,00:50:C2:9B:70:00/36StMichae,00:50:C2:9B:80:00/36SoundPla,00:50:C2:9B:90:00/36ISA-Inte,00:50:C2:9B:A0:00/36Connor-W,00:50:C2:9B:B0:00/36OMICRONe,00:50:C2:9B:C0:00/36VesterEl,00:50:C2:9B:D0:00/36Sensitro,00:50:C2:9B:E0:00/36XadCommu,00:50:C2:9B:F0:00/362NTELEKO,00:50:C2:9C:00:00/36StuytsEn,00:50:C2:9C:10:00/36Tattile,00:50:C2:9C:20:00/36TeamEngi,00:50:C2:9C:30:00/36GESecuri,00:50:C2:9C:40:00/36VitelNet,00:50:C2:9C:50:00/36Scansoni,00:50:C2:9C:60:00/36Protroni,00:50:C2:9C:70:00/36KumeraDr,00:50:C2:9C:80:00/36ethermet,00:50:C2:9C:90:00/36LUMINEXL,00:50:C2:9C:A0:00/36ESAB-ATA,00:50:C2:9C:B0:00/36NIS-time,00:50:C2:9C:C0:00/36Hirotech,00:50:C2:9C:D0:00/36UweSchne,00:50:C2:9C:E0:00/36RonanEng,00:50:C2:9C:F0:00/36Intuitiv,00:50:C2:9D:00:00/36JDITTRIC,00:50:C2:9D:10:00/36Bladeliu,00:50:C2:9D:20:00/36Saia-Bur,00:50:C2:9D:30:00/36Telemetr,00:50:C2:9D:40:00/36First,00:50:C2:9D:50:00/36Netpower,00:50:C2:9D:60:00/36Innovati,00:50:C2:9D:70:00/36Melex,00:50:C2:9D:80:00/36SamsungH,00:50:C2:9D:90:00/36CNS,00:50:C2:9D:A0:00/36NEUTRONI,00:50:C2:9D:B0:00/36WalterGr,00:50:C2:9D:C0:00/36FTMMarke,00:50:C2:9D:D0:00/36Institut,00:50:C2:9D:E0:00/36ChauvinA,00:50:C2:9D:F0:00/36CODEC,00:50:C2:9E:00:00/36DSTSwiss,00:50:C2:9E:10:00/36Enreduce,00:50:C2:9E:20:00/36E-ViEWSS,00:50:C2:9E:30:00/36SAIInfor,00:50:C2:9E:40:00/36PyxisCon,00:50:C2:9E:50:00/36Hallibur,00:50:C2:9E:60:00/36KumhoEle,00:50:C2:9E:70:00/36DorletSa,00:50:C2:9E:80:00/36Hammock,00:50:C2:9E:90:00/36CiemmeSi,00:50:C2:9E:A0:00/36SISMODUL,00:50:C2:9E:B0:00/36AFORESol,00:50:C2:9E:C0:00/36Rohde&Sc,00:50:C2:9E:D0:00/36Picell,00:50:C2:9E:E0:00/36MichaelS,00:50:C2:9E:F0:00/36WoKa-Ele,00:50:C2:9F:00:00/36Veracity,00:50:C2:9F:10:00/36IDEAS,00:50:C2:9F:20:00/36KeithKoe,00:50:C2:9F:30:00/36VisionTe,00:50:C2:9F:40:00/36Fsr,00:50:C2:9F:50:00/36CommexTe,00:50:C2:9F:60:00/36IonSense,00:50:C2:9F:70:00/36DaveJone,00:50:C2:9F:80:00/36AustcoMa,00:50:C2:9F:90:00/36ABBTrans,00:50:C2:9F:A0:00/36TeranexA,00:50:C2:9F:B0:00/36VillbauK,00:50:C2:9F:C0:00/36Ectec,00:50:C2:9F:D0:00/36Bitttech,00:50:C2:9F:E0:00/36SpectraE,00:50:C2:9F:F0:00/36Humphrey,00:50:C2:A0:00:00/36Technova,00:50:C2:A0:10:00/36Patronic,00:50:C2:A0:20:00/36Referenc,00:50:C2:A0:30:00/36EEGEnter,00:50:C2:A0:40:00/36TPRadio,00:50:C2:A0:50:00/36AdgilDes,00:50:C2:A0:60:00/36CloosEle,00:50:C2:A0:70:00/36DynonIns,00:50:C2:A0:80:00/36LabJack,00:50:C2:A0:90:00/36Innovati,00:50:C2:A0:A0:00/36ACDElekt,00:50:C2:A0:B0:00/36IDSGeoRa,00:50:C2:A0:C0:00/36PhytecMe,00:50:C2:A0:D0:00/36Charlyro,00:50:C2:A0:E0:00/36ENGICAM,00:50:C2:A0:F0:00/36Visualwa,00:50:C2:A1:00:00/36Essentia,00:50:C2:A1:10:00/36OJSCRawe,00:50:C2:A1:20:00/36HCEEngin,00:50:C2:A1:30:00/36Talyst,00:50:C2:A1:40:00/36ElbitAme,00:50:C2:A1:50:00/36Industri,00:50:C2:A1:60:00/36Baudisch,00:50:C2:A1:70:00/36WinnersS,00:50:C2:A1:80:00/36Eoslink,00:50:C2:A1:90:00/36IconTime,00:50:C2:A1:A0:00/36Ddl,00:50:C2:A1:B0:00/36Realtime,00:50:C2:A1:C0:00/36Microtec,00:50:C2:A1:D0:00/36SAMHEngi,00:50:C2:A1:E0:00/36MAMAC,00:50:C2:A1:F0:00/36FlightDa,00:50:C2:A2:00:00/36QuorumTe,00:50:C2:A2:10:00/36Isac,00:50:C2:A2:20:00/36NipponMa,00:50:C2:A2:30:00/36AgilityM,00:50:C2:A2:40:00/36GrupoEpe,00:50:C2:A2:50:00/36Saia-Bur,00:50:C2:A2:60:00/36Preferre,00:50:C2:A2:70:00/36meconete,00:50:C2:A2:80:00/36KendaEle,00:50:C2:A2:90:00/36Luminex,00:50:C2:A2:A0:00/36Astronic,00:50:C2:A2:B0:00/36ApriliaR,00:50:C2:A2:C0:00/36KWS-Elec,00:50:C2:A2:D0:00/36Inventur,00:50:C2:A2:E0:00/36YUYAMAMF,00:50:C2:A2:F0:00/36DragonFl,00:50:C2:A3:00:00/36D-TA,00:50:C2:A3:10:00/36Coolit,00:50:C2:A3:20:00/36HarrisDe,00:50:C2:A3:30:00/36FujiFirm,00:50:C2:A3:40:00/36Casabyte,00:50:C2:A3:50:00/36Appareo,00:50:C2:A3:60:00/36Shenzhen,00:50:C2:A3:70:00/36Software,00:50:C2:A3:80:00/36TredDisp,00:50:C2:A3:90:00/36Industri,00:50:C2:A3:A0:00/36Telecor,00:50:C2:A3:B0:00/36IPcontro,00:50:C2:A3:C0:00/36BrählerI,00:50:C2:A3:D0:00/36Owandy,00:50:C2:A3:E0:00/36DueviSnc,00:50:C2:A3:F0:00/36LHACC,00:50:C2:A4:00:00/36Mosberge,00:50:C2:A4:10:00/36MeiryoDe,00:50:C2:A4:20:00/36RealVisi,00:50:C2:A4:30:00/36NKS,00:50:C2:A4:40:00/36TORCTech,00:50:C2:A4:50:00/36Sofradir,00:50:C2:A4:60:00/36Softroni,00:50:C2:A4:70:00/36Primetec,00:50:C2:A4:80:00/36ThalesUK,00:50:C2:A4:90:00/36WayneDal,00:50:C2:A4:A0:00/36DITRON,00:50:C2:A4:B0:00/36L-3Commu,00:50:C2:A4:C0:00/36VasoNova,00:50:C2:A4:D0:00/36LevelSta,00:50:C2:A4:E0:00/36Conduant,00:50:C2:A4:F0:00/36Deuta,00:50:C2:A5:00:00/36i-REDInf,00:50:C2:A5:10:00/36Y-produc,00:50:C2:A5:20:00/36VON,00:50:C2:A5:30:00/36QualityD,00:50:C2:A5:40:00/36DiamondP,00:50:C2:A5:50:00/36Arrowval,00:50:C2:A5:60:00/36ReaMetri,00:50:C2:A5:70:00/36JuiceTec,00:50:C2:A5:80:00/36Epl,00:50:C2:A5:90:00/36GSPSprac,00:50:C2:A5:A0:00/36Itas,00:50:C2:A5:B0:00/36PhytecMe,00:50:C2:A5:C0:00/36Componen,00:50:C2:A5:D0:00/36Mecc,00:50:C2:A5:E0:00/36AnsenInv,00:50:C2:A5:F0:00/36AlgaMicr,00:50:C2:A6:00:00/36ArrowCen,00:50:C2:A6:10:00/36FrSauter,00:50:C2:A6:20:00/36Grossenb,00:50:C2:A6:30:00/36EMSIndus,00:50:C2:A6:40:00/36tetronik,00:50:C2:A6:50:00/36Mark-O-P,00:50:C2:A6:60:00/36DVTech,00:50:C2:A6:70:00/36GSSAvion,00:50:C2:A6:80:00/36X-PertPa,00:50:C2:A6:90:00/36Advanced,00:50:C2:A6:A0:00/36Infocros,00:50:C2:A6:B0:00/36Explorer,00:50:C2:A6:C0:00/36FigmentD,00:50:C2:A6:D0:00/36DTVInnov,00:50:C2:A6:E0:00/36ScreenTe,00:50:C2:A6:F0:00/36Saia-Bur,00:50:C2:A7:00:00/36Reliable,00:50:C2:A7:10:00/36Purite,00:50:C2:A7:20:00/36Gamber-J,00:50:C2:A7:30:00/36KYOEIENG,00:50:C2:A7:40:00/36DspDesig,00:50:C2:A7:50:00/36JTL,00:50:C2:A7:60:00/36RoeschWa,00:50:C2:A7:70:00/36KeithKoe,00:50:C2:A7:80:00/36Apantac,00:50:C2:A7:90:00/36Saintron,00:50:C2:A7:A0:00/36DetNetSo,00:50:C2:A7:B0:00/36OrangeTr,00:50:C2:A7:C0:00/36Pneu-Log,00:50:C2:A7:D0:00/36VitelNet,00:50:C2:A7:E0:00/36Densitro,00:50:C2:A7:F0:00/36PhytecMe,00:50:C2:A8:00:00/36ard,00:50:C2:A8:10:00/36BPCcircu,00:50:C2:A8:20:00/36CT,00:50:C2:A8:30:00/36TechnoSo,00:50:C2:A8:40:00/36LinoManf,00:50:C2:A8:50:00/36Joysyste,00:50:C2:A8:60:00/36Limab,00:50:C2:A8:70:00/36Littlemo,00:50:C2:A8:80:00/36S-Sys,00:50:C2:A8:90:00/36CATraffi,00:50:C2:A8:A0:00/36AudioEng,00:50:C2:A8:B0:00/36Navicron,00:50:C2:A8:C0:00/36Redwire,00:50:C2:A8:D0:00/36Frontier,00:50:C2:A8:E0:00/36BFIIndus,00:50:C2:A8:F0:00/36Quantum3,00:50:C2:A9:00:00/36Stwo,00:50:C2:A9:10:00/36CeronTec,00:50:C2:A9:20:00/36Sicon,00:50:C2:A9:30:00/36SPXFlowT,00:50:C2:A9:40:00/36Par-Tech,00:50:C2:A9:50:00/36INNOVACI,00:50:C2:A9:60:00/36Fep,00:50:C2:A9:70:00/36Microsys,00:50:C2:A9:80:00/36Sentry36,00:50:C2:A9:90:00/36Haivisio,00:50:C2:A9:A0:00/36Absolutr,00:50:C2:A9:B0:00/36PDQManuf,00:50:C2:A9:C0:00/36StarElec,00:50:C2:A9:D0:00/36JoehlKoe,00:50:C2:A9:E0:00/36ProconEn,00:50:C2:A9:F0:00/36YellowSo,00:50:C2:AA:00:00/36SmithMet,00:50:C2:AA:10:00/36ElremEle,00:50:C2:AA:20:00/36ELPAsas,00:50:C2:AA:30:00/36PeekTraf,00:50:C2:AA:40:00/36PSiPrint,00:50:C2:AA:50:00/36TampereU,00:50:C2:AA:60:00/36BassettE,00:50:C2:AA:70:00/36Endeas,00:50:C2:AA:80:00/36NexansCa,00:50:C2:AA:90:00/36SANGIORG,00:50:C2:AA:A0:00/36Flexible,00:50:C2:AA:B0:00/36BRSSiste,00:50:C2:AA:C0:00/36VisiCon,00:50:C2:AA:D0:00/36Update,00:50:C2:AA:E0:00/36OUTLINE,00:50:C2:AA:F0:00/36SantaBar,00:50:C2:AB:00:00/36FRAKOKon,00:50:C2:AB:10:00/36Bitmanuf,00:50:C2:AB:20:00/36ProCom,00:50:C2:AB:30:00/36Compañía,00:50:C2:AB:40:00/36n3kInfor,00:50:C2:AB:50:00/36MettlerT,00:50:C2:AB:60:00/36GygaxEmb,00:50:C2:AB:70:00/36Twinfall,00:50:C2:AB:80:00/36AHMCLiKA,00:50:C2:AB:90:00/36Showtacl,00:50:C2:AB:A0:00/36Saia-Bur,00:50:C2:AB:B0:00/36Volantic,00:50:C2:AB:C0:00/36Barrick,00:50:C2:AB:D0:00/36MonitorB,00:50:C2:AB:E0:00/36APLabs,00:50:C2:AB:F0:00/36MCCCompu,00:50:C2:AC:00:00/36DSPROAud,00:50:C2:AC:10:00/36DAISHIN-,00:50:C2:AC:20:00/36OpenXS,00:50:C2:AC:30:00/36Diversif,00:50:C2:AC:40:00/36OrionTec,00:50:C2:AC:50:00/36E-Motion,00:50:C2:AC:60:00/36Marathon,00:50:C2:AC:70:00/36WaveIP,00:50:C2:AC:80:00/36Palladio,00:50:C2:AC:90:00/36Steinbei,00:50:C2:AC:A0:00/36SoftCont,00:50:C2:AC:B0:00/36U-Care,00:50:C2:AC:C0:00/36ProPhoto,00:50:C2:AC:D0:00/36MeshWork,00:50:C2:AC:E0:00/36ChronoLo,00:50:C2:AC:F0:00/36SPContro,00:50:C2:AD:00:00/36Geonauti,00:50:C2:AD:10:00/36PhytecMe,00:50:C2:AD:20:00/36Rafael,00:50:C2:AD:30:00/36PeekTraf,00:50:C2:AD:40:00/36GlobalRa,00:50:C2:AD:50:00/36MightyLu,00:50:C2:AD:60:00/36Unisenso,00:50:C2:AD:70:00/36AirMonit,00:50:C2:AD:80:00/36Incyma,00:50:C2:AD:90:00/36elettron,00:50:C2:AD:A0:00/36Essepie,00:50:C2:AD:B0:00/36GOengine,00:50:C2:AD:C0:00/36Synthese,00:50:C2:AD:D0:00/36GDMissio,00:50:C2:AD:E0:00/36Neoptix,00:50:C2:AD:F0:00/36Altinex,00:50:C2:AE:00:00/36AT4wirel,00:50:C2:AE:10:00/36Evercare,00:50:C2:AE:20:00/36PowerMed,00:50:C2:AE:30:00/36Psd,00:50:C2:AE:40:00/36Advanced,00:50:C2:AE:50:00/36ABSGesel,00:50:C2:AE:60:00/36VecomUsa,00:50:C2:AE:70:00/36Redwood,00:50:C2:AE:80:00/36Bit-Lab,00:50:C2:AE:90:00/36ClearCor,00:50:C2:AE:A0:00/36Embedia,00:50:C2:AE:B0:00/36UMLogics,00:50:C2:AE:C0:00/36FritzPau,00:50:C2:AE:D0:00/363Roam,00:50:C2:AE:E0:00/36IPtec,00:50:C2:AE:F0:00/36National,00:50:C2:AF:00:00/36FrSauter,00:50:C2:AF:10:00/36GreenGoo,00:50:C2:AF:20:00/36ACDElekt,00:50:C2:AF:30:00/36PalomarP,00:50:C2:AF:40:00/36Dixell,00:50:C2:AF:50:00/36Kramaras,00:50:C2:AF:60:00/36Energid,00:50:C2:AF:70:00/36MidwestM,00:50:C2:AF:80:00/36GlobalSa,00:50:C2:AF:90:00/36Ingenieu,00:50:C2:AF:A0:00/36Absolute,00:50:C2:AF:B0:00/36Vocality,00:50:C2:AF:C0:00/36OdusTech,00:50:C2:AF:D0:00/36HomeScen,00:50:C2:AF:E0:00/36Trolex,00:50:C2:AF:F0:00/36XoByte,00:50:C2:B0:00:00/36Saia-Bur,00:50:C2:B0:10:00/36HSRHaral,00:50:C2:B0:20:00/36Master,00:50:C2:B0:30:00/36SpiderTe,00:50:C2:B0:40:00/36Ubiquiti,00:50:C2:B0:50:00/36POLA,00:50:C2:B0:60:00/36CompuDes,00:50:C2:B0:70:00/36Fareco,00:50:C2:B0:80:00/36Goerlitz,00:50:C2:B0:90:00/36HarperCh,00:50:C2:B0:A0:00/36Induther,00:50:C2:B0:B0:00/36Honeywel,00:50:C2:B0:C0:00/36SmartbTe,00:50:C2:B0:D0:00/36JapanEle,00:50:C2:B0:E0:00/36KYABLule,00:50:C2:B0:F0:00/36NARACont,00:50:C2:B1:00:00/36MarineEn,00:50:C2:B1:10:00/36EXEL,00:50:C2:B1:20:00/36CMElektr,00:50:C2:B1:30:00/36MeasyEle,00:50:C2:B1:40:00/36KeithKoe,00:50:C2:B1:50:00/36PhotoTel,00:50:C2:B1:60:00/36Neothing,00:50:C2:B1:70:00/36ElcoteqD,00:50:C2:B1:80:00/36Rosslare,00:50:C2:B1:90:00/36Polytron,00:50:C2:B1:A0:00/36Elcus,00:50:C2:B1:B0:00/36Integrat,00:50:C2:B1:C0:00/36PhytecMe,00:50:C2:B1:D0:00/36Telegeni,00:50:C2:B1:E0:00/36AbbottMe,00:50:C2:B1:F0:00/36AtlasCop,00:50:C2:B2:00:00/36Five9Net,00:50:C2:B2:10:00/36Phytron-,00:50:C2:B2:20:00/36FarSiteC,00:50:C2:B2:30:00/36RonyoTec,00:50:C2:B2:40:00/36Teledyne,00:50:C2:B2:50:00/36Triax,00:50:C2:B2:60:00/36Elko,00:50:C2:B2:70:00/36Ateme,00:50:C2:B2:80:00/36Micromax,00:50:C2:B2:90:00/36IntegraL,00:50:C2:B2:A0:00/36TrenchAu,00:50:C2:B2:B0:00/36CosmoDat,00:50:C2:B2:C0:00/36Concepte,00:50:C2:B2:D0:00/36DatasatD,00:50:C2:B2:E0:00/36Act,00:50:C2:B2:F0:00/36IntelliV,00:50:C2:B3:00:00/36AppliedM,00:50:C2:B3:10:00/36ShopSafe,00:50:C2:B3:20:00/36ByresSec,00:50:C2:B3:30:00/36Numcore,00:50:C2:B3:40:00/36Meisol,00:50:C2:B3:50:00/36haneron,00:50:C2:B3:60:00/36Crde,00:50:C2:B3:70:00/36IAdea,00:50:C2:B3:80:00/36Grenmore,00:50:C2:B3:90:00/36siXis,00:50:C2:B3:A0:00/36Nikon,00:50:C2:B3:B0:00/36Sportvis,00:50:C2:B3:C0:00/36JanasCar,00:50:C2:B3:D0:00/36Ams,00:50:C2:B3:E0:00/36SageCons,00:50:C2:B3:F0:00/36M-Tronic,00:50:C2:B4:00:00/36TecnintH,00:50:C2:B4:10:00/36TataPowe,00:50:C2:B4:20:00/36ETMElect,00:50:C2:B4:30:00/36J,00:50:C2:B4:40:00/36Ampcontr,00:50:C2:B4:50:00/36Efftroni,00:50:C2:B4:60:00/36Mobileye,00:50:C2:B4:70:00/36MCSMICRO,00:50:C2:B4:80:00/36MTD,00:50:C2:B4:90:00/36AplexTec,00:50:C2:B4:A0:00/36Saia-Bur,00:50:C2:B4:B0:00/36Chitose,00:50:C2:B4:C0:00/36ElectroC,00:50:C2:B4:D0:00/36Troll,00:50:C2:B4:E0:00/36AixContr,00:50:C2:B4:F0:00/36SenconUK,00:50:C2:B5:00:00/36Selco,00:50:C2:B5:10:00/36Aeroflex,00:50:C2:B5:20:00/36SMHTechn,00:50:C2:B5:30:00/36Prodco,00:50:C2:B5:40:00/36APGCashD,00:50:C2:B5:50:00/36SanyoEle,00:50:C2:B5:60:00/36Sinovia,00:50:C2:B5:70:00/36PhytecMe,00:50:C2:B5:80:00/36RealD,00:50:C2:B5:90:00/36SLICANsp,00:50:C2:B5:A0:00/36GREENCen,00:50:C2:B5:B0:00/36Timberli,00:50:C2:B5:C0:00/36ADIVideo,00:50:C2:B5:D0:00/36PlitronM,00:50:C2:B5:E0:00/36Palgiken,00:50:C2:B5:F0:00/36NorthBri,00:50:C2:B6:00:00/36NpfAtis,00:50:C2:B6:10:00/36Nayos,00:50:C2:B6:20:00/36Measurem,00:50:C2:B6:30:00/36ROVERLab,00:50:C2:B6:40:00/36FEWBauer,00:50:C2:B6:50:00/36PeekTraf,00:50:C2:B6:60:00/368185,00:50:C2:B6:70:00/36RC,00:50:C2:B6:80:00/36Electron,00:50:C2:B6:90:00/36Thetis,00:50:C2:B6:A0:00/36PhytecMe,00:50:C2:B6:B0:00/36PhytecMe,00:50:C2:B6:C0:00/36Drinelec,00:50:C2:B6:D0:00/36SoundMet,00:50:C2:B6:E0:00/36Private,00:50:C2:B6:F0:00/36CT,00:50:C2:B7:00:00/36NisshinE,00:50:C2:B7:10:00/36Digitale,00:50:C2:B7:20:00/36Advanced,00:50:C2:B7:30:00/36ARKRAYKy,00:50:C2:B7:40:00/36AXEDJaku,00:50:C2:B7:50:00/36Blankom,00:50:C2:B7:60:00/36ITFFrösc,00:50:C2:B7:70:00/36Kristech,00:50:C2:B7:80:00/36Folink,00:50:C2:B7:90:00/36MitsuyaL,00:50:C2:B7:A0:00/36SchnoorI,00:50:C2:B7:B0:00/36Quartech,00:50:C2:B7:C0:00/36Bettini,00:50:C2:B7:D0:00/36ELETECH,00:50:C2:B7:E0:00/36Naretren,00:50:C2:B7:F0:00/36Enatel,00:50:C2:B8:00:00/36iScreen,00:50:C2:B8:10:00/36GHLAdvan,00:50:C2:B8:20:00/36TANABIKI,00:50:C2:B8:30:00/36Advanced,00:50:C2:B8:40:00/36Innovate,00:50:C2:B8:50:00/36SilverNe,00:50:C2:B8:60:00/36Asto,00:50:C2:B8:70:00/36EMAC,00:50:C2:B8:80:00/36Gigatron,00:50:C2:B8:90:00/36ENTECEle,00:50:C2:B8:A0:00/36MicroPoi,00:50:C2:B8:B0:00/36Fbb,00:50:C2:B8:C0:00/36KeithKoe,00:50:C2:B8:D0:00/36Cemsi,00:50:C2:B8:E0:00/36WACIsrae,00:50:C2:B8:F0:00/36Gentec,00:50:C2:B9:00:00/36NAONWORK,00:50:C2:B9:10:00/36Finnet-S,00:50:C2:B9:20:00/36ABBTrans,00:50:C2:B9:30:00/36EmcPartn,00:50:C2:B9:40:00/36TritechI,00:50:C2:B9:50:00/36RxMonito,00:50:C2:B9:60:00/36OnixElec,00:50:C2:B9:70:00/36ikerlan,00:50:C2:B9:80:00/36Southwes,00:50:C2:B9:90:00/36Greenlig,00:50:C2:B9:A0:00/36Talo,00:50:C2:B9:B0:00/36Telventy,00:50:C2:B9:C0:00/36Dave,00:50:C2:B9:D0:00/36WVershov,00:50:C2:B9:E0:00/36Saia-Bur,00:50:C2:B9:F0:00/36Audiosco,00:50:C2:BA:00:00/36txtr,00:50:C2:BA:10:00/36Transtec,00:50:C2:BA:20:00/36LogicalT,00:50:C2:BA:30:00/36DspDesig,00:50:C2:BA:40:00/36CustosMo,00:50:C2:BA:50:00/36InterCel,00:50:C2:BA:60:00/36Jomitek,00:50:C2:BA:70:00/36RaumComp,00:50:C2:BA:80:00/36PeekTraf,00:50:C2:BA:90:00/36SISSTech,00:50:C2:BA:A0:00/36NetworkF,00:50:C2:BA:B0:00/36iDealTek,00:50:C2:BA:C0:00/36VitecoVn,00:50:C2:BA:D0:00/36Predikto,00:50:C2:BA:E0:00/36FiberCon,00:50:C2:BA:F0:00/36MangoDSP,00:50:C2:BB:00:00/36Gainbrai,00:50:C2:BB:10:00/36Pro4tech,00:50:C2:BB:20:00/36StMichae,00:50:C2:BB:30:00/36ClimateW,00:50:C2:BB:40:00/36Electric,00:50:C2:BB:50:00/36MroadInf,00:50:C2:BB:60:00/36QuarchTe,00:50:C2:BB:70:00/36GDMissio,00:50:C2:BB:80:00/36MoeTroni,00:50:C2:BB:90:00/36Toptech,00:50:C2:BB:A0:00/36Systemte,00:50:C2:BB:B0:00/36GHLBerha,00:50:C2:BB:C0:00/36ImpactSy,00:50:C2:BB:D0:00/36ITSTelec,00:50:C2:BB:E0:00/36Onlinepi,00:50:C2:BB:F0:00/36PhytecMe,00:50:C2:BC:00:00/36GalaxiaE,00:50:C2:BC:10:00/36Sentec,00:50:C2:BC:20:00/36XslentEn,00:50:C2:BC:30:00/36PhytecMe,00:50:C2:BC:40:00/36Wheatsto,00:50:C2:BC:50:00/36Toptechn,00:50:C2:BC:60:00/36MireroTa,00:50:C2:BC:70:00/36PTS,00:50:C2:BC:80:00/36AGWTech,00:50:C2:BC:90:00/36Nextmove,00:50:C2:BC:A0:00/36Aitecsys,00:50:C2:BC:B0:00/36ArteixoT,00:50:C2:BC:C0:00/36VvdnTech,00:50:C2:BC:D0:00/36Highligh,00:50:C2:BC:E0:00/36TVPortal,00:50:C2:BC:F0:00/36Epikoele,00:50:C2:BD:00:00/36EDCwifi,00:50:C2:BD:10:00/36AriemTec,00:50:C2:BD:20:00/36Percello,00:50:C2:BD:30:00/36Postjet,00:50:C2:BD:40:00/36GlobalSe,00:50:C2:BD:50:00/36RF-Embed,00:50:C2:BD:60:00/36BG,00:50:C2:BD:70:00/36Slat,00:50:C2:BD:80:00/36bab-tech,00:50:C2:BD:90:00/36AMSContr,00:50:C2:BD:A0:00/36DigitalL,00:50:C2:BD:B0:00/36GasTOPS,00:50:C2:BD:C0:00/36SS,00:50:C2:BD:E0:00/36Evo-Tehd,00:50:C2:BD:F0:00/36Euro-Kon,00:50:C2:BE:00:00/36Phaedrus,00:50:C2:BE:10:00/36Tattile,00:50:C2:BE:20:00/36Converge,00:50:C2:BE:30:00/36Jiskoot,00:50:C2:BE:40:00/36GrupoEpe,00:50:C2:BE:50:00/36RFCode,00:50:C2:BE:60:00/36Docobo,00:50:C2:BE:70:00/36Genetec,00:50:C2:BE:80:00/36VehicleT,00:50:C2:BE:90:00/36Zucchett,00:50:C2:BE:A0:00/36Daeyoung,00:50:C2:BE:B0:00/36PeekTraf,00:50:C2:BE:C0:00/36DRSLarue,00:50:C2:BE:D0:00/36TouchRev,00:50:C2:BE:E0:00/36Private,00:50:C2:BE:F0:00/36Sociedad,00:50:C2:BF:00:00/36Aim,00:50:C2:BF:10:00/36AmaticIn,00:50:C2:BF:20:00/36Saia-Bur,00:50:C2:BF:30:00/36Wanco,00:50:C2:BF:40:00/36MonarchI,00:50:C2:BF:50:00/36AilesEle,00:50:C2:BF:60:00/36NolamEmb,00:50:C2:BF:70:00/36PhytecMe,00:50:C2:BF:80:00/36Crtiical,00:50:C2:BF:90:00/36VitelNet,00:50:C2:BF:A0:00/36Rohde&Sc,00:50:C2:BF:B0:00/36ECS,00:50:C2:BF:C0:00/36Altronix,00:50:C2:BF:D0:00/36Ernemann,00:50:C2:BF:E0:00/36Ingeteam,00:50:C2:BF:F0:00/36I,00:50:C2:C0:00:00/36ACDElekt,00:50:C2:C0:10:00/36QuercusT,00:50:C2:C0:20:00/36HanningE,00:50:C2:C0:30:00/36Volumati,00:50:C2:C0:40:00/36SoGEME,00:50:C2:C0:50:00/36Doppler,00:50:C2:C0:60:00/36AnalogWa,00:50:C2:C0:70:00/36CIOInfor,00:50:C2:C0:80:00/36juicebos,00:50:C2:C0:90:00/36GlobeWir,00:50:C2:C0:A0:00/36ABBTrans,00:50:C2:C0:B0:00/36ProSourc,00:50:C2:C0:C0:00/36Altierre,00:50:C2:C0:D0:00/36FrSauter,00:50:C2:C0:E0:00/36AVItroni,00:50:C2:C0:F0:00/36Dycec,00:50:C2:C1:00:00/36KeithKoe,00:50:C2:C1:10:00/36ARTAntri,00:50:C2:C1:20:00/36OkiDenki,00:50:C2:C1:30:00/36DantecDy,00:50:C2:C1:40:00/36Spectron,00:50:C2:C1:50:00/36INO-Inst,00:50:C2:C1:60:00/36OMICRONe,00:50:C2:C1:70:00/36Axis-Shi,00:50:C2:C1:80:00/36Linuxsta,00:50:C2:C1:90:00/36Ibercomp,00:50:C2:C1:A0:00/36SAM,00:50:C2:C1:B0:00/36Graessli,00:50:C2:C1:C0:00/36BectonDi,00:50:C2:C1:D0:00/36Powerbas,00:50:C2:C1:E0:00/36Peperoni,00:50:C2:C1:F0:00/36Speciali,00:50:C2:C2:00:00/36SRCCompu,00:50:C2:C2:10:00/36Private,00:50:C2:C2:20:00/36Audient,00:50:C2:C2:30:00/36Vidicon,00:50:C2:C2:40:00/36Qualneti,00:50:C2:C2:50:00/36Private,00:50:C2:C2:60:00/36AustcoMa,00:50:C2:C2:70:00/36Qtechnol,00:50:C2:C2:80:00/36ELREHA,00:50:C2:C2:90:00/36NewtelEn,00:50:C2:C2:A0:00/36RealTime,00:50:C2:C2:B0:00/36Z-App,00:50:C2:C2:C0:00/36bach-mes,00:50:C2:C2:D0:00/36Digitale,00:50:C2:C2:E0:00/36Dismunte,00:50:C2:C2:F0:00/36ReflexCe,00:50:C2:C3:00:00/36WagnerGr,00:50:C2:C3:10:00/364DTechno,00:50:C2:C3:20:00/36ProconEl,00:50:C2:C3:40:00/36Kyuhen,00:50:C2:C3:50:00/36Insitu,00:50:C2:C3:60:00/36SET,00:50:C2:C3:70:00/36BEARSolu,00:50:C2:C3:80:00/36Computer,00:50:C2:C3:90:00/36Secad,00:50:C2:C3:A0:00/36Sicon,00:50:C2:C3:B0:00/36ELEKTRO-,00:50:C2:C3:C0:00/36ELSIST,00:50:C2:C3:D0:00/36PlaElect,00:50:C2:C3:E0:00/36Sysacom,00:50:C2:C3:F0:00/36ANXeBusi,00:50:C2:C4:00:00/36BAEBofor,00:50:C2:C4:10:00/36COMPRION,00:50:C2:C4:20:00/36Saia-Bur,00:50:C2:C4:30:00/36Cammegh,00:50:C2:C4:40:00/36BeijingZ,00:50:C2:C4:50:00/36Galvamat,00:50:C2:C4:60:00/36QNE,00:50:C2:C4:70:00/36WeltekTe,00:50:C2:C4:80:00/36CytekMed,00:50:C2:C4:90:00/36Elektron,00:50:C2:C4:A0:00/36HerrickT,00:50:C2:C4:B0:00/36RVRelett,00:50:C2:C4:C0:00/36LancierM,00:50:C2:C4:D0:00/36Industri,00:50:C2:C4:E0:00/36Elaso,00:50:C2:C4:F0:00/36Powersen,00:50:C2:C5:00:00/36BeceemCo,00:50:C2:C5:10:00/36InForceC,00:50:C2:C5:20:00/36Smartfie,00:50:C2:C5:30:00/36Eilersen,00:50:C2:C5:40:00/36HPCPlatf,00:50:C2:C5:50:00/36Watterot,00:50:C2:C5:60:00/36SpirentC,00:50:C2:C5:70:00/36HighSpee,00:50:C2:C5:80:00/36Foerster,00:50:C2:C5:90:00/36SKDSyste,00:50:C2:C5:A0:00/36Commotiv,00:50:C2:C5:B0:00/36MicroTec,00:50:C2:C5:C0:00/36WavecomE,00:50:C2:C5:D0:00/36SweMet,00:50:C2:C5:E0:00/36CellPlus,00:50:C2:C5:F0:00/36IconTime,00:50:C2:C6:00:00/36Integrat,00:50:C2:C6:10:00/36HaiVisio,00:50:C2:C6:20:00/36ZeusPriv,00:50:C2:C6:30:00/36PotterEl,00:50:C2:C6:40:00/36PalSoftw,00:50:C2:C6:50:00/36MicroIOS,00:50:C2:C6:60:00/36KSBescha,00:50:C2:C6:70:00/36Practica,00:50:C2:C6:80:00/36Broadsof,00:50:C2:C6:90:00/36Rebo,00:50:C2:C6:A0:00/36Electron,00:50:C2:C6:B0:00/36SiGarden,00:50:C2:C6:C0:00/36DorletSa,00:50:C2:C6:D0:00/36Deansoft,00:50:C2:C6:E0:00/36TBS,00:50:C2:C6:F0:00/36MSBElekt,00:50:C2:C7:00:00/36WilkeTec,00:50:C2:C7:10:00/36SequoiaT,00:50:C2:C7:20:00/36Quail,00:50:C2:C7:30:00/36Industry,00:50:C2:C7:40:00/36Wapice,00:50:C2:C7:50:00/36Rovsing,00:50:C2:C7:60:00/36GridMana,00:50:C2:C7:70:00/36AIM,00:50:C2:C7:80:00/369Solutio,00:50:C2:C7:90:00/36CODESYST,00:50:C2:C7:A0:00/36Protonic,00:50:C2:C7:B0:00/36Honeywel,00:50:C2:C7:C0:00/36Keysight,00:50:C2:C7:D0:00/36TAEAntri,00:50:C2:C7:E0:00/36Buerkert,00:50:C2:C7:F0:00/36KinectsS,00:50:C2:C8:00:00/36Reko-vek,00:50:C2:C8:10:00/36OdysseeS,00:50:C2:C8:20:00/36KyoshaIn,00:50:C2:C8:30:00/36Gronic,00:50:C2:C8:40:00/36Domis,00:50:C2:C8:50:00/36PeekTraf,00:50:C2:C8:60:00/36Bruckner,00:50:C2:C8:70:00/36LECO,00:50:C2:C8:80:00/36CSIContr,00:50:C2:C8:90:00/36Creative,00:50:C2:C8:A0:00/36Automate,00:50:C2:C8:B0:00/36OcasAs,00:50:C2:C8:C0:00/36LanmarkC,00:50:C2:C8:D0:00/36Emergenc,00:50:C2:C8:E0:00/36SddItg,00:50:C2:C8:F0:00/36KeithKoe,00:50:C2:C9:00:00/36RealD,00:50:C2:C9:10:00/36MediaTec,00:50:C2:C9:20:00/36EMAC,00:50:C2:C9:30:00/36SENSAIR,00:50:C2:C9:40:00/36IsisEngi,00:50:C2:C9:50:00/36IPSES,00:50:C2:C9:60:00/36CyberCra,00:50:C2:C9:70:00/36Mstronic,00:50:C2:C9:80:00/36Criticar,00:50:C2:C9:90:00/36HJPCdbaP,00:50:C2:C9:A0:00/36PACOMPSp,00:50:C2:C9:B0:00/36Smelectr,00:50:C2:C9:C0:00/36Saia-Bur,00:50:C2:C9:D0:00/36RadiusSw,00:50:C2:C9:E0:00/36Rohde&Sc,00:50:C2:C9:F0:00/36xxter,00:50:C2:CA:00:00/36Kieferte,00:50:C2:CA:10:00/36WayneKer,00:50:C2:CA:20:00/36Logical,00:50:C2:CA:30:00/36CT,00:50:C2:CA:40:00/36VoxTechn,00:50:C2:CA:50:00/36Yokowo,00:50:C2:CA:60:00/36Vidisys,00:50:C2:CA:70:00/36ThermoFi,00:50:C2:CA:80:00/36WithInte,00:50:C2:CA:90:00/36Intellig,00:50:C2:CA:A0:00/36DspDesig,00:50:C2:CA:B0:00/36SAEIT,00:50:C2:CA:C0:00/36PURVIS,00:50:C2:CA:D0:00/36PacificC,00:50:C2:CA:E0:00/36Campbell,00:50:C2:CA:F0:00/36FrSauter,00:50:C2:CB:00:00/36Konsmeta,00:50:C2:CB:10:00/36ZKCellte,00:50:C2:CB:20:00/36Moravian,00:50:C2:CB:30:00/36DEUTA-WE,00:50:C2:CB:40:00/36GEAFarmT,00:50:C2:CB:50:00/36Private,00:50:C2:CB:60:00/36Krontek,00:50:C2:CB:70:00/36inotech,00:50:C2:CB:80:00/36Raith,00:50:C2:CB:90:00/36MicroTec,00:50:C2:CB:A0:00/36DeltaTau,00:50:C2:CB:B0:00/36Coptonix,00:50:C2:CB:C0:00/36CpEletro,00:50:C2:CB:D0:00/36HiTechEl,00:50:C2:CB:E0:00/36CodeBlue,00:50:C2:CB:F0:00/36Megacon,00:50:C2:CC:00:00/36WorldTim,00:50:C2:CC:10:00/36Level3Co,00:50:C2:CC:20:00/36ConectaI,00:50:C2:CC:30:00/36viscount,00:50:C2:CC:40:00/36GDMissio,00:50:C2:CC:50:00/36Tecnovum,00:50:C2:CC:60:00/36Kdt,00:50:C2:CC:70:00/36TOPROOTT,00:50:C2:CC:80:00/36Private,00:50:C2:CC:90:00/36Promess,00:50:C2:CC:A0:00/36SanminaS,00:50:C2:CC:B0:00/36CaptiveA,00:50:C2:CC:C0:00/36Smartech,00:50:C2:CC:D0:00/36FUJIDATA,00:50:C2:CC:E0:00/36Mac-Gray,00:50:C2:CC:F0:00/36TaskSist,00:50:C2:CD:00:00/36MMEMuell,00:50:C2:CD:10:00/36ACDElekt,00:50:C2:CD:20:00/36SIM2Mult,00:50:C2:CD:30:00/36Covidenc,00:50:C2:CD:40:00/36SCHRAML,00:50:C2:CD:50:00/36ArcosTec,00:50:C2:CD:60:00/36Arktan,00:50:C2:CD:70:00/36Saia-Bur,00:50:C2:CD:80:00/36IT-ISInt,00:50:C2:CD:90:00/36NDCInfra,00:50:C2:CD:A0:00/36taskit,00:50:C2:CD:B0:00/36Rutter,00:50:C2:CD:C0:00/36ABBTrans,00:50:C2:CD:D0:00/36KCCShoka,00:50:C2:CD:E0:00/36AxotecTe,00:50:C2:CD:F0:00/36CoreELTE,00:50:C2:CE:00:00/36Industri,00:50:C2:CE:10:00/36Satellin,00:50:C2:CE:20:00/36Sicon,00:50:C2:CE:30:00/36Industri,00:50:C2:CE:40:00/36Tektroni,00:50:C2:CE:50:00/36Maretron,00:50:C2:CE:60:00/36AplicaTe,00:50:C2:CE:70:00/36Echola,00:50:C2:CE:80:00/36ThomasBe,00:50:C2:CE:90:00/36Private,00:50:C2:CE:A0:00/36KeithKoe,00:50:C2:CE:B0:00/36ToyonRes,00:50:C2:CE:C0:00/36Erhardt+,00:50:C2:CE:D0:00/36AeroMech,00:50:C2:CE:E0:00/36Embed-It,00:50:C2:CE:F0:00/36Lupatecn,00:50:C2:CF:00:00/36Inviso,00:50:C2:CF:10:00/36TelGaAs,00:50:C2:CF:20:00/36WeissRob,00:50:C2:CF:30:00/36DaikenAu,00:50:C2:CF:40:00/36Baudisch,00:50:C2:CF:50:00/36GogoBA,00:50:C2:CF:60:00/36Epec,00:50:C2:CF:70:00/36ArmourHo,00:50:C2:CF:80:00/36beksKomm,00:50:C2:CF:90:00/36ElbitAme,00:50:C2:CF:A0:00/36GrupoEpe,00:50:C2:CF:B0:00/36NewEmbed,00:50:C2:CF:C0:00/36Tritium,00:50:C2:CF:D0:00/36Airfolc,00:50:C2:CF:E0:00/36Techlead,00:50:C2:CF:F0:00/36Infrasaf,00:50:C2:D0:00:00/36Bodensee,00:50:C2:D0:10:00/36Aanderaa,00:50:C2:D0:20:00/36Survalen,00:50:C2:D0:30:00/36PeekelIn,00:50:C2:D0:40:00/36TehamaWi,00:50:C2:D0:60:00/36nCkResea,00:50:C2:D0:70:00/36IAF,00:50:C2:D0:80:00/36Reimesch,00:50:C2:D0:90:00/36Guardtec,00:50:C2:D0:A0:00/36Airpoint,00:50:C2:D0:B0:00/36CODACOEL,00:50:C2:D0:C0:00/36JVLIndus,00:50:C2:D0:D0:00/36DECACard,00:50:C2:D0:E0:00/36WeinertE,00:50:C2:D0:F0:00/36Saia-Bur,00:50:C2:D1:00:00/36Rosslare,00:50:C2:D1:10:00/36AplexTec,00:50:C2:D1:20:00/36TokyoWel,00:50:C2:D1:30:00/36GunmaEle,00:50:C2:D1:40:00/36SaetIS,00:50:C2:D1:50:00/36MSR-Offi,00:50:C2:D1:60:00/36ImricorM,00:50:C2:D1:70:00/36CUE,00:50:C2:D1:80:00/36Glyn,00:50:C2:D1:90:00/36AppliedM,00:50:C2:D1:A0:00/36Gillam-F,00:50:C2:D1:B0:00/36TECHKON,00:50:C2:D1:C0:00/36ReconDyn,00:50:C2:D1:D0:00/36MocoMedi,00:50:C2:D1:E0:00/36Tobila,00:50:C2:D1:F0:00/36OlympusN,00:50:C2:D2:00:00/367+Kft,00:50:C2:D2:10:00/36Innovati,00:50:C2:D2:20:00/36eMDeeTec,00:50:C2:D2:30:00/36Blueston,00:50:C2:D2:40:00/36ExproNor,00:50:C2:D2:50:00/36VAFInstr,00:50:C2:D2:60:00/36RchGroup,00:50:C2:D2:70:00/36FrSauter,00:50:C2:D2:80:00/36Digitale,00:50:C2:D2:90:00/36AxibleTe,00:50:C2:D2:A0:00/36Millenni,00:50:C2:D2:B0:00/36VideoTec,00:50:C2:D2:C0:00/36Schneide,00:50:C2:D2:D0:00/36CadiScie,00:50:C2:D2:E0:00/36RSGesell,00:50:C2:D2:F0:00/36Key,00:50:C2:D3:00:00/36ACTIVFin,00:50:C2:D3:10:00/36UNGAVATe,00:50:C2:D3:20:00/36RealTime,00:50:C2:D3:30:00/36Maddalen,00:50:C2:D3:40:00/36GAONTECH,00:50:C2:D3:50:00/36UG,00:50:C2:D3:60:00/36Enatel,00:50:C2:D3:70:00/36LJTAssoc,00:50:C2:D3:80:00/36KyowaEle,00:50:C2:D3:90:00/36Apex,00:50:C2:D3:A0:00/36WellSens,00:50:C2:D3:B0:00/36Gitsn,00:50:C2:D3:C0:00/36ASSYSTEM,00:50:C2:D3:D0:00/36TellabsO,00:50:C2:D3:E0:00/36SynatecE,00:50:C2:D3:F0:00/36Communic,00:50:C2:D4:00:00/36demmelpr,00:50:C2:D4:10:00/36AreaEner,00:50:C2:D4:20:00/36HagenukK,00:50:C2:D4:30:00/36DSP4YOU,00:50:C2:D4:40:00/36Saia-Bur,00:50:C2:D4:50:00/36Technago,00:50:C2:D4:60:00/36ThalesNe,00:50:C2:D4:70:00/36Rohde&Sc,00:50:C2:D4:80:00/36Watermar,00:50:C2:D4:90:00/36SmithMet,00:50:C2:D4:A0:00/36ATHsyste,00:50:C2:D4:B0:00/36IndraAus,00:50:C2:D4:C0:00/36DALOGDia,00:50:C2:D4:D0:00/36YardneyT,00:50:C2:D4:E0:00/36KeithKoe,00:50:C2:D4:F0:00/36SECOM,00:50:C2:D5:00:00/36SolbrigE,00:50:C2:D5:10:00/36Bettini,00:50:C2:D5:20:00/36F+DFeinw,00:50:C2:D5:30:00/36Telemerk,00:50:C2:D5:40:00/36ABtrack,00:50:C2:D5:50:00/36SternaSe,00:50:C2:D5:60:00/36SELEXCom,00:50:C2:D5:70:00/36Hijikata,00:50:C2:D5:80:00/36NIK-ELEK,00:50:C2:D5:90:00/36BuancoSy,00:50:C2:D5:A0:00/36Embedded,00:50:C2:D5:B0:00/36Infiniti,00:50:C2:D5:C0:00/36IbetorSL,00:50:C2:D5:D0:00/36Globalco,00:50:C2:D5:E0:00/36infinite,00:50:C2:D5:F0:00/36Embedded,00:50:C2:D6:00:00/36NihonKes,00:50:C2:D6:10:00/36system2,00:50:C2:D6:20:00/36EMAC,00:50:C2:D6:30:00/36Dataregi,00:50:C2:D6:40:00/36TV1,00:50:C2:D6:50:00/36TXTechno,00:50:C2:D6:60:00/36UvaxConc,00:50:C2:D6:70:00/36KLINGFRE,00:50:C2:D6:80:00/36HiSpeedD,00:50:C2:D6:90:00/36GHLBhd,00:50:C2:D6:A0:00/36A&TElect,00:50:C2:D6:B0:00/36NemecAut,00:50:C2:D6:C0:00/36ALPHA,00:50:C2:D6:D0:00/36Pro-Digi,00:50:C2:D6:E0:00/36BCIllumi,00:50:C2:D6:F0:00/36ImtronMe,00:50:C2:D7:00:00/36CRobHamm,00:50:C2:D7:10:00/36EMAC,00:50:C2:D7:20:00/36Scale-Tr,00:50:C2:D7:30:00/36Saia-Bur,00:50:C2:D7:40:00/36Computec,00:50:C2:D7:50:00/36Collectr,00:50:C2:D7:60:00/36Telvent,00:50:C2:D7:70:00/36FrSauter,00:50:C2:D7:80:00/36P4QElect,00:50:C2:D7:90:00/36DSIRF,00:50:C2:D7:A0:00/36Transbit,00:50:C2:D7:B0:00/36OWITA,00:50:C2:D7:C0:00/36Microcub,00:50:C2:D7:D0:00/36VoltechI,00:50:C2:D7:E0:00/36LYNXTech,00:50:C2:D7:F0:00/36HMITechn,00:50:C2:D8:00:00/36KeithKoe,00:50:C2:D8:10:00/36Tattile,00:50:C2:D8:20:00/36AudioAut,00:50:C2:D8:30:00/36Blankom,00:50:C2:D8:40:00/36ABBTrans,00:50:C2:D8:50:00/36Vitec,00:50:C2:D8:60:00/36EcommEra,00:50:C2:D8:70:00/36Electrol,00:50:C2:D8:80:00/36T+Aelekt,00:50:C2:D8:90:00/36VisualTe,00:50:C2:D8:A0:00/36OptoLink,00:50:C2:D8:B0:00/36Sicon,00:50:C2:D8:C0:00/36iRphoton,00:50:C2:D8:D0:00/36CS-Instr,00:50:C2:D8:E0:00/36LSDScien,00:50:C2:D8:F0:00/36Syes,00:50:C2:D9:00:00/36DumpsEle,00:50:C2:D9:10:00/36ChauvinA,00:50:C2:D9:20:00/36Manz,00:50:C2:D9:30:00/36Axlon,00:50:C2:D9:40:00/36Software,00:50:C2:D9:50:00/36Honeywel,00:50:C2:D9:60:00/36CONTEC,00:50:C2:D9:70:00/36ERSelect,00:50:C2:D9:80:00/36RongShun,00:50:C2:D9:90:00/36T-Indust,00:50:C2:D9:A0:00/36Saia-Bur,00:50:C2:D9:B0:00/36Intuitiv,00:50:C2:D9:C0:00/36GamberJo,00:50:C2:D9:D0:00/36MistralS,00:50:C2:D9:E0:00/36Vocality,00:50:C2:D9:F0:00/36BitWiseC,00:50:C2:DA:00:00/36Precisio,00:50:C2:DA:10:00/36MangoDSP,00:50:C2:DA:20:00/36metraTec,00:50:C2:DA:30:00/36GDMissio,00:50:C2:DA:40:00/36DEUTA-WE,00:50:C2:DA:50:00/36megatece,00:50:C2:DA:60:00/36Manitowo,00:50:C2:DA:70:00/36Capton,00:50:C2:DA:80:00/36Sine,00:50:C2:DA:90:00/36TielineR,00:50:C2:DA:A0:00/36MPaul,00:50:C2:DA:B0:00/36AplexTec,00:50:C2:DA:C0:00/36RFLElect,00:50:C2:DA:D0:00/36KeithKoe,00:50:C2:DA:E0:00/36SpangPow,00:50:C2:DA:F0:00/36eumigind,00:50:C2:DB:00:00/36IMAGOTec,00:50:C2:DB:10:00/36RFCode,00:50:C2:DB:20:00/36Software,00:50:C2:DB:30:00/36LaudaDrR,00:50:C2:DB:40:00/36NPCKompj,00:50:C2:DB:50:00/36DspDesig,00:50:C2:DB:60:00/36Prosoft,00:50:C2:DB:70:00/36SORELMik,00:50:C2:DB:80:00/36ComsatVe,00:50:C2:DB:90:00/36PeekTraf,00:50:C2:DB:A0:00/36MPElectr,00:50:C2:DB:B0:00/36Esensors,00:50:C2:DB:C0:00/36NantesPr,00:50:C2:DB:D0:00/36Margento,00:50:C2:DB:E0:00/36WITHSYST,00:50:C2:DB:F0:00/36One-Nemo,00:50:C2:DC:00:00/36Security,00:50:C2:DC:10:00/36AcruxTec,00:50:C2:DC:20:00/36TesseraT,00:50:C2:DC:30:00/36ZEDZiegl,00:50:C2:DC:40:00/36KeithKoe,00:50:C2:DC:50:00/36Saia-Bur,00:50:C2:DC:60:00/36FluidCom,00:50:C2:DC:70:00/36AGTHoldi,00:50:C2:DC:80:00/36T2M2,00:50:C2:DC:90:00/36KinotonG,00:50:C2:DC:A0:00/36TeleData,00:50:C2:DC:B0:00/36CT,00:50:C2:DC:C0:00/36Instrume,00:50:C2:DC:D0:00/36dilitron,00:50:C2:DC:E0:00/36Mecsel,00:50:C2:DC:F0:00/36MCSEngen,00:50:C2:DD:00:00/36IDCSolut,00:50:C2:DD:10:00/36Brankamp,00:50:C2:DD:20:00/36Electron,00:50:C2:DD:30:00/36Rohde&Sc,00:50:C2:DD:40:00/36Systech,00:50:C2:DD:50:00/36FriendSp,00:50:C2:DD:60:00/36Wartsila,00:50:C2:DD:70:00/36TornadoM,00:50:C2:DD:80:00/36Leonardo,00:50:C2:DD:90:00/36Metrawar,00:50:C2:DD:A0:00/36rbzrobot,00:50:C2:DD:B0:00/36Luceo,00:50:C2:DD:C0:00/36VisionCo,00:50:C2:DD:D0:00/36A&AGener,00:50:C2:DD:E0:00/36DRSImagi,00:50:C2:DD:F0:00/36Device,00:50:C2:DE:00:00/36Internet,00:50:C2:DE:10:00/36ABBTrans,00:50:C2:DE:20:00/36Sequtec,00:50:C2:DE:30:00/36Breakawa,00:50:C2:DE:40:00/36EGSTechn,00:50:C2:DE:50:00/36Neets,00:50:C2:DE:60:00/36FrSauter,00:50:C2:DE:70:00/36Elan,00:50:C2:DE:80:00/36VisualPr,00:50:C2:DE:90:00/36DacomWes,00:50:C2:DE:A0:00/36Cerner,00:50:C2:DE:B0:00/36RuwischK,00:50:C2:DE:C0:00/36VendNova,00:50:C2:DE:D0:00/36LeeLaser,00:50:C2:DE:F0:00/36Powersen,00:50:C2:DF:00:00/36KoncarEl,00:50:C2:DF:10:00/36Saia-Bur,00:50:C2:DF:20:00/36OceanSon,00:50:C2:DF:30:00/36INSEVIS,00:50:C2:DF:40:00/36PotterEl,00:50:C2:DF:50:00/36EtherLig,00:50:C2:DF:60:00/36HinoEngi,00:50:C2:DF:70:00/36Combilen,00:50:C2:DF:80:00/36Tommotek,00:50:C2:DF:90:00/36JennySci,00:50:C2:DF:A0:00/36MACValve,00:50:C2:DF:B0:00/36Bettini,00:50:C2:DF:C0:00/36I-Evo,00:50:C2:DF:D0:00/36Wotbox,00:50:C2:DF:E0:00/36XitekDes,00:50:C2:DF:F0:00/36TantalEl,00:50:C2:E0:00:00/36AplexTec,00:50:C2:E0:10:00/36TycoTraf,00:50:C2:E0:20:00/36Cleversc,00:50:C2:E0:30:00/36ICUScand,00:50:C2:E0:40:00/36SecEng,00:50:C2:E0:50:00/36Nocosium,00:50:C2:E0:60:00/36EbnerEle,00:50:C2:E0:70:00/36Protagon,00:50:C2:E0:80:00/36KSTTechn,00:50:C2:E0:90:00/36Ateme,00:50:C2:E0:A0:00/36Sicon,00:50:C2:E0:B0:00/36Seartech,00:50:C2:E0:C0:00/36YouhoEle,00:50:C2:E0:D0:00/36Unixmedi,00:50:C2:E0:E0:00/36PmacJapa,00:50:C2:E0:F0:00/36Trentino,00:50:C2:E1:00:00/36Radinetw,00:50:C2:E1:10:00/36RadioMob,00:50:C2:E1:20:00/36KagoElec,00:50:C2:E1:30:00/36Automati,00:50:C2:E1:40:00/36Calixto,00:50:C2:E1:50:00/36IHIScube,00:50:C2:E1:60:00/36Jetstrea,00:50:C2:E1:70:00/36GallTank,00:50:C2:E1:80:00/36ABBTrans,00:50:C2:E1:90:00/36ZoeMedic,00:50:C2:E1:A0:00/36Rosslare,00:50:C2:E1:B0:00/36Embedded,00:50:C2:E1:C0:00/36Saia-Bur,00:50:C2:E1:D0:00/36Holdline,00:50:C2:E1:E0:00/36Lo-Q,00:50:C2:E1:F0:00/36Elvees,00:50:C2:E2:00:00/36Divelbis,00:50:C2:E2:10:00/36NorwiaAS,00:50:C2:E2:20:00/36MichaelR,00:50:C2:E2:30:00/36Vitec,00:50:C2:E2:40:00/36DiTESTFa,00:50:C2:E2:50:00/36ACDElekt,00:50:C2:E2:60:00/36Cinetix,00:50:C2:E2:70:00/36CONTROL,00:50:C2:E2:80:00/36Teplovod,00:50:C2:E2:90:00/36FrSauter,00:50:C2:E2:A0:00/36Rohde&Sc,00:50:C2:E2:B0:00/36PlantInt,00:50:C2:E2:C0:00/36ENElectr,00:50:C2:E2:D0:00/36Funkwerk,00:50:C2:E2:E0:00/36DS!Ingen,00:50:C2:E2:F0:00/36Beam,00:50:C2:E3:00:00/36Goennhei,00:50:C2:E3:10:00/36ENSIS,00:50:C2:E3:20:00/36Oshoksh,00:50:C2:E3:30:00/36MoritaTe,00:50:C2:E3:40:00/36HGLDynam,00:50:C2:E3:50:00/36Omnica,00:50:C2:E3:60:00/36Saia-Bur,00:50:C2:E3:70:00/36FUJIDATA,00:50:C2:E3:80:00/36AesirCop,00:50:C2:E3:90:00/36Telemetr,00:50:C2:E3:A0:00/36Vocality,00:50:C2:E3:B0:00/36Nanosolu,00:50:C2:E3:C0:00/36GrupoEpe,00:50:C2:E3:D0:00/36Baudisch,00:50:C2:E3:E0:00/36Monnit,00:50:C2:E3:F0:00/36Visito,00:50:C2:E4:00:00/36Ecrin,00:50:C2:E4:10:00/36Higeco,00:50:C2:E4:20:00/36Wingsfor,00:50:C2:E4:30:00/36Technica,00:50:C2:E4:40:00/36DEUTA-WE,00:50:C2:E4:50:00/36Stichtin,00:50:C2:E4:60:00/36Industre,00:50:C2:E4:70:00/36EnikaCz,00:50:C2:E4:80:00/36ITWReyfl,00:50:C2:E4:90:00/36CtfTechn,00:50:C2:E4:A0:00/36GHLBhd,00:50:C2:E4:B0:00/36Rohde&Sc,00:50:C2:E4:C0:00/36AppliedM,00:50:C2:E4:D0:00/36Pcsc,00:50:C2:E4:E0:00/36Institut,00:50:C2:E4:F0:00/36WineTech,00:50:C2:E5:00:00/36Tattile,00:50:C2:E5:10:00/36Motec,00:50:C2:E5:20:00/36FamasSys,00:50:C2:E5:30:00/36NEXTvide,00:50:C2:E5:40:00/36ArcosTec,00:50:C2:E5:50:00/36TTi,00:50:C2:E5:60:00/36Rfengine,00:50:C2:E5:70:00/36EolaneMo,00:50:C2:E5:80:00/36Agri-hit,00:50:C2:E5:90:00/36Saia-Bur,00:50:C2:E5:A0:00/36Futec,00:50:C2:E5:B0:00/36CAIPOAut,00:50:C2:E5:C0:00/36MCOPIA,00:50:C2:E5:D0:00/36T8,00:50:C2:E5:E0:00/36Orep,00:50:C2:E5:F0:00/36PantecEn,00:50:C2:E6:00:00/36Horizon,00:50:C2:E6:10:00/36DetechEl,00:50:C2:E6:20:00/36SAEIT,00:50:C2:E6:30:00/36Primasis,00:50:C2:E6:40:00/36Edgeware,00:50:C2:E6:50:00/36IBElektr,00:50:C2:E6:60:00/36EMAC,00:50:C2:E6:70:00/36Critical,00:50:C2:E6:80:00/36Kyoritsu,00:50:C2:E6:90:00/36Netmaker,00:50:C2:E6:A0:00/36AplexTec,00:50:C2:E6:B0:00/36SikaTech,00:50:C2:E6:C0:00/36SAMSUNGE,00:50:C2:E6:D0:00/36Allerta,00:50:C2:E6:E0:00/36Power-On,00:50:C2:E6:F0:00/36LeydenEn,00:50:C2:E7:00:00/36DorletSa,00:50:C2:E7:10:00/36trafficn,00:50:C2:E7:30:00/36ACSMotio,00:50:C2:E7:40:00/36Will,00:50:C2:E7:50:00/36Fsm,00:50:C2:E7:60:00/36Embedded,00:50:C2:E7:70:00/36FrSauter,00:50:C2:E7:80:00/36TaskSist,00:50:C2:E7:90:00/36MCSMICRO,00:50:C2:E7:A0:00/36Lightel,00:50:C2:E7:B0:00/36ATOMGIKE,00:50:C2:E7:C0:00/36spcontro,00:50:C2:E7:D0:00/36AELMicro,00:50:C2:E7:E0:00/36Swarefle,00:50:C2:E7:F0:00/36LSContro,00:50:C2:E8:00:00/36Saia-Bur,00:50:C2:E8:10:00/36Adaptive,00:50:C2:E8:20:00/36XploreTe,00:50:C2:E8:30:00/36Witree,00:50:C2:E8:40:00/36ABBTrans,00:50:C2:E8:50:00/36CosmoLif,00:50:C2:E8:60:00/36Multisun,00:50:C2:E8:70:00/36LamsonSa,00:50:C2:E8:80:00/36Pivitec,00:50:C2:E8:90:00/36Proteqse,00:50:C2:E8:A0:00/36Macronet,00:50:C2:E8:B0:00/36RPAElect,00:50:C2:E8:C0:00/36Epec,00:50:C2:E8:D0:00/36SystemAd,00:50:C2:E8:E0:00/36GDMissio,00:50:C2:E8:F0:00/36STTCondi,00:50:C2:E9:00:00/36GSElektr,00:50:C2:E9:10:00/36DspDesig,00:50:C2:E9:20:00/36CT,00:50:C2:E9:30:00/36Percepti,00:50:C2:E9:40:00/36ANA-U,00:50:C2:E9:50:00/36DliteCom,00:50:C2:E9:60:00/36Proyecso,00:50:C2:E9:70:00/36Arista,00:50:C2:E9:80:00/36i3Intern,00:50:C2:E9:90:00/36UVNetwor,00:50:C2:E9:A0:00/36Solace,00:50:C2:E9:B0:00/36Hentsche,00:50:C2:E9:C0:00/36SPARQ,00:50:C2:E9:D0:00/36nicai,00:50:C2:E9:E0:00/36American,00:50:C2:E9:F0:00/36DataSoft,00:50:C2:EA:00:00/36RobertBo,00:50:C2:EA:10:00/36TexCompu,00:50:C2:EA:20:00/36ThinkRF,00:50:C2:EA:30:00/36Subsea,00:50:C2:EA:40:00/36head,00:50:C2:EA:50:00/36Aerodata,00:50:C2:EA:60:00/36Powersen,00:50:C2:EA:70:00/36Saia-Bur,00:50:C2:EA:80:00/36MBconnec,00:50:C2:EA:90:00/36MettlerT,00:50:C2:EA:A0:00/36BAE,00:50:C2:EA:B0:00/36Warp9Tec,00:50:C2:EA:C0:00/36Aliasip,00:50:C2:EA:D0:00/36Rohde&Sc,00:50:C2:EA:E0:00/36AlyricaN,00:50:C2:EA:F0:00/36GogoBA,00:50:C2:EB:00:00/36PulseCom,00:50:C2:EB:10:00/36PduExper,00:50:C2:EB:20:00/36OtakiEle,00:50:C2:EB:30:00/36ARRFMicr,00:50:C2:EB:40:00/36WishtekT,00:50:C2:EB:50:00/36Covidenc,00:50:C2:EB:60:00/36MonsoonS,00:50:C2:EB:70:00/36Saab,00:50:C2:EB:80:00/36dspnor,00:50:C2:EB:90:00/36Alpha-Mo,00:50:C2:EB:A0:00/36West-Com,00:50:C2:EB:B0:00/36TimeTerm,00:50:C2:EB:C0:00/36DiehlAKO,00:50:C2:EB:D0:00/36DropletM,00:50:C2:EB:E0:00/36GlobalTe,00:50:C2:EB:F0:00/36Civoluti,00:50:C2:EC:00:00/36UgMOTech,00:50:C2:EC:10:00/36ANTGroup,00:50:C2:EC:20:00/36Ixonos,00:50:C2:EC:30:00/36AplexTec,00:50:C2:EC:40:00/36LogicalE,00:50:C2:EC:50:00/36RSUPPORT,00:50:C2:EC:60:00/36Infronic,00:50:C2:EC:70:00/36LiquidRo,00:50:C2:EC:80:00/36IbernexI,00:50:C2:EC:90:00/36Amsterda,00:50:C2:EC:A0:00/36BitWiseC,00:50:C2:EC:B0:00/36FAL,00:50:C2:EC:C0:00/36Saia-Bur,00:50:C2:EC:D0:00/36PeekTraf,00:50:C2:EC:E0:00/36easiiica,00:50:C2:EC:F0:00/36TaiwanHi,00:50:C2:ED:00:00/36NipponSy,00:50:C2:ED:10:00/36Arcontia,00:50:C2:ED:20:00/36Klangspe,00:50:C2:ED:30:00/36EcoMonit,00:50:C2:ED:40:00/36Tamagawa,00:50:C2:ED:50:00/36RFLElect,00:50:C2:ED:60:00/36Cat,00:50:C2:ED:70:00/36FBTElett,00:50:C2:ED:80:00/36AVocatio,00:50:C2:ED:90:00/36Plasmatr,00:50:C2:ED:A0:00/36JointSto,00:50:C2:ED:B0:00/36BelikSPR,00:50:C2:ED:C0:00/36Eyelock,00:50:C2:ED:D0:00/36Ebneuro,00:50:C2:ED:E0:00/36SmartGri,00:50:C2:ED:F0:00/36MonitorB,00:50:C2:EE:00:00/36osfHansj,00:50:C2:EE:10:00/36ProconEl,00:50:C2:EE:20:00/36SystemIn,00:50:C2:EE:30:00/36TecnintH,00:50:C2:EE:40:00/36Rohde&Sc,00:50:C2:EE:50:00/36CytecZyl,00:50:C2:EE:60:00/36BTECH,00:50:C2:EE:70:00/36syes,00:50:C2:EE:80:00/36KamachoS,00:50:C2:EE:90:00/36QUANTA,00:50:C2:EE:A0:00/36Position,00:50:C2:EE:B0:00/36fibrisTe,00:50:C2:EE:C0:00/36YUYAMAMF,00:50:C2:EE:D0:00/36FutureDe,00:50:C2:EE:E0:00/36ABBTrans,00:50:C2:EE:F0:00/36IDTRONIC,00:50:C2:EF:00:00/36Homaetri,00:50:C2:EF:10:00/36Saia-Bur,00:50:C2:EF:20:00/36Specialt,00:50:C2:EF:30:00/36SmartPow,00:50:C2:EF:40:00/36ROVERLab,00:50:C2:EF:50:00/36HumanNet,00:50:C2:EF:60:00/36NetlineC,00:50:C2:EF:70:00/36Amstella,00:50:C2:EF:80:00/36HCLTechn,00:50:C2:EF:90:00/36HoribaAb,00:50:C2:EF:A0:00/36Predicti,00:50:C2:EF:B0:00/36NorbitOD,00:50:C2:EF:C0:00/36Private,00:50:C2:EF:D0:00/36Sanmina,00:50:C2:EF:E0:00/36PLRInfor,00:50:C2:EF:F0:00/36Zephyrus,00:50:C2:F0:00:00/36SyscomIn,00:50:C2:F0:10:00/36MangoDSP,00:50:C2:F0:20:00/36Bmr,00:50:C2:F0:30:00/36WrenSoun,00:50:C2:F0:40:00/36KinkiRoe,00:50:C2:F0:50:00/36ESIVentu,00:50:C2:F0:60:00/36Micro-Ke,00:50:C2:F0:70:00/36IconRese,00:50:C2:F0:80:00/36AplexTec,00:50:C2:F0:90:00/36Wheatsto,00:50:C2:F0:A0:00/36HASCOMIn,00:50:C2:F0:B0:00/36Treehave,00:50:C2:F0:C0:00/36Skychann,00:50:C2:F0:D0:00/36Bluetest,00:50:C2:F0:E0:00/36MicroTec,00:50:C2:F0:F0:00/36AeroVisi,00:50:C2:F1:00:00/36WincorNi,00:50:C2:F1:10:00/36Organis,00:50:C2:F1:20:00/36GeneralI,00:50:C2:F1:30:00/36PacketPl,00:50:C2:F1:40:00/36VisionAe,00:50:C2:F1:50:00/36SascalDi,00:50:C2:F1:60:00/36PeterHub,00:50:C2:F1:70:00/36ABBTrans,00:50:C2:F1:80:00/36VitecMul,00:50:C2:F1:90:00/36NetlinkB,00:50:C2:F1:A0:00/36AquaMana,00:50:C2:F1:B0:00/36Saia-Bur,00:50:C2:F1:C0:00/36GDMissio,00:50:C2:F1:D0:00/36Grossenb,00:50:C2:F1:E0:00/36DellOrto,00:50:C2:F1:F0:00/36Verified,00:50:C2:F2:00:00/36UnforsIn,00:50:C2:F2:10:00/36SEITEC,00:50:C2:F2:20:00/36HarlandS,00:50:C2:F2:30:00/36Electro-,00:50:C2:F2:40:00/36CT,00:50:C2:F2:50:00/36SamwayEl,00:50:C2:F2:60:00/36WaveIP,00:50:C2:F2:70:00/36Elan,00:50:C2:F2:80:00/36VertexAn,00:50:C2:F2:90:00/36Radyne,00:50:C2:F2:A0:00/36ACDElekt,00:50:C2:F2:B0:00/36BioGuard,00:50:C2:F2:C0:00/36Terratel,00:50:C2:F2:D0:00/36RobertBo,00:50:C2:F2:E0:00/36H&LInstr,00:50:C2:F2:F0:00/36ArcosTec,00:50:C2:F3:00:00/36Miris,00:50:C2:F3:10:00/36RuetzTec,00:50:C2:F3:20:00/36Net4Thin,00:50:C2:F3:30:00/36AppliedM,00:50:C2:F3:40:00/36SequipS+,00:50:C2:F3:50:00/36GrupoEpe,00:50:C2:F3:60:00/36Visitech,00:50:C2:F3:70:00/36Rosslare,00:50:C2:F3:80:00/36AeroCont,00:50:C2:F3:90:00/36InForceC,00:50:C2:F3:A0:00/36Saia-Bur,00:50:C2:F3:B0:00/36TAMSfirm,00:50:C2:F3:C0:00/36VemcoSpz,00:50:C2:F3:D0:00/36Projects,00:50:C2:F3:E0:00/36Vtron,00:50:C2:F3:F0:00/36DENSEICO,00:50:C2:F4:00:00/36iBWorld,00:50:C2:F4:10:00/36FairyDev,00:50:C2:F4:20:00/36DSPCon,00:50:C2:F4:30:00/36SpecialE,00:50:C2:F4:40:00/36Steinbic,00:50:C2:F4:50:00/36HUSTYMSt,00:50:C2:F4:60:00/36ReasonTe,00:50:C2:F4:70:00/36cadac,00:50:C2:F4:80:00/36PhoenixA,00:50:C2:F4:90:00/36GreenIns,00:50:C2:F4:A0:00/36Z-App,00:50:C2:F4:B0:00/36Supranet,00:50:C2:F4:C0:00/36Enistic,00:50:C2:F4:D0:00/36KnowhowI,00:50:C2:F4:E0:00/36Heinzing,00:50:C2:F4:F0:00/36BAPPreci,00:50:C2:F5:00:00/36Moritex,00:50:C2:F5:10:00/36NDCInfra,00:50:C2:F5:20:00/36Rohde&Sc,00:50:C2:F5:30:00/36BaycomOp,00:50:C2:F5:40:00/36HellaGut,00:50:C2:F5:50:00/36Honeywel,00:50:C2:F5:60:00/36MonsoonS,00:50:C2:F5:70:00/36ReachTec,00:50:C2:F5:80:00/36IEEERegi,00:50:C2:F5:90:00/36G3Techno,00:50:C2:F5:A0:00/36Sentry36,00:50:C2:F5:B0:00/36Saia-Bur,00:50:C2:F5:C0:00/36DspDesig,00:50:C2:F5:D0:00/36SmartbTe,00:50:C2:F5:E0:00/36Y-camSol,00:50:C2:F5:F0:00/36BoryeuTe,00:50:C2:F6:00:00/36Deckma,00:50:C2:F6:10:00/36BrauchEl,00:50:C2:F6:20:00/36EMAC,00:50:C2:F6:30:00/36Triax,00:50:C2:F6:40:00/36ChrissoT,00:50:C2:F6:50:00/36Telebyte,00:50:C2:F6:60:00/36Gwt,00:50:C2:F6:70:00/36Celestia,00:50:C2:F6:80:00/36Newtec,00:50:C2:F6:90:00/36SafePlac,00:50:C2:F6:A0:00/36OFIdba2D,00:50:C2:F6:B0:00/36AlgodueE,00:50:C2:F6:C0:00/36ProDesig,00:50:C2:F6:D0:00/36ProDesig,00:50:C2:F6:E0:00/36SmithMet,00:50:C2:F6:F0:00/36AplexTec,00:50:C2:F7:00:00/36NoraltaT,00:50:C2:F7:10:00/36RFCode,00:50:C2:F7:20:00/36MaxDeTec,00:50:C2:F7:30:00/36Deltacas,00:50:C2:F7:40:00/36ThorTech,00:50:C2:F7:50:00/36PumpWell,00:50:C2:F7:60:00/36RongJieF,00:50:C2:F7:70:00/36SYSTEMTE,00:50:C2:F7:80:00/36GetsMSS,00:50:C2:F7:90:00/36Tattile,00:50:C2:F7:A0:00/36C3,00:50:C2:F7:B0:00/36MCMElect,00:50:C2:F7:C0:00/36Atonomet,00:50:C2:F7:D0:00/36D-HikeEl,00:50:C2:F7:E0:00/36TruTeqWi,00:50:C2:F7:F0:00/36DynamicD,00:50:C2:F8:00:00/36SYSTECel,00:50:C2:F8:10:00/36Plda,00:50:C2:F8:20:00/36SincairI,00:50:C2:F8:30:00/36GSPSprac,00:50:C2:F8:40:00/36DynonIns,00:50:C2:F8:50:00/36Enetics,00:50:C2:F8:60:00/36AudioPow,00:50:C2:F8:70:00/36Vaisala,00:50:C2:F8:80:00/36RTCManuf,00:50:C2:F8:90:00/36CSAEngin,00:50:C2:F8:A0:00/36EMAC,00:50:C2:F8:B0:00/36comletVe,00:50:C2:F8:C0:00/36UBSTechn,00:50:C2:F8:D0:00/36Guangdon,00:50:C2:F8:E0:00/36Gpo,00:50:C2:F8:F0:00/36Computer,00:50:C2:F9:00:00/36SecureTe,00:50:C2:F9:10:00/36RE2,00:50:C2:F9:20:00/36CONETSol,00:50:C2:F9:30:00/36Baudisch,00:50:C2:F9:40:00/36DigitalB,00:50:C2:F9:50:00/36TTiThurl,00:50:C2:F9:60:00/36JLCooper,00:50:C2:F9:70:00/36Sicon,00:50:C2:F9:80:00/36Infotech,00:50:C2:F9:90:00/36DrNeuman,00:50:C2:F9:A0:00/36Telvent,00:50:C2:F9:B0:00/36NewellTe,00:50:C2:F9:C0:00/36R&DKomet,00:50:C2:F9:D0:00/36KalugaTe,00:50:C2:F9:E0:00/36Matsusad,00:50:C2:F9:F0:00/36NanjingS,00:50:C2:FA:00:00/36AmplusCo,00:50:C2:FA:10:00/36N-Handsu,00:50:C2:FA:20:00/36Power-On,00:50:C2:FA:30:00/36Xemex,00:50:C2:FA:50:00/36Intuitiv,00:50:C2:FA:60:00/36Hilkomdi,00:50:C2:FA:70:00/36Exelis,00:50:C2:FA:80:00/36YashSiQu,00:50:C2:FA:90:00/36HijetPri,00:50:C2:FA:A0:00/36Yjsystem,00:50:C2:FA:B0:00/36AplexTec,00:50:C2:FA:C0:00/36AdetelGr,00:50:C2:FA:D0:00/36Finishin,00:50:C2:FA:E0:00/36ATIAutom,00:50:C2:FA:F0:00/36Vremya-C,00:50:C2:FB:00:00/36Tateishi,00:50:C2:FB:10:00/36Matelex,00:50:C2:FB:20:00/36PrestonI,00:50:C2:FB:30:00/36CT,00:50:C2:FB:40:00/36MC-monit,00:50:C2:FB:50:00/36Assembly,00:50:C2:FB:60:00/36Argus-Sp,00:50:C2:FB:70:00/36PounceCo,00:50:C2:FB:80:00/36Techno,00:50:C2:FB:90:00/36CoralTel,00:50:C2:FB:A0:00/36ElbitAme,00:50:C2:FB:B0:00/36ACIDA,00:50:C2:FB:C0:00/36LeroySom,00:50:C2:FB:D0:00/36EATONFHF,00:50:C2:FB:E0:00/36senTecEl,00:50:C2:FB:F0:00/36Mylogic,00:50:C2:FC:00:00/36Rohde&Sc,00:50:C2:FC:10:00/36Motec,00:50:C2:FC:20:00/36Elta,00:50:C2:FC:30:00/36HSDCSp,00:50:C2:FC:40:00/36Kyowaden,00:50:C2:FC:50:00/36SakuraSe,00:50:C2:FC:60:00/36Critical,00:50:C2:FC:70:00/36SERCOMRe,00:50:C2:FC:80:00/36FarSouth,00:50:C2:FC:90:00/36MehtaTec,00:50:C2:FC:A0:00/36Telemisi,00:50:C2:FC:B0:00/36Propagat,00:50:C2:FC:C0:00/36Soudroni,00:50:C2:FC:D0:00/36Jinyoung,00:50:C2:FC:E0:00/36KoyoElec,00:50:C2:FC:F0:00/36DINTEKSh,00:50:C2:FD:00:00/36SimpleSo,00:50:C2:FD:10:00/36Enyx,00:50:C2:FD:20:00/36Autonomi,00:50:C2:FD:30:00/36AsterEle,00:50:C2:FD:40:00/36Insitu,00:50:C2:FD:50:00/36American,00:50:C2:FD:60:00/36CityComp,00:50:C2:FD:70:00/36DEUTA-WE,00:50:C2:FD:80:00/36Ease,00:50:C2:FD:90:00/36FigmentD,00:50:C2:FD:A0:00/36Elan,00:50:C2:FD:B0:00/36Security,00:50:C2:FD:C0:00/36QuercusT,00:50:C2:FD:D0:00/36Toptech,00:50:C2:FD:E0:00/36PeekTraf,00:50:C2:FD:F0:00/36ACDElekt,00:50:C2:FE:00:00/36Azurtest,00:50:C2:FE:10:00/36dotOcean,00:50:C2:FE:20:00/36Pulsotro,00:50:C2:FE:30:00/36Private,00:50:C2:FE:40:00/36RTTMobil,00:50:C2:FE:50:00/36Scandino,00:50:C2:FE:60:00/36Exibea,00:50:C2:FE:70:00/36Erhardt+,00:50:C2:FE:80:00/36MangoDSP,00:50:C2:FE:90:00/36MBconnec,00:50:C2:FE:A0:00/36BrunelSe,00:50:C2:FE:B0:00/36AxibleTe,00:50:C2:FE:C0:00/36FirstSys,00:50:C2:FE:D0:00/36LOGISOLK,00:50:C2:FE:E0:00/36SparksIn,00:50:C2:FE:F0:00/36TaskSist,00:50:C2:FF:00:00/36GDMissio,00:50:C2:FF:10:00/36DiTESTFa,00:50:C2:FF:20:00/36Globalco,00:50:C2:FF:30:00/36CONTROL,00:50:C2:FF:40:00/36BurkTech,00:50:C2:FF:50:00/36FlexkomI,00:50:C2:FF:60:00/36BooycoEl,00:50:C2:FF:70:00/36HumanInt,00:50:C2:FF:80:00/36KSTtechn,00:50:C2:FF:90:00/36AVAMonit,00:50:C2:FF:A0:00/36Nupoint,00:50:C2:FF:B0:00/36Sefram,00:50:C2:FF:C0:00/36SpirentC,00:50:C2:FF:D0:00/36Touchles,00:50:C2:FF:E0:00/36SensataT,00:50:C2:FF:F0:00/36MSR-Solu,00:50:C4Imd,00:50:C5ADSTechn,00:50:C6LoopTele,00:50:C7Private,00:50:C8Addonics,00:50:C9MasproDe,00:50:CANetToNet,00:50:CBJetter,00:50:CCSeagateC,00:50:CDDigiansw,00:50:CELgIntern,00:50:CFVanlinkC,00:50:D0Minerva,00:50:D1Cisco,00:50:D2CMCElect,00:50:D3DigitalA,00:50:D4JoohongI,00:50:D5Ad,00:50:D6AtlasCop,00:50:D7Telstrat,00:50:D8UnicornC,00:50:D9ENGETRON,00:50:DA3Com,00:50:DBContempo,00:50:DCTasTelef,00:50:DDSerraSol,00:50:DESignum,00:50:DFAirFiber,00:50:E1NsTechEl,00:50:E2Cisco,00:50:E3ARRISGro,00:50:E4Apple,00:50:E6Hakusan,00:50:E7Paradise,00:50:E8Nomadix,00:50:EAXelCommu,00:50:EBAlpha-To,00:50:ECOlicom,00:50:EDAndaNetw,00:50:EETekDigit,00:50:EFSPESyste,00:50:F0Cisco,00:50:F1Maxlinea,00:50:F2Microsof,00:50:F3GLOBALNE,00:50:F4Sigmatek,00:50:F6Pan-Inte,00:50:F7VentureM,00:50:F8EntregaT,00:50:F9Sensorma,00:50:FAOxtel,00:50:FBVskElect,00:50:FCEdimaxTe,00:50:FDVisionco,00:50:FEPCTVnetA,00:50:FFHakkoEle,00:51:EDLGInnote,00:52:18WuxiKebo,00:52:C2peikerac,00:54:9FAvaya,00:54:AFContinen,00:54:BDSwelaser,00:55:00Xerox,00:55:B1Shanghai,00:55:DAIEEERegi,00:55:DA:00:00:00/28ShinkoTe,00:55:DA:10:00:00/28KoolPOS,00:55:DA:20:00:00/28BeijingC,00:55:DA:30:00:00/28NovexxSo,00:55:DA:40:00:00/28Datapath,00:55:DA:50:00:00/28Nanoleaf,00:55:DA:60:00:00/28Dekatron,00:55:DA:70:00:00/28Lucistec,00:55:DA:80:00:00/28BroadSof,00:55:DA:90:00:00/28QuantumC,00:55:DA:A0:00:00/28Speechla,00:55:DA:B0:00:00/28Interaxo,00:55:DA:C0:00:00/28DonguanW,00:55:DA:D0:00:00/28ArrowEle,00:55:DA:E0:00:00/28Victorsu,00:55:DA:F0:00:00/28Private,00:56:2BCisco,00:56:CDApple,00:57:C1LGElectr,00:57:D2Cisco,00:58:3FPCAquari,00:59:07LenovoEM,00:59:79Networke,00:59:ACKpn,00:59:DCCisco,00:5A:13HuaweiTe,00:5A:39Shenzhen,00:5B:94Apple,00:5B:A1shanghai,00:5C:86Shenzhen,00:5C:B1GospellD,00:5D:03Xilinx,00:5D:73Cisco,00:5E:0CHMDGloba,00:5F:67TP-Link,00:5F:86Cisco,00:5F:BFToshiba,00:60:00Xycom,00:60:01InnoSys,00:60:02ScreenSu,00:60:03TeraokaW,00:60:04Computad,00:60:05Feedback,00:60:06Sotec,00:60:07AcresGam,00:60:083com,00:60:09CiscoCat,00:60:0ASordComp,00:60:0BLOGWARE,00:60:0CEurotech,00:60:0DDigitalL,00:60:0EWavenetI,00:60:0FWestellT,00:60:10NetworkM,00:60:11CrystalS,00:60:12PowerCom,00:60:13NetstalM,00:60:14Edec,00:60:15Net2Net,00:60:16Clariion,00:60:17Tokimec,00:60:18StellarO,00:60:19RocheDia,00:60:1AKeithley,00:60:1BMesaElec,00:60:1CTelxon,00:60:1DLucentTe,00:60:1ESoftlab,00:60:1FStallion,00:60:20PivotalN,00:60:21Dsc,00:60:22Vicom,00:60:23PericomS,00:60:24Gradient,00:60:25ActiveIm,00:60:26VIKINGMo,00:60:27Superior,00:60:28Macrovis,00:60:29CaryPeri,00:60:2ASymicron,00:60:2BPeakAudi,00:60:2CLINXData,00:60:2DAlertonT,00:60:2ECyclades,00:60:2FCisco,00:60:30Villaget,00:60:31Hrk,00:60:32I-Cube,00:60:33AcuityIm,00:60:34ROBERTBO,00:60:35DallasSe,00:60:36AITAustr,00:60:37NXPSemic,00:60:38NortelNe,00:60:39SanComTe,00:60:3AQuickCon,00:60:3BAMTEC,00:60:3CHagiwara,00:60:3D3Cx,00:60:3ECisco100,00:60:3FPatapsco,00:60:40Netro,00:60:41Yokogawa,00:60:42TksUsa,00:60:43iDirect,00:60:44LittonPo,00:60:45Pathligh,00:60:46Vmetro,00:60:47Cisco,00:60:48DellEMC,00:60:49VinaTech,00:60:4ASaicIdea,00:60:4BSafe-com,00:60:4CSagemcom,00:60:4DMmcNetwo,00:60:4ECycleCom,00:60:4FTattile,00:60:50Internix,00:60:51QualityS,00:60:52PERIPHER,00:60:53ToyodaMa,00:60:54Controlw,00:60:55CornellU,00:60:56NetworkT,00:60:57MurataMa,00:60:58CopperMo,00:60:59Technica,00:60:5ACelcore,00:60:5BIntraSer,00:60:5CCisco,00:60:5DScanival,00:60:5ELibertyT,00:60:5FNipponUn,00:60:60DataInno,00:60:61WhistleC,00:60:62Telesync,00:60:63PsionDac,00:60:64Netcomm,00:60:65B&RIndus,00:60:66LACROIXT,00:60:67AcerLan,00:60:68Dialogic,00:60:69BrocadeC,00:60:6AMitsubis,00:60:6BSynclaye,00:60:6CArescom,00:60:6DDigitalE,00:60:6EDavicomS,00:60:6FClarionA,00:60:70CiscoRou,00:60:71MidasLab,00:60:72VxlInstr,00:60:73Redcreek,00:60:74Qsc,00:60:75Pentek,00:60:76Schlumbe,00:60:77PrisaNet,00:60:78PowerMea,00:60:79Mainstre,00:60:7ADvs,00:60:7BFore,00:60:7CWaveAcce,00:60:7DSentient,00:60:7EGigalabs,00:60:7FAuroraTe,00:60:80Microtro,00:60:81TvComInt,00:60:82Novalink,00:60:83Cisco362,00:60:84DigitalV,00:60:85StorageC,00:60:86LogicRep,00:60:87KansaiEl,00:60:88AnalogDe,00:60:89Xata,00:60:8ACitadelC,00:60:8BConferTe,00:60:8C3com1990,00:60:8DUnipulse,00:60:8EHEELECTR,00:60:8FTekramTe,00:60:90ArtizaNe,00:60:91FirstPac,00:60:92MicroSys,00:60:93Varian,00:60:94AmdPcnet,00:60:95Accu-Tim,00:60:96TSMicrot,00:60:973Com,00:60:98HtCommun,00:60:99SBE,00:60:9ANjkTechn,00:60:9BAstroNov,00:60:9CPerkin-E,00:60:9DPmiFoodE,00:60:9EAscX3-In,00:60:9FPhast,00:60:A0Switched,00:60:A1VPNet,00:60:A2NihonUni,00:60:A3Continuu,00:60:A4GEWTechn,00:60:A5Performa,00:60:A6Particle,00:60:A7MICROSEN,00:60:A8Tidomat,00:60:A9GesytecM,00:60:AAIntellig,00:60:ABLarscom,00:60:ACResilien,00:60:ADMegaChip,00:60:AETrioInfo,00:60:AFPacificM,00:60:B0HewlettP,00:60:B1InputOut,00:60:B2ProcessC,00:60:B3Z-Com,00:60:B4Glenayre,00:60:B5KEBA,00:60:B6LandComp,00:60:B7Channelm,00:60:B8CORELIS,00:60:B9NECPlatf,00:60:BASaharaNe,00:60:BBCabletro,00:60:BCKeunYoun,00:60:BDEnginuit,00:60:BEWebtroni,00:60:BFMacraigo,00:60:C0NeraNetw,00:60:C1WaveSpan,00:60:C2Mpl,00:60:C3Netvisio,00:60:C4Soliton,00:60:C5Ancot,00:60:C6Dcs,00:60:C7AmatiCom,00:60:C8KukaWeld,00:60:C9ControlN,00:60:CAHarmonic,00:60:CBHitachiZ,00:60:CCEmtrak,00:60:CDVideoSer,00:60:CEAcclaimC,00:60:CFAlteonNe,00:60:D0SnmpRese,00:60:D1CascadeC,00:60:D2LucentTe,00:60:D3At&T,00:60:D4EldatCom,00:60:D5AMADAMIY,00:60:D6NovAtel,00:60:D7EcolePol,00:60:D8Elmic,00:60:D9TransysN,00:60:DARedLionC,00:60:DBNtpElekt,00:60:DCNECMagnu,00:60:DDMyricom,00:60:DEKayser-T,00:60:DFBrocadeC,00:60:E0AxiomTec,00:60:E1OrckitCo,00:60:E2QuestEng,00:60:E3ArbinIns,00:60:E4Compuser,00:60:E5FujiAuto,00:60:E6Shomiti,00:60:E7Randata,00:60:E8HitachiC,00:60:E9AtopTech,00:60:EAStreamLo,00:60:EBFourthtr,00:60:ECHermaryO,00:60:EDRicardoT,00:60:EEApollo,00:60:EFFlytechT,00:60:F0JohnsonJ,00:60:F1ExpCompu,00:60:F2Lasergra,00:60:F3Performa,00:60:F4ADVANCED,00:60:F5IconWest,00:60:F6NextestC,00:60:F7Datafusi,00:60:F8LoranInt,00:60:F9DiamondL,00:60:FAEducatio,00:60:FBPacketee,00:60:FCConserva,00:60:FDNetICs,00:60:FELynxSyst,00:60:FFQuVis,00:61:51HuaweiTe,00:61:71Apple,00:62:ECCisco,00:63:DECloudwal,00:64:40Cisco,00:64:A6MaquetCa,00:64:AFDishTech,00:66:19HuaweiDe,00:66:4BHuaweiTe,00:67:62Fiberhom,00:68:2BHuaweiDe,00:68:EBHP,00:69:2DSunnovoI,00:69:67IEEERegi,00:69:67:00:00:00/28Annapurn,00:69:67:10:00:00/28miliwave,00:69:67:20:00:00/28NingboSh,00:69:67:30:00:00/28SuzhouRa,00:69:67:40:00:00/28CommandA,00:69:67:50:00:00/28Shenzhen,00:69:67:60:00:00/28Comcast,00:69:67:70:00:00/28PangaeaS,00:69:67:80:00:00/28Ambient-,00:69:67:90:00:00/28Hangzhou,00:69:67:A0:00:00/28Zhejiang,00:69:67:B0:00:00/28Datapand,00:69:67:C0:00:00/28DesirdDe,00:69:67:D0:00:00/28aversix,00:69:67:E0:00:00/28TianjinL,00:6B:6FHuaweiTe,00:6B:8EShanghai,00:6B:9EVizio,00:6B:A0Shenzhen,00:6B:F1Cisco,00:6C:BCCisco,00:6C:FDSichuanC,00:6D:52Apple,00:6D:61Guangzho,00:6D:FBVutrixTe,00:6E:02Xovis,00:6F:64SamsungE,00:6F:F2MitsumiE,00:70:B0MA-ComCo,00:70:B3DataReca,00:71:47AmazonTe,00:71:C2Pegatron,00:71:CCHonHaiPr,00:72:04SamsungE,00:72:63NetcoreT,00:72:78Cisco,00:73:8DShenzhen,00:73:E0SamsungE,00:74:9CRuijieNe,00:75:32Inid,00:75:E1Ampt,00:76:3DVeea,00:76:86Cisco,00:76:B1Somfy-Pr,00:77:8DCisco,00:77:E4NokiaSol,00:78:88Cisco,00:78:9ESagemcom,00:78:CDIgnition,00:7B:18SENTRY,00:7C:2DSamsungE,00:7D:60Apple,00:7D:FAVolkswag,00:7E:56ChinaDra,00:7E:95Cisco,00:7F:28Actionte,00:80:00Multitec,00:80:01Periphon,00:80:02Satelcom,00:80:03HytecEle,00:80:04AntlowCo,00:80:05CactusCo,00:80:06Compuadd,00:80:07DlogNc-S,00:80:08Dynatech,00:80:09JupiterO,00:80:0AJapanCom,00:80:0BCsk,00:80:0CVidecom,00:80:0DVosswink,00:80:0EAtlantix,00:80:0FStandard,00:80:10Commodor,00:80:11DigitalI,00:80:12ImsImsFa,00:80:13ThomasCo,00:80:14Esprit,00:80:15Seiko,00:80:16WandelGo,00:80:17Pfu,00:80:18KobeStee,00:80:19DaynaCom,00:80:1ABellAtla,00:80:1BKodiakTe,00:80:1CNewportS,00:80:1DIntegrat,00:80:1EXinetron,00:80:1FKruppAtl,00:80:20NetworkP,00:80:21AlcatelC,00:80:22Scan-Opt,00:80:23Integrat,00:80:24Kalpana,00:80:25TelitWir,00:80:26NetworkP,00:80:27Adaptive,00:80:28Tradpost,00:80:29EagleTec,00:80:2ATestSimu,00:80:2BIntegrat,00:80:2CSageGrou,00:80:2DXylogics,00:80:2ECastleRo,00:80:2FNational,00:80:30NexusEle,00:80:31Basys,00:80:32Access,00:80:33EMSAviat,00:80:34SmtGoupi,00:80:35Technolo,00:80:36ReflexMa,00:80:37Ericsson,00:80:38DataRese,00:80:39AlcatelS,00:80:3AVaritype,00:80:3BAptCommu,00:80:3CTvsElect,00:80:3DSurigike,00:80:3ESynernet,00:80:3FTatung,00:80:40JohnFluk,00:80:41VebKombi,00:80:42ForceCom,00:80:43Networld,00:80:44SystechC,00:80:45Matsushi,00:80:46Tattile,00:80:47In-Net,00:80:48CompexUs,00:80:49NissinEl,00:80:4APro-Log,00:80:4BEagleTec,00:80:4CContec,00:80:4DCycloneM,00:80:4EApexComp,00:80:4FDaikinIn,00:80:50Ziatech,00:80:51AdcFiber,00:80:52Technica,00:80:53Intellic,00:80:54Frontier,00:80:55Fermilab,00:80:56SPHINXEl,00:80:57Adsoft,00:80:58Printer,00:80:59StanleyE,00:80:5ATulipCom,00:80:5BCondor,00:80:5CAgilis,00:80:5DCanstar,00:80:5ELsiLogic,00:80:5FHewlettP,00:80:60NetworkI,00:80:61Litton,00:80:62Interfac,00:80:63RichardH,00:80:64WyseTech,00:80:65Cybergra,00:80:66ArcomCon,00:80:67SquareD,00:80:68Yamatech,00:80:69Computon,00:80:6AEriEmpac,00:80:6BSchmidTe,00:80:6CCegelecP,00:80:6DCentury,00:80:6ENipponSt,00:80:6FOnelan,00:80:70Computad,00:80:71SaiTechn,00:80:72Microple,00:80:73DwbAssoc,00:80:74FisherCo,00:80:75Parsytec,00:80:76Mcnc,00:80:77Brotheri,00:80:78Practica,00:80:79Microbus,00:80:7AAitech,00:80:7BArtelCom,00:80:7CFibercom,00:80:7DEquinox,00:80:7ESouthern,00:80:7FDy-4,00:80:80Datamedi,00:80:81KendallS,00:80:82PepModul,00:80:83Amdahl,00:80:84Cloud,00:80:85H-Three,00:80:86Computer,00:80:87OkiElect,00:80:88VictorJa,00:80:89Tecnetic,00:80:8ASummit,00:80:8BDacoll,00:80:8CNetscout,00:80:8DWestcove,00:80:8ERadstone,00:80:8FCItohEle,00:80:90Microtek,00:80:91TokyoEle,00:80:92SilexTec,00:80:93Xyron,00:80:94AlfaLava,00:80:95BasicMer,00:80:96HumanDes,00:80:97Centralp,00:80:98Tdk,00:80:99EatonInd,00:80:9ANovusNet,00:80:9BJustsyst,00:80:9CLuxcom,00:80:9DCommscra,00:80:9EDatus,00:80:9FAlcatelB,00:80:A0HewlettP,00:80:A1Microtes,00:80:A2Creative,00:80:A3Lantroni,00:80:A4LibertyE,00:80:A5SpeedInt,00:80:A6Republic,00:80:A7Honeywel,00:80:A8Vitacom,00:80:A9Clearpoi,00:80:AAMaxpeed,00:80:ABDukaneNe,00:80:ACImlogixD,00:80:ADCnetTech,00:80:AEHughesNe,00:80:AFAllumer,00:80:B0Advanced,00:80:B1Softcom,00:80:B2NetNetwo,00:80:B3AvalData,00:80:B4Sophia,00:80:B5UnitedNe,00:80:B6Themis,00:80:B7StellarC,00:80:B8DmgMoriB,00:80:B9ArcheTec,00:80:BASpeciali,00:80:BBHughesLa,00:80:BCHitachiE,00:80:BDFurukawa,00:80:BEAriesRes,00:80:BFTakaokaE,00:80:C0PenrilDa,00:80:C1Lanex,00:80:C2Ieee8021,00:80:C3BiccInfo,00:80:C4Document,00:80:C5Novellco,00:80:C6National,00:80:C7Xircom,00:80:C8D-LinkAl,00:80:C9AlbertaM,00:80:CANetcomRe,00:80:CBFalcoDat,00:80:CCMicrowav,00:80:CDMicronic,00:80:CEBroadcas,00:80:CFEmbedded,00:80:D0Computer,00:80:D1Kimtron,00:80:D2Shinniho,00:80:D3ShivaApp,00:80:D4Chase,00:80:D5CadreTec,00:80:D6Nuvotech,00:80:D7FantumEl,00:80:D8NetworkP,00:80:D9EMKElekt,00:80:DABruelKja,00:80:DBGraphon,00:80:DCPickerIn,00:80:DDGmxGimix,00:80:DEGipsi,00:80:DFAdcCoden,00:80:E0Xtp,00:80:E1STMicroe,00:80:E2TDI,00:80:E3Coral,00:80:E4Northwes,00:80:E5NetApp,00:80:E6PeerNetw,00:80:E7Leonardo,00:80:E8CumulusC,00:80:E9Madge,00:80:EAADVAOpti,00:80:EBCompcont,00:80:ECSupercom,00:80:EDIqTechno,00:80:EEThomsonC,00:80:EFRational,00:80:F0Panasoni,00:80:F1Opus,00:80:F2Raycom,00:80:F3SunElect,00:80:F4Telemech,00:80:F5Quantel,00:80:F6SynergyM,00:80:F7ZenithCo,00:80:F8Mizar,00:80:F9Heurikon,00:80:FARwt,00:80:FBBvm,00:80:FCAvatar,00:80:FDExsceedC,00:80:FEAzureTec,00:80:FFSocDeTel,00:81:C4Cisco,00:81:F9TexasIns,00:84:EDPrivate,00:86:9CPaloAlto,00:86:A0Private,00:87:01SamsungE,00:87:31Cisco,00:87:64Cisco,00:88:65Apple,00:88:BANc&C,00:8A:55HuaweiDe,00:8A:76Apple,00:8A:96Cisco,00:8B:43Rftech,00:8B:FCmixi,00:8C:10BlackBox,00:8C:54ADBBroad,00:8C:FAInventec,00:8D:4ECjscNiiS,00:8D:DALinkOne,00:8E:73Cisco,00:8E:F2Netgear,00:90:00DiamondM,00:90:01NishimuE,00:90:02Allgon,00:90:03Aplio,00:90:043ComEuro,00:90:05Protech,00:90:06Hamamats,00:90:07DomexTec,00:90:08HanA,00:90:09IControl,00:90:0AProtonEl,00:90:0BLannerEl,00:90:0CCisco,00:90:0DOverland,00:90:0EHandlink,00:90:0FKawasaki,00:90:10Simulati,00:90:11WAVTrace,00:90:12Globespa,00:90:13Samsan,00:90:14RotorkIn,00:90:15Centigra,00:90:16Zac,00:90:17Zypcom,00:90:18ItoElect,00:90:19HermesEl,00:90:1AUnispher,00:90:1BDigitalC,00:90:1CmpsSoftw,00:90:1DPecNz,00:90:1ESelestaI,00:90:1FAdtecPro,00:90:20PhilipsA,00:90:21Cisco,00:90:22Ivex,00:90:23Zilog,00:90:24Pipelink,00:90:25BAEAustr,00:90:26Advanced,00:90:27Intel,00:90:28NipponSi,00:90:29Crypto,00:90:2ACommunic,00:90:2BCiscoEth,00:90:2CDataCont,00:90:2DDataElec,00:90:2ENamco,00:90:2FNetcore,00:90:30Honeywel,00:90:31Mysticom,00:90:32Pelcombe,00:90:33Innovaph,00:90:34Imagic,00:90:35AlphaTel,00:90:36ens,00:90:37Acucomm,00:90:38Fountain,00:90:39ShastaNe,00:90:3ANihonMed,00:90:3BTriEMSRe,00:90:3CAtlantic,00:90:3DBiopac,00:90:3EPhilipsI,00:90:3FAztecRad,00:90:40SiemensN,00:90:41AppliedD,00:90:42ECCS,00:90:43Tattile,00:90:44AssuredD,00:90:45MarconiC,00:90:46Dexdyne,00:90:47GigaFast,00:90:48Zeal,00:90:49Entridia,00:90:4AConcurSy,00:90:4BGemtekTe,00:90:4CEpigram,00:90:4DSpec,00:90:4EDelem,00:90:4FAbbPower,00:90:50Teleste,00:90:51Ultimate,00:90:52SelcomEl,00:90:53DaewooEl,00:90:54Innovati,00:90:55ParkerHa,00:90:56Telestre,00:90:57AANetcom,00:90:58UltraEle,00:90:59TelecomD,00:90:5ADearborn,00:90:5BRaymondA,00:90:5CEdmi,00:90:5DNetcomSi,00:90:5ERauland-,00:90:5FCisco,00:90:60SystemCr,00:90:61PacificR,00:90:62ICPVORTE,00:90:63Coherent,00:90:64Thomson,00:90:65Finisar,00:90:66TroikaNe,00:90:67WalkAbou,00:90:68Dvt,00:90:69JuniperN,00:90:6ATurnston,00:90:6BAppliedR,00:90:6CSartoriu,00:90:6DCisco,00:90:6EPraxon,00:90:6FCisco,00:90:70NeoNetwo,00:90:71AppliedI,00:90:72SimradAs,00:90:73GaioTech,00:90:74ArgonNet,00:90:75NecDoBra,00:90:76FmtAircr,00:90:77Advanced,00:90:78MerTelem,00:90:79ClearOne,00:90:7ASpectral,00:90:7BE-Tech,00:90:7CDigitalc,00:90:7DLakeComm,00:90:7EVetronix,00:90:7FWatchGua,00:90:80Not,00:90:81AlohaNet,00:90:82ForceIns,00:90:83TurboCom,00:90:84AtechSys,00:90:85GoldenEn,00:90:86Cisco,00:90:87Itis,00:90:88BaxallSe,00:90:89SoftcomM,00:90:8ABaylyCom,00:90:8BTattile,00:90:8CEtrendEl,00:90:8DVickersE,00:90:8ENortelNe,00:90:8FAudioCod,00:90:90I-Bus,00:90:91DigitalS,00:90:92Cisco,00:90:93EIZO,00:90:94OspreyTe,00:90:95Universa,00:90:96AskeyCom,00:90:97Sycamore,00:90:98SbcDesig,00:90:99AlliedTe,00:90:9AOneWorld,00:90:9BMarkem-I,00:90:9CARRISGro,00:90:9DNovaTech,00:90:9ECritical,00:90:9FDigi-Dat,00:90:A08X8,00:90:A1FlyingPi,00:90:A2CyberTAN,00:90:A3Corecess,00:90:A4AltigaNe,00:90:A5SpectraL,00:90:A6Cisco,00:90:A7Clientec,00:90:A8NineTile,00:90:A9WesternD,00:90:AAIndigoAc,00:90:ABCisco,00:90:ACOptivisi,00:90:ADAspectEl,00:90:AEITALTELR,00:90:AFJMoritaM,00:90:B0Vadem,00:90:B1Cisco,00:90:B2Avici,00:90:B3Agranat,00:90:B4Willowbr,00:90:B5Nikon,00:90:B6Fibex,00:90:B7DigitalL,00:90:B8RohdeSch,00:90:B9BeranIns,00:90:BAValidNet,00:90:BBTAINETCO,00:90:BCTelemann,00:90:BDOmniaCom,00:90:BEIbcInteg,00:90:BFCisco,00:90:C0KJLawEng,00:90:C1PecoII,00:90:C2JKmicros,00:90:C3TopicSem,00:90:C4Javelin,00:90:C5Internet,00:90:C6Optim,00:90:C7Icom,00:90:C8Waveride,00:90:C9DPACTech,00:90:CAAccordVi,00:90:CBWireless,00:90:CCPlanexCo,00:90:CDEnt-Empr,00:90:CEavateram,00:90:CFNortel,00:90:D0ThomsonT,00:90:D1LeichuEn,00:90:D2ArtelVid,00:90:D3GIESECKE,00:90:D4BindView,00:90:D5Euphonix,00:90:D6CrystalG,00:90:D7NetBoost,00:90:D8Whitecro,00:90:D9Cisco,00:90:DADynarc,00:90:DBNextLeve,00:90:DCTecoInfo,00:90:DDMIHARUCO,00:90:DECardkey,00:90:DFMitsubis,00:90:E0Systran,00:90:E1Telena,00:90:E2Distribu,00:90:E3AvexElec,00:90:E4NecAmeri,00:90:E5Teknema,00:90:E6ALi,00:90:E7HorschEl,00:90:E8MoxaTech,00:90:E9JanzComp,00:90:EAAlphaTec,00:90:EBSentryTe,00:90:ECPyrescom,00:90:EDCentralS,00:90:EEPersonal,00:90:EFIntegrix,00:90:F0Harmonic,00:90:F1SeagateC,00:90:F2CiscoEth,00:90:F3AspectCo,00:90:F4Lightnin,00:90:F5Clevo,00:90:F6Escalate,00:90:F7NbaseCom,00:90:F8Mediatri,00:90:F9ImagineC,00:90:FAEmulex,00:90:FBPortwell,00:90:FCNetworkC,00:90:FDCopperCo,00:90:FEElecomLa,00:90:FFTellusTe,00:91:9EIntelCor,00:91:D6CrystalG,00:91:EBRenesasE,00:91:FASynapseP,00:92:7DFicosaIn,00:92:FAShenzhen,00:93:37IntelCor,00:93:63Uni-Link,00:94:A1F5Networ,00:94:ECHuaweiDe,00:95:69LSDScien,00:97:FFHeimannS,00:9A:CDHuaweiTe,00:9A:D2Cisco,00:9C:02HewlettP,00:9D:6BMurataMa,00:9D:8ECardiacR,00:9E:1ECisco,00:9E:C8XiaomiCo,00:9E:EEPositivo,00:A0:00Centilli,00:A0:01DRSSigna,00:A0:02LeedsNor,00:A0:03Siemens,00:A0:04Netpower,00:A0:05DanielIn,00:A0:06ImageDat,00:A0:07ApexxTec,00:A0:08Netcorp,00:A0:09Whitetre,00:A0:0AAirspan,00:A0:0BComputex,00:A0:0CKingmaxT,00:A0:0DPandaPro,00:A0:0ENetscout,00:A0:0FBroadban,00:A0:10Syslogic,00:A0:11MutohInd,00:A0:12Telco,00:A0:13Teltrend,00:A0:14Csir,00:A0:15Wyle,00:A0:16Micropol,00:A0:17JBM,00:A0:18Creative,00:A0:19NebulaCo,00:A0:1ABinarEle,00:A0:1BPremisys,00:A0:1CNascentN,00:A0:1DRedLionC,00:A0:1EEst,00:A0:1FTricord,00:A0:20Citicorp,00:A0:21GeneralD,00:A0:22CentreFo,00:A0:23AppliedC,00:A0:243Com,00:A0:25RedcomLa,00:A0:26Teldat,00:A0:27Firepowe,00:A0:28ConnerPe,00:A0:29Coulter,00:A0:2ATrancell,00:A0:2BTransiti,00:A0:2CinterWAV,00:A0:2D1394Trad,00:A0:2EBrandCom,00:A0:2FADBBroad,00:A0:30Captor,00:A0:31Hazeltin,00:A0:32GesSinga,00:A0:33imcMeBsy,00:A0:34Axel,00:A0:35Cylink,00:A0:36AppliedN,00:A0:37MindrayD,00:A0:38EmailEle,00:A0:39RossTech,00:A0:3AKubotek,00:A0:3BToshinEl,00:A0:3CEg&GNucl,00:A0:3DOpto-22,00:A0:3EAtmForum,00:A0:3FComputer,00:A0:40ApplePci,00:A0:41Inficon,00:A0:42SpurProd,00:A0:43American,00:A0:44NttIt,00:A0:45PHOENIXC,00:A0:46Scitex,00:A0:47Integrat,00:A0:48Questech,00:A0:49Digitech,00:A0:4ANisshinE,00:A0:4BTflLan,00:A0:4CInnovati,00:A0:4DEdaInstr,00:A0:4EVoelkerT,00:A0:4FAmeritec,00:A0:50CypressS,00:A0:51AngiaCom,00:A0:52Stanilit,00:A0:53CompactD,00:A0:54Private,00:A0:55DataDevi,00:A0:56Micropro,00:A0:57LANCOM,00:A0:58Glory,00:A0:59Hamilton,00:A0:5AKofaxIma,00:A0:5BMarquip,00:A0:5CInventor,00:A0:5DCSCOMPUT,00:A0:5EMyriadLo,00:A0:5FBTGElect,00:A0:60AcerPeri,00:A0:61PuritanB,00:A0:62AesProda,00:A0:63Jrl,00:A0:64KvbAnale,00:A0:65Symantec,00:A0:66Isa,00:A0:67NetworkS,00:A0:68Bhp,00:A0:69Symmetri,00:A0:6AVerilink,00:A0:6BDmsDorsc,00:A0:6CShindeng,00:A0:6DMannesma,00:A0:6EAustron,00:A0:6FColorSen,00:A0:70Coastcom,00:A0:71VideoLot,00:A0:72Ovation,00:A0:73Com21,00:A0:74Percepti,00:A0:75MicronTe,00:A0:76Cardware,00:A0:77FujitsuN,00:A0:78MarconiC,00:A0:79AlpsElec,00:A0:7AAdvanced,00:A0:7BDawnComp,00:A0:7CTonyangN,00:A0:7DSeeqTech,00:A0:7EAvidTech,00:A0:7FGsm-Synt,00:A0:80Tattile,00:A0:81AlcatelD,00:A0:82NktElekt,00:A0:83Asimmpho,00:A0:84Dataplex,00:A0:85Private,00:A0:86AmberWav,00:A0:87Microsem,00:A0:88Essentia,00:A0:89XpointTe,00:A0:8ABrooktro,00:A0:8BAstonEle,00:A0:8CMultiMed,00:A0:8DJacomo,00:A0:8ECheckPoi,00:A0:8FDesknet,00:A0:90TimeStep,00:A0:91Applicom,00:A0:92HBollman,00:A0:93BEAEROSP,00:A0:94Comsat,00:A0:95AcaciaNe,00:A0:96MitsumiE,00:A0:97JcInform,00:A0:98NetApp,00:A0:99K-Net,00:A0:9ANihonKoh,00:A0:9BQpsxComm,00:A0:9CXyplex,00:A0:9DJohnatho,00:A0:9EIctv,00:A0:9FCommvisi,00:A0:A0CompactD,00:A0:A1EpicData,00:A0:A2Digicom,00:A0:A3Reliable,00:A0:A4Oracle,00:A0:A5TeknorMi,00:A0:A6MI,00:A0:A7Vorax,00:A0:A8Renex,00:A0:A9NavtelCo,00:A0:AASpacelab,00:A0:ABNetcsInf,00:A0:ACGilatSat,00:A0:ADMarconi,00:A0:AENucom,00:A0:AFWmsIndus,00:A0:B0I-ODataD,00:A0:B1FirstVir,00:A0:B2ShimaSei,00:A0:B3Zykronix,00:A0:B4TexasMic,00:A0:B53HTechno,00:A0:B6SanritzA,00:A0:B7Cordant,00:A0:B8NetApp,00:A0:B9EagleTec,00:A0:BAPattonEl,00:A0:BBHilan,00:A0:BCViasat,00:A0:BDI-Tech,00:A0:BEIntegrat,00:A0:BFWireless,00:A0:C0DigitalL,00:A0:C1OrtivusM,00:A0:C2RA,00:A0:C3Unicompu,00:A0:C4CristieE,00:A0:C5ZyxelCom,00:A0:C6Qualcomm,00:A0:C7TadiranT,00:A0:C8Adtran,00:A0:C9IntelPro,00:A0:CAFujitsuD,00:A0:CBArkTelec,00:A0:CCLite-OnU,00:A0:CDDRJOHANN,00:A0:CEEcessa,00:A0:CFSotas,00:A0:D0TenXTech,00:A0:D1Inventec,00:A0:D2AlliedTe,00:A0:D3InstemCo,00:A0:D4Radiolan,00:A0:D5SierraWi,00:A0:D6SBE,00:A0:D7KastenCh,00:A0:D8Spectra-,00:A0:D9ConvexCo,00:A0:DAINTEGRAT,00:A0:DBFisherPa,00:A0:DCONElectr,00:A0:DDAzonix,00:A0:DEYamaha,00:A0:DFStsTechn,00:A0:E0Tennyson,00:A0:E1Westport,00:A0:E2Keisokug,00:A0:E3Xkl,00:A0:E4Optiques,00:A0:E5NhcCommu,00:A0:E6Dialogic,00:A0:E7CentralD,00:A0:E8ReutersH,00:A0:E9Electron,00:A0:EAEthercom,00:A0:EBEncoreNe,00:A0:ECTransmit,00:A0:EDBrooksAu,00:A0:EENashobaN,00:A0:EFLucidata,00:A0:F0TorontoM,00:A0:F1Mti,00:A0:F2InfotekC,00:A0:F3Staubli,00:A0:F4Ge,00:A0:F5Radguard,00:A0:F6AutoGas,00:A0:F7VIComput,00:A0:F8ZebraTec,00:A0:F9BintecCo,00:A0:FAMarconiC,00:A0:FBTorayEng,00:A0:FCImageSci,00:A0:FDScitexDi,00:A0:FEBostonTe,00:A0:FFTellabsO,00:A1:DEShenZhen,00:A2:89Cisco,00:A2:DAINAT,00:A2:EECisco,00:A2:F5Guangzho,00:A2:FFabatecgr,00:A3:88BSkyB,00:A3:8ECisco,00:A3:D1Cisco,00:A5:09WigWag,00:A5:BFCisco,00:A6:CACisco,00:A7:42Cisco,00:A7:84ITXsecur,00:AA:00Intel,00:AA:01Intel,00:AA:02Intel,00:AA:3COlivetti,00:AA:6ECisco,00:AA:70LGElectr,00:AB:48eero,00:AC:E0ARRISGro,00:AD:24D-LinkIn,00:AD:63Dedicate,00:AD:D5HuaweiDe,00:AE:CDPensando,00:AE:FAMurataMa,00:AF:1FCisco,00:B0:09GrassVal,00:B0:17InfoGear,00:B0:19UtcCcs,00:B0:1CWestport,00:B0:1ERanticLa,00:B0:2AORSYS,00:B0:2DViaGateT,00:B0:33Izhevski,00:B0:3BHiQNetwo,00:B0:48MarconiC,00:B0:4ACisco,00:B0:52AtherosC,00:B0:64Cisco,00:B0:69Honewell,00:B0:6DJonesFut,00:B0:80Mannesma,00:B0:86LocSoft,00:B0:8ECisco,00:B0:91Transmet,00:B0:94Alaris,00:B0:9AMorrowTe,00:B0:9DPointGre,00:B0:ACSIAE-Mic,00:B0:AESymmetri,00:B0:B3Xstreami,00:B0:C2Cisco,00:B0:C7TellabsO,00:B0:CEViverisT,00:B0:D0Computer,00:B0:DBNextcell,00:B0:DFStarboar,00:B0:E1Cisco,00:B0:E7BritishF,00:B0:ECEacem,00:B0:EEAjile,00:B0:F0CalyNetw,00:B0:F5NetWorth,00:B1:E3Cisco,00:B3:38KontronA,00:B3:42MacroSAN,00:B3:62Apple,00:B4:F5DongGuan,00:B5:6DDavidEle,00:B5:D0SamsungE,00:B5:D6Omnibit,00:B6:00VOIM,00:B6:70Cisco,00:B6:9FLatch,00:B7:71Cisco,00:B7:8DNanjingS,00:B7:A8Heinzing,00:B8:10YichipMi,00:B8:81Newplatf,00:B8:B3Cisco,00:B8:B6Motorola,00:B8:C2HeightsT,00:B9:F6Shenzhen,00:BA:C0Biometri,00:BB:01Octothor,00:BB:1CHuaweiDe,00:BB:3AAmazonTe,00:BB:60IntelCor,00:BB:8EHME,00:BB:C1Canon,00:BB:F0Ungerman,00:BC:60Cisco,00:BD:27Exar,00:BD:3ANokia,00:BD:3EVizio,00:BD:82Shenzhen,00:BE:3BHuaweiTe,00:BE:75Cisco,00:BE:9EFiberhom,00:BE:D5NewH3CTe,00:BF:15Genetec,00:BF:61SamsungE,00:BF:77Cisco,00:BF:AFHuiZhouG,00:C0:00Lanoptic,00:C0:01DiatekPa,00:C0:02Sercomm,00:C0:03Globalne,00:C0:04JapanBus,00:C0:05Livingst,00:C0:06NipponAv,00:C0:07Pinnacle,00:C0:08Seco,00:C0:09KtTechno,00:C0:0AMicroCra,00:C0:0BNorcontr,00:C0:0CReliaTec,00:C0:0DAdvanced,00:C0:0EPsitech,00:C0:0FQnxSoftw,00:C0:10Hirakawa,00:C0:11Interact,00:C0:12Netspan,00:C0:13Netrix,00:C0:14Telemati,00:C0:15NewMedia,00:C0:16Electron,00:C0:17NetAlly,00:C0:18Lanart,00:C0:19LeapTech,00:C0:1ACorometr,00:C0:1BSocketCo,00:C0:1CInterlin,00:C0:1DGrandJun,00:C0:1ELaFranca,00:C0:1FSERCEL,00:C0:20ArcoElec,00:C0:21Netexpre,00:C0:22Lasermas,00:C0:23Tutankha,00:C0:24EdenSist,00:C0:25Dataprod,00:C0:26LansTech,00:C0:27Cipher,00:C0:28Jasco,00:C0:29NexansDe,00:C0:2AOhkuraEl,00:C0:2BGerloffG,00:C0:2CCentrumC,00:C0:2DFujiPhot,00:C0:2ENetwiz,00:C0:2FOkuma,00:C0:30Integrat,00:C0:31DesignRe,00:C0:32I-Cubed,00:C0:33Telebit,00:C0:34Transact,00:C0:35Quintar,00:C0:36RaytechE,00:C0:37Dynatem,00:C0:38RasterIm,00:C0:39Teridian,00:C0:3AMen-Mikr,00:C0:3BMultiacc,00:C0:3CTowerTec,00:C0:3DWieseman,00:C0:3EFaGebrHe,00:C0:3FStoresAu,00:C0:40Ecci,00:C0:41DigitalT,00:C0:42Datalux,00:C0:43Strataco,00:C0:44Emcom,00:C0:45Isolatio,00:C0:46BlueChip,00:C0:47Unimicro,00:C0:48BayTechn,00:C0:49UsRoboti,00:C0:4AGroup200,00:C0:4BCreative,00:C0:4CDepartme,00:C0:4DMitec,00:C0:4EComtrol,00:C0:4FDell,00:C0:50ToyoDenk,00:C0:51Advanced,00:C0:52Burr-Bro,00:C0:53AspectSo,00:C0:54NetworkP,00:C0:55ModularC,00:C0:56Somelec,00:C0:57MycoElec,00:C0:58Dataexpe,00:C0:59Denso,00:C0:5ASemaphor,00:C0:5BNetworks,00:C0:5CElonex,00:C0:5DL&NTechn,00:C0:5EVari-Lit,00:C0:5FFine-Pal,00:C0:60IdScandi,00:C0:61Solectek,00:C0:62ImpulseT,00:C0:63MorningS,00:C0:64GeneralD,00:C0:65ScopeCom,00:C0:66Docupoin,00:C0:67UnitedBa,00:C0:68HMEClear,00:C0:69Axxceler,00:C0:6AZahner-E,00:C0:6BOsiPlus,00:C0:6CSvecComp,00:C0:6DBocaRese,00:C0:6EHaftTech,00:C0:6FKomatsu,00:C0:70SectraSe,00:C0:71AreanexC,00:C0:72Knx,00:C0:73Xedia,00:C0:74ToyodaAu,00:C0:75Xante,00:C0:76I-DataIn,00:C0:77DaewooTe,00:C0:78Computer,00:C0:79Fonsys,00:C0:7APrivaBv,00:C0:7BAscendCo,00:C0:7CHightech,00:C0:7DRiscDeve,00:C0:7EKubotaEl,00:C0:7FNuponCom,00:C0:80Netstar,00:C0:81Metrodat,00:C0:82MoorePro,00:C0:83TraceMou,00:C0:84DataLink,00:C0:85Canon,00:C0:86Lynk,00:C0:87UunetTec,00:C0:88EkfElekt,00:C0:89Telindus,00:C0:8ALauterba,00:C0:8BRisqModu,00:C0:8CPerforma,00:C0:8DTronixPr,00:C0:8ENetworkI,00:C0:8FPanasoni,00:C0:90Praim,00:C0:91JabilCir,00:C0:92MennenMe,00:C0:93AltaRese,00:C0:94Vmx,00:C0:95ZnyxNetw,00:C0:96Tamura,00:C0:97Archipel,00:C0:98ChuntexE,00:C0:99YoshikiI,00:C0:9APhotonic,00:C0:9BReliance,00:C0:9CHiokiEE,00:C0:9DDistribu,00:C0:9ECacheCom,00:C0:9FQuantaCo,00:C0:A0AdvanceM,00:C0:A1TokyoDen,00:C0:A2Intermed,00:C0:A3DualEnte,00:C0:A4Unigraf,00:C0:A5DickensD,00:C0:A6ExicomAu,00:C0:A7Seel,00:C0:A8Gvc,00:C0:A9BarronMc,00:C0:AASiliconV,00:C0:ABTelco,00:C0:ACGambitCo,00:C0:ADMarbenCo,00:C0:AETowercom,00:C0:AFTeklogix,00:C0:B0GccTechn,00:C0:B1GeniusNe,00:C0:B2Norand,00:C0:B3ComstatD,00:C0:B4MysonTec,00:C0:B5Corporat,00:C0:B6HVE,00:C0:B7American,00:C0:B8FraserSH,00:C0:B9FunkSoft,00:C0:BANetvanta,00:C0:BBForvalCr,00:C0:BCTelecomA,00:C0:BDInexTech,00:C0:BEAlcatel-,00:C0:BFTechnolo,00:C0:C0ShoreMic,00:C0:C1QuadGrap,00:C0:C2Infinite,00:C0:C3AcusonCo,00:C0:C4Computer,00:C0:C5SidInfor,00:C0:C6Personal,00:C0:C7Sparktru,00:C0:C8MicroByt,00:C0:C9BaileyCo,00:C0:CAAlfa,00:C0:CBControlT,00:C0:CCTelescie,00:C0:CDComelta,00:C0:CECeiEngin,00:C0:CFImatranV,00:C0:D0RatocSys,00:C0:D1ComtreeT,00:C0:D2Syntelle,00:C0:D3OlympusI,00:C0:D4AxonNetw,00:C0:D5QuancomE,00:C0:D6J1,00:C0:D7TaiwanTr,00:C0:D8Universa,00:C0:D9QuinteNe,00:C0:DANice,00:C0:DBIpcPte,00:C0:DCEosTechn,00:C0:DDQLogic,00:C0:DEZcomm,00:C0:DFKYE,00:C0:E0DscCommu,00:C0:E1SonicSol,00:C0:E2Calcomp,00:C0:E3OsitechC,00:C0:E4SiemensB,00:C0:E5Gespac,00:C0:E6Verilink,00:C0:E7Fiberdat,00:C0:E8Plexcom,00:C0:E9OakSolut,00:C0:EAArrayTec,00:C0:EBSehCompu,00:C0:ECDauphinT,00:C0:EDUsArmyEl,00:C0:EEKyocera,00:C0:EFAbit,00:C0:F0Kingston,00:C0:F1ShinkoEl,00:C0:F2Transiti,00:C0:F3NetworkC,00:C0:F4Interlin,00:C0:F5Metacomp,00:C0:F6CelanTec,00:C0:F7EngageCo,00:C0:F8AboutCom,00:C0:F9ArtesynE,00:C0:FACanaryCo,00:C0:FBAdvanced,00:C0:FCAsdg,00:C0:FDProsum,00:C0:FEAptecCom,00:C0:FFSeagateC,00:C1:4FDDL,00:C1:64Cisco,00:C1:B1Cisco,00:C2:C6IntelCor,00:C3:43E-T-ACir,00:C3:F4SamsungE,00:C5:2CJuniperN,00:C5:DBDatatech,00:C6:10Apple,00:C8:8BCisco,00:CA:E5Cisco,00:CB:00Private,00:CB:51Sagemcom,00:CB:B4Shenzhen,00:CB:BDCambridg,00:CC:34JuniperN,00:CC:3FUniversa,00:CC:FCCisco,00:CD:90MASElekt,00:CD:FEApple,00:CE:30ExpressL,00:CF:1CCommunic,00:CF:C0ChinaMob,00:D0:00FerranSc,00:D0:01VstTechn,00:D0:02Ditech,00:D0:03ComdaEnt,00:D0:04Pentacom,00:D0:05ZhsZeitm,00:D0:06Cisco,00:D0:07MicAssoc,00:D0:08Mactell,00:D0:09HsingTec,00:D0:0ALanacces,00:D0:0BRhkTechn,00:D0:0CSnijderM,00:D0:0DMicromer,00:D0:0EPluris,00:D0:0FSpeechDe,00:D0:10Converge,00:D0:11PrismVid,00:D0:12Gatework,00:D0:13PrimexAe,00:D0:14Root,00:D0:15UnivexMi,00:D0:16ScmMicro,00:D0:17SyntechI,00:D0:18QwesCom,00:D0:19Dainippo,00:D0:1AUrmetTlc,00:D0:1BMimakiEn,00:D0:1CSbsTechn,00:D0:1DFurunoEl,00:D0:1EPingtel,00:D0:1FSenetas,00:D0:20AimSyste,00:D0:21RegentEl,00:D0:22Incredib,00:D0:23Infortre,00:D0:24Cognex,00:D0:25Xrosstec,00:D0:26Hirschma,00:D0:27AppliedA,00:D0:28Harmonic,00:D0:29Wakefern,00:D0:2AVoxent,00:D0:2BJetcell,00:D0:2CCampbell,00:D0:2DResideo,00:D0:2ECommunic,00:D0:2FVlsiTech,00:D0:30Safetran,00:D0:31Industri,00:D0:32YanoElec,00:D0:33DalianDa,00:D0:34Ormec,00:D0:35Behavior,00:D0:36Technolo,00:D0:37ARRISGro,00:D0:38Fivemere,00:D0:39Utilicom,00:D0:3AZoneworx,00:D0:3BVisionPr,00:D0:3CVieo,00:D0:3DGalileoT,00:D0:3ERocketch,00:D0:3FAmerican,00:D0:40Sysmate,00:D0:41AmigoTec,00:D0:42MahloUg,00:D0:43ZonalRet,00:D0:44AlidianN,00:D0:45Kvaser,00:D0:46DolbyLab,00:D0:47XnTechno,00:D0:48Ecton,00:D0:49Impresst,00:D0:4APresence,00:D0:4BLaCieGro,00:D0:4CEurotelT,00:D0:4DDivResea,00:D0:4ELogibag,00:D0:4FBitronic,00:D0:50Iskratel,00:D0:51O2Micro,00:D0:52AscendCo,00:D0:53Connecte,00:D0:54SasInsti,00:D0:55Kathrein,00:D0:56Somat,00:D0:57Ultrak,00:D0:58Cisco,00:D0:59AmbitMic,00:D0:5ASymbioni,00:D0:5BAcroloop,00:D0:5CKATHREIN,00:D0:5DIntelliw,00:D0:5EStratabe,00:D0:5FValcom,00:D0:60Panasoni,00:D0:61TremonEn,00:D0:62Digigram,00:D0:63Cisco,00:D0:64Multitel,00:D0:65TokoElec,00:D0:66Wintriss,00:D0:67CampioCo,00:D0:68Iwill,00:D0:69Technolo,00:D0:6ALinkup,00:D0:6BSrTeleco,00:D0:6CSharewav,00:D0:6DAcrison,00:D0:6ETrendvie,00:D0:6FKmcContr,00:D0:70LongWell,00:D0:71Echelon,00:D0:72Broadlog,00:D0:73AcnAdvan,00:D0:74Taqua,00:D0:75AlarisMe,00:D0:76BankAmer,00:D0:77LucentTe,00:D0:78EltexSwe,00:D0:79Cisco,00:D0:7AAmaquest,00:D0:7BComcamIn,00:D0:7CKoyoElec,00:D0:7DCosineCo,00:D0:7EKeycorp,00:D0:7FStrategy,00:D0:80Exabyte,00:D0:81RTDEmbed,00:D0:82Iowave,00:D0:83Invertex,00:D0:84Nexcomm,00:D0:85OtisElev,00:D0:86Foveon,00:D0:87Microfir,00:D0:88ARRISGro,00:D0:89Dynacolo,00:D0:8APhotronU,00:D0:8BADVAOpti,00:D0:8CGenoaTec,00:D0:8DPhoenixG,00:D0:8EGrassVal,00:D0:8FArdentTe,00:D0:90Cisco,00:D0:91Smartsan,00:D0:92Glenayre,00:D0:93Tq-Compo,00:D0:94SeeionCo,00:D0:95Alcatel-,00:D0:963ComEuro,00:D0:97Cisco,00:D0:98PhotonDy,00:D0:99ElcardWi,00:D0:9AFilanet,00:D0:9BSpectel,00:D0:9CKapadiaC,00:D0:9DVerisInd,00:D0:9E2Wire,00:D0:9FNovtekTe,00:D0:A0MipsDenm,00:D0:A1OskarVie,00:D0:A2Integrat,00:D0:A3VocalDat,00:D0:A4AlantroC,00:D0:A5American,00:D0:A6LanbirdT,00:D0:A7TokyoSok,00:D0:A8NetworkE,00:D0:A9ShinanoK,00:D0:AAChaseCom,00:D0:ABDeltakab,00:D0:ACCommscop,00:D0:ADTlIndust,00:D0:AEOresisCo,00:D0:AFCutler-H,00:D0:B0Bitswitc,00:D0:B1OmegaEle,00:D0:B2Xiotech,00:D0:B3DRSTechn,00:D0:B4Katsujim,00:D0:B5IPricotf,00:D0:B6Crescent,00:D0:B7Intel,00:D0:B8Iomega,00:D0:B9Microtek,00:D0:BACisco,00:D0:BBCisco,00:D0:BCCisco,00:D0:BDLatticeS,00:D0:BEEmutec,00:D0:BFPivotalT,00:D0:C0Cisco,00:D0:C1Harmonic,00:D0:C2Balthaza,00:D0:C3VividTec,00:D0:C4Teratech,00:D0:C5Computat,00:D0:C6ThomasBe,00:D0:C7Pathway,00:D0:C8Prevas,00:D0:C9Advantec,00:D0:CAIntrinsy,00:D0:CBDasan,00:D0:CCTechnolo,00:D0:CDAtanTech,00:D0:CEiSystemL,00:D0:CFMoretonB,00:D0:D0Zhongxin,00:D0:D1Sycamore,00:D0:D2Epilog,00:D0:D3Cisco,00:D0:D4V-Bits,00:D0:D5Grundig,00:D0:D6AethraTe,00:D0:D7B2C2,00:D0:D83Com,00:D0:D9Dedicate,00:D0:DATaicomDa,00:D0:DBMcquayIn,00:D0:DCModularM,00:D0:DDSunriseT,00:D0:DEPhilipsM,00:D0:DFKuzumiEl,00:D0:E0DooinEle,00:D0:E1Avionite,00:D0:E2MrtMicro,00:D0:E3Ele-Chem,00:D0:E4Cisco,00:D0:E5Solidum,00:D0:E6Ibond,00:D0:E7VconTele,00:D0:E8MacSyste,00:D0:E9Advantag,00:D0:EANextoneC,00:D0:EBLightera,00:D0:ECNAKAYO,00:D0:EDXiox,00:D0:EEDictapho,00:D0:EFIgt,00:D0:F0Convisio,00:D0:F1SegaEnte,00:D0:F2Monterey,00:D0:F3SolariDi,00:D0:F4Carinthi,00:D0:F5OrangeMi,00:D0:F6Nokia,00:D0:F7NextNets,00:D0:F8FujianSt,00:D0:F9AcuteCom,00:D0:FAThalese-,00:D0:FBTekMicro,00:D0:FCGraniteM,00:D0:FDOptimaTe,00:D0:FEAstralPo,00:D0:FFCisco,00:D1:1CAcetel,00:D2:79Vingroup,00:D2:B1TPVDispl,00:D3:18SPGContr,00:D3:8DHotelTec,00:D6:32GEEnergy,00:D6:FECisco,00:D7:6DIntelCor,00:D7:8FCisco,00:D8:61Micro-St,00:D9:D1SonyInte,00:DA:55Cisco,00:DB:1EAlbedoTe,00:DB:45Thamway,00:DB:70Apple,00:DB:DFIntelCor,00:DC:B2ExtremeN,00:DD:00Ungerman,00:DD:01Ungerman,00:DD:02Ungerman,00:DD:03Ungerman,00:DD:04Ungerman,00:DD:05Ungerman,00:DD:06Ungerman,00:DD:07Ungerman,00:DD:08Ungerman,00:DD:09Ungerman,00:DD:0AUngerman,00:DD:0BUngerman,00:DD:0CUngerman,00:DD:0DUngerman,00:DD:0EUngerman,00:DD:0FUngerman,00:DD:25Shenzhen,00:DE:FBCisco,00:DF:1DCisco,00:E0:00Fujitsu,00:E0:01StrandLi,00:E0:02Crossroa,00:E0:03NokiaWir,00:E0:04Pmc-Sier,00:E0:05Technica,00:E0:06SiliconI,00:E0:07AvayaECS,00:E0:08AmazingC,00:E0:09StratusT,00:E0:0ADiba,00:E0:0BRooftopC,00:E0:0CMotorola,00:E0:0DRadiant,00:E0:0EAvalonIm,00:E0:0FShanghai,00:E0:10HESSSB-A,00:E0:11Uniden,00:E0:12PlutoTec,00:E0:13EasternE,00:E0:14Cisco,00:E0:15Heiwa,00:E0:16Rapid-Ci,00:E0:17EXXACT,00:E0:18AsustekI,00:E0:19IngGiord,00:E0:1AComtec,00:E0:1BSphereCo,00:E0:1CCradlepo,00:E0:1DWebTVNET,00:E0:1ECisco,00:E0:1FAVIDIA,00:E0:20Tecnomen,00:E0:21Freegate,00:E0:22AnalogDe,00:E0:23Telrad,00:E0:24GadzooxN,00:E0:25dit,00:E0:26RedlakeM,00:E0:27Dux,00:E0:28Aptix,00:E0:29SmcEther,00:E0:2ATandberg,00:E0:2BExtremeN,00:E0:2CAst-Buil,00:E0:2DInnoMedi,00:E0:2ESpcElect,00:E0:2FMcnsHold,00:E0:30MelitaIn,00:E0:31Hagiwara,00:E0:32MisysFin,00:E0:33EEPD,00:E0:34Cisco,00:E0:35ArtesynE,00:E0:36Pioneer,00:E0:37Century,00:E0:38Proxima,00:E0:39Paradyne,00:E0:3ACabletro,00:E0:3BProminet,00:E0:3CAdvanSys,00:E0:3DFoconEle,00:E0:3EAlfatech,00:E0:3FJaton,00:E0:40DeskStat,00:E0:41Cspi,00:E0:42Pacom,00:E0:43VitalCom,00:E0:44Lsics,00:E0:45Touchwav,00:E0:46BentlyNe,00:E0:47InFocus,00:E0:48SdlCommu,00:E0:49MICROWIE,00:E0:4AZXTechno,00:E0:4BJUMPINDU,00:E0:4CRealtekS,00:E0:4DInternet,00:E0:4ESanyoDen,00:E0:4FCisco,00:E0:50Executon,00:E0:51Talx,00:E0:52BrocadeC,00:E0:53Cellport,00:E0:54KodaiHit,00:E0:55Ingenier,00:E0:56Holontec,00:E0:57HanMicro,00:E0:58PhaseOne,00:E0:59Controll,00:E0:5AGaleaNet,00:E0:5BWestEnd,00:E0:5CPHC,00:E0:5DUnitec,00:E0:5EJapanAvi,00:E0:5Fe-Net,00:E0:60Sherwood,00:E0:61EdgePoin,00:E0:62HostEngi,00:E0:63Cabletro,00:E0:64SamsungE,00:E0:65OpticalA,00:E0:66ProMax,00:E0:67eacAUTOM,00:E0:68Merrimac,00:E0:69Jaycor,00:E0:6AKapsch,00:E0:6BW&GSpeci,00:E0:6CUltraEle,00:E0:6DCompuwar,00:E0:6EFAR,00:E0:6FARRISGro,00:E0:70DhTechno,00:E0:71EpisMicr,00:E0:72Lynk,00:E0:73National,00:E0:74TiernanC,00:E0:75Verilink,00:E0:76Developm,00:E0:77Webgear,00:E0:78Berkeley,00:E0:79ATNR,00:E0:7AMikrodid,00:E0:7BBayNetwo,00:E0:7CMettler-,00:E0:7DEncoreNe,00:E0:7EWaltDisn,00:E0:7FLOGISTIS,00:E0:80ControlR,00:E0:81TyanComp,00:E0:82Anerma,00:E0:83JatoTech,00:E0:84Compulit,00:E0:85GlobalMa,00:E0:86EmersonN,00:E0:87LeCroy-N,00:E0:88LTX-Cred,00:E0:89IONNetwo,00:E0:8AGecAvery,00:E0:8BQLogic,00:E0:8CNeoparad,00:E0:8DPressure,00:E0:8EUtstarco,00:E0:8FCiscoSys,00:E0:90BeckmanL,00:E0:91LGElectr,00:E0:92Admtek,00:E0:93AckfinNe,00:E0:94Osai,00:E0:95Advanced,00:E0:96Shimadzu,00:E0:97CarrierA,00:E0:98AboCom,00:E0:99Samson,00:E0:9APositron,00:E0:9BEngageNe,00:E0:9CMii,00:E0:9DSarnoff,00:E0:9EQuantum,00:E0:9FPixelVis,00:E0:A0Wiltron,00:E0:A1HIMAPAUL,00:E0:A2Microsla,00:E0:A3CiscoSys,00:E0:A4ESAOTE,00:E0:A5ComCoreS,00:E0:A6TelogyNe,00:E0:A7IpcInfor,00:E0:A8SAT,00:E0:A9FunaiEle,00:E0:AAElectros,00:E0:ABDimat,00:E0:ACMidsco,00:E0:ADEesTechn,00:E0:AEXaqti,00:E0:AFGeneralD,00:E0:B0CiscoSys,00:E0:B1Alcatel-,00:E0:B2TelmaxCo,00:E0:B3EtherWAN,00:E0:B4TechnoSc,00:E0:B5ArdentCo,00:E0:B6EntradaN,00:E0:B7Cosworth,00:E0:B8AmdPcnet,00:E0:B9Byas,00:E0:BABERGHOFA,00:E0:BBNbx,00:E0:BCSymonCom,00:E0:BDInterfac,00:E0:BEGenrocoI,00:E0:BFTorrentN,00:E0:C0SeiwaEle,00:E0:C1MemorexT,00:E0:C2NECSY,00:E0:C3SakaiSys,00:E0:C4HornerEl,00:E0:C5BcomElec,00:E0:C6Link2ItL,00:E0:C7Eurotech,00:E0:C8VirtualA,00:E0:C9Automate,00:E0:CABestData,00:E0:CBReson,00:E0:CCHero,00:E0:CDSaabSens,00:E0:CEArn,00:E0:CFIntegrat,00:E0:D0Netspeed,00:E0:D1Telsis,00:E0:D2Versanet,00:E0:D3DATENTEC,00:E0:D4Excellen,00:E0:D5Emulex,00:E0:D6Computer,00:E0:D7Sunshine,00:E0:D8LANBitCo,00:E0:D9Tazmo,00:E0:DAAlcatel-,00:E0:DBViaVideo,00:E0:DCNexware,00:E0:DDZenithEl,00:E0:DEDatax,00:E0:DFDZS,00:E0:E0SiElectr,00:E0:E1G2Networ,00:E0:E2Innova,00:E0:E3Sk-Elekt,00:E0:E4FANUCROB,00:E0:E5CincoNet,00:E0:E6INCAACom,00:E0:E7Raytheon,00:E0:E8GRETACOD,00:E0:E9DataLabs,00:E0:EAInnovatC,00:E0:EBDigicom,00:E0:ECCelestic,00:E0:EDSilicom,00:E0:EEMarelHf,00:E0:EFDionex,00:E0:F0AblerTec,00:E0:F1That,00:E0:F2ArlottoC,00:E0:F3WebSprin,00:E0:F4INSIDETe,00:E0:F5Teles,00:E0:F6Decision,00:E0:F7Cisco,00:E0:F8DicnaCon,00:E0:F9Cisco,00:E0:FATrlTechn,00:E0:FBLeightro,00:E0:FCHuaweiTe,00:E0:FDA-TrendT,00:E0:FECisco,00:E0:FFSECURITY,00:E1:6DCisco,00:E1:75AK,00:E1:8CIntelCor,00:E2:2CChinaMob,00:E3:B2SamsungE,00:E4:00SichuanC,00:E4:06HuaweiTe,00:E4:21SonyInte,00:E5:E4SichuanT,00:E6:66ARIMACom,00:E6:D3NixdorfC,00:E6:E8NetzinTe,00:E7:E3zte,00:E8:ABMeggittT,00:E9:3AAzureWav,00:EA:BDCisco,00:EB:2DSony,00:EB:D5Cisco,00:EC:0AXiaomiCo,00:ED:B8KYOCERA,00:EE:ABCisco,00:EE:BDHTC,00:F0:51KWB,00:F2:2CShanghai,00:F2:8BCisco,00:F3:61AmazonTe,00:F3:9FApple,00:F3:DBWOOSport,00:F4:03Orbis,00:F4:6FSamsungE,00:F4:8DLiteonTe,00:F4:B9Apple,00:F6:20Google,00:F6:63Cisco,00:F7:6FApple,00:F8:1CHuaweiTe,00:F8:2CCisco,00:F8:60PTPanggu,00:F8:71DGSDenma,00:FA:21SamsungE,00:FA:3BCloosEle,00:FA:B6KontaktM,00:FC:58WebSilic,00:FC:70Intrepid,00:FC:8BAmazonTe,00:FC:8DHitronTe,00:FC:BACisco,00:FD:22Cisco,00:FD:45HewlettP,00:FD:4CNevatec,00:FE:C8Cisco,01:0E:CFPN-MC,02:04:06BbnInter,02:07:01Racal-Da,02:1C:7CPerq,02:20:48Marconi,02:60:603com,02:60:86LogicRep,02:60:8C3comIbmP,02:70:01Racal-Da,02:70:B0MA-ComCo,02:70:B3DataReca,02:9D:8ECardiacR,02:A0:C9Intel,02:AA:3COlivetti,02:BB:01Octothor,02:C0:8C3Com,02:CF:1CCommunic,02:CF:1FCMC,02:E0:3BProminet,02:E6:D3NixdorfC,04:02:1FHuaweiTe,04:02:CAShenzhen,04:03:D6Nintendo,04:04:EAValensSe,04:05:DDShenzhen,04:07:2EVTechEle,04:09:73HewlettP,04:09:A5HFR,04:0A:83Alcatel-,04:0A:E0XmitComp,04:0C:CEApple,04:0E:3CHP,04:0E:C2ViewSoni,04:11:19IEEERegi,04:11:19:00:00:00/28FORTRobo,04:11:19:10:00:00/28Acentury,04:11:19:20:00:00/28AletheaC,04:11:19:30:00:00/28SuzhouRi,04:11:19:40:00:00/28BolicomI,04:11:19:50:00:00/28CeitaCom,04:11:19:60:00:00/28ZPDtechn,04:11:19:70:00:00/28HerrickT,04:11:19:80:00:00/28Shenzhen,04:11:19:90:00:00/28ACPowerD,04:11:19:A0:00:00/28CyOneSec,04:11:19:B0:00:00/28HubeiBao,04:11:19:C0:00:00/28HaerbinD,04:11:19:D0:00:00/28NuanceHe,04:11:19:E0:00:00/28Julida,04:15:52Apple,04:15:D9Viwone,04:18:0FSamsungE,04:18:B6Private,04:18:D6Ubiquiti,04:19:7FGraspher,04:1A:04WaveIP,04:1B:6DLGElectr,04:1B:94HostMobi,04:1B:BASamsungE,04:1D:10DreamWar,04:1D:C7zte,04:1E:64Apple,04:1E:7ADSPWorks,04:1E:FABISSELLH,04:20:84zte,04:20:9APanasoni,04:21:44SunitecE,04:21:4CInsightE,04:22:34Wireless,04:25:C5HuaweiTe,04:25:E0TaicangT,04:26:05GFRGesel,04:26:65Apple,04:27:28Microsof,04:27:58HuaweiTe,04:2A:E2Cisco,04:2B:58Shenzhen,04:2B:BBPicoCELA,04:2D:B4FirstPro,04:2F:56ATOCSShe,04:31:10InspurGr,04:32:F4Partron,04:33:85Nanchang,04:33:89HuaweiTe,04:33:C2IntelCor,04:34:F6Motorola,04:36:04Gyeyoung,04:38:55ScopusIn,04:39:26ChinaDra,04:3A:0DSMOptics,04:3D:98ChongQin,04:3F:72Mellanox,04:40:A9NewH3CTe,04:41:69GoPro,04:42:1AASUSTekC,04:44:A1TeleconG,04:45:62ANDRASpz,04:45:A1NIRIT-Xi,04:46:65MurataMa,04:46:CFBeijingV,04:48:9AApple,04:49:5DHuaweiDe,04:4A:50RamaxelT,04:4A:6CHuaweiTe,04:4A:C6AiponEle,04:4B:EDApple,04:4B:FFGuangZho,04:4C:EFFujianSa,04:4E:06Ericsson,04:4E:5AARRISGro,04:4E:AFLGInnote,04:4F:17HUMAX,04:4F:4CHuaweiTe,04:4F:8BAdapteva,04:4F:AARuckusWi,04:50:DAQikuInte,04:51:70Zhongsha,04:52:C7Bose,04:52:F3Apple,04:53:D5SysorexG,04:54:53Apple,04:55:CABriViewX,04:56:04GioneeCo,04:56:E5IntelCor,04:57:2FSertelEl,04:58:6FSichuanW,04:5A:95Nokia,04:5C:06ZmodoTec,04:5C:6CJuniperN,04:5C:8EgosundGR,04:5D:4BSony,04:5D:56camtroni,04:5E:A4Shenzhen,04:5F:A7Shenzhen,04:5F:B9Cisco,04:61:69MediaGlo,04:62:73Cisco,04:62:D7AlstomHy,04:63:E0Nome,04:65:65Testop,04:67:85scemtecH,04:68:65Apple,04:69:F8Apple,04:6B:1BSYSDINE,04:6B:25SichuanT,04:6C:59IntelCor,04:6C:9DCisco,04:6D:42Bryston,04:6E:02OpenRTLS,04:6E:49TaiYearE,04:70:BCGlobalst,04:71:4BIEEERegi,04:71:4B:00:00:00/28NeurioTe,04:71:4B:10:00:00/28uAvionix,04:71:4B:20:00:00/28Shenzhen,04:71:4B:30:00:00/28Griesser,04:71:4B:40:00:00/28Apparate,04:71:4B:50:00:00/28BureauEl,04:71:4B:60:00:00/28Armstron,04:71:4B:70:00:00/28OmylisPt,04:71:4B:80:00:00/28Energpor,04:71:4B:90:00:00/28Lighthou,04:71:4B:A0:00:00/28Observab,04:71:4B:B0:00:00/28Digibest,04:71:4B:C0:00:00/28KittyHaw,04:71:4B:D0:00:00/28Shenzhen,04:71:4B:E0:00:00/28GimsoMob,04:71:53SernetSu,04:72:95Apple,04:74:A1AligeraE,04:75:03HuaweiTe,04:75:F5Csst,04:76:6EAlpsalpi,04:76:B0Cisco,04:78:63Shanghai,04:79:70HuaweiTe,04:79:75HonorDev,04:79:B7TexasIns,04:7A:0BBeijingX,04:7A:AEHuaweiDe,04:7B:CBUniversa,04:7D:50Shenzhen,04:7D:7BQuantaCo,04:7E:23ChinaMob,04:7E:4Amoobox,04:7F:0EBarrotTe,04:81:9BBSkyB,04:81:AEClack,04:84:8A7InovaTe,04:88:45BayNetwo,04:88:5FHuaweiTe,04:88:8CEifelwer,04:88:E2BeatsEle,04:8A:15Avaya,04:8A:E1Flextron,04:8B:42Skspruce,04:8C:03ThinPADT,04:8C:16HuaweiTe,04:8C:9AHuaweiDe,04:8D:38NetcoreT,04:90:81Pensando,04:91:62Microchi,04:92:26ASUSTekC,04:92:EEiway,04:94:6BTecnoMob,04:94:A1CatchWin,04:95:73zte,04:95:E6TendaTec,04:96:45WuxiSkyC,04:97:90Lartecht,04:98:F3Alpsalpi,04:99:B9Apple,04:99:E6Shenzhen,04:9B:9CEadingco,04:9C:62BMTMedic,04:9D:FEHivesyst,04:9F:06Smobile,04:9F:15Private,04:9F:81Netscout,04:9F:CAHuaweiTe,04:A1:51Netgear,04:A2:22Arcadyan,04:A2:F3Fiberhom,04:A3:16TexasIns,04:A3:F3Emicon,04:A8:2ANokia,04:AA:E1BeijingM,04:AB:18Elecom,04:AB:6AChun-il,04:AC:44HoltekSe,04:B0:E7HuaweiTe,04:B1:67XiaomiCo,04:B1:A1SamsungE,04:B3:B6SeamapUK,04:B4:29SamsungE,04:B4:66BSP,04:B6:48Zenner,04:B8:6ABSkyB,04:B9:E3SamsungE,04:BA:1CHuaweiDe,04:BA:36LiSengTe,04:BA:8DSamsungE,04:BB:F9Pavilion,04:BC:87Shenzhen,04:BD:70HuaweiTe,04:BD:88ArubaaHe,04:BD:BFSamsungE,04:BF:6DZyxelCom,04:BF:A8ISB,04:C0:5BTigoEner,04:C0:6FHuaweiTe,04:C0:9CTellabs,04:C1:03CloverNe,04:C1:B9Fiberhom,04:C1:D8HuaweiDe,04:C2:3EHTC,04:C2:41Nokia,04:C2:9BAuraHome,04:C3:E6IEEERegi,04:C3:E6:00:00:00/28Dreamkas,04:C3:E6:10:00:00/28Guangdon,04:C3:E6:20:00:00/28SiSTechn,04:C3:E6:30:00:00/28ExtechEl,04:C3:E6:40:00:00/28Innovusi,04:C3:E6:50:00:00/28Invasys,04:C3:E6:60:00:00/28Shenzhen,04:C3:E6:70:00:00/28Advanced,04:C3:E6:80:00:00/28SLOC,04:C3:E6:90:00:00/28EkinTekn,04:C3:E6:A0:00:00/28SealedUn,04:C3:E6:B0:00:00/28FlintecU,04:C3:E6:C0:00:00/28ShantouY,04:C3:E6:D0:00:00/28Amiosec,04:C3:E6:E0:00:00/28Teleepoc,04:C4:61MurataMa,04:C5:A4Cisco,04:C8:07XiaomiCo,04:C8:80Samtec,04:C9:91Phistek,04:C9:D9DishTech,04:CB:1DTraka,04:CB:88Shenzhen,04:CD:15SiliconL,04:CE:14Wilocity,04:CE:7ENXPFranc,04:CF:25Manycolo,04:CF:8CXIAOMIEl,04:D1:3AXiaomiCo,04:D1:6EIEEERegi,04:D1:6E:00:00:00/28INTRIPLE,04:D1:6E:10:00:00/28LaunchTe,04:D1:6E:20:00:00/28sdi,04:D1:6E:30:00:00/28BeijingH,04:D1:6E:40:00:00/28ShenZhen,04:D1:6E:50:00:00/28DspreadT,04:D1:6E:60:00:00/28ETLElekt,04:D1:6E:70:00:00/28Envision,04:D1:6E:80:00:00/28ChengduI,04:D1:6E:90:00:00/28FuzhouZh,04:D1:6E:A0:00:00/28MetraEle,04:D1:6E:B0:00:00/28National,04:D1:6E:C0:00:00/28PacPort,04:D1:6E:D0:00:00/28ElotecFi,04:D1:6E:E0:00:00/28EvoluteP,04:D3:20ItelMobi,04:D3:95Motorola,04:D3:B0IntelCor,04:D3:B5HuaweiDe,04:D3:CFApple,04:D4:37Znv,04:D4:42Guangdon,04:D4:C4ASUSTekC,04:D5:90Fortinet,04:D6:0EFunaiEle,04:D6:AASamsungE,04:D6:F4GDMideaA,04:D7:83Y&HE&C,04:D7:A5NewH3CTe,04:D9:21Occuspac,04:D9:F5ASUSTekC,04:DA:D2Cisco,04:DB:56Apple,04:DB:8ASuntechI,04:DD:4CVelocyte,04:DE:DBRockport,04:DE:F2Shenzhen,04:DF:69CarConne,04:E0:B0Shenzhen,04:E0:C4Triumph-,04:E1:C8IMSSoluç,04:E2:29QingdaoH,04:E2:F8AEPTicke,04:E4:51TexasIns,04:E5:36Apple,04:E5:48CohdaWir,04:E5:6ETHUB,04:E5:98XiaomiCo,04:E6:62Acroname,04:E6:76AMPAKTec,04:E7:7EWe,04:E7:95HuaweiTe,04:E9:E5PjrcCom,04:EA:56IntelCor,04:EB:40Cisco,04:EC:BBFiberhom,04:EC:D8IntelCor,04:ED:33IntelCor,04:EE:03TexasIns,04:EE:91x-fabric,04:EE:EELaplaceS,04:F0:21CompexPt,04:F0:3EHuaweiDe,04:F1:28HMDGloba,04:F1:3EApple,04:F1:69HuaweiDe,04:F1:7DTaranaWi,04:F3:52HuaweiTe,04:F4:BCXenaNetw,04:F5:F4ProximWi,04:F7:E4Apple,04:F8:C2Flaircom,04:F8:F8Edgecore,04:F9:38HuaweiTe,04:F9:93Infinixm,04:F9:D9SpeakerE,04:FA:3FOpticore,04:FA:83QingdaoH,04:FE:31SamsungE,04:FE:7FCisco,04:FE:8DHuaweiTe,04:FE:A1Fihonest,04:FF:51Novamedi,08:00:01Computer,08:00:02BridgeCo,08:00:03Advanced,08:00:04Cromemco,08:00:05Symbolic,08:00:06SiemensN,08:00:07Apple,08:00:08BoltBera,08:00:09HewlettP,08:00:0ANestar,08:00:0BUnisysAl,08:00:0CMiklynDe,08:00:0DIclInter,08:00:0ENcr/At&T,08:00:0FMitel,08:00:10At&T[Mis,08:00:11Tektroni,08:00:12BellAtla,08:00:13Exxon,08:00:14ExcelanB,08:00:15StcBusin,08:00:16Barriste,08:00:17National,08:00:18PirelliF,08:00:19GeneralE,08:00:1ATiara10N,08:00:1BDellEMC,08:00:1CKdd-Koku,08:00:1DAbleComm,08:00:1EApolloCo,08:00:1FSharp,08:00:20Oracle,08:00:213M,08:00:22NbiNothi,08:00:23Panasoni,08:00:2410NetCom,08:00:25ControlD,08:00:26NorskDat,08:00:27PcsCompu,08:00:28TiExplor,08:00:29Megatek,08:00:2AMosaicTe,08:00:2BDigitalE,08:00:2CBrittonL,08:00:2DLan-Tec,08:00:2EMetaphor,08:00:2FPrimeCom,08:00:30NetworkR,08:00:31LittleMa,08:00:32Tigan,08:00:33BauschLo,08:00:34Filenet,08:00:35Microfiv,08:00:36Intergra,08:00:37FUJIFILM,08:00:38BullS,08:00:39Spider,08:00:3AOrcatech,08:00:3BTorus,08:00:3CSchlumbe,08:00:3DCadnetix,08:00:3ECodex,08:00:3FFredKosc,08:00:40Ferranti,08:00:41Racal-Mi,08:00:42MACNICA,08:00:43PixelCom,08:00:44David,08:00:45Concurre,08:00:46Sony,08:00:47SequentC,08:00:48Eurother,08:00:49Univatio,08:00:4ABanyan,08:00:4BPlanning,08:00:4CHydraCom,08:00:4DCorvus,08:00:4E3ComEuro,08:00:4FCygnet,08:00:50Daisy,08:00:51ExperDat,08:00:52Insystec,08:00:53MiddleEa,08:00:55Stanford,08:00:56Stanford,08:00:57EvansSut,08:00:58Concepts,08:00:59Mycron,08:00:5AIBM,08:00:5BVtaTechn,08:00:5CFourPhas,08:00:5DGould,08:00:5ECounterp,08:00:5FSaberTec,08:00:60Industri,08:00:61Jarogate,08:00:62GeneralD,08:00:63Plessey,08:00:64Sitasys,08:00:65Genrad,08:00:66AgfaPrin,08:00:67ComDesig,08:00:68RidgeCom,08:00:69SiliconG,08:00:6AAt&T,08:00:6BAccelTec,08:00:6CSuntekTe,08:00:6DWhitecha,08:00:6EMasscomp,08:00:6FPhilipsA,08:00:70Mitsubis,08:00:71MatraDsi,08:00:72XeroxUni,08:00:73Tecmar,08:00:74CasioCom,08:00:75DdeDanis,08:00:76PcLanTec,08:00:77TslNowRe,08:00:78Accell,08:00:79DroidWor,08:00:7AIndata,08:00:7BSanyoEle,08:00:7CVitalink,08:00:7EAmalgama,08:00:7FCarnegie,08:00:80AesData,08:00:81Crosfiel,08:00:82VeritasS,08:00:83SeikoDen,08:00:84TomenEle,08:00:85Elxsi,08:00:86Imagen/Q,08:00:87XyplexTe,08:00:88BrocadeC,08:00:89Kinetics,08:00:8APerfTech,08:00:8BPyramidT,08:00:8CNetworkR,08:00:8DXyvision,08:00:8ETandem/S,08:00:8FChipcom,08:00:90Sonoma,08:01:0FSichuanT,08:02:05HuaweiTe,08:02:8ENetgear,08:03:42PaloAlto,08:03:71KrgCorpo,08:05:81Roku,08:05:CDDongGuan,08:05:E2JuniperN,08:08:C2SamsungE,08:08:EAAmsc,08:09:B6Masimo,08:09:C7ZhuhaiUn,08:0A:4EPlanetBi,08:0C:0BSysMikDr,08:0C:C9MissionT,08:0D:84GECO,08:0E:A8Velex,08:0F:FAKsp,08:10:86NECPlatf,08:11:5EBitel,08:11:96IntelCor,08:12:A5AmazonTe,08:14:43Unibrain,08:15:2FSamsungE,08:16:05Vodafone,08:16:51Shenzhen,08:16:D5Goertek,08:17:35Cisco,08:17:F4IBM,08:18:1Azte,08:18:4CThomas,08:19:A6HuaweiTe,08:1C:6EXiaomiCo,08:1D:C4ThermoFi,08:1D:FBShanghai,08:1F:3FWondaLin,08:1F:71Tp-LinkT,08:1F:EBBinCube,08:1F:F3Cisco,08:21:EFSamsungE,08:23:B2vivoMobi,08:25:22Advansee,08:25:25XiaomiCo,08:26:97ZyxelCom,08:27:19APSelect,08:27:CENaganoKe,08:2A:D0SRDInnov,08:2C:B0NetworkI,08:2C:B6Apple,08:2C:EDTechnity,08:2E:36HuaweiDe,08:2E:5FHewlettP,08:2F:E9HuaweiTe,08:30:6BPaloAlto,08:31:8BHuaweiTe,08:31:A4HuaweiDe,08:35:1BShenzhen,08:35:71CASwell,08:35:B2CoreEdge,08:36:C9Netgear,08:37:3DSamsungE,08:37:9CTopaz,08:38:69HongKong,08:38:A5Funkwerk,08:38:E6Motorola,08:3A:2FGuangzho,08:3A:38NewH3CTe,08:3A:5CJunilab,08:3A:88Universa,08:3A:B8ShinodaP,08:3A:F2Espressi,08:3D:88SamsungE,08:3E:0CARRISGro,08:3E:5DSagemcom,08:3E:8EHonHaiPr,08:3F:3EWSH,08:3F:76Intellia,08:3F:BCzte,08:40:27Gridstor,08:40:F3TendaTec,08:42:96MobileTe,08:45:D1Cisco,08:46:56Veo-Labs,08:47:4CNokia,08:47:D0NokiaSha,08:48:2CRaycoreT,08:4A:CFGuangdon,08:4E:1CH2A,08:4E:BFSumitomo,08:4F:0AHuaweiTe,08:4F:A9Cisco,08:4F:F9Cisco,08:51:14QingdaoT,08:51:2EOrionDia,08:52:40EbVElekt,08:54:BBShenzhen,08:55:31Routerbo,08:57:00Tp-LinkT,08:58:A5BeijingV,08:5A:11D-LinkIn,08:5A:E0Recovisi,08:5B:0EFortinet,08:5B:D6IntelCor,08:5B:DACliniCar,08:5C:1BHuaweiTe,08:5D:DDMercury,08:60:6EASUSTekC,08:60:83zte,08:61:95Rockwell,08:62:66ASUSTekC,08:63:61HuaweiTe,08:65:F0JMZengge,08:66:1FPaloAlto,08:66:98Apple,08:67:4EHisenseb,08:68:8DNewH3CTe,08:68:D0JapanSys,08:68:EAEitoElec,08:6A:0AAskeyCom,08:6A:C5IntelCor,08:6B:D1Shenzhen,08:6B:D7SiliconL,08:6D:41Apple,08:6D:F2Shenzhen,08:6F:48Shenzhen,08:70:45Apple,08:71:90IntelCor,08:74:02Apple,08:74:F6Winterha,08:75:72Obelux,08:76:18ViETechn,08:76:95AutoIndu,08:76:FFThomsonT,08:78:08SamsungE,08:79:8CHuaweiTe,08:79:99AIM,08:7A:4CHuaweiTe,08:7B:AASvyazkom,08:7C:39AmazonTe,08:7C:BEQuintic,08:7D:21Altasect,08:7E:64Technico,08:7F:98vivoMobi,08:80:39CiscoSPV,08:81:B2Logitech,08:81:BCHongKong,08:81:F4JuniperN,08:84:66Novartis,08:84:9DAmazonTe,08:85:5BKontronE,08:86:20TecnoMob,08:86:3BBelkinIn,08:87:C6IngramMi,08:87:C7Apple,08:8C:2CSamsungE,08:8D:C8RyowaEle,08:8E:4FSFSoftwa,08:8E:90IntelCor,08:8E:DCApple,08:8F:2CAmberTec,08:90:BADanlaw,08:93:56HuaweiTe,08:94:EFWistronI,08:95:2ATechnico,08:96:ADCisco,08:96:D7AVM,08:97:34HewlettP,08:97:58Shenzhen,08:97:98CompalIn,08:99:E8KEMAS,08:9A:C7zte,08:9B:4BiKuaiNet,08:9B:B9NokiaSol,08:9B:F1eero,08:9C:86NokiaSha,08:9E:01QuantaCo,08:9E:08Google,08:9E:84HuaweiTe,08:9F:97LeroyAut,08:A1:2BShenZhen,08:A1:89Hangzhou,08:A5:C8SunnovoI,08:A6:BCAmazonTe,08:A7:C0Technico,08:A8:42HuaweiDe,08:A8:A1Cyclotro,08:A9:5AAzureWav,08:AA:55Motorola,08:AA:89zte,08:AC:A5BenuVide,08:AC:C4FMTech,08:AE:D6SamsungE,08:AF:78TotusSol,08:B0:55AskeyCom,08:B0:A7Truebeyo,08:B2:58JuniperN,08:B2:A3CynnyIta,08:B3:AFvivoMobi,08:B4:9DTecnoMob,08:B4:B1Google,08:B4:CFAbicomIn,08:B7:38Lite-OnT,08:B7:ECWireless,08:BA:22Swaive,08:BA:5FQingdaoH,08:BB:3CFlextron,08:BB:CCAK-NORDE,08:BC:20Hangzhou,08:BD:43Netgear,08:BE:09AstrolEl,08:BE:77GreenEle,08:BE:ACEdimaxTe,08:BF:A0SamsungE,08:C0:21HuaweiTe,08:C0:6CHuaweiDe,08:C0:EBMellanox,08:C5:E1SamsungE,08:C6:B3Qtech,08:C7:29Apple,08:CA:45ToyouFei,08:CB:E5R3-Relia,08:CC:27Motorola,08:CC:68Cisco,08:CC:A7Cisco,08:CD:9Bsamtecau,08:D0:9FCisco,08:D0:B7QingdaoH,08:D2:3EIntelCor,08:D2:9AProforma,08:D3:4BTechmanE,08:D4:0CIntelCor,08:D4:2BSamsungE,08:D4:6ALGElectr,08:D5:9DSagemcom,08:D5:C0SeersTec,08:D8:33Shenzhen,08:DF:1FBose,08:DF:CBSystrome,08:E4:DFShenzhen,08:E5:DANanjingF,08:E6:72JebseeEl,08:E6:89Apple,08:E7:E5HuaweiDe,08:E8:4FHuaweiTe,08:E9:F6AMPAKTec,08:EA:40Shenzhen,08:EA:44ExtremeN,08:EB:29JiangsuH,08:EB:74HUMAX,08:EB:EDWorldEli,08:EC:A9SamsungE,08:EC:F5Cisco,08:ED:02IEEERegi,08:ED:02:00:00:00/28D2SLink,08:ED:02:10:00:00/28Imperx,08:ED:02:20:00:00/28TESTouch,08:ED:02:30:00:00/28JiangsuL,08:ED:02:40:00:00/28Fio,08:ED:02:50:00:00/28Vigitron,08:ED:02:60:00:00/28SangoEle,08:ED:02:70:00:00/28ElevenEn,08:ED:02:80:00:00/28Hantas,08:ED:02:90:00:00/28SavoxCom,08:ED:02:A0:00:00/28Victiana,08:ED:02:B0:00:00/28SzokEner,08:ED:02:C0:00:00/28GuardRFI,08:ED:02:D0:00:00/28OrigamiE,08:ED:02:E0:00:00/28Telstra,08:ED:9DTecnoMob,08:ED:B9HonHaiPr,08:ED:EDZhejiang,08:EE:8BSamsungE,08:EF:3BMCSLogic,08:EF:ABSaymeWir,08:F1:B7Towerstr,08:F1:EAHewlettP,08:F2:F4NetOnePa,08:F4:58HuaweiDe,08:F4:ABApple,08:F6:06zte,08:F6:9CApple,08:F6:F8GETEngin,08:F7:28GLOBOMul,08:F7:E9HRCPRese,08:F8:BCApple,08:FA:28HuaweiTe,08:FA:79vivoMobi,08:FA:E0FohhnAud,08:FB:EAAMPAKTec,08:FC:52OpenXS,08:FC:88SamsungE,08:FD:0ESamsungE,08:FF:24Shenzhen,08:FF:44Apple,09:00:6AAT&T,0A:2A:33Digistor,0A:36:1CSmardii,0A:87:36IEEE1901,0A:CD:8FCiscoNor,0A:E4:71Caterpil,0C:01:DBInfinixm,0C:02:27Technico,0C:02:BDSamsungE,0C:04:00Jantardo,0C:05:35Juniper,0C:08:B4HUMAX,0C:0E:76D-LinkIn,0C:11:05AkuvoxXi,0C:11:67Cisco,0C:12:62zte,0C:13:0BUniqoteq,0C:14:20SamsungE,0C:14:D2ChinaMob,0C:15:39Apple,0C:15:C5SDTEC,0C:17:73HuaweiDe,0C:17:F1Telecsys,0C:19:1FInformEl,0C:19:F8Apple,0C:1A:10Acoustic,0C:1C:19Longconn,0C:1C:20Kakao,0C:1C:57TexasIns,0C:1D:AFXiaomiCo,0C:1D:C2SeAHNetw,0C:20:26noaxTech,0C:20:D3vivoMobi,0C:21:38Hengstle,0C:23:69Honeywel,0C:25:76Longchee,0C:27:24Cisco,0C:27:55Valuable,0C:29:EFDell,0C:2A:69electric,0C:2A:86Fiberhom,0C:2A:E7BeijingG,0C:2C:54HuaweiTe,0C:2D:89QiiQComm,0C:2F:B0SamsungE,0C:30:21Apple,0C:31:DCHuaweiTe,0C:35:4FNokia,0C:35:FEFiberhom,0C:37:47zte,0C:37:96BizlinkT,0C:37:DCHuaweiTe,0C:38:3EFanvilTe,0C:39:56Observat,0C:3A:FANewH3CTe,0C:3B:50Apple,0C:3C:65DomeImag,0C:3C:CDUniversa,0C:3E:9FApple,0C:41:01RuichiAu,0C:41:3EMicrosof,0C:41:E9HuaweiTe,0C:42:A1Mellanox,0C:43:14SiliconL,0C:43:F9AmazonTe,0C:45:BAHuaweiTe,0C:46:9DMSSedco,0C:47:3DHitronTe,0C:47:C9AmazonTe,0C:48:85LGElectr,0C:48:C6Celestic,0C:49:33SichuanJ,0C:4B:54Tp-LinkT,0C:4C:39MitraSta,0C:4D:E9Apple,0C:4F:5AASA-RT,0C:4F:9BHuaweiTe,0C:51:01Apple,0C:51:F7ChauvinA,0C:52:03AgmGroup,0C:53:31ETHZuric,0C:54:15IntelCor,0C:54:A5Pegatron,0C:54:B9Nokia,0C:55:21Axiros,0C:56:5CHyBroadV,0C:57:EBMueller,0C:58:42DMEMicro,0C:59:9CJuniperN,0C:5A:19AxtionSd,0C:5A:9EWi-SUNAl,0C:5C:B5IEEERegi,0C:5C:B5:00:00:00/28Yamasei,0C:5C:B5:10:00:00/28avxavEle,0C:5C:B5:20:00:00/28HongKong,0C:5C:B5:30:00:00/28iH&STech,0C:5C:B5:40:00:00/28Annapurn,0C:5C:B5:50:00:00/28Raymond,0C:5C:B5:60:00:00/28S2C,0C:5C:B5:70:00:00/28Energybo,0C:5C:B5:80:00:00/28Shenzhen,0C:5C:B5:90:00:00/28Colordev,0C:5C:B5:A0:00:00/28Zhengzho,0C:5C:B5:B0:00:00/28Adi,0C:5C:B5:C0:00:00/28HunanNew,0C:5C:B5:D0:00:00/28BSU,0C:5C:B5:E0:00:00/28MuntersE,0C:5C:D8DOLIElek,0C:5F:35NiagaraV,0C:60:46vivoMobi,0C:60:76HonHaiPr,0C:61:11AndaTech,0C:61:27Actionte,0C:61:CFTexasIns,0C:62:A6HuiZhouG,0C:63:FCNanjingS,0C:68:03Cisco,0C:6A:BCFiberhom,0C:6A:E6StanleyS,0C:6E:4FPrimeVOL,0C:6F:9CShawComm,0C:70:4AHuaweiTe,0C:71:5DSamsungE,0C:71:8CTCTmobil,0C:72:2CTp-LinkT,0C:72:D9zte,0C:73:29Sercomm,0C:73:BEDongguan,0C:73:EBIEEERegi,0C:73:EB:00:00:00/28GeminiDa,0C:73:EB:10:00:00/28EversecT,0C:73:EB:20:00:00/28Deltapat,0C:73:EB:30:00:00/28TiinlabA,0C:73:EB:40:00:00/28U-Pass,0C:73:EB:50:00:00/28HustyMSt,0C:73:EB:60:00:00/28GreenFox,0C:73:EB:70:00:00/28DinkleEn,0C:73:EB:80:00:00/28BeijingM,0C:73:EB:90:00:00/28BeijingL,0C:73:EB:A0:00:00/28PiInnovo,0C:73:EB:B0:00:00/28Synacces,0C:73:EB:C0:00:00/28Shenzhen,0C:73:EB:D0:00:00/28D-Link（S,0C:73:EB:E0:00:00/28TaiwanPu,0C:74:C2Apple,0C:75:12Shenzhen,0C:75:23BeijingG,0C:75:6CAnarenMi,0C:75:BDCisco,0C:77:1AApple,0C:7A:15IntelCor,0C:7C:28NokiaSol,0C:7D:7CKexiangI,0C:80:63Tp-LinkT,0C:81:12Private,0C:81:26JuniperN,0C:81:7DEEPElekt,0C:82:30Shenzhen,0C:82:68Tp-LinkT,0C:82:6AWuhanHua,0C:83:9AHuaweiDe,0C:83:CCAlphaNet,0C:84:08HuaweiTe,0C:84:11AOSmithW,0C:84:47Fiberhom,0C:84:84ZenoviaE,0C:84:DCHonHaiPr,0C:85:25Cisco,0C:86:10JuniperN,0C:89:10SamsungE,0C:8A:87AgLogica,0C:8B:7DVizio,0C:8B:D3ItelMobi,0C:8B:FDIntelCor,0C:8C:24Shenzhen,0C:8C:69Shenzhen,0C:8C:8FKamoTech,0C:8C:DCSuunto,0C:8D:98TopEight,0C:8D:CASamsungE,0C:8D:DBCiscoMer,0C:8E:29Arcadyan,0C:8F:FFHuaweiTe,0C:90:43RealmeCh,0C:91:60HuiZhouG,0C:92:4ERiceLake,0C:93:01PTPrasim,0C:93:8FGuangdon,0C:93:FBBNSSolut,0C:95:41ChipseaT,0C:96:BFHuaweiTe,0C:96:CDMercury,0C:96:E6CloudNet,0C:98:38XiaomiCo,0C:9A:3CIntelCor,0C:9A:42Fn-LinkT,0C:9B:13Shanghai,0C:9D:56ConsortC,0C:9D:92ASUSTekC,0C:9E:91Sankosha,0C:A0:6CIndustri,0C:A1:38BlinqWir,0C:A2:F4Chameleo,0C:A4:02Alcatel-,0C:A4:2AOBTeleco,0C:A6:94SunitecE,0C:A8:A7SamsungE,0C:AA:EEAnsjerEl,0C:AC:05UnitendT,0C:AE:7DTexasIns,0C:AF:5AGenusPow,0C:B0:88AITeleco,0C:B2:B7TexasIns,0C:B3:19SamsungE,0C:B3:4FShenzhen,0C:B4:59Marketec,0C:B4:A4XintaiAu,0C:B4:EFDigience,0C:B5:27HuaweiTe,0C:B5:DEAlcatelL,0C:B6:D2D-LinkIn,0C:B7:71ARRISGro,0C:B7:89HonorDev,0C:B9:12JM-DATA,0C:B9:37UbeeInte,0C:BC:9FApple,0C:BD:51TCTmobil,0C:BF:15Genetec,0C:BF:3FShenzhen,0C:BF:74MorseMic,0C:C0:C0MagnetiM,0C:C3:A7Meritec,0C:C4:13Google,0C:C4:7ASuperMic,0C:C4:7EEUCAST,0C:C6:55WuxiYSTe,0C:C6:6ANokia,0C:C6:ACDags,0C:C6:CCHuaweiTe,0C:C6:FDXiaomiCo,0C:C7:31Currant,0C:C8:1FSummerIn,0C:C8:44Cambridg,0C:C9:C6SamwinHo,0C:CA:FBTPVision,0C:CB:0CiSYSRTS,0C:CB:85Motorola,0C:CB:8DASCONuma,0C:CC:26Airenetw,0C:CD:D3Eastrive,0C:CD:FBEDIC,0C:CE:F6GuizhouF,0C:CF:89Shenzhen,0C:CF:D1SPRINGWA,0C:D0:F8Cisco,0C:D2:92IntelCor,0C:D2:B5Binatone,0C:D5:02WestellT,0C:D6:96Amimon,0C:D6:BDHuaweiTe,0C:D7:46Apple,0C:D7:C2AxiumTec,0C:D8:6CShenzhen,0C:D9:96Cisco,0C:D9:C1Visteon,0C:DA:41Hangzhou,0C:DC:7EEspressi,0C:DC:CCInalaTec,0C:DD:24IntelCor,0C:DD:EFNokia,0C:DF:A4SamsungE,0C:E0:41iDruide,0C:E0:DCSamsungE,0C:E0:E4Plantron,0C:E1:59Shenzhen,0C:E4:41Apple,0C:E4:A0HuaweiDe,0C:E5:A3SharkNin,0C:E5:D3DHelectr,0C:E7:09FoxCrypt,0C:E7:25Microsof,0C:E8:2FBonfigli,0C:E9:36ELIMOS,0C:E9:9AAtlsAlte,0C:EA:C9ARRISGro,0C:EC:80TexasIns,0C:EC:84Shenzhen,0C:EC:8DMotorola,0C:EE:99AmazonTe,0C:EE:E6HonHaiPr,0C:EF:7CAnaCom,0C:EF:AFIEEERegi,0C:EF:AF:00:00:00/28Kenmore,0C:EF:AF:10:00:00/28Goerlitz,0C:EF:AF:20:00:00/28Lumel,0C:EF:AF:30:00:00/28Engineer,0C:EF:AF:40:00:00/28Sentry36,0C:EF:AF:50:00:00/28Premium,0C:EF:AF:60:00:00/28Firmware,0C:EF:AF:70:00:00/28Syntrans,0C:EF:AF:80:00:00/28BSXAthle,0C:EF:AF:90:00:00/28Rotel,0C:EF:AF:A0:00:00/28chengduj,0C:EF:AF:B0:00:00/28HubeiCen,0C:EF:AF:C0:00:00/28GainStro,0C:EF:AF:D0:00:00/28CJSC«Svy,0C:EF:AF:E0:00:00/28Infiniso,0C:EF:AF:F0:00:00/28Private,0C:F0:19MalgnTec,0C:F0:B4Globalsa,0C:F3:46XiaomiCo,0C:F3:61JavaInfo,0C:F3:EEEMMicroe,0C:F4:05BeijingS,0C:F4:75ZliideTe,0C:F4:D5RuckusWi,0C:F5:A4Cisco,0C:F8:93ARRISGro,0C:F9:C0BSkyB,0C:FC:83AirohaTe,0C:FD:37SUSELinu,0C:FE:45SonyInte,0C:FE:5DIEEERegi,0C:FE:5D:00:00:00/28ChengduL,0C:FE:5D:10:00:00/28FenderMu,0C:FE:5D:20:00:00/28DspreadI,0C:FE:5D:30:00:00/28BeijingW,0C:FE:5D:40:00:00/28YantaiDo,0C:FE:5D:50:00:00/28SELECTRI,0C:FE:5D:60:00:00/28Antailiy,0C:FE:5D:70:00:00/28VermesMi,0C:FE:5D:80:00:00/28CTKConta,0C:FE:5D:90:00:00/28Celerway,0C:FE:5D:A0:00:00/28FujianJi,0C:FE:5D:B0:00:00/28Yinuo-Li,0C:FE:5D:C0:00:00/28BepalTec,0C:FE:5D:D0:00:00/28MaksatTe,0C:FE:5D:E0:00:00/28Newgreen,10:00:00Private,10:00:20Apple,10:00:5AIBM,10:00:90HP,10:00:D4DEC,10:00:E0AppleA/U,10:00:E8National,10:00:FDLaonPeop,10:01:77HuaweiTe,10:01:CAAshleyBu,10:02:B5IntelCor,10:05:01Pegatron,10:05:B1ARRISGro,10:05:CACisco,10:05:E1Nokia,10:06:45Sagemcom,10:06:EDCisco,10:07:23IEEERegi,10:07:23:00:00:00/28RippleTe,10:07:23:10:00:00/28BeijingA,10:07:23:20:00:00/28DiginetC,10:07:23:30:00:00/28Tongfang,10:07:23:40:00:00/28AudioEng,10:07:23:50:00:00/28BeijingS,10:07:23:60:00:00/28EstoneTe,10:07:23:70:00:00/28nanoTech,10:07:23:80:00:00/28IonProfe,10:07:23:90:00:00/28Wireless,10:07:23:A0:00:00/28TesseraT,10:07:23:B0:00:00/28FujianQu,10:07:23:C0:00:00/28Shenzhen,10:07:23:E0:00:00/28FirstCha,10:07:23:F0:00:00/28Private,10:07:B6SamsungE,10:08:2CTexasIns,10:08:B1HonHaiPr,10:09:0CJanomeSe,10:09:F9AmazonTe,10:0B:A9IntelCor,10:0C:24pomdevic,10:0C:29Shenzhen,10:0C:6BNetgear,10:0D:2FOnlineSe,10:0D:32Embedian,10:0D:7FNetgear,10:0E:2BNECCASIO,10:0E:7EJuniperN,10:0F:18FuGangEl,10:10:81zte,10:10:B6McCain,10:12:12VivoInte,10:12:18Korins,10:12:48ITG,10:12:50Integrat,10:12:B4SichuanT,10:12:FBHangzhou,10:13:31Technico,10:13:EEJustecIn,10:18:9EElmoMoti,10:19:65NewH3CTe,10:1B:54HuaweiTe,10:1C:0CApple,10:1D:518MeshNet,10:1D:C0SamsungE,10:1E:DAIngenico,10:1F:74HewlettP,10:22:79ZeroDesk,10:27:79Sadel,10:27:BETvip,10:27:F5TP-Link,10:28:31Morion,10:29:59Apple,10:29:ABSamsungE,10:2A:B3XiaomiCo,10:2B:41SamsungE,10:2C:6BAMPAKTec,10:2C:83Ximea,10:2C:EFEMUElect,10:2D:31Shenzhen,10:2D:41SichuanA,10:2D:96Looxcie,10:2E:AFTexasIns,10:2F:6BMicrosof,10:2F:A3Shenzhen,10:30:25Apple,10:30:34Cara,10:30:47SamsungE,10:32:1DHuaweiTe,10:32:7EHuaweiDe,10:33:78FLECTRON,10:33:BFTechnico,10:34:1BSpacelin,10:36:4ABostonDy,10:37:11SimlinkA,10:38:1FSichuanA,10:39:17SamsungE,10:39:4EHisenseb,10:39:E9JuniperN,10:3B:59SamsungE,10:3C:59zte,10:3D:0AHuiZhouG,10:3D:1CIntelCor,10:3D:3EChinaMob,10:3D:EAHFCTechn,10:3F:44XiaomiCo,10:40:F3Apple,10:41:21Tellesco,10:41:7FApple,10:43:69Soundmax,10:44:00HuaweiTe,10:44:5AShaanxiH,10:45:BENorphoni,10:45:F8LNT-Auto,10:46:B4Formeric,10:47:38NokiaSha,10:47:80HuaweiTe,10:48:B1BeijingD,10:49:63Harting,10:4A:7DIntelCor,10:4B:46Mitsubis,10:4D:77Innovati,10:4E:07Shanghai,10:4E:89GarminIn,10:4F:58ArubaaHe,10:4F:A8Sony,10:50:72Sercomm,10:51:07IntelCor,10:51:72HuaweiTe,10:52:1CEspressi,10:54:03INTARSO,10:55:E4Shenzhen,10:56:11ARRISGro,10:56:CAPeplinkI,10:58:87Fiberhom,10:59:17Tonal,10:59:32Roku,10:5A:17TuyaSmar,10:5A:F7ADBItali,10:5B:ADMegaWell,10:5C:3BPerma-Pi,10:5C:BFDuroByte,10:5D:DCHuaweiDe,10:5F:06Actionte,10:5F:49CiscoSPV,10:5F:D4Tendyron,10:60:4BHewlettP,10:62:C9Adatis,10:62:D0Technico,10:62:E5HewlettP,10:62:EBD-LinkIn,10:63:C8LiteonTe,10:64:E2ADFwebco,10:65:30Dell,10:65:A3Panamax,10:65:CFIqsim,10:66:82NECPlatf,10:68:3FLGElectr,10:6F:3FBuffalo,10:6F:D9CloudNet,10:6F:EFAd-SolNi,10:70:FDMellanox,10:71:00HuaweiDe,10:71:F9CloudTel,10:72:23Tellesco,10:74:6FMotorola,10:76:8AEoCell,10:77:17Shenzhen,10:77:B0Fiberhom,10:77:B1SamsungE,10:78:5BActionte,10:78:73Shenzhen,10:78:CEHanvitSI,10:78:D2Elitegro,10:7A:86U&UEngin,10:7B:44ASUSTekC,10:7B:A4OliveDov,10:7B:CENokia,10:7B:EFZyxelCom,10:7D:1ADell,10:81:B4HunanGre,10:82:86Luxshare,10:82:D7RealmeCh,10:83:D2Microsev,10:86:8CARRISGro,10:88:0FDarumaTe,10:88:CEFiberhom,10:89:FBSamsungE,10:8A:1BRAONIX,10:8B:6AAntailiy,10:8C:CFCisco,10:8E:BAMolekule,10:8E:E0SamsungE,10:8F:FEHuaweiTe,10:92:66SamsungE,10:93:97ARRISGro,10:93:E9Apple,10:94:97Logitech,10:94:BBApple,10:95:4BMegabyte,10:96:93AmazonTe,10:97:BDEspressi,10:98:36Dell,10:98:C3MurataMa,10:9A:B9Tosibox,10:9A:DDApple,10:9C:70PrusaRes,10:9D:7AHuaweiDe,10:9E:3AZhejiang,10:9F:A9Actionte,10:A1:3BFujikura,10:A2:4EGold3Lin,10:A3:B8Iskratel,10:A4:B9BaiduOnl,10:A4:BEShenzhen,10:A4:DAHuaweiTe,10:A5:1DIntelCor,10:A5:D0MurataMa,10:A6:59MobileCr,10:A7:43SKMtek,10:A9:32BeijingC,10:AE:60Private,10:AF:78Shenzhen,10:B1:F8HuaweiTe,10:B2:6Bbase,10:B3:6FBoweiTec,10:B3:C6Cisco,10:B3:D5Cisco,10:B3:D6Cisco,10:B7:13Private,10:B7:A8CableFre,10:B7:F6Plastofo,10:B9:F7Niko-Ser,10:B9:FELika,10:BA:A5GanaI&C,10:BC:97vivoMobi,10:BD:18Cisco,10:BD:55Q-Lab,10:BE:F5D-LinkIn,10:BF:48ASUSTekC,10:C0:7CBlu-rayD,10:C1:72HuaweiTe,10:C2:2FChinaEnt,10:C2:5ATechnico,10:C2:BAUTT,10:C3:7BASUSTekC,10:C3:ABHuaweiTe,10:C5:86BioSound,10:C5:95Lenovo,10:C6:0CDominoUK,10:C6:1FHuaweiTe,10:C6:5EAdapt-IP,10:C6:7EShenzhen,10:C6:FCGarminIn,10:C7:3FMidasKla,10:C7:53QingdaoI,10:C9:CAAceTechn,10:CA:81Precia,10:CC:1BLiverock,10:CC:DBAximumPr,10:CD:6EFisys,10:CD:AEAvaya,10:CD:B6Essentia,10:CE:45Miromico,10:CE:A9TexasIns,10:CE:E9Apple,10:D0:7AAMPAKTec,10:D0:ABzte,10:D1:DCINSTARDe,10:D3:8ASamsungE,10:D5:42SamsungE,10:D5:61TuyaSmar,10:D7:B0Sagemcom,10:DA:43Netgear,10:DC:4AFiberhom,10:DC:B6IEEERegi,10:DC:B6:00:00:00/28Private,10:DC:B6:10:00:00/28ABBSwitz,10:DC:B6:20:00:00/28Cal-Comp,10:DC:B6:30:00:00/28Hanacns,10:DC:B6:40:00:00/28Annapurn,10:DC:B6:50:00:00/28Milesigh,10:DC:B6:60:00:00/28ProlanZr,10:DC:B6:70:00:00/28MoyaComm,10:DC:B6:80:00:00/28SanofiBe,10:DC:B6:90:00:00/28FuzhouRo,10:DC:B6:A0:00:00/28Pickerin,10:DC:B6:B0:00:00/28EyeballF,10:DC:B6:C0:00:00/28BBPOSInt,10:DC:B6:D0:00:00/28LeoLabs,10:DC:B6:E0:00:00/28Shenzhen,10:DD:B1Apple,10:DD:F4MaxwayEl,10:DE:E4automati,10:DF:8BShenzhen,10:DF:FCSiemens,10:E2:D5QiHardwa,10:E3:C7SeohwaTe,10:E4:AFApr,10:E4:C2SamsungE,10:E6:8FKwangsun,10:E6:AESourceTe,10:E7:7ASTMicrol,10:E7:C6HewlettP,10:E8:78Nokia,10:E8:EEPhaseSpa,10:E9:53HuaweiDe,10:EA:59CiscoSPV,10:EC:81SamsungE,10:EE:D9CanogaPe,10:F0:05IntelCor,10:F1:63Tnk,10:F1:F2LGElectr,10:F3:11Cisco,10:F3:DBGridco,10:F4:9AT3Innova,10:F6:05RealmeCh,10:F6:81vivoMobi,10:F9:20Cisco,10:F9:6FLGElectr,10:F9:EBIndustri,10:F9:EENokia,10:FA:CEReacheng,10:FB:F0KangShen,10:FC:54ShanyEle,10:FC:B6mirusyst,10:FE:EDTp-LinkT,11:00:AAPrivate,11:11:11Private,14:00:20LongSung,14:00:7Dzte,14:00:E9MitelNet,14:01:52SamsungE,14:01:9CUbyon,14:02:ECHewlettP,14:04:67SNKTechn,14:07:08Private,14:07:E0Abrantix,14:09:B4zte,14:09:DCHuaweiTe,14:0A:29Tiinlab,14:0A:C5AmazonTe,14:0C:5BPLNetwor,14:0C:76FreeboxS,14:0D:4FFlextron,14:0F:42Nokia,14:10:9FApple,14:11:14TecnoMob,14:11:5DSkyworth,14:13:30Anakreon,14:13:33AzureWav,14:13:46Skyworth,14:13:57ATPElect,14:13:FBHuaweiTe,14:14:4BRuijieNe,14:14:59Vodafone,14:14:E6NingboSa,14:15:7CTokyoCos,14:16:9DCisco,14:16:9EWingtech,14:17:2AFiberhom,14:18:77Dell,14:18:C3IntelCor,14:19:73BeijingY,14:1A:51Treetech,14:1A:A3Motorola,14:1B:30Shenzhen,14:1B:BDVolex,14:1B:F0Intellim,14:1F:78SamsungE,14:1F:BAIEEERegi,14:1F:BA:00:00:00/28Shenzhen,14:1F:BA:10:00:00/28GloQuad,14:1F:BA:20:00:00/28Deutsche,14:1F:BA:30:00:00/28Private,14:1F:BA:40:00:00/28Byzero,14:1F:BA:50:00:00/28Inttelix,14:1F:BA:60:00:00/28ThalesCo,14:1F:BA:70:00:00/28Wisnetwo,14:1F:BA:80:00:00/28Shenzhen,14:1F:BA:90:00:00/28BlackMot,14:1F:BA:A0:00:00/28Winsonic,14:1F:BA:B0:00:00/28NewingsC,14:1F:BA:C0:00:00/28SwissEle,14:1F:BA:D0:00:00/28AJISDALI,14:1F:BA:E0:00:00/28POSSyste,14:1F:BA:F0:00:00/28Private,14:20:5EApple,14:22:33Fiberhom,14:22:3BGoogle,14:22:DBeero,14:23:0AHuaweiTe,14:23:D7Eutronix,14:24:754DReplay,14:28:82MidicomE,14:29:71NemoaEle,14:2A:14ShenZhen,14:2B:D2Armtel,14:2B:D6Guangdon,14:2C:78GooWiWir,14:2D:27HonHaiPr,14:2D:8BIncipioT,14:2D:F5Amphitec,14:2E:5ESercomm,14:2F:FDLtSecuri,14:30:04HuaweiTe,14:30:7AAvermetr,14:30:C6Motorola,14:32:D1SamsungE,14:33:65TEMMobil,14:35:8BMediabri,14:35:B3FutureDe,14:36:05Nokia,14:36:C6LenovoMo,14:37:19PTPrakar,14:37:3BPROCOM,14:3A:EADynapowe,14:3B:42RealfitS,14:3C:C3HuaweiTe,14:3D:F2BeijingS,14:3E:60Nokia,14:3E:BFzte,14:3F:27Noccela,14:3F:A6SonyHome,14:3F:C3SnapAV,14:41:46Honeywel,14:41:E2MonacoEn,14:42:FCTexasIns,14:43:19Creative,14:44:4AApolloSe,14:46:58HuaweiTe,14:46:E4Avistel,14:47:2DGuangdon,14:48:02YEOLRIM,14:48:8BShenzhen,14:49:20HuaweiTe,14:49:78DigitalC,14:49:BCDrayTek,14:49:E0SamsungE,14:4C:1AMaxCommu,14:4D:67ZioncomE,14:4E:2ACiena,14:4E:34RemoteSo,14:4F:8AIntelCor,14:4F:D7IEEERegi,14:4F:D7:00:00:00/28Annapurn,14:4F:D7:10:00:00/28ZehnderG,14:4F:D7:20:00:00/28FedExSer,14:4F:D7:30:00:00/28QingdaoW,14:4F:D7:40:00:00/28RedTechn,14:4F:D7:50:00:00/28FlsFinla,14:4F:D7:60:00:00/28i-SENS,14:4F:D7:70:00:00/28Shenzhen,14:4F:D7:80:00:00/28NPortNet,14:4F:D7:90:00:00/28EmersonN,14:4F:D7:A0:00:00/28Unirobot,14:4F:D7:B0:00:00/28Arkus-ST,14:4F:D7:C0:00:00/28D&SCable,14:4F:D7:D0:00:00/28Shanghai,14:4F:D7:E0:00:00/28EdanInst,14:50:51SHARP,14:51:20HuaweiDe,14:51:7ENewH3CTe,14:52:90KNSGroup,14:54:12Entis,14:55:94HuaweiDe,14:56:3AHuaweiDe,14:56:45Savitech,14:56:8ESamsungE,14:57:90QingdaoH,14:57:9FHuaweiTe,14:58:08TaicangT,14:58:D0HewlettP,14:59:C0Netgear,14:59:C3Creative,14:5A:05Apple,14:5A:83Logi-D,14:5A:FCLiteonTe,14:5B:D1ARRISGro,14:5B:E1nyantec,14:5E:45BambooGr,14:5E:69Guangdon,14:5F:94HuaweiTe,14:60:80zte,14:60:CBApple,14:61:02AlpsAlpi,14:61:2FAvaya,14:63:08JabilCir,14:69:A2SichuanT,14:6A:0BCypressE,14:6B:72Shenzhen,14:6B:9Azte,14:6B:9CShenzhen,14:6E:0APrivate,14:73:73TubitakU,14:74:11Rim,14:75:90Tp-LinkT,14:77:40HuaweiDe,14:78:0BVarexIma,14:79:F3ChinaMob,14:7B:ACNokia,14:7D:05SercommP,14:7D:B3JoaTelec,14:7D:C5MurataMa,14:7D:DAApple,14:82:5BHefeiRad,14:84:30MitacCom,14:85:7FIntelCor,14:86:92Tp-LinkT,14:87:6AApple,14:88:E6Apple,14:89:192bps,14:89:3EVixtelTe,14:89:CBHuaweiTe,14:89:FDSamsungE,14:8A:70ADS,14:8C:4AHuaweiTe,14:8F:21GarminIn,14:8F:C6Apple,14:90:90KongTopi,14:91:38AmazonTe,14:91:82BelkinIn,14:93:46PNIsenso,14:94:2FUsys,14:94:48BluCastl,14:95:CEApple,14:96:E5SamsungE,14:98:77Apple,14:98:7DTechnico,14:99:E2Apple,14:9A:10Microsof,14:9B:2FJiangSuZ,14:9D:09HuaweiTe,14:9D:99Apple,14:9E:CFDell,14:9F:3CSamsungE,14:9F:B6Guangdon,14:9F:E8LenovoMo,14:A0:F8HuaweiTe,14:A1:BFASSAABLO,14:A2:A0Cisco,14:A3:2FHuaweiDe,14:A3:64SamsungE,14:A3:B4HuaweiDe,14:A5:1AHuaweiTe,14:A6:2CSMDezac,14:A7:2Bcurrento,14:A7:8BZhejiang,14:A8:6BShenZhen,14:A9:D0F5Networ,14:A9:E3Mst,14:AB:02HuaweiTe,14:AB:56WuxiFuni,14:AB:C5IntelCor,14:AB:F0ARRISGro,14:AD:CAChinaMob,14:AE:85IEEERegi,14:AE:85:00:00:00/28Kayamati,14:AE:85:10:00:00/28HenfredT,14:AE:85:20:00:00/28Qingdaoi,14:AE:85:30:00:00/28Iflytek,14:AE:85:40:00:00/28Centervu,14:AE:85:50:00:00/28Az-Techn,14:AE:85:60:00:00/28TMGTE,14:AE:85:70:00:00/28Shenzhen,14:AE:85:80:00:00/28TrimbleL,14:AE:85:90:00:00/28VeoTechn,14:AE:85:A0:00:00/28MTA,14:AE:85:B0:00:00/28NtcSoft,14:AE:85:C0:00:00/28IOIndust,14:AE:85:D0:00:00/28iSolutio,14:AE:85:E0:00:00/28Sercomm,14:AE:DBVTechTel,14:B1:26Industri,14:B1:C8InfiniWi,14:B2:E5Shenzhen,14:B3:1FDell,14:B3:70GigasetD,14:B4:57SiliconL,14:B4:84SamsungE,14:B7:3DARCHEANT,14:B7:F8Technico,14:B8:37Shenzhen,14:B9:68HuaweiTe,14:BB:6ESamsungE,14:BD:61Apple,14:C0:3EARRISGro,14:C0:89DuneHd,14:C0:A1UCloudTe,14:C1:26Nokia,14:C1:4EGoogle,14:C1:FFShenZhen,14:C2:13Apple,14:C2:1DSabtechI,14:C3:C2KASchmer,14:C6:97Guangdon,14:C8:8BApple,14:C9:13LGElectr,14:C9:CFSigmasta,14:CA:A0Hu,14:CB:19HP,14:CB:65Microsof,14:CC:20Tp-LinkT,14:CC:B3AoGkNate,14:CF:8DOhsung,14:CF:92Tp-LinkT,14:CF:E2ARRISGro,14:D0:0DApple,14:D1:1FHuaweiTe,14:D1:69HuaweiTe,14:D1:9EApple,14:D4:FEARRISGro,14:D6:4DD-LinkIn,14:D7:6ECONCHELE,14:DA:E9ASUSTekC,14:DB:85SNetMedi,14:DD:9CvivoMobi,14:DD:A9ASUSTekC,14:DD:E5Mpmkvvcl,14:DE:39HuaweiDe,14:E4:ECmLogic,14:E6:E4Tp-LinkT,14:E7:C8Integrat,14:E9:B2Fiberhom,14:EB:08HuaweiTe,14:EB:33BSMedias,14:EB:B6TP-Link,14:ED:A5Wächter,14:ED:BB2Wire,14:ED:E4Kaiam,14:EE:9DAirNav,14:EF:CFSchreder,14:F0:C5Xtremio,14:F2:8EShenYang,14:F4:2ASamsungE,14:F6:5AXiaomiCo,14:F6:D8IntelCor,14:F8:93WuhanFib,14:FB:70HuaweiDe,14:FE:AFSagittar,14:FE:B5Dell,18:00:2DSony,18:00:DBFitbit,18:01:7DHarbinAr,18:01:E3BittiumW,18:01:F1XiaomiCo,18:02:2DHuaweiTe,18:02:AEvivoMobi,18:03:73Dell,18:03:FAIBTInter,18:04:EDTexasIns,18:06:75DilaxInt,18:06:F5RADDataC,18:06:FFAcerComp,18:07:12Shenzhen,18:0B:52Nanotron,18:0C:14iSonea,18:0C:77Westingh,18:0C:ACCanon,18:0D:2CIntelbra,18:0E:ACShenzhen,18:0F:76D-LinkIn,18:10:4ECedint-U,18:11:71Guangzho,18:12:12CeptonTe,18:13:2Dzte,18:14:20TebSas,18:14:56Nokia,18:14:6CZhejiang,18:16:C9SamsungE,18:17:14Daewoois,18:17:25CameoCom,18:18:8BFujitsuC,18:19:3FTamtron,18:19:D6SamsungE,18:1B:EBActionte,18:1D:EAIntelCor,18:1E:78Sagemcom,18:1E:95AuVerte,18:1E:B0SamsungE,18:20:12AztechAs,18:20:32Apple,18:20:4CKummler+,18:20:A6Sage,18:20:D5ARRISGro,18:21:95SamsungE,18:22:7ESamsungE,18:26:49IntelCor,18:26:66SamsungE,18:28:61AirTiesW,18:2A:44HiroseEl,18:2A:57HuaweiTe,18:2A:7BNintendo,18:2A:D3JuniperN,18:2B:058DTechno,18:2C:91ConceptD,18:2C:B4Nectarso,18:2D:98JinwooIn,18:2D:F7Jy,18:30:09WoojinIn,18:30:0CHisenseE,18:31:BFASUSTekC,18:32:19EMMicroe,18:32:A2LaonTech,18:33:9DCisco,18:34:51Apple,18:35:D1ARRISGro,18:36:72Shaoxing,18:36:FCElecsysI,18:38:25WuhanLin,18:38:64Cap-Tech,18:38:AEConspinS,18:39:19Unicoi,18:39:6ESunseaTe,18:39:9CSkorpios,18:3A:2DSamsungE,18:3A:48VostroNe,18:3B:D2BYDPreci,18:3C:B7HuaweiDe,18:3D:5EHuaweiTe,18:3D:A2IntelCor,18:3E:EFApple,18:3F:47SamsungE,18:40:A4Shenzhen,18:41:FEDigital1,18:42:1DPrivate,18:42:2FAlcatelL,18:42:D4WuhanHos,18:44:62RiavaNet,18:44:E6zte,18:45:16TexasIns,18:45:93TaicangT,18:46:17SamsungE,18:46:44HomeCont,18:47:3DChongqin,18:48:59Castlene,18:48:BEAmazonTe,18:48:CAMurataMa,18:48:D8Fastback,18:4A:6FAlcatel-,18:4B:0DRuckusWi,18:4B:DFCaavo,18:4C:08Rockwell,18:4C:AEContinen,18:4E:16SamsungE,18:4E:94MessoaTe,18:4E:CBSamsungE,18:4F:32HonHaiPr,18:4F:5DJRCMobil,18:50:2ASoarnex,18:52:07SichuanT,18:52:53Pixord,18:52:82Fiberhom,18:53:45Nokia,18:53:E0HanyangD,18:54:CFSamsungE,18:55:0FCiscoSPV,18:55:E3Apple,18:56:44HuaweiTe,18:56:80IntelCor,18:56:C3Apple,18:58:69SailerEl,18:59:33CiscoSPV,18:59:36XiaomiCo,18:59:F5Cisco,18:5A:58Dell,18:5A:E8Zenotech,18:5B:00Nokia,18:5B:B3SamsungE,18:5D:9ABobjGear,18:5E:0FIntelCor,18:60:24HewlettP,18:61:C7lemonbea,18:62:2CSagemcom,18:62:E4TexasIns,18:64:72ArubaaHe,18:65:71TopVicto,18:65:90Apple,18:66:C7Shenzhen,18:66:DADell,18:66:E3Veros,18:67:3FHanoverD,18:67:51KOMEGInd,18:67:B0SamsungE,18:68:6Azte,18:68:82BewardR&,18:68:CBHangzhou,18:69:D8TuyaSmar,18:69:DAChinaMob,18:6D:99Adanis,18:6F:2DShenzhen,18:70:3BHuaweiDe,18:71:17etapluse,18:71:D5HazensAu,18:74:2EAmazonTe,18:74:E2IEEERegi,18:74:E2:00:00:00/28Ensor,18:74:E2:10:00:00/28Sartoriu,18:74:E2:20:00:00/28Shenzhen,18:74:E2:30:00:00/28CT,18:74:E2:40:00:00/28Private,18:74:E2:50:00:00/28Hangzhou,18:74:E2:60:00:00/28BeijingJ,18:74:E2:70:00:00/28SansecTe,18:74:E2:80:00:00/28KanoComp,18:74:E2:90:00:00/28Shenzhen,18:74:E2:A0:00:00/28LinuxAut,18:74:E2:B0:00:00/28Shenzhen,18:74:E2:C0:00:00/28NextGenR,18:74:E2:D0:00:00/28Samriddi,18:74:E2:E0:00:00/28G&OAudio,18:75:32SichuanT,18:77:58AudooUK,18:78:D4Verizon,18:79:A2GmjElect,18:7A:3BArubaaHe,18:7A:93AMICCOME,18:7C:0BRuckusWi,18:7C:81ValeoVis,18:7E:B9Apple,18:7E:D5shenzhen,18:80:90Cisco,18:80:CEBarberry,18:80:F5Alcatel-,18:81:0EApple,18:82:19AlibabaC,18:82:8CArcadyan,18:83:31SamsungE,18:83:BFArcadyan,18:84:10CoreTrus,18:86:3ADigitalA,18:86:ACNokiaDan,18:87:40XiaomiCo,18:87:96HTC,18:88:57BeijingJ,18:89:5BSamsungE,18:89:A0WuhanFun,18:89:DFCerebrEX,18:8A:6AAVProGlo,18:8B:15ShenZhen,18:8B:45Cisco,18:8B:9DCisco,18:8E:D5TPVision,18:8E:F9G2C,18:90:88eero,18:90:D8Sagemcom,18:92:2CVirtualI,18:92:A4Ciena,18:93:7FAMPAKTec,18:93:D7TexasIns,18:94:C6ShenZhen,18:95:521More,18:97:FFTechFait,18:99:F5SichuanC,18:9A:67CSE-Serv,18:9B:A5IEEERegi,18:9B:A5:00:00:00/28Dectris,18:9B:A5:10:00:00/28ChengDuV,18:9B:A5:20:00:00/28Airprote,18:9B:A5:30:00:00/28Phinetwo,18:9B:A5:40:00:00/28Innomind,18:9B:A5:50:00:00/28Starfire,18:9B:A5:60:00:00/28MantraSo,18:9B:A5:70:00:00/28BeijingX,18:9B:A5:80:00:00/28Shenzhen,18:9B:A5:90:00:00/28APANA,18:9B:A5:A0:00:00/28Shenzhen,18:9B:A5:B0:00:00/28Eutron,18:9B:A5:C0:00:00/28ChristEl,18:9B:A5:D0:00:00/28legendsk,18:9B:A5:E0:00:00/28TaiwanNa,18:9C:27ARRISGro,18:9C:5DCisco,18:9E:2CHuaweiDe,18:9E:FCApple,18:A2:8AEssel-T,18:A3:E8Fiberhom,18:A4:A9Vanu,18:A6:F7Tp-LinkT,18:A7:F1QingdaoH,18:A9:05HewlettP,18:A9:58Provisio,18:A9:9BDell,18:A9:A6Nebra,18:AA:0FHuaweiDe,18:AA:45FonTechn,18:AA:CASichuant,18:AB:1DSamsungE,18:AB:F5UltraEle,18:AC:9EItelMobi,18:AD:4DPolostar,18:AE:BBSiemensC,18:AF:61Apple,18:AF:8FApple,18:AF:9FDIGITRON,18:B1:69Sonicwal,18:B2:09TorreyPi,18:B3:BANetlogic,18:B4:30NestLabs,18:B5:91I-Storm,18:B6:CCWe,18:B6:F7NewPosTe,18:B7:9EInvoxia,18:B8:1FARRISGro,18:B9:05HongKong,18:B9:6EDongguan,18:BB:26Fn-LinkT,18:BB:41HuaweiDe,18:BC:5AZhejiang,18:BD:ADL-Tech,18:BE:92DeltaNet,18:BF:1CJiangsuH,18:BF:B3SamsungE,18:C0:4DGiga-Byt,18:C0:86Broadcom,18:C1:9DIntegrat,18:C2:41SonicWal,18:C2:BFBuffalo,18:C4:51TucsonEm,18:C5:01Shenzhen,18:C5:8AHuaweiTe,18:C8:E7Shenzhen,18:CC:18IntelCor,18:CC:23PhilioTe,18:CC:88HitachiJ,18:CE:94SamsungE,18:CF:24HuaweiTe,18:CF:5ELiteonTe,18:D0:71Dasan,18:D0:C5Guangdon,18:D2:25Fiberhom,18:D2:76HuaweiTe,18:D5:B6SMGHoldi,18:D6:1CShenzhen,18:D6:6AInmarsat,18:D6:C7Tp-LinkT,18:D6:CFKurthEle,18:D7:17Guangdon,18:D9:49QvisLabs,18:D9:8FHuaweiDe,18:D9:EFShuttle,18:DB:F2Dell,18:DC:56YulongCo,18:DE:D7HuaweiTe,18:DF:B4BosungPo,18:DF:C1Aetheros,18:E1:CAwanze,18:E1:DEChengduC,18:E2:15Nokia,18:E2:88STTCondi,18:E2:9FvivoMobi,18:E2:C2SamsungE,18:E3:BCTCTmobil,18:E7:28Cisco,18:E7:77vivoMobi,18:E7:F4Apple,18:E8:0FVikingEl,18:E8:29Ubiquiti,18:E8:DDModulete,18:EC:E7Buffalo,18:EE:69Apple,18:EE:86NovatelW,18:EF:3ASichuanA,18:EF:63Cisco,18:F0:E4XiaomiCo,18:F1:45NetCommW,18:F1:8EChipERTe,18:F1:D8Apple,18:F2:2CTp-LinkT,18:F2:92Shannon,18:F4:6AHonHaiPr,18:F6:43Apple,18:F6:50Multimed,18:F7:6BZhejiang,18:F8:7Ai3Intern,18:F8:7FWhaYuInd,18:F9:C4BAE,18:FA:6FISCappli,18:FB:7BDell,18:FC:26QorvoInt,18:FC:9FChangheE,18:FD:CBIEEERegi,18:FD:CB:00:00:00/28Shenzhen,18:FD:CB:10:00:00/28SOTHISCI,18:FD:CB:20:00:00/28Cabtroni,18:FD:CB:30:00:00/28Staclar,18:FD:CB:40:00:00/28GosuncnT,18:FD:CB:50:00:00/28AccelRob,18:FD:CB:60:00:00/28SKAOrgan,18:FD:CB:70:00:00/28EnergieI,18:FD:CB:80:00:00/28CISTECHS,18:FD:CB:90:00:00/28CreyNox,18:FD:CB:A0:00:00/28Sercomm,18:FD:CB:B0:00:00/28Translit,18:FD:CB:C0:00:00/28ArkVisio,18:FD:CB:D0:00:00/28StreamLo,18:FD:CB:E0:00:00/28KwangYan,18:FE:34Espressi,18:FF:0FIntelCor,18:FF:2EShenzhen,1A:75:FBAutosar,1A:BB:FBUHDAllia,1C:00:42NARITech,1C:01:2DFicerTec,1C:02:19Guangdon,1C:05:B7Chongqin,1C:06:56IDY,1C:08:C1LGInnote,1C:0B:52Epicom,1C:0F:AFLucidVis,1C:0F:CFSyproOpt,1C:11:61Ciena,1C:11:E1Wartsila,1C:12:9DIeeePesP,1C:12:B0AmazonTe,1C:13:38KimballE,1C:13:86HuaweiDe,1C:14:48ARRISGro,1C:14:B3AirwireT,1C:15:1FHuaweiTe,1C:17:D3Cisco,1C:18:4AShenZhen,1C:19:DEeyevis,1C:1A:C0Apple,1C:1A:DFMicrosof,1C:1B:0DGiga-Byt,1C:1B:68ARRISGro,1C:1B:B5IntelCor,1C:1C:FDDalianHi,1C:1D:67HuaweiTe,1C:1D:86Cisco,1C:1E:38PCCWGlob,1C:1E:E3HuiZhouG,1C:1F:D4LifeBEAM,1C:1F:F1HuaweiDe,1C:20:DBHuaweiTe,1C:21:D1IEEERegi,1C:21:D1:00:00:00/28ToyoSyst,1C:21:D1:10:00:00/28Ognios,1C:21:D1:20:00:00/28VaraaniW,1C:21:D1:30:00:00/28Microvie,1C:21:D1:40:00:00/28Scientif,1C:21:D1:50:00:00/28B-Scada,1C:21:D1:60:00:00/28WuhanTie,1C:21:D1:70:00:00/28Soundtra,1C:21:D1:80:00:00/28Reliatro,1C:21:D1:90:00:00/28DynojetR,1C:21:D1:A0:00:00/28LgCns,1C:21:D1:B0:00:00/28GlobalDe,1C:21:D1:C0:00:00/28Private,1C:21:D1:D0:00:00/28Liscotec,1C:21:D1:E0:00:00/28p2-plus,1C:21:D1:F0:00:00/28Private,1C:23:2CSamsungE,1C:23:4FEDMIEuro,1C:24:CDAskeyCom,1C:24:EBBurlywoo,1C:25:E1ChinaMob,1C:27:04zte,1C:27:DDDatangGo,1C:28:AFArubaaHe,1C:2A:8BNokia,1C:2A:A3Shenzhen,1C:2C:E0Shanghai,1C:2E:1BSuzhouTr,1C:30:08HuiZhouG,1C:33:0EPernixDa,1C:33:4DITSTelec,1C:34:77Innovati,1C:34:DAMellanox,1C:35:F1NEWLiftN,1C:36:BBApple,1C:37:BFCloudium,1C:39:29Ohsung,1C:39:47CompalIn,1C:39:8AFiberhom,1C:3A:4FAccuSpec,1C:3A:60RuckusWi,1C:3A:DESamsungE,1C:3B:8FSelve,1C:3B:F3Tp-LinkT,1C:3C:D4HuaweiTe,1C:3D:2FHuaweiTe,1C:3D:E7SigmaKok,1C:3E:84HonHaiPr,1C:40:24Dell,1C:40:E8Shenzhen,1C:41:58GemaltoM,1C:41:76ChinaMob,1C:41:90Universa,1C:42:7DGuangdon,1C:43:63HuaweiTe,1C:43:ECJapanCir,1C:44:19Tp-LinkT,1C:44:55SiebMeye,1C:45:86Nintendo,1C:45:93TexasIns,1C:45:C2HuizhouC,1C:47:2FHuaweiDe,1C:48:40IMSMesss,1C:48:CEGuangdon,1C:48:F9GNNetcom,1C:49:7BGemtekTe,1C:4A:F7Amon,1C:4B:B9SmgEnter,1C:4B:D6AzureWav,1C:4C:48ItelMobi,1C:4D:66AmazonTe,1C:4D:70IntelCor,1C:50:1ESunplusT,1C:51:B5Techaya,1C:52:16Dongguan,1C:52:D6FlatDisp,1C:53:F9Google,1C:54:9EUniversa,1C:55:3AQianGua,1C:56:8EZioncomE,1C:56:FEMotorola,1C:57:3EAlticeLa,1C:57:D8Kraftway,1C:59:9BHuaweiTe,1C:5A:0BTegile,1C:5A:3ESamsungE,1C:5A:6BPhilipsE,1C:5C:55PRIMACin,1C:5C:60Shenzhen,1C:5C:F2Apple,1C:5D:80Mitubish,1C:5E:E6Shenzhen,1C:5F:2BD-LinkIn,1C:5F:FFBeijingE,1C:60:D2Fiberhom,1C:60:DEMercuryC,1C:62:B8SamsungE,1C:63:B7OpenProd,1C:63:BFShenzhen,1C:64:99Comtrend,1C:65:9DLiteonTe,1C:66:6DHonHaiPr,1C:66:AASamsungE,1C:67:58HuaweiTe,1C:68:7EShenzhen,1C:69:7AEliteGro,1C:69:A5BlackBer,1C:6A:7ACisco,1C:6B:CAMitsunam,1C:6E:4CLogistic,1C:6E:76QuarionT,1C:6E:E6Nhnetwor,1C:6F:65Giga-Byt,1C:70:22MurataMa,1C:72:1DDell,1C:73:28Connecte,1C:73:70Neotech,1C:73:E2HuaweiTe,1C:74:0DZyxelCom,1C:75:08CompalIn,1C:76:CATerasicT,1C:77:F6Guangdon,1C:78:39Shenzhen,1C:78:4EChinaMob,1C:7B:21Sony,1C:7B:23QingdaoH,1C:7C:11Eid,1C:7C:45VitekInd,1C:7C:C7Coriant,1C:7D:22FUJIFILM,1C:7E:513bumenco,1C:7E:E5D-LinkIn,1C:7F:2CHuaweiTe,1C:82:59IEEERegi,1C:82:59:00:00:00/28Shandong,1C:82:59:10:00:00/283xLOGIC,1C:82:59:20:00:00/28Diatrend,1C:82:59:30:00:00/28C&AMarke,1C:82:59:40:00:00/28winsun,1C:82:59:50:00:00/28Fagus-Gr,1C:82:59:60:00:00/28CgiItUk,1C:82:59:70:00:00/28JumpTrad,1C:82:59:80:00:00/28Shenzhen,1C:82:59:90:00:00/28Shanghai,1C:82:59:A0:00:00/28ESTec,1C:82:59:B0:00:00/28KeyWestN,1C:82:59:C0:00:00/28Evondos,1C:82:59:D0:00:00/28AppliedC,1C:82:59:E0:00:00/28Microtro,1C:83:41HefeiBit,1C:83:B0LinkedIP,1C:84:64FormosaW,1C:86:ADMct,1C:87:2CASUSTekC,1C:87:74:00:00:00/28PhilipsP,1C:87:74:10:00:00/28Sigfox,1C:87:74:20:00:00/28Nichigak,1C:87:74:30:00:00/28SiloraR&,1C:87:74:40:00:00/28WeberMar,1C:87:74:50:00:00/28Xiaoxing,1C:87:74:60:00:00/28Schawbel,1C:87:74:70:00:00/28IngBuero,1C:87:74:80:00:00/28SurtecIn,1C:87:74:90:00:00/28WideWorl,1C:87:74:A0:00:00/28Nebbiolo,1C:87:74:B0:00:00/28HABEYUSA,1C:87:74:C0:00:00/28NewNordi,1C:87:74:D0:00:00/28Claber,1C:87:74:E0:00:00/28QuestInt,1C:87:76:00:00:00/28DspreadT,1C:87:76:10:00:00/28EBSSp,1C:87:76:20:00:00/28IbeoAuto,1C:87:76:30:00:00/28Unjo,1C:87:76:40:00:00/28RdpRu,1C:87:76:50:00:00/28ZhuhaiMY,1C:87:76:60:00:00/28philandr,1C:87:76:70:00:00/28Corporat,1C:87:76:80:00:00/28Guangzho,1C:87:76:90:00:00/28TokyoDra,1C:87:76:A0:00:00/28JiangsuE,1C:87:76:B0:00:00/28Hekatron,1C:87:76:C0:00:00/28StroneTe,1C:87:76:D0:00:00/28Qivivo,1C:87:76:E0:00:00/28Artis,1C:87:79:00:00:00/28WurmElek,1C:87:79:10:00:00/28A-Gear,1C:87:79:20:00:00/28SMARTMOV,1C:87:79:30:00:00/28VisualLa,1C:87:79:40:00:00/28Novetta,1C:87:79:50:00:00/28BeidianG,1C:87:79:60:00:00/28Shenzhen,1C:87:79:70:00:00/28TASC,1C:87:79:80:00:00/28Zhejiang,1C:87:79:90:00:00/28Istriaso,1C:87:79:A0:00:00/28Hangzhou,1C:87:79:B0:00:00/28BeijingG,1C:87:79:C0:00:00/28AllThing,1C:87:79:D0:00:00/28Shenzhen,1C:87:79:E0:00:00/28ASSYSTEM,1C:87:E3TecnoMob,1C:88:0CShenzhen,1C:88:79:00:00:00/28Newps,1C:88:79:10:00:00/28ANDRASpz,1C:88:79:20:00:00/28Airsmart,1C:88:79:30:00:00/28Shenzhen,1C:88:79:40:00:00/28Ultraflu,1C:88:79:50:00:00/28Shenzhen,1C:88:79:60:00:00/28EolosIT,1C:88:79:70:00:00/28SensysNe,1C:88:79:80:00:00/28ToshibaT,1C:88:79:90:00:00/28Xingtera,1C:88:79:A0:00:00/28Itw-Feg,1C:88:79:B0:00:00/28gekartel,1C:88:79:C0:00:00/28Accriva,1C:88:79:D0:00:00/28BeijingR,1C:88:79:E0:00:00/28OrionLab,1C:8E:5CHuaweiTe,1C:8E:8EDBCommun,1C:8F:8APhaseMot,1C:90:BEEricsson,1C:91:48Apple,1C:91:79Integrat,1C:91:80Apple,1C:91:9DDongguan,1C:93:7CARRISGro,1C:93:C4AmazonTe,1C:94:92RUAGSchw,1C:95:5DI-LaxEle,1C:95:9FVeethree,1C:96:5AWeifangG,1C:97:3DPRICOMDe,1C:97:C5Ynomia,1C:98:C1CloudNet,1C:98:ECHewlettP,1C:99:4CMurataMa,1C:99:57IntelCor,1C:9C:26ZoovelTe,1C:9C:8CJuniperN,1C:9D:3EIntegrat,1C:9D:C2Espressi,1C:9E:46Apple,1C:9E:CBBeijingN,1C:9E:CCTechnico,1C:9F:4ECooseaGr,1C:A0:B8HonHaiPr,1C:A0:D3IEEERegi,1C:A0:D3:00:00:00/28Tekhnotr,1C:A0:D3:10:00:00/28Jabilcir,1C:A0:D3:20:00:00/28NovTech,1C:A0:D3:30:00:00/28Savelec,1C:A0:D3:40:00:00/28NpoTelec,1C:A0:D3:50:00:00/28DynamicC,1C:A0:D3:60:00:00/28Intertec,1C:A0:D3:70:00:00/28Private,1C:A0:D3:80:00:00/28Desarrol,1C:A0:D3:90:00:00/28CirqueAu,1C:A0:D3:A0:00:00/28DSMMesst,1C:A0:D3:B0:00:00/28GuangDon,1C:A0:D3:C0:00:00/28LYT,1C:A0:D3:D0:00:00/28ERATOHK,1C:A0:D3:E0:00:00/28ExicomTe,1C:A0:EFIEEERegi,1C:A0:EF:00:00:00/28Tangshan,1C:A0:EF:10:00:00/28Wisnuand,1C:A0:EF:20:00:00/28Schneide,1C:A0:EF:30:00:00/28Sequent,1C:A0:EF:40:00:00/28Leviatha,1C:A0:EF:50:00:00/28NanjingB,1C:A0:EF:60:00:00/28HanjenCh,1C:A0:EF:70:00:00/28tec5AG,1C:A0:EF:80:00:00/28Zillnk,1C:A0:EF:90:00:00/28AtlasAer,1C:A0:EF:A0:00:00/28HenrichE,1C:A0:EF:B0:00:00/28BMKprofe,1C:A0:EF:C0:00:00/28GagarIn,1C:A0:EF:D0:00:00/28Shenzhen,1C:A0:EF:E0:00:00/28RDAMicro,1C:A2:B1ruwidoau,1C:A5:32Shenzhen,1C:A6:81HuaweiTe,1C:A7:70Shenzhen,1C:A8:52SensaioP,1C:AA:07Cisco,1C:AB:01Innovolt,1C:AB:34NewH3CTe,1C:AB:A7Apple,1C:AB:C0HitronTe,1C:AD:D1BosungEl,1C:AE:3EIEEERegi,1C:AE:3E:00:00:00/28DaoQinTe,1C:AE:3E:10:00:00/28IPROAD,1C:AE:3E:20:00:00/28Linkwise,1C:AE:3E:30:00:00/28HagerEne,1C:AE:3E:40:00:00/28PHUMeter,1C:AE:3E:50:00:00/28Netvio,1C:AE:3E:60:00:00/28Annapurn,1C:AE:3E:70:00:00/28NextDriv,1C:AE:3E:80:00:00/28JingQiti,1C:AE:3E:90:00:00/28ChinaCon,1C:AE:3E:A0:00:00/28BeijingS,1C:AE:3E:B0:00:00/28BeijingB,1C:AE:3E:C0:00:00/28QuESTRai,1C:AE:3E:D0:00:00/28Forme,1C:AE:3E:E0:00:00/28Broachli,1C:AE:CBHuaweiTe,1C:AF:05SamsungE,1C:AF:F7D-LinkIn,1C:B0:44AskeyCom,1C:B0:94HTC,1C:B1:7FNECPlatf,1C:B2:43Tdc,1C:B3:C9Apple,1C:B3:E9Shenzhen,1C:B7:2CASUSTekC,1C:B7:96HuaweiTe,1C:B8:57BeconTec,1C:B9:C4RuckusWi,1C:BA:8CTexasIns,1C:BB:A8OJSCUfim,1C:BD:0EAmplifie,1C:BD:B9D-LinkIn,1C:BF:C0Chongqin,1C:BF:CEShenzhen,1C:C0:35PlanexCo,1C:C0:E1IEEERegi,1C:C0:E1:00:00:00/28Shenzhen,1C:C0:E1:10:00:00/28Hangzhou,1C:C0:E1:20:00:00/28AbbottMe,1C:C0:E1:30:00:00/28Hangzhou,1C:C0:E1:40:00:00/28Videri,1C:C0:E1:50:00:00/28KidsWire,1C:C0:E1:60:00:00/28Monument,1C:C0:E1:70:00:00/28Shenzhen,1C:C0:E1:80:00:00/28LX,1C:C0:E1:90:00:00/28Ospicon,1C:C0:E1:A0:00:00/28Secheron,1C:C0:E1:B0:00:00/28ExigentS,1C:C0:E1:C0:00:00/28NittoSei,1C:C0:E1:D0:00:00/28NewLandN,1C:C0:E1:E0:00:00/28YunYangF,1C:C1:0CIntelCor,1C:C1:1AWavetron,1C:C1:BCYichipMi,1C:C1:DEHewlettP,1C:C3:16MileSigh,1C:C3:EBGuangdon,1C:C5:86Absolute,1C:C6:3CArcadyan,1C:C7:2DShenzhen,1C:CA:E3IEEERegi,1C:CA:E3:00:00:00/28Private,1C:CA:E3:10:00:00/28PgaElect,1C:CA:E3:20:00:00/28Insigma,1C:CA:E3:30:00:00/28Shenzhen,1C:CA:E3:40:00:00/28SunrayMe,1C:CA:E3:50:00:00/28TengFeng,1C:CA:E3:60:00:00/28TokaiRik,1C:CA:E3:70:00:00/28BirdHome,1C:CA:E3:80:00:00/28OxySec,1C:CA:E3:90:00:00/28Shin-Yos,1C:CA:E3:A0:00:00/28Sirea,1C:CA:E3:B0:00:00/28DreamVis,1C:CA:E3:C0:00:00/28Gahdeung,1C:CA:E3:D0:00:00/28eSight,1C:CA:E3:E0:00:00/28DabiAtla,1C:CA:E3:F0:00:00/28Private,1C:CB:99TCTmobil,1C:CC:D6XiaomiCo,1C:CD:E5Shanghai,1C:D1:07RealmeCh,1C:D1:BAFiberhom,1C:D1:E0Cisco,1C:D4:0CKriwanIn,1C:D5:E2Shenzhen,1C:D6:BDLeedarso,1C:D6:BEWistronN,1C:DA:27vivoMobi,1C:DD:EAGuangdon,1C:DE:57Fiberhom,1C:DE:A7Cisco,1C:DF:0FCisco,1C:DF:52TexasIns,1C:E1:65Marshal,1C:E1:92Qisda,1C:E2:CCTexasIns,1C:E5:7FSamsungE,1C:E6:1DSamsungE,1C:E6:2BApple,1C:E6:39HuaweiTe,1C:E6:ADHuaweiDe,1C:E6:C7Cisco,1C:E8:5DCisco,1C:EA:0BEdgecore,1C:EA:1BNokia,1C:EC:72Allradio,1C:ED:6FAVMAudio,1C:EE:C9Elotouch,1C:EE:E8IlshinEl,1C:EF:CEbebroele,1C:F0:3EWearhaus,1C:F0:61SCAPS,1C:F2:9AGoogle,1C:F4:CAPrivate,1C:F5:E7TurtleIn,1C:FA:68Tp-LinkT,1C:FC:BBRealfict,1C:FD:08IEEERegi,1C:FD:08:00:00:00/28InSeatSo,1C:FD:08:10:00:00/28Shenzhen,1C:FD:08:20:00:00/28HiHi,1C:FD:08:30:00:00/28UmeoxInn,1C:FD:08:40:00:00/28SABIKOff,1C:FD:08:50:00:00/28BeijingH,1C:FD:08:60:00:00/28A&BTechn,1C:FD:08:70:00:00/28sunweiti,1C:FD:08:80:00:00/28ShenZhen,1C:FD:08:90:00:00/28CobhamSl,1C:FD:08:A0:00:00/28BanmakTe,1C:FD:08:B0:00:00/28guangzho,1C:FD:08:C0:00:00/28Shanghai,1C:FD:08:D0:00:00/28TianjinK,1C:FD:08:E0:00:00/28MeshboxF,1C:FE:2BAmazonTe,1C:FE:A7IDentyte,1C:FF:59SichuanT,20:01:4FLineaRes,20:02:AFMurataMa,20:04:0FDell,20:05:05RadmaxCo,20:05:E8InProMed,20:08:EDHuaweiTe,20:0A:0DIEEERegi,20:0A:0D:00:00:00/28halstrup,20:0A:0D:10:00:00/28Wideband,20:0A:0D:20:00:00/28Netinovo,20:0A:0D:30:00:00/28ClearlyI,20:0A:0D:40:00:00/28Virtium,20:0A:0D:50:00:00/28Shenzhen,20:0A:0D:60:00:00/28AustinHu,20:0A:0D:70:00:00/28TecnintH,20:0A:0D:80:00:00/28bcheck,20:0A:0D:90:00:00/28WelzekBe,20:0A:0D:A0:00:00/28Irsap,20:0A:0D:B0:00:00/28AmazonTe,20:0A:0D:C0:00:00/28sehwa,20:0A:0D:D0:00:00/28BentlyEL,20:0A:0D:E0:00:00/28HANGZHOU,20:0A:5EXiangsha,20:0B:C7HuaweiTe,20:0B:CFNintendo,20:0C:C8Netgear,20:0D:B0Shenzhen,20:0E:95Iec–Tc9W,20:0F:70Foxtech,20:10:7AGemtekTe,20:11:4EMeteRSit,20:12:57MostLuck,20:12:D5Scientec,20:13:E0SamsungE,20:16:3DIntegrat,20:16:B9IntelCor,20:16:D8LiteonTe,20:17:42LGElectr,20:18:0EShenzhen,20:1A:06CompalIn,20:1B:88Dongguan,20:1B:C9JuniperN,20:1D:03Elatec,20:1E:88IntelCor,20:1F:31IntenoBr,20:1F:3BGoogle,20:20:27Shenzhen,20:21:A5LGElectr,20:25:64Pegatron,20:25:98Teleview,20:25:D2Fiberhom,20:26:81TecnoMob,20:28:3EHuaweiTe,20:28:BCVisionsc,20:2A:C5Petite-E,20:2B:20CloudNet,20:2B:C1HuaweiTe,20:2C:B7KongYueE,20:2D:07SamsungE,20:2D:23Collinea,20:2D:F8DigitalM,20:31:1CvivoMobi,20:31:EBHdsn,20:32:33Shenzhen,20:32:6CSamsungE,20:32:C6Apple,20:34:FBXiaomiCo,20:36:5BMegafone,20:36:D7Shanghai,20:37:06Cisco,20:37:A5Apple,20:37:BCKuipersE,20:39:56HMDGloba,20:3A:07Cisco,20:3A:EFSivantos,20:3B:69vivoMobi,20:3C:AEApple,20:3C:C0BeijingT,20:3D:66ARRISGro,20:3D:B2HuaweiTe,20:3D:BDLGInnote,20:40:05feno,20:41:5ASmartehd,20:41:81ESYSEEmb,20:44:3ASchneide,20:44:41RemoteSo,20:46:A1VECOW,20:46:F9Advanced,20:47:47Dell,20:47:DAXiaomiCo,20:47:EDBSkyB,20:4A:AAHanscanS,20:4B:22SunnovoI,20:4C:03ArubaaHe,20:4C:6DHugoBren,20:4C:9ECisco,20:4E:6BAxxanais,20:4E:71JuniperN,20:4E:7FNetgear,20:4E:F6AzureWav,20:50:E7AMPAKTec,20:53:83HuaweiTe,20:53:CARiskTech,20:54:76Sony,20:54:FAHuaweiTe,20:55:31SamsungE,20:55:32GotechIn,20:57:21SalixTec,20:57:9EHunanFn-,20:57:AFShenzhen,20:58:69RuckusWi,20:59:A0ParagonT,20:5A:00Coval,20:5B:2APrivate,20:5B:5EShenzhen,20:5C:FAYangzhou,20:5D:47vivoMobi,20:5E:64HuaweiDe,20:5E:F7SamsungE,20:5F:3DCambridg,20:62:74Microsof,20:63:5FAbeeway,20:64:32SamsungE,20:64:CBGuangdon,20:65:8EHuaweiTe,20:66:CFFreeboxS,20:66:FDConstell,20:67:7CHewlettP,20:67:B1Pluto,20:67:E0Shenzhen,20:68:9DLiteonTe,20:69:80Apple,20:6A:8AWistronI,20:6A:94HitronTe,20:6A:FFAtlasEle,20:6B:E7Tp-LinkT,20:6C:8AExtremeN,20:6D:31Firewall,20:6E:9CSamsungE,20:6F:ECBraemacC,20:71:9ESFTechno,20:73:55ARRISGro,20:74:54vivoMobi,20:74:CFShenzhen,20:76:00Actionte,20:76:8FApple,20:76:93LenovoBe,20:77:59OpticalN,20:78:0BDeltaFau,20:78:52NokiaSol,20:78:F0Apple,20:79:18IntelCor,20:7B:D2ASIXElec,20:7C:14Qotom,20:7C:8FQuantaMi,20:7D:74Apple,20:80:58Ciena,20:80:97Shenzhen,20:82:6AGuangdon,20:82:C0XiaomiCo,20:83:F8Advanced,20:85:8CAssa,20:85:93IEEERegi,20:85:93:00:00:00/28Hemina,20:85:93:10:00:00/28Networki,20:85:93:20:00:00/28MidConti,20:85:93:30:00:00/28Unilumin,20:85:93:40:00:00/28Kloudspo,20:85:93:50:00:00/28Wave-InC,20:85:93:60:00:00/28Eilersen,20:85:93:70:00:00/28GreatLit,20:85:93:80:00:00/28AassetSe,20:85:93:90:00:00/28Mastodon,20:85:93:A0:00:00/28H3Indust,20:85:93:B0:00:00/28IOGProdu,20:85:93:C0:00:00/28Reglopla,20:85:93:D0:00:00/28Shanghai,20:85:93:E0:00:00/28Dynaudio,20:87:56Siemens,20:87:ACAESmotom,20:87:ECHuaweiTe,20:89:6FFiberhom,20:89:84CompalIn,20:89:86zte,20:8B:37Skyworth,20:8C:47Tenstorr,20:90:6FShenzhen,20:91:48TexasIns,20:91:8AProfalux,20:91:D9IM,20:93:4DFujianSt,20:96:8AChinaMob,20:98:D8Shenzhen,20:9A:7DSagemcom,20:9A:E9Volacomm,20:9B:A5JIAXINGG,20:9B:CDApple,20:9E:79Universa,20:9E:F7ExtremeN,20:A1:71AmazonTe,20:A2:E4Apple,20:A2:E7Lee-Dick,20:A6:0CXiaomiCo,20:A6:80HuaweiTe,20:A6:CDHewlettP,20:A7:83miContro,20:A7:87BointecT,20:A7:F9Shenzhen,20:A8:B9Siemens,20:A9:0ETCTmobil,20:A9:9BMicrosof,20:AA:25Ip-Net,20:AA:4BCisco-Li,20:AB:37Apple,20:AB:48HuaweiTe,20:AC:9CChinaTel,20:AD:56Continen,20:B0:01Technico,20:B0:F7Enclustr,20:B3:99Enterasy,20:B5:C6MimosaNe,20:B7:30TeconGro,20:B7:80ToshibaV,20:B7:C0OMICRONe,20:B8:68Motorola,20:BB:76COLGIOVA,20:BB:C0Cisco,20:BB:C6JabilCir,20:BE:CDeero,20:BF:DBDvl,20:C0:47Verizon,20:C0:6DShenzhen,20:C1:9BIntelCor,20:C1:AFiWitDigi,20:C3:8FTexasIns,20:C3:A4RetailNe,20:C6:0DShanghai,20:C6:EBPanasoni,20:C7:4FSensorPu,20:C8:B3Shenzhen,20:C9:D0Apple,20:CD:39TexasIns,20:CD:6ERealmeCh,20:CE:2AIEEERegi,20:CE:2A:00:00:00/28Annapurn,20:CE:2A:10:00:00/28Shanghai,20:CE:2A:20:00:00/28Jabil,20:CE:2A:30:00:00/28Cuculus,20:CE:2A:40:00:00/28Annapurn,20:CE:2A:50:00:00/28ZaberTec,20:CE:2A:60:00:00/28Radarxen,20:CE:2A:70:00:00/28BeijingH,20:CE:2A:80:00:00/28Intellig,20:CE:2A:90:00:00/28RuggedMo,20:CE:2A:A0:00:00/28MeshPlus,20:CE:2A:B0:00:00/28Swarovsk,20:CE:2A:C0:00:00/28AristonT,20:CE:2A:D0:00:00/28LaudaDrR,20:CE:2A:E0:00:00/28Funkwerk,20:CE:C4PerasoTe,20:CF:30ASUSTekC,20:CF:AECisco,20:D1:60Private,20:D2:1FWincalTe,20:D2:5FSmartCap,20:D2:76ItelMobi,20:D3:90SamsungE,20:D5:ABKoreaInf,20:D5:BFSamsungE,20:D6:07Nokia,20:D7:5APoshMobi,20:D7:78TexasIns,20:D8:0BJuniperN,20:D9:06Iota,20:DA:22HuaweiTe,20:DB:ABSamsungE,20:DC:93CheetahH,20:DC:E6Tp-LinkT,20:DC:FDHuaweiDe,20:DE:88ICRealti,20:DF:3FNanjingS,20:DF:73HuaweiTe,20:DF:B9Google,20:E0:9CNokia,20:E2:A8Apple,20:E4:07Spark,20:E5:2ANetgear,20:E5:64ARRISGro,20:E7:91SiemensH,20:E7:B6Universa,20:E8:74Apple,20:E8:82zte,20:EA:C7Shenzhen,20:ED:74Abilitye,20:EE:28Apple,20:EE:C6Elefirst,20:EF:BDRoku,20:F0:02MTDataDe,20:F1:7CHuaweiTe,20:F1:9EARRISGro,20:F3:75ARRISGro,20:F3:A3HuaweiTe,20:F4:1BShenzhen,20:F4:4FNokia,20:F4:52Shanghai,20:F4:78XiaomiCo,20:F5:10CodexDig,20:F5:43HuiZhouG,20:F7:7CvivoMobi,20:F8:5EDeltaEle,20:FA:BBCambridg,20:FD:F13ComEuro,20:FE:CDSystemIn,20:FE:DBM2MSolut,20:FF:36Iflytek,24:00:BAHuaweiTe,24:00:FAChinaMob,24:01:6FHuaweiDe,24:01:C7Cisco,24:05:0FMTNElect,24:05:88Google,24:05:F5Integrat,24:06:AAGuangdon,24:08:5DContinen,24:09:17DevlinEl,24:09:95HuaweiTe,24:0A:11TCTmobil,24:0A:63ARRISGro,24:0A:64AzureWav,24:0A:C4Espressi,24:0B:0APaloAlto,24:0B:2AViettelG,24:0B:88TaicangT,24:0B:B1KOSTALIn,24:0D:65Shenzhen,24:0D:6CSmnd,24:0D:C2TCTmobil,24:0F:9BHangzhou,24:10:64Shenzhen,24:11:25Hutek,24:11:45XiaomiCo,24:11:48Entropix,24:11:D0Chongqin,24:12:81ChinaMob,24:14:07XiamenSi,24:15:10IEEERegi,24:15:10:00:00:00/28Safetrus,24:15:10:10:00:00/28SMaBiT,24:15:10:20:00:00/28NileGlob,24:15:10:30:00:00/28Kaiyun,24:15:10:40:00:00/28Annapurn,24:15:10:50:00:00/28GanzhouD,24:15:10:60:00:00/28Shandong,24:15:10:70:00:00/28SuZhouA-,24:15:10:80:00:00/28Private,24:15:10:90:00:00/28TopgolfS,24:15:10:A0:00:00/28Unitronu,24:15:10:B0:00:00/28Teknic,24:15:10:C0:00:00/28Shenzhen,24:15:10:D0:00:00/28HelenTro,24:15:10:E0:00:00/28Satellit,24:16:6DHuaweiTe,24:16:9DCisco,24:18:1DSamsungE,24:18:C6HunanFn-,24:1A:8CSquarehe,24:1A:E6HuaweiDe,24:1B:13Shanghai,24:1B:44Hangzhou,24:1B:7AApple,24:1C:04Shenzhen,24:1D:48SichuanT,24:1E:EBApple,24:1F:2CCalsys,24:1F:A0HuaweiTe,24:20:C7Sagemcom,24:21:24Nokia,24:21:ABSony,24:24:0EApple,24:26:42SHARP,24:26:BAShenzhen,24:28:FDHangzhou,24:29:FEKYOCERA,24:2E:02HuaweiTe,24:2E:90PalitMic,24:2F:FAToshibaG,24:30:F8HuaweiDe,24:31:54HuaweiTe,24:31:84SHARP,24:33:6CPrivate,24:35:CCZhongsha,24:36:DACisco,24:37:4CCiscoSPV,24:37:EFEMCElect,24:3A:82Irts,24:3C:20Dynamode,24:3F:30OxygenBr,24:3F:AAHuaweiDe,24:40:AENIICTech,24:41:8CIntelCor,24:42:BCAlinco,24:43:E2DASANNet,24:44:27HuaweiTe,24:45:6BHuaweiDe,24:45:97GEMUEGeb,24:46:C8Motorola,24:46:E4HuaweiTe,24:47:0EPentroni,24:49:7BInnovati,24:4B:03SamsungE,24:4B:81SamsungE,24:4B:FEASUSTekC,24:4C:07HuaweiTe,24:4C:E3AmazonTe,24:4E:7BIEEERegi,24:4E:7B:00:00:00/28TekelekE,24:4E:7B:10:00:00/28sonoscap,24:4E:7B:20:00:00/28RccTime,24:4E:7B:30:00:00/28Shenzhen,24:4E:7B:40:00:00/28LeshiInt,24:4E:7B:50:00:00/28JiangsuX,24:4E:7B:60:00:00/28OwasysAd,24:4E:7B:70:00:00/28NanjingW,24:4E:7B:80:00:00/28Cyber1st,24:4E:7B:90:00:00/28UniMATAu,24:4E:7B:A0:00:00/28Shenzhen,24:4E:7B:B0:00:00/28MightyAu,24:4E:7B:C0:00:00/28Chunghsi,24:4E:7B:D0:00:00/28ChurchDw,24:4E:7B:E0:00:00/28WithWinT,24:4F:1DiRule,24:52:6AZhejiang,24:53:BFEnernet,24:58:6Ezte,24:58:80Vizeo,24:59:0BWhiteSky,24:5A:4CUbiquiti,24:5A:B5SamsungE,24:5B:83RenesasE,24:5B:A7Apple,24:5B:F0Liteon,24:5C:BFNcse,24:5C:CBAXIeCons,24:5D:FCIEEERegi,24:5D:FC:00:00:00/28CompanyD,24:5D:FC:10:00:00/28ARTICONA,24:5D:FC:20:00:00/28BlueIris,24:5D:FC:30:00:00/28Shenzhen,24:5D:FC:40:00:00/28SuzhouJi,24:5D:FC:50:00:00/28ContactP,24:5D:FC:60:00:00/28Guangzho,24:5D:FC:70:00:00/28Lty,24:5D:FC:80:00:00/28Cosmicno,24:5D:FC:90:00:00/28Torgovyy,24:5D:FC:A0:00:00/28TataSky,24:5D:FC:B0:00:00/28Only,24:5D:FC:C0:00:00/28Senix,24:5D:FC:D0:00:00/28HunanHon,24:5D:FC:E0:00:00/28Dodge,24:5E:48Apple,24:5E:BEQNAP,24:5F:9FHuaweiDe,24:5F:DFKyocera,24:60:81razberit,24:61:5AChinaMob,24:62:78sysmocom,24:62:ABEspressi,24:62:CEArubaaHe,24:64:9FHuaweiDe,24:64:EFCygSunri,24:65:11AVM,24:68:80Braverid,24:68:B0SamsungE,24:69:3Einnodisk,24:69:4AJasmine,24:69:68Tp-LinkT,24:69:8EShenzhen,24:69:A5HuaweiTe,24:6A:ABIT-ISInt,24:6C:8AYUKAIEng,24:6E:96Dell,24:6F:28Espressi,24:6F:8CHuaweiDe,24:71:52Dell,24:71:89TexasIns,24:72:60IOTTECH,24:74:F7GoPro,24:76:25TexasIns,24:76:56Shanghai,24:76:7DCiscoSPV,24:77:03IntelCor,24:79:2ARuckusWi,24:79:EFGreenpac,24:79:F3Guangdon,24:79:F8KUPSONsp,24:7C:4CHermanMi,24:7D:4DTexasIns,24:7E:12Cisco,24:7E:51zte,24:7F:20Sagemcom,24:7F:3CHuaweiTe,24:80:00Westcont,24:81:AAKSHInter,24:81:C7HuaweiDe,24:82:8AProwaveT,24:84:98BeijingJ,24:86:F4Ctek,24:87:07SEnergy,24:88:94shenzhen,24:8A:07Mellanox,24:8B:E0SichuanT,24:91:BBHuaweiTe,24:92:0ESamsungE,24:93:CAVoxtroni,24:94:42OpenRoad,24:94:93FibRSolG,24:94:94HongKong,24:94:CBARRISGro,24:95:04Sfr,24:97:45HuaweiTe,24:97:EDTechvisi,24:9A:C8Shenzhen,24:9A:D8YealinkX,24:9E:ABHuaweiTe,24:9F:89TexasIns,24:A0:74Apple,24:A1:60Espressi,24:A2:E1Apple,24:A4:2CKOUKAAM,24:A4:3CUbiquiti,24:A4:87HuaweiDe,24:A4:95ThalesCa,24:A5:2CHuaweiTe,24:A5:34SynTrust,24:A6:5Ezte,24:A7:99HuaweiDe,24:A7:DCBSkyB,24:A8:7DPanasoni,24:A9:37PUREStor,24:AB:81Apple,24:AF:4AAlcatel-,24:AF:54NEXGENMe,24:B0:A9Shanghai,24:B1:05PramaHik,24:B2:09Avaya,24:B2:DEEspressi,24:B6:57Cisco,24:B6:B8Friem,24:B6:FDDell,24:B7:2AChinaDra,24:B8:8CCrenus,24:B8:D2OpzoonTe,24:BA:13RisoKaga,24:BA:30Technica,24:BB:C1Absolute,24:BC:82DaliWire,24:BC:F8HuaweiTe,24:BE:05HewlettP,24:BE:18Dadoutek,24:BF:74Private,24:C0:B3Rsf,24:C1:7ABeijingI,24:C1:BDCrrcDali,24:C3:F9Securita,24:C4:2FPhilipsL,24:C4:4Azte,24:C6:13SamsungE,24:C6:96SamsungE,24:C8:48mywerkPo,24:C8:6EChaneyIn,24:C8:D3McWaneIn,24:C9:A1RuckusWi,24:C9:DEGenoray,24:CA:CBFiberhom,24:CB:E7MYK,24:CD:8DMurataMa,24:CE:33AmazonTe,24:CF:21Shenzhen,24:D0:DFApple,24:D1:3FMexus,24:D2:CCSmartDri,24:D3:F2zte,24:D5:1CZhongtia,24:D7:6BSyntroni,24:D7:EBEspressi,24:D8:1EMirWifiJ,24:D9:04SichuanC,24:D9:21Avaya,24:DA:11NONDA,24:DA:33HuaweiTe,24:DA:9BMotorola,24:DA:B6Sistemas,24:DB:ACHuaweiTe,24:DB:ADShopperT,24:DB:EDSamsungE,24:DE:C6ArubaaHe,24:DF:6AHuaweiTe,24:DF:A7Hangzhou,24:E1:24XiamenMi,24:E2:71QingdaoH,24:E3:14Apple,24:E4:3FWenzhouK,24:E4:C8Fiberhom,24:E5:AAPhilipsO,24:E6:BAZavodimK,24:E8:53LGInnote,24:E9:27TomTomIn,24:E9:B3Cisco,24:E9:CAHuaweiDe,24:EA:40Helmholz,24:EB:65SAETIS,24:EC:51ADFTechn,24:EC:99AskeyCom,24:EC:D6CSGScien,24:ED:FDSiemensC,24:EE:3AChengduY,24:EE:9AIntelCor,24:F0:94Apple,24:F0:FFGHT,24:F1:28Telstra,24:F1:50Guangzho,24:F2:7FHewlettP,24:F2:DDRadiantZ,24:F5:7EHwh,24:F5:A2BelkinIn,24:F5:AASamsungE,24:F6:03HuaweiTe,24:F6:77Apple,24:FA:F3Shanghai,24:FB:65HuaweiTe,24:FC:4EJuniperN,24:FC:E5SamsungE,24:FD:0DIntelbra,24:FD:52LiteonTe,24:FD:5BSmartThi,28:02:45KonzeSys,28:02:D8SamsungE,28:04:C6WananHon,28:04:E0FermaxEl,28:05:2EDematic,28:06:1ENingboGl,28:06:8DItl,28:07:0DGuangzho,28:0B:5CApple,28:0C:28UnigenDa,28:0C:B8Mikrosay,28:0D:FCSonyInte,28:0E:8BBeijingS,28:0F:C5BeijingL,28:0F:EBLGInnote,28:10:1BMagnaCom,28:10:7BD-LinkIn,28:11:A5Bose,28:11:A8IntelCor,28:11:ECHuaweiTe,28:14:71Lantis,28:16:2E2Wire,28:16:7FXiaomiCo,28:16:A8Microsof,28:16:ADIntelCor,28:17:09HuaweiTe,28:17:CEOmnisens,28:18:78Microsof,28:18:FDAdityaIn,28:1B:04Zalliant,28:1D:21InOneSma,28:22:46BeijingS,28:23:73Digita,28:23:F5ChinaMob,28:24:FFWistronN,28:25:36Shenzhen,28:26:A6PBRelect,28:27:BFSamsungE,28:28:5DZyxelCom,28:29:86APCbySch,28:29:CCCorsaTec,28:29:D9GlobalBe,28:2A:87ItelMobi,28:2B:96HuaweiDe,28:2C:02IEEERegi,28:2C:02:00:00:00/28SAKATADE,28:2C:02:10:00:00/28Astronic,28:2C:02:20:00:00/28Shenzhen,28:2C:02:30:00:00/28DexinDig,28:2C:02:40:00:00/28EfentoTP,28:2C:02:50:00:00/28Microteh,28:2C:02:60:00:00/28LookmanE,28:2C:02:70:00:00/28Telecoma,28:2C:02:80:00:00/28Shenzhen,28:2C:02:90:00:00/28SystecIn,28:2C:02:A0:00:00/28Tokin,28:2C:02:B0:00:00/28ThirdRea,28:2C:02:C0:00:00/28EpochInt,28:2C:02:D0:00:00/28Shenzhen,28:2C:02:E0:00:00/28Capintec,28:2C:B2Tp-LinkT,28:2D:06AMPAKTec,28:2F:C2Automoti,28:30:ACFrontiir,28:31:52HuaweiTe,28:31:66vivoMobi,28:31:7EHongkong,28:32:C5HUMAX,28:33:34HuaweiDe,28:34:10EnigmaDi,28:34:A2Cisco,28:35:45Shenzhen,28:36:13IEEERegi,28:36:13:00:00:00/28Shandong,28:36:13:10:00:00/28Hi-pSuzh,28:36:13:20:00:00/28Shenzhen,28:36:13:30:00:00/28LinearCo,28:36:13:40:00:00/28ElytoneE,28:36:13:50:00:00/28TuringVi,28:36:13:60:00:00/28ESIVentu,28:36:13:70:00:00/28shenzhen,28:36:13:80:00:00/28FuzhouLe,28:36:13:90:00:00/28QingdaoA,28:36:13:A0:00:00/28MAKEENEn,28:36:13:B0:00:00/28Qorvo,28:36:13:C0:00:00/28midBitTe,28:36:13:D0:00:00/28Avycon,28:36:13:E0:00:00/28EGMedica,28:36:38IEEERegi,28:36:38:00:00:00/28KnowlesE,28:36:38:10:00:00/28Panasoni,28:36:38:20:00:00/28Shenzhen,28:36:38:30:00:00/28Sabinete,28:36:38:40:00:00/28DspreadT,28:36:38:50:00:00/28Chargeli,28:36:38:60:00:00/28GeorgNeu,28:36:38:70:00:00/28Innovati,28:36:38:80:00:00/28HavellsI,28:36:38:90:00:00/28Shenzhen,28:36:38:A0:00:00/28Bluekey,28:36:38:B0:00:00/28ShangHai,28:36:38:C0:00:00/28Swisson,28:36:38:D0:00:00/28APPEAKTe,28:36:38:E0:00:00/28SCAHygie,28:37:13Shenzhen,28:37:37Apple,28:38:5CFlextron,28:38:CFGen2wave,28:39:26CyberTAN,28:39:5ESamsungE,28:39:E7PrecenoT,28:3A:4DCloudNet,28:3B:82D-LinkIn,28:3B:96CoolCont,28:3C:E4HuaweiTe,28:3E:76CommonNe,28:3F:69Sony,28:40:1AC8MediSe,28:41:21OptiSens,28:41:C6HuaweiTe,28:41:ECHuaweiTe,28:44:30GenesisT,28:47:AANokia,28:48:46GridCent,28:48:E7HuaweiDe,28:4C:53IntuneNe,28:4D:92Luminato,28:4E:D7OutSmart,28:4F:CELiaoning,28:51:32Shenzhen,28:52:61Cisco,28:52:E0Layonint,28:52:F9Zhongxin,28:53:4EHuaweiTe,28:54:71HuaweiDe,28:56:3AFiberhom,28:56:5AHonHaiPr,28:56:C1HarmanBe,28:57:67DishTech,28:57:BEHangzhou,28:5A:EBApple,28:5B:0CSichuanJ,28:5F:2FRNware,28:5F:DBHuaweiTe,28:60:46LantechC,28:60:94Capelec,28:63:36Siemens,28:63:BDAptivSer,28:64:B0HuaweiDe,28:64:EFShenzhen,28:65:6BKeystone,28:66:E3AzureWav,28:68:D2HuaweiTe,28:6A:B8Apple,28:6A:BAApple,28:6C:07XIAOMIEl,28:6D:97SAMJIN,28:6D:CDBeijingW,28:6E:D4HuaweiTe,28:6F:7FCisco,28:71:84SpirePay,28:72:C5Smartmat,28:72:F0Athena,28:75:D8FujianSt,28:76:10IgniteNe,28:76:CDFunshion,28:77:77zte,28:77:F1Apple,28:79:94Realplay,28:7A:EEARRISGro,28:7B:09zte,28:7C:DBHefeiToy,28:7F:CFIntelCor,28:80:23HewlettP,28:80:88Netgear,28:80:A2NovatelW,28:83:35SamsungE,28:84:0Esiliconv,28:84:FASHARP,28:85:2DTouchNet,28:89:15CashGuar,28:8A:1CJuniperN,28:8C:B8zte,28:91:D0StageTec,28:92:4AHewlettP,28:93:7DSichuanT,28:93:FECisco,28:94:0FCisco,28:94:AFSamhwaTe,28:97:B8myenergi,28:98:7BSamsungE,28:99:3AAristaNe,28:99:C7LindsayB,28:9A:4BSteelSer,28:9A:F7ADVAOpti,28:9A:FATCTmobil,28:9C:6EShanghai,28:9E:97HuaweiTe,28:9E:DFDanfossT,28:9E:FCSagemcom,28:A0:2BApple,28:A1:83Alpsalpi,28:A1:86enblink,28:A1:92GERPSolu,28:A1:EBEtekTech,28:A2:41exlar,28:A2:4BJuniperN,28:A5:74MillerEl,28:A5:EEShenzhen,28:A6:ACseca,28:A6:DBHuaweiTe,28:AC:67MachPowe,28:AC:9ECisco,28:AD:18HuiZhouG,28:AD:3EShenzhen,28:AF:0ASiriusXM,28:AF:FDCisco,28:B0:CCXenyadoo,28:B2:BDIntelCor,28:B3:71RuckusWi,28:B3:ABGenmarkA,28:B4:48HuaweiTe,28:B4:FBSprocomm,28:B7:7CIEEERegi,28:B7:7C:00:00:00/28Shenzhen,28:B7:7C:10:00:00/28SolarEdg,28:B7:7C:20:00:00/28ZhuhaiRo,28:B7:7C:30:00:00/28BeijingK,28:B7:7C:40:00:00/28Annapurn,28:B7:7C:50:00:00/28GROTHE,28:B7:7C:60:00:00/28Shanghai,28:B7:7C:70:00:00/28Converte,28:B7:7C:80:00:00/28Shenzhen,28:B7:7C:90:00:00/28AnserCod,28:B7:7C:A0:00:00/28Simaudio,28:B7:7C:B0:00:00/28VehantTe,28:B7:7C:C0:00:00/28AnyLink,28:B7:7C:D0:00:00/28EnedoFin,28:B7:7C:E0:00:00/28RayPte,28:B9:D9Radisys,28:BA:18NextNav,28:BA:B5SamsungE,28:BB:59RNETTech,28:BC:18Sourcing,28:BC:56EMAC,28:BD:89Google,28:BE:03TCTmobil,28:BE:9BTechnico,28:BF:89Fiberhom,28:C0:1BShenzhen,28:C0:DAJuniperN,28:C1:3CHonHaiPr,28:C2:1FSamsungE,28:C2:DDAzureWav,28:C5:38Apple,28:C6:3FIntelCor,28:C6:71YotaDevi,28:C6:8ENetgear,28:C7:09Apple,28:C7:18Altierre,28:C7:CECisco,28:C8:25DellKing,28:C8:7AARRISGro,28:C8:7Czte,28:C9:14Taimag,28:CA:09ThyssenK,28:CB:EBOne,28:CC:01SamsungE,28:CC:FFCorporac,28:CD:1CEspotel,28:CD:4CIndividu,28:CD:9CShenzhen,28:CD:C4Chongqin,28:CF:08Essys,28:CF:DAApple,28:CF:E9Apple,28:D0:44Shenzhen,28:D0:CBCambridg,28:D0:EAIntelCor,28:D1:27BeijingX,28:D1:AFNokia,28:D1:B7Shenzhen,28:D2:44LCFCHeFe,28:D3:EAHuaweiDe,28:D4:36Jiangsud,28:D5:76PremierW,28:D9:3ETelecor,28:D9:8AHangzhou,28:D9:97YuduanMo,28:DB:81Shanghai,28:DE:65ArubaaHe,28:DE:A8zte,28:DE:E5HuaweiTe,28:DE:F6bioMerie,28:DF:EBIntelCor,28:E0:2CApple,28:E1:4CApple,28:E2:97Shanghai,28:E3:1FXiaomiCo,28:E3:47LiteonTe,28:E3:4EHuaweiTe,28:E4:76Pi-Coral,28:E5:B0HuaweiTe,28:E6:08Tokheim,28:E6:E9SISSatIn,28:E7:94Microtim,28:E7:CFApple,28:E9:8EMitsubis,28:EC:95Apple,28:EC:9ATexasIns,28:ED:58JAGJakob,28:ED:6AApple,28:ED:E0AMPAKTec,28:EE:2CFrontlin,28:EE:52Tp-LinkT,28:EE:D3Shenzhen,28:EF:01AmazonTe,28:F0:33Apple,28:F0:76Apple,28:F1:0EDell,28:F3:582C-Trifo,28:F3:66Shenzhen,28:F4:9BLeetek,28:F5:32ADD-Engi,28:F5:37IEEERegi,28:F5:37:00:00:00/28ValeoSie,28:F5:37:10:00:00/28Umojo,28:F5:37:20:00:00/28UnicairC,28:F5:37:30:00:00/28Primetec,28:F5:37:40:00:00/28Phyn,28:F5:37:50:00:00/28Atomrock,28:F5:37:60:00:00/28MyOmega,28:F5:37:70:00:00/28Shenzhen,28:F5:37:80:00:00/281More,28:F5:37:90:00:00/28HerbertW,28:F5:37:A0:00:00/28Honeywel,28:F5:37:B0:00:00/28LogiMSof,28:F5:37:C0:00:00/28MatricxS,28:F5:37:D0:00:00/28Skyrocke,28:F5:37:E0:00:00/28Performa,28:F6:06Syes,28:FA:19Shenzhen,28:FA:7AZhejiang,28:FA:A0vivoMobi,28:FB:AEHuaweiTe,28:FB:D3Ragentek,28:FC:51Electric,28:FC:F6Shenzhen,28:FD:80IEEERegi,28:FD:80:00:00:00/28Millcode,28:FD:80:10:00:00/28Galileo,28:FD:80:20:00:00/28Zhixiang,28:FD:80:30:00:00/28NUUO,28:FD:80:40:00:00/28DigitalS,28:FD:80:50:00:00/28Xiaocong,28:FD:80:60:00:00/28VigilMon,28:FD:80:70:00:00/28Universi,28:FD:80:80:00:00/28JascoPro,28:FD:80:90:00:00/28Jinliton,28:FD:80:A0:00:00/28ApolloDi,28:FD:80:B0:00:00/28PoketHar,28:FD:80:C0:00:00/28AirbusDe,28:FD:80:D0:00:00/28Grandway,28:FD:80:E0:00:00/28T-RadioA,28:FD:80:F0:00:00/28Private,28:FE:65DongGuan,28:FE:CDLemobile,28:FE:DECOMESTA,28:FF:3CApple,28:FF:3Ezte,28:FF:B2Toshiba,2A:A1:39Pointgua,2A:EA:15TibitCom,2A:FD:6ACharterC,2C:00:2CUnowhy,2C:00:33EControl,2C:00:ABARRISGro,2C:00:F7Xos,2C:01:0BNASCENTT,2C:01:B5Cisco,2C:02:9F3ALogics,2C:05:47Shenzhen,2C:06:23WinLeade,2C:07:3CDevline,2C:07:86HuaweiDe,2C:08:1COvh,2C:08:23SercommF,2C:08:8CHUMAX,2C:09:4DRaptorEn,2C:09:CBCobs,2C:0B:E9Cisco,2C:0D:A7IntelCor,2C:0E:3DSamsungE,2C:10:C1Nintendo,2C:11:65SiliconL,2C:15:BFSamsungE,2C:15:E1PhicommS,2C:16:BDIEEERegi,2C:16:BD:00:00:00/28BeijingJ,2C:16:BD:10:00:00/28Curtiss-,2C:16:BD:20:00:00/28Aimco,2C:16:BD:30:00:00/28Saft,2C:16:BD:40:00:00/28Sunit,2C:16:BD:50:00:00/28BeijingZ,2C:16:BD:60:00:00/28Cloudwal,2C:16:BD:70:00:00/28SctOptro,2C:16:BD:80:00:00/28Shenzhen,2C:16:BD:90:00:00/28Shanghai,2C:16:BD:A0:00:00/28Shenzhen,2C:16:BD:B0:00:00/28Lingdong,2C:16:BD:C0:00:00/28BeijingC,2C:16:BD:D0:00:00/28Hangzhou,2C:16:BD:E0:00:00/28Molex,2C:17:E0SYSTEMES,2C:18:75Skyworth,2C:18:AETrendEle,2C:19:84IDNTelec,2C:1A:01HuaweiTe,2C:1A:05Cisco,2C:1A:31Electron,2C:1B:C8HunanTop,2C:1C:F6AlienGre,2C:1D:B8ARRISGro,2C:1E:4FChengduQ,2C:1E:EAAerodev,2C:1F:23Apple,2C:20:0BApple,2C:21:31JuniperN,2C:21:72JuniperN,2C:21:D7IMAX,2C:22:8BCtr,2C:23:3AHewlettP,2C:24:5FBabolatV,2C:26:17OculusVR,2C:26:5FIEEERegi,2C:26:5F:00:00:00/28XiamenVo,2C:26:5F:10:00:00/28Griessba,2C:26:5F:20:00:00/28JiangsuJ,2C:26:5F:30:00:00/28shenzhen,2C:26:5F:40:00:00/28GTAElect,2C:26:5F:50:00:00/28Motec,2C:26:5F:60:00:00/28Appostar,2C:26:5F:70:00:00/28Coremate,2C:26:5F:80:00:00/28ItusNetw,2C:26:5F:90:00:00/28BrüelKja,2C:26:5F:A0:00:00/28PolaraEn,2C:26:5F:B0:00:00/28Rexgen,2C:26:5F:C0:00:00/28AatonDig,2C:26:5F:D0:00:00/28ECore,2C:26:5F:E0:00:00/28Hysentel,2C:26:5F:F0:00:00/28Private,2C:26:C5zte,2C:27:68HuaweiTe,2C:27:9EIEEERegi,2C:27:9E:00:00:00/28Changzho,2C:27:9E:10:00:00/28Electron,2C:27:9E:20:00:00/28Kunyiele,2C:27:9E:30:00:00/28Private,2C:27:9E:40:00:00/28Shijiazh,2C:27:9E:50:00:00/28AudioNor,2C:27:9E:60:00:00/28Rutledge,2C:27:9E:70:00:00/28FOCAL-JM,2C:27:9E:80:00:00/28Institut,2C:27:9E:90:00:00/28octoScop,2C:27:9E:A0:00:00/28Exegy,2C:27:9E:B0:00:00/28Forties,2C:27:9E:C0:00:00/28WAYCOMTe,2C:27:9E:D0:00:00/28JiangsuJ,2C:27:9E:E0:00:00/28Amaryllo,2C:27:D7HewlettP,2C:28:2DBbkEduca,2C:28:B7Hangzhou,2C:29:97Microsof,2C:2B:F9LGInnote,2C:2D:48bctelect,2C:30:33Netgear,2C:30:68Pantech,2C:31:24Cisco,2C:33:11Cisco,2C:33:61Apple,2C:33:7AHonHaiPr,2C:34:27ErcoGene,2C:35:57ELIIYPow,2C:36:A0Capisco,2C:36:F8Cisco,2C:37:31Shenzhen,2C:37:96Cybo,2C:37:C5QingdaoH,2C:39:96Sagemcom,2C:39:C1Ciena,2C:3A:28FagorEle,2C:3A:91HuaweiDe,2C:3A:E8Espressi,2C:3A:FDAVMAudio,2C:3B:FDNetstorT,2C:3E:CFCisco,2C:3F:0BCiscoMer,2C:3F:38Cisco,2C:3F:3EAlge-Tim,2C:40:2BSmartiBl,2C:40:53SamsungE,2C:41:38HewlettP,2C:41:A1Bose,2C:42:05Lytx,2C:43:1AShenzhen,2C:43:BESunnovoI,2C:44:01SamsungE,2C:44:1BSpectrum,2C:44:FDHewlettP,2C:47:59BeijingM,2C:48:35IEEERegi,2C:48:35:00:00:00/28Progress,2C:48:35:10:00:00/28Advanced,2C:48:35:20:00:00/28RheonikM,2C:48:35:30:00:00/28NewtraxT,2C:48:35:40:00:00/28Geartech,2C:48:35:50:00:00/28ScoutSec,2C:48:35:60:00:00/28Exertus,2C:48:35:70:00:00/28Fast,2C:48:35:80:00:00/28DPSElect,2C:48:35:90:00:00/28SureFlap,2C:48:35:A0:00:00/28Collatz+,2C:48:35:B0:00:00/28Shanghai,2C:48:35:C0:00:00/28Santec,2C:48:35:D0:00:00/28PhasorSo,2C:48:35:E0:00:00/28Irootech,2C:48:81vivoMobi,2C:4A:11Ciena,2C:4C:C6MurataMa,2C:4D:54ASUSTekC,2C:4D:79WeifangG,2C:4E:7DChunghua,2C:4F:52Cisco,2C:50:89Shenzhen,2C:52:AFHuaweiTe,2C:53:4AShenzhen,2C:53:D7Sonova,2C:54:2DCisco,2C:54:91Microsof,2C:54:CFLGElectr,2C:55:3CGainspee,2C:55:7CShenzhen,2C:55:D3HuaweiTe,2C:56:DCASUSTekC,2C:57:31Wingtech,2C:57:41Cisco,2C:58:4FARRISGro,2C:58:E8HuaweiTe,2C:59:8ALGElectr,2C:59:E5HewlettP,2C:5A:05Nokia,2C:5A:0FCisco,2C:5A:8DSYSTRONI,2C:5A:A3PromateE,2C:5B:B8Guangdon,2C:5B:E1Centripe,2C:5D:34Guangdon,2C:5D:93RuckusWi,2C:5F:F3Pertroni,2C:60:0CQuantaCo,2C:61:04Shenzhen,2C:61:F6Apple,2C:62:5AFinestSe,2C:62:89Regeners,2C:63:73SichuanT,2C:64:1FVizio,2C:67:98InTalTec,2C:67:FBShenZhen,2C:69:BARFContro,2C:6A:6FIEEERegi,2C:6A:6F:00:00:00/28Shanghai,2C:6A:6F:10:00:00/28ELKOEPsr,2C:6A:6F:20:00:00/28NanChang,2C:6A:6F:30:00:00/28Cloudpro,2C:6A:6F:40:00:00/28Tinyco,2C:6A:6F:50:00:00/28ShenZhen,2C:6A:6F:60:00:00/28Beep,2C:6A:6F:70:00:00/28SmDsp,2C:6A:6F:80:00:00/28MilbankM,2C:6A:6F:90:00:00/28LogicIOA,2C:6A:6F:A0:00:00/28Wellntel,2C:6A:6F:B0:00:00/28Schneide,2C:6A:6F:C0:00:00/28Sensity,2C:6A:6F:D0:00:00/28Holjeron,2C:6A:6F:E0:00:00/28EATONFHF,2C:6A:6F:F0:00:00/28Private,2C:6B:7DTexasIns,2C:6B:F5JuniperN,2C:6D:C1IntelCor,2C:6E:85IntelCor,2C:6F:51Herospee,2C:6F:C9HonHaiPr,2C:71:55HiveMoti,2C:71:FFAmazonTe,2C:72:C3Soundmat,2C:73:60EardaTec,2C:73:A0Cisco,2C:75:0FShanghai,2C:76:8AHewlettP,2C:78:0EHuaweiDe,2C:79:3DBoditech,2C:79:D7Sagemcom,2C:7B:5AMilper,2C:7B:84PetrTele,2C:7C:E4WuhanTia,2C:7E:81ARRISGro,2C:7E:CFOnzo,2C:80:65HARTINGN,2C:81:58HonHaiPr,2C:86:D2Cisco,2C:8A:72HTC,2C:8B:F2HitachiM,2C:8D:B1IntelCor,2C:91:27Eintechn,2C:91:ABAVMAudio,2C:92:2CKishuGik,2C:94:64Cincoze,2C:95:69ARRISGro,2C:95:7Fzte,2C:96:62Invenit,2C:97:17ICY,2C:97:B1HuaweiTe,2C:97:EDSonyImag,2C:99:24ARRISGro,2C:9A:A4Eolo,2C:9D:1EHuaweiTe,2C:9E:5FARRISGro,2C:9E:ECJabilCir,2C:9E:FCCanon,2C:9F:FBWistronN,2C:A0:2FVeroguar,2C:A0:42HuaweiDe,2C:A1:57acromate,2C:A1:7DARRISGro,2C:A2:B4FortifyT,2C:A3:0EPowerDra,2C:A5:39Parallel,2C:A5:9CHangzhou,2C:A7:80TrueTech,2C:A8:35Rim,2C:A8:9CCreatz,2C:A9:F0Guangdon,2C:AA:8EWyzeLabs,2C:AB:00HuaweiTe,2C:AB:25Shenzhen,2C:AB:33TexasIns,2C:AB:A4CiscoSPV,2C:AB:EBCisco,2C:AC:44Conextop,2C:AD:13Shenzhen,2C:AE:2BSamsungE,2C:B0:5DNetgear,2C:B0:DFSolitonT,2C:B0:FDShenzhen,2C:B1:15Integrat,2C:B2:1APhicommS,2C:B4:3AApple,2C:B6:93Radware,2C:B6:9DREDDigit,2C:B8:EDSonicWal,2C:BA:BASamsungE,2C:BC:87Apple,2C:BE:08Apple,2C:BE:97Ingenieu,2C:BE:EBNothingT,2C:C2:60Oracle,2C:C4:07machineQ,2C:C5:46HuaweiDe,2C:C5:48IAdea,2C:C5:D3RuckusWi,2C:C8:1BRouterbo,2C:CA:0CWithusPl,2C:CC:15Nokia,2C:CC:44SonyInte,2C:CC:E6Skyworth,2C:CD:27Precor,2C:CD:43SummitTe,2C:CD:69Aqavicom,2C:CE:1ECloudtro,2C:CF:58HuaweiTe,2C:D0:2DCisco,2C:D0:5ALiteonTe,2C:D0:66XiaomiCo,2C:D1:41IEEERegi,2C:D1:41:00:00:00/28iCIRROUN,2C:D1:41:10:00:00/28Ezee,2C:D1:41:20:00:00/28IntelliL,2C:D1:41:30:00:00/28AOptixTe,2C:D1:41:40:00:00/28Shanghai,2C:D1:41:50:00:00/28Zenic,2C:D1:41:60:00:00/28BoweiTec,2C:D1:41:70:00:00/28XiaMen35,2C:D1:41:80:00:00/28Minno,2C:D1:41:90:00:00/28BeijingH,2C:D1:41:A0:00:00/28Fiberroa,2C:D1:41:B0:00:00/28ResusInd,2C:D1:41:C0:00:00/28PINSHANG,2C:D1:41:D0:00:00/28Square,2C:D1:41:E0:00:00/28CitaSmar,2C:D1:41:F0:00:00/28Private,2C:D1:DAKeysight,2C:D2:6BFn-LinkT,2C:D2:E3Guangzho,2C:D2:E7Nokia,2C:D4:44Fujitsu,2C:D9:74HuiZhouG,2C:DB:07IntelCor,2C:DC:ADWistronN,2C:DC:D7AzureWav,2C:DD:0CDiscover,2C:DD:95TaicangT,2C:DD:A3PointGre,2C:DD:E9AristaNe,2C:E0:32TCLKingE,2C:E2:A8DeviceDe,2C:E3:10Strataca,2C:E4:12Sagemcom,2C:E6:CCRuckusWi,2C:E8:71AlertMet,2C:EA:7FDell,2C:EA:DCAskeyCom,2C:ED:EBAlpheusD,2C:EE:26Petroleu,2C:F0:5DMicro-St,2C:F0:A2Apple,2C:F0:EEApple,2C:F1:BBzte,2C:F2:03EmkoElek,2C:F4:32Espressi,2C:F4:C5Avaya,2C:F7:F1SeeedTec,2C:F8:9BCisco,2C:FA:A2Alcatel-,2C:FC:E4CTEKSwed,2C:FD:37BlueCaly,2C:FD:A1ASUSTekC,2C:FD:ABMotorola,2C:FD:B3TCLTechn,2C:FF:65OkiElect,2C:FF:EEvivoMobi,2E:2E:2ELaaLocal,30:03:C8CloudNet,30:05:3FJTI,30:05:5CBrotheri,30:07:4DSamsungE,30:09:C0Motorola,30:09:F9IEEERegi,30:09:F9:00:00:00/28HurrayCl,30:09:F9:10:00:00/28Shenzhen,30:09:F9:20:00:00/28BeijingN,30:09:F9:30:00:00/28Microlin,30:09:F9:40:00:00/28PunktTro,30:09:F9:50:00:00/28Velsitec,30:09:F9:60:00:00/28BeijingM,30:09:F9:70:00:00/28Maytroni,30:09:F9:80:00:00/28essences,30:09:F9:90:00:00/28Bonraybi,30:09:F9:A0:00:00/28Shenzhen,30:09:F9:B0:00:00/28SichuanN,30:09:F9:C0:00:00/28Honeywel,30:09:F9:D0:00:00/28Technolo,30:09:F9:E0:00:00/28ZhongLiH,30:0A:60IEEERegi,30:0A:60:00:00:00/28KAZUtech,30:0A:60:10:00:00/28BeijingR,30:0A:60:20:00:00/28Advanced,30:0A:60:30:00:00/28Private,30:0A:60:40:00:00/28AvicJonh,30:0A:60:50:00:00/28A9,30:0A:60:60:00:00/28Realtime,30:0A:60:70:00:00/28Newtons4,30:0A:60:80:00:00/28Bronkhor,30:0A:60:90:00:00/28WINTEKSy,30:0A:60:A0:00:00/28Ampetron,30:0A:60:B0:00:00/28Giax,30:0A:60:C0:00:00/28ThermoPr,30:0A:60:D0:00:00/28SixthEne,30:0A:60:E0:00:00/28Imageosr,30:0A:C5Ruiotele,30:0B:9CDeltaMob,30:0C:23zte,30:0D:2AZhejiang,30:0D:43Microsof,30:0D:9ERuijieNe,30:0E:B8LGElectr,30:0E:D5HonHaiPr,30:0E:E3Aquantia,30:10:B3LiteonTe,30:10:E4Apple,30:13:89SiemensA,30:14:2DPiciorgr,30:14:4AWistronN,30:15:18Ubiquito,30:16:8DProLon,30:17:C8Sony,30:18:CFDEOScont,30:19:66SamsungE,30:1A:28MakoNetw,30:1A:30MakoNetw,30:1B:97LierdaSc,30:1F:9AIEEERegi,30:1F:9A:00:00:00/28IlsanEle,30:1F:9A:10:00:00/28Dewesoft,30:1F:9A:20:00:00/28CHISONMe,30:1F:9A:30:00:00/28Micomsof,30:1F:9A:40:00:00/28NCMSuppl,30:1F:9A:50:00:00/28BeijingS,30:1F:9A:60:00:00/28YiShengt,30:1F:9A:70:00:00/28Triax,30:1F:9A:80:00:00/28FineTriu,30:1F:9A:90:00:00/28Private,30:1F:9A:A0:00:00/28HunanCha,30:1F:9A:B0:00:00/28SmartCom,30:1F:9A:C0:00:00/28OrigamiG,30:1F:9A:D0:00:00/28OLIMEX,30:1F:9A:E0:00:00/28Shenzhen,30:21:5BShenzhen,30:23:03BelkinIn,30:24:32IntelCor,30:24:78Sagemcom,30:24:A9HP,30:27:CFPrivate,30:29:52Hillston,30:29:BEShanghai,30:2D:E8JDAJDA,30:31:7DHosiden,30:32:35QingdaoI,30:32:94W-IE-NE-,30:32:D4Hanilstm,30:33:35Boosty,30:34:D2Availink,30:35:ADApple,30:35:C5HuaweiDe,30:37:A6Cisco,30:37:B3HuaweiTe,30:38:55Nokia,30:39:26Sony,30:39:55Shenzhen,30:39:A9Hongshan,30:39:F2ADBBroad,30:3A:64IntelCor,30:3A:BAGuangzho,30:3D:08GlinttTe,30:3E:ADSonavoxC,30:3F:BBHewlettP,30:41:74AltecLan,30:42:25Burg-Wäc,30:42:40zte,30:42:A1ilumisys,30:44:49PLATH,30:44:87HefeiRad,30:44:A1Shanghai,30:45:11TexasIns,30:45:96HuaweiTe,30:46:9ANetgear,30:49:3BNanjingZ,30:49:50IEEERegi,30:49:50:00:00:00/28Guangzho,30:49:50:10:00:00/28AtliWorl,30:49:50:20:00:00/28Sercomm,30:49:50:30:00:00/28MorganSc,30:49:50:40:00:00/28Advanced,30:49:50:50:00:00/28IKElektr,30:49:50:60:00:00/28Curb,30:49:50:70:00:00/28Shenzhen,30:49:50:80:00:00/28Shenzhen,30:49:50:90:00:00/28Shanghai,30:49:50:A0:00:00/28Ledworks,30:49:50:B0:00:00/28Hangzhou,30:49:50:C0:00:00/28Anacove,30:49:50:D0:00:00/28MerlynMi,30:49:50:E0:00:00/28IoTmaxx,30:49:9EHuaweiTe,30:4A:26Shenzhen,30:4B:07Motorola,30:4C:7EPanasoni,30:4E:1BHuaweiDe,30:4E:C3TianjinT,30:4F:00Guangdon,30:4F:75DASANNet,30:50:75GNAudio,30:50:FDSkyworth,30:51:F8BYK-Gard,30:52:5ANST,30:52:CBLiteonTe,30:55:EDTrexNetw,30:56:84Shenzhen,30:56:96Infinixm,30:57:14Apple,30:57:8Eeero,30:57:ACIrlab,30:58:90Frontier,30:59:5Bstreamno,30:59:B7Microsof,30:5A:3AASUSTekC,30:5A:99SichuanT,30:5D:38Beissbar,30:5D:A6ADVALYSY,30:5F:77NewH3CTe,30:60:23ARRISGro,30:61:12PAV,30:61:18Paradom,30:63:6BApple,30:65:ECWistronC,30:66:D0HuaweiDe,30:68:8CReachTec,30:69:4BRim,30:6A:85SamsungE,30:6C:BESkymotio,30:6E:5CValidusT,30:6F:07NationsT,30:71:B2Hangzhou,30:73:50Inpeco,30:74:67SamsungE,30:74:96HuaweiTe,30:75:12Sony,30:76:6FLGElectr,30:77:CBMaikeInd,30:78:5CPartowTa,30:78:6BTIANJING,30:78:C2Innowire,30:78:D3Virgilan,30:7B:ACNewH3CTe,30:7B:C9Shenzhen,30:7C:30Rim,30:7C:4AHuaweiDe,30:7C:5EJuniperN,30:7C:B2AnovFran,30:7E:CBSfr,30:80:9BNewH3CTe,30:83:98Espressi,30:84:54Guangdon,30:85:A9ASUSTekC,30:85:EBFiberhom,30:86:2DAristaNe,30:87:30HuaweiTe,30:87:D9RuckusWi,30:88:41SichuanA,30:89:44DEVABroa,30:89:76DalianLa,30:89:99Guangdon,30:89:D3Hongkong,30:8A:F7HuaweiDe,30:8B:B2Cisco,30:8C:FBDropcam,30:8D:99HewlettP,30:8E:7AShenzhen,30:90:48Apple,30:90:ABApple,30:91:76Skyworth,30:91:8FTechnico,30:92:F6Shanghai,30:93:BCSagemcom,30:94:35vivoMobi,30:95:87HunanFn-,30:95:E3Shanghai,30:96:10HuaweiDe,30:96:FBSamsungE,30:99:35zte,30:9B:ADBbkEduca,30:9C:23Micro-St,30:9E:1DOhsung,30:9F:FBArdomusN,30:A0:23RockPath,30:A1:76Fiberhom,30:A1:FAHuaweiTe,30:A2:20ARGTelec,30:A2:43Shenzhen,30:A2:C2HuaweiDe,30:A4:52ArrivalE,30:A6:12ShenZhen,30:A8:89Decimato,30:A8:DBSony,30:A9:98HuaweiDe,30:A9:DELGInnote,30:AA:BDShanghai,30:AA:E4HuaweiDe,30:AB:6ASamsungE,30:AE:7BDeqingDu,30:AE:A4Espressi,30:AE:F6RadioMob,30:AF:CEvivoMobi,30:B0:37NewH3CTe,30:B1:64PowerEle,30:B1:B5Arcadyan,30:B2:16HitachiA,30:B2:37GDMideaA,30:B3:46CjscNors,30:B3:A2Shenzhen,30:B4:9ETp-LinkT,30:B4:B8LGElectr,30:B5:C2Tp-LinkT,30:B5:F1AitexinT,30:B6:2DMojoNetw,30:B6:4FJuniperN,30:B7:D4HitronTe,30:B9:30zte,30:B9:B0Intracom,30:BE:3BMitsubis,30:C0:1BShenzhen,30:C3:D9Alpsalpi,30:C5:07ECITelec,30:C5:0FHuaweiTe,30:C6:F7Espressi,30:C7:50MICTechn,30:C7:AESamsungE,30:C8:2AWI-BIZ,30:C9:ABCloudNet,30:CB:F8SamsungE,30:CC:21zte,30:CD:A7SamsungE,30:D0:42Dell,30:D1:6BLiteonTe,30:D1:7EHuaweiTe,30:D3:2Ddevolo,30:D3:57Logosol,30:D3:86zte,30:D4:6AAutosale,30:D5:87SamsungE,30:D6:59MergingT,30:D6:C9SamsungE,30:D9:41RaydiumS,30:D9:D9Apple,30:DE:86CedacSof,30:DF:8DShenzhen,30:E0:90Linctron,30:E1:71HewlettP,30:E2:83TexasIns,30:E3:7AIntelCor,30:E3:96HuaweiDe,30:E3:D6SpotifyU,30:E4:8EVodafone,30:E4:DBCisco,30:E9:8EHuaweiTe,30:EA:26Sycada,30:EB:1FSkylabM&,30:EB:25IntekDig,30:EB:5ALandis+G,30:EF:D1AlstomSt,30:F3:1Dzte,30:F3:35HuaweiTe,30:F3:3A#NOME?,30:F4:2FEsp,30:F6:B9Ecocentr,30:F7:0DCisco,30:F7:72HonHaiPr,30:F7:7FSMobileD,30:F7:C5Apple,30:F7:D7ThreadTe,30:F9:4BUniversa,30:F9:EDSony,30:FA:B7TunaiCre,30:FB:94Shanghai,30:FB:B8HuaweiTe,30:FC:68Tp-LinkT,30:FC:EBLGElectr,30:FD:11Macrotec,30:FD:38Google,30:FD:65HuaweiTe,30:FE:31Nokia,30:FF:F6HangZhou,34:00:8AIEEERegi,34:00:8A:00:00:00/28AngeeTec,34:00:8A:10:00:00/28ZQAMComm,34:00:8A:20:00:00/28RPEMonit,34:00:8A:30:00:00/28Globex99,34:00:8A:40:00:00/28Fotonici,34:00:8A:50:00:00/28FederalA,34:00:8A:60:00:00/28SithonTe,34:00:8A:70:00:00/28uberGARD,34:00:8A:80:00:00/28Shenzhen,34:00:8A:90:00:00/28KeruyunT,34:00:8A:A0:00:00/28Hibertek,34:00:8A:B0:00:00/28ProjectE,34:00:8A:C0:00:00/28Shenzhen,34:00:8A:D0:00:00/28ChengDuH,34:00:8A:E0:00:00/28Shenzhen,34:00:A3HuaweiTe,34:02:86IntelCor,34:02:9BPlexonic,34:03:DETexasIns,34:04:9EIEEERegi,34:04:9E:00:00:00/28GoChip,34:04:9E:10:00:00/28Connecte,34:04:9E:20:00:00/28EFDInduc,34:04:9E:30:00:00/28NanjingM,34:04:9E:40:00:00/28HarbinYa,34:04:9E:50:00:00/28SeeinerT,34:04:9E:60:00:00/28LifeInte,34:04:9E:70:00:00/28PebbleTe,34:04:9E:80:00:00/28EclipseI,34:04:9E:90:00:00/28ChurchDw,34:04:9E:A0:00:00/28i3Intern,34:04:9E:B0:00:00/28Eginity,34:04:9E:C0:00:00/28ClearCap,34:04:9E:D0:00:00/28uikismar,34:04:9E:E0:00:00/28NDSatCom,34:07:4FAccelSto,34:07:FBEricsson,34:08:04D-Link,34:08:BCApple,34:08:E1TexasIns,34:0A:22Top-Acce,34:0A:33D-LinkIn,34:0A:98HuaweiTe,34:0A:FFQingdaoH,34:0B:40MiosElet,34:0C:EDModuel,34:0F:66WebSensi,34:12:90Treeview,34:12:98Apple,34:12:F9HuaweiTe,34:13:A8Mediplan,34:13:E8IntelCor,34:14:5FSamsungE,34:14:B5TexasIns,34:15:13TexasIns,34:15:9EApple,34:17:EBDell,34:1A:35Fiberhom,34:1A:4CShenzhen,34:1B:22Grandbei,34:1C:F0XiaomiCo,34:1E:6BHuaweiTe,34:1F:E4ARRISGro,34:20:03Shenzhen,34:20:E3RuckusWi,34:21:09JensenSc,34:23:87HonHaiPr,34:23:BASamsungE,34:24:3Ezte,34:25:5DShenzhen,34:26:06CarePred,34:27:92FreeboxS,34:28:F0ATNInter,34:29:12HuaweiTe,34:29:8FIEEERegi,34:29:8F:00:00:00/28BlackEdg,34:29:8F:10:00:00/28ChengduM,34:29:8F:20:00:00/28Shenzhen,34:29:8F:30:00:00/28BeijingV,34:29:8F:40:00:00/28ISRAVisi,34:29:8F:50:00:00/28Highlite,34:29:8F:60:00:00/28BellmanS,34:29:8F:70:00:00/28Dongguan,34:29:8F:80:00:00/28NanjingS,34:29:8F:90:00:00/28Wiesheu,34:29:8F:A0:00:00/28VirtualT,34:29:8F:B0:00:00/28Schnick-,34:29:8F:C0:00:00/28AlbertHa,34:29:8F:D0:00:00/28Keystone,34:29:8F:E0:00:00/28ARCTechn,34:29:EAMcdElect,34:29:EFQingdaoH,34:2A:F1TexasIns,34:2B:70Arris,34:2C:C4CompalBr,34:2D:0DSamsungE,34:2E:B6HuaweiTe,34:2E:B7IntelCor,34:2F:6EAnywire,34:2F:BDNintendo,34:31:11SamsungE,34:31:7FPanasoni,34:31:8FApple,34:31:C4AVM,34:32:E6Panasoni,34:36:3BApple,34:36:54zte,34:37:59zte,34:37:94Hamee,34:38:AFInlabSof,34:38:B7HUMAX,34:3D:98JinQianM,34:3D:C4Buffalo,34:3E:A4Ring,34:40:B5Ibm,34:41:5DIntelCor,34:41:A8ER-Telec,34:42:62Apple,34:46:6FHiTEMEng,34:46:ECHuaweiDe,34:48:EDDell,34:49:5BSagemcom,34:4A:C3HuNanZiK,34:4B:3DFiberhom,34:4B:50zte,34:4C:A4amazipoi,34:4C:C8Echodyne,34:4D:EAzte,34:4D:F7LGElectr,34:4F:3FIO-Power,34:4F:5CR&amp;M,34:4F:69EkinopsS,34:51:80TCLKingE,34:51:84HuaweiDe,34:51:AAJidGloba,34:51:C9Apple,34:53:D2Sagemcom,34:54:3CTakaokaT,34:55:94FujianSt,34:56:FECiscoMer,34:57:60MitraSta,34:58:40HuaweiTe,34:58:7CMiraeInf,34:5A:06SHARP,34:5A:BAtcloudin,34:5B:11EviHeat,34:5B:98EMMicroe,34:5B:BBGDMideaA,34:5C:40CargtHol,34:5D:10Wytek,34:61:78Boeing,34:62:88Cisco,34:63:D4BionixSu,34:64:A9HewlettP,34:66:EAVertuInt,34:68:4ATerawork,34:68:93Tecnovid,34:68:95HonHaiPr,34:69:87zte,34:6A:C2HuaweiTe,34:6B:46Sagemcom,34:6B:5BNewH3CTe,34:6B:D3HuaweiTe,34:6C:0FPramodTe,34:6D:9CCarrier,34:6E:8AEcosense,34:6E:9DEricsson,34:6F:24AzureWav,34:6F:90Cisco,34:6F:92WhiteRod,34:6F:EDEnovatio,34:71:46HuaweiDe,34:73:2DCisco,34:73:5ADell,34:75:63Shenzhen,34:75:C7Avaya,34:76:C5I-ODataD,34:78:39zte,34:78:77O-NetCom,34:78:D7GioneeCo,34:79:16HuaweiTe,34:7A:60ARRISGro,34:7C:25Apple,34:7D:F6IntelCor,34:7E:00HuaweiDe,34:7E:39NokiaDan,34:7E:5CSonos,34:7E:CANextwill,34:80:0DCavium,34:80:B3XiaomiCo,34:81:37Unicard,34:81:C4AVM,34:81:F4SSTTaiwa,34:82:C5SamsungE,34:82:DEKiio,34:83:02iFORCOM,34:84:46Ericsson,34:84:E4TexasIns,34:85:84ExtremeN,34:86:2AHeinzLac,34:86:5DEspressi,34:87:3DQuectelW,34:88:5DLogitech,34:8A:12ArubaaHe,34:8A:7BSamsungE,34:8A:AESagemcom,34:8B:75LavaInte,34:8F:27RuckusWi,34:91:6FUserGate,34:93:42TTE,34:95:DBLogitec,34:96:72Tp-LinkT,34:97:F6ASUSTekC,34:97:FBAdvanced,34:98:B5Netgear,34:99:6FVPIEngin,34:99:71QuantaSt,34:99:D7Universa,34:9A:0DZBDDispl,34:9B:5BMaquet,34:9D:90Heinzman,34:9E:34Evervict,34:9F:7BCanon,34:A1:83AWare,34:A2:A2HuaweiTe,34:A3:95Apple,34:A3:BFTerewave,34:A5:5DTechnoso,34:A5:B4NavtechP,34:A5:E1Sensoris,34:A6:8CShinePro,34:A7:09Trevil,34:A7:BAFischerI,34:A8:43KYOCERAD,34:A8:4ECisco,34:A8:EBApple,34:AA:31Shenzhen,34:AA:8BSamsungE,34:AA:99Nokia,34:AA:EEMikrovis,34:AB:37Apple,34:AB:95Espressi,34:AD:E4Shanghai,34:AF:2CNintendo,34:AF:B3AmazonTe,34:B1:F7TexasIns,34:B2:0AHuaweiDe,34:B3:54HuaweiTe,34:B4:72Espressi,34:B5:71Plds,34:B5:A3CigShang,34:B7:FDGuangzho,34:B9:8DXiaomiCo,34:BA:38PalMohan,34:BA:51Se-KureC,34:BA:75EverestN,34:BA:9AAsiatelc,34:BB:1FBlackBer,34:BB:26Motorola,34:BC:A6BeijingD,34:BD:C8Cisco,34:BD:F9Shanghai,34:BD:FACiscoSPV,34:BE:00SamsungE,34:BF:90Fiberhom,34:C0:59Apple,34:C0:F9Rockwell,34:C1:03Hangzhou,34:C3:ACSamsungE,34:C3:D2Fn-LinkT,34:C5:D0Hagleitn,34:C6:9AEnecsys,34:C7:31Alpsalpi,34:C8:03Nokia,34:C9:3DIntelCor,34:C9:9DEidolonC,34:C9:F0LMTechno,34:CB:1AProcterG,34:CC:28Nexpring,34:CD:6DCommSkyT,34:CD:BEHuaweiTe,34:CE:00XIAOMIEl,34:CE:94Parsec,34:CF:F6IntelCor,34:D0:9BMobilMAX,34:D0:B8IEEERegi,34:D0:B8:00:00:00/28Captec,34:D0:B8:10:00:00/28Shenzhen,34:D0:B8:20:00:00/28Blustrea,34:D0:B8:30:00:00/28Tascent,34:D0:B8:40:00:00/28EQPlayIn,34:D0:B8:50:00:00/28eesy-inn,34:D0:B8:60:00:00/28NumberFo,34:D0:B8:70:00:00/28Shenzhen,34:D0:B8:80:00:00/28VtrekGro,34:D0:B8:90:00:00/28SkytechC,34:D0:B8:A0:00:00/28Meatests,34:D0:B8:B0:00:00/28Orosound,34:D0:B8:C0:00:00/28GloryMar,34:D0:B8:D0:00:00/28NTXEmbed,34:D0:B8:E0:00:00/28Kongqigu,34:D2:62SzDjiTec,34:D2:70AmazonTe,34:D2:C4RENAPrin,34:D6:93HuaweiDe,34:D7:12Smartisa,34:D7:72XiamenYu,34:D7:B4Tributar,34:D9:54WiBotic,34:DA:B7zte,34:DA:C1SAETechn,34:DB:9CSagemcom,34:DB:FDCisco,34:DD:7EUmeoxInn,34:DE:1AIntelCor,34:DE:34zte,34:DF:2AFujikonI,34:E0:CFzte,34:E0:D7Dongguan,34:E1:2DIntelCor,34:E1:D1IEEERegi,34:E1:D1:00:00:00/28TianjinS,34:E1:D1:10:00:00/28SAMANext,34:E1:D1:20:00:00/28TetonCam,34:E1:D1:30:00:00/28RincoUlt,34:E1:D1:40:00:00/28ASAInnov,34:E1:D1:50:00:00/28DokiTech,34:E1:D1:60:00:00/28NingboHu,34:E1:D1:70:00:00/28GeniusPr,34:E1:D1:80:00:00/28Hubitat,34:E1:D1:90:00:00/28Biamp,34:E1:D1:A0:00:00/28OrCamTec,34:E1:D1:B0:00:00/28APGCashD,34:E1:D1:C0:00:00/28CREWbyTr,34:E1:D1:D0:00:00/28Hi-TechO,34:E1:D1:E0:00:00/28Annapurn,34:E2:FDApple,34:E3:80Genexis,34:E3:DAHovalAkt,34:E4:2AAutomati,34:E5:ECPaloAlto,34:E6:ADIntelCor,34:E6:D7Dell,34:E7:0BHANNetwo,34:E7:1CShenzhen,34:E8:94Tp-LinkT,34:E9:11vivoMobi,34:E9:FEMetis,34:EA:34HangZhou,34:EA:E7Shanghai,34:ED:0BShanghai,34:ED:1BCisco,34:EF:442Wire,34:EF:8BNTTCommu,34:EF:B6Edgecore,34:F0:CAShenzhen,34:F1:50HuiZhouG,34:F3:9AIntelCor,34:F3:9BWizLAN,34:F6:2DSHARP,34:F6:4BIntelCor,34:F6:D2Panasoni,34:F7:16Tp-LinkT,34:F8:E7Cisco,34:F9:68ATEKProd,34:FA:40Guangzho,34:FA:9FRuckusWi,34:FC:6FAlcea,34:FC:A1Micronet,34:FC:B9HewlettP,34:FC:EFLGElectr,34:FD:6AApple,34:FE:9EFujitsu,34:FE:C5Shenzhen,38:00:25IntelCor,38:01:18ULVAC,38:01:46Shenzhen,38:01:95SamsungE,38:01:97TSSTGlob,38:01:9FShenzhen,38:02:DESercomm,38:05:46FoctekPh,38:05:ACPillerGr,38:06:B4ADC,38:07:D4Zeppelin,38:08:FDSilca,38:09:A4FireflyI,38:0A:0ASky-City,38:0A:94SamsungE,38:0A:ABFormlabs,38:0B:3CTexasIns,38:0B:40SamsungE,38:0D:D4PrimaxEl,38:0E:4DCisco,38:0E:7BVPSThai,38:0F:4AApple,38:0F:E4Dedicate,38:10:D5AVMAudio,38:10:F0ArubaaHe,38:14:28Dell,38:14:4EFiberhom,38:16:D1SamsungE,38:17:30UlrichLi,38:17:66Promzaka,38:17:C3HewlettP,38:17:E1Technico,38:18:4CSonyHome,38:19:2FNokia,38:1A:52SeikoEps,38:1C:1ACisco,38:1C:23HilanTec,38:1C:4ASIMComWi,38:1D:14Skydio,38:1D:D9Fn-LinkT,38:1F:8DTuyaSmar,38:20:28HuaweiTe,38:20:56Cisco,38:20:A8ColorTok,38:21:87MideaGro,38:21:C7ArubaaHe,38:22:9DADBBroad,38:22:D6Hangzhou,38:22:E2HP,38:25:6BMicrosof,38:26:2BUTranTec,38:26:CDAndtek,38:28:EAFujianNe,38:29:5AGuangdon,38:29:DDONvocal,38:2A:19Technica,38:2B:78EcoPlugs,38:2C:4AASUSTekC,38:2D:D1SamsungE,38:2D:E8SamsungE,38:30:F9LGElectr,38:31:ACWeg,38:35:FBSagemcom,38:37:8BHuaweiTe,38:38:4BvivoMobi,38:3A:21IEEERegi,38:3A:21:00:00:00/28R3CInfor,38:3A:21:10:00:00/28HOBART,38:3A:21:20:00:00/28Shenzhen,38:3A:21:30:00:00/28Shanghai,38:3A:21:40:00:00/28Dongguan,38:3A:21:50:00:00/28NPPUralt,38:3A:21:60:00:00/28Shenzhen,38:3A:21:70:00:00/28ChengduK,38:3A:21:80:00:00/28AlicatSc,38:3A:21:90:00:00/28SkylarkW,38:3A:21:A0:00:00/28Foresigh,38:3A:21:B0:00:00/28Pactron,38:3A:21:C0:00:00/28MissionE,38:3A:21:D0:00:00/28Colooc,38:3A:21:E0:00:00/28SDNwaret,38:3B:26JiangsuQ,38:3B:C82Wire,38:3C:9CFujianNe,38:3D:5BFiberhom,38:3F:10DBLTechn,38:3F:B3Technico,38:42:0BSonos,38:42:33Wildeboe,38:42:A6Ingenieu,38:43:69PatrolPr,38:43:7DCompalBr,38:43:E5Grotech,38:45:3BRuckusWi,38:45:4CLightLab,38:45:54HarmanBe,38:45:8CMyCloudT,38:46:08zte,38:47:BCHuaweiTe,38:48:4CApple,38:4B:24Siemens,38:4B:5BZtronTec,38:4B:76AIRTAMEA,38:4C:4FHuaweiTe,38:4C:90ARRISGro,38:4F:49JuniperN,38:4F:F0AzureWav,38:52:1ANokia,38:52:47HuaweiDe,38:53:9CApple,38:54:9Bzte,38:56:10CANDYHOU,38:56:3DMicrosof,38:56:B5Peerbrid,38:58:0CPanacces,38:59:F8MindMade,38:59:F9HonHaiPr,38:5A:A8BeijingZ,38:5C:76Plantron,38:5F:66CiscoSPV,38:5F:C3YuJeongS,38:60:77Pegatron,38:61:A5Grabango,38:63:BBHewlettP,38:63:F63NodMult,38:65:B2Apple,38:66:45OOSICTec,38:66:F0Apple,38:67:93AsiaOpti,38:68:93IntelCor,38:68:A4SamsungE,38:68:DDInventec,38:6A:77SamsungE,38:6B:1CShenzhen,38:6B:BBARRISGro,38:6C:9BIvyBiome,38:6E:21WasionGr,38:6E:88zte,38:6E:A2vivoMobi,38:70:0CARRISGro,38:71:DEApple,38:72:C0Comtrend,38:73:EAIEEERegi,38:73:EA:00:00:00/28L-3Commu,38:73:EA:10:00:00/28KingWayI,38:73:EA:20:00:00/28Eyesight,38:73:EA:30:00:00/28Prochpla,38:73:EA:40:00:00/28LightBlu,38:73:EA:50:00:00/28Istcontr,38:73:EA:60:00:00/28LiveSent,38:73:EA:70:00:00/28PingGPS,38:73:EA:80:00:00/28RockElec,38:73:EA:90:00:00/28Lightfor,38:73:EA:A0:00:00/28Shenzhen,38:73:EA:B0:00:00/28Shanghai,38:73:EA:C0:00:00/28LGElectr,38:73:EA:D0:00:00/28Annapurn,38:73:EA:E0:00:00/28Shenzhen,38:76:CAShenzhen,38:76:D1Euronda,38:78:62Sony,38:7A:0EIntelCor,38:7A:3CFiberhom,38:7B:47AKELA,38:80:DFMotorola,38:81:D7TexasIns,38:83:45Tp-LinkT,38:83:9AShenzhen,38:84:79CiscoMer,38:86:02Flexopti,38:87:D5IntelCor,38:88:1EHuaweiTe,38:89:2CApple,38:89:DCOpticonS,38:8A:B7ITCNetwo,38:8A:BEGuangdon,38:8B:59Google,38:8C:50LGElectr,38:8E:7AAutoit,38:8E:E7Fanhatta,38:90:52HuaweiTe,38:90:A5Cisco,38:91:D5Hangzhou,38:91:FBXenox,38:94:61RenesasE,38:94:96SamsungE,38:94:E0Syrotech,38:94:EDNetgear,38:95:92BeijingT,38:97:A4Elecom,38:97:D6Hangzhou,38:98:D8Meritech,38:98:E9HuaweiDe,38:9A:F6SamsungE,38:9D:92SeikoEps,38:9F:5AC-KurTV,38:9F:83OTN,38:A0:67NokiaSol,38:A2:8CShenzhen,38:A4:EDXiaomiCo,38:A5:3CCOMECERN,38:A5:B6Shenzhen,38:A6:59Sagemcom,38:A6:CEBSkyB,38:A8:51MoogIng,38:A8:6BOrga,38:A8:CDIEEERegi,38:A8:CD:00:00:00/28ACiiSTSm,38:A8:CD:10:00:00/28FujicaSy,38:A8:CD:20:00:00/28BeijingP,38:A8:CD:30:00:00/28Dongguan,38:A8:CD:40:00:00/28WHITEvoi,38:A8:CD:50:00:00/28RevoInfr,38:A8:CD:60:00:00/28cal4care,38:A8:CD:70:00:00/28Shenzhen,38:A8:CD:80:00:00/28MaxWayEl,38:A8:CD:90:00:00/28PTSupert,38:A8:CD:A0:00:00/28NICTechn,38:A8:CD:B0:00:00/28QingdaoH,38:A8:CD:C0:00:00/28BeijingA,38:A8:CD:D0:00:00/28Annapurn,38:A8:CD:E0:00:00/28Outform,38:A9:1CNewH3CTe,38:A9:5FActifio,38:A9:EAPrivate,38:AA:3CSamsungE,38:AC:3DNephos,38:AD:8ENewH3CTe,38:AD:BENewH3CTe,38:AF:29Zhejiang,38:AF:D0Nevro,38:AF:D7Fujitsu,38:B1:2DSonotron,38:B1:9EIEEERegi,38:B1:9E:00:00:00/28TripleJu,38:B1:9E:10:00:00/28Freedomp,38:B1:9E:20:00:00/28Hdanywhe,38:B1:9E:30:00:00/28AvoDevel,38:B1:9E:40:00:00/28BasalteB,38:B1:9E:50:00:00/28StarElec,38:B1:9E:60:00:00/28ThrustNe,38:B1:9E:70:00:00/28BeijingM,38:B1:9E:80:00:00/28BoCo,38:B1:9E:90:00:00/28DoepkeSc,38:B1:9E:A0:00:00/28Aeroespa,38:B1:9E:B0:00:00/28SystemQ,38:B1:9E:C0:00:00/28Gesellsc,38:B1:9E:D0:00:00/28DallasDe,38:B1:9E:E0:00:00/28ShenZhen,38:B1:DBHonHaiPr,38:B3:F7HuaweiDe,38:B4:D3BSHHausg,38:B5:4DApple,38:B5:BDEGOElekt,38:B5:D3SecuWork,38:B7:25WistronI,38:B7:4DFijowave,38:B8:00WistronN,38:B8:EBIEEERegi,38:B8:EB:00:00:00/28BumjinC&,38:B8:EB:10:00:00/281AConnec,38:B8:EB:20:00:00/28baroxKom,38:B8:EB:30:00:00/28AinaWire,38:B8:EB:40:00:00/28Umlogics,38:B8:EB:50:00:00/28Dojo-Lab,38:B8:EB:60:00:00/28Matrixst,38:B8:EB:70:00:00/28SirinMob,38:B8:EB:80:00:00/28CeeNex,38:B8:EB:90:00:00/28NHSSiste,38:B8:EB:A0:00:00/28Secad,38:B8:EB:B0:00:00/28ExaScale,38:B8:EB:C0:00:00/28Ajax,38:B8:EB:D0:00:00/28Yellowbr,38:B8:EB:E0:00:00/28WyresSAS,38:BA:B0Broadcom,38:BA:F8IntelCor,38:BB:23OzVision,38:BB:3CAvaya,38:BC:01HuaweiTe,38:BC:1AMEIZUTec,38:BE:ABAltoBeam,38:BF:2FEspec,38:BF:33NECCASIO,38:C0:96Alpsalpi,38:C2:BACctvNeot,38:C4:E8NSSSp,38:C7:0AWiFiSong,38:C7:BACSServic,38:C8:5CCiscoSPV,38:C9:86Apple,38:C9:A9SMARTHig,38:CA:73Shenzhen,38:CA:97ContourD,38:CA:DAApple,38:CD:07BeijingF,38:D1:35EasyIOSd,38:D2:69TexasIns,38:D2:CAZhejiang,38:D4:0BSamsungE,38:D5:47ASUSTekC,38:D5:7ACloudNet,38:D6:20LimideaC,38:D7:CA7HugsLab,38:D8:2Fzte,38:D9:A5MikotekI,38:DB:BBSunbowTe,38:DE:60Mohlenho,38:DE:ADIntelCor,38:E0:8EMitsubis,38:E1:AAzte,38:E2:6EShenZhen,38:E2:DDzte,38:E3:9FMotorola,38:E3:C5TaicangT,38:E5:95Shenzhen,38:E6:0AXiaomiCo,38:E7:D8HTC,38:E8:DFbmedien+,38:E8:EENanjingY,38:E9:8CReco,38:EA:A7HewlettP,38:EB:47HuaweiTe,38:EC:0DApple,38:EC:11NovatekM,38:EC:E4SamsungE,38:ED:18Cisco,38:EE:9DAnedo,38:EF:E3Ingenico,38:F0:98VaporSto,38:F0:C8Mevo,38:F1:35SensorTe,38:F2:3EMicrosof,38:F3:2ESkullcan,38:F3:3FTatsuno,38:F3:ABLCFCHeFe,38:F3:FBAsperiq,38:F5:54HisenseE,38:F5:57Jolata,38:F5:97home2net,38:F6:01SolidSta,38:F7:08National,38:F7:3DAmazonTe,38:F7:B2SeojunEl,38:F7:CDIEEERegi,38:F7:CD:00:00:00/28PolskaFa,38:F7:CD:10:00:00/28NZIAConn,38:F7:CD:20:00:00/28RIPower,38:F7:CD:30:00:00/28Vanguard,38:F7:CD:40:00:00/28NordiTel,38:F7:CD:50:00:00/28Shanghai,38:F7:CD:60:00:00/28FastCott,38:F7:CD:70:00:00/28Arunas,38:F7:CD:80:00:00/28BlastWav,38:F7:CD:90:00:00/28RFbeamMi,38:F7:CD:A0:00:00/28DistechC,38:F7:CD:B0:00:00/28Fibergat,38:F7:CD:C0:00:00/28Shenzhen,38:F7:CD:D0:00:00/28Macherey,38:F7:CD:E0:00:00/28AptMobil,38:F8:5EHUMAX,38:F8:89HuaweiTe,38:F8:B7V2ComPar,38:F8:CAOWIN,38:F9:D3Apple,38:FA:CASkyworth,38:FB:14HuaweiTe,38:FC:98IntelCor,38:FD:F8Cisco,38:FD:FEIEEERegi,38:FD:FE:00:00:00/28EdgeI&D,38:FD:FE:10:00:00/28WaytoneB,38:FD:FE:20:00:00/28SmartSol,38:FD:FE:30:00:00/28SiemensP,38:FD:FE:40:00:00/28NewTelec,38:FD:FE:50:00:00/28CaptiveA,38:FD:FE:60:00:00/28Inspero,38:FD:FE:70:00:00/28Rademach,38:FD:FE:80:00:00/28IndraNav,38:FD:FE:90:00:00/28GroupInd,38:FD:FE:A0:00:00/28Manageme,38:FD:FE:B0:00:00/28SwedishA,38:FD:FE:C0:00:00/28NewGarde,38:FD:FE:D0:00:00/28FUBAAuto,38:FD:FE:E0:00:00/28iSmartel,38:FE:C5Ellips,38:FF:13JointSto,38:FF:36RuckusWi,3A:02:92VideoEle,3A:35:41Raspberr,3A:5A:27Shenzhen,3A:5E:03CirrusDa,3A:78:CASoundTal,3A:A3:F8IEEE8021,3A:BA:37Cirrent,3A:CF:C5Quicklin,3A:F5:21SingleDi,3C:00:003Com,3C:01:EFSony,3C:02:B1Creation,3C:04:61ARRISGro,3C:04:BFPRAVIS,3C:05:18SamsungE,3C:05:ABProductC,3C:06:30Apple,3C:06:A7Tp-LinkT,3C:07:54Apple,3C:07:71Sony,3C:08:1EBeijingY,3C:08:F6Cisco,3C:09:6DPowerhou,3C:0C:48Servergy,3C:0C:7DTinyMesh,3C:0C:DBUnionman,3C:0E:23Cisco,3C:0F:C1KBCNetwo,3C:10:40daesungn,3C:10:6FAlbahith,3C:10:E6PHAZR,3C:11:B2Fraunhof,3C:13:CCCisco,3C:15:12Shenzhen,3C:15:C2Apple,3C:15:EATescom,3C:15:FBHuaweiTe,3C:17:10Sagemcom,3C:18:9FNokia,3C:18:A0Luxshare,3C:19:15GFIChron,3C:19:5ESamsungE,3C:19:7DEricsson,3C:1A:0FClearSky,3C:1A:57Cardiopu,3C:1A:79HuayuanT,3C:1A:9EVitalThi,3C:1C:BEJadak,3C:1E:04D-LinkIn,3C:1E:13Hangzhou,3C:20:93GDMideaA,3C:20:F6SamsungE,3C:21:9CIntelCor,3C:22:FBApple,3C:24:F0IEEERegi,3C:24:F0:00:00:00/28Shenzhen,3C:24:F0:10:00:00/28Abrites,3C:24:F0:20:00:00/28LaipacTe,3C:24:F0:30:00:00/28Wisycom,3C:24:F0:40:00:00/28Inter-Co,3C:24:F0:50:00:00/28CASKYeTe,3C:24:F0:60:00:00/28InterAct,3C:24:F0:70:00:00/28Swissdot,3C:24:F0:80:00:00/28SivatTec,3C:24:F0:90:00:00/28Siemens-,3C:24:F0:A0:00:00/28Shenzhen,3C:24:F0:B0:00:00/28Comatis,3C:24:F0:C0:00:00/28Authenti,3C:24:F0:D0:00:00/28Travis,3C:24:F0:E0:00:00/28Getmobit,3C:25:D7Nokia,3C:26:D5SoteraWi,3C:27:63SLEquali,3C:28:6DGoogle,3C:28:A6Alcatel-,3C:2A:F4BrotherI,3C:2C:30Dell,3C:2C:94杭州德澜科技有限,3C:2C:99Edgecore,3C:2D:B7TexasIns,3C:2E:F9Apple,3C:2E:FFApple,3C:2F:3ASFORZATO,3C:30:0CDewarEle,3C:30:6FHuaweiTe,3C:31:78Qolsys,3C:33:00Shenzhen,3C:35:56Cognitec,3C:36:3DNokia,3C:36:E4ARRISGro,3C:37:12AVMAudio,3C:37:86Netgear,3C:38:88ConnectQ,3C:38:F4Sony,3C:39:C3JWElectr,3C:39:E7IEEERegi,3C:39:E7:00:00:00/28Hannstar,3C:39:E7:10:00:00/28BEWATECK,3C:39:E7:20:00:00/28HomeWiza,3C:39:E7:30:00:00/28ELSAJapa,3C:39:E7:40:00:00/28Universi,3C:39:E7:50:00:00/28Attrackt,3C:39:E7:60:00:00/28ROVERLab,3C:39:E7:70:00:00/28Sensorto,3C:39:E7:80:00:00/28MartemAS,3C:39:E7:90:00:00/28ZoneCont,3C:39:E7:A0:00:00/28iiM,3C:39:E7:B0:00:00/28chipsgui,3C:39:E7:C0:00:00/28Vanstone,3C:39:E7:E0:00:00/28Marposs,3C:39:E7:F0:00:00/28Private,3C:3A:73Avaya,3C:3F:512Crsi,3C:40:4FGuangdon,3C:41:0ECisco,3C:42:7EIEEERegi,3C:42:7E:00:00:00/28Grandway,3C:42:7E:10:00:00/28Dongguan,3C:42:7E:20:00:00/28Starloop,3C:42:7E:30:00:00/28Shenzhen,3C:42:7E:40:00:00/28Teknowar,3C:42:7E:50:00:00/28GeoplanK,3C:42:7E:60:00:00/28Edit,3C:42:7E:70:00:00/28GJS,3C:42:7E:80:00:00/28UbtechRo,3C:42:7E:90:00:00/28Taitex,3C:42:7E:A0:00:00/28snap40,3C:42:7E:B0:00:00/28CompalEl,3C:42:7E:C0:00:00/28PrivacyL,3C:42:7E:D0:00:00/28RoboxSma,3C:42:7E:E0:00:00/28Xiaoniun,3C:43:8EARRISGro,3C:45:7ABSkyB,3C:46:D8Tp-LinkT,3C:47:11HuaweiTe,3C:47:9BTheissen,3C:49:37ASSMANNE,3C:4A:92HewlettP,3C:4C:69Infinity,3C:4C:D0CeragonN,3C:4D:BEApple,3C:4E:47Etronic,3C:51:0ECisco,3C:52:82HewlettP,3C:53:D7Cedes,3C:54:47HuaweiTe,3C:55:76CloudNet,3C:57:31Cisco,3C:57:4FChinaMob,3C:57:6CSamsungE,3C:57:BDKesslerC,3C:57:D5FiveCo,3C:58:C2IntelCor,3C:59:1ETCLKingE,3C:5A:37SamsungE,3C:5A:B4Google,3C:5C:C3Shenzhen,3C:5C:C4AmazonTe,3C:5C:F1eero,3C:5E:C3Cisco,3C:5F:01Synerchi,3C:61:04JuniperN,3C:61:05Espressi,3C:62:00SamsungE,3C:62:78Shenzhen,3C:62:F0Sercomm,3C:67:16LilyRobo,3C:67:2CSciovid,3C:67:8CHuaweiTe,3C:68:16VXi,3C:6A:2CIEEERegi,3C:6A:2C:00:00:00/28RioLagoT,3C:6A:2C:10:00:00/28Olibra,3C:6A:2C:20:00:00/28BoschAut,3C:6A:2C:30:00:00/28figur8,3C:6A:2C:40:00:00/28XiAnYepT,3C:6A:2C:50:00:00/28Qingdaoi,3C:6A:2C:60:00:00/28LaBarriè,3C:6A:2C:70:00:00/28Homegear,3C:6A:2C:80:00:00/28TPRadio,3C:6A:2C:90:00:00/28WICKS,3C:6A:2C:A0:00:00/28Metro,3C:6A:2C:B0:00:00/28PhytiumT,3C:6A:2C:C0:00:00/28EltovSys,3C:6A:2C:D0:00:00/28XiamenSm,3C:6A:2C:E0:00:00/28BeijingD,3C:6A:7DNiigataP,3C:6A:9DDexatekT,3C:6A:A7IntelCor,3C:6E:63Mitron,3C:6F:45Fiberpro,3C:6F:EAPanasoni,3C:6F:F7EnTek,3C:70:59MakerBot,3C:71:BFEspressi,3C:74:37Rim,3C:75:4AARRISGro,3C:77:E6HonHaiPr,3C:78:43HuaweiTe,3C:78:73Airsonic,3C:7A:8AARRISGro,3C:7A:AAChinaDra,3C:7A:C4Chemtron,3C:7A:F0ItelMobi,3C:7C:3FASUSTekC,3C:7D:0AApple,3C:7D:B1TexasIns,3C:7F:6FTelechip,3C:80:6BHunanVoc,3C:80:AARansnetS,3C:81:D8Sagemcom,3C:83:1ECKD,3C:83:75Microsof,3C:83:B5AdvanceV,3C:84:6ATp-LinkT,3C:86:9AHuaweiTe,3C:86:A8Sangshin,3C:86:D1vivoMobi,3C:89:4DDrInghcF,3C:89:70Neosfar,3C:89:94BSkyB,3C:89:A6Kapelse,3C:8A:B0JuniperN,3C:8A:E5TensunIn,3C:8B:CDAlcatel-,3C:8B:FESamsungE,3C:8C:40Hangzhou,3C:8C:93JuniperN,3C:8C:F8TRENDnet,3C:8D:20Google,3C:8F:06Shenzhen,3C:90:66SmartRG,3C:91:2BVexata,3C:91:57YulongCo,3C:91:74AlongCom,3C:91:80LiteonTe,3C:92:DCOctopodT,3C:93:F4HuaweiTe,3C:94:D5JuniperN,3C:95:09LiteonTe,3C:97:0EWistronI,3C:97:7EIPSTechn,3C:98:72Sercomm,3C:98:BFQuestCon,3C:99:F7Lansente,3C:9A:77Technico,3C:9B:C6HuaweiDe,3C:9B:D6Vizio,3C:9C:0FIntelCor,3C:9D:56HuaweiTe,3C:9E:C7BSkyB,3C:9F:81Shenzhen,3C:9F:C3BeijingS,3C:A0:67LiteonTe,3C:A1:0DSamsungE,3C:A3:08TexasIns,3C:A3:15BlessInf,3C:A3:1AOilfindI,3C:A3:48vivoMobi,3C:A3:7EHuaweiTe,3C:A5:81vivoMobi,3C:A6:16vivoMobi,3C:A6:2FAVMAudio,3C:A6:F6Apple,3C:A7:2BMRVCommu,3C:A8:2AHewlettP,3C:A8:EDsmartlig,3C:A9:16HuaweiDe,3C:A9:F4IntelCor,3C:AA:3FiKey,3C:AB:8EApple,3C:AE:69ESAElekt,3C:B1:5BAvaya,3C:B1:7FWattwatc,3C:B2:33HuaweiDe,3C:B5:3DHunanGok,3C:B6:B7vivoMobi,3C:B7:2BPLUMgrid,3C:B7:4BTechnico,3C:B7:92HitachiM,3C:B8:7APrivate,3C:B9:A6BeldenDe,3C:BB:73Shenzhen,3C:BB:FDSamsungE,3C:BD:3EBeijingX,3C:BD:C5Arcadyan,3C:BD:D8LgElectr,3C:BE:E1Nikon,3C:BF:60Apple,3C:C0:79Shenzhen,3C:C0:C6d&baudio,3C:C1:2CAES,3C:C1:F6Melange,3C:C2:43Nokia,3C:C2:E1XinhuaCo,3C:C7:86Dongguan,3C:C9:9EHuiyangT,3C:CA:87Iders,3C:CB:7CTCTmobil,3C:CD:36Apple,3C:CD:57BeijingX,3C:CD:5ATechnisc,3C:CD:5DHuaweiTe,3C:CD:93LgElectr,3C:CE:15Mercedes,3C:CE:73Cisco,3C:CF:5BIcommHk,3C:D0:F8Apple,3C:D1:6ETelepowe,3C:D2:E5NewH3CTe,3C:D4:D6Wireless,3C:D7:DASKMtekmi,3C:D9:2BHewlettP,3C:D9:CEEclipseW,3C:DA:2Azte,3C:DA:6DTiandyTe,3C:DC:BCSamsungE,3C:DD:89SomoHold,3C:DF:1ECisco,3C:DF:A9ARRISGro,3C:DF:BDHuaweiTe,3C:E0:38Plumeria,3C:E0:72Apple,3C:E1:A1Universa,3C:E3:6BZhejiang,3C:E3:E7ChinaMob,3C:E4:B0TexasIns,3C:E5:A6Hangzhou,3C:E5:B4KidasenI,3C:E6:24LGDispla,3C:E8:24HuaweiTe,3C:EA:4F2Wire,3C:EA:F9Jubixcol,3C:EA:FBNse,3C:EC:EFSuperMic,3C:EF:8CZhejiang,3C:F0:11IntelCor,3C:F3:92Virtualt,3C:F4:F9Moda-Inn,3C:F5:2CDSPECIAL,3C:F5:91Guangdon,3C:F5:CCNewH3CTe,3C:F6:52zte,3C:F7:2ANokia,3C:F7:48Shenzhen,3C:F7:A4SamsungE,3C:F8:08HuaweiTe,3C:F8:62IntelCor,3C:FA:43HuaweiTe,3C:FA:D3IEEERegi,3C:FA:D3:00:00:00/28HomeCont,3C:FA:D3:10:00:00/28Annapurn,3C:FA:D3:20:00:00/28NaruidaT,3C:FA:D3:30:00:00/28HarmanCo,3C:FA:D3:40:00:00/28GRGBanki,3C:FA:D3:50:00:00/28GulfSecu,3C:FA:D3:60:00:00/28NoxMedic,3C:FA:D3:70:00:00/28LIPS,3C:FA:D3:80:00:00/28Energous,3C:FA:D3:90:00:00/28Shenzhen,3C:FA:D3:A0:00:00/28UltiMach,3C:FA:D3:B0:00:00/28Corelink,3C:FA:D3:C0:00:00/28Shenzhen,3C:FA:D3:D0:00:00/28AMobileS,3C:FA:D3:E0:00:00/28Mirico,3C:FB:5CFiberhom,3C:FB:96Emcraft,3C:FD:FEIntelCor,3C:FF:D8HuaweiTe,40:00:03NetWare?,40:00:E0DerekSha,40:01:07Arista,40:01:7ACisco,40:01:C63ComEuro,40:04:0CA&T,40:05:89T-Mobile,40:06:34HuaweiDe,40:06:A0TexasIns,40:06:D5Cisco,40:07:C0Railtec,40:0D:10ARRISGro,40:0E:67Tremol,40:0E:85SamsungE,40:11:75IEEERegi,40:11:75:00:00:00/28LexiDevi,40:11:75:10:00:00/28FujianKu,40:11:75:20:00:00/28KandaKog,40:11:75:30:00:00/28BeijingH,40:11:75:40:00:00/28TableTra,40:11:75:50:00:00/28MircElec,40:11:75:60:00:00/28ShenZhen,40:11:75:70:00:00/28Guangzho,40:11:75:80:00:00/28BeijingG,40:11:75:90:00:00/28ADHGuard,40:11:75:A0:00:00/28BWTTianj,40:11:75:B0:00:00/28Chongqin,40:11:75:C0:00:00/28disguise,40:11:75:D0:00:00/28NanJingH,40:11:75:E0:00:00/28Nibble,40:11:C3SamsungE,40:11:DCSonance,40:12:E4Compass-,40:13:D9GlobalES,40:14:ADHuaweiDe,40:15:97ProtectA,40:16:3BSamsungE,40:16:7EASUSTekC,40:16:9FTp-LinkT,40:16:FAEKMMeter,40:17:E2IntaiTec,40:18:B1ExtremeN,40:18:D7Smartron,40:19:20Movon,40:1B:5FWeifangG,40:1C:83IntelCor,40:1D:59Biometri,40:22:EDDigitalP,40:23:43Chongqin,40:24:B2SichuanA,40:25:C2IntelCor,40:26:19Apple,40:27:0BMobileec,40:28:14RFIEngin,40:2B:50ARRISGro,40:2B:69KumhoEle,40:2B:A1Sony,40:2C:76IEEERegi,40:2C:76:00:00:00/28Lista,40:2C:76:10:00:00/28Shanghai,40:2C:76:20:00:00/28Annapurn,40:2C:76:40:00:00/28BeijingS,40:2C:76:50:00:00/28BaumerBo,40:2C:76:60:00:00/28Guangzho,40:2C:76:70:00:00/28Zhejiang,40:2C:76:80:00:00/28SutengIn,40:2C:76:90:00:00/28Annapurn,40:2C:76:A0:00:00/28NowTechn,40:2C:76:B0:00:00/28BeijingK,40:2C:76:C0:00:00/28gridX,40:2C:76:D0:00:00/28Guangzho,40:2C:76:E0:00:00/28LSEnergy,40:2C:F4Universa,40:2E:28MiXTelem,40:2E:71TexasIns,40:2F:86LGInnote,40:30:04Apple,40:30:67Conlog,40:31:3CXIAOMIEl,40:33:1AApple,40:33:6CGodrejBo,40:37:ADMacroIma,40:3C:FCApple,40:3D:ECHUMAX,40:3F:8CTp-LinkT,40:40:22Ziv,40:40:28Ziv,40:40:6BIcomera,40:40:6CIcomera,40:40:A7Sony,40:41:01Rockwell,40:42:29Layer3TV,40:44:FDRealmeCh,40:45:DASpreadtr,40:47:6AAstroGam,40:48:FDIEEERegi,40:48:FD:00:00:00/28BeijingC,40:48:FD:10:00:00/28FastProg,40:48:FD:20:00:00/28MITHRAST,40:48:FD:30:00:00/28RLContro,40:48:FD:40:00:00/28DynamicE,40:48:FD:50:00:00/2852ndRese,40:48:FD:60:00:00/28SwarcoTe,40:48:FD:70:00:00/28Cloud4Wi,40:48:FD:80:00:00/28DorelJuv,40:48:FD:90:00:00/28PlusOneG,40:48:FD:A0:00:00/28Shenzhen,40:48:FD:B0:00:00/28MagentaL,40:48:FD:C0:00:00/28Ecotap,40:48:FD:D0:00:00/28NOX,40:48:FD:E0:00:00/28SmartSen,40:49:0FHonHaiPr,40:49:8ASynaptic,40:4A:03ZyxelCom,40:4A:18AddrekSm,40:4A:D4Widex,40:4C:77ARRISGro,40:4D:7FApple,40:4D:8EHuaweiTe,40:4E:36HTC,40:4E:EBHigherWa,40:50:B5Shenzhen,40:50:E0MiltonSe,40:51:6CGrandexI,40:52:0DPicoTech,40:54:E4Wearsafe,40:55:39Cisco,40:55:82Nokia,40:56:0CInHomeDi,40:56:2DSmartron,40:56:62GuoTengS,40:58:99Logitech,40:5A:9BAnovo,40:5B:D8Chongqin,40:5C:FDDell,40:5D:82Netgear,40:5E:E1Shenzhen,40:5F:7DTCTmobil,40:5F:BERim,40:5F:C2TexasIns,40:60:5AHawkeyeT,40:61:86Micro-St,40:61:8EStella-G,40:62:31Gifa,40:62:34TelinkSe,40:62:B6Telesyst,40:62:EAChinaMob,40:64:A4Furukawa,40:65:A3Sagemcom,40:66:7Amediola-,40:68:26ThalesUK,40:6A:8EHangzhou,40:6A:ABRim,40:6C:8FApple,40:6F:2ABlackBer,40:70:09ARRISGro,40:70:4APowerIde,40:70:74LifeTech,40:70:F5Apple,40:71:83JuniperN,40:74:96aFUNTECH,40:74:E0IntelCor,40:77:A9NewH3CTe,40:78:6AMotorola,40:78:75IMBEL-In,40:7A:80Nokia,40:7B:1BMettleNe,40:7C:7DNokia,40:7D:0FHuaweiTe,40:7F:E0GlorySta,40:82:56Continen,40:83:1DApple,40:83:DEZebraTec,40:84:93Claviste,40:86:2EJdmMobil,40:88:05Motorola,40:88:2FExtremeN,40:88:E0BeijingE,40:89:A8WiredIQ,40:8A:9ATITENG,40:8B:07Actionte,40:8B:F6Shenzhen,40:8C:1FGuangdon,40:8C:4CShenzhen,40:8D:5CGiga-Byt,40:8F:9DJuniperN,40:91:51Espressi,40:95:05AcoinfoT,40:95:58Aisino,40:95:BDNTmore,40:97:D1BKElectr,40:98:4CCasacomS,40:98:4ETexasIns,40:98:7BAisino,40:98:ADApple,40:99:22AzureWav,40:9B:0DShenzhen,40:9B:21Nokia,40:9B:CDD-LinkIn,40:9C:28Apple,40:9C:A6Curvalux,40:9F:38AzureWav,40:9F:87JideTech,40:9F:C7BAEKCHUN,40:A1:08Motorola,40:A2:DBAmazonTe,40:A3:6B:00:00:00/28FinRobot,40:A3:6B:10:00:00/28TW-TeamW,40:A3:6B:20:00:00/28TOPROOTT,40:A3:6B:30:00:00/28Omnitrac,40:A3:6B:40:00:00/28SKS-Kink,40:A3:6B:50:00:00/28National,40:A3:6B:60:00:00/28Securito,40:A3:6B:70:00:00/28Pella,40:A3:6B:80:00:00/28SFT,40:A3:6B:90:00:00/28PHTechni,40:A3:6B:A0:00:00/28Embrioni,40:A3:6B:B0:00:00/28AmobileI,40:A3:6B:C0:00:00/28Onion,40:A3:6B:D0:00:00/28FAOD,40:A3:CCIntelCor,40:A5:EFShenzhen,40:A6:77JuniperN,40:A6:A4PassivSy,40:A6:B7IntelCor,40:A6:D9Apple,40:A6:E8Cisco,40:A8:F0HewlettP,40:A9:3FPivotalC,40:A9:CFAmazonTe,40:AA:56ChinaDra,40:AC:8DDataMana,40:AC:BFHangzhou,40:B0:34HewlettP,40:B0:76ASUSTekC,40:B0:A1Valcom,40:B0:FALGElectr,40:B2:C8NortelNe,40:B3:0EIntegrat,40:B3:1EUniversa,40:B3:95Apple,40:B3:CDChiyodaE,40:B3:FCLogital,40:B4:CDAmazonTe,40:B4:F0JuniperN,40:B5:C1Cisco,40:B6:88LEGICIde,40:B6:B1SUNGSAM,40:B6:E7HuaweiDe,40:B7:F3ARRISGro,40:B8:37Sony,40:B8:9AHonHaiPr,40:B9:3CHewlettP,40:BA:61ARIMACom,40:BC:60Apple,40:BC:68WuhanFun,40:BC:73Cronopla,40:BC:8Bitelio,40:BD:32TexasIns,40:BD:9EPhysio-C,40:BE:EEShenzhen,40:BF:17Digistar,40:C2:45Shenzhen,40:C3:C6SnapRout,40:C4:8CN-iTUS,40:C4:D6ChongQin,40:C6:2AShanghai,40:C7:11Apple,40:C7:29Sagemcom,40:C7:C9Naviit,40:C8:1FShenzhen,40:C8:CBAMTeleco,40:CA:63SeongjiI,40:CB:A8HuaweiTe,40:CB:C0Apple,40:CD:3AZ3Techno,40:CD:7AQingdaoH,40:CE:24Cisco,40:D2:5FItelMobi,40:D2:8ANintendo,40:D3:2DApple,40:D3:57IsonTech,40:D3:AESamsungE,40:D4:0EBiodata,40:D4:BDSKNetwor,40:D5:59MicroSER,40:D6:3CEquitech,40:D8:55IEEERegi,40:D8:55:00:00:00/36Xronos,40:D8:55:00:10:00/36Vemotion,40:D8:55:00:20:00/36Hangzhou,40:D8:55:00:30:00/36AlphaNav,40:D8:55:00:40:00/36CRMagnet,40:D8:55:00:50:00/36MonarchI,40:D8:55:00:60:00/36Bactest,40:D8:55:00:70:00/36DigitalA,40:D8:55:00:80:00/36KaoriInd,40:D8:55:00:90:00/36ClearSit,40:D8:55:00:A0:00/36SaranaSi,40:D8:55:00:B0:00/36GogoBA,40:D8:55:00:C0:00/36AplexTec,40:D8:55:00:D0:00/36HuNS,40:D8:55:00:E0:00/36Brightwe,40:D8:55:00:F0:00/36DigitalD,40:D8:55:01:00:00/36APGCashD,40:D8:55:01:10:00/36FleximSe,40:D8:55:01:20:00/36Sencon,40:D8:55:01:30:00/36GrandeVi,40:D8:55:01:40:00/36ToniStud,40:D8:55:01:50:00/36BITMILL,40:D8:55:01:60:00/36Par-Tech,40:D8:55:01:70:00/36FrankeAq,40:D8:55:01:80:00/36StaneoSa,40:D8:55:01:90:00/36Nautel,40:D8:55:01:A0:00/36MeggittD,40:D8:55:01:B0:00/36AudioEnh,40:D8:55:01:C0:00/36BERGClou,40:D8:55:01:D0:00/36ScharcoE,40:D8:55:01:E0:00/36A2S,40:D8:55:01:F0:00/36SitepIta,40:D8:55:02:00:00/36ENTECEle,40:D8:55:02:10:00/36SmtDOO,40:D8:55:02:20:00/36Digimerg,40:D8:55:02:30:00/36Shanghai,40:D8:55:02:40:00/36Electric,40:D8:55:02:50:00/36Rosemoun,40:D8:55:02:60:00/36Symetric,40:D8:55:02:70:00/36GrupoEpe,40:D8:55:02:80:00/36Integrat,40:D8:55:02:90:00/36DeproEle,40:D8:55:02:A0:00/36Tinkerfo,40:D8:55:02:B0:00/36Nomatron,40:D8:55:02:C0:00/36InventLa,40:D8:55:02:D0:00/36ElgamaSi,40:D8:55:02:E0:00/36Circuite,40:D8:55:02:F0:00/36AdvatekL,40:D8:55:03:00:00/36Tecnolog,40:D8:55:03:10:00/36Dommel,40:D8:55:03:20:00/36Bettini,40:D8:55:03:30:00/36ErmesEle,40:D8:55:03:40:00/36DacomWes,40:D8:55:03:50:00/36Mesotech,40:D8:55:03:60:00/36Schweers,40:D8:55:03:70:00/36Software,40:D8:55:03:80:00/36SpecialM,40:D8:55:03:90:00/36CI,40:D8:55:03:A0:00/36Socusnet,40:D8:55:03:B0:00/36Telcomko,40:D8:55:03:C0:00/36Computer,40:D8:55:03:D0:00/36TekelekE,40:D8:55:03:E0:00/36VishayCe,40:D8:55:03:F0:00/36UniSVRGl,40:D8:55:04:00:00/36GHLBerha,40:D8:55:04:10:00/36TQMItaca,40:D8:55:04:20:00/36MangoCom,40:D8:55:04:30:00/36SchulerC,40:D8:55:04:40:00/36AnChenCo,40:D8:55:04:50:00/36Genadsys,40:D8:55:04:60:00/36Circuitl,40:D8:55:04:70:00/36Dos&Dont,40:D8:55:04:80:00/36GDMissio,40:D8:55:04:90:00/36ThermoFi,40:D8:55:04:A0:00/36GatewayT,40:D8:55:04:B0:00/36VitalTec,40:D8:55:04:C0:00/36Serveron,40:D8:55:04:D0:00/36MACHINEP,40:D8:55:04:E0:00/36Honeywel,40:D8:55:04:F0:00/36HaeinS&S,40:D8:55:05:00:00/36ATGUVTec,40:D8:55:05:10:00/36CSInstru,40:D8:55:05:20:00/36DanElect,40:D8:55:05:30:00/36Amantys,40:D8:55:05:40:00/36Vitec,40:D8:55:05:50:00/36Helmholt,40:D8:55:05:60:00/36Group57,40:D8:55:05:70:00/36Tammerma,40:D8:55:05:80:00/36EnergyTe,40:D8:55:05:90:00/36COLONIAL,40:D8:55:05:A0:00/36UltraEle,40:D8:55:05:B0:00/36DataFlow,40:D8:55:05:C0:00/36Rosslare,40:D8:55:05:D0:00/36LeicaBio,40:D8:55:05:E0:00/36inoage,40:D8:55:05:F0:00/36EPSa,40:D8:55:06:00:00/36AplexTec,40:D8:55:06:10:00/36Cominfo,40:D8:55:06:20:00/36TechSour,40:D8:55:06:30:00/36Protonic,40:D8:55:06:40:00/36Hipodrom,40:D8:55:06:50:00/36Parallel,40:D8:55:06:60:00/36TeraTron,40:D8:55:06:70:00/36TronicCo,40:D8:55:06:80:00/36OkiSeate,40:D8:55:06:90:00/36Smartcom,40:D8:55:06:A0:00/36elgrisUG,40:D8:55:06:B0:00/36BRSSiste,40:D8:55:06:C0:00/36Rohde&Sc,40:D8:55:06:D0:00/36BroadSof,40:D8:55:06:E0:00/36C-COMSat,40:D8:55:06:F0:00/36DorletSa,40:D8:55:07:00:00/36Electric,40:D8:55:07:10:00/36Tattile,40:D8:55:07:20:00/36CT,40:D8:55:07:30:00/36DiamondT,40:D8:55:07:40:00/36SphereMe,40:D8:55:07:50:00/36Teraflop,40:D8:55:07:60:00/36Internet,40:D8:55:07:70:00/36ToecTech,40:D8:55:07:80:00/36Nachi-Fu,40:D8:55:07:90:00/36DelfiSol,40:D8:55:07:A0:00/364embedde,40:D8:55:07:B0:00/36IPSTechn,40:D8:55:07:C0:00/36Agramkow,40:D8:55:07:D0:00/36WuxiSiNe,40:D8:55:07:E0:00/36Tescom,40:D8:55:07:F0:00/36Wheatsto,40:D8:55:08:00:00/36Honeywel,40:D8:55:08:10:00/36Sicon,40:D8:55:08:20:00/36ard,40:D8:55:08:30:00/36Delopt,40:D8:55:08:40:00/36Papendor,40:D8:55:08:50:00/36PeekTraf,40:D8:55:08:60:00/36DspDesig,40:D8:55:08:70:00/36BestelCh,40:D8:55:08:80:00/36JelSyste,40:D8:55:08:90:00/36WuhanXin,40:D8:55:08:A0:00/36LederEle,40:D8:55:08:B0:00/36MeshWork,40:D8:55:08:C0:00/36Magnesca,40:D8:55:08:D0:00/36BoehmeNa,40:D8:55:08:E0:00/36Lyngsoe,40:D8:55:08:F0:00/36Excelita,40:D8:55:09:00:00/36AxxessId,40:D8:55:09:10:00/36Kdt,40:D8:55:09:20:00/36Wasserba,40:D8:55:09:30:00/36Sentry36,40:D8:55:09:40:00/36NomadDig,40:D8:55:09:50:00/36HeartFor,40:D8:55:09:60:00/36ComtelEl,40:D8:55:09:70:00/36BurtonTe,40:D8:55:09:80:00/36Dave,40:D8:55:09:90:00/36idcell,40:D8:55:09:A0:00/36Coherent,40:D8:55:09:B0:00/36TokyoDen,40:D8:55:09:C0:00/36KeywareS,40:D8:55:09:D0:00/36EMAC,40:D8:55:09:E0:00/36NanoPuls,40:D8:55:09:F0:00/36Bascules,40:D8:55:0A:00:00/36Quantron,40:D8:55:0A:10:00/36ADVALYSY,40:D8:55:0A:20:00/36Xemex,40:D8:55:0A:30:00/36Telefran,40:D8:55:0A:40:00/36ReschEle,40:D8:55:0A:50:00/36WooshCom,40:D8:55:0A:60:00/36AlumbraP,40:D8:55:0A:70:00/36FirstDes,40:D8:55:0A:80:00/36Baudisch,40:D8:55:0A:90:00/36Apantac,40:D8:55:0A:A0:00/36ThermalI,40:D8:55:0A:B0:00/36EneldooB,40:D8:55:0A:C0:00/36Fraunhof,40:D8:55:0A:D0:00/36SpaceMic,40:D8:55:0A:E0:00/36GDMissio,40:D8:55:0A:F0:00/36EnVerv,40:D8:55:0B:00:00/36Microlog,40:D8:55:0B:10:00/36NanjingT,40:D8:55:0B:20:00/36EverTren,40:D8:55:0B:30:00/36TWS,40:D8:55:0B:40:00/36Mitsubis,40:D8:55:0B:50:00/36DataShar,40:D8:55:0B:60:00/36Telvent,40:D8:55:0B:70:00/36ACDElekt,40:D8:55:0B:80:00/36FerlinTr,40:D8:55:0B:90:00/36WxBRSist,40:D8:55:0B:A0:00/36PCHEngin,40:D8:55:0B:B0:00/36Whiptail,40:D8:55:0B:C0:00/36AplexTec,40:D8:55:0B:D0:00/36iCOGNIZE,40:D8:55:0B:E0:00/36Manufact,40:D8:55:0B:F0:00/36Shenzhen,40:D8:55:0C:00:00/36Act,40:D8:55:0C:10:00/36XeptoCom,40:D8:55:0C:20:00/36SCTechsw,40:D8:55:0C:30:00/36APGCashD,40:D8:55:0C:40:00/36Inspired,40:D8:55:0C:50:00/36MMElektr,40:D8:55:0C:60:00/36comtime,40:D8:55:0C:70:00/36insensiv,40:D8:55:0C:80:00/36MettlerT,40:D8:55:0C:90:00/36Quantafl,40:D8:55:0C:A0:00/36Neutrik,40:D8:55:0C:B0:00/36ReliOn,40:D8:55:0C:C0:00/36Ateme,40:D8:55:0C:D0:00/36LogicalP,40:D8:55:0C:E0:00/36ESTAnaly,40:D8:55:0C:F0:00/36Clark-MX,40:D8:55:0D:00:00/36Icraft,40:D8:55:0D:10:00/36Cantada,40:D8:55:0D:20:00/36Elan,40:D8:55:0D:30:00/36LECO,40:D8:55:0D:40:00/36Mitsubis,40:D8:55:0D:50:00/36ShimizuE,40:D8:55:0D:60:00/36deskontr,40:D8:55:0D:70:00/36AvantTec,40:D8:55:0D:80:00/36NEXT!scS,40:D8:55:0D:90:00/36YukoElec,40:D8:55:0D:A0:00/36Devialet,40:D8:55:0D:B0:00/36TopConne,40:D8:55:0D:C0:00/36NVSTechn,40:D8:55:0D:D0:00/36Embed,40:D8:55:0D:E0:00/36Vpg,40:D8:55:0D:F0:00/36Xadi,40:D8:55:0E:00:00/36Richter,40:D8:55:0E:10:00/36STVElect,40:D8:55:0E:20:00/36KeockoKf,40:D8:55:0E:30:00/36Medigus,40:D8:55:0E:40:00/36Arago,40:D8:55:0E:50:00/36TritonEl,40:D8:55:0E:60:00/36Kyoritsu,40:D8:55:0E:70:00/36Lightsta,40:D8:55:0E:80:00/36Heitec,40:D8:55:0E:90:00/36HAMEG,40:D8:55:0E:A0:00/36A-Z-E,40:D8:55:0E:B0:00/36WANTECHN,40:D8:55:0E:C0:00/36Sentry36,40:D8:55:0E:D0:00/36IntelliD,40:D8:55:0E:E0:00/36SiegmarZ,40:D8:55:0E:F0:00/36GeneSysE,40:D8:55:0F:00:00/36Redwood,40:D8:55:0F:10:00/36Grossenb,40:D8:55:0F:20:00/36SigmaPhi,40:D8:55:0F:30:00/36ECON,40:D8:55:0F:40:00/36MBconnec,40:D8:55:0F:50:00/36CSTGroup,40:D8:55:0F:60:00/36Private,40:D8:55:0F:70:00/36ComlineE,40:D8:55:0F:80:00/36BetterPl,40:D8:55:0F:90:00/36InvisuaL,40:D8:55:0F:A0:00/36Marmitek,40:D8:55:0F:B0:00/36InfoMacS,40:D8:55:0F:C0:00/36eumigind,40:D8:55:0F:D0:00/36MONOGRAM,40:D8:55:0F:E0:00/36CytechTe,40:D8:55:0F:F0:00/36YUYAMAMF,40:D8:55:10:00:00/36TaskSist,40:D8:55:10:10:00/36epgElett,40:D8:55:10:20:00/36PowerEle,40:D8:55:10:30:00/36PeekTraf,40:D8:55:10:40:00/36ImpleSis,40:D8:55:10:50:00/36TielineR,40:D8:55:10:60:00/36Orbital,40:D8:55:10:70:00/36SmithMet,40:D8:55:10:80:00/36AlphaDes,40:D8:55:10:90:00/36Rosslare,40:D8:55:10:A0:00/36DavisDer,40:D8:55:10:B0:00/36So-Cool,40:D8:55:10:C0:00/36Contrans,40:D8:55:10:D0:00/36Rite-Tec,40:D8:55:10:E0:00/36HKS-Proz,40:D8:55:10:F0:00/36CavalryS,40:D8:55:11:00:00/36AplexTec,40:D8:55:11:10:00/36GrupoEpe,40:D8:55:11:20:00/36Hallibur,40:D8:55:11:30:00/36Testbook,40:D8:55:11:40:00/36GDMissio,40:D8:55:11:50:00/36MESAElec,40:D8:55:11:60:00/36Uniscan,40:D8:55:11:70:00/36RCSEnerg,40:D8:55:11:80:00/36Universi,40:D8:55:11:90:00/36GroupInd,40:D8:55:11:A0:00/36Sicon,40:D8:55:11:B0:00/36nanoTRON,40:D8:55:11:C0:00/36DEUTA-WE,40:D8:55:11:D0:00/36ACDElekt,40:D8:55:11:E0:00/36CEMSI,40:D8:55:11:F0:00/36KOMPANPa,40:D8:55:12:00:00/36ObjectFa,40:D8:55:12:10:00/36shanghai,40:D8:55:12:20:00/36ATXNetwo,40:D8:55:12:30:00/36NPCKompj,40:D8:55:12:40:00/36Debug,40:D8:55:12:50:00/36Scandyna,40:D8:55:12:60:00/36Tti,40:D8:55:12:70:00/36Lightsta,40:D8:55:12:80:00/36Akse,40:D8:55:12:90:00/36DspDesig,40:D8:55:12:A0:00/36JadpodCo,40:D8:55:12:B0:00/36MangoDSP,40:D8:55:12:C0:00/36NSPEurop,40:D8:55:12:D0:00/36BiotageS,40:D8:55:12:E0:00/36Canfield,40:D8:55:12:F0:00/36Private,40:D8:55:13:00:00/36GSPSprac,40:D8:55:13:10:00/36EMAC,40:D8:55:13:20:00/36AeroVisi,40:D8:55:13:30:00/36Tattile,40:D8:55:13:40:00/36digitech,40:D8:55:13:50:00/36Globalco,40:D8:55:13:60:00/36Devrieco,40:D8:55:13:70:00/36GDEPolsk,40:D8:55:13:80:00/36CalonAss,40:D8:55:13:90:00/36WOWSyste,40:D8:55:13:A0:00/36Supplier,40:D8:55:13:B0:00/36DavinTec,40:D8:55:13:C0:00/36shanghai,40:D8:55:13:D0:00/36PermScie,40:D8:55:13:E0:00/36hanatech,40:D8:55:13:F0:00/36Zhejiang,40:D8:55:14:00:00/36InnoTran,40:D8:55:14:10:00/36Key,40:D8:55:14:20:00/36Tetracor,40:D8:55:14:30:00/36TokyoDra,40:D8:55:14:40:00/36Venco,40:D8:55:14:50:00/36WeberMar,40:D8:55:14:60:00/36PleigerE,40:D8:55:14:70:00/36AplexTec,40:D8:55:14:80:00/36SeikoTim,40:D8:55:14:90:00/36EngageTe,40:D8:55:14:A0:00/36GDMissio,40:D8:55:14:B0:00/36Vocality,40:D8:55:14:C0:00/36Plt,40:D8:55:14:D0:00/36SomfySas,40:D8:55:14:E0:00/36Marposs,40:D8:55:14:F0:00/36TDSSoftw,40:D8:55:15:00:00/36ShikinoH,40:D8:55:15:10:00/36Progress,40:D8:55:15:20:00/36HomeAuto,40:D8:55:15:30:00/36BlinkPip,40:D8:55:15:40:00/36iart,40:D8:55:15:50:00/36Telefran,40:D8:55:15:60:00/36Emphysys,40:D8:55:15:70:00/36HitachiP,40:D8:55:15:80:00/36Exibea,40:D8:55:15:90:00/36PLATINUM,40:D8:55:15:A0:00/36DorletSa,40:D8:55:15:B0:00/36SQFSpezi,40:D8:55:15:C0:00/36Spectrat,40:D8:55:15:D0:00/36Actronic,40:D8:55:15:E0:00/36ProdcoIn,40:D8:55:15:F0:00/36CT,40:D8:55:16:00:00/36ThermoFi,40:D8:55:16:10:00/36Solidsca,40:D8:55:16:20:00/36Luna-Nex,40:D8:55:16:30:00/36KMtronic,40:D8:55:16:40:00/36NFTAutom,40:D8:55:16:50:00/36Techboar,40:D8:55:16:60:00/36AnhuiJia,40:D8:55:16:70:00/36Assembly,40:D8:55:16:80:00/36OPASCA,40:D8:55:16:90:00/36PhotopKo,40:D8:55:16:A0:00/36AplexTec,40:D8:55:16:B0:00/36Techway,40:D8:55:16:C0:00/36Private,40:D8:55:16:D0:00/36GDMissio,40:D8:55:16:E0:00/36Secuinfo,40:D8:55:16:F0:00/36BrightLe,40:D8:55:17:00:00/36ICSEletr,40:D8:55:17:10:00/36Sicon,40:D8:55:17:20:00/36YawataEl,40:D8:55:17:30:00/36ContecSt,40:D8:55:17:40:00/36EcoGuard,40:D8:55:17:50:00/36AHBSyste,40:D8:55:17:60:00/36Schneide,40:D8:55:17:70:00/36TRIEngin,40:D8:55:17:80:00/36REDERDom,40:D8:55:17:90:00/36Servo-Ro,40:D8:55:17:A0:00/36Argus-Sp,40:D8:55:17:B0:00/36Luceo,40:D8:55:17:C0:00/36Critical,40:D8:55:17:D0:00/36Kiwigrid,40:D8:55:17:E0:00/36Tokhatec,40:D8:55:17:F0:00/36Telvent,40:D8:55:18:00:00/36BroadSof,40:D8:55:18:10:00/36eROCCA,40:D8:55:18:20:00/36GeorgNeu,40:D8:55:18:30:00/36EMAC,40:D8:55:18:40:00/36Satkirit,40:D8:55:18:50:00/36Standard,40:D8:55:18:60:00/36KSTtechn,40:D8:55:18:70:00/36CDEX,40:D8:55:18:80:00/36Array,40:D8:55:18:90:00/36Yoozma,40:D8:55:18:A0:00/36AplexTec,40:D8:55:18:B0:00/36Diagnosy,40:D8:55:18:C0:00/36EOS,40:D8:55:18:D0:00/36ZoeMedic,40:D8:55:18:E0:00/36KerunVis,40:D8:55:18:F0:00/36BeatSens,40:D8:55:19:00:00/36SpiderTe,40:D8:55:19:10:00/36SoukaiEl,40:D8:55:19:20:00/36GDMissio,40:D8:55:19:30:00/36ForzaSil,40:D8:55:19:40:00/36RFCode,40:D8:55:19:50:00/36TonnaEle,40:D8:55:19:60:00/36Advanced,40:D8:55:19:70:00/36BergClou,40:D8:55:19:80:00/36devboard,40:D8:55:19:90:00/36PRESSOLS,40:D8:55:19:A0:00/36Rohde&Sc,40:D8:55:19:B0:00/36Northern,40:D8:55:19:C0:00/36ParrisSe,40:D8:55:19:D0:00/36EMAC,40:D8:55:19:E0:00/36Thirdway,40:D8:55:19:F0:00/36PatriaAv,40:D8:55:1A:00:00/36Futaba,40:D8:55:1A:10:00/36Kronotec,40:D8:55:1A:20:00/36Hipodrom,40:D8:55:1A:30:00/36Noritake,40:D8:55:1A:40:00/36cibite,40:D8:55:1A:50:00/36DemoPad,40:D8:55:1A:60:00/36RB-LINKW,40:D8:55:1A:70:00/36ENTECEle,40:D8:55:1A:80:00/36Multiobr,40:D8:55:1A:90:00/36Lubinosr,40:D8:55:1A:A0:00/36Broachli,40:D8:55:1A:B0:00/36Rosslare,40:D8:55:1A:C0:00/36Elan,40:D8:55:1A:D0:00/36WicherDi,40:D8:55:1A:E0:00/36Autonomo,40:D8:55:1A:F0:00/36Vigitron,40:D8:55:1B:00:00/36Shin-eiE,40:D8:55:1B:10:00/36Logos01,40:D8:55:1B:20:00/36AGEAGilg,40:D8:55:1B:30:00/36Bettini,40:D8:55:1B:40:00/36InforceC,40:D8:55:1B:50:00/36A+ECKlei,40:D8:55:1B:60:00/36Magic,40:D8:55:1B:70:00/36TEWSElek,40:D8:55:1B:80:00/36Orion,40:D8:55:1B:90:00/36BekingIn,40:D8:55:1B:A0:00/36Creative,40:D8:55:1B:B0:00/36Micromeg,40:D8:55:1B:C0:00/36KbDevice,40:D8:55:1B:D0:00/36HoribaAb,40:D8:55:1B:E0:00/36PeekTraf,40:D8:55:1B:F0:00/36shanghai,40:D8:55:1C:00:00/36NPBAutom,40:D8:55:1C:10:00/36TriamecM,40:D8:55:1C:20:00/36DigitalD,40:D8:55:1C:30:00/36Cornfed,40:D8:55:1C:40:00/36QEDAdvan,40:D8:55:1C:50:00/36Private,40:D8:55:1C:60:00/36DeviceSo,40:D8:55:1C:70:00/36Wexiodis,40:D8:55:1C:80:00/36SensataT,40:D8:55:1C:90:00/36Andy-L,40:D8:55:1C:A0:00/36RigelEng,40:D8:55:1C:B0:00/36MG,40:D8:55:1C:C0:00/36NKTPhoto,40:D8:55:1C:D0:00/36YXLONInt,40:D8:55:1C:E0:00/36PeterHub,40:D8:55:1C:F0:00/36OmnikNew,40:D8:55:1D:00:00/36Webeasy,40:D8:55:1D:10:00/36FounderB,40:D8:55:1D:20:00/36InventLa,40:D8:55:1D:30:00/36KalugaTe,40:D8:55:1D:40:00/36PrismaEn,40:D8:55:1D:50:00/36FST21,40:D8:55:1D:60:00/36EMSCompu,40:D8:55:1D:70:00/36Wheatsto,40:D8:55:1D:80:00/36OwlCyber,40:D8:55:1D:90:00/36Commerci,40:D8:55:1D:A0:00/36EnergyTe,40:D8:55:1D:B0:00/36NipponTe,40:D8:55:1D:C0:00/36AplexTec,40:D8:55:1D:D0:00/36BaOptBen,40:D8:55:1D:E0:00/36Vidisys,40:D8:55:1D:F0:00/36ChengduM,40:D8:55:1E:00:00/36Embedded,40:D8:55:1E:10:00/36AdQualit,40:D8:55:1E:20:00/36ELNECsro,40:D8:55:1E:30:00/36MegaElec,40:D8:55:1E:40:00/36STEK,40:D8:55:50:D0:00/36Shenzhen,40:D8:55:EE:60:00/36Narinet,40:DC:9DHajen,40:DC:A5HuaweiDe,40:DD:D1Beautifu,40:DE:17Shenzhen,40:DE:ADJuniperN,40:DF:02LINEBIZP,40:E1:E4NokiaSol,40:E2:30AzureWav,40:E3:D6ArubaaHe,40:E6:4BApple,40:E7:30DEYStora,40:E7:93Shenzhen,40:EA:CEFounderB,40:EC:99IntelCor,40:EC:F8Siemens,40:ED:98IEEERegi,40:ED:98:00:00:00/28Tsinghua,40:ED:98:10:00:00/28GuangZho,40:ED:98:20:00:00/28A-Iox,40:ED:98:30:00:00/28Knox,40:ED:98:40:00:00/28Kendrion,40:ED:98:50:00:00/28Cape,40:ED:98:60:00:00/28Shanghai,40:ED:98:70:00:00/28Vaisala,40:ED:98:80:00:00/28Guangzho,40:ED:98:90:00:00/28TeraTron,40:ED:98:A0:00:00/28Integrat,40:ED:98:B0:00:00/28SiebertI,40:ED:98:C0:00:00/28BloomSky,40:ED:98:D0:00:00/28Hangzhou,40:ED:98:E0:00:00/28BordaTec,40:EE:15ZioncomE,40:EE:DDHuaweiTe,40:EF:4CFihonest,40:F0:2FLiteonTe,40:F0:4EIntegrat,40:F0:78Cisco,40:F1:4CISEEurop,40:F2:01Sagemcom,40:F2:1CDASANZho,40:F2:E9Ibm,40:F3:08MurataMa,40:F3:85IEEERegi,40:F3:85:00:00:00/28SubPac,40:F3:85:10:00:00/28JohnsonM,40:F3:85:20:00:00/28BeijingZ,40:F3:85:30:00:00/28IntelliD,40:F3:85:40:00:00/28Embedded,40:F3:85:50:00:00/28KatoEngi,40:F3:85:60:00:00/28LennoxIn,40:F3:85:70:00:00/28Palazzet,40:F3:85:80:00:00/28Teleepoc,40:F3:85:90:00:00/28FastPrec,40:F3:85:A0:00:00/28Creanord,40:F3:85:B0:00:00/28URMETHom,40:F3:85:C0:00:00/28ClixxoBr,40:F3:85:D0:00:00/28DigitalB,40:F3:85:E0:00:00/28BBB,40:F4:07Nintendo,40:F4:13Rubezh,40:F4:20SichuanT,40:F4:ECCisco,40:F5:20Espressi,40:F5:2ELeicaMic,40:F9:46Apple,40:F9:D5TecoreNe,40:FA:7FPrehCarC,40:FC:89ARRISGro,40:FE:0DMaxio,44:00:10Apple,44:00:49AmazonTe,44:00:4DHuaweiTe,44:01:BBShenzhen,44:03:2CIntelCor,44:03:77IEEERegi,44:03:77:00:00:00/28MusashiS,44:03:77:10:00:00/28Atari,44:03:77:20:00:00/28ExsomCom,44:03:77:30:00:00/28Annapurn,44:03:77:40:00:00/28LenovoIm,44:03:77:50:00:00/28NordenCo,44:03:77:60:00:00/28ShenZhen,44:03:77:70:00:00/28StaraInd,44:03:77:80:00:00/28GemmyEle,44:03:77:90:00:00/28Shenzhen,44:03:77:A0:00:00/28symplr,44:03:77:B0:00:00/28Hangzhou,44:03:77:C0:00:00/28BIGClima,44:03:77:D0:00:00/28Omnisens,44:03:77:E0:00:00/28BolinTec,44:03:A7Cisco,44:04:44Guangdon,44:07:0BGoogle,44:09:B8SalcompS,44:0C:FDNetMan,44:10:FEHuizhouF,44:11:02EDMIEuro,44:11:C2Telegart,44:13:19WkkTechn,44:13:D0zte,44:14:41AudioCon,44:16:22Microsof,44:17:93Espressi,44:18:47HunanScr,44:18:4FFitview,44:18:FDApple,44:19:B6Hangzhou,44:1A:FANewH3CTe,44:1C:12Technico,44:1C:7FMotorola,44:1C:A8HonHaiPr,44:1E:91ARVIDAIn,44:1E:98RuckusWi,44:1E:A1HewlettP,44:22:7CHuaweiTe,44:22:95ChinaMob,44:22:F1SFac,44:23:7CBeijingX,44:23:AAFarmage,44:25:BBBambooEn,44:27:F370mai,44:28:A3Jiangsuf,44:29:38NietZsch,44:2A:60Apple,44:2A:FFE3Techno,44:2B:03Cisco,44:2C:05AMPAKTec,44:31:92HewlettP,44:32:2AAvaya,44:32:C8Technico,44:33:4CShenzhen,44:34:8FMxtIndus,44:34:A7ARRISGro,44:35:6FNeterix,44:35:83Apple,44:37:08MRVComun,44:37:192SaveEne,44:37:6FYoungEle,44:37:E6HonHaiPr,44:38:39CumulusN,44:39:C4Universa,44:3B:32Intelbra,44:3C:88FicosaMa,44:3C:9CPintschT,44:3D:21Nuvolt,44:3E:07Electrol,44:3E:B2DEOTRON,44:42:2FTestop,44:44:50OttoQ,44:45:53Microsoft,44:46:49DfiDiamo,44:46:87RealmeCh,44:47:CCHangzhou,44:48:91HDMILice,44:48:B9MitraSta,44:48:C1HewlettP,44:48:FFQingdaoH,44:4A:65Silverfl,44:4A:B0Zhejiang,44:4A:DBApple,44:4B:5DGEHealth,44:4B:7EFiberhom,44:4C:0CApple,44:4C:A8AristaNe,44:4E:1ASamsungE,44:4E:6DAVMAudio,44:4F:5EPanStudi,44:4F:8EWiZ,44:51:DBRaytheon,44:54:C0Thompson,44:55:B1HuaweiTe,44:55:C4HuaweiDe,44:56:8DPNCTechn,44:56:B7SpawnLab,44:56:E2SichuanT,44:58:29CiscoSPV,44:59:43zte,44:59:9FCriticar,44:59:E3HuaweiTe,44:5B:EDArubaaHe,44:5C:E9SamsungE,44:5D:5ESHENZHEN,44:5E:CDRazer,44:5E:F3Tonalite,44:5F:7AShihlinE,44:5F:8CIntercel,44:61:32ecobee,44:61:9CFONsyste,44:62:46Comat,44:65:0DAmazonTe,44:65:6AMegaVide,44:65:7FCalix,44:66:6EIp-Line,44:66:FCGuangdon,44:67:47HuaweiTe,44:67:52WistronI,44:67:55OrbitIrr,44:68:0CWacom,44:68:ABJuin,44:6A:2EHuaweiTe,44:6A:B7ARRISGro,44:6C:24ReallinE,44:6D:57LiteonTe,44:6D:6CSamsungE,44:6E:E5HuaweiTe,44:6F:D8IEEERegi,44:6F:D8:00:00:00/28Sichuans,44:6F:D8:10:00:00/28Shenzhen,44:6F:D8:20:00:00/28BAYKONEn,44:6F:D8:30:00:00/28Shenzhen,44:6F:D8:40:00:00/28lbLautsp,44:6F:D8:50:00:00/28Zhejiang,44:6F:D8:60:00:00/28AnhuiGuD,44:6F:D8:70:00:00/28Itc,44:6F:D8:80:00:00/28GlobalTe,44:6F:D8:90:00:00/28Annapurn,44:6F:D8:A0:00:00/28ZHEJIANG,44:6F:D8:B0:00:00/28Beijingg,44:6F:D8:C0:00:00/28Changzho,44:6F:D8:D0:00:00/28Scaime,44:6F:D8:E0:00:00/28Cte,44:6F:F8Dyson,44:70:0BIffu,44:70:98MingHong,44:73:D6Logitech,44:74:6CSony,44:76:54HuaweiTe,44:78:3ESamsungE,44:7B:BBShenzhen,44:7B:C4DualShin,44:7C:7FInnoligh,44:7D:A5VtionInf,44:7E:76TrekTech,44:7E:95Alphaand,44:7F:77Connecte,44:80:EBMotorola,44:82:E5HuaweiTe,44:83:12Star-Net,44:85:00IntelCor,44:85:02Shenzhen,44:86:C1SiemensL,44:87:23HoyaServ,44:87:DBTymphany,44:87:FCElitegro,44:88:CBCamcoTec,44:8A:5BMicro-St,44:8C:52KTIS,44:8D:BFRhinoMob,44:8E:12DTResear,44:8E:81Vig,44:8F:17SamsungE,44:90:BBApple,44:91:60MurataMa,44:91:7CHMDGloba,44:91:DBShanghai,44:94:FCNetgear,44:95:3BRLTechIn,44:95:FAQingdaoS,44:96:2BAidon,44:97:5AShenzhen,44:9B:78NowFacto,44:9B:C1HuaweiTe,44:9C:B5Alcomp,44:9E:F9vivoMobi,44:9F:46HuaweiDe,44:9F:7FDataCore,44:A1:91HuaweiTe,44:A4:2DTCTmobil,44:A4:66GroupeLd,44:A5:4EQorvoInt,44:A5:6ENetgear,44:A6:1EIngramMi,44:A6:42Hangzhou,44:A6:89PromaxEl,44:A6:E5Thinking,44:A7:CFMurataMa,44:A8:42Dell,44:A8:C2SewooTec,44:A8:FCApple,44:A9:2CIEEERegi,44:A9:2C:00:00:00/28Zhejiang,44:A9:2C:10:00:00/28uimcom,44:A9:2C:20:00:00/28AnhuiZho,44:A9:2C:30:00:00/28Luxonis,44:A9:2C:40:00:00/28NetXNetw,44:A9:2C:50:00:00/28Shenzhen,44:A9:2C:60:00:00/28Ningbojo,44:A9:2C:70:00:00/28Efficien,44:A9:2C:80:00:00/28RT-Syste,44:A9:2C:90:00:00/28ChinaEle,44:A9:2C:A0:00:00/28Digiport,44:A9:2C:B0:00:00/28Amethyst,44:A9:2C:C0:00:00/28Cubitech,44:A9:2C:D0:00:00/28NppKomet,44:A9:2C:E0:00:00/28Annapurn,44:AA:27udworks,44:AA:50JuniperN,44:AA:E8NanotecE,44:AA:F5ARRISGro,44:AD:19Xingfei（,44:AD:B1Sagemcom,44:AD:D9Cisco,44:AE:25Cisco,44:AE:44HuaweiDe,44:AE:ABGuangdon,44:AF:28IntelCor,44:B2:95SichuanA,44:B3:2DTp-LinkT,44:B3:82Kuang-ch,44:B4:12Sius,44:B4:33tide,44:B4:62Flextron,44:B6:BECisco,44:B9:94DouglasL,44:BA:46SichuanT,44:BB:3BGoogle,44:BD:DEBHTC,44:BF:E3Shenzhen,44:C1:5CTexasIns,44:C2:33Guangzho,44:C3:06SIFROM,44:C3:46HuaweiTe,44:C3:9BRubezhNp,44:C4:A9OpticomC,44:C5:6FNGNEasyS,44:C6:5DApple,44:C6:9BWuhanFen,44:C7:FCHuaweiDe,44:C8:74ChinaMob,44:C9:A2Greenwal,44:CB:8BLGInnote,44:CD:0EFlextron,44:CE:3AJiangsuH,44:CE:7DSfr,44:D1:5EShanghai,44:D1:FAShenzhen,44:D2:44SeikoEps,44:D2:CAAnviaTV,44:D3:ADShenzhen,44:D3:CACisco,44:D4:37IntenoBr,44:D4:53Sagemcom,44:D4:54Sagemcom,44:D4:E0Sony,44:D5:A5AddOnCom,44:D5:CCAmazonTe,44:D5:F2IEEERegi,44:D5:F2:00:00:00/28TIBARese,44:D5:F2:10:00:00/28Simplere,44:D5:F2:20:00:00/28Shenzhen,44:D5:F2:30:00:00/28Vuro,44:D5:F2:40:00:00/28Appotron,44:D5:F2:50:00:00/28tigaelev,44:D5:F2:60:00:00/28BeamComm,44:D5:F2:70:00:00/28Shenzhen,44:D5:F2:80:00:00/28CETCAvio,44:D5:F2:90:00:00/28AuctusTe,44:D5:F2:A0:00:00/28SYSTECel,44:D5:F2:B0:00:00/28ValeoInt,44:D5:F2:C0:00:00/28neocontr,44:D5:F2:D0:00:00/28Shenzhen,44:D5:F2:E0:00:00/28Joint-St,44:D6:3DTalariNe,44:D6:E1SnuzaInt,44:D7:91HuaweiTe,44:D8:32AzureWav,44:D8:78HuiZhouG,44:D8:84Apple,44:D9:E7Ubiquiti,44:DB:60NanjingB,44:DC:4EItelMobi,44:DC:91PlanexCo,44:DC:CBSemindia,44:E0:8ECiscoSPV,44:E1:37ARRISGro,44:E4:9AOmnitron,44:E4:D9Cisco,44:E4:EEWistronN,44:E5:17IntelCor,44:E6:6EApple,44:E6:B0ChinaMob,44:E8:A5MyrekaTe,44:E9:68HuaweiTe,44:E9:DDSagemcom,44:EA:4BActlas,44:EA:D8TexasIns,44:EB:2EAlpsalpi,44:EC:CEJuniperN,44:ED:57Longicor,44:EE:02MTI,44:EE:30Budelman,44:EF:BFChinaDra,44:EF:CFUGENESOL,44:F0:34Kaonmedi,44:F0:9EApple,44:F2:1BApple,44:F4:36zte,44:F4:59SamsungE,44:F4:77JuniperN,44:F4:E7Cohesity,44:F8:49UnionPac,44:F9:71Shenzhen,44:FB:42Apple,44:FB:5Azte,44:FD:A3Everysig,44:FE:3BArcadyan,44:FF:BAzte,47:54:43GtcNotRe,48:00:31HuaweiTe,48:00:33Technico,48:00:7DDtsElekt,48:01:C5OnePlusT,48:02:2AB-LinkEl,48:03:62DesayEle,48:04:9FElecom,48:05:E2HuaweiDe,48:06:2BPrivate,48:06:6ATempered,48:0B:B2IEEERegi,48:0B:B2:00:00:00/28RidangoA,48:0B:B2:10:00:00/28BajaElec,48:0B:B2:20:00:00/28ThalesCE,48:0B:B2:30:00:00/28shanghai,48:0B:B2:40:00:00/28Hangzhou,48:0B:B2:50:00:00/28Solaredg,48:0B:B2:60:00:00/28Annapurn,48:0B:B2:70:00:00/28BeijingD,48:0B:B2:80:00:00/28BravoCom,48:0B:B2:90:00:00/28Micropro,48:0B:B2:A0:00:00/28XiamenRo,48:0B:B2:B0:00:00/28Popit,48:0B:B2:C0:00:00/28Shenzhen,48:0B:B2:D0:00:00/28M2Lab,48:0B:B2:E0:00:00/28BeijingM,48:0C:49NAKAYO,48:0E:ECTp-LinkT,48:0F:CFHewlettP,48:10:63NTTInnov,48:12:49LuxcomTe,48:12:58HuaweiTe,48:12:8FHuaweiTe,48:13:7ESamsungE,48:13:F3BbkEduca,48:16:93Lear,48:17:4CMicroPow,48:18:42Shanghai,48:18:8DWeifangG,48:18:FANocsys,48:1A:84PointerT,48:1B:D2IntronSc,48:1D:70CiscoSPV,48:1F:2DShenzhen,48:21:0BPegatron,48:21:6CChinaMob,48:22:18Shenzhen,48:23:35DialogSe,48:25:67Poly,48:26:2CApple,48:26:E8Tek-Air,48:27:59LevvenEl,48:27:EASamsungE,48:28:2Fzte,48:29:52Sagemcom,48:29:E4NPKRotek,48:2A:E3WistronI,48:2C:A0XiaomiCo,48:2C:D0HuaweiTe,48:2C:EAMotorola,48:2F:D7HuaweiTe,48:33:DDZennioAv,48:34:3DIEP,48:35:2EShenzhen,48:36:5FWintecro,48:38:71HuaweiDe,48:39:74ProwareT,48:3B:38Apple,48:3C:0CHuaweiTe,48:3D:32SyscorCo,48:3E:5ESernetSu,48:3F:DAEspressi,48:3F:E9HuaweiTe,48:43:5AHuaweiTe,48:43:7CApple,48:43:DDAmazonTe,48:44:53Hds???,48:44:87CiscoSPV,48:44:F7SamsungE,48:45:20IntelCor,48:46:C1Fn-LinkT,48:46:F1Uros,48:46:FBHuaweiTe,48:47:4BHuaweiDe,48:49:C7SamsungE,48:4A:30GeorgeRo,48:4A:E9HewlettP,48:4B:AAApple,48:4B:D4Technico,48:4C:00NetworkS,48:4C:29HuaweiTe,48:4C:86HuaweiDe,48:4D:7EDell,48:4E:FCARRISGro,48:50:73Microsof,48:51:69SamsungE,48:51:B7IntelCor,48:51:C5IntelCor,48:51:CFIntelbra,48:52:61Soreel,48:54:15NetRules,48:54:E8Winbond?,48:55:19Espressi,48:55:5CWuQiTech,48:55:5FFiberhom,48:57:02HuaweiTe,48:57:DDFacebook,48:59:29LGElectr,48:59:A4zte,48:5A:3FWisol,48:5A:67ShaanxiR,48:5A:B6HonHaiPr,48:5B:39ASUSTekC,48:5D:36Verizon,48:5D:60AzureWav,48:5D:EBJustAddP,48:5F:99CloudNet,48:60:5FLGElectr,48:60:BCApple,48:61:A3ConcernA,48:61:EESamsungE,48:62:76HuaweiTe,48:65:EEIEEERegi,48:65:EE:00:00:00/28DefPower,48:65:EE:10:00:00/28GopodGro,48:65:EE:20:00:00/28CaptionC,48:65:EE:30:00:00/28DataTech,48:65:EE:40:00:00/28MissionM,48:65:EE:50:00:00/28Swistec,48:65:EE:60:00:00/28shenzhen,48:65:EE:70:00:00/28VentureR,48:65:EE:80:00:00/28SmartDis,48:65:EE:90:00:00/28VideoSti,48:65:EE:A0:00:00/28Shenzhen,48:65:EE:B0:00:00/28EnBWEner,48:65:EE:C0:00:00/28DnvGl,48:65:EE:D0:00:00/28WinnTech,48:65:EE:E0:00:00/28Cnu,48:68:34SiliconM,48:68:4AIntelCor,48:6B:2CBbkEduca,48:6B:91Fleetwoo,48:6D:BBVestelEl,48:6E:70Zhejiang,48:6E:73Pica8,48:6E:FBDavitSys,48:6F:D2StorSimp,48:70:1ETexasIns,48:71:19SgbGroup,48:73:97NewH3CTe,48:73:CBTiinlab,48:74:6EApple,48:75:83Intellio,48:76:04Private,48:77:46Calix,48:77:BDGuangdon,48:78:5EAmazonTe,48:79:4DSamsungE,48:7A:55ALEInter,48:7A:DAHangzhou,48:7A:F6NcsElect,48:7A:FFEssys,48:7B:5ESmtTelec,48:7B:6BHuaweiTe,48:7D:2ETp-LinkT,48:7E:48EardaTec,48:82:44LifeFitn,48:82:F2AppelEle,48:83:B4Guangdon,48:83:C7Sagemcom,48:86:E8Microsof,48:87:2DShenZhen,48:87:64vivoMobi,48:88:03ManTechn,48:88:1EEthoSwit,48:88:99Shenzhen,48:88:CAMotorola,48:89:E7IntelCor,48:8A:D2MercuryC,48:8B:0ACisco,48:8C:63HuaweiDe,48:8D:36Arcadyan,48:8E:42DIGALOG,48:8E:EFHuaweiTe,48:8F:5ARouterbo,48:90:2FLGElectr,48:91:53Weinmann,48:91:F6Shenzhen,48:95:07Guangdon,48:98:CASichuanA,48:9A:42Technoma,48:9B:D5ExtremeN,48:9B:E2SCIInnov,48:9D:18Flashbay,48:9D:24BlackBer,48:9D:D1SamsungE,48:9E:BDHP,48:A0:F8Fiberhom,48:A1:95Apple,48:A2:2DShenzhen,48:A2:B7Kodofon,48:A2:B8ChengduV,48:A2:E6Resideo,48:A3:80GioneeCo,48:A4:72IntelCor,48:A4:93TaiyoYud,48:A5:16HuaweiDe,48:A5:E7Nintendo,48:A6:B8Sonos,48:A6:D2GJsunOpt,48:A7:3CSichuant,48:A7:4Ezte,48:A9:1CApple,48:A9:D2WistronN,48:AA:5DStoreEle,48:AD:08HuaweiTe,48:B0:2DNVIDIA,48:B2:53Marketax,48:B2:5DHuaweiTe,48:B4:23AmazonTe,48:B5:A7GloryHor,48:B6:20ROLI,48:B8:A3Apple,48:B8:DEHomewins,48:B9:77PulseOn,48:B9:C2Teletics,48:BA:4EHewlettP,48:BC:A6ASUNGTE,48:BD:0EQuantaSt,48:BD:3DNewH3CTe,48:BD:4AHuaweiTe,48:BE:2DSymanitr,48:BF:6BApple,48:BF:74Baicells,48:C0:49BroadTel,48:C0:93Xirrus,48:C1:ACPlantron,48:C3:B0Pharos,48:C5:8DLear,48:C6:63GTOAcces,48:C7:96SamsungE,48:C8:62SimoWire,48:C8:B6SysTec,48:CB:6ECelloEle,48:D0:CFUniversa,48:D1:8EMetisCom,48:D2:24LiteonTe,48:D2:4FSagemcom,48:D3:43ARRISGro,48:D3:5DPrivate,48:D5:39HuaweiTe,48:D5:4CJedaNetw,48:D6:D5Google,48:D7:05Apple,48:D7:FFBLANKOMA,48:D8:45Shenzhen,48:D8:55Telvent,48:D8:75ChinaTra,48:D8:90Fn-LinkT,48:D8:FEClarIDyS,48:DA:96EddySmar,48:DB:50HuaweiTe,48:DC:2DHuaweiTe,48:DC:FBNokia,48:DD:0Ceero,48:DD:9DItelMobi,48:DF:1CWuhanNEC,48:DF:37HewlettP,48:E1:AFVity,48:E1:E9ChengduM,48:E2:44HonHaiPr,48:E3:C3JENOPTIK,48:E6:95Insigma,48:E6:C0SIMComWi,48:E7:DAAzureWav,48:E9:F1Apple,48:EA:63Zhejiang,48:EB:30EternaTe,48:EB:62MurataMa,48:EC:5BNokiaSol,48:ED:80daesunge,48:EE:07SilverPa,48:EE:0CD-LinkIn,48:EE:86UTStarco,48:EF:61HuaweiDe,48:F0:27Chengdun,48:F0:7BAlpsalpi,48:F1:7FIntelCor,48:F2:30Ubizcore,48:F3:17Private,48:F3:F3BaiduOnl,48:F4:7DTechVisi,48:F7:C0Technico,48:F7:F1Nokia,48:F8:B3Cisco-Li,48:F8:DBHuaweiTe,48:F8:E1Nokia,48:F8:FFChengduK,48:F9:25Maestron,48:F9:7CFiberhom,48:FC:B6LavaInte,48:FC:B8Woodstre,48:FD:8EHuaweiTe,48:FD:A3XiaomiCo,48:FE:EAHoma,4A:07:D6IEEE8021,4A:19:1BZigBeeAl,4A:5A:6FAppliedM,4A:72:06CAIRE,4A:7F:74INTEC,4A:AD:CECodanArg,4A:C0:3FWuhanHik,4A:CA:3EZeus,4A:DA:10EmotivaA,4A:E0:3FChengduS,4C:00:82Cisco,4C:01:43eero,4C:02:20XiaomiCo,4C:02:2ECmrKorea,4C:02:89LexCompu,4C:03:4FIntelCor,4C:06:8ABaslerEl,4C:07:C9COMPUTER,4C:09:B4zte,4C:09:D4Arcadyan,4C:0A:3DAdnacom,4C:0B:3ATCTmobil,4C:0B:BEMicrosof,4C:0D:EEJabilCir,4C:0F:6EHonHaiPr,4C:0F:C7EardaTec,4C:11:54Mobiwire,4C:11:59VisionIn,4C:11:AEEspressi,4C:11:BFZhejiang,4C:12:65ARRISGro,4C:13:65EmplusTe,4C:14:80Noregon,4C:14:A3TCLTechn,4C:16:94shenzhen,4C:16:F1zte,4C:16:FCJuniperN,4C:17:44AmazonTe,4C:17:EBSagemcom,4C:18:9AGuangdon,4C:1A:3APRIMARes,4C:1A:3DGuangdon,4C:1A:95Novakon,4C:1B:86Arcadyan,4C:1D:96IntelCor,4C:1F:CCHuaweiTe,4C:20:B8Apple,4C:21:13NokiaSha,4C:21:8CPanasoni,4C:21:D0Sony,4C:22:19Yuanfuda,4C:22:58cozybit,4C:24:98TexasIns,4C:25:78Nokia,4C:26:E7Welgate,4C:2C:80BeijingS,4C:2C:83Zhejiang,4C:2E:FEShenzhen,4C:2F:9DICMContr,4C:2F:D7HuaweiDe,4C:30:89ThalesTr,4C:32:2DTeledata,4C:32:75Apple,4C:32:D9MRuttyHo,4C:33:29Sweroam,4C:33:4EHightech,4C:34:88IntelCor,4C:36:4EPanasoni,4C:38:D5MitacCom,4C:38:D8ARRISGro,4C:39:09HPLElect,4C:39:10NewtekEl,4C:3B:6CGaro,4C:3B:74VOGTECHK,4C:3B:DFMicrosof,4C:3C:16SamsungE,4C:3F:A7uGridNet,4C:3F:D3TexasIns,4C:40:88SanshinE,4C:42:4CInformat,4C:44:5BIntelCor,4C:45:76ChinaMob,4C:48:DABeijingA,4C:49:4Fzte,4C:49:E3XiaomiCo,4C:4B:68MobileDe,4C:4B:F9IEEERegi,4C:4B:F9:00:00:00/28Multitek,4C:4B:F9:10:00:00/28Jiangsua,4C:4B:F9:20:00:00/28Shenzhen,4C:4B:F9:30:00:00/28PowerAct,4C:4B:F9:40:00:00/28Shenzhen,4C:4B:F9:50:00:00/28RemedeeL,4C:4B:F9:60:00:00/28Shandong,4C:4B:F9:70:00:00/28Glonexs,4C:4B:F9:80:00:00/28ZividAS,4C:4B:F9:90:00:00/28Tecnoplu,4C:4B:F9:A0:00:00/28Electrol,4C:4B:F9:B0:00:00/28StoredEn,4C:4B:F9:C0:00:00/28Connecte,4C:4B:F9:D0:00:00/28Shenzhen,4C:4B:F9:E0:00:00/28BeijingA,4C:4D:66NanjingJ,4C:4E:03TCTmobil,4C:4E:35Cisco,4C:4F:EEOnePlusT,4C:50:77HuaweiDe,4C:50:F1Guangdon,4C:52:62FujitsuT,4C:52:ECSOLARWAT,4C:54:27LineproS,4C:54:99HuaweiTe,4C:55:85Hamilton,4C:55:B8Turkcell,4C:55:CCZentri,4C:56:9DApple,4C:56:DFTargusUS,4C:57:CAApple,4C:5D:3CCisco,4C:5D:CDFinnishE,4C:5E:0CRouterbo,4C:5F:D2Alcatel-,4C:60:D5airPoint,4C:60:DENetgear,4C:61:7EHuaweiDe,4C:62:55Sanmina-,4C:63:71XiaomiCo,4C:63:EBApplicat,4C:64:D9Guangdon,4C:65:A8IEEERegi,4C:65:A8:00:00:00/28WELT,4C:65:A8:10:00:00/28BeijingB,4C:65:A8:20:00:00/28OricaEur,4C:65:A8:30:00:00/28Roost,4C:65:A8:40:00:00/28PlusOneJ,4C:65:A8:50:00:00/28TEL-Elec,4C:65:A8:60:00:00/28Nuviz,4C:65:A8:70:00:00/28WuhanMor,4C:65:A8:80:00:00/28InstantB,4C:65:A8:90:00:00/28Shenzhen,4C:65:A8:A0:00:00/28SuzhouEm,4C:65:A8:B0:00:00/28ZMINTech,4C:65:A8:C0:00:00/28Fuse,4C:65:A8:D0:00:00/28Qingping,4C:65:A8:E0:00:00/28HighInfi,4C:66:41SamsungE,4C:6A:F6HMDGloba,4C:6B:E8Apple,4C:6C:13IoTSoluc,4C:6D:58JuniperN,4C:6E:6EComnectT,4C:6F:9CGuangdon,4C:71:0CCisco,4C:71:0DCisco,4C:71:67PoLabsdo,4C:72:B9Pegatron,4C:73:67GeniusBy,4C:73:A5Kove,4C:74:03Bq,4C:74:87LeaderPh,4C:74:BFApple,4C:75:25Espressi,4C:76:25Dell,4C:77:4FEmbedded,4C:77:66Shenzhen,4C:77:6DCisco,4C:77:CBIntelCor,4C:78:72CavUffGi,4C:78:97Arrowhea,4C:79:6EIntelCor,4C:79:BAIntelCor,4C:7A:48NipponSe,4C:7C:5FApple,4C:7C:D9Apple,4C:7F:62Nokia,4C:80:4FArmstron,4C:80:93IntelCor,4C:80:BAWuhanTia,4C:81:20TaicangT,4C:82:CFDishTech,4C:83:DECiscoSPV,4C:87:5DBose,4C:8B:30Actionte,4C:8B:55GrupoDig,4C:8B:EFHuaweiTe,4C:8D:53HuaweiTe,4C:8D:79Apple,4C:8E:CCSilkan,4C:8F:A5Jastec,4C:90:DBJLAudio,4C:91:0CLanixInt,4C:91:57FujianLA,4C:91:7AIEEERegi,4C:91:7A:00:00:00/28Shenzhen,4C:91:7A:10:00:00/28InsterTe,4C:91:7A:20:00:00/28Chongqin,4C:91:7A:30:00:00/28SmartAcc,4C:91:7A:40:00:00/28LumiGrow,4C:91:7A:50:00:00/28mtekvisi,4C:91:7A:60:00:00/28Openeye,4C:91:7A:70:00:00/28SICES,4C:91:7A:80:00:00/28CamsatPr,4C:91:7A:90:00:00/28Hangzhou,4C:91:7A:A0:00:00/28ErlabDFS,4C:91:7A:B0:00:00/28AvertX,4C:91:7A:C0:00:00/28AlibabaB,4C:91:7A:D0:00:00/28Shenzhen,4C:91:7A:E0:00:00/28Annapurn,4C:93:A6IEEERegi,4C:93:A6:00:00:00/28Vestaboa,4C:93:A6:10:00:00/28AtrieTec,4C:93:A6:20:00:00/28DiehlCon,4C:93:A6:30:00:00/28Commsign,4C:93:A6:40:00:00/284TheWall,4C:93:A6:50:00:00/28Private,4C:93:A6:60:00:00/28Shandong,4C:93:A6:70:00:00/285Voxel,4C:93:A6:80:00:00/28Sercomm,4C:93:A6:90:00:00/28Advantic,4C:93:A6:A0:00:00/28HanwangT,4C:93:A6:B0:00:00/28FeltenEl,4C:93:A6:C0:00:00/28WuhanMai,4C:93:A6:D0:00:00/28Cantroni,4C:93:A6:E0:00:00/28Celltron,4C:96:14JuniperN,4C:96:2DFresh,4C:98:EFZeo,4C:9E:80KYOKKOEL,4C:9E:E4HanyangN,4C:9E:FFZyxelCom,4C:A0:03Vitec,4C:A1:61RainBird,4C:A5:15BaikalEl,4C:A5:6DSamsungE,4C:A6:4DCisco,4C:A7:4BAlcatelL,4C:A9:28Insensi,4C:AA:16AzureWav,4C:AB:33KSTtechn,4C:AB:4FApple,4C:AB:F8AskeyCom,4C:AB:FCzte,4C:AC:0Azte,4C:AD:A8Panoptic,4C:AE:13HuaweiTe,4C:AE:1CSaiNXTTe,4C:AE:31ShengHai,4C:AE:A3HewlettP,4C:AE:ECGuangzho,4C:B0:08Shenzhen,4C:B0:E8BeijingR,4C:B1:6CHuaweiTe,4C:B1:99Apple,4C:B1:CDRuckusWi,4C:B2:1CMaxphoto,4C:B4:4ANANOWAVE,4C:B4:EAHrdSPte,4C:B7:6DNoviSecu,4C:B8:1CSAMElect,4C:B8:2CCambridg,4C:B8:B5Shenzhen,4C:B9:10Apple,4C:B9:11Raisecom,4C:B9:9BWeifangG,4C:B9:C8Conet,4C:B9:EAiRobot,4C:BA:A3BisonEle,4C:BB:58ChiconyE,4C:BC:42Shenzhen,4C:BC:48Cisco,4C:BC:72PrimexWi,4C:BC:98IEEERegi,4C:BC:98:00:00:00/28Charge-A,4C:BC:98:10:00:00/28Nic,4C:BC:98:20:00:00/28QuakeGlo,4C:BC:98:30:00:00/28MachineM,4C:BC:98:40:00:00/28Nemon,4C:BC:98:50:00:00/28Gronic,4C:BC:98:60:00:00/28Humanplu,4C:BC:98:70:00:00/28Voegtlin,4C:BC:98:80:00:00/28Shenzhen,4C:BC:98:90:00:00/28AirtexMa,4C:BC:98:A0:00:00/28Shenzhen,4C:BC:98:B0:00:00/28Dongguan,4C:BC:98:C0:00:00/28Heliotis,4C:BC:98:D0:00:00/28ElinkTec,4C:BC:98:E0:00:00/28WonderWo,4C:BC:A5SamsungE,4C:BC:B4ABB-DINR,4C:BC:E9LGInnote,4C:BD:8FHangzhou,4C:C0:0AvivoMobi,4C:C2:06Somfy,4C:C4:52ShangHai,4C:C5:3EZyxelCom,4C:C6:02Radios,4C:C6:81Shenzhen,4C:C7:D6Flextron,4C:C8:A1CiscoMer,4C:C9:4FNokia,4C:C9:5ESamsungE,4C:CA:53Skyera,4C:CB:F5zte,4C:CC:34Motorola,4C:CC:6AMicro-St,4C:CE:2DDanlaw,4C:D0:8AHUMAX,4C:D0:CBHuaweiTe,4C:D1:A1HuaweiTe,4C:D3:AFHMDGloba,4C:D5:77Chongqin,4C:D6:29HuaweiTe,4C:D6:37QsonoEle,4C:D7:B6HelmerSc,4C:D9:8FDell,4C:D9:C4MagnetiM,4C:DC:0DCoralTel,4C:DD:31SamsungE,4C:DD:7DLHPTelem,4C:DF:3DTeamEngi,4C:E1:73IEEERegi,4C:E1:73:00:00:00/28BeijingS,4C:E1:73:10:00:00/28Nexoforg,4C:E1:73:20:00:00/28LenovoDa,4C:E1:73:30:00:00/28outpaceI,4C:E1:73:40:00:00/28HuizhouD,4C:E1:73:50:00:00/28NewVaste,4C:E1:73:60:00:00/28DaikokuD,4C:E1:73:70:00:00/28Ersúles,4C:E1:73:80:00:00/28NanjingT,4C:E1:73:90:00:00/28Shenzhen,4C:E1:73:A0:00:00/28jvi,4C:E1:73:B0:00:00/28Shanghai,4C:E1:73:C0:00:00/28RemondeN,4C:E1:73:D0:00:00/28KtcK-Tel,4C:E1:73:E0:00:00/28PlusOneJ,4C:E1:75Cisco,4C:E1:76Cisco,4C:E1:9ETecnoMob,4C:E1:BBZhuhaiHi,4C:E2:F1Udino,4C:E5:AETianjinB,4C:E6:76Buffalo,4C:E9:33RailComm,4C:E9:E4NewH3CTe,4C:EA:AEGuangdon,4C:EB:42IntelCor,4C:EB:BDChongqin,4C:EB:D6Espressi,4C:EC:EFSoraa,4C:ED:DEAskeyCom,4C:ED:FBASUSTekC,4C:EE:B0SHCNetzw,4C:EF:56Shenzhen,4C:EF:C0AmazonTe,4C:F0:2EVifaDenm,4C:F1:9EGroupeAt,4C:F2:02XiaomiCo,4C:F2:BFCambridg,4C:F4:5BBlueClov,4C:F5:5BHuaweiTe,4C:F5:A0Scalable,4C:F5:DCHangzhou,4C:F7:37SamJiEle,4C:F9:5DHuaweiTe,4C:FA:CACambridg,4C:FB:45HuaweiTe,4C:FB:F4OptimalA,4C:FB:FESercommJ,4C:FC:AATesla,4C:FF:12FuzeEnte,50:00:84SiemensC,50:00:8CHongKong,50:01:6BHuaweiTe,50:01:BBSamsungE,50:01:D9HuaweiTe,50:02:91Espressi,50:04:B8HuaweiTe,50:05:3DCyWeeGro,50:06:04Cisco,50:06:ABCisco,50:09:59Technico,50:09:E5Drimsys,50:0A:52HuiwanTe,50:0B:32FoxdaTec,50:0B:91IEEERegi,50:0B:91:00:00:00/28Igor,50:0B:91:10:00:00/28SPDDevel,50:0B:91:20:00:00/28Annapurn,50:0B:91:30:00:00/28EwinTech,50:0B:91:40:00:00/28Sinopete,50:0B:91:50:00:00/28jiangsuz,50:0B:91:60:00:00/28Security,50:0B:91:70:00:00/28Shenzhen,50:0B:91:80:00:00/28Panasoni,50:0B:91:90:00:00/28Machfu,50:0B:91:A0:00:00/28NewAudio,50:0B:91:B0:00:00/28thumbzup,50:0B:91:C0:00:00/28DiamondT,50:0B:91:D0:00:00/28Shenzhen,50:0B:91:E0:00:00/28Shenzhen,50:0E:6DTrafficC,50:0F:59STMicrol,50:0F:80Cisco,50:0F:F5TendaTec,50:11:EBSilverNe,50:13:95SichuanA,50:14:08AiNET,50:14:79iRobot,50:14:B5RichfitI,50:17:FFCisco,50:18:4CPlatina,50:1A:A5GNNetcom,50:1A:C5Microsof,50:1B:32TaicangT,50:1C:B0Cisco,50:1C:BFCisco,50:1D:93HuaweiTe,50:1E:2DStreamUn,50:1F:C6Apple,50:20:6BEmersonC,50:21:ECHuaweiDe,50:22:67PixeLINK,50:25:2BNethraIm,50:26:90Fujitsu,50:27:C7TECHNART,50:28:73HuaweiDe,50:29:4DNanjingI,50:29:7BChinaMob,50:29:F5Guangdon,50:2A:7ESmartele,50:2A:8BTelekomR,50:2B:73TendaTec,50:2B:98Es-techI,50:2C:C6GreeElec,50:2D:1DNokia,50:2D:A2IntelCor,50:2D:BBGDMideaA,50:2D:F4PhytecMe,50:2D:FBIGShare,50:2E:5CHTC,50:2E:CEAsahiEle,50:2F:9BIntelCor,50:2F:A8Cisco,50:31:ADABBGloba,50:32:37Apple,50:32:5FSiliconL,50:32:75SamsungE,50:33:8BTexasIns,50:33:F0YichenSh,50:38:2FASEGroup,50:39:55CiscoSPV,50:3A:7DAlphaTec,50:3A:A0Shenzhen,50:3C:C4LenovoMo,50:3C:EAGuangdon,50:3D:A1SamsungE,50:3D:C6XiaomiCo,50:3D:E5Cisco,50:3D:EBZhejiang,50:3E:7CLeiShenI,50:3E:AATp-LinkT,50:3F:56Syncmold,50:3F:98Cmitech,50:40:61Nokia,50:41:1CAMPAKTec,50:41:B9I-ODataD,50:43:48ThingsMa,50:43:B9OktoInfo,50:45:F7LiuheInt,50:46:4AHuaweiTe,50:46:5DASUSTekC,50:48:EBBeijingH,50:49:B0SamsungE,50:4A:5EMasimo,50:4A:6ENetgear,50:4B:5BCONTROLt,50:4B:9EHuaweiDe,50:4C:7E41StInst,50:4E:DCPingComm,50:4F:94LoxoneEl,50:50:2AEgardia,50:50:65TAKT,50:50:A4SamsungE,50:50:CEHangzhou,50:51:A9TexasIns,50:52:3BNokia,50:52:D2Hangzhou,50:54:7BNanjingQ,50:55:27LGElectr,50:55:8DChinaMob,50:56:63TexasIns,50:56:A8Jolla,50:56:BFSamsungE,50:57:9CSeikoEps,50:57:A8Cisco,50:58:00WyTecInt,50:58:4Fwaytotec,50:58:6FHuaweiDe,50:59:67IntentSo,50:5A:C6Guangdon,50:5B:C2LiteonTe,50:5D:7Azte,50:5D:ACHuaweiTe,50:5F:B5AskeyCom,50:60:28Xirrus,50:61:84Avaya,50:61:BFCisco,50:61:D6Indu-Sol,50:61:F6Universa,50:62:55IEEERegi,50:62:55:00:00:00/28UfanetSC,50:62:55:10:00:00/28Hagiwara,50:62:55:20:00:00/28ShenZhen,50:62:55:30:00:00/28Hypertec,50:62:55:40:00:00/28XSLAB,50:62:55:50:00:00/28SuzhouRu,50:62:55:60:00:00/28Shenzhen,50:62:55:70:00:00/28AVTECHSo,50:62:55:80:00:00/28Rodaindu,50:62:55:90:00:00/28Southern,50:62:55:A0:00:00/28CCTVManu,50:62:55:B0:00:00/28ChengduC,50:62:55:C0:00:00/28AEDDistr,50:62:55:D0:00:00/28COTTElec,50:62:55:E0:00:00/28Shinsoft,50:63:13HonHaiPr,50:64:2BXIAOMIEl,50:64:41Greenlee,50:65:83TexasIns,50:65:F3HewlettP,50:67:87PlanetNe,50:67:AECisco,50:67:F0ZyxelCom,50:68:0AHuaweiTe,50:6A:03Netgear,50:6B:4BMellanox,50:6B:8DNutanix,50:6C:BEInnosili,50:6E:92Innocent,50:6F:0CSagemcom,50:6F:77HuaweiTe,50:6F:98SehajSyn,50:6F:9AWi-FiAll,50:70:43BSkyB,50:70:97ChinaMob,50:70:E5HeShanWo,50:72:24TexasIns,50:72:4DBEGBruec,50:75:F1ARRISGro,50:76:91Tekpea,50:76:A6EcilInfo,50:76:AFIntelCor,50:77:05SamsungE,50:78:B0HuaweiDe,50:78:B3zte,50:79:5BInterexp,50:7A:55Apple,50:7A:C5Apple,50:7B:9DLCFCHeFe,50:7C:6FIntelCor,50:7D:02Biodit,50:7E:5DArcadyan,50:80:4AQuectelW,50:81:40HP,50:82:D5Apple,50:84:92IntelCor,50:85:69SamsungE,50:87:89Cisco,50:87:B8Nuvyyo,50:89:65Shenzhen,50:8A:06TuyaSmar,50:8A:0FShenzhen,50:8A:42UptmateT,50:8A:CBShenzhen,50:8C:77DIRMEIER,50:8C:B1TexasIns,50:8C:F5ChinaMob,50:8D:6FCHAHOO,50:8E:49XiaomiCo,50:8F:4CXiaomiCo,50:92:B9SamsungE,50:93:4FGradualT,50:95:51ARRISGro,50:97:07XiamenPa,50:97:44Integrat,50:97:72Westingh,50:98:39XiaomiCo,50:98:71Inventum,50:98:B8NewH3CTe,50:98:F3RheemAus,50:9A:46Safetrus,50:9A:4CDell,50:9A:88HuaweiTe,50:9E:A7SamsungE,50:9F:27HuaweiTe,50:9F:3BOiElectr,50:A0:09XiaomiCo,50:A0:54Actineon,50:A0:A4Nokia,50:A0:BFAlbaFibe,50:A1:32Shenzhen,50:A4:C8SamsungE,50:A4:D0IEEERegi,50:A4:D0:00:00:00/28Traxens,50:A4:D0:10:00:00/28BeijingA,50:A4:D0:20:00:00/28Seneco,50:A4:D0:30:00:00/28Guangzho,50:A4:D0:40:00:00/28RavenInd,50:A4:D0:50:00:00/28TREXOM,50:A4:D0:60:00:00/28PointGra,50:A4:D0:70:00:00/28Shanghai,50:A4:D0:80:00:00/28XinLianA,50:A4:D0:90:00:00/28OemProdu,50:A4:D0:A0:00:00/28Changsha,50:A4:D0:B0:00:00/28ZhengDia,50:A4:D0:C0:00:00/28BeijingY,50:A4:D0:D0:00:00/28AxelTech,50:A4:D0:E0:00:00/28Sagetech,50:A5:DCARRISGro,50:A6:7FApple,50:A6:E3DavidCla,50:A7:15Aboundi,50:A7:2BHuaweiTe,50:A7:33RuckusWi,50:A8:3ASMobileD,50:A9:DESmartcom,50:AB:3EQibixx,50:AB:BFHoseoTel,50:AD:71Tessolve,50:AD:92NXTechno,50:AD:D5Dynalec,50:AE:86Linkinte,50:AF:4Dzte,50:AF:73Shenzhen,50:B3:63Digitron,50:B6:95Micropoi,50:B7:C3SamsungE,50:B8:88wi2beTec,50:B8:A2ImTechTe,50:BC:96Apple,50:BD:5FTp-LinkT,50:C0:06Carmanah,50:C0:F0ArtekMic,50:C2:71Securete,50:C2:E8CloudNet,50:C2:EDGNAudio,50:C3:A2nForeTec,50:C4:DDBuffalo,50:C5:8DJuniperN,50:C6:8EBiwinSem,50:C6:ADFiberhom,50:C7:09JuniperN,50:C7:BFTp-LinkT,50:C8:E5SamsungE,50:C9:71GNNetcom,50:C9:A0SkipperA,50:CC:F8SamsungE,50:CD:22Avaya,50:CD:32NanJingC,50:CE:75MeasyEle,50:CE:E3Gigafirm,50:D0:65ESYLUX,50:D2:13CviLux,50:D2:74Steffes,50:D2:F5BeijingX,50:D3:7FYuFlyMik,50:D4:F7Tp-LinkT,50:D5:9CThaiHabe,50:D6:D7Takahata,50:D7:53CONELCOM,50:DA:00Hangzhou,50:DA:D6XiaomiCo,50:DB:3FShenzhen,50:DC:E7AmazonTe,50:DC:FCEcocom,50:DD:4FAutomati,50:DE:06Apple,50:DE:19:00:00:00/28Telic,50:DE:19:10:00:00/28ClearFlo,50:DE:19:20:00:00/28Spii,50:DE:19:30:00:00/28Traxens,50:DE:19:40:00:00/28LangogoT,50:DE:19:50:00:00/28Bliq,50:DE:19:60:00:00/28Oceancct,50:DE:19:70:00:00/28TianjinN,50:DE:19:80:00:00/28Ivativ,50:DE:19:90:00:00/28AEGIdent,50:DE:19:A0:00:00/28TannakIn,50:DE:19:B0:00:00/28Brainwar,50:DE:19:C0:00:00/28Shenzhen,50:DE:19:D0:00:00/28PennyGil,50:DE:19:E0:00:00/28DTEN,50:DF:95Lytx,50:E0:39ZyxelCom,50:E0:85IntelCor,50:E0:C7TurContr,50:E0:EFNokia,50:E1:4APrivate,50:E2:4Ezte,50:E5:49Giga-Byt,50:E6:66Shenzhen,50:E7:A0RenesasE,50:E9:71Jibo,50:EA:D6Apple,50:EB:1ABrocadeC,50:EB:71IntelCor,50:EB:F6ASUSTekC,50:EC:50BeijingX,50:ED:3CApple,50:ED:78Changzho,50:ED:94EgatelSl,50:F0:03OpenStac,50:F0:D3SamsungE,50:F1:4ATexasIns,50:F4:3CLeeo,50:F4:EBApple,50:F5:20SamsungE,50:F5:DAAmazonTe,50:F6:1AKunshanJ,50:F7:22Cisco,50:F7:EDHuaweiDe,50:F8:A5eWBM,50:F9:08Wizardla,50:F9:58HuaweiDe,50:FA:84Tp-LinkT,50:FA:ABL-tekdoo,50:FB:19ChipseaT,50:FC:30Treehous,50:FC:9FSamsungE,50:FE:F2SifyTech,50:FF:20Keenetic,50:FF:99IEEERegi,50:FF:99:00:00:00/28Simicon,50:FF:99:10:00:00/28CoyoteSy,50:FF:99:20:00:00/28Shenzhen,50:FF:99:30:00:00/28Yongjing,50:FF:99:40:00:00/28IPCGloba,50:FF:99:50:00:00/28Garrison,50:FF:99:60:00:00/28LegendWi,50:FF:99:70:00:00/28Honeywel,50:FF:99:80:00:00/28DolphinC,50:FF:99:90:00:00/28SeaEagle,50:FF:99:A0:00:00/28metraTec,50:FF:99:B0:00:00/28SichuanD,50:FF:99:C0:00:00/28Goetting,50:FF:99:D0:00:00/28Shenzhen,50:FF:99:E0:00:00/28Informa,52:54:00RealtekU,52:54:4CNovell20,52:54:ABRealtekA,54:02:37Teltroni,54:03:84Hongkong,54:03:F5EBNTechn,54:04:96Gigawave,54:04:A6ASUSTekC,54:05:36Vivago,54:05:5FAlcatelL,54:05:93WOORIELE,54:05:DBLCFCHeFe,54:06:8BNingboDe,54:07:64HuaweiDe,54:09:10Apple,54:09:55zte,54:09:8Ddeistere,54:0D:F9HuaweiDe,54:0E:2DvivoMobi,54:0E:58Guangdon,54:0F:57SiliconL,54:10:31Smarto,54:10:ECMicrochi,54:11:2FSulzerPu,54:11:5FAtamo,54:13:10HuaweiTe,54:13:79HonHaiPr,54:14:73Wingtech,54:14:F3IntelCor,54:14:FDOrbbec3D,54:15:89MCSLogic,54:19:C8vivoMobi,54:1B:5DTechno-I,54:1D:61YEESTORM,54:1D:FBFreestyl,54:1E:56JuniperN,54:1F:8Dzte,54:1F:D5Advantag,54:20:18TelyLabs,54:21:1DHuaweiDe,54:21:60Alula,54:21:9DSamsungE,54:22:F8zte,54:25:EAHuaweiTe,54:26:96Apple,54:27:1EAzureWav,54:27:58Motorola,54:27:6CJiangsuH,54:27:8DNXPChina,54:2A:1BSonos,54:2A:9CLSYDefen,54:2A:A2AlphaNet,54:2B:57NightOwl,54:2B:8DApple,54:2B:DENewH3CTe,54:2C:EAProtectr,54:2F:89EuclidLa,54:2F:8ATellesco,54:31:31RasterVi,54:33:CBApple,54:34:EFHuaweiTe,54:35:30HonHaiPr,54:35:DFSymeo,54:36:9B1VergeIn,54:37:BBTaicangT,54:39:68Edgewate,54:39:DFHuaweiTe,54:3A:D6SamsungE,54:3B:30duagon,54:3D:37RuckusWi,54:3E:64Fiberhom,54:40:ADSamsungE,54:42:49Sony,54:44:08Nokia,54:44:A3SamsungE,54:46:17zte,54:46:6BShenzhen,54:47:41Xcheng,54:47:D3TsatAs,54:47:E8Syrotech,54:48:10Dell,54:48:9CCdoubles,54:48:E6BeijingX,54:49:DFPelotonI,54:4A:00Cisco,54:4A:05wenglors,54:4A:16TexasIns,54:4B:8CJuniperN,54:4E:45Private,54:4E:90Apple,54:51:1BHuaweiTe,54:51:46AMG,54:52:84HuaweiDe,54:53:EDSony,54:54:14DigitalR,54:54:CFProbedig,54:55:D5HuaweiDe,54:5A:A6Espressi,54:5E:BDNLTechno,54:5F:A9Teracom,54:60:09Google,54:61:72ZodiacAe,54:61:EAZaplox,54:62:E2Apple,54:64:D9Sagemcom,54:65:03QuectelW,54:65:DEARRISGro,54:66:6CShenzhen,54:66:F9ConMet,54:67:06Guangdon,54:67:51CompalBr,54:67:E6Shenzhen,54:69:25PsInodic,54:69:90HuaweiTe,54:6A:D8ElsterWa,54:6C:0ETexasIns,54:6C:EBIntelCor,54:6D:52TopviewO,54:6F:71uAvionix,54:71:DDHuaweiDe,54:72:4FApple,54:72:5EUnionman,54:73:98ToyoElec,54:74:E6WebtechW,54:75:95Tp-LinkT,54:75:D0Cisco,54:77:87EardaTec,54:77:8AHewlettP,54:78:1ACisco,54:79:75Nokia,54:7A:52CTEInter,54:7C:69Cisco,54:7D:40Powervis,54:7D:CDTexasIns,54:7F:54Ingenico,54:7F:A8TELCOsro,54:7F:BCiodyne,54:7F:EECisco,54:80:28HewlettP,54:81:2DPAXCompu,54:81:ADEagleRes,54:83:3AZyxelCom,54:84:7BDigitalD,54:84:DCzte,54:86:BCCisco,54:88:0ESamsungE,54:88:DECisco,54:89:22Zelfy,54:89:98HuaweiTe,54:8A:BACisco,54:8C:A0LiteonTe,54:8D:5AIntelCor,54:92:09HuaweiTe,54:92:BESamsungE,54:93:59Shenzhen,54:94:78Silversh,54:99:63Apple,54:9A:11IEEERegi,54:9A:11:00:00:00/28Shenzhen,54:9A:11:10:00:00/28SpearX,54:9A:11:20:00:00/28TorrapDe,54:9A:11:30:00:00/28RoyalBoo,54:9A:11:40:00:00/28eTauro,54:9A:11:50:00:00/28ElotechI,54:9A:11:60:00:00/28OrientDi,54:9A:11:70:00:00/28NiveoInt,54:9A:11:80:00:00/28Tite,54:9A:11:90:00:00/28Alfen,54:9A:11:A0:00:00/28VendNova,54:9A:11:B0:00:00/28EliteSil,54:9A:11:C0:00:00/28XianHuaF,54:9A:11:D0:00:00/28Hangzhou,54:9A:11:E0:00:00/28BeijingH,54:9A:11:F0:00:00/28Private,54:9A:16UzushioE,54:9A:4CGuangdon,54:9B:12SamsungE,54:9B:72Ericsson,54:9C:27PlasmaCl,54:9D:85EnerAcce,54:9F:06NokiaSha,54:9F:13Apple,54:9F:35Dell,54:9F:AEiBASEGam,54:9F:C6Cisco,54:A0:4Ft-macTec,54:A0:50ASUSTekC,54:A2:74Cisco,54:A3:1BShenzhen,54:A3:FABQTSolut,54:A4:93IEEERegi,54:A4:93:00:00:00/28Intellig,54:A4:93:10:00:00/28ShenZhen,54:A4:93:20:00:00/28genua,54:A4:93:30:00:00/28I-MoonTe,54:A4:93:40:00:00/28Shenzhen,54:A4:93:50:00:00/28Ausounds,54:A4:93:60:00:00/28HanntoTe,54:A4:93:70:00:00/28REDHydro,54:A4:93:80:00:00/28ChengduE,54:A4:93:90:00:00/28DoEasyIn,54:A4:93:A0:00:00/28WondersT,54:A4:93:B0:00:00/28Advice,54:A4:93:C0:00:00/28BjCotyte,54:A4:93:D0:00:00/28AssemTec,54:A4:93:E0:00:00/28Nederman,54:A5:1BHuaweiTe,54:A5:4BNSCCommu,54:A6:19Alcatel-,54:A6:5CTechnico,54:A6:DBHuaweiDe,54:A7:03Tp-LinkT,54:A9:D4Minibar,54:AB:3AQuantaCo,54:AE:27Apple,54:AE:D0DASANNet,54:AE:D2CSLDualc,54:AF:97TP-Link,54:B1:21HuaweiTe,54:B2:03Pegatron,54:B5:6CXianNova,54:B6:20SUHDOLE&,54:B7:53HunanFen,54:B7:E5RaysonTe,54:B8:02SamsungE,54:B8:0AD-LinkIn,54:BA:D6HuaweiTe,54:BD:79SamsungE,54:BE:53zte,54:BE:F7Pegatron,54:BF:64Dell,54:C2:50Iskratel,54:C3:3ECiena,54:C4:15Hangzhou,54:C4:80HuaweiTe,54:C5:7ASunnovoI,54:C8:0FTp-LinkT,54:C9:DFFn-LinkT,54:CD:10Panasoni,54:CD:A7FujianSh,54:CD:EEShenZhen,54:CE:69HikariTr,54:CE:82zte,54:CF:8DHuaweiTe,54:D0:B4XiamenFo,54:D0:EDAXIMComm,54:D1:63Max-Tech,54:D1:7DSamsungE,54:D1:B0Universa,54:D2:72NukiHome,54:D4:6FCiscoSPV,54:D7:51Proximus,54:D9:C6HuaweiDe,54:D9:E4Brillian,54:DB:A2Fibrain,54:DC:1DYulongCo,54:DE:D0Sevio,54:DF:00Ulterius,54:DF:24Fiberhom,54:DF:63Intrakey,54:E0:19Ring,54:E0:32JuniperN,54:E0:61SichuanT,54:E1:40Ingenico,54:E1:ADLCFCHeFe,54:E2:C8Dongguan,54:E2:E0ARRISGro,54:E3:B0JVLIndus,54:E3:F6Alcatel-,54:E4:3AApple,54:E4:A9BHRTech,54:E4:BDFn-LinkT,54:E6:1BApple,54:E6:3FShenZhen,54:E6:FCTp-LinkT,54:E7:D5SunCupid,54:EA:A8Apple,54:EC:2FRuckusWi,54:ED:A3Navdy,54:EE:75WistronI,54:EF:33Shenzhen,54:EF:44LumiUnit,54:EF:92Shenzhen,54:EF:FEFullpowe,54:F1:5FSichuanA,54:F2:01SamsungE,54:F2:94HuaweiDe,54:F5:B6Oriental,54:F6:07HuaweiDe,54:F6:66Berthold,54:F6:C5FujianSt,54:F6:E2HuaweiTe,54:F8:76Abb,54:FA:3ESamsungE,54:FA:96NokiaSol,54:FB:58WISEWARE,54:FC:F0SamsungE,54:FD:BFScheidtB,54:FF:82DavitSol,54:FF:CFMopriaAl,56:58:57AculabPl,58:00:BBJuniperN,58:00:E3LiteonTe,58:03:FBHangzhou,58:04:54IcommHk,58:04:CBTianjinH,58:05:28LabrisNe,58:05:56Elettron,58:08:FAFiberOpt,58:09:43Private,58:09:E5Kivic,58:0A:20Cisco,58:0A:D4Apple,58:10:8CIntelbra,58:10:B7Infinixm,58:12:43AcSiPTec,58:13:D3GemtekTe,58:16:26Avaya,58:16:D7Alpsalpi,58:17:0CSony,58:19:F8ARRISGro,58:1C:BDAffinegy,58:1D:91Advanced,58:1F:28HuaweiTe,58:1F:67Open-mte,58:1F:AAApple,58:1F:EFTuttnaer,58:20:59XiaomiCo,58:20:8AIEEERegi,58:20:8A:00:00:00/28Annapurn,58:20:8A:20:00:00/28MarsDigi,58:20:8A:30:00:00/28Aggregat,58:20:8A:40:00:00/28Tring,58:20:8A:50:00:00/28JiaHuang,58:20:8A:60:00:00/28Shangyin,58:20:8A:70:00:00/28pureLiFi,58:20:8A:80:00:00/28SAMILCTS,58:20:8A:90:00:00/28SuzhouRu,58:20:8A:A0:00:00/28Conducti,58:20:8A:B0:00:00/28InfodevE,58:20:8A:D0:00:00/28SamboHit,58:20:8A:E0:00:00/28UPMTechn,58:20:B1HewlettP,58:21:36KMBsro,58:21:E9Twpi,58:23:8CTechnico,58:24:29Google,58:25:75HuaweiTe,58:27:8CBuffalo,58:2A:F7HuaweiTe,58:2B:DBPax,58:2D:34Qingping,58:2E:FELighting,58:2F:40Nintendo,58:2F:42Universa,58:2F:F7Sagemcom,58:31:12Drust,58:32:77Reliance,58:34:3BGlovastT,58:35:26DeepletT,58:35:5DHuaweiDe,58:35:6BTecnoMob,58:35:D9Cisco,58:38:79Ricoh,58:3B:D9Fiberhom,58:3C:C6Omnealit,58:3F:54LGElectr,58:40:4EApple,58:41:20Tp-LinkT,58:42:E4BaxterIn,58:44:98XiaomiCo,58:45:4CEricsson,58:46:8FKoncarEl,58:46:E1BaxterIn,58:47:04Shenzhen,58:48:22Sony,58:48:49IEEERegi,58:48:49:00:00:00/28BeijingZ,58:48:49:10:00:00/28SKAARHOJ,58:48:49:20:00:00/28X-speedl,58:48:49:30:00:00/28ViperDes,58:48:49:40:00:00/28SernetSu,58:48:49:50:00:00/28HubeiShu,58:48:49:60:00:00/28Shenzhen,58:48:49:70:00:00/28Shandong,58:48:49:80:00:00/28STACKFOR,58:48:49:90:00:00/28Shenzhen,58:48:49:A0:00:00/28Waoo,58:48:49:B0:00:00/28Daatrics,58:48:49:C0:00:00/28Haag-Str,58:48:49:D0:00:00/28Telegaer,58:48:49:E0:00:00/28Avadesig,58:48:C0Coflec,58:49:25E3Enterp,58:49:3BPaloAlto,58:49:BAChitaiEl,58:4C:19Chongqin,58:4C:EEDigitalO,58:4D:42Dragos,58:50:76LinearEq,58:50:ABTLS,58:50:E6BestBuy,58:50:EDHangzhou,58:52:8AMitsubis,58:53:C0BeijingG,58:55:CAApple,58:56:C2HuaweiTe,58:56:E8ARRISGro,58:57:0DDanfossS,58:59:C2ExtremeN,58:5F:F6zte,58:60:5FHuaweiTe,58:60:D8ARRISGro,58:61:63QuantumN,58:63:56Fn-LinkT,58:63:9ATplSyste,58:65:E6infomark,58:66:BAHangzhou,58:67:1ABarnes&N,58:67:7FClareCon,58:68:5DTempoAus,58:69:6CRuijieNe,58:69:F9FusionTr,58:6A:B1Hangzhou,58:6B:14Apple,58:6C:25IntelCor,58:6D:8FCisco-Li,58:6E:D6Private,58:70:C6Shanghai,58:75:21CJSCRTSo,58:76:75BeijingE,58:76:C5DigiIS,58:7A:4DStonesof,58:7A:62TexasIns,58:7A:6AGuangdon,58:7B:E9AirProTe,58:7E:61QingdaoH,58:7F:57Apple,58:7F:66HuaweiTe,58:7F:B7SonarInd,58:7F:C8S2M,58:82:1DHSchomäc,58:82:A8Microsof,58:84:E4IP500All,58:85:6EQsc,58:85:A2RealmeCh,58:85:E9RealmeCh,58:86:94EFMNetwo,58:87:4CLite-OnC,58:87:E2Shenzhen,58:8A:5ADell,58:8B:F3ZyxelCom,58:8D:09Cisco,58:8D:64XianClev,58:8E:81SiliconL,58:90:43Sagemcom,58:91:53ChinaMob,58:91:CFIntelCor,58:92:0DKineticA,58:93:96RuckusWi,58:93:D8TexasIns,58:94:6BIntelCor,58:94:A2KETEK,58:94:AEHuaweiDe,58:94:B2BrainCo,58:94:CFVertexSt,58:95:D8IEEERegi,58:95:D8:00:00:00/28Shenzhen,58:95:D8:10:00:00/28shenzhen,58:95:D8:20:00:00/28Sercomm,58:95:D8:30:00:00/28TonnetTe,58:95:D8:40:00:00/28UnitySur,58:95:D8:50:00:00/28elgrisUG,58:95:D8:60:00:00/28NorgrenM,58:95:D8:70:00:00/28Epiphan,58:95:D8:80:00:00/28Shenzhen,58:95:D8:90:00:00/28Loftie,58:95:D8:A0:00:00/28PeakComm,58:95:D8:B0:00:00/28SuZhouRu,58:95:D8:C0:00:00/28LoctekEr,58:95:D8:D0:00:00/28Alunos,58:95:D8:E0:00:00/28Gmvsiste,58:96:1DIntelCor,58:96:30Technico,58:97:1ECisco,58:97:BDCisco,58:98:35Technico,58:98:6FRevoluti,58:9B:0BShineway,58:9B:4ADWnetTec,58:9C:FCFreeBSDF,58:9E:C6GigasetC,58:A0:23IntelCor,58:A0:CBTrackNet,58:A2:B5LGElectr,58:A4:8EPixArtIm,58:A6:39SamsungE,58:A7:6FiD,58:A8:39IntelCor,58:A8:7BFitbit,58:AC:78Cisco,58:AE:2BHuaweiDe,58:AE:A8HuaweiTe,58:AE:F1Fiberhom,58:B0:35Apple,58:B0:D4ZuniData,58:B0:FETeamEPS,58:B1:0FSamsungE,58:B3:FCShenzhen,58:B4:2DYSTenTec,58:B5:68Securita,58:B6:23BeijingX,58:B6:33RuckusWi,58:B9:61SOLEMEle,58:B9:E1Crystalf,58:BA:D4HuaweiTe,58:BC:27Cisco,58:BC:8FCognitiv,58:BD:A3Nintendo,58:BD:F9Sigrand,58:BE:72HuaweiTe,58:BF:25Espressi,58:BF:EACisco,58:C1:7ACambiumN,58:C2:32NEC,58:C3:8BSamsungE,58:C5:83ItelMobi,58:C5:CBSamsungE,58:C6:F0Guangdon,58:C7:ACNewH3CTe,58:C8:76ChinaMob,58:C9:35ChiunMai,58:CB:52Google,58:CE:2AIntelCor,58:CF:4BLufkinIn,58:D0:61HuaweiTe,58:D0:71BWBroadc,58:D0:8FIEEE1904,58:D3:49Apple,58:D3:91QuectelW,58:D5:0AMurataMa,58:D5:6ED-LinkIn,58:D6:7ATCPlink,58:D6:97Guangdon,58:D6:D3DairyChe,58:D7:59HuaweiTe,58:D9:C3Motorola,58:D9:D5TendaTec,58:DB:15TecnoMob,58:DB:8DFast,58:DC:6DExceptio,58:E0:2CMicroTec,58:E1:6CYingHuaI,58:E2:8FApple,58:E3:26CompassT,58:E4:76CentronC,58:E6:36EVRsafeT,58:E6:BAApple,58:E7:47Deltanet,58:E8:08Autonics,58:E8:73HANGZHOU,58:E8:76IEEERegi,58:E8:76:00:00:00/28ZhuhaiRa,58:E8:76:10:00:00/28BeijingP,58:E8:76:20:00:00/28CoalaLif,58:E8:76:30:00:00/28McWongIn,58:E8:76:40:00:00/28Probit,58:E8:76:50:00:00/28BroadAir,58:E8:76:60:00:00/28DivioTec,58:E8:76:70:00:00/28ChronosT,58:E8:76:80:00:00/28ChengduV,58:E8:76:90:00:00/28TEMMobil,58:E8:76:A0:00:00/28Shenzhen,58:E8:76:B0:00:00/28Annapurn,58:E8:76:C0:00:00/28KustomSi,58:E8:76:D0:00:00/28XiamenCa,58:E8:76:E0:00:00/28BaoruhEl,58:EA:FCELL-IoT,58:EB:14ProteusD,58:EC:E1Newport,58:EC:EDIntegrat,58:EE:CEIconTime,58:EF:68BelkinIn,58:F1:02BLUProdu,58:F2:FCHuaweiDe,58:F3:87Hccp,58:F3:9CCisco,58:F4:96SourceCh,58:F6:7BXiaMenUn,58:F6:BFKyotoUni,58:F9:87HuaweiTe,58:F9:8ESECUDOS,58:FB:84IntelCor,58:FB:96RuckusWi,58:FC:20AlticeLa,58:FC:73ArriaLiv,58:FC:C6Tozo,58:FC:DBIEEERegi,58:FC:DB:00:00:00/28SpangPow,58:FC:DB:10:00:00/28CertisTe,58:FC:DB:20:00:00/28BeseyeCl,58:FC:DB:30:00:00/28CustomBi,58:FC:DB:40:00:00/28InforceC,58:FC:DB:50:00:00/28Shenzhen,58:FC:DB:60:00:00/28TimexGro,58:FC:DB:70:00:00/28Promethe,58:FC:DB:80:00:00/28Shanghai,58:FC:DB:90:00:00/28Hi-Targe,58:FC:DB:A0:00:00/28Xmodus,58:FC:DB:B0:00:00/28SwarcoTr,58:FC:DB:C0:00:00/28ExcenonM,58:FC:DB:D0:00:00/28XiamenLe,58:FC:DB:E0:00:00/28AppliedD,58:FC:DB:F0:00:00/28Private,58:FD:20Systemho,58:FD:5DHangzhou,58:FD:B1LGElectr,58:FD:BEShenzhen,5A:03:BAWireless,5A:18:FFFiRaCons,5A:CB:D3Simaudio,5A:E6:60nyantec,5C:00:38ViasatGr,5C:02:14BeijingX,5C:02:6AAppliedV,5C:02:72SiliconL,5C:03:39HuaweiTe,5C:07:6FThoughtC,5C:09:47Apple,5C:09:79HuaweiTe,5C:0A:5BSamsungE,5C:0B:CATunstall,5C:0C:0EGuizhouH,5C:0C:BBCELIZION,5C:0C:E6Nintendo,5C:0E:8BExtremeN,5C:0F:FBAminoCom,5C:10:C5SamsungE,5C:11:93SealOne,5C:14:37Thyssenk,5C:15:15Advan,5C:15:E1AidcTech,5C:16:C7BigSwitc,5C:17:20HuaweiDe,5C:17:37I-ViewNo,5C:17:CFOnePlusT,5C:17:D3Lge,5C:18:B5TalonCom,5C:1A:6FCambridg,5C:1C:B9vivoMobi,5C:1D:D9Apple,5C:20:D0AsoniCom,5C:22:C4DaeEunEl,5C:23:16Squirrel,5C:24:43O-SungTe,5C:24:79Baltech,5C:25:4CAvireGlo,5C:26:0ADell,5C:26:23WaveLynx,5C:27:D4Shenzhen,5C:2A:EFr2pAsia-,5C:2B:F5VivintWi,5C:2E:59SamsungE,5C:2E:D2ABCXiShe,5C:31:3ETexasIns,5C:32:C5Teracom,5C:33:27SpazioIt,5C:33:5CSwisspho,5C:33:8EAlphaNet,5C:34:00HisenseE,5C:35:3BCompalBr,5C:35:DAThere,5C:36:B8TCLKingE,5C:38:E0Shanghai,5C:3A:3Dzte,5C:3A:45Chongqin,5C:3B:35Gehirn,5C:3C:27SamsungE,5C:40:58Jefferso,5C:41:5AAmazonco,5C:41:E7WiatecIn,5C:43:D2Hazemeye,5C:44:3ESkullcan,5C:45:27JuniperN,5C:46:B0SIMComWi,5C:49:79AVMAudio,5C:49:7DSamsungE,5C:4A:1FSichuanT,5C:4A:26EnguityT,5C:4C:A9HuaweiTe,5C:50:15Cisco,5C:50:D9Apple,5C:51:4FIntelCor,5C:51:81SamsungE,5C:51:88Motorola,5C:52:1ENintendo,5C:52:30Apple,5C:54:6DHuaweiTe,5C:55:78iryx,5C:56:A4WananHon,5C:56:ED3pleplay,5C:57:1AARRISGro,5C:57:C8Nokia,5C:58:19Jingshen,5C:58:E6PaloAlto,5C:59:48Apple,5C:5A:C7Cisco,5C:5A:EAFord,5C:5B:35Mist,5C:5B:C2YIK,5C:5E:ABJuniperN,5C:5F:67IntelCor,5C:61:99CloudNet,5C:62:5ACanon,5C:63:BFTp-LinkT,5C:63:C9Intellit,5C:64:7AHuaweiTe,5C:64:8EZyxelCom,5C:66:6CGuangdon,5C:67:76IDSImagi,5C:68:D0AuroraIn,5C:69:84Nuvico,5C:6A:7DKentkart,5C:6A:80ZyxelCom,5C:6B:32TexasIns,5C:6B:4FHello,5C:6B:D7FoshanVI,5C:6D:20HonHaiPr,5C:6F:4FSistel,5C:6F:69Broadcom,5C:70:17Apple,5C:70:A3LGElectr,5C:71:0DCisco,5C:75:AFFitbit,5C:75:C6ChinaMob,5C:76:95Technico,5C:77:57Haivisio,5C:77:76TCTmobil,5C:78:F8HuaweiDe,5C:7D:5EHuaweiTe,5C:7D:7DTechnico,5C:80:B6IntelCor,5C:81:A7NetworkD,5C:83:82Nokia,5C:83:8FCisco,5C:83:CDNewplatf,5C:84:86Brightso,5C:85:7EIEEERegi,5C:85:7E:00:00:00/2828Gorill,5C:85:7E:10:00:00/28SichuanC,5C:85:7E:20:00:00/28mobilogi,5C:85:7E:30:00:00/28CableMat,5C:85:7E:40:00:00/28Shenzhen,5C:85:7E:50:00:00/28Shanghai,5C:85:7E:60:00:00/28ProdataK,5C:85:7E:70:00:00/28BeijingH,5C:85:7E:80:00:00/28BeiJingX,5C:85:7E:90:00:00/28ExpressL,5C:85:7E:A0:00:00/28Zhejiang,5C:85:7E:B0:00:00/28HHCCPlan,5C:85:7E:C0:00:00/28Annapurn,5C:85:7E:D0:00:00/28NautechE,5C:85:7E:E0:00:00/28GuoyiLia,5C:85:F8Shenzhen,5C:86:13BeijingZ,5C:86:4ASecretLa,5C:86:5CSamsungE,5C:86:C1Dongguan,5C:87:30Apple,5C:87:78Cybertel,5C:87:9CIntelCor,5C:88:16Rockwell,5C:89:9ATp-LinkT,5C:89:D4BeijingB,5C:8A:38HewlettP,5C:8D:2DShanghai,5C:8D:4EApple,5C:8F:40TecnoMob,5C:8F:E0ARRISGro,5C:90:12OwlCyber,5C:91:57HuaweiTe,5C:91:FDJaewoncn,5C:92:5EZioncomE,5C:93:A2LiteonTe,5C:95:AEApple,5C:96:56AzureWav,5C:96:66SonyInte,5C:96:6ARtnet,5C:96:9DApple,5C:97:F3Apple,5C:99:60SamsungE,5C:9A:A1HuaweiDe,5C:9A:D8Fujitsu,5C:A1:76SichuanT,5C:A1:78TableTop,5C:A1:E0EmbedWay,5C:A3:9DSamsungE,5C:A3:EBLokelsro,5C:A4:8ACisco,5C:A4:A4Fiberhom,5C:A5:BCeero,5C:A6:2DCisco,5C:A6:E6TP-Link,5C:A7:21NewH3CTe,5C:A8:6AHuaweiTe,5C:A9:33LumaHome,5C:AA:FDSonos,5C:AC:4CHonHaiPr,5C:AD:76Shenzhen,5C:AD:CFApple,5C:AF:06LGElectr,5C:B0:0AHuaweiTe,5C:B0:66ARRISGro,5C:B1:3ESagemcom,5C:B1:5FOceanblu,5C:B2:9EASCOPowe,5C:B3:95HuaweiTe,5C:B3:F6Human,5C:B4:3EHuaweiTe,5C:B4:E2InspurSo,5C:B5:24Sony,5C:B5:59CNEXLabs,5C:B6:CCNovaComm,5C:B8:CBAllisCom,5C:B9:01HewlettP,5C:BA:2CHewlettP,5C:BA:37Microsof,5C:BA:EFChongqin,5C:BD:9AHuaweiDe,5C:BD:9EHongkong,5C:C0:A0HuaweiTe,5C:C1:D7SamsungE,5C:C2:13FrSauter,5C:C3:07HuaweiTe,5C:C3:36ittim,5C:C5:D4IntelCor,5C:C6:D0Skyworth,5C:C6:E9EdifierI,5C:C7:D7AzroadTe,5C:C8:E3ShintecH,5C:C9:99NewH3CTe,5C:C9:D3Palladiu,5C:CA:1AMicrosof,5C:CA:32Theben,5C:CA:D3ChipseaT,5C:CB:99SamsungE,5C:CB:CAFujianSt,5C:CC:A0Gridwiz,5C:CC:FFTechrout,5C:CD:5BIntelCor,5C:CD:7CMEIZUTec,5C:CE:ADCDYNE,5C:CF:7FEspressi,5C:D0:6EXiaomiCo,5C:D1:35XtremePo,5C:D2:0BYytek,5C:D2:E4IntelCor,5C:D4:1BUCZOONTe,5C:D4:ABZektor,5C:D5:B5Shenzhen,5C:D6:1FQardio,5C:D8:9EHuaweiDe,5C:D9:98D-Link,5C:DA:D4MurataMa,5C:DC:96Arcadyan,5C:DD:70Hangzhou,5C:DE:34Shenzhen,5C:DF:89RuckusWi,5C:DF:B8Shenzhen,5C:E0:C5IntelCor,5C:E0:CAFeiTianU,5C:E0:F6NICbr-Nu,5C:E1:76Cisco,5C:E2:23DelphinT,5C:E2:86NortelNe,5C:E2:8CZyxelCom,5C:E2:F4AcSiPTec,5C:E3:0EARRISGro,5C:E3:B6Fiberhom,5C:E4:2AIntelCor,5C:E5:0CBeijingX,5C:E7:47HuaweiTe,5C:E7:A0Nokia,5C:E7:BFNewSingu,5C:E8:83HuaweiTe,5C:E8:B7OraimoTe,5C:E8:EBSamsungE,5C:EA:1DHonHaiPr,5C:EB:4ERSTAHLHM,5C:EB:68Cheersta,5C:ED:8CHewlettP,5C:EE:79GlobalDi,5C:F2:07SpecoTec,5C:F2:86IEEERegi,5C:F2:86:00:00:00/28Hangzhou,5C:F2:86:10:00:00/28iSonTech,5C:F2:86:20:00:00/28Shanghai,5C:F2:86:30:00:00/28beijingy,5C:F2:86:40:00:00/28CHIPSEN,5C:F2:86:50:00:00/28EUROIMMU,5C:F2:86:60:00:00/28VPInstru,5C:F2:86:70:00:00/28AccessIS,5C:F2:86:80:00:00/28Shenzhen,5C:F2:86:90:00:00/28Shenzhen,5C:F2:86:A0:00:00/28UnforsRa,5C:F2:86:B0:00:00/28ItronUK,5C:F2:86:C0:00:00/28SunpetIn,5C:F2:86:D0:00:00/28BrightSk,5C:F2:86:E0:00:00/28DaisenEl,5C:F3:70CC&CTech,5C:F3:FCIBM,5C:F4:ABZyxelCom,5C:F5:0DInstitut,5C:F5:DAApple,5C:F6:DCSamsungE,5C:F7:C3SyntechH,5C:F7:E6Apple,5C:F8:21TexasIns,5C:F8:A1MurataMa,5C:F9:38Apple,5C:F9:6AHuaweiTe,5C:F9:DDDell,5C:F9:F0AtomosEn,5C:F9:FDTaicangT,5C:FA:FBAcubit,5C:FB:3AChongqin,5C:FB:7CShenzhen,5C:FC:66Cisco,5C:FE:9EWiwynnTa,5C:FF:35Wistron,5C:FF:FFShenzhen,60:01:94Espressi,60:02:92Pegatron,60:02:B4WistronN,60:03:08Apple,60:03:47BillionE,60:03:A6IntenoBr,60:04:17Posbank,60:05:8AHitachiM,60:06:E3Apple,60:07:7CJalaGrou,60:07:C4Guangdon,60:08:10HuaweiTe,60:08:37ivviScie,60:09:C3u-blox,60:0B:03Hangzhou,60:0F:77SilverPl,60:10:A2Crompton,60:11:99Siama,60:12:3CHuaweiTe,60:12:83TsbRealT,60:12:8BCanon,60:14:66zte,60:14:B3CyberTAN,60:15:92IEEERegi,60:15:92:00:00:00/28SLabssp,60:15:92:10:00:00/28RTDSTech,60:15:92:20:00:00/28EDATechn,60:15:92:30:00:00/28OsiTechn,60:15:92:40:00:00/28Zaptec,60:15:92:60:00:00/28BeijingK,60:15:92:70:00:00/28FasterCZ,60:15:92:80:00:00/28Yangzhou,60:15:92:90:00:00/28JiangsuS,60:15:92:A0:00:00/28insensiv,60:15:92:B0:00:00/28Annapurn,60:15:92:C0:00:00/28PSS,60:15:92:D0:00:00/28Remowire,60:15:92:E0:00:00/28Annapurn,60:15:C7IdaTech,60:18:03DaikinAi,60:18:2EShenZhen,60:18:88zte,60:18:95Dell,60:19:0CRramac,60:19:29Voltroni,60:19:70HuizhouQ,60:19:71ARRISGro,60:1D:0FMidniteS,60:1D:91Motorola,60:1D:9DSichuanA,60:1E:02EltexAla,60:21:01Guangdon,60:21:03I4Vine,60:21:C0MurataMa,60:22:32Ubiquiti,60:23:A4SichuanA,60:24:C1JiangsuZ,60:26:EFArubaaHe,60:27:1CVIDEOREH,60:29:D5DAVOLINK,60:2A:54CardioTe,60:2A:D0CiscoSPV,60:2E:20HuaweiTe,60:30:D4Apple,60:31:3BSunnovoI,60:31:97ZyxelCom,60:32:B1Tp-LinkT,60:32:F0Mplustec,60:33:4BApple,60:35:53BuwonTec,60:35:73EardaTec,60:35:C0Sfr,60:36:96Sapling,60:36:DDIntelCor,60:38:0EAlpsalpi,60:38:E0BelkinIn,60:39:1FABB,60:3A:7CTp-LinkT,60:3A:AFSamsungE,60:3C:EELGElectr,60:3D:26Technico,60:3D:29HuaweiTe,60:3E:7BGafachi,60:3E:CACambridg,60:3F:C5Cox,60:42:7FShenzhen,60:44:7AWater-id,60:44:F5EasyDigi,60:45:5ELiptelsr,60:45:BDMicrosof,60:45:CBASUSTekC,60:46:16XiamenVa,60:47:62BeijingS,60:47:D4FORICSEl,60:48:26Newbridg,60:49:C1Avaya,60:4A:1CSUYIN,60:4B:AAMagicLea,60:4D:E1HuaweiTe,60:50:C1KinetekS,60:51:2CTCTmobil,60:52:D0FACTSEng,60:53:17Sandston,60:53:75HuaweiTe,60:54:64EyedroGr,60:55:F9Espressi,60:56:61IXECLOUD,60:57:18IntelCor,60:57:7Deero,60:5B:B4AzureWav,60:5E:4FHuaweiDe,60:5F:8Deero,60:60:1FSzDjiTec,60:61:34GenesisT,60:61:DFZ-metaRe,60:63:4CD-LinkIn,60:63:F9Ciholas,60:63:FDTranscen,60:64:05TexasIns,60:64:53AOD,60:64:A1RADiflow,60:67:20IntelCor,60:68:4ESamsungE,60:69:44Apple,60:69:9Bisepos,60:6B:BDSamsungE,60:6B:FFNintendo,60:6C:63HitronTe,60:6C:66IntelCor,60:6D:3CLuxshare,60:6D:C7HonHaiPr,60:6E:D0Seal,60:6E:E8XiaomiCo,60:70:72Shenzhen,60:70:C0Apple,60:72:0BBLUProdu,60:73:5CCisco,60:73:BCzte,60:74:8DAtmacaEl,60:76:88Velodyne,60:77:71TexasIns,60:77:E2SamsungE,60:7D:DDShenzhen,60:7E:A4Shanghai,60:7E:C9Apple,60:7E:CDHuaweiTe,60:7E:DDMicrosof,60:81:2BAstronic,60:81:F9Helium,60:83:34HuaweiTe,60:83:73Apple,60:83:B2GkWareeK,60:84:3BSoladigm,60:84:BDBuffalo,60:86:45AveryWei,60:89:3CThermoFi,60:89:B1KeyDigit,60:89:B7KaelMühe,60:8A:10Microchi,60:8B:0EApple,60:8C:2BHansonTe,60:8C:4AApple,60:8C:DFPrivate,60:8C:E6ARRISGro,60:8D:17SentrusG,60:8D:26Arcadyan,60:8E:08SamsungE,60:8F:5CSamsungE,60:8F:A4NokiaSol,60:90:84DSSD,60:91:F3vivoMobi,60:92:17Apple,60:92:F5ARRISGro,60:93:16Apple,60:95:CEIEEERegi,60:95:CE:00:00:00/28SiemaApp,60:95:CE:10:00:00/28PonoorEx,60:95:CE:20:00:00/28Q-SENTEC,60:95:CE:30:00:00/28Robot,60:95:CE:40:00:00/28Untangle,60:95:CE:50:00:00/28AdvanWIS,60:95:CE:60:00:00/28XiamenSi,60:95:CE:70:00:00/28CadmoSol,60:95:CE:80:00:00/28TrophySA,60:95:CE:90:00:00/28Jlztlink,60:95:CE:A0:00:00/28UnManned,60:95:CE:B0:00:00/28BeijingS,60:95:CE:C0:00:00/28Synamedi,60:95:CE:D0:00:00/28GovComm,60:95:CE:E0:00:00/28VNS,60:96:20Private,60:97:DDMicroSys,60:98:13Shanghai,60:98:66TexasIns,60:99:D1VuzixLen,60:9A:A4GviSecur,60:9A:C1Apple,60:9B:2DJMACSJap,60:9B:B4HuaweiTe,60:9B:C8HipadInt,60:9C:9FBrocadeC,60:9E:64Vivonic,60:9F:9DCloudSwi,60:A1:0ASamsungE,60:A1:1EWuhanMax,60:A3:7DApple,60:A4:23SiliconL,60:A4:4CASUSTekC,60:A4:B7TP-Link,60:A4:D0SamsungE,60:A5:E2IntelCor,60:A7:30Shenzhen,60:A7:51HuaweiDe,60:A8:FENokiaSol,60:A9:B0Merchand,60:AA:EFHuaweiDe,60:AB:14LGInnote,60:AB:67XiaomiCo,60:AB:D2Bose,60:AC:C8KunTeng,60:AF:6DSamsungE,60:B1:85ATHsyste,60:B3:87Synergic,60:B3:C4Elber,60:B4:F7PlumeDes,60:B6:06Phorus,60:B6:17Fiberhom,60:B6:E1TexasIns,60:B7:6EGoogle,60:B9:33DeutronE,60:B9:82ROVERLab,60:BA:18nextLAP,60:BB:0CBeijingH,60:BC:4CEWMHight,60:BD:91MoveInno,60:BE:B5Motorola,60:BE:C4Apple,60:C0:BFONSemico,60:C1:CBFujianGr,60:C3:972Wire,60:C5:47Apple,60:C5:A8BeijingL,60:C5:ADSamsungE,60:C5:E6Skullcan,60:C6:58PHYTRONI,60:C7:98Verifone,60:C9:80Trymus,60:CB:FBAirScape,60:CD:A9Abloomy,60:CD:C5TaiwanCa,60:CE:41HuaweiTe,60:CE:86Sercomm,60:CE:92RefinedI,60:D0:2CRuckusWi,60:D0:A9SamsungE,60:D1:AAVishalTe,60:D2:1CSunnovoI,60:D2:48ARRISGro,60:D2:62Tzukuri,60:D2:B9RealandB,60:D2:DDShenzhen,60:D3:0AQuatius,60:D4:E9Guangdon,60:D7:55HuaweiTe,60:D7:E3IEEERegi,60:D7:E3:00:00:00/28Avalun,60:D7:E3:10:00:00/28Elap,60:D7:E3:20:00:00/28Novoinno,60:D7:E3:30:00:00/28SKSAutom,60:D7:E3:40:00:00/28Hemisphe,60:D7:E3:50:00:00/28RevolTec,60:D7:E3:60:00:00/28Ameli,60:D7:E3:70:00:00/28PhaseOne,60:D7:E3:80:00:00/28HindlePo,60:D7:E3:90:00:00/28LongSung,60:D7:E3:A0:00:00/28Wilderne,60:D7:E3:B0:00:00/28Nextivit,60:D7:E3:C0:00:00/28Zhejiang,60:D7:E3:D0:00:00/28Quantron,60:D7:E3:E0:00:00/28HuBDIC,60:D8:19HonHaiPr,60:D8:9CHMDGloba,60:D9:A0LenovoMo,60:D9:C7Apple,60:DA:23Estech,60:DA:83Hangzhou,60:DB:15NewH3CTe,60:DB:2AHns,60:DB:98Calix,60:DD:8EIntelCor,60:DE:35GITSN,60:DE:44HuaweiTe,60:DE:F3HuaweiTe,60:E0:0EShinseiE,60:E3:27Tp-LinkT,60:E3:2BIntelCor,60:E3:ACLGElectr,60:E6:BCSino-Tel,60:E6:F0WistronN,60:E7:01HuaweiTe,60:E7:8AUnisem,60:E8:5BTexasIns,60:E9:56AylaNetw,60:EB:5AAsterfus,60:EB:69QuantaCo,60:EE:5CShenzhen,60:EF:C6Shenzhen,60:F1:3DJABLOCOM,60:F1:89MurataMa,60:F1:8AHuaweiTe,60:F2:62IntelCor,60:F2:81TranwoTe,60:F2:EFVisionVe,60:F3:DALogicWay,60:F4:3AEdifierI,60:F4:45Apple,60:F4:94HonHaiPr,60:F5:9CCRU-Data,60:F6:73Terumo,60:F6:77IntelCor,60:F8:1DApple,60:F8:F2Synaptec,60:FA:9DHuaweiTe,60:FA:CDApple,60:FB:42Apple,60:FC:F1Private,60:FD:56WOORISYS,60:FE:1EChinaPal,60:FE:202Wire,60:FE:C5Apple,60:FE:F9ThomasBe,60:FF:12SamsungE,60:FF:DDCEElectr,64:00:2DPowerlin,64:00:6ADell,64:00:F1Cisco,64:01:FBLandis+G,64:02:CBARRISGro,64:03:7FSamsungE,64:05:BENewLight,64:05:E4Alpsalpi,64:05:E9Shenzhen,64:07:F6SamsungE,64:09:4CBeijingS,64:09:80XiaomiCo,64:09:ACTCTmobil,64:0B:4ADigitalT,64:0B:D7Apple,64:0D:22LGElectr,64:0D:CEShenzhen,64:0D:E6Petra,64:0E:36Taztag,64:0E:94Pluribus,64:0E:9BIshidaMe,64:0F:282Wire,64:10:84HEXIUMTe,64:12:25Cisco,64:12:36Technico,64:12:69ARRISGro,64:13:31BoschCar,64:13:6Czte,64:13:ABHuaweiTe,64:16:66NestLabs,64:16:7FPolycom,64:16:8DCisco,64:16:F0HuaweiTe,64:17:59Intelliv,64:1A:22Heliospe,64:1C:67Digibras,64:1C:AESamsungE,64:1C:B0SamsungE,64:1E:81Dowslake,64:20:0CApple,64:20:9FTilgin,64:20:E0T3Techno,64:21:84NipponDe,64:22:16Shandong,64:23:15HuaweiDe,64:24:00Xorcom,64:25:5EObservin,64:26:56Shenzhen,64:26:77BKM-Micr,64:27:37HonHaiPr,64:27:53HuaweiDe,64:29:EDAOPKKMil,64:2B:8AALLBESTI,64:2C:0FvivoMobi,64:2C:ACHuaweiTe,64:2D:B7SeungilE,64:31:39IEEERegi,64:31:39:00:00:00/28Shenzhen,64:31:39:10:00:00/28LivongoH,64:31:39:20:00:00/28Smartplu,64:31:39:30:00:00/28Koangyow,64:31:39:40:00:00/28ActiveBr,64:31:39:50:00:00/28Shenzhen,64:31:39:60:00:00/28HunanVoc,64:31:39:70:00:00/28Dongguan,64:31:39:80:00:00/28Shenzhen,64:31:39:A0:00:00/28ProductD,64:31:39:C0:00:00/28SHENZHEN,64:31:39:D0:00:00/28Zhejiang,64:31:39:E0:00:00/28ATGUVTec,64:31:50HewlettP,64:31:7EDexin,64:32:16WeiduTec,64:32:A8IntelCor,64:33:B5IEEERegi,64:33:B5:00:00:00/28Duomondi,64:33:B5:10:00:00/28HuaqinTe,64:33:B5:20:00:00/28Adesso,64:33:B5:30:00:00/28Wingtech,64:33:B5:40:00:00/28EagleEye,64:33:B5:50:00:00/28RevoSmar,64:33:B5:60:00:00/28Microit,64:33:B5:70:00:00/28ABBElect,64:33:B5:80:00:00/28LACOTech,64:33:B5:90:00:00/28Annapurn,64:33:B5:A0:00:00/28HometekE,64:33:B5:B0:00:00/28electroC,64:33:B5:C0:00:00/28Geksacon,64:33:B5:D0:00:00/28Iiyama,64:33:B5:E0:00:00/28Universi,64:33:DBTexasIns,64:34:09BITwaveP,64:35:1Ce-CONIND,64:3A:B1SichuanT,64:3A:EACisco,64:3E:8CHuaweiTe,64:3F:5FExablaze,64:42:12Shenzhen,64:42:14Swisscom,64:43:46GuangDon,64:44:D5Private,64:47:E0FeitianT,64:4B:C3Shanghai,64:4B:F0CalDigit,64:4C:36IntelCor,64:4D:70dSPACE,64:4F:42JETTER,64:4F:74LENUS,64:4F:B0Hyunjinc,64:50:D6Liquidto,64:51:06HewlettP,64:51:7ELongBenD,64:52:99Chamberl,64:53:5DFrausche,64:54:22EquinoxP,64:55:63Inteligh,64:55:7FNSFOCUSI,64:55:B1ARRISGro,64:56:01Tp-LinkT,64:57:E5BeijingR,64:58:ADChinaMob,64:59:F8Vodafone,64:5A:04ChiconyE,64:5A:36Apple,64:5A:EDApple,64:5C:F3ParanTek,64:5D:86IntelCor,64:5D:92SichuanT,64:5D:D7Shenzhen,64:5E:10HuaweiTe,64:5E:2CIRayTech,64:5E:BEYahoo!JA,64:5F:FFNicoletN,64:60:38Hirschma,64:61:40HuaweiDe,64:61:84Velux,64:62:23Cellient,64:62:66IEEERegi,64:62:66:00:00:00/28MiiViiDy,64:62:66:10:00:00/28Annapurn,64:62:66:20:00:00/28Protectl,64:62:66:30:00:00/28FaceHear,64:62:66:40:00:00/28Redstone,64:62:66:50:00:00/28Bühler,64:62:66:60:00:00/28PassSeym,64:62:66:70:00:00/28Shenzhen,64:62:66:80:00:00/28Leontech,64:62:66:90:00:00/28Chunghwa,64:62:66:A0:00:00/28Sensoro,64:62:66:B0:00:00/28SignalHo,64:62:66:C0:00:00/28JiangsuA,64:62:66:D0:00:00/28KobolInn,64:62:66:E0:00:00/28Shenzhen,64:62:8Aevon,64:64:4ABeijingX,64:64:9BJuniperN,64:65:C0Nuvon,64:66:24Sagemcom,64:66:B3Tp-LinkT,64:67:07BeijingO,64:68:0CComtrend,64:69:4ETexasIns,64:69:BCHyteraCo,64:6A:52Avaya,64:6A:74Auth-Ser,64:6C:80Chongqin,64:6C:B2SamsungE,64:6D:2FApple,64:6D:4EHuaweiTe,64:6D:6CHuaweiTe,64:6E:69LiteonTe,64:6E:6CRadioDat,64:6E:97Tp-LinkT,64:6E:E0IntelCor,64:6E:EAIskratel,64:70:02Tp-LinkT,64:70:33Apple,64:72:D8GooWiTec,64:73:66Shenzhen,64:73:E2Arbiter,64:74:F6ShooterD,64:76:57Innovati,64:76:BAApple,64:77:7DHitronTe,64:77:91SamsungE,64:79:24HuaweiDe,64:79:A7PhisonEl,64:79:F0IntelCor,64:7B:CESamsungE,64:7B:D4TexasIns,64:7C:34UbeeInte,64:7D:81YokotaIn,64:7F:DATEKTELIC,64:80:8BVGContro,64:80:99IntelCor,64:81:25Alphatro,64:87:88JuniperN,64:87:D7ADBBroad,64:88:FFSichuanC,64:89:9ALGElectr,64:89:F1SamsungE,64:8D:9EIVTElect,64:90:C1BeijingX,64:95:6CLGElectr,64:97:14eero,64:98:29Integrat,64:99:5DLge,64:99:68Elentec,64:99:A0Elektron,64:9A:08Shenzhen,64:9A:12P2Mobile,64:9A:BEApple,64:9B:24VTechnol,64:9C:81Qualcomm,64:9C:8ETexasIns,64:9D:99FsCom,64:9E:31BeijingX,64:9E:F3Cisco,64:9F:F7Kone,64:A0:E7Cisco,64:A1:98HuaweiDe,64:A2:00XiaomiCo,64:A2:32Samlight,64:A2:8AHuaweiDe,64:A2:F9OnePlusT,64:A3:41Wonderla,64:A3:CBApple,64:A5:C3Apple,64:A6:51HuaweiTe,64:A6:8FZhongsha,64:A7:69HTC,64:A7:DDAvaya,64:A8:37JuniKore,64:A9:65Linkflow,64:AE:0CCisco,64:AE:88Polytec,64:AE:F1QingdaoH,64:B0:A6Apple,64:B0:E8HuaweiDe,64:B2:1DChengduP,64:B3:10SamsungE,64:B3:70PowerCom,64:B3:79Private,64:B4:73XiaomiCo,64:B5:C6Nintendo,64:B6:23SchrackS,64:B6:4AViVOtech,64:B8:53SamsungE,64:B9:4EDellTech,64:B9:E8Apple,64:BA:BDSDJTechn,64:BC:0CLGElectr,64:BC:11CombiQ,64:BC:58IntelCor,64:BE:63STORDIS,64:BF:6BHuaweiTe,64:C2:DELGElectr,64:C3:54Avaya,64:C3:94HuaweiTe,64:C3:D6JuniperN,64:C4:03QuectelW,64:C5:AASouthAfr,64:C6:67Barnes&N,64:C6:AFAXERRANe,64:C7:53Apple,64:C9:01INVENTEC,64:C9:44LARKTech,64:CB:5DSIATeleS,64:CB:9FTecnoMob,64:CB:A3Pointmob,64:CB:E9LGInnote,64:CC:22Arcadyan,64:CC:2EXiaomiCo,64:CE:6ESierraWi,64:CF:D9TexasIns,64:D0:2DNextGene,64:D1:54Routerbo,64:D1:A3SitecomE,64:D2:41KeithKoe,64:D2:C4Apple,64:D4:BDAlpsalpi,64:D4:DAIntelCor,64:D7:C0HuaweiDe,64:D8:14Cisco,64:D9:12Solidica,64:D9:54TaicangT,64:D9:89Cisco,64:DA:A0RobertBo,64:DB:18OpenPatt,64:DB:43Motorola,64:DB:81Syszone,64:DB:8BHangzhou,64:DB:A0SelectCo,64:DC:01StaticGr,64:DC:DEZheJiang,64:DD:E9XiaomiCo,64:DE:1CKingneti,64:DF:10JingLueS,64:DF:E9Ateme,64:E0:03HuiZhouG,64:E0:ABUnionman,64:E1:61DEP,64:E1:72Shenzhen,64:E5:99EFMNetwo,64:E6:25WoxuWire,64:E6:82Apple,64:E7:D8SamsungE,64:E8:4FSerialwa,64:E8:81ArubaaHe,64:E8:92MorioDen,64:E8:E6globalmo,64:E9:50Cisco,64:EA:C5SiboTech,64:EB:8CSeikoEps,64:ED:57ARRISGro,64:ED:62WOORI,64:EE:B7NetcoreT,64:F2:42GerdesAk,64:F2:FBHangzhou,64:F5:0EKinionTe,64:F5:4EEMMicroe,64:F6:9DCisco,64:F6:BBFibocomW,64:F6:F7AnhuiDyn,64:F7:05HuaweiDe,64:F8:1CHuaweiTe,64:F8:8AChinaMob,64:F9:47Senscomm,64:F9:70KenadeEl,64:F9:87Avvasi,64:F9:C0AnalogDe,64:FB:50RoomRead,64:FB:81IEEERegi,64:FB:81:00:00:00/28Shanghai,64:FB:81:10:00:00/28Narrativ,64:FB:81:20:00:00/28SevenSol,64:FB:81:30:00:00/28MOBILUS,64:FB:81:40:00:00/28Pricer,64:FB:81:50:00:00/28KaySchul,64:FB:81:60:00:00/28XIMOComm,64:FB:81:70:00:00/28Securosy,64:FB:81:80:00:00/28NPGTechn,64:FB:81:90:00:00/28hiQview,64:FB:81:A0:00:00/28Bronkhor,64:FB:81:B0:00:00/28SichuanH,64:FB:81:C0:00:00/28Bridgepo,64:FB:81:D0:00:00/28Dongyang,64:FB:81:E0:00:00/28ChengDuK,64:FB:81:F0:00:00/28Private,64:FB:92PPCBroad,64:FC:8CZonar,64:FF:0AWistronN,68:02:35KontenNe,68:02:B8CompalBr,68:05:71SamsungE,68:05:CAIntelCor,68:07:0ATPVision,68:07:15IntelCor,68:09:27Apple,68:0A:D7Yancheng,68:0A:E2SiliconL,68:12:2DSpecialI,68:12:95LupineLi,68:13:24HuaweiDe,68:14:01HonHaiPr,68:15:90Sagemcom,68:15:D3ZakladyE,68:16:05AndElect,68:17:29IntelCor,68:19:3FDigitalA,68:19:ACGuangzho,68:1A:B2zte,68:1B:EFHuaweiTe,68:1C:A2Rosewill,68:1D:64SunwaveC,68:1D:EFShenzhen,68:1E:8BInfoSigh,68:1F:40BluWirel,68:1F:D8SiemensI,68:21:5FEdgecore,68:22:8EJuniperN,68:23:4BNihonDen,68:26:2ASichuanT,68:27:19Microchi,68:27:37SamsungE,68:28:BADejai,68:28:F6VubiqNet,68:29:DCFicosaEl,68:2C:7BCisco,68:2D:83Shenzhen,68:2D:DCWuhanCha,68:2F:67Apple,68:31:FETeladin,68:33:2CKenstelN,68:34:89LEAProfe,68:35:63Shenzhen,68:36:B5DriveSca,68:37:E9AmazonTe,68:39:43ittim,68:3A:1ECiscoMer,68:3A:48SAMJIN,68:3B:1ECountwis,68:3B:78Cisco,68:3C:7DMagicInt,68:3E:02SIEMENSD,68:3E:26IntelCor,68:3E:34MEIZUTec,68:3E:ECEreca,68:3F:1EEFFECTPh,68:3F:7DIngramMi,68:40:3CFiberhom,68:43:52Bhuu,68:43:D7Agilecom,68:45:71HuaweiDe,68:45:F1ToshibaC,68:47:49TexasIns,68:48:98SamsungE,68:49:B2CarloGav,68:4A:76eero,68:4A:AEHuaweiTe,68:4B:88Galtroni,68:4C:A8Shenzhen,68:4F:64Dell,68:51:B7PowerClo,68:52:D6UGameTec,68:53:6CSPnS,68:53:88P&STechn,68:54:5AIntelCor,68:54:C1ColorTok,68:54:EDAlcatel-,68:54:F5enLighte,68:54:FDAmazonTe,68:57:2DTuyaSmar,68:58:11Fiberhom,68:58:C5ZFTRWAut,68:59:7FAlcatelL,68:5A:CFSamsungE,68:5B:35Apple,68:5B:36Powertec,68:5D:43IntelCor,68:5E:6BPowerRay,68:63:50HellaInd,68:63:59Advanced,68:64:4BApple,68:67:25Espressi,68:69:2EZycoo,68:69:75AnglerLa,68:69:CAHitachi,68:69:F2ComApsro,68:6D:BCHangzhou,68:6E:23Wi3,68:6E:48ProphetE,68:72:51Ubiquiti,68:72:C3SamsungE,68:72:DCCETORYTV,68:76:27ZhuhaiDi,68:76:4FSony,68:77:24Tp-LinkT,68:78:48Westunit,68:78:4CNortelNe,68:79:12IEEERegi,68:79:12:00:00:00/28PCTEL,68:79:12:10:00:00/28Annapurn,68:79:12:20:00:00/28Cndi,68:79:12:30:00:00/28StephanE,68:79:12:40:00:00/28McDonald,68:79:12:50:00:00/28CopperLa,68:79:12:60:00:00/28Appotron,68:79:12:70:00:00/28Babbitan,68:79:12:80:00:00/28ShangHai,68:79:12:90:00:00/28LEAPSsro,68:79:12:A0:00:00/28Wingtech,68:79:12:B0:00:00/28Swisscom,68:79:12:C0:00:00/28GlobusIn,68:79:12:D0:00:00/28Neurolab,68:79:12:E0:00:00/28AmetekSo,68:79:24ELS,68:79:EDSHARP,68:7C:C8Measurem,68:7C:D5YSoft,68:7D:6BSamsungE,68:7D:B4Cisco,68:7F:74Cisco-Li,68:81:E0HuaweiTe,68:82:F2grandcen,68:83:1APandoraM,68:84:70eSSys,68:84:7EFujitsu,68:85:40IGIMobil,68:85:6AOuterLin,68:86:A7Cisco,68:86:E7Orbotix,68:87:6BINQMobil,68:87:C6Cisco,68:88:A1Universa,68:89:75nuoxc,68:89:C1HuaweiTe,68:8A:B5EDPServi,68:8A:F0zte,68:8B:0FChinaMob,68:8D:B6Aetek,68:8F:2EHitronTe,68:8F:84HuaweiTe,68:8F:C9Zhuolian,68:91:D0IEEERegi,68:91:D0:00:00:00/28CentralR,68:91:D0:10:00:00/28MultiAla,68:91:D0:20:00:00/28Shenzhen,68:91:D0:30:00:00/28Ambitio,68:91:D0:40:00:00/28G-TECHIn,68:91:D0:50:00:00/28NIPKElec,68:91:D0:60:00:00/28femrice,68:91:D0:70:00:00/28Omniimpe,68:91:D0:80:00:00/28solvimus,68:91:D0:90:00:00/28Quantex,68:91:D0:A0:00:00/28WiseCube,68:91:D0:B0:00:00/28AltisTec,68:91:D0:C0:00:00/28Spraying,68:91:D0:D0:00:00/28Fuzhoux-,68:91:D0:E0:00:00/28Outstand,68:92:34RuckusWi,68:93:20NewH3CTe,68:93:61Integrat,68:94:23HonHaiPr,68:96:6AOhsung,68:96:7BApple,68:97:4BShenzhen,68:97:E8SocietyM,68:98:61Beacon,68:99:CDCisco,68:9A:87AmazonTe,68:9A:B7AtelierV,68:9C:5EAcSiPTec,68:9C:70Apple,68:9C:E2Cisco,68:9E:0BCisco,68:9E:19TexasIns,68:9E:6AHuaweiDe,68:9F:F0zte,68:A0:3EHuaweiTe,68:A0:F6HuaweiTe,68:A1:B7HonghaoM,68:A3:78FreeboxS,68:A3:C4LiteonTe,68:A4:0EBSHHausg,68:A4:7DSunCupid,68:A6:82Shenzhen,68:A8:28HuaweiTe,68:A8:6DApple,68:A8:78GeoWAN,68:A8:E1Wacom,68:AA:D2Datecs,68:AB:09Nokia,68:AB:1EApple,68:AB:8ARFIDeas,68:AB:BCBeijingX,68:AE:20Apple,68:AF:13FuturaMo,68:AF:FFShanghai,68:B0:94InesaEle,68:B3:5EShenzhen,68:B4:3AWaterFur,68:B5:99HewlettP,68:B6:FCHitronTe,68:B8:D9ActKDE,68:B9:83b-plus,68:B9:D3Shenzhen,68:BC:0CCisco,68:BD:ABCisco,68:BF:C4SamsungE,68:C4:4DMotorola,68:C6:3AEspressi,68:C9:0BTexasIns,68:CA:00Octopus,68:CA:E4Cisco,68:CC:6EHuaweiTe,68:CC:9CMineSite,68:CD:0FUTek,68:CE:4EL-3Commu,68:D1:BAShenzhen,68:D1:FDShenzhen,68:D2:47Portalis,68:D4:82Shenzhen,68:D4:8BHailoTec,68:D6:EDGooWiWir,68:D7:9AUbiquiti,68:D9:25ProSysDe,68:D9:3CApple,68:DB:54PhicommS,68:DB:67NantongC,68:DB:96OPWILLTe,68:DB:CAApple,68:DB:F5AmazonTe,68:DC:E8PacketSt,68:DD:26Shanghai,68:DD:D9HMDGloba,68:DF:DDXiaomiCo,68:E1:66Private,68:E2:09HuaweiTe,68:E4:1FUnglaube,68:E4:78QingdaoH,68:E7:C2SamsungE,68:E8:EBLinktelT,68:EB:AESamsungE,68:EB:C5Angstrem,68:EC:62YODOTech,68:EC:8APrivate,68:EC:C5IntelCor,68:ED:43BlackBer,68:ED:A4Shenzhen,68:EE:96CiscoSPV,68:EF:43Apple,68:EF:BDCisco,68:F0:6DAlongInd,68:F0:BCShenzhen,68:F0:D0SkyBellT,68:F1:25DataCont,68:F3:8EJuniperN,68:F7:28LCFCHeFe,68:F8:95Redflow,68:F9:56Objetivo,68:FB:7EApple,68:FB:95Generalp,68:FC:B3NextLeve,68:FE:DAFiberhom,68:FE:F7Apple,68:FF:7BTp-LinkT,6A:0E:20GeoFrenz,6A:1F:6CiPass,6A:40:65OpenConn,6A:5C:35Secrétar,6A:73:7D25GEther,6A:8E:AFWaoo,6A:9D:A4GlobalRe,6A:B6:F2EliTechn,6A:E6:4AS&OElect,6A:E7:32UNIS-WDC,6C:00:6BSamsungE,6C:02:73Shenzhen,6C:02:E0HP,6C:03:09Cisco,6C:04:60RBHAcces,6C:05:D5Ethertro,6C:06:D6HuaweiDe,6C:09:0AGematica,6C:09:BFFiberhom,6C:09:D6Digiques,6C:0B:84Universa,6C:0D:34Nokia,6C:0D:C4BeijingX,6C:0D:E1Dongguan,6C:0E:0DSony,6C:0E:E6ChengduX,6C:0F:0BChinaMob,6C:0F:6AJDCTech,6C:10:8BWeLinkCo,6C:11:B3WuQiTech,6C:13:D5Cisco,6C:14:14BUJEONEL,6C:14:6EHuaweiTe,6C:14:F7Erhardt+,6C:15:F9Nautroni,6C:16:0EShotTrac,6C:16:32HuaweiTe,6C:18:11DecaturE,6C:19:8FD-LinkIn,6C:19:C0Apple,6C:1A:75HuaweiDe,6C:1B:3FMiraeSig,6C:1C:71Zhejiang,6C:1D:EBu-blox,6C:1E:70Guangzho,6C:1E:90HansolTe,6C:1E:D7vivoMobi,6C:20:56Cisco,6C:21:A2AMPAKTec,6C:22:ABAinswort,6C:23:B9Sony,6C:23:CBWattty,6C:24:83Microsof,6C:24:A6vivoMobi,6C:25:B9BbkEduca,6C:26:36HuaweiTe,6C:27:79Microsof,6C:29:90WiZConne,6C:29:95IntelCor,6C:2A:CBPaxtonAc,6C:2B:59Dell,6C:2C:06NPPSyste,6C:2C:DCSkyworth,6C:2D:24ZhenShiI,6C:2E:33Accelink,6C:2E:72B&BExpor,6C:2E:85Sagemcom,6C:2F:2CSamsungE,6C:2F:8ASamsungE,6C:31:0ECisco,6C:32:DEIndieonT,6C:33:A9Magicjac,6C:34:91HuaweiTe,6C:38:38MarkingS,6C:38:45Fiberhom,6C:38:A1UbeeInte,6C:39:1DBeijingZ,6C:3A:84Shenzhen,6C:3B:6BRouterbo,6C:3B:E5HewlettP,6C:3C:53SoundHaw,6C:3C:7CCanon,6C:3E:6DApple,6C:3E:9CKEKneste,6C:40:08Apple,6C:40:C6NimbusDa,6C:41:0ECisco,6C:41:6ACisco,6C:42:ABSubscrib,6C:43:3CTecnoMob,6C:44:18Zappware,6C:44:2AHuaweiTe,6C:45:98AntexEle,6C:47:60SunitecE,6C:49:C1o2ones,6C:4A:39Bita,6C:4A:74Aerodisk,6C:4A:85Apple,6C:4B:7FVossloh-,6C:4B:90LiteON,6C:4D:51Shenzhen,6C:4D:73Apple,6C:4E:86ThirdMil,6C:50:4DCisco,6C:54:CDLampexEl,6C:55:E8Technico,6C:56:40BLUProdu,6C:56:97AmazonTe,6C:57:79Aclima,6C:59:40MercuryC,6C:59:76Shanghai,6C:5A:34Shenzhen,6C:5A:B0TP-Link,6C:5A:B5TCLTechn,6C:5C:14Guangdon,6C:5C:3DIEEERegi,6C:5C:3D:00:00:00/28ShenZhen,6C:5C:3D:10:00:00/28Shenzhen,6C:5C:3D:20:00:00/28VertivIn,6C:5C:3D:30:00:00/28KwongMin,6C:5C:3D:40:00:00/28HTI,6C:5C:3D:50:00:00/28UnitelEn,6C:5C:3D:60:00:00/28Hangzhou,6C:5C:3D:70:00:00/28Soundkin,6C:5C:3D:80:00:00/28Guangzho,6C:5C:3D:90:00:00/28IskraUra,6C:5C:3D:A0:00:00/28krtkl,6C:5C:3D:B0:00:00/28Reconova,6C:5C:3D:C0:00:00/28choyangp,6C:5C:3D:D0:00:00/28Syowatsu,6C:5C:3D:E0:00:00/28ClintonE,6C:5C:DESunRepor,6C:5D:3AMicrosof,6C:5D:63ShenZhen,6C:5E:3BCisco,6C:5E:7AUbiquito,6C:5F:1CLenovoMo,6C:60:EBZhiYuanE,6C:61:26RinicomH,6C:61:F4Sfr,6C:62:6DMicro-St,6C:63:9CARRISGro,6C:64:1APenguinC,6C:6A:77IntelCor,6C:6C:D3Cisco,6C:6D:09KyowaEle,6C:6E:FECoreLogi,6C:6F:18Stereota,6C:70:39Novar,6C:70:9FApple,6C:71:0DCisco,6C:71:BDEzelinkT,6C:71:D2HuaweiTe,6C:71:D9AzureWav,6C:72:20D-LinkIn,6C:72:E7Apple,6C:75:0DWiFiSONG,6C:76:37HuaweiDe,6C:76:60Kyocera,6C:79:B8TexasIns,6C:81:FEMitsuba,6C:83:36SamsungE,6C:83:66NanjingS,6C:86:86Technoni,6C:88:14IntelCor,6C:8A:ECNantongC,6C:8B:2Fzte,6C:8B:D3Cisco,6C:8C:DBOtusTech,6C:8D:65Wireless,6C:8D:C1Apple,6C:8F:B5Microsof,6C:90:B1SanLogic,6C:91:06Private,6C:92:BFInspurEl,6C:93:54YaojinTe,6C:93:92BEKOTech,6C:94:66IntelCor,6C:94:F8Apple,6C:95:22Scalys,6C:96:CFApple,6C:98:EBRiverbed,6C:99:61Sagemcom,6C:99:89Cisco,6C:9A:C9Valentin,6C:9B:02Nokia,6C:9B:C0Chemopti,6C:9C:E9NimbleSt,6C:9C:EDCisco,6C:9E:7CFiberhom,6C:A0:B4BSkyB,6C:A1:00IntelCor,6C:A4:D1Fiberhom,6C:A6:04ARRISGro,6C:A6:82EDAMinfo,6C:A7:5Fzte,6C:A7:80Nokia,6C:A7:FAYoungboE,6C:A8:49Avaya,6C:A8:58Fiberhom,6C:A9:06Telefiel,6C:A9:28HMDGloba,6C:A9:36DisplayL,6C:A9:6FTransPac,6C:AA:B3RuckusWi,6C:AB:05Cisco,6C:AB:31Apple,6C:AB:4DDigitalP,6C:AC:60Venetex,6C:AD:3FHubbellB,6C:AD:ADChongqin,6C:AD:EFKZBroadb,6C:AD:F8AzureWav,6C:AE:8BIBM,6C:AE:F6eero,6C:AF:15WebastoS,6C:B0:CENetgear,6C:B0:FDShenzhen,6C:B2:27SonyVide,6C:B2:AECisco,6C:B3:11Shenzhen,6C:B3:50Anhuicom,6C:B4:A7Landauer,6C:B4:FDHuaweiDe,6C:B5:6BHUMAX,6C:B6:CADIVUS,6C:B7:49HuaweiTe,6C:B7:F4SamsungE,6C:B8:81zte,6C:B9:C5DeltaNet,6C:BA:B8Sagemcom,6C:BE:E9Alcatel-,6C:BF:B5NoonTech,6C:C1:47XiamenHa,6C:C1:D2ARRISGro,6C:C2:17HewlettP,6C:C2:6BApple,6C:C3:74TexasIns,6C:C4:9FArubaaHe,6C:C4:D5HMDGloba,6C:C6:3BTaicangT,6C:C7:ECSamsungE,6C:CA:08ARRISGro,6C:CD:D6Netgear,6C:CE:441More,6C:CF:39Guangdon,6C:D0:32LGElectr,6C:D1:46FRAMOS,6C:D1:B0WingSing,6C:D1:E5HuaweiTe,6C:D2:BAzte,6C:D3:EEZimi,6C:D6:30RootousS,6C:D6:8ALGElectr,6C:D7:04HuaweiTe,6C:D7:1FGuangdon,6C:D9:4CvivoMobi,6C:DC:6APromethe,6C:DD:30Cisco,6C:DD:BCSamsungE,6C:DD:EFEPCOMM,6C:DE:A9CiscoMer,6C:DF:FBIEEERegi,6C:DF:FB:00:00:00/28Shenzhen,6C:DF:FB:10:00:00/28Chongqin,6C:DF:FB:20:00:00/28Sercomm,6C:DF:FB:30:00:00/28BeijingA,6C:DF:FB:40:00:00/28Lineable,6C:DF:FB:50:00:00/28Greenbir,6C:DF:FB:60:00:00/28Aaon,6C:DF:FB:70:00:00/28Hashtren,6C:DF:FB:80:00:00/28Hardmeie,6C:DF:FB:90:00:00/28YongTech,6C:DF:FB:A0:00:00/28GuilinZh,6C:DF:FB:B0:00:00/28CELLSyst,6C:DF:FB:C0:00:00/28Toucan,6C:DF:FB:D0:00:00/28NanjingB,6C:DF:FB:E0:00:00/28BeijingF,6C:E0:1EModcam,6C:E0:B0Sound4,6C:E3:B6NeraTele,6C:E4:CEVilliger,6C:E4:DANECPlatf,6C:E5:F7NewH3CTe,6C:E8:5CApple,6C:E8:73Tp-LinkT,6C:E8:74HuaweiTe,6C:E8:C6EardaTec,6C:E9:07Nokia,6C:E9:83Gastron,6C:EB:B2Dongguan,6C:EB:B6HuaweiTe,6C:EC:5AHonHaiPr,6C:EC:A1Shenzhen,6C:EC:EBTexasIns,6C:ED:51NEXCONTR,6C:EF:C6Shenzhen,6C:F0:49Giga-Byt,6C:F1:7EZhejiang,6C:F3:73SamsungE,6C:F3:7FArubaaHe,6C:F5:E8Mooredol,6C:F7:12NokiaSol,6C:F7:84XiaomiCo,6C:F9:7CNanoptix,6C:F9:D2ChengduP,6C:FA:58Avaya,6C:FA:89Cisco,6C:FA:A7AMPAKTec,6C:FD:B9ProwareT,6C:FE:54IntelCor,6C:FF:BEMPBCommu,70:01:36FATEKAut,70:01:B5Cisco,70:02:5801Db-Met,70:03:7ETechnico,70:03:9FEspressi,70:04:33Californ,70:05:14LGElectr,70:06:ACEastcomp,70:07:77OnTarget,70:09:71SamsungE,70:0B:01Sagemcom,70:0B:4FCisco,70:0B:C0DewavTec,70:0F:6ACisco,70:0F:C7Shenzhen,70:0F:ECPoindus,70:10:5CCisco,70:10:6FHewlettP,70:11:24Apple,70:11:35Livesecu,70:11:AEMusicLif,70:14:04Liabilit,70:14:A6Apple,70:16:9FEtherCAT,70:18:8BHonHaiPr,70:18:A7Cisco,70:19:2FHuaweiTe,70:1A:04LiteonTe,70:1A:B8IntelCor,70:1A:D5Openpath,70:1A:EDAdvas,70:1B:FBIntegrat,70:1C:E7IntelCor,70:1D:0899IOTShe,70:1D:7FComtechT,70:1D:C4NorthSta,70:1E:68HannaIns,70:1F:0BWilogy,70:1F:3CSamsungE,70:1F:53Cisco,70:20:84HonHaiPr,70:23:93fos4X,70:25:26Nokia,70:25:59CyberTAN,70:26:05SONYVisu,70:28:8BSamsungE,70:29:00Shenzhen,70:2A:7DEpSpot,70:2A:D5SamsungE,70:2B:1DE-DomusI,70:2C:09Nintendo,70:2C:1FWisol,70:2D:84i4CInnov,70:2D:D1NewingsC,70:2E:22zte,70:2E:80DIEHLCon,70:2E:D9Guangzho,70:2F:35HuaweiTe,70:2F:4BSteelcas,70:2F:97AavaMobi,70:30:18Avaya,70:30:5DUbiquoss,70:30:5ENanjingZ,70:31:87ACX,70:32:17IntelCor,70:32:D5AthenaWi,70:35:09Cisco,70:38:11SiemensM,70:38:B4LowTechS,70:38:EEAvaya,70:3A:0EArubaaHe,70:3A:2DShenzhen,70:3A:51XiaomiCo,70:3A:73Shenzhen,70:3A:A6NewH3CTe,70:3A:CBGoogle,70:3A:D8Shenzhen,70:3C:03RadiAnt,70:3C:39SEAWINGK,70:3C:69Apple,70:3D:15Hangzhou,70:3E:97ItonTech,70:3E:ACApple,70:40:FFHuaweiDe,70:41:B7EdwardsL,70:44:1CShenzhen,70:46:42ChyngHon,70:47:E9vivoMobi,70:48:0FApple,70:48:F7Nintendo,70:4A:0EAMPAKTec,70:4A:AEXstreamF,70:4A:E4Rinstrum,70:4C:A5Fortinet,70:4C:EDTMRG,70:4D:7BASUSTekC,70:4E:01Kwangwon,70:4E:66Shenzhen,70:4E:6BHuaweiTe,70:4F:08Shenzhen,70:4F:57Tp-LinkT,70:4F:B8ARRISGro,70:50:AFBSkyB,70:52:C5Avaya,70:52:D8ItelMobi,70:53:3FAlfaInst,70:54:25ARRISGro,70:54:B4VestelEl,70:54:D2Pegatron,70:54:F5HuaweiTe,70:55:F8Cerebras,70:56:81Apple,70:57:BFNewH3CTe,70:58:12Panasoni,70:58:96InShowTe,70:59:57Medallio,70:59:86Ttv,70:5A:0FHewlettP,70:5A:9ETechnico,70:5A:ACSamsungE,70:5A:B6CompalIn,70:5B:2EM2Commun,70:5C:ADKonamiGa,70:5D:CCEFMNetwo,70:5E:55RealmeCh,70:5E:AAActionTa,70:5F:A3XiaomiCo,70:60:DELaVision,70:61:73Calantec,70:61:7BCisco,70:61:BEWistronN,70:61:EESunwodaE,70:62:B8D-LinkIn,70:64:17OrbisTec,70:65:82SuzhouHa,70:65:A3Kandaoli,70:66:1BSonova,70:66:55AzureWav,70:66:E1dntInnov,70:68:79SaijoDen,70:69:5ACisco,70:69:79IEEERegi,70:69:79:00:00:00/28FullSolu,70:69:79:10:00:00/28LinksysT,70:69:79:20:00:00/28Graphcor,70:69:79:30:00:00/28HebeiBai,70:69:79:40:00:00/28SelectTe,70:69:79:50:00:00/28Ibyte,70:69:79:60:00:00/28BeijingS,70:69:79:70:00:00/28Intelite,70:69:79:80:00:00/28AnPhatIn,70:69:79:90:00:00/28Faurecia,70:69:79:A0:00:00/28FoxconnB,70:69:79:B0:00:00/28LiquidIn,70:69:79:C0:00:00/28RivianAu,70:69:79:D0:00:00/28FreundEl,70:69:79:E0:00:00/28Bas-IpLp,70:6B:B9Cisco,70:6D:15Cisco,70:6D:ECWifi-sof,70:6E:6DCisco,70:6F:81Private,70:70:0DApple,70:70:4CPurpleCo,70:70:8BCisco,70:70:AAAmazonTe,70:71:B3Brain,70:71:BCPegatron,70:72:0DLenovoMo,70:72:3CHuaweiTe,70:72:CFEdgeCore,70:73:CBApple,70:74:14MurataMa,70:76:30ARRISGro,70:76:DDOxyGuard,70:76:F0LevelOne,70:76:FFKerlink,70:77:81HonHaiPr,70:78:8BvivoMobi,70:79:38WuxiZhan,70:79:90HuaweiTe,70:79:B3Cisco,70:7B:E8HuaweiTe,70:7C:18ADATATec,70:7C:69Avaya,70:7D:95Shenzhen,70:7D:B9Cisco,70:7E:43ARRISGro,70:7E:DENastec,70:81:05Cisco,70:81:EBApple,70:82:0Easelectr,70:82:8EOleumTec,70:85:40Skyworth,70:85:C2ASRockIn,70:85:C6ARRISGro,70:86:C1TexasIns,70:86:CEGDMideaA,70:87:9EBeken,70:88:4DJapanRad,70:88:6B:00:00:00/28Veracity,70:88:6B:10:00:00/28Bitfinde,70:88:6B:20:00:00/28CVnet,70:88:6B:40:00:00/28Hori,70:88:6B:50:00:00/28ChengduO,70:88:6B:60:00:00/28ChurchDw,70:88:6B:80:00:00/28CableMat,70:88:6B:90:00:00/28Shenzhen,70:88:6B:A0:00:00/28RHXTuneT,70:88:6B:B0:00:00/28BeijingS,70:88:6B:C0:00:00/28MAX4G,70:89:76TuyaSmar,70:89:CCChinaMob,70:89:F5Dongguan,70:8A:09HuaweiTe,70:8B:78citygrow,70:8B:CDASUSTekC,70:8C:B6HuaweiTe,70:8C:BBMimodisp,70:8D:09Nokia,70:8F:47vivoMobi,70:90:B7HuaweiDe,70:91:8FWeber-St,70:91:F3Universa,70:93:83Intellig,70:93:F8SpaceMon,70:97:41Arcadyan,70:97:56Happyele,70:99:1CShenzhen,70:9A:0BItalianI,70:9B:A5Shenzhen,70:9B:FCBryton,70:9C:8FNero,70:9C:D1IntelCor,70:9E:29SonyInte,70:9E:86X6D,70:9F:2Dzte,70:9F:A9TecnoMob,70:A1:91Trendset,70:A2:B3Apple,70:A4:1CAdvanced,70:A5:6AShenzhen,70:A6:6AProxDyna,70:A6:CCIntelCor,70:A7:41Ubiquiti,70:A8:4CMONAD,70:A8:E3HuaweiTe,70:AA:B2BlackBer,70:AC:D7Shenzhen,70:AD:54MalvernI,70:AF:24TPVision,70:AF:25Nishiyam,70:AF:6AShenzhen,70:B0:35Shenzhen,70:B0:8CShenouCo,70:B1:3DSamsungE,70:B1:4EARRISGro,70:B2:65Hiltron,70:B3:17Cisco,70:B3:D5IEEERegi,70:B3:D5:00:10:00/36SOREDIto,70:B3:D5:00:20:00/36GogoBA,70:B3:D5:00:30:00/36Anyroam,70:B3:D5:00:40:00/36Leidos,70:B3:D5:00:50:00/36CT,70:B3:D5:00:60:00/36PiranhaE,70:B3:D5:00:70:00/36Sensoneo,70:B3:D5:00:80:00/36ESYSEEmb,70:B3:D5:00:90:00/36HolidayC,70:B3:D5:00:A0:00/36FUJICOM,70:B3:D5:00:B0:00/36Axing,70:B3:D5:00:C0:00/36EXARAGro,70:B3:D5:00:D0:00/36Scrona,70:B3:D5:00:E0:00/36Magosys,70:B3:D5:00:F0:00/36NeusoftR,70:B3:D5:01:00:00/36HanwaEle,70:B3:D5:01:10:00/36SumerDat,70:B3:D5:01:20:00/36KSTtechn,70:B3:D5:01:30:00/36Sportsbe,70:B3:D5:01:40:00/36FRAKOKon,70:B3:D5:01:50:00/36ENElectr,70:B3:D5:01:60:00/36Guardian,70:B3:D5:01:70:00/36FTG,70:B3:D5:01:80:00/36Delitech,70:B3:D5:01:90:00/36TransitS,70:B3:D5:01:A0:00/36CubroAcr,70:B3:D5:01:B0:00/36Audi,70:B3:D5:01:C0:00/36KumuNetw,70:B3:D5:01:D0:00/36WeiglEle,70:B3:D5:01:E0:00/36ePOINTEm,70:B3:D5:01:F0:00/36SPXFlowT,70:B3:D5:02:00:00/36MicroDeb,70:B3:D5:02:10:00/36HGLDynam,70:B3:D5:02:20:00/36Ravelin,70:B3:D5:02:30:00/36Cambridg,70:B3:D5:02:40:00/36G+DMobil,70:B3:D5:02:50:00/36ElsuhdNe,70:B3:D5:02:60:00/36Telstra,70:B3:D5:02:70:00/36RedcapSo,70:B3:D5:02:80:00/36AT-Autom,70:B3:D5:02:90:00/36Marimoel,70:B3:D5:02:A0:00/36BAESurfa,70:B3:D5:02:B0:00/36Scorpion,70:B3:D5:02:C0:00/36Iylus,70:B3:D5:02:D0:00/36NEXTtec,70:B3:D5:02:E0:00/36Monnit,70:B3:D5:02:F0:00/36Legendai,70:B3:D5:03:00:00/36TresentT,70:B3:D5:03:10:00/36Shenzhen,70:B3:D5:03:20:00/36iFreecom,70:B3:D5:03:30:00/36Sailmon,70:B3:D5:03:40:00/36DigitalE,70:B3:D5:03:50:00/36HKW-Elek,70:B3:D5:03:60:00/36VemaVent,70:B3:D5:03:70:00/36EiffageE,70:B3:D5:03:80:00/36DONGILVI,70:B3:D5:03:90:00/36DoWooDig,70:B3:D5:03:A0:00/36Ochno,70:B3:D5:03:B0:00/36SSL-Elec,70:B3:D5:03:C0:00/36Ultimate,70:B3:D5:03:D0:00/36QuercusT,70:B3:D5:03:E0:00/36GuanShow,70:B3:D5:03:F0:00/36Elesar,70:B3:D5:04:00:00/36Savari,70:B3:D5:04:10:00/36Fibernet,70:B3:D5:04:20:00/36CovelozT,70:B3:D5:04:30:00/36cal4care,70:B3:D5:04:40:00/36DonElect,70:B3:D5:04:50:00/36NavaeroA,70:B3:D5:04:60:00/36Shenzhen,70:B3:D5:04:70:00/36Orion-R,70:B3:D5:04:80:00/36AvMapsrl,70:B3:D5:04:90:00/36APPEngin,70:B3:D5:04:A0:00/36GeckoRob,70:B3:D5:04:B0:00/36DreamISy,70:B3:D5:04:C0:00/36mapnagro,70:B3:D5:04:D0:00/36Sicon,70:B3:D5:04:E0:00/36HUGEL,70:B3:D5:04:F0:00/36EVPUDefe,70:B3:D5:05:00:00/36Compusig,70:B3:D5:05:10:00/36Jt,70:B3:D5:05:20:00/36SudoPrem,70:B3:D5:05:30:00/36Yamakats,70:B3:D5:05:40:00/36Groupeer,70:B3:D5:05:50:00/36Bae,70:B3:D5:05:60:00/36MiraeInf,70:B3:D5:05:70:00/36RchItali,70:B3:D5:05:80:00/36TelinkSe,70:B3:D5:05:90:00/36Pro-Digi,70:B3:D5:05:A0:00/36UniContr,70:B3:D5:05:B0:00/36PAL,70:B3:D5:05:C0:00/36AmberKin,70:B3:D5:05:D0:00/36KOMS,70:B3:D5:05:E0:00/36Vitec,70:B3:D5:05:F0:00/36UnisorMu,70:B3:D5:06:00:00/36RCHVietn,70:B3:D5:06:10:00/36IntelliD,70:B3:D5:06:20:00/36RMMichae,70:B3:D5:06:30:00/36PoolDigi,70:B3:D5:06:40:00/36Precisio,70:B3:D5:06:50:00/36Exatel,70:B3:D5:06:60:00/36NorthPol,70:B3:D5:06:70:00/36NeopathI,70:B3:D5:06:80:00/36Onethinx,70:B3:D5:06:90:00/36ONDEMAND,70:B3:D5:06:A0:00/36Guangdon,70:B3:D5:06:B0:00/36U-Tech,70:B3:D5:06:C0:00/36AppTek,70:B3:D5:06:D0:00/36Panorami,70:B3:D5:06:E0:00/36Global-K,70:B3:D5:06:F0:00/36BeijingD,70:B3:D5:07:00:00/36Lumiplan,70:B3:D5:07:10:00/36Fsr,70:B3:D5:07:20:00/36Lightdro,70:B3:D5:07:30:00/36LiteonTe,70:B3:D5:07:40:00/36OrlacoPr,70:B3:D5:07:50:00/36Mo-SysEn,70:B3:D5:07:60:00/36PrivateE,70:B3:D5:07:70:00/36InAccess,70:B3:D5:07:80:00/36OrbiWise,70:B3:D5:07:90:00/36CheckBil,70:B3:D5:07:A0:00/36Zeo,70:B3:D5:07:B0:00/36wallbe,70:B3:D5:07:C0:00/36Isac,70:B3:D5:07:D0:00/36Panorami,70:B3:D5:07:E0:00/36ENTECEle,70:B3:D5:07:F0:00/36Abalance,70:B3:D5:08:00:00/36Abb,70:B3:D5:08:10:00/36ISTTechn,70:B3:D5:08:20:00/36SakuraSe,70:B3:D5:08:30:00/36Zeo,70:B3:D5:08:40:00/36RakoCont,70:B3:D5:08:50:00/36HumanInt,70:B3:D5:08:60:00/36HustyMSt,70:B3:D5:08:70:00/36TempusFu,70:B3:D5:08:80:00/36OptiScan,70:B3:D5:08:90:00/36Kazdream,70:B3:D5:08:A0:00/36MBconnec,70:B3:D5:08:B0:00/36PeterHub,70:B3:D5:08:C0:00/36AirmarTe,70:B3:D5:08:D0:00/36CloverEl,70:B3:D5:08:E0:00/36BeijingC,70:B3:D5:08:F0:00/36DEUTA-WE,70:B3:D5:09:00:00/36Powercra,70:B3:D5:09:10:00/36PROFITT,70:B3:D5:09:20:00/36inomedMe,70:B3:D5:09:30:00/36LegrandE,70:B3:D5:09:40:00/36Circuitl,70:B3:D5:09:50:00/36plc-tec,70:B3:D5:09:60:00/36Havelsan,70:B3:D5:09:70:00/36AvantTec,70:B3:D5:09:80:00/36AlcodexT,70:B3:D5:09:90:00/36Schwer+K,70:B3:D5:09:A0:00/36Akse,70:B3:D5:09:B0:00/36Jacarta,70:B3:D5:09:C0:00/36Cardinal,70:B3:D5:09:D0:00/36PuSund,70:B3:D5:09:E0:00/36MobiProm,70:B3:D5:09:F0:00/36COMTECHK,70:B3:D5:0A:00:00/36Cominfo,70:B3:D5:0A:10:00/36PTNElect,70:B3:D5:0A:20:00/36TechSign,70:B3:D5:0A:30:00/36Solace,70:B3:D5:0A:40:00/36Communic,70:B3:D5:0A:50:00/36Fuelcell,70:B3:D5:0A:60:00/36PaConsul,70:B3:D5:0A:70:00/36Traffica,70:B3:D5:0A:80:00/36Symetric,70:B3:D5:0A:90:00/36ProConne,70:B3:D5:0A:A0:00/36Wanco,70:B3:D5:0A:B0:00/36KSTtechn,70:B3:D5:0A:C0:00/36RoboCore,70:B3:D5:0A:D0:00/36Vega-Abs,70:B3:D5:0A:E0:00/36NorsatIn,70:B3:D5:0A:F0:00/36KMtronic,70:B3:D5:0B:00:00/36RavenDes,70:B3:D5:0B:10:00/36AirBie,70:B3:D5:0B:20:00/36ndbtechn,70:B3:D5:0B:30:00/36ReonixAu,70:B3:D5:0B:40:00/36Aver,70:B3:D5:0B:50:00/36Capgemin,70:B3:D5:0B:60:00/36LandisGy,70:B3:D5:0B:70:00/36HAIROBOT,70:B3:D5:0B:80:00/36Lucas-Nü,70:B3:D5:0B:90:00/36EasyDigi,70:B3:D5:0B:A0:00/36AyreAcou,70:B3:D5:0B:B0:00/36AnaPico,70:B3:D5:0B:C0:00/36Practica,70:B3:D5:0B:D0:00/36Andium,70:B3:D5:0B:E0:00/36ChamSys,70:B3:D5:0B:F0:00/36DenAutom,70:B3:D5:0C:00:00/36MoluTech,70:B3:D5:0C:10:00/36NexusTec,70:B3:D5:0C:20:00/36LookEasy,70:B3:D5:0C:30:00/36AugWinkh,70:B3:D5:0C:40:00/36Tiama,70:B3:D5:0C:50:00/36Precitec,70:B3:D5:0C:60:00/36Embedded,70:B3:D5:0C:70:00/36PeekTraf,70:B3:D5:0C:80:00/36FinRobot,70:B3:D5:0C:90:00/36LineageP,70:B3:D5:0C:A0:00/36Vitec,70:B3:D5:0C:B0:00/36Nireco,70:B3:D5:0C:C0:00/36ADMiTASC,70:B3:D5:0C:D0:00/36AMLOcean,70:B3:D5:0C:E0:00/36Innomind,70:B3:D5:0C:F0:00/36sohonet,70:B3:D5:0D:00:00/36ProHound,70:B3:D5:0D:10:00/36CommonSe,70:B3:D5:0D:20:00/36Unmanned,70:B3:D5:0D:30:00/36TsatAs,70:B3:D5:0D:40:00/36Guangzho,70:B3:D5:0D:50:00/36KahlerAu,70:B3:D5:0D:60:00/36Tattile,70:B3:D5:0D:70:00/36RussianT,70:B3:D5:0D:80:00/36LaserIma,70:B3:D5:0D:90:00/36Brechbue,70:B3:D5:0D:A0:00/36Aquavisi,70:B3:D5:0D:B0:00/36Cryptotr,70:B3:D5:0D:C0:00/36Talleres,70:B3:D5:0D:D0:00/36Shenzhen,70:B3:D5:0D:E0:00/36Grossenb,70:B3:D5:0D:F0:00/36BEA,70:B3:D5:0E:00:00/36PLCiS,70:B3:D5:0E:10:00/36MiWaveCo,70:B3:D5:0E:20:00/36JESE,70:B3:D5:0E:30:00/36SinTau,70:B3:D5:0E:40:00/36WalterMü,70:B3:D5:0E:50:00/36DeltaSol,70:B3:D5:0E:60:00/36Nasdaq,70:B3:D5:0E:70:00/36PureAirF,70:B3:D5:0E:80:00/36Grossenb,70:B3:D5:0E:90:00/36VNTelect,70:B3:D5:0E:A0:00/36AEVBroad,70:B3:D5:0E:B0:00/36Tomahawk,70:B3:D5:0E:C0:00/36AcsMotio,70:B3:D5:0E:D0:00/36LupaTecn,70:B3:D5:0E:E0:00/36PictureE,70:B3:D5:0E:F0:00/36DexteraL,70:B3:D5:0F:00:00/36Avionica,70:B3:D5:0F:10:00/36BeijingO,70:B3:D5:0F:20:00/36TrexEdge,70:B3:D5:0F:30:00/36MonsoonR,70:B3:D5:0F:40:00/36VisualRo,70:B3:D5:0F:50:00/36SeasonEl,70:B3:D5:0F:60:00/36KSE,70:B3:D5:0F:70:00/36Bespoon,70:B3:D5:0F:80:00/36SpecialS,70:B3:D5:0F:90:00/36Research,70:B3:D5:0F:A0:00/36InsideRF,70:B3:D5:0F:B0:00/36Cygnus,70:B3:D5:0F:C0:00/36vitalcar,70:B3:D5:0F:D0:00/36UralFact,70:B3:D5:0F:E0:00/36Vocality,70:B3:D5:0F:F0:00/36Internet,70:B3:D5:10:00:00/36Gupsy,70:B3:D5:10:10:00/36AdolfNis,70:B3:D5:10:20:00/36OxfordMo,70:B3:D5:10:30:00/36Hanyoung,70:B3:D5:10:40:00/36Plumsp,70:B3:D5:10:50:00/36BeijingN,70:B3:D5:10:60:00/36AplexTec,70:B3:D5:10:70:00/36Alyans,70:B3:D5:10:80:00/36TexCompu,70:B3:D5:10:90:00/36DiTESTFa,70:B3:D5:10:A0:00/36SeasonDe,70:B3:D5:10:B0:00/36SECUREAN,70:B3:D5:10:C0:00/36Vocality,70:B3:D5:10:D0:00/36CoreELTe,70:B3:D5:10:E0:00/36Colorime,70:B3:D5:10:F0:00/36neQis,70:B3:D5:11:00:00/36OrionPow,70:B3:D5:11:10:00/36Leonardo,70:B3:D5:11:20:00/36DiTESTFa,70:B3:D5:11:30:00/36iREASyst,70:B3:D5:11:40:00/36ProjectH,70:B3:D5:11:50:00/36Welltec,70:B3:D5:11:60:00/36Momentum,70:B3:D5:11:70:00/36SysComAu,70:B3:D5:11:80:00/36Macromat,70:B3:D5:11:90:00/36Private,70:B3:D5:11:A0:00/36Private,70:B3:D5:11:B0:00/36HoseoTel,70:B3:D5:11:C0:00/36Samriddi,70:B3:D5:11:D0:00/36DaktonMi,70:B3:D5:11:E0:00/36Kbpr,70:B3:D5:11:F0:00/36Geppetto,70:B3:D5:12:00:00/36GSPSprac,70:B3:D5:12:10:00/36Shenzhen,70:B3:D5:12:20:00/36HenriHol,70:B3:D5:12:30:00/36Amfitech,70:B3:D5:12:40:00/36Forschun,70:B3:D5:12:50:00/36Securoly,70:B3:D5:12:60:00/36AddSecur,70:B3:D5:12:70:00/36Vitec,70:B3:D5:12:80:00/36Akse,70:B3:D5:12:90:00/36Microlin,70:B3:D5:12:A0:00/36Elvyssro,70:B3:D5:12:B0:00/36RICElect,70:B3:D5:12:C0:00/36Cielle,70:B3:D5:12:D0:00/36SEI,70:B3:D5:12:E0:00/36GreenFlu,70:B3:D5:12:F0:00/36DSP4YOU,70:B3:D5:13:00:00/36MG,70:B3:D5:13:10:00/36InovaDes,70:B3:D5:13:20:00/36HagenukK,70:B3:D5:13:30:00/36Vidisys,70:B3:D5:13:40:00/36ConjingN,70:B3:D5:13:50:00/36DorletSa,70:B3:D5:13:60:00/36MiguelCo,70:B3:D5:13:70:00/36SubjectL,70:B3:D5:13:80:00/36SMITEC,70:B3:D5:13:90:00/36Tunstall,70:B3:D5:13:A0:00/36DEUTA-WE,70:B3:D5:13:B0:00/36Sienna,70:B3:D5:13:C0:00/36Detec,70:B3:D5:13:D0:00/36Elsist,70:B3:D5:13:E0:00/36StaraInd,70:B3:D5:13:F0:00/36Farmobil,70:B3:D5:14:00:00/36VirtaLab,70:B3:D5:14:10:00/36MT,70:B3:D5:14:20:00/36Dave,70:B3:D5:14:30:00/36ATTechno,70:B3:D5:14:40:00/36GSElektr,70:B3:D5:14:50:00/36Sicon,70:B3:D5:14:60:00/363CityEle,70:B3:D5:14:70:00/36ROMOWind,70:B3:D5:14:80:00/36PowerEle,70:B3:D5:14:90:00/36eleven-x,70:B3:D5:14:A0:00/36ExSensTe,70:B3:D5:14:B0:00/36C21,70:B3:D5:14:C0:00/36Crde,70:B3:D5:14:D0:00/362-Observ,70:B3:D5:14:E0:00/36Innosoni,70:B3:D5:14:F0:00/36MobileDe,70:B3:D5:15:00:00/36YUYAMAMF,70:B3:D5:15:10:00/36VirsaeGr,70:B3:D5:15:20:00/36Xped,70:B3:D5:15:30:00/36Schneide,70:B3:D5:15:40:00/36WalkHori,70:B3:D5:15:50:00/36SanwaNew,70:B3:D5:15:60:00/36Rivercit,70:B3:D5:15:70:00/36Shanghai,70:B3:D5:15:80:00/36EAXLabss,70:B3:D5:15:90:00/36RCHVietn,70:B3:D5:15:A0:00/36Enabler,70:B3:D5:15:B0:00/36Armstron,70:B3:D5:15:C0:00/36WoodsHol,70:B3:D5:15:D0:00/36Vtron,70:B3:D5:15:E0:00/36SeasonEl,70:B3:D5:15:F0:00/36Savroni̇,70:B3:D5:16:00:00/36European,70:B3:D5:16:10:00/36MBconnec,70:B3:D5:16:20:00/36EspaiDeP,70:B3:D5:16:30:00/36BharatHe,70:B3:D5:16:40:00/36TokyoDra,70:B3:D5:16:50:00/36WuhanXin,70:B3:D5:16:60:00/36SerialIm,70:B3:D5:16:70:00/36Eiden,70:B3:D5:16:80:00/36BiwaveTe,70:B3:D5:16:90:00/36ServiceP,70:B3:D5:16:A0:00/364Jtechsr,70:B3:D5:16:B0:00/36IOTEngin,70:B3:D5:16:C0:00/36Ocean,70:B3:D5:16:D0:00/36BluB0XSe,70:B3:D5:16:E0:00/36JemacSwe,70:B3:D5:16:F0:00/36NimbeLin,70:B3:D5:17:00:00/36Mutelcor,70:B3:D5:17:10:00/36Aetina,70:B3:D5:17:20:00/36LumiGrow,70:B3:D5:17:30:00/36National,70:B3:D5:17:40:00/36CarlsonW,70:B3:D5:17:50:00/36Akribis,70:B3:D5:17:60:00/36Larraioz,70:B3:D5:17:70:00/36WiredBro,70:B3:D5:17:80:00/36GamberJo,70:B3:D5:17:90:00/36AltranUk,70:B3:D5:17:A0:00/36Gencoa,70:B3:D5:17:B0:00/36VistecEl,70:B3:D5:17:C0:00/36Farmpro,70:B3:D5:17:D0:00/36EntechEl,70:B3:D5:17:E0:00/36OculiVis,70:B3:D5:17:F0:00/36MBconnec,70:B3:D5:18:00:00/36LHA,70:B3:D5:18:10:00/36TaskSist,70:B3:D5:18:20:00/36KitronUA,70:B3:D5:18:30:00/36Evco,70:B3:D5:18:40:00/36XV360Opt,70:B3:D5:18:50:00/36R&DGran-,70:B3:D5:18:60:00/36Rohde&Sc,70:B3:D5:18:70:00/36Elektron,70:B3:D5:18:80:00/36BirketEn,70:B3:D5:18:90:00/36Dave,70:B3:D5:18:A0:00/36NSPEurop,70:B3:D5:18:B0:00/36AplexTec,70:B3:D5:18:C0:00/36CMCIndus,70:B3:D5:18:D0:00/36ForoTel,70:B3:D5:18:E0:00/36NipponSe,70:B3:D5:18:F0:00/36Newtec,70:B3:D5:19:00:00/36FantomWi,70:B3:D5:19:10:00/36AlgodueE,70:B3:D5:19:20:00/36Aspt,70:B3:D5:19:30:00/36EraToys,70:B3:D5:19:40:00/36HustyMSt,70:B3:D5:19:50:00/36Ci4Rail,70:B3:D5:19:60:00/36YUYAMAMF,70:B3:D5:19:70:00/36Lattech,70:B3:D5:19:80:00/36BeijingM,70:B3:D5:19:90:00/36SmartCon,70:B3:D5:19:A0:00/36WiSuiteU,70:B3:D5:19:B0:00/36GlobalTe,70:B3:D5:19:C0:00/36Kubu,70:B3:D5:19:D0:00/36Automata,70:B3:D5:19:E0:00/36J-Factor,70:B3:D5:19:F0:00/36KoizumiL,70:B3:D5:1A:00:00/36Ufatech,70:B3:D5:1A:10:00/36HMicro,70:B3:D5:1A:20:00/36XirgoTec,70:B3:D5:1A:30:00/36Telairit,70:B3:D5:1A:40:00/36DaveyBic,70:B3:D5:1A:50:00/36Metronic,70:B3:D5:1A:60:00/36Robotelf,70:B3:D5:1A:70:00/36ElkSolut,70:B3:D5:1A:80:00/36STCRainb,70:B3:D5:1A:90:00/36Oceanix,70:B3:D5:1A:A0:00/36EchoRidg,70:B3:D5:1A:B0:00/36AccessCo,70:B3:D5:1A:C0:00/36SVPBroad,70:B3:D5:1A:D0:00/36Techworl,70:B3:D5:1A:E0:00/36EcoG,70:B3:D5:1A:F0:00/36TeenageE,70:B3:D5:1B:00:00/36NALResea,70:B3:D5:1B:10:00/36Shanghai,70:B3:D5:1B:20:00/36CavagnaG,70:B3:D5:1B:30:00/36Graphcor,70:B3:D5:1B:40:00/365nines,70:B3:D5:1B:50:00/36StarBrid,70:B3:D5:1B:60:00/36DACOMWes,70:B3:D5:1B:70:00/36ULSee,70:B3:D5:1B:80:00/36OES,70:B3:D5:1B:90:00/36RELISTEG,70:B3:D5:1B:A0:00/36GuanShow,70:B3:D5:1B:B0:00/36EfentoTP,70:B3:D5:1B:C0:00/36Flextron,70:B3:D5:1B:D0:00/36Shenzhen,70:B3:D5:1B:E0:00/36PotterEl,70:B3:D5:1B:F0:00/36DEUTA-WE,70:B3:D5:1C:00:00/36WHLeary,70:B3:D5:1C:10:00/36Sphereec,70:B3:D5:1C:20:00/36CENSISUi,70:B3:D5:1C:30:00/36Shanghai,70:B3:D5:1C:40:00/36Smeg,70:B3:D5:1C:50:00/36Elsag,70:B3:D5:1C:60:00/36Bita-Int,70:B3:D5:1C:70:00/36HoshinEl,70:B3:D5:1C:80:00/36LDAaudio,70:B3:D5:1C:90:00/36MBconnec,70:B3:D5:1C:A0:00/36inomatic,70:B3:D5:1C:B0:00/36MatchX,70:B3:D5:1C:C0:00/36AooGeeCo,70:B3:D5:1C:D0:00/36ELEUSI,70:B3:D5:1C:E0:00/36ClearFlo,70:B3:D5:1C:F0:00/36Dalcnet,70:B3:D5:1D:00:00/36Shenzhen,70:B3:D5:1D:10:00/36Eurotek,70:B3:D5:1D:20:00/36Xacti,70:B3:D5:1D:30:00/36Airobot,70:B3:D5:1D:40:00/36Brinkman,70:B3:D5:1D:50:00/36MIVOTech,70:B3:D5:1D:60:00/36MacGrayS,70:B3:D5:1D:70:00/36Private,70:B3:D5:1D:80:00/36BlueSkie,70:B3:D5:1D:90:00/36MondeF,70:B3:D5:1D:A0:00/36Promess,70:B3:D5:1D:B0:00/36HudsonRo,70:B3:D5:1D:C0:00/36TEKVEL,70:B3:D5:1D:D0:00/36RfCreati,70:B3:D5:1D:E0:00/36Dycec,70:B3:D5:1D:F0:00/36ENTECEle,70:B3:D5:1E:00:00/36TOPROOTT,70:B3:D5:1E:10:00/36TexCompu,70:B3:D5:1E:20:00/36Shenzhen,70:B3:D5:1E:30:00/36HatelEle,70:B3:D5:1E:40:00/36Tecnolog,70:B3:D5:1E:50:00/36VendNova,70:B3:D5:1E:60:00/36SanminaI,70:B3:D5:1E:70:00/36DogWatch,70:B3:D5:1E:80:00/36GogoBA,70:B3:D5:1E:90:00/36comtime,70:B3:D5:1E:A0:00/36SenseFor,70:B3:D5:1E:B0:00/36Xavant,70:B3:D5:1E:D0:00/36SUS,70:B3:D5:1E:E0:00/36Meggitt,70:B3:D5:1E:F0:00/36Adtek,70:B3:D5:1F:00:00/36Harmonic,70:B3:D5:1F:10:00/36DIEHLCon,70:B3:D5:1F:20:00/36YUYAMAMF,70:B3:D5:1F:30:00/36SmartEne,70:B3:D5:1F:40:00/36Hangzhou,70:B3:D5:1F:50:00/36Martec,70:B3:D5:1F:60:00/36LinkAVTe,70:B3:D5:1F:70:00/36MorganSc,70:B3:D5:1F:80:00/36Converge,70:B3:D5:1F:90:00/36Automata,70:B3:D5:1F:A0:00/36EBZSysTe,70:B3:D5:1F:B0:00/36Crane-el,70:B3:D5:1F:C0:00/36GuanShow,70:B3:D5:1F:D0:00/36BRSSiste,70:B3:D5:1F:E0:00/36MobiProm,70:B3:D5:1F:F0:00/36Audiodo,70:B3:D5:20:00:00/36NextEV,70:B3:D5:20:10:00/36Leontech,70:B3:D5:20:20:00/36DEUTA-WE,70:B3:D5:20:30:00/36WOOJIN,70:B3:D5:20:40:00/36Twc,70:B3:D5:20:50:00/36Esource,70:B3:D5:20:60:00/36ard,70:B3:D5:20:70:00/36Savari,70:B3:D5:20:80:00/36DspDesig,70:B3:D5:20:90:00/36SmartNod,70:B3:D5:20:A0:00/36GoldenGr,70:B3:D5:20:B0:00/36KSTtechn,70:B3:D5:20:C0:00/36SiemensH,70:B3:D5:20:D0:00/36EngageTe,70:B3:D5:20:E0:00/36AmrehnPa,70:B3:D5:20:F0:00/36TielineR,70:B3:D5:21:00:00/36EastoneC,70:B3:D5:21:10:00/36Fracarro,70:B3:D5:21:20:00/36Semicons,70:B3:D5:21:30:00/36ETONDeut,70:B3:D5:21:40:00/36signalpa,70:B3:D5:21:50:00/36Dataspee,70:B3:D5:21:60:00/36Flextron,70:B3:D5:21:70:00/36TecnintH,70:B3:D5:21:80:00/36Gremeshc,70:B3:D5:21:90:00/36D-E-K,70:B3:D5:21:A0:00/36Acutroni,70:B3:D5:21:B0:00/36Lab241,70:B3:D5:21:C0:00/36Enyx,70:B3:D5:21:D0:00/36iRF-Inte,70:B3:D5:21:E0:00/36Hildebra,70:B3:D5:21:F0:00/36Chronome,70:B3:D5:22:00:00/36Private,70:B3:D5:22:10:00/36LXDesign,70:B3:D5:22:20:00/36Marioff,70:B3:D5:22:30:00/36Research,70:B3:D5:22:40:00/36UrbanaSm,70:B3:D5:22:50:00/36RCDRadio,70:B3:D5:22:60:00/36Yaviar,70:B3:D5:22:70:00/36Montalvo,70:B3:D5:22:80:00/36Heitec,70:B3:D5:22:90:00/36CONTROL,70:B3:D5:22:A0:00/36Shishido,70:B3:D5:22:B0:00/36Vitec,70:B3:D5:22:C0:00/36HiquelEl,70:B3:D5:22:D0:00/36LederEle,70:B3:D5:22:E0:00/36Private,70:B3:D5:22:F0:00/36Instec,70:B3:D5:23:00:00/36CT,70:B3:D5:23:10:00/36DeltaTau,70:B3:D5:23:20:00/36Uconsys,70:B3:D5:23:40:00/36EDFelect,70:B3:D5:23:50:00/36Cameon,70:B3:D5:23:60:00/36Monnit,70:B3:D5:23:70:00/36SikomAS,70:B3:D5:23:80:00/36AreteAss,70:B3:D5:23:90:00/36AppliedS,70:B3:D5:23:A0:00/36MesaLabs,70:B3:D5:23:B0:00/36FinkTele,70:B3:D5:23:C0:00/36Quasonix,70:B3:D5:23:D0:00/36CircleCo,70:B3:D5:23:E0:00/36TornadoM,70:B3:D5:23:F0:00/36Eta-Usa,70:B3:D5:24:00:00/36OrlacoPr,70:B3:D5:24:10:00/36BolideTe,70:B3:D5:24:20:00/36ComeoTec,70:B3:D5:24:30:00/36Rohde&Sc,70:B3:D5:24:40:00/36DATInfor,70:B3:D5:24:50:00/36Newtec,70:B3:D5:24:60:00/36SalineLe,70:B3:D5:24:70:00/36SatskyCo,70:B3:D5:24:80:00/36GlTech,70:B3:D5:24:90:00/36Kospel,70:B3:D5:24:A0:00/36UnmuktiT,70:B3:D5:24:B0:00/36ToseiEng,70:B3:D5:24:C0:00/36Astronom,70:B3:D5:24:D0:00/36InfoCrea,70:B3:D5:24:E0:00/36ChengduC,70:B3:D5:24:F0:00/36ElbitBmd,70:B3:D5:25:00:00/36DatumEle,70:B3:D5:25:10:00/36PixelApp,70:B3:D5:25:20:00/36SierraNe,70:B3:D5:25:30:00/36WimateTe,70:B3:D5:25:40:00/36Spectrum,70:B3:D5:25:50:00/36Asystems,70:B3:D5:25:60:00/36TelcoAnt,70:B3:D5:25:70:00/36LGElectr,70:B3:D5:25:80:00/36BAYKONEn,70:B3:D5:25:90:00/36ZebraEle,70:B3:D5:25:A0:00/36DEUTA-WE,70:B3:D5:25:B0:00/36GIDIndus,70:B3:D5:25:C0:00/36ArclanSy,70:B3:D5:25:D0:00/36MimoNetw,70:B3:D5:25:E0:00/36Rfhic,70:B3:D5:25:F0:00/36Copperni,70:B3:D5:26:00:00/36ModuSyst,70:B3:D5:26:10:00/36PotterEl,70:B3:D5:26:20:00/36Research,70:B3:D5:26:30:00/36Axing,70:B3:D5:26:40:00/36ifaktech,70:B3:D5:26:50:00/36Rapiot,70:B3:D5:26:60:00/36SpectraD,70:B3:D5:26:70:00/36Zehntner,70:B3:D5:26:80:00/36Cardinal,70:B3:D5:26:90:00/36Gilbarco,70:B3:D5:26:A0:00/36Talleres,70:B3:D5:26:B0:00/36Sorama,70:B3:D5:26:C0:00/36EAElektr,70:B3:D5:26:D0:00/36SorionEl,70:B3:D5:26:E0:00/36HI-TECHS,70:B3:D5:26:F0:00/36CompalEl,70:B3:D5:27:00:00/36AmazonTe,70:B3:D5:27:10:00/36CodeBlue,70:B3:D5:27:20:00/36TelecomS,70:B3:D5:27:30:00/36WeVoTech,70:B3:D5:27:40:00/36StercomP,70:B3:D5:27:50:00/36Internet,70:B3:D5:27:60:00/36TELLSoft,70:B3:D5:27:70:00/36Voltawar,70:B3:D5:27:80:00/36Private,70:B3:D5:27:90:00/36Medicomp,70:B3:D5:27:A0:00/36TdEcophi,70:B3:D5:27:B0:00/36Dave,70:B3:D5:27:C0:00/36MOTIONLI,70:B3:D5:27:D0:00/36TelenorC,70:B3:D5:27:E0:00/36MettlerT,70:B3:D5:27:F0:00/36STAerosp,70:B3:D5:28:00:00/36Computec,70:B3:D5:28:10:00/36Itg,70:B3:D5:28:20:00/36SamboHit,70:B3:D5:28:30:00/36TextNinj,70:B3:D5:28:40:00/36Globalco,70:B3:D5:28:50:00/36BentecDr,70:B3:D5:28:60:00/36PedaxDan,70:B3:D5:28:70:00/36HypexEle,70:B3:D5:28:80:00/36Bressler,70:B3:D5:28:90:00/36Shenzhen,70:B3:D5:28:A0:00/36TransitS,70:B3:D5:28:B0:00/36ArnouseD,70:B3:D5:28:C0:00/36StepTech,70:B3:D5:28:D0:00/36Technica,70:B3:D5:28:E0:00/36TexCompu,70:B3:D5:28:F0:00/36Overline,70:B3:D5:29:00:00/36GETTGera,70:B3:D5:29:10:00/36Sequent,70:B3:D5:29:20:00/36BostonDy,70:B3:D5:29:30:00/36SolarRIg,70:B3:D5:29:40:00/36RCHVietn,70:B3:D5:29:50:00/36CelloEle,70:B3:D5:29:60:00/36Rohde&Sc,70:B3:D5:29:70:00/36Grossenb,70:B3:D5:29:80:00/36Reflexio,70:B3:D5:29:90:00/36KMtronic,70:B3:D5:29:A0:00/36Profusio,70:B3:D5:29:B0:00/36DermaLum,70:B3:D5:29:C0:00/36TekoTele,70:B3:D5:29:D0:00/36XTech2SI,70:B3:D5:29:E0:00/36B2cloudl,70:B3:D5:29:F0:00/36CodeHard,70:B3:D5:2A:00:00/36Airthing,70:B3:D5:2A:10:00/36BlinkSer,70:B3:D5:2A:20:00/36Visualwa,70:B3:D5:2A:30:00/36ATTNussb,70:B3:D5:2A:40:00/36GSPSprac,70:B3:D5:2A:50:00/36Taitotek,70:B3:D5:2A:60:00/36GSITechn,70:B3:D5:2A:70:00/36Plasmabi,70:B3:D5:2A:80:00/36DynamicP,70:B3:D5:2A:90:00/36PowerEle,70:B3:D5:2A:A0:00/36Flirtey,70:B3:D5:2A:B0:00/36NASAJohn,70:B3:D5:2A:C0:00/36NewImagi,70:B3:D5:2A:D0:00/36OpgalOpt,70:B3:D5:2A:E0:00/36AlereTec,70:B3:D5:2A:F0:00/36Enlaps,70:B3:D5:2B:00:00/36BeijingZ,70:B3:D5:2B:10:00/36WIXCON,70:B3:D5:2B:20:00/36SunCreat,70:B3:D5:2B:30:00/36HAS,70:B3:D5:2B:40:00/36Foerster,70:B3:D5:2B:50:00/36Dosepack,70:B3:D5:2B:60:00/36HLTMicro,70:B3:D5:2B:70:00/36MatrixOr,70:B3:D5:2B:80:00/36WideNort,70:B3:D5:2B:90:00/36BELECTRI,70:B3:D5:2B:A0:00/36ActiveBr,70:B3:D5:2B:B0:00/36Automati,70:B3:D5:2B:C0:00/36EquiposD,70:B3:D5:2B:D0:00/36mg-senso,70:B3:D5:2B:E0:00/36Coherent,70:B3:D5:2B:F0:00/36FoshanVo,70:B3:D5:2C:00:00/36Sensativ,70:B3:D5:2C:10:00/36Avlinkpr,70:B3:D5:2C:20:00/36QuantumD,70:B3:D5:2C:30:00/36Proterra,70:B3:D5:2C:40:00/36Hodwa,70:B3:D5:2C:50:00/36Mect,70:B3:D5:2C:60:00/36AMGenera,70:B3:D5:2C:70:00/36Worldsen,70:B3:D5:2C:80:00/36Slat,70:B3:D5:2C:90:00/36SeasonDe,70:B3:D5:2C:A0:00/36Tattile,70:B3:D5:2C:B0:00/36Yongtong,70:B3:D5:2C:C0:00/36WeWorkCo,70:B3:D5:2C:D0:00/36KoreaAir,70:B3:D5:2C:E0:00/36Kdt,70:B3:D5:2C:F0:00/36MBconnec,70:B3:D5:2D:00:00/36ijin,70:B3:D5:2D:10:00/36Integerp,70:B3:D5:2D:20:00/36Shanghai,70:B3:D5:2D:30:00/36Hensoldt,70:B3:D5:2D:40:00/36CT,70:B3:D5:2D:50:00/36TeucoGuz,70:B3:D5:2D:60:00/36Kvazar,70:B3:D5:2D:70:00/36Private,70:B3:D5:2D:80:00/36Unisight,70:B3:D5:2D:90:00/36Zpas,70:B3:D5:2D:A0:00/36SkywaveN,70:B3:D5:2D:B0:00/36ProtoPix,70:B3:D5:2D:C0:00/36BolideTe,70:B3:D5:2D:D0:00/36MelissaC,70:B3:D5:2D:E0:00/36YUYAMAMF,70:B3:D5:2D:F0:00/36EasternS,70:B3:D5:2E:00:00/36PeterHub,70:B3:D5:2E:10:00/36hiSkySCS,70:B3:D5:2E:20:00/36SparkLas,70:B3:D5:2E:30:00/36Meiknolo,70:B3:D5:2E:40:00/36Schneide,70:B3:D5:2E:50:00/36FläktWoo,70:B3:D5:2E:60:00/36IPGPhoto,70:B3:D5:2E:70:00/36Atos,70:B3:D5:2E:80:00/36Telefire,70:B3:D5:2E:90:00/36NeurITsr,70:B3:D5:2E:A0:00/36Schneide,70:B3:D5:2E:B0:00/36Brnet,70:B3:D5:2E:C0:00/36GrupoEpe,70:B3:D5:2E:D0:00/36Signalsa,70:B3:D5:2E:E0:00/36AplexTec,70:B3:D5:2E:F0:00/36Iem,70:B3:D5:2F:00:00/36Clock-O-,70:B3:D5:2F:10:00/36Inspike,70:B3:D5:2F:20:00/36HealthCa,70:B3:D5:2F:30:00/36ScameSis,70:B3:D5:2F:40:00/36Radixons,70:B3:D5:2F:50:00/36ezeSyste,70:B3:D5:2F:60:00/36Tattile,70:B3:D5:2F:70:00/36Military,70:B3:D5:2F:80:00/36Tunstall,70:B3:D5:2F:90:00/36Consospy,70:B3:D5:2F:A0:00/36TorayMed,70:B3:D5:2F:B0:00/36IkMultim,70:B3:D5:2F:C0:00/36Loanguar,70:B3:D5:2F:D0:00/36SpecialP,70:B3:D5:2F:E0:00/36YahamOpt,70:B3:D5:2F:F0:00/36Sunstone,70:B3:D5:30:00:00/36NovoDR,70:B3:D5:30:10:00/36WayneAna,70:B3:D5:30:20:00/36DogWatch,70:B3:D5:30:30:00/36FuchuGik,70:B3:D5:30:40:00/36Wartsila,70:B3:D5:30:50:00/36CAITRONI,70:B3:D5:30:60:00/36Lemz-T,70:B3:D5:30:70:00/36Energiin,70:B3:D5:30:80:00/36DsdMicro,70:B3:D5:30:A0:00/36HongSeok,70:B3:D5:30:B0:00/36AshTechn,70:B3:D5:30:C0:00/36Sicon,70:B3:D5:30:D0:00/36Fiberbas,70:B3:D5:30:E0:00/36Private,70:B3:D5:30:F0:00/36Cardinal,70:B3:D5:31:00:00/36ConservS,70:B3:D5:31:10:00/36GüntherS,70:B3:D5:31:20:00/36SMITEC,70:B3:D5:31:30:00/36DIEHLCon,70:B3:D5:31:40:00/36GrauElek,70:B3:D5:31:50:00/36Private,70:B3:D5:31:60:00/36AustcoMa,70:B3:D5:31:70:00/36IotopiaS,70:B3:D5:31:80:00/36Exemplar,70:B3:D5:31:90:00/36IsoTc22S,70:B3:D5:31:A0:00/36Terratel,70:B3:D5:31:B0:00/36SilTerra,70:B3:D5:31:C0:00/36FINANCIE,70:B3:D5:31:D0:00/36AVAMonit,70:B3:D5:31:E0:00/36Gillam-F,70:B3:D5:31:F0:00/36Elcoma,70:B3:D5:32:00:00/36CYNIX,70:B3:D5:32:10:00/36Yitetech,70:B3:D5:32:20:00/36PuSund,70:B3:D5:32:30:00/36Tattile,70:B3:D5:32:40:00/36ThalesNe,70:B3:D5:32:50:00/36BlueMark,70:B3:D5:32:60:00/36Nemeus-S,70:B3:D5:32:70:00/36Seneco,70:B3:D5:32:80:00/36Hipodrom,70:B3:D5:32:90:00/36Primaluc,70:B3:D5:32:A0:00/36WuhanXin,70:B3:D5:32:B0:00/36RTA,70:B3:D5:32:C0:00/36ATION,70:B3:D5:32:D0:00/36HanwellT,70:B3:D5:32:E0:00/36A&T,70:B3:D5:32:F0:00/36Movidius,70:B3:D5:33:00:00/36iOne,70:B3:D5:33:10:00/36Firecom,70:B3:D5:33:20:00/36InnoSenT,70:B3:D5:33:30:00/36OrlacoPr,70:B3:D5:33:40:00/36Dokuen,70:B3:D5:33:50:00/36JonsaAus,70:B3:D5:33:60:00/36Synacces,70:B3:D5:33:70:00/36Laborie,70:B3:D5:33:80:00/36Opti-Sci,70:B3:D5:33:90:00/36SierraNe,70:B3:D5:33:A0:00/36AudioTEC,70:B3:D5:33:B0:00/36SealShie,70:B3:D5:33:C0:00/36Videri,70:B3:D5:33:D0:00/36Schneide,70:B3:D5:33:E0:00/36DynamicC,70:B3:D5:33:F0:00/36Xantia,70:B3:D5:34:00:00/36RenesasE,70:B3:D5:34:10:00/36Vtron,70:B3:D5:34:20:00/36Solectri,70:B3:D5:34:30:00/36Elektro-,70:B3:D5:34:40:00/36IHIInspe,70:B3:D5:34:50:00/36AT-Autom,70:B3:D5:34:60:00/36Ultamati,70:B3:D5:34:70:00/36OASSwede,70:B3:D5:34:80:00/36BÄRBahns,70:B3:D5:34:90:00/36Slat,70:B3:D5:34:A0:00/36PavoTasa,70:B3:D5:34:B0:00/36LeaffEng,70:B3:D5:34:C0:00/36GLTExpor,70:B3:D5:34:D0:00/36EquosRes,70:B3:D5:34:E0:00/36RiskExpe,70:B3:D5:34:F0:00/36RoyalEng,70:B3:D5:35:00:00/36Tickster,70:B3:D5:35:10:00/36KSTtechn,70:B3:D5:35:20:00/36Globalco,70:B3:D5:35:30:00/36DigitalO,70:B3:D5:35:40:00/36IMP-Comp,70:B3:D5:35:50:00/36Hongin,70:B3:D5:35:60:00/36BRSSiste,70:B3:D5:35:70:00/36Moviment,70:B3:D5:35:80:00/36Nevotek,70:B3:D5:35:90:00/36Boutroni,70:B3:D5:35:A0:00/36AppliedR,70:B3:D5:35:B0:00/36NuanceHe,70:B3:D5:35:C0:00/36ACSelect,70:B3:D5:35:D0:00/36FreshIde,70:B3:D5:35:E0:00/36EIDOS,70:B3:D5:35:F0:00/36AplexTec,70:B3:D5:36:00:00/36PTEmsoni,70:B3:D5:36:10:00/36ParentPo,70:B3:D5:36:20:00/36Asiga,70:B3:D5:36:30:00/36ContecAm,70:B3:D5:36:40:00/36ADAMCZEW,70:B3:D5:36:50:00/36CircuitM,70:B3:D5:36:60:00/36Solarlyt,70:B3:D5:36:70:00/36LivingWa,70:B3:D5:36:80:00/36WhiteMat,70:B3:D5:36:90:00/36ALVATsro,70:B3:D5:36:A0:00/36BectonDi,70:B3:D5:36:C0:00/36Sicon,70:B3:D5:36:D0:00/36Cybertea,70:B3:D5:36:E0:00/36Electrón,70:B3:D5:36:F0:00/36BuddyGua,70:B3:D5:37:00:00/36Inphi,70:B3:D5:37:10:00/36BEDEROV,70:B3:D5:37:20:00/36Matelex,70:B3:D5:37:30:00/36Hangzhou,70:B3:D5:37:40:00/36NPPMars-,70:B3:D5:37:50:00/36AdelSyst,70:B3:D5:37:60:00/36MagentaL,70:B3:D5:37:70:00/36Monnit,70:B3:D5:37:80:00/36synchrot,70:B3:D5:37:90:00/36Vensi,70:B3:D5:37:A0:00/36APGCashD,70:B3:D5:37:B0:00/36Power,70:B3:D5:37:C0:00/36MerusPow,70:B3:D5:37:D0:00/36DXShop,70:B3:D5:37:E0:00/36Elinkgat,70:B3:D5:37:F0:00/36IDSInnom,70:B3:D5:38:00:00/36SeaTechI,70:B3:D5:38:10:00/36Crde,70:B3:D5:38:20:00/36NavalGro,70:B3:D5:38:30:00/36LPAExcil,70:B3:D5:38:40:00/36Sensohiv,70:B3:D5:38:50:00/36KamachoS,70:B3:D5:38:60:00/36GPSat,70:B3:D5:38:70:00/36GWFMessS,70:B3:D5:38:80:00/36Xitron,70:B3:D5:38:90:00/36Private,70:B3:D5:38:A0:00/36KSE,70:B3:D5:38:B0:00/36LookmanE,70:B3:D5:38:C0:00/36MiraeSig,70:B3:D5:38:D0:00/36Imp-Tele,70:B3:D5:38:E0:00/36ChinaTel,70:B3:D5:38:F0:00/36Sorynory,70:B3:D5:39:00:00/36TexCompu,70:B3:D5:39:10:00/36Changshu,70:B3:D5:39:20:00/36ContecAm,70:B3:D5:39:30:00/36Monnit,70:B3:D5:39:40:00/36RomteckA,70:B3:D5:39:50:00/36ICsec,70:B3:D5:39:60:00/36CTGspzoo,70:B3:D5:39:70:00/36GuangxiH,70:B3:D5:39:80:00/36SIPRO,70:B3:D5:39:90:00/36SPESmart,70:B3:D5:39:A0:00/36Videotre,70:B3:D5:39:B0:00/36Iroc,70:B3:D5:39:C0:00/36GDMissio,70:B3:D5:39:D0:00/36ComarkIn,70:B3:D5:39:E0:00/36LanmarkC,70:B3:D5:39:F0:00/36CT,70:B3:D5:3A:00:00/36chiconyp,70:B3:D5:3A:10:00/36ReckeenH,70:B3:D5:3A:20:00/36Daifuku,70:B3:D5:3A:30:00/36CDSInsti,70:B3:D5:3A:40:00/36Ascenix,70:B3:D5:3A:50:00/36KMtronic,70:B3:D5:3A:60:00/36myenergi,70:B3:D5:3A:70:00/36Varikore,70:B3:D5:3A:80:00/36JamHub,70:B3:D5:3A:90:00/36Vivalnk,70:B3:D5:3A:A0:00/36Rcatsone,70:B3:D5:3A:B0:00/36CamozziA,70:B3:D5:3A:C0:00/36RF-Tuote,70:B3:D5:3A:D0:00/36CT,70:B3:D5:3A:E0:00/36ExicomTe,70:B3:D5:3A:F0:00/36TurboTec,70:B3:D5:3B:00:00/36Millenni,70:B3:D5:3B:10:00/36GlobalPo,70:B3:D5:3B:20:00/36Sicon,70:B3:D5:3B:30:00/36MovicomE,70:B3:D5:3B:40:00/36Yousung,70:B3:D5:3B:50:00/36PrestonI,70:B3:D5:3B:60:00/36MedRx,70:B3:D5:3B:70:00/36PaulSche,70:B3:D5:3B:80:00/36nVideon,70:B3:D5:3B:90:00/36BirdDogA,70:B3:D5:3B:A0:00/36SilexIns,70:B3:D5:3B:B0:00/36A-M,70:B3:D5:3B:C0:00/36SciTroni,70:B3:D5:3B:D0:00/36DaoQinTe,70:B3:D5:3B:E0:00/36MyDefenc,70:B3:D5:3B:F0:00/36StarElec,70:B3:D5:3C:00:00/36DK-Techn,70:B3:D5:3C:10:00/36thingdus,70:B3:D5:3C:20:00/36Cellular,70:B3:D5:3C:30:00/36Aimco,70:B3:D5:3C:40:00/36Hagiwara,70:B3:D5:3C:50:00/36P4QElect,70:B3:D5:3C:60:00/36ACDElekr,70:B3:D5:3C:70:00/36Softcrea,70:B3:D5:3C:80:00/36ABCElect,70:B3:D5:3C:90:00/36Duerkopp,70:B3:D5:3C:A0:00/36TTI,70:B3:D5:3C:B0:00/36GeoSpect,70:B3:D5:3C:C0:00/36TerOpta,70:B3:D5:3C:D0:00/36BRSSiste,70:B3:D5:3C:E0:00/36Aditec,70:B3:D5:3C:F0:00/36Engineer,70:B3:D5:3D:00:00/36ORtekTec,70:B3:D5:3D:10:00/36Imenco,70:B3:D5:3D:20:00/36Imagine,70:B3:D5:3D:30:00/36GSElektr,70:B3:D5:3D:40:00/36SanminaI,70:B3:D5:3D:50:00/36oxynetSo,70:B3:D5:3D:60:00/36AristonT,70:B3:D5:3D:70:00/36RemoteSe,70:B3:D5:3D:80:00/36Abitsoft,70:B3:D5:3D:90:00/36AplexTec,70:B3:D5:3D:A0:00/36LoopLabs,70:B3:D5:3D:B0:00/36KSTtechn,70:B3:D5:3D:C0:00/36Xia,70:B3:D5:3D:D0:00/36Kniggend,70:B3:D5:3D:E0:00/36ELOMACEl,70:B3:D5:3D:F0:00/36MultiDyn,70:B3:D5:3E:00:00/36GogoBusi,70:B3:D5:3E:10:00/36Barnstor,70:B3:D5:3E:20:00/36AVI,70:B3:D5:3E:30:00/36Head,70:B3:D5:3E:40:00/36NeptecTe,70:B3:D5:3E:50:00/36Ateme,70:B3:D5:3E:60:00/36machineQ,70:B3:D5:3E:70:00/36JNRSport,70:B3:D5:3E:80:00/36COSMOSwe,70:B3:D5:3E:90:00/36APOLLOGI,70:B3:D5:3E:A0:00/36Dave,70:B3:D5:3E:B0:00/36Grossenb,70:B3:D5:3E:C0:00/36Outsight,70:B3:D5:3E:D0:00/36UltraEle,70:B3:D5:3E:E0:00/36LaserIma,70:B3:D5:3E:F0:00/36Vtron,70:B3:D5:3F:00:00/36Interval,70:B3:D5:3F:10:00/36OlympusN,70:B3:D5:3F:20:00/36H3D,70:B3:D5:3F:30:00/36Spea,70:B3:D5:3F:40:00/36WincodeT,70:B3:D5:3F:50:00/36DolbyLab,70:B3:D5:3F:60:00/36SycompEl,70:B3:D5:3F:70:00/36Advansid,70:B3:D5:3F:80:00/36FireHorn,70:B3:D5:3F:90:00/36HerrickT,70:B3:D5:3F:A0:00/36ZakladEn,70:B3:D5:3F:B0:00/36LibertyR,70:B3:D5:3F:C0:00/36TangRenC,70:B3:D5:3F:D0:00/36NaraCont,70:B3:D5:3F:E0:00/36SiemensI,70:B3:D5:3F:F0:00/36HydraCon,70:B3:D5:40:00:00/36Vtron,70:B3:D5:40:10:00/36Private,70:B3:D5:40:20:00/36AKIStech,70:B3:D5:40:30:00/36MightyCu,70:B3:D5:40:40:00/36RANIX,70:B3:D5:40:50:00/36MG,70:B3:D5:40:60:00/36Acrodea,70:B3:D5:40:70:00/36Idosens,70:B3:D5:40:80:00/36ComrodAS,70:B3:D5:40:90:00/36BeijingY,70:B3:D5:40:A0:00/36MonroeEl,70:B3:D5:40:B0:00/36QuercusT,70:B3:D5:40:C0:00/36TornadoM,70:B3:D5:40:D0:00/36GrupoEpe,70:B3:D5:40:E0:00/36LiaoyunI,70:B3:D5:40:F0:00/36Nexelec,70:B3:D5:41:00:00/36AvantTec,70:B3:D5:41:10:00/36Mi-FiNet,70:B3:D5:41:20:00/36Tattile,70:B3:D5:41:30:00/36Axess,70:B3:D5:41:40:00/36SmithMet,70:B3:D5:41:50:00/36Idea,70:B3:D5:41:60:00/36Antlia,70:B3:D5:41:70:00/36FigmentD,70:B3:D5:41:80:00/36DEVSyste,70:B3:D5:41:A0:00/36HYOSUNGP,70:B3:D5:41:B0:00/36SYSTECel,70:B3:D5:41:C0:00/36TwowayCo,70:B3:D5:41:D0:00/36AzmoonKe,70:B3:D5:41:E0:00/36RedlerCo,70:B3:D5:41:F0:00/36Orion,70:B3:D5:42:00:00/36Ecoinet,70:B3:D5:42:10:00/36NorthSta,70:B3:D5:42:20:00/36SUS,70:B3:D5:42:30:00/36HarmanCo,70:B3:D5:42:40:00/36Undergro,70:B3:D5:42:50:00/36SinterCa,70:B3:D5:42:60:00/36ZehnderG,70:B3:D5:42:70:00/36KeyChemi,70:B3:D5:42:80:00/36Presenta,70:B3:D5:42:90:00/36RedcoAud,70:B3:D5:42:A0:00/36Critical,70:B3:D5:42:B0:00/36Guangzho,70:B3:D5:42:C0:00/36DMarchio,70:B3:D5:42:D0:00/36RchItali,70:B3:D5:42:E0:00/36DrZinngr,70:B3:D5:42:F0:00/36Sintokog,70:B3:D5:43:00:00/36AlgodueE,70:B3:D5:43:10:00/36PowerEle,70:B3:D5:43:20:00/36DEUTA-WE,70:B3:D5:43:30:00/36Flexsolu,70:B3:D5:43:40:00/36Witcom,70:B3:D5:43:50:00/36WuhanXin,70:B3:D5:43:60:00/36HenrichE,70:B3:D5:43:70:00/36DigitalW,70:B3:D5:43:90:00/36TriLED,70:B3:D5:43:A0:00/36ARKSEnte,70:B3:D5:43:B0:00/36Kalycito,70:B3:D5:43:C0:00/36Scenario,70:B3:D5:43:D0:00/36VeryxTec,70:B3:D5:43:E0:00/36PelotonT,70:B3:D5:43:F0:00/36biosilve,70:B3:D5:44:00:00/36Discover,70:B3:D5:44:10:00/36Videopor,70:B3:D5:44:20:00/36BlairCom,70:B3:D5:44:30:00/36Slot3,70:B3:D5:44:40:00/36AMSContr,70:B3:D5:44:50:00/36Advanced,70:B3:D5:44:60:00/36SantaBar,70:B3:D5:44:70:00/36AvidCont,70:B3:D5:44:80:00/36BEAerosp,70:B3:D5:44:90:00/36Edgeware,70:B3:D5:44:A0:00/36CanonEle,70:B3:D5:44:B0:00/36OpenSyst,70:B3:D5:44:C0:00/36ejoinsro,70:B3:D5:44:D0:00/36VesselTe,70:B3:D5:44:E0:00/36Solace,70:B3:D5:44:F0:00/36Velvac,70:B3:D5:45:00:00/36Apantac,70:B3:D5:45:10:00/36Perform3,70:B3:D5:45:20:00/36Italiana,70:B3:D5:45:30:00/36Foerster,70:B3:D5:45:40:00/36GoldingA,70:B3:D5:45:50:00/36Heartlan,70:B3:D5:45:60:00/36Technolo,70:B3:D5:45:70:00/36VivaldiC,70:B3:D5:45:80:00/36Ongisul,70:B3:D5:45:90:00/36ProtiumT,70:B3:D5:45:A0:00/36Palarum,70:B3:D5:45:B0:00/36Komz-Izm,70:B3:D5:45:C0:00/36AlyTech,70:B3:D5:45:D0:00/36Sensapex,70:B3:D5:45:E0:00/36eSOL,70:B3:D5:45:F0:00/36Cloud4Wi,70:B3:D5:46:00:00/36GuilinTr,70:B3:D5:46:10:00/36TESEC,70:B3:D5:46:20:00/36EarTex,70:B3:D5:46:30:00/36Warecube,70:B3:D5:46:50:00/36Energism,70:B3:D5:46:60:00/36SYLinkTe,70:B3:D5:46:70:00/36GreenWak,70:B3:D5:46:80:00/36Shanghai,70:B3:D5:46:90:00/36Gentec,70:B3:D5:46:A0:00/36Shenzhen,70:B3:D5:46:B0:00/36Airborne,70:B3:D5:46:C0:00/36Shanghai,70:B3:D5:46:D0:00/36GuanShow,70:B3:D5:46:E0:00/36ZamirRec,70:B3:D5:46:F0:00/36servatra,70:B3:D5:47:00:00/36KitronUa,70:B3:D5:47:10:00/36SYSCOSic,70:B3:D5:47:20:00/36QuadioDe,70:B3:D5:47:30:00/36KeyProd,70:B3:D5:47:40:00/36Ctrogers,70:B3:D5:47:50:00/36Ewattch,70:B3:D5:47:60:00/36FR-TeamI,70:B3:D5:47:70:00/36digitrol,70:B3:D5:47:80:00/36Touchnet,70:B3:D5:47:90:00/36LineageP,70:B3:D5:47:A0:00/36GlooVir,70:B3:D5:47:B0:00/36Monixo,70:B3:D5:47:C0:00/36Par-Tech,70:B3:D5:47:D0:00/36Shenyang,70:B3:D5:47:E0:00/36FiberOpt,70:B3:D5:47:F0:00/36ASE,70:B3:D5:48:00:00/36Emergenc,70:B3:D5:48:10:00/36STEP,70:B3:D5:48:20:00/36AeryonLa,70:B3:D5:48:30:00/36LitumBil,70:B3:D5:48:40:00/36HermannS,70:B3:D5:48:50:00/36Claresys,70:B3:D5:48:60:00/36ChongQin,70:B3:D5:48:70:00/36ECS,70:B3:D5:48:80:00/36Cardinal,70:B3:D5:48:90:00/36ard,70:B3:D5:48:A0:00/36GeorgeWi,70:B3:D5:48:B0:00/36Tattile,70:B3:D5:48:C0:00/36Integrat,70:B3:D5:48:D0:00/36OmegaBil,70:B3:D5:48:E0:00/36AllimSys,70:B3:D5:48:F0:00/36SeiwaGik,70:B3:D5:49:00:00/36XiamenBe,70:B3:D5:49:10:00/36Vonsch,70:B3:D5:49:20:00/36JiangsuJ,70:B3:D5:49:30:00/36ImpulseN,70:B3:D5:49:40:00/36Schildkn,70:B3:D5:49:50:00/36FiemIndu,70:B3:D5:49:60:00/36Profcon,70:B3:D5:49:70:00/36AlbiralD,70:B3:D5:49:80:00/36XgemSas,70:B3:D5:49:90:00/36Pycom,70:B3:D5:49:A0:00/36HaxeSyst,70:B3:D5:49:B0:00/36AlgodueE,70:B3:D5:49:C0:00/36ACPower,70:B3:D5:49:D0:00/36Shenzhen,70:B3:D5:49:E0:00/36CAPTEMPL,70:B3:D5:49:F0:00/36BPA,70:B3:D5:4A:00:00/36Fludia,70:B3:D5:4A:10:00/36Herholdt,70:B3:D5:4A:20:00/36DEVAULem,70:B3:D5:4A:30:00/36TualcomE,70:B3:D5:4A:40:00/36DEUTA-WE,70:B3:D5:4A:50:00/36Intermin,70:B3:D5:4A:60:00/36HzhyTech,70:B3:D5:4A:70:00/36aelettro,70:B3:D5:4A:80:00/36Acrodea,70:B3:D5:4A:90:00/36Warecube,70:B3:D5:4A:A0:00/36TwowayCo,70:B3:D5:4A:B0:00/36TruTeqWi,70:B3:D5:4A:C0:00/36Microsof,70:B3:D5:4A:D0:00/36Gaci,70:B3:D5:4A:E0:00/36Reinhard,70:B3:D5:4A:F0:00/36Agramkow,70:B3:D5:4B:00:00/36Tecogen,70:B3:D5:4B:10:00/36Lace,70:B3:D5:4B:20:00/36CertusOp,70:B3:D5:4B:30:00/36Bacsoft,70:B3:D5:4B:40:00/36HiTech,70:B3:D5:4B:50:00/36Toolplan,70:B3:D5:4B:60:00/36Veilux,70:B3:D5:4B:70:00/36AplexTec,70:B3:D5:4B:80:00/36Internat,70:B3:D5:4B:90:00/36ShenZhen,70:B3:D5:4B:A0:00/36Sinftech,70:B3:D5:4B:B0:00/36Plazma-T,70:B3:D5:4B:C0:00/36Tiama,70:B3:D5:4B:D0:00/36BoulderA,70:B3:D5:4B:E0:00/36Gy-FxSas,70:B3:D5:4B:F0:00/36ExsomCom,70:B3:D5:4C:00:00/36Technica,70:B3:D5:4C:10:00/36QuercusT,70:B3:D5:4C:20:00/36heraLabo,70:B3:D5:4C:30:00/36EAElektr,70:B3:D5:4C:40:00/36Research,70:B3:D5:4C:50:00/36MovingiM,70:B3:D5:4C:60:00/36BlueBoxV,70:B3:D5:4C:70:00/36SOLVERIS,70:B3:D5:4C:80:00/36Hosokawa,70:B3:D5:4C:90:00/36Elsist,70:B3:D5:4C:A0:00/36PCBPiezo,70:B3:D5:4C:B0:00/36CucosRet,70:B3:D5:4C:C0:00/36Freseniu,70:B3:D5:4C:D0:00/36PowerEle,70:B3:D5:4C:E0:00/36Agilack,70:B3:D5:4C:F0:00/36GreenHou,70:B3:D5:4D:00:00/36Codewerk,70:B3:D5:4D:10:00/36Contrave,70:B3:D5:4D:20:00/36BiotageS,70:B3:D5:4D:30:00/36HefeiSTA,70:B3:D5:4D:40:00/36NortekGl,70:B3:D5:4D:50:00/36MoogReko,70:B3:D5:4D:60:00/36Operatio,70:B3:D5:4D:70:00/36Technolo,70:B3:D5:4D:80:00/36Versilis,70:B3:D5:4D:90:00/36CodaOcto,70:B3:D5:4D:A0:00/36Private,70:B3:D5:4D:B0:00/36Temperat,70:B3:D5:4D:C0:00/36JkDevice,70:B3:D5:4D:D0:00/36Velvac,70:B3:D5:4D:E0:00/36OsoTechn,70:B3:D5:4D:F0:00/36NidecAvt,70:B3:D5:4E:00:00/36Microvid,70:B3:D5:4E:10:00/36GrupoEpe,70:B3:D5:4E:20:00/36TransitS,70:B3:D5:4E:30:00/36adnexo,70:B3:D5:4E:40:00/36WABenjam,70:B3:D5:4E:50:00/36viZaarin,70:B3:D5:4E:60:00/36SantaBar,70:B3:D5:4E:70:00/36DigitalD,70:B3:D5:4E:80:00/36CopiousI,70:B3:D5:4E:90:00/36AdetecSa,70:B3:D5:4E:A0:00/36Vocality,70:B3:D5:4E:B0:00/36Infosoft,70:B3:D5:4E:C0:00/36Hangzhou,70:B3:D5:4E:D0:00/36Panorami,70:B3:D5:4E:E0:00/36NOA,70:B3:D5:4E:F0:00/36CMI,70:B3:D5:4F:00:00/36LiSengTe,70:B3:D5:4F:10:00/36LGElectr,70:B3:D5:4F:20:00/36CompalEl,70:B3:D5:4F:30:00/36XpsEletr,70:B3:D5:4F:40:00/36WiTagg,70:B3:D5:4F:50:00/36OrlacoPr,70:B3:D5:4F:60:00/36DorletSa,70:B3:D5:4F:70:00/36Foxtel,70:B3:D5:4F:80:00/36Private,70:B3:D5:4F:90:00/36OptoPrec,70:B3:D5:4F:A0:00/36Thruvisi,70:B3:D5:4F:B0:00/36MASElett,70:B3:D5:4F:C0:00/36MettlerT,70:B3:D5:4F:D0:00/36EnlessWi,70:B3:D5:4F:E0:00/36WiTagg,70:B3:D5:4F:F0:00/36Shanghai,70:B3:D5:50:00:00/36MistralS,70:B3:D5:50:10:00/36PeekTraf,70:B3:D5:50:20:00/36Glidewel,70:B3:D5:50:30:00/36Itestcom,70:B3:D5:50:40:00/36Xsight,70:B3:D5:50:50:00/36MC2-Tech,70:B3:D5:50:60:00/36TonboIma,70:B3:D5:50:70:00/36HumanOri,70:B3:D5:50:80:00/36INSEVIS,70:B3:D5:50:90:00/36Tinkerfo,70:B3:D5:50:A0:00/36AMEDTECM,70:B3:D5:50:B0:00/36Nordson,70:B3:D5:50:C0:00/36Hangzhou,70:B3:D5:50:D0:00/36CT,70:B3:D5:50:E0:00/36MicroTre,70:B3:D5:50:F0:00/36SarovInn,70:B3:D5:51:00:00/36PslElekt,70:B3:D5:51:10:00/36NextSigh,70:B3:D5:51:20:00/36TechnoBr,70:B3:D5:51:30:00/36MBconnec,70:B3:D5:51:40:00/36Intellig,70:B3:D5:51:50:00/36Pcsc,70:B3:D5:51:60:00/36LineageP,70:B3:D5:51:70:00/36Ispher,70:B3:D5:51:80:00/36CRUXELL,70:B3:D5:51:90:00/36MBconnec,70:B3:D5:51:A0:00/36Shachiha,70:B3:D5:51:B0:00/36VitreaSm,70:B3:D5:51:C0:00/36ATXNetwo,70:B3:D5:51:D0:00/36TecnintH,70:B3:D5:51:E0:00/36Fundació,70:B3:D5:51:F0:00/36ValeoCda,70:B3:D5:52:00:00/36promedia,70:B3:D5:52:10:00/36SelexES,70:B3:D5:52:20:00/36Syncopat,70:B3:D5:52:30:00/36TibitCom,70:B3:D5:52:40:00/36WuxiNewO,70:B3:D5:52:50:00/36Plantiga,70:B3:D5:52:60:00/36FlowNet,70:B3:D5:52:70:00/36ProconEl,70:B3:D5:52:80:00/36AplexTec,70:B3:D5:52:90:00/36Inventeq,70:B3:D5:52:A0:00/36Dataflex,70:B3:D5:52:B0:00/36GEAviati,70:B3:D5:52:C0:00/36Centurya,70:B3:D5:52:D0:00/36TanakaEl,70:B3:D5:52:E0:00/36Swisspon,70:B3:D5:52:F0:00/36RC,70:B3:D5:53:00:00/36iSiS-Ex,70:B3:D5:53:10:00/36Ateme,70:B3:D5:53:20:00/36Talleres,70:B3:D5:53:30:00/36NipponMa,70:B3:D5:53:40:00/36WeihaiWe,70:B3:D5:53:50:00/36SITAMess,70:B3:D5:53:60:00/36Larimart,70:B3:D5:53:70:00/36Biennebi,70:B3:D5:53:80:00/36sydetion,70:B3:D5:53:90:00/36Tempris,70:B3:D5:53:A0:00/36Panorami,70:B3:D5:53:B0:00/36MrLoop,70:B3:D5:53:C0:00/36Airthing,70:B3:D5:53:D0:00/36Accel,70:B3:D5:53:E0:00/36Asiga,70:B3:D5:53:F0:00/36AbbottDi,70:B3:D5:54:00:00/36KMtronic,70:B3:D5:54:10:00/36NanjingP,70:B3:D5:54:20:00/36RTDSTech,70:B3:D5:54:30:00/36wallbe,70:B3:D5:54:40:00/36SiliconS,70:B3:D5:54:50:00/36AirityTe,70:B3:D5:54:60:00/36Sensefar,70:B3:D5:54:70:00/36CeLink,70:B3:D5:54:80:00/36Digiverv,70:B3:D5:54:90:00/36Proconau,70:B3:D5:54:A0:00/36DigitalI,70:B3:D5:54:B0:00/36BrakelsI,70:B3:D5:54:C0:00/36HustyMSt,70:B3:D5:54:D0:00/36QingdaoH,70:B3:D5:54:E0:00/36RFLElect,70:B3:D5:54:F0:00/36Assembly,70:B3:D5:55:00:00/36MertenGm,70:B3:D5:55:10:00/36infrachi,70:B3:D5:55:20:00/36ALTIT,70:B3:D5:55:30:00/36TAALEXSy,70:B3:D5:55:40:00/36Teletype,70:B3:D5:55:50:00/36SoftLab-,70:B3:D5:55:60:00/36OhashiEn,70:B3:D5:55:70:00/36Heitec,70:B3:D5:55:80:00/36Multiple,70:B3:D5:55:90:00/36EagleMou,70:B3:D5:55:A0:00/36Sontay,70:B3:D5:55:B0:00/36ProconEl,70:B3:D5:55:C0:00/36Saratoga,70:B3:D5:55:D0:00/36LunaNexu,70:B3:D5:55:E0:00/36BRSSiste,70:B3:D5:55:F0:00/36Deep,70:B3:D5:56:00:00/36DaiShinI,70:B3:D5:56:10:00/36Liberato,70:B3:D5:56:20:00/36JDSquare,70:B3:D5:56:30:00/36Zhejiang,70:B3:D5:56:40:00/36christma,70:B3:D5:56:50:00/36Clecell,70:B3:D5:56:60:00/36DataInfo,70:B3:D5:56:70:00/36DogWatch,70:B3:D5:56:80:00/36SmallDat,70:B3:D5:56:90:00/36NuanceHe,70:B3:D5:56:A0:00/36HarvardT,70:B3:D5:56:B0:00/36SEI,70:B3:D5:56:C0:00/36Telensa,70:B3:D5:56:D0:00/36Pro-Digi,70:B3:D5:56:E0:00/36PowerEle,70:B3:D5:56:F0:00/36Radikald,70:B3:D5:57:00:00/36BayernEn,70:B3:D5:57:10:00/36Echogear,70:B3:D5:57:20:00/36Crde,70:B3:D5:57:30:00/36GegaElec,70:B3:D5:57:40:00/36CloudInt,70:B3:D5:57:50:00/36Konrad,70:B3:D5:57:60:00/36Shandong,70:B3:D5:57:70:00/36Dsilog,70:B3:D5:57:80:00/36ImageTec,70:B3:D5:57:90:00/36ChelseaT,70:B3:D5:57:A0:00/36RhythmEn,70:B3:D5:57:B0:00/36ELAMAKAT,70:B3:D5:57:C0:00/36Automata,70:B3:D5:57:D0:00/36WICOM1,70:B3:D5:57:E0:00/36AsconTec,70:B3:D5:57:F0:00/36MBioDiag,70:B3:D5:58:00:00/36Private,70:B3:D5:58:10:00/36Thermoko,70:B3:D5:58:20:00/36VAGLERIn,70:B3:D5:58:30:00/36Ducommun,70:B3:D5:58:40:00/36Sertonea,70:B3:D5:58:50:00/36Nefteavt,70:B3:D5:58:60:00/36AliterTe,70:B3:D5:58:70:00/36INCAACom,70:B3:D5:58:80:00/36NPOSvyaz,70:B3:D5:58:90:00/36Cityntel,70:B3:D5:58:A0:00/36ITKDrKas,70:B3:D5:58:B0:00/36Williams,70:B3:D5:58:C0:00/36Optsys,70:B3:D5:58:D0:00/36DorletSa,70:B3:D5:58:E0:00/36Veilux,70:B3:D5:58:F0:00/36LSL,70:B3:D5:59:00:00/36812thAIT,70:B3:D5:59:10:00/36Private,70:B3:D5:59:20:00/36Crde,70:B3:D5:59:30:00/36AsisPro,70:B3:D5:59:40:00/36ATE,70:B3:D5:59:50:00/36PLRPruef,70:B3:D5:59:60:00/36Mencom,70:B3:D5:59:70:00/36VapeRail,70:B3:D5:59:80:00/36RuagDefe,70:B3:D5:59:90:00/36LECO,70:B3:D5:59:A0:00/36WagnerGr,70:B3:D5:59:B0:00/36Automati,70:B3:D5:59:C0:00/36Dave,70:B3:D5:59:D0:00/36servicio,70:B3:D5:59:E0:00/36i2-elect,70:B3:D5:59:F0:00/36MeggerGe,70:B3:D5:5A:00:00/36AsconTec,70:B3:D5:5A:10:00/36BOETechn,70:B3:D5:5A:20:00/36WallnerA,70:B3:D5:5A:30:00/36CT,70:B3:D5:5A:40:00/36MBconnec,70:B3:D5:5A:50:00/36Rehwork,70:B3:D5:5A:60:00/36TimeMach,70:B3:D5:5A:70:00/36ABB,70:B3:D5:5A:80:00/36Farmobil,70:B3:D5:5A:90:00/36BunkaShu,70:B3:D5:5A:A0:00/36ChugokuE,70:B3:D5:5A:B0:00/36SeaAiran,70:B3:D5:5A:C0:00/36LM-Instr,70:B3:D5:5A:D0:00/36Profotec,70:B3:D5:5A:E0:00/36TinTec,70:B3:D5:5A:F0:00/36JENGIoT,70:B3:D5:5B:00:00/36QxpertsI,70:B3:D5:5B:10:00/36EPDElect,70:B3:D5:5B:20:00/36PeterHub,70:B3:D5:5B:30:00/36STENTORI,70:B3:D5:5B:40:00/36Technolo,70:B3:D5:5B:50:00/36LehighEl,70:B3:D5:5B:60:00/36EthicalL,70:B3:D5:5B:70:00/36on,70:B3:D5:5B:80:00/36HellaGut,70:B3:D5:5B:90:00/36EizoRugg,70:B3:D5:5B:A0:00/36Infrasaf,70:B3:D5:5B:B0:00/36OlympusN,70:B3:D5:5B:C0:00/36LAMTECMe,70:B3:D5:5B:D0:00/36nexgenwa,70:B3:D5:5B:E0:00/36Caswa,70:B3:D5:5B:F0:00/36Aton,70:B3:D5:5C:00:00/36Shenzhen,70:B3:D5:5C:10:00/36Shanghai,70:B3:D5:5C:20:00/36Sono-Tek,70:B3:D5:5C:30:00/36DIC,70:B3:D5:5C:40:00/36Tattile,70:B3:D5:5C:50:00/36Haag-Str,70:B3:D5:5C:60:00/36C4I,70:B3:D5:5C:70:00/36QSnetVis,70:B3:D5:5C:80:00/36YUYAMAMF,70:B3:D5:5C:90:00/36ICTKHold,70:B3:D5:5C:A0:00/36ACDElekr,70:B3:D5:5C:B0:00/36ECoCoMS,70:B3:D5:5C:C0:00/36Akse,70:B3:D5:5C:D0:00/36MVTVideo,70:B3:D5:5C:E0:00/36IPDevice,70:B3:D5:5C:F0:00/36PROELTSI,70:B3:D5:5D:00:00/36InterTal,70:B3:D5:5D:10:00/36Software,70:B3:D5:5D:20:00/36ContecAm,70:B3:D5:5D:30:00/36Supracon,70:B3:D5:5D:40:00/36RchItali,70:B3:D5:5D:50:00/36CT,70:B3:D5:5D:60:00/36BMTMesst,70:B3:D5:5D:70:00/36Clockwor,70:B3:D5:5D:80:00/36LYNXTech,70:B3:D5:5D:90:00/36olympus-,70:B3:D5:5D:A0:00/36ValkWeld,70:B3:D5:5D:B0:00/36Movicom,70:B3:D5:5D:C0:00/36FactoryL,70:B3:D5:5D:D0:00/36Theatrix,70:B3:D5:5D:E0:00/36Hangzhou,70:B3:D5:5D:F0:00/36SemaconB,70:B3:D5:5E:00:00/36HexagonM,70:B3:D5:5E:10:00/36Arevita,70:B3:D5:5E:20:00/36Grossenb,70:B3:D5:5E:30:00/36ImeconEn,70:B3:D5:5E:40:00/36DspDesig,70:B3:D5:5E:50:00/36HaiyangO,70:B3:D5:5E:60:00/36Mechatro,70:B3:D5:5E:70:00/36HeroicTe,70:B3:D5:5E:80:00/36Vitec,70:B3:D5:5E:90:00/36Zehetner,70:B3:D5:5E:A0:00/36Kys,70:B3:D5:5E:B0:00/36Lomasro,70:B3:D5:5E:C0:00/36Creative,70:B3:D5:5E:D0:00/36EAElektr,70:B3:D5:5E:E0:00/36Mikrotro,70:B3:D5:5E:F0:00/36StarInte,70:B3:D5:5F:00:00/36managee,70:B3:D5:5F:10:00/36FaterRas,70:B3:D5:5F:20:00/36Invisibl,70:B3:D5:5F:30:00/36Rtone,70:B3:D5:5F:40:00/36FDSTimin,70:B3:D5:5F:50:00/36Microvis,70:B3:D5:5F:60:00/36FreeFlig,70:B3:D5:5F:70:00/36JFAElect,70:B3:D5:5F:80:00/36ForciteH,70:B3:D5:5F:90:00/36MBconnec,70:B3:D5:5F:A0:00/36TexCompu,70:B3:D5:5F:B0:00/36Teleplat,70:B3:D5:5F:C0:00/36Surtec,70:B3:D5:5F:D0:00/36WindarPh,70:B3:D5:5F:E0:00/36Grossenb,70:B3:D5:5F:F0:00/36Vaisala,70:B3:D5:60:00:00/36Stellwer,70:B3:D5:60:10:00/36TricomRe,70:B3:D5:60:20:00/36QuantumO,70:B3:D5:60:30:00/36Egistech,70:B3:D5:60:40:00/36FoxtrotR,70:B3:D5:60:50:00/36AplexTec,70:B3:D5:60:60:00/36Research,70:B3:D5:60:70:00/36Ateme,70:B3:D5:60:80:00/36Eiit,70:B3:D5:60:90:00/36PBSIGrou,70:B3:D5:60:A0:00/36TataPowe,70:B3:D5:60:B0:00/36Edgeware,70:B3:D5:60:C0:00/36ISTElekt,70:B3:D5:60:D0:00/36LinkElec,70:B3:D5:60:E0:00/36Hdanywhe,70:B3:D5:60:F0:00/36TanakaIn,70:B3:D5:61:00:00/36Polvisio,70:B3:D5:61:10:00/36Avionica,70:B3:D5:61:20:00/36EdgePowe,70:B3:D5:61:30:00/36SuprockT,70:B3:D5:61:40:00/36Qualitte,70:B3:D5:61:50:00/36Otzvuk,70:B3:D5:61:60:00/36AxxessId,70:B3:D5:61:70:00/36Cominfo,70:B3:D5:61:80:00/36Motec,70:B3:D5:61:90:00/36Zeo,70:B3:D5:61:A0:00/36RocketLa,70:B3:D5:61:B0:00/36Nubewell,70:B3:D5:61:C0:00/36EarthWor,70:B3:D5:61:D0:00/36TelonicB,70:B3:D5:61:E0:00/36PKEElect,70:B3:D5:61:F0:00/36Labotect,70:B3:D5:62:00:00/36OrlacoPr,70:B3:D5:62:10:00/36Sertec,70:B3:D5:62:20:00/36PCS,70:B3:D5:62:30:00/36BeijingH,70:B3:D5:62:40:00/36EBEMobil,70:B3:D5:62:50:00/36VXInstru,70:B3:D5:62:60:00/36Kronotec,70:B3:D5:62:70:00/36EarTex,70:B3:D5:62:80:00/36Mect,70:B3:D5:62:90:00/36Ozray,70:B3:D5:62:A0:00/36Doga,70:B3:D5:62:B0:00/36Silicann,70:B3:D5:62:C0:00/36NTCRotek,70:B3:D5:62:D0:00/36elements,70:B3:D5:62:E0:00/36LineageP,70:B3:D5:62:F0:00/36BARCOsro,70:B3:D5:63:00:00/36Lge,70:B3:D5:63:10:00/36Senso2Me,70:B3:D5:63:20:00/36PowerEle,70:B3:D5:63:30:00/36Observer,70:B3:D5:63:40:00/36idaqs,70:B3:D5:63:50:00/36Cosylabd,70:B3:D5:63:60:00/36Globalco,70:B3:D5:63:70:00/36Ineo-Sen,70:B3:D5:63:80:00/36Parkalot,70:B3:D5:63:90:00/36DorletSa,70:B3:D5:63:A0:00/36Dave,70:B3:D5:63:B0:00/36LazerSaf,70:B3:D5:63:C0:00/36Pivothea,70:B3:D5:63:D0:00/36TopicEmb,70:B3:D5:63:E0:00/36RikenOpt,70:B3:D5:63:F0:00/36DARBS,70:B3:D5:64:00:00/36Electron,70:B3:D5:64:10:00/36BurkTech,70:B3:D5:64:20:00/36MBconnec,70:B3:D5:64:30:00/36Marques,70:B3:D5:64:40:00/36ATXNetwo,70:B3:D5:64:50:00/36ProjectD,70:B3:D5:64:60:00/36XirgoTec,70:B3:D5:64:70:00/36Kzta,70:B3:D5:64:80:00/36Magnamed,70:B3:D5:64:90:00/36swissled,70:B3:D5:64:A0:00/36NetbricT,70:B3:D5:64:B0:00/36Kalfire,70:B3:D5:64:C0:00/36AcemisFr,70:B3:D5:64:D0:00/36SanminaI,70:B3:D5:64:E0:00/36BigStuff,70:B3:D5:64:F0:00/36GunmaEle,70:B3:D5:65:00:00/36GIFAS-EL,70:B3:D5:65:10:00/36Roxford,70:B3:D5:65:20:00/36RobertBo,70:B3:D5:65:30:00/36LuxarTec,70:B3:D5:65:40:00/36EMAC,70:B3:D5:65:50:00/36AOTSyste,70:B3:D5:65:60:00/36SonoSoun,70:B3:D5:65:70:00/36IDQuanti,70:B3:D5:65:80:00/36emperorb,70:B3:D5:65:90:00/36E2G,70:B3:D5:65:A0:00/36AplexTec,70:B3:D5:65:B0:00/36Roush,70:B3:D5:65:C0:00/36AplexTec,70:B3:D5:65:D0:00/36GegaElec,70:B3:D5:65:E0:00/36SeasonEl,70:B3:D5:65:F0:00/36AxnesAS,70:B3:D5:66:00:00/36SmartSer,70:B3:D5:66:10:00/36DesignAE,70:B3:D5:66:20:00/36IconIndu,70:B3:D5:66:30:00/36Intrinsi,70:B3:D5:66:40:00/36SankyoIn,70:B3:D5:66:50:00/36CertUsus,70:B3:D5:66:60:00/36AplexTec,70:B3:D5:66:70:00/36CT,70:B3:D5:66:80:00/36Öresunds,70:B3:D5:66:90:00/36Panorami,70:B3:D5:66:A0:00/36Private,70:B3:D5:66:B0:00/36Innitive,70:B3:D5:66:C0:00/36KRISTECH,70:B3:D5:66:D0:00/36SanminaI,70:B3:D5:66:E0:00/36Siame,70:B3:D5:66:F0:00/36Simplifi,70:B3:D5:67:00:00/36Particle,70:B3:D5:67:10:00/36SeaShell,70:B3:D5:67:20:00/36KLEIBERI,70:B3:D5:67:30:00/36ACDElekr,70:B3:D5:67:40:00/36Fortress,70:B3:D5:67:50:00/36alfamati,70:B3:D5:67:60:00/36samwooel,70:B3:D5:67:70:00/36Fraunhof,70:B3:D5:67:80:00/36DiniGrou,70:B3:D5:67:90:00/36EMAC,70:B3:D5:67:A0:00/36Micatu,70:B3:D5:67:B0:00/36Stesalit,70:B3:D5:67:C0:00/36Benchmar,70:B3:D5:67:D0:00/36Acrodea,70:B3:D5:67:E0:00/36SeasonEl,70:B3:D5:67:F0:00/36IAAN,70:B3:D5:68:00:00/36BASF,70:B3:D5:68:10:00/36DEUTA-WE,70:B3:D5:68:20:00/36Rosslare,70:B3:D5:68:30:00/36Decyben,70:B3:D5:68:40:00/36LECO,70:B3:D5:68:50:00/36LDAAudio,70:B3:D5:68:60:00/36AccessPr,70:B3:D5:68:70:00/36Volution,70:B3:D5:68:80:00/36MG,70:B3:D5:68:90:00/36PrismaTe,70:B3:D5:68:A0:00/36Advanced,70:B3:D5:68:B0:00/36Sadel,70:B3:D5:68:C0:00/36NdMeter,70:B3:D5:68:D0:00/36Meta-chr,70:B3:D5:68:E0:00/36CEATechn,70:B3:D5:68:F0:00/36PeekTraf,70:B3:D5:69:00:00/36Sicon,70:B3:D5:69:10:00/36PeekTraf,70:B3:D5:69:20:00/36HosinInd,70:B3:D5:69:30:00/36Altron,70:B3:D5:69:40:00/36MoviTHER,70:B3:D5:69:50:00/36GSPSprac,70:B3:D5:69:60:00/36OpenGrow,70:B3:D5:69:70:00/36AlazarTe,70:B3:D5:69:80:00/36Ziehl-Ab,70:B3:D5:69:90:00/36Flextron,70:B3:D5:69:A0:00/36Altaneos,70:B3:D5:69:B0:00/36Horizon,70:B3:D5:69:C0:00/36Keepen,70:B3:D5:69:D0:00/36JPEmbedd,70:B3:D5:69:E0:00/36PTYPE,70:B3:D5:69:F0:00/36T+Aelekt,70:B3:D5:6A:00:00/36ActiveRe,70:B3:D5:6A:10:00/36GlialTec,70:B3:D5:6A:20:00/36RootAuto,70:B3:D5:6A:30:00/36OutdoorL,70:B3:D5:6A:40:00/36Acrodea,70:B3:D5:6A:50:00/36AkenoriP,70:B3:D5:6A:60:00/36WOWSyste,70:B3:D5:6A:70:00/36Partilin,70:B3:D5:6A:80:00/36VitschEl,70:B3:D5:6A:90:00/36OhmoriEl,70:B3:D5:6A:A0:00/36Intermob,70:B3:D5:6A:B0:00/36ArrowChi,70:B3:D5:6A:C0:00/36Ketronix,70:B3:D5:6A:D0:00/36Connit,70:B3:D5:6A:E0:00/36Hangzhou,70:B3:D5:6A:F0:00/36Sensorbe,70:B3:D5:6B:00:00/36PTYPE,70:B3:D5:6B:10:00/36TTCTELEK,70:B3:D5:6B:20:00/36Crde,70:B3:D5:6B:30:00/36DuraComm,70:B3:D5:6B:40:00/36NudronIo,70:B3:D5:6B:50:00/36Art,70:B3:D5:6B:60:00/36INRADIOS,70:B3:D5:6B:70:00/36Grossenb,70:B3:D5:6B:80:00/36Bt9,70:B3:D5:6B:90:00/36BectonDi,70:B3:D5:6B:A0:00/36Integrot,70:B3:D5:6B:B0:00/36Luceo,70:B3:D5:6B:C0:00/36EAElektr,70:B3:D5:6B:D0:00/36RCHVietn,70:B3:D5:6B:E0:00/36VantageI,70:B3:D5:6B:F0:00/36OttoBihl,70:B3:D5:6C:00:00/36NTZMekha,70:B3:D5:6C:10:00/36Labtroni,70:B3:D5:6C:20:00/36TexCompu,70:B3:D5:6C:30:00/36BeijingZ,70:B3:D5:6C:40:00/36VeoRobot,70:B3:D5:6C:50:00/36CJSC«Rus,70:B3:D5:6C:60:00/36AbbottDi,70:B3:D5:6C:70:00/36BectonDi,70:B3:D5:6C:80:00/36Sicon,70:B3:D5:6C:90:00/36Redstone,70:B3:D5:6C:A0:00/36LineageP,70:B3:D5:6C:B0:00/36NAJINaut,70:B3:D5:6C:C0:00/36Arinax,70:B3:D5:6C:D0:00/36Northbou,70:B3:D5:6C:E0:00/36ErediGiu,70:B3:D5:6C:F0:00/36Private,70:B3:D5:6D:00:00/36CodeBlue,70:B3:D5:6D:10:00/36VisualEn,70:B3:D5:6D:20:00/36AhrensBi,70:B3:D5:6D:30:00/36DEUTA-WE,70:B3:D5:6D:40:00/36TelerobG,70:B3:D5:6D:50:00/36PotterEl,70:B3:D5:6D:60:00/36KMtronic,70:B3:D5:6D:70:00/36MBconnec,70:B3:D5:6D:80:00/36Shanghai,70:B3:D5:6D:90:00/36VECTARE,70:B3:D5:6D:A0:00/36Enovativ,70:B3:D5:6D:B0:00/36Techimp-,70:B3:D5:6D:C0:00/36DEUTA-WE,70:B3:D5:6D:D0:00/36AbbottDi,70:B3:D5:6D:E0:00/36AmetekSo,70:B3:D5:6D:F0:00/36MangoDSP,70:B3:D5:6E:00:00/36Abb-Dmpc,70:B3:D5:6E:10:00/36Shanghai,70:B3:D5:6E:20:00/36E-Contro,70:B3:D5:6E:30:00/36ShenZhen,70:B3:D5:6E:40:00/36Institut,70:B3:D5:6E:50:00/36DEUTA-WE,70:B3:D5:6E:60:00/36ElevenEn,70:B3:D5:6E:70:00/36Aml,70:B3:D5:6E:80:00/36BluWirel,70:B3:D5:6E:90:00/36Krontech,70:B3:D5:6E:A0:00/36Edgeware,70:B3:D5:6E:B0:00/36Quantafl,70:B3:D5:6E:C0:00/36Crde,70:B3:D5:6E:D0:00/36Wiingtec,70:B3:D5:6E:E0:00/36HankookC,70:B3:D5:6E:F0:00/36Beringar,70:B3:D5:6F:00:00/36iTelaSof,70:B3:D5:6F:10:00/36Discover,70:B3:D5:6F:20:00/36P&CMicro,70:B3:D5:6F:30:00/36iungo,70:B3:D5:6F:40:00/36Private,70:B3:D5:6F:50:00/36Cominfo,70:B3:D5:6F:60:00/36AccoBran,70:B3:D5:6F:70:00/36Egicon,70:B3:D5:6F:80:00/36SENSEON,70:B3:D5:6F:90:00/36ENVItech,70:B3:D5:6F:A0:00/36Datafort,70:B3:D5:6F:B0:00/36Shachiha,70:B3:D5:6F:C0:00/36MI,70:B3:D5:6F:D0:00/36CoreAkıl,70:B3:D5:6F:E0:00/36NtoIre-P,70:B3:D5:6F:F0:00/36AkeoPlus,70:B3:D5:70:00:00/36Universi,70:B3:D5:70:10:00/36COMPARCo,70:B3:D5:70:20:00/36SensorHi,70:B3:D5:70:30:00/36StromIde,70:B3:D5:70:40:00/36MelecsEW,70:B3:D5:70:50:00/36DigitalM,70:B3:D5:70:60:00/36SmithMet,70:B3:D5:70:70:00/36KocoMoti,70:B3:D5:70:80:00/36IBMResea,70:B3:D5:70:90:00/36Aml,70:B3:D5:70:A0:00/36PullnetT,70:B3:D5:70:B0:00/36AlereTec,70:B3:D5:70:C0:00/36PotterEl,70:B3:D5:70:D0:00/36Omnisens,70:B3:D5:70:E0:00/36WuhanXin,70:B3:D5:70:F0:00/36AlionSci,70:B3:D5:71:00:00/36Guardian,70:B3:D5:71:10:00/36X-Laser,70:B3:D5:71:20:00/36APGCashD,70:B3:D5:71:30:00/36Coloet,70:B3:D5:71:40:00/36AlturnaN,70:B3:D5:71:50:00/36Riot,70:B3:D5:71:60:00/36Lode,70:B3:D5:71:70:00/36SecureSe,70:B3:D5:71:80:00/36PeekTraf,70:B3:D5:71:90:00/362MTechno,70:B3:D5:71:A0:00/36MBconnec,70:B3:D5:71:B0:00/36elsys,70:B3:D5:71:C0:00/36KonzeptI,70:B3:D5:71:D0:00/36Connido,70:B3:D5:71:E0:00/36Motec,70:B3:D5:71:F0:00/36Grayshif,70:B3:D5:72:00:00/36JeioTech,70:B3:D5:72:10:00/36ZoeMedic,70:B3:D5:72:20:00/36Uman,70:B3:D5:72:30:00/36LGElectr,70:B3:D5:72:40:00/36QuanInte,70:B3:D5:72:50:00/36SwissTim,70:B3:D5:72:60:00/36Atgs,70:B3:D5:72:70:00/36LPTechno,70:B3:D5:72:80:00/36BCDAudio,70:B3:D5:72:90:00/36EMAC,70:B3:D5:72:A0:00/36MRC,70:B3:D5:72:B0:00/36Medipens,70:B3:D5:72:C0:00/36NuRi&GEn,70:B3:D5:72:D0:00/36KronMedi,70:B3:D5:72:E0:00/36Maharsys,70:B3:D5:72:F0:00/36AVATechn,70:B3:D5:73:00:00/36Videogen,70:B3:D5:73:10:00/36Phoniro,70:B3:D5:73:20:00/36Tofwerk,70:B3:D5:73:30:00/36Instrume,70:B3:D5:73:40:00/36MansionI,70:B3:D5:73:50:00/36SwissAud,70:B3:D5:73:60:00/36Jabil,70:B3:D5:73:70:00/36SDBiosen,70:B3:D5:73:80:00/36GryphonS,70:B3:D5:73:90:00/36Zigencor,70:B3:D5:73:A0:00/36DolbyLab,70:B3:D5:73:B0:00/36S-I-C,70:B3:D5:73:C0:00/36Centrode,70:B3:D5:73:D0:00/36NETWAYS,70:B3:D5:73:E0:00/36TridentR,70:B3:D5:73:F0:00/36OpenConv,70:B3:D5:74:00:00/36PrismaTe,70:B3:D5:74:10:00/36How-E,70:B3:D5:74:20:00/36YUYAMAMF,70:B3:D5:74:30:00/36EAElektr,70:B3:D5:74:40:00/36PHYZHONH,70:B3:D5:74:50:00/36Tmsi,70:B3:D5:74:60:00/36Smart,70:B3:D5:74:70:00/36EvaAutom,70:B3:D5:74:80:00/36Kdt,70:B3:D5:74:90:00/36GraniteR,70:B3:D5:74:A0:00/36MettlerT,70:B3:D5:74:B0:00/36CodeBlue,70:B3:D5:74:C0:00/36KwantCon,70:B3:D5:74:D0:00/36SpeechTe,70:B3:D5:74:E0:00/36PushCorp,70:B3:D5:74:F0:00/36UnitedSt,70:B3:D5:75:00:00/36NeurioTe,70:B3:D5:75:10:00/36Gnf,70:B3:D5:75:20:00/36GuanShow,70:B3:D5:75:30:00/36HCHKündi,70:B3:D5:75:40:00/36Cosmoit,70:B3:D5:75:50:00/36Landmark,70:B3:D5:75:60:00/36TimeMach,70:B3:D5:75:70:00/36Gabo,70:B3:D5:75:80:00/36Grossenb,70:B3:D5:75:90:00/36Aml,70:B3:D5:75:A0:00/36Standard,70:B3:D5:75:B0:00/36Netool,70:B3:D5:75:C0:00/36UPMTechn,70:B3:D5:75:D0:00/36NanjingM,70:B3:D5:75:E0:00/36Cardinal,70:B3:D5:75:F0:00/36Vocality,70:B3:D5:76:00:00/36Qualitte,70:B3:D5:76:10:00/36Critical,70:B3:D5:76:20:00/36Transfor,70:B3:D5:76:30:00/36ATrapUSA,70:B3:D5:76:40:00/36SCHMIDel,70:B3:D5:76:50:00/36LGElectr,70:B3:D5:76:60:00/36Tirasoft,70:B3:D5:76:70:00/36Franklin,70:B3:D5:76:80:00/36KazanNet,70:B3:D5:76:A0:00/36Swiftnet,70:B3:D5:76:B0:00/36EMPELOR,70:B3:D5:76:C0:00/36Aural,70:B3:D5:76:D0:00/36Trimble,70:B3:D5:76:E0:00/36GrupoEpe,70:B3:D5:76:F0:00/36Oti,70:B3:D5:77:00:00/36Strega,70:B3:D5:77:10:00/36ApatorMi,70:B3:D5:77:20:00/36enModus,70:B3:D5:77:30:00/36RuggedSc,70:B3:D5:77:40:00/36MicramIn,70:B3:D5:77:50:00/36Sonel,70:B3:D5:77:60:00/36Power,70:B3:D5:77:70:00/36QuercusT,70:B3:D5:77:80:00/36Lumacron,70:B3:D5:77:90:00/36DrBridge,70:B3:D5:77:A0:00/36TecsagIn,70:B3:D5:77:B0:00/36AeroVisi,70:B3:D5:77:C0:00/36HUSTYMSt,70:B3:D5:77:D0:00/36APGCashD,70:B3:D5:77:E0:00/36BlueMarb,70:B3:D5:77:F0:00/36Microchi,70:B3:D5:78:00:00/36NidecLer,70:B3:D5:78:10:00/36ProjectS,70:B3:D5:78:20:00/36thou&tec,70:B3:D5:78:30:00/36CHIeru,70:B3:D5:78:40:00/36Shenzhen,70:B3:D5:78:50:00/36Density,70:B3:D5:78:60:00/36RCHVietn,70:B3:D5:78:70:00/36DenAutom,70:B3:D5:78:80:00/36Slan,70:B3:D5:78:90:00/36SEMEX-En,70:B3:D5:78:A0:00/36HillsHea,70:B3:D5:78:B0:00/36JingtuPr,70:B3:D5:78:C0:00/36Survalen,70:B3:D5:78:D0:00/36AVLDiTES,70:B3:D5:78:E0:00/36effectas,70:B3:D5:78:F0:00/36SoFiHa,70:B3:D5:79:00:00/36AVI,70:B3:D5:79:10:00/36RomteckA,70:B3:D5:79:20:00/36Immolas,70:B3:D5:79:30:00/36GastechA,70:B3:D5:79:40:00/36ShadinAv,70:B3:D5:79:50:00/36TIECHEEn,70:B3:D5:79:60:00/36GAMPTmbH,70:B3:D5:79:70:00/36Mitsubis,70:B3:D5:79:80:00/36Tiama,70:B3:D5:79:90:00/36VitecSys,70:B3:D5:79:A0:00/36Innerspe,70:B3:D5:79:B0:00/36Soniclea,70:B3:D5:79:C0:00/36Adde,70:B3:D5:79:D0:00/36Editech,70:B3:D5:79:E0:00/36CW2,70:B3:D5:79:F0:00/36GreenIns,70:B3:D5:7A:00:00/36Reactec,70:B3:D5:7A:10:00/36Excelfor,70:B3:D5:7A:20:00/36AlphaESS,70:B3:D5:7A:30:00/36ImpulseA,70:B3:D5:7A:40:00/36PotterEl,70:B3:D5:7A:50:00/36TritonEl,70:B3:D5:7A:60:00/36Electrol,70:B3:D5:7A:70:00/36Symbicon,70:B3:D5:7A:80:00/36dieEntwi,70:B3:D5:7A:90:00/36adidas,70:B3:D5:7A:A0:00/36Sadel,70:B3:D5:7A:B0:00/36Microgat,70:B3:D5:7A:C0:00/36VeritySt,70:B3:D5:7A:D0:00/36Insitu,70:B3:D5:7A:E0:00/36ExiFlowM,70:B3:D5:7A:F0:00/36Hessware,70:B3:D5:7B:00:00/36Medisafe,70:B3:D5:7B:10:00/36Panamera,70:B3:D5:7B:20:00/36RailPowe,70:B3:D5:7B:30:00/36BroadSof,70:B3:D5:7B:40:00/36ZumbachE,70:B3:D5:7B:50:00/36VOCALTec,70:B3:D5:7B:60:00/36AmadaMiy,70:B3:D5:7B:70:00/36Lsb-LaSa,70:B3:D5:7B:80:00/36SerEnerg,70:B3:D5:7B:90:00/36QIAGENIn,70:B3:D5:7B:A0:00/36Decentla,70:B3:D5:7B:B0:00/36Aloxy,70:B3:D5:7B:C0:00/36FIRSTRF,70:B3:D5:7B:D0:00/36TableCon,70:B3:D5:7B:E0:00/36Phytron,70:B3:D5:7B:F0:00/36StoneThr,70:B3:D5:7C:00:00/36Torgovyy,70:B3:D5:7C:10:00/36DataScie,70:B3:D5:7C:20:00/36MorganSc,70:B3:D5:7C:30:00/36FleximSe,70:B3:D5:7C:40:00/36Mect,70:B3:D5:7C:50:00/36Projects,70:B3:D5:7C:60:00/36UtrendTe,70:B3:D5:7C:70:00/36Sicon,70:B3:D5:7C:80:00/36Crde,70:B3:D5:7C:90:00/36CouncilR,70:B3:D5:7C:A0:00/36HunanShe,70:B3:D5:7C:B0:00/36KeyW,70:B3:D5:7C:C0:00/36Mitsubis,70:B3:D5:7C:D0:00/36Molekule,70:B3:D5:7C:E0:00/36AplexTec,70:B3:D5:7C:F0:00/36ORCATech,70:B3:D5:7D:00:00/36Cubitech,70:B3:D5:7D:10:00/36Schneide,70:B3:D5:7D:20:00/36SDKKrist,70:B3:D5:7D:30:00/36Oledcomm,70:B3:D5:7D:40:00/36Computec,70:B3:D5:7D:50:00/36SICSSwed,70:B3:D5:7D:60:00/36Yukilab,70:B3:D5:7D:70:00/36Gedomo,70:B3:D5:7D:80:00/36Nuand,70:B3:D5:7D:90:00/36ATOMGIKE,70:B3:D5:7D:A0:00/36GrupoEpe,70:B3:D5:7D:B0:00/36aquilabi,70:B3:D5:7D:C0:00/36Software,70:B3:D5:7D:D0:00/36ExcelMed,70:B3:D5:7D:E0:00/36Telaeris,70:B3:D5:7D:F0:00/36RDT,70:B3:D5:7E:00:00/36Sanko-sh,70:B3:D5:7E:10:00/36AppliedM,70:B3:D5:7E:20:00/36DeproÉle,70:B3:D5:7E:30:00/36RedLeafS,70:B3:D5:7E:40:00/36C21,70:B3:D5:7E:50:00/36Megaflex,70:B3:D5:7E:60:00/3611811347,70:B3:D5:7E:70:00/36Atessa,70:B3:D5:7E:80:00/36Mannkind,70:B3:D5:7E:90:00/36Mecsel,70:B3:D5:7E:A0:00/36Waterkot,70:B3:D5:7E:B0:00/36XeroxInt,70:B3:D5:7E:C0:00/36CubicITS,70:B3:D5:7E:D0:00/36ThingsNe,70:B3:D5:7E:E0:00/36Adveez,70:B3:D5:7E:F0:00/36Cravis,70:B3:D5:7F:00:00/36YDKTechn,70:B3:D5:7F:10:00/36AeroVisi,70:B3:D5:7F:20:00/36Tci,70:B3:D5:7F:30:00/36Shenzhen,70:B3:D5:7F:40:00/36KSTtechn,70:B3:D5:7F:50:00/36Incusens,70:B3:D5:7F:60:00/36IDZ,70:B3:D5:7F:70:00/36JASCOApp,70:B3:D5:7F:80:00/36SolveraL,70:B3:D5:7F:90:00/36Communic,70:B3:D5:7F:A0:00/36meoENERG,70:B3:D5:7F:B0:00/36dbBroadc,70:B3:D5:7F:C0:00/36Surion,70:B3:D5:7F:D0:00/36SYSTECel,70:B3:D5:7F:E0:00/36RchItali,70:B3:D5:7F:F0:00/36eumigind,70:B3:D5:80:00:00/36Headsafe,70:B3:D5:80:10:00/36GloryTec,70:B3:D5:80:20:00/36QingdaoC,70:B3:D5:80:30:00/36Grossenb,70:B3:D5:80:40:00/36PMT,70:B3:D5:80:50:00/36Eurotron,70:B3:D5:80:60:00/36Internat,70:B3:D5:80:70:00/36CamsatPr,70:B3:D5:80:80:00/36BectonDi,70:B3:D5:80:90:00/36TecnintH,70:B3:D5:80:A0:00/36SensingL,70:B3:D5:80:B0:00/36FischerB,70:B3:D5:80:C0:00/36Algratec,70:B3:D5:80:D0:00/36DataPhys,70:B3:D5:80:E0:00/36Utopi,70:B3:D5:80:F0:00/36Quickwar,70:B3:D5:81:00:00/36Advice,70:B3:D5:81:10:00/36Cjsc«Int,70:B3:D5:81:20:00/36TESCANBr,70:B3:D5:81:30:00/36Wavemed,70:B3:D5:81:40:00/36Ingenieu,70:B3:D5:81:50:00/36WacoGike,70:B3:D5:81:60:00/36SmithMet,70:B3:D5:81:70:00/36AplexTec,70:B3:D5:81:80:00/36Crde,70:B3:D5:81:90:00/36«Intelle,70:B3:D5:81:A0:00/36JoehlKoe,70:B3:D5:81:B0:00/36bobz,70:B3:D5:81:C0:00/36QIT,70:B3:D5:81:D0:00/36DEUTA-WE,70:B3:D5:81:E0:00/36Novathin,70:B3:D5:81:F0:00/36CAR-conn,70:B3:D5:82:00:00/36BeckerNa,70:B3:D5:82:10:00/36HL2group,70:B3:D5:82:20:00/36AngoraNe,70:B3:D5:82:30:00/36SPContro,70:B3:D5:82:40:00/36SongwooI,70:B3:D5:82:50:00/36Tattile,70:B3:D5:82:60:00/36ElbitAme,70:B3:D5:82:70:00/36Metromat,70:B3:D5:82:80:00/36Xacti,70:B3:D5:82:90:00/36GuanShow,70:B3:D5:82:A0:00/36CWFHamil,70:B3:D5:82:B0:00/36Shangnuo,70:B3:D5:82:C0:00/36NELS,70:B3:D5:82:D0:00/36Elektron,70:B3:D5:82:E0:00/36PlayAliv,70:B3:D5:82:F0:00/36SIANA,70:B3:D5:83:00:00/36Nordson,70:B3:D5:83:10:00/36ArnouseD,70:B3:D5:83:20:00/36PotterEl,70:B3:D5:83:30:00/36AlpiqInT,70:B3:D5:83:40:00/36NCENetwo,70:B3:D5:83:50:00/36CommBoxP,70:B3:D5:83:60:00/36Authenti,70:B3:D5:83:70:00/36HiDes,70:B3:D5:83:80:00/36Tofino,70:B3:D5:83:90:00/36Rockwell,70:B3:D5:83:A0:00/36EmdepCen,70:B3:D5:83:B0:00/36Telefoni,70:B3:D5:83:C0:00/36Sinoembe,70:B3:D5:83:D0:00/36Gentec,70:B3:D5:83:E0:00/36DiniGrou,70:B3:D5:83:F0:00/36LumineLi,70:B3:D5:84:00:00/36xm,70:B3:D5:84:10:00/36Stanet,70:B3:D5:84:20:00/36PLUTOSol,70:B3:D5:84:30:00/36Research,70:B3:D5:84:40:00/36SANSFILT,70:B3:D5:84:50:00/36Harborsi,70:B3:D5:84:60:00/36National,70:B3:D5:84:70:00/36Ai-Lynx,70:B3:D5:84:80:00/36Aldridge,70:B3:D5:84:90:00/36RF-Tuote,70:B3:D5:84:A0:00/36MOGLabor,70:B3:D5:84:B0:00/36QuestHou,70:B3:D5:84:C0:00/36CoreKine,70:B3:D5:84:D0:00/36QuantumD,70:B3:D5:84:E0:00/36Chromalo,70:B3:D5:84:F0:00/36MettlerT,70:B3:D5:85:00:00/36Reo,70:B3:D5:85:10:00/36EXASCEND,70:B3:D5:85:20:00/36NetBoxSC,70:B3:D5:85:30:00/36HghSyste,70:B3:D5:85:40:00/36AdimecAd,70:B3:D5:85:50:00/36Crde,70:B3:D5:85:60:00/36Shanghai,70:B3:D5:85:70:00/36RchItali,70:B3:D5:85:80:00/36HubbellP,70:B3:D5:85:90:00/36HanChang,70:B3:D5:85:A0:00/36Brushies,70:B3:D5:85:B0:00/36Tsubakim,70:B3:D5:85:C0:00/36RobotPub,70:B3:D5:85:D0:00/36Athreya,70:B3:D5:85:E0:00/36XLOGIC,70:B3:D5:85:F0:00/36YUYAMAMF,70:B3:D5:86:00:00/36KBSIndus,70:B3:D5:86:10:00/36KSTtechn,70:B3:D5:86:20:00/36TripleOr,70:B3:D5:86:30:00/36Shenzhen,70:B3:D5:86:40:00/36BORMANNE,70:B3:D5:86:50:00/36Insitu,70:B3:D5:86:60:00/36MEPSReal,70:B3:D5:86:70:00/36Speciali,70:B3:D5:86:80:00/36U-JINMes,70:B3:D5:86:90:00/36chargeBI,70:B3:D5:86:A0:00/36StealthC,70:B3:D5:86:B0:00/36AVLDiTES,70:B3:D5:86:C0:00/36eeas,70:B3:D5:86:D0:00/36CensusDi,70:B3:D5:86:E0:00/36Profcon,70:B3:D5:86:F0:00/36NtcActor,70:B3:D5:87:00:00/36bentrupI,70:B3:D5:87:10:00/36OsoTechn,70:B3:D5:87:20:00/36NipponSa,70:B3:D5:87:30:00/36VishayNo,70:B3:D5:87:40:00/36Northbou,70:B3:D5:87:50:00/36PeekTraf,70:B3:D5:87:60:00/36Ionetech,70:B3:D5:87:70:00/36PolynetT,70:B3:D5:87:80:00/36PackageG,70:B3:D5:87:90:00/36ZIGPOS,70:B3:D5:87:A0:00/36Accolade,70:B3:D5:87:B0:00/36LiquidIn,70:B3:D5:87:C0:00/36Nautel,70:B3:D5:87:D0:00/36Invixium,70:B3:D5:87:E0:00/36Septentr,70:B3:D5:87:F0:00/36NACPlann,70:B3:D5:88:00:00/36Skopei,70:B3:D5:88:10:00/36Tattile,70:B3:D5:88:20:00/36SimonTec,70:B3:D5:88:30:00/36ContecAm,70:B3:D5:88:40:00/36LGElectr,70:B3:D5:88:50:00/36QuirkLog,70:B3:D5:88:60:00/36MBconnec,70:B3:D5:88:70:00/36EntecSol,70:B3:D5:88:80:00/36Zetechti,70:B3:D5:88:90:00/36Innovati,70:B3:D5:88:A0:00/36Percepti,70:B3:D5:88:B0:00/36WUHANEAS,70:B3:D5:88:D0:00/36LGElectr,70:B3:D5:88:E0:00/36RCHVietn,70:B3:D5:88:F0:00/36QuaestaI,70:B3:D5:89:00:00/36EIDOS,70:B3:D5:89:10:00/36neocontr,70:B3:D5:89:20:00/36Abb,70:B3:D5:89:30:00/36Cubitech,70:B3:D5:89:40:00/36UnISyste,70:B3:D5:89:50:00/36Integrat,70:B3:D5:89:60:00/36Shanghai,70:B3:D5:89:70:00/36EFGCZspo,70:B3:D5:89:80:00/36SalupoSa,70:B3:D5:89:90:00/36ViotecUS,70:B3:D5:89:A0:00/36AlgodueE,70:B3:D5:89:B0:00/36ControlW,70:B3:D5:89:C0:00/36IHIRotat,70:B3:D5:89:D0:00/36e-Matix,70:B3:D5:89:E0:00/36Innovati,70:B3:D5:89:F0:00/36Levelup,70:B3:D5:8A:00:00/36DmRadioc,70:B3:D5:8A:10:00/36Tiama,70:B3:D5:8A:20:00/36WinnersD,70:B3:D5:8A:30:00/36Loehnert,70:B3:D5:8A:40:00/36PhytonMi,70:B3:D5:8A:50:00/36KSTtechn,70:B3:D5:8A:60:00/36Crde,70:B3:D5:8A:70:00/36TucsenPh,70:B3:D5:8A:80:00/36megatece,70:B3:D5:8A:90:00/36WoKa-Ele,70:B3:D5:8A:A0:00/36Tattile,70:B3:D5:8A:B0:00/36EMAC,70:B3:D5:8A:C0:00/36ASUNGTE,70:B3:D5:8A:D0:00/36GlobalCo,70:B3:D5:8A:E0:00/36Fareco,70:B3:D5:8A:F0:00/36QbicComm,70:B3:D5:8B:00:00/36IES,70:B3:D5:8B:10:00/36M-TechIn,70:B3:D5:8B:20:00/36NPFModem,70:B3:D5:8B:30:00/36FireflyR,70:B3:D5:8B:40:00/36Scenario,70:B3:D5:8B:50:00/36xTom,70:B3:D5:8B:60:00/36Eldes,70:B3:D5:8B:70:00/36ContecAm,70:B3:D5:8B:80:00/36GDITechn,70:B3:D5:8B:90:00/36Toptech,70:B3:D5:8B:A0:00/36Tiama,70:B3:D5:8B:B0:00/36KSTtechn,70:B3:D5:8B:C0:00/36GSIGeoSo,70:B3:D5:8B:D0:00/36MahleEle,70:B3:D5:8B:E0:00/36Connoise,70:B3:D5:8B:F0:00/36Hangzhou,70:B3:D5:8C:00:00/36SenseNL,70:B3:D5:8C:10:00/36Rievtech,70:B3:D5:8C:20:00/36F-domain,70:B3:D5:8C:30:00/36Wyebot,70:B3:D5:8C:40:00/36APE,70:B3:D5:8C:50:00/36HMicro,70:B3:D5:8C:60:00/36Onosokki,70:B3:D5:8C:70:00/36Henschel,70:B3:D5:8C:80:00/36Kronotec,70:B3:D5:8C:90:00/36ArwinTec,70:B3:D5:8C:A0:00/36AlliedDa,70:B3:D5:8C:B0:00/36WELT,70:B3:D5:8C:C0:00/36PiranhaE,70:B3:D5:8C:D0:00/36EAElektr,70:B3:D5:8C:E0:00/36CORES,70:B3:D5:8C:F0:00/36Dainichi,70:B3:D5:8D:00:00/36RaftTech,70:B3:D5:8D:10:00/36FieldDes,70:B3:D5:8D:20:00/36Wizapply,70:B3:D5:8D:30:00/36Performa,70:B3:D5:8D:40:00/36Guangdon,70:B3:D5:8D:50:00/36Guangzho,70:B3:D5:8D:60:00/36BeijingX,70:B3:D5:8D:70:00/36Schneide,70:B3:D5:8D:80:00/36VNG,70:B3:D5:8D:90:00/36MBconnec,70:B3:D5:8D:A0:00/36MicroEle,70:B3:D5:8D:B0:00/36KratosAn,70:B3:D5:8D:C0:00/36NiveoInt,70:B3:D5:8D:D0:00/36Vertex,70:B3:D5:8D:E0:00/36Induther,70:B3:D5:8D:F0:00/36DorletSa,70:B3:D5:8E:00:00/36SoudaxEq,70:B3:D5:8E:10:00/36WoKa-Ele,70:B3:D5:8E:20:00/36ZhiyeEle,70:B3:D5:8E:30:00/36DorletSa,70:B3:D5:8E:40:00/36AplexTec,70:B3:D5:8E:50:00/36Shanghai,70:B3:D5:8E:60:00/36Mothonic,70:B3:D5:8E:70:00/36Reo,70:B3:D5:8E:80:00/36PreoIndu,70:B3:D5:8E:90:00/36COONTROL,70:B3:D5:8E:A0:00/36JLCooper,70:B3:D5:8E:B0:00/36ProconEl,70:B3:D5:8E:C0:00/36RudyTell,70:B3:D5:8E:D0:00/36NanoSens,70:B3:D5:8E:E0:00/36NetworkA,70:B3:D5:8E:F0:00/36BeeperCo,70:B3:D5:8F:00:00/36ERAESEED,70:B3:D5:8F:10:00/36Paramoun,70:B3:D5:8F:20:00/36Rimota,70:B3:D5:8F:30:00/36Tattile,70:B3:D5:8F:40:00/36ACQUAsrl,70:B3:D5:8F:50:00/36Stmovic,70:B3:D5:8F:60:00/36Dofuntec,70:B3:D5:8F:70:00/36IESevkoA,70:B3:D5:8F:80:00/36Wi6labs,70:B3:D5:8F:90:00/36IWSGloba,70:B3:D5:8F:A0:00/36DeaSyste,70:B3:D5:8F:B0:00/36MBconnec,70:B3:D5:8F:C0:00/36MianjieT,70:B3:D5:8F:D0:00/36sonatest,70:B3:D5:8F:E0:00/36Selmatec,70:B3:D5:8F:F0:00/36IMST,70:B3:D5:90:10:00/36Ats-Conv,70:B3:D5:90:20:00/36Unlimite,70:B3:D5:90:30:00/36Cymtec,70:B3:D5:90:40:00/36PHBEletr,70:B3:D5:90:50:00/36Wexiodis,70:B3:D5:90:60:00/36AplexTec,70:B3:D5:90:70:00/36NingboCr,70:B3:D5:90:80:00/36Accusoni,70:B3:D5:90:90:00/36tetronik,70:B3:D5:90:A0:00/36Hangzhou,70:B3:D5:90:B0:00/36MatrixSw,70:B3:D5:90:C0:00/36ANTEK,70:B3:D5:90:D0:00/36Modtroni,70:B3:D5:90:E0:00/36Maytroni,70:B3:D5:90:F0:00/36DTRONCom,70:B3:D5:91:00:00/36Eginity,70:B3:D5:91:10:00/36Equatel,70:B3:D5:91:20:00/36VertelDi,70:B3:D5:91:30:00/36Shenzhen,70:B3:D5:91:40:00/36ContecAm,70:B3:D5:91:50:00/36DHKStora,70:B3:D5:91:60:00/36TechnoMa,70:B3:D5:91:70:00/36KSJ,70:B3:D5:91:80:00/36GlovaRai,70:B3:D5:91:90:00/36Thesycon,70:B3:D5:91:A0:00/36FujianLa,70:B3:D5:91:B0:00/36Dolotron,70:B3:D5:91:C0:00/36AlereTec,70:B3:D5:91:D0:00/36Cubitech,70:B3:D5:91:E0:00/36Creotech,70:B3:D5:91:F0:00/36InformIn,70:B3:D5:92:00:00/36Slat,70:B3:D5:92:10:00/36QDevil,70:B3:D5:92:20:00/36AdcoleSp,70:B3:D5:92:30:00/36eumigind,70:B3:D5:92:40:00/36Meridian,70:B3:D5:92:50:00/36Diamante,70:B3:D5:92:60:00/36Advice,70:B3:D5:92:70:00/36LGElectr,70:B3:D5:92:80:00/36DoneDesi,70:B3:D5:92:90:00/36OutSys,70:B3:D5:92:A0:00/36Miravue,70:B3:D5:92:B0:00/36ENTECEle,70:B3:D5:92:C0:00/36Dismunte,70:B3:D5:92:D0:00/36SuzhouWa,70:B3:D5:92:E0:00/36MedicalM,70:B3:D5:92:F0:00/36SiFive,70:B3:D5:93:00:00/36Institut,70:B3:D5:93:10:00/36MarineIn,70:B3:D5:93:20:00/36Rohde&Sc,70:B3:D5:93:30:00/36S@Tis,70:B3:D5:93:40:00/36RBSNetko,70:B3:D5:93:50:00/36SensorDe,70:B3:D5:93:60:00/36FaroTech,70:B3:D5:93:70:00/36Tattile,70:B3:D5:93:80:00/36JETITech,70:B3:D5:93:90:00/36Invertek,70:B3:D5:93:A0:00/36BraemarM,70:B3:D5:93:B0:00/36Changchu,70:B3:D5:93:C0:00/36GSPSprac,70:B3:D5:93:D0:00/36Elmeasur,70:B3:D5:93:E0:00/36WithInte,70:B3:D5:93:F0:00/36VisionSe,70:B3:D5:94:00:00/36Paradigm,70:B3:D5:94:10:00/36Triax,70:B3:D5:94:20:00/36TruTeqDe,70:B3:D5:94:30:00/36AbbottMe,70:B3:D5:94:40:00/36Chromate,70:B3:D5:94:50:00/36Symbotic,70:B3:D5:94:60:00/36GREATWAL,70:B3:D5:94:70:00/36Checkbil,70:B3:D5:94:80:00/36VisionAu,70:B3:D5:94:90:00/36National,70:B3:D5:94:A0:00/36Shenzhen,70:B3:D5:94:B0:00/36RFCode,70:B3:D5:94:C0:00/36Honeywel,70:B3:D5:94:D0:00/36SeasonDe,70:B3:D5:94:E0:00/36BPLubric,70:B3:D5:94:F0:00/36MartNetw,70:B3:D5:95:00:00/36CMTMedic,70:B3:D5:95:10:00/36Trident,70:B3:D5:95:20:00/36Requea,70:B3:D5:95:30:00/36Spectrum,70:B3:D5:95:40:00/36DotSyste,70:B3:D5:95:50:00/36Dynacard,70:B3:D5:95:60:00/36AeroVisi,70:B3:D5:95:70:00/36EAElektr,70:B3:D5:95:80:00/36pureLiFi,70:B3:D5:95:90:00/36ZulexInt,70:B3:D5:95:A0:00/36SigmannE,70:B3:D5:95:B0:00/36SRSGroup,70:B3:D5:95:C0:00/36WilsonEl,70:B3:D5:95:D0:00/36Giordano,70:B3:D5:95:E0:00/36Blocksi,70:B3:D5:95:F0:00/36WiFiNati,70:B3:D5:96:00:00/36HorizonT,70:B3:D5:96:10:00/36TaskSist,70:B3:D5:96:20:00/36Senquire,70:B3:D5:96:30:00/36Triax,70:B3:D5:96:40:00/36Visility,70:B3:D5:96:50:00/36LineageP,70:B3:D5:96:60:00/36dATomato,70:B3:D5:96:70:00/36Tattile,70:B3:D5:96:80:00/36LGMIngén,70:B3:D5:96:90:00/36EmtelSys,70:B3:D5:96:A0:00/36AnelloPh,70:B3:D5:96:B0:00/36FOCAL-JM,70:B3:D5:96:C0:00/36WebleSàr,70:B3:D5:96:D0:00/36MSBElekt,70:B3:D5:96:E0:00/36MyostatM,70:B3:D5:96:F0:00/364CAM,70:B3:D5:97:00:00/36Bintel,70:B3:D5:97:10:00/36RchItali,70:B3:D5:97:20:00/36AixContr,70:B3:D5:97:30:00/36Autonomi,70:B3:D5:97:40:00/36JirehInd,70:B3:D5:97:50:00/36CoesterA,70:B3:D5:97:60:00/36AtonarpM,70:B3:D5:97:70:00/36EngageTe,70:B3:D5:97:80:00/36SatixfyI,70:B3:D5:97:90:00/36eSMARTTe,70:B3:D5:97:A0:00/36Orion,70:B3:D5:97:B0:00/36WIKAAlex,70:B3:D5:97:C0:00/36Nu-TekPo,70:B3:D5:97:D0:00/36RCHVietn,70:B3:D5:97:E0:00/36PublicJo,70:B3:D5:97:F0:00/36BISTOS,70:B3:D5:98:00:00/36BeijingY,70:B3:D5:98:10:00/36ZamirRec,70:B3:D5:98:20:00/363S-Senso,70:B3:D5:98:30:00/36ENSEngin,70:B3:D5:98:40:00/36SanminaI,70:B3:D5:98:50:00/36BurkTech,70:B3:D5:98:60:00/36AplexTec,70:B3:D5:98:70:00/36Axis,70:B3:D5:98:80:00/36Arris,70:B3:D5:98:90:00/36Dcns,70:B3:D5:98:A0:00/36visionsa,70:B3:D5:98:B0:00/36RichardP,70:B3:D5:98:C0:00/36Universi,70:B3:D5:98:D0:00/36Motohaus,70:B3:D5:98:E0:00/36AutocomD,70:B3:D5:98:F0:00/36Spacefli,70:B3:D5:99:00:00/36EnergyWa,70:B3:D5:99:10:00/36Javaspar,70:B3:D5:99:20:00/36Kaeonit,70:B3:D5:99:30:00/36ioThings,70:B3:D5:99:40:00/36KeFFNetw,70:B3:D5:99:50:00/36LayTec,70:B3:D5:99:60:00/36XpertSea,70:B3:D5:99:70:00/36ProTomIn,70:B3:D5:99:80:00/36KitaKirm,70:B3:D5:99:90:00/36Logicube,70:B3:D5:99:A0:00/36KEVIC,70:B3:D5:99:B0:00/36RchItali,70:B3:D5:99:C0:00/36Enerwise,70:B3:D5:99:D0:00/36Opsys-Te,70:B3:D5:99:E0:00/36TrinityC,70:B3:D5:99:F0:00/36Confed,70:B3:D5:9A:00:00/36Eldes,70:B3:D5:9A:10:00/36ITSIndus,70:B3:D5:9A:20:00/36O-NetCom,70:B3:D5:9A:30:00/36Shanghai,70:B3:D5:9A:40:00/36Nordmann,70:B3:D5:9A:50:00/36Softel,70:B3:D5:9A:60:00/36QunuLabs,70:B3:D5:9A:70:00/36Honeywel,70:B3:D5:9A:80:00/36Egag,70:B3:D5:9A:90:00/36PABLOAIR,70:B3:D5:9A:A0:00/36Tecsysdo,70:B3:D5:9A:B0:00/36GroupePa,70:B3:D5:9A:C0:00/36SuzhouSa,70:B3:D5:9A:D0:00/36FortunaI,70:B3:D5:9A:E0:00/36Volansys,70:B3:D5:9A:F0:00/36Shanghai,70:B3:D5:9B:00:00/36ClearlyI,70:B3:D5:9B:10:00/36AplexTec,70:B3:D5:9B:20:00/36CONTINEN,70:B3:D5:9B:30:00/36K&JSchmi,70:B3:D5:9B:40:00/36MyoungSu,70:B3:D5:9B:50:00/36Ideetron,70:B3:D5:9B:60:00/36Intercom,70:B3:D5:9B:70:00/36Itronics,70:B3:D5:9B:80:00/36Lomasro,70:B3:D5:9B:90:00/36AetheraT,70:B3:D5:9B:A0:00/36ATIMRadi,70:B3:D5:9B:B0:00/36Jinga-hi,70:B3:D5:9B:C0:00/36RadianRe,70:B3:D5:9B:D0:00/36SignalPr,70:B3:D5:9B:E0:00/36Izome,70:B3:D5:9B:F0:00/36XirisAut,70:B3:D5:9C:00:00/36Schneide,70:B3:D5:9C:10:00/36Zeroplus,70:B3:D5:9C:20:00/36Sportsbe,70:B3:D5:9C:30:00/36Sevensen,70:B3:D5:9C:40:00/36aelettro,70:B3:D5:9C:50:00/36LineageP,70:B3:D5:9C:60:00/36Overspee,70:B3:D5:9C:70:00/36YUYAMAMF,70:B3:D5:9C:80:00/36AppliedE,70:B3:D5:9C:90:00/36PKSound,70:B3:D5:9C:A0:00/36KomsisEl,70:B3:D5:9C:B0:00/36Alligato,70:B3:D5:9C:C0:00/36Zaxcom,70:B3:D5:9C:D0:00/36WEPTECHe,70:B3:D5:9C:E0:00/36Terragen,70:B3:D5:9C:F0:00/36Iotize,70:B3:D5:9D:00:00/36RJ45Tech,70:B3:D5:9D:10:00/36OS42UGha,70:B3:D5:9D:20:00/36AcsMotio,70:B3:D5:9D:30:00/36Communic,70:B3:D5:9D:40:00/36Wartsila,70:B3:D5:9D:50:00/36Southern,70:B3:D5:9D:60:00/36CrownSol,70:B3:D5:9D:70:00/36KMOptoEl,70:B3:D5:9D:80:00/36JOLANYEE,70:B3:D5:9D:90:00/36ATXNetwo,70:B3:D5:9D:A0:00/36BlakeUK,70:B3:D5:9D:B0:00/36CASMedic,70:B3:D5:9D:C0:00/36Shanghai,70:B3:D5:9D:D0:00/36HumanEye,70:B3:D5:9D:E0:00/36System11,70:B3:D5:9D:F0:00/36DOBEComp,70:B3:D5:9E:00:00/36ESIndust,70:B3:D5:9E:10:00/36BolideTe,70:B3:D5:9E:20:00/36OfilUSA,70:B3:D5:9E:30:00/36LGElectr,70:B3:D5:9E:40:00/36K&AElect,70:B3:D5:9E:50:00/36AntekTec,70:B3:D5:9E:60:00/36Blocksi,70:B3:D5:9E:70:00/36XiamenMa,70:B3:D5:9E:80:00/36Zerospac,70:B3:D5:9E:90:00/36LiveCopp,70:B3:D5:9E:A0:00/36BlueStor,70:B3:D5:9E:B0:00/36PrestonI,70:B3:D5:9E:C0:00/36eSoftThi,70:B3:D5:9E:D0:00/36Benchmar,70:B3:D5:9E:E0:00/36Lockheed,70:B3:D5:9E:F0:00/36Cottonwo,70:B3:D5:9F:00:00/36FUJICOM,70:B3:D5:9F:10:00/36RFEL,70:B3:D5:9F:20:00/36AcordeTe,70:B3:D5:9F:30:00/36IEEERegi,70:B3:D5:9F:40:00/36Tband,70:B3:D5:9F:50:00/36VickersE,70:B3:D5:9F:60:00/36Edgeware,70:B3:D5:9F:70:00/36Foerster,70:B3:D5:9F:80:00/36Asymmetr,70:B3:D5:9F:90:00/36FluidCom,70:B3:D5:9F:A0:00/36Ideas,70:B3:D5:9F:B0:00/36UnicomGl,70:B3:D5:9F:C0:00/36TruecomT,70:B3:D5:9F:D0:00/36amakiden,70:B3:D5:9F:E0:00/36SurugaSe,70:B3:D5:9F:F0:00/36NetworkI,70:B3:D5:A0:00:00/36AtxNetwo,70:B3:D5:A0:10:00/36FeldTech,70:B3:D5:A0:20:00/36GreenFlu,70:B3:D5:A0:30:00/36Proemion,70:B3:D5:A0:40:00/36GaleaEle,70:B3:D5:A0:50:00/36Wartsila,70:B3:D5:A0:60:00/36KopisMob,70:B3:D5:A0:70:00/36IoTrekTe,70:B3:D5:A0:80:00/36BioBusin,70:B3:D5:A0:90:00/36SmartEmb,70:B3:D5:A0:A0:00/36Capsys,70:B3:D5:A0:B0:00/36ambiHome,70:B3:D5:A0:C0:00/36Lumiplan,70:B3:D5:A0:D0:00/36Globalco,70:B3:D5:A0:E0:00/36Vetaphon,70:B3:D5:A0:F0:00/36OsakiDat,70:B3:D5:A1:00:00/36w-tec,70:B3:D5:A1:10:00/36Trioptic,70:B3:D5:A1:20:00/36QuercusT,70:B3:D5:A1:30:00/36Uplevel,70:B3:D5:A1:40:00/36aelettro,70:B3:D5:A1:50:00/36Intercor,70:B3:D5:A1:60:00/36devAIs,70:B3:D5:A1:70:00/36Tunstall,70:B3:D5:A1:80:00/36Embedded,70:B3:D5:A1:90:00/36Qualitro,70:B3:D5:A1:A0:00/36Nueon-CO,70:B3:D5:A1:B0:00/36PotterEl,70:B3:D5:A1:C0:00/36MecaSyst,70:B3:D5:A1:D0:00/36FluidCom,70:B3:D5:A1:E0:00/36Monnit,70:B3:D5:A1:F0:00/36GlobalTe,70:B3:D5:A2:00:00/36DesignFo,70:B3:D5:A2:10:00/36PPI,70:B3:D5:A2:20:00/36eSysSolu,70:B3:D5:A2:30:00/36LGElectr,70:B3:D5:A2:40:00/36BoozAlle,70:B3:D5:A2:50:00/36PulseTor,70:B3:D5:A2:60:00/36HearGear,70:B3:D5:A2:70:00/36HDLdaAma,70:B3:D5:A2:80:00/36PeekTraf,70:B3:D5:A2:90:00/36QIAGENIn,70:B3:D5:A2:A0:00/36Redwood,70:B3:D5:A2:B0:00/36CleverDe,70:B3:D5:A2:C0:00/36Tlv,70:B3:D5:A2:D0:00/36ProjectS,70:B3:D5:A2:E0:00/36Kokam,70:B3:D5:A2:F0:00/36Botek,70:B3:D5:A3:00:00/36ShenZhen,70:B3:D5:A3:10:00/36Private,70:B3:D5:A3:20:00/36Toughdog,70:B3:D5:A3:30:00/36Tiama,70:B3:D5:A3:40:00/36RchItali,70:B3:D5:A3:50:00/36Sicon,70:B3:D5:A3:60:00/36BeijingD,70:B3:D5:A3:70:00/36Mitsubis,70:B3:D5:A3:80:00/36Aditec,70:B3:D5:A3:90:00/36SPETSSTR,70:B3:D5:A3:A0:00/36EPSOFT,70:B3:D5:A3:B0:00/36GraceDes,70:B3:D5:A3:C0:00/36WaveMusi,70:B3:D5:A3:D0:00/36SMARTINO,70:B3:D5:A3:E0:00/36Vigorclo,70:B3:D5:A3:F0:00/36PHPower,70:B3:D5:A4:00:00/36STRACKLI,70:B3:D5:A4:10:00/36THELIGHT,70:B3:D5:A4:20:00/36iMARNavi,70:B3:D5:A4:30:00/36Oledcomm,70:B3:D5:A4:40:00/36Fsr,70:B3:D5:A4:50:00/36ViperInn,70:B3:D5:A4:60:00/36Foxconn4,70:B3:D5:A4:70:00/36Kanoa,70:B3:D5:A4:80:00/36AppliedS,70:B3:D5:A4:90:00/36Unipower,70:B3:D5:A4:A0:00/36BeijingA,70:B3:D5:A4:B0:00/36McKayBro,70:B3:D5:A4:C0:00/36AlereTec,70:B3:D5:A4:D0:00/36Lansitec,70:B3:D5:A4:E0:00/36ArrayTec,70:B3:D5:A4:F0:00/36WeltekTe,70:B3:D5:A5:00:00/36Lecip,70:B3:D5:A5:10:00/36RFCode,70:B3:D5:A5:20:00/36APEXStab,70:B3:D5:A5:30:00/36GSIndust,70:B3:D5:A5:40:00/36provedo,70:B3:D5:A5:50:00/36EmbestTe,70:B3:D5:A5:60:00/36DorletSa,70:B3:D5:A5:70:00/36Pcsc,70:B3:D5:A5:80:00/36MCQTECH,70:B3:D5:A5:90:00/36Muuntosä,70:B3:D5:A5:A0:00/36RCSEnerg,70:B3:D5:A5:B0:00/36ChristEl,70:B3:D5:A5:C0:00/36Molekule,70:B3:D5:A5:D0:00/36Position,70:B3:D5:A5:E0:00/36ConectaI,70:B3:D5:A5:F0:00/36Daatrics,70:B3:D5:A6:00:00/36Pneumax,70:B3:D5:A6:10:00/36OmskManu,70:B3:D5:A6:20:00/36Environe,70:B3:D5:A6:30:00/36DesignAE,70:B3:D5:A6:40:00/36Newshine,70:B3:D5:A6:50:00/36Creative,70:B3:D5:A6:60:00/36TrapezeS,70:B3:D5:A6:70:00/36GstarCre,70:B3:D5:A6:80:00/36Zhejiang,70:B3:D5:A6:90:00/36Leviatha,70:B3:D5:A6:A0:00/36Privafy,70:B3:D5:A6:B0:00/36xmi,70:B3:D5:A6:C0:00/36Controle,70:B3:D5:A6:D0:00/36MetekMet,70:B3:D5:A6:E0:00/36Electric,70:B3:D5:A6:F0:00/368Cups,70:B3:D5:A7:00:00/36Gateview,70:B3:D5:A7:10:00/36SamwellI,70:B3:D5:A7:20:00/36Business,70:B3:D5:A7:30:00/36MobiProm,70:B3:D5:A7:40:00/36Sadel,70:B3:D5:A7:50:00/36TaejinIn,70:B3:D5:A7:60:00/36PietroFi,70:B3:D5:A7:70:00/36SPXRadio,70:B3:D5:A7:80:00/36Bionics,70:B3:D5:A7:90:00/36NOREYATe,70:B3:D5:A7:A0:00/36FluidMan,70:B3:D5:A7:B0:00/36SmartSaf,70:B3:D5:A7:C0:00/36Transele,70:B3:D5:A7:D0:00/36PriorSci,70:B3:D5:A7:E0:00/36QUICCOSO,70:B3:D5:A7:F0:00/36AudioVis,70:B3:D5:A8:00:00/36Evco,70:B3:D5:A8:10:00/36SiendaNe,70:B3:D5:A8:20:00/36Telefran,70:B3:D5:A8:30:00/36SHENZHEN,70:B3:D5:A8:40:00/36SORELMik,70:B3:D5:A8:50:00/36exceetel,70:B3:D5:A8:60:00/36Divigrap,70:B3:D5:A8:70:00/36TornadoM,70:B3:D5:A8:80:00/36Shangdon,70:B3:D5:A8:90:00/36GbsCommu,70:B3:D5:A8:A0:00/36VISTGrou,70:B3:D5:A8:B0:00/36GiantPow,70:B3:D5:A8:C0:00/36CygContr,70:B3:D5:A8:D0:00/36CodeBlue,70:B3:D5:A8:E0:00/36OmeshCit,70:B3:D5:A8:F0:00/36VKIntegr,70:B3:D5:A9:00:00/36ERA,70:B3:D5:A9:10:00/36IDEALIND,70:B3:D5:A9:20:00/36Grossenb,70:B3:D5:A9:30:00/36MesCommu,70:B3:D5:A9:40:00/36ETATechn,70:B3:D5:A9:50:00/36DEUTA-WE,70:B3:D5:A9:60:00/36ÖstlingM,70:B3:D5:A9:70:00/36Bizwerks,70:B3:D5:A9:80:00/36Pantec,70:B3:D5:A9:90:00/36Bandelin,70:B3:D5:A9:A0:00/36Amphenol,70:B3:D5:A9:B0:00/36Osmozis,70:B3:D5:A9:C0:00/36VeoTechn,70:B3:D5:A9:D0:00/36VitecMul,70:B3:D5:A9:E0:00/36ArgonST,70:B3:D5:A9:F0:00/36MasterMe,70:B3:D5:AA:00:00/36SimpleWo,70:B3:D5:AA:10:00/36Shenzhen,70:B3:D5:AA:20:00/36eumigind,70:B3:D5:AA:30:00/36LineageP,70:B3:D5:AA:40:00/36PullnetT,70:B3:D5:AA:50:00/36MBconnec,70:B3:D5:AA:60:00/36Proximus,70:B3:D5:AA:70:00/36Ateme,70:B3:D5:AA:80:00/36West-Com,70:B3:D5:AA:90:00/36Datamars,70:B3:D5:AA:A0:00/36Xemex,70:B3:D5:AA:B0:00/36QUISS,70:B3:D5:AA:C0:00/36SensoTec,70:B3:D5:AA:D0:00/36Bartec,70:B3:D5:AA:E0:00/36Nuviz,70:B3:D5:AA:F0:00/36ExiFlowM,70:B3:D5:AB:00:00/36OsrR&DIs,70:B3:D5:AB:10:00/36ISRVZrt,70:B3:D5:AB:20:00/36PowerEle,70:B3:D5:AB:30:00/36Micas,70:B3:D5:AB:40:00/36SYSTECel,70:B3:D5:AB:50:00/36BroadSof,70:B3:D5:AB:60:00/36SmartDTe,70:B3:D5:AB:70:00/36Siglead,70:B3:D5:AB:80:00/36HoribaAb,70:B3:D5:AB:90:00/36DynamicC,70:B3:D5:AB:A0:00/36CLIntern,70:B3:D5:AB:B0:00/36DavidHor,70:B3:D5:AB:C0:00/36BKM-Micr,70:B3:D5:AB:D0:00/36wtec,70:B3:D5:AB:E0:00/36MartNetw,70:B3:D5:AB:F0:00/36AGRInter,70:B3:D5:AC:00:00/36Ritec,70:B3:D5:AC:10:00/36AEMSinga,70:B3:D5:AC:20:00/36Wisebox,70:B3:D5:AC:30:00/36Novoptel,70:B3:D5:AC:40:00/36LexiDevi,70:B3:D5:AC:50:00/36ATOMGIKE,70:B3:D5:AC:60:00/36SMTC,70:B3:D5:AC:70:00/36vivaMOS,70:B3:D5:AC:80:00/36Heartlan,70:B3:D5:AC:90:00/36TrinityS,70:B3:D5:AC:A0:00/36TecnintH,70:B3:D5:AC:B0:00/36Tattile,70:B3:D5:AC:C0:00/36Schneide,70:B3:D5:AC:D0:00/36Crde,70:B3:D5:AC:E0:00/36FarhoDom,70:B3:D5:AC:F0:00/36APGCashD,70:B3:D5:AD:00:00/36Reo,70:B3:D5:AD:10:00/36SensileT,70:B3:D5:AD:20:00/36Wart-Ele,70:B3:D5:AD:30:00/36Warecube,70:B3:D5:AD:40:00/36Invissys,70:B3:D5:AD:50:00/36Birdland,70:B3:D5:AD:60:00/36Lemonade,70:B3:D5:AD:70:00/36OctopusI,70:B3:D5:AD:80:00/36Euklisby,70:B3:D5:AD:90:00/36aelettro,70:B3:D5:AD:A0:00/36Private,70:B3:D5:AD:B0:00/36RFCode,70:B3:D5:AD:C0:00/36Sodaq,70:B3:D5:AD:D0:00/36GHLBerha,70:B3:D5:AD:E0:00/36Isac,70:B3:D5:AD:F0:00/36Seraphim,70:B3:D5:AE:00:00/36AnyComm,70:B3:D5:AE:10:00/36DimoCore,70:B3:D5:AE:20:00/36Wartsila,70:B3:D5:AE:30:00/36Zhejiang,70:B3:D5:AE:40:00/36NuanceHe,70:B3:D5:AE:50:00/36BeatCraf,70:B3:D5:AE:60:00/36YaBathoT,70:B3:D5:AE:70:00/36E-T-AEle,70:B3:D5:AE:80:00/36Innoknig,70:B3:D5:AE:90:00/36CariElec,70:B3:D5:AE:A0:00/36BBRVerke,70:B3:D5:AE:B0:00/36Associat,70:B3:D5:AE:C0:00/36Paratec,70:B3:D5:AE:D0:00/36Cubitech,70:B3:D5:AE:E0:00/36DiTESTFa,70:B3:D5:AE:F0:00/36Baumtec,70:B3:D5:AF:00:00/36SeasonDe,70:B3:D5:AF:10:00/36EmkaTech,70:B3:D5:AF:20:00/36TrueNetw,70:B3:D5:AF:30:00/36NewJapan,70:B3:D5:AF:40:00/36Tattile,70:B3:D5:AF:50:00/36NetAndPr,70:B3:D5:AF:60:00/36SCE,70:B3:D5:AF:70:00/36DimoSyst,70:B3:D5:AF:80:00/36boekel,70:B3:D5:AF:90:00/36Critical,70:B3:D5:AF:A0:00/36PowerSec,70:B3:D5:AF:B0:00/36Shanghai,70:B3:D5:AF:C0:00/36BAE,70:B3:D5:AF:D0:00/36dongshen,70:B3:D5:AF:E0:00/36Mesotech,70:B3:D5:AF:F0:00/36digital-,70:B3:D5:B0:00:00/36HoribaAb,70:B3:D5:B0:10:00/36GSDGroup,70:B3:D5:B0:20:00/36NordicAu,70:B3:D5:B0:30:00/36Sprintsh,70:B3:D5:B0:40:00/36Herrmann,70:B3:D5:B0:50:00/36E-PlusTe,70:B3:D5:B0:60:00/36Multivoi,70:B3:D5:B0:70:00/36Arrowval,70:B3:D5:B0:80:00/36Secuinfo,70:B3:D5:B0:90:00/36FirstLig,70:B3:D5:B0:B0:00/36Internet,70:B3:D5:B0:C0:00/36Vigilate,70:B3:D5:B0:D0:00/36Alfi,70:B3:D5:B0:E0:00/36Servotro,70:B3:D5:B0:F0:00/36merkurFu,70:B3:D5:B1:00:00/36ZumbachE,70:B3:D5:B1:10:00/36Cab,70:B3:D5:B1:20:00/36Vteq,70:B3:D5:B1:30:00/36Omwave,70:B3:D5:B1:40:00/36Pantheru,70:B3:D5:B1:50:00/36EtaBeta,70:B3:D5:B1:60:00/36XiAnShen,70:B3:D5:B1:70:00/36Intesens,70:B3:D5:B1:80:00/36Abbas,70:B3:D5:B1:90:00/36BraydenA,70:B3:D5:B1:A0:00/36Aaronia,70:B3:D5:B1:B0:00/36Technolo,70:B3:D5:B1:C0:00/36Serveron,70:B3:D5:B1:D0:00/36Safelet,70:B3:D5:B1:E0:00/36Fen,70:B3:D5:B1:F0:00/36Tecnowat,70:B3:D5:B2:00:00/36ICTBUSIN,70:B3:D5:B2:10:00/36Tattile,70:B3:D5:B2:20:00/36YUYAMAMF,70:B3:D5:B2:30:00/36Supervis,70:B3:D5:B2:40:00/36DatasatD,70:B3:D5:B2:50:00/36HifocusE,70:B3:D5:B2:60:00/36INTECInt,70:B3:D5:B2:70:00/36NavalGro,70:B3:D5:B2:80:00/36HUSTYMSt,70:B3:D5:B2:90:00/36WiViCom,70:B3:D5:B2:A0:00/36MyroCont,70:B3:D5:B2:B0:00/36Vtron,70:B3:D5:B2:C0:00/36Elman,70:B3:D5:B2:D0:00/36Plexus,70:B3:D5:B2:E0:00/36GreenAcc,70:B3:D5:B2:F0:00/36HermannA,70:B3:D5:B3:00:00/36SystoléH,70:B3:D5:B3:10:00/36Qwave,70:B3:D5:B3:30:00/36AplexTec,70:B3:D5:B3:40:00/36Medtroni,70:B3:D5:B3:50:00/36Rexxam,70:B3:D5:B3:60:00/36Cetitec,70:B3:D5:B3:70:00/36CODEC,70:B3:D5:B3:80:00/36GoTrustI,70:B3:D5:B3:90:00/36MBconnec,70:B3:D5:B3:A0:00/36Adigital,70:B3:D5:B3:B0:00/36Insitu,70:B3:D5:B3:C0:00/36DorletSa,70:B3:D5:B3:D0:00/36Inras,70:B3:D5:B3:E0:00/36Paradigm,70:B3:D5:B3:F0:00/36OrbitInt,70:B3:D5:B4:00:00/36WuhanXin,70:B3:D5:B4:10:00/36T&MMedia,70:B3:D5:B4:20:00/36SamwellI,70:B3:D5:B4:30:00/36Zeo,70:B3:D5:B4:40:00/36ENTECEle,70:B3:D5:B4:50:00/36HonHaiPr,70:B3:D5:B4:60:00/36FASElect,70:B3:D5:B4:70:00/36DSITSolu,70:B3:D5:B4:80:00/36DWQInfor,70:B3:D5:B4:90:00/36Analogic,70:B3:D5:B4:A0:00/36Medex,70:B3:D5:B4:B0:00/36NetworkC,70:B3:D5:B4:C0:00/36American,70:B3:D5:B4:D0:00/36Avidbots,70:B3:D5:B4:F0:00/36AvMapsrl,70:B3:D5:B5:00:00/36iGridT&D,70:B3:D5:B5:10:00/36Critical,70:B3:D5:B5:20:00/36AEye,70:B3:D5:B5:30:00/36Revoluti,70:B3:D5:B5:40:00/36PacketPo,70:B3:D5:B5:50:00/36Ctag-Esg,70:B3:D5:B5:60:00/36PowerEle,70:B3:D5:B5:70:00/36Shanghai,70:B3:D5:B5:80:00/36Internet,70:B3:D5:B5:90:00/36FutureTe,70:B3:D5:B5:A0:00/36GTITechn,70:B3:D5:B5:B0:00/36DynaMoun,70:B3:D5:B5:C0:00/36ProzessT,70:B3:D5:B5:D0:00/36Shandhai,70:B3:D5:B5:E0:00/36Dynics,70:B3:D5:B5:F0:00/36Crdmdeve,70:B3:D5:B6:00:00/36Zeo,70:B3:D5:B6:10:00/36WuXiankt,70:B3:D5:B6:20:00/36SakuraSe,70:B3:D5:B6:30:00/36Ideas,70:B3:D5:B6:40:00/36OsungLst,70:B3:D5:B6:50:00/36RotemInd,70:B3:D5:B6:60:00/36SilentGl,70:B3:D5:B6:70:00/36RedWaveL,70:B3:D5:B6:80:00/36S-RainCo,70:B3:D5:B6:90:00/36Daatrics,70:B3:D5:B6:A0:00/36YUYAMAMF,70:B3:D5:B6:B0:00/36Cambria,70:B3:D5:B6:C0:00/36GHM-Mess,70:B3:D5:B6:D0:00/36Movis,70:B3:D5:B6:E0:00/36Edgeware,70:B3:D5:B6:F0:00/36IntegraM,70:B3:D5:B7:00:00/36TorionPl,70:B3:D5:B7:10:00/36Optiver,70:B3:D5:B7:20:00/36UB330net,70:B3:D5:B7:30:00/36CettoInd,70:B3:D5:B7:40:00/36OnYield,70:B3:D5:B7:50:00/36Grossenb,70:B3:D5:B7:60:00/36Atl-Sd,70:B3:D5:B7:70:00/36Motec,70:B3:D5:B7:80:00/36HOERMANN,70:B3:D5:B7:90:00/36Dadacon,70:B3:D5:B7:A0:00/36Mahle,70:B3:D5:B7:B0:00/36DoosanDi,70:B3:D5:B7:C0:00/36Electron,70:B3:D5:B7:D0:00/36LOGIXITS,70:B3:D5:B7:E0:00/36ElbitAme,70:B3:D5:B7:F0:00/36JSKSyste,70:B3:D5:B8:00:00/36Bighouse,70:B3:D5:B8:10:00/36InstroPr,70:B3:D5:B8:20:00/36LookoutP,70:B3:D5:B8:30:00/36MatrixTe,70:B3:D5:B8:40:00/36Research,70:B3:D5:B8:50:00/36Fenotech,70:B3:D5:B8:60:00/36Hilo,70:B3:D5:B8:70:00/36CAITRON,70:B3:D5:B8:80:00/36ARP,70:B3:D5:B8:90:00/36Ida,70:B3:D5:B8:A0:00/36NexusTec,70:B3:D5:B8:B0:00/36Profound,70:B3:D5:B8:C0:00/36ePOINTEm,70:B3:D5:B8:D0:00/36JungwooE,70:B3:D5:B8:E0:00/36UrFog,70:B3:D5:B8:F0:00/36Assembly,70:B3:D5:B9:00:00/36Amico,70:B3:D5:B9:10:00/36Dynetics,70:B3:D5:B9:20:00/36NACommun,70:B3:D5:B9:30:00/36Internet,70:B3:D5:B9:40:00/36Cygnetic,70:B3:D5:B9:60:00/36Oculii,70:B3:D5:B9:70:00/36CanamTec,70:B3:D5:B9:80:00/36GSFPte,70:B3:D5:B9:90:00/36DomoSafe,70:B3:D5:B9:A0:00/36PotterEl,70:B3:D5:B9:B0:00/36Elektron,70:B3:D5:B9:C0:00/36EDCOTech,70:B3:D5:B9:D0:00/36Conclusi,70:B3:D5:B9:E0:00/36Polsyste,70:B3:D5:B9:F0:00/36YuksekKa,70:B3:D5:BA:00:00/36SeasonEl,70:B3:D5:BA:10:00/36Cathwell,70:B3:D5:BA:20:00/36MAMAC,70:B3:D5:BA:30:00/36Tiama,70:B3:D5:BA:40:00/36EiwaGike,70:B3:D5:BA:50:00/36fpgalabs,70:B3:D5:BA:60:00/36GluonSol,70:B3:D5:BA:70:00/36DigitalY,70:B3:D5:BA:80:00/36Controll,70:B3:D5:BA:90:00/36Alma,70:B3:D5:BA:A0:00/36DeviceSo,70:B3:D5:BA:B0:00/36AxotecTe,70:B3:D5:BA:C0:00/36AdInte,70:B3:D5:BA:D0:00/36TechnikD,70:B3:D5:BA:E0:00/36Warecube,70:B3:D5:BA:F0:00/36SYSTECel,70:B3:D5:BB:00:00/36WicellTe,70:B3:D5:BB:10:00/36Lumiplan,70:B3:D5:BB:20:00/36MettlerT,70:B3:D5:BB:30:00/36APGCashD,70:B3:D5:BB:40:00/36Integrit,70:B3:D5:BB:50:00/36Grossenb,70:B3:D5:BB:60:00/36FrankeAq,70:B3:D5:BB:70:00/36Innoflig,70:B3:D5:BB:80:00/36AlKamelS,70:B3:D5:BB:90:00/36KOSMEK,70:B3:D5:BB:A0:00/36Samriddi,70:B3:D5:BB:B0:00/36YUYAMAMF,70:B3:D5:BB:C0:00/36Boundary,70:B3:D5:BB:D0:00/36Providiu,70:B3:D5:BB:E0:00/36SunriseE,70:B3:D5:BB:F0:00/36Ensys,70:B3:D5:BC:00:00/36Senso2Me,70:B3:D5:BC:10:00/36Abionic,70:B3:D5:BC:20:00/36DWEWOONG,70:B3:D5:BC:30:00/36eWireles,70:B3:D5:BC:40:00/36DigitalM,70:B3:D5:BC:50:00/36U&RHardw,70:B3:D5:BC:60:00/36Hattelan,70:B3:D5:BC:70:00/36Autonomi,70:B3:D5:BC:80:00/36Lomasro,70:B3:D5:BC:90:00/36Yitetech,70:B3:D5:BC:A0:00/36DeymedDi,70:B3:D5:BC:B0:00/36SmartVis,70:B3:D5:BC:C0:00/36MBconnec,70:B3:D5:BC:D0:00/36SaskenTe,70:B3:D5:BC:E0:00/36YawataEl,70:B3:D5:BC:F0:00/36APGCashD,70:B3:D5:BD:00:00/36Shs,70:B3:D5:BD:10:00/36CableLab,70:B3:D5:BD:20:00/36BurkTech,70:B3:D5:BD:30:00/36FotonaDD,70:B3:D5:BD:40:00/36YUYAMAMF,70:B3:D5:BD:50:00/36Synics,70:B3:D5:BD:60:00/36Consarc,70:B3:D5:BD:70:00/36TTGroup,70:B3:D5:BD:80:00/36MBconnec,70:B3:D5:BD:90:00/36SolwayTe,70:B3:D5:BD:A0:00/365-D,70:B3:D5:BD:B0:00/36PowerEle,70:B3:D5:BD:C0:00/36EDFLab,70:B3:D5:BD:D0:00/36Cdr,70:B3:D5:BD:E0:00/36CASTGrou,70:B3:D5:BD:F0:00/36H2O-Yug,70:B3:D5:BE:00:00/36Cognosos,70:B3:D5:BE:10:00/36FeCon,70:B3:D5:BE:20:00/36Nocix,70:B3:D5:BE:30:00/36SaratovE,70:B3:D5:BE:40:00/36Kunshane,70:B3:D5:BE:50:00/36PantecEn,70:B3:D5:BE:60:00/36CCII,70:B3:D5:BE:70:00/36SyscomIn,70:B3:D5:BE:80:00/36AndFun,70:B3:D5:BE:90:00/36Telecast,70:B3:D5:BE:A0:00/36Virtuosy,70:B3:D5:BE:B0:00/36PotterEl,70:B3:D5:BE:C0:00/36TokyoCom,70:B3:D5:BE:D0:00/36Itrinegy,70:B3:D5:BE:E0:00/36Sicon,70:B3:D5:BE:F0:00/36Sensorte,70:B3:D5:BF:00:00/36AlfaElet,70:B3:D5:BF:10:00/36Flashnet,70:B3:D5:BF:20:00/36TwinDeve,70:B3:D5:BF:30:00/36Cg-Wirel,70:B3:D5:BF:40:00/36CreevX,70:B3:D5:BF:50:00/36AcaciaRe,70:B3:D5:BF:60:00/36comtac,70:B3:D5:BF:70:00/36FischerC,70:B3:D5:BF:80:00/36RchItali,70:B3:D5:BF:90:00/36Okolab,70:B3:D5:BF:A0:00/36Nesa,70:B3:D5:BF:B0:00/36Sensor42,70:B3:D5:BF:C0:00/36VishayNo,70:B3:D5:BF:D0:00/36Lumentum,70:B3:D5:BF:E0:00/36AplexTec,70:B3:D5:BF:F0:00/36Sunsa,70:B3:D5:C0:00:00/36BESOsp,70:B3:D5:C0:10:00/36SmartGua,70:B3:D5:C0:20:00/36GarmoIns,70:B3:D5:C0:30:00/36XAViTech,70:B3:D5:C0:40:00/36ProlanZr,70:B3:D5:C0:50:00/36KSTtechn,70:B3:D5:C0:60:00/36Xotonics,70:B3:D5:C0:70:00/36Areco,70:B3:D5:C0:80:00/36Talleres,70:B3:D5:C0:90:00/36RCHVietn,70:B3:D5:C0:A0:00/36Infosock,70:B3:D5:C0:B0:00/36Fstudio,70:B3:D5:C0:C0:00/36Tech4Rac,70:B3:D5:C0:D0:00/36ClarityM,70:B3:D5:C0:E0:00/36SYSDEV,70:B3:D5:C0:F0:00/36Honeywel,70:B3:D5:C1:00:00/36Scanvaeg,70:B3:D5:C1:10:00/36AristonT,70:B3:D5:C1:20:00/36BeijingW,70:B3:D5:C1:30:00/36Guangzho,70:B3:D5:C1:40:00/36GrupoEpe,70:B3:D5:C1:50:00/36Sensobox,70:B3:D5:C1:60:00/36Southern,70:B3:D5:C1:70:00/36PotterEl,70:B3:D5:C1:80:00/36SanminaI,70:B3:D5:C1:90:00/36ZumbachE,70:B3:D5:C1:A0:00/36Xylon,70:B3:D5:C1:B0:00/36Labinven,70:B3:D5:C1:C0:00/36DEM,70:B3:D5:C1:D0:00/36KranzeTe,70:B3:D5:C1:E0:00/36KronMedi,70:B3:D5:C1:F0:00/36BehrTech,70:B3:D5:C2:00:00/36Mipot,70:B3:D5:C2:10:00/36AplexTec,70:B3:D5:C2:20:00/36Skyriver,70:B3:D5:C2:30:00/36Sumitomo,70:B3:D5:C2:40:00/36ElbitAme,70:B3:D5:C2:50:00/36speedsig,70:B3:D5:C2:60:00/36TriplePl,70:B3:D5:C2:70:00/36GDMissio,70:B3:D5:C2:80:00/36MitechIn,70:B3:D5:C2:90:00/36Softland,70:B3:D5:C2:A0:00/36ArrayTel,70:B3:D5:C2:B0:00/36YUYAMAMF,70:B3:D5:C2:C0:00/36Dromont,70:B3:D5:C2:D0:00/36Ensotech,70:B3:D5:C2:E0:00/36Triax,70:B3:D5:C2:F0:00/36ATBiS,70:B3:D5:C3:00:00/36PolskieS,70:B3:D5:C3:10:00/36GermanPo,70:B3:D5:C3:20:00/36Infrasaf,70:B3:D5:C3:30:00/36DandongD,70:B3:D5:C3:40:00/36Technica,70:B3:D5:C3:50:00/36Vibratio,70:B3:D5:C3:60:00/36Knowledg,70:B3:D5:C3:70:00/36Keycom,70:B3:D5:C3:80:00/36Cresprit,70:B3:D5:C3:90:00/36MeshWork,70:B3:D5:C3:A0:00/36HanChang,70:B3:D5:C3:B0:00/36Vironova,70:B3:D5:C3:C0:00/36PeekTraf,70:B3:D5:C3:D0:00/36CISTECHS,70:B3:D5:C3:E0:00/36Dosadore,70:B3:D5:C3:F0:00/36CodeBlue,70:B3:D5:C4:00:00/36HongSeok,70:B3:D5:C4:10:00/36MerlinCS,70:B3:D5:C4:20:00/36Crde,70:B3:D5:C4:30:00/36FutureSk,70:B3:D5:C4:40:00/36FranzKes,70:B3:D5:C4:50:00/36StiebelE,70:B3:D5:C4:60:00/36eumigind,70:B3:D5:C4:70:00/36Abb,70:B3:D5:C4:80:00/36WeltekTe,70:B3:D5:C4:90:00/36BTGInstr,70:B3:D5:C4:A0:00/36Tiama,70:B3:D5:C4:B0:00/36Anker-Ea,70:B3:D5:C4:C0:00/36VTCDigic,70:B3:D5:C4:D0:00/36RADAElec,70:B3:D5:C4:E0:00/36ARKRAYKy,70:B3:D5:C4:F0:00/36AEVandeV,70:B3:D5:C5:00:00/36Combilen,70:B3:D5:C5:10:00/36InnotasE,70:B3:D5:C5:20:00/36sensorwa,70:B3:D5:C5:30:00/36SLabssp,70:B3:D5:C5:40:00/36Flexsolu,70:B3:D5:C5:50:00/36Intellig,70:B3:D5:C5:60:00/36Teletask,70:B3:D5:C5:70:00/36eBZ,70:B3:D5:C5:80:00/36RMILaser,70:B3:D5:C5:90:00/36RCubedEn,70:B3:D5:C5:A0:00/36Commsign,70:B3:D5:C5:B0:00/36ACDElekt,70:B3:D5:C5:C0:00/36LayerLog,70:B3:D5:C5:D0:00/36FoshanSh,70:B3:D5:C5:E0:00/36FrogCell,70:B3:D5:C5:F0:00/36Clean-La,70:B3:D5:C6:00:00/36GogoBA,70:B3:D5:C6:10:00/36JcHunter,70:B3:D5:C6:20:00/36Wiznova,70:B3:D5:C6:30:00/36XentechS,70:B3:D5:C6:40:00/36SYSTECel,70:B3:D5:C6:50:00/36PeekTraf,70:B3:D5:C6:60:00/36BlueAcce,70:B3:D5:C6:70:00/36ColliniD,70:B3:D5:C6:80:00/36MiniSolu,70:B3:D5:C6:90:00/36Az-Techn,70:B3:D5:C6:A0:00/36Private,70:B3:D5:C6:B0:00/36Herholdt,70:B3:D5:C6:C0:00/36McQ,70:B3:D5:C6:D0:00/36CyvizAS,70:B3:D5:C6:E0:00/36OrionTec,70:B3:D5:C6:F0:00/36nyantec,70:B3:D5:C7:00:00/36Magnetek,70:B3:D5:C7:10:00/36Engineer,70:B3:D5:C7:20:00/36ScharcoE,70:B3:D5:C7:30:00/36CDN,70:B3:D5:C7:40:00/36Qtechnol,70:B3:D5:C7:50:00/36BITGroup,70:B3:D5:C7:60:00/36ElaInnov,70:B3:D5:C7:70:00/36YönnetAk,70:B3:D5:C7:80:00/36NETAElek,70:B3:D5:C7:90:00/36MBconnec,70:B3:D5:C7:A0:00/36ENTECEle,70:B3:D5:C7:B0:00/36EMClarit,70:B3:D5:C7:C0:00/36BeijingA,70:B3:D5:C7:D0:00/36Metatron,70:B3:D5:C7:E0:00/36BirdDogA,70:B3:D5:C7:F0:00/36Tattile,70:B3:D5:C8:00:00/36LinkCare,70:B3:D5:C8:10:00/36DspDesig,70:B3:D5:C8:20:00/36Sicon,70:B3:D5:C8:30:00/36CertusNe,70:B3:D5:C8:40:00/36LincTech,70:B3:D5:C8:50:00/36SolidSta,70:B3:D5:C8:60:00/36Woodam,70:B3:D5:C8:70:00/36Siemens,70:B3:D5:C8:80:00/36SINED,70:B3:D5:C8:90:00/36Ard,70:B3:D5:C8:A0:00/36WTE,70:B3:D5:C8:B0:00/36AsiaPaci,70:B3:D5:C8:C0:00/36Rollogo,70:B3:D5:C8:D0:00/36KSTtechn,70:B3:D5:C8:E0:00/36CoralTel,70:B3:D5:C8:F0:00/36TridentI,70:B3:D5:C9:00:00/36Diretta,70:B3:D5:C9:10:00/36Grossenb,70:B3:D5:C9:20:00/36UnitroFl,70:B3:D5:C9:30:00/36GMI,70:B3:D5:C9:40:00/36VarsTech,70:B3:D5:C9:50:00/36ChengduM,70:B3:D5:C9:60:00/36UniDimen,70:B3:D5:C9:70:00/36Csinfote,70:B3:D5:C9:80:00/36TrustAut,70:B3:D5:C9:90:00/36RemoteDi,70:B3:D5:C9:A0:00/36ToddDigi,70:B3:D5:C9:B0:00/36TietoSwe,70:B3:D5:C9:C0:00/36Connecte,70:B3:D5:C9:D0:00/36APGCashD,70:B3:D5:C9:E0:00/36FukudaSa,70:B3:D5:C9:F0:00/36Triax,70:B3:D5:CA:00:00/36XirgoTec,70:B3:D5:CA:10:00/36WaldoSys,70:B3:D5:CA:20:00/36DeHaardt,70:B3:D5:CA:30:00/36Saankhya,70:B3:D5:CA:40:00/36Netemera,70:B3:D5:CA:50:00/36PTSTechn,70:B3:D5:CA:60:00/36Axing,70:B3:D5:CA:70:00/36i-ViewCo,70:B3:D5:CA:80:00/36GrupoEpe,70:B3:D5:CA:90:00/36Nxcontro,70:B3:D5:CA:A0:00/36BelPower,70:B3:D5:CA:B0:00/36NOTICE,70:B3:D5:CA:C0:00/36Crde,70:B3:D5:CA:D0:00/36YUYAMAMF,70:B3:D5:CA:E0:00/36Thema,70:B3:D5:CA:F0:00/36Dave,70:B3:D5:CB:00:00/36Ossiaco,70:B3:D5:CB:10:00/36Radar,70:B3:D5:CB:20:00/36Seclab,70:B3:D5:CB:30:00/36KSTtechn,70:B3:D5:CB:40:00/36Planewav,70:B3:D5:CB:50:00/36AtlasLig,70:B3:D5:CB:60:00/36Kuebrich,70:B3:D5:CB:70:00/36HKC,70:B3:D5:CB:80:00/36VertiTec,70:B3:D5:CB:90:00/36«Satis-T,70:B3:D5:CB:A0:00/36YUYAMAMF,70:B3:D5:CB:B0:00/36Postmark,70:B3:D5:CB:C0:00/36ProconEl,70:B3:D5:CB:D0:00/36PreoIndu,70:B3:D5:CB:E0:00/36EnsuraSo,70:B3:D5:CB:F0:00/36CubicITS,70:B3:D5:CC:00:00/36Avionica,70:B3:D5:CC:10:00/36BEEcube,70:B3:D5:CC:20:00/36LSCLight,70:B3:D5:CC:30:00/36FidaliaN,70:B3:D5:CC:40:00/36Benchmar,70:B3:D5:CC:50:00/36Intecom,70:B3:D5:CC:60:00/36MBconnec,70:B3:D5:CC:70:00/36SOtM,70:B3:D5:CC:80:00/36ProfenCo,70:B3:D5:CC:90:00/36Rapiscan,70:B3:D5:CC:A0:00/36SiemensA,70:B3:D5:CC:B0:00/36RealD,70:B3:D5:CC:C0:00/36AEC,70:B3:D5:CC:D0:00/36SuzhouPo,70:B3:D5:CC:E0:00/36Proconex,70:B3:D5:CC:F0:00/36Netberg,70:B3:D5:CD:00:00/36Ellenex,70:B3:D5:CD:10:00/36CannexTe,70:B3:D5:CD:20:00/36HBHMicro,70:B3:D5:CD:30:00/36Controlr,70:B3:D5:CD:40:00/36Southern,70:B3:D5:CD:50:00/36Apantac,70:B3:D5:CD:60:00/36VideoRay,70:B3:D5:CD:70:00/36Automati,70:B3:D5:CD:80:00/36NexusEle,70:B3:D5:CD:90:00/36PeterHub,70:B3:D5:CD:A0:00/36Vitec,70:B3:D5:CD:B0:00/36WuhanXin,70:B3:D5:CD:C0:00/36Dat-Cond,70:B3:D5:CD:D0:00/36TeneoIoT,70:B3:D5:CD:E0:00/36Multipur,70:B3:D5:CD:F0:00/363DPrinti,70:B3:D5:CE:00:00/36MSContro,70:B3:D5:CE:10:00/36EAElektr,70:B3:D5:CE:20:00/36Centero,70:B3:D5:CE:30:00/36Dalcnet,70:B3:D5:CE:40:00/36WavesSys,70:B3:D5:CE:50:00/36GridBrid,70:B3:D5:CE:60:00/36Dynim,70:B3:D5:CE:70:00/36JuneAuto,70:B3:D5:CE:80:00/36Grossenb,70:B3:D5:CE:90:00/36Kinemetr,70:B3:D5:CE:A0:00/36Computer,70:B3:D5:CE:B0:00/36XirgoTec,70:B3:D5:CE:C0:00/36Deltroni,70:B3:D5:CE:D0:00/36Advanced,70:B3:D5:CE:E0:00/36ACRIOSsr,70:B3:D5:CE:F0:00/36EllegoPo,70:B3:D5:CF:00:00/36Shenzhen,70:B3:D5:CF:10:00/36LightDec,70:B3:D5:CF:20:00/36tinnos,70:B3:D5:CF:30:00/36MeshMoti,70:B3:D5:CF:40:00/36HarbinCh,70:B3:D5:CF:50:00/36PetringE,70:B3:D5:CF:60:00/36TornadoM,70:B3:D5:CF:70:00/36GentecEl,70:B3:D5:CF:80:00/36IdneoTec,70:B3:D5:CF:90:00/36BreasMed,70:B3:D5:CF:A0:00/36Scheiber,70:B3:D5:CF:B0:00/36ScreenIn,70:B3:D5:CF:C0:00/36Veilux,70:B3:D5:CF:D0:00/36iLOQ,70:B3:D5:CF:E0:00/36Secturio,70:B3:D5:CF:F0:00/36DTECHLab,70:B3:D5:D0:00:00/36DKITechn,70:B3:D5:D0:10:00/36Vision4c,70:B3:D5:D0:20:00/36ArctosSh,70:B3:D5:D0:30:00/36Digitell,70:B3:D5:D0:40:00/36PlentyUn,70:B3:D5:D0:50:00/36Colmek,70:B3:D5:D0:60:00/36YUYAMAMF,70:B3:D5:D0:70:00/36Waversa,70:B3:D5:D0:80:00/36VeecoIns,70:B3:D5:D0:90:00/36RishaadB,70:B3:D5:D0:A0:00/36Private,70:B3:D5:D0:B0:00/36Vendanor,70:B3:D5:D0:C0:00/36ConnorWi,70:B3:D5:D0:D0:00/36Logiwast,70:B3:D5:D0:E0:00/36BeijingA,70:B3:D5:D0:F0:00/36AltoAvia,70:B3:D5:D1:00:00/36ContecAm,70:B3:D5:D1:10:00/36EREEElec,70:B3:D5:D1:20:00/36Fideltro,70:B3:D5:D1:30:00/36IRTTechn,70:B3:D5:D1:40:00/36Ligpt,70:B3:D5:D1:50:00/363DGences,70:B3:D5:D1:60:00/36Monnit,70:B3:D5:D1:70:00/36PowerEle,70:B3:D5:D1:80:00/36MetComSo,70:B3:D5:D1:90:00/36SeniorGr,70:B3:D5:D1:A0:00/36Monnit,70:B3:D5:D1:B0:00/36GrupoEpe,70:B3:D5:D1:C0:00/36Speciali,70:B3:D5:D1:D0:00/36StuytsEn,70:B3:D5:D1:E0:00/36HoustonR,70:B3:D5:D1:F0:00/36Embsec,70:B3:D5:D2:00:00/36Rheonics,70:B3:D5:D2:10:00/36biosilve,70:B3:D5:D2:20:00/36DEKTechn,70:B3:D5:D2:30:00/36COTTElec,70:B3:D5:D2:40:00/36Microtro,70:B3:D5:D2:50:00/36ENGenesi,70:B3:D5:D2:60:00/36MI,70:B3:D5:D2:70:00/36Lightfie,70:B3:D5:D2:80:00/36ToshibaE,70:B3:D5:D2:90:00/36Sportzca,70:B3:D5:D2:A0:00/36ITsynerg,70:B3:D5:D2:B0:00/36StreamPl,70:B3:D5:D2:C0:00/36microWer,70:B3:D5:D2:D0:00/36EvoluteP,70:B3:D5:D2:E0:00/36Coheros,70:B3:D5:D2:F0:00/36LIFE,70:B3:D5:D3:00:00/36LeicaMic,70:B3:D5:D3:10:00/36Solace,70:B3:D5:D3:20:00/36Euklisby,70:B3:D5:D3:30:00/36Vector,70:B3:D5:D3:40:00/36G-Philos,70:B3:D5:D3:50:00/36King-OnT,70:B3:D5:D3:60:00/36Insitu,70:B3:D5:D3:70:00/36Sicon,70:B3:D5:D3:80:00/36VistaRes,70:B3:D5:D3:90:00/36ASHIDAEl,70:B3:D5:D3:A0:00/36Promomed,70:B3:D5:D3:B0:00/36NimbeLin,70:B3:D5:D3:C0:00/36Hrt,70:B3:D5:D3:D0:00/36Netzikon,70:B3:D5:D3:E0:00/36enders,70:B3:D5:D3:F0:00/36Globalco,70:B3:D5:D4:00:00/36Crde,70:B3:D5:D4:10:00/36KSE,70:B3:D5:D4:20:00/36DspDesig,70:B3:D5:D4:30:00/36EZSYS,70:B3:D5:D4:40:00/36ic-autom,70:B3:D5:D4:50:00/36VemcoSpz,70:B3:D5:D4:60:00/36Contineo,70:B3:D5:D4:70:00/36YotaScop,70:B3:D5:D4:80:00/36HEADROOM,70:B3:D5:D4:90:00/36Sicon,70:B3:D5:D4:A0:00/36ELIKOTeh,70:B3:D5:D4:B0:00/36HermannL,70:B3:D5:D4:C0:00/36ElystecT,70:B3:D5:D4:D0:00/36Morey,70:B3:D5:D4:E0:00/36FLSmidth,70:B3:D5:D4:F0:00/36C-COMSat,70:B3:D5:D5:00:00/36CubicITS,70:B3:D5:D5:10:00/36AzcomTec,70:B3:D5:D5:20:00/36Sensoron,70:B3:D5:D5:30:00/36BeiLieTe,70:B3:D5:D5:40:00/36JLWorld,70:B3:D5:D5:50:00/36WMDesign,70:B3:D5:D5:60:00/36Kronotec,70:B3:D5:D5:70:00/36TRIUMPHB,70:B3:D5:D5:80:00/36IdyllicE,70:B3:D5:D5:90:00/36WyreStor,70:B3:D5:D5:A0:00/36WyreStor,70:B3:D5:D5:B0:00/36WyreStor,70:B3:D5:D5:C0:00/36Critical,70:B3:D5:D5:D0:00/36Seasons4,70:B3:D5:D5:E0:00/36Barcelon,70:B3:D5:D5:F0:00/36CoreBala,70:B3:D5:D6:00:00/36Flintab,70:B3:D5:D6:10:00/36Vitec,70:B3:D5:D6:20:00/36AndasisE,70:B3:D5:D6:30:00/36Crde,70:B3:D5:D6:40:00/36MettlerT,70:B3:D5:D6:50:00/36Crde,70:B3:D5:D6:60:00/36Ascenden,70:B3:D5:D6:70:00/36ALPHA,70:B3:D5:D6:80:00/36TobiTrib,70:B3:D5:D6:90:00/36ThermoFi,70:B3:D5:D6:A0:00/36KnowRoam,70:B3:D5:D6:B0:00/36Uwinloc,70:B3:D5:D6:C0:00/36GP,70:B3:D5:D6:D0:00/36ACDElekr,70:B3:D5:D6:E0:00/36ard,70:B3:D5:D6:F0:00/36X-SPEX,70:B3:D5:D7:00:00/36Rational,70:B3:D5:D7:10:00/36RZBRudol,70:B3:D5:D7:20:00/36OnYield,70:B3:D5:D7:30:00/36ERMINE,70:B3:D5:D7:40:00/36SandiaNa,70:B3:D5:D7:50:00/36HyundaiM,70:B3:D5:D7:60:00/36attocube,70:B3:D5:D7:70:00/36Portrait,70:B3:D5:D7:80:00/36NxviMicr,70:B3:D5:D7:90:00/36GOMAELET,70:B3:D5:D7:A0:00/36Speedifi,70:B3:D5:D7:B0:00/36PeterHub,70:B3:D5:D7:C0:00/36DTSIllum,70:B3:D5:D7:D0:00/36BESOsp,70:B3:D5:D7:E0:00/36Triax,70:B3:D5:D7:F0:00/36ConectaI,70:B3:D5:D8:00:00/36AMMT,70:B3:D5:D8:10:00/36PDDGroup,70:B3:D5:D8:20:00/36SunElect,70:B3:D5:D8:30:00/36Akasakat,70:B3:D5:D8:40:00/36Sentry36,70:B3:D5:D8:50:00/36BTGInstr,70:B3:D5:D8:60:00/36WPGSYSPt,70:B3:D5:D8:70:00/36Zigen,70:B3:D5:D8:80:00/36Nidecasi,70:B3:D5:D8:90:00/36Resoluti,70:B3:D5:D8:A0:00/36JiangsuH,70:B3:D5:D8:B0:00/36LenoxiAu,70:B3:D5:D8:C0:00/36Damerell,70:B3:D5:D8:D0:00/36PullnetT,70:B3:D5:D8:E0:00/36Axatel,70:B3:D5:D8:F0:00/36MoluTech,70:B3:D5:D9:00:00/36AplexTec,70:B3:D5:D9:10:00/36FoodALYT,70:B3:D5:D9:20:00/36ZamirRec,70:B3:D5:D9:30:00/36PAMIR,70:B3:D5:D9:40:00/36Dewetron,70:B3:D5:D9:50:00/36SANOSERV,70:B3:D5:D9:60:00/36ThermoFi,70:B3:D5:D9:70:00/36BRSSiste,70:B3:D5:D9:80:00/36ACDElekr,70:B3:D5:D9:90:00/36Nilar,70:B3:D5:D9:A0:00/36WuhanXin,70:B3:D5:D9:B0:00/36RussianT,70:B3:D5:D9:C0:00/36Subiniti,70:B3:D5:D9:D0:00/36Electroi,70:B3:D5:D9:E0:00/36GrupoEpe,70:B3:D5:D9:F0:00/36DigitalS,70:B3:D5:DA:00:00/36JiangsuE,70:B3:D5:DA:10:00/36Qprel,70:B3:D5:DA:20:00/36ACDElekr,70:B3:D5:DA:30:00/36Voleatec,70:B3:D5:DA:40:00/36Crde,70:B3:D5:DA:50:00/36Roboteq,70:B3:D5:DA:60:00/36RedfishG,70:B3:D5:DA:70:00/36NetworkI,70:B3:D5:DA:80:00/36TagarnoA,70:B3:D5:DA:90:00/36RCHVietn,70:B3:D5:DA:A0:00/36AmToteAu,70:B3:D5:DA:B0:00/36SETPower,70:B3:D5:DA:C0:00/36DalianLa,70:B3:D5:DA:D0:00/36GDMissio,70:B3:D5:DA:E0:00/36Lge,70:B3:D5:DA:F0:00/36Innovati,70:B3:D5:DB:00:00/36ArnouseD,70:B3:D5:DB:10:00/36Biovigil,70:B3:D5:DB:20:00/36MicroEle,70:B3:D5:DB:30:00/36Klaxoon,70:B3:D5:DB:40:00/36YUYAMAMF,70:B3:D5:DB:50:00/36XiamenPo,70:B3:D5:DB:60:00/36csintech,70:B3:D5:DB:70:00/36PengoTec,70:B3:D5:DB:80:00/36Sistem,70:B3:D5:DB:90:00/36PULOONTe,70:B3:D5:DB:A0:00/36Kodenshi,70:B3:D5:DB:B0:00/36FuhrFilt,70:B3:D5:DB:C0:00/36GamberJo,70:B3:D5:DB:D0:00/36Translit,70:B3:D5:DB:E0:00/36Hiber,70:B3:D5:DB:F0:00/36InfodevE,70:B3:D5:DC:00:00/36Ateme,70:B3:D5:DC:10:00/36Metralig,70:B3:D5:DC:20:00/36SwineTec,70:B3:D5:DC:30:00/36FathMech,70:B3:D5:DC:40:00/36PeterHub,70:B3:D5:DC:50:00/36ExcelMed,70:B3:D5:DC:60:00/36Idem,70:B3:D5:DC:70:00/36NUBURU,70:B3:D5:DC:80:00/36EnertexB,70:B3:D5:DC:90:00/36Sensoter,70:B3:D5:DC:A0:00/36DSan,70:B3:D5:DC:B0:00/36Mijienet,70:B3:D5:DC:C0:00/36Eutron,70:B3:D5:DC:D0:00/36CTechBil,70:B3:D5:DC:E0:00/36Stahl,70:B3:D5:DC:F0:00/36KLSNethe,70:B3:D5:DD:00:00/36DeepSecu,70:B3:D5:DD:10:00/36em-tec,70:B3:D5:DD:20:00/36Insitu,70:B3:D5:DD:30:00/36Vitec,70:B3:D5:DD:40:00/36ResIOTUB,70:B3:D5:DD:50:00/36Cooltera,70:B3:D5:DD:60:00/36Umweltan,70:B3:D5:DD:70:00/36DETECTAu,70:B3:D5:DD:80:00/36EMSCAN,70:B3:D5:DD:90:00/36MaNimaTe,70:B3:D5:DD:A0:00/36HubbellP,70:B3:D5:DD:B0:00/36Intra,70:B3:D5:DD:C0:00/36SyscomIn,70:B3:D5:DD:D0:00/36BioRadLa,70:B3:D5:DD:E0:00/36AbbottDi,70:B3:D5:DD:F0:00/36AeroVisi,70:B3:D5:DE:00:00/36eCozy,70:B3:D5:DE:10:00/36Duplomat,70:B3:D5:DE:20:00/36ACDElekr,70:B3:D5:DE:30:00/36ETLElekt,70:B3:D5:DE:40:00/36MaviliEl,70:B3:D5:DE:50:00/36Asml,70:B3:D5:DE:60:00/36MBconnec,70:B3:D5:DE:70:00/36Innomind,70:B3:D5:DE:80:00/36Nation-E,70:B3:D5:DE:90:00/36EkspertS,70:B3:D5:DE:A0:00/36Advanced,70:B3:D5:DE:B0:00/36DorletSa,70:B3:D5:DE:C0:00/36Condev-A,70:B3:D5:DE:D0:00/36Simpulse,70:B3:D5:DE:E0:00/36Crde,70:B3:D5:DE:F0:00/36ISGNordi,70:B3:D5:DF:00:00/36astozico,70:B3:D5:DF:10:00/36CoXlab,70:B3:D5:DF:20:00/36Aml,70:B3:D5:DF:30:00/36SPCBiocl,70:B3:D5:DF:40:00/36Heim-Bür,70:B3:D5:DF:50:00/36BeijingH,70:B3:D5:DF:60:00/36Tiab,70:B3:D5:DF:70:00/36ScopeSen,70:B3:D5:DF:80:00/36RMAMess-,70:B3:D5:DF:90:00/36KoreaPla,70:B3:D5:DF:A0:00/36Newtouch,70:B3:D5:DF:B0:00/36Yamamoto,70:B3:D5:DF:C0:00/36Electron,70:B3:D5:DF:D0:00/36Contiweb,70:B3:D5:DF:E0:00/36microtec,70:B3:D5:DF:F0:00/36Spanawav,70:B3:D5:E0:00:00/36JeawayCC,70:B3:D5:E0:10:00/36EarTex,70:B3:D5:E0:20:00/36YehlJord,70:B3:D5:E0:30:00/36Mbj,70:B3:D5:E0:40:00/36Combilen,70:B3:D5:E0:50:00/36Lobaro,70:B3:D5:E0:60:00/36SystemWe,70:B3:D5:E0:70:00/36BaaderPl,70:B3:D5:E0:80:00/36Olssen,70:B3:D5:E0:90:00/36L-3commu,70:B3:D5:E0:A0:00/36Acouva,70:B3:D5:E0:B0:00/36ENTECEle,70:B3:D5:E0:C0:00/36Communic,70:B3:D5:E0:D0:00/36SigmaCon,70:B3:D5:E0:E0:00/36VulcanFo,70:B3:D5:E0:F0:00/36Vtron,70:B3:D5:E1:00:00/36Leidos,70:B3:D5:E1:10:00/36EngageTe,70:B3:D5:E1:20:00/36SNK,70:B3:D5:E1:30:00/36SuzhouZh,70:B3:D5:E1:40:00/36Automata,70:B3:D5:E1:50:00/36Benetel,70:B3:D5:E1:60:00/36ChinaEnt,70:B3:D5:E1:70:00/36Photonic,70:B3:D5:E1:80:00/36Plasmapp,70:B3:D5:E1:90:00/36BABTECHN,70:B3:D5:E1:A0:00/36BizerbaL,70:B3:D5:E1:B0:00/36Neuron,70:B3:D5:E1:C0:00/36RoomMate,70:B3:D5:E1:D0:00/36GalaxyNe,70:B3:D5:E1:E0:00/36UmanoMed,70:B3:D5:E1:F0:00/36Theta432,70:B3:D5:E2:00:00/36Signatur,70:B3:D5:E2:10:00/36Llvision,70:B3:D5:E2:20:00/36Federate,70:B3:D5:E2:30:00/36SmithMet,70:B3:D5:E2:40:00/36GogoBusi,70:B3:D5:E2:50:00/36GJDManuf,70:B3:D5:E2:60:00/36Feitian,70:B3:D5:E2:70:00/36Woodside,70:B3:D5:E2:80:00/36iotec,70:B3:D5:E2:90:00/36InventVi,70:B3:D5:E2:A0:00/36CONTESsp,70:B3:D5:E2:B0:00/36GuanShow,70:B3:D5:E2:C0:00/36FourthFr,70:B3:D5:E2:D0:00/36Private,70:B3:D5:E2:E0:00/36Merzsro,70:B3:D5:E2:F0:00/36Flextron,70:B3:D5:E3:00:00/36Quiss,70:B3:D5:E3:10:00/36NEUROPHE,70:B3:D5:E3:20:00/36HerutuEl,70:B3:D5:E3:30:00/36DEUTA-WE,70:B3:D5:E3:40:00/36GamberJo,70:B3:D5:E3:50:00/36Nanospee,70:B3:D5:E3:60:00/36Guidance,70:B3:D5:E3:70:00/36Eurotemp,70:B3:D5:E3:80:00/36Cursor,70:B3:D5:E3:90:00/36Thinnect,70:B3:D5:E3:A0:00/36Cyanview,70:B3:D5:E3:B0:00/36ComNavTe,70:B3:D5:E3:C0:00/36Densitro,70:B3:D5:E3:D0:00/36LeoBodna,70:B3:D5:E3:E0:00/36SolWeldi,70:B3:D5:E3:F0:00/36Bestcode,70:B3:D5:E4:00:00/36SiemensM,70:B3:D5:E4:10:00/364neXts,70:B3:D5:E4:20:00/36NeusoftR,70:B3:D5:E4:30:00/36SLAudio,70:B3:D5:E4:40:00/36Brainbox,70:B3:D5:E4:50:00/36Momentum,70:B3:D5:E4:60:00/367thSense,70:B3:D5:E4:70:00/36DEUTA-WE,70:B3:D5:E4:80:00/36TDI,70:B3:D5:E4:90:00/36Kendrion,70:B3:D5:E4:A0:00/36ICPNewTe,70:B3:D5:E4:B0:00/36Delta,70:B3:D5:E4:C0:00/36IAI-Isra,70:B3:D5:E4:D0:00/36VulcanWi,70:B3:D5:E4:E0:00/36Midfin,70:B3:D5:E4:F0:00/36RWSAutom,70:B3:D5:E5:00:00/36Advanced,70:B3:D5:E5:10:00/36NooliTIC,70:B3:D5:E5:20:00/36Guangzho,70:B3:D5:E5:30:00/36Mi,70:B3:D5:E5:40:00/36BeijingP,70:B3:D5:E5:50:00/36BELT,70:B3:D5:E5:60:00/36Hipodrom,70:B3:D5:E5:70:00/36Iradimed,70:B3:D5:E5:80:00/36ThurlbyT,70:B3:D5:E5:90:00/36Fracarro,70:B3:D5:E5:A0:00/36Cardinal,70:B3:D5:E5:B0:00/36ArgosyLa,70:B3:D5:E5:C0:00/36WaltonHi,70:B3:D5:E5:D0:00/36BoffinsT,70:B3:D5:E5:E0:00/36Critical,70:B3:D5:E5:F0:00/36CesiumAs,70:B3:D5:E6:00:00/36Davitor,70:B3:D5:E6:10:00/36Adeli,70:B3:D5:E6:20:00/36Eon,70:B3:D5:E6:30:00/36PotomacE,70:B3:D5:E6:40:00/36HongJian,70:B3:D5:E6:50:00/36BirtechT,70:B3:D5:E6:60:00/36Eneonsp,70:B3:D5:E6:70:00/36AppliedP,70:B3:D5:E6:80:00/36TransitS,70:B3:D5:E6:90:00/36Fire4UK,70:B3:D5:E6:A0:00/36MACSolut,70:B3:D5:E6:B0:00/36Shenzhen,70:B3:D5:E6:C0:00/36FusarTec,70:B3:D5:E6:D0:00/36DomusSC,70:B3:D5:E6:E0:00/36LieronBV,70:B3:D5:E6:F0:00/36AmazonTe,70:B3:D5:E7:00:00/36DISKMult,70:B3:D5:E7:10:00/36SiSTechn,70:B3:D5:E7:20:00/36KDT,70:B3:D5:E7:30:00/36ZeusCont,70:B3:D5:E7:40:00/36Exfronti,70:B3:D5:E7:50:00/36Nke,70:B3:D5:E7:60:00/36DorsettT,70:B3:D5:E7:70:00/36Optix,70:B3:D5:E7:80:00/36CamwellI,70:B3:D5:E7:90:00/36Acrodea,70:B3:D5:E7:A0:00/36Art,70:B3:D5:E7:B0:00/36Shenzhen,70:B3:D5:E7:C0:00/36AplexTec,70:B3:D5:E7:D0:00/36NanjingD,70:B3:D5:E7:E0:00/36GroupeCi,70:B3:D5:E7:F0:00/36SankyoIn,70:B3:D5:E8:00:00/36Changzho,70:B3:D5:E8:10:00/36Slat,70:B3:D5:E8:20:00/36RFTrack,70:B3:D5:E8:30:00/36Talleres,70:B3:D5:E8:40:00/36ENTECEle,70:B3:D5:E8:50:00/36Explorer,70:B3:D5:E8:60:00/36YUYAMAMF,70:B3:D5:E8:70:00/36STACKFOR,70:B3:D5:E8:80:00/36BreasMed,70:B3:D5:E8:90:00/36KalugaAs,70:B3:D5:E8:A0:00/36MelecsEW,70:B3:D5:E8:B0:00/36DreamD&S,70:B3:D5:E8:C0:00/36Fracarro,70:B3:D5:E8:D0:00/36NatavSer,70:B3:D5:E8:E0:00/36MacnicaT,70:B3:D5:E8:F0:00/36Dismunte,70:B3:D5:E9:00:00/36GeteinBi,70:B3:D5:E9:10:00/36NASAustr,70:B3:D5:E9:20:00/36FujiData,70:B3:D5:E9:30:00/36ECONTech,70:B3:D5:E9:40:00/36Lumiplan,70:B3:D5:E9:50:00/36BroadSof,70:B3:D5:E9:60:00/36CellierD,70:B3:D5:E9:70:00/36Toptech,70:B3:D5:E9:80:00/36KalugaAs,70:B3:D5:E9:90:00/36Advitron,70:B3:D5:E9:A0:00/36MetaComp,70:B3:D5:E9:B0:00/36NUMATAR&,70:B3:D5:E9:C0:00/36ATGUVTec,70:B3:D5:E9:D0:00/36Intech,70:B3:D5:E9:E0:00/36MSBElekt,70:B3:D5:E9:F0:00/36Gigaband,70:B3:D5:EA:00:00/36Park24,70:B3:D5:EA:10:00/36QntraTec,70:B3:D5:EA:20:00/36Transpor,70:B3:D5:EA:30:00/36Gridless,70:B3:D5:EA:40:00/36GrupoEpe,70:B3:D5:EA:50:00/36LotesTm,70:B3:D5:EA:60:00/36Galios,70:B3:D5:EA:70:00/36SICES,70:B3:D5:EA:80:00/36Dia-Stro,70:B3:D5:EA:90:00/36ZhuhaiLo,70:B3:D5:EA:A0:00/36Druck,70:B3:D5:EA:B0:00/36APENGROU,70:B3:D5:EA:C0:00/36KentechI,70:B3:D5:EA:D0:00/36Cobo,70:B3:D5:EA:E0:00/36OrlacoPr,70:B3:D5:EA:F0:00/36Sicon,70:B3:D5:EB:00:00/36NautelLi,70:B3:D5:EB:10:00/36CPcontec,70:B3:D5:EB:20:00/36ShooterD,70:B3:D5:EB:30:00/36KWS-Elec,70:B3:D5:EB:40:00/36RoboticR,70:B3:D5:EB:50:00/36Justek,70:B3:D5:EB:60:00/36Energize,70:B3:D5:EB:70:00/36Skreens,70:B3:D5:EB:80:00/36EmporiaR,70:B3:D5:EB:90:00/36ThielAud,70:B3:D5:EB:A0:00/36LastMile,70:B3:D5:EB:B0:00/36BeijingW,70:B3:D5:EB:C0:00/36RefineTe,70:B3:D5:EB:D0:00/36midBitTe,70:B3:D5:EB:E0:00/36SierraPa,70:B3:D5:EB:F0:00/36Automati,70:B3:D5:EC:00:00/36ProtoCon,70:B3:D5:EC:10:00/36XafaxNed,70:B3:D5:EC:20:00/36Lightsid,70:B3:D5:EC:30:00/36VirtualC,70:B3:D5:EC:40:00/36hmttelem,70:B3:D5:EC:50:00/36Tattile,70:B3:D5:EC:60:00/36Esii,70:B3:D5:EC:70:00/36Neoptix,70:B3:D5:EC:80:00/36Panasoni,70:B3:D5:EC:90:00/36QlinxTec,70:B3:D5:EC:A0:00/36Transtro,70:B3:D5:EC:B0:00/36Re-Contr,70:B3:D5:EC:C0:00/36Digifocu,70:B3:D5:EC:D0:00/36SBS-Fein,70:B3:D5:EC:E0:00/36COMM-con,70:B3:D5:EC:F0:00/36Ipitek,70:B3:D5:ED:00:00/36shanghai,70:B3:D5:ED:10:00/36Przemysl,70:B3:D5:ED:20:00/36PCTEL,70:B3:D5:ED:30:00/36BeijingL,70:B3:D5:ED:40:00/36WilmoreE,70:B3:D5:ED:50:00/36hangzhou,70:B3:D5:ED:60:00/36Metrasen,70:B3:D5:ED:70:00/36Wave,70:B3:D5:ED:80:00/36Wartsila,70:B3:D5:ED:90:00/36AADONACo,70:B3:D5:ED:A0:00/36BreasMed,70:B3:D5:ED:B0:00/36NetfortS,70:B3:D5:ED:C0:00/36JDKoftin,70:B3:D5:ED:D0:00/36SolarNet,70:B3:D5:ED:E0:00/36Agrident,70:B3:D5:ED:F0:00/36GridNavi,70:B3:D5:EE:00:00/36Stecomp,70:B3:D5:EE:10:00/36alloraFa,70:B3:D5:EE:20:00/36Montrade,70:B3:D5:EE:30:00/36LitheTec,70:B3:D5:EE:40:00/36O-NetAut,70:B3:D5:EE:50:00/36BeijingH,70:B3:D5:EE:60:00/36VaunixTe,70:B3:D5:EE:70:00/36Blue-Sol,70:B3:D5:EE:80:00/36robertju,70:B3:D5:EE:90:00/36SC3Autom,70:B3:D5:EE:A0:00/36Dameca,70:B3:D5:EE:B0:00/36shenzhen,70:B3:D5:EE:C0:00/36Impolux,70:B3:D5:EE:D0:00/36COMM-con,70:B3:D5:EE:E0:00/36Sociedad,70:B3:D5:EE:F0:00/36Tattile,70:B3:D5:EF:00:00/36Pnetwork,70:B3:D5:EF:10:00/36Nanotok,70:B3:D5:EF:20:00/36Kongsber,70:B3:D5:EF:30:00/36octoScop,70:B3:D5:EF:40:00/36OrangeTr,70:B3:D5:EF:50:00/36DEUTA-WE,70:B3:D5:EF:60:00/36Chargeli,70:B3:D5:EF:70:00/36Dave,70:B3:D5:EF:80:00/36DKSDiens,70:B3:D5:EF:90:00/36Critical,70:B3:D5:EF:A0:00/36NextEraE,70:B3:D5:EF:B0:00/36PXM,70:B3:D5:EF:C0:00/36Absolent,70:B3:D5:EF:D0:00/36Cambridg,70:B3:D5:EF:E0:00/36MeidenSy,70:B3:D5:EF:F0:00/36CarloGav,70:B3:D5:F0:00:00/36AplexTec,70:B3:D5:F0:10:00/36Software,70:B3:D5:F0:20:00/36ABECOInd,70:B3:D5:F0:30:00/36GMI,70:B3:D5:F0:40:00/36ScameSis,70:B3:D5:F0:50:00/36Motomuto,70:B3:D5:F0:60:00/36Warecube,70:B3:D5:F0:70:00/36DuvalMes,70:B3:D5:F0:80:00/36SzaboSof,70:B3:D5:F0:90:00/36Mictrotr,70:B3:D5:F0:A0:00/36Neuronal,70:B3:D5:F0:B0:00/36RFIndust,70:B3:D5:F0:C0:00/36ModulaTe,70:B3:D5:F0:D0:00/36MeQ,70:B3:D5:F0:E0:00/36TextSpea,70:B3:D5:F0:F0:00/36KyotoDen,70:B3:D5:F1:00:00/36RieglLas,70:B3:D5:F1:10:00/36BroadSof,70:B3:D5:F1:20:00/36IncoilIn,70:B3:D5:F1:30:00/36MEDIAMSp,70:B3:D5:F1:40:00/36SanyuSwi,70:B3:D5:F1:50:00/36ArecaEmb,70:B3:D5:F1:60:00/36BRSSiste,70:B3:D5:F1:70:00/36Vitec,70:B3:D5:F1:80:00/36HDVision,70:B3:D5:F1:90:00/36VitroTec,70:B3:D5:F1:A0:00/36SatorCon,70:B3:D5:F1:B0:00/36IndiNatu,70:B3:D5:F1:C0:00/36BavariaD,70:B3:D5:F1:D0:00/36Critical,70:B3:D5:F1:E0:00/36AtxNetwo,70:B3:D5:F1:F0:00/36HKC,70:B3:D5:F2:10:00/36dds,70:B3:D5:F2:20:00/36ShengliF,70:B3:D5:F2:30:00/36LyseAS,70:B3:D5:F2:40:00/36Daavlin,70:B3:D5:F2:50:00/36“Scienti,70:B3:D5:F2:60:00/36XjElectr,70:B3:D5:F2:70:00/36NIRIT-Xi,70:B3:D5:F2:80:00/36YiAnElec,70:B3:D5:F2:90:00/36SamabaNo,70:B3:D5:F2:A0:00/36WIBONDIn,70:B3:D5:F2:B0:00/36SENSYS,70:B3:D5:F2:C0:00/36HengenTe,70:B3:D5:F2:D0:00/36IDLockAS,70:B3:D5:F2:E0:00/36Shanghai,70:B3:D5:F2:F0:00/36Teleplat,70:B3:D5:F3:00:00/36ADETechn,70:B3:D5:F3:10:00/36The-BoxD,70:B3:D5:F3:20:00/36Elektron,70:B3:D5:F3:30:00/36BeijingV,70:B3:D5:F3:40:00/36MacGrayS,70:B3:D5:F3:50:00/36carbonTR,70:B3:D5:F3:60:00/36dinosys,70:B3:D5:F3:70:00/36Mitsubis,70:B3:D5:F3:80:00/36Scanvaeg,70:B3:D5:F3:90:00/36ZenrosAp,70:B3:D5:F3:A0:00/36Research,70:B3:D5:F3:B0:00/36Epdm,70:B3:D5:F3:C0:00/36Gigaray,70:B3:D5:F3:D0:00/36KAYAInst,70:B3:D5:F3:E0:00/36ОооРонек,70:B3:D5:F3:F0:00/36comtac,70:B3:D5:F4:00:00/36Horizon,70:B3:D5:F4:10:00/36Duevi,70:B3:D5:F4:20:00/36Matsuhis,70:B3:D5:F4:30:00/36Divelbis,70:B3:D5:F4:40:00/36MagnetiM,70:B3:D5:F4:50:00/36NorbitOD,70:B3:D5:F4:60:00/36SeasonEl,70:B3:D5:F4:70:00/36TXMissio,70:B3:D5:F4:80:00/36Heitec,70:B3:D5:F4:90:00/36ZmbiziAp,70:B3:D5:F4:A0:00/36Lacs,70:B3:D5:F4:B0:00/36ChengduL,70:B3:D5:F4:C0:00/36PolyTech,70:B3:D5:F4:D0:00/36Honeywel,70:B3:D5:F4:E0:00/36HunanLia,70:B3:D5:F4:F0:00/36PowerEle,70:B3:D5:F5:00:00/36Vectolog,70:B3:D5:F5:10:00/36IoTRoute,70:B3:D5:F5:20:00/36AlereTec,70:B3:D5:F5:30:00/36HighTech,70:B3:D5:F5:40:00/36Revoluti,70:B3:D5:F5:50:00/36KohlerMi,70:B3:D5:F5:60:00/36VirtualH,70:B3:D5:F5:70:00/36AplexTec,70:B3:D5:F5:80:00/36Cdr,70:B3:D5:F5:90:00/36KoreaSpe,70:B3:D5:F5:A0:00/36HAMEG,70:B3:D5:F5:B0:00/36AFMensah,70:B3:D5:F5:C0:00/36NableCom,70:B3:D5:F5:D0:00/36PotterEl,70:B3:D5:F5:E0:00/36SelexES,70:B3:D5:F5:F0:00/36RFRain,70:B3:D5:F6:00:00/36MPMMicro,70:B3:D5:F6:10:00/36PowerDia,70:B3:D5:F6:20:00/36FRS,70:B3:D5:F6:30:00/36ArsProdu,70:B3:D5:F6:40:00/36silicom,70:B3:D5:F6:50:00/36MarkusLa,70:B3:D5:F6:60:00/36Seznamcz,70:B3:D5:F6:70:00/36winsun,70:B3:D5:F6:80:00/36AlZajelM,70:B3:D5:F6:90:00/36CopperLa,70:B3:D5:F6:A0:00/36GuanShow,70:B3:D5:F6:B0:00/36DEUTA-WE,70:B3:D5:F6:C0:00/36VisioGre,70:B3:D5:F6:D0:00/36Qowisio,70:B3:D5:F6:E0:00/36Streambo,70:B3:D5:F6:F0:00/36Smashtag,70:B3:D5:F7:00:00/36Honeywel,70:B3:D5:F7:10:00/36Sonel,70:B3:D5:F7:20:00/36HanshinE,70:B3:D5:F7:30:00/36ASLHoldi,70:B3:D5:F7:40:00/36TessaAgr,70:B3:D5:F7:50:00/36Enlaps,70:B3:D5:F7:60:00/36ThermoFi,70:B3:D5:F7:70:00/36Satcube,70:B3:D5:F7:80:00/36Manvishe,70:B3:D5:F7:90:00/36Firehose,70:B3:D5:F7:A0:00/36Senso2Me,70:B3:D5:F7:B0:00/36KSTtechn,70:B3:D5:F7:C0:00/36Medicomp,70:B3:D5:F7:D0:00/362MTechno,70:B3:D5:F7:E0:00/36AlphaEle,70:B3:D5:F7:F0:00/36ABLSpace,70:B3:D5:F8:00:00/36GuanShow,70:B3:D5:F8:10:00/36Littlemo,70:B3:D5:F8:20:00/36PrestonI,70:B3:D5:F8:30:00/36TataComm,70:B3:D5:F8:40:00/36DEUTA-WE,70:B3:D5:F8:50:00/36Solystic,70:B3:D5:F8:60:00/36Wireless,70:B3:D5:F8:70:00/36ShinwaIn,70:B3:D5:F8:80:00/36Odawarak,70:B3:D5:F8:90:00/36SoehnleI,70:B3:D5:F8:A0:00/36FRS,70:B3:D5:F8:B0:00/36IOOOTA,70:B3:D5:F8:C0:00/36European,70:B3:D5:F8:D0:00/36Flextron,70:B3:D5:F8:E0:00/36Isabelle,70:B3:D5:F8:F0:00/36Dimastec,70:B3:D5:F9:00:00/36AtmanTec,70:B3:D5:F9:10:00/36SolidSta,70:B3:D5:F9:20:00/36TechOne,70:B3:D5:F9:30:00/36HellaGut,70:B3:D5:F9:40:00/36MBconnec,70:B3:D5:F9:50:00/36GetSAT,70:B3:D5:F9:60:00/36Ecologic,70:B3:D5:F9:70:00/36TyphonTr,70:B3:D5:F9:80:00/36MetrumSw,70:B3:D5:F9:90:00/36TexCompu,70:B3:D5:F9:A0:00/36Krabbenh,70:B3:D5:F9:B0:00/36EvoLogic,70:B3:D5:F9:C0:00/36SureFlap,70:B3:D5:F9:D0:00/36Teledyne,70:B3:D5:F9:E0:00/36Internat,70:B3:D5:F9:F0:00/36MACSolut,70:B3:D5:FA:00:00/36Tiama,70:B3:D5:FA:10:00/36BBIEngin,70:B3:D5:FA:20:00/36SarokalT,70:B3:D5:FA:30:00/36Elva-1Mi,70:B3:D5:FA:40:00/36Energybo,70:B3:D5:FA:50:00/36Shenzhen,70:B3:D5:FA:60:00/36RFLElect,70:B3:D5:FA:70:00/36Nordson,70:B3:D5:FA:80:00/36Munters,70:B3:D5:FA:90:00/36CorDes,70:B3:D5:FA:A0:00/36LogiMSof,70:B3:D5:FA:B0:00/36OpenSyst,70:B3:D5:FA:C0:00/36Integrat,70:B3:D5:FA:D0:00/36ARCTechn,70:B3:D5:FA:E0:00/36Silixa,70:B3:D5:FA:F0:00/36RadigHar,70:B3:D5:FB:00:00/36Rohde&Sc,70:B3:D5:FB:10:00/36TomeiTsu,70:B3:D5:FB:20:00/36KJ3Elekt,70:B3:D5:FB:30:00/363PS,70:B3:D5:FB:40:00/36ArrayTec,70:B3:D5:FB:50:00/36OrangeTr,70:B3:D5:FB:60:00/36Kronotec,70:B3:D5:FB:70:00/36Saice,70:B3:D5:FB:80:00/36HyannisP,70:B3:D5:FB:90:00/36Eyedea,70:B3:D5:FB:A0:00/36ApogeeAp,70:B3:D5:FB:B0:00/36VenaEngi,70:B3:D5:FB:C0:00/36TwowayCo,70:B3:D5:FB:D0:00/36MBconnec,70:B3:D5:FB:E0:00/36HanbatNa,70:B3:D5:FB:F0:00/36SenSysDe,70:B3:D5:FC:00:00/36CODESYST,70:B3:D5:FC:10:00/36InDiCor,70:B3:D5:FC:20:00/36HunterLi,70:B3:D5:FC:30:00/36myUpTech,70:B3:D5:FC:40:00/36AERIALCA,70:B3:D5:FC:50:00/36Eltwin,70:B3:D5:FC:60:00/36TecnintH,70:B3:D5:FC:70:00/36InvertRo,70:B3:D5:FC:80:00/36Moduware,70:B3:D5:FC:90:00/36Shanghai,70:B3:D5:FC:A0:00/36M2MCyber,70:B3:D5:FC:B0:00/36TielineR,70:B3:D5:FC:C0:00/36DIgSILEN,70:B3:D5:FC:D0:00/36EngageTe,70:B3:D5:FC:E0:00/36FxTechno,70:B3:D5:FC:F0:00/36Acc+Ess,70:B3:D5:FD:00:00/36AlcoholC,70:B3:D5:FD:10:00/36RedRat,70:B3:D5:FD:20:00/36DalianLe,70:B3:D5:FD:30:00/36AKIStech,70:B3:D5:FD:40:00/36Getralin,70:B3:D5:FD:50:00/36Oceancct,70:B3:D5:FD:60:00/36VisualFa,70:B3:D5:FD:70:00/36CentumAd,70:B3:D5:FD:80:00/36MBconnec,70:B3:D5:FD:90:00/36eSight,70:B3:D5:FD:A0:00/36ACDElekt,70:B3:D5:FD:B0:00/36DesignSH,70:B3:D5:FD:C0:00/36Tapdn,70:B3:D5:FD:D0:00/36LaserIma,70:B3:D5:FD:E0:00/36Aeronaut,70:B3:D5:FD:F0:00/36NaraCont,70:B3:D5:FE:00:00/36Blueprin,70:B3:D5:FE:10:00/36Shenzhen,70:B3:D5:FE:20:00/36GalileoT,70:B3:D5:FE:30:00/36CSMMACHI,70:B3:D5:FE:40:00/36Care,70:B3:D5:FE:50:00/36MalinSpa,70:B3:D5:FE:60:00/36ShizukiE,70:B3:D5:FE:70:00/36Veilux,70:B3:D5:FE:80:00/36PCME,70:B3:D5:FE:90:00/36CamsatPr,70:B3:D5:FE:A0:00/36HengDian,70:B3:D5:FE:B0:00/36Lesdistr,70:B3:D5:FE:C0:00/36Finder,70:B3:D5:FE:D0:00/36NironPro,70:B3:D5:FE:E0:00/36Kawasaki,70:B3:D5:FE:F0:00/36Hangzhou,70:B3:D5:FF:00:00/36E-MetroT,70:B3:D5:FF:10:00/36DataStra,70:B3:D5:FF:20:00/36tigaelev,70:B3:D5:FF:30:00/36AplexTec,70:B3:D5:FF:40:00/36Serveron,70:B3:D5:FF:50:00/36ProlanPr,70:B3:D5:FF:60:00/36ElektroA,70:B3:D5:FF:70:00/36Cybercom,70:B3:D5:FF:80:00/36DutileGl,70:B3:D5:FF:90:00/36InOutCom,70:B3:D5:FF:A0:00/36Barracud,70:B3:D5:FF:C0:00/36Symetric,70:B3:D5:FF:D0:00/36i2System,70:B3:D5:FF:E0:00/36Private,70:B3:D5:FF:F0:00/36Private,70:B5:99Embedded,70:B5:E8Dell,70:B6:4FGuangzho,70:B7:AAvivoMobi,70:B7:E2JiangsuM,70:B9:21Fiberhom,70:B9:50TexasIns,70:B9:BBShenzhen,70:BA:EFHangzhou,70:BB:E9XiaomiCo,70:BC:10Microsof,70:BF:3ECharlesR,70:BF:92GNAudio,70:C6:ACBoschAut,70:C6:DDNewH3CTe,70:C7:6FInnoS,70:C7:F2HuaweiTe,70:C8:33Wirepas,70:C9:4ELiteonTe,70:C9:C6Cisco,70:CA:4DShenzhen,70:CA:97RuckusWi,70:CA:9BCisco,70:CD:0DIntelCor,70:CD:60Apple,70:CD:91TeracomT,70:CE:8CSamsungE,70:CF:49IntelCor,70:D0:81BeijingN,70:D3:13HuaweiTe,70:D3:79Cisco,70:D4:F2Rim,70:D5:7EScalar,70:D5:E7Wellcore,70:D6:B6MetrumTe,70:D8:80UposSyst,70:D9:23vivoMobi,70:D9:31Cambridg,70:DA:17Austrian,70:DA:9CTecsen,70:DB:98Cisco,70:DD:A1Tellabs,70:DD:A8Guangdon,70:DD:EFHuaweiDe,70:DE:E2Apple,70:DE:F9FaiWahIn,70:DF:2FCisco,70:DF:F7ARRISGro,70:E0:27HongyuCo,70:E1:393view,70:E1:FDFlextron,70:E2:4CSAEIT,70:E2:84WistronI,70:E4:22Cisco,70:E5:6ETexasIns,70:E7:2CApple,70:E8:43BeijingC,70:EA:1ACisco,70:EA:5AApple,70:EC:E4Apple,70:EE:50Netatmo,70:EE:A3Eoptolin,70:EF:00Apple,70:F0:87Apple,70:F0:88Nintendo,70:F0:96Cisco,70:F1:1CShenzhen,70:F1:76DataModu,70:F1:96Actionte,70:F1:A1LiteonTe,70:F1:E5Xetawave,70:F2:20Actionte,70:F3:5ACisco,70:F3:95Universa,70:F7:54AMPAKTec,70:F8:2BDWnetTec,70:F8:E7IEEERegi,70:F8:E7:00:00:00/28SHENZHEN,70:F8:E7:10:00:00/28SystemLe,70:F8:E7:20:00:00/28VOXXInte,70:F8:E7:30:00:00/28DrSimonC,70:F8:E7:40:00:00/28CLIP,70:F8:E7:50:00:00/28BeijingE,70:F8:E7:60:00:00/28FleximSe,70:F8:E7:70:00:00/28NSTTechn,70:F8:E7:80:00:00/28EclipseS,70:F8:E7:90:00:00/28KontechE,70:F8:E7:A0:00:00/28TiVACIPT,70:F8:E7:B0:00:00/28Photonfo,70:F8:E7:C0:00:00/28Fixstars,70:F8:E7:D0:00:00/28System-o,70:F8:E7:E0:00:00/28Cuav,70:F9:27SamsungE,70:F9:6DHangzhou,70:FC:8COneAcces,70:FC:8FFreeboxS,70:FD:45HuaweiTe,70:FD:46SamsungE,70:FD:88NanjingJ,70:FF:5CCheerzin,70:FF:76TexasIns,74:03:BDBuffalo,74:04:2BLenovoMo,74:04:F0Mobiwire,74:05:A5Tp-LinkT,74:09:ACQuext,74:0A:BCLightwav,74:0A:E1HuaweiDe,74:0C:EEHuaweiDe,74:0E:DBOptowiz,74:12:B3Chongqin,74:12:BBFiberhom,74:14:89SRTWirel,74:15:75XiaomiCo,74:15:E2Tri-Sen,74:18:65Shanghai,74:19:F8IEEERegi,74:19:F8:00:00:00/28Marmitek,74:19:F8:10:00:00/28Trend-te,74:19:F8:20:00:00/28SymtopIn,74:19:F8:30:00:00/28Essentia,74:19:F8:40:00:00/28Cloudvue,74:19:F8:50:00:00/28StarcorB,74:19:F8:60:00:00/28Baudisch,74:19:F8:70:00:00/28Heptagon,74:19:F8:80:00:00/28QuestPay,74:19:F8:90:00:00/28Princip,74:19:F8:A0:00:00/28Tanjarin,74:19:F8:B0:00:00/28IDEXXLab,74:19:F8:C0:00:00/28BachIcon,74:19:F8:D0:00:00/28AnsjerEl,74:19:F8:E0:00:00/28Volacomm,74:19:F8:F0:00:00/28Private,74:1A:E0IEEERegi,74:1A:E0:00:00:00/28HuanoInt,74:1A:E0:10:00:00/28Socionex,74:1A:E0:20:00:00/28NuraHold,74:1A:E0:30:00:00/28PhilipsP,74:1A:E0:40:00:00/28Revl,74:1A:E0:50:00:00/28FujianTa,74:1A:E0:60:00:00/28BlocksWe,74:1A:E0:70:00:00/28BÄRBahns,74:1A:E0:80:00:00/28Broadcas,74:1A:E0:90:00:00/28Private,74:1A:E0:A0:00:00/28Saiercom,74:1A:E0:B0:00:00/28ShenZhen,74:1A:E0:C0:00:00/28bistos,74:1A:E0:D0:00:00/28Voltawar,74:1A:E0:E0:00:00/28ITSPartn,74:1B:B2Apple,74:1C:27ItelMobi,74:1E:93Fiberhom,74:1F:4AHangzhou,74:1F:79Youngkoo,74:22:BBHuaweiDe,74:23:44XiaomiCo,74:25:8AHangzhou,74:26:ACCisco,74:27:3CChangYan,74:27:EAElitegro,74:28:57Mayfield,74:29:AFHonHaiPr,74:2B:0FInfinida,74:2B:62Fujitsu,74:2D:0ANorfolkE,74:2E:DBPerinet,74:2E:FCDirectPa,74:2F:68AzureWav,74:31:70Arcadyan,74:32:56NT-wareS,74:32:C2Kyolis,74:33:A6Shenzhen,74:34:00MTG,74:34:AEthisisen,74:36:6DVodafone,74:37:2FTongfang,74:37:3BUNINET,74:38:89ANNAXAnz,74:38:B7Canon,74:3A:20NewH3CTe,74:3A:65NEC,74:3A:EFKaonmedi,74:3C:18TaicangT,74:3E:2BRuckusWi,74:3E:CBGentrice,74:40:BBHonHaiPr,74:40:BELGInnote,74:42:7FAVMAudio,74:42:8BApple,74:44:01Netgear,74:45:2DHuaweiDe,74:45:8ASamsungE,74:45:CECresyn,74:46:A0HewlettP,74:4A:A4zte,74:4B:E9Explorer,74:4C:A1LiteonTe,74:4D:28Routerbo,74:4D:79Arrive,74:50:4ENewH3CTe,74:51:BAXiaomiCo,74:53:27Commsen,74:54:27Shenzhen,74:54:7DCiscoSPV,74:56:12ARRISGro,74:57:98TRUMPFLa,74:58:F3AmazonTe,74:59:09HuaweiTe,74:59:33DanalEnt,74:5A:AAHuaweiTe,74:5B:C5IEEERegi,74:5B:C5:00:00:00/28IRSSyste,74:5B:C5:10:00:00/28BeijingI,74:5B:C5:20:00:00/28SiglentT,74:5B:C5:30:00:00/28Oxon,74:5B:C5:40:00:00/28uGridNet,74:5B:C5:50:00:00/28SpringCa,74:5B:C5:60:00:00/28YekaniMa,74:5B:C5:70:00:00/28Shenzhen,74:5B:C5:80:00:00/28EDOMO,74:5B:C5:90:00:00/28HaikouFr,74:5B:C5:A0:00:00/28FournieG,74:5B:C5:B0:00:00/28Smartipl,74:5B:C5:C0:00:00/28ComNot,74:5B:C5:D0:00:00/28CelyssSa,74:5B:C5:E0:00:00/28QingdaoW,74:5C:4BGNAudio,74:5C:9FTCTmobil,74:5C:FAShenzhen,74:5D:43BSHHausg,74:5D:68Fiberhom,74:5E:1CPioneer,74:5F:00SamsungS,74:5F:90LAMTechn,74:5F:AETslPpl,74:60:FAHuaweiTe,74:61:4BChongqin,74:63:C2HuaweiDe,74:63:DFVTS,74:65:0CApple,74:65:D1Atlinks,74:66:30TmiYtti,74:67:F7ExtremeN,74:6A:3AAperi,74:6A:89Rezolt,74:6A:8FVSVision,74:6B:82Movek,74:6B:ABGuangdon,74:6E:E4AsiaVita,74:6F:19Icarvisi,74:6F:3DContec,74:6F:F7WistronN,74:70:69HuaweiDe,74:70:FDIntelCor,74:72:1EEdisonLa,74:72:B0Guangzho,74:72:F2ChipsipT,74:73:1Difmelect,74:73:36MICRODIG,74:75:48AmazonTe,74:76:5BQuectelW,74:78:18Jurumani,74:78:27Dell,74:7A:90MurataMa,74:7B:7AETH,74:7D:24PhicommS,74:7D:B6AliweiCo,74:7E:1ARedEmbed,74:7E:2DBeijingT,74:81:14Apple,74:81:9APTHarton,74:83:C2Ubiquiti,74:83:EFAristaNe,74:84:E1Dongguan,74:85:2APegatron,74:85:C4NewH3CTe,74:86:0BCisco,74:86:7ADell,74:87:A9OCTTechn,74:87:BBCiena,74:88:2AHuaweiTe,74:88:8BADBBroad,74:88:BBCisco,74:8A:0DARRISGro,74:8A:28HMDGloba,74:8A:69KoreaIma,74:8B:29Micobiom,74:8B:34Shanghai,74:8D:08Apple,74:8E:08Bestek,74:8E:F8BrocadeC,74:8F:1BMasterIm,74:8F:3CApple,74:8F:4DduagonGe,74:90:1FRagileNe,74:90:50RenesasE,74:91:1ARuckusWi,74:91:BDFour,74:93:A4ZebraTec,74:94:3DAgJuncti,74:95:ECAlpsalpi,74:96:37TodaairE,74:97:81zte,74:99:75IBM,74:9A:C0Cachengo,74:9B:E8HitronTe,74:9C:52HuizhouD,74:9C:E3KodaClou,74:9D:79Sercomm,74:9D:8FHuaweiTe,74:9D:DC2Wire,74:9E:A5Ohsung,74:9E:AFApple,74:9E:F5SamsungE,74:A0:2FCisco,74:A0:63HuaweiTe,74:A2:E6Cisco,74:A3:4AZimi,74:A4:A7QRSMusic,74:A4:B5Powerlea,74:A5:28HuaweiTe,74:A7:22LGElectr,74:A7:8Ezte,74:A7:EAAmazonTe,74:AB:93BlinkbyA,74:AC:5FQikuInte,74:AC:B9Ubiquiti,74:AD:98Cisco,74:AD:B7ChinaMob,74:AE:76iNovoBro,74:B0:0CNetworkV,74:B4:72Ciesse,74:B5:7Ezte,74:B5:87Apple,74:B6:B6eero,74:B7:B3Shenzhen,74:B9:1ENanjingB,74:B9:EBJinQianM,74:BA:DBLongconn,74:BB:D3Shenzhen,74:BE:08ATEKProd,74:BF:A1Hyunteck,74:BF:B7Nusoft,74:BF:C0Canon,74:C1:4FHuaweiTe,74:C1:7DInfinixm,74:C2:46AmazonTe,74:C3:30Shenzhen,74:C6:21Zhejiang,74:C6:3BAzureWav,74:C9:29Zhejiang,74:C9:9AEricsson,74:C9:A3Fiberhom,74:CA:25Calxeda,74:CB:F3Lavainte,74:CC:39Fiberhom,74:CD:0CSmithMye,74:CE:56PacketFo,74:CF:00Shenzhen,74:D0:2BASUSTekC,74:D0:DCEricsson,74:D2:1DHuaweiTe,74:D2:85TexasIns,74:D4:35Giga-Byt,74:D6:37AmazonTe,74:D6:54Gint,74:D6:75WYMATecn,74:D6:CBNewH3CTe,74:D6:EATexasIns,74:D7:CAPanasoni,74:D8:3EIntelCor,74:D8:50Evrisko,74:DA:38EdimaxTe,74:DA:88Tp-LinkT,74:DA:DAD-LinkIn,74:DA:EATexasIns,74:DB:D1Ebay,74:DE:2BLiteonTe,74:DF:BFLiteonTe,74:E0:6EErgophon,74:E1:4AIEEERegi,74:E1:4A:00:00:00/28Altenbur,74:E1:4A:10:00:00/28Cerevo,74:E1:4A:20:00:00/28KLIMATSO,74:E1:4A:30:00:00/28emz-Hana,74:E1:4A:40:00:00/28YUG-SIST,74:E1:4A:50:00:00/28UTU,74:E1:4A:60:00:00/28Emerging,74:E1:4A:70:00:00/28APMTechn,74:E1:4A:80:00:00/28aritec,74:E1:4A:90:00:00/28KantoAir,74:E1:4A:A0:00:00/28AStarDes,74:E1:4A:B0:00:00/28LoctekVi,74:E1:4A:C0:00:00/28WuhanShe,74:E1:4A:D0:00:00/28Knog,74:E1:4A:E0:00:00/28DiamondK,74:E1:4A:F0:00:00/28Private,74:E1:82TexasIns,74:E1:9AFiberhom,74:E1:B6Apple,74:E2:0CAmazonTe,74:E2:77Vizmonet,74:E2:8CMicrosof,74:E2:F5Apple,74:E3:36FujianSt,74:E4:24Apiste,74:E5:0BIntelCor,74:E5:37Radspin,74:E5:43LiteonTe,74:E5:F9IntelCor,74:E6:0FTecnoMob,74:E6:E2Dell,74:E7:C6ARRISGro,74:E9:BFHuaweiTe,74:EA:3ATp-LinkT,74:EA:C8NewH3CTe,74:EA:CBNewH3CTe,74:EA:E8ARRISGro,74:EB:80SamsungE,74:EC:42Fiberhom,74:EC:B2AmazonTe,74:EC:F1Acumen,74:EE:2AShenzhen,74:F0:6DAzureWav,74:F0:7DBnCOM,74:F1:02BeijingH,74:F4:13MaxwellF,74:F6:12ARRISGro,74:F6:1CHTC,74:F6:61Schneide,74:F7:26NeuronRo,74:F7:37Kce,74:F7:F6Shanghai,74:F8:5DBerkeley,74:F8:DBIEEERegi,74:F8:DB:00:00:00/28EnerconT,74:F8:DB:10:00:00/28GHLAdvan,74:F8:DB:20:00:00/28Shenzhen,74:F8:DB:30:00:00/28Atx,74:F8:DB:40:00:00/28WiFiHots,74:F8:DB:50:00:00/28Provisio,74:F8:DB:60:00:00/28Shenzhen,74:F8:DB:70:00:00/28WuhanTia,74:F8:DB:80:00:00/28SongamSy,74:F8:DB:90:00:00/28Avantree,74:F8:DB:A0:00:00/28BallardT,74:F8:DB:B0:00:00/28CapwaveT,74:F8:DB:C0:00:00/28Tbm,74:F8:DB:D0:00:00/28SimonEle,74:F8:DB:E0:00:00/28BernardK,74:F8:DB:F0:00:00/28Private,74:F9:1AOnface,74:F9:CANintendo,74:FD:A0Compupal,74:FE:48Advantec,74:FF:4CSkyworth,74:FF:7DWrenSoun,78:00:9ESamsungE,78:02:8BApple,78:02:8FAdaptive,78:02:B1Cisco,78:02:B7ShenZhen,78:02:F8XiaomiCo,78:04:73TexasIns,78:04:E3HuaweiDe,78:05:41Queclink,78:05:5FShenzhen,78:05:8CmMaxComm,78:06:C9HuaweiDe,78:07:38ZUKElzab,78:0A:C7BaofengT,78:0C:B8IntelCor,78:0C:F0Cisco,78:0E:D1TRUMPFWe,78:0F:77HangZhou,78:10:53ChinaMob,78:11:00Quantums,78:11:85NBSPayme,78:11:DCXIAOMIEl,78:12:B8Orantek,78:13:05IEEERegi,78:13:05:00:00:00/28InnoSenT,78:13:05:10:00:00/28GlobalMe,78:13:05:20:00:00/28LeaffEng,78:13:05:30:00:00/28microtec,78:13:05:40:00:00/28JiangxiW,78:13:05:50:00:00/28Ats-Conv,78:13:05:60:00:00/28CRRCNang,78:13:05:70:00:00/28E-StoneE,78:13:05:80:00:00/28Shenzhen,78:13:05:90:00:00/28Shenzhen,78:13:05:A0:00:00/28Leonardo,78:13:05:B0:00:00/28Bithouse,78:13:05:C0:00:00/28Brigates,78:13:05:D0:00:00/28Shanghai,78:13:05:E0:00:00/28Dongguan,78:13:E0FujianSt,78:17:35NokiaSha,78:17:BEHuaweiTe,78:18:81AzureWav,78:18:A8HuaweiDe,78:19:2ENASCENTT,78:19:F7JuniperN,78:1C:5ASHARP,78:1D:4Azte,78:1D:BAHuaweiTe,78:1D:FDJabil,78:1F:11RABLight,78:1F:DBSamsungE,78:20:79IDTech,78:22:3DAffirmed,78:23:27SamsungE,78:23:AEARRISGro,78:24:AFASUSTekC,78:25:44Omnima,78:25:7ALEOInnov,78:25:ADSamsungE,78:28:CASonos,78:29:EDAskeyCom,78:2A:79Integrat,78:2B:46IntelCor,78:2B:64Bose,78:2B:CBDell,78:2C:29NewH3CTe,78:2D:7ETRENDnet,78:2E:56ChinaMob,78:2E:EFNokia,78:2F:17Xlab,78:30:3BStephenT,78:30:E1UltraCle,78:31:2Bzte,78:31:C1Apple,78:32:1BD-LinkIn,78:32:4FMillenni,78:35:A0ZurnIndu,78:36:07CermateT,78:36:90YulongCo,78:36:CCGuangdon,78:37:16SamsungE,78:3A:6CTecnoMob,78:3A:84Apple,78:3C:E3Kai-EE,78:3D:5BTELNETRe,78:3E:53BSkyB,78:3F:15EasySYNC,78:40:E4SamsungE,78:44:05FUJITUHO,78:44:4AShenzhen,78:44:76ZioncomE,78:44:FDTp-LinkT,78:45:01Biamp,78:45:58Ubiquiti,78:45:61CyberTAN,78:45:B3HuaweiDe,78:45:C4Dell,78:46:C4DaehapHy,78:46:D4SamsungE,78:47:1DSamsungE,78:47:E3SichuanT,78:48:59HewlettP,78:49:1DWill-Bur,78:4B:08frobotic,78:4B:87MurataMa,78:4F:43Apple,78:4F:9BJuniperN,78:50:05MOKOTECH,78:50:7CJuniperN,78:51:0CLiveU,78:52:1ASamsungE,78:52:4AOptonic,78:52:62Shenzhen,78:53:0DShenzhen,78:53:64SHIFT,78:53:F2Roxton,78:54:2ED-LinkIn,78:55:17SankyuEl,78:57:12MobileIn,78:57:73HuaweiTe,78:57:B0GertecBr,78:58:60HuaweiTe,78:58:F3Vachen,78:59:3ERAFI,78:59:5ESamsungE,78:59:68HonHaiPr,78:5C:28PrimeMot,78:5C:72HiosoTec,78:5D:C8LGElectr,78:5E:A2SunitecE,78:5E:E8IEEERegi,78:5E:E8:00:00:00/28Youtrans,78:5E:E8:10:00:00/28RIKENKEI,78:5E:E8:20:00:00/28Vega-Abs,78:5E:E8:30:00:00/28Incontro,78:5E:E8:40:00:00/28beijingA,78:5E:E8:50:00:00/28Infomobi,78:5E:E8:60:00:00/28Guangdon,78:5E:E8:70:00:00/28MtB?Lg?T,78:5E:E8:80:00:00/28Jiangxig,78:5E:E8:90:00:00/28TOPDONTE,78:5E:E8:A0:00:00/28YakeTian,78:5E:E8:B0:00:00/28LanternE,78:5E:E8:C0:00:00/28Finetool,78:5E:E8:D0:00:00/28TachyonN,78:5E:E8:E0:00:00/28SuzhouTi,78:5F:36Shenzhen,78:5F:4CArgoxInf,78:61:7CMitsumiE,78:62:56HuaweiTe,78:64:C0Apple,78:64:E6GreenMot,78:65:3BShaoxing,78:65:59Sagemcom,78:66:AEZTECInst,78:67:0EWistronN,78:67:D7Apple,78:68:F7YSTenTec,78:69:D4Shenyang,78:6A:1FARRISGro,78:6A:89HuaweiTe,78:6C:1CApple,78:6D:94PaloAlto,78:6D:EBGELighti,78:70:52Welotec,78:71:9CARRISGro,78:72:5DCisco,78:72:64IEEERegi,78:72:64:00:00:00/28SmartMor,78:72:64:10:00:00/28Zhengzho,78:72:64:20:00:00/28Bzbgear,78:72:64:30:00:00/28Guangdon,78:72:64:40:00:00/28Asustor,78:72:64:50:00:00/28CalttaTe,78:72:64:60:00:00/28Shenzhen,78:72:64:70:00:00/28ConjingN,78:72:64:80:00:00/28GsouTech,78:72:64:90:00:00/28Shenzhen,78:72:64:A0:00:00/28TyphoonH,78:72:64:B0:00:00/28digades,78:72:64:C0:00:00/28Comcast,78:72:64:D0:00:00/28QT,78:72:64:E0:00:00/28HeltecAu,78:76:D9EXARAGro,78:7A:6FJuiceTec,78:7B:8AApple,78:7D:48ItelMobi,78:7D:53ExtremeN,78:7D:F3Sterlite,78:7E:61Apple,78:7F:62GiKmbH,78:80:38FunaiEle,78:81:02Sercomm,78:81:8FServerRa,78:81:CEChinaMob,78:84:3CSony,78:84:EEIndraEsp,78:85:F4HuaweiDe,78:86:B6Shenzhen,78:87:0DUnifiedg,78:88:6DApple,78:88:8ACDRSpSpk,78:89:73Cmc,78:8A:20Ubiquiti,78:8B:2AZhenShiI,78:8B:77StandarT,78:8C:4DIndymeSo,78:8C:54PingComm,78:8C:77LexmarkI,78:8D:F7HitronTe,78:8E:33JiangsuS,78:90:A2zte,78:91:E9Raisecom,78:92:3ENokia,78:92:9CIntelCor,78:94:B4Sercomm,78:94:E8RadioBri,78:95:EBItelMobi,78:96:82zte,78:96:84ARRISGro,78:97:C3DingxinI,78:98:E8D-LinkIn,78:98:FDQ9Networ,78:99:5CNationzT,78:99:66MusilabE,78:99:8FMediline,78:9C:85AugustHo,78:9C:E7Shenzhen,78:9E:D0SamsungE,78:9F:4CHOERBIGE,78:9F:70Apple,78:9F:87SiemensI,78:9F:AAHuaweiDe,78:A0:51iiNetLab,78:A1:06Tp-LinkT,78:A1:83Advidia,78:A2:A0Nintendo,78:A3:51Shenzhen,78:A3:E4Apple,78:A5:04TexasIns,78:A5:DDShenzhen,78:A6:83Precidat,78:A6:A0Hangzhou,78:A6:BDDAEYEONC,78:A6:E1BrocadeC,78:A7:14Amphenol,78:A7:EB1More,78:A8:73SamsungE,78:AA:82NewH3CTe,78:AB:60ABBAustr,78:AB:BBSamsungE,78:AC:44Dell,78:AC:BFIgneous,78:AC:C0HewlettP,78:AE:0CFarSouth,78:AF:58Gimasi,78:AF:E4Comau,78:B2:13DWnetTec,78:B2:8DBeijingT,78:B3:B9ShangHai,78:B3:CEElotouch,78:B4:6AHuaweiTe,78:B5:54HuaweiDe,78:B5:D2EverTrea,78:B6:C1AOBOTele,78:B6:ECScufGami,78:B8:1AInterSal,78:B8:4BSichuanT,78:B8:D6ZebraTec,78:BA:D0Shinybow,78:BA:F9Cisco,78:BB:88MaxioTec,78:BC:1ACisco,78:BD:BCSamsungE,78:BE:B6Enhanced,78:BE:BDSTULZ,78:C1:A7zte,78:C2:C0IEEERegi,78:C2:C0:00:00:00/28Shenzhen,78:C2:C0:10:00:00/28Xronos,78:C2:C0:20:00:00/28RONIX,78:C2:C0:30:00:00/28NingboSa,78:C2:C0:40:00:00/28OryLabor,78:C2:C0:50:00:00/28ShenZhen,78:C2:C0:60:00:00/28SichuanT,78:C2:C0:70:00:00/28Guangzho,78:C2:C0:80:00:00/28BeijingC,78:C2:C0:90:00:00/28Ses,78:C2:C0:A0:00:00/28Ombitron,78:C2:C0:B0:00:00/28WanChaoA,78:C2:C0:C0:00:00/28Shanghai,78:C2:C0:D0:00:00/28KORF,78:C2:C0:E0:00:00/28Huwomobi,78:C2:C0:F0:00:00/28Private,78:C3:13ChinaMob,78:C3:E9SamsungE,78:C4:0EH&DWirel,78:C4:ABShenzhen,78:C5:E5TexasIns,78:C5:F8HuaweiDe,78:C6:BBAnalogDe,78:C8:81SonyInte,78:C9:5EMidmarkR,78:CA:04Nokia,78:CA:39Apple,78:CA:5EElno,78:CA:83IEEERegi,78:CA:83:00:00:00/28Daincube,78:CA:83:10:00:00/28Exceloci,78:CA:83:20:00:00/28Apc,78:CA:83:30:00:00/28Neofon,78:CA:83:40:00:00/28PinholeB,78:CA:83:50:00:00/28HuatuneT,78:CA:83:60:00:00/28Nomiku,78:CA:83:70:00:00/28BeijingC,78:CA:83:80:00:00/28Ihm,78:CA:83:90:00:00/28LouroeEl,78:CA:83:A0:00:00/28Eksagate,78:CA:83:B0:00:00/28Zhejiang,78:CA:83:C0:00:00/28Elanview,78:CA:83:D0:00:00/28HubeiBoy,78:CA:83:E0:00:00/28Konecran,78:CB:2CJoinDigi,78:CB:33DHCSoftw,78:CB:68DaehapHy,78:CC:2BSinewyTe,78:CD:8ESMCNetwo,78:CF:2FHuaweiTe,78:CF:F9HuaweiDe,78:D0:04NeousysT,78:D1:29Vicos,78:D1:62Apple,78:D2:94Netgear,78:D3:47Ericsson,78:D3:4FPace-O-M,78:D3:8DHongkong,78:D3:EDNorma,78:D4:F1IEEERegi,78:D4:F1:00:00:00/28BurischE,78:D4:F1:10:00:00/28Cartende,78:D4:F1:20:00:00/28Lyngsoe,78:D4:F1:30:00:00/28Ekoenerg,78:D4:F1:40:00:00/28BYDAutol,78:D4:F1:50:00:00/28HuaqinTe,78:D4:F1:60:00:00/28Guangzho,78:D4:F1:70:00:00/28FamarFue,78:D4:F1:80:00:00/28XiamenCh,78:D4:F1:90:00:00/28shanghai,78:D4:F1:A0:00:00/28BonengTr,78:D4:F1:B0:00:00/28Jiangsub,78:D4:F1:C0:00:00/28Tnb,78:D4:F1:D0:00:00/28Quidel,78:D4:F1:E0:00:00/28BlueSpar,78:D5:B5Navielek,78:D6:6FAristocr,78:D6:B2Toshiba,78:D6:DCMotorola,78:D6:F0SamsungE,78:D7:1ACiena,78:D7:52HuaweiTe,78:D7:5FApple,78:D8:00IEEERegi,78:D8:00:00:00:00/28Kvernela,78:D8:00:10:00:00/28Shenzhen,78:D8:00:20:00:00/28Shanghai,78:D8:00:30:00:00/28Shenzhen,78:D8:00:40:00:00/28CSInstru,78:D8:00:50:00:00/28Björkvik,78:D8:00:60:00:00/28AlangoTe,78:D8:00:70:00:00/28NimbeLin,78:D8:00:80:00:00/28Salunda,78:D8:00:90:00:00/28SightLin,78:D8:00:A0:00:00/28Insignal,78:D8:00:B0:00:00/28Maddalen,78:D8:00:C0:00:00/28Shenzhen,78:D8:00:D0:00:00/28KoreaMic,78:D8:00:E0:00:00/28CLIntern,78:D9:9FNuComHK,78:D9:E9Momentum,78:DA:07Zhejiang,78:DA:6ECisco,78:DA:A2Cynosure,78:DA:B3GBOTechn,78:DB:2FTexasIns,78:DD:08HonHaiPr,78:DD:12Arcadyan,78:DD:33HuaweiTe,78:DD:D6c-scape,78:DD:D9Guangzho,78:DE:E4TexasIns,78:E1:03AmazonTe,78:E2:2CHuaweiDe,78:E2:BDVodafone,78:E3:6DEspressi,78:E3:B5HewlettP,78:E3:DEApple,78:E4:00HonHaiPr,78:E7:D1HewlettP,78:E8:B6zte,78:E9:80RainUs,78:E9:CFTellesco,78:EB:14Shenzhen,78:EB:39Institut,78:EC:22Shanghai,78:EC:74Kyland-U,78:EF:4CUnetconv,78:F0:9BHuaweiDe,78:F2:35SichuanA,78:F2:9EPegatron,78:F5:57HuaweiTe,78:F5:E5BEGAGant,78:F5:FDHuaweiTe,78:F7:BESamsungE,78:F7:D0Silverbr,78:F8:82LGElectr,78:F8:B8RakoCont,78:F9:44Private,78:F9:B4NokiaSol,78:FC:14FamilyZo,78:FD:94Apple,78:FE:3DJuniperN,78:FE:41Socusnet,78:FE:E2Shanghai,78:FF:57IntelCor,78:FF:CATecnoMob,7A:15:ECYatomiTe,7A:35:EASHARP,7A:4B:A4Innostor,7A:4B:F3DilonTec,7A:4D:2FOpenComp,7A:84:F0McDowell,7A:AB:77Orange,7A:FD:44Rosenber,7C:00:4DHuaweiTe,7C:01:0ATexasIns,7C:01:87CurtisIn,7C:01:91Apple,7C:02:BCHansungE,7C:03:4CSagemcom,7C:03:5EXiaomiCo,7C:03:ABXiaomiCo,7C:03:C9Shenzhen,7C:03:D8Sagemcom,7C:04:D0Apple,7C:05:07Pegatron,7C:05:1ERafael,7C:06:23UltraEle,7C:08:D9Shanghai,7C:09:2BBekey,7C:0A:3FSamsungE,7C:0A:50J-MEX,7C:0B:C6SamsungE,7C:0C:F6Guangdon,7C:0E:CECisco,7C:10:15Brillian,7C:10:C9ASUSTekC,7C:11:BEApple,7C:11:CBHuaweiTe,7C:11:CDQianTang,7C:14:76DamallTe,7C:16:0DSaia-Bur,7C:16:89Sagemcom,7C:18:CDE-TRON,7C:1A:038Locatio,7C:1A:C0HuaweiTe,7C:1A:FCDalianCo,7C:1B:93HuaweiDe,7C:1C:4ELGInnote,7C:1C:68SamsungE,7C:1C:F1HuaweiTe,7C:1D:D9XiaomiCo,7C:1E:06NewH3CTe,7C:1E:52Microsof,7C:1E:B32NTELEKO,7C:20:48KoamTac,7C:20:64Alcatel-,7C:21:0DCisco,7C:21:0ECisco,7C:21:4AIntelCor,7C:21:D8Shenzhen,7C:23:02SamsungE,7C:24:0CTelechip,7C:24:99Apple,7C:25:86JuniperN,7C:25:87chaowifi,7C:25:DAFn-LinkT,7C:26:34ARRISGro,7C:26:64Sagemcom,7C:27:BCHuiZhouG,7C:2A:31IntelCor,7C:2A:DBXiaomiCo,7C:2B:E1Shenzhen,7C:2C:F3SecureEl,7C:2E:0DBlackmag,7C:2E:BDGoogle,7C:2E:DDSamsungE,7C:2F:80GigasetC,7C:31:0ECisco,7C:33:6EMEGElect,7C:35:48Transcen,7C:38:66TexasIns,7C:38:6CRealTime,7C:38:ADSamsungE,7C:39:20SsomaSec,7C:39:53zte,7C:39:85HuaweiTe,7C:3B:D5ImagoGro,7C:3C:B6Shenzhen,7C:3D:2BHuaweiDe,7C:3E:74HuaweiDe,7C:3E:9DPatech,7C:41:A2Nokia,7C:43:8FE-BandCo,7C:44:4CEntertai,7C:46:85Motorola,7C:47:7CIEEERegi,7C:47:7C:00:00:00/28BungBung,7C:47:7C:10:00:00/28Photosyn,7C:47:7C:20:00:00/28Powerlan,7C:47:7C:30:00:00/28EyeLock,7C:47:7C:40:00:00/28RLCElect,7C:47:7C:50:00:00/28MidwestM,7C:47:7C:60:00:00/28Zerosyst,7C:47:7C:70:00:00/28BlueSmar,7C:47:7C:80:00:00/28Shenzhen,7C:47:7C:90:00:00/28DaLianCh,7C:47:7C:A0:00:00/28DspreadT,7C:47:7C:B0:00:00/28Hangzhou,7C:47:7C:C0:00:00/28Annapurn,7C:47:7C:D0:00:00/28Speedifi,7C:47:7C:E0:00:00/28I-Conver,7C:48:B2VidaReso,7C:49:B9PlexusMa,7C:49:EBXIAOMIEl,7C:4A:82Portsmit,7C:4A:A8MindTree,7C:4B:78RedSunSy,7C:4C:58ScaleCom,7C:4C:A5BSkyB,7C:4E:09Shenzhen,7C:4F:7DSawwave,7C:4F:B5Arcadyan,7C:50:49Apple,7C:50:79IntelCor,7C:50:DAPrivate,7C:51:89SGWirele,7C:52:59SichuanJ,7C:53:4AMetamako,7C:55:A7Kastle,7C:55:E7YSI,7C:57:3CArubaaHe,7C:57:4ECOBI,7C:5A:1CSophos,7C:5A:67JNC,7C:5C:F8IntelCor,7C:60:4AAvelon,7C:60:97HuaweiTe,7C:61:66AmazonTe,7C:61:93HTC,7C:64:56SamsungE,7C:66:9DTexasIns,7C:66:EFHonHaiPr,7C:67:A2IntelCor,7C:69:6BAtmosicT,7C:69:F6Cisco,7C:6A:B3IbcTechn,7C:6A:C3GatesAir,7C:6A:DBSafeTone,7C:6A:F3Integrat,7C:6B:33TenyuTec,7C:6B:52TigaroWi,7C:6B:9CGuangdon,7C:6B:F7NTI,7C:6C:39Pixsys,7C:6C:8FAmsNeve,7C:6D:62Apple,7C:6D:A6Superwav,7C:6D:F8Apple,7C:6F:06Caterpil,7C:6F:F8ShenZhen,7C:70:BCIEEERegi,7C:70:BC:00:00:00/28Shanghai,7C:70:BC:10:00:00/28XD-GEAut,7C:70:BC:20:00:00/28DigitalL,7C:70:BC:30:00:00/28FLEXIM,7C:70:BC:40:00:00/28K-Vision,7C:70:BC:50:00:00/28CanaryCo,7C:70:BC:60:00:00/28Bidgely,7C:70:BC:70:00:00/28NomadDig,7C:70:BC:80:00:00/28Mennekes,7C:70:BC:90:00:00/28dogtra,7C:70:BC:A0:00:00/28AmetekVI,7C:70:BC:B0:00:00/28TohanEng,7C:70:BC:C0:00:00/28LukupMed,7C:70:BC:D0:00:00/28mk-messt,7C:70:BC:E0:00:00/28HoperunM,7C:70:BC:F0:00:00/28Private,7C:70:DBIntelCor,7C:71:76WuxiiDat,7C:72:6EEricsson,7C:72:E4UnikeyTe,7C:73:8BCocoonAl,7C:73:EBHuaweiDe,7C:76:30Shenzhen,7C:76:35IntelCor,7C:76:68HuaweiTe,7C:76:73ENMAS,7C:78:7ESamsungE,7C:78:B2WyzeLabs,7C:79:E8PayRange,7C:7A:53PhytrexT,7C:7A:91IntelCor,7C:7B:8BControlC,7C:7B:E4ZSedaiKe,7C:7D:3DHuaweiTe,7C:7D:41JinmuyuE,7C:82:2DNortec,7C:82:74Shenzhen,7C:83:06GlenDimp,7C:83:34IEEERegi,7C:83:34:00:00:00/28Thermali,7C:83:34:10:00:00/28LinearLo,7C:83:34:20:00:00/28PEMtroni,7C:83:34:30:00:00/28BeijingC,7C:83:34:40:00:00/28Fusus,7C:83:34:50:00:00/28ProBrand,7C:83:34:60:00:00/28Wojinxin,7C:83:34:70:00:00/28ChengDUY,7C:83:34:80:00:00/28SiliconX,7C:83:34:90:00:00/28SernetSu,7C:83:34:A0:00:00/28Enginete,7C:83:34:B0:00:00/28Shenzhen,7C:83:34:C0:00:00/28HunanDat,7C:83:34:D0:00:00/28MSVelekt,7C:83:34:E0:00:00/28Balter,7C:84:37ChinaPos,7C:85:30Nokia,7C:87:CEEspressi,7C:89:56SamsungE,7C:89:C1PaloAlto,7C:8A:E1CompalIn,7C:8B:B5SamsungE,7C:8B:CATp-LinkT,7C:8D:91Shanghai,7C:8E:E4TexasIns,7C:8F:DEDWnetTec,7C:91:22SamsungE,7C:94:2AHuaweiTe,7C:94:B2PhilipsH,7C:95:B1ExtremeN,7C:95:F3Cisco,7C:96:D2Fihonest,7C:97:63Openmati,7C:9A:1DApple,7C:9A:54Technico,7C:9A:9BVSEvalen,7C:9E:BDEspressi,7C:9F:07CigShang,7C:A1:5DGNReSoun,7C:A1:77HuaweiTe,7C:A1:AEApple,7C:A2:37KingSlid,7C:A2:3EHuaweiTe,7C:A2:9BDSignT,7C:A6:1DMhl,7C:A6:2AHewlettP,7C:A7:B0Shenzhen,7C:A9:6BSyrotech,7C:A9:7DObjeniou,7C:AB:25MesmoTec,7C:AB:60Apple,7C:AC:B2BoschSof,7C:AD:4FCisco,7C:AD:74Cisco,7C:B0:3EOSRAM,7C:B0:73RealmeCh,7C:B0:C2IntelCor,7C:B1:5DHuaweiTe,7C:B1:77Satelco,7C:B2:1BCiscoSPV,7C:B2:32HuiZhouG,7C:B2:5CAcaciaCo,7C:B2:7DIntelCor,7C:B3:7BQingdaoI,7C:B5:42ACESTech,7C:B5:9BTp-LinkT,7C:B7:33AskeyCom,7C:B7:7BParadigm,7C:B9:4CBouffalo,7C:B9:60Shanghai,7C:BA:CCIEEERegi,7C:BA:CC:00:00:00/28TGT,7C:BA:CC:10:00:00/28Changsha,7C:BA:CC:20:00:00/28MacoLigh,7C:BA:CC:30:00:00/28Izkare,7C:BA:CC:40:00:00/28SunAsiaT,7C:BA:CC:50:00:00/28FortemTe,7C:BA:CC:60:00:00/28FossilPo,7C:BA:CC:70:00:00/28VirginOr,7C:BA:CC:80:00:00/28Collinea,7C:BA:CC:90:00:00/28Yongguan,7C:BA:CC:A0:00:00/28Annapurn,7C:BA:CC:B0:00:00/28Briowire,7C:BA:CC:C0:00:00/28FlyingLo,7C:BA:CC:D0:00:00/28SIGMA-EL,7C:BA:CC:E0:00:00/28AlphaTec,7C:BB:6FCoscoEle,7C:BB:8ANintendo,7C:BC:84IEEERegi,7C:BC:84:00:00:00/28Neovo,7C:BC:84:10:00:00/28XiamenMa,7C:BC:84:20:00:00/283STechno,7C:BC:84:30:00:00/28Shanghai,7C:BC:84:40:00:00/28Continen,7C:BC:84:50:00:00/28Nanninga,7C:BC:84:60:00:00/28Sociétéd,7C:BC:84:70:00:00/28XujiChan,7C:BC:84:80:00:00/28Shenzhen,7C:BC:84:90:00:00/28Hitiq,7C:BC:84:A0:00:00/28Opnt,7C:BC:84:B0:00:00/28Guangzho,7C:BC:84:C0:00:00/28TibitCom,7C:BC:84:D0:00:00/28VantageI,7C:BC:84:E0:00:00/28BeijingT,7C:BD:06AEREFUso,7C:BF:88Mobilico,7C:BF:B1ARRISGro,7C:C1:77IngramMi,7C:C2:25SamsungE,7C:C2:55SuperMic,7C:C2:94BeijingX,7C:C2:C6TP-Link,7C:C3:85HuaweiTe,7C:C3:A1Apple,7C:C4:EFDevialet,7C:C5:37Apple,7C:C6:C4KolffCom,7C:C7:09Shenzhen,7C:C7:7EFiberhom,7C:C8:ABAcroAsso,7C:C8:D0TianjinY,7C:C8:D7Damalisk,7C:C9:26WuhanGre,7C:C9:5ADellEMC,7C:CB:0DAntairaT,7C:CB:E2IEEERegi,7C:CB:E2:00:00:00/28HeyuanYo,7C:CB:E2:10:00:00/28CeoTroni,7C:CB:E2:20:00:00/281000eyes,7C:CB:E2:30:00:00/28AstrumTe,7C:CB:E2:40:00:00/28Ningbobi,7C:CB:E2:50:00:00/28DTECHLab,7C:CB:E2:60:00:00/28SYElectr,7C:CB:E2:70:00:00/28Hangzhou,7C:CB:E2:80:00:00/28Polartek,7C:CB:E2:90:00:00/28Hangzhou,7C:CB:E2:A0:00:00/28Shanghai,7C:CB:E2:B0:00:00/28EasyBroa,7C:CB:E2:C0:00:00/28mirakont,7C:CB:E2:D0:00:00/28optilink,7C:CB:E2:E0:00:00/28AplexTec,7C:CC:1FSichuanT,7C:CC:B8IntelCor,7C:CD:11MS-Magne,7C:CD:3CGuangzho,7C:CF:CFShanghai,7C:D1:C3Apple,7C:D3:0AInventec,7C:D5:66AmazonTe,7C:D6:61XiaomiCo,7C:D7:62Freestyl,7C:D8:44Enmotus,7C:D9:5CGoogle,7C:D9:A0HuaweiTe,7C:D9:F4UABTelto,7C:D9:FENewCosmo,7C:DA:84Dongnian,7C:DB:98AskeyCom,7C:DD:11Chongqin,7C:DD:20IOXOSTec,7C:DD:76SuzhouHa,7C:DD:90Shenzhen,7C:DD:E9ATOMtech,7C:DF:A1Espressi,7C:E0:44NEON,7C:E1:FFComputer,7C:E2:CAJuniperN,7C:E4:AAPrivate,7C:E5:24Quirky,7C:E5:6BESENOpto,7C:E9:7CItelMobi,7C:E9:D3HonHaiPr,7C:EB:7FDmetProd,7C:EB:AERidgelin,7C:EB:EAAsct,7C:EC:79TexasIns,7C:EC:9BFuzhouTe,7C:ED:8DMicrosof,7C:EF:18Creative,7C:EF:61STRElekt,7C:EF:8AInhonInt,7C:F0:5FApple,7C:F0:98BeeBeans,7C:F0:BALinkwell,7C:F2:DDVence,7C:F3:1BLGElectr,7C:F4:29NUUO,7C:F4:62BeijingH,7C:F6:66TuyaSmar,7C:F8:54SamsungE,7C:F8:80Cisco,7C:F9:0ESamsungE,7C:F9:5CUILapp,7C:F9:A0Fiberhom,7C:FA:DFApple,7C:FC:16Apple,7C:FC:3CVisteon,7C:FD:6BXiaomiCo,7C:FD:82Guangdon,7C:FE:28Salutron,7C:FE:4EShenzhen,7C:FE:90Mellanox,7C:FF:4DAVMAudio,7C:FF:62HuizhouS,80:00:0BIntelCor,80:00:10At&T[Mis,80:00:6EApple,80:01:84HTC,80:02:9CGemtekTe,80:02:DFORA,80:03:84RuckusWi,80:05:88RuijieNe,80:05:DFMontageT,80:07:1BVsolutio,80:07:A2EssonTec,80:09:02Keysight,80:0A:06COMTEC,80:0A:80IEEERegi,80:0A:80:00:00:00/28GolanaTe,80:0A:80:10:00:00/28Dongguan,80:0A:80:20:00:00/28Sumitomo,80:0A:80:30:00:00/28BeijingV,80:0A:80:40:00:00/28Llvision,80:0A:80:50:00:00/28Shenzhen,80:0A:80:60:00:00/28BeijingG,80:0A:80:F0:00:00/28Private,80:0B:51ChengduX,80:0C:67Apple,80:0D:D7Latticew,80:0E:24ForgetBo,80:12:DFShenzhen,80:13:82HuaweiTe,80:14:40SunlitSy,80:14:A8Guangzho,80:16:05Vodafone,80:16:09SleepNum,80:16:B7BrunelUn,80:17:7DNortelNe,80:18:44Dell,80:18:A7SamsungE,80:19:34IntelCor,80:19:67Shanghai,80:19:FEJianLing,80:1D:AAAvaya,80:1F:02EdimaxTe,80:1F:12Microchi,80:20:AFTradeFID,80:20:DASagemcom,80:20:E1BVBADPTe,80:20:FDSamsungE,80:22:75BeijingB,80:22:78ChinaMob,80:22:A7NECPlatf,80:24:8FCisco,80:25:11ItelMobi,80:26:89D-LinkIn,80:29:94Technico,80:2A:A8Ubiquiti,80:2A:FAGermanee,80:2B:F9HonHaiPr,80:2D:BFCisco,80:2D:E1Solarbri,80:2E:14azetiNet,80:2F:DEZurichIn,80:30:49LiteonTe,80:30:DCTexasIns,80:30:E0HewlettP,80:31:F0SamsungE,80:32:53IntelCor,80:34:28Microchi,80:34:57OT,80:35:C1XiaomiCo,80:37:73Netgear,80:38:96SHARP,80:38:BCHuaweiTe,80:38:FBIntelCor,80:38:FDLeapFrog,80:39:E5Patlite,80:3A:0AIntegrat,80:3A:59At&T,80:3A:F4Fiberhom,80:3B:2AABBXiame,80:3B:9Aghe-cese,80:3B:F6LookEasy,80:3E:48Shenzhen,80:3F:5DWinstars,80:3F:D6bytesatw,80:41:26HuaweiTe,80:41:4EBbkEduca,80:42:7CAdolfTed,80:44:FDChinaMob,80:45:DDIntelCor,80:47:31PacketDe,80:47:86SamsungE,80:48:A5SichuanT,80:49:71Apple,80:4A:14Apple,80:4B:20Ventilat,80:4B:50SiliconL,80:4E:70SamsungE,80:4E:81SamsungE,80:4F:58ThinkEco,80:50:1BNokia,80:50:67WDTechno,80:50:F6ItelMobi,80:54:6AShenzhen,80:54:D9HuaweiTe,80:56:F2HonHaiPr,80:57:19SamsungE,80:58:C5NovaTecK,80:58:F8Motorola,80:59:FDNoviga,80:5A:04LGElectr,80:5E:0CYealinkX,80:5E:4FFn-LinkT,80:5E:C0YealinkX,80:5F:C5Apple,80:60:07Rim,80:60:B7CloudNet,80:61:5FBeijingS,80:61:8FShenzhen,80:64:59Nimbus,80:64:7AOlaSense,80:65:59EMMicroe,80:65:6DSamsungE,80:65:E9BenQ,80:66:29Prescope,80:69:33HuaweiTe,80:69:40Lexar,80:6A:10WhiskerL,80:6A:B0Shenzhen,80:6C:1BMotorola,80:6C:8BKaeserKo,80:6C:BCNETNewEl,80:6D:71AmazonTe,80:6D:97Private,80:6F:1CHuaweiDe,80:6F:B0TexasIns,80:71:1FJuniperN,80:71:7AHuaweiTe,80:72:15BSkyB,80:72:64HuaweiDe,80:73:9FKyocera,80:74:59Ks,80:74:84ALLWinne,80:75:1FBSkyB,80:76:93Newag,80:78:71AskeyCom,80:79:5DInfinixm,80:79:AEShanDong,80:7A:7FABBGenwa,80:7A:BFHTC,80:7B:1ECorsairM,80:7B:3ESamsungE,80:7B:85IEEERegi,80:7B:85:00:00:00/28Shiroshi,80:7B:85:10:00:00/28Hangzhou,80:7B:85:20:00:00/28Phoenix,80:7B:85:30:00:00/28ZhuhaiTO,80:7B:85:40:00:00/28QuantelU,80:7B:85:50:00:00/28Efco,80:7B:85:60:00:00/28QuickteT,80:7B:85:70:00:00/28ChenduNi,80:7B:85:80:00:00/28IDair,80:7B:85:90:00:00/28SmartEle,80:7B:85:A0:00:00/28Interpla,80:7B:85:B0:00:00/28Oliotalo,80:7B:85:C0:00:00/28NingboPl,80:7B:85:D0:00:00/28KaynesTe,80:7B:85:E0:00:00/28Mersen,80:7B:85:F0:00:00/28Private,80:7D:14HuaweiTe,80:7D:1BNeosyste,80:7D:3AEspressi,80:7D:E3Chongqin,80:7E:B4Shenzhen,80:7F:F8JuniperN,80:81:A5TONGQING,80:82:23Apple,80:82:87ATCOMTec,80:84:A9oshkosh,80:86:98Netronic,80:86:D9SamsungE,80:86:F2IntelCor,80:89:17Tp-LinkT,80:8A:8BvivoMobi,80:8A:BDSamsungE,80:8A:F7Nanoleaf,80:8B:5CShenzhen,80:8C:97Kaonmedi,80:8D:B7HewlettP,80:8F:1DTp-LinkT,80:8F:E8Intelbra,80:91:2ALihRonge,80:91:33AzureWav,80:91:C0AgileMes,80:92:9FApple,80:93:93Xapt,80:94:6CTokyoRad,80:96:21Lenovo,80:96:B1ARRISGro,80:96:CAHonHaiPr,80:97:1BAltenerg,80:9B:20IntelCor,80:9F:9BSichuanA,80:9F:ABFiberhom,80:9F:F5SamsungE,80:A0:36Shanghai,80:A1:ABIntellis,80:A1:D7Shanghai,80:A2:35Edgecore,80:A5:89AzureWav,80:A7:96Neurotek,80:A8:5DOsterhou,80:AA:A4Usag,80:AC:7CSichuanA,80:AC:ACJuniperN,80:AD:00CnetTech,80:AD:16XiaomiCo,80:AD:67KasdaNet,80:B0:3DApple,80:B0:7Bzte,80:B2:19Elektron,80:B2:34Technico,80:B2:89Forworld,80:B3:2AUKGridSo,80:B5:75HuaweiTe,80:B6:24Ivs,80:B6:55IntelCor,80:B6:86HuaweiTe,80:B7:08BlueDanu,80:B7:09Viptela,80:B7:45SilkTech,80:B9:5CELFTECH,80:B9:7Aeero,80:BA:ACTeleAdap,80:BA:E6Neets,80:BB:EBSatmap,80:BE:05Apple,80:C1:6EHewlettP,80:C3:BASennheis,80:C5:01OctoGate,80:C5:48Shenzhen,80:C5:E6Microsof,80:C5:F2AzureWav,80:C6:3FRemecBro,80:C6:ABTechnico,80:C6:CAEndian,80:C7:55Panasoni,80:C7:C5Fiberhom,80:C8:62Openpeak,80:C9:55RedpineS,80:CA:4BShenzhen,80:CB:BCQingdaoI,80:CC:12HuaweiDe,80:CC:9CNetgear,80:CE:62HewlettP,80:CE:B1Theissen,80:CE:B9SamsungE,80:CF:41LenovoMo,80:CF:A2HuaweiDe,80:D0:19Embed,80:D0:4ATechnico,80:D0:65CKS,80:D0:9BHuaweiTe,80:D1:60Integrat,80:D1:8BHangzhou,80:D2:1DAzureWav,80:D2:E5Nintendo,80:D3:36Cern,80:D4:33LzLabs,80:D4:A5HuaweiTe,80:D6:05Apple,80:D7:33QSRAutom,80:DA:13eero,80:DA:BCMegafone,80:DB:31PowerQuo,80:E0:1DCisco,80:E1:BFHuaweiTe,80:E4:55NewH3CTe,80:E4:DAIEEERegi,80:E4:DA:00:00:00/28Wheatsto,80:E4:DA:10:00:00/28Guangzho,80:E4:DA:20:00:00/28ThurlbyT,80:E4:DA:30:00:00/28BeijingG,80:E4:DA:40:00:00/28BeijingY,80:E4:DA:50:00:00/28CavalryS,80:E4:DA:60:00:00/28BroadMed,80:E4:DA:70:00:00/28Shortcut,80:E4:DA:80:00:00/28Krizerin,80:E4:DA:90:00:00/28Elcus,80:E4:DA:A0:00:00/28Neutroni,80:E4:DA:B0:00:00/28NanjingL,80:E4:DA:C0:00:00/28EVERSp,80:E4:DA:D0:00:00/28DalianRo,80:E4:DA:E0:00:00/28AkenoriP,80:E4:DA:F0:00:00/28Private,80:E5:40ARRISGro,80:E6:50Apple,80:E8:2CHewlettP,80:E8:6FCisco,80:EA:07Tp-LinkT,80:EA:23WistronN,80:EA:96Apple,80:EA:CADialogSe,80:EB:77Wistron,80:ED:2CApple,80:EE:25Shenzhen,80:EE:73Shuttle,80:F1:F1Tech4hom,80:F2:5EKyynel,80:F3:EFFacebook,80:F5:03ARRISGro,80:F5:93IRCOSist,80:F5:B5TexasIns,80:F6:2EHangzhou,80:F8:EBRayTight,80:FA:5BClevo,80:FB:06HuaweiTe,80:FB:F0QuectelW,80:FB:F1Freescal,80:FD:7ABLUProdu,80:FF:A8Unidis,84:00:2DPegatron,84:00:D2Sony,84:01:12Kaonmedi,84:01:A7Greyware,84:02:83HUMAX,84:03:28JuniperN,84:04:D2KiraleTe,84:06:FAFiberhom,84:0B:2DSamsungE,84:0B:7CHitronTe,84:0D:8EEspressi,84:0F:45Shanghai,84:10:0DMotorola,84:11:9ESamsungE,84:11:C2IEEERegi,84:11:C2:00:00:00/28Kazdream,84:11:C2:10:00:00/28BeijingD,84:11:C2:20:00:00/28Futureco,84:11:C2:30:00:00/28FUJIFILM,84:11:C2:40:00:00/28StcMzta,84:11:C2:50:00:00/28AIBIoT,84:11:C2:60:00:00/28Kessel,84:11:C2:70:00:00/28Ei3,84:11:C2:80:00:00/28Leybold,84:11:C2:90:00:00/28CTechBil,84:11:C2:A0:00:00/28igus,84:11:C2:B0:00:00/28Guangdon,84:11:C2:C0:00:00/28Provisio,84:11:C2:D0:00:00/28Goldmund,84:11:C2:E0:00:00/28Dangerou,84:13:9Fzte,84:14:4DIntelCor,84:15:D3HuaweiTe,84:16:F9Tp-LinkT,84:17:15GPElectr,84:17:66WeifangG,84:17:EFTechnico,84:18:26Osram,84:18:3ARuckusWi,84:18:88JuniperN,84:1B:38Shenzhen,84:1B:5ENetgear,84:1B:77IntelCor,84:1C:70zte,84:1E:26KERNEL-I,84:1E:A3Sagemcom,84:20:96Shenzhen,84:21:41Shenzhen,84:21:F1HuaweiTe,84:22:5EShenzhen,84:23:88RuckusWi,84:24:8DZebraTec,84:25:19SamsungE,84:25:3Fsilextec,84:25:A4Tariox,84:25:DBSamsungE,84:26:15ADBBroad,84:26:2BNokia,84:26:7AGuangdon,84:26:90BeijingT,84:27:B6ChinaMob,84:27:CEPresidin,84:28:5ASaffronS,84:29:14EMPORIAT,84:29:99Apple,84:2A:FDHP,84:2B:2BDell,84:2B:50Huria,84:2B:BCModellei,84:2C:80SichuanC,84:2E:14SiliconL,84:2E:27SamsungE,84:2F:75InnokasG,84:30:95HonHaiPr,84:30:E5SkyHawke,84:32:6FGuangzho,84:32:EAAnhuiWan,84:34:97HewlettP,84:36:11hyungseu,84:37:D5SamsungE,84:38:35Apple,84:38:38SamsungE,84:39:BE:00:00:00/28HinoEngi,84:39:BE:10:00:00/28Guangzho,84:39:BE:20:00:00/28ChengDuv,84:39:BE:30:00:00/28ShenZhen,84:39:BE:40:00:00/28Shenzhen,84:39:BE:50:00:00/28Neat,84:39:BE:60:00:00/28Shenzhen,84:39:BE:80:00:00/28DiamondP,84:39:BE:90:00:00/28Guangdon,84:39:BE:A0:00:00/28Emotiq,84:39:BE:B0:00:00/28Shenzhen,84:39:BE:C0:00:00/28EDCElect,84:39:BE:D0:00:00/28Shenzhen,84:3A:4BIntelCor,84:3A:5BInventec,84:3B:10Lvswitch,84:3D:C6Cisco,84:3E:79Shenzhen,84:3E:92HuaweiTe,84:3F:4ETri-Tech,84:40:76Drivenet,84:41:67Apple,84:44:64ServerU,84:44:AFZhejiang,84:46:FEHuaweiTe,84:47:09Shenzhen,84:47:65HuaweiTe,84:48:23WOXTERTE,84:49:15vArmourN,84:4B:B7BeijingS,84:4B:F5HonHaiPr,84:4F:03Ablelink,84:50:9AEasySoft,84:51:81SamsungE,84:54:DFHuaweiDe,84:55:A5SamsungE,84:56:9CCohoData,84:57:33Microsof,84:57:87DVRC&C,84:5A:81ffly4u,84:5B:12HuaweiTe,84:5C:93Chabrier,84:5C:F3IntelCor,84:5D:D7Shenzhen,84:60:82Private,84:61:A0ARRISGro,84:62:23Shenzhen,84:62:A6EuroCBPh,84:63:D6Microsof,84:65:69NewH3CTe,84:68:3EIntelCor,84:68:78Apple,84:68:C8Totolink,84:69:91Nokia,84:6A:66Sumitomo,84:6A:EDWireless,84:6B:48ShenZhen,84:6E:B1ParkAssi,84:6F:CEGuangdon,84:71:27SiliconL,84:71:6AHuaweiDe,84:72:07I&CTechn,84:73:03LetvMobi,84:74:2Azte,84:74:60zte,84:76:16Addatsro,84:76:37HuaweiTe,84:77:78Cochlear,84:78:8BApple,84:78:ACCisco,84:79:33profichi,84:79:73Shanghai,84:7A:88HTC,84:7A:B6AltoBeam,84:7B:57IntelCor,84:7B:EBDell,84:7C:9BGDMideaA,84:7D:50HolleyMe,84:7E:40TexasIns,84:7F:3DIntegrat,84:80:2DCisco,84:80:94Meter,84:82:F4BeijingH,84:83:19Hangzhou,84:83:36Newrun,84:83:71Avaya,84:84:33ParadoxE,84:85:06Apple,84:85:0AHellaSon,84:85:E6Guangdon,84:86:F3Greenvit,84:89:ADApple,84:89:ECIEEERegi,84:89:EC:00:00:00/28SmartGia,84:89:EC:10:00:00/28Research,84:89:EC:20:00:00/28thousand,84:89:EC:30:00:00/28Aerionic,84:89:EC:40:00:00/28VayyarIm,84:89:EC:50:00:00/28ZephyrEn,84:89:EC:60:00:00/28POCTbiot,84:89:EC:70:00:00/28BYDA,84:89:EC:80:00:00/28ArtsDigi,84:89:EC:90:00:00/28Shenzhen,84:89:EC:A0:00:00/28NewellBr,84:89:EC:B0:00:00/28EPSaElek,84:89:EC:C0:00:00/28Shinkawa,84:89:EC:D0:00:00/28PriceInd,84:89:EC:E0:00:00/28Shenzhen,84:8A:8DCisco,84:8B:CDIEEERegi,84:8B:CD:00:00:00/28SouXinCo,84:8B:CD:10:00:00/28Shenzhen,84:8B:CD:20:00:00/28CCXTechn,84:8B:CD:30:00:00/28Annapurn,84:8B:CD:40:00:00/28LogicSup,84:8B:CD:50:00:00/28exodraft,84:8B:CD:60:00:00/28TwtgR&D,84:8B:CD:70:00:00/28SmartCod,84:8B:CD:80:00:00/28Dunsttro,84:8B:CD:90:00:00/28Noralsy,84:8B:CD:A0:00:00/28SpheraTe,84:8B:CD:B0:00:00/28Chongqin,84:8B:CD:C0:00:00/28Wormit,84:8B:CD:D0:00:00/28EngisatL,84:8B:CD:E0:00:00/28Emotiv,84:8C:8DApple,84:8D:84Rajant,84:8D:C7CiscoSPV,84:8E:0CApple,84:8E:96Embertec,84:8E:DFSony,84:8F:69Dell,84:90:00Arnold&R,84:93:0CInCoaxNe,84:93:A0HuaweiDe,84:94:8CHitronTe,84:96:81CathayCo,84:96:D8ARRISGro,84:97:B8Memjet,84:98:66SamsungE,84:9A:40Hangzhou,84:9C:A6Arcadyan,84:9D:64SMC,84:9D:C2Shanghai,84:9D:C5CenteraP,84:9F:B5HuaweiTe,84:A0:6ESagemcom,84:A1:34Apple,84:A1:D1Sagemcom,84:A2:4DBirdsEye,84:A3:B5Propulsi,84:A4:23Sagemcom,84:A4:66SamsungE,84:A6:C8IntelCor,84:A7:83AlcatelL,84:A7:88Perples,84:A8:E4HuaweiTe,84:A9:38LCFCHeFe,84:A9:3EHewlettP,84:A9:91CyberTra,84:A9:C4HuaweiTe,84:A9:EACareerTe,84:AA:9CMitraSta,84:AB:1AApple,84:AB:26Tiinlab,84:AC:16Apple,84:AC:A4BeijingN,84:AC:FBCrouzetA,84:AD:58HuaweiTe,84:AD:8DApple,84:AF:1FBeatSyst,84:AF:ECBuffalo,84:B1:53Apple,84:B2:61Cisco,84:B3:1BKinexon,84:B5:17Cisco,84:B5:41SamsungE,84:B5:9CJuniperN,84:B6:30SichuanT,84:B8:02Cisco,84:B8:66BeijingX,84:B8:B8Motorola,84:BA:20SiliconL,84:BA:3BCanon,84:BB:69ARRISGro,84:BE:52HuaweiTe,84:C0:EFSamsungE,84:C1:C1JuniperN,84:C2:E4JiangsuQ,84:C3:E8Vaillant,84:C5:A6IntelCor,84:C7:27Gnodal,84:C7:8FAPSNetwo,84:C7:A9C3Po,84:C7:EASony,84:C8:07ADVAOpti,84:C8:B1Incognit,84:C9:B2D-LinkIn,84:C9:C6Shenzhen,84:CC:63HuaweiDe,84:CC:A8Espressi,84:CD:62ShenZhen,84:CF:BFFairphon,84:D1:5ATCTmobil,84:D3:2AIeee1905,84:D3:43Calix,84:D3:D5HuaweiDe,84:D4:12PaloAlto,84:D4:7EArubaaHe,84:D4:C8Widex,84:D6:08Wingtech,84:D6:C5SolarEdg,84:D6:D0AmazonTe,84:D8:1BTp-LinkT,84:D9:31Hangzhou,84:D9:C8Unipatte,84:DB:2FSierraWi,84:DB:9EPinkNect,84:DB:ACHuaweiTe,84:DB:FCNokia,84:DD:20TexasIns,84:DD:B7CilagInt,84:DE:3DCrystalV,84:DF:0CNet2Grid,84:DF:19ChuangoS,84:E0:58ARRISGro,84:E0:F4IEEERegi,84:E0:F4:00:00:00/28ShenZhen,84:E0:F4:10:00:00/28MedicusT,84:E0:F4:20:00:00/28Hangzhou,84:E0:F4:30:00:00/28ASLInter,84:E0:F4:40:00:00/28PetroInT,84:E0:F4:50:00:00/28Hangzhou,84:E0:F4:60:00:00/28Liaoning,84:E0:F4:70:00:00/28Dantherm,84:E0:F4:80:00:00/28RAY,84:E0:F4:90:00:00/28Shenzhen,84:E0:F4:A0:00:00/28iSolutio,84:E0:F4:B0:00:00/28OrchardE,84:E0:F4:C0:00:00/28Aimtron,84:E0:F4:D0:00:00/28Logos01,84:E0:F4:E0:00:00/28Scale-Te,84:E3:23GreenWav,84:E3:27TailynTe,84:E3:42TuyaSmar,84:E4:D9Shenzhen,84:E5:D8Guangdon,84:E6:29Bluwan,84:E7:14LiangHer,84:E8:92Actionte,84:E9:86HuaweiDe,84:EA:97Shenzhen,84:EA:99Vieworks,84:EA:EDRoku,84:EB:18TexasIns,84:EB:3EVivintSm,84:EB:EFCisco,84:ED:33BBMC,84:EF:18IntelCor,84:F1:29Metrasca,84:F1:47Cisco,84:F3:EBEspressi,84:F4:93OMSspols,84:F6:4CCrossPoi,84:F6:FAMiovisio,84:F7:03Espressi,84:F8:83LuminarT,84:FC:ACApple,84:FC:FEApple,84:FD:27SiliconL,84:FD:D1IntelCor,84:FE:9ERTCIndus,84:FE:DCBorqsBei,88:01:18BLT,88:01:F2VitecSys,88:03:55Arcadyan,88:03:E9Guangdon,88:07:4BLGElectr,88:09:05MTMCommu,88:09:07MKTSyste,88:09:AFMasimo,88:0F:10HuamiInf,88:0F:B6JabilCir,88:10:36PanodicS,88:10:8FHuaweiTe,88:11:96HuaweiTe,88:12:3DSuzhouAq,88:12:4EQualcomm,88:14:2BProtonic,88:15:44CiscoMer,88:15:C5HuaweiDe,88:17:A3Integrat,88:18:AETamron,88:19:08Apple,88:1B:99Shenzhen,88:1C:95ItelMobi,88:1D:FCCisco,88:1F:A1Apple,88:20:12LMITechn,88:21:E3Nebusens,88:23:1FFibocomW,88:23:64Watchnet,88:23:8CFiberhom,88:23:FETTTechCo,88:25:2CArcadyan,88:25:93Tp-LinkT,88:28:B3HuaweiTe,88:29:49RenesasE,88:29:50NetmoonT,88:29:9CSamsungE,88:2A:5ENewH3CTe,88:2B:94MADOKASY,88:2B:D7Addénerg,88:2D:53BaiduOnl,88:2E:5AstorONE,88:30:8AMurataMa,88:32:9BSamsungE,88:33:14TexasIns,88:33:BEIvenix,88:34:FEBoschAut,88:35:4CTransics,88:35:C1OiElectr,88:36:12SRCCompu,88:36:5FLGElectr,88:36:6CEFMNetwo,88:36:CFHuaweiDe,88:3A:30ArubaaHe,88:3B:8BCheering,88:3C:1CMercury,88:3D:24Google,88:3F:4ATexasIns,88:3F:99Siemens,88:3F:D3HuaweiTe,88:40:33HuaweiTe,88:40:3BHuaweiTe,88:40:67infomark,88:41:57Shenzhen,88:41:C1OrbisatD,88:41:FCAirTiesW,88:43:E1Cisco,88:44:77HuaweiTe,88:44:F6Nokia,88:46:04XiaomiCo,88:46:2ATelechip,88:4A:18Opulinks,88:4A:70Wacom,88:4A:EATexasIns,88:4B:39SiemensH,88:4C:CFPulzze,88:4D:7CApple,88:50:DDInfiniba,88:50:F6Shenzhen,88:51:7AHMDGloba,88:51:FBHewlettP,88:53:2EIntelCor,88:53:95Apple,88:53:D4HuaweiTe,88:54:1FGoogle,88:57:1DSeongjiI,88:57:6DXTAElect,88:57:EEBuffalo,88:5A:06Guangdon,88:5A:85WistronN,88:5A:92Cisco,88:5B:DDExtremeN,88:5C:47AlcatelL,88:5D:90IEEERegi,88:5D:90:00:00:00/28FoshanHu,88:5D:90:10:00:00/28ShenZhen,88:5D:90:20:00:00/28DAIDONGI,88:5D:90:30:00:00/28CPAC,88:5D:90:40:00:00/28WuhanStr,88:5D:90:50:00:00/28Shenzhen,88:5D:90:60:00:00/28Hi-Profi,88:5D:90:70:00:00/28SchmidtH,88:5D:90:80:00:00/28Creative,88:5D:90:90:00:00/28Gigatech,88:5D:90:A0:00:00/28Shenzhen,88:5D:90:B0:00:00/28PremierM,88:5D:90:C0:00:00/28iRoom,88:5D:90:D0:00:00/28Hexaglob,88:5D:90:E0:00:00/28UnitacTe,88:5D:90:F0:00:00/28Private,88:5D:FBzte,88:5F:E8IEEERegi,88:5F:E8:00:00:00/28Junghein,88:5F:E8:10:00:00/28ApoideaT,88:5F:E8:20:00:00/28OptoEngi,88:5F:E8:30:00:00/28SonnetLa,88:5F:E8:40:00:00/28Beijingl,88:5F:E8:50:00:00/28HauchBac,88:5F:E8:60:00:00/28Shenzhen,88:5F:E8:70:00:00/28RedTechn,88:5F:E8:80:00:00/28Changsha,88:5F:E8:90:00:00/28Sowee,88:5F:E8:A0:00:00/28LisleDes,88:5F:E8:B0:00:00/28Shenzhen,88:5F:E8:C0:00:00/28InorProc,88:5F:E8:D0:00:00/28zhejiang,88:5F:E8:E0:00:00/28UnicomGl,88:61:5ASianoMob,88:63:DFApple,88:64:40Apple,88:66:39HuaweiTe,88:66:5AApple,88:66:A5Apple,88:68:5CShenzhen,88:6A:B1vivoMobi,88:6A:E3AlphaNet,88:6B:0FBluegiga,88:6B:44SunnovoI,88:6B:6EApple,88:6B:76ChinaHop,88:6E:E1ErbeElek,88:6F:D4Dell,88:70:33Hangzhou,88:70:8CLenovoMo,88:70:EFSCProfes,88:71:B1ARRISGro,88:71:E5AmazonTe,88:73:84Toshiba,88:73:98K2ETekpo,88:75:56Cisco,88:75:98SamsungE,88:78:73IntelCor,88:78:9CGameTech,88:79:5BKonkaGro,88:79:7EMotorola,88:7A:31Velankan,88:7E:25ExtremeN,88:7F:03ComperTe,88:81:B9HuaweiDe,88:82:79Shenzhen,88:83:22SamsungE,88:83:5DFn-LinkT,88:86:03HuaweiTe,88:86:A0SimtonTe,88:86:C2STABILOI,88:87:17Canon,88:87:DDDarbeeVi,88:89:14AllCompo,88:89:2FHuaweiTe,88:89:64GSIElect,88:8B:5DStorageA,88:8C:19BradyAsi,88:8E:68HuaweiDe,88:8E:7FAtop,88:90:09JuniperN,88:90:8DCisco,88:91:66Viewcoop,88:91:DDRacktivi,88:94:71BrocadeC,88:94:7EFiberhom,88:94:8FXianZhis,88:94:F9GemicomT,88:95:B9UnifiedP,88:96:4EARRISGro,88:96:55Zitte,88:96:76TTCMARCO,88:96:B6GlobalFi,88:96:F2ValeoSch,88:97:46SichuanA,88:97:65exands,88:97:DFEntrypas,88:98:21Teraon,88:9B:39SamsungE,88:9C:A6BTBKorea,88:9D:98Allied-t,88:9E:33TCTmobil,88:9E:68Technico,88:9F:6FSamsungE,88:9F:AAHellaGut,88:9F:FAHonHaiPr,88:A0:84Formatio,88:A0:BEHuaweiTe,88:A2:5EJuniperN,88:A2:D7HuaweiTe,88:A3:03SamsungE,88:A3:CCAmatisCo,88:A4:79Apple,88:A5:BDQpcom,88:A6:C6Sagemcom,88:A7:3CRagentek,88:A9:A7IEEERegi,88:A9:A7:00:00:00/28Shenzhen,88:A9:A7:10:00:00/28Solaredg,88:A9:A7:20:00:00/28Honeywel,88:A9:A7:30:00:00/28Mikroele,88:A9:A7:40:00:00/28ThomasDa,88:A9:A7:50:00:00/28Volterma,88:A9:A7:60:00:00/28SieperLü,88:A9:A7:70:00:00/28kimuragi,88:A9:A7:80:00:00/28psbintra,88:A9:A7:90:00:00/28FlashFor,88:A9:A7:A0:00:00/28Zhejiang,88:A9:A7:B0:00:00/28Twk-Elek,88:A9:A7:C0:00:00/28AndroVid,88:A9:A7:D0:00:00/28AvlinkIn,88:A9:A7:E0:00:00/28ImpactDi,88:A9:B7Apple,88:AC:9EShenzhen,88:AC:C0ZyxelCom,88:AC:C1Generito,88:AD:43Pegatron,88:AD:D2SamsungE,88:AE:07Apple,88:AE:1DCompalIn,88:AE:DDEliteGro,88:B1:11IntelCor,88:B1:68DeltaCon,88:B1:E1MojoNetw,88:B2:91Apple,88:B3:62NokiaSha,88:B4:36Private,88:B4:A6Motorola,88:B6:27GembirdE,88:B6:6Beasynetw,88:B6:EEDishTech,88:B8:D0Dongguan,88:B9:45Apple,88:BA:7FQfiednet,88:BC:C1HuaweiTe,88:BD:45SamsungE,88:BD:78Flaircom,88:BF:D5SimpleAu,88:BF:E4HuaweiTe,88:C0:8BApple,88:C2:27HuaweiTe,88:C2:42Poynt,88:C2:55TexasIns,88:C3:6EBeijingE,88:C3:97BeijingX,88:C3:B3Sovico,88:C3:E5BetopTec,88:C6:26Logitech,88:C6:63Apple,88:C9:B3IEEERegi,88:C9:B3:00:00:00/28AdoptNet,88:C9:B3:10:00:00/28CervozTe,88:C9:B3:20:00:00/28shenzhen,88:C9:B3:30:00:00/28FortiveS,88:C9:B3:40:00:00/28Hasbro,88:C9:B3:50:00:00/28Brabende,88:C9:B3:60:00:00/28HugoTech,88:C9:B3:70:00:00/28RobertBo,88:C9:B3:80:00:00/28Divelbis,88:C9:B3:90:00:00/28Richbeam,88:C9:B3:A0:00:00/28GefranDr,88:C9:B3:B0:00:00/28Shenzhen,88:C9:B3:C0:00:00/28Shenzhen,88:C9:B3:D0:00:00/28OriginsT,88:C9:B3:E0:00:00/28Sercomm,88:C9:D0LGElectr,88:CB:87Apple,88:CB:A5SuzhouTo,88:CC:45Skyworth,88:CE:FAHuaweiTe,88:CF:98HuaweiTe,88:D0:39TCLTechn,88:D1:71Beghelli,88:D1:99Vencer,88:D2:11EkoDevic,88:D2:74zte,88:D2:BFGermanAu,88:D3:7BFirmTek,88:D5:0CGuangdon,88:D5:A8ItelMobi,88:D6:52AMERGINT,88:D7:BCDEP,88:D7:F6ASUSTekC,88:D8:2EIntelCor,88:D9:62CanopusU,88:D9:8FJuniperN,88:DA:1ARedpineS,88:DA:33BeijingX,88:DC:96EnGenius,88:DD:79Voltaire,88:DE:7CAskeyCom,88:DE:A9Roku,88:DF:9ENewH3CTe,88:E0:34Shinwain,88:E0:56HuaweiTe,88:E0:A0Shenzhen,88:E0:F3JuniperN,88:E1:61ArtBeiji,88:E3:ABHuaweiTe,88:E6:03Avotek,88:E6:28Shenzhen,88:E6:4BJuniperN,88:E7:12Whirlpoo,88:E7:A6iKnowled,88:E8:7FApple,88:E8:F8YongTaiE,88:E9:0Finnomdle,88:E9:17Tamaggo,88:E9:A4HewlettP,88:E9:FEApple,88:ED:1CCudoComm,88:EF:16ARRISGro,88:F0:31Cisco,88:F0:77Cisco,88:F4:88cellonco,88:F4:90Jetmobil,88:F5:6EHuaweiTe,88:F7:BFvivoMobi,88:F7:C7Technico,88:F8:72HuaweiTe,88:FC:A6devolo,88:FD:15Lineeye,88:FE:D6ShangHai,8A:00:B0Eoptolin,8A:0A:F7MontageT,8A:34:BCFiberwor,8A:4F:8BIrdeto,8A:94:ADNexgen,8A:A5:C1RanovusU,8A:B3:DAHomePlug,8A:C7:2ERoku,8A:CB:A4Resideo,8A:DA:26Eleven,8A:FB:16TeraByte,8C:00:6DApple,8C:02:FACOMMANDO,8C:04:BADell,8C:04:FFTechnico,8C:05:51Koubachi,8C:07:8CFlowData,8C:08:8BRemoteSo,8C:09:F4ARRISGro,8C:0C:87Nokia,8C:0C:90RuckusWi,8C:0C:A3Amper,8C:0D:76HuaweiTe,8C:0E:60NanjingJ,8C:0E:E3Guangdon,8C:0F:6FPegatron,8C:0F:83AngieHos,8C:0F:A0di-soric,8C:0F:C9HuaweiDe,8C:0F:FAHutec,8C:10:D4Sagemcom,8C:11:CBABUSSecu,8C:14:7DIEEERegi,8C:14:7D:00:00:00/28Nio,8C:14:7D:10:00:00/28Private,8C:14:7D:20:00:00/28Agilent,8C:14:7D:30:00:00/28RemotecT,8C:14:7D:40:00:00/28Nanjingb,8C:14:7D:50:00:00/28UnwiredN,8C:14:7D:60:00:00/28Shenzhen,8C:14:7D:70:00:00/28UrbanHel,8C:14:7D:80:00:00/28V2,8C:14:7D:90:00:00/28AnywareS,8C:14:7D:A0:00:00/28Bluemega,8C:14:7D:B0:00:00/28BauschDa,8C:14:7D:C0:00:00/28Reynaers,8C:14:7D:D0:00:00/28Shenzhen,8C:14:7D:E0:00:00/28Electric,8C:14:B4zte,8C:15:C7HuaweiTe,8C:16:45LCFCHeFe,8C:18:50ChinaMob,8C:18:D9Shenzhen,8C:19:2DIEEERegi,8C:19:2D:00:00:00/28NoritsuP,8C:19:2D:10:00:00/28Shenzhen,8C:19:2D:20:00:00/28DataRemo,8C:19:2D:30:00:00/28Greenfie,8C:19:2D:40:00:00/28Charmlin,8C:19:2D:50:00:00/28ElcoTian,8C:19:2D:60:00:00/28smartHom,8C:19:2D:70:00:00/28Srett,8C:19:2D:80:00:00/28Shenzhen,8C:19:2D:90:00:00/28ViaWear,8C:19:2D:A0:00:00/28TeleAlar,8C:19:2D:B0:00:00/28AbsideNe,8C:19:2D:C0:00:00/28YouZheng,8C:19:2D:D0:00:00/28PyrasTec,8C:19:2D:E0:00:00/28Elcon,8C:19:B5Arcadyan,8C:1A:BFSamsungE,8C:1C:DAIEEERegi,8C:1C:DA:00:00:00/28CEOS,8C:1C:DA:10:00:00/28GESAS,8C:1C:DA:20:00:00/28Geomc,8C:1C:DA:30:00:00/28Structur,8C:1C:DA:40:00:00/28Anntec（B,8C:1C:DA:50:00:00/28Septentr,8C:1C:DA:60:00:00/28LocoLabs,8C:1C:DA:70:00:00/28KTechnol,8C:1C:DA:80:00:00/28Atol,8C:1C:DA:90:00:00/28RaychemR,8C:1C:DA:A0:00:00/28ChinaPot,8C:1C:DA:B0:00:00/28T+Aelekt,8C:1C:DA:C0:00:00/28Alcidae,8C:1C:DA:D0:00:00/28RieglLas,8C:1C:DA:E0:00:00/28Electron,8C:1D:96IntelCor,8C:1E:D9BeijingU,8C:1F:64IEEERegi,8C:1F:64:00:00:00/36SuzhouXi,8C:1F:64:00:30:00/36Brighten,8C:1F:64:06:D0:00/36Monnit,8C:1F:64:07:E0:00/36FLOYD,8C:1F:64:08:60:00/36WEPTECHe,8C:1F:64:08:B0:00/36Shanghai,8C:1F:64:0A:80:00/36SamabaNo,8C:1F:64:0A:B0:00/36NorbitOD,8C:1F:64:0C:50:00/36TechnipF,8C:1F:64:0E:00:00/36Autophar,8C:1F:64:0F:00:00/36Xylon,8C:1F:64:0F:90:00/36ikanInte,8C:1F:64:10:10:00/36ASW-ATI,8C:1F:64:11:F0:00/36NodeUDes,8C:1F:64:12:80:00/36YulistaI,8C:1F:64:13:50:00/36YuvalFic,8C:1F:64:14:40:00/36Langfang,8C:1F:64:15:10:00/36GogoBusi,8C:1F:64:15:E0:00/36Dynomoti,8C:1F:64:16:40:00/36Revo-Tec,8C:1F:64:16:60:00/36HikariAl,8C:1F:64:19:30:00/36Sicon,8C:1F:64:19:C0:00/36Aton,8C:1F:64:1B:60:00/36RedSenso,8C:1F:64:1C:20:00/36SolidInv,8C:1F:64:1D:10:00/36ASStrömu,8C:1F:64:1D:A0:00/36Chongqin,8C:1F:64:1E:10:00/36VAF,8C:1F:64:1E:30:00/36WBNet,8C:1F:64:20:40:00/36castcore,8C:1F:64:21:90:00/36Guangzho,8C:1F:64:21:C0:00/36EMS-Expe,8C:1F:64:22:70:00/36Digilens,8C:1F:64:24:20:00/36Giordano,8C:1F:64:25:A0:00/36WuhanXin,8C:1F:64:25:E0:00/36R2Sonic,8C:1F:64:26:40:00/36BRVossIn,8C:1F:64:27:00:00/36Xi‘anHan,8C:1F:64:28:A0:00/36Arcopie,8C:1F:64:29:60:00/36Roogzhit,8C:1F:64:29:F0:00/36Nagtech,8C:1F:64:2A:50:00/36Nonet,8C:1F:64:2C:80:00/36BRSSiste,8C:1F:64:2E:80:00/36SonoraNe,8C:1F:64:2E:F0:00/36Invisens,8C:1F:64:2F:50:00/36FloridaR,8C:1F:64:30:40:00/36JemacSwe,8C:1F:64:30:A0:00/36XCOMLabs,8C:1F:64:33:00:00/36VisionSa,8C:1F:64:37:00:00/36WOLFAdva,8C:1F:64:37:20:00/36WINKStre,8C:1F:64:38:20:00/36Shenzhen,8C:1F:64:38:50:00/36Multilan,8C:1F:64:39:10:00/36CpcUk,8C:1F:64:39:70:00/36IntelCor,8C:1F:64:39:80:00/36Software,8C:1F:64:3A:40:00/36QLMTechn,8C:1F:64:3A:D0:00/36TowerIQ,8C:1F:64:3B:50:00/36Svms,8C:1F:64:3C:40:00/36NavSysTe,8C:1F:64:3E:00:00/36YPP,8C:1F:64:3E:80:00/36Ruichuan,8C:1F:64:3F:F0:00/36UiseeSha,8C:1F:64:41:40:00/36INSEVIS,8C:1F:64:42:B0:00/36GamberJo,8C:1F:64:43:80:00/36Integerp,8C:1F:64:44:50:00/36FigmentD,8C:1F:64:46:00:00/36Solace,8C:1F:64:46:60:00/36Intamsys,8C:1F:64:49:30:00/36Security,8C:1F:64:4B:00:00/36U-Mei-Da,8C:1F:64:4C:10:00/36Clock-O-,8C:1F:64:4C:70:00/36SBS,8C:1F:64:4C:D0:00/36GuanShow,8C:1F:64:4D:B0:00/36Private,8C:1F:64:4D:D0:00/36GriffynR,8C:1F:64:4E:C0:00/36XORUK,8C:1F:64:4F:A0:00/36Sanskrut,8C:1F:64:51:20:00/36BlikSens,8C:1F:64:53:40:00/36SuryaEle,8C:1F:64:53:60:00/36BeijingL,8C:1F:64:53:A0:00/36TPVision,8C:1F:64:53:D0:00/36Nexconte,8C:1F:64:53:F0:00/36Velvac,8C:1F:64:54:20:00/36Landis+G,8C:1F:64:54:90:00/36BradTech,8C:1F:64:54:C0:00/36GeminiEl,8C:1F:64:57:20:00/36ZmbiziAp,8C:1F:64:57:50:00/36Yu-HengE,8C:1F:64:57:A0:00/36NPOECO-I,8C:1F:64:5D:30:00/36EloyWate,8C:1F:64:60:E0:00/36ICTInter,8C:1F:64:61:10:00/36SiemensI,8C:1F:64:61:90:00/36Labtrino,8C:1F:64:63:80:00/36ThunderD,8C:1F:64:63:B0:00/36Tiama,8C:1F:64:66:30:00/36mal-tech,8C:1F:64:66:C0:00/36LineageP,8C:1F:64:6A:D0:00/36PotterEl,8C:1F:64:6A:E0:00/36BrayInte,8C:1F:64:6C:60:00/36Fit,8C:1F:64:6C:D0:00/36WuhanXin,8C:1F:64:6E:30:00/36ViewSoni,8C:1F:64:6E:A0:00/36KMtronic,8C:1F:64:70:80:00/36Zuum,8C:1F:64:70:E0:00/36OvercomT,8C:1F:64:71:20:00/36NexionDa,8C:1F:64:72:60:00/36Dave,8C:1F:64:72:C0:00/36Antaitec,8C:1F:64:73:D0:00/36NewAgeMi,8C:1F:64:73:F0:00/36Ubiscale,8C:1F:64:75:F0:00/36ASTRACOM,8C:1F:64:76:80:00/36mapnagro,8C:1F:64:77:C0:00/36OrangeTr,8C:1F:64:78:20:00/36Atm,8C:1F:64:79:D0:00/36MurataMa,8C:1F:64:79:E0:00/36AccemicT,8C:1F:64:7A:10:00/36Guardian,8C:1F:64:7A:60:00/36OTMetric,8C:1F:64:7C:80:00/36JacquetD,8C:1F:64:7D:60:00/36AlgodueE,8C:1F:64:7D:D0:00/36TAKASAKI,8C:1F:64:7F:10:00/36AEMSinga,8C:1F:64:80:10:00/36Zhejiang,8C:1F:64:83:A0:00/36Grossenb,8C:1F:64:83:C0:00/36XtendTec,8C:1F:64:85:60:00/36GartenAu,8C:1F:64:87:80:00/36GreenAcc,8C:1F:64:89:20:00/36MDIIndus,8C:1F:64:8A:90:00/36GuanShow,8C:1F:64:8A:A0:00/36ForeverE,8C:1F:64:8A:C0:00/36BOZHONPr,8C:1F:64:8A:E0:00/36Shenzhen,8C:1F:64:8B:50:00/36AshtonBe,8C:1F:64:8C:20:00/36Cirrus,8C:1F:64:8D:10:00/36OrlacoPr,8C:1F:64:8E:20:00/36ALPHA,8C:1F:64:8E:E0:00/36AbbottDi,8C:1F:64:90:30:00/36Portrait,8C:1F:64:90:90:00/36Matelex,8C:1F:64:90:E0:00/36Xacti,8C:1F:64:91:80:00/36AbbottDi,8C:1F:64:92:30:00/36MBconnec,8C:1F:64:92:A0:00/36ThermoOn,8C:1F:64:92:D0:00/36IVORInte,8C:1F:64:94:70:00/36TCVympel,8C:1F:64:95:A0:00/36Shenzhen,8C:1F:64:97:10:00/36Infrasaf,8C:1F:64:97:30:00/36DorsettT,8C:1F:64:98:40:00/36AbacusPe,8C:1F:64:99:10:00/36DBSystel,8C:1F:64:99:80:00/36EVLOStoc,8C:1F:64:9C:10:00/36RealWear,8C:1F:64:9C:E0:00/36ExiFlowM,8C:1F:64:9C:F0:00/36ASAPElec,8C:1F:64:9D:80:00/36Integerp,8C:1F:64:9F:00:00/36ePlant,8C:1F:64:9F:20:00/36MBconnec,8C:1F:64:9F:D0:00/36VishayNo,8C:1F:64:A2:90:00/36Ringtail,8C:1F:64:A4:C0:00/36Flextron,8C:1F:64:A4:E0:00/36SyscomIn,8C:1F:64:A5:70:00/36EkspertS,8C:1F:64:A5:C0:00/36Prosys,8C:1F:64:A7:60:00/36DEUTA-WE,8C:1F:64:A9:40:00/36Futurewa,8C:1F:64:AA:40:00/36HEINENEL,8C:1F:64:AA:B0:00/36BlueSwor,8C:1F:64:AB:50:00/36Justmorp,8C:1F:64:AC:E0:00/36RayhaanN,8C:1F:64:AD:20:00/36YUYAMAMF,8C:1F:64:AE:10:00/36YUYAMAMF,8C:1F:64:AE:D0:00/36MBconnec,8C:1F:64:AE:F0:00/36Scenario,8C:1F:64:AF:70:00/36ard,8C:1F:64:B0:30:00/36Shenzhen,8C:1F:64:B0:C0:00/36Barkodes,8C:1F:64:B2:C0:00/36SanminaI,8C:1F:64:B4:60:00/36Phygital,8C:1F:64:B6:40:00/36GSPSprac,8C:1F:64:B8:20:00/36SeedCore,8C:1F:64:B8:40:00/36SPXFlowT,8C:1F:64:B9:A0:00/36QuercusT,8C:1F:64:BA:30:00/36DEUTA-WE,8C:1F:64:BC:00:00/36GSElektr,8C:1F:64:BE:E0:00/36Sirius,8C:1F:64:C0:30:00/36AbimanEn,8C:1F:64:C1:F0:00/36Esys,8C:1F:64:C2:70:00/36LiftVent,8C:1F:64:C2:F0:00/36PowerEle,8C:1F:64:C4:00:00/36Sciospec,8C:1F:64:C4:10:00/36Katronic,8C:1F:64:C5:00:00/36Spacee,8C:1F:64:C5:40:00/36FirstMod,8C:1F:64:C7:C0:00/36MERKLESc,8C:1F:64:CA:10:00/36Pantheru,8C:1F:64:CA:60:00/36ReliaSpe,8C:1F:64:CA:D0:00/36GeneralM,8C:1F:64:CB:E0:00/36CircaEnt,8C:1F:64:CD:60:00/36USM,8C:1F:64:CD:80:00/36GogoBusi,8C:1F:64:CD:F0:00/36CanwayTe,8C:1F:64:CE:30:00/36PixelDes,8C:1F:64:CE:F0:00/36GoertekR,8C:1F:64:CF:10:00/36ROBOfibe,8C:1F:64:CF:30:00/36ABB,8C:1F:64:D2:90:00/36SecureBi,8C:1F:64:D3:C0:00/36KIBEnerg,8C:1F:64:D4:A0:00/36Caproc,8C:1F:64:D5:40:00/36GrupoEpe,8C:1F:64:D7:80:00/36HunanOus,8C:1F:64:D7:E0:00/36ThalesBe,8C:1F:64:DA:A0:00/36Davetech,8C:1F:64:DA:E0:00/36Maincoau,8C:1F:64:DB:50:00/36victtron,8C:1F:64:DB:90:00/36ErmesEle,8C:1F:64:DB:D0:00/36Giordano,8C:1F:64:DC:90:00/36PeterHub,8C:1F:64:DE:10:00/36FrankeAq,8C:1F:64:E2:10:00/36LG-LHTAi,8C:1F:64:E4:30:00/36Daedalea,8C:1F:64:E5:20:00/36LcmVeloc,8C:1F:64:E5:C0:00/36Scientif,8C:1F:64:E7:30:00/36GTRIndus,8C:1F:64:E7:70:00/36Gy-FxSas,8C:1F:64:E9:80:00/36Luxshare,8C:1F:64:EA:A0:00/36KBModul,8C:1F:64:EA:C0:00/36MiracleH,8C:1F:64:EC:10:00/36Actronik,8C:1F:64:ED:40:00/36Zhejiang,8C:1F:64:EE:80:00/36GlobalOr,8C:1F:64:EE:F0:00/36AiUnion,8C:1F:64:EF:10:00/36BiotageG,8C:1F:64:F0:40:00/36IoTSecur,8C:1F:64:F3:F0:00/36Industri,8C:1F:64:F4:10:00/36Automati,8C:1F:64:F7:80:00/36TernaryR,8C:1F:64:F8:60:00/36INFOSTEC,8C:1F:64:F9:40:00/36EAElektr,8C:1F:64:F9:E0:00/36DREAMSWE,8C:1F:64:FB:10:00/36Abb,8C:1F:64:FB:D0:00/36SAN-AIEl,8C:1F:64:FD:10:00/36Edgeware,8C:1F:64:FE:00:00/36PotterEl,8C:1F:94RFSurgic,8C:21:0ATp-LinkT,8C:25:05HuaweiTe,8C:27:1DQuantHou,8C:27:8AVocollec,8C:29:37Apple,8C:2A:8EDongGuan,8C:2D:AAApple,8C:2F:39IBADosim,8C:2F:A6SolidOpt,8C:31:E2Dayouplu,8C:33:30EmFirst,8C:33:57HiteVisi,8C:34:01Guangdon,8C:34:46HuaweiDe,8C:34:FDHuaweiTe,8C:35:79QDIQOSp,8C:36:7APaloAlto,8C:39:5CBit4id,8C:3A:7EUniversa,8C:3A:E3LGElectr,8C:3B:32Microfan,8C:3B:ADNetgear,8C:3C:07SkivaTec,8C:3C:4ANAKAYO,8C:3D:B1BeijingH,8C:41:F2RDATechn,8C:41:F4IPmotion,8C:42:6DHuaweiTe,8C:43:61HailoDig,8C:44:35Shanghai,8C:44:4FHUMAX,8C:45:00MurataMa,8C:47:6EIEEERegi,8C:47:6E:00:00:00/28Chipsafe,8C:47:6E:10:00:00/28TelWare,8C:47:6E:20:00:00/28HuiZhouM,8C:47:6E:30:00:00/28Shanghai,8C:47:6E:40:00:00/28Shenzhen,8C:47:6E:50:00:00/28Square,8C:47:6E:60:00:00/28OxfordNa,8C:47:6E:70:00:00/28Private,8C:47:6E:80:00:00/28IntelliV,8C:47:6E:90:00:00/28Xertifie,8C:47:6E:A0:00:00/28AUOptron,8C:47:6E:B0:00:00/28FaravidC,8C:47:6E:C0:00:00/28EdgeNetw,8C:47:6E:D0:00:00/28innolect,8C:47:6E:E0:00:00/28Annapurn,8C:47:BEDell,8C:49:62Roku,8C:4A:EEGigaTms,8C:4B:14Espressi,8C:4B:593DImagin,8C:4C:ADEvoluzn,8C:4C:DCPlanexCo,8C:4D:B9Unmonday,8C:4D:EACerio,8C:51:05Shenzhen,8C:53:C3BeijingX,8C:53:D2ChinaMob,8C:53:F7A&DEngin,8C:54:1DLge,8C:55:4AIntelCor,8C:55:BBSongwooI,8C:56:46LGElectr,8C:56:9DImagingS,8C:56:C5Nintendo,8C:57:9BWistronN,8C:57:FDLVXWeste,8C:58:77Apple,8C:59:3CIEEERegi,8C:59:3C:00:00:00/28FujianCh,8C:59:3C:10:00:00/28FutureRo,8C:59:3C:20:00:00/28BeidaJad,8C:59:3C:30:00:00/28Chongqin,8C:59:3C:40:00:00/28Guralp,8C:59:3C:50:00:00/28Spectran,8C:59:3C:60:00:00/28QbicTech,8C:59:3C:70:00:00/28OBOPro2,8C:59:3C:80:00:00/28Nanonord,8C:59:3C:90:00:00/28Genis,8C:59:3C:A0:00:00/28ecominst,8C:59:3C:B0:00:00/28Scharfe-,8C:59:3C:C0:00:00/28Dantherm,8C:59:3C:D0:00:00/28Idro-Ele,8C:59:3C:E0:00:00/28Shenzhen,8C:59:73ZyxelCom,8C:59:8BCTechnol,8C:59:C3ADBItali,8C:59:DCASRMicro,8C:5A:25ARRISGro,8C:5A:C1HuaweiDe,8C:5A:F0Exeltech,8C:5A:F8BeijingX,8C:5B:F0ARRISGro,8C:5C:A1d-broad,8C:5D:60UCI,8C:5E:BDHuaweiDe,8C:5F:48Continen,8C:5F:ADFiberhom,8C:5F:DFBeijingR,8C:60:4FCisco,8C:60:78Swissbit,8C:60:E7Mpgio,8C:61:02BeijingB,8C:61:A3ARRISGro,8C:64:0BBeyondDe,8C:64:22Sony,8C:64:A2OnePlusT,8C:64:D4HyecoSma,8C:67:94vivoMobi,8C:68:3AHuaweiTe,8C:68:78Nortek-A,8C:68:C8zte,8C:6A:8DTechnico,8C:6A:E4Viogem,8C:6D:50Shenzhen,8C:6D:77HuaweiTe,8C:6D:C4Megapixe,8C:70:5AIntelCor,8C:70:86Gesellsc,8C:71:F8SamsungE,8C:73:6EFujitsu,8C:73:A0Fiberhom,8C:76:C1GodenTec,8C:77:12SamsungE,8C:77:16Longchee,8C:78:D7Shenzhen,8C:79:67zte,8C:79:F5SamsungE,8C:7A:15RuckusWi,8C:7A:3DXiaomiCo,8C:7A:AAApple,8C:7B:9DApple,8C:7B:F0XufengDe,8C:7C:92Apple,8C:7C:B5HonHaiPr,8C:7C:FFBrocadeC,8C:7E:B3Lytro,8C:7F:3BARRISGro,8C:81:26Arcom,8C:81:72SichuanT,8C:82:A8InsigmaT,8C:83:9DShenzhen,8C:83:DFNokia,8C:83:E1SamsungE,8C:83:E8HuaweiTe,8C:83:FCAxiomaMe,8C:84:01Private,8C:85:80SmartInn,8C:85:90Apple,8C:85:C1ArubaaHe,8C:85:E6Cleondri,8C:86:1EApple,8C:87:3BLeicaCam,8C:89:7AAugtek,8C:89:A5Micro-St,8C:89:FAZhejiang,8C:8A:6EEstunAut,8C:8A:BBBeijingO,8C:8B:83TexasIns,8C:8C:AALCFCHeFe,8C:8D:28IntelCor,8C:8E:76taskit,8C:8E:F2Apple,8C:8F:8BChinaMob,8C:8F:E9Apple,8C:90:D3Nokia,8C:91:09Toyoshim,8C:92:36AusLinxT,8C:92:46Oerlikon,8C:93:51Jigowatt,8C:94:1FCisco,8C:94:CCSfr,8C:94:CFEncellTe,8C:96:5FShandong,8C:97:EAFreeboxS,8C:99:E6TCTmobil,8C:9F:3BQingdaoH,8C:A0:48BeijingN,8C:A2:FDStarry,8C:A3:99Serverco,8C:A5:A1Oregano-,8C:A6:DFTp-LinkT,8C:A9:6FD&MHoldi,8C:A9:82IntelCor,8C:AA:B5Espressi,8C:AA:CEXiaomiCo,8C:AB:8EShanghai,8C:AE:49IEEERegi,8C:AE:49:00:00:00/28Ouman,8C:AE:49:10:00:00/28H3Platfo,8C:AE:49:20:00:00/28SEVERINE,8C:AE:49:30:00:00/28BertinTe,8C:AE:49:40:00:00/28JiangsuS,8C:AE:49:50:00:00/28GatiInfo,8C:AE:49:60:00:00/28ChengduB,8C:AE:49:70:00:00/28Precitec,8C:AE:49:80:00:00/28Taipit-M,8C:AE:49:90:00:00/28TTR,8C:AE:49:A0:00:00/28Gigawave,8C:AE:49:B0:00:00/28SuzhouGu,8C:AE:49:C0:00:00/28Parametr,8C:AE:49:D0:00:00/28LarchNet,8C:AE:49:E0:00:00/28Shenzhen,8C:AE:4CPlugable,8C:AE:89Y-camSol,8C:AE:DBNagtech,8C:B0:94AirtechI,8C:B0:E9SamsungE,8C:B6:4FCisco,8C:B7:F7Shenzhen,8C:B8:2CIPitomyC,8C:B8:4ASamsungE,8C:B8:64AcSiPTec,8C:B8:7EIntelCor,8C:BA:25Unionman,8C:BE:24TashangS,8C:BE:BEXiaomiCo,8C:BF:9DShanghai,8C:BF:A6SamsungE,8C:C1:21Panasoni,8C:C5:B4Sagemcom,8C:C5:E1ShenZhen,8C:C6:61Currentp,8C:C6:81IntelCor,8C:C7:AARadinetC,8C:C7:D0zhejiang,8C:C8:4BChongqin,8C:C8:CDSamsungE,8C:C8:F4IEEERegi,8C:C8:F4:00:00:00/28Guardtec,8C:C8:F4:10:00:00/28Lanhomex,8C:C8:F4:20:00:00/28DarkHors,8C:C8:F4:30:00:00/28TohoDenk,8C:C8:F4:40:00:00/28ITECHEle,8C:C8:F4:50:00:00/28BeijingK,8C:C8:F4:60:00:00/28SHENZHEN,8C:C8:F4:70:00:00/28Private,8C:C8:F4:80:00:00/28Strongby,8C:C8:F4:90:00:00/28SwiftNav,8C:C8:F4:A0:00:00/28TriluxGr,8C:C8:F4:B0:00:00/28PTYPE,8C:C8:F4:C0:00:00/28Shenzhen,8C:C8:F4:D0:00:00/28BeijingX,8C:C8:F4:E0:00:00/28Evaporco,8C:CD:A2ACTP,8C:CD:E8Nintendo,8C:CE:4EEspressi,8C:CE:FDShenzhen,8C:CF:09DellEMC,8C:CF:5CBEFEGA,8C:CF:8FITC,8C:D1:7BCGMobile,8C:D2:E9YokoteSe,8C:D3:A2VisSimAS,8C:D4:8EItelMobi,8C:D6:28IkorMete,8C:D6:7FEMMicroe,8C:D9:D6XiaomiCo,8C:DB:25ESGSolut,8C:DC:02zte,8C:DC:D4HewlettP,8C:DD:8DWifly-Ci,8C:DE:52ISSCTech,8C:DE:99Comlab,8C:DE:E6SamsungE,8C:DE:F9BeijingX,8C:DF:9DNEC,8C:E0:81zte,8C:E1:17zte,8C:E2:DACircleMe,8C:E3:8EKioxia,8C:E4:68Guangzho,8C:E5:C0SamsungE,8C:E5:EFHuaweiTe,8C:E7:48Hangzhou,8C:E7:8CDKNetwor,8C:E7:B3Sonardyn,8C:E9:B4Zhejiang,8C:EA:12Shenzhen,8C:EA:1BEdgecore,8C:EA:48SamsungE,8C:EB:C6HuaweiTe,8C:EC:4BDell,8C:EC:7BApple,8C:EE:C6Precepsc,8C:F1:12Motorola,8C:F2:28MercuryC,8C:F3:19SiemensI,8C:F5:A3SamsungE,8C:F6:81SiliconL,8C:F7:10AMPAKTec,8C:F7:73Nokia,8C:F8:13OrangePo,8C:F9:45PowerAut,8C:F9:57RuiXingH,8C:F9:C9MESADATe,8C:FA:BAApple,8C:FC:A0Shenzhen,8C:FD:15ImagineM,8C:FD:18HuaweiTe,8C:FD:DESagemcom,8C:FD:F0Qualcomm,8C:FE:57Apple,8C:FE:74RuckusWi,8C:FE:B4Vsoontec,90:00:4EHonHaiPr,90:00:DBSamsungE,90:01:3BSagemcom,90:02:18BSkyB,90:02:7AShenzhen,90:02:8AShenzhen,90:02:A9Zhejiang,90:03:25HuaweiTe,90:03:72LongnanJ,90:03:B7Parrot,90:06:28SamsungE,90:09:17Far-sigh,90:09:D0Synology,90:0A:1ATaicangT,90:0A:39Wiio,90:0A:3APSGPlast,90:0A:84Mellanox,90:0B:C1Sprocomm,90:0C:B4AlinketE,90:0C:C8Google,90:0D:66Digimore,90:0D:CBARRISGro,90:0E:83MonicoMo,90:0E:B3Shenzhen,90:0F:0CCloudNet,90:12:34Shenzhen,90:12:A1We,90:13:DAAthom,90:16:BAHuaweiTe,90:17:11HagenukM,90:17:3FHuaweiTe,90:17:9BNanomega,90:17:ACHuaweiTe,90:17:C8HuaweiTe,90:18:5EApexTool,90:18:7CSamsungE,90:18:AEShanghai,90:19:00Scs,90:1A:4FEMMicroe,90:1A:CAARRISGro,90:1B:0EFujitsuT,90:1D:27zte,90:1E:DDGreatCom,90:20:3ABYDPreci,90:20:83GeneralE,90:20:C2ArubaaHe,90:21:06BSkyB,90:21:55HTC,90:21:81Shanghai,90:23:B4NewH3CTe,90:23:ECAvailink,90:27:2BAlgorab,90:27:59NanjingJ,90:27:E4Apple,90:2B:34Giga-Byt,90:2B:D2HuaweiTe,90:2C:C7C-MAXAsi,90:2E:16LCFCHeFe,90:2E:1CIntelCor,90:2E:87LabJack,90:31:CDOnyxHeal,90:32:4BHonHaiPr,90:34:2BGatekeep,90:34:FCHonHaiPr,90:35:6EVodafone,90:35:EASiliconL,90:38:09Ericsson,90:38:0CEspressi,90:38:DFChangzho,90:3A:72RuckusWi,90:3A:A0Nokia,90:3A:E6Parrot,90:3C:92Apple,90:3C:AEYunnanKS,90:3C:B3Edgecore,90:3D:5AShenzhen,90:3D:68G-Printe,90:3D:6BZiconTec,90:3D:BDSecureMe,90:3E:7FFiberhom,90:3E:ABARRISGro,90:3F:EAHuaweiTe,90:43:E2Cornami,90:45:06TokyoBoe,90:46:A2TedipayU,90:46:B7VadaroPt,90:47:16Rorze,90:47:3CChinaMob,90:48:9AHonHaiPr,90:49:FAIntelCor,90:4C:81HewlettP,90:4C:E5HonHaiPr,90:4D:4ASagemcom,90:4D:C3Flonidan,90:4E:2BHuaweiTe,90:4E:91IEEERegi,90:4E:91:00:00:00/28Spirtech,90:4E:91:10:00:00/28ApolloVi,90:4E:91:20:00:00/28NorthPol,90:4E:91:30:00:00/28Teleepoc,90:4E:91:40:00:00/28Wrtnodet,90:4E:91:50:00:00/28mcf88,90:4E:91:60:00:00/28NuwaRobo,90:4E:91:70:00:00/28Ibm,90:4E:91:80:00:00/28CommandS,90:4E:91:90:00:00/28CUTTERsp,90:4E:91:A0:00:00/28Kaertech,90:4E:91:B0:00:00/28Shanghai,90:4E:91:C0:00:00/28Showtacl,90:4E:91:D0:00:00/28SKODAELE,90:4E:91:E0:00:00/28Shenzhen,90:50:5AunGlue,90:50:7BAdvanced,90:50:CAHitronTe,90:51:3FElettron,90:54:46TesElect,90:55:AEEricsson,90:55:DEFiberhom,90:56:82Lenbrook,90:56:92Autotalk,90:56:FCTecnoMob,90:58:51Technico,90:59:3CAz-Techn,90:59:AFTexasIns,90:5C:34SiriusEl,90:5C:44CompalBr,90:5D:7CNewH3CTe,90:5F:2ETCTmobil,90:5F:8Dmodas,90:60:F1Apple,90:61:0CFidaInte,90:61:AEIntelCor,90:63:3BSamsungE,90:67:17AlphionI,90:67:1CHuaweiTe,90:67:B5Alcatel-,90:67:F3AlcatelL,90:68:C3Motorola,90:69:76Withrobo,90:6A:94hangzhou,90:6C:ACFortinet,90:6D:05BxbElect,90:6D:C8DLGAutom,90:6E:BBHonHaiPr,90:6F:18Private,90:6F:A9NanjingP,90:70:25GareaMic,90:70:65TexasIns,90:72:40Apple,90:72:82Sagemcom,90:73:5AMotorola,90:74:9DIRayTech,90:76:9FShenzhen,90:77:EECisco,90:78:41IntelCor,90:78:B2XiaomiCo,90:79:10Integrat,90:79:90Benchmar,90:7A:0AGebrBode,90:7A:28BeijingM,90:7A:58Zegna-Da,90:7A:F1Wally,90:7E:30Lars,90:7E:BAUtekTech,90:7F:61ChiconyE,90:80:60Nilfisk,90:80:8FHuaweiDe,90:81:2AApple,90:81:58Apple,90:81:75SamsungE,90:82:60IEEE1904,90:83:4BBeijingY,90:83:7AGeneralE,90:84:0DApple,90:84:2BLEGOSyst,90:84:8BHDR10+Te,90:86:74SichuanT,90:86:9Bzte,90:88:A2IonicsTe,90:89:5FWeifangG,90:8C:09TotalPha,90:8C:43Apple,90:8C:44HKZongmu,90:8C:63GZWeedon,90:8D:1DGHTechno,90:8D:6CApple,90:8D:6EDell,90:8D:78D-LinkIn,90:8F:CFUNOSyste,90:90:3CTrisonTe,90:90:60RsiVideo,90:91:64ChongQin,90:92:B4DiehlBGT,90:94:0AAnalogDe,90:94:97HuaweiTe,90:94:E4D-LinkIn,90:96:F3Buffalo,90:97:D5Espressi,90:97:F3SamsungE,90:98:38HuaweiDe,90:98:64Impex-Sa,90:99:16ELVEESNe,90:9A:4ATp-LinkT,90:9A:77TexasIns,90:9C:4AApple,90:9D:7DARRISGro,90:9D:E0NewlandD,90:9F:33EFMNetwo,90:9F:43Accutron,90:A1:37BeijingS,90:A2:10UnitedTe,90:A2:5BApple,90:A2:DAGheo,90:A3:65HMDGloba,90:A4:6ASisnet,90:A4:DEWistronN,90:A5:AFHuaweiTe,90:A6:2FNaver,90:A7:83JswPacif,90:A7:C1PakedgeD,90:A8:22AmazonTe,90:A9:35JWEntert,90:AA:C3HitronTe,90:AC:3FBrightSi,90:AD:F7vivoMobi,90:AD:FCTelechip,90:AE:1BTp-LinkT,90:AF:D1netKTI,90:B0:EDApple,90:B1:1CDell,90:B1:34ARRISGro,90:B1:44SamsungE,90:B1:E0BeijingN,90:B2:1FApple,90:B4:DDPrivate,90:B5:7FShenzhen,90:B6:22SamsungE,90:B6:7AShenzhen,90:B6:86MurataMa,90:B8:32ExtremeN,90:B8:D0Joyent,90:B8:E0Shenzhen,90:B9:31Apple,90:B9:7DJohnsonO,90:BD:E6QuectelW,90:C1:15Sony,90:C1:19Nokia,90:C1:C6Apple,90:C3:5FNanjingJ,90:C5:4AvivoMobi,90:C6:82IEEERegi,90:C6:82:00:00:00/28Shenzhen,90:C6:82:10:00:00/28Shenzhen,90:C6:82:20:00:00/28ekeybiom,90:C6:82:30:00:00/28Innovati,90:C6:82:40:00:00/28Neone,90:C6:82:50:00:00/28ET,90:C6:82:60:00:00/28NanjingJ,90:C6:82:70:00:00/28Cinet,90:C6:82:80:00:00/28TeletekE,90:C6:82:90:00:00/28Act,90:C6:82:A0:00:00/28BeijingA,90:C6:82:B0:00:00/28Lachmann,90:C6:82:C0:00:00/28LiSengTe,90:C6:82:D0:00:00/28PowerShi,90:C6:82:E0:00:00/28Shanghai,90:C6:82:F0:00:00/28Private,90:C7:92ARRISGro,90:C7:D8zte,90:C9:9BTesorion,90:CC:24Synaptic,90:CC:DFIntelCor,90:CD:B6HonHaiPr,90:CF:15Nokia,90:CF:6FDlogixs,90:CF:7DQingdaoH,90:D1:1BPalomarM,90:D7:4FBookeen,90:D7:BEWavelabG,90:D7:EBTexasIns,90:D8:52Comtec,90:D8:F3zte,90:D9:2CHug-Wits,90:DA:4EAvanu,90:DA:6AFOCUSH&S,90:DB:46E-LeadEl,90:DD:5DApple,90:DE:80Shenzhen,90:DF:B7smssmart,90:DF:FBHomeride,90:E0:F0IEEE1722,90:E1:7BApple,90:E2:02TexasIns,90:E2:BAIntelCor,90:E2:FCIEEERegi,90:E2:FC:00:00:00/28ParsErte,90:E2:FC:10:00:00/28Yitetech,90:E2:FC:20:00:00/28ShenZhen,90:E2:FC:30:00:00/28Shenzhen,90:E2:FC:40:00:00/28Dongguan,90:E2:FC:50:00:00/28Totalone,90:E2:FC:60:00:00/28SindohTe,90:E2:FC:70:00:00/28FairWind,90:E2:FC:80:00:00/28bitsensi,90:E2:FC:90:00:00/28HuddlyAS,90:E2:FC:A0:00:00/28PowerEng,90:E2:FC:B0:00:00/28Shenzhen,90:E2:FC:C0:00:00/28StanleyS,90:E2:FC:D0:00:00/28BeijingL,90:E2:FC:E0:00:00/28DevComsp,90:E6:BAASUSTekC,90:E7:10NewH3CTe,90:E7:C4HTC,90:E8:68AzureWav,90:EA:60SPILaser,90:EB:48Shanghai,90:EC:50COBO,90:EC:77silicom,90:EE:C7SamsungE,90:EE:D9Universa,90:EF:68ZyxelCom,90:F0:52MEIZUTec,90:F1:57GarminIn,90:F1:AASamsungE,90:F1:B0Hangzhou,90:F2:78RadiusGa,90:F3:05HUMAX,90:F3:B7KirisunC,90:F3:B8ChinaMob,90:F4:C1RandMcNa,90:F6:44HuaweiDe,90:F6:52Tp-LinkT,90:F7:2FPhillips,90:F8:91Kaonmedi,90:F9:B7HuaweiTe,90:FB:5BAvaya,90:FB:A6HonHaiPr,90:FD:61Apple,90:FD:73zte,90:FD:9FSiliconL,90:FF:79MetroEth,90:FF:D6HonorDev,94:00:06jinyoung,94:00:70Nokia,94:00:B0HuaweiTe,94:01:49AutoHotB,94:01:C2SamsungE,94:02:30Logitech,94:02:6BOptictim,94:04:9CHuaweiTe,94:05:B6LilingFu,94:05:BBIEEERegi,94:05:BB:00:00:00/28QingdaoM,94:05:BB:10:00:00/28Dongguan,94:05:BB:20:00:00/28Dongguan,94:05:BB:30:00:00/28Neutrik,94:05:BB:40:00:00/28Shenzhen,94:05:BB:50:00:00/28ChengduZ,94:05:BB:60:00:00/28ZIGPOS,94:05:BB:70:00:00/28closip,94:05:BB:80:00:00/28iungo,94:05:BB:90:00:00/28Zimmer,94:05:BB:A0:00:00/28SolarEdg,94:05:BB:B0:00:00/28AustarHe,94:05:BB:C0:00:00/28LaoIndus,94:05:BB:D0:00:00/28Sunthink,94:05:BB:E0:00:00/28BAE,94:08:53LiteonTe,94:08:C7HuaweiDe,94:09:37HUMAX,94:09:D3shenzhen,94:0B:19HuaweiTe,94:0B:2DNetViewT,94:0B:D5HimaxTec,94:0C:6DTp-LinkT,94:0C:98Apple,94:0E:6BHuaweiTe,94:10:3EBelkinIn,94:11:DAITFFrösc,94:14:57Shenzhen,94:14:7AvivoMobi,94:16:25Apple,94:16:73PointCor,94:17:00XiaomiCo,94:18:82HewlettP,94:19:3AElvaco,94:1C:56Actionte,94:1D:1CTLabWest,94:1F:3AAmbiq,94:20:53Nokia,94:21:97Stalmart,94:23:6EShenzhen,94:24:B8GreeElec,94:24:E1Alcatel-,94:25:33HuaweiTe,94:27:90TCTmobil,94:28:2ENewH3CTe,94:29:0CShenyang,94:29:2FNewH3CTe,94:29:57AirpoNet,94:29:8DShanghai,94:2A:3FDiversey,94:2C:B3HUMAX,94:2D:DCSamsungE,94:2E:17Schneide,94:2E:63Finsécur,94:31:9BAlphatro,94:31:CBvivoMobi,94:33:DDTaco,94:35:0ASamsungE,94:36:E0SichuanB,94:37:F7HuaweiDe,94:39:E5HonHaiPr,94:3A:91AmazonTe,94:3A:F0Nokia,94:3B:B0NewH3CTe,94:3B:B1Kaonmedi,94:3C:C6Espressi,94:3D:C9AsahiNet,94:3F:C2HewlettP,94:40:A2AnywaveC,94:40:C9HewlettP,94:41:C1Mini-Cam,94:43:4DCiena,94:44:44LGInnote,94:44:52BelkinIn,94:46:96BaudTec,94:47:B0BeijingE,94:49:96WiSilica,94:4A:09BitWiseC,94:4A:0CSercomm,94:4F:4CSoundUni,94:50:47Rechnerb,94:50:89SimonsVo,94:51:03SamsungE,94:51:3DiSmartAl,94:51:BFHyundaiE,94:53:30HonHaiPr,94:54:93Rigado,94:54:CEGuangdon,94:54:DFYst,94:56:41PaloAlto,94:57:A5HewlettP,94:58:CBNintendo,94:59:07Shanghai,94:59:2DEKEBuild,94:5A:FCAmazonTe,94:5B:7ETrilobit,94:5C:9AApple,94:5F:34RenesasE,94:60:10HuaweiDe,94:60:D5ArubaaHe,94:61:1EWataElec,94:61:24Pason,94:62:69ARRISGro,94:63:72vivoMobi,94:63:D1SamsungE,94:64:24ArubaaHe,94:65:2DOnePlusT,94:65:9CIntelCor,94:66:E7WOMEngin,94:67:7EBeldenIn,94:6A:77Technico,94:6A:B0Arcadyan,94:70:D2WinfirmT,94:71:ACTCTmobil,94:75:6EQinetiQN,94:76:B7SamsungE,94:77:2BHuaweiTe,94:7B:BEUbicquia,94:7B:E7SamsungE,94:7C:3EPolewall,94:7E:B9National,94:7F:1DShenzhen,94:81:A4AzurayTe,94:83:C4GLTechno,94:85:7AEvantage,94:86:CDSeoulEle,94:86:D4Surveill,94:87:7CARRISGro,94:87:E0XiaomiCo,94:88:15Infiniqu,94:88:54TexasIns,94:88:5ESurfilte,94:8A:C6RealmeCh,94:8B:03EAGETInn,94:8B:C1SamsungE,94:8D:50Beamex,94:8D:EFOetikerS,94:8E:89Industri,94:8E:D3AristaNe,94:8F:CFARRISGro,94:8F:EEVerizonT,94:90:10HuaweiTe,94:90:34Shenzhen,94:91:7FAskeyCom,94:92:BCSyntechH,94:92:D2KCFTechn,94:94:26Apple,94:95:A0Google,94:98:69zte,94:98:A2Shanghai,94:99:01Shenzhen,94:99:90VTCTelec,94:9A:A9Microsof,94:9B:2CExtremeN,94:9B:FDTransNew,94:9C:55AltaData,94:9D:57Panasoni,94:9F:3ESonos,94:9F:3FOptekDig,94:9F:B4ChengDuJ,94:A0:4EBostexTe,94:A1:A2AMPAKTec,94:A3:CAKonnectO,94:A4:08Shenzhen,94:A4:0CDiehlMet,94:A4:F9HuaweiTe,94:A6:7ENetgear,94:A7:B7zte,94:A7:BCBodyMedi,94:A9:A8TexasIns,94:AA:0AFiberhom,94:AA:B8JoviewBe,94:AB:DEOMXTechn,94:AC:CAtrivumte,94:AE:61AlcatelL,94:AE:E3BeldenHi,94:AE:F0Cisco,94:B0:1FApple,94:B1:0ASamsungE,94:B2:71HuaweiTe,94:B2:CCPioneer,94:B3:4FRuckusWi,94:B4:0FArubaaHe,94:B8:19Nokia,94:B8:6DIntelCor,94:B8:C5RuggedCo,94:B9:7EEspressi,94:B9:B4AptosTec,94:BA:31Visionte,94:BA:56Shenzhen,94:BB:AEHusqvarn,94:BE:46Motorola,94:BF:1EeflowSma,94:BF:2DApple,94:BF:80zte,94:BF:94JuniperN,94:BF:95Shenzhen,94:BF:C4RuckusWi,94:C0:14SorterSp,94:C0:38TallacNe,94:C1:502Wire,94:C2:BDTecnobit,94:C3:E4AtlasCop,94:C4:E9PowerLay,94:C6:91EliteGro,94:C6:EBNOVAelec,94:C7:AFRayliosT,94:C9:60Zhongsha,94:C9:62Teseq,94:CA:0FHoneywel,94:CC:04IEEERegi,94:CC:04:00:00:00/28Hangzhou,94:CC:04:10:00:00/28Gocoax,94:CC:04:20:00:00/28NanjingY,94:CC:04:30:00:00/28Shenzhen,94:CC:04:40:00:00/28ProConne,94:CC:04:50:00:00/28Shenzhen,94:CC:04:60:00:00/28SamNazar,94:CC:04:70:00:00/28GowingBu,94:CC:04:80:00:00/28CircuitW,94:CC:04:90:00:00/28ENTECEle,94:CC:04:A0:00:00/28hyBee,94:CC:04:B0:00:00/28Shandong,94:CC:04:C0:00:00/28ShanxiBa,94:CC:04:D0:00:00/28HanzhuoI,94:CC:04:E0:00:00/28Synchron,94:CC:B9ARRISGro,94:CD:ACCreowave,94:CE:2CSony,94:CE:31CTS,94:D0:0DHuaweiTe,94:D0:19Cydle,94:D0:29Guangdon,94:D0:75CISCrypt,94:D2:99Techmati,94:D4:17GpiKorea,94:D4:69Cisco,94:D5:05Fiberhom,94:D6:0Eshenzhen,94:D6:DBNexFi,94:D7:23Shanghai,94:D7:71SamsungE,94:D8:59TCTmobil,94:D9:3CEnelps,94:D9:B3Tp-LinkT,94:DB:49Sitcorp,94:DB:56SonyHome,94:DB:C9AzureWav,94:DB:DAHuaweiTe,94:DC:4EAEVspols,94:DD:3FA+VLinkT,94:DE:0ESmartOpt,94:DE:80Giga-Byt,94:DF:4EWistronI,94:DF:58IJElectr,94:E0:D0HealthSt,94:E0:D6ChinaDra,94:E1:29SamsungE,94:E1:ACHangzhou,94:E2:26DORtizCo,94:E2:3CIntelCor,94:E2:FDBogeKomp,94:E3:6DTexasIns,94:E3:EEzte,94:E4:BAHuaweiDe,94:E6:F7IntelCor,94:E7:0BIntelCor,94:E7:11XirkaDam,94:E7:EAHuaweiTe,94:E8:48FyldeMic,94:E8:C5ARRISGro,94:E9:6AApple,94:E9:79LiteonTe,94:E9:8CNokia,94:E9:EEHuaweiDe,94:EA:32Apple,94:EA:EATellesco,94:EB:2CGoogle,94:EB:CDBlackBer,94:EE:9FHMDGloba,94:F1:28HewlettP,94:F1:9EHuizhouM,94:F2:78ElmaElec,94:F2:BBValeoVis,94:F5:51CadiScie,94:F6:65RuckusWi,94:F6:92Geminico,94:F6:A3Apple,94:F6:D6Apple,94:F7:20TianjinD,94:F7:ADJuniperN,94:FA:E8Shenzhen,94:FB:29ZebraTec,94:FB:A7IEEERegi,94:FB:A7:00:00:00/28Reichert,94:FB:A7:10:00:00/28InaxsysS,94:FB:A7:20:00:00/28BeijingL,94:FB:A7:30:00:00/28GuangDon,94:FB:A7:40:00:00/28UoiTechn,94:FB:A7:50:00:00/28CavityEy,94:FB:A7:60:00:00/28Sercomm,94:FB:A7:70:00:00/28AnvilGro,94:FB:A7:80:00:00/28Silver-I,94:FB:A7:90:00:00/28Shanghai,94:FB:A7:A0:00:00/28Elkron,94:FB:A7:B0:00:00/28Shenzhen,94:FB:A7:C0:00:00/28Solabora,94:FB:A7:D0:00:00/28Rosenber,94:FB:A7:E0:00:00/28SkyringS,94:FB:B2Shenzhen,94:FD:1DWhereWhe,94:FD:2EShanghai,94:FE:22HuaweiTe,94:FE:9DShenzhen,94:FE:F4Sagemcom,94:FF:3CFortinet,94:FF:61ChinaMob,98:00:6Azte,98:00:74Raisecom,98:00:C1GuangZho,98:00:C6Apple,98:01:A7Apple,98:02:84Theobrom,98:02:D8IEEERegi,98:02:D8:00:00:00/28Stoerk-T,98:02:D8:10:00:00/28Shenzhen,98:02:D8:20:00:00/28UnitedPo,98:02:D8:30:00:00/28GrammerE,98:02:D8:40:00:00/28Zedi,98:02:D8:50:00:00/28EBI,98:02:D8:60:00:00/28FritzKue,98:02:D8:70:00:00/28Ormazaba,98:02:D8:80:00:00/28SimploTe,98:02:D8:90:00:00/28NavroomB,98:02:D8:A0:00:00/28HySecuri,98:02:D8:B0:00:00/28HanshinM,98:02:D8:C0:00:00/28AGV,98:02:D8:D0:00:00/28Promicon,98:02:D8:E0:00:00/28Private,98:02:D8:F0:00:00/28Private,98:03:9BMellanox,98:03:A0ABBPower,98:03:D8Apple,98:06:37IEEERegi,98:06:37:00:00:00/28Zoleo,98:06:37:10:00:00/28EPSchlum,98:06:37:20:00:00/28Summa,98:06:37:30:00:00/28Hangzhou,98:06:37:40:00:00/28ChengduS,98:06:37:50:00:00/28GsGlobal,98:06:37:60:00:00/28BoeingSs,98:06:37:70:00:00/28Samwonte,98:06:37:80:00:00/28Shenzhen,98:06:37:90:00:00/28NAB,98:06:37:A0:00:00/28AngoraNe,98:06:37:B0:00:00/28Petersim,98:06:37:C0:00:00/28HwaCom,98:06:37:D0:00:00/28VRTechno,98:06:37:E0:00:00/28Shanghai,98:06:3AHomeCont,98:06:3CSamsungE,98:07:2DTexasIns,98:09:CFOnePlusT,98:0C:82SamsungE,98:0C:A5Motorola,98:0D:2EHTC,98:0D:51HuaweiDe,98:0D:67ZyxelCom,98:0E:24PhytiumT,98:0E:E4Private,98:10:82Nsolutio,98:10:94Shenzhen,98:10:E8Apple,98:13:33zte,98:14:D2Avonic,98:16:ECICIntrac,98:18:88CiscoMer,98:19:2CEdgecore,98:1A:35HuaweiTe,98:1B:B5ASSAABLO,98:1D:FASamsungE,98:1E:0FJeelanSh,98:1E:19Sagemcom,98:1F:B1Shenzhen,98:20:8EDefinium,98:22:EFLiteonTe,98:23:4EMicromed,98:26:2AAppliedR,98:27:82IEEERegi,98:27:82:00:00:00/28Shenzhen,98:27:82:10:00:00/28INFODAS,98:27:82:20:00:00/28AnhuiShe,98:27:82:30:00:00/28DanfossP,98:27:82:40:00:00/28DspreadT,98:27:82:50:00:00/28Guangzho,98:27:82:60:00:00/28WesternS,98:27:82:70:00:00/28Kortek,98:27:82:80:00:00/28CATSPowe,98:27:82:90:00:00/28WuxiGuoY,98:27:82:A0:00:00/28NanjingB,98:27:82:B0:00:00/28RayTron,98:27:82:C0:00:00/28KRISTECH,98:27:82:D0:00:00/28Thorlabs,98:27:82:E0:00:00/28SureFlap,98:28:A6CompalIn,98:29:1DJaguarde,98:29:3FFujianSt,98:29:A6CompalIn,98:2C:BCIntelCor,98:2C:BE2Wire,98:2D:56Resoluti,98:2D:68SamsungE,98:2D:BAFibergat,98:2F:3CSichuanC,98:2F:F8HuaweiDe,98:30:00BeijingK,98:30:71Daikyung,98:34:9DKraussMa,98:35:71Sub10,98:35:B8Assemble,98:35:EDHuaweiTe,98:37:13PTNavico,98:38:7DItronicT,98:39:8ESamsungE,98:3B:16AMPAKTec,98:3B:67DWnetTec,98:3B:8FIntelCor,98:3F:60HuaweiTe,98:3F:66WuhanFun,98:3F:9FChinaSSJ,98:40:BBDell,98:41:5CNintendo,98:42:46SolIndus,98:42:65Sagemcom,98:43:DAIntertec,98:43:FAIntelCor,98:44:B6Infranor,98:44:CEHuaweiTe,98:45:62Shanghai,98:46:0AApple,98:47:3CShanghai,98:48:27Tp-LinkT,98:48:74HuaweiTe,98:49:14WistronN,98:49:9FDomoTact,98:49:E1BoeingDe,98:4A:47CHGHospi,98:4B:4AARRISGro,98:4B:E1HewlettP,98:4C:04Zhangzho,98:4C:D3MantisDe,98:4E:97Starligh,98:4F:EEIntelCor,98:50:2EApple,98:52:3DSunitecE,98:52:4ATechnico,98:52:B1SamsungE,98:54:1BIntelCor,98:57:D3HonHai-C,98:58:8ASYSGRATI,98:59:45TexasIns,98:59:49Luxottic,98:5A:EBApple,98:5B:B0Kmdata,98:5C:93SBGSAS,98:5D:46PeopleNe,98:5D:82AristaNe,98:5D:ADTexasIns,98:5E:1BConversD,98:5F:D3Microsof,98:60:22EMW,98:60:CAApple,98:66:EAIndustri,98:6B:3DARRISGro,98:6C:5CJiangxiG,98:6C:F5zte,98:6D:35IEEERegi,98:6D:35:00:00:00/28Shenzhen,98:6D:35:10:00:00/28Shenzhen,98:6D:35:20:00:00/28Shenzhen,98:6D:35:30:00:00/28DHMechat,98:6D:35:40:00:00/28blossomc,98:6D:35:50:00:00/28Pdahl,98:6D:35:60:00:00/28Vitronic,98:6D:35:70:00:00/28Zhejiang,98:6D:35:80:00:00/28Beijing3,98:6D:35:90:00:00/28Advanced,98:6D:35:A0:00:00/28iWaveJap,98:6D:35:B0:00:00/28Intech,98:6D:35:C0:00:00/28my-PV,98:6D:35:D0:00:00/28Praeside,98:6D:35:E0:00:00/28BaycomOp,98:6D:C8ToshibaM,98:6E:E8IEEERegi,98:6E:E8:00:00:00/28SbarcoTe,98:6E:E8:10:00:00/28Shanghai,98:6E:E8:20:00:00/28UgreenGr,98:6E:E8:30:00:00/28ReeR,98:6E:E8:40:00:00/28Fujitsuc,98:6E:E8:50:00:00/28SuzhouAu,98:6E:E8:60:00:00/28BlairCom,98:6E:E8:70:00:00/28Centrode,98:6E:E8:80:00:00/28Sisgeo,98:6E:E8:90:00:00/28SpanIO,98:6E:E8:A0:00:00/28LogosPay,98:6E:E8:B0:00:00/28Private,98:6E:E8:C0:00:00/28Sercomm,98:6E:E8:D0:00:00/28Changzho,98:6E:E8:E0:00:00/28FirstDes,98:6F:60Guangdon,98:70:E8Innatech,98:73:C4SageElec,98:74:3DShenzhen,98:74:DAInfinixm,98:75:1AHuaweiDe,98:76:B6Adafruit,98:77:70PepDigit,98:77:CBVorteksE,98:77:E7Kaonmedi,98:7A:10Ericsson,98:7A:14Microsof,98:7B:F3TexasIns,98:7D:DDChinaMob,98:7E:46EmizonNe,98:7E:CAInventus,98:7E:E3vivoMobi,98:80:BBIEEERegi,98:80:BB:00:00:00/28RYEEXTec,98:80:BB:10:00:00/28GreatWal,98:80:BB:20:00:00/28Shanghai,98:80:BB:30:00:00/28Annapurn,98:80:BB:40:00:00/28Shenzhen,98:80:BB:50:00:00/28MelexisT,98:80:BB:60:00:00/28NeusoftR,98:80:BB:70:00:00/28Guangdon,98:80:BB:80:00:00/28JyhEngTe,98:80:BB:90:00:00/28Shenzhen,98:80:BB:A0:00:00/28Guangzho,98:80:BB:B0:00:00/28Hilo,98:80:BB:C0:00:00/28Shenzhen,98:80:BB:D0:00:00/28Wyebot,98:80:BB:E0:00:00/28DMedTech,98:80:EESamsungE,98:82:17Disrupti,98:83:89SamsungE,98:84:E3TexasIns,98:86:5DNokiaSha,98:86:B1Flyaudio,98:87:44WuxiHong,98:89:EDAnademIn,98:8B:0AHangzhou,98:8B:5DSagemcom,98:8B:69Shenzhen,98:8B:ADCorintec,98:8D:46IntelCor,98:8E:34Zhejiang,98:8E:4ANoxusBei,98:8E:79Qudelix,98:8E:D4ItelMobi,98:8E:DDTEConnec,98:90:80Linkpowe,98:90:96Dell,98:93:CCLgElectr,98:94:49Skyworth,98:97:CCTp-LinkT,98:97:D1MitraSta,98:9A:B9zte,98:9B:CBAVMAudio,98:9C:57HuaweiTe,98:9D:5DTechnico,98:9E:63Apple,98:A4:04Ericsson,98:A4:0ESnap,98:A7:B0Mcst,98:A9:42Guangzho,98:AA:3CWilli-te,98:AA:D7BlueWave,98:AA:FCIEEERegi,98:AA:FC:00:00:00/28DalianEa,98:AA:FC:10:00:00/28Surtec,98:AA:FC:20:00:00/28Shenzhen,98:AA:FC:30:00:00/28NexusEle,98:AA:FC:40:00:00/28RpeRadic,98:AA:FC:50:00:00/28SPMInstr,98:AA:FC:60:00:00/28Mekotron,98:AA:FC:70:00:00/28Shenzhen,98:AA:FC:80:00:00/28BeijingT,98:AA:FC:90:00:00/28BEAMAuth,98:AA:FC:A0:00:00/28SENKO,98:AA:FC:B0:00:00/28Resonant,98:AA:FC:C0:00:00/28dots,98:AA:FC:D0:00:00/28MCSMicro,98:AA:FC:E0:00:00/28Comarch,98:AD:1DHuaweiDe,98:AE:71VVDNTech,98:AF:65IntelCor,98:B0:39Nokia,98:B1:77Landis+G,98:B3:EFHuaweiDe,98:B6:E9Nintendo,98:B8:BALGElectr,98:B8:BCSamsungE,98:B8:E3Apple,98:BA:39Doro,98:BB:1EBYDPreci,98:BB:99PhicommS,98:BC:57SvaTechn,98:BC:99Edeltech,98:BE:94Ibm,98:BE:DCHonorDev,98:C0:EBGlobalRe,98:C3:D2NingboSa,98:C5:DBEricsson,98:C7:A4Shenzhen,98:C8:45PacketAc,98:C8:54ChiunMai,98:C8:B8vivoMobi,98:C9:7CShenzhen,98:CA:33Apple,98:CB:27GaloreNe,98:CB:A4Benchmar,98:CC:4DShenzhen,98:CD:ACEspressi,98:CD:B4Virident,98:CF:53BbkEduca,98:D2:93Google,98:D3:31Shenzhen,98:D3:D2MEKRALan,98:D3:E7NetafimL,98:D6:86ChyiLeei,98:D6:BBApple,98:D6:F7LGElectr,98:D8:63Shanghai,98:D8:8CNortelNe,98:DA:92Vuzix,98:DA:C4Tp-LinkT,98:DC:D9UNITEC,98:DD:5BTakumiJa,98:DD:60Apple,98:DD:EAInfinixm,98:DE:D0Tp-LinkT,98:DF:82Hangzhou,98:E0:D9Apple,98:E1:65Accutome,98:E4:76Zentan,98:E7:43Dell,98:E7:9AFoxconnN,98:E7:F4HewlettP,98:E7:F5HuaweiTe,98:E8:48Axiim,98:E8:FANintendo,98:EC:65CosesyAp,98:ED:5CTesla,98:ED:7Eeero,98:EE:CBWistronI,98:EF:9BOhsung,98:F0:58Lynxspri,98:F0:7BTexasIns,98:F0:83HuaweiTe,98:F0:ABApple,98:F1:70MurataMa,98:F1:81NewH3CTe,98:F1:99NECPlatf,98:F2:17Castlene,98:F2:B3HewlettP,98:F4:28zte,98:F4:ABEspressi,98:F5:37zte,98:F5:A9Ohsung,98:F6:21XiaomiCo,98:F7:81ARRISGro,98:F7:D7ARRISGro,98:F8:C1IDTTechn,98:F8:DBMariniIm,98:F9:C7IEEERegi,98:F9:C7:00:00:00/28Shenzhen,98:F9:C7:10:00:00/28HighSecL,98:F9:C7:20:00:00/28Pozyx,98:F9:C7:30:00:00/28BeijingH,98:F9:C7:40:00:00/28Promess,98:F9:C7:50:00:00/28Tonycore,98:F9:C7:60:00:00/28GoodBox,98:F9:C7:70:00:00/28ARIMACom,98:F9:C7:80:00:00/28Renalsen,98:F9:C7:90:00:00/28KoalaTec,98:F9:C7:A0:00:00/28MSBElekt,98:F9:C7:B0:00:00/28HIROIACo,98:F9:C7:C0:00:00/28ShenZhen,98:F9:C7:D0:00:00/28hangzhou,98:F9:C7:E0:00:00/28NC-LINKT,98:FA:9BLCFCHeFe,98:FA:A7Innonet,98:FA:E3XiaomiCo,98:FB:12GrandEle,98:FC:11Cisco-Li,98:FC:84IEEERegi,98:FC:84:00:00:00/28Leia,98:FC:84:10:00:00/28go-e,98:FC:84:20:00:00/28Juketek,98:FC:84:30:00:00/28ZeXinSha,98:FC:84:40:00:00/28Sferrum,98:FC:84:50:00:00/28Zymbit,98:FC:84:60:00:00/28Zerone,98:FC:84:70:00:00/28Broadtec,98:FC:84:80:00:00/28Guangdon,98:FC:84:90:00:00/28FathMech,98:FC:84:A0:00:00/28Shield,98:FC:84:B0:00:00/28chiconyp,98:FC:84:C0:00:00/28Shenzhen,98:FC:84:D0:00:00/28Jazwares,98:FC:84:E0:00:00/28Dongguan,98:FD:74Act,98:FD:B4PrimaxEl,98:FE:03Ericsson,98:FE:94Apple,98:FF:6AOTECShan,98:FF:D0LenovoMo,9A:1B:22InfoCore,9A:56:4BCubix,9A:AD:CACredo,9A:B2:F8SFFCommi,9A:D2:63FirmTek,9C:00:D3SHENZHEN,9C:01:11Shenzhen,9C:02:98SamsungE,9C:03:9EBeijingW,9C:04:73Tecmobil,9C:04:EBApple,9C:05:67HonorDev,9C:06:1BHangzhou,9C:06:6EHyteraCo,9C:0C:DFGuangdon,9C:0D:ACTymphany,9C:0E:4AShenzhen,9C:13:ABChansonW,9C:14:63Zhejiang,9C:14:65EdataEle,9C:18:74NokiaDan,9C:19:C2Dongguan,9C:1C:12ArubaaHe,9C:1C:37AltoBeam,9C:1C:6DHefeiDat,9C:1D:36HuaweiTe,9C:1D:58TexasIns,9C:1E:95Actionte,9C:1E:A4RenesasE,9C:1F:DDAccupix,9C:20:7BApple,9C:21:6ATp-LinkT,9C:22:0ETASCAN,9C:25:BEWildlife,9C:28:40Discover,9C:28:B3Apple,9C:28:BFContinen,9C:28:EFHuaweiTe,9C:28:F7XiaomiCo,9C:29:3FApple,9C:29:76IntelCor,9C:2A:70HonHaiPr,9C:2A:83SamsungE,9C:2B:A6RuijieNe,9C:2D:CFShishiTo,9C:2E:A1XiaomiCo,9C:2F:4Ezte,9C:2F:73Universa,9C:30:5BHonHaiPr,9C:30:66RWEEffiz,9C:31:78FoshanHu,9C:31:B6KuliteSe,9C:31:C3BSkyB,9C:32:A9SichuanT,9C:32:CECanon,9C:34:26ARRISGro,9C:35:83NiproDia,9C:35:EBApple,9C:36:F8HyundaiK,9C:37:F4HuaweiTe,9C:3A:9AShenzhen,9C:3A:AFSamsungE,9C:3D:CFNetgear,9C:3E:AAEnvyLogi,9C:40:CDSynclaye,9C:41:7CHameTech,9C:43:1EIEEERegi,9C:43:1E:00:00:00/28Antailiy,9C:43:1E:10:00:00/28SymfunTe,9C:43:1E:20:00:00/28HaesungD,9C:43:1E:30:00:00/28Advanced,9C:43:1E:40:00:00/28Wireless,9C:43:1E:50:00:00/28ProMOSTe,9C:43:1E:60:00:00/28R-S-IEle,9C:43:1E:70:00:00/28Optris,9C:43:1E:80:00:00/28WundaGro,9C:43:1E:90:00:00/28CONTINEN,9C:43:1E:A0:00:00/28STAccess,9C:43:1E:B0:00:00/28JNLTechn,9C:43:1E:C0:00:00/28SuZhouJi,9C:43:1E:D0:00:00/28HKELEPHO,9C:43:1E:E0:00:00/28PhoenixA,9C:44:3DChengduX,9C:44:A6SwiftTes,9C:45:63DIMEPSis,9C:49:7FIntegrat,9C:4A:7BNokia,9C:4C:AEMesaLabs,9C:4E:20Cisco,9C:4E:36IntelCor,9C:4E:8EALT,9C:4E:BFBoxCast,9C:4F:5FTAPSound,9C:4F:CFTCTmobil,9C:4F:DAApple,9C:50:D1MurataMa,9C:50:EECambridg,9C:52:F8HuaweiTe,9C:53:CDENGICAM,9C:54:1CShenzhen,9C:54:40Private,9C:54:C2NewH3CTe,9C:54:CAZhengzho,9C:54:DASkyBellT,9C:55:B4ISE,9C:56:36HuaweiDe,9C:57:11FeitianX,9C:57:ADCisco,9C:58:3CApple,9C:5A:44CompalIn,9C:5A:81XiaomiCo,9C:5B:96NMR,9C:5C:8DFiremaxI,9C:5C:8EASUSTekC,9C:5C:F9Sony,9C:5D:12ExtremeN,9C:5D:95VTCElect,9C:5E:73CalibreU,9C:5F:5AGuangdon,9C:5F:B0SamsungE,9C:61:1DPanasoni,9C:61:21SichuanT,9C:62:ABSumavisi,9C:63:EDzte,9C:64:5EHarmanCo,9C:64:8BApple,9C:65:B0SamsungE,9C:65:EEDASANNet,9C:65:F9AcSiPTec,9C:66:50GlodioTe,9C:68:5BOctonion,9C:68:65Fiberhom,9C:69:37QorvoInt,9C:69:B4IEEERegi,9C:69:B4:00:00:00/28SuzhouFi,9C:69:B4:10:00:00/28EATechno,9C:69:B4:20:00:00/28MOZIShen,9C:69:B4:30:00:00/28Appareo,9C:69:B4:40:00:00/28Globalco,9C:69:B4:50:00:00/28Elesta,9C:69:B4:60:00:00/28Shenzhen,9C:69:B4:70:00:00/28PCI,9C:69:B4:80:00:00/28Skydockd,9C:69:B4:90:00:00/28Teptron,9C:69:B4:A0:00:00/28BeijingP,9C:69:B4:B0:00:00/28Toughdog,9C:69:B4:C0:00:00/28Guangdon,9C:69:B4:D0:00:00/28Intellec,9C:69:B4:E0:00:00/28NingboSh,9C:69:D1HuaweiTe,9C:6A:BEQEESApS,9C:6B:37RenesasE,9C:6B:72RealmeCh,9C:6B:F0Shenzhen,9C:6C:15Microsof,9C:6F:52zte,9C:71:3AHuaweiTe,9C:73:70HuaweiTe,9C:74:1AHuaweiTe,9C:74:6FHuaweiTe,9C:75:14Wildix,9C:75:6EAjaxDMCC,9C:76:0EApple,9C:76:13Ring,9C:77:AANadasnv,9C:79:ACSuntecSo,9C:7A:03Ciena,9C:7B:D2NEOLABCo,9C:7B:EFHewlettP,9C:7D:A3HuaweiTe,9C:7F:57UNICMemo,9C:7F:81Shenzhen,9C:80:7DSYSCABLE,9C:80:DFArcadyan,9C:82:3FHuaweiDe,9C:82:75YichipMi,9C:82:81vivoMobi,9C:83:BFPRO-VISI,9C:84:BFApple,9C:85:66Wingtech,9C:86:DAPhoenixG,9C:88:88SimacTec,9C:88:ADFiberhom,9C:8A:CBJuniperN,9C:8B:A0Apple,9C:8B:F1Warehous,9C:8C:6ESamsungE,9C:8C:D8HewlettP,9C:8D:1AINTEGpro,9C:8D:7CAlpsalpi,9C:8D:D3LeontonT,9C:8E:99HewlettP,9C:8E:9CHuaweiDe,9C:8E:CDAmcrestT,9C:8E:DCTeracom,9C:93:4EXerox,9C:93:B0Megatron,9C:93:E4Private,9C:95:67HuaweiDe,9C:95:F8SmartDoo,9C:97:26Technico,9C:97:891More,9C:98:11Guangzho,9C:99:A0XiaomiCo,9C:99:CDVoippart,9C:9A:C0LEGOSyst,9C:9C:1DStarkeyL,9C:9C:1FEspressi,9C:9C:40SichuanT,9C:9D:5DRaden,9C:9D:7EBeijingX,9C:9E:71HuaweiDe,9C:A1:0AScleSfe,9C:A1:34Nike,9C:A3:A9Guangzho,9C:A3:BASAKURAIn,9C:A5:13SamsungE,9C:A5:25Shandong,9C:A5:70eero,9C:A5:77OsornoEn,9C:A5:C0vivoMobi,9C:A6:15Tp-LinkT,9C:A6:9DWhaleyTe,9C:A9:E4zte,9C:AA:1BMicrosof,9C:AC:6DUniversa,9C:AD:97HonHaiPr,9C:AD:EFObihaiTe,9C:AE:D3SeikoEps,9C:AF:6FItelMobi,9C:AF:CACisco,9C:B0:08Ubiquito,9C:B2:06Procente,9C:B2:B2HuaweiTe,9C:B2:E8HuaweiTe,9C:B6:54HewlettP,9C:B6:D0RivetNet,9C:B7:0DLiteonTe,9C:B7:93Creatcom,9C:BB:98ShenZhen,9C:BC:F0XiaomiCo,9C:BD:6EDERA,9C:BD:9DSkyDisk,9C:BE:E0Biosound,9C:C0:77PrintCou,9C:C0:D2Conducti,9C:C1:72HuaweiTe,9C:C2:C4InspurEl,9C:C7:A6AVM,9C:C7:D1SHARP,9C:C8:AEBectonDi,9C:C8:FCARRISGro,9C:C9:50Baumer,9C:C9:EBNetgear,9C:CA:D9Nokia,9C:CC:83JuniperN,9C:CD:82ChengUei,9C:D2:1EHonHaiPr,9C:D2:4Bzte,9C:D3:32Technolo,9C:D3:5BSamsungE,9C:D3:6DNetgear,9C:D4:8BInnoluxT,9C:D5:7DCisco,9C:D6:43D-LinkIn,9C:D9:17Motorola,9C:D9:CBLesiraMa,9C:DA:3EIntelCor,9C:DB:07Thum+Mah,9C:DB:CBWuhanFun,9C:DC:71HewlettP,9C:DD:1FIntellig,9C:DF:03HarmanBe,9C:DF:B1Shenzhen,9C:E0:63SamsungE,9C:E1:0ENCTech,9C:E1:76Cisco,9C:E1:D6JungerAu,9C:E2:30Julong,9C:E3:3FApple,9C:E3:74HuaweiTe,9C:E6:35Nintendo,9C:E6:5EApple,9C:E6:E7SamsungE,9C:E7:BDWindusko,9C:E8:2BvivoMobi,9C:E8:95NewH3CTe,9C:E9:1Czte,9C:E9:51Shenzhen,9C:EA:97HonorDev,9C:EB:E8BizLinkK,9C:EC:61HuaweiDe,9C:ED:FAEVUlutio,9C:EF:D5PandaWir,9C:F0:29Integrat,9C:F3:87Apple,9C:F4:8EApple,9C:F5:31Guangdon,9C:F6:1ACarrierF,9C:F6:7DRicardoP,9C:F6:DDIEEERegi,9C:F6:DD:00:00:00/28Annapurn,9C:F6:DD:10:00:00/28IthorIT,9C:F6:DD:20:00:00/28BeijingS,9C:F6:DD:30:00:00/28RYEEXTec,9C:F6:DD:40:00:00/28CapitalE,9C:F6:DD:50:00:00/28b8ta,9C:F6:DD:60:00:00/28Shenzhen,9C:F6:DD:70:00:00/28KXTTechn,9C:F6:DD:80:00:00/28Savari,9C:F6:DD:90:00:00/28CAMA（Luo,9C:F6:DD:A0:00:00/28AVI,9C:F6:DD:B0:00:00/28Guangzho,9C:F6:DD:C0:00:00/28Lighting,9C:F6:DD:D0:00:00/28FoshanSy,9C:F6:DD:E0:00:00/28ShanxiZh,9C:F8:DBshenzhen,9C:F9:38AREVANP,9C:FB:D5vivoMobi,9C:FB:F1MESOMATI,9C:FC:01Apple,9C:FC:28Apple,9C:FC:D1Aetheris,9C:FC:E8IntelCor,9C:FE:A1Fiberhom,9C:FF:BEOTSL,9C:FF:C2AVI,A0:02:4A:00:00:00/28Zhejiang,A0:02:4A:10:00:00/28VitecIma,A0:02:4A:20:00:00/28Danriver,A0:02:4A:30:00:00/28SomaDete,A0:02:4A:40:00:00/28ArgosSol,A0:02:4A:50:00:00/28DonguanA,A0:02:4A:60:00:00/28XiaojieT,A0:02:4A:80:00:00/28BeijingL,A0:02:4A:90:00:00/28KontaktM,A0:02:4A:A0:00:00/28Guangdon,A0:02:4A:B0:00:00/28XianYing,A0:02:4A:C0:00:00/28Encroute,A0:02:4A:D0:00:00/28bitbee,A0:02:4A:E0:00:00/28IoTecha,A0:02:DCAmazonTe,A0:03:63RobertBo,A0:04:3EParkerHa,A0:04:60Netgear,A0:06:27NEXPASys,A0:07:98SamsungE,A0:07:B6Advanced,A0:08:6FHuaweiTe,A0:09:4CCenturyL,A0:09:EDAvaya,A0:0A:BFWiesonTe,A0:0B:BASamsungE,A0:0C:A1SKTBSKiT,A0:0F:37Cisco,A0:10:81SamsungE,A0:12:90Avaya,A0:12:DBTabuchiE,A0:13:3BHiTiDigi,A0:13:CBFiberhom,A0:14:3DParrot,A0:16:5CTriteka,A0:18:28Apple,A0:18:42Comtrend,A0:18:59Shenzhen,A0:19:17Bertel,A0:19:B2IEEERegi,A0:19:B2:00:00:00/28VastProd,A0:19:B2:10:00:00/28ElSewedy,A0:19:B2:20:00:00/28BeijingD,A0:19:B2:30:00:00/28PowerDia,A0:19:B2:40:00:00/28Osatec,A0:19:B2:50:00:00/28SzbroadT,A0:19:B2:60:00:00/28GfGmbH,A0:19:B2:70:00:00/28ARIMACom,A0:19:B2:80:00:00/28MISIndus,A0:19:B2:90:00:00/28LonMicro,A0:19:B2:A0:00:00/28Adomi,A0:19:B2:B0:00:00/28HangZhou,A0:19:B2:C0:00:00/28LDATechn,A0:19:B2:D0:00:00/28RYDElect,A0:19:B2:E0:00:00/28AhgoraSi,A0:1B:29Sagemcom,A0:1C:05NimaxTel,A0:1C:8DHuaweiTe,A0:1D:48HewlettP,A0:1E:0BMINIXTec,A0:20:A6Espressi,A0:21:95SamsungE,A0:21:B7Netgear,A0:22:4EIEEERegi,A0:22:4E:00:00:00/28KyungInE,A0:22:4E:10:00:00/28rNETCont,A0:22:4E:20:00:00/28ClosedJo,A0:22:4E:30:00:00/28ProPhoto,A0:22:4E:40:00:00/28TMGcore,A0:22:4E:50:00:00/28ZhuhaiCh,A0:22:4E:60:00:00/28MESITasd,A0:22:4E:70:00:00/28AppliedI,A0:22:4E:80:00:00/28EISSTInt,A0:22:4E:90:00:00/28DeltaTau,A0:22:4E:A0:00:00/28ISTElekt,A0:22:4E:B0:00:00/28AllInspi,A0:22:4E:C0:00:00/28Standart,A0:22:4E:D0:00:00/28Digifocu,A0:22:4E:E0:00:00/28HunanYou,A0:22:DEvivoMobi,A0:23:1BTeleComp,A0:23:9FCisco,A0:27:B6SamsungE,A0:28:33IEEERegi,A0:28:33:00:00:00/28GERSYS,A0:28:33:10:00:00/28Ordercub,A0:28:33:20:00:00/28Shanghai,A0:28:33:30:00:00/28Shanghai,A0:28:33:40:00:00/28FirmINFO,A0:28:33:50:00:00/28JGROptic,A0:28:33:60:00:00/28XiamenCa,A0:28:33:70:00:00/28KryptusI,A0:28:33:80:00:00/28HzhyTech,A0:28:33:90:00:00/28Imeshx,A0:28:33:A0:00:00/28MedicalE,A0:28:33:B0:00:00/28FlexLink,A0:28:33:C0:00:00/28Kalray,A0:28:33:D0:00:00/28Audix,A0:28:33:E0:00:00/28Precisio,A0:28:EDHMDGloba,A0:29:19Dell,A0:29:BDTeamGrou,A0:2B:B8HewlettP,A0:2C:36Fn-LinkT,A0:2D:13AirTiesW,A0:2E:F3UnitedIn,A0:31:DBHuaweiTe,A0:32:99LenovoBe,A0:34:1BAdero,A0:36:79HuaweiTe,A0:36:9FIntelCor,A0:36:F0Comprehe,A0:36:FAEttusRes,A0:38:F8OURAHeal,A0:39:EESagemcom,A0:39:F7LGElectr,A0:3A:75PSSBelgi,A0:3B:01KyungInE,A0:3B:1BInspireT,A0:3B:E3Apple,A0:3C:31Shenzhen,A0:3D:6ECisco,A0:3D:6FCisco,A0:3E:6BIEEERegi,A0:3E:6B:00:00:00/28s&tembed,A0:3E:6B:10:00:00/28Business,A0:3E:6B:20:00:00/28VidexEle,A0:3E:6B:30:00:00/28iLodaSol,A0:3E:6B:40:00:00/28Shenzhen,A0:3E:6B:50:00:00/28FridayLa,A0:3E:6B:60:00:00/28WuhanRui,A0:3E:6B:70:00:00/28SinoGrid,A0:3E:6B:80:00:00/28718thRes,A0:3E:6B:90:00:00/28Incognit,A0:3E:6B:A0:00:00/28Shenzhen,A0:3E:6B:B0:00:00/28KoCoSMes,A0:3E:6B:C0:00:00/28Qunarcom,A0:3E:6B:D0:00:00/28JiningSm,A0:3E:6B:E0:00:00/28Nanjingz,A0:3E:6B:F0:00:00/28Private,A0:40:25Actionca,A0:40:41SAMWONFA,A0:40:6FHuaweiTe,A0:40:A0Netgear,A0:41:2DLansen,A0:41:47HuaweiDe,A0:41:5EOpsensSo,A0:41:A7NLMinist,A0:42:3FTyanComp,A0:42:46ITTeleco,A0:43:B0Hangzhou,A0:43:DBSitael,A0:44:5CHuaweiTe,A0:47:D7BestITWo,A0:48:1CHewlettP,A0:4A:5EMicrosof,A0:4C:0CShenzhen,A0:4C:5BShenzhen,A0:4C:C1Helixtec,A0:4E:01CENTRALE,A0:4E:04Nokia,A0:4E:A7Apple,A0:4E:CFApple,A0:4F:85LGElectr,A0:4F:D4ADBBroad,A0:51:0BIntelCor,A0:51:C6Avaya,A0:55:4FCisco,A0:55:DEARRISGro,A0:56:B2HarmanBe,A0:56:F3Apple,A0:57:E3HuaweiTe,A0:59:3AVDSVideo,A0:5A:A4GrandPro,A0:5B:21ENVINET,A0:5D:C1TMCT,A0:5D:E7DIRECTV,A0:5E:6BMELPER,A0:60:90SamsungE,A0:62:60Private,A0:62:FBHisenseV,A0:63:91Netgear,A0:64:8FAskeyCom,A0:65:18VnptTech,A0:66:10Fujitsu,A0:67:BESicon,A0:68:1CGDMideaA,A0:68:7EARRISGro,A0:69:74HonorDev,A0:69:86WellavTe,A0:6A:00Verilink,A0:6A:44Vizio,A0:6C:65TexasIns,A0:6C:ECRim,A0:6D:09Intelcan,A0:6E:50NanotekE,A0:6F:AALGInnote,A0:70:99BeijingH,A0:70:B7HuaweiTe,A0:71:A9Nokia,A0:72:2CHUMAX,A0:72:E4NjSystem,A0:73:32Cashmast,A0:73:FCRancoreT,A0:75:91SamsungE,A0:75:EABoxLock,A0:76:4EEspressi,A0:77:51ASMediaT,A0:77:71Vialis,A0:78:17Apple,A0:78:BAPantech,A0:82:1FSamsungE,A0:82:22QingdaoH,A0:82:ACLinearDM,A0:82:C7PTI,A0:84:CBSonicSen,A0:85:FCMicrosof,A0:86:1DChengduF,A0:86:C6XiaomiCo,A0:86:ECSAEHANHI,A0:88:69IntelCor,A0:88:B4IntelCor,A0:89:E4Skyworth,A0:8A:87HuiZhouK,A0:8C:15GerhardD,A0:8C:9BXtremeTe,A0:8C:F8HuaweiTe,A0:8C:FDHewlettP,A0:8D:16HuaweiTe,A0:8E:78Sagemcom,A0:90:DEVeedims,A0:91:69LGElectr,A0:91:A2OnePlusE,A0:91:C8zte,A0:93:47Guangdon,A0:93:51Cisco,A0:94:1AGuangdon,A0:94:6AShenzhen,A0:95:0CChinaMob,A0:98:05OpenVoxC,A0:98:EDShandong,A0:99:9BApple,A0:9A:5ATimeDoma,A0:9B:12ChinaMob,A0:9B:17TaicangT,A0:9B:BDTotalAvi,A0:9D:86Alcatel-,A0:9D:91SoundBri,A0:9D:C1ChinaDra,A0:9E:1APolarEle,A0:9F:10Shenzhen,A0:9F:7AD-LinkMi,A0:A0:DCHuaweiDe,A0:A1:30DLITaiwa,A0:A2:3CGpms,A0:A3:09Apple,A0:A3:3BHuaweiTe,A0:A3:B8Wiscloud,A0:A3:E2Actionte,A0:A3:F0D-LinkIn,A0:A4:C5IntelCor,A0:A6:5CSupercom,A0:A7:63Polytron,A0:A8:CDIntelCor,A0:AA:FDEraThink,A0:AB:1BD-LinkIn,A0:AB:51WeifangG,A0:AC:69SamsungE,A0:AD:A1JMRElect,A0:AF:BDIntelCor,A0:B0:45HalongMi,A0:B0:86Hirschma,A0:B1:00ShenZhen,A0:B3:CCHewlettP,A0:B4:37GDMissio,A0:B4:39Cisco,A0:B4:A5SamsungE,A0:B4:BFInfinet-,A0:B5:3CTechnico,A0:B5:49Arcadyan,A0:B5:DAHongKong,A0:B6:62Acutvist,A0:B8:F8AmgenU,A0:B9:EDSkytap,A0:BA:B8PixonIma,A0:BB:3EIEEERegi,A0:BB:3E:00:00:00/28LinkLabs,A0:BB:3E:10:00:00/28IVisionE,A0:BB:3E:20:00:00/28DirectOu,A0:BB:3E:30:00:00/28WiteRive,A0:BB:3E:40:00:00/28COMSYSCo,A0:BB:3E:50:00:00/28ManTechI,A0:BB:3E:60:00:00/28XiamenKe,A0:BB:3E:70:00:00/28SIMTECEl,A0:BB:3E:80:00:00/28AutarcTe,A0:BB:3E:90:00:00/28Sandal,A0:BB:3E:A0:00:00/28Filo,A0:BB:3E:B0:00:00/28BeijingT,A0:BB:3E:C0:00:00/28EwigIndu,A0:BB:3E:D0:00:00/28Shenzhen,A0:BB:3E:E0:00:00/28Messtech,A0:BB:3E:F0:00:00/28Private,A0:BD:1DZhejiang,A0:BD:CDBSkyB,A0:BF:50SCAdd-Pr,A0:BF:A5Coresys,A0:C2:DECostarVi,A0:C3:DETritonEl,A0:C4:A5SygnHous,A0:C5:62ARRISGro,A0:C5:89IntelCor,A0:C5:F2IEEERegi,A0:C5:F2:00:00:00/28Quantlab,A0:C5:F2:10:00:00/28KNSGroup,A0:C5:F2:20:00:00/28Speedgoa,A0:C5:F2:30:00:00/28Shenzhen,A0:C5:F2:40:00:00/28AiCare,A0:C5:F2:50:00:00/28Spacepat,A0:C5:F2:60:00:00/28ShenZhen,A0:C5:F2:70:00:00/28Viettron,A0:C5:F2:80:00:00/28CoolRGro,A0:C5:F2:90:00:00/28ImpulseN,A0:C5:F2:A0:00:00/28SeriousI,A0:C5:F2:B0:00:00/28Oraycom,A0:C5:F2:C0:00:00/28Glooko,A0:C5:F2:D0:00:00/28UnaliWea,A0:C5:F2:E0:00:00/28Synapsys,A0:C6:ECShenZhen,A0:C9:A0MurataMa,A0:CA:A5Intellig,A0:CB:FDSamsungE,A0:CC:2BMurataMa,A0:CE:C8CeLink,A0:CF:5BCisco,A0:CF:F5zte,A0:D0:5BSamsungE,A0:D0:DCAmazonTe,A0:D1:2AAXPROTec,A0:D2:B1AmazonTe,A0:D3:7AIntelCor,A0:D3:85AUMARies,A0:D3:C1HewlettP,A0:D6:35WBSTechn,A0:D7:22SamsungE,A0:D7:95Apple,A0:D7:A0HuaweiDe,A0:D7:F3SamsungE,A0:D8:07HuaweiDe,A0:D8:3DFiberhom,A0:D8:6FArgoAi,A0:DA:92NanjingG,A0:DC:04Becker-A,A0:DD:97PolarLin,A0:DD:E5SHARP,A0:DE:05Irbis-T,A0:DE:0FHuaweiDe,A0:DF:15HuaweiTe,A0:E0:AFCisco,A0:E2:01AVTraceC,A0:E2:5AAmicusSK,A0:E2:95DATSyste,A0:E4:53Sony,A0:E4:CBZyxelCom,A0:E5:34StratecB,A0:E5:E9enimai,A0:E6:17Matis,A0:E6:F8TexasIns,A0:E7:0BIntelCor,A0:E9:DBNingboFr,A0:EB:76AirCUVE,A0:EC:80zte,A0:EC:F9Cisco,A0:ED:CDApple,A0:EF:84SeineIma,A0:F2:17GEMedica,A0:F3:C1Tp-LinkT,A0:F3:E4Alcatel-,A0:F4:19Nokia,A0:F4:50HTC,A0:F4:59Fn-LinkT,A0:F4:79HuaweiTe,A0:F6:FDTexasIns,A0:F8:49Cisco,A0:F8:95Shenzhen,A0:F9:B7AdemcoSm,A0:F9:E0Vivatel,A0:FB:C5Apple,A0:FC:6ETelegraf,A0:FE:61VivintWi,A0:FE:91AVATAuto,A0:FF:22Shenzhen,A0:FF:70Technico,A4:00:E2HuaweiTe,A4:01:30ABIsyste,A4:02:B9IntelCor,A4:04:50nForeTec,A4:05:6ETiinlab,A4:05:9ESTAInfin,A4:06:E9TexasIns,A4:07:B6SamsungE,A4:08:01AmazonTe,A4:08:EAMurataMa,A4:08:F5Sagemcom,A4:09:CBAlfredKa,A4:0B:EDCarryTec,A4:0C:66Shenzhen,A4:0C:C3Cisco,A4:0D:BCXiamenIn,A4:0E:2BFacebook,A4:11:15RobertBo,A4:11:62ArloTech,A4:11:63IEEERegi,A4:11:63:00:00:00/28AdetelEq,A4:11:63:10:00:00/28INTERCON,A4:11:63:20:00:00/28AllgoTec,A4:11:63:30:00:00/28Pax,A4:11:63:40:00:00/28AlterG,A4:11:63:50:00:00/28Carbon,A4:11:63:60:00:00/28BeijingX,A4:11:63:70:00:00/28Shenzhen,A4:11:63:80:00:00/28DspreadT,A4:11:63:90:00:00/28accessoT,A4:11:63:A0:00:00/28ISE,A4:11:63:B0:00:00/28MoogMusi,A4:11:63:C0:00:00/28Viloc,A4:11:63:D0:00:00/28Shenzhen,A4:11:63:E0:00:00/28tinylogi,A4:11:94Lenovo,A4:12:32Guangdon,A4:12:42NECPlatf,A4:13:4ELuxul,A4:14:37Hangzhou,A4:15:66WeifangG,A4:15:88ARRISGro,A4:16:E7HuaweiTe,A4:17:31HonHaiPr,A4:17:8BHuaweiTe,A4:17:91Shenzhen,A4:18:75Cisco,A4:19:08Fiberhom,A4:1A:3ATp-LinkT,A4:1B:34ChinaMob,A4:1B:C0FastecIm,A4:1F:72Dell,A4:21:8ANortelNe,A4:23:05OpenNetw,A4:24:B3FlatFrog,A4:24:DDCambrion,A4:25:1BAvaya,A4:26:18Integrat,A4:26:55LTIMotio,A4:28:B7YangtzeM,A4:29:40Shenzhen,A4:29:83BoeingDe,A4:29:85SichuanA,A4:29:B7bluesky,A4:2A:71SichuanT,A4:2B:8CNetgear,A4:2B:B0Tp-LinkT,A4:2C:08Masterwo,A4:30:7ASamsungE,A4:31:11Ziv,A4:31:35Apple,A4:33:D1Fibrlink,A4:33:D7MitraSta,A4:34:12ThalesAl,A4:34:D9IntelCor,A4:34:F1TexasIns,A4:35:23Guangdon,A4:35:2DTRIZNetw,A4:38:31RFelemen,A4:38:CCNintendo,A4:38:FCPlasticL,A4:39:B6Shenzhen,A4:3A:69Vers,A4:3B:0EHuaweiDe,A4:3B:FAIEEERegi,A4:3B:FA:00:00:00/28ChengduT,A4:3B:FA:10:00:00/28BeijingU,A4:3B:FA:20:00:00/28PowellIn,A4:3B:FA:30:00:00/28CircusWo,A4:3B:FA:40:00:00/28MaxonAus,A4:3B:FA:50:00:00/28BOISolut,A4:3B:FA:60:00:00/28Recognit,A4:3B:FA:70:00:00/28Deatroni,A4:3B:FA:80:00:00/28Alpwise,A4:3B:FA:90:00:00/28ShenZhen,A4:3B:FA:A0:00:00/28PlusOneJ,A4:3B:FA:B0:00:00/28ALSTOMSt,A4:3B:FA:C0:00:00/28Shanghai,A4:3B:FA:D0:00:00/28“Compone,A4:3B:FA:E0:00:00/28Magstim,A4:3B:FA:F0:00:00/28Private,A4:3D:78Guangdon,A4:3E:51AnovFran,A4:3E:A0iCommHK,A4:40:27zte,A4:42:3BIntelCor,A4:44:D1Wingtech,A4:45:19XiaomiCo,A4:45:CDIoTDiagn,A4:46:6BEOCTechn,A4:46:B4HuaweiDe,A4:46:FAAmTRANVi,A4:4A:D3STElectr,A4:4B:15SunCupid,A4:4B:D5XiaomiCo,A4:4C:11Cisco,A4:4C:62Hangzhou,A4:4C:C8Dell,A4:4E:2DAdaptive,A4:4E:31IntelCor,A4:4F:29IEEERegi,A4:4F:29:00:00:00/28Dermalog,A4:4F:29:10:00:00/28Olssen,A4:4F:29:20:00:00/28Luceor,A4:4F:29:30:00:00/28ComselSy,A4:4F:29:40:00:00/28DGCAcces,A4:4F:29:50:00:00/28Shanghai,A4:4F:29:60:00:00/28Selektro,A4:4F:29:70:00:00/28ProteanP,A4:4F:29:80:00:00/28Innovati,A4:4F:29:90:00:00/28CertiNet,A4:4F:29:A0:00:00/28Htd,A4:4F:29:B0:00:00/28Guangdon,A4:4F:29:C0:00:00/28Shenzhen,A4:4F:29:D0:00:00/28Hallibur,A4:4F:29:E0:00:00/28Neotech,A4:4F:29:F0:00:00/28Private,A4:50:06Shenzhen,A4:50:46XiaomiCo,A4:50:55BuswareD,A4:51:29Xag,A4:51:6FMicrosof,A4:52:6FADBBroad,A4:53:0ECisco,A4:53:85WeifangG,A4:53:EEIEEERegi,A4:53:EE:00:00:00/28MahleEle,A4:53:EE:10:00:00/28Stellamo,A4:53:EE:20:00:00/28UbisafeS,A4:53:EE:30:00:00/28Larvaio,A4:53:EE:40:00:00/28Williams,A4:53:EE:50:00:00/28FoshanYi,A4:53:EE:60:00:00/28Shenzhen,A4:53:EE:70:00:00/28BeijingL,A4:53:EE:80:00:00/28T-Touchi,A4:53:EE:90:00:00/28Dongguan,A4:53:EE:B0:00:00/28ViperDes,A4:53:EE:C0:00:00/28SOSLAB,A4:53:EE:D0:00:00/28Ssk,A4:55:90XiaomiCo,A4:56:02fenglian,A4:56:1BMCOT,A4:56:30Cisco,A4:56:CCTechnico,A4:58:02Shin-IlT,A4:58:0FIEEERegi,A4:58:0F:00:00:00/28Innopro,A4:58:0F:10:00:00/28StoneLoc,A4:58:0F:20:00:00/28BLOKS,A4:58:0F:30:00:00/28Engineer,A4:58:0F:40:00:00/28Shenzhen,A4:58:0F:50:00:00/28CoAsiaMi,A4:58:0F:60:00:00/28Astro,A4:58:0F:70:00:00/28Changsha,A4:58:0F:80:00:00/28AirLiqui,A4:58:0F:90:00:00/28KseniaSe,A4:58:0F:A0:00:00/28Guangzho,A4:58:0F:B0:00:00/28AbbPghv,A4:58:0F:C0:00:00/28Homebeav,A4:58:0F:D0:00:00/28EyeIo,A4:58:0F:E0:00:00/28Finetree,A4:5A:1Csmart-el,A4:5C:27Nintendo,A4:5D:36HewlettP,A4:5D:A1ADBBroad,A4:5E:5AACTIVIO,A4:5E:60Apple,A4:5F:9BNexell,A4:60:11Verifone,A4:60:32MRVCommu,A4:61:91NamJunSa,A4:62:DFDSGlobal,A4:63:A1Inventus,A4:67:06Apple,A4:68:BCOakley,A4:6B:B6IntelCor,A4:6C:2ACisco,A4:6C:C1LTiREEne,A4:6C:F1SamsungE,A4:6D:A4HuaweiTe,A4:6E:79DFTSyste,A4:70:D6Motorola,A4:71:74HuaweiTe,A4:77:33Google,A4:77:58NingboFr,A4:77:60Nokia,A4:77:F3Apple,A4:78:06Cisco,A4:78:86Avaya,A4:79:E4KLINFO,A4:7A:A4ARRISGro,A4:7A:CFVibicomC,A4:7B:1AHuaweiDe,A4:7B:2CNokia,A4:7B:85ULTIMEDI,A4:7B:9DEspressi,A4:7C:14ChargeSt,A4:7C:1FCobham,A4:7C:C9HuaweiTe,A4:7D:9FShenzhen,A4:7E:36EMMicroe,A4:7E:39zte,A4:81:7ACigShang,A4:81:EENokia,A4:82:69Datrium,A4:83:E7Apple,A4:84:31SamsungE,A4:85:6BQElectro,A4:86:AEQuectelW,A4:88:73Cisco,A4:89:5BArkInfos,A4:8C:C0JLGIndus,A4:8C:DBLenovo,A4:8D:3BVizio,A4:8E:0ADeLavalI,A4:90:05ChinaGre,A4:91:B1Technico,A4:92:CBNokia,A4:93:3FHuaweiTe,A4:93:40BeijingS,A4:93:4CCisco,A4:94:26Elgama-E,A4:97:33AskeyCom,A4:97:5CVTechTel,A4:97:B1Chongqin,A4:97:BBHitachiI,A4:98:13ARRISGro,A4:99:47HuaweiTe,A4:99:81FuJianEl,A4:9A:58SamsungE,A4:9B:13DigitalC,A4:9B:4FHuaweiTe,A4:9B:CDCisco,A4:9B:F5Hybridse,A4:9D:49Ketra,A4:9E:DBAutoCrib,A4:9F:85LyveMind,A4:9F:89Shanghai,A4:A1:79Nanjingd,A4:A1:C2Ericsson,A4:A1:E4Innotube,A4:A2:4ACiscoSPV,A4:A4:6BHuaweiTe,A4:A4:D3Bluebank,A4:A6:A9Private,A4:A8:0FShenzhen,A4:AA:FEHuaweiDe,A4:AC:0FHuaweiDe,A4:AD:00Ragsdale,A4:AD:B8VitecGro,A4:AE:11HonHaiPr,A4:AE:12HonHaiPr,A4:AE:9AMaestroW,A4:B1:21Arantia2,A4:B1:97Apple,A4:B1:C1IntelCor,A4:B1:E9Technico,A4:B1:EEHZANDER,A4:B2:39Cisco,A4:B2:A7AdaxysSo,A4:B3:6ASDOChrom,A4:B4:39Cisco,A4:B5:2EIntegrat,A4:B6:1EHuaweiDe,A4:B8:05Apple,A4:B8:18PENTAGes,A4:B9:80ParkingB,A4:BA:76HuaweiTe,A4:BA:DBDell,A4:BB:6DDell,A4:BB:AFLimeInst,A4:BD:C4HuaweiTe,A4:BE:2BHuaweiTe,A4:BE:61EutroVis,A4:BF:01IntelCor,A4:C0:C7ShenZhen,A4:C0:E1Nintendo,A4:C1:38TelinkSe,A4:C2:ABHangzhou,A4:C3:61Apple,A4:C3:F0IntelCor,A4:C4:94IntelCor,A4:C5:4EHuaweiDe,A4:C6:4FHuaweiTe,A4:C6:9ASamsungE,A4:C7:4BHuaweiDe,A4:C7:DECambridg,A4:C9:39Guangdon,A4:CA:A0HuaweiTe,A4:CC:32Inficomm,A4:CC:B9RealmeCh,A4:CD:23Shenzhen,A4:CE:DAArcadyan,A4:CF:12Espressi,A4:CF:D2UbeeInte,A4:D0:94Vivavis,A4:D1:8CApple,A4:D1:8FShenzhen,A4:D1:D1ECOtalit,A4:D1:D2Apple,A4:D3:B5GLITELSt,A4:D4:B2Shenzhen,A4:D5:78TexasIns,A4:D7:3CSeikoEps,A4:D7:95Wingtech,A4:D8:56Gimbal,A4:D8:CAHongKong,A4:D9:31Apple,A4:D9:90SamsungE,A4:D9:A4neXusIDS,A4:DA:22IEEERegi,A4:DA:22:00:00:00/28GeneralE,A4:DA:22:10:00:00/28T2TSyste,A4:DA:22:20:00:00/28WyzeLabs,A4:DA:22:30:00:00/28DURATECH,A4:DA:22:40:00:00/28Loriot,A4:DA:22:50:00:00/28Original,A4:DA:22:60:00:00/28Auranext,A4:DA:22:70:00:00/28HydroEle,A4:DA:22:80:00:00/28SolidPro,A4:DA:22:90:00:00/28MalldonT,A4:DA:22:A0:00:00/28Grundig,A4:DA:22:B0:00:00/28Klashwer,A4:DA:22:C0:00:00/28EhoLink,A4:DA:22:D0:00:00/28ShenZhen,A4:DA:22:E0:00:00/28Quuppa,A4:DA:32TexasIns,A4:DA:3FBionics,A4:DA:D4YamatoDe,A4:DB:2EKingspan,A4:DB:30LiteonTe,A4:DC:BEHuaweiTe,A4:DE:26Sumitomo,A4:DE:50TotalWal,A4:DE:C9QLoveMob,A4:E0:E6Filizola,A4:E1:1AJuniperN,A4:E3:1BNokia,A4:E3:2ESiliconS,A4:E3:91DenyFont,A4:E4:B8BlackBer,A4:E5:7CEspressi,A4:E5:97Gessler,A4:E6:15Shenzhen,A4:E6:B1Shanghai,A4:E7:31Nokia,A4:E7:E4Connex,A4:E9:75Apple,A4:E9:91Sistemas,A4:E9:A3HonestTe,A4:EA:8EExtremeN,A4:EB:D3SamsungE,A4:ED:43IEEERegi,A4:ED:43:00:00:00/28Sweam,A4:ED:43:10:00:00/28Ingelabs,A4:ED:43:20:00:00/28Shanghai,A4:ED:43:30:00:00/28Dongguan,A4:ED:43:40:00:00/28NetasTel,A4:ED:43:50:00:00/28BeijingI,A4:ED:43:60:00:00/28Shanghai,A4:ED:43:70:00:00/28WuxiJunc,A4:ED:43:80:00:00/28LinseisM,A4:ED:43:90:00:00/28Heyuanin,A4:ED:43:A0:00:00/28Guangzho,A4:ED:43:B0:00:00/28ParagonB,A4:ED:43:C0:00:00/28leakSMAR,A4:ED:43:D0:00:00/28BrandNew,A4:ED:43:E0:00:00/28ToecTech,A4:ED:4EARRISGro,A4:EE:57SeikoEps,A4:EF:52Telewave,A4:F0:5EGuangdon,A4:F1:E8Apple,A4:F3:C1OpenSour,A4:F3:E7Integrat,A4:F4:65ItelMobi,A4:F4:C2VnptTech,A4:F5:22ChofuSei,A4:F7:D0LANAcces,A4:F9:E4AirVineS,A4:FA:76NewH3CTe,A4:FB:8DHangzhou,A4:FC:77MegaWell,A4:FC:CESecurity,A4:FF:95Nokia,A8:01:6DAiwa,A8:01:80IMAGOTec,A8:02:DBzte,A8:03:2AEspressi,A8:05:77Netlist,A8:06:00SamsungE,A8:0C:0DCisco,A8:0C:63HuaweiTe,A8:0C:CAShenzhen,A8:10:87TexasIns,A8:11:FCARRISGro,A8:13:74Panasoni,A8:15:4DTp-LinkT,A8:15:59Breathom,A8:15:D6Shenzhen,A8:16:B2LGElectr,A8:16:D0SamsungE,A8:17:58Elektron,A8:1B:18Xts,A8:1B:5AGuangdon,A8:1B:5DFoxtelMa,A8:1B:6ATexasIns,A8:1D:16AzureWav,A8:1E:84QuantaCo,A8:1F:AFKryptonP,A8:20:66Apple,A8:23:16Nokia,A8:23:FELGElectr,A8:24:B8Nokia,A8:24:EBNPOIntro,A8:25:EBCambridg,A8:26:D9HTC,A8:29:4CPrecisio,A8:2B:B5Edgecore,A8:2B:B9SamsungE,A8:2B:CDHuaweiTe,A8:2B:D6ShinaSys,A8:30:1CQingdaoI,A8:30:ADWeifangG,A8:30:BCSamsungE,A8:32:9ADigicomF,A8:34:6ASamsungE,A8:35:12HuaweiDe,A8:36:7Afrogblue,A8:37:59HuaweiDe,A8:39:44Actionte,A8:3B:5CHuaweiTe,A8:3C:CBRossma,A8:3E:0EHMDGloba,A8:3F:A1IEEERegi,A8:3F:A1:00:00:00/28ImeconEn,A8:3F:A1:10:00:00/28GTDevice,A8:3F:A1:20:00:00/28Medcapta,A8:3F:A1:30:00:00/28Guangzho,A8:3F:A1:40:00:00/28Zhejiang,A8:3F:A1:50:00:00/28Sercomm,A8:3F:A1:60:00:00/28Beglec,A8:3F:A1:70:00:00/28Plejd,A8:3F:A1:80:00:00/28NeosVent,A8:3F:A1:90:00:00/28Shenzhen,A8:3F:A1:A0:00:00/28Shanghai,A8:3F:A1:B0:00:00/28Exelunip,A8:3F:A1:C0:00:00/28Laonz,A8:3F:A1:D0:00:00/28Shenzhen,A8:3F:A1:E0:00:00/28Guangzho,A8:40:25OxideCom,A8:40:41DraginoT,A8:40:7DGDMideaA,A8:41:22ChinaMob,A8:42:A7JiangsuH,A8:43:97Innogrit,A8:44:81Nokia,A8:45:CDSiselect,A8:45:E9FirichEn,A8:46:9DCiscoMer,A8:47:4AHonHaiPr,A8:48:FAEspressi,A8:49:4DHuaweiTe,A8:49:A5Lisantec,A8:4A:28Apple,A8:4A:63TPVDispl,A8:4B:4DSamsungE,A8:4D:4AAudiowis,A8:4E:3FHitronTe,A8:50:81HuaweiTe,A8:51:5BSamsungE,A8:54:B2WistronN,A8:55:6A3SSystem,A8:57:4ETp-LinkT,A8:58:40Cambridg,A8:5A:E0HuaweiDe,A8:5A:F3Shanghai,A8:5B:36IEEERegi,A8:5B:36:00:00:00/28BluesooT,A8:5B:36:10:00:00/28Parma,A8:5B:36:20:00:00/28Loomanet,A8:5B:36:30:00:00/28Shenzhen,A8:5B:36:40:00:00/28LuoxianG,A8:5B:36:50:00:00/28Juganu,A8:5B:36:60:00:00/28Dap,A8:5B:36:70:00:00/28LouisVui,A8:5B:36:80:00:00/28ShangHai,A8:5B:36:90:00:00/28AvistaEd,A8:5B:36:A0:00:00/28TaidenIn,A8:5B:36:B0:00:00/28Lampyris,A8:5B:36:C0:00:00/28ATERTech,A8:5B:36:D0:00:00/28AdamHall,A8:5B:36:E0:00:00/28ORBITVUS,A8:5B:6CRobertBo,A8:5B:78Apple,A8:5B:B0Shenzhen,A8:5B:F3Audivo,A8:5B:F7ArubaaHe,A8:5C:2CApple,A8:5E:45ASUSTekC,A8:5E:E412SidedT,A8:60:B6Apple,A8:61:0AArduino,A8:61:AACloudvie,A8:61:DFChinaMob,A8:62:A2Jiwumedi,A8:63:7DD-LinkIn,A8:63:DFDisplair,A8:63:F2TexasIns,A8:64:05nimbus9,A8:64:F1IntelCor,A8:65:B2Dongguan,A8:66:7FApple,A8:67:1ERatp,A8:69:8COracle,A8:6A:6FRim,A8:6A:BBSagemcom,A8:6A:C1HanbitED,A8:6B:7CShenzhen,A8:6B:ADHonHaiPr,A8:6D:5FRaisecom,A8:6D:AAIntelCor,A8:6E:4EHuaweiDe,A8:70:5DARRISGro,A8:70:A5UniComm,A8:72:85Idt,A8:74:1DPHOENIXC,A8:74:84zte,A8:75:D6FreeTekI,A8:75:E2Aventura,A8:76:50SamsungE,A8:77:6FZonoff,A8:77:E5Shenzhen,A8:7B:39Nokia,A8:7C:01SamsungE,A8:7D:12HuaweiTe,A8:7E:33NokiaDan,A8:7E:EAIntelCor,A8:80:38ShenZhen,A8:81:7EApple,A8:81:95SamsungE,A8:81:F1Bmeye,A8:82:00HisenseE,A8:82:7FCIBNOrie,A8:85:D7SangforT,A8:86:DDApple,A8:87:92Broadban,A8:87:B3SamsungE,A8:87:EDARCWirel,A8:88:08Apple,A8:89:40HuaweiDe,A8:8C:EEMicroMad,A8:8D:7BSunDroid,A8:8E:24Apple,A8:90:08BeijingY,A8:90:42BeijingW,A8:91:3DApple,A8:92:2CLGElectr,A8:93:4AChongqin,A8:93:52Shanghai,A8:93:E6JiangxiJ,A8:95:B0AkerSubs,A8:96:75Motorola,A8:96:8AApple,A8:97:CDARRISGro,A8:97:DCIbm,A8:98:92Guangdon,A8:98:C6Shinbo,A8:99:5Caizo,A8:99:69Dell,A8:99:DCi-TOPDES,A8:9A:93Sagemcom,A8:9A:D7Nokia,A8:9B:10inMotion,A8:9C:A4Furrion,A8:9C:EDXiaomiCo,A8:9D:21Cisco,A8:9D:D2Shanghai,A8:9F:BASamsungE,A8:9F:ECARRISGro,A8:A0:89Tactical,A8:A0:97ScioTeqb,A8:A1:59ASRockIn,A8:A1:98TCTmobil,A8:A5:E2MSF-Vath,A8:A6:48QingdaoH,A8:A6:68zte,A8:A7:95HonHaiPr,A8:AD:3DAlcatel-,A8:B0:88eero,A8:B0:AELeoni,A8:B1:D4Cisco,A8:B2:DAFujitsu,A8:B4:56Cisco,A8:B5:7CRoku,A8:B8:6ELGElectr,A8:B9:B3Essys,A8:BB:50WiZIoT,A8:BB:CFApple,A8:BC:9CCloudLig,A8:BD:1AHoneyBee,A8:BD:27HewlettP,A8:BD:3AUnionman,A8:BE:27Apple,A8:BF:3CHDVPhoel,A8:C0:92HuaweiDe,A8:C0:EAPepwave,A8:C2:22TM-Resea,A8:C2:52HuaweiDe,A8:C2:66HUMAX,A8:C8:3AHuaweiTe,A8:C8:7FRoqos,A8:CA:7BHuaweiTe,A8:CA:B9SamsungE,A8:CB:95EastBest,A8:CC:6FHMDGloba,A8:CC:C5Saabpubl,A8:CE:90Cvc,A8:D0:81HuaweiDe,A8:D0:E3SystechE,A8:D0:E5JuniperN,A8:D2:36Lightwar,A8:D3:C8TopconEl,A8:D3:F7Arcadyan,A8:D4:09USA111,A8:D4:98AviraOpe,A8:D4:E0HuaweiTe,A8:D5:79BeijingC,A8:D8:28Ascensia,A8:D8:8AWyconn,A8:DA:01Shenzhen,A8:DA:0CServerco,A8:DB:03SamsungE,A8:E0:18Nokia,A8:E2:C1TexasIns,A8:E2:C3Shenzhen,A8:E3:EESonyInte,A8:E5:39Moimston,A8:E5:44HuaweiTe,A8:E5:52JUWELAqu,A8:E7:05Fiberhom,A8:E7:7DTexasIns,A8:E8:1EAtwTechn,A8:E8:24InimElec,A8:E9:78HuaweiDe,A8:EE:C6Muuselab,A8:EF:26Tritonwa,A8:F0:38ShenZhen,A8:F2:66HuaweiDe,A8:F2:74SamsungE,A8:F4:70FujianNe,A8:F5:ACHuaweiTe,A8:F5:DDARRISGro,A8:F7:66ITETech,A8:F7:E0PLANETTe,A8:F9:4BEltexEnt,A8:FA:D8Apple,A8:FB:70WiseSecL,A8:FC:B7Consolid,A8:FE:9DApple,A8:FF:BAHuaweiTe,AA:00:00DecObsol,AA:00:01DecObsol,AA:00:02DecObsol,AA:00:03DecGloba,AA:00:04DecLocal,AA:0F:85Yamaichi,AA:34:D4CDVIWire,AA:38:C7Kaloom,AA:54:8BTintri,AA:DC:47Cachengo,AC:00:D0zte,AC:01:42UrielTec,AC:02:CAHISoluti,AC:02:CFRWTecnol,AC:02:EFComsis,AC:04:0BPelotonI,AC:04:81JiangsuH,AC:06:13Senselog,AC:06:C7ServerNe,AC:07:5FHuaweiTe,AC:0A:61Labor,AC:0B:FBEspressi,AC:0D:1BLGElectr,AC:0D:FEEkon-myG,AC:11:D3SuzhouHO,AC:12:03IntelCor,AC:12:2FFantasia,AC:13:9CAdtran,AC:14:61ATAW,AC:14:D2wi-daq,AC:15:85silergy,AC:15:F4Apple,AC:16:2DHewlettP,AC:17:02FibarGro,AC:17:C8CiscoMer,AC:18:26SeikoEps,AC:19:9FSungrowP,AC:1D:06Apple,AC:1D:DFIEEERegi,AC:1D:DF:00:00:00/28PiOctave,AC:1D:DF:10:00:00/28HellaSto,AC:1D:DF:20:00:00/28ConectaI,AC:1D:DF:30:00:00/28Crde,AC:1D:DF:40:00:00/28Motec,AC:1D:DF:50:00:00/28Shenzhen,AC:1D:DF:60:00:00/28Shenzhen,AC:1D:DF:70:00:00/28GreenITK,AC:1D:DF:80:00:00/28SichuanO,AC:1D:DF:90:00:00/28SolareDa,AC:1D:DF:A0:00:00/28WescoInt,AC:1D:DF:B0:00:00/28FINEpowe,AC:1D:DF:C0:00:00/28BeijingC,AC:1D:DF:D0:00:00/28Elekon,AC:1D:DF:E0:00:00/28Duravit,AC:1E:92SamsungE,AC:1E:D0TemicAut,AC:1F:09shenzhen,AC:1F:0FTexasIns,AC:1F:6BSuperMic,AC:1F:74Apple,AC:1F:D7RealVisi,AC:20:2EHitronTe,AC:20:3EWuhanTia,AC:20:AADMATEK,AC:22:05CompalBr,AC:22:0BASUSTekC,AC:23:16Mist,AC:23:34Infinixm,AC:23:3FShenzhen,AC:29:3AApple,AC:2A:0CCsrZhuzh,AC:2B:6EIntelCor,AC:2D:A3TXTR,AC:2D:A9TecnoMob,AC:2F:A8Humannix,AC:31:9DShenzhen,AC:33:28HuaweiDe,AC:34:CBShanhaiG,AC:35:EEFn-LinkT,AC:36:13SamsungE,AC:36:51JiangsuH,AC:37:28TaicangT,AC:37:43HTC,AC:37:C9RAID,AC:38:70LenovoMo,AC:3A:67Cisco,AC:3A:7ARoku,AC:3B:77Sagemcom,AC:3C:0BApple,AC:3C:8EFlextron,AC:3C:B4Nilan,AC:3D:05Instores,AC:3D:75Hangzhou,AC:3F:A4TaiyoYud,AC:40:EAC&TSolut,AC:41:22EclipseE,AC:42:28PartaNet,AC:43:30VersaNet,AC:44:F2Yamaha,AC:47:1BHuaweiDe,AC:47:23Genelec,AC:48:2DRalinwiN,AC:49:DBApple,AC:4A:56Cisco,AC:4A:67Cisco,AC:4A:FEHisenseB,AC:4B:1EIntegri-,AC:4B:C8JuniperN,AC:4D:16TexasIns,AC:4E:2EShenzhen,AC:4E:91HuaweiTe,AC:4F:FCSVS-VIST,AC:50:36Pi-Coral,AC:50:93MagnaEle,AC:51:2CInfinixm,AC:51:35MpiTech,AC:51:EECambridg,AC:54:74ChinaMob,AC:54:ECIEEEP182,AC:56:2CLavaInte,AC:57:75HMDGloba,AC:58:3BHumanAss,AC:58:7BJCTHealt,AC:5A:14SamsungE,AC:5A:EEChinaMob,AC:5A:FCIntelCor,AC:5D:10PaceAmer,AC:5D:5CFn-LinkT,AC:5E:14HuaweiTe,AC:5E:8CUtillink,AC:5F:3ESamsungE,AC:5F:EAOnePlusT,AC:60:89HuaweiTe,AC:60:B6Ericsson,AC:61:23Drivven,AC:61:75HuaweiTe,AC:61:B9WAMATech,AC:61:EAApple,AC:62:0DJabilCir,AC:63:BEAmazonTe,AC:64:17Siemens,AC:64:62zte,AC:64:90HuaweiTe,AC:64:CFFn-LinkT,AC:64:DDIEEERegi,AC:64:DD:00:00:00/28Jia-Teng,AC:64:DD:10:00:00/28InfoTeCS,AC:64:DD:20:00:00/28Shenzhen,AC:64:DD:30:00:00/28infypowe,AC:64:DD:40:00:00/288Cups,AC:64:DD:50:00:00/28Shanghai,AC:64:DD:60:00:00/28Kpnetwor,AC:64:DD:70:00:00/28Wittmann,AC:64:DD:80:00:00/28PfdcElan,AC:64:DD:90:00:00/28MicroCon,AC:64:DD:A0:00:00/28Bluewave,AC:64:DD:B0:00:00/28GroupeCi,AC:64:DD:C0:00:00/28BeijingH,AC:64:DD:D0:00:00/28HMicro,AC:64:DD:E0:00:00/28Digibird,AC:67:06RuckusWi,AC:67:5DIntelCor,AC:67:6FElectroc,AC:67:84Google,AC:67:B2Espressi,AC:6A:A3Shenzhen,AC:6B:0FCadenceD,AC:6B:ACJennySci,AC:6C:90SamsungE,AC:6E:1AShenzhen,AC:6F:4FEnspert,AC:6F:BBTATUNGTe,AC:6F:D9Valueplu,AC:71:0CChinaMob,AC:72:36LexkingT,AC:72:89IntelCor,AC:74:09Hangzhou,AC:74:B1IntelCor,AC:74:C4Maytroni,AC:75:1DHuaweiTe,AC:76:4CGuangdon,AC:77:13Honeywel,AC:78:D1JuniperN,AC:7A:42iConnect,AC:7A:4DAlpsalpi,AC:7A:56Cisco,AC:7B:A1IntelCor,AC:7E:01HuaweiDe,AC:7E:8ACisco,AC:7F:3EApple,AC:80:D6Hexatron,AC:81:12GemtekTe,AC:81:F3Nokia,AC:82:26QingdaoH,AC:82:47IntelCor,AC:83:17Shenzhen,AC:83:E9BeijingZ,AC:83:F0ImmediaT,AC:83:F3AMPAKTec,AC:84:C6Tp-LinkT,AC:84:C9Sagemcom,AC:85:3DHuaweiTe,AC:86:74OpenMesh,AC:86:7ECreateNe,AC:87:A3Apple,AC:88:FDApple,AC:89:95AzureWav,AC:8A:CDROGERDWe,AC:8B:6AChinaMob,AC:8B:9CPrimeraT,AC:8B:A9Ubiquiti,AC:8D:14Smartrov,AC:8D:34HuaweiTe,AC:8F:F8Nokia,AC:90:85Apple,AC:92:32HuaweiTe,AC:93:2FNokia,AC:94:03Envision,AC:95:72Jovision,AC:97:6CGreenlia,AC:99:29HuaweiTe,AC:9A:22NXPSemic,AC:9A:96Maxlinea,AC:9B:0ASony,AC:9B:84SmakTecn,AC:9C:E4Alcatel-,AC:9E:17ASUSTekC,AC:A0:16Cisco,AC:A2:13Shenzhen,AC:A2:2CBaycityT,AC:A3:1EArubaaHe,AC:A4:30Peerless,AC:A4:6EShenzhen,AC:A6:67Electron,AC:A8:8ESHARP,AC:A9:19TrekStor,AC:A9:A0Audioeng,AC:AB:2EBeijingL,AC:AB:8DLyngsoMa,AC:AB:BFAthenTek,AC:AC:E2Changhon,AC:AE:19Roku,AC:AF:B9SamsungE,AC:B1:EEShenzhen,AC:B3:13ARRISGro,AC:B3:B5HuaweiTe,AC:B5:7DLiteonTe,AC:B7:4FMETELsro,AC:B8:59UnibandE,AC:B9:2FHangzhou,AC:BB:61YSTenTec,AC:BC:32Apple,AC:BC:D9Cisco,AC:BD:0BLeimac,AC:BD:70HuaweiDe,AC:BE:75UfineTec,AC:BE:B6Visualed,AC:C1:EEXiaomiCo,AC:C2:5DFiberhom,AC:C2:ECCltIntLI,AC:C3:3ASamsungE,AC:C3:58Continen,AC:C5:1BZhuhaiPa,AC:C5:95Graphite,AC:C6:62MitraSta,AC:C6:98KohzuPre,AC:C7:3FVitsmo,AC:C9:35Ness,AC:CA:54TelldusT,AC:CA:8EODATechn,AC:CA:ABVirtualE,AC:CA:BAMidokura,AC:CB:09HefcomMe,AC:CB:51Hangzhou,AC:CC:8EAxisComm,AC:CE:8FHwaYaoTe,AC:CF:23Hi-flyin,AC:CF:5CApple,AC:CF:85HuaweiTe,AC:D0:74Espressi,AC:D1:80Crexendo,AC:D1:B8HonHaiPr,AC:D3:64AbbAbbSa,AC:D5:64Chongqin,AC:D6:18OnePlusT,AC:D6:57ShaanxiG,AC:D8:29Bouffalo,AC:D9:D6tci,AC:DB:48ARRISGro,AC:DB:DAShenzhen,AC:DC:CAHuaweiTe,AC:DC:E5ProcterG,AC:DE:48Private,AC:E0:10LiteonTe,AC:E0:69ISAACIns,AC:E1:4FAutonomi,AC:E2:15HuaweiTe,AC:E2:D3HewlettP,AC:E3:42HuaweiTe,AC:E3:48MadgeTec,AC:E4:2ESKhynix,AC:E4:B5Apple,AC:E5:F0DopplerL,AC:E6:4BShenzhen,AC:E7:7BSichuanT,AC:E8:7BHuaweiTe,AC:E8:7EBytemark,AC:E9:7FIoTTech,AC:E9:AAHay,AC:EA:6AGenixInf,AC:EB:51Universa,AC:EC:80ARRISGro,AC:EC:85eero,AC:ED:5CIntelCor,AC:EE:3B6harmoni,AC:EE:70FontemVe,AC:EE:9ESamsungE,AC:F0:B2BeckerEl,AC:F1:08LGInnote,AC:F1:DFD-LinkIn,AC:F2:C5Cisco,AC:F5:E6Cisco,AC:F6:F7LGElectr,AC:F7:F3XiaomiCo,AC:F8:5CChengduH,AC:F8:CCARRISGro,AC:F9:70HuaweiTe,AC:F9:7EElesys,AC:FA:A5digitron,AC:FD:93WeifangG,AC:FD:CEIntelCor,AC:FD:ECApple,AC:FE:05ItelMobi,B0:00:73WistronN,B0:00:B4Cisco,B0:02:47AMPAKTec,B0:02:7EMullerSe,B0:05:94LiteonTe,B0:08:75HuaweiTe,B0:08:BFVitalCon,B0:09:D3Avizia,B0:09:DARingSolu,B0:0A:D5zte,B0:0C:D1HewlettP,B0:10:41HonHaiPr,B0:10:A0TexasIns,B0:12:03Dynamics,B0:12:66Futaba-K,B0:14:08Lightspe,B0:16:56HuaweiTe,B0:17:43EdisonGl,B0:18:86SmarDTV,B0:19:C6Apple,B0:1B:7COntrol,B0:1B:D2LeShiZhi,B0:1C:0CShenzhen,B0:1C:91Elim,B0:1F:29Helvetia,B0:1F:81IEEERegi,B0:1F:81:00:00:00/28DalianGi,B0:1F:81:10:00:00/28UvaxConc,B0:1F:81:20:00:00/28Private,B0:1F:81:30:00:00/28SoundUni,B0:1F:81:40:00:00/28Shenzhen,B0:1F:81:50:00:00/28Shenzhen,B0:1F:81:60:00:00/28COMOTA,B0:1F:81:70:00:00/28AetherSe,B0:1F:81:80:00:00/28Technion,B0:1F:81:90:00:00/28CIDEInte,B0:1F:81:A0:00:00/28Steffens,B0:1F:81:B0:00:00/28Rademach,B0:1F:81:C0:00:00/28AccessDe,B0:1F:81:D0:00:00/28TAIWANAn,B0:1F:81:E0:00:00/28Advanced,B0:1F:81:F0:00:00/28Private,B0:22:7AHP,B0:24:91HuaweiDe,B0:24:F3Progeny,B0:25:AAPrivate,B0:26:28Broadcom,B0:26:80Cisco,B0:27:CFExtremeN,B0:2A:1FWingtech,B0:2A:43Google,B0:30:55ChinaMob,B0:30:C8TealDron,B0:33:66vivoMobi,B0:33:A6JuniperN,B0:34:95Apple,B0:35:0BMobiwire,B0:35:8DNokia,B0:35:9FIntelCor,B0:35:B5Apple,B0:37:95LGElectr,B0:38:29Siliconw,B0:38:50NanjingC,B0:39:56Netgear,B0:3A:CEHuaweiDe,B0:3D:96VisionVa,B0:3D:C2Wasparti,B0:3E:51BSkyB,B0:3E:B0MICRODIA,B0:40:89Senient,B0:41:1DITTIMTec,B0:41:6FShenzhen,B0:43:5DNuLEDs,B0:44:14NewH3CTe,B0:45:02HuaweiDe,B0:45:15mirafitn,B0:45:19TCTmobil,B0:45:30BSkyB,B0:45:45YACOUBAu,B0:46:92Guangdon,B0:46:FCMitraSta,B0:47:BFSamsungE,B0:48:1AApple,B0:48:7ATp-LinkT,B0:49:5FOMRONHEA,B0:4A:39BeijingR,B0:4B:BFPtHanSun,B0:4C:05Freseniu,B0:4E:26Tp-LinkT,B0:4F:13Dell,B0:4F:C3Shenzhen,B0:50:BCShenzhen,B0:51:8EHolltech,B0:52:16HonHaiPr,B0:53:65ChinaMob,B0:55:08HuaweiTe,B0:57:06Vallox,B0:58:C4Broadcas,B0:59:47Shenzhen,B0:5A:DAHewlettP,B0:5B:1FThermoFi,B0:5B:67HuaweiTe,B0:5C:DAHP,B0:5C:E5Nokia,B0:5D:D4ARRISGro,B0:60:88IntelCor,B0:61:C7Ericsson,B0:65:63Shanghai,B0:65:BDApple,B0:65:F1WIOManuf,B0:67:2FBowersWi,B0:68:B6Hangzhou,B0:68:E6Chongqin,B0:69:71DEISales,B0:6A:41Google,B0:6C:BF3alityDi,B0:6E:BFASUSTekC,B0:6F:E0SamsungE,B0:70:0DNokia,B0:70:2DApple,B0:72:BFMurataMa,B0:73:5DHuaweiDe,B0:73:9CAmazonTe,B0:75:0CQACafe,B0:75:4DNokia,B0:75:D5zte,B0:76:1BHuaweiTe,B0:77:ACARRISGro,B0:78:70Wi-NEXT,B0:78:F0BeijingH,B0:79:08Cummings,B0:79:3CRevolv,B0:79:94Motorola,B0:7B:25Dell,B0:7D:47Cisco,B0:7D:62Dipl-Ing,B0:7D:64IntelCor,B0:7E:11TexasIns,B0:7E:70ZadaraSt,B0:7F:B9Netgear,B0:80:8CLaserLig,B0:81:D8I-sys,B0:83:D6ARRISGro,B0:83:FEDell,B0:86:9EChloride,B0:88:07StrataWo,B0:89:00HuaweiTe,B0:89:91Lge,B0:89:C2Zyptonit,B0:8B:CFCisco,B0:8B:D0Cisco,B0:8C:75Apple,B0:8E:1AURadio,B0:90:74FulanEle,B0:90:7ECisco,B0:90:D4Shenzhen,B0:91:22TexasIns,B0:91:34Taleo,B0:91:37ISisImag,B0:93:5BARRISGro,B0:95:75Tp-LinkT,B0:95:8ETp-LinkT,B0:96:6CLanbowan,B0:97:3AE-Fuel,B0:98:2BSagemcom,B0:98:9FLgCns,B0:98:BCHuaweiDe,B0:99:28Fujitsu,B0:9A:E2STEMMERI,B0:9B:D4GNHSoftw,B0:9F:BAApple,B0:A1:0APivotal,B0:A2:E7Shenzhen,B0:A3:7EQingDaoH,B0:A4:54Tripwire,B0:A4:60IntelCor,B0:A6:51Cisco,B0:A6:F5Xaptum,B0:A7:2AEnsemble,B0:A7:37Roku,B0:A7:B9TP-Link,B0:A8:6EJuniperN,B0:AA:36Guangdon,B0:AA:77Cisco,B0:AA:D2Sichuant,B0:AC:D2zte,B0:AC:FAFujitsu,B0:AD:AAAvaya,B0:AE:25Varikore,B0:B1:13TexasIns,B0:B1:94zte,B0:B2:8FSagemcom,B0:B2:DCZyxelCom,B0:B3:2BSlicanSp,B0:B3:53IEEERegi,B0:B3:53:00:00:00/28BlakeUK,B0:B3:53:10:00:00/28Sprocomm,B0:B3:53:20:00:00/28RizhaoSU,B0:B3:53:30:00:00/28AdHocDev,B0:B3:53:40:00:00/28InnotasE,B0:B3:53:50:00:00/28Zenlayer,B0:B3:53:60:00:00/28Hangzhou,B0:B3:53:70:00:00/28WuukLabs,B0:B3:53:80:00:00/28Voxiscom,B0:B3:53:90:00:00/28Hanmecip,B0:B3:53:A0:00:00/28Ledger,B0:B3:53:B0:00:00/28Zoox,B0:B3:53:C0:00:00/28BeijingG,B0:B3:53:D0:00:00/28IPvideo,B0:B3:53:E0:00:00/28NanjingY,B0:B3:ADHUMAX,B0:B4:48TexasIns,B0:B5:C3Guangdon,B0:B5:E8Ruroc,B0:B8:67HewlettP,B0:B8:D5NanjingN,B0:B9:8ANetgear,B0:BB:8BWavetelT,B0:BB:E5Sagemcom,B0:BD:1BDongguan,B0:BD:6DEchostre,B0:BD:A1ZakladEl,B0:BE:76Tp-LinkT,B0:BE:83Apple,B0:BF:99Wizitdon,B0:C0:90ChiconyE,B0:C1:28AdlerELR,B0:C1:9Ezte,B0:C2:05Bionime,B0:C2:87Technico,B0:C3:87GOEFER,B0:C4:6CSenseit,B0:C4:E7SamsungE,B0:C5:3CCisco,B0:C5:54D-LinkIn,B0:C5:59SamsungE,B0:C5:CAIEEERegi,B0:C5:CA:00:00:00/28EM-Tech,B0:C5:CA:10:00:00/28Ivk-Saya,B0:C5:CA:20:00:00/28LOWOTEC,B0:C5:CA:30:00:00/28abode,B0:C5:CA:40:00:00/28shanghai,B0:C5:CA:50:00:00/28Systovi,B0:C5:CA:60:00:00/28SunTechM,B0:C5:CA:70:00:00/28Shenzhen,B0:C5:CA:80:00:00/28Astyx,B0:C5:CA:90:00:00/28D&T,B0:C5:CA:A0:00:00/28TEMMobil,B0:C5:CA:B0:00:00/28Risecomm,B0:C5:CA:C0:00:00/28XMetrics,B0:C5:CA:D0:00:00/28Private,B0:C5:CA:E0:00:00/28AudioEle,B0:C5:CA:F0:00:00/28Private,B0:C6:9AJuniperN,B0:C7:45Buffalo,B0:C7:87HuaweiTe,B0:C8:3FJiangsuC,B0:C8:ADPeoplePo,B0:C9:52Guangdon,B0:C9:5BBeijingS,B0:CA:68Apple,B0:CC:FEHuaweiDe,B0:CE:18Zhejiang,B0:CF:4DMI-ZoneT,B0:D0:9CSamsungE,B0:D2:F5Vello,B0:D5:68Shenzhen,B0:D5:9DShenzhen,B0:D5:CCTexasIns,B0:D7:C5Logipix,B0:D7:CCTridonic,B0:D8:88Panasoni,B0:DA:00CeraElec,B0:DA:F9ARRISGro,B0:DF:3ASamsungE,B0:DF:C1TendaTec,B0:E0:3CTCTmobil,B0:E1:7EHuaweiTe,B0:E2:35XiaomiCo,B0:E2:E5Fiberhom,B0:E3:9DCatSyste,B0:E4:D5Google,B0:E5:0ENrg,B0:E5:EDHuaweiTe,B0:E5:F9Apple,B0:E7:1DShanghai,B0:E7:542Wire,B0:E7:DEHomaTech,B0:E8:92SeikoEps,B0:E9:7EAdvanced,B0:E9:FEWoanTech,B0:EA:BCAskeyCom,B0:EB:57HuaweiTe,B0:EC:71SamsungE,B0:EC:8FGmxSas,B0:EC:DDHuaweiTe,B0:EC:E1Private,B0:EE:45AzureWav,B0:EE:7BRoku,B0:F1:A3FengfanB,B0:F1:BCDhemaxIn,B0:F1:ECAMPAKTec,B0:F5:30HitronTe,B0:F7:C4AmazonTe,B0:F8:93Shanghai,B0:F9:63Hangzhou,B0:FA:EBCisco,B0:FC:0DAmazonTe,B0:FC:36CyberTAN,B0:FD:0BIEEERegi,B0:FD:0B:00:00:00/28TAEHYUNG,B0:FD:0B:10:00:00/28IDspire,B0:FD:0B:20:00:00/28VistaMan,B0:FD:0B:30:00:00/28DMACSecu,B0:FD:0B:40:00:00/28FasiiInf,B0:FD:0B:50:00:00/28TaianYuq,B0:FD:0B:60:00:00/28DNESOTEN,B0:FD:0B:70:00:00/28Everynet,B0:FD:0B:80:00:00/28eSenseLa,B0:FD:0B:90:00:00/28EagleAco,B0:FD:0B:A0:00:00/28TemcoJap,B0:FD:0B:B0:00:00/28MartinLo,B0:FD:0B:C0:00:00/28HaltianP,B0:FD:0B:D0:00:00/28HabanaLa,B0:FD:0B:E0:00:00/28Shenzhen,B0:FE:BDPrivate,B0:FE:E5HuaweiDe,B4:00:16Ingenico,B4:00:9CCableWor,B4:01:42GCIScien,B4:02:16Cisco,B4:04:18Smartchi,B4:05:5DInspurEl,B4:05:66SPBest,B4:07:F9SamsungE,B4:08:32TCCommun,B4:09:31HuaweiTe,B4:0A:C6DEXON,B4:0B:44Smartisa,B4:0B:78BrusaEle,B4:0B:7ABrusaEle,B4:0C:25PaloAlto,B4:0E:96Heran,B4:0E:CFBouffalo,B4:0E:DCLG-Erics,B4:0E:DEIntelCor,B4:0F:3BTendaTec,B4:0F:B3vivoMobi,B4:10:7BTexasIns,B4:14:89Cisco,B4:14:E6HuaweiTe,B4:15:13HuaweiTe,B4:15:7ECelona,B4:17:80DTIGroup,B4:18:D1Apple,B4:1A:1DSamsungE,B4:1B:B0Apple,B4:1C:30zte,B4:1C:ABICR,B4:1D:2BShenzhen,B4:1D:EFInternet,B4:20:46eero,B4:20:5BGuangdon,B4:21:1DBeijingG,B4:21:8ADogHunte,B4:22:00BrotherI,B4:23:30Itron,B4:24:E7CodetekT,B4:26:5DTaicangT,B4:28:F1E-Prime,B4:29:3DShenzhen,B4:2A:0ETechnico,B4:2A:39ORBITMER,B4:2C:92Zhejiang,B4:2C:BEDirectPa,B4:2D:56ExtremeN,B4:2E:99Giga-Byt,B4:2E:F8ElineTec,B4:30:52HuaweiTe,B4:30:C0YorkInst,B4:31:B8Aviwest,B4:34:6CMatsunic,B4:35:64FujianTi,B4:35:F7Zhejiang,B4:36:A9FibocomW,B4:36:D1RenesasE,B4:36:E3Kbvision,B4:37:41Consert,B4:37:D1IEEERegi,B4:37:D1:00:00:00/28LezyneUS,B4:37:D1:10:00:00/28AlturnaN,B4:37:D1:20:00:00/28Fibersys,B4:37:D1:30:00:00/28Dimton,B4:37:D1:40:00:00/28KomsisEl,B4:37:D1:50:00:00/28Stratom,B4:37:D1:60:00:00/28YirehAut,B4:37:D1:70:00:00/28GEPowerM,B4:37:D1:80:00:00/28eInfochi,B4:37:D1:90:00:00/28Nanjingy,B4:37:D1:A0:00:00/28Axiomati,B4:37:D1:B0:00:00/28NSI,B4:37:D1:C0:00:00/28NanjingP,B4:37:D1:D0:00:00/28ZXYSport,B4:37:D1:E0:00:00/28UnionTec,B4:37:D1:F0:00:00/28Private,B4:37:D8D-LinkSh,B4:39:34PenGener,B4:39:39Shenzhen,B4:39:D6ProCurve,B4:3A:28SamsungE,B4:3D:08GXIntern,B4:3D:B2Degreane,B4:3E:3BViablewa,B4:40:A4Apple,B4:41:7AShenzhen,B4:43:0DBroadlin,B4:43:26HuaweiTe,B4:47:5EAvaya,B4:47:F5EardaTec,B4:4B:D2Apple,B4:4B:D6IEEERegi,B4:4B:D6:00:00:00/28G4SMonit,B4:4B:D6:10:00:00/28Shenzhen,B4:4B:D6:20:00:00/28Shenzhen,B4:4B:D6:30:00:00/28HuizhouS,B4:4B:D6:40:00:00/28Shenzhen,B4:4B:D6:50:00:00/28ShenZhen,B4:4B:D6:60:00:00/28Perspica,B4:4B:D6:70:00:00/28Taizhouc,B4:4B:D6:80:00:00/28ArnouseD,B4:4B:D6:90:00:00/28QstarTec,B4:4B:D6:A0:00:00/28Shenzhen,B4:4B:D6:B0:00:00/28DongYoun,B4:4B:D6:C0:00:00/28Impakt,B4:4B:D6:D0:00:00/28ElletaSo,B4:4B:D6:E0:00:00/28Chunghsi,B4:4C:3BZhejiang,B4:4C:C2NrElectr,B4:4F:96Zhejiang,B4:50:62EmBestor,B4:51:F9NBSoftwa,B4:52:53SeagateT,B4:52:7DSony,B4:52:7ESony,B4:52:A9TexasIns,B4:54:59ChinaMob,B4:55:70Borea,B4:56:B9Teraspek,B4:56:E3Apple,B4:58:61CRemote,B4:5C:A4Thing-ta,B4:5D:50ArubaaHe,B4:5F:84zte,B4:60:77SichuanC,B4:60:8CFiberhom,B4:60:EDBeijingX,B4:61:FFLumigon,B4:62:38Exablox,B4:62:93SamsungE,B4:62:ADElysiaGe,B4:66:98Zealabs,B4:67:E9QingdaoG,B4:69:21IntelCor,B4:6B:FCIntelCor,B4:6C:47Panasoni,B4:6D:35DalianSe,B4:6D:83IntelCor,B4:6E:08HuaweiTe,B4:6F:2DWahooFit,B4:73:56Hangzhou,B4:74:43SamsungE,B4:74:47CoreOS,B4:74:9FAskeyCom,B4:75:0EBelkinIn,B4:77:48Shenzhen,B4:79:47Nutanix,B4:79:A7SamsungE,B4:79:C8RuckusWi,B4:7A:F1HewlettP,B4:7C:29Shenzhen,B4:7C:59JiangsuH,B4:7C:9CAmazonTe,B4:7F:5EForesigh,B4:81:07Shenzhen,B4:81:BFMeta-Net,B4:82:55Research,B4:82:7BAKGAcous,B4:82:C5Relay2,B4:82:FEAskeyCom,B4:85:47AmptownS,B4:85:E1Apple,B4:86:55HuaweiTe,B4:89:01HuaweiTe,B4:89:10CosterTE,B4:8A:5FJuniperN,B4:8B:19Apple,B4:94:4EWeTeleco,B4:96:91IntelCor,B4:98:42zte,B4:99:4CTexasIns,B4:99:BAHewlettP,B4:9A:95Shenzhen,B4:9C:DFApple,B4:9D:02SamsungE,B4:9D:0BBq,B4:9D:B4AxionTec,B4:9E:80SichuanC,B4:9E:ACImagikIn,B4:9E:E6Shenzhen,B4:A2:5CCambiumN,B4:A2:EBIEEERegi,B4:A2:EB:00:00:00/28QKMTechn,B4:A2:EB:10:00:00/28DCIInter,B4:A2:EB:20:00:00/28Katerra,B4:A2:EB:30:00:00/28CanaanCr,B4:A2:EB:40:00:00/28Softel,B4:A2:EB:50:00:00/28Annapurn,B4:A2:EB:60:00:00/28ShenZhen,B4:A2:EB:70:00:00/28KonaI,B4:A2:EB:80:00:00/28Shenzhen,B4:A2:EB:90:00:00/28CurrentW,B4:A2:EB:A0:00:00/28Hengkang,B4:A2:EB:B0:00:00/28Quantite,B4:A2:EB:C0:00:00/28Shanghai,B4:A2:EB:D0:00:00/28SALZBREN,B4:A2:EB:E0:00:00/28Dongguan,B4:A3:05XiamenYa,B4:A3:82Hangzhou,B4:A4:B5ZenEye,B4:A4:E3Cisco,B4:A5:A9MODI,B4:A5:ACGuangdon,B4:A5:EFSercomm,B4:A8:28Shenzhen,B4:A8:2BHistarDi,B4:A8:98HuaweiDe,B4:A8:B9Cisco,B4:A9:4FMercury,B4:A9:5AAvaya,B4:A9:84Symantec,B4:A9:FCQuantaCo,B4:A9:FEGHIATech,B4:AA:4DEnsequen,B4:AB:2CMtMTechn,B4:AC:8CBernUniv,B4:AD:A3Guangzho,B4:AE:2BMicrosof,B4:AE:6FCircleRe,B4:B0:17Avaya,B4:B0:24TP-Link,B4:B0:55HuaweiTe,B4:B1:5ASiemensE,B4:B2:65DaehoI&T,B4:B2:91LGElectr,B4:B3:62zte,B4:B3:84ShenZhen,B4:B5:2FHewlettP,B4:B5:42HubbellP,B4:B5:AFMinsungE,B4:B5:B6Chongqin,B4:B6:76IntelCor,B4:B6:86HewlettP,B4:B8:59Texa,B4:B8:8DThuh,B4:BA:02Agatel,B4:BA:12ChinaMob,B4:BC:7CTexasIns,B4:BF:F6SamsungE,B4:C0:F5Shenzhen,B4:C1:70YichipMi,B4:C2:6AGarminIn,B4:C4:4EVXLeTech,B4:C4:76WuhanMar,B4:C4:FCXiaomiCo,B4:C6:2EMolexCMS,B4:C6:F8Axilspot,B4:C7:99ExtremeN,B4:C8:10Umpi,B4:C9:B9SichuanA,B4:CB:57Guangdon,B4:CC:04Piranti,B4:CC:E9Prosyst,B4:CD:27HuaweiTe,B4:CD:F5CubElecp,B4:CE:40SamsungE,B4:CE:F6HTC,B4:CE:FEJamesCze,B4:CF:DBShenzhen,B4:CF:E0Sichuant,B4:D0:A9ChinaMob,B4:D1:35Cloudist,B4:D5:BDIntelCor,B4:D6:4ECaldero,B4:D8:A9BetterBo,B4:D8:DEiotaComp,B4:DC:09Guangzho,B4:DD:15ControlT,B4:DD:D0Continen,B4:DE:31Cisco,B4:DE:DFzte,B4:DF:3BChromlec,B4:DF:FALitemaxE,B4:E0:1DConcepti,B4:E0:CDFusion-i,B4:E1:0FDell,B4:E1:C4Microsof,B4:E1:EBPrivate,B4:E3:F9SiliconL,B4:E4:54AmazonTe,B4:E6:2ALGInnote,B4:E6:2DEspressi,B4:E7:82Vivalnk,B4:E8:42HongKong,B4:E8:C9XADATech,B4:E9:A3portindu,B4:E9:B0Cisco,B4:EC:02Alpsalpi,B4:EC:F2Shanghai,B4:EC:FFWuhanIPG,B4:ED:19PieDigit,B4:ED:54WohlerTe,B4:EE:25Shenzhen,B4:EE:B4AskeyCom,B4:EE:D4TexasIns,B4:EF:04DAIHANSc,B4:EF:1C360AITec,B4:EF:39SamsungE,B4:EF:FALemobile,B4:F0:ABApple,B4:F1:8CHuaweiDe,B4:F1:DALGElectr,B4:F2:E8ARRISGro,B4:F3:23Petatel,B4:F5:8EHuaweiTe,B4:F6:1CApple,B4:F7:A1LGElectr,B4:F8:1EKinova,B4:F9:49optilink,B4:FA:48Apple,B4:FB:E3AltoBeam,B4:FB:E4Ubiquiti,B4:FB:F9HuaweiTe,B4:FC:75SEMAElec,B4:FE:8CCentroSi,B4:FF:98HuaweiTe,B8:00:18Htel,B8:02:A4Aeonsemi,B8:03:05IntelCor,B8:04:15BayanAud,B8:05:ABzte,B8:07:16vivoMobi,B8:07:56CiscoMer,B8:08:CFIntelCor,B8:08:D7HuaweiTe,B8:09:8AApple,B8:0B:9DROPEXInd,B8:10:D4Masimo,B8:11:4BCisco,B8:12:DALvswitch,B8:13:32AMPAKTec,B8:13:E9TraceLiv,B8:14:13KeenHigh,B8:14:5CHuaweiDe,B8:14:DBOhsung,B8:16:19ARRISGro,B8:16:DBChantSin,B8:17:C2Apple,B8:18:6FOriental,B8:19:04NokiaSha,B8:19:99Nesys,B8:1D:AALGElectr,B8:1F:5EApptionL,B8:20:8EPanasoni,B8:20:E7Guangzho,B8:22:4FSichuanT,B8:24:10MagnetiM,B8:24:1ASwedaInf,B8:24:F0SOYOTech,B8:25:9AThalmicL,B8:25:B5Trakm8,B8:26:6CANOVFran,B8:26:D4Furukawa,B8:27:C5HuaweiDe,B8:27:EBRaspberr,B8:28:8BParkerHa,B8:29:F7BlasterT,B8:2A:72Dell,B8:2A:A9Apple,B8:2A:DCEFREurop,B8:2C:A0Resideo,B8:2D:28AMPAKTec,B8:2F:CBCMSElect,B8:30:A8Road-Tra,B8:31:B5Microsof,B8:32:41WuhanTia,B8:36:D8Videoswi,B8:37:4AApple,B8:37:65Guangdon,B8:38:61Cisco,B8:38:CAKyokkoTs,B8:3A:08TendaTec,B8:3A:5AArubaaHe,B8:3A:7BWorldpla,B8:3A:9DAlarmcom,B8:3B:CCXiaomiCo,B8:3D:4EShenzhen,B8:3E:59Roku,B8:41:5FAsp,B8:41:A4Apple,B8:43:E4Vlatacom,B8:44:AETCTmobil,B8:44:D9Apple,B8:45:F4NewH3CTe,B8:47:7ADasanEle,B8:47:C6SanJetTe,B8:48:AAEMMicroe,B8:4D:43HunanFn-,B8:4D:EEHisenseb,B8:4F:D5Microsof,B8:50:01ExtremeN,B8:53:ACApple,B8:55:10ZioncomE,B8:56:00HuaweiTe,B8:56:BDItt,B8:57:76lignex1,B8:57:D8SamsungE,B8:58:10Numera,B8:59:9FMellanox,B8:59:CEEardaTec,B8:5A:73SamsungE,B8:5A:F7Ouya,B8:5A:FEHandaerC,B8:5D:0AApple,B8:5E:7BSamsungE,B8:5F:98AmazonTe,B8:5F:B0HuaweiTe,B8:60:91OnnetTec,B8:61:42BeijingT,B8:61:6FAcctonTe,B8:62:1FCisco,B8:63:4DApple,B8:63:92Guangdon,B8:63:BCROBOTIS,B8:64:91CKTeleco,B8:65:3BBolymin,B8:66:85Sagemcom,B8:69:C2SunitecE,B8:69:F4Routerbo,B8:6A:97Edgecore,B8:6B:23Toshiba,B8:6C:E8SamsungE,B8:70:F4CompalIn,B8:74:24Viessman,B8:74:47Converge,B8:75:C0PayPal,B8:76:3FHonHaiPr,B8:77:C3METERGro,B8:78:26Nintendo,B8:78:2EApple,B8:78:79RocheDia,B8:79:7ESecureMe,B8:7A:C9Siemens,B8:7B:C5Apple,B8:7C:6FNXPChina,B8:7C:F2ExtremeN,B8:80:35Shenzhen,B8:80:4FTexasIns,B8:81:98IntelCor,B8:81:FAApple,B8:83:03HewlettP,B8:85:7BHuaweiTe,B8:85:84Dell,B8:86:87LiteonTe,B8:87:1EGoodMind,B8:87:6EYandexSe,B8:87:A8StepAhea,B8:87:C6Prudenti,B8:88:E3CompalIn,B8:89:81ChengduI,B8:89:CAILJINELE,B8:8A:60IntelCor,B8:8A:72RenesasE,B8:8A:ECNintendo,B8:8C:29GDMideaA,B8:8D:12Apple,B8:8D:F1NanjingB,B8:8E:3AInfinite,B8:8E:82HuaweiDe,B8:8E:C6Stateles,B8:8E:DFZencheer,B8:8F:14Analytic,B8:8F:B4JabilCir,B8:90:47Apple,B8:91:C9Handream,B8:92:1DBgT&A,B8:94:36HuaweiTe,B8:94:70Calix,B8:94:D2RetailIn,B8:94:E7XiaomiCo,B8:96:74AllDSP,B8:97:5ABIOSTARM,B8:98:ADMotorola,B8:98:B0Atlona,B8:98:F7GioneeCo,B8:99:197signalS,B8:99:AEShenzhen,B8:99:B0CohereTe,B8:9A:2AIntelCor,B8:9A:9AXinShiJi,B8:9A:CDEliteOpt,B8:9A:EDOceanSer,B8:9B:C9SMCNetwo,B8:9B:E4ABBPower,B8:9E:A6Spbec-Mi,B8:9F:09WistronN,B8:A1:4ARaisecom,B8:A1:75Roku,B8:A3:77Cisco,B8:A3:86D-LinkIn,B8:A3:E0BenRuiTe,B8:A4:4FAxisComm,B8:A5:8DAxeGroup,B8:A8:AFLogic,B8:AC:6FDell,B8:AD:3EBluecom,B8:AE:1CSmartCub,B8:AE:1DGuangzho,B8:AE:6ENintendo,B8:AE:EDElitegro,B8:AF:67HewlettP,B8:B1:C7Bt&Com,B8:B2:EBGoogolTe,B8:B2:F8Apple,B8:B3:DCDerekSha,B8:B4:2EGioneeCo,B8:B7:7DGuangdon,B8:B7:D72GIGTech,B8:B7:F1WistronN,B8:B8:1EIntelCor,B8:B9:4EShenzhen,B8:BA:68XianJizh,B8:BA:72Cynove,B8:BB:23Guangdon,B8:BB:6DENERES,B8:BB:AFSamsungE,B8:BC:1BHuaweiTe,B8:BC:5BSamsungE,B8:BD:79TrendPoi,B8:BE:BFCisco,B8:BE:F4devolo,B8:BF:83IntelCor,B8:C1:11Apple,B8:C1:A2DragonPa,B8:C2:27PSTec,B8:C2:53JuniperN,B8:C3:85HuaweiTe,B8:C3:BFHenanChe,B8:C4:6FPrimmcon,B8:C6:8ESamsungE,B8:C6:AAEardaTec,B8:C7:16Fiberhom,B8:C7:4AGuangdon,B8:C7:5DApple,B8:C8:55Shanghai,B8:C8:EBItelMobi,B8:C9:B5Guangdon,B8:CA:04HoltekSe,B8:CA:3ADell,B8:CB:29Dell,B8:CD:93Penetek,B8:CD:A7MaxelerT,B8:CE:F6Mellanox,B8:D0:6FGuangzho,B8:D3:09CoxCommu,B8:D4:3EvivoMobi,B8:D4:9DMSevenSy,B8:D4:E7ArubaaHe,B8:D5:0BSunitecE,B8:D5:26ZyxelCom,B8:D5:6BMirka,B8:D6:F6HuaweiTe,B8:D7:AFMurataMa,B8:D8:12IEEERegi,B8:D8:12:00:00:00/28Glamo,B8:D8:12:10:00:00/28Votem,B8:D8:12:20:00:00/28IPMSales,B8:D8:12:30:00:00/28iModesty,B8:D8:12:40:00:00/28V5Techno,B8:D8:12:50:00:00/28XiamenXi,B8:D8:12:60:00:00/28VongerEl,B8:D8:12:70:00:00/28Neuropac,B8:D8:12:80:00:00/28VisualPr,B8:D8:12:90:00:00/28Entotem,B8:D8:12:A0:00:00/28Kiwigrid,B8:D8:12:B0:00:00/28Docobo,B8:D8:12:C0:00:00/28YuweiInf,B8:D8:12:D0:00:00/28LamResea,B8:D8:12:E0:00:00/28ZheJiang,B8:D8:12:F0:00:00/28Private,B8:D9:4DSagemcom,B8:D9:CESamsungE,B8:DA:E8HuaweiDe,B8:DA:F1Strahlen,B8:DA:F7Advanced,B8:DB:1CIntegrat,B8:DC:87IAI,B8:DD:71zte,B8:DE:5ELongchee,B8:DF:6BSpotCam,B8:E3:B1HuaweiTe,B8:E3:EEUniversa,B8:E5:89Payter,B8:E6:252Wire,B8:E7:799Solutio,B8:E8:56Apple,B8:E9:37Sonos,B8:EA:AAICGNETWO,B8:EC:A3ZyxelCom,B8:EE:0ESagemcom,B8:EE:65LiteonTe,B8:EE:79YWireTec,B8:EF:8BShenzhen,B8:F0:09Espressi,B8:F0:80Sps,B8:F1:2AApple,B8:F3:17iSunSmas,B8:F4:D0Herrmann,B8:F5:E7WayTools,B8:F6:53Shenzhen,B8:F6:B1Apple,B8:F7:32AryakaNe,B8:F7:4ARcntec,B8:F8:28Changshu,B8:F8:53Arcadyan,B8:F8:83Tp-LinkT,B8:F8:BEBluecom,B8:F9:34Sony,B8:FC:9ALeShiZhi,B8:FD:32Zhejiang,B8:FF:61Apple,B8:FF:6FShanghai,B8:FF:B3MitraSta,B8:FF:FETexasIns,BA:07:DAInfinixm,BA:30:54GeorgeWi,BA:41:59Anapass,BA:4D:EAEnvirone,BA:55:ECIeee8021,BA:A2:D0Wireless,BA:C4:85Fraunhof,BA:F2:1CIsoTc22S,BC:02:00StewartA,BC:02:4AHMDGloba,BC:03:A7MfpMiche,BC:05:43AVM,BC:06:2DWacom,BC:09:1BIntelCor,BC:09:63Apple,BC:0D:A5TexasIns,BC:0F:2BFortuneT,BC:0F:64IntelCor,BC:0F:9AD-LinkIn,BC:0F:A7Ouster,BC:12:5EBeijingW,BC:13:A8Shenzhen,BC:14:01HitronTe,BC:14:85SamsungE,BC:14:EFITONTech,BC:15:A6TaiwanJa,BC:15:ACVodafone,BC:16:65Cisco,BC:16:95zte,BC:16:F5Cisco,BC:17:B8IntelCor,BC:1A:67YFTechno,BC:1A:E4HuaweiDe,BC:1C:81Sichuani,BC:1E:85HuaweiTe,BC:20:A4SamsungE,BC:20:BAInspurSh,BC:22:28D-LinkIn,BC:22:FBRFIndust,BC:23:92BYDPreci,BC:25:E0HuaweiTe,BC:25:F03DDispla,BC:26:1DHongKong,BC:26:43Elprotro,BC:26:A1FACTORYF,BC:26:C7Cisco,BC:28:2Ce-Smart,BC:28:46NextBITC,BC:28:D6RowleyAs,BC:2B:6BBeijingH,BC:2B:D7RevogiIn,BC:2C:55BearFlag,BC:2C:E6Cisco,BC:2D:98ThinGlob,BC:2D:EFRealmeCh,BC:2E:48ARRISGro,BC:2E:F6HuaweiDe,BC:2F:3DvivoMobi,BC:30:5BDell,BC:30:7DWistronN,BC:30:7EWistronN,BC:30:D9Arcadyan,BC:32:5FZhejiang,BC:33:29SonyInte,BC:33:ACSiliconL,BC:34:00IEEERegi,BC:34:00:00:00:00/28Redvisio,BC:34:00:10:00:00/28IPLINKTe,BC:34:00:20:00:00/28LifeSmar,BC:34:00:30:00:00/28Altronix,BC:34:00:40:00:00/28DexcelDe,BC:34:00:50:00:00/28NDSL,BC:34:00:60:00:00/28Cameron,BC:34:00:70:00:00/28Q-PRODUC,BC:34:00:80:00:00/28MaticaTe,BC:34:00:90:00:00/28Shenzhen,BC:34:00:A0:00:00/28Auralic,BC:34:00:B0:00:00/28FaroTech,BC:34:00:C0:00:00/28ParlayLa,BC:34:00:D0:00:00/28Hangzhou,BC:34:00:E0:00:00/28LLDTechn,BC:34:00:F0:00:00/28Private,BC:35:E5Hydro,BC:38:65Jwcnetwo,BC:38:D2Pandachi,BC:39:A6CSUNSyst,BC:39:D9Z-Tec,BC:3A:EAGuangdon,BC:3B:AFApple,BC:3D:85HuaweiTe,BC:3E:07HitronTe,BC:3E:13Accordan,BC:3E:CBvivoMobi,BC:3F:4ETeleepoc,BC:3F:8FHuaweiTe,BC:41:00CODACOEL,BC:41:01Shenzhen,BC:42:8CAlpsalpi,BC:43:77HangZhou,BC:44:34Shenzhen,BC:44:86SamsungE,BC:44:B0Elastifi,BC:45:2EKnowledg,BC:46:99Tp-LinkT,BC:47:60SamsungE,BC:4A:56Cisco,BC:4B:79SensingT,BC:4C:C4Apple,BC:4D:FBHitronTe,BC:4E:3CCoreStaf,BC:4E:5DZhongMia,BC:51:FESwanncom,BC:52:B4Nokia,BC:52:B7Apple,BC:54:2FIntelCor,BC:54:36Apple,BC:54:51SamsungE,BC:54:F9DrogooTe,BC:54:FCShenzhen,BC:57:29Shenzhen,BC:5A:56Cisco,BC:5B:D5ARRISGro,BC:5C:4CElecom,BC:5E:A1PsiKick,BC:5F:F4ASRockIn,BC:5F:F6MercuryC,BC:60:10QingdaoH,BC:60:A7SonyInte,BC:62:0EHuaweiTe,BC:62:9FTelenetP,BC:62:CEShenzhen,BC:62:D2GenexisI,BC:64:4BARRISGro,BC:66:41IEEERegi,BC:66:41:00:00:00/28InSyncTe,BC:66:41:10:00:00/28GlobalCh,BC:66:41:20:00:00/28Process-,BC:66:41:30:00:00/28Solectri,BC:66:41:40:00:00/28Argus-Sp,BC:66:41:50:00:00/28Scientif,BC:66:41:60:00:00/28Intuitiv,BC:66:41:70:00:00/28VSNMobil,BC:66:41:80:00:00/28Shenzhen,BC:66:41:90:00:00/28Shenzhen,BC:66:41:A0:00:00/28EBlink,BC:66:41:B0:00:00/28SidusNov,BC:66:41:C0:00:00/28Shenzhen,BC:66:41:D0:00:00/28UtilLigh,BC:66:41:E0:00:00/28LucentTr,BC:66:41:F0:00:00/28Private,BC:66:DEShadowCr,BC:67:1CCisco,BC:67:78Apple,BC:67:84Environi,BC:69:CBPanasoni,BC:6A:16tdvine,BC:6A:29TexasIns,BC:6A:2FHengeDoc,BC:6A:44CommendI,BC:6A:D1XiaomiCo,BC:6B:4DNokia,BC:6C:21Apple,BC:6D:05DusunEle,BC:6E:64Sony,BC:6E:76GreenEne,BC:6E:E2IntelCor,BC:71:C1XTrillio,BC:72:B1SamsungE,BC:74:D7HangZhou,BC:75:36Alpsalpi,BC:75:74HuaweiTe,BC:75:96BeijingB,BC:76:4ERackspac,BC:76:5ESamsungE,BC:76:70HuaweiTe,BC:76:C5HuaweiTe,BC:77:37IntelCor,BC:77:9FSBM,BC:79:ADSamsungE,BC:7A:BFSamsungE,BC:7D:D1RadioDat,BC:7E:8BSamsungE,BC:7F:7BHuaweiDe,BC:7F:A4XiaomiCo,BC:81:1FIngate,BC:81:99BASIC,BC:82:5DMitsumiE,BC:83:85Microsof,BC:83:A7Shenzhen,BC:85:1FSamsungE,BC:85:56HonHaiPr,BC:88:93VILLBAU,BC:88:C3NingboDo,BC:8A:A3NHNEnter,BC:8A:E8QingDaoH,BC:8B:55NPPELIKS,BC:8C:CDSamsungE,BC:8D:0ENokia,BC:90:3ARobertBo,BC:91:B5Infinixm,BC:92:6BApple,BC:93:25NingboJo,BC:96:80Shenzhen,BC:97:40IEEERegi,BC:97:40:00:00:00/28AlphaESS,BC:97:40:10:00:00/28comtac,BC:97:40:20:00:00/28LattecIS,BC:97:40:30:00:00/28Precisio,BC:97:40:40:00:00/28WindMobi,BC:97:40:50:00:00/28Shanghai,BC:97:40:60:00:00/28Shenzhen,BC:97:40:70:00:00/28Airfi,BC:97:40:80:00:00/28GaodiRus,BC:97:40:90:00:00/28DirectCo,BC:97:40:A0:00:00/28AmapInfo,BC:97:40:B0:00:00/28ForoTel,BC:97:40:C0:00:00/28LISTEC,BC:97:40:D0:00:00/28Rollock,BC:97:40:E0:00:00/28B4ComTec,BC:97:89HuaweiDe,BC:97:E1Broadcom,BC:98:89Fiberhom,BC:98:DFMotorola,BC:99:11ZyxelCom,BC:99:30HuaweiTe,BC:99:BCFonSeeTe,BC:9A:53HuaweiDe,BC:9B:68Technico,BC:9C:31HuaweiTe,BC:9C:C5BeijingH,BC:9D:42Shenzhen,BC:9D:A5DASCOMEu,BC:9F:E4ArubaaHe,BC:9F:EFApple,BC:A0:42Shanghai,BC:A1:3ASES-imag,BC:A3:7FRail-Mil,BC:A4:E1Nabto,BC:A5:11Netgear,BC:A5:8BSamsungE,BC:A5:A9Apple,BC:A8:A6IntelCor,BC:A9:20Apple,BC:A9:93CambiumN,BC:A9:D6Cyber-Ra,BC:AB:7CTRnPKORE,BC:AD:28Hangzhou,BC:AD:ABAvaya,BC:AE:C5ASUSTekC,BC:AF:91TEConnec,BC:B0:E7HuaweiTe,BC:B1:81Sharp,BC:B1:F3SamsungE,BC:B2:2BEM-Tech,BC:B3:08Hongkong,BC:B8:52Cybera,BC:B8:63Apple,BC:BA:C2Hangzhou,BC:BA:E1AREC,BC:BB:C9Kellendo,BC:BC:46SKSWeldi,BC:BD:9EItelMobi,BC:C0:0FFiberhom,BC:C1:68DinBoxSv,BC:C2:3AThomsonV,BC:C3:1BKygoLife,BC:C3:42Panasoni,BC:C4:93Cisco,BC:C6:1ASpectraE,BC:C6:DBNokia,BC:C8:10CiscoSPV,BC:CA:B5ARRISGro,BC:CD:45Voismart,BC:CE:25Nintendo,BC:CF:4FZyxelCom,BC:CF:CCHTC,BC:D1:1FSamsungE,BC:D1:65CiscoSPV,BC:D1:77Tp-LinkT,BC:D1:D3Shenzhen,BC:D2:95Cisco,BC:D5:B6d2dtechn,BC:D7:13OwlLabs,BC:D7:67Private,BC:D7:A5ArubaaHe,BC:D7:CEChinaMob,BC:D7:D4Roku,BC:D9:40ASR,BC:DB:09CiscoMer,BC:DD:C2Espressi,BC:E0:9DEoslink,BC:E1:43Apple,BC:E2:65HuaweiTe,BC:E5:9FWATERWOR,BC:E6:3FSamsungE,BC:E6:7CCambiumN,BC:E7:12Cisco,BC:E7:67Quanzhou,BC:E7:96Wireless,BC:E9:2FHP,BC:EA:2BCityCom,BC:EA:FAHewlettP,BC:EB:5FFujianBe,BC:EC:23Shenzhen,BC:EC:5DApple,BC:EC:A0CompalIn,BC:EE:7BASUSTekC,BC:F1:71IntelCor,BC:F1:F2Cisco,BC:F2:92Plantron,BC:F2:AFdevolo,BC:F3:10ExtremeN,BC:F4:5Fzte,BC:F5:ACLGElectr,BC:F6:1CGeomodel,BC:F6:85D-LinkIn,BC:F8:11XiamenDN,BC:F9:F2Teko,BC:FA:B8Guangzho,BC:FE:8CAltronic,BC:FE:D9Apple,BC:FF:21SmartCod,BC:FF:4DEspressi,BC:FF:ACTopcon,BC:FF:EBMotorola,C0:00:00WesternD,C0:02:8DWINSTARD,C0:03:80JuniperN,C0:05:C2ARRISGro,C0:06:C3TP-Link,C0:07:4ABrita,C0:0D:7EAdditech,C0:10:B1HMDGloba,C0:11:73SamsungE,C0:11:A6Fort-Tel,C0:12:42AlphaSec,C0:13:2BSichuanC,C0:14:3DHonHaiPr,C0:14:B8Nokia,C0:14:FECisco,C0:16:92ChinaMob,C0:17:4DSamsungE,C0:18:50QuantaCo,C0:18:85HonHaiPr,C0:1A:DAApple,C0:1B:23SichuanT,C0:1C:30Shenzhen,C0:1E:9BPixaviAS,C0:21:0DShenzhen,C0:22:50Koss,C0:23:8DSamsungE,C0:25:06AVM,C0:25:2FShenzhen,C0:25:5CCisco,C0:25:67NexxtSol,C0:25:A2NECPlatf,C0:25:A5Dell,C0:25:E9Tp-LinkT,C0:27:B9BeijingN,C0:28:0BHonorDev,C0:28:8DLogitech,C0:29:73Audyssey,C0:29:F3XySystem,C0:2B:FCiNESappl,C0:2C:5CApple,C0:2C:7AShenzhen,C0:2D:EECuff,C0:2E:25Guangdon,C0:2E:26Private,C0:2F:F1VoltaNet,C0:33:5EMicrosof,C0:33:DAShenzhen,C0:34:B4Gigaston,C0:35:80A&RTech,C0:35:BDVelocyte,C0:35:C5Prosoft,C0:36:53eero,C0:36:56Fiberhom,C0:38:96HonHaiPr,C0:38:F9NokiaDan,C0:39:37GreeElec,C0:39:5AZhejiang,C0:3B:8FMinicomD,C0:3C:04Sagemcom,C0:3C:59IntelCor,C0:3D:03SamsungE,C0:3D:46Shanghai,C0:3D:D9MitraSta,C0:3E:0FBSkyB,C0:3E:BADell,C0:3F:0ENetgear,C0:3F:2ABiscotti,C0:3F:D5Elitegro,C0:3F:DDHuaweiTe,C0:40:04Medicaro,C0:41:21NokiaSol,C0:41:F6LgElectr,C0:42:D0JuniperN,C0:43:01Epec,C0:44:42Apple,C0:44:E3Shenzhen,C0:47:54vivoMobi,C0:48:E6SamsungE,C0:48:FBShenzhen,C0:49:3DMaitrise,C0:4A:00Tp-LinkT,C0:4A:09Zhejiang,C0:4B:13WonderSo,C0:4D:F7Serelec,C0:51:7EHangzhou,C0:53:36BeijingN,C0:56:27BelkinIn,C0:56:E3Hangzhou,C0:57:BCAvaya,C0:58:A7Pico,C0:5E:6FVStonkau,C0:5E:79Shenzhen,C0:61:18Tp-LinkT,C0:61:9AIEEERegi,C0:61:9A:00:00:00/28ParagonR,C0:61:9A:10:00:00/28KidKraft,C0:61:9A:20:00:00/28GrupArge,C0:61:9A:30:00:00/28LyandAco,C0:61:9A:40:00:00/28Stello,C0:61:9A:50:00:00/28NanjingB,C0:61:9A:60:00:00/28IPGAutom,C0:61:9A:70:00:00/28MadPiece,C0:61:9A:80:00:00/28NanjingS,C0:61:9A:90:00:00/28Wingtech,C0:61:9A:A0:00:00/28GronnKon,C0:61:9A:B0:00:00/28VictronE,C0:61:9A:C0:00:00/28JAM-Labs,C0:61:9A:D0:00:00/28Uhnder,C0:61:9A:E0:00:00/28Zhejiang,C0:62:6BCisco,C0:63:69BinxinTe,C0:63:94Apple,C0:64:C6Nokia,C0:64:E4Cisco,C0:65:99SamsungE,C0:67:AFCisco,C0:6B:55Motorola,C0:6C:0FDobbsSta,C0:6C:6DMagneMot,C0:6D:1ATianjinH,C0:70:09HuaweiTe,C0:74:2BShenzhen,C0:74:ADGrandstr,C0:78:31HuaweiDe,C0:78:78Flextron,C0:7B:BCCisco,C0:7C:D1Pegatron,C0:7E:40Shenzhen,C0:81:35NingboFo,C0:81:70EffigisG,C0:83:0A2Wire,C0:83:59IEEERegi,C0:83:59:00:00:00/28Chongqin,C0:83:59:10:00:00/28GemvaxTe,C0:83:59:20:00:00/28HuaxinSM,C0:83:59:30:00:00/28PCHEngin,C0:83:59:40:00:00/28Ants,C0:83:59:50:00:00/28ViperDes,C0:83:59:60:00:00/28BeijingC,C0:83:59:70:00:00/28FuzhouFd,C0:83:59:80:00:00/28istaInte,C0:83:59:90:00:00/28Shenzhen,C0:83:59:A0:00:00/28Shanghai,C0:83:59:B0:00:00/28SuzhouSi,C0:83:59:C0:00:00/28Private,C0:83:59:D0:00:00/28GardnerD,C0:83:59:E0:00:00/28CyberSci,C0:83:C9HuaweiDe,C0:84:7AApple,C0:84:7DAMPAKTec,C0:84:88Finis,C0:85:4CRagentek,C0:86:B3Shenzhen,C0:87:EBSamsungE,C0:88:5BSnDTech,C0:89:97SamsungE,C0:89:ABARRISGro,C0:8A:CDGuangzho,C0:8A:DERuckusWi,C0:8B:05HuaweiTe,C0:8B:6FSISistem,C0:8C:60Cisco,C0:8C:71Motorola,C0:8F:20Shenzhen,C0:91:32PatriotM,C0:91:34ProCurve,C0:92:96zte,C0:94:35ARRISGro,C0:94:ADzte,C0:95:DANXPIndia,C0:97:27SamsungE,C0:98:79Acer,C0:98:DAChinaMob,C0:98:E5Universi,C0:9A:71XiamenMe,C0:9A:D0Apple,C0:9B:F4IEEERegi,C0:9B:F4:00:00:00/28Annapurn,C0:9B:F4:10:00:00/28Connecte,C0:9B:F4:20:00:00/28HitachiH,C0:9B:F4:30:00:00/28OspreyVi,C0:9B:F4:40:00:00/28NpkAtron,C0:9B:F4:50:00:00/28Infiot,C0:9B:F4:60:00:00/28DelovoyO,C0:9B:F4:70:00:00/28BigDutch,C0:9B:F4:80:00:00/28Shenzhen,C0:9B:F4:90:00:00/28Alcatraz,C0:9B:F4:A0:00:00/28Inveo,C0:9B:F4:B0:00:00/28Nuctech,C0:9B:F4:C0:00:00/28Pinpark,C0:9B:F4:D0:00:00/28Professi,C0:9B:F4:E0:00:00/28Continen,C0:9C:04ShaanxiG,C0:9C:92Coby,C0:9D:26TopiconH,C0:9F:05Guangdon,C0:9F:42Apple,C0:9F:E1zte,C0:A0:0DARRISGro,C0:A0:BBD-LinkIn,C0:A0:C7Fairfiel,C0:A0:DEMultiTou,C0:A0:E2EdenInno,C0:A1:A2MarqMetr,C0:A2:6DAbbottPo,C0:A3:643DMassac,C0:A3:6EBSkyB,C0:A3:9EEarthCam,C0:A5:3EApple,C0:A5:DDShenzhen,C0:A6:00Apple,C0:A6:6DInspurGr,C0:A8:F0AdamsonE,C0:AA:68OSASITec,C0:AC:54Sagemcom,C0:AE:FDShenzhen,C0:B1:01zte,C0:B3:39Comigo,C0:B3:57YoshikiE,C0:B4:7DHuaweiDe,C0:B5:CDHuaweiDe,C0:B5:D7Chongqin,C0:B6:58Apple,C0:B6:F9IntelCor,C0:B7:13BeijingX,C0:B8:83IntelCor,C0:B8:B1BitBox,C0:B8:E6RuijieNe,C0:BA:E6Applicat,C0:BC:9AHuaweiTe,C0:BD:42ZPASmart,C0:BD:C8SamsungE,C0:BD:D1SamsungE,C0:BF:A7JuniperN,C0:BF:C0HuaweiTe,C0:C1:C0Cisco-Li,C0:C3:B6Automati,C0:C5:20RuckusWi,C0:C5:22ARRISGro,C0:C5:69Shanghai,C0:C6:87CiscoSPV,C0:C9:46MitsuyaL,C0:C9:76Shenzhen,C0:C9:E3Tp-LinkT,C0:CB:38HonHaiPr,C0:CB:F1Mobiwire,C0:CC:42SichuanT,C0:CC:F8Apple,C0:CE:CDApple,C0:CF:A3Creative,C0:D0:12Apple,C0:D0:26HuaweiDe,C0:D0:44Sagemcom,C0:D0:63EMMicroe,C0:D0:FFChinaMob,C0:D1:93HuaweiDe,C0:D2:DDSamsungE,C0:D2:F3HuiZhouG,C0:D3:91IEEERegi,C0:D3:91:00:00:00/28FuzhouJi,C0:D3:91:10:00:00/28B9Creati,C0:D3:91:20:00:00/28HofonAut,C0:D3:91:30:00:00/28Ixon,C0:D3:91:40:00:00/28VernierS,C0:D3:91:50:00:00/28WiTagg,C0:D3:91:60:00:00/28Ernitec,C0:D3:91:70:00:00/28ALNETz,C0:D3:91:80:00:00/28XenaSecu,C0:D3:91:90:00:00/28xxter,C0:D3:91:A0:00:00/28AlphaAud,C0:D3:91:B0:00:00/28Private,C0:D3:91:C0:00:00/28Zhinengg,C0:D3:91:D0:00:00/28Regulus,C0:D3:91:E0:00:00/28SamsaraN,C0:D3:C0SamsungE,C0:D4:6BHuaweiDe,C0:D6:82AristaNe,C0:D7:AAArcadyan,C0:D8:34xvtec,C0:D9:62AskeyCom,C0:D9:F7ShanDong,C0:DA:74Hangzhou,C0:DC:6AQingdaoE,C0:DC:D7HuaweiDe,C0:DC:DASamsungE,C0:DF:77ConradEl,C0:E0:18HuaweiTe,C0:E1:BEHuaweiTe,C0:E3:A0RenesasE,C0:E3:FBHuaweiTe,C0:E4:22TexasIns,C0:E4:2DTp-LinkT,C0:E4:34AzureWav,C0:E5:4EARIESEmb,C0:E7:BFSichuanA,C0:E8:62Apple,C0:EA:E4Sonicwal,C0:EE:40LairdTec,C0:EE:B5EniceNet,C0:EE:FBOnePlusT,C0:F1:C4Pacidal,C0:F2:FBApple,C0:F4:E6HuaweiTe,C0:F5:35AMPAKTec,C0:F6:36Hangzhou,C0:F6:C2HuaweiTe,C0:F6:ECHuaweiTe,C0:F7:9DPowercod,C0:F8:27Rapidmax,C0:F8:DAHonHaiPr,C0:F9:45ToshibaT,C0:F9:91GMEStand,C0:F9:B0HuaweiTe,C0:F9:D2arkonate,C0:FB:F9IEEERegi,C0:FB:F9:00:00:00/28Xerox,C0:FB:F9:10:00:00/28LIXIL,C0:FB:F9:20:00:00/28Dongguan,C0:FB:F9:30:00:00/28Shenzhen,C0:FB:F9:40:00:00/28MinatoAd,C0:FB:F9:50:00:00/28Haguenet,C0:FB:F9:60:00:00/28IVT,C0:FB:F9:70:00:00/28LongSung,C0:FB:F9:80:00:00/28Dongmeng,C0:FB:F9:90:00:00/28zxsoluti,C0:FB:F9:A0:00:00/28TiandiCh,C0:FB:F9:B0:00:00/28Shenzhen,C0:FB:F9:C0:00:00/28Shenzhen,C0:FB:F9:D0:00:00/28Dropbeat,C0:FB:F9:E0:00:00/28NavitasD,C0:FD:84zte,C0:FF:A8HuaweiTe,C0:FF:D4Netgear,C4:00:06LipiData,C4:00:49Kamama,C4:00:ADAdvantec,C4:01:42MaxMedia,C4:01:7CRuckusWi,C4:01:B1SeekTech,C4:01:CEPresitio,C4:02:E1Khwahish,C4:03:A8IntelCor,C4:04:15Netgear,C4:04:7BShenzhen,C4:05:28HuaweiTe,C4:06:83HuaweiTe,C4:07:2FHuaweiTe,C4:08:4ANokia,C4:08:80Shenzhen,C4:09:38FujianSt,C4:0A:CBCisco,C4:0B:31Apple,C4:0B:CBXiaomiCo,C4:0D:96HuaweiTe,C4:0E:45ACKNetwo,C4:0F:09Hermesel,C4:10:8ARuckusWi,C4:11:E0BullGrou,C4:12:34Apple,C4:12:F5D-LinkIn,C4:13:E2ExtremeN,C4:14:11Apple,C4:14:3CCisco,C4:16:88HuaweiDe,C4:16:FAPrysm,C4:17:0EHuaweiDe,C4:17:FEHonHaiPr,C4:18:E9SamsungE,C4:19:8BDominion,C4:19:D1TelinkSe,C4:19:ECQualisys,C4:1C:07SamsungE,C4:1C:9CJiQiDao,C4:1C:FFVizio,C4:1E:CEHMISourc,C4:21:C8Kyocera,C4:23:60IntelCor,C4:23:7AWhizNets,C4:23:A2PTEmsoni,C4:24:2EGalvanic,C4:24:56PaloAlto,C4:26:28AiroWire,C4:27:8CHuaweiDe,C4:27:95Technico,C4:28:2DEmbedded,C4:29:1DKlemsanE,C4:29:96Signify,C4:2A:D0Apple,C4:2B:44HuaweiDe,C4:2C:03Apple,C4:2C:4FQingdaoH,C4:2F:90Hangzhou,C4:30:18MCSLogic,C4:30:CASDBiosen,C4:32:D1FarlinkT,C4:33:06ChinaMob,C4:34:5BHuaweiTe,C4:34:6BHewlettP,C4:36:55Shenzhen,C4:36:6CLGInnote,C4:36:C0Buffalo,C4:36:DARustelet,C4:37:72Virtuozz,C4:38:D3Tagatec,C4:39:3ASMCNetwo,C4:39:60GDMideaA,C4:3A:35Fn-LinkT,C4:3A:9FSiconix,C4:3A:BESony,C4:3C:3CCybelec,C4:3C:EABuffalo,C4:3D:C7Netgear,C4:40:44RackTop,C4:40:F6Guangdon,C4:41:1EBelkinIn,C4:41:37QuectelW,C4:42:02SamsungE,C4:42:68Crestron,C4:43:8FLGElectr,C4:44:7DHuaweiTe,C4:44:A0Cisco,C4:45:67SAMBONPR,C4:45:ECShanghai,C4:46:19HonHaiPr,C4:47:3FHuaweiTe,C4:48:38SatcomDi,C4:48:FATaicangT,C4:49:BBMitsumiE,C4:4A:D0Fireflie,C4:4B:44Omniprin,C4:4B:D1WallysCo,C4:4D:84Cisco,C4:4E:1FBlueN,C4:4E:ACShenzhen,C4:4F:33Espressi,C4:4F:96AlpsAlpi,C4:50:06SamsungE,C4:51:8DShenzhen,C4:54:44QuantaCo,C4:55:A6CadacHol,C4:55:C2Bach-Sim,C4:56:00GalleonE,C4:56:FELavaInte,C4:57:1FJuneLife,C4:57:6ESamsungE,C4:58:C2Shenzhen,C4:59:76FugooCoo,C4:5A:86HuaweiDe,C4:5B:BEEspressi,C4:5B:F7ants,C4:5D:83SamsungE,C4:5D:D8HDMIForu,C4:5E:5CHuaweiTe,C4:60:44EverexEl,C4:61:8BApple,C4:62:6BZPTVigan,C4:62:EASamsungE,C4:63:54U-Raku,C4:63:FBNeatfram,C4:64:13Cisco,C4:64:B7Fiberhom,C4:64:E3TexasIns,C4:65:16HewlettP,C4:66:99vivoMobi,C4:67:B5Libraton,C4:67:D1HuaweiTe,C4:68:D0VTechTel,C4:69:3ETurbulen,C4:69:F0HuaweiTe,C4:6A:B7XiaomiCo,C4:6B:B4myIDkey,C4:6D:F1DataGrav,C4:6E:1FTp-LinkT,C4:6E:33ZhongGeS,C4:6E:7BShenzhen,C4:70:0BGuangzho,C4:70:ABRuijieNe,C4:71:30FonTechn,C4:71:54Tp-LinkT,C4:71:FECisco,C4:72:95Cisco,C4:73:1ESamsungE,C4:74:1Ezte,C4:74:69Bt9,C4:74:F8HotPeppe,C4:75:ABIntelCor,C4:77:ABBeijingA,C4:77:AFAdvanced,C4:78:A2HuaweiDe,C4:7B:2FBeijingJ,C4:7B:A3NAVIS,C4:7C:8DIEEERegi,C4:7C:8D:00:00:00/28Ati,C4:7C:8D:10:00:00/28LynxInno,C4:7C:8D:20:00:00/28Star2Sta,C4:7C:8D:30:00:00/28Watec,C4:7C:8D:40:00:00/28Robostar,C4:7C:8D:50:00:00/28PASCAL,C4:7C:8D:60:00:00/28HHCCPlan,C4:7C:8D:70:00:00/28Awiselin,C4:7C:8D:80:00:00/28GETEMEDM,C4:7C:8D:90:00:00/28AirbusDS,C4:7C:8D:A0:00:00/28Silvuste,C4:7C:8D:B0:00:00/28GcAutoma,C4:7C:8D:C0:00:00/28INOTECSi,C4:7C:8D:D0:00:00/28AnhuiGua,C4:7C:8D:E0:00:00/28LaborStr,C4:7D:46Fujitsu,C4:7D:4FCisco,C4:7D:9FSamsungE,C4:7D:CCZebraTec,C4:7D:FEANSoluti,C4:7F:51Inventek,C4:80:25HuaweiDe,C4:80:8ACloudDia,C4:82:3FFujianNe,C4:82:4EChangzho,C4:83:6FCiena,C4:84:66Apple,C4:85:08IntelCor,C4:86:E9HuaweiTe,C4:88:E5SamsungE,C4:89:EDSolidOpt,C4:8A:5AJfcontro,C4:8B:A3CiscoMer,C4:8E:8FHonHaiPr,C4:8F:07Shenzhen,C4:8F:C1Deeptrac,C4:91:0CApple,C4:91:3AShenzhen,C4:91:CFLuxul,C4:92:4CKeisokuk,C4:93:008Devices,C4:93:13100fione,C4:93:80Speedyte,C4:93:D9SamsungE,C4:95:00AmazonTe,C4:95:4DIEEERegi,C4:95:4D:00:00:00/28BAIntern,C4:95:4D:10:00:00/28Teletron,C4:95:4D:20:00:00/28ShenZhen,C4:95:4D:30:00:00/28Sercomm,C4:95:4D:40:00:00/28GLSoluti,C4:95:4D:50:00:00/28MarbleAu,C4:95:4D:60:00:00/28AKKAGerm,C4:95:4D:70:00:00/28TechnoEn,C4:95:4D:80:00:00/28Xinjiang,C4:95:4D:90:00:00/28Shenzhen,C4:95:4D:A0:00:00/28KATMekat,C4:95:4D:B0:00:00/28Multicom,C4:95:4D:C0:00:00/28SolidGea,C4:95:4D:D0:00:00/28NewlandE,C4:95:4D:E0:00:00/28CanareEl,C4:95:A2Shenzhen,C4:98:05MinieumN,C4:98:5CHuiZhouG,C4:98:78Shanghai,C4:98:80Apple,C4:98:86QorvoInt,C4:9A:02LGElectr,C4:9D:EDMicrosof,C4:9E:41G24Power,C4:9F:4CHuaweiTe,C4:9F:F3MciaoTec,C4:A1:51SichuanT,C4:A3:66zte,C4:A4:02HuaweiTe,C4:A7:2BShenzhen,C4:A8:1DD-LinkIn,C4:AA:A1SUMMITDE,C4:AB:B2vivoMobi,C4:AC:59MurataMa,C4:AD:21MEDIAEDG,C4:AD:34Routerbo,C4:AD:F1GOPEACE,C4:AE:12SamsungE,C4:B2:39Cisco,C4:B3:01Apple,C4:B3:6ACisco,C4:B5:12GeneralE,C4:B8:B4HuaweiTe,C4:B9:CDCisco,C4:BA:99I+MEActi,C4:BA:A3BeijingW,C4:BB:4CZebraInf,C4:BB:EAPakedgeD,C4:BC:D7NewRyate,C4:BD:6ASKF,C4:BD:E5IntelCor,C4:BE:84TexasIns,C4:BE:D4Avaya,C4:BF:60TecnoMob,C4:C0:AEMidoriEl,C4:C1:38OWLinkTe,C4:C1:9FNational,C4:C3:6BApple,C4:C5:63TecnoMob,C4:C6:03Cisco,C4:C7:55BeijingH,C4:C9:19EnergyIm,C4:C9:ECGugaooHK,C4:CA:2BAristaNe,C4:CA:D9Hangzhou,C4:CB:54FibocomA,C4:CB:6BAiristaF,C4:CD:45BeijingB,C4:CD:82Hangzhou,C4:D0:E3IntelCor,C4:D1:97VentiaUt,C4:D4:38HuaweiTe,C4:D4:89JiangSuJ,C4:D6:55Tercelte,C4:D7:38HuaweiDe,C4:D7:FDBouffalo,C4:D8:F3iZotope,C4:D9:87IntelCor,C4:DA:26Noblex,C4:DA:7DIviumTec,C4:DD:57Espressi,C4:DE:7BHuaweiDe,C4:E0:32IEEE1904,C4:E0:DEZhengzho,C4:E1:7CU2S,C4:E1:A1Guangdon,C4:E2:87HuaweiTe,C4:E3:9FGuangdon,C4:E5:06PiperNet,C4:E5:10Mechatro,C4:E5:32Arcadyan,C4:E7:BESCSpro,C4:E9:0AD-LinkIn,C4:E9:2FSciex,C4:E9:84Tp-LinkT,C4:EA:1DTechnico,C4:EB:E3RrcnSas,C4:ED:BATexasIns,C4:EE:AEVSSMonit,C4:EE:F5II-VI,C4:EF:70HomeSkin,C4:F0:81HuaweiTe,C4:F0:ECFiberhom,C4:F1:22Nexar,C4:F1:74eero,C4:F1:D1BeijingS,C4:F3:12TexasIns,C4:F4:64Spicaint,C4:F5:7CBrocadeC,C4:F5:A5Kumalift,C4:F7:D5Cisco,C4:F8:39ActiaAut,C4:FB:AAHuaweiTe,C4:FB:C8Shenzhen,C4:FC:E4DishTVNZ,C4:FC:EFSambaNov,C4:FD:E6Drtech,C4:FE:5BGuangdon,C4:FE:E2AMICCOME,C4:FF:1FHuaweiTe,C4:FF:22HuaweiDe,C4:FF:BCIEEERegi,C4:FF:BC:00:00:00/28Danego,C4:FF:BC:10:00:00/28Visatech,C4:FF:BC:20:00:00/28Mobiletr,C4:FF:BC:30:00:00/28Shenzhen,C4:FF:BC:40:00:00/28iMageTec,C4:FF:BC:50:00:00/28comtime,C4:FF:BC:60:00:00/28Shenzhen,C4:FF:BC:70:00:00/28Critical,C4:FF:BC:80:00:00/28ShenZhen,C4:FF:BC:90:00:00/28GSMInnov,C4:FF:BC:A0:00:00/28Advanced,C4:FF:BC:B0:00:00/28KagaElec,C4:FF:BC:C0:00:00/28KyongBoE,C4:FF:BC:D0:00:00/28BeijingK,C4:FF:BC:E0:00:00/28viRaTec,C8:00:84Cisco,C8:02:10LGInnote,C8:02:58ITWGSEAp,C8:02:8FNovaElec,C8:02:A6BeijingN,C8:03:F5RuckusWi,C8:05:9EHefeiSym,C8:07:18TDSi,C8:07:39NAKAYO,C8:08:73RuckusWi,C8:08:E9LGElectr,C8:09:A8IntelCor,C8:0A:A9QuantaCo,C8:0C:C8HuaweiTe,C8:0D:32Holoplot,C8:0E:14AVMAudio,C8:0E:77LeShiZhi,C8:0E:95OmniLync,C8:10:73CenturyO,C8:13:8BShenzhen,C8:14:51HuaweiTe,C8:14:79SamsungE,C8:16:A5Masimo,C8:16:BDQingdaoH,C8:16:DARealmeCh,C8:17:39ItelMobi,C8:19:F7SamsungE,C8:1A:FEDLOGIC,C8:1B:5CBCTech,C8:1B:6BInnovaSe,C8:1C:FEZebraTec,C8:1E:8EADVSecur,C8:1E:E7Apple,C8:1F:66Dell,C8:1F:BEHuaweiTe,C8:1F:EAAvaya,C8:20:8EStoraged,C8:21:58IntelCor,C8:21:DAShenzhen,C8:25:E1Lemobile,C8:28:32BeijingX,C8:29:2ABarunEle,C8:2A:14Apple,C8:2B:96Espressi,C8:2C:2BIEEERegi,C8:2C:2B:00:00:00/28Fungible,C8:2C:2B:10:00:00/28Galgus,C8:2C:2B:20:00:00/28ReppHeal,C8:2C:2B:30:00:00/28RFEngine,C8:2C:2B:40:00:00/28iWaveTec,C8:2C:2B:50:00:00/28Dalco,C8:2C:2B:60:00:00/28GravIT,C8:2C:2B:70:00:00/28MerpaBil,C8:2C:2B:80:00:00/28Verifone,C8:2C:2B:90:00:00/28BIOTSp,C8:2C:2B:A0:00:00/28Shiftall,C8:2C:2B:B0:00:00/28KunshanS,C8:2C:2B:C0:00:00/28SmartWir,C8:2C:2B:D0:00:00/28UBITRON,C8:2C:2B:E0:00:00/28Fränkisc,C8:2E:47SuzhouSm,C8:2E:94HalfaEnt,C8:31:68eZEX,C8:32:32HuntingI,C8:33:4BApple,C8:33:E5HuaweiTe,C8:34:8EIntelCor,C8:35:B8Ericsson,C8:38:70SamsungE,C8:39:ACHuaweiDe,C8:3A:35TendaTec,C8:3A:6BRoku,C8:3B:45Jri,C8:3C:85Apple,C8:3D:97Nokia,C8:3D:D4CyberTAN,C8:3D:DCXiaomiCo,C8:3D:FCAlphaThe,C8:3E:99TexasIns,C8:3E:A7KUNBUS,C8:3F:26Microsof,C8:3F:B4ARRISGro,C8:40:29Fiberhom,C8:41:8ASamsungE,C8:45:29IMKNetwo,C8:45:44AsiaPaci,C8:45:8FWyler,C8:47:82AresonTe,C8:47:8CBeken,C8:48:F5MEDISONX,C8:4C:75Cisco,C8:4D:34LIONSTai,C8:4D:44Shenzhen,C8:4F:0EIntegrat,C8:4F:86Sophos,C8:50:CEHuaweiTe,C8:50:E9Raisecom,C8:51:42SamsungE,C8:51:95HuaweiTe,C8:52:61ARRISGro,C8:53:E1BeijingB,C8:54:4BZyxelCom,C8:56:45Intermas,C8:56:63SunflexE,C8:58:C0IntelCor,C8:5A:9Fzte,C8:5B:76LCFCHeFe,C8:5B:A0Shenzhen,C8:5C:CCBeijingX,C8:5D:38HUMAX,C8:60:00ASUSTekC,C8:63:14IEEERegi,C8:63:14:00:00:00/28WesternR,C8:63:14:10:00:00/28Autonics,C8:63:14:20:00:00/28Tymphany,C8:63:14:30:00:00/28TrackMan,C8:63:14:40:00:00/28Shenzhen,C8:63:14:50:00:00/28MeyerEle,C8:63:14:60:00:00/28GrinbiPa,C8:63:14:70:00:00/28Shenzhen,C8:63:14:80:00:00/28Thinci,C8:63:14:90:00:00/28Maxcom,C8:63:14:A0:00:00/28Optictim,C8:63:14:B0:00:00/28Shenzhen,C8:63:14:C0:00:00/28Freeus,C8:63:14:D0:00:00/28Telemati,C8:63:14:E0:00:00/28TaylorDy,C8:63:F1SonyInte,C8:63:FCARRISGro,C8:64:C7zte,C8:66:2CBeijingH,C8:66:5DExtremeN,C8:67:5EExtremeN,C8:68:DEHuaweiDe,C8:69:CDApple,C8:6C:1EDisplay,C8:6C:3DAmazonTe,C8:6C:87ZyxelCom,C8:6C:B6Optcom,C8:6F:1DApple,C8:71:25JohnsonO,C8:72:48Aplicom,C8:73:24SowCheng,C8:75:5BQuantify,C8:77:65Tiesse,C8:77:8BMercury–,C8:7B:23Bose,C8:7B:5Bzte,C8:7C:BCValink,C8:7D:77Shenzhen,C8:7E:75SamsungE,C8:7E:A1TCLMOKAI,C8:83:14TempoCom,C8:84:39SunriseT,C8:84:47Beautifu,C8:84:A1Cisco,C8:84:CFHuaweiTe,C8:85:50Apple,C8:86:29Shenzhen,C8:87:22Lumenpul,C8:87:3BNetOptic,C8:8A:83Dongguan,C8:8B:47Nolangro,C8:8B:E8Masimo,C8:8D:83HuaweiTe,C8:8E:D1IEEERegi,C8:8E:D1:00:00:00/28Aisworld,C8:8E:D1:10:00:00/28GermanPi,C8:8E:D1:20:00:00/28Rotronic,C8:8E:D1:30:00:00/28LinxTech,C8:8E:D1:40:00:00/28Comlab,C8:8E:D1:50:00:00/28Fibergat,C8:8E:D1:60:00:00/28Shenyang,C8:8E:D1:70:00:00/28UbedbaPl,C8:8E:D1:80:00:00/28Electron,C8:8E:D1:90:00:00/28Focalcre,C8:8E:D1:A0:00:00/28APSensin,C8:8E:D1:B0:00:00/28Advanced,C8:8E:D1:C0:00:00/28Shanghai,C8:8E:D1:D0:00:00/28PhoenixE,C8:8E:D1:E0:00:00/28Aventics,C8:8E:D1:F0:00:00/28Private,C8:8F:26Skyworth,C8:90:3EPaktonTe,C8:91:F9Sagemcom,C8:93:46MXCHIP,C8:93:83Embedded,C8:94:02Chongqin,C8:94:BBHuaweiTe,C8:94:D2JiangsuD,C8:96:65Microsof,C8:97:9FNokia,C8:9B:ADHonorDev,C8:9C:13Inspirem,C8:9C:1DCisco,C8:9C:DCElitegro,C8:9D:18HuaweiDe,C8:9E:43Netgear,C8:9F:1AHuaweiTe,C8:9F:1DShenzhen,C8:9F:42VDIIInno,C8:A0:30TexasIns,C8:A1:B6Shenzhen,C8:A1:BANeul,C8:A2:CEOasisMed,C8:A4:0DCoolerMa,C8:A6:20Nebula,C8:A7:0AVerizonB,C8:A7:29SYStroni,C8:A7:76HuaweiTe,C8:A8:23SamsungE,C8:A9:FCGoyooNet,C8:AA:21ARRISGro,C8:AA:55HunanCom,C8:AA:CCPrivate,C8:AE:9CShanghai,C8:AF:40marcoSys,C8:AF:E3HefeiRad,C8:B1:CDApple,C8:B1:EEQorvo,C8:B2:1EChipseaT,C8:B2:9BIntelCor,C8:B3:73Cisco-Li,C8:B4:22AskeyCom,C8:B5:ADHewlettP,C8:B5:B7Apple,C8:B6:D3HuaweiTe,C8:BA:94SamsungE,C8:BA:E9Qdis,C8:BB:81HuaweiDe,C8:BB:D3Embrane,C8:BC:9CHuaweiDe,C8:BC:C8Apple,C8:BC:E5SenseThi,C8:BD:4DSamsungE,C8:BD:69SamsungE,C8:BE:19D-LinkIn,C8:BF:FEHuaweiDe,C8:C1:26ZPMIndus,C8:C1:3CRuggedTe,C8:C2:C6Shanghai,C8:C2:F5Flextron,C8:C2:FAHuaweiTe,C8:C4:65HuaweiTe,C8:C5:0EShenzhen,C8:C6:4AFlextron,C8:C7:50Motorola,C8:C7:91Zero1tv,C8:C9:A3Espressi,C8:CA:63HuaweiDe,C8:CA:79Ciena,C8:CB:9EIntelCor,C8:CB:B8HewlettP,C8:CD:72Sagemcom,C8:D0:19Shanghai,C8:D0:83Apple,C8:D1:0BNokia,C8:D1:2AComtrend,C8:D1:5EHuaweiTe,C8:D1:D1AGAiTTec,C8:D2:C1JetlunSh,C8:D3:A3D-LinkIn,C8:D3:FFHewlettP,C8:D4:29Muehlbau,C8:D5:90FlightDa,C8:D5:FEShenzhen,C8:D6:9DArabInte,C8:D7:19Cisco-Li,C8:D7:78BSHHausg,C8:D7:79QingDaoH,C8:D7:B0SamsungE,C8:D8:84Universa,C8:D9:D2HewlettP,C8:DB:26Logitech,C8:DD:C9LenovoMo,C8:DE:51IntegraO,C8:DE:C9Coriant,C8:DF:7CNokia,C8:DF:84TexasIns,C8:E0:EBApple,C8:E1:30Milkyway,C8:E1:A7Vertu,C8:E2:65IntelCor,C8:E4:2FTechnica,C8:E6:00HuaweiTe,C8:E7:76PTCOMTec,C8:E7:D8MercuryC,C8:E7:F0JuniperN,C8:EA:F8zte,C8:EE:08TangtopT,C8:EE:75PishionI,C8:EE:A6Shenzhen,C8:EF:2EBeijingG,C8:F2:30Guangdon,C8:F3:19LGElectr,C8:F3:6BYamatoSc,C8:F3:86Shenzhen,C8:F4:06Avaya,C8:F5:D6IEEERegi,C8:F5:D6:00:00:00/28MeiryoTe,C8:F5:D6:10:00:00/28ValeoInt,C8:F5:D6:20:00:00/28QbicTech,C8:F5:D6:30:00:00/28BBPOSInt,C8:F5:D6:40:00:00/28Evotor,C8:F5:D6:50:00:00/28Pinmicro,C8:F5:D6:60:00:00/28Jabil,C8:F5:D6:70:00:00/28OscarsPr,C8:F5:D6:80:00:00/28YarwardE,C8:F5:D6:90:00:00/28Shanghai,C8:F5:D6:A0:00:00/28HENANFOX,C8:F5:D6:B0:00:00/28UnitedBa,C8:F5:D6:C0:00:00/28Eltako,C8:F5:D6:D0:00:00/28Volansys,C8:F5:D6:E0:00:00/28Heitec,C8:F6:50Apple,C8:F6:8DSETechno,C8:F6:C8Fiberhom,C8:F7:04Building,C8:F7:33IntelCor,C8:F7:42HangZhou,C8:F7:50Dell,C8:F8:6DAlcatel-,C8:F9:46LOCOSYST,C8:F9:81Seneca,C8:F9:C8NewSharp,C8:F9:F9Cisco,C8:FA:84Trusonus,C8:FA:E1ARQDigit,C8:FB:26CiscoSPV,C8:FD:19TexasIns,C8:FE:30BejingDA,C8:FE:6AJuniperN,C8:FF:28LiteonTe,C8:FF:77Dyson,CA:12:5CMicrosof,CA:1E:45ASMediaT,CA:2B:5ESuzhouMo,CA:30:BFIEEE8021,CA:37:82Storbyte,CA:44:6CCypressC,CA:8F:A5Advanced,CA:C8:7EAllenOrg,CA:E5:05Mobiliti,CA:F2:5BLexarEle,CC:00:80Bettini,CC:03:D9CiscoMer,CC:03:FATechnico,CC:04:7CG-WAYMic,CC:04:B4SelectCo,CC:05:1BSamsungE,CC:05:77HuaweiTe,CC:06:77Fiberhom,CC:07:ABSamsungE,CC:07:E4LenovoMo,CC:08:8DApple,CC:08:E0Apple,CC:08:FBTp-LinkT,CC:09:C8Imaqliq,CC:0C:DAMiljovak,CC:0D:E7BMeters,CC:0D:ECCiscoSPV,CC:0D:F2Motorola,CC:10:A3BeijingN,CC:14:A6YichunMy,CC:15:31IntelCor,CC:16:7ECisco,CC:18:7BManzanit,CC:19:A8PTInovaç,CC:1A:FAzte,CC:1B:E0IEEERegi,CC:1B:E0:00:00:00/28Microtec,CC:1B:E0:10:00:00/28BeijingD,CC:1B:E0:20:00:00/28i-Trinet,CC:1B:E0:30:00:00/28Shenzhen,CC:1B:E0:40:00:00/28Laserwor,CC:1B:E0:50:00:00/28Earphone,CC:1B:E0:60:00:00/28ICRealTe,CC:1B:E0:70:00:00/28SichuanD,CC:1B:E0:80:00:00/28MDTtechn,CC:1B:E0:90:00:00/28MobiStor,CC:1B:E0:A0:00:00/28MatterLa,CC:1B:E0:B0:00:00/28ART&CORE,CC:1B:E0:C0:00:00/28Guangzho,CC:1B:E0:D0:00:00/28NewstarH,CC:1B:E0:E0:00:00/28CassiaNe,CC:1B:E0:F0:00:00/28Private,CC:1E:FFMetrolog,CC:1F:C4InVue,CC:20:8CHuaweiTe,CC:20:E8Apple,CC:21:19SamsungE,CC:22:18InnoDigi,CC:22:37IEEERegi,CC:22:37:00:00:00/28MEDCOMsp,CC:22:37:10:00:00/28TermaSp,CC:22:37:20:00:00/28ApeironD,CC:22:37:30:00:00/28XConnect,CC:22:37:40:00:00/28Shanghai,CC:22:37:50:00:00/28BeijingS,CC:22:37:60:00:00/28SiemensA,CC:22:37:70:00:00/28Shanghai,CC:22:37:80:00:00/28Safilo,CC:22:37:90:00:00/28EInk,CC:22:37:A0:00:00/28shenzhen,CC:22:37:B0:00:00/28Tolomati,CC:22:37:C0:00:00/28HebeiZHS,CC:22:37:D0:00:00/28Shenzhen,CC:22:37:E0:00:00/28Manufact,CC:24:2EShenzhen,CC:25:EFApple,CC:26:2DVerifi,CC:29:F5Apple,CC:2A:80Micro-Bi,CC:2C:83DarkMatt,CC:2D:1BSfr,CC:2D:21TendaTec,CC:2D:83Guangdon,CC:2D:8CLgElectr,CC:2D:B7Apple,CC:2D:E0Routerbo,CC:2F:71IntelCor,CC:30:80VAIO,CC:31:2AHuizhouT,CC:32:96HuaweiDe,CC:32:E5Tp-LinkT,CC:33:31TexasIns,CC:33:BBSagemcom,CC:34:29Tp-LinkT,CC:34:D7Gewiss,CC:35:40Technico,CC:35:5ASecuGen,CC:37:ABEdgecore,CC:39:8CShiningt,CC:3A:61SamsungE,CC:3A:DFNeptuneT,CC:3B:27TecnoMob,CC:3B:3ELesterEl,CC:3B:58Curiouse,CC:3C:3FSSDatent,CC:3D:82IntelCor,CC:3E:5FHewlettP,CC:3F:1DIntesisS,CC:3F:8AKomatsu,CC:3F:EABAE,CC:40:D0Netgear,CC:41:8EMSAInnov,CC:43:E3Trump,CC:44:63Apple,CC:46:39WAAV,CC:46:4ESamsungE,CC:46:D6Cisco,CC:47:03Intercon,CC:47:BDRhombus,CC:48:3ADell,CC:4A:E1fourtec-,CC:4B:73AMPAKTec,CC:4B:FBHellberg,CC:4D:38Carnegie,CC:4E:24BrocadeC,CC:4E:ECHUMAX,CC:4F:5CIEEERegi,CC:4F:5C:10:00:00/28lesswire,CC:4F:5C:20:00:00/28MatchX,CC:4F:5C:30:00:00/28Shanghai,CC:4F:5C:40:00:00/28SparkBio,CC:4F:5C:50:00:00/28Kymati,CC:4F:5C:60:00:00/28Watertec,CC:4F:5C:70:00:00/28SmithsUS,CC:4F:5C:80:00:00/28Feelmore,CC:4F:5C:90:00:00/28Dtrovisi,CC:4F:5C:A0:00:00/28Az-Techn,CC:4F:5C:B0:00:00/28Ontex,CC:4F:5C:C0:00:00/28BeijingC,CC:4F:5C:D0:00:00/28BeijingN,CC:4F:5C:E0:00:00/28ButtonsB,CC:50:0AFiberhom,CC:50:1CKVHIndus,CC:50:76OcomComm,CC:50:E3Espressi,CC:51:B4Integrat,CC:52:89Shenzhen,CC:52:AFUniversa,CC:53:B5HuaweiTe,CC:54:59OnTimeNe,CC:55:ADRim,CC:59:3ESensiumH,CC:5A:53Cisco,CC:5C:75Weightec,CC:5C:DEChinaMob,CC:5D:4EZyxelCom,CC:5D:57Informat,CC:5D:78JTDConsu,CC:5F:BFTopwise3,CC:60:BBEmpowerR,CC:61:E5Motorola,CC:64:A6HuaweiTe,CC:65:ADARRISGro,CC:66:0AApple,CC:66:B2Nokia,CC:68:B6TP-Link,CC:69:B0GlobalTr,CC:69:FAApple,CC:6A:10Chamberl,CC:6B:1ECloudNet,CC:6B:98MinetecW,CC:6B:F1SoundMas,CC:6D:A0Roku,CC:6D:EFTJKTieto,CC:6E:A4SamsungE,CC:70:EDCisco,CC:71:90VietnamP,CC:72:0FViscount,CC:72:86XianFeng,CC:73:14HongKong,CC:74:98Filmetri,CC:75:E2ARRISGro,CC:76:69Seetech,CC:78:5FApple,CC:78:ABTexasIns,CC:79:4ABLUProdu,CC:79:CFShenzhen,CC:7A:30CMAXWire,CC:7B:35zte,CC:7B:61Nikkiso,CC:7D:37ARRISGro,CC:7E:E7Panasoni,CC:7F:75Cisco,CC:7F:76Cisco,CC:81:2AvivoMobi,CC:81:DAPhicommS,CC:82:EBKyocera,CC:85:6CShenzhen,CC:86:ECSiliconL,CC:87:4ANokia,CC:88:26LGInnote,CC:88:C7ArubaaHe,CC:89:5EHuaweiTe,CC:89:6CGNHearin,CC:89:FDNokia,CC:8C:DAShenzhen,CC:8C:E3TexasIns,CC:8E:71Cisco,CC:90:70Cisco,CC:90:93HansongT,CC:90:E8Shenzhen,CC:91:2BTEConnec,CC:93:4ASierraWi,CC:94:4APfeiffer,CC:94:70Kinestra,CC:95:D7Vizio,CC:96:35LVS,CC:96:A0HuaweiTe,CC:98:8BSONYVisu,CC:98:91Cisco,CC:99:16Integrat,CC:9C:3ECiscoMer,CC:9D:A2EltexEnt,CC:9E:00Nintendo,CC:9E:A2AmazonTe,CC:9E:CAHMDGloba,CC:9F:35Transbit,CC:9F:7AChiunMai,CC:A0:E5DZGMeter,CC:A1:2BTCLKingE,CC:A2:19Shenzhen,CC:A2:23HuaweiTe,CC:A2:60SichuanT,CC:A3:74Guangdon,CC:A4:62ARRISGro,CC:A4:AFShenzhen,CC:A6:14AifaTech,CC:A7:C1Google,CC:AB:2CHUMAX,CC:AF:78HonHaiPr,CC:B0:A8HuaweiDe,CC:B0:DALiteonTe,CC:B1:1ASamsungE,CC:B1:82HuaweiTe,CC:B2:55D-LinkIn,CC:B3:ABshenzhen,CC:B3:F8FujitsuI,CC:B5:5AFraunhof,CC:B5:D1BeijingX,CC:B6:91NECMagnu,CC:B8:88AnBSecur,CC:B8:A8AMPAKTec,CC:B8:F1EagleKin,CC:BB:FEHuaweiTe,CC:BC:E3HuaweiTe,CC:BD:35Steinel,CC:BD:D3Ultimake,CC:BE:59Calix,CC:BE:71OptiLogi,CC:C0:79MurataMa,CC:C1:04AppliedT,CC:C2:61IEEERegi,CC:C2:61:00:00:00/28Ebiologi,CC:C2:61:10:00:00/28NWL,CC:C2:61:20:00:00/28Tecnoide,CC:C2:61:30:00:00/28Netradyn,CC:C2:61:40:00:00/28EDAGEngi,CC:C2:61:50:00:00/28ViperDes,CC:C2:61:60:00:00/28Guardiar,CC:C2:61:70:00:00/28AbilityE,CC:C2:61:90:00:00/28Byterg,CC:C2:61:A0:00:00/28Shenzhen,CC:C2:61:B0:00:00/28Winterth,CC:C2:61:C0:00:00/28NortekSe,CC:C2:61:D0:00:00/28DspreadT,CC:C2:61:E0:00:00/28ToongInE,CC:C2:E0Raisecom,CC:C3:EAMotorola,CC:C5:0AShenzhen,CC:C5:E5Dell,CC:C5:EFCo-CommS,CC:C6:2BTri,CC:C7:60Apple,CC:C8:D7CIASElet,CC:C9:2CSchindle,CC:C9:5DApple,CC:CC:4ESunFount,CC:CC:81HuaweiTe,CC:CC:CCSiliconL,CC:CD:64SM-Elect,CC:CE:1EAVMAudio,CC:CE:40Janteq,CC:D0:83ArubaaHe,CC:D2:81Apple,CC:D2:9BShenzhen,CC:D3:1EIEEERegi,CC:D3:1E:00:00:00/28SAMIM,CC:D3:1E:10:00:00/28RondoBur,CC:D3:1E:20:00:00/28Neptune,CC:D3:1E:30:00:00/28Ken,CC:D3:1E:40:00:00/28PJGSyste,CC:D3:1E:50:00:00/28NTmore,CC:D3:1E:60:00:00/28BBPOSInt,CC:D3:1E:70:00:00/28Shenzhen,CC:D3:1E:80:00:00/28inoage,CC:D3:1E:90:00:00/28SiemensM,CC:D3:1E:A0:00:00/28HaishuTe,CC:D3:1E:B0:00:00/28ElkProdu,CC:D3:1E:C0:00:00/28NantEner,CC:D3:1E:D0:00:00/28Cujo,CC:D3:1E:E0:00:00/28ShenZhen,CC:D3:9DIEEERegi,CC:D3:9D:00:00:00/28Inx,CC:D3:9D:10:00:00/28EvokoUnl,CC:D3:9D:20:00:00/28Continen,CC:D3:9D:30:00:00/28MagTarge,CC:D3:9D:40:00:00/28Shenzhen,CC:D3:9D:50:00:00/28Shenzhen,CC:D3:9D:60:00:00/28Krontech,CC:D3:9D:70:00:00/28Glenair,CC:D3:9D:80:00:00/28Obelisk,CC:D3:9D:90:00:00/28BejingNe,CC:D3:9D:A0:00:00/28Lubelski,CC:D3:9D:B0:00:00/28Q-Branch,CC:D3:9D:C0:00:00/28Hangzhou,CC:D3:9D:D0:00:00/28Ethernit,CC:D3:9D:E0:00:00/28Shanghai,CC:D3:C1VestelEl,CC:D3:E2JiangsuY,CC:D4:2EArcadyan,CC:D4:A1MitraSta,CC:D5:39Cisco,CC:D7:3CHuaweiTe,CC:D8:11AiconnTe,CC:D8:1FMaipuCom,CC:D8:C1Cisco,CC:D9:ACIntelCor,CC:D9:E9SCREngin,CC:DB:04DataRemo,CC:DB:93Cisco,CC:DC:55Dragonch,CC:E0:C3EXTENTec,CC:E0:DABaiduOnl,CC:E1:7FJuniperN,CC:E1:94JuniperN,CC:E1:D5Buffalo,CC:E2:36Hangzhou,CC:E7:98MySocial,CC:E7:DFAmerican,CC:E8:ACSOYEATec,CC:EA:1CDCONWORK,CC:ED:21NokiaSha,CC:ED:4DCisco,CC:ED:DCMitraSta,CC:EE:D9VAHLEAut,CC:EF:03HunanKey,CC:EF:48Cisco,CC:F0:FDChinaMob,CC:F3:A5ChiMeiCo,CC:F4:07EukreaEl,CC:F4:11Google,CC:F5:383isysnet,CC:F5:5FEFocusIn,CC:F6:7AAyeckaCo,CC:F7:35AmazonTe,CC:F8:41Lumewave,CC:F8:F0XianHISU,CC:F9:54Avaya,CC:F9:57u-blox,CC:F9:E4IntelCor,CC:F9:E8SamsungE,CC:FA:00LGElectr,CC:FA:66HuaweiDe,CC:FB:65Nintendo,CC:FC:6DRizTrans,CC:FC:B1Wireless,CC:FD:17TCTmobil,CC:FE:3CSamsungE,CC:FF:90HuaweiDe,D0:03:4BApple,D0:03:DFSamsungE,D0:03:EBTexasIns,D0:04:01Motorola,D0:04:92Fiberhom,D0:05:2AArcadyan,D0:05:E4HuaweiDe,D0:07:90TexasIns,D0:07:CAJuniperN,D0:0A:ABYokogawa,D0:0D:F7HuaweiDe,D0:0E:A4PorscheC,D0:0E:D9TaicangT,D0:0F:6DT&WElect,D0:12:42BIOS,D0:12:CBAVMAudio,D0:13:1ESunrexTe,D0:13:FDLGElectr,D0:14:11IEEERegi,D0:14:11:00:00:00/28EkkoSens,D0:14:11:10:00:00/28Private,D0:14:11:20:00:00/28EvocoLab,D0:14:11:30:00:00/28iLOQ,D0:14:11:40:00:00/28powerall,D0:14:11:60:00:00/28Ahnnet,D0:14:11:70:00:00/28Realwave,D0:14:11:80:00:00/28VideoSec,D0:14:11:90:00:00/28Airthing,D0:14:11:A0:00:00/28AbbEvi,D0:14:11:B0:00:00/28CYLTek,D0:14:11:C0:00:00/28ShenZhen,D0:14:11:D0:00:00/28Guangdon,D0:14:11:E0:00:00/28Tecnosof,D0:15:4Azte,D0:15:A6ArubaaHe,D0:16:7Ceero,D0:16:B4HuaweiTe,D0:17:69MurataMa,D0:17:6ASamsungE,D0:17:C2ASUSTekC,D0:19:6ACiena,D0:1A:A7UniPrint,D0:1B:49SamsungE,D0:1C:3CTecnoMob,D0:1C:BBBeijingC,D0:1E:1DSaiNXTTe,D0:21:ACYoLabs,D0:21:F9Ubiquiti,D0:22:12IEEERegi,D0:22:12:00:00:00/28SpiritIT,D0:22:12:10:00:00/28Aim,D0:22:12:20:00:00/28RHENAC,D0:22:12:40:00:00/28Viatron,D0:22:12:50:00:00/28Shanghai,D0:22:12:60:00:00/28UranoInd,D0:22:12:70:00:00/28Cliptech,D0:22:12:80:00:00/28Shenzhen,D0:22:12:90:00:00/28UAB&quot,D0:22:12:A0:00:00/28GNS,D0:22:12:B0:00:00/28Schleife,D0:22:12:C0:00:00/28XperioLa,D0:22:12:D0:00:00/28Shenzhen,D0:22:12:E0:00:00/28uLux,D0:22:12:F0:00:00/28Private,D0:22:BESamsungE,D0:23:DBApple,D0:25:16MercuryC,D0:25:44SamsungE,D0:25:98Apple,D0:27:88HonHaiPr,D0:28:BARealmeCh,D0:2B:20Apple,D0:2C:45littleBi,D0:2D:B3HuaweiTe,D0:2E:ABTexasIns,D0:31:10IngenicS,D0:31:69SamsungE,D0:33:11Apple,D0:35:E5EMMicroe,D0:37:42YulongCo,D0:37:45Tp-LinkT,D0:37:61TexasIns,D0:39:72TexasIns,D0:39:B3ARRISGro,D0:39:EANetApp,D0:3C:1FIntelCor,D0:3D:52AvaSecur,D0:3D:C3AQ,D0:3E:5CHuaweiTe,D0:3E:7DChipseaT,D0:3F:27WyzeLabs,D0:3F:AAApple,D0:40:EFMurataMa,D0:41:C9Fiberhom,D0:43:1EDell,D0:46:DCSouthwes,D0:47:C1ElmaElec,D0:48:F3DATTUS,D0:49:7COnePlusT,D0:49:8BZoomServ,D0:4C:C1SINTRONE,D0:4D:2CRoku,D0:4D:C6ArubaaHe,D0:4E:50Mobiwire,D0:4F:7EApple,D0:50:99ASRockIn,D0:51:57LEAXArki,D0:51:62Sony,D0:52:A8Physical,D0:53:49LiteonTe,D0:54:2DCambridg,D0:54:75SAVICont,D0:55:09Nintendo,D0:55:B2Integrat,D0:56:BFAmosense,D0:57:4CCisco,D0:57:7BIntelCor,D0:57:85Pantech,D0:57:94Sagemcom,D0:57:A1WermaSig,D0:58:75ActiveCo,D0:58:A8zte,D0:58:C0QingdaoH,D0:58:FCBSkyB,D0:59:19zte,D0:59:95Fiberhom,D0:59:C3CeraMicr,D0:59:E4SamsungE,D0:5A:00Technico,D0:5A:0FI-BtDigi,D0:5A:F1Shenzhen,D0:5A:FDRealmeCh,D0:5B:A8zte,D0:5C:7ASarturad,D0:5F:64IEEERegi,D0:5F:64:00:00:00/28Decathlo,D0:5F:64:10:00:00/28Hangzhou,D0:5F:64:20:00:00/28Shanghai,D0:5F:64:30:00:00/28HuaqinTe,D0:5F:64:40:00:00/28wallbe,D0:5F:64:50:00:00/28AtollSol,D0:5F:64:60:00:00/28CyrusTec,D0:5F:64:70:00:00/28BeijingC,D0:5F:64:80:00:00/28TytoCare,D0:5F:64:90:00:00/28Shanghai,D0:5F:64:A0:00:00/28PartnerN,D0:5F:64:B0:00:00/28NorthAme,D0:5F:64:C0:00:00/28NanjingH,D0:5F:64:D0:00:00/28Shenzhen,D0:5F:64:E0:00:00/28Montblan,D0:5F:B8TexasIns,D0:5F:CEHitachiD,D0:60:8Czte,D0:62:A0ChinaEss,D0:63:4DMeikoMas,D0:63:B4SolidRun,D0:65:44Apple,D0:65:CAHuaweiTe,D0:66:6DShenzhen,D0:66:7BSamsungE,D0:67:26HewlettP,D0:67:E5Dell,D0:69:9ELUMINEXL,D0:69:D0VertoMed,D0:6A:1FBse,D0:6E:DESagemcom,D0:6F:4ATopwellI,D0:6F:82HuaweiTe,D0:71:C4zte,D0:72:DCCisco,D0:73:7FMini-Cir,D0:73:8EDongOhPr,D0:73:D5LifiLabs,D0:75:BERenoA&E,D0:76:50IEEERegi,D0:76:50:00:00:00/28CentrAle,D0:76:50:10:00:00/28DaikenAu,D0:76:50:20:00:00/28HappoSol,D0:76:50:30:00:00/28TAPKOTec,D0:76:50:40:00:00/28Private,D0:76:50:50:00:00/28Annapurn,D0:76:50:60:00:00/28Picobrew,D0:76:50:70:00:00/28ENCOREDT,D0:76:50:80:00:00/28Accumula,D0:76:50:90:00:00/28Greenwav,D0:76:50:A0:00:00/28InventDe,D0:76:50:B0:00:00/28PelKorea,D0:76:50:C0:00:00/28Electro-,D0:76:50:D0:00:00/28tecnotro,D0:76:50:E0:00:00/28Revox,D0:76:50:F0:00:00/28Private,D0:76:8FCalix,D0:76:E7Tp-LinkT,D0:77:14Motorola,D0:78:80Fiberhom,D0:7A:B5HuaweiTe,D0:7C:2DLeieIOTt,D0:7D:33HuaweiDe,D0:7D:E5ForwardP,D0:7E:28HewlettP,D0:7E:35IntelCor,D0:7F:A0SamsungE,D0:7F:C4OuWeiTec,D0:81:7AApple,D0:83:D4XtelWire,D0:84:B0Sagemcom,D0:87:E2SamsungE,D0:89:99APCON,D0:8A:55Skullcan,D0:8A:91Technico,D0:8B:7EPassifSe,D0:8C:B5TexasIns,D0:8C:FFUpwis,D0:8E:79Dell,D0:92:9EMicrosof,D0:92:FAFiberhom,D0:93:80DucereTe,D0:93:F8Stonestr,D0:94:66Dell,D0:95:C7Pantech,D0:96:FBDASANNet,D0:97:FERealmeCh,D0:99:D5Alcatel-,D0:9B:05Emtronix,D0:9C:30FosterEl,D0:9C:7AXiaomiCo,D0:9C:AEvivoMobi,D0:9D:0ALinkcom,D0:9D:ABTCTmobil,D0:9F:D9IEEERegi,D0:9F:D9:00:00:00/28LemeiInt,D0:9F:D9:10:00:00/28elecgato,D0:9F:D9:20:00:00/28WestarDi,D0:9F:D9:30:00:00/28GSYuasaI,D0:9F:D9:40:00:00/28PotenSha,D0:9F:D9:50:00:00/28CarbonMo,D0:9F:D9:60:00:00/28ElevocTe,D0:9F:D9:70:00:00/28RaymaxTe,D0:9F:D9:80:00:00/28Queclink,D0:9F:D9:90:00:00/28ENTTEC,D0:9F:D9:A0:00:00/28Eurolan,D0:9F:D9:B0:00:00/28Cablewir,D0:9F:D9:C0:00:00/28FujianNe,D0:9F:D9:D0:00:00/28Shenzhen,D0:9F:D9:E0:00:00/28Minibems,D0:A0:D6ChengDuT,D0:A3:11Neuberge,D0:A4:B1Sonifex,D0:A5:A6Cisco,D0:A6:37Apple,D0:AB:D5IntelCor,D0:AE:ECAlphaNet,D0:AF:B6LinktopT,D0:B0:CDMoen,D0:B1:28SamsungE,D0:B2:14PoeWit,D0:B2:C4Technico,D0:B3:3FShenzhen,D0:B4:5DHuaweiDe,D0:B4:98RobertBo,D0:B5:23Bestcare,D0:B5:3DSeproRob,D0:B5:C2TexasIns,D0:B6:0AXingluoT,D0:B6:6FSernetSu,D0:BA:E4Shanghai,D0:BB:80SHLTelem,D0:BC:C1WeifangG,D0:BD:01DSIntern,D0:BE:2CCNSLink,D0:BF:9CHewlettP,D0:C0:BFActionsM,D0:C1:93Skybell,D0:C1:B1SamsungE,D0:C2:4ESamsungE,D0:C2:82Cisco,D0:C3:1EJUNGJINE,D0:C3:5AJabilCir,D0:C4:2FTamagawa,D0:C5:D3AzureWav,D0:C5:D8Latecoer,D0:C5:F3Apple,D0:C6:37IntelCor,D0:C6:5BHuaweiTe,D0:C7:89Cisco,D0:C7:C0Tp-LinkT,D0:C8:57IEEERegi,D0:C8:57:00:00:00/28YUANHigh,D0:C8:57:10:00:00/28Dali,D0:C8:57:20:00:00/28Forgamer,D0:C8:57:30:00:00/28Mobicon,D0:C8:57:40:00:00/28IminTech,D0:C8:57:50:00:00/28BeijingI,D0:C8:57:60:00:00/28Innovati,D0:C8:57:70:00:00/28EcoMobil,D0:C8:57:80:00:00/28NanjingM,D0:C8:57:90:00:00/28Shenzhen,D0:C8:57:A0:00:00/28shenzhen,D0:C8:57:B0:00:00/28Chunghsi,D0:C8:57:C0:00:00/28DanteSec,D0:C8:57:D0:00:00/28Iflytek,D0:C8:57:E0:00:00/28E-T-AEle,D0:CD:E1Scientec,D0:CF:5EEnergyMi,D0:CF:D8HuizhouB,D0:D0:03SamsungE,D0:D0:4BHuaweiTe,D0:D0:FDCisco,D0:D2:12K2NET,D0:D2:3CApple,D0:D2:86BeckmanC,D0:D2:B0Apple,D0:D3:E0ArubaaHe,D0:D3:FCMios,D0:D4:12ADBBroad,D0:D4:71MVTECH,D0:D6:CCWintop,D0:D7:83HuaweiTe,D0:D9:4FIEEERegi,D0:D9:4F:00:00:00/28PerfantT,D0:D9:4F:10:00:00/28mycable,D0:D9:4F:20:00:00/28TecoImag,D0:D9:4F:30:00:00/28BeijingY,D0:D9:4F:40:00:00/28peikerCE,D0:D9:4F:50:00:00/28OptigoNe,D0:D9:4F:60:00:00/28HyundaiA,D0:D9:4F:70:00:00/28Mitsubis,D0:D9:4F:80:00:00/28ApptionL,D0:D9:4F:90:00:00/28Hangzhou,D0:D9:4F:A0:00:00/28Shenzhen,D0:D9:4F:B0:00:00/28MAXSmart,D0:D9:4F:C0:00:00/28Arrowave,D0:D9:4F:D0:00:00/28Duksanme,D0:D9:4F:E0:00:00/28Appotron,D0:DB:32Nokia,D0:DB:B7Casa,D0:DD:49JuniperN,D0:DF:9ALiteonTe,D0:DF:B2GenieNet,D0:DF:C7SamsungE,D0:E0:42Cisco,D0:E1:40Apple,D0:E3:47Yoga,D0:E4:0BWearable,D0:E4:4AMurataMa,D0:E5:4DARRISGro,D0:E7:82AzureWav,D0:EB:03Zhehuate,D0:EB:9ESeowoo,D0:EC:35Cisco,D0:EF:C1HuaweiTe,D0:F0:DBEricsson,D0:F1:21XianLINK,D0:F2:7FSteadySe,D0:F3:F5HuaweiDe,D0:F5:20KYOCERA,D0:F7:3BHelmutMa,D0:F8:65ItelMobi,D0:F8:8CMotorola,D0:FA:1DQihoo360,D0:FC:CCSamsungE,D0:FF:50TexasIns,D0:FF:98HuaweiTe,D4:00:0DPhoenixB,D4:00:57MCTechno,D4:01:29Broadcom,D4:01:6DTp-LinkT,D4:02:4ADelphian,D4:04:CDARRISGro,D4:04:FFJuniperN,D4:05:98ARRISGro,D4:05:DEeero,D4:08:68BeijingL,D4:0A:A9ARRISGro,D4:0B:1AHTC,D4:0B:B9SolidSem,D4:0F:B2AppliedM,D4:10:90iNFORM,D4:10:CFHuanshun,D4:11:A3SamsungE,D4:11:D6ShotSpot,D4:12:43AMPAKTec,D4:12:96AnobitTe,D4:12:BBQuadrant,D4:13:6FAsiaPaci,D4:13:F8PeplinkI,D4:19:F6NXPSemic,D4:1A:3FGuangdon,D4:1A:C8NipponPr,D4:1A:D1ZyxelCom,D4:1B:81Chongqin,D4:1C:1CRcf,D4:1D:71PaloAlto,D4:1E:35TOHOElec,D4:1F:0CJAIManuf,D4:20:6DHTC,D4:20:B0Mist,D4:21:22Sercomm,D4:22:3FLenovoMo,D4:22:4EAlcatelL,D4:22:CDXsensTec,D4:24:93GWTechno,D4:25:8BIntelCor,D4:25:CCIEEERegi,D4:25:CC:00:00:00/28NordiTel,D4:25:CC:10:00:00/28EwareInf,D4:25:CC:20:00:00/28MusicLen,D4:25:CC:30:00:00/28EISST,D4:25:CC:40:00:00/28Barobo,D4:25:CC:50:00:00/28bvktechn,D4:25:CC:60:00:00/28NanjingL,D4:25:CC:70:00:00/28BlueCats,D4:25:CC:80:00:00/28DolbyLab,D4:25:CC:90:00:00/28TakumiJa,D4:25:CC:A0:00:00/28E-MetroT,D4:25:CC:B0:00:00/28Veea,D4:25:CC:C0:00:00/28POSNETPo,D4:25:CC:D0:00:00/28Combined,D4:25:CC:E0:00:00/28Coperion,D4:27:51Infopia,D4:28:B2ioBridge,D4:28:D5TCTmobil,D4:29:EAZimory,D4:2C:0FARRISGro,D4:2C:3DSkyLight,D4:2C:44Cisco,D4:2D:C5Panasoni,D4:2F:23AkenoriP,D4:31:9DSinwatec,D4:32:60GoPro,D4:32:66Fike,D4:35:1DTechnico,D4:36:39TexasIns,D4:36:DBJiangsuT,D4:37:D7zte,D4:38:9CSony,D4:39:B8Ciena,D4:3A:2EShenzhen,D4:3A:65IGRSEngi,D4:3A:E9DONGGUAN,D4:3B:04IntelCor,D4:3D:39DialogSe,D4:3D:67CarmaInd,D4:3D:7EMicro-St,D4:3D:F3ZyxelCom,D4:3F:CBARRISGro,D4:40:D0OCOSMOS,D4:40:F0HuaweiTe,D4:41:65SichuanT,D4:43:A8Changzho,D4:45:E8JiangxiH,D4:46:49HuaweiTe,D4:46:E1Apple,D4:47:5AScreenBe,D4:48:2DShenzhen,D4:4B:5ETaiyoYud,D4:4B:B6Zhejiang,D4:4C:24Vuppalam,D4:4C:9CShenzhen,D4:4C:A7Informte,D4:4D:A4MurataMa,D4:4F:67HuaweiTe,D4:4F:68EideticC,D4:4F:80KemperDi,D4:50:3FGuangdon,D4:50:7ACEIVALog,D4:52:2ATangoWiF,D4:52:51IBTIngen,D4:52:97nSTREAMS,D4:52:EEBSkyB,D4:53:83MurataMa,D4:53:AFVIGOSyst,D4:54:8BIntelCor,D4:55:56FiberMou,D4:55:BEShenzhen,D4:57:63Apple,D4:58:00Fiberhom,D4:5A:B2Galleon,D4:5C:70Wi-FiAll,D4:5D:42Nokia,D4:5D:64ASUSTekC,D4:5D:DFPegatron,D4:5E:ECBeijingX,D4:5F:25Shenzhen,D4:60:75BaiduOnl,D4:60:E3Sercomm,D4:61:2EHuaweiTe,D4:61:32ProConce,D4:61:9DApple,D4:61:DAApple,D4:61:FEHangzhou,D4:62:EAHuaweiTe,D4:63:C6Motorola,D4:63:DEvivoMobi,D4:63:FEArcadyan,D4:64:F7ChengduU,D4:66:A8RiedoNet,D4:67:61XonTelTe,D4:67:D3Guangdon,D4:67:E7Fiberhom,D4:68:4DRuckusWi,D4:68:67Neoventu,D4:68:AAApple,D4:68:BAShenzhen,D4:69:A5Miura,D4:6A:35Cisco,D4:6A:6AHonHaiPr,D4:6A:91SnapAV,D4:6A:A8HuaweiTe,D4:6B:A6HuaweiTe,D4:6C:BFGoodrich,D4:6C:DACSM,D4:6D:50Cisco,D4:6D:6DIntelCor,D4:6E:0ETp-LinkT,D4:6E:5CHuaweiTe,D4:6F:42WAXESSUS,D4:72:08Bragi,D4:72:26zte,D4:73:50DBGCommn,D4:74:15HuaweiDe,D4:74:1BBeijingH,D4:76:A0Fortinet,D4:76:EAzte,D4:77:2BNanjingZ,D4:77:98Cisco,D4:77:B2NetixGlo,D4:78:56Avaya,D4:78:9BCisco,D4:79:54HuaweiDe,D4:79:C3Camerone,D4:7A:E2SamsungE,D4:7B:35NEOMonit,D4:7B:75HARTINGE,D4:7B:B0AskeyCom,D4:7C:44IEEERegi,D4:7C:44:00:00:00/28Exafore,D4:7C:44:10:00:00/28InnovizT,D4:7C:44:20:00:00/28YunDingN,D4:7C:44:30:00:00/28OmronSen,D4:7C:44:40:00:00/28SammiOnf,D4:7C:44:50:00:00/28LSCommun,D4:7C:44:60:00:00/28ASDAICT,D4:7C:44:70:00:00/28PongeeIn,D4:7C:44:80:00:00/28BeijingM,D4:7C:44:90:00:00/28SuzhouWa,D4:7C:44:A0:00:00/28Tendzone,D4:7C:44:B0:00:00/28OPTiM,D4:7C:44:C0:00:00/28StriveOr,D4:7C:44:D0:00:00/28HuaqinTe,D4:7C:44:E0:00:00/28Shenzhen,D4:7D:FCTecnoMob,D4:7E:E4ChinaMob,D4:81:CAiDevices,D4:81:D7Dell,D4:82:3EArgosyTe,D4:83:04Shenzhen,D4:84:57GDMideaA,D4:85:64HewlettP,D4:86:60Arcadyan,D4:87:D8SamsungE,D4:88:3FHdpro,D4:88:66HuaweiTe,D4:88:90SamsungE,D4:8A:39SamsungE,D4:8A:3BHunanFn-,D4:8C:B5Cisco,D4:8D:D9MeldTech,D4:8F:33Microsof,D4:8F:A2HuaweiDe,D4:8F:AASogecamI,D4:90:9CApple,D4:90:E0TopconEl,D4:91:0FAmazonTe,D4:91:AFElectroa,D4:92:34NEC,D4:93:90Clevo,D4:93:98Nokia,D4:93:A0Fidelix,D4:94:00HuaweiTe,D4:94:5ACosmo,D4:94:A1TexasIns,D4:94:E8HuaweiTe,D4:94:FBContinen,D4:95:24CloverNe,D4:96:DFSungjinC,D4:97:0BXiaomiCo,D4:9A:20Apple,D4:9A:A0VnptTech,D4:9B:5CChongqin,D4:9C:28JayBird,D4:9C:8EUniversi,D4:9C:DDAMPAKTec,D4:9C:F4PaloAlto,D4:9D:C0SamsungE,D4:9E:05zte,D4:9E:3BGuangzho,D4:9E:6DWuhanZho,D4:A0:2ACisco,D4:A1:48HuaweiTe,D4:A3:3DApple,D4:A4:25SMAXTech,D4:A4:99InViewTe,D4:A6:51TuyaSmar,D4:A9:28GreenWav,D4:AA:FFMicroWor,D4:AB:82ARRISGro,D4:AB:CDHuiZhouG,D4:AC:4EBODirS,D4:AD:2DFiberhom,D4:AD:71Cisco,D4:AD:BDCisco,D4:AD:FCPrivate,D4:AE:05SamsungE,D4:AE:52Dell,D4:AF:F7AristaNe,D4:B1:10HuaweiTe,D4:B1:69LeShiZhi,D4:B2:7AARRISGro,D4:B4:3EMesscomp,D4:B7:09zte,D4:B7:61SichuanA,D4:B7:D0Ciena,D4:B8:FFHomeCont,D4:B9:2FTechnico,D4:BB:C8vivoMobi,D4:BB:E6HuaweiDe,D4:BD:1E5VTTechn,D4:BE:D9Dell,D4:BF:2DSEContro,D4:BF:7FUpvel,D4:C1:9ERuckusWi,D4:C1:C8zte,D4:C1:FCNokia,D4:C3:B0Gearlinx,D4:C7:66Acentic,D4:C8:B0PrimeEle,D4:C9:3CCisco,D4:C9:4BMotorola,D4:C9:B2Quanergy,D4:C9:EFHewlettP,D4:CA:6DRouterbo,D4:CA:6Eu-blox,D4:CB:AFNokia,D4:CE:B8Enatel,D4:CF:37Symbolic,D4:CF:F9Shenzhen,D4:D1:84ADBBroad,D4:D2:49PowerEth,D4:D2:52IntelCor,D4:D2:D6Fn-LinkT,D4:D2:E5BKAV,D4:D5:0DSouthwes,D4:D5:1BHuaweiTe,D4:D7:48Cisco,D4:D7:A9Shanghai,D4:D7:CFRealmeCh,D4:D8:98KoreaCNO,D4:D9:19GoPro,D4:DA:CDBSkyB,D4:DC:09Mist,D4:DC:CDApple,D4:DF:57Alpinion,D4:E0:8EValueHD,D4:E2:CBTechnico,D4:E3:2CSSiedleS,D4:E3:3FNokia,D4:E6:B7SamsungE,D4:E8:53Hangzhou,D4:E8:80Cisco,D4:E8:B2SamsungE,D4:E9:0BCvt,D4:EA:0EAvaya,D4:EB:68Cisco,D4:EC:0CHarley-D,D4:EC:86LinkedHo,D4:EC:ABvivoMobi,D4:EE:07HIWIFI,D4:EE:DESichuanT,D4:F0:27TrustPow,D4:F0:57Nintendo,D4:F0:B4NapcoSec,D4:F1:43IPROAD,D4:F2:07DIAODIAO,D4:F3:37Xunison,D4:F4:6FApple,D4:F4:BEPaloAlto,D4:F5:13TexasIns,D4:F5:27Siemens,D4:F5:47Google,D4:F5:EFHewlettP,D4:F6:3FIea,D4:F7:56zte,D4:F7:86Fiberhom,D4:F8:29Sagemcom,D4:F9:A1HuaweiTe,D4:FC:13Fiberhom,D8:00:4DApple,D8:00:93Aurender,D8:05:2ESkyviia,D8:06:D1Honeywel,D8:07:B6Tp-LinkT,D8:08:31SamsungE,D8:08:F5ArcadiaN,D8:09:C3Cercacor,D8:09:D6Zexelon,D8:0A:60HuaweiTe,D8:0B:9ASamsungE,D8:0B:CBTelinkSe,D8:0C:CFCGVS,D8:0D:17Tp-LinkT,D8:0D:E3FxiTechn,D8:0F:99HonHaiPr,D8:10:9FHuaweiTe,D8:10:CBAndreaIn,D8:12:65Chongqin,D8:13:99HuiZhouG,D8:14:D6SURESYST,D8:14:DFTCLKingE,D8:15:0DTp-LinkT,D8:16:0ANipponEl,D8:16:C1DewavHkE,D8:18:2BContiTem,D8:18:D3JuniperN,D8:19:7ANuheara,D8:19:CETelesqua,D8:1B:FETwinlinx,D8:1C:14Compacta,D8:1C:79Apple,D8:1D:72Apple,D8:1E:DDGuangdon,D8:1E:DEB&WGroup,D8:1F:12TuyaSmar,D8:1F:CCBrocadeC,D8:20:9FCubroAcr,D8:22:F4AvnetSil,D8:24:77Universa,D8:24:BDCisco,D8:25:22ARRISGro,D8:25:B0Rockeete,D8:26:B9Guangdon,D8:27:0CMaxTroni,D8:28:C9GeneralE,D8:29:16AscentCo,D8:29:18HuaweiTe,D8:29:86BestWish,D8:2A:15Leitner,D8:2A:7ENokia,D8:2D:9BShenzhen,D8:2D:E1Tricasca,D8:2F:E6Zhejiang,D8:30:62Apple,D8:31:34Roku,D8:31:CFSamsungE,D8:32:14TendaTec,D8:32:5AShenzhen,D8:32:E3XiaomiCo,D8:33:7FOfficeFA,D8:34:EEStemAudi,D8:37:3BShenzhen,D8:37:BEShenzhen,D8:38:0DSHENZHEN,D8:38:FCRuckusWi,D8:3A:F5Wideband,D8:3B:BFIntelCor,D8:3C:69Shenzhen,D8:42:ACShanghai,D8:42:E2CanaryCo,D8:43:EDSuzuken,D8:44:5CDEVTecno,D8:45:2BIntegrat,D8:46:06SiliconV,D8:47:10SichuanC,D8:47:32Tp-LinkT,D8:47:BBHuaweiDe,D8:48:EEHangzhou,D8:49:0BHuaweiTe,D8:49:2FCanon,D8:4A:87OiElectr,D8:4B:2ACognitas,D8:4C:90Apple,D8:4D:B9WuQiTech,D8:4F:37Proxissp,D8:4F:B8LgElectr,D8:50:E6ASUSTekC,D8:54:3ATexasIns,D8:54:A2ExtremeN,D8:55:75SamsungE,D8:55:A3zte,D8:57:EFSamsungE,D8:58:D7CZNICzsp,D8:59:82HuaweiTe,D8:5B:2ASamsungE,D8:5D:4CTp-LinkT,D8:5D:84CAxsoft,D8:5D:E2HonHaiPr,D8:5D:EFBusch-Ja,D8:5D:FBPrivate,D8:5E:D3Giga-Byt,D8:5F:77TelinkSe,D8:60:B0bioMérie,D8:60:B3Guangdon,D8:61:62WistronN,D8:61:94Objetivo,D8:62:DBEno,D8:63:75XiaomiCo,D8:65:95ToysMyth,D8:66:C6Shenzhen,D8:66:EEBoxinCom,D8:67:D3HuaweiDe,D8:67:D9Cisco,D8:68:52HuaweiTe,D8:68:C3SamsungE,D8:69:60Steinsvi,D8:6B:F7Nintendo,D8:6C:02HuaqinTe,D8:6C:5AHUMAX,D8:6C:63Google,D8:6C:E9Sagemcom,D8:6D:17HuaweiTe,D8:71:4DTexasIns,D8:71:57LenovoMo,D8:74:95zte,D8:75:33Nokia,D8:76:0AEscort,D8:76:AEHuaweiTe,D8:77:8BIntelbra,D8:78:7FUbeeInte,D8:78:E5Kuhn,D8:79:88HonHaiPr,D8:7C:DDSanix,D8:7D:7FSagemcom,D8:7E:76ItelMobi,D8:7E:B1xoware,D8:80:39Microchi,D8:80:3CAnhuiHua,D8:80:83CloudNet,D8:81:CEAhn,D8:84:66ExtremeN,D8:86:0BIEEERegi,D8:86:0B:00:00:00/28InspurGr,D8:86:0B:10:00:00/28Krspace,D8:86:0B:20:00:00/28GetSAT,D8:86:0B:30:00:00/28Auvidea,D8:86:0B:40:00:00/28Teplovod,D8:86:0B:50:00:00/28Camtrace,D8:86:0B:60:00:00/28Scanmati,D8:86:0B:70:00:00/28Grünbeck,D8:86:0B:80:00:00/28VrindaNa,D8:86:0B:90:00:00/28DigitalC,D8:86:0B:A0:00:00/28GLOScien,D8:86:0B:B0:00:00/28LibraryI,D8:86:0B:C0:00:00/28YusanInd,D8:86:0B:D0:00:00/28ComNavTe,D8:86:0B:E0:00:00/28Shenzhen,D8:87:D5Leadcore,D8:88:CERFTechno,D8:8A:3BUnit-Em,D8:8A:DCHuaweiDe,D8:8B:4CKingTing,D8:8C:73zte,D8:8C:79Google,D8:8D:5CElentec,D8:8D:C8AtilTech,D8:8F:76Apple,D8:90:E8SamsungE,D8:91:2AZyxelCom,D8:91:36DoverFue,D8:93:41GeneralE,D8:94:03HewlettP,D8:95:2FTexasIns,D8:96:85GoPro,D8:96:95Apple,D8:96:E0AlibabaC,D8:97:3BArtesynE,D8:97:60C2Develo,D8:97:7CGreyInno,D8:97:90Commonwe,D8:97:BAPegatron,D8:9A:34BeijingS,D8:9A:C1Nokia,D8:9B:3BHuaweiTe,D8:9C:67HonHaiPr,D8:9D:67HewlettP,D8:9D:B9eMegatec,D8:9E:3FApple,D8:9E:61HuaweiDe,D8:9E:D4Fiberhom,D8:9E:F3Dell,D8:A0:11WiZ,D8:A0:1DEspressi,D8:A1:05Syslane,D8:A2:5EApple,D8:A3:15vivoMobi,D8:A3:5CSamsungE,D8:A4:91HuaweiDe,D8:A5:34Spectron,D8:A6:F0WuQiTech,D8:A6:FDGhostLoc,D8:A7:56Sagemcom,D8:A8:C8zte,D8:A9:8BTexasIns,D8:AD:DDSonavati,D8:AE:90ItibiaTe,D8:AE:D0Shanghai,D8:AF:3BHangzhou,D8:AF:81NPKRotek,D8:AF:F1Panasoni,D8:B0:2EGuangzho,D8:B0:4CJinanUSR,D8:B0:53XiaomiCo,D8:B1:22JuniperN,D8:B1:2APanasoni,D8:B1:90Cisco,D8:B3:77HTC,D8:B6:B7Comtrend,D8:B6:C1NetworkA,D8:B6:D6BluTethe,D8:B8:F6Nantwork,D8:B9:0ETripleDo,D8:BB:2CApple,D8:BB:C1Micro-St,D8:BC:59Shenzhen,D8:BE:1FApple,D8:BE:65AmazonTe,D8:BF:4CVictoryC,D8:BF:C0Espressi,D8:C0:68Netgenet,D8:C0:6AHunantvc,D8:C0:A6AzureWav,D8:C3:FBDetracom,D8:C4:6AMurataMa,D8:C4:97QuantaCo,D8:C4:E9SamsungE,D8:C5:61CommFron,D8:C6:78MitraSta,D8:C6:91HichanTe,D8:C7:71HuaweiTe,D8:C7:C8ArubaaHe,D8:C8:E9PhicommS,D8:C9:9DEaDispla,D8:CA:06TitanDat,D8:CB:8AMicro-St,D8:CC:98HuaweiDe,D8:CD:2CWuxiNeih,D8:CE:3AXiaomiCo,D8:CF:89BeijingD,D8:CF:9CApple,D8:D0:90Dell,D8:D1:CBApple,D8:D2:7CJemaEner,D8:D3:85HewlettP,D8:D4:3CSony,D8:D4:E6HytecInt,D8:D5:B9Rainfore,D8:D6:7EGskCncEq,D8:D6:F3Integrat,D8:D7:23IDS,D8:D7:75Sagemcom,D8:D8:66Shenzhen,D8:DA:52Apator,D8:DC:40Apple,D8:DC:E9KunshanE,D8:DD:5FBALMUDA,D8:DD:FDTexasIns,D8:DE:3AApple,D8:DE:CEIsung,D8:DF:0DberoNet,D8:DF:7AQuestSof,D8:E0:04VodiaNet,D8:E0:B8Bulat,D8:E0:E1SamsungE,D8:E3:AECirtecMe,D8:E5:6DTCTmobil,D8:E7:2BNetscout,D8:E7:43Wush,D8:E9:52Keopsys,D8:EB:46Google,D8:EB:97TRENDnet,D8:EC:5EBelkinIn,D8:EC:E5ZyxelCom,D8:ED:1CMagnaTec,D8:EE:78MoogProt,D8:EF:42HuaweiDe,D8:EF:CDNokiaSol,D8:F0:F2Zeebo,D8:F1:5BEspressi,D8:F1:F0PepximIn,D8:F2:CAIntelCor,D8:F3:BCLiteonTe,D8:F3:DBPostCH,D8:F7:10LibreWir,D8:F8:83IntelCor,D8:F8:AFDaontec,D8:FB:11Axacore,D8:FB:5EAskeyCom,D8:FB:68CloudCor,D8:FC:38GiantecS,D8:FC:93IntelCor,D8:FE:8FIDFone,D8:FE:E3D-LinkIn,DA:0D:38Farpoint,DA:1C:21Sesam,DA:28:ECHGGenuin,DA:2B:DDAT&T,DA:41:62Xperi,DA:A1:19Google,DA:C8:7DSEDevelo,DA:FF:6BTeamPrec,DC:00:77Tp-LinkT,DC:00:B0FreeboxS,DC:02:65Meditech,DC:02:8Ezte,DC:03:98LGInnote,DC:05:2FNational,DC:05:75SiemensE,DC:05:EDNabtesco,DC:07:C1HangZhou,DC:08:0FApple,DC:08:56Alcatel-,DC:09:14Talk-A-P,DC:09:4CHuaweiTe,DC:0B:1AADBBroad,DC:0B:34LGElectr,DC:0C:2DWeifangG,DC:0C:5CApple,DC:0D:30Shenzhen,DC:0E:96PaloAlto,DC:0E:A1CompalIn,DC:15:C8AVMAudio,DC:15:DBGeRuiliI,DC:16:A2Medtroni,DC:16:B2HuaweiTe,DC:17:5AHitachiH,DC:17:92Captivat,DC:1A:01EcolivTe,DC:1A:C5vivoMobi,DC:1B:A1IntelCor,DC:1D:9FUBtech,DC:1D:D4Microste,DC:1E:A3Accensus,DC:20:08ASDElect,DC:21:48IntelCor,DC:21:5CIntelCor,DC:21:B9Sentec,DC:21:E2HuaweiTe,DC:27:27HuaweiDe,DC:28:34HAKKO,DC:29:19AltoBeam,DC:29:3AShenzhen,DC:2A:14Shanghai,DC:2A:A1MedHab,DC:2B:2AApple,DC:2B:61Apple,DC:2B:66InfoBLOC,DC:2B:CAZera,DC:2C:26ItonTech,DC:2C:6ERouterbo,DC:2D:3CHuaweiDe,DC:2D:CBBeijingU,DC:2E:6AHCT,DC:2F:03Stepforw,DC:30:9CHeyrex,DC:31:D1vivoMobi,DC:33:0DQingDaoH,DC:33:3DHuaweiDe,DC:33:50TechSAT,DC:35:F1Positivo,DC:37:14Apple,DC:37:52Ge,DC:37:57Integrat,DC:37:D2HunanHKT,DC:38:E1JuniperN,DC:39:6FAVMAudio,DC:39:79Cisco,DC:3A:5ERoku,DC:3C:2EManufact,DC:3C:84TicomGeo,DC:3C:F6AtomicRu,DC:3E:51SolbergA,DC:3E:F8Nokia,DC:41:5FApple,DC:41:A9IntelCor,DC:41:E5Shenzhen,DC:44:27IEEERegi,DC:44:27:00:00:00/28Suritel,DC:44:27:10:00:00/28Tesla,DC:44:27:20:00:00/28SkywaveT,DC:44:27:30:00:00/28GeneralM,DC:44:27:40:00:00/28NexTechn,DC:44:27:50:00:00/28CenturyA,DC:44:27:60:00:00/28EK-TEAME,DC:44:27:70:00:00/28EcoGuard,DC:44:27:80:00:00/28WhartonE,DC:44:27:90:00:00/28Neusoft,DC:44:27:A0:00:00/28Shanghai,DC:44:27:B0:00:00/28Nautilus,DC:44:27:C0:00:00/28PyrexxTe,DC:44:27:D0:00:00/28Rohde&Sc,DC:44:27:E0:00:00/28VerifEye,DC:44:27:F0:00:00/28Private,DC:44:6DAllwinne,DC:44:B6SamsungE,DC:45:17ARRISGro,DC:48:B2Baraja,DC:49:C9CascoSig,DC:4A:3EHewlettP,DC:4A:9EIEEERegi,DC:4A:9E:00:00:00/28Dongguan,DC:4A:9E:10:00:00/28Advanced,DC:4A:9E:20:00:00/28Annapurn,DC:4A:9E:30:00:00/28LeachInt,DC:4A:9E:40:00:00/28Adial,DC:4A:9E:50:00:00/28NuoveTec,DC:4A:9E:60:00:00/28Tattile,DC:4A:9E:70:00:00/28Astrogat,DC:4A:9E:80:00:00/28Methodex,DC:4A:9E:90:00:00/28AiSight,DC:4A:9E:A0:00:00/28LongSung,DC:4A:9E:B0:00:00/28Maxvisio,DC:4A:9E:C0:00:00/28HefeiDat,DC:4A:9E:D0:00:00/28Happiest,DC:4A:9E:E0:00:00/28SES-imag,DC:4B:DDShenzhen,DC:4B:FEShenzhen,DC:4D:23MRVComun,DC:4E:DEShinyeiT,DC:4E:F4Shenzhen,DC:4F:22Espressi,DC:50:3ANanjingT,DC:52:85Apple,DC:53:60IntelCor,DC:53:7CCompalBr,DC:53:92Apple,DC:54:3DItelMobi,DC:54:D7AmazonTe,DC:55:83Guangdon,DC:56:E6Shenzhen,DC:56:E7Apple,DC:57:26Power-On,DC:58:BCThomas-K,DC:5E:36Paterson,DC:60:A1Teledyne,DC:62:94Guangzho,DC:63:73ObaraKor,DC:64:7CCRSiiMot,DC:64:B8Shenzhen,DC:66:3AApacerTe,DC:66:72SamsungE,DC:67:23baroxKom,DC:68:0CHewlettP,DC:68:EBNintendo,DC:6A:EAInfinixm,DC:6B:12worldcns,DC:6D:CDGuangdon,DC:6F:00Livescri,DC:6F:08BayStora,DC:70:14Private,DC:71:37zte,DC:71:44SamsungE,DC:71:96IntelCor,DC:72:23HuiZhouG,DC:72:9BHuaweiTe,DC:73:85HuaweiDe,DC:74:A8SamsungE,DC:77:4CCisco,DC:78:34Logicom,DC:7B:94Cisco,DC:7F:A42Wire,DC:82:5BJANUSspo,DC:82:F6iPort,DC:84:E9Shenzhen,DC:85:DEAzureWav,DC:86:D8Apple,DC:87:CBBeijingP,DC:89:83SamsungE,DC:8B:28IntelCor,DC:8C:1BvivoMobi,DC:8C:37Cisco,DC:8D:8ANokiaSol,DC:90:20RuruTekP,DC:90:88HuaweiTe,DC:91:66HuaweiDe,DC:91:BFAmazonTe,DC:96:2CNSTAudio,DC:97:3AVeranaNe,DC:98:40Microsof,DC:99:14HuaweiTe,DC:9A:8ENanjingC,DC:9B:1EIntercom,DC:9B:9CApple,DC:9B:D6TCTmobil,DC:9C:52Sapphire,DC:9C:9FShenzhen,DC:9F:A4Nokia,DC:9F:DBUbiquiti,DC:A1:20Nokia,DC:A2:66HonHaiPr,DC:A3:33Shenzhen,DC:A3:A2FengmiBe,DC:A3:ACRBcloudt,DC:A4:CAApple,DC:A5:F4Cisco,DC:A6:32Raspberr,DC:A6:BDBeijingL,DC:A7:82HuaweiTe,DC:A7:D9Compress,DC:A8:CFNewSpinG,DC:A9:04Apple,DC:A9:71IntelCor,DC:A9:89Macandc,DC:AD:9EGreenPri,DC:AE:04CELOXICA,DC:AE:EBRuckusWi,DC:AF:68WeifangG,DC:B0:58BürkertW,DC:B0:82Nokia,DC:B1:31Shenzhen,DC:B3:B4Honeywel,DC:B4:ACFlextron,DC:B4:C4Microsof,DC:B5:4FApple,DC:B7:2EXiaomiCo,DC:B7:FCAlpsElec,DC:B8:08ExtremeN,DC:BB:96FullSolu,DC:BD:7AGuangzho,DC:BE:7AZhejiang,DC:BF:90HuizhouQ,DC:BF:E9Motorola,DC:C0:DBShenzhen,DC:C0:EBAssaAblo,DC:C1:01SOLiDTec,DC:C4:22Systemba,DC:C6:22BuheungS,DC:C6:4BHuaweiTe,DC:C7:93Nokia,DC:C8:F5Shanghai,DC:CB:A8ExploraT,DC:CC:8DIntegrat,DC:CD:2FSeikoEps,DC:CD:74JapanEMS,DC:CE:41FeGlobal,DC:CE:BCShenzhen,DC:CE:C1Cisco,DC:CF:94BeijingR,DC:CF:96SamsungE,DC:D0:F7Bentek,DC:D2:55KinpoEle,DC:D2:FCHuaweiTe,DC:D3:21HUMAX,DC:D3:A2Apple,DC:D4:44HuaweiDe,DC:D5:2ASunnyHea,DC:D7:A0HuaweiDe,DC:D8:7CBeijingJ,DC:D8:7FShenzhen,DC:D9:16HuaweiTe,DC:D9:AENokiaSha,DC:DA:4FGetckTec,DC:DA:80NewH3CTe,DC:DB:70TonfunkS,DC:DC:07TRP,DC:DC:E2SamsungE,DC:DD:24Energica,DC:DE:4FGioneeCo,DC:DE:CAAkyllor,DC:DF:D6zte,DC:E0:26PatrolTa,DC:E0:EBNanjingA,DC:E1:ADShenzhen,DC:E2:ACLumensDi,DC:E3:05NPKRotek,DC:E5:33IEEERegi,DC:E5:33:00:00:00/28FLYHTAer,DC:E5:33:10:00:00/28AmbiLabs,DC:E5:33:20:00:00/28Remko,DC:E5:33:30:00:00/28ShenZhen,DC:E5:33:40:00:00/28shenzhen,DC:E5:33:50:00:00/28Controls,DC:E5:33:60:00:00/28WECANSol,DC:E5:33:70:00:00/28SANEngin,DC:E5:33:80:00:00/28JB-Light,DC:E5:33:90:00:00/28Tiertime,DC:E5:33:A0:00:00/28Amazingl,DC:E5:33:B0:00:00/28TintelHo,DC:E5:33:C0:00:00/28Brck,DC:E5:33:D0:00:00/28SuzhouAT,DC:E5:33:E0:00:00/28GiantPow,DC:E5:5BGoogle,DC:E5:78Experime,DC:E7:1CAUGElekt,DC:E8:38CKTeleco,DC:E9:94CloudNet,DC:EB:53WuhanQia,DC:EB:69Technico,DC:EB:94Cisco,DC:EC:06HeimiNet,DC:ED:83BeijingX,DC:ED:84Haverfor,DC:EE:06HuaweiTe,DC:EF:09Netgear,DC:EF:80HuaweiTe,DC:EF:CAMurataMa,DC:F0:5DLettaTek,DC:F0:90NubiaTec,DC:F1:10Nokia,DC:F4:01Dell,DC:F4:CAApple,DC:F5:05AzureWav,DC:F5:6EWellysis,DC:F7:19Cisco,DC:F7:55Sitronik,DC:F7:56SamsungE,DC:F8:58LorentNe,DC:F8:B9zte,DC:FA:D5STRONGGe,DC:FB:02Buffalo,DC:FB:48IntelCor,DC:FE:07Pegatron,DC:FE:18Tp-LinkT,E0:00:84HuaweiTe,E0:02:A5ABBRobot,E0:03:70ShenZhen,E0:05:C5Tp-LinkT,E0:06:E6HonHaiPr,E0:07:1BHewlettP,E0:09:BFSHENZHEN,E0:0A:F6LiteonTe,E0:0B:28Inovonic,E0:0C:7FNintendo,E0:0C:E5HuaweiTe,E0:0D:B9Cree,E0:0E:DACisco,E0:0E:E1We,E0:0E:E4DWnetTec,E0:10:7FRuckusWi,E0:12:83Shenzhen,E0:13:B5vivoMobi,E0:14:3EModoosis,E0:18:77Fujitsu,E0:18:9FEMMicroe,E0:19:1DHuaweiTe,E0:19:54zte,E0:19:95Nutanix,E0:19:D8BhTechno,E0:1A:EAAlliedTe,E0:1C:41ExtremeN,E0:1C:EEBravoTec,E0:1C:FCD-LinkIn,E0:1D:38BeijingH,E0:1D:3BCambridg,E0:1E:07AniteTel,E0:1F:0AXslentEn,E0:1F:2BNokiaSol,E0:1F:88XiaomiCo,E0:1F:EDNokiaSha,E0:22:02ARRISGro,E0:23:D7SleepNum,E0:23:FFFortinet,E0:24:7FHuaweiTe,E0:24:81HuaweiTe,E0:25:38TitanPet,E0:26:30Intrigue,E0:26:36NortelNe,E0:27:1ATTCNext-,E0:28:61HuaweiTe,E0:28:6DAVMAudio,E0:29:67HMDGloba,E0:2A:82Universa,E0:2A:E6Fiberhom,E0:2B:96Apple,E0:2B:E9IntelCor,E0:2C:B2LenovoMo,E0:2C:F3MRSElect,E0:2E:3FHuaweiDe,E0:2F:6DCisco,E0:30:05Alcatel-,E0:30:F9JuniperN,E0:31:9EValve,E0:31:D0SZTelsta,E0:33:8EApple,E0:34:E4FeitElec,E0:35:60Challeng,E0:36:76HuaweiTe,E0:36:E3StageOne,E0:37:17Technico,E0:37:BFWistronN,E0:38:3Fzte,E0:39:D7Plexxi,E0:3C:5BShenzhen,E0:3E:44Broadcom,E0:3E:4ACavanagh,E0:3E:7Ddata-com,E0:3F:49ASUSTekC,E0:40:07HuaweiDe,E0:41:36MitraSta,E0:43:DBShenzhen,E0:45:6DChinaMob,E0:46:9ANetgear,E0:46:E5GosuncnT,E0:48:AFPremiete,E0:48:D3Mobiwire,E0:49:EDAudeze,E0:4B:45Hi-PElec,E0:4B:A6HuaweiTe,E0:4F:43Universa,E0:4F:BDSichuanT,E0:50:8BZhejiang,E0:51:24NXPSemic,E0:51:63Arcadyan,E0:55:3DCiscoMer,E0:55:97Emergent,E0:56:F4AxesNetw,E0:58:9ELaerdalM,E0:5A:9FIEEERegi,E0:5A:9F:00:00:00/28Annapurn,E0:5A:9F:10:00:00/28AitecSys,E0:5A:9F:20:00:00/28ChengduS,E0:5A:9F:30:00:00/28LinkThin,E0:5A:9F:40:00:00/28HaleProd,E0:5A:9F:50:00:00/28Tryen,E0:5A:9F:60:00:00/28Fibrain,E0:5A:9F:70:00:00/28OMBGuita,E0:5A:9F:80:00:00/28FujianNe,E0:5A:9F:90:00:00/28GemaltoD,E0:5A:9F:A0:00:00/28Contempo,E0:5A:9F:B0:00:00/28Shenzhen,E0:5A:9F:C0:00:00/28ShenZhen,E0:5A:9F:D0:00:00/28Mountz,E0:5A:9F:E0:00:00/28ShenZhen,E0:5B:70Innovid,E0:5D:5CEveron,E0:5D:A6DetlefFi,E0:5F:45Apple,E0:5F:B9Cisco,E0:60:66Sercomm,E0:60:89Cloudlea,E0:61:B2Hangzhou,E0:62:34TexasIns,E0:62:67XiaomiCo,E0:62:90JinanJov,E0:63:DAUbiquiti,E0:63:E5Sony,E0:64:BBDigiView,E0:66:78Apple,E0:67:81Dongguan,E0:67:B3Shenzhen,E0:68:6DRaybased,E0:69:3AInnophas,E0:69:95Pegatron,E0:69:BACisco,E0:6C:4EShenzhen,E0:6C:A6Creotech,E0:6D:17Apple,E0:70:EAHP,E0:72:0AShenzhen,E0:73:5FNucom,E0:75:0AAlpsalpi,E0:75:7DMotorola,E0:75:AABeijingJ,E0:76:D0AMPAKTec,E0:77:26HuaweiDe,E0:78:A3Shanghai,E0:79:5EWuxiXiao,E0:79:C4iRayTech,E0:7C:13zte,E0:7C:62WhistleL,E0:7D:EATexasIns,E0:7E:5FRenesasE,E0:7F:53Techboar,E0:7F:88EVIDENCE,E0:81:77GreenByt,E0:84:F3HighGrad,E0:85:9AShenzhen,E0:87:B1Nata-Inf,E0:88:5DTechnico,E0:89:7EApple,E0:89:9DCisco,E0:8A:7EExponent,E0:8E:3CAztechEl,E0:8F:ECRepotec,E0:91:3CKyeungin,E0:91:53XAViTech,E0:91:F5Netgear,E0:92:5CApple,E0:92:A7FeitianT,E0:94:67IntelCor,E0:95:79ORTHOsof,E0:97:96HuaweiTe,E0:97:F2Atomax,E0:98:06Espressi,E0:98:61Motorola,E0:99:71SamsungE,E0:9D:13SamsungE,E0:9D:31IntelCor,E0:9D:B8PlanexCo,E0:9D:FAWananHon,E0:9F:2AItonTech,E0:A1:98NOJAPowe,E0:A1:D7Sfr,E0:A2:58WanbangD,E0:A3:0FPevco,E0:A3:ACHuaweiTe,E0:A5:09BitmainT,E0:A6:70Nokia,E0:A7:00Verkada,E0:A8:B8LeShiZhi,E0:AA:96SamsungE,E0:AA:B0Suntaili,E0:AA:DBNanjingP,E0:AB:FEOrbNetwo,E0:AC:CBApple,E0:AC:F1Cisco,E0:AE:5EAlpsalpi,E0:AE:B2Bender&a,E0:AE:EDLoenk,E0:AF:4BPluribus,E0:AF:4FDeutsche,E0:B2:60TenoNetw,E0:B2:F1Fn-LinkT,E0:B5:2DApple,E0:B5:5FApple,E0:B6:55BeijingX,E0:B6:F5IEEERegi,E0:B6:F5:00:00:00/28BeSTAR,E0:B6:F5:10:00:00/28StartTod,E0:B6:F5:20:00:00/28Shanghai,E0:B6:F5:30:00:00/28HuizhouG,E0:B6:F5:40:00:00/28Agora,E0:B6:F5:50:00:00/28Shenzhen,E0:B6:F5:60:00:00/28POMCube,E0:B6:F5:70:00:00/28Shenzhen,E0:B6:F5:80:00:00/28YuneecIn,E0:B6:F5:90:00:00/28Motivepr,E0:B6:F5:A0:00:00/28Folksam,E0:B6:F5:B0:00:00/28MoogCros,E0:B6:F5:C0:00:00/28funktel,E0:B6:F5:D0:00:00/28ItelMobi,E0:B6:F5:E0:00:00/28AdvatekL,E0:B7:0AARRISGro,E0:B7:2EShenZhen,E0:B7:B1ARRISGro,E0:B9:4DShenzhen,E0:B9:A5AzureWav,E0:B9:BAApple,E0:B9:E5Technico,E0:BA:B4Arrcus,E0:BB:9ESeikoEps,E0:BC:43C2Micros,E0:BE:03Lite-OnN,E0:C0:D1CKTeleco,E0:C2:86AisaiCom,E0:C2:B7Masimo,E0:C3:77SamsungE,E0:C3:F3zte,E0:C5:8FChinaMob,E0:C6:3CSichuanT,E0:C6:B3MilDef,E0:C7:67Apple,E0:C7:9DTexasIns,E0:C8:6ASHENZHEN,E0:C9:22JirehEne,E0:C9:7AApple,E0:CA:4DShenzhen,E0:CA:94AskeyCom,E0:CB:1DPrivate,E0:CB:4EASUSTekC,E0:CB:56Shenzhen,E0:CB:BCCiscoMer,E0:CB:EESamsungE,E0:CC:7AHuaweiTe,E0:CC:F8XiaomiCo,E0:CD:FDBeijingE,E0:CE:C3AskeyCom,E0:CF:2DGemintek,E0:D0:83SamsungE,E0:D1:0AKatouden,E0:D1:73Cisco,E0:D1:E6Aliphdba,E0:D3:1AEQUESTec,E0:D4:62HuaweiDe,E0:D4:64IntelCor,E0:D4:E8IntelCor,E0:D5:5EGiga-Byt,E0:D7:BATexasIns,E0:D8:48Dell,E0:D9:A2Hippihap,E0:D9:E3EltexEnt,E0:DA:90HuaweiTe,E0:DA:DCJVCKENWO,E0:DB:10SamsungE,E0:DB:55Dell,E0:DB:88OpenStan,E0:DB:D1Technico,E0:DC:A0SiemensI,E0:DC:FFXiaomiCo,E0:DD:C0vivoMobi,E0:E0:C2ChinaMob,E0:E0:FCHuaweiDe,E0:E1:A9Shenzhen,E0:E2:E6Espressi,E0:E3:7CHuaweiDe,E0:E5:CFTexasIns,E0:E6:2ETCTmobil,E0:E6:31SnbTechn,E0:E6:56Nethesis,E0:E7:51Nintendo,E0:E7:BBNureva,E0:E8:BBUnicomVs,E0:E8:E6Shenzhen,E0:E8:E8OliveTel,E0:EB:40Apple,E0:EB:62Shanghai,E0:ED:1Avastrive,E0:ED:C7Shenzhen,E0:EE:1BPanasoni,E0:EF:25LintesTe,E0:F2:11Digitalw,E0:F3:79Vaddio,E0:F4:42HuaweiDe,E0:F5:C6Apple,E0:F5:CAChengUei,E0:F6:B5Nintendo,E0:F8:47Apple,E0:F9:BECloudena,E0:FA:ECPlatansp,E0:FF:F7Softiron,E2:0C:0FKingston,E4:02:9BIntelCor,E4:04:39TomTomSo,E4:05:F8Bytedanc,E4:07:2BHuaweiDe,E4:08:E7QuectelW,E4:0C:FDGuangdon,E4:0E:EEHuaweiTe,E4:11:5BHewlettP,E4:12:18ShenZhen,E4:12:1DSamsungE,E4:12:89topsyste,E4:15:F6TexasIns,E4:17:D88BitdoTe,E4:18:6BZyxelCom,E4:19:C1HuaweiTe,E4:1A:2CZPE,E4:1C:4BV2Techno,E4:1D:2DMellanox,E4:1E:0AIEEERegi,E4:1E:0A:00:00:00/28Zavod№42,E4:1E:0A:10:00:00/28Connecte,E4:1E:0A:20:00:00/28IDvacoPr,E4:1E:0A:30:00:00/28AvastSof,E4:1E:0A:40:00:00/28XPRGroup,E4:1E:0A:50:00:00/28Aeroel,E4:1E:0A:60:00:00/28SFCEnerg,E4:1E:0A:70:00:00/28Tritium,E4:1E:0A:80:00:00/28SAGEGlas,E4:1E:0A:90:00:00/28BMeters,E4:1E:0A:A0:00:00/28FireAnge,E4:1E:0A:B0:00:00/28SafetyVi,E4:1E:0A:C0:00:00/28Teletask,E4:1E:0A:D0:00:00/28ROMOWind,E4:1E:0A:E0:00:00/28Shanghai,E4:1F:13IBM,E4:1F:7BCisco,E4:1F:E9Dunkermo,E4:22:A5Plantron,E4:23:54Shenzhen,E4:24:6CZhejiang,E4:25:E7Apple,E4:25:E9Color-Ch,E4:26:86DWnetTec,E4:26:8BHuaweiDe,E4:27:61HonorDev,E4:27:71Smartlab,E4:28:05PivotalO,E4:28:A4PramaInd,E4:29:3DShenzhen,E4:2A:ACMicrosof,E4:2A:D3MagnetiM,E4:2B:34Apple,E4:2C:56Lilee,E4:2D:02TCTmobil,E4:2D:7BChinaMob,E4:2F:26Fiberhom,E4:2F:56OptoMET,E4:2F:F6Unicorec,E4:30:22HanwhaTe,E4:32:CBSamsungE,E4:33:AEGuangdon,E4:34:93HuaweiTe,E4:35:93Hangzhou,E4:35:C8HuaweiTe,E4:35:FBSabreTec,E4:37:D7HenriDep,E4:38:8CDigitalP,E4:38:F2Advantag,E4:3A:65MofiNetw,E4:3A:6EShenzhen,E4:3B:C9HisenseV,E4:3C:80Universi,E4:3D:1ABroadcom,E4:3E:C6HuaweiTe,E4:3E:D7Arcadyan,E4:3F:A2WuxiDSPT,E4:40:E2SamsungE,E4:41:22OnePlusT,E4:41:64Nokia,E4:41:E6OttecTec,E4:42:A6IntelCor,E4:43:4BDell,E4:44:E5ExtremeN,E4:46:B0FujitsuC,E4:46:BDC&CTechn,E4:46:DAXiaomiCo,E4:47:90Guangdon,E4:47:91IrisID,E4:47:B3zte,E4:48:C7CiscoSPV,E4:4C:6CShenzhen,E4:4C:C7IEEERegi,E4:4C:C7:00:00:00/28AlertAla,E4:4C:C7:10:00:00/28ACS-Solu,E4:4C:C7:20:00:00/28DoowonEl,E4:4C:C7:30:00:00/28SvyazIng,E4:4C:C7:40:00:00/28BeijingZ,E4:4C:C7:50:00:00/28CeLabs,E4:4C:C7:60:00:00/28Hangzhou,E4:4C:C7:70:00:00/28ChannelE,E4:4C:C7:80:00:00/28IagGroup,E4:4C:C7:90:00:00/28Ottomate,E4:4C:C7:A0:00:00/28Muzik,E4:4C:C7:B0:00:00/28SmallHD,E4:4C:C7:C0:00:00/28EPSBio,E4:4C:C7:D0:00:00/28TeloLimi,E4:4C:C7:E0:00:00/28FLKinfor,E4:4E:18Gardasof,E4:4E:2DCisco,E4:4E:76Champion,E4:4F:29MALighti,E4:4F:5FEDSElekt,E4:50:9AHWCommun,E4:50:EBApple,E4:54:E8Dell,E4:55:A8CiscoMer,E4:55:EADedicate,E4:56:14SuttleAp,E4:57:40ARRISGro,E4:57:A8StuartMa,E4:58:B8SamsungE,E4:58:E7SamsungE,E4:5A:A2vivoMobi,E4:5A:D4EltexEnt,E4:5D:37JuniperN,E4:5D:51Sfr,E4:5D:52Avaya,E4:5D:75SamsungE,E4:5E:1BGoogle,E4:5E:37IntelCor,E4:5F:01Raspberr,E4:60:59Pingtek,E4:62:51HaoCheng,E4:64:49ARRISGro,E4:67:1ESHENZHEN,E4:67:BADanishIn,E4:68:A3HuaweiTe,E4:69:5ADictumHe,E4:6C:21messMa,E4:6F:13D-LinkIn,E4:70:B8IntelCor,E4:71:85Securifi,E4:72:E2HuaweiTe,E4:75:1EGetingeS,E4:75:DCArcadyan,E4:76:84Apple,E4:77:23zte,E4:77:27HuaweiTe,E4:77:6BAartesys,E4:77:D4MinrrayI,E4:7B:3FBeijingC,E4:7C:65SunstarC,E4:7C:F9SamsungE,E4:7D:5ABeijingH,E4:7D:BDSamsungE,E4:7D:EBShanghai,E4:7E:66HuaweiTe,E4:7E:9Azte,E4:7F:B2Fujitsu,E4:81:84Nokia,E4:81:B3Shenzhen,E4:82:10HuaweiTe,E4:82:CCJumptron,E4:83:26HuaweiTe,E4:83:99ARRISGro,E4:84:2BHangzhou,E4:85:01GeberitI,E4:8A:D5RfWindow,E4:8B:7FApple,E4:8C:0FDiscover,E4:8D:8CRouterbo,E4:8F:1DHuaweiDe,E4:8F:34Vodafone,E4:8F:65YelatmaI,E4:90:69Rockwell,E4:90:7EMotorola,E4:90:FDApple,E4:92:2ADbgHoldi,E4:92:E7Gridlink,E4:92:FBSamsungE,E4:93:6AGuangdon,E4:95:6EIEEERegi,E4:95:6E:00:00:00/28SMCNetwo,E4:95:6E:10:00:00/28Tband,E4:95:6E:20:00:00/28Shanghai,E4:95:6E:30:00:00/28Shanghai,E4:95:6E:40:00:00/28GuangLia,E4:95:6E:50:00:00/28Elan,E4:95:6E:60:00:00/28Shenzhen,E4:95:6E:70:00:00/28National,E4:95:6E:80:00:00/28PTMLWTel,E4:95:6E:90:00:00/28eZeLink,E4:95:6E:A0:00:00/28RedPoint,E4:95:6E:B0:00:00/28iConserv,E4:95:6E:C0:00:00/28Shenzhen,E4:95:6E:D0:00:00/28Shanghai,E4:95:6E:E0:00:00/28TacomPro,E4:95:6E:F0:00:00/28Private,E4:96:AEALTOGRAP,E4:97:F0Shanghai,E4:98:BBPhyplusM,E4:98:D1Microsof,E4:98:D6Apple,E4:9A:79Apple,E4:9A:DCApple,E4:9E:12FreeboxS,E4:9F:1EARRISGro,E4:A1:E6Alcatel-,E4:A3:2FShanghai,E4:A3:87ControlS,E4:A4:71IntelCor,E4:A5:EFTronLink,E4:A7:49PaloAlto,E4:A7:A0IntelCor,E4:A7:C5HuaweiTe,E4:A7:FDCellcoPa,E4:A8:B6HuaweiTe,E4:A8:DFCompalIn,E4:AA:5DCisco,E4:AA:EALiteonTe,E4:AA:ECTianjinH,E4:AB:46UABSelte,E4:AB:89MitraSta,E4:AD:7DSCLEleme,E4:AF:A1Hes-So,E4:B0:05BeijingI,E4:B0:21SamsungE,E4:B2:FBApple,E4:B3:18IntelCor,E4:B5:03RealmeCh,E4:B9:7ADell,E4:BA:D9360Fly,E4:BD:4Bzte,E4:BE:EDNetcoreT,E4:BF:FATechnico,E4:C0:CCChinaMob,E4:C0:E2Sagemcom,E4:C1:46Objetivo,E4:C1:F1SHENZHEN,E4:C2:D1HuaweiTe,E4:C3:2ATp-LinkT,E4:C4:83Guangdon,E4:C6:2BAirware,E4:C6:3DApple,E4:C6:E6Mophie,E4:C7:22Cisco,E4:C8:01BLUProdu,E4:C8:06CeiecEle,E4:C9:0BRadwin,E4:CA:12zte,E4:CB:59BeijingL,E4:CC:9DIntegrat,E4:CE:02WyreStor,E4:CE:70HealthLi,E4:CE:8FApple,E4:D1:24MojoNetw,E4:D3:32Tp-LinkT,E4:D3:73HuaweiTe,E4:D3:AAFujitsuC,E4:D3:F1Cisco,E4:D5:3DHonHaiPr,E4:D7:1DOrayaThe,E4:DB:6DBeijingX,E4:DC:43HuaweiDe,E4:DC:5FCofracta,E4:DD:79En-Visio,E4:E0:A6Apple,E4:E0:C5SamsungE,E4:E1:12TexasIns,E4:E1:30TCTmobil,E4:E4:09Leifheit,E4:E4:ABApple,E4:E7:49HewlettP,E4:EA:83Shenzhen,E4:EC:10Nokia,E4:EE:FDMR&DManu,E4:F0:04Dell,E4:F0:42Google,E4:F1:4CPrivate,E4:F1:D4vivoMobi,E4:F3:27Atol,E4:F3:65Time-O-M,E4:F3:C4SamsungE,E4:F3:E3Shanghai,E4:F3:E8Shenzhen,E4:F3:F5Shenzhen,E4:F4:C6Netgear,E4:F7:5BARRISGro,E4:F7:A1Datafox,E4:F8:9CIntelCor,E4:F8:EFSamsungE,E4:F9:39MinxonHo,E4:FA:1DPADPerip,E4:FA:EDSamsungE,E4:FA:FDIntelCor,E4:FB:5DHuaweiTe,E4:FB:8FMobiwire,E4:FC:82JuniperN,E4:FD:45IntelCor,E4:FD:A1HuaweiTe,E4:FE:D9EDMIEuro,E4:FF:DDElectron,E8:00:36Befs,E8:01:15COOCAANe,E8:01:8DFiberhom,E8:03:9ASamsungE,E8:04:0BApple,E8:04:10Private,E8:04:62Cisco,E8:04:F3Throught,E8:05:6DNortelNe,E8:05:DCVerifone,E8:06:88Apple,E8:07:34Champion,E8:07:BFShenzhen,E8:08:8BHuaweiTe,E8:09:45Integrat,E8:09:59Guoguang,E8:0A:ECJiangsuH,E8:0B:13AkibTaiw,E8:0C:38Daeyoung,E8:0C:75Syncbak,E8:0F:C8Universa,E8:10:2EReallySi,E8:11:32SamsungE,E8:11:CAShandong,E8:13:24GuangZho,E8:13:63Comstock,E8:13:67AIRSOUND,E8:13:6EHuaweiTe,E8:15:0ENokia,E8:16:2BIDEOSecu,E8:17:FCFujitsuC,E8:18:63IEEERegi,E8:18:63:00:00:00/28DigiMagu,E8:18:63:10:00:00/28clabsys,E8:18:63:20:00:00/28AVCONInf,E8:18:63:30:00:00/28DongGuan,E8:18:63:40:00:00/28Guangzho,E8:18:63:50:00:00/28WetekEle,E8:18:63:60:00:00/28ArtechSo,E8:18:63:70:00:00/28Siliconc,E8:18:63:90:00:00/28BSMWirel,E8:18:63:A0:00:00/28JDMMobil,E8:18:63:B0:00:00/28ProtekEl,E8:18:63:C0:00:00/28Shenzhen,E8:18:63:D0:00:00/28DigitalD,E8:18:63:E0:00:00/28AcopianT,E8:18:63:F0:00:00/28Private,E8:1A:58Technolo,E8:1A:ACORFEOSOU,E8:1B:4Bamnimo,E8:1B:69Sercomm,E8:1C:BAFortinet,E8:1C:D8Apple,E8:1D:A8RuckusWi,E8:1E:92HuaweiDe,E8:20:E2HUMAX,E8:26:89ArubaaHe,E8:26:B6InsideBi,E8:28:77TMY,E8:28:C1EltexEnt,E8:28:D5CotsTech,E8:2A:44LiteonTe,E8:2A:EAIntelCor,E8:2C:6DSmartRG,E8:2E:0CNETINTTe,E8:2E:24OutFogRe,E8:33:0DXaptec,E8:33:81ARRISGro,E8:34:3EBeijingI,E8:36:17Apple,E8:36:1DSenseLab,E8:37:7AZyxelCom,E8:39:35HewlettP,E8:39:DFAskeyCom,E8:3A:12SamsungE,E8:3A:97Toshiba,E8:3E:B6Rim,E8:3E:FBGeodesic,E8:3E:FCARRISGro,E8:3F:67HuaweiDe,E8:40:40Cisco,E8:40:F2Pegatron,E8:43:B6QNAP,E8:44:7EBitdefen,E8:47:27QuectelW,E8:48:1FAdvanced,E8:48:B8TP-Link,E8:49:43YUGEInfo,E8:4C:56Intercep,E8:4D:74HuaweiTe,E8:4D:D0HuaweiTe,E8:4E:06EdupInte,E8:4E:84SamsungE,E8:4E:CENintendo,E8:4F:25MurataMa,E8:4F:4BShenzhen,E8:4F:A7HuaweiDe,E8:50:8BSamsungE,E8:51:6ETSMART,E8:51:9DYeonhabP,E8:54:84NEOInfor,E8:55:B4SAITechn,E8:56:59Advanced,E8:56:D6NCTech,E8:5A:8BXiaomiCo,E8:5A:A7Emzior,E8:5A:D1Fiberhom,E8:5B:5BLgElectr,E8:5B:B7Ample,E8:5B:F0ImagingD,E8:5C:0ACisco,E8:5D:6BLuminate,E8:5D:86ChangYow,E8:5E:53Infratec,E8:61:1FDawningI,E8:61:7ELiteonTe,E8:61:83BlackDia,E8:61:BEMelec,E8:65:49Cisco,E8:65:D4TendaTec,E8:66:C4Diamanti,E8:68:19HuaweiTe,E8:68:E7Espressi,E8:6A:64LCFCHeFe,E8:6C:C7IEEERegi,E8:6C:C7:00:00:00/28TrapezeS,E8:6C:C7:10:00:00/28ASSAABLO,E8:6C:C7:20:00:00/28XirgoTec,E8:6C:C7:30:00:00/28Shenzhen,E8:6C:C7:40:00:00/28KoalSoft,E8:6C:C7:50:00:00/28Shenzhen,E8:6C:C7:60:00:00/28Klab,E8:6C:C7:70:00:00/28HuaqinTe,E8:6C:C7:80:00:00/28Lighthou,E8:6C:C7:90:00:00/28Hangzhou,E8:6C:C7:A0:00:00/28CoxSpace,E8:6C:C7:B0:00:00/28MORNSUNG,E8:6C:C7:C0:00:00/28Liabilit,E8:6C:C7:D0:00:00/28z-maxmed,E8:6C:C7:E0:00:00/28Annapurn,E8:6C:DASupercom,E8:6D:52ARRISGro,E8:6D:54DigitMob,E8:6D:65AUDIOMOB,E8:6D:6Evoestalp,E8:6D:CBSamsungE,E8:6D:E9HuaweiTe,E8:6E:44zte,E8:6F:38Chongqin,E8:6F:F2Actionte,E8:71:8DElsysEqu,E8:74:C7Sentinhe,E8:74:E6ADBBroad,E8:75:7FFIRSTech,E8:78:29IEEERegi,E8:78:29:00:00:00/28TanzSecu,E8:78:29:10:00:00/28Shenzhen,E8:78:29:20:00:00/28Galcon,E8:78:29:30:00:00/28Electron,E8:78:29:40:00:00/28Annapurn,E8:78:29:50:00:00/28ShenZhen,E8:78:29:60:00:00/28Axing,E8:78:29:70:00:00/28FAIOT,E8:78:29:80:00:00/28JVISMall,E8:78:29:90:00:00/28RyuTech,E8:78:29:A0:00:00/28METZCONN,E8:78:29:B0:00:00/28Private,E8:78:29:C0:00:00/28FairPhon,E8:78:29:D0:00:00/28BerndWal,E8:78:29:E0:00:00/28SolosTec,E8:78:65Apple,E8:78:A1BeoviewI,E8:7A:F3S5Tech,E8:7F:6BSamsungE,E8:7F:95Apple,E8:80:2EApple,E8:80:D8GNTEKEle,E8:81:52Apple,E8:82:5BARRISGro,E8:84:A5IntelCor,E8:84:C6HuaweiTe,E8:85:4BApple,E8:87:A3LoxleyPu,E8:88:6CShenzhen,E8:89:2CARRISGro,E8:8D:28Apple,E8:8D:F5ZNYXNetw,E8:8E:60NSD,E8:91:0FFiberhom,E8:91:20Motorola,E8:92:18Arcontia,E8:92:A4LGElectr,E8:93:09SamsungE,E8:93:63Nokia,E8:94:4CCogentHe,E8:94:F6Tp-LinkT,E8:95:26Luxshare,E8:96:06testoIns,E8:97:9AQuectelW,E8:98:6DPaloAlto,E8:98:C2ZETLAB,E8:99:5APiiGABPr,E8:99:C4HTC,E8:9A:8FQuantaCo,E8:9A:FFFujianLA,E8:9D:87Toshiba,E8:9E:0CMax8UsaD,E8:9E:B4HonHaiPr,E8:9F:39Nokia,E8:9F:6DEspressi,E8:9F:80BelkinIn,E8:9F:ECChengduK,E8:A0:CDNintendo,E8:A1:F8zte,E8:A2:45JuniperN,E8:A3:64SignalPa,E8:A4:C1DeepSeaE,E8:A6:60HuaweiTe,E8:A6:CAHuaweiDe,E8:A7:2FMicrosof,E8:A7:30Apple,E8:A7:88XiamenLe,E8:A7:F2sTraffic,E8:AB:F3HuaweiTe,E8:AB:FAShenzhen,E8:AC:ADzte,E8:AD:A6Sagemcom,E8:B1:FCIntelCor,E8:B2:ACApple,E8:B2:FEHUMAX,E8:B4:70IEEERegi,E8:B4:70:00:00:00/28DongGuan,E8:B4:70:10:00:00/28AutocomD,E8:B4:70:20:00:00/28internet,E8:B4:70:30:00:00/28Webfleet,E8:B4:70:40:00:00/28YawataEl,E8:B4:70:50:00:00/28AlperiaF,E8:B4:70:60:00:00/28Elcoma,E8:B4:70:70:00:00/28TibitCom,E8:B4:70:80:00:00/28DEHNSE+,E8:B4:70:90:00:00/28MiltekIn,E8:B4:70:A0:00:00/28plc2Desi,E8:B4:70:B0:00:00/28Digifocu,E8:B4:70:C0:00:00/28AndurilI,E8:B4:70:D0:00:00/28Medica,E8:B4:70:E0:00:00/28Unicacce,E8:B4:AEShenzhen,E8:B4:C8SamsungE,E8:B5:41zte,E8:B5:D0Dell,E8:B6:C2JuniperN,E8:B7:48Cisco,E8:BA:70Cisco,E8:BB:3DSinoPrim,E8:BB:A8Guangdon,E8:BD:D1HuaweiTe,E8:BE:81Sagemcom,E8:C1:B8NanjingB,E8:C1:D7Philips,E8:C1:E8Shenzhen,E8:C2:29H-Displa,E8:C2:DDInfinixm,E8:C3:20AustcoMa,E8:C4:17Fiberhom,E8:C5:7AUfispace,E8:C7:4FLiteonTe,E8:CA:C8HuiZhouG,E8:CB:A1Nokia,E8:CB:EDChipseaT,E8:CC:18D-LinkIn,E8:CC:32Micronet,E8:CD:2DHuaweiTe,E8:CE:06SkyHawke,E8:D0:3CShenzhen,E8:D0:99Fiberhom,E8:D0:B9TaicangT,E8:D0:FAMKSInstr,E8:D0:FCLiteonTe,E8:D1:1BAskeyCom,E8:D2:FFSagemcom,E8:D4:83ULTIMATE,E8:D4:E0BeijingB,E8:D7:65HuaweiTe,E8:D8:19AzureWav,E8:D8:D1HP,E8:DA:00KivoTech,E8:DA:20Nintendo,E8:DA:96ZhuhaiTi,E8:DA:AAVideoHom,E8:DB:84Espressi,E8:DE:00ChongQin,E8:DE:27Tp-LinkT,E8:DE:8EIntegrat,E8:DE:D6Intrisin,E8:DE:FBMesoticS,E8:DF:70AVMAudio,E8:DF:F2PRF,E8:E0:8FGravotec,E8:E0:B7Toshiba,E8:E1:E1GemtekTe,E8:E1:E2Energote,E8:E5:D6SamsungE,E8:E7:32Alcatel-,E8:E7:70Warp9Tec,E8:E7:76Shenzhen,E8:E8:75iS5Commu,E8:E8:B7MurataMa,E8:E9:8ESOLARcon,E8:EA:4DHuaweiTe,E8:EA:6AStarTech,E8:EA:DADenkoviA,E8:EB:11TexasIns,E8:EB:1BMicrochi,E8:EB:34Cisco,E8:EC:A3Dongguan,E8:ED:05ARRISGro,E8:ED:D6Fortinet,E8:ED:F3Cisco,E8:EF:89OPMEXTec,E8:F1:B0Sagemcom,E8:F2:26MillsonC,E8:F2:E2LGInnote,E8:F2:E3StarcorB,E8:F4:08IntelCor,E8:F6:54HuaweiTe,E8:F7:24HewlettP,E8:F9:28Rftech,E8:F9:D4HuaweiTe,E8:FA:F7Guangdon,E8:FB:E9Apple,E8:FC:60ELCOMInn,E8:FC:AFNetgear,E8:FD:35HuaweiDe,E8:FD:72Shanghai,E8:FD:90Turbosto,E8:FD:E8CeLaLink,EA:11:5DAutovisi,EA:2A:DBLabsNetw,EA:34:B4ThinkRF,EA:60:76CloudSim,EA:9F:B1PhilipsI,EA:B8:9BThreadGr,EA:BC:04G3Allian,EA:BE:A7Sonos,EA:DD:88IEEEPES-,EA:E0:D9Berk-tek,EC:01:33Trinus,EC:01:D5Cisco,EC:01:E2FoxconnI,EC:01:EEGuangdon,EC:02:73ArubaaHe,EC:04:41ShenZhen,EC:08:6BTp-LinkT,EC:08:E5Motorola,EC:0B:AEHangzhou,EC:0D:9AMellanox,EC:0D:E4AmazonTe,EC:0E:C4HonHaiPr,EC:0E:D6ItechIns,EC:10:00EnanceSo,EC:10:7BSamsungE,EC:11:20FloDesig,EC:11:27TexasIns,EC:13:B2Netonix,EC:13:DBJuniperN,EC:14:F6BioContr,EC:15:3DBeijingY,EC:17:2FTp-LinkT,EC:17:66Research,EC:1A:59BelkinIn,EC:1B:BDSiliconL,EC:1C:5DSiemens,EC:1D:7Fzte,EC:1D:8BCisco,EC:1F:72SamsungE,EC:21:9FVidaBox,EC:21:E5Toshiba,EC:22:57JiangSuN,EC:22:80D-LinkIn,EC:23:3DHuaweiTe,EC:23:68IntelliV,EC:23:7Bzte,EC:24:B8TexasIns,EC:26:51Apple,EC:26:CATp-LinkT,EC:26:FBTecc,EC:2A:72Dell,EC:2A:F0Ypsomed,EC:2B:EBAmazonTe,EC:2C:11CwdInnov,EC:2C:49Universi,EC:2C:E2Apple,EC:2E:4EHitachi-,EC:2E:98AzureWav,EC:30:91Cisco,EC:31:6DHansgroh,EC:35:4DWingtech,EC:35:86Apple,EC:36:3FMarkov,EC:38:73JuniperN,EC:38:8FHuaweiTe,EC:3B:F0NovelSat,EC:3C:5AShenZhen,EC:3C:88MCNEX,EC:3C:BBHuaweiDe,EC:3D:FDShenzhen,EC:3E:09Performa,EC:3E:B3ZyxelCom,EC:3E:F7JuniperN,EC:3F:05Institut,EC:41:18XIAOMIEl,EC:42:69HMDGloba,EC:42:B4ADC,EC:42:F0ADLEmbed,EC:43:8BYaptv,EC:43:E6AWCER,EC:43:F6ZyxelCom,EC:44:76Cisco,EC:46:44TtkSas,EC:46:70Meinberg,EC:47:3CRedwire,EC:49:93QihanTec,EC:4C:4DNPKRoTeK,EC:4D:3EBeijingX,EC:4D:47HuaweiTe,EC:4F:82Calix,EC:51:BCGuangdon,EC:52:DCWORLDMED,EC:54:2EShanghai,EC:55:F9HonHaiPr,EC:56:23HuaweiTe,EC:57:0DAFE,EC:58:EARuckusWi,EC:59:E7Microsof,EC:5A:86YulongCo,EC:5B:73Advanced,EC:5C:68Chongqin,EC:5C:69Mitsubis,EC:5F:23QinghaiK,EC:60:E0Avi-OnLa,EC:62:64Global41,EC:63:D7IntelCor,EC:63:E5ePBoardD,EC:63:EDHyundaiA,EC:64:88HonorDev,EC:64:E7MOCACARE,EC:65:CCPanasoni,EC:66:D1B&WGroup,EC:68:81PaloAlto,EC:6C:9AArcadyan,EC:6C:9FChengduV,EC:6C:B5zte,EC:6F:0BFADU,EC:70:97ARRISGro,EC:71:DBShenzhen,EC:74:BAHirschma,EC:75:3EHuaweiTe,EC:75:EDCitrix,EC:79:49Fujitsu,EC:79:F2Startel,EC:7C:2CHuaweiTe,EC:7C:74JustoneT,EC:7C:B6SamsungE,EC:7D:11vivoMobi,EC:7D:9DCpi,EC:7E:91ItelMobi,EC:7F:C6EccelSas,EC:80:09NovaSpar,EC:81:93Logitech,EC:82:63zte,EC:83:50Microsof,EC:83:6CRMTech,EC:83:D5GIRD,EC:84:B4CigShang,EC:85:2FApple,EC:88:8FTp-LinkT,EC:88:92Motorola,EC:89:14HuaweiTe,EC:89:F5LenovoMo,EC:8A:4Czte,EC:8A:C4AmazonTe,EC:8A:C7Fiberhom,EC:8C:9AHuaweiTe,EC:8C:A2RuckusWi,EC:8E:ADDlx,EC:8E:AENagravis,EC:8E:B5HewlettP,EC:92:33EddyfiND,EC:93:27MEMMERT+,EC:93:65Mapperai,EC:93:7DTechnico,EC:93:EDDDoS-Gua,EC:94:68MetaSyst,EC:94:CBEspressi,EC:94:D5JuniperN,EC:96:812276427O,EC:97:B2SUMECMac,EC:97:E0Hangzhou,EC:98:6CLufftMes,EC:98:C1BeijingR,EC:9A:74HewlettP,EC:9B:5BNokia,EC:9B:8BHewlettP,EC:9B:F3SamsungE,EC:9C:32SichuanA,EC:9E:CDArtesynE,EC:9F:0DIEEERegi,EC:9F:0D:00:00:00/28HesaiPho,EC:9F:0D:10:00:00/28SimulaTe,EC:9F:0D:20:00:00/28DRB,EC:9F:0D:30:00:00/28WaverlyL,EC:9F:0D:40:00:00/28WisIOE,EC:9F:0D:50:00:00/28Paw-Taw-,EC:9F:0D:60:00:00/28Shenzhen,EC:9F:0D:70:00:00/28BeijingL,EC:9F:0D:80:00:00/28Zhejiang,EC:9F:0D:90:00:00/28Fci,EC:9F:0D:A0:00:00/28flexlog,EC:9F:0D:B0:00:00/28CrrcQing,EC:9F:0D:C0:00:00/28Sarcos,EC:9F:0D:D0:00:00/28SKSContr,EC:9F:0D:E0:00:00/28MAXTechn,EC:A1:D1HuaweiTe,EC:A2:9BKemppi,EC:A5:DEONYXWIFI,EC:A8:1FTechnico,EC:A8:6BElitegro,EC:A9:40ARRISGro,EC:A9:FAGuangdon,EC:AA:25SamsungE,EC:AA:A0Pegatron,EC:AD:B8Apple,EC:AD:E0D-LinkIn,EC:AF:97Git,EC:B0:E1Ciena,EC:B1:06AcuroNet,EC:B1:D7HewlettP,EC:B3:13Shenzhen,EC:B4:E8WistronM,EC:B5:41SHINANOE,EC:B5:FAPhilipsL,EC:B8:70BeijingH,EC:B9:07CloudGen,EC:B9:70RuijieNe,EC:BA:FEGiroptic,EC:BB:AEDigivoic,EC:BD:09FUSIONEl,EC:BD:1DCisco,EC:BE:5FVestelEl,EC:BE:DDSagemcom,EC:C0:1BHuaweiTe,EC:C0:6APowerCho,EC:C1:ABGuangzho,EC:C3:02HUMAX,EC:C3:8AAccuener,EC:C4:0DNintendo,EC:C5:7FSuzhouPa,EC:C5:D2HuaweiDe,EC:C8:82Cisco,EC:C8:9CHangzhou,EC:CB:30HuaweiTe,EC:CD:6DAlliedTe,EC:CE:13Cisco,EC:CE:D7Apple,EC:D0:0EMiraeRec,EC:D0:40GEAFarmT,EC:D0:9FXiaomiCo,EC:D1:9AZhuhaiLi,EC:D6:8AShenzhen,EC:D9:25Rami,EC:D9:50Irt,EC:D9:D1Shenzhen,EC:DB:86Api-K,EC:DE:3DLampreyN,EC:DF:3AvivoMobi,EC:E0:9BSamsungE,EC:E1:54BeijingU,EC:E1:A9Cisco,EC:E2:FDSKGElect,EC:E5:12tado,EC:E5:55Hirschma,EC:E7:44Omntecmf,EC:E9:0BSistemaS,EC:E9:15STI,EC:E9:F8GuangZho,EC:EA:03DarfonLi,EC:EB:B8HewlettP,EC:EE:D8ZTLXNetw,EC:F0:0EAboCom,EC:F0:FEzte,EC:F2:2BTecnoMob,EC:F2:36Neomonta,EC:F3:42Guangdon,EC:F3:5BNokia,EC:F4:0CCisco,EC:F4:51Arcadyan,EC:F4:BBDell,EC:F6:BDSncfMobi,EC:F7:2BHdDigita,EC:F8:EBSichuanT,EC:FA:03Fca,EC:FA:5CBeijingX,EC:FA:AAIMS,EC:FA:BCEspressi,EC:FA:F4SenRaTec,EC:FC:55AEberle,EC:FE:7EBlueRadi,F0:00:7FJanz-Con,F0:01:6ETianyiTe,F0:02:2BChrontel,F0:02:48SmarteBu,F0:03:8CAzureWav,F0:07:86Shandong,F0:08:D1Espressi,F0:08:F1SamsungE,F0:0D:5CJinQianM,F0:0D:F5ACOMAMed,F0:0E:1DMegafone,F0:0E:BFZettaHas,F0:0F:ECHuaweiTe,F0:10:90NewH3CTe,F0:10:ABChinaMob,F0:13:C1HanntoTe,F0:13:C3Shenzhen,F0:15:A0KyungDon,F0:15:B9PlayFusi,F0:16:28Technico,F0:18:2BLGChem,F0:18:98Apple,F0:1B:6CvivoMobi,F0:1C:13LGElectr,F0:1C:2DJuniperN,F0:1D:2DCisco,F0:1D:BCMicrosof,F0:1E:34ORICOTec,F0:1F:AFDell,F0:21:9DCal-Comp,F0:21:E0eero,F0:22:4EEsanelec,F0:23:29ShowaDen,F0:23:AEAMPAKTec,F0:23:B9IEEERegi,F0:23:B9:00:00:00/28Aquametr,F0:23:B9:10:00:00/28Ubiant,F0:23:B9:20:00:00/28RaysgemE,F0:23:B9:30:00:00/28BSPRUS,F0:23:B9:40:00:00/28Ezvis,F0:23:B9:50:00:00/28Audeara,F0:23:B9:60:00:00/28XiamenJi,F0:23:B9:70:00:00/28Transcen,F0:23:B9:80:00:00/28G3Techno,F0:23:B9:90:00:00/28EmuTechn,F0:23:B9:A0:00:00/28Annapurn,F0:23:B9:B0:00:00/28QCoreMed,F0:23:B9:C0:00:00/28Shenzhen,F0:23:B9:D0:00:00/28Shenyang,F0:23:B9:E0:00:00/28Domotz,F0:24:05OPUSHigh,F0:24:08TalarisS,F0:24:75Apple,F0:25:72Cisco,F0:25:8EHuaweiTe,F0:25:B7SamsungE,F0:26:24WafaTech,F0:26:4CSigrist-,F0:27:2DAmazonTe,F0:27:45F-Secure,F0:27:65MurataMa,F0:29:29Cisco,F0:2A:23Creative,F0:2A:2BIEEERegi,F0:2A:2B:00:00:00/28MerlinSe,F0:2A:2B:10:00:00/28TobiTrib,F0:2A:2B:20:00:00/28Shanghai,F0:2A:2B:30:00:00/28Frigotel,F0:2A:2B:40:00:00/28OnclaveN,F0:2A:2B:50:00:00/28AgileSpo,F0:2A:2B:60:00:00/28Shenzhen,F0:2A:2B:70:00:00/28Protroni,F0:2A:2B:80:00:00/28TenwaysE,F0:2A:2B:90:00:00/28ZiGongPe,F0:2A:2B:A0:00:00/28Navigil,F0:2A:2B:B0:00:00/28ELMO,F0:2A:2B:C0:00:00/28Comexio,F0:2A:2B:D0:00:00/28Definite,F0:2A:2B:E0:00:00/28Shenzhen,F0:2A:61WaldoNet,F0:2E:51Casa,F0:2F:4BApple,F0:2F:74ASUSTekC,F0:2F:A7HuaweiTe,F0:2F:D8Bi2-Visi,F0:32:1AMita-Tek,F0:33:E5HuaweiTe,F0:34:04TCTmobil,F0:35:75HuiZhouG,F0:37:A1HuikeEle,F0:39:65SamsungE,F0:3A:4BBloombas,F0:3A:55OmegaEle,F0:3D:03TecnoMob,F0:3D:29Actility,F0:3E:90RuckusWi,F0:3E:BFGogoroTa,F0:3F:95HuaweiTe,F0:3F:F8RLDrake,F0:40:7BFiberhom,F0:41:C6HeatTech,F0:41:C8IEEERegi,F0:41:C8:00:00:00/28LinpaAco,F0:41:C8:10:00:00/28DongGuan,F0:41:C8:20:00:00/28Shenzhen,F0:41:C8:30:00:00/28Shenzhen,F0:41:C8:40:00:00/28Candelic,F0:41:C8:50:00:00/28XIANMEIS,F0:41:C8:60:00:00/28AEDEngin,F0:41:C8:70:00:00/28Nanchang,F0:41:C8:80:00:00/28PostiumK,F0:41:C8:90:00:00/28Shenzhen,F0:41:C8:A0:00:00/28Telstra,F0:41:C8:B0:00:00/28Powervau,F0:41:C8:C0:00:00/28Shanghai,F0:41:C8:D0:00:00/28ATNMedia,F0:41:C8:E0:00:00/28Shenzhen,F0:42:1CIntelCor,F0:42:F5HuaweiDe,F0:43:35DVNShang,F0:43:47HuaweiTe,F0:45:DATexasIns,F0:46:3BComcastC,F0:4A:02Cisco,F0:4A:2BPYRAMIDC,F0:4B:3AJuniperN,F0:4B:6AScientif,F0:4B:F2JTECHCom,F0:4C:D5Maxlinea,F0:4D:A2Dell,F0:4F:7CAmazonTe,F0:51:36TCTmobil,F0:51:EAFitbit,F0:54:94Honeywel,F0:55:01HuaweiDe,F0:57:A6IntelCor,F0:58:49CareView,F0:5A:09SamsungE,F0:5B:7BSamsungE,F0:5C:19ArubaaHe,F0:5C:77Google,F0:5C:D5Apple,F0:5D:89Dycon,F0:5D:C8Duracell,F0:5E:CDTexasIns,F0:5F:5AGetriebe,F0:61:30Advantag,F0:62:0DShenzhen,F0:62:5ARealmeCh,F0:62:81ProCurve,F0:63:F9HuaweiTe,F0:64:26ExtremeN,F0:65:C2YanfengV,F0:65:DDPrimaxEl,F0:67:28Guangdon,F0:68:53Integrat,F0:68:65TaicangT,F0:6B:CASamsungE,F0:6C:73Nokia,F0:6D:78Guangdon,F0:6E:0BMicrosof,F0:6E:32Microtel,F0:6F:46Ubiik,F0:70:4FSamsungE,F0:72:8CSamsungE,F0:72:EAGoogle,F0:73:AEPEAK-Sys,F0:74:85NGD,F0:74:E4Thunderc,F0:76:1CCompalIn,F0:76:6FApple,F0:77:65Sourcefi,F0:77:C3IntelCor,F0:77:D0Xcellen,F0:78:07Apple,F0:78:16Cisco,F0:79:59ASUSTekC,F0:79:60Apple,F0:79:E8Guangdon,F0:7B:CBHonHaiPr,F0:7C:C7JuniperN,F0:7D:68D-Link,F0:7F:06Cisco,F0:7F:0CLeopoldK,F0:81:73AmazonTe,F0:81:75Sagemcom,F0:81:AFIrzAutom,F0:82:61Sagemcom,F0:84:2FADBBroad,F0:84:C9zte,F0:85:C1Shenzhen,F0:86:20Arcadyan,F0:8A:28JIANGSUH,F0:8A:76SamsungE,F0:8B:FECostel,F0:8C:FBFiberhom,F0:8E:DBVeloClou,F0:92:1CHewlettP,F0:92:B4SichuanT,F0:93:3ANxtConec,F0:93:C5GarlandT,F0:95:F1CarlZeis,F0:97:E5Tamio,F0:98:38HuaweiTe,F0:98:9DApple,F0:99:19GarminIn,F0:99:B6Apple,F0:99:BFApple,F0:9A:51Shanghai,F0:9B:B8HuaweiTe,F0:9C:BBRaonThin,F0:9C:D7Guangzho,F0:9C:E9ExtremeN,F0:9E:4AIntelCor,F0:9E:63Cisco,F0:9F:C2Ubiquiti,F0:9F:FCSHARP,F0:A2:25AmazonTe,F0:A3:5AApple,F0:A3:B2HuiZhouG,F0:A7:64GST,F0:A7:B2Futaba,F0:A9:68Antailiy,F0:AA:0BArraNetw,F0:AB:54MitsumiE,F0:AC:A4HBC-radi,F0:AC:D7IEEERegi,F0:AC:D7:00:00:00/28Guilingl,F0:AC:D7:10:00:00/28Intenta,F0:AC:D7:20:00:00/28QuantumP,F0:AC:D7:30:00:00/28Med-PatI,F0:AC:D7:40:00:00/28Sercomm,F0:AC:D7:50:00:00/28PavoTasa,F0:AC:D7:60:00:00/28SuzhouPa,F0:AC:D7:70:00:00/28HanjuNet,F0:AC:D7:80:00:00/28Telefoni,F0:AC:D7:90:00:00/28U3storag,F0:AC:D7:A0:00:00/28Groupeer,F0:AC:D7:B0:00:00/28Zhejiang,F0:AC:D7:C0:00:00/28Simprint,F0:AC:D7:D0:00:00/28SmartPow,F0:AC:D7:E0:00:00/28Fiziico,F0:AD:4EGlobalsc,F0:AE:51Xi3,F0:AE:66CosonicI,F0:AF:50PhantomI,F0:AF:85ARRISGro,F0:B0:14AVMAudio,F0:B0:22TOHOElec,F0:B0:52RuckusWi,F0:B0:E7Apple,F0:B1:07Ericsson,F0:B1:1DNokia,F0:B1:3FHuaweiDe,F0:B2:E5Cisco,F0:B3:1EUniversa,F0:B3:ECApple,F0:B4:29XiaomiCo,F0:B4:79Apple,F0:B4:D2D-LinkIn,F0:B5:B7Disrupti,F0:B5:D1TexasIns,F0:B6:1EIntelCor,F0:B6:EBPoslabTe,F0:B9:68ItelMobi,F0:BC:C8MaxID,F0:BC:C9Pfu,F0:BD:2EH+SPolat,F0:BD:F1Sipod,F0:BF:97Sony,F0:C1:F1Apple,F0:C2:4CZhejiang,F0:C2:7CMianyang,F0:C3:71Apple,F0:C4:2FHuaweiDe,F0:C4:78HuaweiTe,F0:C7:7FTexasIns,F0:C8:14Shenzhen,F0:C8:50HuaweiTe,F0:C8:8CLeddarTe,F0:C9:D1GDMideaA,F0:CB:A1Apple,F0:CD:31SamsungE,F0:D0:8CTCTmobil,F0:D1:4FLinear,F0:D1:A9Apple,F0:D1:B8Ledvance,F0:D2:F1AmazonTe,F0:D3:A7CobaltRa,F0:D3:E7Sensomet,F0:D4:E2Dell,F0:D4:F6LarsThra,F0:D4:F7varramsy,F0:D5:BFIntelCor,F0:D6:57Echosens,F0:D7:67AxemaPas,F0:D7:AAMotorola,F0:D7:AFIEEERegi,F0:D7:AF:00:00:00/28IDTechJa,F0:D7:AF:10:00:00/28BeijingS,F0:D7:AF:20:00:00/28Blacknig,F0:D7:AF:30:00:00/28720?beij,F0:D7:AF:40:00:00/28ADAMAudi,F0:D7:AF:50:00:00/28Dongguan,F0:D7:AF:60:00:00/28AnordMar,F0:D7:AF:70:00:00/28Rievtech,F0:D7:AF:80:00:00/28ShenZhen,F0:D7:AF:90:00:00/28NewITPro,F0:D7:AF:A0:00:00/28MstarTec,F0:D7:AF:B0:00:00/28Evco,F0:D7:AF:C0:00:00/28Shenzhen,F0:D7:AF:D0:00:00/28Dongguan,F0:D7:AF:E0:00:00/28WrenAsso,F0:D7:DCWesineWu,F0:D9:B2Exo,F0:DA:7CRlhIndus,F0:DB:30Yottabyt,F0:DB:E2Apple,F0:DB:F8Apple,F0:DC:E2Apple,F0:DE:71Shanghai,F0:DE:B9ShangHai,F0:DE:F1WistronI,F0:E3:DCTeconMT,F0:E4:A2HuaweiTe,F0:E5:C3Drägerwe,F0:E7:7ESamsungE,F0:EB:D0Shanghai,F0:EC:39Essec,F0:ED:1EBilkonBi,F0:EE:10SamsungE,F0:EE:58PACETele,F0:EE:BBVIPAR,F0:EF:86Google,F0:EF:D2TfPaymen,F0:F0:02HonHaiPr,F0:F0:8FNextekSo,F0:F0:A4AmazonTe,F0:F2:49HitronTe,F0:F2:60Mobitec,F0:F3:36Tp-LinkT,F0:F5:64SamsungE,F0:F5:AEAdaptrum,F0:F6:1CApple,F0:F6:44Whitesky,F0:F6:69MotionAn,F0:F6:C1Sonos,F0:F7:55Cisco,F0:F7:B3Phorm,F0:F7:E7HuaweiTe,F0:F8:42KEEBOX,F0:F8:F2TexasIns,F0:F9:F7IES,F0:FA:C7HuaweiDe,F0:FC:C8ARRISGro,F0:FD:A0AcurixNe,F0:FE:6BShanghai,F0:FE:E7HuaweiDe,F4:02:23PAXCompu,F4:02:28SamsungE,F4:02:70Dell,F4:03:04Google,F4:03:21BeNeXt,F4:03:2AAmazonTe,F4:03:2FReduxio,F4:03:43HewlettP,F4:04:4CValenceT,F4:06:16Apple,F4:06:69IntelCor,F4:06:8Ddevolo,F4:06:A5Hangzhou,F4:09:D8SamsungE,F4:0A:4AINDUSNET,F4:0B:93BlackBer,F4:0B:9FCigShang,F4:0E:01Apple,F4:0E:11IEEERegi,F4:0E:11:00:00:00/28realphon,F4:0E:11:10:00:00/28BeijingD,F4:0E:11:20:00:00/28Axel,F4:0E:11:30:00:00/28Shenzhen,F4:0E:11:40:00:00/28DayangTe,F4:0E:11:50:00:00/28E-Song,F4:0E:11:60:00:00/28AlphaDes,F4:0E:11:70:00:00/28Shenzhen,F4:0E:11:80:00:00/28Zeepro,F4:0E:11:90:00:00/28SternaSe,F4:0E:11:A0:00:00/28Kodpro,F4:0E:11:B0:00:00/28BradarIn,F4:0E:11:C0:00:00/28NihonMeg,F4:0E:11:D0:00:00/28DXGTechn,F4:0E:11:E0:00:00/28Elektron,F4:0E:11:F0:00:00/28Private,F4:0E:22SamsungE,F4:0E:83ARRISGro,F4:0F:1BCisco,F4:0F:24Apple,F4:0F:9BWavelink,F4:15:35SPONComm,F4:15:63F5Networ,F4:15:FDShanghai,F4:17:B8AirTiesW,F4:19:E2Volterra,F4:1B:A1Apple,F4:1C:95BeijingY,F4:1D:6BHuaweiTe,F4:1E:26Simon-Ka,F4:1E:5ERtBrick,F4:1F:0BYAMABISH,F4:1F:88zte,F4:1F:C2Cisco,F4:20:12Cucinial,F4:23:9CSernetSu,F4:26:79IntelCor,F4:28:33MMPC,F4:28:53ZioncomE,F4:28:96SpectoPa,F4:29:81vivoMobi,F4:2A:7DTp-LinkT,F4:2B:48Ubiqam,F4:2C:56SenorTec,F4:2E:7FArubaaHe,F4:30:8BXiaomiCo,F4:30:B9HewlettP,F4:31:C3Apple,F4:32:3DSichuant,F4:33:28CIMCONLi,F4:34:F0Apple,F4:36:E1Abilis,F4:37:B7Apple,F4:38:14Shanghai,F4:39:09HewlettP,F4:3D:80FAGIndus,F4:3E:61Shenzhen,F4:3E:66BeeCompu,F4:3E:9DBenuNetw,F4:41:56Arrikto,F4:41:9EHuaweiDe,F4:42:27SSResear,F4:42:8FSamsungE,F4:44:50BND,F4:45:88HuaweiTe,F4:45:EDPortable,F4:46:37IntelCor,F4:47:13LeadingP,F4:47:2ANanjingR,F4:48:48Amscreen,F4:49:55MIMOTECH,F4:49:EFEmstone,F4:4B:2ACiscoSPV,F4:4C:70Skyworth,F4:4C:7FHuaweiTe,F4:4D:17Goldcard,F4:4D:30Elitegro,F4:4E:05Cisco,F4:4E:38Olibra,F4:4E:E3IntelCor,F4:4E:FDActionsS,F4:4F:D3shenzhen,F4:50:EBTelechip,F4:52:14Mellanox,F4:54:20Tellesco,F4:54:33Rockwell,F4:55:95HENGBAO,F4:55:9CHuaweiTe,F4:55:E0NicewayC,F4:57:3EFiberhom,F4:58:42BoxxTV,F4:5B:73Wanjiaan,F4:5C:89Apple,F4:5E:ABTexasIns,F4:5F:69MatsufuE,F4:5F:D4CiscoSPV,F4:5F:F7DQTechno,F4:60:0DPanoptic,F4:60:77TexasIns,F4:60:E2XiaomiCo,F4:62:D0NotforRa,F4:63:1FHuaweiTe,F4:63:49Diffon,F4:63:E7NanjingM,F4:64:5DToshiba,F4:65:A6Apple,F4:67:2DShenZhen,F4:69:42AskeyCom,F4:69:D5IEEERegi,F4:69:D5:00:00:00/28Mossman,F4:69:D5:10:00:00/28Junchuan,F4:69:D5:20:00:00/28PulsarEn,F4:69:D5:30:00:00/28ITS,F4:69:D5:40:00:00/28StypeCSd,F4:69:D5:50:00:00/28HefeiSTA,F4:69:D5:60:00:00/28TianJinK,F4:69:D5:70:00:00/28Rosco,F4:69:D5:90:00:00/28Terminus,F4:69:D5:A0:00:00/28ShenZhen,F4:69:D5:B0:00:00/28Konntek,F4:69:D5:C0:00:00/28HuaqinTe,F4:69:D5:D0:00:00/28NantongZ,F4:69:D5:E0:00:00/28ORtekTec,F4:6A:92Shenzhen,F4:6A:BCAdonit,F4:6A:D7Microsof,F4:6B:8CHonHaiPr,F4:6B:EFSagemcom,F4:6D:04ASUSTekC,F4:6D:E2zte,F4:6E:24NECPerso,F4:6E:95ExtremeN,F4:6F:4EEchowell,F4:6F:A4PhysikIn,F4:6F:EDFiberhom,F4:70:0CIEEERegi,F4:70:0C:00:00:00/28Hyunsung,F4:70:0C:10:00:00/28Shenzhen,F4:70:0C:20:00:00/28BeijingA,F4:70:0C:30:00:00/28UnionSou,F4:70:0C:40:00:00/28Shenzhen,F4:70:0C:50:00:00/28Shenzhen,F4:70:0C:60:00:00/28JinanUSR,F4:70:0C:70:00:00/28Changdex,F4:70:0C:80:00:00/28Shenzhen,F4:70:0C:90:00:00/28Annapurn,F4:70:0C:A0:00:00/28JinanHua,F4:70:0C:B0:00:00/28Shanghai,F4:70:0C:C0:00:00/28GSDGroup,F4:70:0C:D0:00:00/28Freeus,F4:70:0C:E0:00:00/28Shenzhen,F4:70:ABvivoMobi,F4:71:90SamsungE,F4:73:35Logitech,F4:73:CAConversi,F4:74:88NewH3CTe,F4:76:26Viltechm,F4:79:60HuaweiTe,F4:7A:4EWoojeon&,F4:7A:CCSolidFir,F4:7B:09IntelCor,F4:7B:5ESamsungE,F4:7D:EFSamsungE,F4:7F:35Cisco,F4:81:39Canon,F4:83:CDTp-LinkT,F4:83:E1Shanghai,F4:84:4CTexasIns,F4:85:C6FDTTechn,F4:87:71Infoblox,F4:87:C5HuaweiDe,F4:8B:32XiaomiCo,F4:8C:50IntelCor,F4:8C:EBD-LinkIn,F4:8E:09Nokia,F4:8E:38Dell,F4:8E:92HuaweiTe,F4:90:CATensorco,F4:90:CBIEEERegi,F4:90:CB:00:00:00/28Epitel,F4:90:CB:10:00:00/28Delem,F4:90:CB:20:00:00/28ICEGatew,F4:90:CB:30:00:00/28RickerLy,F4:90:CB:40:00:00/28OmniNet,F4:90:CB:50:00:00/28Avilutio,F4:90:CB:60:00:00/28AirbeamW,F4:90:CB:70:00:00/28Teq,F4:90:CB:80:00:00/28BeijingP,F4:90:CB:90:00:00/28FractylL,F4:90:CB:A0:00:00/28Private,F4:90:CB:B0:00:00/28A-dec,F4:90:CB:C0:00:00/28CheetahM,F4:90:CB:D0:00:00/28Simavita,F4:90:CB:E0:00:00/28RSAELabs,F4:90:EADeciso,F4:91:1EZhuhaiEw,F4:92:BFUbiquiti,F4:93:9FHonHaiPr,F4:94:61NexGenSt,F4:94:66CountMax,F4:95:1BHefeiRad,F4:96:34IntelCor,F4:96:51NAKAYO,F4:97:C2Nebulon,F4:99:ACWEBERSch,F4:9C:12Structab,F4:9E:EFTaicangT,F4:9F:54SamsungE,F4:9F:F3HuaweiTe,F4:A2:94EagleWor,F4:A4:54IEEERegi,F4:A4:54:00:00:00/28NKTPhoto,F4:A4:54:30:00:00/28UbiqcomI,F4:A4:54:40:00:00/28Earshots,F4:A4:54:50:00:00/28Denshiji,F4:A4:54:60:00:00/28IntrolDe,F4:A4:54:70:00:00/28Advanced,F4:A4:54:80:00:00/28Shenzhen,F4:A4:54:B0:00:00/28Graco,F4:A4:75IntelCor,F4:A4:D6HuaweiTe,F4:A5:2AHawaTech,F4:A5:9DHuaweiDe,F4:A7:39JuniperN,F4:A8:0DWistronI,F4:A9:97Canon,F4:AC:C1Cisco,F4:AF:E7Apple,F4:B1:64Lightnin,F4:B1:9CAltoBeam,F4:B1:C2Zhejiang,F4:B3:01IntelCor,F4:B3:81WindowMa,F4:B5:20BiostarM,F4:B5:2FJuniperN,F4:B5:49XiamenYe,F4:B5:AAzte,F4:B5:BBCeragonN,F4:B6:88Plantron,F4:B6:E5TerraSem,F4:B7:2ATimeInte,F4:B7:8DHuaweiTe,F4:B7:B3vivoMobi,F4:B7:E2HonHaiPr,F4:B8:5ETexasIns,F4:B8:A7zte,F4:BC:97Shenzhen,F4:BC:DAShenzhen,F4:BD:7CChengduj,F4:BD:9ECisco,F4:BE:ECApple,F4:BF:80HuaweiTe,F4:BF:A8JuniperN,F4:C0:2FBlueBite,F4:C1:14Technico,F4:C2:48SamsungE,F4:C4:47CoagentI,F4:C4:D6Shenzhen,F4:C6:13Alcatel-,F4:C6:D7blackned,F4:C7:14HuaweiTe,F4:C7:95WEYTechn,F4:C7:AAMarvellS,F4:C7:C8Kelvin,F4:CA:24FreeBit,F4:CA:E5FreeboxS,F4:CB:52HuaweiTe,F4:CC:55JuniperN,F4:CD:90Vispiron,F4:CE:36NordicSe,F4:CE:46HewlettP,F4:CE:48ExtremeN,F4:CF:A2Espressi,F4:CF:E2Cisco,F4:D0:32YunnanId,F4:D1:08IntelCor,F4:D2:61SEMOCON,F4:D4:88Apple,F4:D6:20Guangdon,F4:D7:B2LGSInnov,F4:D9:C6Unionman,F4:D9:FBSamsungE,F4:DB:E3Apple,F4:DB:E6Cisco,F4:DC:41Youngzon,F4:DC:4DBeijingC,F4:DC:A5DawonDns,F4:DC:DAZhuhaiJi,F4:DC:F9HuaweiTe,F4:DD:9EGoPro,F4:DE:0CESPOD,F4:DE:AFHuaweiTe,F4:E1:1ETexasIns,F4:E1:42DeltaEle,F4:E2:04Traqueur,F4:E3:FBHuaweiTe,F4:E4:51HuaweiTe,F4:E4:ADzte,F4:E4:D7FujianSt,F4:E5:78Proizvod,F4:E5:F2HuaweiTe,F4:E6:D7SolarPow,F4:E9:26TianjinZ,F4:E9:D4QLogic,F4:EA:67Cisco,F4:EA:B5ExtremeN,F4:EB:38Sagemcom,F4:EB:9FEllu2019,F4:EC:38Tp-LinkT,F4:ED:5FShenzhen,F4:EE:08Dell,F4:EE:14MercuryC,F4:EF:9ESgsgScie,F4:F1:5AApple,F4:F1:97EMTAKE,F4:F1:E1Motorola,F4:F2:6DTp-LinkT,F4:F3:09SamsungE,F4:F3:AAJBL,F4:F5:24Motorola,F4:F5:A5Nokia,F4:F5:D8Google,F4:F5:DBXiaomiCo,F4:F5:E8Google,F4:F6:46Dediprog,F4:F9:51Apple,F4:FB:B8HuaweiTe,F4:FC:32TexasIns,F4:FC:B1JJ,F4:FD:2BZOYI,F4:FE:FBSamsungE,F8:00:A1HuaweiTe,F8:01:13HuaweiTe,F8:02:78IEEERegi,F8:02:78:00:00:00/28Digatron,F8:02:78:10:00:00/28ReasonTe,F8:02:78:20:00:00/28Innodisk,F8:02:78:30:00:00/283Shape,F8:02:78:40:00:00/28CLARUSKo,F8:02:78:50:00:00/28Electric,F8:02:78:60:00:00/28Witium,F8:02:78:70:00:00/28Bettini,F8:02:78:80:00:00/28EMBUXTec,F8:02:78:90:00:00/28BeijingR,F8:02:78:A0:00:00/28LuxulTec,F8:02:78:B0:00:00/28Rosemoun,F8:02:78:C0:00:00/28Technolo,F8:02:78:D0:00:00/28Duetonsr,F8:02:78:E0:00:00/28LitTechn,F8:02:78:F0:00:00/28Private,F8:03:32Khomp,F8:03:77Apple,F8:04:2ESamsungE,F8:05:1CDRSImagi,F8:08:4FSagemcom,F8:0B:BEARRISGro,F8:0B:CBCisco,F8:0B:D0DatangTe,F8:0C:58TaicangT,F8:0C:F3LGElectr,F8:0D:43HonHaiPr,F8:0D:60Canon,F8:0D:ACHP,F8:0D:EAZyCastTe,F8:0D:F0zte,F8:0D:F1Sontex,F8:0F:41WistronI,F8:0F:6FCisco,F8:0F:84NaturalS,F8:0F:F9Google,F8:10:37AtopiaLP,F8:10:93Apple,F8:13:08Nokia,F8:15:47Avaya,F8:16:54IntelCor,F8:18:972Wire,F8:1A:2BGoogle,F8:1A:67Tp-LinkT,F8:1B:04ZhongSha,F8:1C:E5Telefonb,F8:1D:0FHitronTe,F8:1D:78IEEERegi,F8:1D:78:00:00:00/28Dongguan,F8:1D:78:10:00:00/28ADTECHNO,F8:1D:78:20:00:00/28XperioLa,F8:1D:78:30:00:00/28Shanghai,F8:1D:78:40:00:00/28DigitalI,F8:1D:78:50:00:00/28Dacons,F8:1D:78:60:00:00/28Zengge,F8:1D:78:70:00:00/28WuhanGui,F8:1D:78:80:00:00/28Teleofis,F8:1D:78:90:00:00/28OphrysSy,F8:1D:78:A0:00:00/28AVProGlo,F8:1D:78:B0:00:00/28SigmaCon,F8:1D:78:C0:00:00/28Shenzhuo,F8:1D:78:D0:00:00/28Tofino,F8:1D:78:E0:00:00/28Guangdon,F8:1D:90Solidwin,F8:1D:93Longdhua,F8:1E:6FEBGcompl,F8:1E:DFApple,F8:1F:32Motorola,F8:20:55GreenInf,F8:20:A9HuaweiDe,F8:22:85CypressT,F8:23:87Shenzhen,F8:23:B2HuaweiTe,F8:24:41Yeelink,F8:27:2EMercku,F8:27:93Apple,F8:28:19LiteonTe,F8:29:C0Availink,F8:2B:C8JiangsuS,F8:2C:182Wire,F8:2D:7CApple,F8:2D:C0ARRISGro,F8:2E:3FHuaweiTe,F8:2E:8ENanjingK,F8:2E:DBRTW,F8:2F:08MolexCMS,F8:2F:5BeGauge,F8:2F:65HuaweiDe,F8:2F:6AItelMobi,F8:2F:A8HonHaiPr,F8:30:02TexasIns,F8:30:94Alcatel-,F8:31:3Eendeavou,F8:32:E4ASUSTekC,F8:33:31TexasIns,F8:33:76GoodMind,F8:34:41IntelCor,F8:35:53MagentaR,F8:35:DDGemtekTe,F8:36:9BTexasIns,F8:38:69LGElectr,F8:38:80Apple,F8:3B:1DTechnico,F8:3B:7EHuaweiDe,F8:3C:BFBotatoEl,F8:3D:4ESoftlink,F8:3D:FFHuaweiTe,F8:3E:95HuaweiTe,F8:3F:51SamsungE,F8:42:FBYasudaJo,F8:44:E3TaicangT,F8:45:ADKonkaGro,F8:45:C4Shenzhen,F8:46:1CSonyInte,F8:46:2DSYNTECIn,F8:47:2DX2genDig,F8:48:97Hitachi,F8:48:FDChinaMob,F8:4A:73Eumtech,F8:4A:7FInnometr,F8:4A:BFHuaweiTe,F8:4C:DAHuaweiTe,F8:4D:33Fiberhom,F8:4D:89Apple,F8:4D:FCHangzhou,F8:4E:17Sony,F8:4E:73Apple,F8:4F:57Cisco,F8:4F:ADHuiZhouG,F8:50:1CTianjinG,F8:50:63Verathon,F8:51:28SimpliSa,F8:51:6DDenwaTec,F8:52:DFVNLEurop,F8:53:29HuaweiTe,F8:54:AFECITelec,F8:54:B8AmazonTe,F8:55:CDVisteon,F8:56:C3zte,F8:57:2ECoreBran,F8:59:71IntelCor,F8:5A:00SanfordL,F8:5B:3BAskeyCom,F8:5B:9CSB,F8:5B:C9M-Cube,F8:5C:45ICNexus,F8:5C:4DNokia,F8:5C:7DShenzhen,F8:5E:3CShenzhen,F8:5E:42Technico,F8:5E:A0IntelCor,F8:5F:2ANokia,F8:60:F0ArubaaHe,F8:62:14Apple,F8:62:AAxn,F8:63:3FIntelCor,F8:64:65AnovaApp,F8:64:B8zte,F8:66:01SuzhouCh,F8:66:5AApple,F8:66:D1HonHaiPr,F8:66:F2Cisco,F8:69:71SeibuEle,F8:6B:D9Cisco,F8:6C:03Shenzhen,F8:6C:E1TaicangT,F8:6D:73Zengge,F8:6E:CFArcx,F8:6E:EEHuaweiTe,F8:6F:C1Apple,F8:6F:DEShenzhen,F8:71:FEGoldmanS,F8:72:EACisco,F8:73:94Netgear,F8:73:A2Avaya,F8:75:88HuaweiTe,F8:75:A4LCFCHeFe,F8:76:9BNeopis,F8:77:B8SamsungE,F8:79:0AARRISGro,F8:7A:41Cisco,F8:7A:EFRosonixT,F8:7B:20Cisco,F8:7B:62FASTWELI,F8:7B:7AARRISGro,F8:7B:8CAmpedWir,F8:7F:A5Greatek,F8:80:96ElsysEqu,F8:81:1AOverkiz,F8:82:00CaptionC,F8:84:79YaojinTe,F8:84:F2SamsungE,F8:85:F9Calix,F8:87:F1Apple,F8:89:3CInventec,F8:89:D2CloudNet,F8:8A:3CIEEERegi,F8:8A:3C:00:00:00/28Art,F8:8A:3C:10:00:00/28Carefree,F8:8A:3C:20:00:00/28KLATUNet,F8:8A:3C:30:00:00/28Shenzhen,F8:8A:3C:40:00:00/28Go-LinkT,F8:8A:3C:50:00:00/28Kokkia,F8:8A:3C:60:00:00/28BeijingZ,F8:8A:3C:70:00:00/28Joshai,F8:8A:3C:80:00:00/28CadmusEl,F8:8A:3C:90:00:00/28withus,F8:8A:3C:A0:00:00/28Protos,F8:8A:3C:B0:00:00/28FaraAs,F8:8A:3C:C0:00:00/28ExcetopT,F8:8A:3C:D0:00:00/28THK,F8:8A:3C:E0:00:00/28Avateq,F8:8A:5ETexasIns,F8:8B:37ARRISGro,F8:8C:1CKaishunE,F8:8C:21Tp-LinkT,F8:8D:EFTenebrae,F8:8E:85Comtrend,F8:8E:A1Edgecore,F8:8F:07SamsungE,F8:8F:CAGoogle,F8:90:66Nain,F8:91:2AGLPGerma,F8:91:73AedleSas,F8:93:F3Volans,F8:94:C2IntelCor,F8:95:22HuaweiTe,F8:95:50ProtonPr,F8:95:C7LGElectr,F8:95:EAApple,F8:97:53HuaweiDe,F8:97:CFDaeshin-,F8:98:3ALeemanIn,F8:98:B9HuaweiTe,F8:98:EFHuaweiTe,F8:99:10Integrat,F8:99:55Fortress,F8:9A:78HuaweiTe,F8:9D:0DControlT,F8:9D:BBTintri,F8:9E:28CiscoMer,F8:9E:94IntelCor,F8:9F:B8YAZAKIEn,F8:A0:3DDinstarT,F8:A0:97ARRISGro,F8:A1:88LEDRoadw,F8:A2:6DCanon,F8:A2:B4RHEWA-WA,F8:A2:D6LiteonTe,F8:A3:4Fzte,F8:A4:5FXiaomiCo,F8:A5:C5Cisco,F8:A7:3ACisco,F8:A7:63Zhejiang,F8:A9:63CompalIn,F8:A9:D0LGElectr,F8:A9:DEPuissanc,F8:AA:3FDWnetTec,F8:AA:8AAxviewTe,F8:AB:05Sagemcom,F8:AB:E5shenzhen,F8:AC:65IntelCor,F8:AC:6DDeltenna,F8:AD:CBHMDGloba,F8:AE:27JohnDeer,F8:AF:05HuaweiDe,F8:AF:DBFiberhom,F8:B1:32HuaweiTe,F8:B1:56Dell,F8:B1:DDApple,F8:B2:F3Guangzho,F8:B4:6AHewlettP,F8:B5:4DIntelCor,F8:B5:68IEEERegi,F8:B5:68:00:00:00/28LifePrin,F8:B5:68:10:00:00/28PTEyroDi,F8:B5:68:20:00:00/28Shenzhen,F8:B5:68:30:00:00/28DongwooE,F8:B5:68:40:00:00/28Combiwin,F8:B5:68:50:00:00/28etectRx,F8:B5:68:60:00:00/28PackageG,F8:B5:68:70:00:00/28CloudMin,F8:B5:68:80:00:00/28MavenWir,F8:B5:68:90:00:00/28BeijingW,F8:B5:68:A0:00:00/28SinePuls,F8:B5:68:B0:00:00/28Whizpace,F8:B5:68:C0:00:00/283SISecur,F8:B5:68:D0:00:00/28Solarius,F8:B5:68:E0:00:00/28&quot;RA,F8:B5:99Guangzho,F8:B7:97NECPlatf,F8:B7:E2Cisco,F8:B9:5ALGInnote,F8:BA:E6Nokia,F8:BB:BFeero,F8:BC:0Eeero,F8:BC:12Dell,F8:BC:41Rosslare,F8:BE:0DA2UICT,F8:BF:09HuaweiTe,F8:C0:01JuniperN,F8:C0:91Highgate,F8:C1:20XianLink,F8:C2:49Private,F8:C2:88Cisco,F8:C3:72TsuzukiD,F8:C3:97NZXT,F8:C3:9EHuaweiTe,F8:C3:CCApple,F8:C4:F3Shanghai,F8:C6:78Carefusi,F8:C9:6CFiberhom,F8:CA:59NetCommW,F8:CA:85NEC,F8:CA:B8Dell,F8:CC:6EDEPOElec,F8:CE:72Wistron,F8:CF:C5Motorola,F8:D0:27SeikoEps,F8:D0:ACSonyInte,F8:D0:BDSamsungE,F8:D1:11Tp-LinkT,F8:D3:A9AXANNetw,F8:D4:62Pumatron,F8:D4:78Flextron,F8:D7:56SimmTron,F8:D7:BFREVRitte,F8:D9:B8OpenMesh,F8:DA:0CHonHaiPr,F8:DA:DFEcoTech,F8:DA:E2NDCTechn,F8:DA:F4TaishanO,F8:DB:4CPNYTechn,F8:DB:7FHTC,F8:DB:88Dell,F8:DC:7AVariscit,F8:DF:15SunitecE,F8:DF:A8zte,F8:DF:E1MyLight,F8:E0:79Motorola,F8:E4:3BASIXElec,F8:E4:4EMcot,F8:E4:E3IntelCor,F8:E4:FBActionte,F8:E5:CFCgiItUk,F8:E6:1ASamsungE,F8:E7:1ERuckusWi,F8:E7:A0vivoMobi,F8:E7:B5µTechTec,F8:E8:11HuaweiTe,F8:E8:77HarmanBe,F8:E9:03D-LinkIn,F8:E9:4EApple,F8:E9:68EgkerKft,F8:EA:0ADipl-Mat,F8:ED:A5ARRISGro,F8:F0:05NewportM,F8:F0:14RackWare,F8:F0:82Nagtech,F8:F1:B6Motorola,F8:F1:E6SamsungE,F8:F2:1EIntelCor,F8:F2:5AG-Lab,F8:F4:64RaweElec,F8:F5:32ARRISGro,F8:F7:B9HuaweiTe,F8:F7:D3Internat,F8:F7:FFSyn-Tech,F8:FB:2FSantur,F8:FC:E1AmazonTe,F8:FE:5CReciproc,F8:FE:A8Technico,F8:FF:0BElectron,F8:FF:5FShenzhen,F8:FF:C2Apple,FA:07:3ENBASE-TA,FA:0B:BCAsd-Stan,FA:14:66Allegion,FA:16:19trendyte,FA:2E:E9OCAAllia,FA:4C:77Occitali,FA:55:6FSymbolic,FA:61:0ELaborato,FA:63:E1SamsungE,FA:66:73NellsEle,FA:94:F1IEEE8021,FA:BA:85APATOpto,FA:D9:88Momentum,FA:E1:90InWinDev,FA:E5:1ARSAELabs,FA:EB:6ExRANorg,FA:F9:C0RAID,FC:00:12ToshibaS,FC:01:7CHonHaiPr,FC:01:9EVievu,FC:01:CDFundacio,FC:02:96XiaomiCo,FC:03:9FSamsungE,FC:04:1CGuangdon,FC:06:47Cortland,FC:06:EDM2Motive,FC:07:A0LREMedic,FC:08:4AFujitsu,FC:08:77PrentkeR,FC:09:D8ACTEONGr,FC:09:F6Guangdon,FC:0A:81ExtremeN,FC:0C:45Shenzhen,FC:0F:4BTexasIns,FC:0F:E6SonyInte,FC:0F:E7Microchi,FC:10:BDControlS,FC:10:C6TaicangT,FC:11:86Logic3,FC:12:2CHuaweiTe,FC:13:49GlobalAp,FC:13:F0Bouffalo,FC:14:99AimoreAc,FC:15:B4HewlettP,FC:16:07TaianTec,FC:17:94InterCre,FC:18:3CApple,FC:19:10SamsungE,FC:19:28ActionsM,FC:19:99XiaomiCo,FC:19:D0CloudVis,FC:1A:11vivoMobi,FC:1B:D1HuaweiTe,FC:1B:FFV-Zug,FC:1C:A1Nokia,FC:1D:2AvivoMobi,FC:1D:43Apple,FC:1D:59ISmartCi,FC:1D:84Autobase,FC:1E:16IPEVO,FC:1F:19SamsungE,FC:1F:C0Eurecam,FC:22:9CHanKyung,FC:23:25EosTekSh,FC:25:3FApple,FC:27:A2TransEle,FC:29:E3Infinixm,FC:29:F3McPay,FC:2A:54Connecte,FC:2A:9CApple,FC:2B:B2Actionte,FC:2D:5Ezte,FC:2E:19ChinaMob,FC:2E:2DLoromInd,FC:2F:40Calxeda,FC:2F:6BEverspin,FC:2F:AANokia,FC:2F:EFUTTTechn,FC:32:88CELOTWir,FC:33:42JuniperN,FC:33:5FPolyera,FC:34:97ASUSTekC,FC:35:98Favite,FC:35:E6Visteon,FC:37:2BSichuanT,FC:39:64ItelMobi,FC:3C:E9Tsington,FC:3D:93Longchee,FC:3D:A5Arcadyan,FC:3F:7CHuaweiTe,FC:3F:ABHenanLan,FC:3F:DBHewlettP,FC:40:09zte,FC:42:03SamsungE,FC:42:65Zhejiang,FC:44:63Universa,FC:44:82IntelCor,FC:44:99SwarcoLE,FC:44:9Fzte,FC:45:5FJiangxiS,FC:45:96CompalIn,FC:45:C3TexasIns,FC:48:EFHuaweiTe,FC:49:2DAmazonTe,FC:4A:E9Castlene,FC:4B:1CIntersen,FC:4B:57Peerless,FC:4B:BCSunplusT,FC:4D:8CShenzhen,FC:4D:A6HuaweiTe,FC:4D:D4Universa,FC:4E:A4Apple,FC:50:90SIMEXSp,FC:51:A4ARRISGro,FC:52:8DTechnico,FC:52:CEControli,FC:53:9EShanghai,FC:55:DCBalticLa,FC:58:4Axiamensh,FC:58:9ACisco,FC:58:DFInterpho,FC:58:FAShenZhen,FC:5A:1DHitronTe,FC:5B:24WeibelSc,FC:5B:26MikroBit,FC:5B:39Cisco,FC:5C:45RuckusWi,FC:5F:49Zhejiang,FC:60:18Zhejiang,FC:60:9BNewH3CTe,FC:61:98NECPerso,FC:61:E9Fiberhom,FC:62:6EBeijingM,FC:62:B9Alpsalpi,FC:64:3ASamsungE,FC:64:BAXiaomiCo,FC:65:B3HuaweiDe,FC:65:DEAmazonTe,FC:66:CFApple,FC:68:3EDirected,FC:69:47TexasIns,FC:69:8CANDREASS,FC:6B:F0TopwellI,FC:6C:31LXinstru,FC:6D:C0Bme,FC:6D:D1APRESIA,FC:6F:B7ARRISGro,FC:71:FATraneTec,FC:73:FBHuaweiTe,FC:75:16D-LinkIn,FC:75:E6Handream,FC:76:92Semptian,FC:77:74IntelCor,FC:77:7BHitronTe,FC:79:0BHitachiH,FC:7C:02PhicommS,FC:7C:E7FciUsa,FC:7D:6CHYESUNGT,FC:7F:56CoSystCo,FC:7F:F1ArubaaHe,FC:83:29Treitech,FC:83:99Avaya,FC:83:C6N-RadioT,FC:85:96Axonne,FC:86:2AHuaweiDe,FC:87:43HuaweiTe,FC:8A:3Dzte,FC:8B:97Shenzhen,FC:8D:3DLeapfive,FC:8E:5BChinaMob,FC:8E:6EStreamCC,FC:8E:7EARRISGro,FC:8F:7DShenzhen,FC:8F:90SamsungE,FC:8F:C4Intellig,FC:90:FAIndepend,FC:91:14Technico,FC:92:3BNokia,FC:92:57RenesasE,FC:94:35HuaweiTe,FC:94:6CUbivelox,FC:94:CEzte,FC:94:E3Technico,FC:95:6AOctagon,FC:96:43JuniperN,FC:99:47Cisco,FC:9A:FAMotusGlo,FC:9B:C6Sumavisi,FC:9B:D4EdgeQ,FC:9C:98ArloTech,FC:9D:D8BeijingT,FC:9F:AEFidus,FC:9F:E1CONWINTe,FC:A1:3ESamsungE,FC:A1:83AmazonTe,FC:A2:2APTCallys,FC:A3:86Shenzhen,FC:A4:7AIEEERegi,FC:A4:7A:00:00:00/28Broadcom,FC:A4:7A:10:00:00/28Shenzhen,FC:A4:7A:20:00:00/28AntFinan,FC:A4:7A:30:00:00/28Cliptech,FC:A4:7A:40:00:00/28Hooc,FC:A4:7A:50:00:00/28Syfer,FC:A4:7A:60:00:00/28Token,FC:A4:7A:70:00:00/28Innovati,FC:A4:7A:80:00:00/28KarryCom,FC:A4:7A:90:00:00/28OberixGr,FC:A4:7A:A0:00:00/28Shenzhen,FC:A4:7A:B0:00:00/28Shenzhen,FC:A4:7A:C0:00:00/28Shenzhen,FC:A4:7A:D0:00:00/28Shenzhen,FC:A4:7A:E0:00:00/28HefeiFei,FC:A5:D0Guangdon,FC:A6:21SamsungE,FC:A6:4CAlibabac,FC:A6:67AmazonTe,FC:A6:CDFiberhom,FC:A8:41Avaya,FC:A8:4ASentinum,FC:A8:9ASunitecE,FC:A8:9BTexasIns,FC:A9:B0Miartech,FC:A9:DCRenesasE,FC:AA:14Giga-Byt,FC:AA:81Apple,FC:AA:B6SamsungE,FC:AB:90HuaweiTe,FC:AD:0FQtsNetwo,FC:AE:34ARRISGro,FC:AF:6AQulsar,FC:AF:ACSocionex,FC:AF:BETireChec,FC:B0:C4Shanghai,FC:B1:0DShenzhen,FC:B3:BCIntelCor,FC:B4:E6AskeyCom,FC:B5:8AWapice,FC:B6:62ICHoldin,FC:B6:98Cambridg,FC:B6:9DZhejiang,FC:B6:D8Apple,FC:B7:F0IdahoNat,FC:BB:A1Shenzhen,FC:BC:0EZhejiang,FC:BC:9CVimar,FC:BC:D1HuaweiTe,FC:BD:67AristaNe,FC:BE:7BvivoMobi,FC:C2:33ASUSTekC,FC:C2:3DAtmel,FC:C2:DEMurataMa,FC:C7:34SamsungE,FC:C8:97zte,FC:CA:C4LifeHeal,FC:CC:E4Ascon,FC:CD:2FIEEERegi,FC:CD:2F:00:00:00/28NingboBu,FC:CD:2F:10:00:00/28SirenCar,FC:CD:2F:20:00:00/28Loupedec,FC:CD:2F:30:00:00/28XmitechT,FC:CD:2F:40:00:00/28GenitekE,FC:CD:2F:50:00:00/28Qctek,FC:CD:2F:60:00:00/28Annapurn,FC:CD:2F:70:00:00/28Suzhoule,FC:CD:2F:80:00:00/28Asesoria,FC:CD:2F:90:00:00/28AromaRet,FC:CD:2F:A0:00:00/28ScopusIn,FC:CD:2F:B0:00:00/28HEAD-DIR,FC:CD:2F:C0:00:00/28SpedosAD,FC:CD:2F:D0:00:00/28Shenzhen,FC:CD:2F:E0:00:00/28Eltekbro,FC:CF:43HuizhouC,FC:CF:62IBM,FC:D2:B6IEEERegi,FC:D2:B6:00:00:00/28CgPowerA,FC:D2:B6:10:00:00/28LinkFar-,FC:D2:B6:20:00:00/28Soma,FC:D2:B6:30:00:00/28CoetCost,FC:D2:B6:40:00:00/28ShenZhen,FC:D2:B6:50:00:00/28Grandway,FC:D2:B6:60:00:00/28CirqueAu,FC:D2:B6:70:00:00/28TeamlyDi,FC:D2:B6:80:00:00/28OvissLab,FC:D2:B6:90:00:00/28Winglet,FC:D2:B6:A0:00:00/28NrealTec,FC:D2:B6:B0:00:00/28TChipDig,FC:D2:B6:C0:00:00/28SiliconS,FC:D2:B6:D0:00:00/28BeeSmart,FC:D2:B6:E0:00:00/28Univer,FC:D4:36Motorola,FC:D4:F2CocaCola,FC:D4:F6MessanaA,FC:D5:D9Shenzhen,FC:D6:BDRobertBo,FC:D7:33Tp-LinkT,FC:D8:17BeijingH,FC:D8:48Apple,FC:D9:08XiaomiCo,FC:DB:21SamsaraN,FC:DB:96Enervall,FC:DB:B3MurataMa,FC:DC:4AG-Wearab,FC:DD:55Shenzhen,FC:DE:90SamsungE,FC:E1:4FBRKBrand,FC:E1:86A3M,FC:E1:92SichuanJ,FC:E1:D9StableIm,FC:E1:FBArrayNet,FC:E2:3FClayPaky,FC:E3:3CHuaweiTe,FC:E5:57Nokia,FC:E6:6AIndustri,FC:E8:06EdifierI,FC:E8:92Hangzhou,FC:E9:98Apple,FC:EA:50Integrat,FC:EC:DAUbiquiti,FC:ED:B9Arrayent,FC:EE:E6FormikeE,FC:F1:36SamsungE,FC:F1:52Sony,FC:F1:CDOptex-Fa,FC:F2:9FChinaMob,FC:F5:28ZyxelCom,FC:F5:C4Espressi,FC:F6:47Fiberhom,FC:F7:7BHuaweiDe,FC:F8:AEIntelCor,FC:F8:B7TRONTEQE,FC:FA:F7Shanghai,FC:FB:FBCisco,FC:FC:48Apple,FC:FE:77HitachiR,FC:FE:C2Invensys,FC:FF:AAIEEERegi,")
liveList2 = []
liveList_manufacturer = []
for i in range(len(liveList)):
    flag = liveList[i][0:8].upper()
    flag2 = manufacture.split(flag,1)
    if len(flag2) == 1:
        flag4 = liveList[i][0:17].upper()
        flag3 = flag4
        liveList_manufacturer_flag = str(flag4)
    if len(flag2) == 2:
        flag3 = flag2[1].split(',')
        flag4 = flag3[0]
        
        if flag4 == 'IEEERegi':
            flag = liveList[i][0:17].upper()
            flag2 = manufacture.split(flag,1)
            if len(flag2) == 2:
                flag3 = flag2[1].split(',',1)
                flag4 = flag3[0][3:]
        liveList_manufacturer_flag = str(flag4+'_'+liveList[i][9:])
        if flag4 == 'Broadcast':
            liveList_manufacturer_flag = str('255.255.255.255')
            flag3[0] = liveList[i]
        if flag4 == 'LLDP_Multicast':
            liveList_manufacturer_flag = str('LLDP_Multicast')
            flag3[0] = liveList[i]
        liveList_manufacturer.append(liveList_manufacturer_flag)
    srcCounter = srcList.count(liveList[i])
    dstCounter = dstList.count(liveList[i])
    flagLivelist = i+1,liveList_manufacturer_flag,flag4,srcCounter,dstCounter #MAC-ADRESS,sent packets, received packets
    liveList2.append(flagLivelist)

#################################    
#################################
#################################
#################################

plot_throughput = []
plot_rtt = []
plot_nrtt = []
for i in range (len(rtThroughputSecond)):
    x = '%.3f'%float(rtThroughputSecond[i])
    x = float(x)
    xnrt = '%.3f'%float(nRtThroughputSecond[i])
    xnrt = float(xnrt)
    x2 = '%.3f'%float(x+xnrt)
    x2=float(x2)
    plot_throughput.append(x2)
    plot_nrtt.append(xnrt)
    plot_rtt.append(x)

#################################    
#################################
#################################
#################################    
    
communicationID = [] #Communication ID
communicationMACAdress = [] #Source and Destination
communicationType = [] # Protocol
for i in range(len(all_db)):
    if all_db[i][0] not in communicationID:
        flagID = all_db[i][0]
        flagSRC = all_db[i][2][0]
        flagDST = all_db[i][2][1]
        flagCtype = all_db[i][8]
        communicationType.append(flagCtype)
        communicationID.append(flagID)
        for i1 in range(len(liveList)):
            if flagSRC == liveList[i1]:
                flagSRC = liveList2[i1][1]
            if flagDST == liveList[i1]:
                flagDST = liveList2[i1][1]
        flagMACAdress = [flagSRC,flagDST]
        communicationMACAdress.append(flagMACAdress)

#################################    
#################################
#################################
#################################    

jitterAvg = [] # Jitter on list
setTDE = [] # TDE Set
avgTDE = [] #TDE Avarage
graphTDEAxisY = [] # TDE Avarage graph axis Y
graphTDEAxisY_FLOAT = []
graphAlarm = []
graphJitterAxisY = []
chronology = [] 
chronology2 = []# TDE Avarage graph axis X, graph Jitter Axis X, 
packetsSum = []
idTDE = []
alarmCount = []
alarmTime = []
alarmType = []
alarmDetail = []
packetLost = []
packetLostCount = []
packetLostTime = []
throughputPerID = []
communicationProtocol = []

for i in range(len(communicationID)):
    startPoint = '%.9f'%float(0.000000000)
    flagChronology = [startPoint]
    flagChronology2 = []
    flaggraphJitterAxisY = []
    flagPackets = 0
    flagTDEAxisY = [0]
    flagTDEAxisY_FLOAT = []    
    flagjitterAvg = []
    flagjitterAvg_count = 0
    flagAlarmtime = []
    flagAlarmtype = []
    flagAlarmDetail = []
    flagTDEAVG = []
    flagPacketLost = []
    flagPacketLostCount = []
    flagAlarmCount = 0
    flagThroughput = []
    flagProtocol = []
    flagpacketLostTime = []
    for i1 in range(len(all_db)):
        if all_db[i1][0] == communicationID[i]:
            flagProtocol1 = all_db[i1][8]
            flagProtocol.append(flagProtocol1)
    
    for i1 in range(len(all_db)):
        if all_db[i1][0] == communicationID[i]:
            flagProtocol1 = all_db[i1][8]
            flagProtocol.append(flagProtocol1)            
            if all_db[i1][7] != 'None':
                flagAlarmCount = flagAlarmCount +1
                flagAlarmtype1 = all_db[i1][6]
                flagAlarmDetail1 = all_db[i1][7]
                flagAlarmtime1 = '%.9f'%float(all_db[i1][10])
                #flagAlarmtime1 = float(flagAlarmtime1)
                flagAlarmtime.append(flagAlarmtime1)
                flagAlarmtype.append(flagAlarmtype1)
                flagAlarmDetail.append(flagAlarmDetail1)
                
            flagPackets = flagPackets +1

            if len(flagChronology) <= 1:
                flaggraphJitterAxisY.append(0)
                flagTDEAxisY.append(float(0))
                flag = all_db[i1][10]
                flag1 = '%.2f'%float(flag)
                flag1 = float(flag1)
                flagChronology2.append(flag1)
                flagThroughput1 = float(all_db[i1][4])
                flagThroughput.append(flagThroughput1)
            else:
                
                flagTDE1= float(all_db[i1][10])-float(all_db[i1-1][10])
                flagTDE1='%.9f'%float(flagTDE1)
                flagTDEAxisY.append(flagTDE1)
                flagTDE1 = float(float(flagTDE1)*1000)
                flagTDEAxisY_FLOAT.append(flagTDE1)
                flagTDEAVG1 = 0
                flagTDEAVG.append(flagTDEAVG1)
                flagThroughput1 = float(all_db[i1][4])
                flagThroughput.append(flagThroughput1)

                if len(flagTDEAxisY) >3:
                    flagJitter1 = abs(float(flagTDE1)/1000 - float(flagTDEAxisY[-2]))*1000000
                    flagJitter1 = '%.2f'%float(flagJitter1)
                    flagJitter1 = float(flagJitter1)
                    flaggraphJitterAxisY.append(flagJitter1)
                    flagJitter2 = float(flagJitter1)
                    flagjitterAvg.append(flagJitter2)
                    flagjitterAvg_count = flagjitterAvg_count +1
                    
                    flag = all_db[i1][10]
                    flag1 = '%.2f'%float(flag)
                    flag1 = float(flag1)
                    flagChronology2.append(flag1)    

            flag = all_db[i1][10]
            flagChronology.append(flag)
    flagThroughput2 = '%.3f'%float((sum(flagThroughput)*8)/float(flagChronology[-1])/1000000) 
    throughputPerID.append(flagThroughput2)
    alarmTime.append(flagAlarmtime)
    alarmType.append(flagAlarmtype)
    alarmDetail.append(flagAlarmDetail)
    graphTDEAxisY_FLOAT.append(flagTDEAxisY_FLOAT)
    if len(flagjitterAvg) == 0:
        flagJitter3= "low data"
    else:
        flagJitter3 = '%.2f'%float(sum(flagjitterAvg)/len(flagjitterAvg))
    jitterAvg.append(flagJitter3)
    graphJitterAxisY.append(flaggraphJitterAxisY)        
    graphTDEAxisY.append(flagTDEAxisY)
    chronology.append(flagChronology)
    chronology2.append(flagChronology2)
    packetsSum.append(flagPackets)
    if len(flagTDEAxisY_FLOAT) == 0:
        flagTDEAxisY_FLOAT = [1]
    else:
        if len(flagTDEAxisY_FLOAT[1:]) ==0:
               opTDEAVG = float(sum(flagTDEAxisY_FLOAT[1:]))/1               
        else:
               opTDEAVG = float(sum(flagTDEAxisY_FLOAT[1:]))/(len(flagTDEAxisY_FLOAT[1:]))
    flagTDESET2 = '%.9f'%float(opTDEAVG)
    flagTDESET2 = str(flagTDESET2[0:4])
    setTDE.append(flagTDESET2)
    otPacketLost = ((float(opTDEAVG)*5)/100) + opTDEAVG  # porcentagem a considerar para pacotes perdidos
    if opTDEAVG < 0.9:
        opTDEAVG = float('%.1f'%float(opTDEAVG))
    else:
        opTDEAVG = round(opTDEAVG)
                         
    for index, value in enumerate(flagTDEAVG):
        if value == 0:
            flagTDEAVG[index] = opTDEAVG
    for index, value in enumerate(flagTDEAxisY_FLOAT):
        if value < otPacketLost or index == 0:
            flagPacketLost.append(otPacketLost)
        else:
            flagPacketLost1 = otPacketLost
            flagPacketLost.append(flagPacketLost1)
            flagPacketLostCount.append(1)
            flagpacketLostTime1 = index,chronology[i][index]
            flagpacketLostTime.append(flagpacketLostTime1)
            #print(i,index, chronology)
    packetLostTime.append(flagpacketLostTime)        
    flagAlarmCount1 = flagAlarmCount
    alarmCount.append(flagAlarmCount1)
    packetLostCount1 = sum(flagPacketLostCount)
    packetLostCount.append(packetLostCount1)
    packetLost.append(flagPacketLost)        
    avgTDE.append(flagTDEAVG)
    protocolFLAG=0
    for i3 in range(len(flagProtocol)):
        if flagProtocol[i3][0:2] == 'RT':
            flagProtocolRT= str(flagProtocol[i3])
            flagProtocolRT2 = flagProtocolRT
            protocolFLAG = protocolFLAG + 1
        else:
            if protocolFLAG == 0:
                flagProtocolRT = str(flagProtocol[i3])
            else:
                flagProtocolRT = flagProtocolRT2
    communicationProtocol.append(flagProtocolRT)
    protocolFLAG = 0
#################################    
#################################
#################################
#################################    
dateTime = datetime.today().strftime('%Y-%m-%d %H:%M')
captureType = 'Kunbus TAP'
arrivalTime = datetime.fromtimestamp(packets[0].time).strftime('%Y-%m-%d - %H:%M')

plt.rcParams.update({'font.size':12})
pie_index = ['Packets RT','Packets nRT']
pie_value = [packetsRT,packetsNRT]
pie_explode = (0.2,0)

plt.figure (figsize=(10,4))
plt.pie(x=pie_value,labels=pie_index, explode=pie_explode, autopct='%1.1f%%',startangle=75, shadow=True )
plt.savefig('pie_graph.png')


from fpdf import FPDF

total_time = "%.9f"%float(chronology_list[-1])

class PDF(FPDF):
    def header(self):
        # Logo
        self.image('logo_ifsp.jpg', 10, 8, 30)
        # helvetica bold 15
        self.image('logo2.jpg', 165, 13, 32)
        # helvetica bold 15
        self.set_font('Arial', 'B', 15)
        # Move to the right
        
        # Title
        self.ln(20)
        self.cell(40, 10, ' ', 0, 0, 'C')
        self.cell(110, 20, 'PROFINET NETWORK\nPASSIVE ANALYSIS', 0, 0, 'C')
        self.cell(40, 10, ' ', 0, 0, 'C')
        self.ln(15)

    # Page footer
    def footer(self):
        # Position at 1.5 cm from bottom
        self.set_y(-15)
        # helvetica italic 8
        self.set_font('helvetica', 'I', 8)
        # Page number
        self.cell(0, 10, str(self.page_no()) + 'pg', 0, 0, True)


pdf = PDF()

pdf.alias_nb_pages(0)
pdf.add_page()
pdf.set_font('Arial', '', 9)

################################# Live List
pdf.set_font('Arial', 'B', 15)
pdf.cell(190, 7, 'User:   '+str(userID), 2, 0, 'L')
pdf.ln(7)
pdf.set_font('Arial','',9)
pdf.cell(60,7,'Analysis datetime :   '+ str(dateTime),1,0,"L")
pdf.cell(130,7,'Filename:   '+str(fileName),2,0,"L")
pdf.ln(7)
pdf.cell(60,7,'Packets Real Time:   '+ str(packetsRT),1,0,"L")
pdf.cell(130,7,'Devices online:   '+str(len(liveList2)),1,0,"L")
pdf.ln(7)
pdf.cell(60,7,'Packets non Real Time:   '+ str(packetsNRT),1,0,"L")
pdf.cell(130,7,'Time:   '+ '%.2f'%float(chronology_list[-1])+' seconds',1,0,"L")
pdf.ln(7)
pdf.cell(60,7,'Capture Type:   '+ str(captureType),1,0,"L")
pdf.cell(130,7,'Arrival Time:   '+str(arrivalTime),1,0,"L")    
pdf.ln(7)
pdf.ln(5)
pdf.set_font('Arial', 'B', 15)
pdf.set_line_width(0.2)
pdf.cell(190, 10, 'Live List', 1, 0, 'C')
pdf.ln(10)
pdf.set_font('Arial', 'B', 9)
pdf.cell(38,7,'Device ID',1,0,"C")
pdf.cell(38,7,'MAC address',1,0,"C")
pdf.cell(38,7,'Manufacturer',1,0,"C")
pdf.cell(38,7,'Sent Packets',1,0,"C")
pdf.cell(38,7,'Packets received',1,0,"C")
pdf.ln(7)
pdf.set_font('Arial', '', 9)
for i in range(len(liveList2)):
    flag= str(liveList2[i][0])
    pdf.cell(38,7,flag,1,0,"C")
    flag= str(liveList2[i][1])
    pdf.cell(38,7,flag,1,0,"C")
    flag= str(liveList2[i][2])
    pdf.cell(38,7,flag,1,0,"C")
    flag= str(liveList2[i][3])
    pdf.cell(38,7,flag,1,0,"C")
    flag= str(liveList2[i][4])
    pdf.cell(38,7,flag,1,0,"C")
    pdf.ln(7)
    
######################Grafico Throughput
    
x = []
for i in range(len(chronology_throughput)):
    x1 =float('%.3f'%float(chronology_throughput[i]))
    x.append(x1)
    
pdf.cell(190,118,'',1,0,"C")
pdf.ln(7)
pdf.set_font('Arial', 'B', 9)
pdf.cell(190,7,'Throughput',1,0,"C")

rtt = sum(plot_rtt)
nrtt = sum(plot_nrtt)
rtt_nrtt = sum(plot_rtt) + sum(plot_nrtt)
rttpercentual = (rtt*100)/rtt_nrtt
nrttpercentual = (nrtt*100)/rtt_nrtt
print(rtt, nrtt,rtt_nrtt,'---', rttpercentual, nrttpercentual)

pdf.ln(7)
pdf.set_font('Arial','',9)
pdf.cell(95,7,'Data:   '+str('%.3f'%float(rtt_nrtt))+'Mbit',1,0,"L")
pdf.cell(95,7,'Time:   '+str(chronology_list[-1])+'s',1,0,"L")
pdf.ln(7)
pdf.cell(95,7,'Data Real Time:   ' + str('%.3f'%float(rtt)+'Mbit (' + str('%.2f'%float(rttpercentual))+'%)' ),1,0,"L")
pdf.cell(95,7,'Data non Real Time:   '+ str('%.3f'%float(nrtt)+'Mbit (' + str('%.2f'%float(nrttpercentual))+'%)' ),1,0,"L")
pdf.ln(7)
        
plt.figure(figsize=(15,5))
plt.plot(plot_throughput,'g', label='RT+nRT')
plt.plot(chronology_throughput, plot_rtt,label='Real Time')
plt.plot(plot_nrtt,'r', label='non Real Time')
plt.xticks(np.arange(min(x), max(x)+1, 3),fontsize=15)
plt.yticks(fontsize=15)
plt.title("Throughput", fontsize=15)
plt.ylabel('Mbps', fontsize=15)
plt.xlabel('Time (s)', fontsize=15)
plt.legend(bbox_to_anchor=(0.5,-0.3), fontsize= 15, loc= 'lower center', ncol = 3)
plt.grid(linestyle = '--', linewidth = 0.8)
plt.savefig('throughput'+str(i)+'.png', 
            dpi=64, 
            format='png', 
 
            bbox_inches='tight')
pdf.ln(7)
pdf.image('throughput'+str(i)+'.png',w=190)
pdf.add_page('P','A4',0)
#################################Analise por comunicação

pdf.ln(5)
pdf.set_font('Arial', 'B', 15)
pdf.cell(190, 10, 'PROFINET communications analysis', 1, 0, 'C')
pdf.ln(10)
pdf.set_font('Arial', 'B', 9)
pdf.cell(15,7,'ID',1,0,"C")
pdf.cell(30,7,'Source',1,0,"C")
pdf.cell(30,7,'Destination',1,0,"C") #pdf.cell(23.2,7,'TDE SET [ms]',1,0,"C")           TDE SET
pdf.cell(18,7,'TDE AVG',1,0,"C")
pdf.cell(21,7,'Jitter',1,0,"C") 
pdf.cell(23,7,'Throughput ',1,0,"C")
pdf.cell(15,7,'Packets',1,0,"C")
pdf.cell(15,7,'Alarm(s)',1,0,"C")
pdf.cell(23,7,'Protocol',1,0,"C")

pdf.ln(7)
pdf.set_font('Arial', '', 9)
for i in range(len(communicationID)):
    if jitterAvg[i] == 'low data':
        flag= str(communicationID[i])
        pdf.cell(15,7,flag,1,0,"C")
        flag= str(communicationMACAdress[i][0])
        pdf.cell(30,7,flag,1,0,"C")
        flag= str(communicationMACAdress[i][1])
        pdf.cell(30,7,flag,1,0,"C")
        flag= str(setTDE[i])+'ms'
        pdf.cell(18,7,flag,1,0,"C")
        flag= str("insufficient data")
        pdf.cell(44,7,flag,1,0,"C")
        flag= str(packetsSum[i])
        pdf.cell(15,7,flag,1,0,"C")
        flag= str(alarmCount[i])
        pdf.cell(15,7,flag,1,0,"C")
        flag= str(communicationProtocol[i])
        pdf.cell(23,7,flag,1,0,"C")
        pdf.ln(7)
    else:    
        flag= str(communicationID[i])
        pdf.cell(15,7,flag,1,0,"C")
        flag= str(communicationMACAdress[i][0])
        pdf.cell(30,7,flag,1,0,"C")
        flag= str(communicationMACAdress[i][1])
        pdf.cell(30,7,flag,1,0,"C")
        flag= str(setTDE[i])+'ms'
        pdf.cell(18,7,flag,1,0,"C")
        flag= str(jitterAvg[i])+ 'µs'
        pdf.cell(21,7,flag,1,0,"C")
        flag= str(throughputPerID[i])+ ' Mbps'
        pdf.cell(23,7,flag,1,0,"C")
        flag= str(packetsSum[i])
        pdf.cell(15,7,flag,1,0,"C")
        flag= str(alarmCount[i])
        pdf.cell(15,7,flag,1,0,"C")
        flag= str(communicationProtocol[i])
        pdf.cell(23,7,flag,1,0,"C")
        pdf.ln(7)
#pdf.set_font('Arial', '', 6)
#pdf.cell(2,5,"Non-profinet communication(s) ID: " + str(non_profinet_index),1,0,0)
pdf.ln(10)
pdf.add_page('P','A4',0)
for i in range(len(communicationID)):
    if communicationProtocol[i][0:2] == 'RT' and packetsSum[i] > 3 :
        #pdf.ln(5)
        pdf.set_font('Arial', 'B', 15)
        pdf.cell(190, 7, 'Communication ID:   '+str(communicationID[i]), 2, 0, 'L')
        pdf.ln(7)
        pdf.set_font('Arial','',9)
        pdf.cell(95,7,'Source:   '+str(communicationMACAdress[i][0]),1,0,"L")
        pdf.cell(95,7,'Destination:   '+str(communicationMACAdress[i][1]),1,0,"L")
        pdf.ln(7)
        pdf.cell(95,7,'Packets:   ' + str(packetsSum[i]),1,0,"L")
        pdf.cell(95,7,'TDE Set:   '+ str(avgTDE[i][0])+'ms',1,0,"L")
        pdf.ln(7)
        pdf.cell(95,7,'Packets Loss:   '+ str(packetLostCount[i]),1,0,"L")
        pdf.cell(95,7,'Alarms:   '+str(alarmCount[i]),2,0,"L")
        pdf.ln(7)
        if packetLostCount[i] == 0:
            #print('erro 1')
            pdf.cell(190,165,'',1,0,"C")
            pdf.ln(10)
            maximo = float('%.3f'%float(max(graphTDEAxisY_FLOAT[i][1:])))
            minimo = float('%.3f'%float(min(graphTDEAxisY_FLOAT[i][1:])))
            difference = float('%.3f'%float(maximo-minimo))
            #print(difference)
            plt.figure(figsize=(15,5))
            plt.scatter(chronology2[i][1:],graphTDEAxisY_FLOAT[i][1:],c=graphTDEAxisY_FLOAT[i][1:], cmap='jet', label = 'Packets' )
            #plt.plot(chronology2[i],packetLost[i],'r-',label='Limit for packet loss',linewidth=0.8)
            plt.plot(chronology2[i],avgTDE[i],'k--',label='TDE avarage',linewidth=0.8)

            plt.xticks(np.arange(0, max(chronology2[1])+1,1),fontsize=15)

            if difference < 9:
                plt.yticks(np.arange(minimo, maximo,step=1),fontsize=12)
                if difference < 0.9:
                    #print('aqui1')
                    plt.yticks(np.arange(minimo, maximo,0.05),fontsize=12)
                    if difference < 0.09:
                        #print('aqui2')
                        plt.yticks(np.arange(minimo, maximo,step=0.005),fontsize=12)
                        if difference < 0.009:
                            #print('aqui3')
                            plt.yticks(np.arange(minimo, maximo,step=0.0005),fontsize=12) 
            #plt.yticks(minimo,maximo)
            plt.legend(bbox_to_anchor=(0.5,-0.30), fontsize= 15, loc= 'lower center', ncol = 3)
            plt.title('Packets x TDE - communication ID:   '+str(communicationID[i]), fontsize=15)
            plt.ylabel('TDE per packet (ms)',fontsize=15)
            plt.xlabel('Time(s)',fontsize=15)
            plt.grid(linestyle = '--', linewidth = 0.8)
            plt.savefig('graphID'+str(i)+'.png', 
                dpi=64, 
                format='png', 

                bbox_inches='tight')
            pdf.image('graphID'+str(i)+'.png',w=190)
            #pdf.cell(190,80,'',1,0,"C") 
            pdf.ln(10)
            plt.figure(figsize=(15,4))
            plt.bar(chronology2[i],graphJitterAxisY[i],width=0.03, label = 'Jitter' )
            plt.xticks(np.arange(0, max(chronology2[i])+0.5,1),fontsize=15)
            plt.yticks(fontsize=15)
            plt.title('Jitter of communication ID:   '+str(communicationID[i]), fontsize=15)
            plt.ylabel('Jitter (µs)',fontsize=15)
            plt.xlabel('Time (s)',fontsize=15)
            plt.grid(linestyle = '--', linewidth = 0.8)
            plt.savefig('jitterGraph'+str(i)+'.png', 
                dpi=64, 
                format='png', 

                bbox_inches='tight')
            pdf.image('jitterGraph'+str(i)+'.png',w=190)
            pdf.ln(5)
            if alarmCount[i] != 0:
                pdf.set_font('Arial', 'B', 10)
                pdf.cell(190, 7, 'Alarms Details:', 1, 0, 'C')
                pdf.ln(7)
                pdf.set_font('Arial', 'B', 9)
                pdf.cell(63.333,7,'Capture time',1,0,"L")
                pdf.cell(63.333,7,'Alarm type',1,0,"L")
                pdf.cell(63.333,7,'Detail',1,0,"L")
                pdf.ln(7)
                pdf.set_font('Arial', '', 9)
                for y in range(len(alarmTime[i])):
                    pdf.cell(63.333,7,str(alarmTime[i][y]),1,0,"L")
                    pdf.cell(63.333,7,str(alarmType[i][y]),1,0,"L")
                    pdf.cell(63.333,7,str(alarmDetail[i][y]),1,0,"L")
                    pdf.ln(7)
            
        else:
            #print('erro 2')
            pdf.cell(190,165,'',1,0,"C")
            pdf.ln(10)
            maximo = float('%.3f'%float(max(graphTDEAxisY_FLOAT[i][1:])))
            minimo = float('%.3f'%float(min(graphTDEAxisY_FLOAT[i][1:])))
            difference = float('%.3f'%float(maximo-minimo))
            #print(difference)
            plt.figure(figsize=(15,5))
            plt.scatter(chronology2[i][1:],graphTDEAxisY_FLOAT[i][1:],c=graphTDEAxisY_FLOAT[i][1:], cmap='jet', label = 'Packets' )
            plt.plot(chronology2[i],packetLost[i],'r-',label='Limit for packet loss',linewidth=0.8)
            plt.plot(chronology2[i],avgTDE[i],'k--',label='TDE avarage',linewidth=0.8)

            plt.xticks(np.arange(0, max(chronology2[1])+1,1),fontsize=15)

            if difference < 9:
                plt.yticks(np.arange(minimo, maximo,step=1),fontsize=12)
                if difference < 0.9:
                    #print('aqui1')
                    plt.yticks(np.arange(minimo, maximo,0.05),fontsize=12)
                    if difference < 0.09:
                        #print('aqui2')
                        plt.yticks(np.arange(minimo, maximo,step=0.005),fontsize=12)
                        if difference < 0.009:
                            #print('aqui3')
                            plt.yticks(np.arange(minimo, maximo,step=0.0005),fontsize=12) 
            #plt.yticks(minimo,maximo)
            plt.legend(bbox_to_anchor=(0.5,-0.30), fontsize= 15, loc= 'lower center', ncol = 3)
            plt.title('Packets x TDE - communication ID:   '+str(communicationID[i]), fontsize=15)
            plt.ylabel('TDE per packet (ms)',fontsize=15)
            plt.xlabel('Time(s)',fontsize=15)
            plt.grid(linestyle = '--', linewidth = 0.8)
            plt.savefig('graphID'+str(i)+'.png', 
                dpi=64, 
                format='png', 

                bbox_inches='tight')
            pdf.image('graphID'+str(i)+'.png',w=190)
            #pdf.cell(190,80,'',1,0,"C") 
            pdf.ln(10)
            plt.figure(figsize=(15,4))
            plt.bar(chronology2[i],graphJitterAxisY[i],width=0.03, label = 'Jitter' )
            plt.xticks(np.arange(0, max(chronology2[i])+0.5,1),fontsize=15)
            plt.yticks(fontsize=15)
            plt.title('Jitter of communication ID:   '+str(communicationID[i]), fontsize=15)
            plt.ylabel('Jitter (µs)',fontsize=15)
            plt.xlabel('Time (s)',fontsize=15)
            plt.grid(linestyle = '--', linewidth = 0.8)
            plt.savefig('jitterGraph'+str(i)+'.png', 
                dpi=64, 
                format='png', 

                bbox_inches='tight')
            pdf.image('jitterGraph'+str(i)+'.png',w=190)
            pdf.ln(5)
            if alarmCount[i] != 0:
                pdf.set_font('Arial', 'B', 10)
                pdf.cell(190, 7, 'Alarms Details:', 1, 0, 'C')
                pdf.ln(7)
                pdf.set_font('Arial', 'B', 9)
                pdf.cell(63.333,7,'Capture time',1,0,"L")
                pdf.cell(63.333,7,'Alarm type',1,0,"L")
                pdf.cell(63.333,7,'Detail',1,0,"L")
                pdf.ln(7)
                pdf.set_font('Arial', '', 9)
                for y in range(len(alarmTime[i])):
                    pdf.cell(63.333,7,str(alarmTime[i][y]),1,0,"L")
                    pdf.cell(63.333,7,str(alarmType[i][y]),1,0,"L")
                    pdf.cell(63.333,7,str(alarmDetail[i][y]),1,0,"L")
                    pdf.ln(7)                        
            
        if communicationID[i] != communicationID[-1]:    
            pdf.add_page()            
        #pdf.cell(190,10,,2,0,"C")
        #pdf.cell(190,' ',1,0,"C")
        


    
pdf.output(fileName+'_PNPA_TAP.pdf')


fim = time.time()
print('tempo de execução: ',fim - inicio)  

In [ ]:
#VVVVVVVVVVVVVVV----TESTES
#VVVVVVVVVVVVVVV----TESTES
#VVVVVVVVVVVVVVV----TESTES
#VVVVVVVVVVVVVVV----TESTES
#VVVVVVVVVVVVVVV----TESTES
#VVVVVVVVVVVVVVV----TESTES
#VVVVVVVVVVVVVVV----TESTES
#VVVVVVVVVVVVVVV----TESTES
#VVVVVVVVVVVVVVV----TESTES
#VVVVVVVVVVVVVVV----TESTES
